# Search engine demonstration

## Βήμα 1
Αρχικά ξεκινάμε τρέχοντας μέσω CMD τις επόμενες εντολές:
- pip install notebook
- pip install beautifulsoup4
- pip install requests
- pip install pandas

In [ ]:
import requests
from bs4 import BeautifulSoup
import json
import re

# Λίστα με τα 10 URLs
urls = [
    'https://en.wikipedia.org/wiki/Python_(programming_language)',
    'https://en.wikipedia.org/wiki/JavaScript',
    'https://en.wikipedia.org/wiki/Java_(programming_language)',
    'https://en.wikipedia.org/wiki/C_(programming_language)',
    'https://en.wikipedia.org/wiki/Ruby_(programming_language)',
    'https://en.wikipedia.org/wiki/HTML',
    'https://en.wikipedia.org/wiki/C%2B%2B',
    'https://en.wikipedia.org/wiki/Go_(programming_language)',
    'https://en.wikipedia.org/wiki/Swift_(programming_language)',
    'https://en.wikipedia.org/wiki/Kotlin'
]

# Συνάρτηση για να τραβήξουμε δεδομένα από κάθε URL
def crawl_page(url):
    print(f"Crawling: {url}")
    try:
        response = requests.get(url)
        if response.status_code == 200:
            soup = BeautifulSoup(response.content, 'html.parser')
            
            # Εξαγωγή όλων των λέξεων από την σελίδα
            # Επιλέγουμε τα tags που περιέχουν κείμενο: p (paragraphs), li (list items), span, a (links) και άλλα
            text = []
            for tag in soup.find_all(['p', 'li', 'span', 'a', 'h1', 'h2', 'h3', 'h4', 'h5', 'h6']):
                text.append(tag.get_text())
            
            # Συνενώνουμε όλα τα κείμενα και καθαρίζουμε από περιττά κενά
            full_text = ' '.join(text)
            # Καθαρισμός κειμένου (π.χ. αφαιρούμε περιττές λευκές περιοχές)
            cleaned_text = re.sub(r'\s+', ' ', full_text).strip()
            
            return cleaned_text
        else:
            print(f"Failed to retrieve {url}")
            return ""
    except Exception as e:
        print(f"Error while crawling {url}: {e}")
        return ""

# Δημιουργία λεξικού για αποθήκευση του πλήρους κειμένου ανά URL
text_dict = {}

# Κάνουμε crawl τις 10 σελίδες
for url in urls:
    page_text = crawl_page(url)
    text_dict[url] = page_text

# Αποθήκευση των δεδομένων σε αρχείο JSON
with open('programming_languages_text.json', 'w') as file:
    json.dump(text_dict, file, indent=4)

print("Data saved to programming_languages_text.json")


Ο παραπάνω κώδικας υλοποιεί μια διαδικασία web scraping για να τραβήξει κείμενο από 10 σελίδες της Wikipedia που σχετίζονται με γλώσσες προγραμματισμού. Αναλυτικά:

1. **Λίστα URLs**: Ορίζεται μια λίστα με 10 URLs που περιέχουν πληροφορίες για γλώσσες προγραμματισμού, όπως Python, Java, C, HTML κ.λπ.

2. **Συνάρτηση crawl_page**:
- Κάνει HTTP αίτημα (GET) σε κάθε URL.
- Αν η απάντηση είναι επιτυχής (κωδικός 200), χρησιμοποιεί το BeautifulSoup για να αναλύσει το HTML περιεχόμενο της σελίδας.
- Εξάγει κείμενο από διάφορα HTML tags όπως *p, li, span, a, h1-h6*.
- Συνενώνει το κείμενο από όλα τα tags σε μια ενιαία συμβολοσειρά.
- Καθαρίζει το κείμενο από περιττά κενά και επιστρέφει το τελικό καθαρό κείμενο.

3. **Αποθήκευση Δεδομένων**:
- Τα δεδομένα αποθηκεύονται σε ένα λεξικό text_dict όπου το κλειδί είναι το URL και η τιμή είναι το εξαγόμενο κείμενο από τη σελίδα.
- Τέλος, τα δεδομένα αποθηκεύονται σε αρχείο JSON με το όνομα *programming_languages_text.json*.

4. **Αρχειοθέτηση**:
Το JSON αρχείο που δημιουργείται, περιέχει τα κείμενα από όλες τις σελίδες, τα οποία μπορούν να χρησιμοποιηθούν για περαιτέρω επεξεργασία, όπως δημιουργία ενός inverted index ή κατάταξη εγγράφων.

![step 1](./project/images/step_1.png) 

Το αρχείο JSON που δημιουργήθηκε:

In [ ]:
{
    "https://en.wikipedia.org/wiki/Python_(programming_language)": "Jump to content Main menu Main page Main page Main page Contents Contents Contents Current events Current events Current events Random article Random article Random article About Wikipedia About Wikipedia About Wikipedia Contact us Contact us Contact us Help Help Help Learn to edit Learn to edit Learn to edit Community portal Community portal Community portal Recent changes Recent changes Recent changes Upload file Upload file Upload file Search Search Appearance Donate Donate Donate Create account Create account Create account Log in Log in Log in Personal tools Donate Donate Donate Create account Create account Create account Log in Log in Log in learn more learn more Contributions Contributions Contributions Talk Talk Talk Contents (Top) (Top) 1 History 1 History 1 History 2 Design philosophy and features 2 Design philosophy and features 2 Design philosophy and features 3 Syntax and semantics Toggle Syntax and semantics subsection 3.1 Indentation 3.2 Statements and control flow 3.3 Expressions 3.4 Methods 3.5 Typing 3.6 Arithmetic operations 3.7 Function syntax 3 Syntax and semantics 3 Syntax and semantics Toggle Syntax and semantics subsection 3.1 Indentation 3.1 Indentation 3.1 Indentation 3.2 Statements and control flow 3.2 Statements and control flow 3.2 Statements and control flow 3.3 Expressions 3.3 Expressions 3.3 Expressions 3.4 Methods 3.4 Methods 3.4 Methods 3.5 Typing 3.5 Typing 3.5 Typing 3.6 Arithmetic operations 3.6 Arithmetic operations 3.6 Arithmetic operations 3.7 Function syntax 3.7 Function syntax 3.7 Function syntax 4 Programming examples 4 Programming examples 4 Programming examples 5 Libraries 5 Libraries 5 Libraries 6 Development environments 6 Development environments 6 Development environments 7 Implementations Toggle Implementations subsection 7.1 Reference implementation 7.2 Other implementations 7.3 No longer supported implementations 7.4 Cross-compilers to other languages 7.5 Performance 7 Implementations 7 Implementations Toggle Implementations subsection 7.1 Reference implementation 7.1 Reference implementation 7.1 Reference implementation 7.2 Other implementations 7.2 Other implementations 7.2 Other implementations 7.3 No longer supported implementations 7.3 No longer supported implementations 7.3 No longer supported implementations 7.4 Cross-compilers to other languages 7.4 Cross-compilers to other languages 7.4 Cross-compilers to other languages 7.5 Performance 7.5 Performance 7.5 Performance 8 Development 8 Development 8 Development 9 API documentation generators 9 API documentation generators 9 API documentation generators 10 Naming 10 Naming 10 Naming 11 Popularity 11 Popularity 11 Popularity 12 Uses 12 Uses 12 Uses 13 Languages influenced by Python 13 Languages influenced by Python 13 Languages influenced by Python 14 See also 14 See also 14 See also 15 References Toggle References subsection 15.1 Sources 15 References 15 References Toggle References subsection 15.1 Sources 15.1 Sources 15.1 Sources 16 Further reading 16 Further reading 16 Further reading 17 External links 17 External links 17 External links Toggle the table of contents Python (programming language) Python (programming language) 112 languages Afrikaans Afrikaans Afrikaans Alemannisch Alemannisch Alemannisch \u0627\u0644\u0639\u0631\u0628\u064a\u0629 \u0627\u0644\u0639\u0631\u0628\u064a\u0629 \u0627\u0644\u0639\u0631\u0628\u064a\u0629 Aragon\u00e9s Aragon\u00e9s Aragon\u00e9s \u0985\u09b8\u09ae\u09c0\u09af\u09bc\u09be \u0985\u09b8\u09ae\u09c0\u09af\u09bc\u09be \u0985\u09b8\u09ae\u09c0\u09af\u09bc\u09be Asturianu Asturianu Asturianu Az\u0259rbaycanca Az\u0259rbaycanca Az\u0259rbaycanca \u062a\u06c6\u0631\u06a9\u062c\u0647 \u062a\u06c6\u0631\u06a9\u062c\u0647 \u062a\u06c6\u0631\u06a9\u062c\u0647 Basa Bali Basa Bali Basa Bali \u09ac\u09be\u0982\u09b2\u09be \u09ac\u09be\u0982\u09b2\u09be \u09ac\u09be\u0982\u09b2\u09be \u95a9\u5357\u8a9e / B\u00e2n-l\u00e2m-g\u00fa \u95a9\u5357\u8a9e / B\u00e2n-l\u00e2m-g\u00fa \u95a9\u5357\u8a9e / B\u00e2n-l\u00e2m-g\u00fa \u0411\u0435\u043b\u0430\u0440\u0443\u0441\u043a\u0430\u044f \u0411\u0435\u043b\u0430\u0440\u0443\u0441\u043a\u0430\u044f \u0411\u0435\u043b\u0430\u0440\u0443\u0441\u043a\u0430\u044f \u0411\u0435\u043b\u0430\u0440\u0443\u0441\u043a\u0430\u044f (\u0442\u0430\u0440\u0430\u0448\u043a\u0435\u0432\u0456\u0446\u0430) \u0411\u0435\u043b\u0430\u0440\u0443\u0441\u043a\u0430\u044f (\u0442\u0430\u0440\u0430\u0448\u043a\u0435\u0432\u0456\u0446\u0430) \u0411\u0435\u043b\u0430\u0440\u0443\u0441\u043a\u0430\u044f (\u0442\u0430\u0440\u0430\u0448\u043a\u0435\u0432\u0456\u0446\u0430) \u092d\u094b\u091c\u092a\u0941\u0930\u0940 \u092d\u094b\u091c\u092a\u0941\u0930\u0940 \u092d\u094b\u091c\u092a\u0941\u0930\u0940 \u0411\u044a\u043b\u0433\u0430\u0440\u0441\u043a\u0438 \u0411\u044a\u043b\u0433\u0430\u0440\u0441\u043a\u0438 \u0411\u044a\u043b\u0433\u0430\u0440\u0441\u043a\u0438 Bosanski Bosanski Bosanski Brezhoneg Brezhoneg Brezhoneg Catal\u00e0 Catal\u00e0 Catal\u00e0 Cebuano Cebuano Cebuano \u010ce\u0161tina \u010ce\u0161tina \u010ce\u0161tina Cymraeg Cymraeg Cymraeg Dansk Dansk Dansk Deutsch Deutsch Deutsch Eesti Eesti Eesti \u0395\u03bb\u03bb\u03b7\u03bd\u03b9\u03ba\u03ac \u0395\u03bb\u03bb\u03b7\u03bd\u03b9\u03ba\u03ac \u0395\u03bb\u03bb\u03b7\u03bd\u03b9\u03ba\u03ac Espa\u00f1ol Espa\u00f1ol Espa\u00f1ol Esperanto Esperanto Esperanto Euskara Euskara Euskara \u0641\u0627\u0631\u0633\u06cc \u0641\u0627\u0631\u0633\u06cc \u0641\u0627\u0631\u0633\u06cc Fran\u00e7ais Fran\u00e7ais Fran\u00e7ais Galego Galego Galego \u0a97\u0ac1\u0a9c\u0ab0\u0abe\u0aa4\u0ac0 \u0a97\u0ac1\u0a9c\u0ab0\u0abe\u0aa4\u0ac0 \u0a97\u0ac1\u0a9c\u0ab0\u0abe\u0aa4\u0ac0 \ud55c\uad6d\uc5b4 \ud55c\uad6d\uc5b4 \ud55c\uad6d\uc5b4 Hausa Hausa Hausa \u0540\u0561\u0575\u0565\u0580\u0565\u0576 \u0540\u0561\u0575\u0565\u0580\u0565\u0576 \u0540\u0561\u0575\u0565\u0580\u0565\u0576 \u0939\u093f\u0928\u094d\u0926\u0940 \u0939\u093f\u0928\u094d\u0926\u0940 \u0939\u093f\u0928\u094d\u0926\u0940 Hrvatski Hrvatski Hrvatski Ido Ido Ido Bahasa Indonesia Bahasa Indonesia Bahasa Indonesia Interlingua Interlingua Interlingua \u00cdslenska \u00cdslenska \u00cdslenska Italiano Italiano Italiano \u05e2\u05d1\u05e8\u05d9\u05ea \u05e2\u05d1\u05e8\u05d9\u05ea \u05e2\u05d1\u05e8\u05d9\u05ea \u10e5\u10d0\u10e0\u10d7\u10e3\u10da\u10d8 \u10e5\u10d0\u10e0\u10d7\u10e3\u10da\u10d8 \u10e5\u10d0\u10e0\u10d7\u10e3\u10da\u10d8 \u049a\u0430\u0437\u0430\u049b\u0448\u0430 \u049a\u0430\u0437\u0430\u049b\u0448\u0430 \u049a\u0430\u0437\u0430\u049b\u0448\u0430 Kiswahili Kiswahili Kiswahili Kurd\u00ee Kurd\u00ee Kurd\u00ee \u041a\u044b\u0440\u0433\u044b\u0437\u0447\u0430 \u041a\u044b\u0440\u0433\u044b\u0437\u0447\u0430 \u041a\u044b\u0440\u0433\u044b\u0437\u0447\u0430 \u0ea5\u0eb2\u0ea7 \u0ea5\u0eb2\u0ea7 \u0ea5\u0eb2\u0ea7 Latina Latina Latina Latvie\u0161u Latvie\u0161u Latvie\u0161u Lietuvi\u0173 Lietuvi\u0173 Lietuvi\u0173 La .lojban. La .lojban. La .lojban. Lombard Lombard Lombard Magyar Magyar Magyar \u041c\u0430\u043a\u0435\u0434\u043e\u043d\u0441\u043a\u0438 \u041c\u0430\u043a\u0435\u0434\u043e\u043d\u0441\u043a\u0438 \u041c\u0430\u043a\u0435\u0434\u043e\u043d\u0441\u043a\u0438 \u0d2e\u0d32\u0d2f\u0d3e\u0d33\u0d02 \u0d2e\u0d32\u0d2f\u0d3e\u0d33\u0d02 \u0d2e\u0d32\u0d2f\u0d3e\u0d33\u0d02 \u092e\u0930\u093e\u0920\u0940 \u092e\u0930\u093e\u0920\u0940 \u092e\u0930\u093e\u0920\u0940 Bahasa Melayu Bahasa Melayu Bahasa Melayu \u041c\u043e\u043d\u0433\u043e\u043b \u041c\u043e\u043d\u0433\u043e\u043b \u041c\u043e\u043d\u0433\u043e\u043b \u1019\u103c\u1014\u103a\u1019\u102c\u1018\u102c\u101e\u102c \u1019\u103c\u1014\u103a\u1019\u102c\u1018\u102c\u101e\u102c \u1019\u103c\u1014\u103a\u1019\u102c\u1018\u102c\u101e\u102c Na Vosa Vakaviti Na Vosa Vakaviti Na Vosa Vakaviti Nederlands Nederlands Nederlands \u0928\u0947\u092a\u093e\u0932\u0940 \u0928\u0947\u092a\u093e\u0932\u0940 \u0928\u0947\u092a\u093e\u0932\u0940 \u65e5\u672c\u8a9e \u65e5\u672c\u8a9e \u65e5\u672c\u8a9e \u07d2\u07de\u07cf \u07d2\u07de\u07cf \u07d2\u07de\u07cf Norsk bokm\u00e5l Norsk bokm\u00e5l Norsk bokm\u00e5l Norsk nynorsk Norsk nynorsk Norsk nynorsk \u0b13\u0b21\u0b3c\u0b3f\u0b06 \u0b13\u0b21\u0b3c\u0b3f\u0b06 \u0b13\u0b21\u0b3c\u0b3f\u0b06 O\u02bbzbekcha / \u045e\u0437\u0431\u0435\u043a\u0447\u0430 O\u02bbzbekcha / \u045e\u0437\u0431\u0435\u043a\u0447\u0430 O\u02bbzbekcha / \u045e\u0437\u0431\u0435\u043a\u0447\u0430 \u0a2a\u0a70\u0a1c\u0a3e\u0a2c\u0a40 \u0a2a\u0a70\u0a1c\u0a3e\u0a2c\u0a40 \u0a2a\u0a70\u0a1c\u0a3e\u0a2c\u0a40 \u067e\u0646\u062c\u0627\u0628\u06cc \u067e\u0646\u062c\u0627\u0628\u06cc \u067e\u0646\u062c\u0627\u0628\u06cc \u1797\u17b6\u179f\u17b6\u1781\u17d2\u1798\u17c2\u179a \u1797\u17b6\u179f\u17b6\u1781\u17d2\u1798\u17c2\u179a \u1797\u17b6\u179f\u17b6\u1781\u17d2\u1798\u17c2\u179a Plattd\u00fc\u00fctsch Plattd\u00fc\u00fctsch Plattd\u00fc\u00fctsch Polski Polski Polski Portugu\u00eas Portugu\u00eas Portugu\u00eas Qaraqalpaqsha Qaraqalpaqsha Qaraqalpaqsha Rom\u00e2n\u0103 Rom\u00e2n\u0103 Rom\u00e2n\u0103 Runa Simi Runa Simi Runa Simi \u0420\u0443\u0441\u0441\u043a\u0438\u0439 \u0420\u0443\u0441\u0441\u043a\u0438\u0439 \u0420\u0443\u0441\u0441\u043a\u0438\u0439 \u0421\u0430\u0445\u0430 \u0442\u044b\u043b\u0430 \u0421\u0430\u0445\u0430 \u0442\u044b\u043b\u0430 \u0421\u0430\u0445\u0430 \u0442\u044b\u043b\u0430 \u1c65\u1c5f\u1c71\u1c5b\u1c5f\u1c72\u1c64 \u1c65\u1c5f\u1c71\u1c5b\u1c5f\u1c72\u1c64 \u1c65\u1c5f\u1c71\u1c5b\u1c5f\u1c72\u1c64 Scots Scots Scots Shqip Shqip Shqip \u0dc3\u0dd2\u0d82\u0dc4\u0dbd \u0dc3\u0dd2\u0d82\u0dc4\u0dbd \u0dc3\u0dd2\u0d82\u0dc4\u0dbd Simple English Simple English Simple English Sloven\u010dina Sloven\u010dina Sloven\u010dina Sloven\u0161\u010dina Sloven\u0161\u010dina Sloven\u0161\u010dina \u06a9\u0648\u0631\u062f\u06cc \u06a9\u0648\u0631\u062f\u06cc \u06a9\u0648\u0631\u062f\u06cc \u0421\u0440\u043f\u0441\u043a\u0438 / srpski \u0421\u0440\u043f\u0441\u043a\u0438 / srpski \u0421\u0440\u043f\u0441\u043a\u0438 / srpski Srpskohrvatski / \u0441\u0440\u043f\u0441\u043a\u043e\u0445\u0440\u0432\u0430\u0442\u0441\u043a\u0438 Srpskohrvatski / \u0441\u0440\u043f\u0441\u043a\u043e\u0445\u0440\u0432\u0430\u0442\u0441\u043a\u0438 Srpskohrvatski / \u0441\u0440\u043f\u0441\u043a\u043e\u0445\u0440\u0432\u0430\u0442\u0441\u043a\u0438 Suomi Suomi Suomi Svenska Svenska Svenska Tagalog Tagalog Tagalog \u0ba4\u0bae\u0bbf\u0bb4\u0bcd \u0ba4\u0bae\u0bbf\u0bb4\u0bcd \u0ba4\u0bae\u0bbf\u0bb4\u0bcd \u0422\u0430\u0442\u0430\u0440\u0447\u0430 / tatar\u00e7a \u0422\u0430\u0442\u0430\u0440\u0447\u0430 / tatar\u00e7a \u0422\u0430\u0442\u0430\u0440\u0447\u0430 / tatar\u00e7a \u107d\u1083\u1087\u101e\u1083\u1087\u1010\u1086\u1038 \u107d\u1083\u1087\u101e\u1083\u1087\u1010\u1086\u1038 \u107d\u1083\u1087\u101e\u1083\u1087\u1010\u1086\u1038 \u0c24\u0c46\u0c32\u0c41\u0c17\u0c41 \u0c24\u0c46\u0c32\u0c41\u0c17\u0c41 \u0c24\u0c46\u0c32\u0c41\u0c17\u0c41 \u0e44\u0e17\u0e22 \u0e44\u0e17\u0e22 \u0e44\u0e17\u0e22 \u0422\u043e\u04b7\u0438\u043a\u04e3 \u0422\u043e\u04b7\u0438\u043a\u04e3 \u0422\u043e\u04b7\u0438\u043a\u04e3 T\u00fcrk\u00e7e T\u00fcrk\u00e7e T\u00fcrk\u00e7e Basa Ugi Basa Ugi Basa Ugi \u0423\u043a\u0440\u0430\u0457\u043d\u0441\u044c\u043a\u0430 \u0423\u043a\u0440\u0430\u0457\u043d\u0441\u044c\u043a\u0430 \u0423\u043a\u0440\u0430\u0457\u043d\u0441\u044c\u043a\u0430 \u0627\u0631\u062f\u0648 \u0627\u0631\u062f\u0648 \u0627\u0631\u062f\u0648 \u0626\u06c7\u064a\u063a\u06c7\u0631\u0686\u06d5 / Uyghurche \u0626\u06c7\u064a\u063a\u06c7\u0631\u0686\u06d5 / Uyghurche \u0626\u06c7\u064a\u063a\u06c7\u0631\u0686\u06d5 / Uyghurche Ti\u1ebfng Vi\u1ec7t Ti\u1ebfng Vi\u1ec7t Ti\u1ebfng Vi\u1ec7t Walon Walon Walon \u6587\u8a00 \u6587\u8a00 \u6587\u8a00 Winaray Winaray Winaray \u5434\u8bed \u5434\u8bed \u5434\u8bed \u7cb5\u8a9e \u7cb5\u8a9e \u7cb5\u8a9e \u4e2d\u6587 \u4e2d\u6587 \u4e2d\u6587 Edit links Edit links Article Article Article Talk Talk Talk English Read Read Read Edit Edit Edit View history View history View history Tools Read Read Read Edit Edit Edit View history View history View history What links here What links here What links here Related changes Related changes Related changes Upload file Upload file Upload file Special pages Special pages Special pages Permanent link Permanent link Permanent link Page information Page information Page information Cite this page Cite this page Cite this page Get shortened URL Get shortened URL Get shortened URL Download QR code Download QR code Download QR code Download as PDF Download as PDF Download as PDF Printable version Printable version Printable version Wikimedia Commons Wikimedia Commons Wikimedia Commons MediaWiki MediaWiki MediaWiki Wikibooks Wikibooks Wikibooks Wikifunctions Wikifunctions Wikifunctions Wikiquote Wikiquote Wikiquote Wikiversity Wikiversity Wikiversity Wikidata item Wikidata item Wikidata item Python (genus) Paradigm Multi-paradigm object-oriented [1] [ ] procedural imperative functional structured reflective Designed by Guido van Rossum Developer Python Software Foundation ; 33 years ago (1991-02-20) 1991-02-20 [2] [ ] Stable release ; 29 days ago (3 December 2024) 3 December 2024 Typing discipline duck dynamic strong [3] [ ] optional type annotations [4] [ ] OS Tier 1: 64-bit Linux, macOS; 64- and 32-bit Windows 10+[5] Linux macOS Windows [5] [ ] Tier 2: E.g. 32-bit WebAssembly (WASI) WebAssembly Tier 3: 64-bit Android,[6] iOS, FreeBSD, and (32-bit) Raspberry Pi OSUnofficial (or has been known to work): Other Unix-like/BSD variants) and a few other platforms[7][8][9] Android [6] [ ] iOS FreeBSD Raspberry Pi OS Unix-like BSD [7] [ ] [8] [ ] [9] [ ] License Python Software Foundation License Filename extensions [10] [ ] python.org python.org implementations CPython PyPy Stackless Python MicroPython CircuitPython IronPython Jython Dialects Cython RPython Starlark [11] [ ] ABC [12] [ ] Ada [13] [ ] ALGOL 68 [14] [ ] APL [15] [ ] C [16] [ ] C++ [17] [ ] CLU [18] [ ] Dylan [19] [ ] Haskell [20] [ ] [15] [ ] Icon [21] [ ] Lisp [22] [ ] Modula-3 Modula-3 [14] [ ] [17] [ ] Perl [23] [ ] Standard ML [15] [ ] Apache Groovy Boo Cobra CoffeeScript [24] [ ] D F# GDScript Go JavaScript [25] [ ] [26] [ ] Julia [27] [ ] Mojo [28] [ ] Nim Ring [29] [ ] Ruby [30] [ ] Swift [31] [ ] V [32] [ ] Python Programming at Wikibooks Python Programming Python is a high-level, general-purpose programming language. Its design philosophy emphasizes code readability with the use of significant indentation.[33] high-level general-purpose programming language code readability significant indentation [33] [ ] Python is dynamically type-checked and garbage-collected. It supports multiple programming paradigms, including structured (particularly procedural), object-oriented and functional programming. It is often described as a \"batteries included\" language due to its comprehensive standard library.[34][35] dynamically type-checked garbage-collected programming paradigms structured procedural object-oriented functional programming standard library [34] [ ] [35] [ ] Guido van Rossum began working on Python in the late 1980s as a successor to the ABC programming language and first released it in 1991 as Python 0.9.0.[36] Python 2.0 was released in 2000. Python 3.0, released in 2008, was a major revision not completely backward-compatible with earlier versions. Python 2.7.18, released in 2020, was the last release of Python 2.[37] Guido van Rossum ABC [36] [ ] backward-compatible [37] [ ] Python consistently ranks as one of the most popular programming languages, and has gained widespread use in the machine learning community.[38][39][40][41] machine learning [38] [ ] [39] [ ] [40] [ ] [41] [ ] History [edit] [ edit edit ] History of Python Guido van Rossum OSCON Python was conceived in the late 1980s[42] by Guido van Rossum at Centrum Wiskunde & Informatica (CWI) in the Netherlands as a successor to the ABC programming language, which was inspired by SETL,[43] capable of exception handling and interfacing with the Amoeba operating system.[12] Its implementation began in December 1989.[44] Van Rossum shouldered sole responsibility for the project, as the lead developer, until 12 July 2018, when he announced his \"permanent vacation\" from his responsibilities as Python's \"benevolent dictator for life\" (BDFL), a title the Python community bestowed upon him to reflect his long-term commitment as the project's chief decision-maker[45] (he has since come out of retirement and is self-titled \"BDFL-emeritus\"). In January 2019, active Python core developers elected a five-member Steering Council to lead the project.[46][47] [42] [ ] Guido van Rossum Centrum Wiskunde & Informatica Netherlands ABC SETL [43] [ ] exception handling Amoeba [12] [ ] [44] [ ] benevolent dictator for life [45] [ ] [46] [ ] [47] [ ] The name Python is said to come from the British comedy series Monty Python's Flying Circus.[48] Monty Python's Flying Circus [48] [ ] Python 2.0 was released on 16 October 2000, with many major new features such as list comprehensions, cycle-detecting garbage collection, reference counting, and Unicode support.[49] Python 2.7's end-of-life was initially set for 2015, then postponed to 2020 out of concern that a large body of existing code could not easily be forward-ported to Python 3.[50][51] No further security patches or other improvements will be released for it.[52][53] While Python 2.7 and older versions are officially unsupported, a different unofficial Python implementation, PyPy, continues to support Python 2, i.e. \"2.7.18+\" (plus 3.10), with the plus meaning (at least some) \"backported security updates\".[54] list comprehensions cycle-detecting reference counting Unicode [49] [ ] end-of-life [50] [ ] [51] [ ] [52] [ ] [53] [ ] PyPy backported [54] [ ] Python 3.0 was released on 3 December 2008, with some new semantics and changed syntax. At least every Python release since (now unsupported) 3.5 has added some syntax to the language, and a few later releases have dropped outdated modules, or changed semantics, at least in a minor way. Since 7 October 2024[update], Python 3.13 is the latest stable release, and it and, for few more months, 3.12 are the only releases with active support including for bug fixes (as opposed to just for security) and Python 3.9,[55] is the oldest supported version of Python (albeit in the 'security support' phase), due to Python 3.8 reaching end-of-life.[56][57] Starting with 3.13, it and later versions have 2 years of full support (up from one and a half), followed by 3 years of security support (for same total support as before). [update] [55] [ ] end-of-life [56] [ ] [57] [ ] Security updates were expedited in 2021 (and again twice in 2022, and more fixed in 2023 and in September 2024 for Python 3.12.6 down to 3.8.20), since all Python versions were insecure (including 2.7[58]) because of security issues leading to possible remote code execution[59] and web-cache poisoning.[60] [58] [ ] remote code execution [59] [ ] web-cache poisoning [60] [ ] Python 3.10 added the | union type operator[61] and the match and case keywords (for structural pattern matching statements). 3.11 expanded exception handling functionality. Python 3.12 added the new keyword type. Notable changes in 3.11 from 3.10 include increased program execution speed and improved error reporting.[62] Python 3.11 claims to be between 10 and 60% faster than Python 3.10, and Python 3.12 adds another 5% on top of that. It also has improved error messages (again improved in 3.14), and many other changes. [61] [ ] pattern matching exception handling [62] [ ] Python 3.13 introduces more syntax for types, a new and improved interactive interpreter (REPL), featuring multi-line editing and color support; an incremental garbage collector (producing shorter pauses for collection in programs with a lot of objects, and addition to the improved speed in 3.11 and 3.12), and an experimental just-in-time (JIT) compiler (such features, can/needs to be enabled specifically for the increase in speed),[63] and an experimental free-threaded build mode, which disables the global interpreter lock (GIL), allowing threads to run more concurrently, that latter feature enabled with python3.13t or python3.13t.exe. REPL just-in-time (JIT) compiler [63] [ ] global interpreter lock Python 3.13 introduces some change in behavior, i.e. new \"well-defined semantics\", fixing bugs (plus many removals of deprecated classes, functions and methods, and removed some of the C API and outdated modules): \"The [old] implementation of locals() and frame.f_locals is slow, inconsistent and buggy [and it] has many corner cases and oddities. Code that works around those may need to be changed. Code that uses locals() for simple templating, or print debugging, will continue to work correctly.\"[64] [64] [ ] Some (more) standard library modules and many deprecated classes, functions and methods, will be removed in Python 3.15 or 3.16.[65][66] [65] [ ] [66] [ ] Python 3.14 is now in alpha 2;[67] regarding possible change to annotations: \"In Python 3.14, from __future__ import annotations will continue to work as it did before, converting annotations into strings.\" [68] [67] [ ] [68] [ ] PEP 711 proposes PyBI: a standard format for distributing Python Binaries.[69] [69] [ ] Python 3.15 will \"Make UTF-8 mode default\",[70] the mode exists in all current Python versions, but currently needs to be opted into. UTF-8 is already used, by default, on Windows (and elsewhere), for most things, but e.g. to open files it's not and enabling also makes code fully cross-platform, i.e. use UTF-8 for everything on all platforms. [70] [ ] UTF-8 Design philosophy and features [edit] [ edit edit ] Python is a multi-paradigm programming language. Object-oriented programming and structured programming are fully supported, and many of their features support functional programming and aspect-oriented programming (including metaprogramming[71] and metaobjects).[72] Many other paradigms are supported via extensions, including design by contract[73][74] and logic programming.[75] Python is known as a glue language,[76] able to work very well with many other languages with ease of access. multi-paradigm programming language Object-oriented programming structured programming aspect-oriented programming metaprogramming [71] [ ] metaobjects [72] [ ] design by contract [73] [ ] [74] [ ] logic programming [75] [ ] glue language [76] [ ] Python uses dynamic typing and a combination of reference counting and a cycle-detecting garbage collector for memory management.[77] It uses dynamic name resolution (late binding), which binds method and variable names during program execution. dynamic typing reference counting memory management [77] [ ] name resolution late binding Its design offers some support for functional programming in the Lisp tradition. It has filter,mapandreduce functions; list comprehensions, dictionaries, sets, and generator expressions.[78] The standard library has two modules (itertools and functools) that implement functional tools borrowed from Haskell and Standard ML.[79] Lisp list comprehensions dictionaries generator [78] [ ] Haskell Standard ML [79] [ ] Its core philosophy is summarized in the Zen of Python (PEP 20), which includes aphorisms such as:[80] Zen of Python aphorisms [80] [ ] Beautiful is better than ugly. Explicit is better than implicit. Simple is better than complex. Complex is better than complicated. Readability counts. However, Python features regularly violate these principles and have received criticism for adding unnecessary language bloat.[81] Responses to these criticisms are that the Zen of Python is a guideline rather than a rule.[82] The addition of some new features had been so controversial that Guido van Rossum resigned as Benevolent Dictator for Life following vitriol over the addition of the assignment expression operator in Python 3.8.[83][84] [81] [ ] [82] [ ] [83] [ ] [84] [ ] Nevertheless, rather than building all of its functionality into its core, Python was designed to be highly extensible via modules. This compact modularity has made it particularly popular as a means of adding programmable interfaces to existing applications. Van Rossum's vision of a small core language with a large standard library and easily extensible interpreter stemmed from his frustrations with ABC, which espoused the opposite approach.[42] extensible ABC [42] [ ] Python claims to strive for a simpler, less-cluttered syntax and grammar while giving developers a choice in their coding methodology. In contrast to Perl's \"there is more than one way to do it\" motto, Python embraces a \"there should be one\u2014and preferably only one\u2014obvious way to do it.\" philosophy.[80] In practice, however, Python provides many ways to achieve the same task. There are, for example, at least three ways to format a string literal, with no certainty as to which one a programmer should use.[85] Alex Martelli, a Fellow at the Python Software Foundation and Python book author, wrote: \"To describe something as 'clever' is not considered a compliment in the Python culture.\"[86] Perl there is more than one way to do it [80] [ ] [85] [ ] Alex Martelli Fellow Python Software Foundation [86] [ ] Python's developers usually strive to avoid premature optimization and reject patches to non-critical parts of the CPython reference implementation that would offer marginal increases in speed at the cost of clarity.[87] Execution speed can be improved by moving speed-critical functions to extension modules written in languages such as C, or by using a just-in-time compiler like PyPy. It is also possible to cross-compile to other languages, but it either doesn't provide the full speed-up that might be expected, since Python is a very dynamic language, or a restricted subset of Python is compiled, and possibly semantics are slightly changed.[88] premature optimization CPython [87] [ ] just-in-time compiler PyPy cross-compile to other languages dynamic language [88] [ ] Python's developers aim for it to be fun to use. This is reflected in its name\u2014a tribute to the British comedy group Monty Python[89]\u2014and in occasionally playful approaches to tutorials and reference materials, such as the use of the terms \"spam\" and \"eggs\" (a reference to a Monty Python sketch) in examples, instead of the often-used \"foo\" and \"bar\".[90][91] A common neologism in the Python community is pythonic, which has a wide range of meanings related to program style. \"Pythonic\" code may use Python idioms well, be natural or show fluency in the language, or conform with Python's minimalist philosophy and emphasis on readability. Code that is difficult to understand or reads like a rough transcription from another programming language is called unpythonic.[92] Monty Python [89] [ ] a Monty Python sketch \"foo\" and \"bar\" [90] [ ] [91] [ ] neologism idioms [92] [ ] Syntax and semantics [edit] [ edit edit ] Python syntax and semantics C# Python is meant to be an easily readable language. Its formatting is visually uncluttered and often uses English keywords where other languages use punctuation. Unlike many other languages, it does not use curly brackets to delimit blocks, and semicolons after statements are allowed but rarely used. It has fewer syntactic exceptions and special cases than C or Pascal.[93] curly brackets C Pascal [93] [ ] Indentation [edit] [ edit edit ] Python syntax and semantics \u00a7 Indentation Python uses whitespace indentation, rather than curly brackets or keywords, to delimit blocks. An increase in indentation comes after certain statements; a decrease in indentation signifies the end of the current block.[94] Thus, the program's visual structure accurately represents its semantic structure.[95] This feature is sometimes termed the off-side rule. Some other languages use indentation this way; but in most, indentation has no semantic meaning. The recommended indent size is four spaces.[96] whitespace curly brackets blocks [94] [ ] [95] [ ] off-side rule [96] [ ] Statements and control flow [edit] [ edit edit ] Python's statements include: statements The assignment statement, using a single equals sign = assignment The if statement, which conditionally executes a block of code, along with else and elif (a contraction of else-if) if The for statement, which iterates over an iterable object, capturing each element to a local variable for use by the attached block for The while statement, which executes a block of code as long as its condition is true while The try statement, which allows exceptions raised in its attached code block to be caught and handled by except clauses (or new syntax except* in Python 3.11 for exception groups[97]); it also ensures that clean-up code in a finally block is always run regardless of how the block exits try [97] [ ] The raise statement, used to raise a specified exception or re-raise a caught exception The class statement, which executes a block of code and attaches its local namespace to a class, for use in object-oriented programming class The def statement, which defines a function or method function method The with statement, which encloses a code block within a context manager (for example, acquiring a lock before it is run, then releasing the lock; or opening and closing a file), allowing resource-acquisition-is-initialization (RAII)-like behavior and replacing a common try/finally idiom[98] with lock file resource-acquisition-is-initialization [98] [ ] The break statement, which exits a loop break The continue statement, which skips the rest of the current iteration and continues with the next The del statement, which removes a variable\u2014deleting the reference from the name to the value, and producing an error if the variable is referred to before it is redefined The pass statement, serving as a NOP, syntactically needed to create an empty code block NOP The assert statement, used in debugging to check for conditions that should apply assert The yield statement, which returns a value from a generator function (and also an operator); used to implement coroutines generator coroutines The return statement, used to return a value from a function The import and from statements, used to import modules whose functions or variables can be used in the current program import The match and case statements, an analog of the switch statement construct, that compares an expression against one or more cases as a control-of-flow measure. switch statement The assignment statement (=) binds a name as a reference to a separate, dynamically allocated object. Variables may subsequently be rebound at any time to any object. In Python, a variable name is a generic reference holder without a fixed data type; however, it always refers to some object with a type. This is called dynamic typing\u2014in contrast to statically-typed languages, where each variable may contain only a value of a certain type. reference object data type dynamic typing statically-typed Python does not support tail call optimization or first-class continuations, and, according to Van Rossum, it never will.[99][100] However, better support for coroutine-like functionality is provided by extending Python's generators.[101] Before 2.5, generators were lazy iterators; data was passed unidirectionally out of the generator. From Python 2.5 on, it is possible to pass data back into a generator function; and from version 3.3, it can be passed through multiple stack levels.[102] tail call first-class continuations [99] [ ] [100] [ ] coroutine generators [101] [ ] lazy iterators [102] [ ] Expressions [edit] [ edit edit ] Python's expressions include: expressions The +, -, and * operators for mathematical addition, subtraction, and multiplication are similar to other languages, but the behavior of division differs. There are two types of divisions in Python: floor division (or integer division) // and floating-point/division.[103] Python uses the ** operator for exponentiation. floor division [103] [ ] Python uses the + operator for string concatenation. Python uses the * operator for duplicating a string a specified number of times. The @ infix operator is intended to be used by libraries such as NumPy for matrix multiplication.[104][105] NumPy matrix multiplication [104] [ ] [105] [ ] The syntax :=, called the \"walrus operator\", was introduced in Python 3.8. It assigns values to variables as part of a larger expression.[106] [106] [ ] In Python, == compares by value. Python's is operator may be used to compare object identities (comparison by reference), and comparisons may be chained\u2014for example, a <= b <= c. a <= b <= c Python uses and, or, and not as Boolean operators. Python has a type of expression named a list comprehension, and a more general expression named a generator expression.[78] list comprehension generator [78] [ ] Anonymous functions are implemented using lambda expressions; however, there may be only one expression in each body. Anonymous functions lambda expressions Conditional expressions are written as x if c else y[107] (different in order of operands from the c ? x : y operator common to many other languages). x if c else y [107] [ ] c ? x : y Python makes a distinction between lists and tuples. Lists are written as [1, 2, 3], are mutable, and cannot be used as the keys of dictionaries (dictionary keys must be immutable in Python). Tuples, written as (1, 2, 3), are immutable and thus can be used as keys of dictionaries, provided all of the tuple's elements are immutable. The + operator can be used to concatenate two tuples, which does not directly modify their contents, but produces a new tuple containing the elements of both. Thus, given the variable t initially equal to (1, 2, 3), executing t = t + (4, 5) first evaluates t + (4, 5), which yields (1, 2, 3, 4, 5), which is then assigned back to t\u2014thereby effectively \"modifying the contents\" of t while conforming to the immutable nature of tuple objects. Parentheses are optional for tuples in unambiguous contexts.[108] lists tuples [ 1 , 2 , 3 ] immutable ( 1 , 2 , 3 ) ( 1 , 2 , 3 ) t = t + ( 4 , 5 ) t + ( 4 , 5 ) ( 1 , 2 , 3 , 4 , 5 ) [108] [ ] Python features sequence unpacking where multiple expressions, each evaluating to anything that can be assigned (to a variable, writable property, etc.) are associated in an identical manner to that forming tuple literals\u2014and, as a whole, are put on the left-hand side of the equal sign in an assignment statement. The statement expects an iterable object on the right-hand side of the equal sign that produces the same number of values as the provided writable expressions; when iterated through them, it assigns each of the produced values to the corresponding expression on the left.[109] [109] [ ] Python has a \"string format\" operator % that functions analogously to printf format strings in C\u2014e.g. \"spam=%s eggs=%d\" % (\"blah\", 2) evaluates to \"spam=blah eggs=2\". In Python 2.6+ and 3+, this was supplemented by the format() method of the str class, e.g. \"spam={0} eggs={1}\".format(\"blah\", 2). Python 3.6 added \"f-strings\": spam = \"blah\"; eggs = 2; f'spam={spam} eggs={eggs}'.[110] printf \"spam= %s eggs= %d \" % ( \"blah\" , 2 ) \"spam= {0} eggs= {1} \" . format ( \"blah\" , 2 ) spam = \"blah\" ; eggs = 2 ; f 'spam= { spam } eggs= { eggs } ' [110] [ ] Strings in Python can be concatenated by \"adding\" them (with the same operator as for adding integers and floats), e.g. \"spam\" + \"eggs\" returns \"spameggs\". If strings contain numbers, they are added as strings rather than integers, e.g. \"2\" + \"2\" returns \"22\". concatenated \"spam\" + \"eggs\" \"2\" + \"2\" Python has various string literals: Delimited by single or double quotes; unlike in Unix shells, Perl, and Perl-influenced languages, single and double quotes work the same. Both use the backslash (\\) as an escape character. String interpolation became available in Python 3.6 as \"formatted string literals\".[110] Triple-quoted (beginning and ending with three single or double quotes), which may span multiple lines and function like here documents in shells, Perl, and Ruby. Raw string varieties, denoted by prefixing the string literal with r. Escape sequences are not interpreted; hence raw strings are useful where literal backslashes are common, such as regular expressions and Windows-style paths. (Compare \"@-quoting\" in C#.) string literals Delimited by single or double quotes; unlike in Unix shells, Perl, and Perl-influenced languages, single and double quotes work the same. Both use the backslash (\\) as an escape character. String interpolation became available in Python 3.6 as \"formatted string literals\".[110] Unix shells Perl escape character String interpolation [110] [ ] Triple-quoted (beginning and ending with three single or double quotes), which may span multiple lines and function like here documents in shells, Perl, and Ruby. here documents Ruby Raw string varieties, denoted by prefixing the string literal with r. Escape sequences are not interpreted; hence raw strings are useful where literal backslashes are common, such as regular expressions and Windows-style paths. (Compare \"@-quoting\" in C#.) Raw string regular expressions Windows C# Python has array index and array slicing expressions in lists, denoted as a[key], a[start:stop] or a[start:stop:step]. Indexes are zero-based, and negative indexes are relative to the end. Slices take elements from the start index up to, but not including, the stop index. The third slice parameter, called step or stride, allows elements to be skipped and reversed. Slice indexes may be omitted\u2014for example, a[:] returns a copy of the entire list. Each element of a slice is a shallow copy. array index array slicing a [ start : stop ] a [ start : stop : step ] zero-based a [:] shallow copy In Python, a distinction between expressions and statements is rigidly enforced, in contrast to languages such as Common Lisp, Scheme, or Ruby. This leads to duplicating some functionality. For example: Common Lisp Scheme Ruby List comprehensions vs. for-loops List comprehensions Conditional expressions vs. if blocks Conditional The eval() vs. exec() built-in functions (in Python 2, exec is a statement); the former is for expressions, the latter is for statements Statements cannot be a part of an expression\u2014so list and other comprehensions or lambda expressions, all being expressions, cannot contain statements. A particular case is that an assignment statement such as a = 1 cannot form part of the conditional expression of a conditional statement. lambda expressions a = 1 Methods [edit] [ edit edit ] Methods on objects are functions attached to the object's class; the syntax instance.method(argument) is, for normal methods and functions, syntactic sugar for Class.method(instance, argument). Python methods have an explicit self parameter to access instance data, in contrast to the implicit self (or this) in some other object-oriented programming languages (e.g., C++, Java, Objective-C, Ruby).[111] Python also provides methods, often called dunder methods (due to their names beginning and ending with double-underscores), to allow user-defined classes to modify how they are handled by native operations including length, comparison, in arithmetic operations and type conversion.[112] Methods functions instance . method ( argument ) syntactic sugar Class . method ( instance , argument ) self instance data C++ Java Objective-C Ruby [111] [ ] arithmetic operations [112] [ ] Typing [edit] [ edit edit ] Python uses duck typing and has typed objects but untyped variable names. Type constraints are not checked at compile time; rather, operations on an object may fail, signifying that it is not of a suitable type. Despite being dynamically typed, Python is strongly typed, forbidding operations that are not well-defined (for example, adding a number to a string) rather than silently attempting to make sense of them. duck typing compile time dynamically typed strongly typed Python allows programmers to define their own types using classes, most often used for object-oriented programming. New instances of classes are constructed by calling the class (for example, SpamClass() or EggsClass()), and the classes are instances of the metaclass type (itself an instance of itself), allowing metaprogramming and reflection. classes object-oriented programming instances SpamClass () EggsClass () metaclass reflection Before version 3.0, Python had two kinds of classes (both using the same syntax): old-style and new-style;[113] current Python versions only support the semantics of the new style. [113] [ ] Python supports optional type annotations.[4][114] These annotations are not enforced by the language, but may be used by external tools such as mypy to catch errors.[115][116] Mypy also supports a Python compiler called mypyc, which leverages type annotations for optimization.[117] optional type annotations [4] [ ] [114] [ ] [115] [ ] [116] [ ] [117] [ ] Mutability Boolean value True False bytes bytearray ( b 'Some ASCII' ) bytearray ( b \"Some ASCII\" ) bytearray ([ 119 , 105 , 107 , 105 ]) b 'Some ASCII' b \"Some ASCII\" bytes ([ 119 , 105 , 107 , 105 ]) Complex number 3 + 2.7 j 3 + 2.7 j Associative array { 'key1' : 1.0 , 3 : False } {} ellipsis NumPy ... Ellipsis Double-precision floating-point number IEEE 754 [118] [ ] 1.33333 1.33333 set frozenset([4.0, 'string', True]) frozenset ([ 4.0 , 'string' , True ]) Integer [119] [ ] 42 List [ 4.0 , 'string' , True ] [] null None overloaded operators NotImplemented [120] [ ] range ( \u2212 1 , 10 ) range ( 10 , \u2212 5 , \u2212 2 ) set { 4.0 , 'string' , True } set () character string 'Wikipedia' \"Wikipedia\" \"\"\"Spanning multiple lines\"\"\" Spanning multiple lines ( 4.0 , 'string' , True ) ( 'single element' ,) () Arithmetic operations [edit] [ edit edit ] Python has the usual symbols for arithmetic operators (+, -, *, /), the floor division operator // and the modulo operation % (where the remainder can be negative, e.g. 4 % -3 == -2). It also has ** for exponentiation, e.g. 5**3 == 125 and 9**0.5 == 3.0, and a matrix\u2011multiplication operator @ .[121] These operators work like in traditional math; with the same precedence rules, the operators infix (+ and - can also be unary to represent positive and negative numbers respectively). modulo operation exponentiation [121] [ ] precedence rules infix unary The division between integers produces floating-point results. The behavior of division has changed significantly over time:[122] [122] [ ] Current Python (i.e. since 3.0) changed / to always be floating-point division, e.g. 5/2 == 2.5. 5 / 2 == 2.5 The floor division // operator was introduced. So 7//3 == 2, -7//3 == -3, 7.5//3 == 2.0 and -7.5//3 == -3.0. Adding from __future__ import division causes a module used in Python 2.7 to use Python 3.0 rules for division (see above). from __future__ import division In Python terms, / is true division (or simply division), and // is floor division. / before version 3.0 is classic division.[122] [122] [ ] Rounding towards negative infinity, though different from most languages, adds consistency. For instance, it means that the equation (a + b)//b == a//b + 1 is always true. It also means that the equation b*(a//b) + a%b == a is valid for both positive and negative values of a. However, maintaining the validity of this equation means that while the result of a%b is, as expected, in the half-open interval [0, b), where b is a positive integer, it has to lie in the interval (b, 0] when b is negative.[123] ( a + b ) // b == a // b + 1 b * ( a // b ) + a % b == a half-open interval [123] [ ] Python provides a round function for rounding a float to the nearest integer. For tie-breaking, Python 3 uses round to even: round(1.5) and round(2.5) both produce 2.[124] Versions before 3 used round-away-from-zero: round(0.5) is 1.0, round(-0.5) is \u22121.0.[125] rounding tie-breaking round to even [124] [ ] round-away-from-zero [125] [ ] Python allows Boolean expressions with multiple equality relations in a manner that is consistent with general use in mathematics. For example, the expression a < b < c tests whether a is less than b and b is less than c.[126] C-derived languages interpret this expression differently: in C, the expression would first evaluate a < b, resulting in 0 or 1, and that result would then be compared with c.[127] [126] [ ] [127] [ ] Python uses arbitrary-precision arithmetic for all integer operations. The Decimal type/class in the decimal module provides decimal floating-point numbers to a pre-defined arbitrary precision and several rounding modes.[128] The Fraction class in the fractions module provides arbitrary precision for rational numbers.[129] arbitrary-precision arithmetic decimal floating-point numbers [128] [ ] rational numbers [129] [ ] Due to Python's extensive mathematics library, and the third-party library NumPy that further extends the native capabilities, it is frequently used as a scientific scripting language to aid in problems such as numerical data processing and manipulation.[130][131] NumPy [130] [ ] [131] [ ] Function syntax [edit] [ edit edit ] Functions are created in Python using the def keyword. In Python, you define the function as if you were calling it, by typing the function name and then the attributes required. Here is an example of a function that will print whatever is given: def printer ( input1 , input2 = \"already there\" ): print ( input1 ) print ( input2 ) printer ( \"hello\" ) # Example output: # hello # already there If you want the attribute to have a set value if no value is given, use the variable-defining syntax inside the function definition. Programming examples [edit] [ edit edit ] \"Hello, World!\" program: \"Hello, World!\" program print ( 'Hello, world!' ) Program to calculate the factorial of a positive integer: factorial n = int ( input ( 'Type a number, and its factorial will be printed: ' )) if n < 0 : raise ValueError ( 'You must enter a non-negative integer' ) factorial = 1 for i in range ( 2 , n + 1 ): factorial *= i print ( factorial ) Libraries [edit] [ edit edit ] Python's large standard library[132] provides tools suited to many tasks and is commonly cited as one of its greatest strengths. For Internet-facing applications, many standard formats and protocols such as MIME and HTTP are supported. It includes modules for creating graphical user interfaces, connecting to relational databases, generating pseudorandom numbers, arithmetic with arbitrary-precision decimals,[128] manipulating regular expressions, and unit testing. [132] [ ] MIME HTTP graphical user interfaces relational databases generating pseudorandom numbers [128] [ ] regular expressions unit testing Some parts of the standard library are covered by specifications\u2014for example, the Web Server Gateway Interface (WSGI) implementation wsgiref follows PEP 333[133]\u2014but most are specified by their code, internal documentation, and test suites. However, because most of the standard library is cross-platform Python code, only a few modules need altering or rewriting for variant implementations. Web Server Gateway Interface [133] [ ] test suites As of 17 March 2024,[update] the Python Package Index (PyPI), the official repository for third-party Python software, contains over 523,000[134] packages with a wide range of functionality, including: [update] Python Package Index [134] [ ] Automation Automation Data analytics Data analytics Databases Databases Documentation Documentation Graphical user interfaces Graphical user interfaces Image processing Image processing Machine learning Machine learning Mobile apps Mobile apps Multimedia Multimedia Computer networking Computer networking Scientific computing Scientific computing System administration System administration Test frameworks Test frameworks Text processing Text processing Web frameworks Web frameworks Web scraping Web scraping Development environments [edit] [ edit edit ] Comparison of integrated development environments \u00a7 Python Most Python implementations (including CPython) include a read\u2013eval\u2013print loop (REPL), permitting them to function as a command line interpreter for which users enter statements sequentially and receive results immediately. read\u2013eval\u2013print loop command line interpreter Python also comes with an Integrated development environment (IDE) called IDLE, which is more beginner-oriented. Integrated development environment (IDE) IDLE Other shells, including IDLE and IPython, add further abilities such as improved auto-completion, session state retention, and syntax highlighting. IDLE IPython syntax highlighting As well as standard desktop integrated development environments including PyCharm, IntelliJ Idea, Visual Studio Code etc, there are web browser-based IDEs, including SageMath, for developing science- and math-related programs; PythonAnywhere, a browser-based IDE and hosting environment; and Canopy IDE, a commercial IDE emphasizing scientific computing.[135] integrated development environments web browser SageMath PythonAnywhere scientific computing [135] [ ] Implementations [edit] [ edit edit ] List of Python software \u00a7 Python implementations Reference implementation [edit] [ edit edit ] CPython is the reference implementation of Python. It is written in C, meeting the C89 standard (Python 3.11 uses C11[136]) with several select C99 features. CPython includes its own C extensions, but third-party extensions are not limited to older C versions\u2014e.g. they can be implemented with C11 or C++.[137][138] CPython compiles Python programs into an intermediate bytecode[139] which is then executed by its virtual machine.[140] CPython is distributed with a large standard library written in a mixture of C and native Python, and is available for many platforms, including Windows (starting with Python 3.9, the Python installer deliberately fails to install on Windows 7 and 8;[141][142] Windows XP was supported until Python 3.5) and most modern Unix-like systems, including macOS (and Apple M1 Macs, since Python 3.9.1, with experimental installer), with unofficial support for VMS.[143] Platform portability was one of its earliest priorities.[144] (During Python 1 and 2 development, even OS/2 and Solaris were supported,[145] but support has since been dropped for many platforms.) CPython reference implementation C89 C11 [136] [ ] C99 C11 [137] [ ] [138] [ ] compiles bytecode [139] [ ] virtual machine [140] [ ] Windows 7 [141] [ ] [142] [ ] Windows XP Unix-like Apple M1 VMS [143] [ ] [144] [ ] OS/2 Solaris [145] [ ] All current Python versions (i.e. since 3.7) only support operating systems with multi-threading support. Other implementations [edit] [ edit edit ] All alternative implementations have at least slightly different semantics (e.g. may have unordered dictionaries, unlike all current Python versions), e.g. with the larger Python ecosystem, such as with supporting the C Python API of with PyPy: PyPy is a fast, compliant interpreter of Python 2.7 and 3.10.[146][147] Its just-in-time compiler often brings a significant speed improvement over CPython, but some libraries written in C cannot be used with it.[148] It has e.g. RISC-V support. PyPy [146] [ ] [147] [ ] just-in-time compiler [148] [ ] RISC-V Codon is a language with an ahead-of-time (AOT) compiler, that (AOT) compiles a statically-typed Python-like language with \"syntax and semantics are nearly identical to Python's, there are some notable differences\"[149] e.g. it uses 64-bit machine integers, for speed, not arbitrary like Python, and it claims speedups over CPython are usually on the order of 10\u2013100x. It compiles to machine code (via LLVM) and supports native multithreading.[150] Codon can also compile to Python extension modules that can be imported and used from Python. Codon ahead-of-time (AOT) compiler [149] [ ] LLVM [150] [ ] Stackless Python is a significant fork of CPython that implements microthreads; it does not use the call stack in the same way, thus allowing massively concurrent programs. PyPy also has a stackless version.[151] Stackless Python microthreads call stack [151] [ ] MicroPython and CircuitPython are Python 3 variants optimized for microcontrollers, including Lego Mindstorms EV3.[152] MicroPython CircuitPython microcontrollers Lego Mindstorms EV3 [152] [ ] Pyston is a variant of the Python runtime that uses just-in-time compilation to speed up the execution of Python programs.[153] [153] [ ] Cinder is a performance-oriented fork of CPython 3.8 that contains a number of optimizations, including bytecode inline caching, eager evaluation of coroutines, a method-at-a-time JIT, and an experimental bytecode compiler.[154] JIT [154] [ ] Snek[155][156][157] Embedded Computing Language (compatible with e.g. 8-bit AVR microcontrollers such as ATmega 328P-based Arduino, as well as larger ones compatible with MicroPython) \"is Python-inspired, but it is not Python. It is possible to write Snek programs that run under a full Python system, but most Python programs will not run under Snek.\"[158] It is an imperative language not including OOP / classes, unlike Python, and simplifying to one number type with 32-bit single-precision (similar to JavaScript, except smaller). Snek [155] [ ] [156] [ ] [157] [ ] AVR microcontrollers ATmega 328P MicroPython [158] [ ] OOP single-precision JavaScript No longer supported implementations [edit] [ edit edit ] Other just-in-time Python compilers have been developed, but are now unsupported: Google began a project named Unladen Swallow in 2009, with the aim of speeding up the Python interpreter five-fold by using the LLVM, and of improving its multithreading ability to scale to thousands of cores,[159] while ordinary implementations suffer from the global interpreter lock. Unladen Swallow LLVM multithreading [159] [ ] global interpreter lock Psyco is a discontinued just-in-time specializing compiler that integrates with CPython and transforms bytecode to machine code at runtime. The emitted code is specialized for certain data types and is faster than the standard Python code. Psyco does not support Python 2.7 or later. Psyco just-in-time specializing data types PyS60 was a Python 2 interpreter for Series 60 mobile phones released by Nokia in 2005. It implemented many of the modules from the standard library and some additional modules for integrating with the Symbian operating system. The Nokia N900 also supports Python with GTK widget libraries, enabling programs to be written and run on the target device.[160] PyS60 Series 60 Nokia Symbian N900 GTK [160] [ ] Cross-compilers to other languages [edit] [ edit edit ] There are several compilers/transpilers to high-level object languages, with either unrestricted Python, a restricted subset of Python, or a language similar to Python as the source language: transpilers Brython,[161] Transcrypt[162][163] and Pyjs (latest release in 2012) compile Python to JavaScript. [161] [ ] [162] [ ] [163] [ ] Pyjs JavaScript Cython compiles (a superset of) Python to C. The resulting code is also usable with Python via direct C-level API calls into the Python interpreter. Cython PyJL compiles/transpiles a subset of Python to \"human-readable, maintainable, and high-performance Julia source code\".[88] Despite claiming high performance, no tool can claim to do that for arbitrary Python code; i.e. it's known not possible to compile to a faster language or machine code. Unless semantics of Python are changed, but in many cases speedup is possible with few or no changes in the Python code. The faster Julia source code can then be used from Python, or compiled to machine code, and based that way. [88] [ ] Nuitka compiles Python into C.[164] It works with Python 3.4 to 3.12 (and 2.6 and 2.7), for Python's main supported platforms (and Windows 7 or even Windows XP) and for Android. It claims complete support for Python 3.10, some support for 3.11 and 3.12 and experimental support for Python 3.13. It supports macOS including Apple Silicon-based. It's a free compiler, though it also has commercial add-ons (e.g. for hiding source code). Nuitka [164] [ ] Numba is used from Python, as a tool (enabled by adding a decorator to relevant Python code), a JIT compiler that translates a subset of Python and NumPy code into fast machine code. Numba Pythran compiles a subset of Python 3 to C++ (C++11).[165] C++11 [165] [ ] RPython can be compiled to C, and is used to build the PyPy interpreter of Python. RPython The Python \u2192 11l \u2192 C++ transpiler[166] compiles a subset of Python 3 to C++ (C++17). [166] [ ] C++17 Specialized: MyHDL is a Python-based hardware description language (HDL), that converts MyHDL code to Verilog or VHDL code. MyHDL hardware description language Verilog VHDL Older projects (or not to be used with Python 3.x and latest syntax): Google's Grumpy (latest release in 2017) transpiles Python 2 to Go.[167][168][169] transpiles Go [167] [ ] [168] [ ] [169] [ ] IronPython allows running Python 2.7 programs (and an alpha, released in 2021, is also available for \"Python 3.4, although features and behaviors from later versions may be included\"[170]) on the .NET Common Language Runtime.[171] IronPython alpha [170] [ ] Common Language Runtime [171] [ ] Jython compiles Python 2.7 to Java bytecode, allowing the use of the Java libraries from a Python program.[172] Jython [172] [ ] Pyrex (latest release in 2010) and Shed Skin (latest release in 2013) compile to C and C++ respectively. Pyrex Shed Skin Performance [edit] [ edit edit ] Performance comparison of various Python implementations on a non-numerical (combinatorial) workload was presented at EuroSciPy '13.[173] Python's performance compared to other programming languages is also benchmarked by The Computer Language Benchmarks Game.[174] [173] [ ] The Computer Language Benchmarks Game [174] [ ] Development [edit] [ edit edit ] Python's development is conducted largely through the Python Enhancement Proposal (PEP) process, the primary mechanism for proposing major new features, collecting community input on issues, and documenting Python design decisions.[175] Python coding style is covered in PEP 8.[176] Outstanding PEPs are reviewed and commented on by the Python community and the steering council.[175] [175] [ ] [176] [ ] [175] [ ] Enhancement of the language corresponds with the development of the CPython reference implementation. The mailing list python-dev is the primary forum for the language's development. Specific issues were originally discussed in the Roundup bug tracker hosted at by the foundation.[177] In 2022, all issues and discussions were migrated to GitHub.[178] Development originally took place on a self-hosted source-code repository running Mercurial, until Python moved to GitHub in January 2017.[179] Roundup bug tracker [177] [ ] GitHub [178] [ ] self-hosted Mercurial GitHub [179] [ ] CPython's public releases come in three types, distinguished by which part of the version number is incremented: Backward-incompatible versions, where code is expected to break and needs to be manually ported. The first part of the version number is incremented. These releases happen infrequently\u2014version 3.0 was released 8 years after 2.0. According to Guido van Rossum, a version 4.0 is very unlikely to ever happen.[180] ported [180] [ ] Major or \"feature\" releases are largely compatible with the previous version but introduce new features. The second part of the version number is incremented. Starting with Python 3.9, these releases are expected to happen annually.[181][182] Each major version is supported by bug fixes for several years after its release.[183] [181] [ ] [182] [ ] [183] [ ] Bug fix releases,[184] which introduce no new features, occur about every 3 months and are made when a sufficient number of bugs have been fixed upstream since the last release. Security vulnerabilities are also patched in these releases. The third and final part of the version number is incremented.[184] [184] [ ] [184] [ ] Many alpha, beta, and release-candidates are also released as previews and for testing before final releases. Although there is a rough schedule for each release, they are often delayed if the code is not ready. Python's development team monitors the state of the code by running the large unit test suite during development.[185] alpha, beta, and release-candidates unit test [185] [ ] The major academic conference on Python is PyCon. There are also special Python mentoring programs, such as PyLadies. academic conference PyCon PyLadies Python 3.12 removed wstr meaning Python extensions[186] need to be modified,[187] and 3.10 added pattern matching to the language.[188] [186] [ ] [187] [ ] pattern matching [188] [ ] Python 3.12 dropped some outdated modules, and more will be dropped in the future, deprecated as of 3.13; already deprecated array 'u' format code will emit DeprecationWarning since 3.13 and will be removed in Python 3.16. The 'w' format code should be used instead. Part of ctypes is also deprecated and http.server.CGIHTTPRequestHandler will emit a DeprecationWarning, and will be removed in 3.15. Using that code already has a high potential for both security and functionality bugs. Parts of the typing module are deprecated, e.g. creating a typing.NamedTuple class using keyword arguments to denote the fields and such (and more) will be disallowed in Python 3.15. API documentation generators [edit] [ edit edit ] Tools that can generate documentation for Python API include pydoc (available as part of the standard library), Sphinx, Pdoc and its forks, Doxygen and Graphviz, among others.[189] API pydoc Sphinx Pdoc Doxygen Graphviz [189] [ ] Naming [edit] [ edit edit ] Python's name is derived from the British comedy group Monty Python, whom Python creator Guido van Rossum enjoyed while developing the language. Monty Python references appear frequently in Python code and culture;[190] for example, the metasyntactic variables often used in Python literature are spam and eggs instead of the traditional foo and bar.[190][191] The official Python documentation also contains various references to Monty Python routines.[192][193] Users of Python are sometimes referred to as \"Pythonistas\".[194] Monty Python [190] [ ] metasyntactic variables spam and eggs foo and bar [190] [ ] [191] [ ] [192] [ ] [193] [ ] [194] [ ] The prefix Py- is used to show that something is related to Python. Examples of the use of this prefix in names of Python applications or libraries include Pygame, a binding of Simple DirectMedia Layer to Python (commonly used to create games); PyQt and PyGTK, which bind Qt and GTK to Python respectively; and PyPy, a Python implementation originally written in Python. Pygame binding Simple DirectMedia Layer PyQt PyGTK Qt PyPy Popularity [edit] [ edit edit ] Since 2003, Python has consistently ranked in the top ten most popular programming languages in the TIOBE Programming Community Index where as of December 2022[update] it was the most popular language (ahead of C, C++, and Java).[40] It was selected as Programming Language of the Year (for \"the highest rise in ratings in a year\") in 2007, 2010, 2018, and 2020 (the only language to have done so four times as of 2020[update][195]). TIOBE Programming Community Index [update] Java [40] [ ] [update] [195] [ ] Large organizations that use Python include Wikipedia, Google,[196] Yahoo!,[197] CERN,[198] NASA,[199] Facebook,[200] Amazon, Instagram,[201] Spotify,[202] and some smaller entities like Industrial Light & Magic[203] and ITA.[204] The social news networking site Reddit was written mostly in Python.[205] Organizations that partially use Python include Discord[206] and Baidu.[207] Wikipedia Google [196] [ ] Yahoo! [197] [ ] CERN [198] [ ] NASA [199] [ ] Facebook [200] [ ] Amazon Instagram [201] [ ] Spotify [202] [ ] Industrial Light & Magic [203] [ ] ITA [204] [ ] Reddit [205] [ ] Discord [206] [ ] Baidu [207] [ ] Uses [edit] [ edit edit ] List of Python software Python can serve as a scripting language for web applications, e.g. via mod_wsgi for the Apache webserver.[208] With Web Server Gateway Interface, a standard API has evolved to facilitate these applications. Web frameworks like Django, Pylons, Pyramid, TurboGears, web2py, Tornado, Flask, Bottle, and Zope support developers in the design and maintenance of complex applications. Pyjs and IronPython can be used to develop the client-side of Ajax-based applications. SQLAlchemy can be used as a data mapper to a relational database. Twisted is a framework to program communications between computers, and is used (for example) by Dropbox. scripting language web applications mod_wsgi Apache webserver [208] [ ] Web Server Gateway Interface Web frameworks Django Pylons Pyramid TurboGears web2py Tornado Flask Zope IronPython SQLAlchemy data mapper Twisted Dropbox Libraries such as NumPy, SciPy and Matplotlib allow the effective use of Python in scientific computing,[209][210] with specialized libraries such as Biopython and Astropy providing domain-specific functionality. SageMath is a computer algebra system with a notebook interface programmable in Python: its library covers many aspects of mathematics, including algebra, combinatorics, numerical mathematics, number theory, and calculus.[211] OpenCV has Python bindings with a rich set of features for computer vision and image processing.[212] NumPy SciPy Matplotlib [209] [ ] [210] [ ] Biopython Astropy SageMath computer algebra system notebook interface mathematics algebra combinatorics numerical mathematics number theory calculus [211] [ ] OpenCV computer vision image processing [212] [ ] Python is commonly used in artificial intelligence projects and machine learning projects with the help of libraries like TensorFlow, Keras, Pytorch, scikit-learn and the Logic language ProbLog.[213][214][215][216][217] As a scripting language with a modular architecture, simple syntax, and rich text processing tools, Python is often used for natural language processing.[218] artificial intelligence TensorFlow Keras Pytorch scikit-learn ProbLog [213] [ ] [214] [ ] [215] [ ] [216] [ ] [217] [ ] modular architecture natural language processing [218] [ ] The combination of Python and Prolog has proved to be particularly useful for AI applications, with Prolog providing knowledge representation and reasoning capabilities. The Janus system, in particular, exploits the similarities between these two languages, in part because of their use of dynamic typing, and the simple recursive nature of their data structures. Typical applications of this combination include natural language processing, visual query answering, geospatial reasoning, and handling of semantic web data.[219][220] The Natlog system, implemented in Python, uses Definite Clause Grammars (DCGs) as prompt generators for text-to-text generators like GPT3 and text-to-image generators like DALL-E or Stable Diffusion.[221] Prolog [219] [ ] [220] [ ] Definite Clause Grammars [221] [ ] Python can also be used for graphical user interface (GUI) by using libraries like Tkinter.[222][223] graphical user interface Tkinter [222] [ ] [223] [ ] Python has been successfully embedded in many software products as a scripting language, including in finite element method software such as Abaqus, 3D parametric modelers like FreeCAD, 3D animation packages such as 3ds Max, Blender, Cinema 4D, Lightwave, Houdini, Maya, modo, MotionBuilder, Softimage, the visual effects compositor Nuke, 2D imaging programs like GIMP,[224] Inkscape, Scribus and Paint Shop Pro,[225] and musical notation programs like scorewriter and capella. GNU Debugger uses Python as a pretty printer to show complex structures such as C++ containers. Esri promotes Python as the best choice for writing scripts in ArcGIS.[226] It has also been used in several video games,[227][228] and has been adopted as first of the three available programming languages in Google App Engine, the other two being Java and Go.[229] finite element method Abaqus FreeCAD 3ds Max Blender Cinema 4D Lightwave Houdini Maya modo MotionBuilder Softimage Nuke GIMP [224] [ ] Inkscape Scribus Paint Shop Pro [225] [ ] musical notation scorewriter capella GNU Debugger pretty printer Esri ArcGIS [226] [ ] [227] [ ] [228] [ ] programming languages Google App Engine Java Go [229] [ ] Many operating systems include Python as a standard component. It ships with most Linux distributions,[230] AmigaOS 4 (using Python 2.7), FreeBSD (as a package), NetBSD, and OpenBSD (as a package) and can be used from the command line (terminal). Many Linux distributions use installers written in Python: Ubuntu uses the Ubiquity installer, while Red Hat Linux and Fedora Linux use the Anaconda installer. Gentoo Linux uses Python in its package management system, Portage. Linux distributions [230] [ ] AmigaOS 4 FreeBSD NetBSD OpenBSD Ubuntu Ubiquity Red Hat Linux Fedora Linux Anaconda Gentoo Linux package management system Portage Python is used extensively in the information security industry, including in exploit development.[231][232] information security [231] [ ] [232] [ ] Most of the Sugar software for the One Laptop per Child XO, developed at Sugar Labs as of 2008[update], is written in Python.[233] The Raspberry Pi single-board computer project has adopted Python as its main user-programming language. Sugar One Laptop per Child Sugar Labs [update] [233] [ ] Raspberry Pi single-board computer LibreOffice includes Python and intends to replace Java with Python. Its Python Scripting Provider is a core feature[234] since Version 4.0 from 7 February 2013. LibreOffice [234] [ ] Languages influenced by Python [edit] [ edit edit ] Python's design and philosophy have influenced many other programming languages: Boo uses indentation, a similar syntax, and a similar object model.[235] Boo [235] [ ] Cobra uses indentation and a similar syntax, and its Acknowledgements document lists Python first among languages that influenced it.[236] Cobra [236] [ ] CoffeeScript, a programming language that cross-compiles to JavaScript, has Python-inspired syntax. CoffeeScript ECMAScript\u2013JavaScript borrowed iterators and generators from Python.[237] ECMAScript JavaScript generators [237] [ ] GDScript, a scripting language very similar to Python, built-in to the Godot game engine.[238] GDScript Godot [238] [ ] Go is designed for the \"speed of working in a dynamic language like Python\"[239] and shares the same syntax for slicing arrays. Go [239] [ ] Groovy was motivated by the desire to bring the Python design philosophy to Java.[240] Groovy Java [240] [ ] Julia was designed to be \"as usable for general programming as Python\".[27] Julia [27] [ ] Mojo is a non-strict[28][241] superset of Python (e.g. still missing classes, and adding e.g. struct).[242] Mojo [28] [ ] [241] [ ] struct [242] [ ] Nim uses indentation and similar syntax.[243] Nim [243] [ ] Ruby's creator, Yukihiro Matsumoto, has said: \"I wanted a scripting language that was more powerful than Perl, and more object-oriented than Python. That's why I decided to design my own language.\"[244] Ruby Yukihiro Matsumoto [244] [ ] Swift, a programming language developed by Apple, has some Python-inspired syntax.[245] Swift [245] [ ] Kotlin blends Python and Java features, minimizing boilerplate code for enhanced developer efficiency.[246] Kotlin [246] [ ] Python's development practices have also been emulated by other languages. For example, the practice of requiring a document describing the rationale for, and issues surrounding, a change to the language (in Python, a PEP) is also used in Tcl,[247] Erlang,[248] and Swift.[249] Tcl [247] [ ] Erlang [248] [ ] [249] [ ] See also [edit] [ edit edit ] Computer programming portal Computer programming portal Computer programming portal Free and open-source software portal Free and open-source software portal Free and open-source software portal Python syntax and semantics Python syntax and semantics pip (package manager) pip (package manager) List of programming languages List of programming languages History of programming languages History of programming languages Comparison of programming languages Comparison of programming languages References [edit] [ edit edit ] ^ \"General Python FAQ \u2013 Python 3 documentation\". docs.python.org. Retrieved 7 July 2024. ^ ^ \"General Python FAQ \u2013 Python 3 documentation\". docs.python.org. Retrieved 7 July 2024. \"General Python FAQ \u2013 Python 3 documentation\" . Retrieved 7 July 2024 7 July ^ \"Python 0.9.1 part 01/21\". alt.sources archives. Archived from the original on 11 August 2021. Retrieved 11 August 2021. ^ ^ \"Python 0.9.1 part 01/21\". alt.sources archives. Archived from the original on 11 August 2021. Retrieved 11 August 2021. \"Python 0.9.1 part 01/21\" Archived . Retrieved 11 August 2021 11 August ^ \"Why is Python a dynamic language and also a strongly typed language\". Python Wiki. Archived from the original on 14 March 2021. Retrieved 27 January 2021. ^ ^ \"Why is Python a dynamic language and also a strongly typed language\". Python Wiki. Archived from the original on 14 March 2021. Retrieved 27 January 2021. \"Why is Python a dynamic language and also a strongly typed language\" Archived . Retrieved 27 January 2021 27 January ^ a b \"PEP 483 \u2013 The Theory of Type Hints\". Python.org. Archived from the original on 14 June 2020. Retrieved 14 June 2018. ^ a b a b \"PEP 483 \u2013 The Theory of Type Hints\". Python.org. Archived from the original on 14 June 2020. Retrieved 14 June 2018. \"PEP 483 \u2013 The Theory of Type Hints\" Archived . Retrieved 14 June 2018 14 June ^ \"PEP 11 \u2013 CPython platform support | peps.python.org\". Python Enhancement Proposals (PEPs). Retrieved 22 April 2024. ^ ^ \"PEP 11 \u2013 CPython platform support | peps.python.org\". Python Enhancement Proposals (PEPs). Retrieved 22 April 2024. \"PEP 11 \u2013 CPython platform support | peps.python.org\" . Retrieved 22 April 2024 22 April ^ \"PEP 738 \u2013 Adding Android as a supported platform | peps.python.org\". Python Enhancement Proposals (PEPs). Retrieved 19 May 2024. ^ ^ \"PEP 738 \u2013 Adding Android as a supported platform | peps.python.org\". Python Enhancement Proposals (PEPs). Retrieved 19 May 2024. \"PEP 738 \u2013 Adding Android as a supported platform | peps.python.org\" . Retrieved 19 May 2024 19 May ^ \"Download Python for Other Platforms\". Python.org. Archived from the original on 27 November 2020. Retrieved 18 August 2023. ^ ^ \"Download Python for Other Platforms\". Python.org. Archived from the original on 27 November 2020. Retrieved 18 August 2023. \"Download Python for Other Platforms\" Archived . Retrieved 18 August 2023 18 August ^ \"test \u2013 Regression tests package for Python \u2013 Python 3.7.13 documentation\". docs.python.org. Archived from the original on 17 May 2022. Retrieved 17 May 2022. ^ ^ \"test \u2013 Regression tests package for Python \u2013 Python 3.7.13 documentation\". docs.python.org. Archived from the original on 17 May 2022. Retrieved 17 May 2022. \"test \u2013 Regression tests package for Python \u2013 Python 3.7.13 documentation\" Archived . Retrieved 17 May 2022 17 May ^ \"platform \u2013 Access to underlying platform's identifying data \u2013 Python 3.10.4 documentation\". docs.python.org. Archived from the original on 17 May 2022. Retrieved 17 May 2022. ^ ^ \"platform \u2013 Access to underlying platform's identifying data \u2013 Python 3.10.4 documentation\". docs.python.org. Archived from the original on 17 May 2022. Retrieved 17 May 2022. \"platform \u2013 Access to underlying platform's identifying data \u2013 Python 3.10.4 documentation\" Archived . Retrieved 17 May 2022 17 May ^ Holth, Moore (30 March 2014). \"PEP 0441 \u2013 Improving Python ZIP Application Support\". Archived from the original on 26 December 2018. Retrieved 12 November 2015. ^ ^ Holth, Moore (30 March 2014). \"PEP 0441 \u2013 Improving Python ZIP Application Support\". Archived from the original on 26 December 2018. Retrieved 12 November 2015. \"PEP 0441 \u2013 Improving Python ZIP Application Support\" Archived . Retrieved 12 November 2015 12 November ^ \"Starlark Language\". Archived from the original on 15 June 2020. Retrieved 25 May 2019. ^ ^ \"Starlark Language\". Archived from the original on 15 June 2020. Retrieved 25 May 2019. \"Starlark Language\" Archived . Retrieved 25 May 2019 25 May ^ a b \"Why was Python created in the first place?\". General Python FAQ. Python Software Foundation. Archived from the original on 24 October 2012. Retrieved 22 March 2007. I had extensive experience with implementing an interpreted language in the ABC group at CWI, and from working with this group I had learned a lot about language design. This is the origin of many Python features, including the use of indentation for statement grouping and the inclusion of very high-level data types (although the details are all different in Python). ^ a b a b \"Why was Python created in the first place?\". General Python FAQ. Python Software Foundation. Archived from the original on 24 October 2012. Retrieved 22 March 2007. I had extensive experience with implementing an interpreted language in the ABC group at CWI, and from working with this group I had learned a lot about language design. This is the origin of many Python features, including the use of indentation for statement grouping and the inclusion of very high-level data types (although the details are all different in Python). \"Why was Python created in the first place?\" Archived . Retrieved 22 March 2007 22 March ^ \"Ada 83 Reference Manual (raise statement)\". Archived from the original on 22 October 2019. Retrieved 7 January 2020. ^ ^ \"Ada 83 Reference Manual (raise statement)\". Archived from the original on 22 October 2019. Retrieved 7 January 2020. \"Ada 83 Reference Manual (raise statement)\" Archived . Retrieved 7 January 2020 7 January ^ a b Kuchling, Andrew M. (22 December 2006). \"Interview with Guido van Rossum (July 1998)\". amk.ca. Archived from the original on 1 May 2007. Retrieved 12 March 2012. I'd spent a summer at DEC's Systems Research Center, which introduced me to Modula-2+; the Modula-3 final report was being written there at about the same time. What I learned there later showed up in Python's exception handling, modules, and the fact that methods explicitly contain 'self' in their parameter list. String slicing came from Algol-68 and Icon. ^ a b a b Kuchling, Andrew M. (22 December 2006). \"Interview with Guido van Rossum (July 1998)\". amk.ca. Archived from the original on 1 May 2007. Retrieved 12 March 2012. I'd spent a summer at DEC's Systems Research Center, which introduced me to Modula-2+; the Modula-3 final report was being written there at about the same time. What I learned there later showed up in Python's exception handling, modules, and the fact that methods explicitly contain 'self' in their parameter list. String slicing came from Algol-68 and Icon. \"Interview with Guido van Rossum (July 1998)\" the original . Retrieved 12 March 2012 12 March ^ a b c \"itertools \u2013 Functions creating iterators for efficient looping \u2013 Python 3.7.1 documentation\". docs.python.org. Archived from the original on 14 June 2020. Retrieved 22 November 2016. This module implements a number of iterator building blocks inspired by constructs from APL, Haskell, and SML. ^ a b c a b c \"itertools \u2013 Functions creating iterators for efficient looping \u2013 Python 3.7.1 documentation\". docs.python.org. Archived from the original on 14 June 2020. Retrieved 22 November 2016. This module implements a number of iterator building blocks inspired by constructs from APL, Haskell, and SML. \"itertools \u2013 Functions creating iterators for efficient looping \u2013 Python 3.7.1 documentation\" Archived . Retrieved 22 November 2016 22 November ^ van Rossum, Guido (1993). \"An Introduction to Python for UNIX/C Programmers\". Proceedings of the NLUUG Najaarsconferentie (Dutch UNIX Users Group). CiteSeerX 10.1.1.38.2023. even though the design of C is far from ideal, its influence on Python is considerable. ^ ^ van Rossum, Guido (1993). \"An Introduction to Python for UNIX/C Programmers\". Proceedings of the NLUUG Najaarsconferentie (Dutch UNIX Users Group). CiteSeerX 10.1.1.38.2023. even though the design of C is far from ideal, its influence on Python is considerable. CiteSeerX 10.1.1.38.2023 10.1.1.38.2023 ^ a b \"Classes\". The Python Tutorial. Python Software Foundation. Archived from the original on 23 October 2012. Retrieved 20 February 2012. It is a mixture of the class mechanisms found in C++ and Modula-3 ^ a b a b \"Classes\". The Python Tutorial. Python Software Foundation. Archived from the original on 23 October 2012. Retrieved 20 February 2012. It is a mixture of the class mechanisms found in C++ and Modula-3 \"Classes\" Archived . Retrieved 20 February 2012 20 February ^ Lundh, Fredrik. \"Call By Object\". effbot.org. Archived from the original on 23 November 2019. Retrieved 21 November 2017. replace \"CLU\" with \"Python\", \"record\" with \"instance\", and \"procedure\" with \"function or method\", and you get a pretty accurate description of Python's object model. ^ ^ Lundh, Fredrik. \"Call By Object\". effbot.org. Archived from the original on 23 November 2019. Retrieved 21 November 2017. replace \"CLU\" with \"Python\", \"record\" with \"instance\", and \"procedure\" with \"function or method\", and you get a pretty accurate description of Python's object model. \"Call By Object\" Archived . Retrieved 21 November 2017 21 November ^ Simionato, Michele. \"The Python 2.3 Method Resolution Order\". Python Software Foundation. Archived from the original on 20 August 2020. Retrieved 29 July 2014. The C3 method itself has nothing to do with Python, since it was invented by people working on Dylan and it is described in a paper intended for lispers ^ ^ Simionato, Michele. \"The Python 2.3 Method Resolution Order\". Python Software Foundation. Archived from the original on 20 August 2020. Retrieved 29 July 2014. The C3 method itself has nothing to do with Python, since it was invented by people working on Dylan and it is described in a paper intended for lispers \"The Python 2.3 Method Resolution Order\" Archived . Retrieved 29 July 2014 29 July ^ Kuchling, A. M. \"Functional Programming HOWTO\". Python v2.7.2 documentation. Python Software Foundation. Archived from the original on 24 October 2012. Retrieved 9 February 2012. List comprehensions and generator expressions [...] are a concise notation for such operations, borrowed from the functional programming language Haskell. ^ ^ Kuchling, A. M. \"Functional Programming HOWTO\". Python v2.7.2 documentation. Python Software Foundation. Archived from the original on 24 October 2012. Retrieved 9 February 2012. List comprehensions and generator expressions [...] are a concise notation for such operations, borrowed from the functional programming language Haskell. \"Functional Programming HOWTO\" Archived . Retrieved 9 February 2012 9 February ^ Schemenauer, Neil; Peters, Tim; Hetland, Magnus Lie (18 May 2001). \"PEP 255 \u2013 Simple Generators\". Python Enhancement Proposals. Python Software Foundation. Archived from the original on 5 June 2020. Retrieved 9 February 2012. ^ ^ Schemenauer, Neil; Peters, Tim; Hetland, Magnus Lie (18 May 2001). \"PEP 255 \u2013 Simple Generators\". Python Enhancement Proposals. Python Software Foundation. Archived from the original on 5 June 2020. Retrieved 9 February 2012. \"PEP 255 \u2013 Simple Generators\" Archived . Retrieved 9 February 2012 9 February ^ \"More Control Flow Tools\". Python 3 documentation. Python Software Foundation. Archived from the original on 4 June 2016. Retrieved 24 July 2015. By popular demand, a few features commonly found in functional programming languages like Lisp have been added to Python. With the lambda keyword, small anonymous functions can be created. ^ ^ \"More Control Flow Tools\". Python 3 documentation. Python Software Foundation. Archived from the original on 4 June 2016. Retrieved 24 July 2015. By popular demand, a few features commonly found in functional programming languages like Lisp have been added to Python. With the lambda keyword, small anonymous functions can be created. \"More Control Flow Tools\" Archived . Retrieved 24 July 2015 24 July ^ \"re \u2013 Regular expression operations \u2013 Python 3.10.6 documentation\". docs.python.org. Archived from the original on 18 July 2018. Retrieved 6 September 2022. This module provides regular expression matching operations similar to those found in Perl. ^ ^ \"re \u2013 Regular expression operations \u2013 Python 3.10.6 documentation\". docs.python.org. Archived from the original on 18 July 2018. Retrieved 6 September 2022. This module provides regular expression matching operations similar to those found in Perl. \"re \u2013 Regular expression operations \u2013 Python 3.10.6 documentation\" Archived . Retrieved 6 September 2022 6 September ^ \"CoffeeScript\". coffeescript.org. Archived from the original on 12 June 2020. Retrieved 3 July 2018. ^ ^ \"CoffeeScript\". coffeescript.org. Archived from the original on 12 June 2020. Retrieved 3 July 2018. \"CoffeeScript\" Archived . Retrieved 3 July 2018 3 July ^ \"Perl and Python influences in JavaScript\". www.2ality.com. 24 February 2013. Archived from the original on 26 December 2018. Retrieved 15 May 2015. ^ ^ \"Perl and Python influences in JavaScript\". www.2ality.com. 24 February 2013. Archived from the original on 26 December 2018. Retrieved 15 May 2015. \"Perl and Python influences in JavaScript\" Archived . Retrieved 15 May 2015 15 May ^ Rauschmayer, Axel. \"Chapter 3: The Nature of JavaScript; Influences\". O'Reilly, Speaking JavaScript. Archived from the original on 26 December 2018. Retrieved 15 May 2015. ^ ^ Rauschmayer, Axel. \"Chapter 3: The Nature of JavaScript; Influences\". O'Reilly, Speaking JavaScript. Archived from the original on 26 December 2018. Retrieved 15 May 2015. \"Chapter 3: The Nature of JavaScript; Influences\" Archived . Retrieved 15 May 2015 15 May ^ a b \"Why We Created Julia\". Julia website. February 2012. Archived from the original on 2 May 2020. Retrieved 5 June 2014. We want something as usable for general programming as Python [...] ^ a b a b \"Why We Created Julia\". Julia website. February 2012. Archived from the original on 2 May 2020. Retrieved 5 June 2014. We want something as usable for general programming as Python [...] \"Why We Created Julia\" Archived . Retrieved 5 June 2014 5 June ^ a b Krill, Paul (4 May 2023). \"Mojo language marries Python and MLIR for AI development\". InfoWorld. Archived from the original on 5 May 2023. Retrieved 5 May 2023. ^ a b a b Krill, Paul (4 May 2023). \"Mojo language marries Python and MLIR for AI development\". InfoWorld. Archived from the original on 5 May 2023. Retrieved 5 May 2023. \"Mojo language marries Python and MLIR for AI development\" Archived . Retrieved 5 May 2023 5 May ^ Ring Team (4 December 2017). \"Ring and other languages\". ring-lang.net. ring-lang. Archived from the original on 25 December 2018. Retrieved 4 December 2017. ^ ^ Ring Team (4 December 2017). \"Ring and other languages\". ring-lang.net. ring-lang. Archived from the original on 25 December 2018. Retrieved 4 December 2017. \"Ring and other languages\" ring-lang Archived . Retrieved 4 December 2017 4 December ^ Bini, Ola (2007). Practical JRuby on Rails Web 2.0 Projects: bringing Ruby on Rails to the Java platform. Berkeley: APress. p. 3. ISBN 978-1-59059-881-8. ^ ^ Bini, Ola (2007). Practical JRuby on Rails Web 2.0 Projects: bringing Ruby on Rails to the Java platform. Berkeley: APress. p. 3. ISBN 978-1-59059-881-8. Practical JRuby on Rails Web 2.0 Projects: bringing Ruby on Rails to the Java platform Practical JRuby on Rails Web 2.0 Projects: bringing Ruby on Rails to the Java platform 3 ISBN 978-1-59059-881-8 ^ Lattner, Chris (3 June 2014). \"Chris Lattner's Homepage\". Chris Lattner. Archived from the original on 25 December 2018. Retrieved 3 June 2014. The Swift language is the product of tireless effort from a team of language experts, documentation gurus, compiler optimization ninjas, and an incredibly important internal dogfooding group who provided feedback to help refine and battle-test ideas. Of course, it also greatly benefited from the experiences hard-won by many other languages in the field, drawing ideas from Objective-C, Rust, Haskell, Ruby, Python, C#, CLU, and far too many others to list. ^ ^ Lattner, Chris (3 June 2014). \"Chris Lattner's Homepage\". Chris Lattner. Archived from the original on 25 December 2018. Retrieved 3 June 2014. The Swift language is the product of tireless effort from a team of language experts, documentation gurus, compiler optimization ninjas, and an incredibly important internal dogfooding group who provided feedback to help refine and battle-test ideas. Of course, it also greatly benefited from the experiences hard-won by many other languages in the field, drawing ideas from Objective-C, Rust, Haskell, Ruby, Python, C#, CLU, and far too many others to list. \"Chris Lattner's Homepage\" Archived . Retrieved 3 June 2014 3 June ^ \"V documentation (Introduction)\". GitHub. Retrieved 24 December 2024. ^ ^ \"V documentation (Introduction)\". GitHub. Retrieved 24 December 2024. \"V documentation (Introduction)\" . Retrieved 24 December 2024 24 December ^ Kuhlman, Dave. \"A Python Book: Beginning Python, Advanced Python, and Python Exercises\". Section 1.1. Archived from the original (PDF) on 23 June 2012. ^ ^ Kuhlman, Dave. \"A Python Book: Beginning Python, Advanced Python, and Python Exercises\". Section 1.1. Archived from the original (PDF) on 23 June 2012. \"A Python Book: Beginning Python, Advanced Python, and Python Exercises\" the original (PDF) ^ \"About Python\". Python Software Foundation. Archived from the original on 20 April 2012. Retrieved 24 April 2012., second section \"Fans of Python use the phrase \"batteries included\" to describe the standard library, which covers everything from asynchronous processing to zip files.\" ^ ^ \"About Python\". Python Software Foundation. Archived from the original on 20 April 2012. Retrieved 24 April 2012., second section \"Fans of Python use the phrase \"batteries included\" to describe the standard library, which covers everything from asynchronous processing to zip files.\" \"About Python\" Archived . Retrieved 24 April 2012 24 April ^ \"PEP 206 \u2013 Python Advanced Library\". Python.org. Archived from the original on 5 May 2021. Retrieved 11 October 2021. ^ ^ \"PEP 206 \u2013 Python Advanced Library\". Python.org. Archived from the original on 5 May 2021. Retrieved 11 October 2021. \"PEP 206 \u2013 Python Advanced Library\" Archived . Retrieved 11 October 2021 11 October ^ Rossum, Guido Van (20 January 2009). \"The History of Python: A Brief Timeline of Python\". The History of Python. Archived from the original on 5 June 2020. Retrieved 5 March 2021. ^ ^ Rossum, Guido Van (20 January 2009). \"The History of Python: A Brief Timeline of Python\". The History of Python. Archived from the original on 5 June 2020. Retrieved 5 March 2021. \"The History of Python: A Brief Timeline of Python\" Archived . Retrieved 5 March 2021 5 March ^ Peterson, Benjamin (20 April 2020). \"Python 2.7.18, the last release of Python 2\". Python Insider. Archived from the original on 26 April 2020. Retrieved 27 April 2020. ^ ^ Peterson, Benjamin (20 April 2020). \"Python 2.7.18, the last release of Python 2\". Python Insider. Archived from the original on 26 April 2020. Retrieved 27 April 2020. \"Python 2.7.18, the last release of Python 2\" Archived . Retrieved 27 April 2020 27 April ^ \"Stack Overflow Developer Survey 2022\". Stack Overflow. Archived from the original on 27 June 2022. Retrieved 12 August 2022. ^ ^ \"Stack Overflow Developer Survey 2022\". Stack Overflow. Archived from the original on 27 June 2022. Retrieved 12 August 2022. \"Stack Overflow Developer Survey 2022\" Archived . Retrieved 12 August 2022 12 August ^ \"The State of Developer Ecosystem in 2020 Infographic\". JetBrains: Developer Tools for Professionals and Teams. Archived from the original on 1 March 2021. Retrieved 5 March 2021. ^ ^ \"The State of Developer Ecosystem in 2020 Infographic\". JetBrains: Developer Tools for Professionals and Teams. Archived from the original on 1 March 2021. Retrieved 5 March 2021. \"The State of Developer Ecosystem in 2020 Infographic\" Archived . Retrieved 5 March 2021 5 March ^ a b \"TIOBE Index\". TIOBE. Archived from the original on 25 February 2018. Retrieved 3 January 2023. The TIOBE Programming Community index is an indicator of the popularity of programming languages Updated as required. ^ a b a b \"TIOBE Index\". TIOBE. Archived from the original on 25 February 2018. Retrieved 3 January 2023. The TIOBE Programming Community index is an indicator of the popularity of programming languages Updated as required. \"TIOBE Index\" Archived . Retrieved 3 January 2023 3 January ^ \"PYPL PopularitY of Programming Language index\". pypl.github.io. Archived from the original on 14 March 2017. Retrieved 26 March 2021. ^ ^ \"PYPL PopularitY of Programming Language index\". pypl.github.io. Archived from the original on 14 March 2017. Retrieved 26 March 2021. \"PYPL PopularitY of Programming Language index\" Archived . Retrieved 26 March 2021 26 March ^ a b Venners, Bill (13 January 2003). \"The Making of Python\". Artima Developer. Artima. Archived from the original on 1 September 2016. Retrieved 22 March 2007. ^ a b a b Venners, Bill (13 January 2003). \"The Making of Python\". Artima Developer. Artima. Archived from the original on 1 September 2016. Retrieved 22 March 2007. \"The Making of Python\" Archived . Retrieved 22 March 2007 22 March ^ van Rossum, Guido (29 August 2000). \"SETL (was: Lukewarm about range literals)\". Python-Dev (Mailing list). Archived from the original on 14 July 2018. Retrieved 13 March 2011. ^ ^ van Rossum, Guido (29 August 2000). \"SETL (was: Lukewarm about range literals)\". Python-Dev (Mailing list). Archived from the original on 14 July 2018. Retrieved 13 March 2011. van Rossum, Guido \"SETL (was: Lukewarm about range literals)\" Archived . Retrieved 13 March 2011 13 March ^ van Rossum, Guido (20 January 2009). \"A Brief Timeline of Python\". The History of Python. Archived from the original on 5 June 2020. Retrieved 20 January 2009. ^ ^ van Rossum, Guido (20 January 2009). \"A Brief Timeline of Python\". The History of Python. Archived from the original on 5 June 2020. Retrieved 20 January 2009. \"A Brief Timeline of Python\" Archived . Retrieved 20 January 2009 20 January ^ Fairchild, Carlie (12 July 2018). \"Guido van Rossum Stepping Down from Role as Python's Benevolent Dictator For Life\". Linux Journal. Archived from the original on 13 July 2018. Retrieved 13 July 2018. ^ ^ Fairchild, Carlie (12 July 2018). \"Guido van Rossum Stepping Down from Role as Python's Benevolent Dictator For Life\". Linux Journal. Archived from the original on 13 July 2018. Retrieved 13 July 2018. \"Guido van Rossum Stepping Down from Role as Python's Benevolent Dictator For Life\" Archived . Retrieved 13 July 2018 13 July ^ \"PEP 8100\". Python Software Foundation. Archived from the original on 4 June 2020. Retrieved 4 May 2019. ^ ^ \"PEP 8100\". Python Software Foundation. Archived from the original on 4 June 2020. Retrieved 4 May 2019. \"PEP 8100\" Archived . Retrieved 4 May 2019 4 May ^ \"PEP 13 \u2013 Python Language Governance\". Python.org. Archived from the original on 27 May 2021. Retrieved 25 August 2021. ^ ^ \"PEP 13 \u2013 Python Language Governance\". Python.org. Archived from the original on 27 May 2021. Retrieved 25 August 2021. \"PEP 13 \u2013 Python Language Governance\" Archived . Retrieved 25 August 2021 25 August ^ Briggs, Jason R.; Lipova\u010da, Miran (2013). Python for kids: a playful introduction to programming. San Francisco, Calif: No Starch Press. ISBN 978-1-59327-407-8. ^ ^ Briggs, Jason R.; Lipova\u010da, Miran (2013). Python for kids: a playful introduction to programming. San Francisco, Calif: No Starch Press. ISBN 978-1-59327-407-8. ISBN 978-1-59327-407-8 ^ Kuchling, A. M.; Zadka, Moshe (16 October 2000). \"What's New in Python 2.0\". Python Software Foundation. Archived from the original on 23 October 2012. Retrieved 11 February 2012. ^ ^ Kuchling, A. M.; Zadka, Moshe (16 October 2000). \"What's New in Python 2.0\". Python Software Foundation. Archived from the original on 23 October 2012. Retrieved 11 February 2012. \"What's New in Python 2.0\" Archived . Retrieved 11 February 2012 11 February ^ \"PEP 373 \u2013 Python 2.7 Release Schedule\". python.org. Archived from the original on 19 May 2020. Retrieved 9 January 2017. ^ ^ \"PEP 373 \u2013 Python 2.7 Release Schedule\". python.org. Archived from the original on 19 May 2020. Retrieved 9 January 2017. \"PEP 373 \u2013 Python 2.7 Release Schedule\" Archived . Retrieved 9 January 2017 9 January ^ \"PEP 466 \u2013 Network Security Enhancements for Python 2.7.x\". python.org. Archived from the original on 4 June 2020. Retrieved 9 January 2017. ^ ^ \"PEP 466 \u2013 Network Security Enhancements for Python 2.7.x\". python.org. Archived from the original on 4 June 2020. Retrieved 9 January 2017. \"PEP 466 \u2013 Network Security Enhancements for Python 2.7.x\" Archived . Retrieved 9 January 2017 9 January ^ \"Sunsetting Python 2\". Python.org. Archived from the original on 12 January 2020. Retrieved 22 September 2019. ^ ^ \"Sunsetting Python 2\". Python.org. Archived from the original on 12 January 2020. Retrieved 22 September 2019. \"Sunsetting Python 2\" Archived . Retrieved 22 September 2019 22 September ^ \"PEP 373 \u2013 Python 2.7 Release Schedule\". Python.org. Archived from the original on 13 January 2020. Retrieved 22 September 2019. ^ ^ \"PEP 373 \u2013 Python 2.7 Release Schedule\". Python.org. Archived from the original on 13 January 2020. Retrieved 22 September 2019. \"PEP 373 \u2013 Python 2.7 Release Schedule\" Archived . Retrieved 22 September 2019 22 September ^ mattip (25 December 2023). \"PyPy v7.3.14 release\". PyPy. Archived from the original on 5 January 2024. Retrieved 5 January 2024. ^ ^ mattip (25 December 2023). \"PyPy v7.3.14 release\". PyPy. Archived from the original on 5 January 2024. Retrieved 5 January 2024. \"PyPy v7.3.14 release\" Archived . Retrieved 5 January 2024 5 January ^ Langa, \u0141ukasz (17 May 2022). \"Python 3.9.13 is now available\". Python Insider. Archived from the original on 17 May 2022. Retrieved 21 May 2022. ^ ^ Langa, \u0141ukasz (17 May 2022). \"Python 3.9.13 is now available\". Python Insider. Archived from the original on 17 May 2022. Retrieved 21 May 2022. \"Python 3.9.13 is now available\" Archived . Retrieved 21 May 2022 21 May ^ \"Status of Python versions\". Python Developer's Guide. Retrieved 7 October 2024. ^ ^ \"Status of Python versions\". Python Developer's Guide. Retrieved 7 October 2024. \"Status of Python versions\" . Retrieved 7 October 2024 7 October ^ \"Python\". endoflife.date. 8 October 2024. Retrieved 20 November 2024. ^ ^ \"Python\". endoflife.date. 8 October 2024. Retrieved 20 November 2024. \"Python\" . Retrieved 20 November 2024 20 November ^ \"CVE-2021-3177\". Red Hat Customer Portal. Archived from the original on 6 March 2021. Retrieved 26 February 2021. ^ ^ \"CVE-2021-3177\". Red Hat Customer Portal. Archived from the original on 6 March 2021. Retrieved 26 February 2021. \"CVE-2021-3177\" Archived . Retrieved 26 February 2021 26 February ^ \"CVE-2021-3177\". CVE. Archived from the original on 27 February 2021. Retrieved 26 February 2021. ^ ^ \"CVE-2021-3177\". CVE. Archived from the original on 27 February 2021. Retrieved 26 February 2021. \"CVE-2021-3177\" Archived . Retrieved 26 February 2021 26 February ^ \"CVE-2021-23336\". CVE. Archived from the original on 24 February 2021. Retrieved 26 February 2021. ^ ^ \"CVE-2021-23336\". CVE. Archived from the original on 24 February 2021. Retrieved 26 February 2021. \"CVE-2021-23336\" Archived . Retrieved 26 February 2021 26 February ^ \"Built-in Types\". ^ ^ \"Built-in Types\". \"Built-in Types\" ^ corbet (24 October 2022). \"Python 3.11 released [LWN.net]\". lwn.net. Retrieved 15 November 2022. ^ ^ corbet (24 October 2022). \"Python 3.11 released [LWN.net]\". lwn.net. Retrieved 15 November 2022. \"Python 3.11 released [LWN.net]\" . Retrieved 15 November 2022 15 November ^ \"What's New In Python 3.13\". Python documentation. Retrieved 30 April 2024. ^ ^ \"What's New In Python 3.13\". Python documentation. Retrieved 30 April 2024. \"What's New In Python 3.13\" . Retrieved 30 April 2024 30 April ^ \"PEP 667 \u2013 Consistent views of namespaces | peps.python.org\". Python Enhancement Proposals (PEPs). Retrieved 7 October 2024. ^ ^ \"PEP 667 \u2013 Consistent views of namespaces | peps.python.org\". Python Enhancement Proposals (PEPs). Retrieved 7 October 2024. \"PEP 667 \u2013 Consistent views of namespaces | peps.python.org\" . Retrieved 7 October 2024 7 October ^ Wouters, Thomas (9 April 2024). \"Python Insider: Python 3.12.3 and 3.13.0a6 released\". Python Insider. Retrieved 29 April 2024. ^ ^ Wouters, Thomas (9 April 2024). \"Python Insider: Python 3.12.3 and 3.13.0a6 released\". Python Insider. Retrieved 29 April 2024. \"Python Insider: Python 3.12.3 and 3.13.0a6 released\" . Retrieved 29 April 2024 29 April ^ \"PEP 594 \u2013 Removing dead batteries from the standard library\". Python Enhancement Proposals. Python Softtware Foundation. 20 May 2019. ^ ^ \"PEP 594 \u2013 Removing dead batteries from the standard library\". Python Enhancement Proposals. Python Softtware Foundation. 20 May 2019. \"PEP 594 \u2013 Removing dead batteries from the standard library\" ^ Hugo (15 October 2024). \"Python Insider: Python 3.14.0 alpha 1 is now available\". Python Insider. Retrieved 16 October 2024. ^ ^ Hugo (15 October 2024). \"Python Insider: Python 3.14.0 alpha 1 is now available\". Python Insider. Retrieved 16 October 2024. \"Python Insider: Python 3.14.0 alpha 1 is now available\" . Retrieved 16 October 2024 16 October ^ \"PEP 749 \u2013 Implementing PEP 649 | peps.python.org\". Python Enhancement Proposals (PEPs). Retrieved 20 November 2024. ^ ^ \"PEP 749 \u2013 Implementing PEP 649 | peps.python.org\". Python Enhancement Proposals (PEPs). Retrieved 20 November 2024. \"PEP 749 \u2013 Implementing PEP 649 | peps.python.org\" . Retrieved 20 November 2024 20 November ^ \"PEP 711: PyBI: a standard format for distributing Python Binaries\". Discussions on Python.org. 7 April 2023. Retrieved 20 November 2024. ^ ^ \"PEP 711: PyBI: a standard format for distributing Python Binaries\". Discussions on Python.org. 7 April 2023. Retrieved 20 November 2024. \"PEP 711: PyBI: a standard format for distributing Python Binaries\" . Retrieved 20 November 2024 20 November ^ \"PEP 686 \u2013 Make UTF-8 mode default | peps.python.org\". Python Enhancement Proposals (PEPs). Retrieved 20 November 2024. ^ ^ \"PEP 686 \u2013 Make UTF-8 mode default | peps.python.org\". Python Enhancement Proposals (PEPs). Retrieved 20 November 2024. \"PEP 686 \u2013 Make UTF-8 mode default | peps.python.org\" . Retrieved 20 November 2024 20 November ^ The Cain Gang Ltd. \"Python Metaclasses: Who? Why? When?\" (PDF). Archived from the original (PDF) on 30 May 2009. Retrieved 27 June 2009. ^ ^ The Cain Gang Ltd. \"Python Metaclasses: Who? Why? When?\" (PDF). Archived from the original (PDF) on 30 May 2009. Retrieved 27 June 2009. \"Python Metaclasses: Who? Why? When?\" (PDF) the original (PDF) . Retrieved 27 June 2009 27 June ^ \"3.3. Special method names\". The Python Language Reference. Python Software Foundation. Archived from the original on 15 December 2018. Retrieved 27 June 2009. ^ ^ \"3.3. Special method names\". The Python Language Reference. Python Software Foundation. Archived from the original on 15 December 2018. Retrieved 27 June 2009. \"3.3. Special method names\" Archived . Retrieved 27 June 2009 27 June ^ \"PyDBC: method preconditions, method postconditions and class invariants for Python\". Archived from the original on 23 November 2019. Retrieved 24 September 2011. ^ ^ \"PyDBC: method preconditions, method postconditions and class invariants for Python\". Archived from the original on 23 November 2019. Retrieved 24 September 2011. \"PyDBC: method preconditions, method postconditions and class invariants for Python\" Archived . Retrieved 24 September 2011 24 September ^ \"Contracts for Python\". Archived from the original on 15 June 2020. Retrieved 24 September 2011. ^ ^ \"Contracts for Python\". Archived from the original on 15 June 2020. Retrieved 24 September 2011. \"Contracts for Python\" Archived . Retrieved 24 September 2011 24 September ^ \"PyDatalog\". Archived from the original on 13 June 2020. Retrieved 22 July 2012. ^ ^ \"PyDatalog\". Archived from the original on 13 June 2020. Retrieved 22 July 2012. \"PyDatalog\" Archived . Retrieved 22 July 2012 22 July ^ \"Glue It All Together With Python\". Python.org. Retrieved 30 September 2024. ^ ^ \"Glue It All Together With Python\". Python.org. Retrieved 30 September 2024. \"Glue It All Together With Python\" . Retrieved 30 September 2024 30 September ^ \"Extending and Embedding the Python Interpreter: Reference Counts\". Docs.python.org. Archived from the original on 18 October 2012. Retrieved 5 June 2020. Since Python makes heavy use of malloc() and free(), it needs a strategy to avoid memory leaks as well as the use of freed memory. The chosen method is called reference counting. ^ ^ \"Extending and Embedding the Python Interpreter: Reference Counts\". Docs.python.org. Archived from the original on 18 October 2012. Retrieved 5 June 2020. Since Python makes heavy use of malloc() and free(), it needs a strategy to avoid memory leaks as well as the use of freed memory. The chosen method is called reference counting. \"Extending and Embedding the Python Interpreter: Reference Counts\" Archived . Retrieved 5 June 2020 5 June ^ a b Hettinger, Raymond (30 January 2002). \"PEP 289 \u2013 Generator Expressions\". Python Enhancement Proposals. Python Software Foundation. Archived from the original on 14 June 2020. Retrieved 19 February 2012. ^ a b a b Hettinger, Raymond (30 January 2002). \"PEP 289 \u2013 Generator Expressions\". Python Enhancement Proposals. Python Software Foundation. Archived from the original on 14 June 2020. Retrieved 19 February 2012. \"PEP 289 \u2013 Generator Expressions\" Archived . Retrieved 19 February 2012 19 February ^ \"6.5 itertools \u2013 Functions creating iterators for efficient looping\". Docs.python.org. Archived from the original on 14 June 2020. Retrieved 22 November 2016. ^ ^ \"6.5 itertools \u2013 Functions creating iterators for efficient looping\". Docs.python.org. Archived from the original on 14 June 2020. Retrieved 22 November 2016. \"6.5 itertools \u2013 Functions creating iterators for efficient looping\" Archived . Retrieved 22 November 2016 22 November ^ a b Peters, Tim (19 August 2004). \"PEP 20 \u2013 The Zen of Python\". Python Enhancement Proposals. Python Software Foundation. Archived from the original on 26 December 2018. Retrieved 24 November 2008. ^ a b a b Peters, Tim (19 August 2004). \"PEP 20 \u2013 The Zen of Python\". Python Enhancement Proposals. Python Software Foundation. Archived from the original on 26 December 2018. Retrieved 24 November 2008. \"PEP 20 \u2013 The Zen of Python\" Archived . Retrieved 24 November 2008 24 November ^ Lutz, Mark (January 2022). \"Python Changes 2014+\". Learning Python. Archived from the original on 15 March 2024. Retrieved 25 February 2024. ^ ^ Lutz, Mark (January 2022). \"Python Changes 2014+\". Learning Python. Archived from the original on 15 March 2024. Retrieved 25 February 2024. \"Python Changes 2014+\" Archived . Retrieved 25 February 2024 25 February ^ \"Confusion regarding a rule in The Zen of Python\". Python Help - Discussions on Python.org. 3 May 2022. Archived from the original on 25 February 2024. Retrieved 25 February 2024. ^ ^ \"Confusion regarding a rule in The Zen of Python\". Python Help - Discussions on Python.org. 3 May 2022. Archived from the original on 25 February 2024. Retrieved 25 February 2024. \"Confusion regarding a rule in The Zen of Python\" Archived . Retrieved 25 February 2024 25 February ^ Ambi, Chetan (4 July 2021). \"The Most Controversial Python Walrus Operator\". Python Simplified. Archived from the original on 27 August 2023. Retrieved 5 February 2024. ^ ^ Ambi, Chetan (4 July 2021). \"The Most Controversial Python Walrus Operator\". Python Simplified. Archived from the original on 27 August 2023. Retrieved 5 February 2024. \"The Most Controversial Python Walrus Operator\" Archived . Retrieved 5 February 2024 5 February ^ Grifski, Jeremy (24 May 2020). \"The Controversy Behind The Walrus Operator in Python\". The Renegade Coder. Archived from the original on 28 December 2023. Retrieved 25 February 2024. ^ ^ Grifski, Jeremy (24 May 2020). \"The Controversy Behind The Walrus Operator in Python\". The Renegade Coder. Archived from the original on 28 December 2023. Retrieved 25 February 2024. \"The Controversy Behind The Walrus Operator in Python\" Archived . Retrieved 25 February 2024 25 February ^ Bader, Dan. \"Python String Formatting Best Practices\". Real Python. Archived from the original on 18 February 2024. Retrieved 25 February 2024. ^ ^ Bader, Dan. \"Python String Formatting Best Practices\". Real Python. Archived from the original on 18 February 2024. Retrieved 25 February 2024. \"Python String Formatting Best Practices\" Archived . Retrieved 25 February 2024 25 February ^ Martelli, Alex; Ravenscroft, Anna; Ascher, David (2005). Python Cookbook, 2nd Edition. O'Reilly Media. p. 230. ISBN 978-0-596-00797-3. Archived from the original on 23 February 2020. Retrieved 14 November 2015. ^ ^ Martelli, Alex; Ravenscroft, Anna; Ascher, David (2005). Python Cookbook, 2nd Edition. O'Reilly Media. p. 230. ISBN 978-0-596-00797-3. Archived from the original on 23 February 2020. Retrieved 14 November 2015. Python Cookbook, 2nd Edition O'Reilly Media ISBN 978-0-596-00797-3 Archived . Retrieved 14 November 2015 14 November ^ \"Python Culture\". ebeab. 21 January 2014. Archived from the original on 30 January 2014. ^ ^ \"Python Culture\". ebeab. 21 January 2014. Archived from the original on 30 January 2014. \"Python Culture\" the original ^ a b \"Transpiling Python to Julia using PyJL\" (PDF). Archived (PDF) from the original on 19 November 2023. Retrieved 20 September 2023. After manually modifying one line of code by specifying the necessary type information, we obtained a speedup of 52.6\u00d7, making the translated Julia code 19.5\u00d7 faster than the original Python code. ^ a b a b \"Transpiling Python to Julia using PyJL\" (PDF). Archived (PDF) from the original on 19 November 2023. Retrieved 20 September 2023. After manually modifying one line of code by specifying the necessary type information, we obtained a speedup of 52.6\u00d7, making the translated Julia code 19.5\u00d7 faster than the original Python code. \"Transpiling Python to Julia using PyJL\" (PDF) Archived (PDF) . Retrieved 20 September 2023 20 September ^ \"Why is it called Python?\". General Python FAQ. Docs.python.org. Archived from the original on 24 October 2012. Retrieved 3 January 2023. ^ ^ \"Why is it called Python?\". General Python FAQ. Docs.python.org. Archived from the original on 24 October 2012. Retrieved 3 January 2023. \"Why is it called Python?\" Archived . Retrieved 3 January 2023 3 January ^ \"15 Ways Python Is a Powerful Force on the Web\". Archived from the original on 11 May 2019. Retrieved 3 July 2018. ^ ^ \"15 Ways Python Is a Powerful Force on the Web\". Archived from the original on 11 May 2019. Retrieved 3 July 2018. \"15 Ways Python Is a Powerful Force on the Web\" the original . Retrieved 3 July 2018 3 July ^ \"pprint \u2013 Data pretty printer \u2013 Python 3.11.0 documentation\". docs.python.org. Archived from the original on 22 January 2021. Retrieved 5 November 2022. stuff=['spam', 'eggs', 'lumberjack', 'knights', 'ni'] ^ ^ \"pprint \u2013 Data pretty printer \u2013 Python 3.11.0 documentation\". docs.python.org. Archived from the original on 22 January 2021. Retrieved 5 November 2022. stuff=['spam', 'eggs', 'lumberjack', 'knights', 'ni'] \"pprint \u2013 Data pretty printer \u2013 Python 3.11.0 documentation\" Archived . Retrieved 5 November 2022 5 November ^ \"Code Style \u2013 The Hitchhiker's Guide to Python\". docs.python-guide.org. Archived from the original on 27 January 2021. Retrieved 20 January 2021. ^ ^ \"Code Style \u2013 The Hitchhiker's Guide to Python\". docs.python-guide.org. Archived from the original on 27 January 2021. Retrieved 20 January 2021. \"Code Style \u2013 The Hitchhiker's Guide to Python\" Archived . Retrieved 20 January 2021 20 January ^ \"Is Python a good language for beginning programmers?\". General Python FAQ. Python Software Foundation. Archived from the original on 24 October 2012. Retrieved 21 March 2007. ^ ^ \"Is Python a good language for beginning programmers?\". General Python FAQ. Python Software Foundation. Archived from the original on 24 October 2012. Retrieved 21 March 2007. \"Is Python a good language for beginning programmers?\" Archived . Retrieved 21 March 2007 21 March ^ \"Myths about indentation in Python\". Secnetix.de. Archived from the original on 18 February 2018. Retrieved 19 April 2011. ^ ^ \"Myths about indentation in Python\". Secnetix.de. Archived from the original on 18 February 2018. Retrieved 19 April 2011. \"Myths about indentation in Python\" the original . Retrieved 19 April 2011 19 April ^ Guttag, John V. (12 August 2016). Introduction to Computation and Programming Using Python: With Application to Understanding Data. MIT Press. ISBN 978-0-262-52962-4. ^ ^ Guttag, John V. (12 August 2016). Introduction to Computation and Programming Using Python: With Application to Understanding Data. MIT Press. ISBN 978-0-262-52962-4. ISBN 978-0-262-52962-4 ^ \"PEP 8 \u2013 Style Guide for Python Code\". Python.org. Archived from the original on 17 April 2019. Retrieved 26 March 2019. ^ ^ \"PEP 8 \u2013 Style Guide for Python Code\". Python.org. Archived from the original on 17 April 2019. Retrieved 26 March 2019. \"PEP 8 \u2013 Style Guide for Python Code\" Archived . Retrieved 26 March 2019 26 March ^ \"8. Errors and Exceptions \u2013 Python 3.12.0a0 documentation\". docs.python.org. Archived from the original on 9 May 2022. Retrieved 9 May 2022. ^ ^ \"8. Errors and Exceptions \u2013 Python 3.12.0a0 documentation\". docs.python.org. Archived from the original on 9 May 2022. Retrieved 9 May 2022. \"8. Errors and Exceptions \u2013 Python 3.12.0a0 documentation\" Archived . Retrieved 9 May 2022 9 May ^ \"Highlights: Python 2.5\". Python.org. Archived from the original on 4 August 2019. Retrieved 20 March 2018. ^ ^ \"Highlights: Python 2.5\". Python.org. Archived from the original on 4 August 2019. Retrieved 20 March 2018. \"Highlights: Python 2.5\" Archived . Retrieved 20 March 2018 20 March ^ van Rossum, Guido (22 April 2009). \"Tail Recursion Elimination\". Neopythonic.blogspot.be. Archived from the original on 19 May 2018. Retrieved 3 December 2012. ^ ^ van Rossum, Guido (22 April 2009). \"Tail Recursion Elimination\". Neopythonic.blogspot.be. Archived from the original on 19 May 2018. Retrieved 3 December 2012. \"Tail Recursion Elimination\" Archived . Retrieved 3 December 2012 3 December ^ van Rossum, Guido (9 February 2006). \"Language Design Is Not Just Solving Puzzles\". Artima forums. Artima. Archived from the original on 17 January 2020. Retrieved 21 March 2007. ^ ^ van Rossum, Guido (9 February 2006). \"Language Design Is Not Just Solving Puzzles\". Artima forums. Artima. Archived from the original on 17 January 2020. Retrieved 21 March 2007. \"Language Design Is Not Just Solving Puzzles\" Archived . Retrieved 21 March 2007 21 March ^ van Rossum, Guido; Eby, Phillip J. (10 May 2005). \"PEP 342 \u2013 Coroutines via Enhanced Generators\". Python Enhancement Proposals. Python Software Foundation. Archived from the original on 29 May 2020. Retrieved 19 February 2012. ^ ^ van Rossum, Guido; Eby, Phillip J. (10 May 2005). \"PEP 342 \u2013 Coroutines via Enhanced Generators\". Python Enhancement Proposals. Python Software Foundation. Archived from the original on 29 May 2020. Retrieved 19 February 2012. \"PEP 342 \u2013 Coroutines via Enhanced Generators\" Archived . Retrieved 19 February 2012 19 February ^ \"PEP 380\". Python.org. Archived from the original on 4 June 2020. Retrieved 3 December 2012. ^ ^ \"PEP 380\". Python.org. Archived from the original on 4 June 2020. Retrieved 3 December 2012. \"PEP 380\" Archived . Retrieved 3 December 2012 3 December ^ \"division\". python.org. Archived from the original on 20 July 2006. Retrieved 30 July 2014. ^ ^ \"division\". python.org. Archived from the original on 20 July 2006. Retrieved 30 July 2014. \"division\" Archived . Retrieved 30 July 2014 30 July ^ \"PEP 0465 \u2013 A dedicated infix operator for matrix multiplication\". python.org. Archived from the original on 4 June 2020. Retrieved 1 January 2016. ^ ^ \"PEP 0465 \u2013 A dedicated infix operator for matrix multiplication\". python.org. Archived from the original on 4 June 2020. Retrieved 1 January 2016. \"PEP 0465 \u2013 A dedicated infix operator for matrix multiplication\" Archived . Retrieved 1 January 2016 1 January ^ \"Python 3.5.1 Release and Changelog\". python.org. Archived from the original on 14 May 2020. Retrieved 1 January 2016. ^ ^ \"Python 3.5.1 Release and Changelog\". python.org. Archived from the original on 14 May 2020. Retrieved 1 January 2016. \"Python 3.5.1 Release and Changelog\" Archived . Retrieved 1 January 2016 1 January ^ \"What's New in Python 3.8\". Archived from the original on 8 June 2020. Retrieved 14 October 2019. ^ ^ \"What's New in Python 3.8\". Archived from the original on 8 June 2020. Retrieved 14 October 2019. \"What's New in Python 3.8\" Archived . Retrieved 14 October 2019 14 October ^ van Rossum, Guido; Hettinger, Raymond (7 February 2003). \"PEP 308 \u2013 Conditional Expressions\". Python Enhancement Proposals. Python Software Foundation. Archived from the original on 13 March 2016. Retrieved 13 July 2011. ^ ^ van Rossum, Guido; Hettinger, Raymond (7 February 2003). \"PEP 308 \u2013 Conditional Expressions\". Python Enhancement Proposals. Python Software Foundation. Archived from the original on 13 March 2016. Retrieved 13 July 2011. \"PEP 308 \u2013 Conditional Expressions\" Archived . Retrieved 13 July 2011 13 July ^ \"4. Built-in Types \u2013 Python 3.6.3rc1 documentation\". python.org. Archived from the original on 14 June 2020. Retrieved 1 October 2017. ^ ^ \"4. Built-in Types \u2013 Python 3.6.3rc1 documentation\". python.org. Archived from the original on 14 June 2020. Retrieved 1 October 2017. \"4. Built-in Types \u2013 Python 3.6.3rc1 documentation\" Archived . Retrieved 1 October 2017 1 October ^ \"5.3. Tuples and Sequences \u2013 Python 3.7.1rc2 documentation\". python.org. Archived from the original on 10 June 2020. Retrieved 17 October 2018. ^ ^ \"5.3. Tuples and Sequences \u2013 Python 3.7.1rc2 documentation\". python.org. Archived from the original on 10 June 2020. Retrieved 17 October 2018. \"5.3. Tuples and Sequences \u2013 Python 3.7.1rc2 documentation\" Archived . Retrieved 17 October 2018 17 October ^ a b \"PEP 498 \u2013 Literal String Interpolation\". python.org. Archived from the original on 15 June 2020. Retrieved 8 March 2017. ^ a b a b \"PEP 498 \u2013 Literal String Interpolation\". python.org. Archived from the original on 15 June 2020. Retrieved 8 March 2017. \"PEP 498 \u2013 Literal String Interpolation\" Archived . Retrieved 8 March 2017 8 March ^ \"Why must 'self' be used explicitly in method definitions and calls?\". Design and History FAQ. Python Software Foundation. Archived from the original on 24 October 2012. Retrieved 19 February 2012. ^ ^ \"Why must 'self' be used explicitly in method definitions and calls?\". Design and History FAQ. Python Software Foundation. Archived from the original on 24 October 2012. Retrieved 19 February 2012. \"Why must 'self' be used explicitly in method definitions and calls?\" Archived . Retrieved 19 February 2012 19 February ^ Sweigart, Al (2020). Beyond the Basic Stuff with Python: Best Practices for Writing Clean Code. No Starch Press. p. 322. ISBN 978-1-59327-966-0. Archived from the original on 13 August 2021. Retrieved 7 July 2021. ^ ^ Sweigart, Al (2020). Beyond the Basic Stuff with Python: Best Practices for Writing Clean Code. No Starch Press. p. 322. ISBN 978-1-59327-966-0. Archived from the original on 13 August 2021. Retrieved 7 July 2021. Beyond the Basic Stuff with Python: Best Practices for Writing Clean Code ISBN 978-1-59327-966-0 Archived . Retrieved 7 July 2021 7 July ^ \"The Python Language Reference, section 3.3. New-style and classic classes, for release 2.7.1\". Archived from the original on 26 October 2012. Retrieved 12 January 2011. ^ ^ \"The Python Language Reference, section 3.3. New-style and classic classes, for release 2.7.1\". Archived from the original on 26 October 2012. Retrieved 12 January 2011. \"The Python Language Reference, section 3.3. New-style and classic classes, for release 2.7.1\" the original . Retrieved 12 January 2011 12 January ^ \"PEP 484 \u2013 Type Hints | peps.python.org\". peps.python.org. Archived from the original on 27 November 2023. Retrieved 29 November 2023. ^ ^ \"PEP 484 \u2013 Type Hints | peps.python.org\". peps.python.org. Archived from the original on 27 November 2023. Retrieved 29 November 2023. \"PEP 484 \u2013 Type Hints | peps.python.org\" Archived . Retrieved 29 November 2023 29 November ^ \"typing \u2014 Support for type hints\". Python documentation. Python Software Foundation. Archived from the original on 21 February 2020. Retrieved 22 December 2023. ^ ^ \"typing \u2014 Support for type hints\". Python documentation. Python Software Foundation. Archived from the original on 21 February 2020. Retrieved 22 December 2023. \"typing \u2014 Support for type hints\" Archived . Retrieved 22 December 2023 22 December ^ \"mypy \u2013 Optional Static Typing for Python\". Archived from the original on 6 June 2020. Retrieved 28 January 2017. ^ ^ \"mypy \u2013 Optional Static Typing for Python\". Archived from the original on 6 June 2020. Retrieved 28 January 2017. \"mypy \u2013 Optional Static Typing for Python\" Archived . Retrieved 28 January 2017 28 January ^ \"Introduction\". mypyc.readthedocs.io. Archived from the original on 22 December 2023. Retrieved 22 December 2023. ^ ^ \"Introduction\". mypyc.readthedocs.io. Archived from the original on 22 December 2023. Retrieved 22 December 2023. \"Introduction\" Archived . Retrieved 22 December 2023 22 December ^ \"15. Floating Point Arithmetic: Issues and Limitations \u2013 Python 3.8.3 documentation\". docs.python.org. Archived from the original on 6 June 2020. Retrieved 6 June 2020. Almost all machines today (November 2000) use IEEE-754 floating point arithmetic, and almost all platforms map Python floats to IEEE-754 \"double precision\". ^ ^ \"15. Floating Point Arithmetic: Issues and Limitations \u2013 Python 3.8.3 documentation\". docs.python.org. Archived from the original on 6 June 2020. Retrieved 6 June 2020. Almost all machines today (November 2000) use IEEE-754 floating point arithmetic, and almost all platforms map Python floats to IEEE-754 \"double precision\". \"15. Floating Point Arithmetic: Issues and Limitations \u2013 Python 3.8.3 documentation\" Archived . Retrieved 6 June 2020 6 June ^ Zadka, Moshe; van Rossum, Guido (11 March 2001). \"PEP 237 \u2013 Unifying Long Integers and Integers\". Python Enhancement Proposals. Python Software Foundation. Archived from the original on 28 May 2020. Retrieved 24 September 2011. ^ ^ Zadka, Moshe; van Rossum, Guido (11 March 2001). \"PEP 237 \u2013 Unifying Long Integers and Integers\". Python Enhancement Proposals. Python Software Foundation. Archived from the original on 28 May 2020. Retrieved 24 September 2011. \"PEP 237 \u2013 Unifying Long Integers and Integers\" Archived . Retrieved 24 September 2011 24 September ^ \"Built-in Types\". Archived from the original on 14 June 2020. Retrieved 3 October 2019. ^ ^ \"Built-in Types\". Archived from the original on 14 June 2020. Retrieved 3 October 2019. \"Built-in Types\" Archived . Retrieved 3 October 2019 3 October ^ \"PEP 465 \u2013 A dedicated infix operator for matrix multiplication\". python.org. Archived from the original on 29 May 2020. Retrieved 3 July 2018. ^ ^ \"PEP 465 \u2013 A dedicated infix operator for matrix multiplication\". python.org. Archived from the original on 29 May 2020. Retrieved 3 July 2018. \"PEP 465 \u2013 A dedicated infix operator for matrix multiplication\" Archived . Retrieved 3 July 2018 3 July ^ a b Zadka, Moshe; van Rossum, Guido (11 March 2001). \"PEP 238 \u2013 Changing the Division Operator\". Python Enhancement Proposals. Python Software Foundation. Archived from the original on 28 May 2020. Retrieved 23 October 2013. ^ a b a b Zadka, Moshe; van Rossum, Guido (11 March 2001). \"PEP 238 \u2013 Changing the Division Operator\". Python Enhancement Proposals. Python Software Foundation. Archived from the original on 28 May 2020. Retrieved 23 October 2013. \"PEP 238 \u2013 Changing the Division Operator\" Archived . Retrieved 23 October 2013 23 October ^ \"Why Python's Integer Division Floors\". 24 August 2010. Archived from the original on 5 June 2020. Retrieved 25 August 2010. ^ ^ \"Why Python's Integer Division Floors\". 24 August 2010. Archived from the original on 5 June 2020. Retrieved 25 August 2010. \"Why Python's Integer Division Floors\" Archived . Retrieved 25 August 2010 25 August ^ \"round\", The Python standard library, release 3.2, \u00a72: Built-in functions, archived from the original on 25 October 2012, retrieved 14 August 2011 ^ ^ \"round\", The Python standard library, release 3.2, \u00a72: Built-in functions, archived from the original on 25 October 2012, retrieved 14 August 2011 \"round\" archived , retrieved 14 August 2011 14 August ^ \"round\", The Python standard library, release 2.7, \u00a72: Built-in functions, archived from the original on 27 October 2012, retrieved 14 August 2011 ^ ^ \"round\", The Python standard library, release 2.7, \u00a72: Built-in functions, archived from the original on 27 October 2012, retrieved 14 August 2011 \"round\" archived , retrieved 14 August 2011 14 August ^ Beazley, David M. (2009). Python Essential Reference (4th ed.). Addison-Wesley Professional. p. 66. ISBN 9780672329784. ^ ^ Beazley, David M. (2009). Python Essential Reference (4th ed.). Addison-Wesley Professional. p. 66. ISBN 9780672329784. Python Essential Reference Python Essential Reference 66 ISBN 9780672329784 ^ Kernighan, Brian W.; Ritchie, Dennis M. (1988). The C Programming Language (2nd ed.). p. 206. ^ ^ Kernighan, Brian W.; Ritchie, Dennis M. (1988). The C Programming Language (2nd ed.). p. 206. The C Programming Language 206 ^ a b Batista, Facundo (17 October 2003). \"PEP 327 \u2013 Decimal Data Type\". Python Enhancement Proposals. Python Software Foundation. Archived from the original on 4 June 2020. Retrieved 24 November 2008. ^ a b a b Batista, Facundo (17 October 2003). \"PEP 327 \u2013 Decimal Data Type\". Python Enhancement Proposals. Python Software Foundation. Archived from the original on 4 June 2020. Retrieved 24 November 2008. \"PEP 327 \u2013 Decimal Data Type\" Archived . Retrieved 24 November 2008 24 November ^ \"What's New in Python 2.6\". Python v2.6.9 documentation. 29 October 2013. Archived from the original on 23 December 2019. Retrieved 26 September 2015. ^ ^ \"What's New in Python 2.6\". Python v2.6.9 documentation. 29 October 2013. Archived from the original on 23 December 2019. Retrieved 26 September 2015. \"What's New in Python 2.6\" Archived . Retrieved 26 September 2015 26 September ^ \"10 Reasons Python Rocks for Research (And a Few Reasons it Doesn't) \u2013 Hoyt Koepke\". University of Washington Department of Statistics. Archived from the original on 31 May 2020. Retrieved 3 February 2019. ^ ^ \"10 Reasons Python Rocks for Research (And a Few Reasons it Doesn't) \u2013 Hoyt Koepke\". University of Washington Department of Statistics. Archived from the original on 31 May 2020. Retrieved 3 February 2019. \"10 Reasons Python Rocks for Research (And a Few Reasons it Doesn't) \u2013 Hoyt Koepke\" the original . Retrieved 3 February 2019 3 February ^ Shell, Scott (17 June 2014). \"An introduction to Python for scientific computing\" (PDF). Archived (PDF) from the original on 4 February 2019. Retrieved 3 February 2019. ^ ^ Shell, Scott (17 June 2014). \"An introduction to Python for scientific computing\" (PDF). Archived (PDF) from the original on 4 February 2019. Retrieved 3 February 2019. \"An introduction to Python for scientific computing\" (PDF) Archived (PDF) . Retrieved 3 February 2019 3 February ^ Piotrowski, Przemyslaw (July 2006). \"Build a Rapid Web Development Environment for Python Server Pages and Oracle\". Oracle Technology Network. Oracle. Archived from the original on 2 April 2019. Retrieved 12 March 2012. ^ ^ Piotrowski, Przemyslaw (July 2006). \"Build a Rapid Web Development Environment for Python Server Pages and Oracle\". Oracle Technology Network. Oracle. Archived from the original on 2 April 2019. Retrieved 12 March 2012. \"Build a Rapid Web Development Environment for Python Server Pages and Oracle\" Archived . Retrieved 12 March 2012 12 March ^ Eby, Phillip J. (7 December 2003). \"PEP 333 \u2013 Python Web Server Gateway Interface v1.0\". Python Enhancement Proposals. Python Software Foundation. Archived from the original on 14 June 2020. Retrieved 19 February 2012. ^ ^ Eby, Phillip J. (7 December 2003). \"PEP 333 \u2013 Python Web Server Gateway Interface v1.0\". Python Enhancement Proposals. Python Software Foundation. Archived from the original on 14 June 2020. Retrieved 19 February 2012. \"PEP 333 \u2013 Python Web Server Gateway Interface v1.0\" Archived . Retrieved 19 February 2012 19 February ^ \"PyPI\". PyPI. 17 March 2024. Archived from the original on 17 March 2024. ^ ^ \"PyPI\". PyPI. 17 March 2024. Archived from the original on 17 March 2024. \"PyPI\" Archived ^ Enthought, Canopy. \"Canopy\". www.enthought.com. Archived from the original on 15 July 2017. Retrieved 20 August 2016. ^ ^ Enthought, Canopy. \"Canopy\". www.enthought.com. Archived from the original on 15 July 2017. Retrieved 20 August 2016. \"Canopy\" the original . Retrieved 20 August 2016 20 August ^ \"PEP 7 \u2013 Style Guide for C Code | peps.python.org\". peps.python.org. Archived from the original on 24 April 2022. Retrieved 28 April 2022. ^ ^ \"PEP 7 \u2013 Style Guide for C Code | peps.python.org\". peps.python.org. Archived from the original on 24 April 2022. Retrieved 28 April 2022. \"PEP 7 \u2013 Style Guide for C Code | peps.python.org\" Archived . Retrieved 28 April 2022 28 April ^ \"4. Building C and C++ Extensions \u2013 Python 3.9.2 documentation\". docs.python.org. Archived from the original on 3 March 2021. Retrieved 1 March 2021. ^ ^ \"4. Building C and C++ Extensions \u2013 Python 3.9.2 documentation\". docs.python.org. Archived from the original on 3 March 2021. Retrieved 1 March 2021. \"4. Building C and C++ Extensions \u2013 Python 3.9.2 documentation\" Archived . Retrieved 1 March 2021 1 March ^ van Rossum, Guido (5 June 2001). \"PEP 7 \u2013 Style Guide for C Code\". Python Enhancement Proposals. Python Software Foundation. Archived from the original on 1 June 2020. Retrieved 24 November 2008. ^ ^ van Rossum, Guido (5 June 2001). \"PEP 7 \u2013 Style Guide for C Code\". Python Enhancement Proposals. Python Software Foundation. Archived from the original on 1 June 2020. Retrieved 24 November 2008. \"PEP 7 \u2013 Style Guide for C Code\" Archived . Retrieved 24 November 2008 24 November ^ \"CPython byte code\". Docs.python.org. Archived from the original on 5 June 2020. Retrieved 16 February 2016. ^ ^ \"CPython byte code\". Docs.python.org. Archived from the original on 5 June 2020. Retrieved 16 February 2016. \"CPython byte code\" Archived . Retrieved 16 February 2016 16 February ^ \"Python 2.5 internals\" (PDF). Archived (PDF) from the original on 6 August 2012. Retrieved 19 April 2011. ^ ^ \"Python 2.5 internals\" (PDF). Archived (PDF) from the original on 6 August 2012. Retrieved 19 April 2011. \"Python 2.5 internals\" (PDF) Archived (PDF) . Retrieved 19 April 2011 19 April ^ \"Changelog \u2013 Python 3.9.0 documentation\". docs.python.org. Archived from the original on 7 February 2021. Retrieved 8 February 2021. ^ ^ \"Changelog \u2013 Python 3.9.0 documentation\". docs.python.org. Archived from the original on 7 February 2021. Retrieved 8 February 2021. \"Changelog \u2013 Python 3.9.0 documentation\" Archived . Retrieved 8 February 2021 8 February ^ \"Download Python\". Python.org. Archived from the original on 8 December 2020. Retrieved 13 December 2020. ^ ^ \"Download Python\". Python.org. Archived from the original on 8 December 2020. Retrieved 13 December 2020. \"Download Python\" Archived . Retrieved 13 December 2020 13 December ^ \"history [vmspython]\". www.vmspython.org. Archived from the original on 2 December 2020. Retrieved 4 December 2020. ^ ^ \"history [vmspython]\". www.vmspython.org. Archived from the original on 2 December 2020. Retrieved 4 December 2020. \"history [vmspython]\" Archived . Retrieved 4 December 2020 4 December ^ \"An Interview with Guido van Rossum\". Oreilly.com. Archived from the original on 16 July 2014. Retrieved 24 November 2008. ^ ^ \"An Interview with Guido van Rossum\". Oreilly.com. Archived from the original on 16 July 2014. Retrieved 24 November 2008. \"An Interview with Guido van Rossum\" Archived . Retrieved 24 November 2008 24 November ^ \"Download Python for Other Platforms\". Python.org. Archived from the original on 27 November 2020. Retrieved 4 December 2020. ^ ^ \"Download Python for Other Platforms\". Python.org. Archived from the original on 27 November 2020. Retrieved 4 December 2020. \"Download Python for Other Platforms\" Archived . Retrieved 4 December 2020 4 December ^ \"PyPy compatibility\". Pypy.org. Archived from the original on 6 June 2020. Retrieved 3 December 2012. ^ ^ \"PyPy compatibility\". Pypy.org. Archived from the original on 6 June 2020. Retrieved 3 December 2012. \"PyPy compatibility\" Archived . Retrieved 3 December 2012 3 December ^ Team, The PyPy (28 December 2019). \"Download and Install\". PyPy. Archived from the original on 8 January 2022. Retrieved 8 January 2022. ^ ^ Team, The PyPy (28 December 2019). \"Download and Install\". PyPy. Archived from the original on 8 January 2022. Retrieved 8 January 2022. \"Download and Install\" Archived . Retrieved 8 January 2022 8 January ^ \"speed comparison between CPython and Pypy\". Speed.pypy.org. Archived from the original on 10 May 2021. Retrieved 3 December 2012. ^ ^ \"speed comparison between CPython and Pypy\". Speed.pypy.org. Archived from the original on 10 May 2021. Retrieved 3 December 2012. \"speed comparison between CPython and Pypy\" Archived . Retrieved 3 December 2012 3 December ^ \"Codon: Differences with Python\". Archived from the original on 25 May 2023. Retrieved 28 August 2023. ^ ^ \"Codon: Differences with Python\". Archived from the original on 25 May 2023. Retrieved 28 August 2023. \"Codon: Differences with Python\" Archived . Retrieved 28 August 2023 28 August ^ Lawson, Loraine (14 March 2023). \"MIT-Created Compiler Speeds up Python Code\". The New Stack. Archived from the original on 6 April 2023. Retrieved 28 August 2023. ^ ^ Lawson, Loraine (14 March 2023). \"MIT-Created Compiler Speeds up Python Code\". The New Stack. Archived from the original on 6 April 2023. Retrieved 28 August 2023. \"MIT-Created Compiler Speeds up Python Code\" Archived . Retrieved 28 August 2023 28 August ^ \"Application-level Stackless features \u2013 PyPy 2.0.2 documentation\". Doc.pypy.org. Archived from the original on 4 June 2020. Retrieved 17 July 2013. ^ ^ \"Application-level Stackless features \u2013 PyPy 2.0.2 documentation\". Doc.pypy.org. Archived from the original on 4 June 2020. Retrieved 17 July 2013. \"Application-level Stackless features \u2013 PyPy 2.0.2 documentation\" Archived . Retrieved 17 July 2013 17 July ^ \"Python-for-EV3\". LEGO Education. Archived from the original on 7 June 2020. Retrieved 17 April 2019. ^ ^ \"Python-for-EV3\". LEGO Education. Archived from the original on 7 June 2020. Retrieved 17 April 2019. \"Python-for-EV3\" Archived . Retrieved 17 April 2019 17 April ^ Yegulalp, Serdar (29 October 2020). \"Pyston returns from the dead to speed Python\". InfoWorld. Archived from the original on 27 January 2021. Retrieved 26 January 2021. ^ ^ Yegulalp, Serdar (29 October 2020). \"Pyston returns from the dead to speed Python\". InfoWorld. Archived from the original on 27 January 2021. Retrieved 26 January 2021. \"Pyston returns from the dead to speed Python\" InfoWorld Archived . Retrieved 26 January 2021 26 January ^ \"cinder: Instagram's performance-oriented fork of CPython\". GitHub. Archived from the original on 4 May 2021. Retrieved 4 May 2021. ^ ^ \"cinder: Instagram's performance-oriented fork of CPython\". GitHub. Archived from the original on 4 May 2021. Retrieved 4 May 2021. \"cinder: Instagram's performance-oriented fork of CPython\" GitHub Archived . Retrieved 4 May 2021 4 May ^ Aroca, Rafael (7 August 2021). \"Snek Lang: feels like Python on Arduinos\". Yet Another Technology Blog. Archived from the original on 5 January 2024. Retrieved 4 January 2024. ^ ^ Aroca, Rafael (7 August 2021). \"Snek Lang: feels like Python on Arduinos\". Yet Another Technology Blog. Archived from the original on 5 January 2024. Retrieved 4 January 2024. \"Snek Lang: feels like Python on Arduinos\" Archived . Retrieved 4 January 2024 4 January ^ Aufranc (CNXSoft), Jean-Luc (16 January 2020). \"Snekboard Controls LEGO Power Functions with CircuitPython or Snek Programming Languages (Crowdfunding) \u2013 CNX Software\". CNX Software \u2013 Embedded Systems News. Archived from the original on 5 January 2024. Retrieved 4 January 2024. ^ ^ Aufranc (CNXSoft), Jean-Luc (16 January 2020). \"Snekboard Controls LEGO Power Functions with CircuitPython or Snek Programming Languages (Crowdfunding) \u2013 CNX Software\". CNX Software \u2013 Embedded Systems News. Archived from the original on 5 January 2024. Retrieved 4 January 2024. \"Snekboard Controls LEGO Power Functions with CircuitPython or Snek Programming Languages (Crowdfunding) \u2013 CNX Software\" Archived . Retrieved 4 January 2024 4 January ^ Kennedy (@mkennedy), Michael. \"Ready to find out if you're git famous?\". pythonbytes.fm. Archived from the original on 5 January 2024. Retrieved 4 January 2024. ^ ^ Kennedy (@mkennedy), Michael. \"Ready to find out if you're git famous?\". pythonbytes.fm. Archived from the original on 5 January 2024. Retrieved 4 January 2024. \"Ready to find out if you're git famous?\" Archived . Retrieved 4 January 2024 4 January ^ Packard, Keith (20 December 2022). \"The Snek Programming Language: A Python-inspired Embedded Computing Language\" (PDF). Archived (PDF) from the original on 4 January 2024. Retrieved 4 January 2024. ^ ^ Packard, Keith (20 December 2022). \"The Snek Programming Language: A Python-inspired Embedded Computing Language\" (PDF). Archived (PDF) from the original on 4 January 2024. Retrieved 4 January 2024. \"The Snek Programming Language: A Python-inspired Embedded Computing Language\" (PDF) Archived (PDF) . Retrieved 4 January 2024 4 January ^ \"Plans for optimizing Python\". Google Project Hosting. 15 December 2009. Archived from the original on 11 April 2016. Retrieved 24 September 2011. ^ ^ \"Plans for optimizing Python\". Google Project Hosting. 15 December 2009. Archived from the original on 11 April 2016. Retrieved 24 September 2011. \"Plans for optimizing Python\" Archived . Retrieved 24 September 2011 24 September ^ \"Python on the Nokia N900\". Stochastic Geometry. 29 April 2010. Archived from the original on 20 June 2019. Retrieved 9 July 2015. ^ ^ \"Python on the Nokia N900\". Stochastic Geometry. 29 April 2010. Archived from the original on 20 June 2019. Retrieved 9 July 2015. \"Python on the Nokia N900\" Archived . Retrieved 9 July 2015 9 July ^ \"Brython\". brython.info. Archived from the original on 3 August 2018. Retrieved 21 January 2021. ^ ^ \"Brython\". brython.info. Archived from the original on 3 August 2018. Retrieved 21 January 2021. \"Brython\" Archived . Retrieved 21 January 2021 21 January ^ \"Transcrypt \u2013 Python in the browser\". transcrypt.org. Archived from the original on 19 August 2018. Retrieved 22 December 2020. ^ ^ \"Transcrypt \u2013 Python in the browser\". transcrypt.org. Archived from the original on 19 August 2018. Retrieved 22 December 2020. \"Transcrypt \u2013 Python in the browser\" Archived . Retrieved 22 December 2020 22 December ^ \"Transcrypt: Anatomy of a Python to JavaScript Compiler\". InfoQ. Archived from the original on 5 December 2020. Retrieved 20 January 2021. ^ ^ \"Transcrypt: Anatomy of a Python to JavaScript Compiler\". InfoQ. Archived from the original on 5 December 2020. Retrieved 20 January 2021. \"Transcrypt: Anatomy of a Python to JavaScript Compiler\" Archived . Retrieved 20 January 2021 20 January ^ \"Nuitka Home | Nuitka Home\". nuitka.net. Archived from the original on 30 May 2020. Retrieved 18 August 2017. ^ ^ \"Nuitka Home | Nuitka Home\". nuitka.net. Archived from the original on 30 May 2020. Retrieved 18 August 2017. \"Nuitka Home | Nuitka Home\" Archived . Retrieved 18 August 2017 18 August ^ Guelton, Serge; Brunet, Pierrick; Amini, Mehdi; Merlini, Adrien; Corbillon, Xavier; Raynaud, Alan (16 March 2015). \"Pythran: enabling static optimization of scientific Python programs\". Computational Science & Discovery. 8 (1). IOP Publishing: 014001. Bibcode:2015CS&D....8a4001G. doi:10.1088/1749-4680/8/1/014001. ISSN 1749-4699. ^ ^ Guelton, Serge; Brunet, Pierrick; Amini, Mehdi; Merlini, Adrien; Corbillon, Xavier; Raynaud, Alan (16 March 2015). \"Pythran: enabling static optimization of scientific Python programs\". Computational Science & Discovery. 8 (1). IOP Publishing: 014001. Bibcode:2015CS&D....8a4001G. doi:10.1088/1749-4680/8/1/014001. ISSN 1749-4699. \"Pythran: enabling static optimization of scientific Python programs\" Bibcode 2015CS&D....8a4001G doi 10.1088/1749-4680/8/1/014001 10.1088/1749-4680/8/1/014001 ISSN 1749-4699 ^ \"The Python \u2192 11l \u2192 C++ transpiler\". Archived from the original on 24 September 2022. Retrieved 17 July 2022. ^ ^ \"The Python \u2192 11l \u2192 C++ transpiler\". Archived from the original on 24 September 2022. Retrieved 17 July 2022. \"The Python \u2192 11l \u2192 C++ transpiler\" Archived . Retrieved 17 July 2022 17 July ^ \"google/grumpy\". 10 April 2020. Archived from the original on 15 April 2020. Retrieved 25 March 2020 \u2013 via GitHub. ^ ^ \"google/grumpy\". 10 April 2020. Archived from the original on 15 April 2020. Retrieved 25 March 2020 \u2013 via GitHub. \"google/grumpy\" Archived . Retrieved 25 March 2020 25 March ^ \"Projects\". opensource.google. Archived from the original on 24 April 2020. Retrieved 25 March 2020. ^ ^ \"Projects\". opensource.google. Archived from the original on 24 April 2020. Retrieved 25 March 2020. \"Projects\" Archived . Retrieved 25 March 2020 25 March ^ Francisco, Thomas Claburn in San. \"Google's Grumpy code makes Python Go\". www.theregister.com. Archived from the original on 7 March 2021. Retrieved 20 January 2021. ^ ^ Francisco, Thomas Claburn in San. \"Google's Grumpy code makes Python Go\". www.theregister.com. Archived from the original on 7 March 2021. Retrieved 20 January 2021. \"Google's Grumpy code makes Python Go\" Archived . Retrieved 20 January 2021 20 January ^ \"GitHub \u2013 IronLanguages/ironpython3: Implementation of Python 3.x for .NET Framework that is built on top of the Dynamic Language Runtime\". GitHub. Archived from the original on 28 September 2021. ^ ^ \"GitHub \u2013 IronLanguages/ironpython3: Implementation of Python 3.x for .NET Framework that is built on top of the Dynamic Language Runtime\". GitHub. Archived from the original on 28 September 2021. \"GitHub \u2013 IronLanguages/ironpython3: Implementation of Python 3.x for .NET Framework that is built on top of the Dynamic Language Runtime\" GitHub Archived ^ \"IronPython.net /\". ironpython.net. Archived from the original on 17 April 2021. ^ ^ \"IronPython.net /\". ironpython.net. Archived from the original on 17 April 2021. \"IronPython.net /\" Archived ^ \"Jython FAQ\". www.jython.org. Archived from the original on 22 April 2021. Retrieved 22 April 2021. ^ ^ \"Jython FAQ\". www.jython.org. Archived from the original on 22 April 2021. Retrieved 22 April 2021. \"Jython FAQ\" Archived . Retrieved 22 April 2021 22 April ^ Murri, Riccardo (2013). Performance of Python runtimes on a non-numeric scientific code. European Conference on Python in Science (EuroSciPy). arXiv:1404.6388. Bibcode:2014arXiv1404.6388M. ^ ^ Murri, Riccardo (2013). Performance of Python runtimes on a non-numeric scientific code. European Conference on Python in Science (EuroSciPy). arXiv:1404.6388. Bibcode:2014arXiv1404.6388M. arXiv 1404.6388 1404.6388 Bibcode 2014arXiv1404.6388M ^ \"The Computer Language Benchmarks Game\". Archived from the original on 14 June 2020. Retrieved 30 April 2020. ^ ^ \"The Computer Language Benchmarks Game\". Archived from the original on 14 June 2020. Retrieved 30 April 2020. \"The Computer Language Benchmarks Game\" Archived . Retrieved 30 April 2020 30 April ^ a b Warsaw, Barry; Hylton, Jeremy; Goodger, David (13 June 2000). \"PEP 1 \u2013 PEP Purpose and Guidelines\". Python Enhancement Proposals. Python Software Foundation. Archived from the original on 6 June 2020. Retrieved 19 April 2011. ^ a b a b Warsaw, Barry; Hylton, Jeremy; Goodger, David (13 June 2000). \"PEP 1 \u2013 PEP Purpose and Guidelines\". Python Enhancement Proposals. Python Software Foundation. Archived from the original on 6 June 2020. Retrieved 19 April 2011. \"PEP 1 \u2013 PEP Purpose and Guidelines\" Archived . Retrieved 19 April 2011 19 April ^ \"PEP 8 \u2013 Style Guide for Python Code\". Python.org. Archived from the original on 17 April 2019. Retrieved 26 March 2019. ^ ^ \"PEP 8 \u2013 Style Guide for Python Code\". Python.org. Archived from the original on 17 April 2019. Retrieved 26 March 2019. \"PEP 8 \u2013 Style Guide for Python Code\" Archived . Retrieved 26 March 2019 26 March ^ Cannon, Brett. \"Guido, Some Guys, and a Mailing List: How Python is Developed\". python.org. Python Software Foundation. Archived from the original on 1 June 2009. Retrieved 27 June 2009. ^ ^ Cannon, Brett. \"Guido, Some Guys, and a Mailing List: How Python is Developed\". python.org. Python Software Foundation. Archived from the original on 1 June 2009. Retrieved 27 June 2009. \"Guido, Some Guys, and a Mailing List: How Python is Developed\" the original . Retrieved 27 June 2009 27 June ^ \"Moving Python's bugs to GitHub [LWN.net]\". Archived from the original on 2 October 2022. Retrieved 2 October 2022. ^ ^ \"Moving Python's bugs to GitHub [LWN.net]\". Archived from the original on 2 October 2022. Retrieved 2 October 2022. \"Moving Python's bugs to GitHub [LWN.net]\" Archived . Retrieved 2 October 2022 2 October ^ \"Python Developer's Guide \u2013 Python Developer's Guide\". devguide.python.org. Archived from the original on 9 November 2020. Retrieved 17 December 2019. ^ ^ \"Python Developer's Guide \u2013 Python Developer's Guide\". devguide.python.org. Archived from the original on 9 November 2020. Retrieved 17 December 2019. \"Python Developer's Guide \u2013 Python Developer's Guide\" Archived . Retrieved 17 December 2019 17 December ^ Hughes, Owen (24 May 2021). \"Programming languages: Why Python 4.0 might never arrive, according to its creator\". TechRepublic. Archived from the original on 14 July 2022. Retrieved 16 May 2022. ^ ^ Hughes, Owen (24 May 2021). \"Programming languages: Why Python 4.0 might never arrive, according to its creator\". TechRepublic. Archived from the original on 14 July 2022. Retrieved 16 May 2022. \"Programming languages: Why Python 4.0 might never arrive, according to its creator\" Archived . Retrieved 16 May 2022 16 May ^ \"PEP 602 \u2013 Annual Release Cycle for Python\". Python.org. Archived from the original on 14 June 2020. Retrieved 6 November 2019. ^ ^ \"PEP 602 \u2013 Annual Release Cycle for Python\". Python.org. Archived from the original on 14 June 2020. Retrieved 6 November 2019. \"PEP 602 \u2013 Annual Release Cycle for Python\" Archived . Retrieved 6 November 2019 6 November ^ \"Changing the Python release cadence [LWN.net]\". lwn.net. Archived from the original on 6 November 2019. Retrieved 6 November 2019. ^ ^ \"Changing the Python release cadence [LWN.net]\". lwn.net. Archived from the original on 6 November 2019. Retrieved 6 November 2019. \"Changing the Python release cadence [LWN.net]\" Archived . Retrieved 6 November 2019 6 November ^ Norwitz, Neal (8 April 2002). \"[Python-Dev] Release Schedules (was Stability & change)\". Archived from the original on 15 December 2018. Retrieved 27 June 2009. ^ ^ Norwitz, Neal (8 April 2002). \"[Python-Dev] Release Schedules (was Stability & change)\". Archived from the original on 15 December 2018. Retrieved 27 June 2009. \"[Python-Dev] Release Schedules (was Stability & change)\" Archived . Retrieved 27 June 2009 27 June ^ a b Aahz; Baxter, Anthony (15 March 2001). \"PEP 6 \u2013 Bug Fix Releases\". Python Enhancement Proposals. Python Software Foundation. Archived from the original on 5 June 2020. Retrieved 27 June 2009. ^ a b a b Aahz; Baxter, Anthony (15 March 2001). \"PEP 6 \u2013 Bug Fix Releases\". Python Enhancement Proposals. Python Software Foundation. Archived from the original on 5 June 2020. Retrieved 27 June 2009. \"PEP 6 \u2013 Bug Fix Releases\" Archived . Retrieved 27 June 2009 27 June ^ \"Python Buildbot\". Python Developer's Guide. Python Software Foundation. Archived from the original on 5 June 2020. Retrieved 24 September 2011. ^ ^ \"Python Buildbot\". Python Developer's Guide. Python Software Foundation. Archived from the original on 5 June 2020. Retrieved 24 September 2011. \"Python Buildbot\" Archived . Retrieved 24 September 2011 24 September ^ \"1. Extending Python with C or C++ \u2013 Python 3.9.1 documentation\". docs.python.org. Archived from the original on 23 June 2020. Retrieved 14 February 2021. ^ ^ \"1. Extending Python with C or C++ \u2013 Python 3.9.1 documentation\". docs.python.org. Archived from the original on 23 June 2020. Retrieved 14 February 2021. \"1. Extending Python with C or C++ \u2013 Python 3.9.1 documentation\" Archived . Retrieved 14 February 2021 14 February ^ \"PEP 623 \u2013 Remove wstr from Unicode\". Python.org. Archived from the original on 5 March 2021. Retrieved 14 February 2021. ^ ^ \"PEP 623 \u2013 Remove wstr from Unicode\". Python.org. Archived from the original on 5 March 2021. Retrieved 14 February 2021. \"PEP 623 \u2013 Remove wstr from Unicode\" Archived . Retrieved 14 February 2021 14 February ^ \"PEP 634 \u2013 Structural Pattern Matching: Specification\". Python.org. Archived from the original on 6 May 2021. Retrieved 14 February 2021. ^ ^ \"PEP 634 \u2013 Structural Pattern Matching: Specification\". Python.org. Archived from the original on 6 May 2021. Retrieved 14 February 2021. \"PEP 634 \u2013 Structural Pattern Matching: Specification\" Archived . Retrieved 14 February 2021 14 February ^ \"Documentation Tools\". Python.org. Archived from the original on 11 November 2020. Retrieved 22 March 2021. ^ ^ \"Documentation Tools\". Python.org. Archived from the original on 11 November 2020. Retrieved 22 March 2021. \"Documentation Tools\" Archived . Retrieved 22 March 2021 22 March ^ a b \"Whetting Your Appetite\". The Python Tutorial. Python Software Foundation. Archived from the original on 26 October 2012. Retrieved 20 February 2012. ^ a b a b \"Whetting Your Appetite\". The Python Tutorial. Python Software Foundation. Archived from the original on 26 October 2012. Retrieved 20 February 2012. \"Whetting Your Appetite\" Archived . Retrieved 20 February 2012 20 February ^ \"In Python, should I use else after a return in an if block?\". Stack Overflow. Stack Exchange. 17 February 2011. Archived from the original on 20 June 2019. Retrieved 6 May 2011. ^ ^ \"In Python, should I use else after a return in an if block?\". Stack Overflow. Stack Exchange. 17 February 2011. Archived from the original on 20 June 2019. Retrieved 6 May 2011. \"In Python, should I use else after a return in an if block?\" Stack Overflow Archived . Retrieved 6 May 2011 6 May ^ Lutz, Mark (2009). Learning Python: Powerful Object-Oriented Programming. O'Reilly Media, Inc. p. 17. ISBN 9781449379322. Archived from the original on 17 July 2017. Retrieved 9 May 2017. ^ ^ Lutz, Mark (2009). Learning Python: Powerful Object-Oriented Programming. O'Reilly Media, Inc. p. 17. ISBN 9781449379322. Archived from the original on 17 July 2017. Retrieved 9 May 2017. Learning Python: Powerful Object-Oriented Programming ISBN 9781449379322 Archived . Retrieved 9 May 2017 9 May ^ Fehily, Chris (2002). Python. Peachpit Press. p. xv. ISBN 9780201748840. Archived from the original on 17 July 2017. Retrieved 9 May 2017. ^ ^ Fehily, Chris (2002). Python. Peachpit Press. p. xv. ISBN 9780201748840. Archived from the original on 17 July 2017. Retrieved 9 May 2017. Python ISBN 9780201748840 Archived . Retrieved 9 May 2017 9 May ^ Lubanovic, Bill (2014). Introducing Python. Sebastopol, CA : O'Reilly Media. p. 305. ISBN 978-1-4493-5936-2. Retrieved 31 July 2023. ^ ^ Lubanovic, Bill (2014). Introducing Python. Sebastopol, CA : O'Reilly Media. p. 305. ISBN 978-1-4493-5936-2. Retrieved 31 July 2023. Introducing Python ISBN 978-1-4493-5936-2 . Retrieved 31 July 2023 31 July ^ Blake, Troy (18 January 2021). \"TIOBE Index for January 2021\". Technology News and Information by SeniorDBA. Archived from the original on 21 March 2021. Retrieved 26 February 2021. ^ ^ Blake, Troy (18 January 2021). \"TIOBE Index for January 2021\". Technology News and Information by SeniorDBA. Archived from the original on 21 March 2021. Retrieved 26 February 2021. \"TIOBE Index for January 2021\" Archived . Retrieved 26 February 2021 26 February ^ \"Quotes about Python\". Python Software Foundation. Archived from the original on 3 June 2020. Retrieved 8 January 2012. ^ ^ \"Quotes about Python\". Python Software Foundation. Archived from the original on 3 June 2020. Retrieved 8 January 2012. \"Quotes about Python\" Archived . Retrieved 8 January 2012 8 January ^ \"Organizations Using Python\". Python Software Foundation. Archived from the original on 21 August 2018. Retrieved 15 January 2009. ^ ^ \"Organizations Using Python\". Python Software Foundation. Archived from the original on 21 August 2018. Retrieved 15 January 2009. \"Organizations Using Python\" Archived . Retrieved 15 January 2009 15 January ^ \"Python : the holy grail of programming\". CERN Bulletin (31/2006). CERN Publications. 31 July 2006. Archived from the original on 15 January 2013. Retrieved 11 February 2012. ^ ^ \"Python : the holy grail of programming\". CERN Bulletin (31/2006). CERN Publications. 31 July 2006. Archived from the original on 15 January 2013. Retrieved 11 February 2012. \"Python : the holy grail of programming\" Archived . Retrieved 11 February 2012 11 February ^ Shafer, Daniel G. (17 January 2003). \"Python Streamlines Space Shuttle Mission Design\". Python Software Foundation. Archived from the original on 5 June 2020. Retrieved 24 November 2008. ^ ^ Shafer, Daniel G. (17 January 2003). \"Python Streamlines Space Shuttle Mission Design\". Python Software Foundation. Archived from the original on 5 June 2020. Retrieved 24 November 2008. \"Python Streamlines Space Shuttle Mission Design\" Archived . Retrieved 24 November 2008 24 November ^ \"Tornado: Facebook's Real-Time Web Framework for Python \u2013 Facebook for Developers\". Facebook for Developers. Archived from the original on 19 February 2019. Retrieved 19 June 2018. ^ ^ \"Tornado: Facebook's Real-Time Web Framework for Python \u2013 Facebook for Developers\". Facebook for Developers. Archived from the original on 19 February 2019. Retrieved 19 June 2018. \"Tornado: Facebook's Real-Time Web Framework for Python \u2013 Facebook for Developers\" Archived . Retrieved 19 June 2018 19 June ^ \"What Powers Instagram: Hundreds of Instances, Dozens of Technologies\". Instagram Engineering. 11 December 2016. Archived from the original on 15 June 2020. Retrieved 27 May 2019. ^ ^ \"What Powers Instagram: Hundreds of Instances, Dozens of Technologies\". Instagram Engineering. 11 December 2016. Archived from the original on 15 June 2020. Retrieved 27 May 2019. \"What Powers Instagram: Hundreds of Instances, Dozens of Technologies\" Archived . Retrieved 27 May 2019 27 May ^ \"How we use Python at Spotify\". Spotify Labs. 20 March 2013. Archived from the original on 10 June 2020. Retrieved 25 July 2018. ^ ^ \"How we use Python at Spotify\". Spotify Labs. 20 March 2013. Archived from the original on 10 June 2020. Retrieved 25 July 2018. \"How we use Python at Spotify\" Archived . Retrieved 25 July 2018 25 July ^ Fortenberry, Tim (17 January 2003). \"Industrial Light & Magic Runs on Python\". Python Software Foundation. Archived from the original on 6 June 2020. Retrieved 11 February 2012. ^ ^ Fortenberry, Tim (17 January 2003). \"Industrial Light & Magic Runs on Python\". Python Software Foundation. Archived from the original on 6 June 2020. Retrieved 11 February 2012. \"Industrial Light & Magic Runs on Python\" Archived . Retrieved 11 February 2012 11 February ^ Taft, Darryl K. (5 March 2007). \"Python Slithers into Systems\". eWeek.com. Ziff Davis Holdings. Archived from the original on 13 August 2021. Retrieved 24 September 2011. ^ ^ Taft, Darryl K. (5 March 2007). \"Python Slithers into Systems\". eWeek.com. Ziff Davis Holdings. Archived from the original on 13 August 2021. Retrieved 24 September 2011. \"Python Slithers into Systems\" Archived . Retrieved 24 September 2011 24 September ^ GitHub \u2013 reddit-archive/reddit: historical code from reddit.com., The Reddit Archives, archived from the original on 1 June 2020, retrieved 20 March 2019 ^ ^ GitHub \u2013 reddit-archive/reddit: historical code from reddit.com., The Reddit Archives, archived from the original on 1 June 2020, retrieved 20 March 2019 GitHub \u2013 reddit-archive/reddit: historical code from reddit.com. archived , retrieved 20 March 2019 20 March ^ \"Real time communication at scale with Elixir at Discord\". 8 October 2020. ^ ^ \"Real time communication at scale with Elixir at Discord\". 8 October 2020. \"Real time communication at scale with Elixir at Discord\" ^ \"What Programming Language is Baidu Built In?\". 5 July 2018. ^ ^ \"What Programming Language is Baidu Built In?\". 5 July 2018. \"What Programming Language is Baidu Built In?\" ^ \"Usage statistics and market share of Python for websites\". 2012. Archived from the original on 13 August 2021. Retrieved 18 December 2012. ^ ^ \"Usage statistics and market share of Python for websites\". 2012. Archived from the original on 13 August 2021. Retrieved 18 December 2012. \"Usage statistics and market share of Python for websites\" Archived . Retrieved 18 December 2012 18 December ^ Oliphant, Travis (2007). \"Python for Scientific Computing\". Computing in Science and Engineering. 9 (3): 10\u201320. Bibcode:2007CSE.....9c..10O. CiteSeerX 10.1.1.474.6460. doi:10.1109/MCSE.2007.58. ISSN 1521-9615. S2CID 206457124. Archived from the original on 15 June 2020. Retrieved 10 April 2015. ^ ^ Oliphant, Travis (2007). \"Python for Scientific Computing\". Computing in Science and Engineering. 9 (3): 10\u201320. Bibcode:2007CSE.....9c..10O. CiteSeerX 10.1.1.474.6460. doi:10.1109/MCSE.2007.58. ISSN 1521-9615. S2CID 206457124. Archived from the original on 15 June 2020. Retrieved 10 April 2015. \"Python for Scientific Computing\" 10\u2013 Bibcode 2007CSE.....9c..10O CiteSeerX 10.1.1.474.6460 10.1.1.474.6460 doi 10.1109/MCSE.2007.58 ISSN 1521-9615 S2CID 206457124 Archived . Retrieved 10 April 2015 10 April ^ Millman, K. Jarrod; Aivazis, Michael (2011). \"Python for Scientists and Engineers\". Computing in Science and Engineering. 13 (2): 9\u201312. Bibcode:2011CSE....13b...9M. doi:10.1109/MCSE.2011.36. Archived from the original on 19 February 2019. Retrieved 7 July 2014. ^ ^ Millman, K. Jarrod; Aivazis, Michael (2011). \"Python for Scientists and Engineers\". Computing in Science and Engineering. 13 (2): 9\u201312. Bibcode:2011CSE....13b...9M. doi:10.1109/MCSE.2011.36. Archived from the original on 19 February 2019. Retrieved 7 July 2014. \"Python for Scientists and Engineers\" 9\u2013 Bibcode 2011CSE....13b...9M doi 10.1109/MCSE.2011.36 Archived . Retrieved 7 July 2014 7 July ^ Science education with SageMath, Innovative Computing in Science Education, archived from the original on 15 June 2020, retrieved 22 April 2019 ^ ^ Science education with SageMath, Innovative Computing in Science Education, archived from the original on 15 June 2020, retrieved 22 April 2019 Science education with SageMath the original , retrieved 22 April 2019 22 April ^ \"OpenCV: OpenCV-Python Tutorials\". docs.opencv.org. Archived from the original on 23 September 2020. Retrieved 14 September 2020. ^ ^ \"OpenCV: OpenCV-Python Tutorials\". docs.opencv.org. Archived from the original on 23 September 2020. Retrieved 14 September 2020. \"OpenCV: OpenCV-Python Tutorials\" Archived . Retrieved 14 September 2020 14 September ^ Dean, Jeff; Monga, Rajat; et al. (9 November 2015). \"TensorFlow: Large-scale machine learning on heterogeneous systems\" (PDF). TensorFlow.org. Google Research. Archived (PDF) from the original on 20 November 2015. Retrieved 10 November 2015. ^ ^ Dean, Jeff; Monga, Rajat; et al. (9 November 2015). \"TensorFlow: Large-scale machine learning on heterogeneous systems\" (PDF). TensorFlow.org. Google Research. Archived (PDF) from the original on 20 November 2015. Retrieved 10 November 2015. Dean, Jeff \"TensorFlow: Large-scale machine learning on heterogeneous systems\" (PDF) Archived (PDF) . Retrieved 10 November 2015 10 November ^ Piatetsky, Gregory. \"Python eats away at R: Top Software for Analytics, Data Science, Machine Learning in 2018: Trends and Analysis\". KDnuggets. Archived from the original on 15 November 2019. Retrieved 30 May 2018. ^ ^ Piatetsky, Gregory. \"Python eats away at R: Top Software for Analytics, Data Science, Machine Learning in 2018: Trends and Analysis\". KDnuggets. Archived from the original on 15 November 2019. Retrieved 30 May 2018. \"Python eats away at R: Top Software for Analytics, Data Science, Machine Learning in 2018: Trends and Analysis\" Archived . Retrieved 30 May 2018 30 May ^ \"Who is using scikit-learn? \u2013 scikit-learn 0.20.1 documentation\". scikit-learn.org. Archived from the original on 6 May 2020. Retrieved 30 November 2018. ^ ^ \"Who is using scikit-learn? \u2013 scikit-learn 0.20.1 documentation\". scikit-learn.org. Archived from the original on 6 May 2020. Retrieved 30 November 2018. \"Who is using scikit-learn? \u2013 scikit-learn 0.20.1 documentation\" Archived . Retrieved 30 November 2018 30 November ^ Jouppi, Norm. \"Google supercharges machine learning tasks with TPU custom chip\". Google Cloud Platform Blog. Archived from the original on 18 May 2016. Retrieved 19 May 2016. ^ ^ Jouppi, Norm. \"Google supercharges machine learning tasks with TPU custom chip\". Google Cloud Platform Blog. Archived from the original on 18 May 2016. Retrieved 19 May 2016. Jouppi, Norm \"Google supercharges machine learning tasks with TPU custom chip\" Archived . Retrieved 19 May 2016 19 May ^ De Raedt, Luc; Kimmig, Angelika (2015). \"Probabilistic (logic) programming concepts\". Machine Learning. 100 (1): 5\u201347. doi:10.1007/s10994-015-5494-z. S2CID 3166992. ^ ^ De Raedt, Luc; Kimmig, Angelika (2015). \"Probabilistic (logic) programming concepts\". Machine Learning. 100 (1): 5\u201347. doi:10.1007/s10994-015-5494-z. S2CID 3166992. \"Probabilistic (logic) programming concepts\" 5\u2013 doi 10.1007/s10994-015-5494-z 10.1007/s10994-015-5494-z S2CID 3166992 ^ \"Natural Language Toolkit \u2013 NLTK 3.5b1 documentation\". www.nltk.org. Archived from the original on 13 June 2020. Retrieved 10 April 2020. ^ ^ \"Natural Language Toolkit \u2013 NLTK 3.5b1 documentation\". www.nltk.org. Archived from the original on 13 June 2020. Retrieved 10 April 2020. \"Natural Language Toolkit \u2013 NLTK 3.5b1 documentation\" Archived . Retrieved 10 April 2020 10 April ^ Andersen, C. and Swift, T., 2023. The Janus System: a bridge to new prolog applications. In Prolog: The Next 50 Years (pp. 93\u2013104). Cham: Springer Nature Switzerland. ^ ^ Andersen, C. and Swift, T., 2023. The Janus System: a bridge to new prolog applications. In Prolog: The Next 50 Years (pp. 93\u2013104). Cham: Springer Nature Switzerland. ^ \"SWI-Prolog Python interface\". Archived from the original on 15 March 2024. Retrieved 15 March 2024. ^ ^ \"SWI-Prolog Python interface\". Archived from the original on 15 March 2024. Retrieved 15 March 2024. \"SWI-Prolog Python interface\" Archived . Retrieved 15 March 2024 15 March ^ Tarau, P., 2023. Reflections on automation, learnability and expressiveness in logic-based programming languages. In Prolog: The Next 50 Years (pp. 359\u2013371). Cham: Springer Nature Switzerland. ^ ^ Tarau, P., 2023. Reflections on automation, learnability and expressiveness in logic-based programming languages. In Prolog: The Next 50 Years (pp. 359\u2013371). Cham: Springer Nature Switzerland. ^ \"Tkinter \u2014 Python interface to TCL/Tk\". Archived from the original on 18 October 2012. Retrieved 9 June 2023. ^ ^ \"Tkinter \u2014 Python interface to TCL/Tk\". Archived from the original on 18 October 2012. Retrieved 9 June 2023. \"Tkinter \u2014 Python interface to TCL/Tk\" Archived . Retrieved 9 June 2023 9 June ^ \"Python Tkinter Tutorial\". 3 June 2020. Archived from the original on 9 June 2023. Retrieved 9 June 2023. ^ ^ \"Python Tkinter Tutorial\". 3 June 2020. Archived from the original on 9 June 2023. Retrieved 9 June 2023. \"Python Tkinter Tutorial\" Archived . Retrieved 9 June 2023 9 June ^ \"Installers for GIMP for Windows \u2013 Frequently Asked Questions\". 26 July 2013. Archived from the original on 17 July 2013. Retrieved 26 July 2013. ^ ^ \"Installers for GIMP for Windows \u2013 Frequently Asked Questions\". 26 July 2013. Archived from the original on 17 July 2013. Retrieved 26 July 2013. \"Installers for GIMP for Windows \u2013 Frequently Asked Questions\" the original . Retrieved 26 July 2013 26 July ^ \"jasc psp9components\". Archived from the original on 19 March 2008. ^ ^ \"jasc psp9components\". Archived from the original on 19 March 2008. \"jasc psp9components\" the original ^ \"About getting started with writing geoprocessing scripts\". ArcGIS Desktop Help 9.2. Environmental Systems Research Institute. 17 November 2006. Archived from the original on 5 June 2020. Retrieved 11 February 2012. ^ ^ \"About getting started with writing geoprocessing scripts\". ArcGIS Desktop Help 9.2. Environmental Systems Research Institute. 17 November 2006. Archived from the original on 5 June 2020. Retrieved 11 February 2012. \"About getting started with writing geoprocessing scripts\" Archived . Retrieved 11 February 2012 11 February ^ CCP porkbelly (24 August 2010). \"Stackless Python 2.7\". EVE Community Dev Blogs. CCP Games. Archived from the original on 11 January 2014. Retrieved 11 January 2014. As you may know, EVE has at its core the programming language known as Stackless Python. ^ ^ CCP porkbelly (24 August 2010). \"Stackless Python 2.7\". EVE Community Dev Blogs. CCP Games. Archived from the original on 11 January 2014. Retrieved 11 January 2014. As you may know, EVE has at its core the programming language known as Stackless Python. \"Stackless Python 2.7\" CCP Games Archived . Retrieved 11 January 2014 11 January ^ Caudill, Barry (20 September 2005). \"Modding Sid Meier's Civilization IV\". Sid Meier's Civilization IV Developer Blog. Firaxis Games. Archived from the original on 2 December 2010. we created three levels of tools ... The next level offers Python and XML support, letting modders with more experience manipulate the game world and everything in it. ^ ^ Caudill, Barry (20 September 2005). \"Modding Sid Meier's Civilization IV\". Sid Meier's Civilization IV Developer Blog. Firaxis Games. Archived from the original on 2 December 2010. we created three levels of tools ... The next level offers Python and XML support, letting modders with more experience manipulate the game world and everything in it. \"Modding Sid Meier's Civilization IV\" Firaxis Games the original ^ \"Python Language Guide (v1.0)\". Google Documents List Data API v1.0. Archived from the original on 15 July 2010. ^ ^ \"Python Language Guide (v1.0)\". Google Documents List Data API v1.0. Archived from the original on 15 July 2010. \"Python Language Guide (v1.0)\" the original ^ \"Python Setup and Usage\". Python Software Foundation. Archived from the original on 17 June 2020. Retrieved 10 January 2020. ^ ^ \"Python Setup and Usage\". Python Software Foundation. Archived from the original on 17 June 2020. Retrieved 10 January 2020. \"Python Setup and Usage\" Archived . Retrieved 10 January 2020 10 January ^ \"Immunity: Knowing You're Secure\". Archived from the original on 16 February 2009. ^ ^ \"Immunity: Knowing You're Secure\". Archived from the original on 16 February 2009. \"Immunity: Knowing You're Secure\" the original ^ \"Core Security\". Core Security. Archived from the original on 9 June 2020. Retrieved 10 April 2020. ^ ^ \"Core Security\". Core Security. Archived from the original on 9 June 2020. Retrieved 10 April 2020. \"Core Security\" Archived . Retrieved 10 April 2020 10 April ^ \"What is Sugar?\". Sugar Labs. Archived from the original on 9 January 2009. Retrieved 11 February 2012. ^ ^ \"What is Sugar?\". Sugar Labs. Archived from the original on 9 January 2009. Retrieved 11 February 2012. \"What is Sugar?\" Archived . Retrieved 11 February 2012 11 February ^ \"4.0 New Features and Fixes\". LibreOffice.org. The Document Foundation. 2013. Archived from the original on 9 February 2014. Retrieved 25 February 2013. ^ ^ \"4.0 New Features and Fixes\". LibreOffice.org. The Document Foundation. 2013. Archived from the original on 9 February 2014. Retrieved 25 February 2013. \"4.0 New Features and Fixes\" The Document Foundation Archived . Retrieved 25 February 2013 25 February ^ \"Gotchas for Python Users\". boo.codehaus.org. Codehaus Foundation. Archived from the original on 11 December 2008. Retrieved 24 November 2008. ^ ^ \"Gotchas for Python Users\". boo.codehaus.org. Codehaus Foundation. Archived from the original on 11 December 2008. Retrieved 24 November 2008. \"Gotchas for Python Users\" the original . Retrieved 24 November 2008 24 November ^ Esterbrook, Charles. \"Acknowledgements\". cobra-language.com. Cobra Language. Archived from the original on 8 February 2008. Retrieved 7 April 2010. ^ ^ Esterbrook, Charles. \"Acknowledgements\". cobra-language.com. Cobra Language. Archived from the original on 8 February 2008. Retrieved 7 April 2010. \"Acknowledgements\" the original . Retrieved 7 April 2010 7 April ^ \"Proposals: iterators and generators [ES4 Wiki]\". wiki.ecmascript.org. Archived from the original on 20 October 2007. Retrieved 24 November 2008. ^ ^ \"Proposals: iterators and generators [ES4 Wiki]\". wiki.ecmascript.org. Archived from the original on 20 October 2007. Retrieved 24 November 2008. \"Proposals: iterators and generators [ES4 Wiki]\" the original . Retrieved 24 November 2008 24 November ^ \"Frequently asked questions\". Godot Engine documentation. Archived from the original on 28 April 2021. Retrieved 10 May 2021. ^ ^ \"Frequently asked questions\". Godot Engine documentation. Archived from the original on 28 April 2021. Retrieved 10 May 2021. \"Frequently asked questions\" Archived . Retrieved 10 May 2021 10 May ^ Kincaid, Jason (10 November 2009). \"Google's Go: A New Programming Language That's Python Meets C++\". TechCrunch. Archived from the original on 18 January 2010. Retrieved 29 January 2010. ^ ^ Kincaid, Jason (10 November 2009). \"Google's Go: A New Programming Language That's Python Meets C++\". TechCrunch. Archived from the original on 18 January 2010. Retrieved 29 January 2010. \"Google's Go: A New Programming Language That's Python Meets C++\" Archived . Retrieved 29 January 2010 29 January ^ Strachan, James (29 August 2003). \"Groovy \u2013 the birth of a new dynamic language for the Java platform\". Archived from the original on 5 April 2007. Retrieved 11 June 2007. ^ ^ Strachan, James (29 August 2003). \"Groovy \u2013 the birth of a new dynamic language for the Java platform\". Archived from the original on 5 April 2007. Retrieved 11 June 2007. \"Groovy \u2013 the birth of a new dynamic language for the Java platform\" the original . Retrieved 11 June 2007 11 June ^ \"Modular Docs \u2013 Why Mojo\". docs.modular.com. Archived from the original on 5 May 2023. Retrieved 5 May 2023. Mojo as a member of the Python family [..] Embracing Python massively simplifies our design efforts, because most of the syntax is already specified. [..] we decided that the right long-term goal for Mojo is to provide a superset of Python (i.e. be compatible with existing programs) and to embrace the CPython immediately for long-tail ecosystem enablement. To a Python programmer, we expect and hope that Mojo will be immediately familiar, while also providing new tools for developing systems-level code that enable you to do things that Python falls back to C and C++ for. ^ ^ \"Modular Docs \u2013 Why Mojo\". docs.modular.com. Archived from the original on 5 May 2023. Retrieved 5 May 2023. Mojo as a member of the Python family [..] Embracing Python massively simplifies our design efforts, because most of the syntax is already specified. [..] we decided that the right long-term goal for Mojo is to provide a superset of Python (i.e. be compatible with existing programs) and to embrace the CPython immediately for long-tail ecosystem enablement. To a Python programmer, we expect and hope that Mojo will be immediately familiar, while also providing new tools for developing systems-level code that enable you to do things that Python falls back to C and C++ for. \"Modular Docs \u2013 Why Mojo\" Archived . Retrieved 5 May 2023 5 May ^ Spencer, Michael (4 May 2023). \"What is Mojo Programming Language?\". datasciencelearningcenter.substack.com. Archived from the original on 5 May 2023. Retrieved 5 May 2023. ^ ^ Spencer, Michael (4 May 2023). \"What is Mojo Programming Language?\". datasciencelearningcenter.substack.com. Archived from the original on 5 May 2023. Retrieved 5 May 2023. \"What is Mojo Programming Language?\" Archived . Retrieved 5 May 2023 5 May ^ Yegulalp, Serdar (16 January 2017). \"Nim language draws from best of Python, Rust, Go, and Lisp\". InfoWorld. Archived from the original on 13 October 2018. Retrieved 7 June 2020. Nim's syntax is strongly reminiscent of Python's, as it uses indented code blocks and some of the same syntax (such as the way if/elif/then/else blocks are constructed). ^ ^ Yegulalp, Serdar (16 January 2017). \"Nim language draws from best of Python, Rust, Go, and Lisp\". InfoWorld. Archived from the original on 13 October 2018. Retrieved 7 June 2020. Nim's syntax is strongly reminiscent of Python's, as it uses indented code blocks and some of the same syntax (such as the way if/elif/then/else blocks are constructed). \"Nim language draws from best of Python, Rust, Go, and Lisp\" Archived . Retrieved 7 June 2020 7 June ^ \"An Interview with the Creator of Ruby\". Linuxdevcenter.com. Archived from the original on 28 April 2018. Retrieved 3 December 2012. ^ ^ \"An Interview with the Creator of Ruby\". Linuxdevcenter.com. Archived from the original on 28 April 2018. Retrieved 3 December 2012. \"An Interview with the Creator of Ruby\" Archived . Retrieved 3 December 2012 3 December ^ Lattner, Chris (3 June 2014). \"Chris Lattner's Homepage\". Chris Lattner. Archived from the original on 22 December 2015. Retrieved 3 June 2014. I started work on the Swift Programming Language in July of 2010. I implemented much of the basic language structure, with only a few people knowing of its existence. A few other (amazing) people started contributing in earnest late in 2011, and it became a major focus for the Apple Developer Tools group in July 2013 [...] drawing ideas from Objective-C, Rust, Haskell, Ruby, Python, C#, CLU, and far too many others to list. ^ ^ Lattner, Chris (3 June 2014). \"Chris Lattner's Homepage\". Chris Lattner. Archived from the original on 22 December 2015. Retrieved 3 June 2014. I started work on the Swift Programming Language in July of 2010. I implemented much of the basic language structure, with only a few people knowing of its existence. A few other (amazing) people started contributing in earnest late in 2011, and it became a major focus for the Apple Developer Tools group in July 2013 [...] drawing ideas from Objective-C, Rust, Haskell, Ruby, Python, C#, CLU, and far too many others to list. Lattner, Chris \"Chris Lattner's Homepage\" Archived . Retrieved 3 June 2014 3 June ^ Jalan, Nishant Aanjaney (10 November 2022). \"Programming in Kotlin\". CodeX. Retrieved 29 April 2024. ^ ^ Jalan, Nishant Aanjaney (10 November 2022). \"Programming in Kotlin\". CodeX. Retrieved 29 April 2024. \"Programming in Kotlin\" . Retrieved 29 April 2024 29 April ^ Kupries, Andreas; Fellows, Donal K. (14 September 2000). \"TIP #3: TIP Format\". tcl.tk. Tcl Developer Xchange. Archived from the original on 13 July 2017. Retrieved 24 November 2008. ^ ^ Kupries, Andreas; Fellows, Donal K. (14 September 2000). \"TIP #3: TIP Format\". tcl.tk. Tcl Developer Xchange. Archived from the original on 13 July 2017. Retrieved 24 November 2008. \"TIP #3: TIP Format\" Archived . Retrieved 24 November 2008 24 November ^ Gustafsson, Per; Niskanen, Raimo (29 January 2007). \"EEP 1: EEP Purpose and Guidelines\". erlang.org. Archived from the original on 15 June 2020. Retrieved 19 April 2011. ^ ^ Gustafsson, Per; Niskanen, Raimo (29 January 2007). \"EEP 1: EEP Purpose and Guidelines\". erlang.org. Archived from the original on 15 June 2020. Retrieved 19 April 2011. \"EEP 1: EEP Purpose and Guidelines\" Archived . Retrieved 19 April 2011 19 April ^ \"Swift Evolution Process\". Swift Programming Language Evolution repository on GitHub. 18 February 2020. Archived from the original on 27 April 2020. Retrieved 27 April 2020. ^ ^ \"Swift Evolution Process\". Swift Programming Language Evolution repository on GitHub. 18 February 2020. Archived from the original on 27 April 2020. Retrieved 27 April 2020. \"Swift Evolution Process\" Archived . Retrieved 27 April 2020 27 April Sources [edit] [ edit edit ] \"Python for Artificial Intelligence\". Python Wiki. 19 July 2012. Archived from the original on 1 November 2012. Retrieved 3 December 2012. \"Python for Artificial Intelligence\" the original . Retrieved 3 December 2012 3 December Paine, Jocelyn, ed. (August 2005). \"AI in Python\". AI Expert Newsletter. Amzi!. Archived from the original on 26 March 2012. Retrieved 11 February 2012. \"AI in Python\" the original . Retrieved 11 February 2012 11 February \"PyAIML 0.8.5 : Python Package Index\". Pypi.python.org. Retrieved 17 July 2013. \"PyAIML 0.8.5 : Python Package Index\" . Retrieved 17 July 2013 17 July Russell, Stuart J. & Norvig, Peter (2009). Artificial Intelligence: A Modern Approach (3rd ed.). Upper Saddle River, NJ: Prentice Hall. ISBN 978-0-13-604259-4. Russell, Stuart J. Norvig, Peter ISBN 978-0-13-604259-4 Further reading [edit] [ edit edit ] Downey, Allen (July 2024). Think Python: How to Think Like a Computer Scientist (3rd ed.). O'Reilly Media. ISBN 978-1098155438. Think Python: How to Think Like a Computer Scientist ISBN 978-1098155438 Lutz, Mark (2013). Learning Python (5th ed.). O'Reilly Media. ISBN 978-0-596-15806-4. ISBN 978-0-596-15806-4 Summerfield, Mark (2009). Programming in Python 3 (2nd ed.). Addison-Wesley Professional. ISBN 978-0-321-68056-3. ISBN 978-0-321-68056-3 Ramalho, Luciano (May 2022). Fluent Python. O'Reilly Media. ISBN 978-1-4920-5632-4. Fluent Python ISBN 978-1-4920-5632-4 External links [edit] [ edit edit ] sister projects sister projects Media from Commons Media from Commons Media Quotations from Wikiquote Quotations from Wikiquote Quotations Textbooks from Wikibooks Textbooks from Wikibooks Textbooks Resources from Wikiversity Resources from Wikiversity Resources Data from Wikidata Data from Wikidata Data Official website Official website Official website Official website The Python Tutorial The Python Tutorial v v t t e e Python Python Implementations CircuitPython CircuitPython CLPython CLPython CPython CPython Cython Cython MicroPython MicroPython Numba Numba IronPython IronPython Jython Jython Psyco Psyco PyPy PyPy Python for S60 Python for S60 Shed Skin Shed Skin Stackless Python Stackless Python Unladen Swallow Unladen Swallow more... more IDEs eric eric IDLE IDLE Ninja-IDE Ninja-IDE PyCharm PyCharm PyDev PyDev Spyder Spyder more... more WSGI WSGI ASGI ASGI Designer Guido van Rossum Guido van Rossum Software (list) Software Python Software Foundation Python Software Foundation Python Conference (PyCon) Python Conference v v t t e e Programming languages Comparison Comparison Timeline Timeline History History Ada Ada ALGOL Simula ALGOL Simula Simula APL APL Assembly Assembly BASIC Visual Basic classic .NET BASIC Visual Basic classic .NET Visual Basic classic classic .NET .NET C C C++ C++ C# C# COBOL COBOL Erlang Erlang Forth Forth Fortran Fortran Go Go Haskell Haskell Java Java JavaScript JavaScript Julia Julia Kotlin Kotlin Lisp Lisp Lua Lua MATLAB MATLAB ML ML Pascal Object Pascal Pascal Object Pascal Object Pascal Perl Perl PHP PHP Prolog Prolog Python Python R R Ruby Ruby Rust Rust SQL SQL Scratch Scratch Shell Shell Smalltalk Smalltalk Swift Swift more... more... Lists: Alphabetical Alphabetical Categorical Categorical Generational Generational Non-English-based Non-English-based Category Category v v t t e e Python web frameworks Bottle CherryPy CherryPy CubicWeb CubicWeb Django Django FastAPI FastAPI Flask Flask Grok Grok Nevow Nevow Pylons Pylons Pyramid Pyramid Quixote Quixote TACTIC TACTIC Tornado Tornado TurboGears TurboGears TwistedWeb TwistedWeb web2py web2py Zope 2 Zope 2 more... more Comparison Comparison v v t t e e General Differentiable programming Differentiable programming Information geometry Information geometry Statistical manifold Statistical manifold Automatic differentiation Automatic differentiation Neuromorphic computing Neuromorphic computing Pattern recognition Pattern recognition Ricci calculus Ricci calculus Computational learning theory Computational learning theory Inductive bias Inductive bias IPU IPU TPU TPU VPU VPU Memristor Memristor SpiNNaker SpiNNaker TensorFlow TensorFlow PyTorch PyTorch Keras Keras scikit-learn scikit-learn Theano Theano JAX JAX Flux.jl Flux.jl MindSpore MindSpore Portals Computer programming Technology Computer programming Computer programming Technology Technology v v t t e e Free and open-source software Alternative terms for free software Alternative terms for free software Comparison of open-source and closed-source software Comparison of open-source and closed-source software Comparison of source-code-hosting facilities Comparison of source-code-hosting facilities Free software Free software Free software project directories Free software project directories Gratis versus libre Gratis versus libre Long-term support Long-term support Open-source software Open-source software Open-source software development Open-source software development Outline Outline Timeline Timeline Softwarepackages Audio Audio Bioinformatics Bioinformatics Codecs Codecs Configuration management Configuration management Drivers Graphics Wireless Drivers Graphics Graphics Wireless Wireless Health Health Mathematics Mathematics Office suites Office suites Operating systems Operating systems Routing Routing Television Television Video games Video games Web applications E-commerce Web applications E-commerce E-commerce Android apps Android apps iOS apps iOS apps Commercial Commercial Formerly proprietary Formerly proprietary Formerly open-source Formerly open-source Community Free software movement Free software movement History History Open-source-software movement Open-source-software movement Events Events Advocacy Advocacy Organisations Free Software Movement of India Free Software Movement of India Free Software Foundation Free Software Foundation Licenses AFL AFL Apache Apache APSL APSL Artistic Artistic Beerware Beerware BSD BSD Creative Commons Creative Commons CDDL CDDL EPL EPL Free Software Foundation GNU GPL GNU AGPL GNU LGPL Free Software Foundation GNU GPL GNU GPL GNU AGPL GNU AGPL GNU LGPL GNU LGPL ISC ISC MIT MIT MPL MPL Python Python Python Software Foundation License Python Software Foundation License Shared Source Initiative Shared Source Initiative Sleepycat Sleepycat Unlicense Unlicense WTFPL WTFPL zlib zlib Comparison of licenses Comparison of licenses Contributor License Agreement Contributor License Agreement Copyleft Copyleft Debian Free Software Guidelines Debian Free Software Guidelines Definition of Free Cultural Works Definition of Free Cultural Works Free license Free license The Free Software Definition The Free Software Definition The Open Source Definition The Open Source Definition Open-source license Open-source license Permissive software license Permissive software license Public domain Public domain Digital rights management Digital rights management License proliferation License proliferation Mozilla software rebranding Mozilla software rebranding Proprietary device drivers Proprietary device drivers Proprietary firmware Proprietary firmware Proprietary software Proprietary software SCO/Linux controversies SCO/Linux controversies Software patents Software patents Software security Software security Tivoization Tivoization Trusted Computing Trusted Computing Forking Forking GNU Manifesto GNU Manifesto Microsoft Open Specification Promise Microsoft Open Specification Promise Open-core model Open-core model Open-source hardware Open-source hardware Shared Source Initiative Shared Source Initiative Source-available software Source-available software The Cathedral and the Bazaar The Cathedral and the Bazaar Revolution OS Revolution OS Portal Portal Category Category v v t t e e Statistical software Public domain Dataplot Dataplot Epi Info Epi Info CSPro CSPro X-12-ARIMA X-12-ARIMA Open-source ADMB ADMB DAP DAP gretl gretl jamovi jamovi JASP JASP JAGS JAGS JMulTi JMulTi Julia Julia Jupyter (Julia, Python, R) Jupyter GNU Octave GNU Octave OpenBUGS OpenBUGS Orange Orange PSPP PSPP Python (statsmodels, PyMC3, IPython, IDLE) Python PyMC3 IPython IDLE R (RStudio) R RStudio SageMath SageMath SimFiT SimFiT SOFA Statistics SOFA Statistics Stan Stan XLispStat XLispStat Freeware BV4.1 BV4.1 CumFreq CumFreq SegReg SegReg XploRe XploRe WinBUGS WinBUGS Commercial Cross-platform Data Desk Data Desk GAUSS GAUSS GraphPad InStat GraphPad InStat GraphPad Prism GraphPad Prism IBM SPSS Statistics SPSS IBM SPSS Modeler SPSS Modeler JMP JMP Maple Maple Mathcad Mathcad Mathematica Mathematica MATLAB MATLAB OxMetrics OxMetrics RATS RATS Revolution Analytics Revolution Analytics SAS SAS SmartPLS SmartPLS Stata Stata StatView StatView SUDAAN SUDAAN S-PLUS S-PLUS TSP TSP World Programming System (WPS) World Programming System Windows BMDP BMDP EViews EViews GenStat GenStat LIMDEP LIMDEP LISREL LISREL MedCalc MedCalc Microfit Microfit Minitab Minitab MLwiN MLwiN NCSS NCSS SHAZAM SHAZAM SigmaStat SigmaStat Statistica Statistica StatsDirect StatsDirect StatXact StatXact SYSTAT SYSTAT The Unscrambler The Unscrambler UNISTAT UNISTAT Excel Analyse-it Analyse-it UNISTAT for Excel UNISTAT XLfit XLfit RExcel RExcel Category Category Comparison Comparison v v t t e e Numerical-analysis software Advanced Simulation Library Advanced Simulation Library ADMB ADMB Chapel Chapel Euler Mathematical Toolbox Euler Mathematical Toolbox FreeFem++ FreeFem++ FreeMat FreeMat Genius Genius Gmsh Gmsh GNU Octave GNU Octave gretl gretl Julia Julia Jupyter (Julia, Python, R; IPython) Jupyter IPython MFEM MFEM OpenFOAM OpenFOAM Python Python R R SageMath SageMath Salome Salome ScicosLab ScicosLab Scilab Scilab X10 X10 Weka Weka Fortress Fortress DADiSP DADiSP FEATool Multiphysics FEATool Multiphysics GAUSS GAUSS LabVIEW LabVIEW Maple Maple Mathcad Mathcad Mathematica Mathematica MATLAB MATLAB Speakeasy Speakeasy VisSim VisSim Comparison Comparison Authority control databases FAST FAST FAST Germany Germany Germany United States United States United States France France France BnF data BnF data BnF data Czech Republic Czech Republic Czech Republic Czech Republic Israel Israel Israel IdRef IdRef IdRef https://en.wikipedia.org/w/index.php?title=Python_(programming_language)&oldid=1266145019 Categories Python (programming language) Python (programming language) Class-based programming languages Class-based programming languages Notebook interface Notebook interface Computer science in the Netherlands Computer science in the Netherlands Concurrent programming languages Concurrent programming languages Cross-platform free software Cross-platform free software Cross-platform software Cross-platform software Dutch inventions Dutch inventions Dynamically typed programming languages Dynamically typed programming languages Educational programming languages Educational programming languages High-level programming languages High-level programming languages Information technology in the Netherlands Information technology in the Netherlands Multi-paradigm programming languages Multi-paradigm programming languages Object-oriented programming languages Object-oriented programming languages Pattern matching programming languages Pattern matching programming languages Programming languages Programming languages Programming languages created in 1991 Programming languages created in 1991 Scripting languages Scripting languages Text-oriented programming languages Text-oriented programming languages Articles with short description Articles with short description Short description matches Wikidata Short description matches Wikidata Use dmy dates from November 2021 Use dmy dates from November 2021 Use American English from December 2024 Use American English from December 2024 All Wikipedia articles written in American English All Wikipedia articles written in American English Articles containing potentially dated statements from October 2024 Articles containing potentially dated statements from October 2024 All articles containing potentially dated statements All articles containing potentially dated statements Articles containing potentially dated statements from March 2024 Articles containing potentially dated statements from March 2024 Articles containing potentially dated statements from December 2022 Articles containing potentially dated statements from December 2022 Articles containing potentially dated statements from 2020 Articles containing potentially dated statements from 2020 Articles containing potentially dated statements from 2008 Articles containing potentially dated statements from 2008 Pages using Sister project links with wikidata namespace mismatch Pages using Sister project links with wikidata namespace mismatch Pages using Sister project links with hidden wikidata Pages using Sister project links with hidden wikidata Articles with example Python (programming language) code Articles with example Python (programming language) code This page was last edited on 30 December 2024, at 08:10 (UTC). (UTC) Text is available under the Creative Commons Attribution-ShareAlike 4.0 License; additional terms may apply. By using this site, you agree to the Terms of Use and Privacy Policy. Wikipedia\u00ae is a registered trademark of the Wikimedia Foundation, Inc., a non-profit organization. Creative Commons Attribution-ShareAlike 4.0 License Terms of Use Privacy Policy Wikimedia Foundation, Inc. Privacy policy Privacy policy About Wikipedia About Wikipedia Disclaimers Disclaimers Contact Wikipedia Contact Wikipedia Code of Conduct Code of Conduct Developers Developers Statistics Statistics Cookie statement Cookie statement Mobile view Mobile view",
    "https://en.wikipedia.org/wiki/JavaScript": "Jump to content Main menu Main page Main page Main page Contents Contents Contents Current events Current events Current events Random article Random article Random article About Wikipedia About Wikipedia About Wikipedia Contact us Contact us Contact us Help Help Help Learn to edit Learn to edit Learn to edit Community portal Community portal Community portal Recent changes Recent changes Recent changes Upload file Upload file Upload file Search Search Appearance Donate Donate Donate Create account Create account Create account Log in Log in Log in Personal tools Donate Donate Donate Create account Create account Create account Log in Log in Log in learn more learn more Contributions Contributions Contributions Talk Talk Talk Contents (Top) (Top) 1 History Toggle History subsection 1.1 Creation at Netscape 1.2 Adoption by Microsoft 1.3 The rise of JScript 1.4 Growth and standardization 1.5 Reaching maturity 1 History 1 History Toggle History subsection 1.1 Creation at Netscape 1.1 Creation at Netscape 1.1 Creation at Netscape 1.2 Adoption by Microsoft 1.2 Adoption by Microsoft 1.2 Adoption by Microsoft 1.3 The rise of JScript 1.3 The rise of JScript 1.3 The rise of JScript 1.4 Growth and standardization 1.4 Growth and standardization 1.4 Growth and standardization 1.5 Reaching maturity 1.5 Reaching maturity 1.5 Reaching maturity 2 Trademark 2 Trademark 2 Trademark 3 Website client-side usage Toggle Website client-side usage subsection 3.1 Examples of scripted behavior 3.2 Libraries and frameworks 3 Website client-side usage 3 Website client-side usage Toggle Website client-side usage subsection 3.1 Examples of scripted behavior 3.1 Examples of scripted behavior 3.1 Examples of scripted behavior 3.2 Libraries and frameworks 3.2 Libraries and frameworks 3.2 Libraries and frameworks 4 Other usage 4 Other usage 4 Other usage 5 Execution Toggle Execution subsection 5.1 JavaScript engine 5.2 Runtime system 5 Execution 5 Execution Toggle Execution subsection 5.1 JavaScript engine 5.1 JavaScript engine 5.1 JavaScript engine 5.2 Runtime system 5.2 Runtime system 5.2 Runtime system 6 Features Toggle Features subsection 6.1 Imperative and structured 6.2 Weakly typed 6.3 Dynamic 6.3.1 Typing 6.3.2 Run-time evaluation 6.4 Object-orientation (prototype-based) 6.4.1 Prototypes 6.4.2 Functions as object constructors 6.4.3 Functions as methods 6.5 Functional 6.5.1 Lexical closure 6.5.2 Anonymous function 6.6 Delegative 6.6.1 Functions as roles (Traits and Mixins) 6.6.2 Object composition and inheritance 6.7 Miscellaneous 6.7.1 Zero-based numbering 6.7.2 Variadic functions 6.7.3 Array and object literals 6.7.4 Regular expressions 6.7.5 Promises and Async/await 6.7.5.1 Promises 6.7.5.2 Async/await 6.8 Vendor-specific extensions 6 Features 6 Features Toggle Features subsection 6.1 Imperative and structured 6.1 Imperative and structured 6.1 Imperative and structured 6.2 Weakly typed 6.2 Weakly typed 6.2 Weakly typed 6.3 Dynamic 6.3.1 Typing 6.3.2 Run-time evaluation 6.3 Dynamic 6.3 Dynamic 6.3.1 Typing 6.3.1 Typing 6.3.1 Typing 6.3.2 Run-time evaluation 6.3.2 Run-time evaluation 6.3.2 Run-time evaluation 6.4 Object-orientation (prototype-based) 6.4.1 Prototypes 6.4.2 Functions as object constructors 6.4.3 Functions as methods 6.4 Object-orientation (prototype-based) 6.4 Object-orientation (prototype-based) 6.4.1 Prototypes 6.4.1 Prototypes 6.4.1 Prototypes 6.4.2 Functions as object constructors 6.4.2 Functions as object constructors 6.4.2 Functions as object constructors 6.4.3 Functions as methods 6.4.3 Functions as methods 6.4.3 Functions as methods 6.5 Functional 6.5.1 Lexical closure 6.5.2 Anonymous function 6.5 Functional 6.5 Functional 6.5.1 Lexical closure 6.5.1 Lexical closure 6.5.1 Lexical closure 6.5.2 Anonymous function 6.5.2 Anonymous function 6.5.2 Anonymous function 6.6 Delegative 6.6.1 Functions as roles (Traits and Mixins) 6.6.2 Object composition and inheritance 6.6 Delegative 6.6 Delegative 6.6.1 Functions as roles (Traits and Mixins) 6.6.1 Functions as roles (Traits and Mixins) 6.6.1 Functions as roles (Traits and Mixins) 6.6.2 Object composition and inheritance 6.6.2 Object composition and inheritance 6.6.2 Object composition and inheritance 6.7 Miscellaneous 6.7.1 Zero-based numbering 6.7.2 Variadic functions 6.7.3 Array and object literals 6.7.4 Regular expressions 6.7.5 Promises and Async/await 6.7.5.1 Promises 6.7.5.2 Async/await 6.7 Miscellaneous 6.7 Miscellaneous 6.7.1 Zero-based numbering 6.7.1 Zero-based numbering 6.7.1 Zero-based numbering 6.7.2 Variadic functions 6.7.2 Variadic functions 6.7.2 Variadic functions 6.7.3 Array and object literals 6.7.3 Array and object literals 6.7.3 Array and object literals 6.7.4 Regular expressions 6.7.4 Regular expressions 6.7.4 Regular expressions 6.7.5 Promises and Async/await 6.7.5.1 Promises 6.7.5.2 Async/await 6.7.5 Promises and Async/await 6.7.5 Promises and Async/await 6.7.5.1 Promises 6.7.5.1 Promises 6.7.5.1 Promises 6.7.5.2 Async/await 6.7.5.2 Async/await 6.7.5.2 Async/await 6.8 Vendor-specific extensions 6.8 Vendor-specific extensions 6.8 Vendor-specific extensions 7 Syntax Toggle Syntax subsection 7.1 Simple examples 7.2 More advanced example 7 Syntax 7 Syntax Toggle Syntax subsection 7.1 Simple examples 7.1 Simple examples 7.1 Simple examples 7.2 More advanced example 7.2 More advanced example 7.2 More advanced example 8 Security Toggle Security subsection 8.1 Cross-site scripting 8.2 Cross-site request forgery 8.3 Misplaced trust in the client 8.4 Misplaced trust in developers 8.5 Browser and plugin coding errors 8.6 Sandbox implementation errors 8.7 Hardware vulnerabilities 8 Security 8 Security Toggle Security subsection 8.1 Cross-site scripting 8.1 Cross-site scripting 8.1 Cross-site scripting 8.2 Cross-site request forgery 8.2 Cross-site request forgery 8.2 Cross-site request forgery 8.3 Misplaced trust in the client 8.3 Misplaced trust in the client 8.3 Misplaced trust in the client 8.4 Misplaced trust in developers 8.4 Misplaced trust in developers 8.4 Misplaced trust in developers 8.5 Browser and plugin coding errors 8.5 Browser and plugin coding errors 8.5 Browser and plugin coding errors 8.6 Sandbox implementation errors 8.6 Sandbox implementation errors 8.6 Sandbox implementation errors 8.7 Hardware vulnerabilities 8.7 Hardware vulnerabilities 8.7 Hardware vulnerabilities 9 Development tools 9 Development tools 9 Development tools 10 Related technologies Toggle Related technologies subsection 10.1 Java 10.2 JSON 10.3 Transpilers 10.4 WebAssembly 10 Related technologies 10 Related technologies Toggle Related technologies subsection 10.1 Java 10.1 Java 10.1 Java 10.2 JSON 10.2 JSON 10.2 JSON 10.3 Transpilers 10.3 Transpilers 10.3 Transpilers 10.4 WebAssembly 10.4 WebAssembly 10.4 WebAssembly 11 References 11 References 11 References 12 Sources 12 Sources 12 Sources 13 Further reading 13 Further reading 13 Further reading 14 External links 14 External links 14 External links Toggle the table of contents JavaScript JavaScript 109 languages Afrikaans Afrikaans Afrikaans \u00c6nglisc \u00c6nglisc \u00c6nglisc \u0627\u0644\u0639\u0631\u0628\u064a\u0629 \u0627\u0644\u0639\u0631\u0628\u064a\u0629 \u0627\u0644\u0639\u0631\u0628\u064a\u0629 Aragon\u00e9s Aragon\u00e9s Aragon\u00e9s Asturianu Asturianu Asturianu Az\u0259rbaycanca Az\u0259rbaycanca Az\u0259rbaycanca \u062a\u06c6\u0631\u06a9\u062c\u0647 \u062a\u06c6\u0631\u06a9\u062c\u0647 \u062a\u06c6\u0631\u06a9\u062c\u0647 \u09ac\u09be\u0982\u09b2\u09be \u09ac\u09be\u0982\u09b2\u09be \u09ac\u09be\u0982\u09b2\u09be \u95a9\u5357\u8a9e / B\u00e2n-l\u00e2m-g\u00fa \u95a9\u5357\u8a9e / B\u00e2n-l\u00e2m-g\u00fa \u95a9\u5357\u8a9e / B\u00e2n-l\u00e2m-g\u00fa \u0411\u0435\u043b\u0430\u0440\u0443\u0441\u043a\u0430\u044f \u0411\u0435\u043b\u0430\u0440\u0443\u0441\u043a\u0430\u044f \u0411\u0435\u043b\u0430\u0440\u0443\u0441\u043a\u0430\u044f \u0411\u0435\u043b\u0430\u0440\u0443\u0441\u043a\u0430\u044f (\u0442\u0430\u0440\u0430\u0448\u043a\u0435\u0432\u0456\u0446\u0430) \u0411\u0435\u043b\u0430\u0440\u0443\u0441\u043a\u0430\u044f (\u0442\u0430\u0440\u0430\u0448\u043a\u0435\u0432\u0456\u0446\u0430) \u0411\u0435\u043b\u0430\u0440\u0443\u0441\u043a\u0430\u044f (\u0442\u0430\u0440\u0430\u0448\u043a\u0435\u0432\u0456\u0446\u0430) \u0411\u044a\u043b\u0433\u0430\u0440\u0441\u043a\u0438 \u0411\u044a\u043b\u0433\u0430\u0440\u0441\u043a\u0438 \u0411\u044a\u043b\u0433\u0430\u0440\u0441\u043a\u0438 Brezhoneg Brezhoneg Brezhoneg Catal\u00e0 Catal\u00e0 Catal\u00e0 \u0427\u04d1\u0432\u0430\u0448\u043b\u0430 \u0427\u04d1\u0432\u0430\u0448\u043b\u0430 \u0427\u04d1\u0432\u0430\u0448\u043b\u0430 \u010ce\u0161tina \u010ce\u0161tina \u010ce\u0161tina Dansk Dansk Dansk Deutsch Deutsch Deutsch Eesti Eesti Eesti \u0395\u03bb\u03bb\u03b7\u03bd\u03b9\u03ba\u03ac \u0395\u03bb\u03bb\u03b7\u03bd\u03b9\u03ba\u03ac \u0395\u03bb\u03bb\u03b7\u03bd\u03b9\u03ba\u03ac Espa\u00f1ol Espa\u00f1ol Espa\u00f1ol Esperanto Esperanto Esperanto Euskara Euskara Euskara \u0641\u0627\u0631\u0633\u06cc \u0641\u0627\u0631\u0633\u06cc \u0641\u0627\u0631\u0633\u06cc Fran\u00e7ais Fran\u00e7ais Fran\u00e7ais Galego Galego Galego \u5ba2\u5bb6\u8a9e / Hak-k\u00e2-ng\u00ee \u5ba2\u5bb6\u8a9e / Hak-k\u00e2-ng\u00ee \u5ba2\u5bb6\u8a9e / Hak-k\u00e2-ng\u00ee \ud55c\uad6d\uc5b4 \ud55c\uad6d\uc5b4 \ud55c\uad6d\uc5b4 Hausa Hausa Hausa \u0540\u0561\u0575\u0565\u0580\u0565\u0576 \u0540\u0561\u0575\u0565\u0580\u0565\u0576 \u0540\u0561\u0575\u0565\u0580\u0565\u0576 \u0939\u093f\u0928\u094d\u0926\u0940 \u0939\u093f\u0928\u094d\u0926\u0940 \u0939\u093f\u0928\u094d\u0926\u0940 Hrvatski Hrvatski Hrvatski Ido Ido Ido Bahasa Indonesia Bahasa Indonesia Bahasa Indonesia Interlingua Interlingua Interlingua \u00cdslenska \u00cdslenska \u00cdslenska Italiano Italiano Italiano \u05e2\u05d1\u05e8\u05d9\u05ea \u05e2\u05d1\u05e8\u05d9\u05ea \u05e2\u05d1\u05e8\u05d9\u05ea Jawa Jawa Jawa \u10e5\u10d0\u10e0\u10d7\u10e3\u10da\u10d8 \u10e5\u10d0\u10e0\u10d7\u10e3\u10da\u10d8 \u10e5\u10d0\u10e0\u10d7\u10e3\u10da\u10d8 \u049a\u0430\u0437\u0430\u049b\u0448\u0430 \u049a\u0430\u0437\u0430\u049b\u0448\u0430 \u049a\u0430\u0437\u0430\u049b\u0448\u0430 Kiswahili Kiswahili Kiswahili Kurd\u00ee Kurd\u00ee Kurd\u00ee \u041a\u044b\u0440\u0433\u044b\u0437\u0447\u0430 \u041a\u044b\u0440\u0433\u044b\u0437\u0447\u0430 \u041a\u044b\u0440\u0433\u044b\u0437\u0447\u0430 Latina Latina Latina Latvie\u0161u Latvie\u0161u Latvie\u0161u Lietuvi\u0173 Lietuvi\u0173 Lietuvi\u0173 Lingua Franca Nova Lingua Franca Nova Lingua Franca Nova Lombard Lombard Lombard Magyar Magyar Magyar \u041c\u0430\u043a\u0435\u0434\u043e\u043d\u0441\u043a\u0438 \u041c\u0430\u043a\u0435\u0434\u043e\u043d\u0441\u043a\u0438 \u041c\u0430\u043a\u0435\u0434\u043e\u043d\u0441\u043a\u0438 \u0d2e\u0d32\u0d2f\u0d3e\u0d33\u0d02 \u0d2e\u0d32\u0d2f\u0d3e\u0d33\u0d02 \u0d2e\u0d32\u0d2f\u0d3e\u0d33\u0d02 \u092e\u0930\u093e\u0920\u0940 \u092e\u0930\u093e\u0920\u0940 \u092e\u0930\u093e\u0920\u0940 \u0645\u0635\u0631\u0649 \u0645\u0635\u0631\u0649 \u0645\u0635\u0631\u0649 \u0645\u0627\u0632\u0650\u0631\u0648\u0646\u06cc \u0645\u0627\u0632\u0650\u0631\u0648\u0646\u06cc \u0645\u0627\u0632\u0650\u0631\u0648\u0646\u06cc Bahasa Melayu Bahasa Melayu Bahasa Melayu \u95a9\u6771\u8a9e / M\u00ecng-d\u0115\u0324ng-ng\u1e73\u0304 \u95a9\u6771\u8a9e / M\u00ecng-d\u0115\u0324ng-ng\u1e73\u0304 \u95a9\u6771\u8a9e / M\u00ecng-d\u0115\u0324ng-ng\u1e73\u0304 \u041c\u043e\u043d\u0433\u043e\u043b \u041c\u043e\u043d\u0433\u043e\u043b \u041c\u043e\u043d\u0433\u043e\u043b \u1019\u103c\u1014\u103a\u1019\u102c\u1018\u102c\u101e\u102c \u1019\u103c\u1014\u103a\u1019\u102c\u1018\u102c\u101e\u102c \u1019\u103c\u1014\u103a\u1019\u102c\u1018\u102c\u101e\u102c Nederlands Nederlands Nederlands \u0928\u0947\u092a\u093e\u0932\u0940 \u0928\u0947\u092a\u093e\u0932\u0940 \u0928\u0947\u092a\u093e\u0932\u0940 \u65e5\u672c\u8a9e \u65e5\u672c\u8a9e \u65e5\u672c\u8a9e Nordfriisk Nordfriisk Nordfriisk Norsk bokm\u00e5l Norsk bokm\u00e5l Norsk bokm\u00e5l \u0b13\u0b21\u0b3c\u0b3f\u0b06 \u0b13\u0b21\u0b3c\u0b3f\u0b06 \u0b13\u0b21\u0b3c\u0b3f\u0b06 O\u02bbzbekcha / \u045e\u0437\u0431\u0435\u043a\u0447\u0430 O\u02bbzbekcha / \u045e\u0437\u0431\u0435\u043a\u0447\u0430 O\u02bbzbekcha / \u045e\u0437\u0431\u0435\u043a\u0447\u0430 \u067e\u0646\u062c\u0627\u0628\u06cc \u067e\u0646\u062c\u0627\u0628\u06cc \u067e\u0646\u062c\u0627\u0628\u06cc \u1015\u1021\u102d\u102f\u101d\u103a\u108f\u1018\u102c\u108f\u101e\u102c\u108f \u1015\u1021\u102d\u102f\u101d\u103a\u108f\u1018\u102c\u108f\u101e\u102c\u108f \u1015\u1021\u102d\u102f\u101d\u103a\u108f\u1018\u102c\u108f\u101e\u102c\u108f \u1797\u17b6\u179f\u17b6\u1781\u17d2\u1798\u17c2\u179a \u1797\u17b6\u179f\u17b6\u1781\u17d2\u1798\u17c2\u179a \u1797\u17b6\u179f\u17b6\u1781\u17d2\u1798\u17c2\u179a Pinayuanan Pinayuanan Pinayuanan Polski Polski Polski Portugu\u00eas Portugu\u00eas Portugu\u00eas Qaraqalpaqsha Qaraqalpaqsha Qaraqalpaqsha Rom\u00e2n\u0103 Rom\u00e2n\u0103 Rom\u00e2n\u0103 Runa Simi Runa Simi Runa Simi \u0420\u0443\u0441\u0441\u043a\u0438\u0439 \u0420\u0443\u0441\u0441\u043a\u0438\u0439 \u0420\u0443\u0441\u0441\u043a\u0438\u0439 \u0421\u0430\u0445\u0430 \u0442\u044b\u043b\u0430 \u0421\u0430\u0445\u0430 \u0442\u044b\u043b\u0430 \u0421\u0430\u0445\u0430 \u0442\u044b\u043b\u0430 Scots Scots Scots Shqip Shqip Shqip \u0dc3\u0dd2\u0d82\u0dc4\u0dbd \u0dc3\u0dd2\u0d82\u0dc4\u0dbd \u0dc3\u0dd2\u0d82\u0dc4\u0dbd Simple English Simple English Simple English Sloven\u010dina Sloven\u010dina Sloven\u010dina Sloven\u0161\u010dina Sloven\u0161\u010dina Sloven\u0161\u010dina Soomaaliga Soomaaliga Soomaaliga \u06a9\u0648\u0631\u062f\u06cc \u06a9\u0648\u0631\u062f\u06cc \u06a9\u0648\u0631\u062f\u06cc \u0421\u0440\u043f\u0441\u043a\u0438 / srpski \u0421\u0440\u043f\u0441\u043a\u0438 / srpski \u0421\u0440\u043f\u0441\u043a\u0438 / srpski Srpskohrvatski / \u0441\u0440\u043f\u0441\u043a\u043e\u0445\u0440\u0432\u0430\u0442\u0441\u043a\u0438 Srpskohrvatski / \u0441\u0440\u043f\u0441\u043a\u043e\u0445\u0440\u0432\u0430\u0442\u0441\u043a\u0438 Srpskohrvatski / \u0441\u0440\u043f\u0441\u043a\u043e\u0445\u0440\u0432\u0430\u0442\u0441\u043a\u0438 Sunda Sunda Sunda Suomi Suomi Suomi Svenska Svenska Svenska Tagalog Tagalog Tagalog \u0ba4\u0bae\u0bbf\u0bb4\u0bcd \u0ba4\u0bae\u0bbf\u0bb4\u0bcd \u0ba4\u0bae\u0bbf\u0bb4\u0bcd Taqbaylit Taqbaylit Taqbaylit \u0422\u0430\u0442\u0430\u0440\u0447\u0430 / tatar\u00e7a \u0422\u0430\u0442\u0430\u0440\u0447\u0430 / tatar\u00e7a \u0422\u0430\u0442\u0430\u0440\u0447\u0430 / tatar\u00e7a \u0c24\u0c46\u0c32\u0c41\u0c17\u0c41 \u0c24\u0c46\u0c32\u0c41\u0c17\u0c41 \u0c24\u0c46\u0c32\u0c41\u0c17\u0c41 Tetun Tetun Tetun \u0e44\u0e17\u0e22 \u0e44\u0e17\u0e22 \u0e44\u0e17\u0e22 T\u00fcrk\u00e7e T\u00fcrk\u00e7e T\u00fcrk\u00e7e T\u00fcrkmen\u00e7e T\u00fcrkmen\u00e7e T\u00fcrkmen\u00e7e Basa Ugi Basa Ugi Basa Ugi \u0423\u043a\u0440\u0430\u0457\u043d\u0441\u044c\u043a\u0430 \u0423\u043a\u0440\u0430\u0457\u043d\u0441\u044c\u043a\u0430 \u0423\u043a\u0440\u0430\u0457\u043d\u0441\u044c\u043a\u0430 \u0627\u0631\u062f\u0648 \u0627\u0631\u062f\u0648 \u0627\u0631\u062f\u0648 V\u00e8neto V\u00e8neto V\u00e8neto Ti\u1ebfng Vi\u1ec7t Ti\u1ebfng Vi\u1ec7t Ti\u1ebfng Vi\u1ec7t \u6587\u8a00 \u6587\u8a00 \u6587\u8a00 \u5434\u8bed \u5434\u8bed \u5434\u8bed \u7cb5\u8a9e \u7cb5\u8a9e \u7cb5\u8a9e \u017demait\u0117\u0161ka \u017demait\u0117\u0161ka \u017demait\u0117\u0161ka \u4e2d\u6587 \u4e2d\u6587 \u4e2d\u6587 Edit links Edit links Article Article Article Talk Talk Talk English Read Read Read View source View source View source View history View history View history Tools Read Read Read View source View source View source View history View history View history What links here What links here What links here Related changes Related changes Related changes Upload file Upload file Upload file Special pages Special pages Special pages Permanent link Permanent link Permanent link Page information Page information Page information Cite this page Cite this page Cite this page Get shortened URL Get shortened URL Get shortened URL Download QR code Download QR code Download QR code Download as PDF Download as PDF Download as PDF Printable version Printable version Printable version Wikimedia Commons Wikimedia Commons Wikimedia Commons MediaWiki MediaWiki MediaWiki Wikibooks Wikibooks Wikibooks Wikifunctions Wikifunctions Wikifunctions Wikiversity Wikiversity Wikiversity Wikidata item Wikidata item Wikidata item Page version status This is an accepted version of this page latest accepted revision reviewed Java (programming language) Javanese script ECMAScript Paradigm Multi-paradigm event-driven functional imperative procedural object-oriented Designed by Brendan Eich Netscape ECMAScript ; 29 years ago (1995-12-04) 1995-12-04 [1] [ ] Stable release [2] [ ] ; 7 months ago (June 2024) June 2024 Preview release [3] [ ] ; 9 months ago (27 March 2024) 27 March 2024 Typing discipline Dynamic weak duck Memory management Garbage collection Filename extensions .js .cjs .mjs[4] [4] [ ] ecma-international.org/publications-and-standards/standards/ecma-262/ ecma-international.org/publications-and-standards/standards/ecma-262/ implementations V8 JavaScriptCore SpiderMonkey Chakra Java [5] [ ] [6] [ ] Scheme [6] [ ] Self [7] [ ] AWK [8] [ ] HyperTalk [9] [ ] ActionScript ArkTS AssemblyScript CoffeeScript Dart Haxe JS++ Opa TypeScript JavaScript at Wikibooks JavaScript JavaScript (/\u02c8d\u0292\u0251\u02d0v\u0259skr\u026apt/), often abbreviated as JS, is a programming language and core technology of the Web, alongside HTML and CSS. 99% of websites use JavaScript on the client side for webpage behavior.[10] /\u02c8d\u0292\u0251\u02d0v\u0259skr\u026apt/ /\u02c8d\u0292\u0251\u02d0v\u0259skr\u026apt/ /\u02c8d\u0292\u0251\u02d0v\u0259skr\u026apt/ \u02c8d\u0292\u0251\u02d0v\u0259skr\u026apt \u02c8 d\u0292 \u0251\u02d0 v \u0259 s k r \u026a p t programming language the Web HTML CSS websites client webpage [10] [ ] Web browsers have a dedicated JavaScript engine that executes the client code. These engines are also utilized in some servers and a variety of apps. The most popular runtime system for non-browser usage is Node.js. Web browsers JavaScript engine code servers apps runtime system Node.js JavaScript is a high-level, often just-in-time compiled language that conforms to the ECMAScript standard.[11] It has dynamic typing, prototype-based object-orientation, and first-class functions. It is multi-paradigm, supporting event-driven, functional, and imperative programming styles. It has application programming interfaces (APIs) for working with text, dates, regular expressions, standard data structures, and the Document Object Model (DOM). high-level just-in-time compiled ECMAScript [11] [ ] dynamic typing prototype-based object-orientation first-class functions multi-paradigm event-driven functional imperative programming styles application programming interfaces regular expressions data structures Document Object Model The ECMAScript standard does not include any input/output (I/O), such as networking, storage, or graphics facilities. In practice, the web browser or other runtime system provides JavaScript APIs for I/O. input/output networking storage graphics Although Java and JavaScript are similar in name and syntax, the two languages are distinct and differ greatly in design. Java syntax History Creation at Netscape The first popular web browser with a graphical user interface, Mosaic, was released in 1993. Accessible to non-technical people, it played a prominent role in the rapid growth of the early World Wide Web.[12] The lead developers of Mosaic then founded the Netscape corporation, which released a more polished browser, Netscape Navigator, in 1994. This quickly became the most-used.[13] web browser graphical user interface Mosaic World Wide Web [12] [ ] Netscape Netscape Navigator [13] [ ] During these formative years of the Web, web pages could only be static, lacking the capability for dynamic behavior after the page was loaded in the browser. There was a desire in the flourishing web development scene to remove this limitation, so in 1995, Netscape decided to add a programming language to Navigator. They pursued two routes to achieve this: collaborating with Sun Microsystems to embed the Java language, while also hiring Brendan Eich to embed the Scheme language.[6] web pages programming language Sun Microsystems Java Brendan Eich Scheme [6] [ ] The goal was a \"language for the masses\",[14] \"to help nonprogrammers create dynamic, interactive Web sites\".[15] Netscape management soon decided that the best option was for Eich to devise a new language, with syntax similar to Java and less like Scheme or other extant scripting languages.[5][6] Although the new language and its interpreter implementation were called LiveScript when first shipped as part of a Navigator beta in September 1995, the name was changed to JavaScript for the official release in December.[6][1][16][17] [14] [ ] Web sites [15] [ ] scripting languages [5] [ ] [6] [ ] interpreter beta [6] [ ] [1] [ ] [16] [ ] [17] [ ] The choice of the JavaScript name has caused confusion, implying that it is directly related to Java. At the time, the dot-com boom had begun and Java was a popular new language, so Eich considered the JavaScript name a marketing ploy by Netscape.[14] dot-com boom [14] [ ] Adoption by Microsoft Microsoft debuted Internet Explorer in 1995, leading to a browser war with Netscape. On the JavaScript front, Microsoft created its own interpreter called JScript.[18] Microsoft Internet Explorer browser war interpreter JScript [18] [ ] Microsoft first released JScript in 1996, alongside initial support for CSS and extensions to HTML. Each of these implementations was noticeably different from their counterparts in Netscape Navigator.[19][20] These differences made it difficult for developers to make their websites work well in both browsers, leading to widespread use of \"best viewed in Netscape\" and \"best viewed in Internet Explorer\" logos for several years.[19][21] CSS HTML implementations Netscape Navigator [19] [ ] [20] [ ] [19] [ ] [21] [ ] The rise of JScript Brendan Eich later said of this period: \"It's still kind of a sidekick language. It's considered slow or annoying. People do pop-ups or those scrolling messages in the old status bar at the bottom of your old browser.\"[14] Brendan Eich sidekick pop-ups status bar browser [14] [ ] In November 1996, Netscape submitted JavaScript to Ecma International, as the starting point for a standard specification that all browser vendors could conform to. This led to the official release of the first ECMAScript language specification in June 1997. Netscape Ecma International ECMAScript The standards process continued for a few years, with the release of ECMAScript 2 in June 1998 and ECMAScript 3 in December 1999. Work on ECMAScript 4 began in 2000.[18] [18] [ ] However, the effort to fully standardize the language was undermined by Microsoft gaining an increasingly dominant position in the browser market. By the early 2000s, Internet Explorer's market share reached 95%.[22] This meant that JScript became the de facto standard for client-side scripting on the Web. Microsoft Internet Explorer [22] [ ] JScript client-side scripting Microsoft initially participated in the standards process and implemented some proposals in its JScript language, but eventually it stopped collaborating on ECMA work. Thus ECMAScript 4 was mothballed. Growth and standardization During the period of Internet Explorer dominance in the early 2000s, client-side scripting was stagnant. This started to change in 2004, when the successor of Netscape, Mozilla, released the Firefox browser. Firefox was well received by many, taking significant market share from Internet Explorer.[23] Internet Explorer Mozilla Firefox [23] [ ] In 2005, Mozilla joined ECMA International, and work started on the ECMAScript for XML (E4X) standard. This led to Mozilla working jointly with Macromedia (later acquired by Adobe Systems), who were implementing E4X in their ActionScript 3 language, which was based on an ECMAScript 4 draft. The goal became standardizing ActionScript 3 as the new ECMAScript 4. To this end, Adobe Systems released the Tamarin implementation as an open source project. However, Tamarin and ActionScript 3 were too different from established client-side scripting, and without cooperation from Microsoft, ECMAScript 4 never reached fruition. ECMAScript for XML Macromedia Adobe Systems Tamarin open source Microsoft Meanwhile, very important developments were occurring in open-source communities not affiliated with ECMA work. In 2005, Jesse James Garrett released a white paper in which he coined the term Ajax and described a set of technologies, of which JavaScript was the backbone, to create web applications where data can be loaded in the background, avoiding the need for full page reloads. This sparked a renaissance period of JavaScript, spearheaded by open-source libraries and the communities that formed around them. Many new libraries were created, including jQuery, Prototype, Dojo Toolkit, and MooTools. Jesse James Garrett Ajax web applications jQuery Prototype Dojo Toolkit MooTools Google debuted its Chrome browser in 2008, with the V8 JavaScript engine that was faster than its competition.[24][25] The key innovation was just-in-time compilation (JIT),[26] so other browser vendors needed to overhaul their engines for JIT.[27] Google Chrome V8 [24] [ ] [25] [ ] just-in-time compilation [26] [ ] [27] [ ] In July 2008, these disparate parties came together for a conference in Oslo. This led to the eventual agreement in early 2009 to combine all relevant work and drive the language forward. The result was the ECMAScript 5 standard, released in December 2009. Oslo Reaching maturity Ambitious work on the language continued for several years, culminating in an extensive collection of additions and refinements being formalized with the publication of ECMAScript 6 in 2015.[28] ECMAScript 6 [28] [ ] The creation of Node.js in 2009 by Ryan Dahl sparked a significant increase in the usage of JavaScript outside of web browsers. Node combines the V8 engine, an event loop, and I/O APIs, thereby providing a stand-alone JavaScript runtime system.[29][30] As of 2018, Node had been used by millions of developers,[31] and npm had the most modules of any package manager in the world.[32] Node.js Ryan Dahl V8 event loop I/O APIs [29] [ ] [30] [ ] [31] [ ] npm package manager [32] [ ] The ECMAScript draft specification is currently maintained openly on GitHub,[33] and editions are produced via regular annual snapshots.[33] Potential revisions to the language are vetted through a comprehensive proposal process.[34][35] Now, instead of edition numbers, developers check the status of upcoming features individually.[33] GitHub [33] [ ] [33] [ ] [34] [ ] [35] [ ] [33] [ ] The current JavaScript ecosystem has many libraries and frameworks, established programming practices, and substantial usage of JavaScript outside of web browsers.[17] Plus, with the rise of single-page applications and other JavaScript-heavy websites, several transpilers have been created to aid the development process.[36] libraries frameworks [17] [ ] single-page applications transpilers [36] [ ] Trademark \"JavaScript\" is a trademark of Oracle Corporation in the United States.[37][38] The trademark was originally issued to Sun Microsystems on 6 May 1997, and was transferred to Oracle when they acquired Sun in 2009.[39] trademark Oracle Corporation [37] [ ] [38] [ ] Sun Microsystems [39] [ ] A letter was circulated in September 2024, spearheaded by Ryan Dahl, calling on Oracle to free the JavaScript trademark.[40] Brendan Eich the original creator of JavaScript, was among the over 14,000 signatories who supported the initiative. Ryan Dahl [40] [ ] Brendan Eich Website client-side usage JavaScript is the dominant client-side scripting language of the Web, with 99% of all websites using it for this purpose.[10] Scripts are embedded in or included from HTML documents and interact with the DOM. client-side scripting language websites [10] [ ] HTML DOM All major web browsers have a built-in JavaScript engine that executes the code on the user's device. web browsers JavaScript engine code Examples of scripted behavior Loading new web page content without reloading the page, via Ajax or a WebSocket. For example, users of social media can send and receive messages without leaving the current page. web page Ajax WebSocket users social media Web page animations, such as fading objects in and out, resizing, and moving them. Playing browser games. browser games Controlling the playback of streaming media. playback streaming media Generating pop-up ads or alert boxes. pop-up ads Validating input values of a web form before the data is sent to a web server. Validating web form web server Logging data about the user's behavior then sending it to a server. The website owner can use this data for analytics, ad tracking, and personalization. analytics ad tracking personalization Redirecting a user to another page. Storing and retrieving data on the user's device, via the storage or IndexedDB standards. storage IndexedDB Libraries and frameworks Over 80% of websites use a third-party JavaScript library or web framework as part of their client-side scripting.[41] library web framework [41] [ ] jQuery is by far the most-used.[41] Other notable ones include Angular, Bootstrap, Lodash, Modernizr, React, Underscore, and Vue.[41] Multiple options can be used in conjunction, such as jQuery and Bootstrap.[42] jQuery [41] [ ] Angular Bootstrap Lodash Modernizr React Underscore Vue [41] [ ] [42] [ ] However, the term \"Vanilla JS\" was coined for websites not using any libraries or frameworks at all, instead relying entirely on standard JavaScript functionality.[43] [43] [ ] Other usage The use of JavaScript has expanded beyond its web browser roots. JavaScript engines are now embedded in a variety of other software systems, both for server-side website deployments and non-browser applications. web browser JavaScript engines server-side applications Initial attempts at promoting server-side JavaScript usage were Netscape Enterprise Server and Microsoft's Internet Information Services,[44][45] but they were small niches.[46] Server-side usage eventually started to grow in the late 2000s, with the creation of Node.js and other approaches.[46] Netscape Enterprise Server Microsoft Internet Information Services [44] [ ] [45] [ ] [46] [ ] Node.js other approaches [46] [ ] Electron, Cordova, React Native, and other application frameworks have been used to create many applications with behavior implemented in JavaScript. Other non-browser applications include Adobe Acrobat support for scripting PDF documents[47] and GNOME Shell extensions written in JavaScript.[48] Electron Cordova React Native application frameworks Adobe Acrobat PDF [47] [ ] GNOME Shell [48] [ ] JavaScript has been used in some embedded systems, usually by leveraging Node.js.[49][50][51] embedded systems [49] [ ] [50] [ ] [51] [ ] Execution JavaScript engine JavaScript engine [edit] [ edit ] A JavaScript engine is a software component that executes JavaScript code. The first JavaScript engines were mere interpreters, but all relevant modern engines use just-in-time compilation for improved performance.[52] JavaScript engine software component code engines interpreters just-in-time compilation [52] [ ] JavaScript engines are typically developed by web browser vendors, and every major browser has one. In a browser, the JavaScript engine runs in concert with the rendering engine via the Document Object Model and Web IDL bindings.[53] However, the use of JavaScript engines is not limited to browsers; for example, the V8 engine is a core component of the Node.js runtime system.[54] web browser rendering engine Document Object Model Web IDL [53] [ ] V8 engine Node.js runtime system [54] [ ] ECMAScript implementations WebAssembly sandbox [55] [ ] [54] [ ] Runtime system A JavaScript engine must be embedded within a runtime system (such as a web browser or a standalone system) to enable scripts to interact with the broader environment. The runtime system includes the necessary APIs for input/output operations, such as networking, storage, and graphics, and provides the ability to import scripts. runtime system web browser input/output networking storage graphics JavaScript is a single-threaded language. The runtime processes messages from a queue one at a time, and it calls a function associated with each new message, creating a call stack frame with the function's arguments and local variables. The call stack shrinks and grows based on the function's needs. When the call stack is empty upon function completion, JavaScript proceeds to the next message in the queue. This is called the event loop, described as \"run to completion\" because each message is fully processed before the next message is considered. However, the language's concurrency model describes the event loop as non-blocking: program I/O is performed using events and callback functions. This means, for example, that JavaScript can process a mouse click while waiting for a database query to return information.[56] threaded messages queue function call stack arguments local variables event loop concurrency model non-blocking events callback functions [56] [ ] The notable standalone runtimes are Node.js, Deno, and Bun. Node.js Deno Bun Features The following features are common to all conforming ECMAScript implementations unless explicitly specified otherwise. Imperative and structured Structured programming JavaScript supports much of the structured programming syntax from C (e.g., if statements, while loops, switch statements, do while loops, etc.). One partial exception is scoping: originally JavaScript only had function scoping with var; block scoping was added in ECMAScript 2015 with the keywords let and const. Like C, JavaScript makes a distinction between expressions and statements. One syntactic difference from C is automatic semicolon insertion, which allow semicolons (which terminate statements) to be omitted.[57] structured programming C scoping function scoping block scoping const expressions statements automatic semicolon insertion [57] [ ] Weakly typed Weakly typed JavaScript is weakly typed, which means certain types are implicitly cast depending on the operation used.[58] weakly typed [58] [ ] The binary + operator casts both operands to a string unless both operands are numbers. This is because the addition operator doubles as a concatenation operator The binary - operator always casts both operands to a number Both unary operators (+, -) always cast the operand to a number. However, + always casts to Number (binary64) while - preserves BigInt (integer)[59] binary64 integer [59] [ ] Values are cast to strings like the following:[58] [58] [ ] Strings are left as-is Numbers are converted to their string representation Arrays have their elements cast to strings after which they are joined by commas (,) Other objects are converted to the string [object Object] where Object is the name of the constructor of the object Values are cast to numbers by casting to strings and then casting the strings to numbers. These processes can be modified by defining toString and valueOf functions on the prototype for string and number casting respectively. prototype JavaScript has received criticism for the way it implements these conversions as the complexity of the rules can be mistaken for inconsistency.[60][58] For example, when adding a number to a string, the number will be cast to a string before performing concatenation, but when subtracting a number from a string, the string is cast to a number before performing subtraction. [60] [ ] [58] [ ] NaN Often also mentioned is {} + [] resulting in 0 (number). This is misleading: the {} is interpreted as an empty code block instead of an empty object, and the empty array is cast to a number by the remaining unary + operator. If the expression is wrapped in parentheses - ({} + []) \u2013 the curly brackets are interpreted as an empty object and the result of the expression is \"[object Object]\" as expected.[58] [58] [ ] Dynamic Dynamic Programming Typing Dynamic typing JavaScript is dynamically typed like most other scripting languages. A type is associated with a value rather than an expression. For example, a variable initially bound to a number may be reassigned to a string.[61] JavaScript supports various ways to test the type of objects, including duck typing.[62] dynamically typed scripting languages type value variable string [61] [ ] duck typing [62] [ ] Run-time evaluation eval JavaScript includes an eval function that can execute statements provided as strings at run-time. eval Object-orientation (prototype-based) Prototypal inheritance in JavaScript is described by Douglas Crockford as: Douglas Crockford You make prototype objects, and then ... make new instances. Objects are mutable in JavaScript, so we can augment the new instances, giving them new fields and methods. These can then act as prototypes for even newer objects. We don't need classes to make lots of similar objects... Objects inherit from objects. What could be more object oriented than that?[63] [63] [ ] In JavaScript, an object is an associative array, augmented with a prototype (see below); each key provides the name for an object property, and there are two syntactical ways to specify such a name: dot notation (obj.x = 10) and bracket notation (obj['x'] = 10). A property may be added, rebound, or deleted at run-time. Most properties of an object (and any property that belongs to an object's prototype inheritance chain) can be enumerated using a for...in loop. object associative array property properties Prototypes Prototype-based programming JavaScript uses prototypes where many other object-oriented languages use classes for inheritance.[64] It is possible to simulate many class-based features with prototypes in JavaScript.[65] prototypes classes inheritance [64] [ ] [65] [ ] Functions as object constructors Functions double as object constructors, along with their typical role. Prefixing a function call with new will create an instance of a prototype, inheriting properties and methods from the constructor (including properties from the Object prototype).[66] ECMAScript 5 offers the Object.create method, allowing explicit creation of an instance without automatically inheriting from the Object prototype (older environments can assign the prototype to null).[67] The constructor's prototype property determines the object used for the new object's internal prototype. New methods can be added by modifying the prototype of the function used as a constructor. JavaScript's built-in constructors, such as Array or Object, also have prototypes that can be modified. While it is possible to modify the Object prototype, it is generally considered bad practice because most objects in JavaScript will inherit methods and properties from the Object prototype, and they may not expect the prototype to be modified.[68] [66] [ ] [67] [ ] [68] [ ] Functions as methods Method (computer science) Unlike in many object-oriented languages, in JavaScript there is no distinction between a function definition and a method definition. Rather, the distinction occurs during function calling. When a function is called as a method of an object, the function's local this keyword is bound to that object for that invocation. method Functional Functional programming JavaScript functions are first-class; a function is considered to be an object.[69] As such, a function may have properties and methods, such as .call() and .bind().[70] functions first-class [69] [ ] [70] [ ] Lexical closure Closure (computer programming) A nested function is a function defined within another function. It is created each time the outer function is invoked. In addition, each nested function forms a lexical closure: the lexical scope of the outer function (including any constant, local variable, or argument value) becomes part of the internal state of each inner function object, even after execution of the outer function concludes.[71] lexical closure lexical scope [71] [ ] Anonymous function Anonymous function JavaScript also supports anonymous functions. anonymous functions Delegative Delegation (object-oriented programming) JavaScript supports implicit and explicit delegation. delegation Functions as roles (Traits and Mixins) Role-oriented programming Traits (computer science) Mixin JavaScript natively supports various function-based implementations of Role[72] patterns like Traits[73][74] and Mixins.[75] Such a function defines additional behavior by at least one method bound to the this keyword within its function body. A Role then has to be delegated explicitly via call or apply to objects that need to feature additional behavior that is not shared via the prototype chain. Role [72] [ ] Traits [73] [ ] [74] [ ] Mixins [75] [ ] Object composition and inheritance Whereas explicit function-based delegation does cover composition in JavaScript, implicit delegation already happens every time the prototype chain is walked in order to, e.g., find a method that might be related to but is not directly owned by an object. Once the method is found it gets called within this object's context. Thus inheritance in JavaScript is covered by a delegation automatism that is bound to the prototype property of constructor functions. composition inheritance Miscellaneous Zero-based numbering JavaScript is a zero-index language. zero-index Variadic functions Variadic function An indefinite number of parameters can be passed to a function. The function can access them through formal parameters and also through the local arguments object. Variadic functions can also be created by using the bind method. formal parameters Variadic functions bind Array and object literals Associative arrays Object literal Like in many scripting languages, arrays and objects (associative arrays in other languages) can each be created with a succinct shortcut syntax. In fact, these literals form the basis of the JSON data format. associative arrays literals JSON Regular expressions Regular expression In a manner similar to Perl, JavaScript also supports regular expressions, which provide a concise and powerful syntax for text manipulation that is more sophisticated than the built-in string functions.[76] Perl regular expressions [76] [ ] Promises and Async/await JavaScript supports promises and Async/await for handling asynchronous operations.[citation needed] promises Async/await citation needed citation needed Promises Futures and promises A built-in Promise object provides functionality for handling promises and associating handlers with an asynchronous action's eventual result. Recently, the JavaScript specification introduced combinator methods, which allow developers to combine multiple JavaScript promises and do operations based on different scenarios. The methods introduced are: Promise.race, Promise.all, Promise.allSettled and Promise.any. Async/await Async/await Async/await allows an asynchronous, non-blocking function to be structured in a way similar to an ordinary synchronous function. Asynchronous, non-blocking code can be written, with minimal overhead, structured similarly to traditional synchronous, blocking code. Vendor-specific extensions Historically, some JavaScript engines supported these non-standard features: JavaScript engines conditional catch clauses (like Java) array comprehensions and generator expressions (like Python) array comprehensions concise function expressions (function(args) expr; this experimental syntax predated arrow functions) ECMAScript for XML (E4X), an extension that adds native XML support to ECMAScript (unsupported in Firefox since version 21[77]) ECMAScript for XML [77] [ ] Syntax JavaScript syntax Simple examples Variables in JavaScript can be defined using either the var,[78] let[79] or const[80] keywords. Variables defined without keywords will be defined at the global scope. Variables [78] [ ] [79] [ ] [80] [ ] // Declares a function-scoped variable named `x`, and implicitly assigns the // special value `undefined` to it. Variables without value are automatically // set to undefined. // var is generally considered bad practice and let and const are usually preferred. var x ; // Variables can be manually set to `undefined` like so let x2 = undefined ; // Declares a block-scoped variable named `y`, and implicitly sets it to // `undefined`. The `let` keyword was introduced in ECMAScript 2015. let y ; // Declares a block-scoped, un-reassignable variable named `z`, and sets it to // a string literal. The `const` keyword was also introduced in ECMAScript 2015, // and must be explicitly assigned to. // The keyword `const` means constant, hence the variable cannot be reassigned // as the value is `constant`. const z = \"this value cannot be reassigned!\" ; // Declares a global-scoped variable and assigns 3. This is generally considered // bad practice, and will not work if strict mode is on. t = 3 ; // Declares a variable named `myNumber`, and assigns a number literal (the value // `2`) to it. let myNumber = 2 ; // Reassigns `myNumber`, setting it to a string literal (the value `\"foo\"`). // JavaScript is a dynamically-typed language, so this is legal. myNumber = \"foo\" ; Note the comments in the examples above, all of which were preceded with two forward slashes. comments forward slashes There is no built-in Input/output functionality in JavaScript, instead it is provided by the run-time environment. The ECMAScript specification in edition 5.1 mentions that \"there are no provisions in this specification for input of external data or output of computed results\".[81] However, most runtime environments have a console object that can be used to print output.[82] Here is a minimalist \"Hello, World!\" program in JavaScript in a runtime environment with a console object: Input/output [81] [ ] [82] [ ] \"Hello, World!\" program console . log ( \"Hello, World!\" ); In HTML documents, a program like this is required for an output: // Text nodes can be made using the \"write\" method. // This is frowned upon, as it can overwrite the document if the document is fully loaded. document . write ( 'foo' ); // Elements can be made too. First, they have to be created in the DOM. const myElem = document . createElement ( 'span' ); // Attributes like classes and the id can be set as well myElem . classList . add ( 'foo' ); myElem . id = 'bar' ; // After setting this, the tag will look like this: `<span class=\"foo\" id=\"bar\" data-attr=\"baz\"></span>` myElem . setAttribute ( 'data-attr' , 'baz' ); // Which could also be written as `myElem.dataset.attr = 'baz'` // Finally append it as a child element to the <body> in the HTML document . body . appendChild ( myElem ); // Elements can be imperatively grabbed with querySelector for one element, or querySelectorAll for multiple elements that can be looped with forEach document . querySelector ( '.class' ); // Selects the first element with the \"class\" class document . querySelector ( '#id' ); // Selects the first element with an `id` of \"id\" document . querySelector ( '[data-other]' ); // Selects the first element with the \"data-other\" attribute document . querySelectorAll ( '.multiple' ); // Returns an Array-like NodeList of all elements with the \"multiple\" class A simple recursive function to calculate the factorial of a natural number: recursive factorial natural number function factorial ( n ) { // Checking the argument for legitimacy. Factorial is defined for positive integers. if ( isNaN ( n )) { console . error ( \"Non-numerical argument not allowed.\" ); return NaN ; // The special value: Not a Number } if ( n === 0 ) return 1 ; // 0! = 1 if ( n < 0 ) return undefined ; // Factorial of negative numbers is not defined. if ( n % 1 ) { console . warn ( ` ${ n } will be rounded to the closest integer. For non-integers consider using gamma function instead.` ); n = Math . round ( n ); } // The above checks need not be repeated in the recursion, hence defining the actual recursive part separately below. // The following line is a function expression to recursively compute the factorial. It uses the arrow syntax introduced in ES6. const recursivelyCompute = a => a > 1 ? a * recursivelyCompute ( a - 1 ) : 1 ; // Note the use of the ternary operator `?`. return recursivelyCompute ( n ); } factorial ( 3 ); // Returns 6 An anonymous function (or lambda): anonymous function const counter = function () { let count = 0 ; return function () { return ++ count ; } }; const x = counter (); x (); // Returns 1 x (); // Returns 2 x (); // Returns 3 This example shows that, in JavaScript, function closures capture their non-local variables by reference. function closures Arrow functions were first introduced in 6th Edition \u2013 ECMAScript 2015. They shorten the syntax for writing functions in JavaScript. Arrow functions are anonymous, so a variable is needed to refer to them in order to invoke them after their creation, unless surrounded by parenthesis and executed immediately. 6th Edition \u2013 ECMAScript 2015 Example of arrow function: // Arrow functions let us omit the `function` keyword. // Here `long_example` points to an anonymous function value. const long_example = ( input1 , input2 ) => { console . log ( \"Hello, World!\" ); const output = input1 + input2 ; return output ; }; // If there are no braces, the arrow function simply returns the expression // So here it's (input1 + input2) const short_example = ( input1 , input2 ) => input1 + input2 ; long_example ( 2 , 3 ); // Prints \"Hello, World!\" and returns 5 short_example ( 2 , 5 ); // Returns 7 // If an arrow function has only one parameter, the parentheses can be removed. const no_parentheses = input => input + 2 ; no_parentheses ( 3 ); // Returns 5 // An arrow function, like other function definitions, can be executed in the same statement as they are created. // This is useful when writing libraries to avoid filling the global scope, and for closures. let three = (( a , b ) => a + b ) ( 1 , 2 ); const generate_multiplier_function = a => ( b => isNaN ( b ) || ! b ? a : a *= b ); const five_multiples = generate_multiplier_function ( 5 ); // The supplied argument \"seeds\" the expression and is retained by a. five_multiples ( 1 ); // Returns 5 five_multiples ( 3 ); // Returns 15 five_multiples ( 4 ); // Returns 60 In JavaScript, objects can be created as instances of a class. objects instances class Object class example: class Ball { constructor ( radius ) { this . radius = radius ; this . area = Math . PI * ( radius ** 2 ); } // Classes (and thus objects) can contain functions known as methods show () { console . log ( this . radius ); } }; const myBall = new Ball ( 5 ); // Creates a new instance of the ball object with radius 5 myBall . radius ++ ; // Object properties can usually be modified from the outside myBall . show (); // Using the inherited \"show\" function logs \"6\" In JavaScript, objects can be instantiated directly from a function. objects Object functional example: function Ball ( radius ) { const area = Math . PI * ( radius ** 2 ); const obj = { radius , area }; // Objects are mutable, and functions can be added as properties. obj . show = () => console . log ( obj . radius ); return obj ; }; const myBall = Ball ( 5 ); // Creates a new ball object with radius 5. No \"new\" keyword needed. myBall . radius ++ ; // The instance property can be modified. myBall . show (); // Using the \"show\" function logs \"6\" - the new instance value. Variadic function demonstration (arguments is a special variable):[83] Variadic function variable [83] [ ] function sum () { let x = 0 ; for ( let i = 0 ; i < arguments . length ; ++ i ) x += arguments [ i ]; return x ; } sum ( 1 , 2 ); // Returns 3 sum ( 1 , 2 , 3 ); // Returns 6 // As of ES6, using the rest operator. function sum (... args ) { return args . reduce (( a , b ) => a + b ); } sum ( 1 , 2 ); // Returns 3 sum ( 1 , 2 , 3 ); // Returns 6 Immediately-invoked function expressions are often used to create closures. Closures allow gathering properties and methods in a namespace and making some of them private: Immediately-invoked function expressions let counter = ( function () { let i = 0 ; // Private property return { // Public methods get : function () { alert ( i ); }, set : function ( value ) { i = value ; }, increment : function () { alert ( ++ i ); } }; })(); // Module counter . get (); // Returns 0 counter . set ( 6 ); counter . increment (); // Returns 7 counter . increment (); // Returns 8 Generator objects (in the form of generator functions) provide a function which can be called, exited, and re-entered while maintaining internal context (statefulness).[84] Generator [84] [ ] function * rawCounter () { yield 1 ; yield 2 ; } function * dynamicCounter () { let count = 0 ; while ( true ) { // It is not recommended to utilize while true loops in most cases. yield ++ count ; } } // Instances const counter1 = rawCounter (); const counter2 = dynamicCounter (); // Implementation counter1 . next (); // {value: 1, done: false} counter1 . next (); // {value: 2, done: false} counter1 . next (); // {value: undefined, done: true} counter2 . next (); // {value: 1, done: false} counter2 . next (); // {value: 2, done: false} counter2 . next (); // {value: 3, done: false} // ...infinitely JavaScript can export and import from modules:[85] [85] [ ] Export example: /* mymodule.js */ // This function remains private, as it is not exported let sum = ( a , b ) => { return a + b ; } // Export variables export let name = 'Alice' ; export let age = 23 ; // Export named functions export function add ( num1 , num2 ) { return num1 + num2 ; } // Export class export class Multiplication { constructor ( num1 , num2 ) { this . num1 = num1 ; this . num2 = num2 ; } add () { return sum ( this . num1 , this . num2 ); } } Import example: // Import one property import { add } from './mymodule.js' ; console . log ( add ( 1 , 2 )); //> 3 // Import multiple properties import { name , age } from './mymodule.js' ; console . log ( name , age ); //> \"Alice\", 23 // Import all properties from a module import * from './module.js' console . log ( name , age ); //> \"Alice\", 23 console . log ( add ( 1 , 2 )); //> 3 More advanced example This sample code displays various JavaScript features. /* Finds the lowest common multiple (LCM) of two numbers */ function LCMCalculator ( x , y ) { // constructor function if ( isNaN ( x * y )) throw new TypeError ( \"Non-numeric arguments not allowed.\" ); const checkInt = function ( x ) { // inner function if ( x % 1 !== 0 ) throw new TypeError ( x + \"is not an integer\" ); return x ; }; this . a = checkInt ( x ) // semicolons ^^^^ are optional, a newline is enough this . b = checkInt ( y ); } // The prototype of object instances created by a constructor is // that constructor's \"prototype\" property. LCMCalculator . prototype = { // object literal constructor : LCMCalculator , // when reassigning a prototype, set the constructor property appropriately gcd : function () { // method that calculates the greatest common divisor // Euclidean algorithm: let a = Math . abs ( this . a ), b = Math . abs ( this . b ), t ; if ( a < b ) { // swap variables // t = b; b = a; a = t; [ a , b ] = [ b , a ]; // swap using destructuring assignment (ES6) } while ( b !== 0 ) { t = b ; b = a % b ; a = t ; } // Only need to calculate GCD once, so \"redefine\" this method. // (Actually not redefinition\u2014it's defined on the instance itself, // so that this.gcd refers to this \"redefinition\" instead of LCMCalculator.prototype.gcd. // Note that this leads to a wrong result if the LCMCalculator object members \"a\" or \"b\" are altered afterwards.) // Also, 'gcd' === \"gcd\", this['gcd'] === this.gcd this [ 'gcd' ] = function () { return a ; }; return a ; }, // Object property names can be specified by strings delimited by double (\") or single (') quotes. \"lcm\" : function () { // Variable names do not collide with object properties, e.g., |lcm| is not |this.lcm|. // not using |this.a*this.b| to avoid FP precision issues let lcm = this . a / this . gcd () * this . b ; // Only need to calculate lcm once, so \"redefine\" this method. this . lcm = function () { return lcm ; }; return lcm ; }, // Methods can also be declared using ES6 syntax toString () { // Using both ES6 template literals and the (+) operator to concatenate values return `LCMCalculator: a = ${ this . a } , b = ` + this . b ; } }; // Define generic output function; this implementation only works for Web browsers function output ( x ) { document . body . appendChild ( document . createTextNode ( x )); document . body . appendChild ( document . createElement ( 'br' )); } // Note: Array's map() and forEach() are defined in JavaScript 1.6. // They are used here to demonstrate JavaScript's inherent functional nature. [ [ 25 , 55 ], [ 21 , 56 ], [ 22 , 58 ], [ 28 , 56 ] ]. map ( function ( pair ) { // array literal + mapping function return new LCMCalculator ( pair [ 0 ], pair [ 1 ]); }). sort (( a , b ) => a . lcm () - b . lcm ()) // sort with this comparative function; => is a shorthand form of a function, called \"arrow function\" . forEach ( printResult ); function printResult ( obj ) { output ( obj + \", gcd = \" + obj . gcd () + \", lcm = \" + obj . lcm ()); } The following output should be displayed in the browser window. LCMCalculator: a = 28, b = 56, gcd = 28, lcm = 56 LCMCalculator: a = 21, b = 56, gcd = 7, lcm = 168 LCMCalculator: a = 25, b = 55, gcd = 5, lcm = 275 LCMCalculator: a = 22, b = 58, gcd = 2, lcm = 638 Security Browser security JavaScript and the DOM provide the potential for malicious authors to deliver scripts to run on a client computer via the Web. Browser authors minimize this risk using two restrictions. First, scripts run in a sandbox in which they can only perform Web-related actions, not general-purpose programming tasks like creating files. Second, scripts are constrained by the same-origin policy: scripts from one website do not have access to information such as usernames, passwords, or cookies sent to another site. Most JavaScript-related security bugs are breaches of either the same origin policy or the sandbox. DOM sandbox same-origin policy There are subsets of general JavaScript\u2014ADsafe, Secure ECMAScript (SES)\u2014that provide greater levels of security, especially on code created by third parties (such as advertisements).[86][87] Closure Toolkit is another project for safe embedding and isolation of third-party JavaScript and HTML.[88] [86] [ ] [87] [ ] [88] [ ] Content Security Policy is the main intended method of ensuring that only trusted code is executed on a Web page. Content Security Policy Cross-site scripting Cross-site scripting A common JavaScript-related security problem is cross-site scripting (XSS), a violation of the same-origin policy. XSS vulnerabilities occur when an attacker can cause a target Website, such as an online banking website, to include a malicious script in the webpage presented to a victim. The script in this example can then access the banking application with the privileges of the victim, potentially disclosing secret information or transferring money without the victim's authorization. One important solution to XSS vulnerabilities is HTML sanitization. cross-site scripting same-origin policy HTML sanitization Some browsers include partial protection against reflected XSS attacks, in which the attacker provides a URL including malicious script. However, even users of those browsers are vulnerable to other XSS attacks, such as those where the malicious code is stored in a database. Only correct design of Web applications on the server-side can fully prevent XSS. XSS vulnerabilities can also occur because of implementation mistakes by browser authors.[89] [89] [ ] Cross-site request forgery Cross-site request forgery Another cross-site vulnerability is cross-site request forgery (CSRF). In CSRF, code on an attacker's site tricks the victim's browser into taking actions the user did not intend at a target site (like transferring money at a bank). When target sites rely solely on cookies for request authentication, requests originating from code on the attacker's site can carry the same valid login credentials of the initiating user. In general, the solution to CSRF is to require an authentication value in a hidden form field, and not only in the cookies, to authenticate any request that might have lasting effects. Checking the HTTP Referrer header can also help. cross-site request forgery \"JavaScript hijacking\" is a type of CSRF attack in which a <script> tag on an attacker's site exploits a page on the victim's site that returns private information such as JSON or JavaScript. Possible solutions include: JSON requiring an authentication token in the POST and GET parameters for any response that returns private information. POST GET Misplaced trust in the client Developers of client-server applications must recognize that untrusted clients may be under the control of attackers. The application author cannot assume that their JavaScript code will run as intended (or at all) because any secret embedded in the code could be extracted by a determined adversary. Some implications are: Website authors cannot perfectly conceal how their JavaScript operates because the raw source code must be sent to the client. The code can be obfuscated, but obfuscation can be reverse-engineered. obfuscated JavaScript form validation only provides convenience for users, not security. If a site verifies that the user agreed to its terms of service, or filters invalid characters out of fields that should only contain numbers, it must do so on the server, not only the client. Scripts can be selectively disabled, so JavaScript cannot be relied on to prevent operations such as right-clicking on an image to save it.[90] [90] [ ] It is considered very bad practice to embed sensitive information such as passwords in JavaScript because it can be extracted by an attacker.[91] [91] [ ] Prototype pollution is a runtime vulnerability in which attackers can overwrite arbitrary properties in an object's prototype. Prototype pollution Misplaced trust in developers Package management systems such as npm and Bower are popular with JavaScript developers. Such systems allow a developer to easily manage their program's dependencies upon other developers' program libraries. Developers trust that the maintainers of the libraries will keep them secure and up to date, but that is not always the case. A vulnerability has emerged because of this blind trust. Relied-upon libraries can have new releases that cause bugs or vulnerabilities to appear in all programs that rely upon the libraries. Inversely, a library can go unpatched with known vulnerabilities out in the wild. In a study done looking over a sample of 133,000 websites, researchers found 37% of the websites included a library with at least one known vulnerability.[92] \"The median lag between the oldest library version used on each website and the newest available version of that library is 1,177 days in ALEXA, and development of some libraries still in active use ceased years ago.\"[92] Another possibility is that the maintainer of a library may remove the library entirely. This occurred in March 2016 when Azer Ko\u00e7ulu removed his repository from npm. This caused tens of thousands of programs and websites depending upon his libraries to break.[93][94] npm [92] [ ] [92] [ ] [93] [ ] [94] [ ] Browser and plugin coding errors Buffer overflow JavaScript provides an interface to a wide range of browser capabilities, some of which may have flaws such as buffer overflows. These flaws can allow attackers to write scripts that would run any code they wish on the user's system. This code is not by any means limited to another JavaScript application. For example, a buffer overrun exploit can allow an attacker to gain access to the operating system's API with superuser privileges. buffer overflows API These flaws have affected major browsers including Firefox,[95] Internet Explorer,[96] and Safari.[97] [95] [ ] [96] [ ] [97] [ ] Plugins, such as video players, Adobe Flash, and the wide range of ActiveX controls enabled by default in Microsoft Internet Explorer, may also have flaws exploitable via JavaScript (such flaws have been exploited in the past).[98][99] Adobe Flash ActiveX [98] [ ] [99] [ ] In Windows Vista, Microsoft has attempted to contain the risks of bugs such as buffer overflows by running the Internet Explorer process with limited privileges.[100] Google Chrome similarly confines its page renderers to their own \"sandbox\". [100] [ ] Google Chrome sandbox Sandbox implementation errors Web browsers are capable of running JavaScript outside the sandbox, with the privileges necessary to, for example, create or delete files. Such privileges are not intended to be granted to code from the Web. Incorrectly granting privileges to JavaScript from the Web has played a role in vulnerabilities in both Internet Explorer[101] and Firefox.[102] In Windows XP Service Pack 2, Microsoft demoted JScript's privileges in Internet Explorer.[103] [101] [ ] [102] [ ] [103] [ ] Microsoft Windows allows JavaScript source files on a computer's hard drive to be launched as general-purpose, non-sandboxed programs (see: Windows Script Host). This makes JavaScript (like VBScript) a theoretically viable vector for a Trojan horse, although JavaScript Trojan horses are uncommon in practice.[104][failed verification] Microsoft Windows Windows Script Host VBScript Trojan horse [104] [ ] failed verification failed verification Hardware vulnerabilities In 2015, a JavaScript-based proof-of-concept implementation of a rowhammer attack was described in a paper by security researchers.[105][106][107][108] rowhammer [105] [ ] [106] [ ] [107] [ ] [108] [ ] In 2017, a JavaScript-based attack via browser was demonstrated that could bypass ASLR. It is called \"ASLR\u2295Cache\" or AnC.[109][110] ASLR [109] [ ] [110] [ ] In 2018, the paper that announced the Spectre attacks against Speculative Execution in Intel and other processors included a JavaScript implementation.[111] Spectre [111] [ ] Development tools Important tools have evolved with the language. Every major web browser has built-in web development tools, including a JavaScript debugger. web development tools debugger Static program analysis tools, such as ESLint and JSLint, scan JavaScript code for conformance to a set of standards and guidelines. Static program analysis ESLint JSLint Some browsers have built-in profilers. Stand-alone profiling libraries have also been created, such as benchmark.js and jsbench.[112][113] profilers [112] [ ] [113] [ ] Many text editors have syntax highlighting support for JavaScript code. text editors Related technologies Java A common misconception is that JavaScript is directly related to Java. Both indeed have a C-like syntax (the C language being their most immediate common ancestor language). They are also typically sandboxed, and JavaScript was designed with Java's syntax and standard library in mind. In particular, all Java keywords were reserved in original JavaScript, JavaScript's standard library follows Java's naming conventions, and JavaScript's Math and Date objects are based on classes from Java 1.0.[114] Java sandboxed Math Date [114] [ ] Both languages first appeared in 1995, but Java was developed by James Gosling of Sun Microsystems and JavaScript by Brendan Eich of Netscape Communications. James Gosling Brendan Eich The differences between the two languages are more prominent than their similarities. Java has static typing, while JavaScript's typing is dynamic. Java is loaded from compiled bytecode, while JavaScript is loaded as human-readable source code. Java's objects are class-based, while JavaScript's are prototype-based. Finally, Java did not support functional programming until Java 8, while JavaScript has done so from the beginning, being influenced by Scheme. static typing dynamic compiled class-based prototype-based Scheme JSON JSON is a data format derived from JavaScript; hence the name JavaScript Object Notation. It is a widely used format supported by many other programming languages. JSON Transpilers Many websites are JavaScript-heavy, so transpilers have been created to convert code written in other languages, which can aid the development process.[36] transpilers [36] [ ] TypeScript and CoffeeScript are two notable languages that transpile to JavaScript. TypeScript CoffeeScript WebAssembly WebAssembly is a newer language with a bytecode format designed to complement JavaScript, especially the performance-critical portions of web page scripts. All of the major JavaScript engines support WebAssembly,[115] which runs in the same sandbox as regular JavaScript code. WebAssembly bytecode web page JavaScript engines [115] [ ] sandbox asm.js is a subset of JavaScript that served as the forerunner of WebAssembly.[116] asm.js [116] [ ] References ^ a b \"Netscape and Sun announce JavaScript, the Open, Cross-platform Object Scripting Language for Enterprise Networks and the Internet\" (Press release). 4 December 1995. Archived from the original on 16 September 2007. ^ a b a b \"Netscape and Sun announce JavaScript, the Open, Cross-platform Object Scripting Language for Enterprise Networks and the Internet\" (Press release). 4 December 1995. Archived from the original on 16 September 2007. \"Netscape and Sun announce JavaScript, the Open, Cross-platform Object Scripting Language for Enterprise Networks and the Internet\" the original ^ \"ECMAScript\u00ae 2024 Language Specification\". June 2024. Retrieved 30 August 2024. ^ ^ \"ECMAScript\u00ae 2024 Language Specification\". June 2024. Retrieved 30 August 2024. \"ECMAScript\u00ae 2024 Language Specification\" . Retrieved 30 August 2024 30 August ^ \"ECMAScript\u00ae 2025 Language Specification\". 27 March 2024. Retrieved 17 April 2024. ^ ^ \"ECMAScript\u00ae 2025 Language Specification\". 27 March 2024. Retrieved 17 April 2024. \"ECMAScript\u00ae 2025 Language Specification\" . Retrieved 17 April 2024 17 April ^ \"nodejs/node-eps\". GitHub. Archived from the original on 29 August 2020. Retrieved 5 July 2018. ^ ^ \"nodejs/node-eps\". GitHub. Archived from the original on 29 August 2020. Retrieved 5 July 2018. \"nodejs/node-eps\" Archived . Retrieved 5 July 2018 5 July ^ a b Seibel, Peter (16 September 2009). Coders at Work: Reflections on the Craft of Programming. Apress. ISBN 978-1-4302-1948-4. Archived from the original on 24 December 2020. Retrieved 25 December 2018. Eich: The immediate concern at Netscape was it must look like Java. ^ a b a b Seibel, Peter (16 September 2009). Coders at Work: Reflections on the Craft of Programming. Apress. ISBN 978-1-4302-1948-4. Archived from the original on 24 December 2020. Retrieved 25 December 2018. Eich: The immediate concern at Netscape was it must look like Java. Coders at Work: Reflections on the Craft of Programming ISBN 978-1-4302-1948-4 Archived . Retrieved 25 December 2018 25 December ^ a b c d e \"Chapter 4. How JavaScript Was Created\". speakingjs.com. Archived from the original on 27 February 2020. Retrieved 21 November 2017. ^ a b c d e a b c d e \"Chapter 4. How JavaScript Was Created\". speakingjs.com. Archived from the original on 27 February 2020. Retrieved 21 November 2017. \"Chapter 4. How JavaScript Was Created\" Archived . Retrieved 21 November 2017 21 November ^ \"Popularity \u2013 Brendan Eich\". ^ ^ \"Popularity \u2013 Brendan Eich\". \"Popularity \u2013 Brendan Eich\" ^ \"Brendan Eich: An Introduction to JavaScript, JSConf 2010\". YouTube. 20 January 2013. p. 22m. Archived from the original on 29 August 2020. Retrieved 25 November 2019. Eich: \"function\", eight letters, I was influenced by AWK. ^ ^ \"Brendan Eich: An Introduction to JavaScript, JSConf 2010\". YouTube. 20 January 2013. p. 22m. Archived from the original on 29 August 2020. Retrieved 25 November 2019. Eich: \"function\", eight letters, I was influenced by AWK. \"Brendan Eich: An Introduction to JavaScript, JSConf 2010\" YouTube Archived . Retrieved 25 November 2019 25 November ^ Eich, Brendan (1998). \"Foreword\". In Goodman, Danny (ed.). JavaScript Bible (3rd ed.). John Wiley & Sons. ISBN 0-7645-3188-3. LCCN 97078208. OCLC 38888873. OL 712205M. ^ ^ Eich, Brendan (1998). \"Foreword\". In Goodman, Danny (ed.). JavaScript Bible (3rd ed.). John Wiley & Sons. ISBN 0-7645-3188-3. LCCN 97078208. OCLC 38888873. OL 712205M. Eich, Brendan Goodman, Danny JavaScript Bible JavaScript Bible John Wiley & Sons ISBN 0-7645-3188-3 LCCN 97078208 OCLC 38888873 OL 712205M ^ a b \"Usage Statistics of JavaScript as Client-side Programming Language on Websites\". W3Techs. Retrieved 27 February 2024. ^ a b a b \"Usage Statistics of JavaScript as Client-side Programming Language on Websites\". W3Techs. Retrieved 27 February 2024. \"Usage Statistics of JavaScript as Client-side Programming Language on Websites\" . Retrieved 27 February 2024 27 February ^ \"ECMAScript 2020 Language Specification\". Archived from the original on 8 May 2020. Retrieved 8 May 2020. ^ ^ \"ECMAScript 2020 Language Specification\". Archived from the original on 8 May 2020. Retrieved 8 May 2020. \"ECMAScript 2020 Language Specification\" Archived . Retrieved 8 May 2020 8 May ^ \"Bloomberg Game Changers: Marc Andreessen\". Bloomberg. Bloomberg. 17 March 2011. Archived from the original on 16 May 2012. Retrieved 7 December 2011. ^ ^ \"Bloomberg Game Changers: Marc Andreessen\". Bloomberg. Bloomberg. 17 March 2011. Archived from the original on 16 May 2012. Retrieved 7 December 2011. \"Bloomberg Game Changers: Marc Andreessen\" Bloomberg Archived . Retrieved 7 December 2011 7 December ^ Enzer, Larry (31 August 2018). \"The Evolution of the Web Browsers\". Monmouth Web Developers. Archived from the original on 31 August 2018. Retrieved 31 August 2018. ^ ^ Enzer, Larry (31 August 2018). \"The Evolution of the Web Browsers\". Monmouth Web Developers. Archived from the original on 31 August 2018. Retrieved 31 August 2018. \"The Evolution of the Web Browsers\" the original . Retrieved 31 August 2018 31 August ^ a b c Fin JS (17 June 2016), \"Brendan Eich \u2013 CEO of Brave\", YouTube, retrieved 7 February 2018 ^ a b c a b c Fin JS (17 June 2016), \"Brendan Eich \u2013 CEO of Brave\", YouTube, retrieved 7 February 2018 \"Brendan Eich \u2013 CEO of Brave\" , retrieved 7 February 2018 7 February ^ \"Netscape Communications Corp.\", Browser enhancements. Encyclop\u00e6dia Britannica 2006 Ultimate Reference Suite DVD ^ ^ \"Netscape Communications Corp.\", Browser enhancements. Encyclop\u00e6dia Britannica 2006 Ultimate Reference Suite DVD ^ \"TechVision: Innovators of the Net: Brendan Eich and JavaScript\". Archived from the original on 8 February 2008. ^ ^ \"TechVision: Innovators of the Net: Brendan Eich and JavaScript\". Archived from the original on 8 February 2008. \"TechVision: Innovators of the Net: Brendan Eich and JavaScript\" the original ^ a b Han, Sheon (4 March 2024). \"JavaScript Runs the World\u2014Maybe Even Literally\". Wired. Retrieved 21 August 2024. ^ a b a b Han, Sheon (4 March 2024). \"JavaScript Runs the World\u2014Maybe Even Literally\". Wired. Retrieved 21 August 2024. \"JavaScript Runs the World\u2014Maybe Even Literally\" . Retrieved 21 August 2024 21 August ^ a b \"Chapter 5. Standardization: ECMAScript\". speakingjs.com. Archived from the original on 1 November 2021. Retrieved 1 November 2021. ^ a b a b \"Chapter 5. Standardization: ECMAScript\". speakingjs.com. Archived from the original on 1 November 2021. Retrieved 1 November 2021. \"Chapter 5. Standardization: ECMAScript\" the original . Retrieved 1 November 2021 1 November ^ a b Champeon, Steve (6 April 2001). \"JavaScript, How Did We Get Here?\". oreilly.com. Archived from the original on 19 July 2016. Retrieved 16 July 2016. ^ a b a b Champeon, Steve (6 April 2001). \"JavaScript, How Did We Get Here?\". oreilly.com. Archived from the original on 19 July 2016. Retrieved 16 July 2016. \"JavaScript, How Did We Get Here?\" the original . Retrieved 16 July 2016 16 July ^ \"Microsoft Internet Explorer 3.0 Beta Now Available\". microsoft.com. Microsoft. 29 May 1996. Archived from the original on 24 November 2020. Retrieved 16 July 2016. ^ ^ \"Microsoft Internet Explorer 3.0 Beta Now Available\". microsoft.com. Microsoft. 29 May 1996. Archived from the original on 24 November 2020. Retrieved 16 July 2016. \"Microsoft Internet Explorer 3.0 Beta Now Available\" Archived . Retrieved 16 July 2016 16 July ^ McCracken, Harry (16 September 2010). \"The Unwelcome Return of \"Best Viewed with Internet Explorer\"\". technologizer.com. Archived from the original on 23 June 2018. Retrieved 16 July 2016. ^ ^ McCracken, Harry (16 September 2010). \"The Unwelcome Return of \"Best Viewed with Internet Explorer\"\". technologizer.com. Archived from the original on 23 June 2018. Retrieved 16 July 2016. \"The Unwelcome Return of \"Best Viewed with Internet Explorer\"\" Archived . Retrieved 16 July 2016 16 July ^ Baker, Loren (24 November 2004). \"Mozilla Firefox Internet Browser Market Share Gains to 7.4%\". Search Engine Journal. Archived from the original on 7 May 2021. Retrieved 8 May 2021. ^ ^ Baker, Loren (24 November 2004). \"Mozilla Firefox Internet Browser Market Share Gains to 7.4%\". Search Engine Journal. Archived from the original on 7 May 2021. Retrieved 8 May 2021. \"Mozilla Firefox Internet Browser Market Share Gains to 7.4%\" Archived . Retrieved 8 May 2021 8 May ^ Weber, Tim (9 May 2005). \"The assault on software giant Microsoft\". BBC News. Archived from the original on 25 September 2017. ^ ^ Weber, Tim (9 May 2005). \"The assault on software giant Microsoft\". BBC News. Archived from the original on 25 September 2017. \"The assault on software giant Microsoft\" BBC News the original ^ \"Big browser comparison test: Internet Explorer vs. Firefox, Opera, Safari and Chrome\". PC Games Hardware. Computec Media AG. 3 July 2009. Archived from the original on 2 May 2012. Retrieved 28 June 2010. ^ ^ \"Big browser comparison test: Internet Explorer vs. Firefox, Opera, Safari and Chrome\". PC Games Hardware. Computec Media AG. 3 July 2009. Archived from the original on 2 May 2012. Retrieved 28 June 2010. \"Big browser comparison test: Internet Explorer vs. Firefox, Opera, Safari and Chrome\" Archived . Retrieved 28 June 2010 28 June ^ Purdy, Kevin (11 June 2009). \"Lifehacker Speed Tests: Safari 4, Chrome 2\". Lifehacker. Archived from the original on 14 April 2021. Retrieved 8 May 2021. ^ ^ Purdy, Kevin (11 June 2009). \"Lifehacker Speed Tests: Safari 4, Chrome 2\". Lifehacker. Archived from the original on 14 April 2021. Retrieved 8 May 2021. \"Lifehacker Speed Tests: Safari 4, Chrome 2\" Lifehacker Archived . Retrieved 8 May 2021 8 May ^ \"TraceMonkey: JavaScript Lightspeed, Brendan Eich's Blog\". Archived from the original on 4 December 2015. Retrieved 22 July 2020. ^ ^ \"TraceMonkey: JavaScript Lightspeed, Brendan Eich's Blog\". Archived from the original on 4 December 2015. Retrieved 22 July 2020. \"TraceMonkey: JavaScript Lightspeed, Brendan Eich's Blog\" Archived . Retrieved 22 July 2020 22 July ^ \"Mozilla asks, 'Are we fast yet?'\". Wired. Archived from the original on 22 June 2018. Retrieved 18 January 2019. ^ ^ \"Mozilla asks, 'Are we fast yet?'\". Wired. Archived from the original on 22 June 2018. Retrieved 18 January 2019. \"Mozilla asks, 'Are we fast yet?'\" Archived . Retrieved 18 January 2019 18 January ^ \"ECMAScript 6: New Features: Overview and Comparison\". es6-features.org. Archived from the original on 18 March 2018. Retrieved 19 March 2018. ^ ^ \"ECMAScript 6: New Features: Overview and Comparison\". es6-features.org. Archived from the original on 18 March 2018. Retrieved 19 March 2018. \"ECMAScript 6: New Features: Overview and Comparison\" . Retrieved 19 March 2018 19 March ^ Professional Node.js: Building JavaScript Based Scalable Software Archived 2017-03-24 at the Wayback Machine, John Wiley & Sons, 01-Oct-2012 ^ ^ Professional Node.js: Building JavaScript Based Scalable Software Archived 2017-03-24 at the Wayback Machine, John Wiley & Sons, 01-Oct-2012 Professional Node.js: Building JavaScript Based Scalable Software Archived Wayback Machine ^ Sams Teach Yourself Node.js in 24 Hours Archived 2017-03-23 at the Wayback Machine, Sams Publishing, 05-Sep-2012 ^ ^ Sams Teach Yourself Node.js in 24 Hours Archived 2017-03-23 at the Wayback Machine, Sams Publishing, 05-Sep-2012 Sams Teach Yourself Node.js in 24 Hours Archived Wayback Machine ^ Lawton, George (19 July 2018). \"The secret history behind the success of npm and Node\". TheServerSide. Archived from the original on 2 August 2021. Retrieved 2 August 2021. ^ ^ Lawton, George (19 July 2018). \"The secret history behind the success of npm and Node\". TheServerSide. Archived from the original on 2 August 2021. Retrieved 2 August 2021. \"The secret history behind the success of npm and Node\" Archived . Retrieved 2 August 2021 2 August ^ Brown, Paul (13 January 2017). \"State of the Union: npm\". Linux.com. Archived from the original on 2 August 2021. Retrieved 2 August 2021. ^ ^ Brown, Paul (13 January 2017). \"State of the Union: npm\". Linux.com. Archived from the original on 2 August 2021. Retrieved 2 August 2021. \"State of the Union: npm\" Archived . Retrieved 2 August 2021 2 August ^ a b c Branscombe, Mary (4 May 2016). \"JavaScript Standard Moves to Yearly Release Schedule; Here is What's New for ES16\". The New Stack. Archived from the original on 16 January 2021. Retrieved 15 January 2021. ^ a b c a b c Branscombe, Mary (4 May 2016). \"JavaScript Standard Moves to Yearly Release Schedule; Here is What's New for ES16\". The New Stack. Archived from the original on 16 January 2021. Retrieved 15 January 2021. \"JavaScript Standard Moves to Yearly Release Schedule; Here is What's New for ES16\" Archived . Retrieved 15 January 2021 15 January ^ \"The TC39 Process\". tc39.es. Ecma International. Archived from the original on 7 February 2021. Retrieved 15 January 2021. ^ ^ \"The TC39 Process\". tc39.es. Ecma International. Archived from the original on 7 February 2021. Retrieved 15 January 2021. \"The TC39 Process\" Archived . Retrieved 15 January 2021 15 January ^ \"ECMAScript proposals\". TC39. Archived from the original on 4 December 2020. Retrieved 15 January 2021. ^ ^ \"ECMAScript proposals\". TC39. Archived from the original on 4 December 2020. Retrieved 15 January 2021. \"ECMAScript proposals\" Archived . Retrieved 15 January 2021 15 January ^ a b Ashkenas, Jeremy. \"List of languages that compile to JS\". GitHub. Archived from the original on 31 January 2020. Retrieved 6 February 2020. ^ a b a b Ashkenas, Jeremy. \"List of languages that compile to JS\". GitHub. Archived from the original on 31 January 2020. Retrieved 6 February 2020. Ashkenas, Jeremy \"List of languages that compile to JS\" GitHub Archived . Retrieved 6 February 2020 6 February ^ \"U.S. Trademark Serial No. 75026640\". uspto.gov. United States Patent and Trademark Office. 6 May 1997. Archived from the original on 13 July 2021. Retrieved 8 May 2021. ^ ^ \"U.S. Trademark Serial No. 75026640\". uspto.gov. United States Patent and Trademark Office. 6 May 1997. Archived from the original on 13 July 2021. Retrieved 8 May 2021. \"U.S. Trademark Serial No. 75026640\" United States Patent and Trademark Office Archived . Retrieved 8 May 2021 8 May ^ \"Legal Notices\". oracle.com. Oracle Corporation. Archived from the original on 5 June 2021. Retrieved 8 May 2021. ^ ^ \"Legal Notices\". oracle.com. Oracle Corporation. Archived from the original on 5 June 2021. Retrieved 8 May 2021. \"Legal Notices\" Oracle Corporation Archived . Retrieved 8 May 2021 8 May ^ \"Oracle to buy Sun in $7.4-bn deal\". The Economic Times. 21 April 2009. ^ ^ \"Oracle to buy Sun in $7.4-bn deal\". The Economic Times. 21 April 2009. \"Oracle to buy Sun in $7.4-bn deal\" ^ \"JavaScript\u2122\ufe0f\". ^ ^ \"JavaScript\u2122\ufe0f\". \"JavaScript\u2122\ufe0f\" ^ a b c \"Usage statistics of JavaScript libraries for websites\". W3Techs. Retrieved 9 April 2021. ^ a b c a b c \"Usage statistics of JavaScript libraries for websites\". W3Techs. Retrieved 9 April 2021. \"Usage statistics of JavaScript libraries for websites\" . Retrieved 9 April 2021 9 April ^ \"Using jQuery with Bootstrap\". clouddevs.com. 10 June 2019. Retrieved 17 March 2024. ^ ^ \"Using jQuery with Bootstrap\". clouddevs.com. 10 June 2019. Retrieved 17 March 2024. \"Using jQuery with Bootstrap\" . Retrieved 17 March 2024 17 March ^ \"Vanilla JS\". vanilla-js.com. 16 June 2020. Archived from the original on 16 June 2020. Retrieved 17 June 2020. ^ ^ \"Vanilla JS\". vanilla-js.com. 16 June 2020. Archived from the original on 16 June 2020. Retrieved 17 June 2020. \"Vanilla JS\" Archived . Retrieved 17 June 2020 17 June ^ \"Server-Side JavaScript Guide\". oracle.com. Oracle Corporation. 11 December 1998. Archived from the original on 11 March 2021. Retrieved 8 May 2021. ^ ^ \"Server-Side JavaScript Guide\". oracle.com. Oracle Corporation. 11 December 1998. Archived from the original on 11 March 2021. Retrieved 8 May 2021. \"Server-Side JavaScript Guide\" Oracle Corporation Archived . Retrieved 8 May 2021 8 May ^ Clinick, Andrew (14 July 2000). \"Introducing JScript .NET\". Microsoft Developer Network. Microsoft. Archived from the original on 10 November 2017. Retrieved 10 April 2018. [S]ince the 1996 introduction of JScript version 1.0 ... we've been seeing a steady increase in the usage of JScript on the server\u2014particularly in Active Server Pages (ASP) ^ ^ Clinick, Andrew (14 July 2000). \"Introducing JScript .NET\". Microsoft Developer Network. Microsoft. Archived from the original on 10 November 2017. Retrieved 10 April 2018. [S]ince the 1996 introduction of JScript version 1.0 ... we've been seeing a steady increase in the usage of JScript on the server\u2014particularly in Active Server Pages (ASP) \"Introducing JScript .NET\" Archived . Retrieved 10 April 2018 10 April ^ a b Mahemoff, Michael (17 December 2009). \"Server-Side JavaScript, Back with a Vengeance\". readwrite.com. Archived from the original on 17 June 2016. Retrieved 16 July 2016. ^ a b a b Mahemoff, Michael (17 December 2009). \"Server-Side JavaScript, Back with a Vengeance\". readwrite.com. Archived from the original on 17 June 2016. Retrieved 16 July 2016. \"Server-Side JavaScript, Back with a Vengeance\" Archived . Retrieved 16 July 2016 16 July ^ \"JavaScript for Acrobat\". adobe.com. 7 August 2009. Archived from the original on 7 August 2009. Retrieved 18 August 2009. ^ ^ \"JavaScript for Acrobat\". adobe.com. 7 August 2009. Archived from the original on 7 August 2009. Retrieved 18 August 2009. \"JavaScript for Acrobat\" Archived . Retrieved 18 August 2009 18 August ^ treitter (2 February 2013). \"Answering the question: \"How do I develop an app for GNOME?\"\". livejournal.com. Archived from the original on 11 February 2013. Retrieved 7 February 2013. ^ ^ treitter (2 February 2013). \"Answering the question: \"How do I develop an app for GNOME?\"\". livejournal.com. Archived from the original on 11 February 2013. Retrieved 7 February 2013. \"Answering the question: \"How do I develop an app for GNOME?\"\" Archived . Retrieved 7 February 2013 7 February ^ \"Tessel 2... Leverage all the libraries of Node.JS to create useful devices in minutes with Tessel\". tessel.io. Archived from the original on 26 May 2021. Retrieved 8 May 2021. ^ ^ \"Tessel 2... Leverage all the libraries of Node.JS to create useful devices in minutes with Tessel\". tessel.io. Archived from the original on 26 May 2021. Retrieved 8 May 2021. \"Tessel 2... Leverage all the libraries of Node.JS to create useful devices in minutes with Tessel\" Archived . Retrieved 8 May 2021 8 May ^ \"Node.js Raspberry Pi GPIO Introduction\". w3schools.com. Archived from the original on 13 August 2021. Retrieved 3 May 2020. ^ ^ \"Node.js Raspberry Pi GPIO Introduction\". w3schools.com. Archived from the original on 13 August 2021. Retrieved 3 May 2020. \"Node.js Raspberry Pi GPIO Introduction\" Archived . Retrieved 3 May 2020 3 May ^ \"Espruino \u2013 JavaScript for Microcontrollers\". espruino.com. Archived from the original on 1 May 2020. Retrieved 3 May 2020. ^ ^ \"Espruino \u2013 JavaScript for Microcontrollers\". espruino.com. Archived from the original on 1 May 2020. Retrieved 3 May 2020. \"Espruino \u2013 JavaScript for Microcontrollers\" Archived . Retrieved 3 May 2020 3 May ^ Looper, Jen (21 September 2015). \"A Guide to JavaScript Engines for Idiots\". Telerik Developer Network. Archived from the original on 8 December 2018. Retrieved 8 December 2018. ^ ^ Looper, Jen (21 September 2015). \"A Guide to JavaScript Engines for Idiots\". Telerik Developer Network. Archived from the original on 8 December 2018. Retrieved 8 December 2018. \"A Guide to JavaScript Engines for Idiots\" Telerik the original . Retrieved 8 December 2018 8 December ^ \"How Blink Works\". Google. Retrieved 12 March 2024. ^ ^ \"How Blink Works\". Google. Retrieved 12 March 2024. \"How Blink Works\" . Retrieved 12 March 2024 12 March ^ a b \"Documentation \u00b7 V8\". Google. Retrieved 3 March 2024. ^ a b a b \"Documentation \u00b7 V8\". Google. Retrieved 3 March 2024. \"Documentation \u00b7 V8\" . Retrieved 3 March 2024 3 March ^ Nelaturu, Keerthi. \"WebAssembly: What's the big deal?\". medium.com. Retrieved 3 March 2024. ^ ^ Nelaturu, Keerthi. \"WebAssembly: What's the big deal?\". medium.com. Retrieved 3 March 2024. \"WebAssembly: What's the big deal?\" . Retrieved 3 March 2024 3 March ^ \"Concurrency model and Event Loop\". Mozilla Developer Network. Archived from the original on 5 September 2015. Retrieved 28 August 2015. ^ ^ \"Concurrency model and Event Loop\". Mozilla Developer Network. Archived from the original on 5 September 2015. Retrieved 28 August 2015. \"Concurrency model and Event Loop\" Archived . Retrieved 28 August 2015 28 August ^ Flanagan, David (17 August 2006). JavaScript: The Definitive Guide: The Definitive Guide. \"O'Reilly Media, Inc.\". p. 16. ISBN 978-0-596-55447-7. Archived from the original on 1 August 2020. Retrieved 29 March 2019. ^ ^ Flanagan, David (17 August 2006). JavaScript: The Definitive Guide: The Definitive Guide. \"O'Reilly Media, Inc.\". p. 16. ISBN 978-0-596-55447-7. Archived from the original on 1 August 2020. Retrieved 29 March 2019. JavaScript: The Definitive Guide: The Definitive Guide ISBN 978-0-596-55447-7 Archived . Retrieved 29 March 2019 29 March ^ a b c d Korolev, Mikhail (1 March 2019). \"JavaScript quirks in one image from the Internet\". The DEV Community. Archived from the original on 28 October 2019. Retrieved 28 October 2019. ^ a b c d a b c d Korolev, Mikhail (1 March 2019). \"JavaScript quirks in one image from the Internet\". The DEV Community. Archived from the original on 28 October 2019. Retrieved 28 October 2019. \"JavaScript quirks in one image from the Internet\" Archived . Retrieved 28 October 2019 28 October ^ \"Proposal-bigint/ADVANCED.md at master \u00b7 tc39/Proposal-bigint\". GitHub. ^ ^ \"Proposal-bigint/ADVANCED.md at master \u00b7 tc39/Proposal-bigint\". GitHub. \"Proposal-bigint/ADVANCED.md at master \u00b7 tc39/Proposal-bigint\" GitHub ^ Bernhardt, Gary (2012). \"Wat\". Destroy All Software. Archived from the original on 28 October 2019. Retrieved 28 October 2019. ^ ^ Bernhardt, Gary (2012). \"Wat\". Destroy All Software. Archived from the original on 28 October 2019. Retrieved 28 October 2019. \"Wat\" Archived . Retrieved 28 October 2019 28 October ^ \"JavaScript data types and data structures\". MDN. 16 February 2017. Archived from the original on 14 March 2017. Retrieved 24 February 2017. ^ ^ \"JavaScript data types and data structures\". MDN. 16 February 2017. Archived from the original on 14 March 2017. Retrieved 24 February 2017. \"JavaScript data types and data structures\" Archived . Retrieved 24 February 2017 24 February ^ Flanagan 2006, pp. 176\u2013178. ^ ^ Flanagan 2006, pp. 176\u2013178. Flanagan 2006 ^ Crockford, Douglas. \"Prototypal Inheritance in JavaScript\". Archived from the original on 13 August 2013. Retrieved 20 August 2013. ^ ^ Crockford, Douglas. \"Prototypal Inheritance in JavaScript\". Archived from the original on 13 August 2013. Retrieved 20 August 2013. \"Prototypal Inheritance in JavaScript\" Archived . Retrieved 20 August 2013 20 August ^ \"Inheritance and the prototype chain\". Mozilla Developer Network. Archived from the original on 25 April 2013. Retrieved 6 April 2013. ^ ^ \"Inheritance and the prototype chain\". Mozilla Developer Network. Archived from the original on 25 April 2013. Retrieved 6 April 2013. \"Inheritance and the prototype chain\" Mozilla Archived . Retrieved 6 April 2013 6 April ^ Herman, David (2013). Effective JavaScript. Addison-Wesley. p. 83. ISBN 978-0-321-81218-6. ^ ^ Herman, David (2013). Effective JavaScript. Addison-Wesley. p. 83. ISBN 978-0-321-81218-6. Effective JavaScript ISBN 978-0-321-81218-6 ^ Haverbeke, Marijn (2011). Eloquent JavaScript. No Starch Press. pp. 95\u201397. ISBN 978-1-59327-282-1. ^ ^ Haverbeke, Marijn (2011). Eloquent JavaScript. No Starch Press. pp. 95\u201397. ISBN 978-1-59327-282-1. Eloquent JavaScript 95\u2013 ISBN 978-1-59327-282-1 ^ Katz, Yehuda (12 August 2011). \"Understanding \"Prototypes\" in JavaScript\". Archived from the original on 5 April 2013. Retrieved 6 April 2013. ^ ^ Katz, Yehuda (12 August 2011). \"Understanding \"Prototypes\" in JavaScript\". Archived from the original on 5 April 2013. Retrieved 6 April 2013. \"Understanding \"Prototypes\" in JavaScript\" Archived . Retrieved 6 April 2013 6 April ^ Herman, David (2013). Effective JavaScript. Addison-Wesley. pp. 125\u2013127. ISBN 978-0-321-81218-6. ^ ^ Herman, David (2013). Effective JavaScript. Addison-Wesley. pp. 125\u2013127. ISBN 978-0-321-81218-6. Effective JavaScript 125\u2013 ISBN 978-0-321-81218-6 ^ \"Function \u2013 JavaScript\". MDN Web Docs. Retrieved 30 October 2021. ^ ^ \"Function \u2013 JavaScript\". MDN Web Docs. Retrieved 30 October 2021. \"Function \u2013 JavaScript\" MDN Web Docs . Retrieved 30 October 2021 30 October ^ \"Properties of the Function Object\". Es5.github.com. Archived from the original on 28 January 2013. Retrieved 26 May 2013. ^ ^ \"Properties of the Function Object\". Es5.github.com. Archived from the original on 28 January 2013. Retrieved 26 May 2013. \"Properties of the Function Object\" Archived . Retrieved 26 May 2013 26 May ^ Flanagan 2006, p. 141. ^ ^ Flanagan 2006, p. 141. Flanagan 2006 ^ The many talents of JavaScript for generalizing Role-Oriented Programming approaches like Traits and Mixins Archived 2017-10-05 at the Wayback Machine, Peterseliger.blogspot.de, April 11, 2014. ^ ^ The many talents of JavaScript for generalizing Role-Oriented Programming approaches like Traits and Mixins Archived 2017-10-05 at the Wayback Machine, Peterseliger.blogspot.de, April 11, 2014. The many talents of JavaScript for generalizing Role-Oriented Programming approaches like Traits and Mixins Archived Wayback Machine ^ Traits for JavaScript Archived 2014-07-24 at the Wayback Machine, 2010. ^ ^ Traits for JavaScript Archived 2014-07-24 at the Wayback Machine, 2010. Traits for JavaScript Archived Wayback Machine ^ \"Home | CocktailJS\". Cocktailjs.github.io. Archived from the original on 4 February 2017. Retrieved 24 February 2017. ^ ^ \"Home | CocktailJS\". Cocktailjs.github.io. Archived from the original on 4 February 2017. Retrieved 24 February 2017. \"Home | CocktailJS\" Archived . Retrieved 24 February 2017 24 February ^ Croll, Angus (31 May 2011). \"A fresh look at JavaScript Mixins\". JavaScript, JavaScript\u2026. Archived from the original on 15 April 2020. ^ ^ Croll, Angus (31 May 2011). \"A fresh look at JavaScript Mixins\". JavaScript, JavaScript\u2026. Archived from the original on 15 April 2020. \"A fresh look at JavaScript Mixins\" Archived ^ Haverbeke, Marijn (2011). Eloquent JavaScript. No Starch Press. pp. 139\u2013149. ISBN 978-1-59327-282-1. ^ ^ Haverbeke, Marijn (2011). Eloquent JavaScript. No Starch Press. pp. 139\u2013149. ISBN 978-1-59327-282-1. Eloquent JavaScript 139\u2013 ISBN 978-1-59327-282-1 ^ \"E4X \u2013 Archive of obsolete content\". Mozilla Developer Network. Mozilla Foundation. 14 February 2014. Archived from the original on 24 July 2014. Retrieved 13 July 2014. ^ ^ \"E4X \u2013 Archive of obsolete content\". Mozilla Developer Network. Mozilla Foundation. 14 February 2014. Archived from the original on 24 July 2014. Retrieved 13 July 2014. \"E4X \u2013 Archive of obsolete content\" the original . Retrieved 13 July 2014 13 July ^ \"var \u2013 JavaScript\". The Mozilla Developer Network. Archived from the original on 23 December 2012. Retrieved 22 December 2012. ^ ^ \"var \u2013 JavaScript\". The Mozilla Developer Network. Archived from the original on 23 December 2012. Retrieved 22 December 2012. \"var \u2013 JavaScript\" Mozilla Developer Network Archived . Retrieved 22 December 2012 22 December ^ \"let\". MDN web docs. Mozilla. Archived from the original on 28 May 2019. Retrieved 27 June 2018. ^ ^ \"let\". MDN web docs. Mozilla. Archived from the original on 28 May 2019. Retrieved 27 June 2018. \"let\" Archived . Retrieved 27 June 2018 27 June ^ \"const\". MDN web docs. Mozilla. Archived from the original on 28 June 2018. Retrieved 27 June 2018. ^ ^ \"const\". MDN web docs. Mozilla. Archived from the original on 28 June 2018. Retrieved 27 June 2018. \"const\" Archived . Retrieved 27 June 2018 27 June ^ \"ECMAScript Language Specification \u2013 ECMA-262 Edition 5.1\". Ecma International. Archived from the original on 26 November 2012. Retrieved 22 December 2012. ^ ^ \"ECMAScript Language Specification \u2013 ECMA-262 Edition 5.1\". Ecma International. Archived from the original on 26 November 2012. Retrieved 22 December 2012. \"ECMAScript Language Specification \u2013 ECMA-262 Edition 5.1\" Ecma International Archived . Retrieved 22 December 2012 22 December ^ \"console\". Mozilla Developer Network. Mozilla. Archived from the original on 28 February 2013. Retrieved 6 April 2013. ^ ^ \"console\". Mozilla Developer Network. Mozilla. Archived from the original on 28 February 2013. Retrieved 6 April 2013. \"console\" Mozilla Archived . Retrieved 6 April 2013 6 April ^ \"arguments\". Mozilla Developer Network. Mozilla. Archived from the original on 13 April 2013. Retrieved 6 April 2013. ^ ^ \"arguments\". Mozilla Developer Network. Mozilla. Archived from the original on 13 April 2013. Retrieved 6 April 2013. \"arguments\" Mozilla Archived . Retrieved 6 April 2013 6 April ^ \"function* - JavaScript | MDN\". developer.mozilla.org. Retrieved 27 September 2022. ^ ^ \"function* - JavaScript | MDN\". developer.mozilla.org. Retrieved 27 September 2022. \"function* - JavaScript | MDN\" . Retrieved 27 September 2022 27 September ^ \"JavaScript modules\". MDN Web Docs. Mozilla. Archived from the original on 17 July 2022. Retrieved 28 July 2022. ^ ^ \"JavaScript modules\". MDN Web Docs. Mozilla. Archived from the original on 17 July 2022. Retrieved 28 July 2022. \"JavaScript modules\" Archived . Retrieved 28 July 2022 28 July ^ \"Making JavaScript Safe for Advertising\". ADsafe. Archived from the original on 6 July 2021. Retrieved 8 May 2021. ^ ^ \"Making JavaScript Safe for Advertising\". ADsafe. Archived from the original on 6 July 2021. Retrieved 8 May 2021. \"Making JavaScript Safe for Advertising\" Archived . Retrieved 8 May 2021 8 May ^ \"Secure ECMA Script (SES)\". Archived from the original on 15 May 2013. Retrieved 26 May 2013. ^ ^ \"Secure ECMA Script (SES)\". Archived from the original on 15 May 2013. Retrieved 26 May 2013. \"Secure ECMA Script (SES)\" Archived . Retrieved 26 May 2013 26 May ^ \"Google Caja Project\". Google. Archived from the original on 22 January 2021. Retrieved 9 July 2021. ^ ^ \"Google Caja Project\". Google. Archived from the original on 22 January 2021. Retrieved 9 July 2021. \"Google Caja Project\" Google Archived . Retrieved 9 July 2021 9 July ^ \"Mozilla Cross-Site Scripting Vulnerability Reported and Fixed \u2013 MozillaZine Talkback\". Mozillazine.org. Archived from the original on 21 July 2011. Retrieved 24 February 2017. ^ ^ \"Mozilla Cross-Site Scripting Vulnerability Reported and Fixed \u2013 MozillaZine Talkback\". Mozillazine.org. Archived from the original on 21 July 2011. Retrieved 24 February 2017. \"Mozilla Cross-Site Scripting Vulnerability Reported and Fixed \u2013 MozillaZine Talkback\" Archived . Retrieved 24 February 2017 24 February ^ Kottelin, Thor (17 June 2008). \"Right-click \"protection\"? Forget about it\". blog.anta.net. Archived from the original on 9 August 2011. Retrieved 28 July 2022. ^ ^ Kottelin, Thor (17 June 2008). \"Right-click \"protection\"? Forget about it\". blog.anta.net. Archived from the original on 9 August 2011. Retrieved 28 July 2022. \"Right-click \"protection\"? Forget about it\" the original . Retrieved 28 July 2022 28 July ^ Rehorik, Jan (29 November 2016). \"Why You Should Never Put Sensitive Data in Your JavaScript\". ServiceObjects Blog. ServiceObjects. Archived from the original on 3 June 2019. Retrieved 3 June 2019. ^ ^ Rehorik, Jan (29 November 2016). \"Why You Should Never Put Sensitive Data in Your JavaScript\". ServiceObjects Blog. ServiceObjects. Archived from the original on 3 June 2019. Retrieved 3 June 2019. \"Why You Should Never Put Sensitive Data in Your JavaScript\" Archived . Retrieved 3 June 2019 3 June ^ a b Lauinger, Tobias; Chaabane, Abdelberi; Arshad, Sajjad; Robertson, William; Wilson, Christo; Kirda, Engin (21 December 2016), \"Thou Shalt Not Depend on Me: Analysing the Use of Outdated JavaScript Libraries on the Web\" (PDF), Northeastern University, arXiv:1811.00918, doi:10.14722/ndss.2017.23414, ISBN 978-1-891562-46-4, S2CID 17885720, archived from the original (PDF) on 29 March 2017, retrieved 28 July 2022 ^ a b a b Lauinger, Tobias; Chaabane, Abdelberi; Arshad, Sajjad; Robertson, William; Wilson, Christo; Kirda, Engin (21 December 2016), \"Thou Shalt Not Depend on Me: Analysing the Use of Outdated JavaScript Libraries on the Web\" (PDF), Northeastern University, arXiv:1811.00918, doi:10.14722/ndss.2017.23414, ISBN 978-1-891562-46-4, S2CID 17885720, archived from the original (PDF) on 29 March 2017, retrieved 28 July 2022 \"Thou Shalt Not Depend on Me: Analysing the Use of Outdated JavaScript Libraries on the Web\" (PDF) arXiv 1811.00918 1811.00918 doi 10.14722/ndss.2017.23414 ISBN 978-1-891562-46-4 S2CID 17885720 the original (PDF) , retrieved 28 July 2022 28 July ^ Collins, Keith (27 March 2016). \"How one programmer broke the internet by deleting a tiny piece of code\". Quartz. Archived from the original on 22 February 2017. Retrieved 22 February 2017. ^ ^ Collins, Keith (27 March 2016). \"How one programmer broke the internet by deleting a tiny piece of code\". Quartz. Archived from the original on 22 February 2017. Retrieved 22 February 2017. \"How one programmer broke the internet by deleting a tiny piece of code\" Archived . Retrieved 22 February 2017 22 February ^ SC Magazine UK, Developer's 11 lines of deleted code 'breaks the internet' Archived February 23, 2017, at the Wayback Machine ^ ^ SC Magazine UK, Developer's 11 lines of deleted code 'breaks the internet' Archived February 23, 2017, at the Wayback Machine Developer's 11 lines of deleted code 'breaks the internet' Archived Wayback Machine ^ Mozilla Corporation, Buffer overflow in crypto.signText() Archived 2014-06-04 at the Wayback Machine ^ ^ Mozilla Corporation, Buffer overflow in crypto.signText() Archived 2014-06-04 at the Wayback Machine Buffer overflow in crypto.signText() Archived Wayback Machine ^ Festa, Paul (19 August 1998). \"Buffer-overflow bug in IE\". CNET. Archived from the original on 25 December 2002. ^ ^ Festa, Paul (19 August 1998). \"Buffer-overflow bug in IE\". CNET. Archived from the original on 25 December 2002. \"Buffer-overflow bug in IE\" CNET the original ^ SecurityTracker.com, Apple Safari JavaScript Buffer Overflow Lets Remote Users Execute Arbitrary Code and HTTP Redirect Bug Lets Remote Users Access Files Archived 2010-02-18 at the Wayback Machine ^ ^ SecurityTracker.com, Apple Safari JavaScript Buffer Overflow Lets Remote Users Execute Arbitrary Code and HTTP Redirect Bug Lets Remote Users Access Files Archived 2010-02-18 at the Wayback Machine Apple Safari JavaScript Buffer Overflow Lets Remote Users Execute Arbitrary Code and HTTP Redirect Bug Lets Remote Users Access Files Archived Wayback Machine ^ SecurityFocus, Microsoft WebViewFolderIcon ActiveX Control Buffer Overflow Vulnerability Archived 2011-10-11 at the Wayback Machine ^ ^ SecurityFocus, Microsoft WebViewFolderIcon ActiveX Control Buffer Overflow Vulnerability Archived 2011-10-11 at the Wayback Machine Microsoft WebViewFolderIcon ActiveX Control Buffer Overflow Vulnerability Archived Wayback Machine ^ Fusion Authority, Macromedia Flash ActiveX Buffer Overflow Archived August 13, 2011, at the Wayback Machine ^ ^ Fusion Authority, Macromedia Flash ActiveX Buffer Overflow Archived August 13, 2011, at the Wayback Machine Macromedia Flash ActiveX Buffer Overflow Archived Wayback Machine ^ \"Protected Mode in Vista IE7 \u2013 IEBlog\". Blogs.msdn.com. 9 February 2006. Archived from the original on 23 January 2010. Retrieved 24 February 2017. ^ ^ \"Protected Mode in Vista IE7 \u2013 IEBlog\". Blogs.msdn.com. 9 February 2006. Archived from the original on 23 January 2010. Retrieved 24 February 2017. \"Protected Mode in Vista IE7 \u2013 IEBlog\" Archived . Retrieved 24 February 2017 24 February ^ US CERT, Vulnerability Note VU#713878: Microsoft Internet Explorer does not properly validate source of redirected frame Archived 2009-10-30 at the Wayback Machine ^ ^ US CERT, Vulnerability Note VU#713878: Microsoft Internet Explorer does not properly validate source of redirected frame Archived 2009-10-30 at the Wayback Machine Vulnerability Note VU#713878: Microsoft Internet Explorer does not properly validate source of redirected frame Archived Wayback Machine ^ Mozilla Foundation, Mozilla Foundation Security Advisory 2005\u201341: Privilege escalation via DOM property overrides Archived 2014-06-04 at the Wayback Machine ^ ^ Mozilla Foundation, Mozilla Foundation Security Advisory 2005\u201341: Privilege escalation via DOM property overrides Archived 2014-06-04 at the Wayback Machine Mozilla Foundation Security Advisory 2005\u201341: Privilege escalation via DOM property overrides Archived Wayback Machine ^ Andersen, Starr (9 August 2004). \"Part 5: Enhanced Browsing Security\". TechNet. Microsoft Docs. Changes to Functionality in Windows XP Service Pack 2. Retrieved 20 October 2021. ^ ^ Andersen, Starr (9 August 2004). \"Part 5: Enhanced Browsing Security\". TechNet. Microsoft Docs. Changes to Functionality in Windows XP Service Pack 2. Retrieved 20 October 2021. \"Part 5: Enhanced Browsing Security\" TechNet Microsoft Docs . Retrieved 20 October 2021 20 October ^ For one example of a rare JavaScript Trojan Horse, see Symantec Corporation, JS.Seeker.K Archived 2011-09-13 at the Wayback Machine ^ ^ For one example of a rare JavaScript Trojan Horse, see Symantec Corporation, JS.Seeker.K Archived 2011-09-13 at the Wayback Machine JS.Seeker.K Archived Wayback Machine ^ Gruss, Daniel; Maurice, Cl\u00e9mentine; Mangard, Stefan (24 July 2015). \"Rowhammer.js: A Remote Software-Induced Fault Attack in JavaScript\". arXiv:1507.06955 [cs.CR]. ^ ^ Gruss, Daniel; Maurice, Cl\u00e9mentine; Mangard, Stefan (24 July 2015). \"Rowhammer.js: A Remote Software-Induced Fault Attack in JavaScript\". arXiv:1507.06955 [cs.CR]. arXiv 1507.06955 1507.06955 cs.CR ^ Jean-Pharuns, Alix (30 July 2015). \"Rowhammer.js Is the Most Ingenious Hack I've Ever Seen\". Motherboard. Vice. Archived from the original on 27 January 2018. Retrieved 26 January 2018. ^ ^ Jean-Pharuns, Alix (30 July 2015). \"Rowhammer.js Is the Most Ingenious Hack I've Ever Seen\". Motherboard. Vice. Archived from the original on 27 January 2018. Retrieved 26 January 2018. \"Rowhammer.js Is the Most Ingenious Hack I've Ever Seen\" Vice Archived . Retrieved 26 January 2018 26 January ^ Goodin, Dan (4 August 2015). \"DRAM 'Bitflipping' exploit for attacking PCs: Just add JavaScript\". Ars Technica. Archived from the original on 27 January 2018. Retrieved 26 January 2018. ^ ^ Goodin, Dan (4 August 2015). \"DRAM 'Bitflipping' exploit for attacking PCs: Just add JavaScript\". Ars Technica. Archived from the original on 27 January 2018. Retrieved 26 January 2018. \"DRAM 'Bitflipping' exploit for attacking PCs: Just add JavaScript\" Ars Technica Archived . Retrieved 26 January 2018 26 January ^ Auerbach, David (28 July 2015). \"Rowhammer security exploit: Why a new security attack is truly terrifying\". slate.com. Archived from the original on 30 July 2015. Retrieved 29 July 2015. ^ ^ Auerbach, David (28 July 2015). \"Rowhammer security exploit: Why a new security attack is truly terrifying\". slate.com. Archived from the original on 30 July 2015. Retrieved 29 July 2015. Auerbach, David \"Rowhammer security exploit: Why a new security attack is truly terrifying\" Archived . Retrieved 29 July 2015 29 July ^ AnC Archived 2017-03-16 at the Wayback Machine VUSec, 2017 ^ ^ AnC Archived 2017-03-16 at the Wayback Machine VUSec, 2017 AnC Archived Wayback Machine ^ New ASLR-busting JavaScript is about to make drive-by exploits much nastier Archived 2017-03-16 at the Wayback Machine Ars Technica, 2017 ^ ^ New ASLR-busting JavaScript is about to make drive-by exploits much nastier Archived 2017-03-16 at the Wayback Machine Ars Technica, 2017 New ASLR-busting JavaScript is about to make drive-by exploits much nastier Archived Wayback Machine ^ Spectre Attack Archived 2018-01-03 at the Wayback Machine Spectre Attack ^ ^ Spectre Attack Archived 2018-01-03 at the Wayback Machine Spectre Attack Spectre Attack Archived Wayback Machine ^ \"Benchmark.js\". benchmarkjs.com. Archived from the original on 19 December 2016. Retrieved 6 November 2016. ^ ^ \"Benchmark.js\". benchmarkjs.com. Archived from the original on 19 December 2016. Retrieved 6 November 2016. \"Benchmark.js\" Archived . Retrieved 6 November 2016 6 November ^ JSBEN.CH. \"JSBEN.CH Performance Benchmarking Playground for JavaScript\". jsben.ch. Archived from the original on 27 February 2021. Retrieved 13 August 2021. ^ ^ JSBEN.CH. \"JSBEN.CH Performance Benchmarking Playground for JavaScript\". jsben.ch. Archived from the original on 27 February 2021. Retrieved 13 August 2021. \"JSBEN.CH Performance Benchmarking Playground for JavaScript\" Archived . Retrieved 13 August 2021 13 August ^ Eich, Brendan (3 April 2008). \"Popularity\". Archived from the original on 3 July 2011. Retrieved 19 January 2012. ^ ^ Eich, Brendan (3 April 2008). \"Popularity\". Archived from the original on 3 July 2011. Retrieved 19 January 2012. Eich, Brendan \"Popularity\" Archived . Retrieved 19 January 2012 19 January ^ \"Edge Browser Switches WebAssembly to 'On' -- Visual Studio Magazine\". Visual Studio Magazine. Archived from the original on 10 February 2018. Retrieved 9 February 2018. ^ ^ \"Edge Browser Switches WebAssembly to 'On' -- Visual Studio Magazine\". Visual Studio Magazine. Archived from the original on 10 February 2018. Retrieved 9 February 2018. \"Edge Browser Switches WebAssembly to 'On' -- Visual Studio Magazine\" Archived . Retrieved 9 February 2018 9 February ^ \"frequently asked questions\". asm.js. Archived from the original on 4 June 2014. Retrieved 13 April 2014. ^ ^ \"frequently asked questions\". asm.js. Archived from the original on 4 June 2014. Retrieved 13 April 2014. \"frequently asked questions\" Archived . Retrieved 13 April 2014 13 April Sources Dere, Mohan (21 December 2017). \"How to integrate create-react-app with all the libraries you need to make a great app\". freeCodeCamp. Retrieved 14 June 2018. \"How to integrate create-react-app with all the libraries you need to make a great app\" . Retrieved 14 June 2018 14 June Panchal, Krunal (26 April 2022). \"Angular vs React Detailed Comparison\". Groovy Web. Retrieved 5 June 2023. \"Angular vs React Detailed Comparison\" . Retrieved 5 June 2023 5 June Further reading ECMAScript Specification Documents Flanagan, David. JavaScript: The Definitive Guide. 7th edition. Sebastopol, California: O'Reilly, 2020. ISBN 978-1-491-95202-3. ISBN 978-1-491-95202-3 Haverbeke, Marijn. Eloquent JavaScript. 3rd edition. No Starch Press, 2018. 472 pages. ISBN 978-1593279509.(download) ISBN 978-1593279509 (download) Zakas, Nicholas. Principles of Object-Oriented JavaScript, 1st edition. No Starch Press, 2014. 120 pages. ISBN 978-1593275402. ISBN 978-1593275402 External links sister projects sister projects Definitions from Wiktionary Definitions from Wiktionary Definitions Media from Commons Media from Commons Media Textbooks from Wikibooks Textbooks from Wikibooks Textbooks Resources from Wikiversity Resources from Wikiversity Resources Documentation from MediaWiki Documentation from MediaWiki Documentation Listen to this article 48 minutes 48 This audio file (2013-08-20) 2013-08-20 Audio help More spoken articles The Modern JavaScript Tutorial. A community maintained continuously updated collection of tutorials on the entirety of the language. The Modern JavaScript Tutorial \"JavaScript: The First 20 Years\". Retrieved 6 February 2022. \"JavaScript: The First 20 Years\" . Retrieved 6 February 2022 6 February v v t t e e JavaScript Code analysis ESLint ESLint JSHint JSHint JSLint JSLint Supersets JS++ JS++ TypeScript TypeScript ArkTS ArkTS Transpilers AtScript AtScript Babel Babel ClojureScript ClojureScript CoffeeScript CoffeeScript Dart Dart Elm Elm Emscripten Emscripten Google Closure Compiler Google Closure Compiler Google Web Toolkit Google Web Toolkit Haxe Haxe LiveScript LiveScript Morfik Morfik Nim Nim Opa Opa PureScript PureScript Reason Reason WebSharper WebSharper JavaScript library JavaScript library JavaScript syntax JavaScript syntax Debuggers Chrome DevTools Chrome DevTools Firefox Inspector Firefox Inspector Komodo IDE Komodo IDE Microsoft Edge DevTools Microsoft Edge DevTools Opera DevTools Opera DevTools Safari Web Inspector Safari Web Inspector Doc generators JSDoc JSDoc (comparison) comparison Ace Cloud9 IDE Ace Cloud9 IDE Cloud9 IDE Atom Atom CodeMirror Brackets Light Table CodeMirror Brackets Brackets Light Table Light Table PhpStorm PhpStorm Orion Orion Visual Studio Visual Studio Express Visual Studio Visual Studio Express Visual Studio Express Visual Studio Code Visual Studio Code Visual Studio Team Services Visual Studio Team Services Vim Vim Engines List of ECMAScript engines List of ECMAScript engines Frameworks Comparison of JavaScript frameworks Comparison of JavaScript frameworks List of JavaScript libraries List of JavaScript libraries technologies Ajax Ajax AssemblyScript AssemblyScript asm.js asm.js Cascading Style Sheets Cascading Style Sheets Document Object Model Document Object Model HTML HTML5 HTML HTML5 HTML5 JSON JSON WebAssembly WebAssembly WebAuthn WebAuthn Package managers npm npm pnpm pnpm yarn yarn Webpack Webpack Vite Vite esbuild esbuild Server-side Active Server Pages Active Server Pages Bun Bun CommonJS CommonJS Deno Deno JSGI JSGI Node.js Node.js Unit testing (list) list Jasmine Jasmine Mocha Mocha QUnit QUnit Douglas Crockford Douglas Crockford Ryan Dahl Ryan Dahl Brendan Eich Brendan Eich John Resig John Resig v v t t e e Programming languages Comparison Comparison Timeline Timeline History History Ada Ada ALGOL Simula ALGOL Simula Simula APL APL Assembly Assembly BASIC Visual Basic classic .NET BASIC Visual Basic classic .NET Visual Basic classic classic .NET .NET C C C++ C++ C# C# COBOL COBOL Erlang Erlang Forth Forth Fortran Fortran Go Go Haskell Haskell Java Java JavaScript JavaScript Julia Julia Kotlin Kotlin Lisp Lisp Lua Lua MATLAB MATLAB ML ML Pascal Object Pascal Pascal Object Pascal Object Pascal Perl Perl PHP PHP Prolog Prolog Python Python R R Ruby Ruby Rust Rust SQL SQL Scratch Scratch Shell Shell Smalltalk Smalltalk Swift Swift more... more... Lists: Alphabetical Alphabetical Categorical Categorical Generational Generational Non-English-based Non-English-based Category Category v v t t e e ECMAScript ActionScript ActionScript Caja Caja JavaScript engines asm.js JavaScript engines engines asm.js asm.js JScript JScript JScript .NET JScript .NET QtScript QtScript TypeScript TypeScript WMLScript WMLScript Engines Carakan Carakan Futhark Futhark InScript InScript JavaScriptCore JavaScriptCore JScript JScript KJS KJS Linear B Linear B QtScript QtScript Rhino Rhino SpiderMonkey TraceMonkey J\u00e4gerMonkey SpiderMonkey TraceMonkey TraceMonkey J\u00e4gerMonkey J\u00e4gerMonkey Tamarin Tamarin V8 V8 ChakraCore Chakra ChakraCore Chakra Chakra JScript .NET JScript .NET Nashorn Nashorn Frameworks Client-side Dojo Dojo Echo Echo Ext JS Ext JS Google Web Toolkit Google Web Toolkit jQuery jQuery Lively Kernel Lively Kernel midori MochiKit MochiKit MooTools MooTools Prototype Prototype Pyjs Pyjs qooxdoo qooxdoo SproutCore SproutCore Spry Spry Wakanda Framework Server-side Node.js Node.js Deno Deno Bun Bun Jaxer Jaxer AppJet AppJet WakandaDB Cappuccino Libraries Backbone.js Backbone.js SWFObject SWFObject Underscore.js Underscore.js Brendan Eich Brendan Eich Douglas Crockford Douglas Crockford John Resig John Resig Scott Isaacs DHTML DHTML Ecma International Ecma International JSDoc JSDoc JSGI JSGI JSHint JSHint JSLint JSLint JSON JSON JSSS JSSS Sputnik Sputnik SunSpider SunSpider Asynchronous module definition Asynchronous module definition CommonJS CommonJS JavaScript libraries Ajax frameworks JavaScript frameworks server-side JavaScript v v t t e e Web browsers Features standards protocols Bookmarks Bookmarks Extensions Extensions Privacy mode Privacy mode Web standards HTML v5 HTML v5 v5 CSS CSS DOM DOM JavaScript WebAssembly Web storage IndexedDB WebGL WebGPU JavaScript WebAssembly WebAssembly Web storage Web storage IndexedDB IndexedDB WebGL WebGL WebGPU WebGPU HTTP Encryption Cookies third-party HTTP Encryption Encryption Cookies third-party Cookies third-party third-party OCSP OCSP WebRTC WebRTC WebSocket WebSocket Blink Proprietary Google Chrome Google Chrome Arc Arc Avast Avast Coc Coc Coc Coc Comodo Comodo Epic Epic Maxthon Maxthon Microsoft Edge Microsoft Edge Opera (Mobile) Opera Mobile Puffin Puffin QQ QQ Samsung Samsung Silk Silk Sleipnir Sleipnir SRWare SRWare UC UC Vivaldi Vivaldi Whale Whale Yandex Yandex FOSS Chromium Chromium Brave Brave Dooble Dooble Falkon Falkon Otter Otter Supermium Supermium ungoogled ungoogled Gecko Firefox Firefox Floorp Floorp GNU IceCat GNU IceCat LibreWolf LibreWolf Midori Midori SlimBrowser SlimBrowser Tor Tor Zen Zen Gecko forks Basilisk K-Meleon Pale Moon SeaMonkey Waterfox forks Basilisk Basilisk K-Meleon K-Meleon Pale Moon Pale Moon SeaMonkey SeaMonkey Waterfox Waterfox WebKit Safari Safari GNOME Web GNOME Web iCab iCab Orion Orion engine 360 360 DuckDuckGo DuckDuckGo Konqueror Konqueror Lunascape Lunascape NetFront NetFront qutebrowser qutebrowser eww eww Flow Flow Ladybird Ladybird Links Links Lynx Lynx NetSurf NetSurf Opera Mini Opera Mini w3m w3m Blink Beaker Beaker Citrio Citrio Flock Flock Redcore Redcore Rockmelt Rockmelt SalamWeb SalamWeb Sputnik Sputnik Torch Torch Gecko Beonex Beonex Camino Camino Classilla Classilla Conkeror Conkeror Firefox Lite Firefox Lite Galeon Galeon Ghostzilla Ghostzilla IceDragon IceDragon Kazehakase Kazehakase Kylo Kylo Lotus Lotus MicroB MicroB Minimo Minimo Mozilla suite Mozilla suite PirateBrowser PirateBrowser Pogo Pogo Strata Strata Swiftfox Swiftfox Swiftweasel Swiftweasel TenFourFox TenFourFox Timberwolf Timberwolf xB xB MSHTML Internet Explorer Internet Explorer AOL AOL Deepnet Deepnet GreenBrowser GreenBrowser MediaBrowser MediaBrowser NeoPlanet NeoPlanet NetCaptor NetCaptor SpaceTime SpaceTime ZAC ZAC WebKit Arora Arora BOLT BOLT Dolphin Dolphin Fluid Fluid Google TV Google TV Iris Iris Mercury Mercury Nokia Symbian Nokia Symbian OmniWeb OmniWeb Opera Coast Opera Coast Origyn Origyn QtWeb QtWeb Shiira Shiira Steel Steel surf surf Uzbl Uzbl WebPositive WebPositive xombrero xombrero abaco abaco Amaya Amaya Arachne Arachne Arena Arena Blazer Blazer Cake Cake Charon Charon CM CM Deepfish Deepfish Dillo Dillo Edge Legacy Edge Legacy ELinks ELinks Gazelle Gazelle HotJava HotJava IBM Home Page Reader IBM Home Page Reader IBM WebExplorer IBM WebExplorer IBrowse IBrowse Internet Explorer for Mac Internet Explorer for Mac KidZui KidZui Line Mode Line Mode Mosaic Mosaic MSN TV MSN TV NetPositive NetPositive Netscape Netscape Skweezer Skweezer Skyfire Skyfire ThunderHawk ThunderHawk Vision Vision WinWAP WinWAP WorldWideWeb WorldWideWeb Category Category Comparisons Comparisons List List v v t t e e Node.js Node.js Node.js npm npm V8 V8 CommonJS CommonJS MEAN MongoDB Express.js AngularJS/Angular MEAN MongoDB MongoDB Express.js Express.js AngularJS/Angular AngularJS Angular MEEN (substituted with Ember.js) Ember.js Backbone.js Backbone.js Fastify Fastify Meteor Meteor Sails.js (uses Express.js) Sails.js NestJS NestJS Next.js Next.js Nuxt.js Nuxt.js Lodash Lodash Underscore.js Underscore.js React.js React.js Vue.js Vue.js JavaScript JavaScript CoffeeScript CoffeeScript TypeScript TypeScript WebAssembly AssemblyScript C Rust WebAssembly AssemblyScript AssemblyScript C C Rust Rust Portal: Portal Computer programming Computer programming Authority control databases FAST FAST FAST Germany Germany Germany United States United States United States France France France BnF data BnF data BnF data Czech Republic Czech Republic Czech Republic Czech Republic Spain Spain Spain Israel Israel Israel IdRef IdRef IdRef ELMCIP ELMCIP ELMCIP https://en.wikipedia.org/w/index.php?title=JavaScript&oldid=1263283332 Categories JavaScript JavaScript American inventions American inventions Cross-platform software Cross-platform software Dynamically typed programming languages Dynamically typed programming languages Functional languages Functional languages Object-based programming languages Object-based programming languages High-level programming languages High-level programming languages Programming languages created in 1995 Programming languages created in 1995 Programming languages with an ISO standard Programming languages with an ISO standard Prototype-based programming languages Prototype-based programming languages Scripting languages Scripting languages Web programming Web programming Programming languages Programming languages CS1: unfit URL CS1: unfit URL Webarchive template wayback links Webarchive template wayback links Articles with short description Articles with short description Short description matches Wikidata Short description matches Wikidata Wikipedia indefinitely semi-protected pages Wikipedia indefinitely semi-protected pages Use dmy dates from March 2024 Use dmy dates from March 2024 Articles with excerpts Articles with excerpts All articles with unsourced statements All articles with unsourced statements Articles with unsourced statements from August 2023 Articles with unsourced statements from August 2023 All articles with failed verification All articles with failed verification Articles with failed verification from March 2017 Articles with failed verification from March 2017 Pages using Sister project links with wikidata namespace mismatch Pages using Sister project links with wikidata namespace mismatch Pages using Sister project links with hidden wikidata Pages using Sister project links with hidden wikidata Pages using Sister project links with default search Pages using Sister project links with default search Articles with hAudio microformats Articles with hAudio microformats Spoken articles Spoken articles Articles with example JavaScript code Articles with example JavaScript code This page was last edited on 15 December 2024, at 19:41 (UTC). (UTC) Text is available under the Creative Commons Attribution-ShareAlike 4.0 License; additional terms may apply. By using this site, you agree to the Terms of Use and Privacy Policy. Wikipedia\u00ae is a registered trademark of the Wikimedia Foundation, Inc., a non-profit organization. Creative Commons Attribution-ShareAlike 4.0 License Terms of Use Privacy Policy Wikimedia Foundation, Inc. Privacy policy Privacy policy About Wikipedia About Wikipedia Disclaimers Disclaimers Contact Wikipedia Contact Wikipedia Code of Conduct Code of Conduct Developers Developers Statistics Statistics Cookie statement Cookie statement Mobile view Mobile view",
    "https://en.wikipedia.org/wiki/Java_(programming_language)": "Jump to content Main menu Main page Main page Main page Contents Contents Contents Current events Current events Current events Random article Random article Random article About Wikipedia About Wikipedia About Wikipedia Contact us Contact us Contact us Help Help Help Learn to edit Learn to edit Learn to edit Community portal Community portal Community portal Recent changes Recent changes Recent changes Upload file Upload file Upload file Search Search Appearance Donate Donate Donate Create account Create account Create account Log in Log in Log in Personal tools Donate Donate Donate Create account Create account Create account Log in Log in Log in learn more learn more Contributions Contributions Contributions Talk Talk Talk Contents (Top) (Top) 1 History Toggle History subsection 1.1 Principles 1.2 Versions 1 History 1 History Toggle History subsection 1.1 Principles 1.1 Principles 1.1 Principles 1.2 Versions 1.2 Versions 1.2 Versions 2 Editions 2 Editions 2 Editions 3 Execution system Toggle Execution system subsection 3.1 Java JVM and bytecode 3.1.1 Performance 3.2 Non-JVM 3.3 Automatic memory management 3 Execution system 3 Execution system Toggle Execution system subsection 3.1 Java JVM and bytecode 3.1.1 Performance 3.1 Java JVM and bytecode 3.1 Java JVM and bytecode 3.1.1 Performance 3.1.1 Performance 3.1.1 Performance 3.2 Non-JVM 3.2 Non-JVM 3.2 Non-JVM 3.3 Automatic memory management 3.3 Automatic memory management 3.3 Automatic memory management 4 Syntax Toggle Syntax subsection 4.1 Hello world 4 Syntax 4 Syntax Toggle Syntax subsection 4.1 Hello world 4.1 Hello world 4.1 Hello world 5 Special classes Toggle Special classes subsection 5.1 Applet 5.2 Servlet 5.3 JavaServer Pages 5.4 Swing application 5.5 JavaFX application 5.6 Generics 5 Special classes 5 Special classes Toggle Special classes subsection 5.1 Applet 5.1 Applet 5.1 Applet 5.2 Servlet 5.2 Servlet 5.2 Servlet 5.3 JavaServer Pages 5.3 JavaServer Pages 5.3 JavaServer Pages 5.4 Swing application 5.4 Swing application 5.4 Swing application 5.5 JavaFX application 5.5 JavaFX application 5.5 JavaFX application 5.6 Generics 5.6 Generics 5.6 Generics 6 Criticism 6 Criticism 6 Criticism 7 Class libraries 7 Class libraries 7 Class libraries 8 Documentation 8 Documentation 8 Documentation 9 Implementations 9 Implementations 9 Implementations 10 Use outside the Java platform Toggle Use outside the Java platform subsection 10.1 Android 10.1.1 Controversy 10 Use outside the Java platform 10 Use outside the Java platform Toggle Use outside the Java platform subsection 10.1 Android 10.1.1 Controversy 10.1 Android 10.1 Android 10.1.1 Controversy 10.1.1 Controversy 10.1.1 Controversy 11 See also 11 See also 11 See also 12 References 12 References 12 References 13 Bibliography 13 Bibliography 13 Bibliography 14 External links 14 External links 14 External links Toggle the table of contents Java (programming language) Java (programming language) 117 languages Ac\u00e8h Ac\u00e8h Ac\u00e8h Afrikaans Afrikaans Afrikaans \u12a0\u121b\u122d\u129b \u12a0\u121b\u122d\u129b \u12a0\u121b\u122d\u129b \u0627\u0644\u0639\u0631\u0628\u064a\u0629 \u0627\u0644\u0639\u0631\u0628\u064a\u0629 \u0627\u0644\u0639\u0631\u0628\u064a\u0629 Aragon\u00e9s Aragon\u00e9s Aragon\u00e9s \u0985\u09b8\u09ae\u09c0\u09af\u09bc\u09be \u0985\u09b8\u09ae\u09c0\u09af\u09bc\u09be \u0985\u09b8\u09ae\u09c0\u09af\u09bc\u09be Asturianu Asturianu Asturianu Az\u0259rbaycanca Az\u0259rbaycanca Az\u0259rbaycanca \u062a\u06c6\u0631\u06a9\u062c\u0647 \u062a\u06c6\u0631\u06a9\u062c\u0647 \u062a\u06c6\u0631\u06a9\u062c\u0647 \u09ac\u09be\u0982\u09b2\u09be \u09ac\u09be\u0982\u09b2\u09be \u09ac\u09be\u0982\u09b2\u09be \u95a9\u5357\u8a9e / B\u00e2n-l\u00e2m-g\u00fa \u95a9\u5357\u8a9e / B\u00e2n-l\u00e2m-g\u00fa \u95a9\u5357\u8a9e / B\u00e2n-l\u00e2m-g\u00fa \u0411\u0435\u043b\u0430\u0440\u0443\u0441\u043a\u0430\u044f \u0411\u0435\u043b\u0430\u0440\u0443\u0441\u043a\u0430\u044f \u0411\u0435\u043b\u0430\u0440\u0443\u0441\u043a\u0430\u044f \u0411\u0435\u043b\u0430\u0440\u0443\u0441\u043a\u0430\u044f (\u0442\u0430\u0440\u0430\u0448\u043a\u0435\u0432\u0456\u0446\u0430) \u0411\u0435\u043b\u0430\u0440\u0443\u0441\u043a\u0430\u044f (\u0442\u0430\u0440\u0430\u0448\u043a\u0435\u0432\u0456\u0446\u0430) \u0411\u0435\u043b\u0430\u0440\u0443\u0441\u043a\u0430\u044f (\u0442\u0430\u0440\u0430\u0448\u043a\u0435\u0432\u0456\u0446\u0430) \u0411\u044a\u043b\u0433\u0430\u0440\u0441\u043a\u0438 \u0411\u044a\u043b\u0433\u0430\u0440\u0441\u043a\u0438 \u0411\u044a\u043b\u0433\u0430\u0440\u0441\u043a\u0438 Bosanski Bosanski Bosanski Catal\u00e0 Catal\u00e0 Catal\u00e0 \u0427\u04d1\u0432\u0430\u0448\u043b\u0430 \u0427\u04d1\u0432\u0430\u0448\u043b\u0430 \u0427\u04d1\u0432\u0430\u0448\u043b\u0430 \u010ce\u0161tina \u010ce\u0161tina \u010ce\u0161tina Corsu Corsu Corsu Cymraeg Cymraeg Cymraeg Dansk Dansk Dansk \u0627\u0644\u062f\u0627\u0631\u062c\u0629 \u0627\u0644\u062f\u0627\u0631\u062c\u0629 \u0627\u0644\u062f\u0627\u0631\u062c\u0629 Deutsch Deutsch Deutsch Eesti Eesti Eesti \u0395\u03bb\u03bb\u03b7\u03bd\u03b9\u03ba\u03ac \u0395\u03bb\u03bb\u03b7\u03bd\u03b9\u03ba\u03ac \u0395\u03bb\u03bb\u03b7\u03bd\u03b9\u03ba\u03ac Espa\u00f1ol Espa\u00f1ol Espa\u00f1ol Esperanto Esperanto Esperanto Estreme\u00f1u Estreme\u00f1u Estreme\u00f1u Euskara Euskara Euskara \u0641\u0627\u0631\u0633\u06cc \u0641\u0627\u0631\u0633\u06cc \u0641\u0627\u0631\u0633\u06cc Fran\u00e7ais Fran\u00e7ais Fran\u00e7ais Gaeilge Gaeilge Gaeilge Galego Galego Galego \u0a97\u0ac1\u0a9c\u0ab0\u0abe\u0aa4\u0ac0 \u0a97\u0ac1\u0a9c\u0ab0\u0abe\u0aa4\u0ac0 \u0a97\u0ac1\u0a9c\u0ab0\u0abe\u0aa4\u0ac0 \u5ba2\u5bb6\u8a9e / Hak-k\u00e2-ng\u00ee \u5ba2\u5bb6\u8a9e / Hak-k\u00e2-ng\u00ee \u5ba2\u5bb6\u8a9e / Hak-k\u00e2-ng\u00ee \ud55c\uad6d\uc5b4 \ud55c\uad6d\uc5b4 \ud55c\uad6d\uc5b4 Hausa Hausa Hausa \u0540\u0561\u0575\u0565\u0580\u0565\u0576 \u0540\u0561\u0575\u0565\u0580\u0565\u0576 \u0540\u0561\u0575\u0565\u0580\u0565\u0576 \u0939\u093f\u0928\u094d\u0926\u0940 \u0939\u093f\u0928\u094d\u0926\u0940 \u0939\u093f\u0928\u094d\u0926\u0940 Hrvatski Hrvatski Hrvatski Ido Ido Ido Bahasa Indonesia Bahasa Indonesia Bahasa Indonesia Interlingua Interlingua Interlingua \u00cdslenska \u00cdslenska \u00cdslenska Italiano Italiano Italiano \u05e2\u05d1\u05e8\u05d9\u05ea \u05e2\u05d1\u05e8\u05d9\u05ea \u05e2\u05d1\u05e8\u05d9\u05ea \u0c95\u0ca8\u0ccd\u0ca8\u0ca1 \u0c95\u0ca8\u0ccd\u0ca8\u0ca1 \u0c95\u0ca8\u0ccd\u0ca8\u0ca1 \u10e5\u10d0\u10e0\u10d7\u10e3\u10da\u10d8 \u10e5\u10d0\u10e0\u10d7\u10e3\u10da\u10d8 \u10e5\u10d0\u10e0\u10d7\u10e3\u10da\u10d8 \u049a\u0430\u0437\u0430\u049b\u0448\u0430 \u049a\u0430\u0437\u0430\u049b\u0448\u0430 \u049a\u0430\u0437\u0430\u049b\u0448\u0430 Kiswahili Kiswahili Kiswahili Krey\u00f2l ayisyen Krey\u00f2l ayisyen Krey\u00f2l ayisyen Kurd\u00ee Kurd\u00ee Kurd\u00ee \u041a\u044b\u0440\u0433\u044b\u0437\u0447\u0430 \u041a\u044b\u0440\u0433\u044b\u0437\u0447\u0430 \u041a\u044b\u0440\u0433\u044b\u0437\u0447\u0430 Latina Latina Latina Latvie\u0161u Latvie\u0161u Latvie\u0161u Lietuvi\u0173 Lietuvi\u0173 Lietuvi\u0173 Ligure Ligure Ligure Lombard Lombard Lombard Magyar Magyar Magyar \u041c\u0430\u043a\u0435\u0434\u043e\u043d\u0441\u043a\u0438 \u041c\u0430\u043a\u0435\u0434\u043e\u043d\u0441\u043a\u0438 \u041c\u0430\u043a\u0435\u0434\u043e\u043d\u0441\u043a\u0438 \u0d2e\u0d32\u0d2f\u0d3e\u0d33\u0d02 \u0d2e\u0d32\u0d2f\u0d3e\u0d33\u0d02 \u0d2e\u0d32\u0d2f\u0d3e\u0d33\u0d02 \u092e\u0930\u093e\u0920\u0940 \u092e\u0930\u093e\u0920\u0940 \u092e\u0930\u093e\u0920\u0940 \u0645\u0635\u0631\u0649 \u0645\u0635\u0631\u0649 \u0645\u0635\u0631\u0649 Bahasa Melayu Bahasa Melayu Bahasa Melayu Minangkabau Minangkabau Minangkabau \u95a9\u6771\u8a9e / M\u00ecng-d\u0115\u0324ng-ng\u1e73\u0304 \u95a9\u6771\u8a9e / M\u00ecng-d\u0115\u0324ng-ng\u1e73\u0304 \u95a9\u6771\u8a9e / M\u00ecng-d\u0115\u0324ng-ng\u1e73\u0304 \u041c\u043e\u043d\u0433\u043e\u043b \u041c\u043e\u043d\u0433\u043e\u043b \u041c\u043e\u043d\u0433\u043e\u043b \u1019\u103c\u1014\u103a\u1019\u102c\u1018\u102c\u101e\u102c \u1019\u103c\u1014\u103a\u1019\u102c\u1018\u102c\u101e\u102c \u1019\u103c\u1014\u103a\u1019\u102c\u1018\u102c\u101e\u102c Na Vosa Vakaviti Na Vosa Vakaviti Na Vosa Vakaviti Nederlands Nederlands Nederlands \u0928\u0947\u092a\u093e\u0932\u0940 \u0928\u0947\u092a\u093e\u0932\u0940 \u0928\u0947\u092a\u093e\u0932\u0940 \u65e5\u672c\u8a9e \u65e5\u672c\u8a9e \u65e5\u672c\u8a9e Norsk bokm\u00e5l Norsk bokm\u00e5l Norsk bokm\u00e5l Norsk nynorsk Norsk nynorsk Norsk nynorsk Occitan Occitan Occitan \u0b13\u0b21\u0b3c\u0b3f\u0b06 \u0b13\u0b21\u0b3c\u0b3f\u0b06 \u0b13\u0b21\u0b3c\u0b3f\u0b06 O\u02bbzbekcha / \u045e\u0437\u0431\u0435\u043a\u0447\u0430 O\u02bbzbekcha / \u045e\u0437\u0431\u0435\u043a\u0447\u0430 O\u02bbzbekcha / \u045e\u0437\u0431\u0435\u043a\u0447\u0430 \u0a2a\u0a70\u0a1c\u0a3e\u0a2c\u0a40 \u0a2a\u0a70\u0a1c\u0a3e\u0a2c\u0a40 \u0a2a\u0a70\u0a1c\u0a3e\u0a2c\u0a40 \u067e\u0646\u062c\u0627\u0628\u06cc \u067e\u0646\u062c\u0627\u0628\u06cc \u067e\u0646\u062c\u0627\u0628\u06cc Polski Polski Polski Portugu\u00eas Portugu\u00eas Portugu\u00eas Qaraqalpaqsha Qaraqalpaqsha Qaraqalpaqsha Rom\u00e2n\u0103 Rom\u00e2n\u0103 Rom\u00e2n\u0103 Runa Simi Runa Simi Runa Simi \u0420\u0443\u0441\u0441\u043a\u0438\u0439 \u0420\u0443\u0441\u0441\u043a\u0438\u0439 \u0420\u0443\u0441\u0441\u043a\u0438\u0439 \u0421\u0430\u0445\u0430 \u0442\u044b\u043b\u0430 \u0421\u0430\u0445\u0430 \u0442\u044b\u043b\u0430 \u0421\u0430\u0445\u0430 \u0442\u044b\u043b\u0430 \u0938\u0902\u0938\u094d\u0915\u0943\u0924\u092e\u094d \u0938\u0902\u0938\u094d\u0915\u0943\u0924\u092e\u094d \u0938\u0902\u0938\u094d\u0915\u0943\u0924\u092e\u094d Shqip Shqip Shqip \u0dc3\u0dd2\u0d82\u0dc4\u0dbd \u0dc3\u0dd2\u0d82\u0dc4\u0dbd \u0dc3\u0dd2\u0d82\u0dc4\u0dbd Simple English Simple English Simple English \u0633\u0646\u068c\u064a \u0633\u0646\u068c\u064a \u0633\u0646\u068c\u064a Sloven\u010dina Sloven\u010dina Sloven\u010dina Sloven\u0161\u010dina Sloven\u0161\u010dina Sloven\u0161\u010dina Soomaaliga Soomaaliga Soomaaliga \u06a9\u0648\u0631\u062f\u06cc \u06a9\u0648\u0631\u062f\u06cc \u06a9\u0648\u0631\u062f\u06cc \u0421\u0440\u043f\u0441\u043a\u0438 / srpski \u0421\u0440\u043f\u0441\u043a\u0438 / srpski \u0421\u0440\u043f\u0441\u043a\u0438 / srpski Srpskohrvatski / \u0441\u0440\u043f\u0441\u043a\u043e\u0445\u0440\u0432\u0430\u0442\u0441\u043a\u0438 Srpskohrvatski / \u0441\u0440\u043f\u0441\u043a\u043e\u0445\u0440\u0432\u0430\u0442\u0441\u043a\u0438 Srpskohrvatski / \u0441\u0440\u043f\u0441\u043a\u043e\u0445\u0440\u0432\u0430\u0442\u0441\u043a\u0438 Suomi Suomi Suomi Svenska Svenska Svenska Tagalog Tagalog Tagalog \u0ba4\u0bae\u0bbf\u0bb4\u0bcd \u0ba4\u0bae\u0bbf\u0bb4\u0bcd \u0ba4\u0bae\u0bbf\u0bb4\u0bcd Taqbaylit Taqbaylit Taqbaylit \u0422\u0430\u0442\u0430\u0440\u0447\u0430 / tatar\u00e7a \u0422\u0430\u0442\u0430\u0440\u0447\u0430 / tatar\u00e7a \u0422\u0430\u0442\u0430\u0440\u0447\u0430 / tatar\u00e7a \u0c24\u0c46\u0c32\u0c41\u0c17\u0c41 \u0c24\u0c46\u0c32\u0c41\u0c17\u0c41 \u0c24\u0c46\u0c32\u0c41\u0c17\u0c41 \u0e44\u0e17\u0e22 \u0e44\u0e17\u0e22 \u0e44\u0e17\u0e22 T\u00fcrk\u00e7e T\u00fcrk\u00e7e T\u00fcrk\u00e7e Basa Ugi Basa Ugi Basa Ugi \u0423\u043a\u0440\u0430\u0457\u043d\u0441\u044c\u043a\u0430 \u0423\u043a\u0440\u0430\u0457\u043d\u0441\u044c\u043a\u0430 \u0423\u043a\u0440\u0430\u0457\u043d\u0441\u044c\u043a\u0430 \u0627\u0631\u062f\u0648 \u0627\u0631\u062f\u0648 \u0627\u0631\u062f\u0648 V\u00e8neto V\u00e8neto V\u00e8neto Ti\u1ebfng Vi\u1ec7t Ti\u1ebfng Vi\u1ec7t Ti\u1ebfng Vi\u1ec7t \u6587\u8a00 \u6587\u8a00 \u6587\u8a00 Winaray Winaray Winaray \u5434\u8bed \u5434\u8bed \u5434\u8bed \u7cb5\u8a9e \u7cb5\u8a9e \u7cb5\u8a9e \u017demait\u0117\u0161ka \u017demait\u0117\u0161ka \u017demait\u0117\u0161ka \u4e2d\u6587 \u4e2d\u6587 \u4e2d\u6587 Edit links Edit links Article Article Article Talk Talk Talk English Read Read Read View source View source View source View history View history View history Tools Read Read Read View source View source View source View history View history View history What links here What links here What links here Related changes Related changes Related changes Upload file Upload file Upload file Special pages Special pages Special pages Permanent link Permanent link Permanent link Page information Page information Page information Cite this page Cite this page Cite this page Get shortened URL Get shortened URL Get shortened URL Download QR code Download QR code Download QR code Download as PDF Download as PDF Download as PDF Printable version Printable version Printable version Wikimedia Commons Wikimedia Commons Wikimedia Commons Wikibooks Wikibooks Wikibooks Wikiquote Wikiquote Wikiquote Wikiversity Wikiversity Wikiversity Wikidata item Wikidata item Wikidata item Java (software platform) JavaScript Java Javanese language Open frame Paradigm Multi-paradigm generic object-oriented class-based functional imperative reflective concurrent Designed by James Gosling Developer Oracle Corporation ; 29 years ago (1995-05-23) 1995-05-23 [1] [ ] Typing discipline Static, strong, safe nominative manifest Memory management Automatic garbage collection Filename extensions .class .jar .war oracle.com/java/ oracle.com/java/ oracle.com/java/ java.com java.com java.com dev.java dev.java dev.java CLU [2] [ ] Simula67 [2] [ ] Lisp [2] [ ] Smalltalk [2] [ ] Ada 83 C++ [3] [ ] C# [4] [ ] Eiffel [5] [ ] Mesa [6] [ ] Modula-3 [7] [ ] Oberon [8] [ ] Objective-C [9] [ ] UCSD Pascal [10] [ ] [11] [ ] Object Pascal [12] [ ] Ada 2005 BeanShell C# Chapel [13] [ ] Clojure ECMAScript Fantom Gambas [14] [ ] Groovy Hack [15] [ ] Haxe J# Kotlin PHP Python Scala Seed7 Vala JavaScript JS++ ArkTS Java Programming at Wikibooks Java Programming Java is a high-level, class-based, object-oriented programming language that is designed to have as few implementation dependencies as possible. It is a general-purpose programming language intended to let programmers write once, run anywhere (WORA),[16] meaning that compiled Java code can run on all platforms that support Java without the need to recompile.[17] Java applications are typically compiled to bytecode that can run on any Java virtual machine (JVM) regardless of the underlying computer architecture. The syntax of Java is similar to C and C++, but has fewer low-level facilities than either of them. The Java runtime provides dynamic capabilities (such as reflection and runtime code modification) that are typically not available in traditional compiled languages. high-level class-based object-oriented programming language dependencies general-purpose programmers WORA [16] [ ] compiled [17] [ ] bytecode Java virtual machine computer architecture syntax C C++ low-level reflection Java gained popularity shortly after its release, and has been a popular programming language since then.[18] Java was the third most popular programming language in 2022[update] according to GitHub.[19] Although still widely popular, there has been a gradual decline in use of Java in recent years with other languages using JVM gaining popularity.[20] [18] [ ] [update] GitHub [19] [ ] other languages using JVM [20] [ ] Java was originally developed by James Gosling at Sun Microsystems. It was released in May 1995 as a core component of Sun's Java platform. The original and reference implementation Java compilers, virtual machines, and class libraries were originally released by Sun under proprietary licenses. As of May 2007, in compliance with the specifications of the Java Community Process, Sun had relicensed most of its Java technologies under the GPL-2.0-only license. Oracle offers its own HotSpot Java Virtual Machine, however the official reference implementation is the OpenJDK JVM which is free open-source software and used by most developers and is the default JVM for almost all Linux distributions. James Gosling Sun Microsystems Java platform reference implementation compilers class libraries proprietary licenses Java Community Process relicensed GPL-2.0-only Oracle HotSpot reference implementation OpenJDK As of September 2024[update], Java 23 is the latest version (Java 22, and 20 are no longer maintained). Java 8, 11, 17, and 21 are previous LTS versions still officially supported. [update] Java 23 History Java (software platform) \u00a7 History James Gosling James Gosling, Mike Sheridan, and Patrick Naughton initiated the Java language project in June 1991.[21] Java was originally designed for interactive television, but it was too advanced for the digital cable television industry at the time.[22] The language was initially called Oak after an oak tree that stood outside Gosling's office. Later the project went by the name Green and was finally renamed Java, from Java coffee, a type of coffee from Indonesia.[23] Gosling designed Java with a C/C++-style syntax that system and application programmers would find familiar.[24] James Gosling Patrick Naughton [21] [ ] [22] [ ] Oak oak Java coffee Indonesia [23] [ ] C C++ [24] [ ] Sun Microsystems released the first public implementation as Java 1.0 in 1996.[25] It promised write once, run anywhere (WORA) functionality, providing no-cost run-times on popular platforms. Fairly secure and featuring configurable security, it allowed network- and file-access restrictions. Major web browsers soon incorporated the ability to run Java applets within web pages, and Java quickly became popular. The Java 1.0 compiler was re-written in Java by Arthur van Hoff to comply strictly with the Java 1.0 language specification.[26] With the advent of Java 2 (released initially as J2SE 1.2 in December 1998 \u2013 1999), new versions had multiple configurations built for different types of platforms. J2EE included technologies and APIs for enterprise applications typically run in server environments, while J2ME featured APIs optimized for mobile applications. The desktop version was renamed J2SE. In 2006, for marketing purposes, Sun renamed new J2 versions as Java EE, Java ME, and Java SE, respectively. [25] [ ] write once, run anywhere platforms web browsers Java applets in Java Arthur van Hoff [26] [ ] J2EE Java EE Java ME Java SE In 1997, Sun Microsystems approached the ISO/IEC JTC 1 standards body and later the Ecma International to formalize Java, but it soon withdrew from the process.[27][28][29] Java remains a de facto standard, controlled through the Java Community Process.[30] At one time, Sun made most of its Java implementations available without charge, despite their proprietary software status. Sun generated revenue from Java through the selling of licenses for specialized products such as the Java Enterprise System. ISO/IEC JTC 1 Ecma International [27] [ ] [28] [ ] [29] [ ] de facto standard Java Community Process [30] [ ] proprietary software On November 13, 2006, Sun released much of its Java virtual machine (JVM) as free and open-source software (FOSS), under the terms of the GPL-2.0-only license. On May 8, 2007, Sun finished the process, making all of its JVM's core code available under free software/open-source distribution terms, aside from a small portion of code to which Sun did not hold the copyright.[31] free and open-source software GPL-2.0-only free software [31] [ ] Sun's vice-president Rich Green said that Sun's ideal role with regard to Java was as an evangelist.[32] Following Oracle Corporation's acquisition of Sun Microsystems in 2009\u201310, Oracle has described itself as the steward of Java technology with a relentless commitment to fostering a community of participation and transparency.[33] This did not prevent Oracle from filing a lawsuit against Google shortly after that for using Java inside the Android SDK (see the Android section). [32] [ ] Oracle Corporation [33] [ ] Android SDK Android On April 2, 2010, James Gosling resigned from Oracle.[34] Oracle [34] [ ] In January 2016, Oracle announced that Java run-time environments based on JDK 9 will discontinue the browser plugin.[35] [35] [ ] Java software runs on everything from laptops to data centers, game consoles to scientific supercomputers.[36] data centers game consoles supercomputers [36] [ ] Oracle (and others) highly recommend uninstalling outdated and unsupported versions of Java, due to unresolved security issues in older versions.[37] Oracle [37] [ ] Principles There were five primary goals in creating the Java language:[17] [17] [ ] It must be simple, object-oriented, and familiar. object-oriented It must be robust and secure. robust It must be architecture-neutral and portable. It must execute with high performance. It must be interpreted, threaded, and dynamic. interpreted threaded dynamic Versions Java version history As of November 2024[update], Java 8, 11, 17, and 21 are supported as long-term support (LTS) versions, with Java 25, releasing in September 2025, as the next scheduled LTS version.[38] [update] long-term support [38] [ ] Oracle released the last zero-cost public update for the legacy version Java 8 LTS in January 2019 for commercial use, although it will otherwise still support Java 8 with public updates for personal use indefinitely. Other vendors such as Adoptium continue to offer free builds of OpenJDK's long-term support (LTS) versions. These builds may include additional security patches and bug fixes.[39] legacy Java 8 Adoptium [39] [ ] Major release versions of Java, along with their release dates: Beta [40] [ ] [41] [ ] [42] [ ] [43] [ ] Editions Free Java implementations \u00a7 Class library Java platform Java Card Java Card Java ME (Micro Edition) Java ME Java SE (Standard Edition) Java SE Jakarta EE (Enterprise Edition) Jakarta EE JavaFX (bundled in Oracle's JDK from versions 8 to 10 but separately since 11) JavaFX PersonalJava (Discontinued) PersonalJava v v t t e e Sun has defined and supports four editions of Java targeting different application environments and segmented many of its APIs so that they belong to one of the platforms. The platforms are: APIs Java Card for smart-cards.[44] Java Card [44] [ ] Java Platform, Micro Edition (Java ME) \u2013 targeting environments with limited resources.[45] Java Platform, Micro Edition [45] [ ] Java Platform, Standard Edition (Java SE) \u2013 targeting workstation environments.[46] Java Platform, Standard Edition [46] [ ] Java Platform, Enterprise Edition (Java EE) \u2013 targeting large distributed enterprise or Internet environments.[47] Java Platform, Enterprise Edition [47] [ ] The classes in the Java APIs are organized into separate groups called packages. Each package contains a set of related interfaces, classes, subpackages and exceptions. classes packages interfaces exceptions Sun also provided an edition called Personal Java that has been superseded by later, standards-based Java ME configuration-profile pairings. Personal Java Execution system Java JVM and bytecode Java (software platform) Java virtual machine One design goal of Java is portability, which means that programs written for the Java platform must run similarly on any combination of hardware and operating system with adequate run time support. This is achieved by compiling the Java language code to an intermediate representation called Java bytecode, instead of directly to architecture-specific machine code. Java bytecode instructions are analogous to machine code, but they are intended to be executed by a virtual machine (VM) written specifically for the host hardware. End-users commonly use a Java Runtime Environment (JRE) installed on their device for standalone Java applications or a web browser for Java applets. portability Java bytecode machine code virtual machine End-users Java Runtime Environment Java applets Standard libraries provide a generic way to access host-specific features such as graphics, threading, and networking. threading networking The use of universal bytecode makes porting simple. However, the overhead of interpreting bytecode into machine instructions made interpreted programs almost always run more slowly than native executables. Just-in-time (JIT) compilers that compile byte-codes to machine code during runtime were introduced from an early stage. Java's Hotspot compiler is actually two compilers in one; and with GraalVM (included in e.g. Java 11, but removed as of Java 16) allowing tiered compilation.[48] Java itself is platform-independent and is adapted to the particular platform it is to run on by a Java virtual machine (JVM), which translates the Java bytecode into the platform's machine language.[49] interpreting executables Just-in-time GraalVM tiered compilation [48] [ ] Java virtual machine Java bytecode [49] [ ] Performance Java performance Programs written in Java have a reputation for being slower and requiring more memory than those written in C++.[50][51] However, Java programs' execution speed improved significantly with the introduction of just-in-time compilation in 1997/1998 for Java 1.1,[52] the addition of language features supporting better code analysis (such as inner classes, the StringBuilder class, optional assertions, etc.), and optimizations in the Java virtual machine, such as HotSpot becoming Sun's default JVM in 2000. With Java 1.5, the performance was improved with the addition of the java.util.concurrent package, including lock-free implementations of the ConcurrentMaps and other multi-core collections, and it was improved further with Java 1.6. C++ [50] [ ] [51] [ ] just-in-time compilation Java 1.1 [52] [ ] HotSpot lock-free ConcurrentMaps Non-JVM Some platforms offer direct hardware support for Java; there are micro controllers that can run Java bytecode in hardware instead of a software Java virtual machine,[53] and some ARM-based processors could have hardware support for executing Java bytecode through their Jazelle option, though support has mostly been dropped in current implementations of ARM. [53] [ ] ARM Jazelle Automatic memory management Java uses an automatic garbage collector to manage memory in the object lifecycle. The programmer determines when objects are created, and the Java runtime is responsible for recovering the memory once objects are no longer in use. Once no references to an object remain, the unreachable memory becomes eligible to be freed automatically by the garbage collector. Something similar to a memory leak may still occur if a programmer's code holds a reference to an object that is no longer needed, typically when objects that are no longer needed are stored in containers that are still in use.[54] If methods for a non-existent object are called, a null pointer exception is thrown.[55][56] automatic garbage collector object lifecycle unreachable memory memory leak [54] [ ] null pointer [55] [ ] [56] [ ] One of the ideas behind Java's automatic memory management model is that programmers can be spared the burden of having to perform manual memory management. In some languages, memory for the creation of objects is implicitly allocated on the stack or explicitly allocated and deallocated from the heap. In the latter case, the responsibility of managing memory resides with the programmer. If the program does not deallocate an object, a memory leak occurs.[54] If the program attempts to access or deallocate memory that has already been deallocated, the result is undefined and difficult to predict, and the program is likely to become unstable or crash. This can be partially remedied by the use of smart pointers, but these add overhead and complexity. Garbage collection does not prevent logical memory leaks, i.e. those where the memory is still referenced but never used.[54] stack heap memory leak [54] [ ] smart pointers logical memory [54] [ ] Garbage collection may happen at any time. Ideally, it will occur when a program is idle. It is guaranteed to be triggered if there is insufficient free memory on the heap to allocate a new object; this can cause a program to stall momentarily. Explicit memory management is not possible in Java. Java does not support C/C++ style pointer arithmetic, where object addresses can be arithmetically manipulated (e.g. by adding or subtracting an offset). This allows the garbage collector to relocate referenced objects and ensures type safety and security. pointer arithmetic As in C++ and some other object-oriented languages, variables of Java's primitive data types are either stored directly in fields (for objects) or on the stack (for methods) rather than on the heap, as is commonly true for non-primitive data types (but see escape analysis). This was a conscious decision by Java's designers for performance reasons. primitive data types stack escape analysis Java contains multiple types of garbage collectors. Since Java 9, HotSpot uses the Garbage First Garbage Collector (G1GC) as the default.[57] However, there are also several other garbage collectors that can be used to manage the heap, such as the Z Garbage Collector (ZGC) introduced in Java 11, and Shenandoah GC, introduced in Java 12 but unavailable in Oracle-produced OpenJDK builds. Shenandoah is instead available in third-party builds of OpenJDK, such as Eclipse Temurin. For most applications in Java, G1GC is sufficient. In prior versions of Java, such as Java 8, the Parallel Garbage Collector was used as the default garbage collector. Garbage First Garbage Collector [57] [ ] Eclipse Temurin Parallel Garbage Collector Having solved the memory management problem does not relieve the programmer of the burden of handling properly other kinds of resources, like network or database connections, file handles, etc., especially in the presence of exceptions. Syntax Java syntax Gephi The syntax of Java is largely influenced by C++ and C. Unlike C++, which combines the syntax for structured, generic, and object-oriented programming, Java was built almost exclusively as an object-oriented language.[17] All code is written inside classes, and every data item is an object, with the exception of the primitive data types, (i.e. integers, floating-point numbers, boolean values, and characters), which are not objects for performance reasons. Java reuses some popular aspects of C++ (such as the printf method). C++ C [17] [ ] boolean values printf Unlike C++, Java does not support operator overloading[58] or multiple inheritance for classes, though multiple inheritance is supported for interfaces.[59] operator overloading [58] [ ] multiple inheritance interfaces [59] [ ] Java uses comments similar to those of C++. There are three different styles of comments: a single line style marked with two slashes (//), a multiple line style opened with /* and closed with */, and the Javadoc commenting style opened with /** and closed with */. The Javadoc style of commenting allows the user to run the Javadoc executable to create documentation for the program and can be read by some integrated development environments (IDEs) such as Eclipse to allow developers to access documentation within the IDE. comments Javadoc integrated development environments Eclipse Hello world The following is a simple example of a \"Hello, World!\" program that writes a message to the standard output: \"Hello, World!\" program standard output public class Example { public static void main ( String [] args ) { System . out . println ( \"Hello World!\" ); } } Special classes verification Please help improve this article by adding citations to reliable sources in this section. Unsourced material may be challenged and removed.Find sources: \"Java\" programming language \u2013 news \u00b7 newspapers \u00b7 books \u00b7 scholar \u00b7 JSTOR improve this article adding citations to reliable sources Find sources: \"Java\" programming language \u2013 news \u00b7 newspapers \u00b7 books \u00b7 scholar \u00b7 JSTOR \"Java\" programming language news newspapers books scholar JSTOR (May 2019) May 2019 (Learn how and when to remove this message) Learn how and when to remove this message Applet Java applet Java applets are programs embedded in other applications, mainly in web pages displayed in web browsers. The Java applet API was deprecated with the release of Java 9 in 2017.[60][61] embedded [60] [ ] [61] [ ] Servlet Java servlet Java servlet technology provides Web developers with a simple, consistent mechanism for extending the functionality of a Web server and for accessing existing business systems. Servlets are server-side Java EE components that generate responses to requests from clients. Most of the time, this means generating HTML pages in response to HTTP requests, although there are a number of other standard servlet classes available, for example for WebSocket communication. Java servlet server-side clients HTML HTTP WebSocket The Java servlet API has to some extent been superseded (but still used under the hood) by two standard Java technologies for web services: the Java API for RESTful Web Services (JAX-RS 2.0) useful for AJAX, JSON and REST services, and Java API for RESTful Web Services the Java API for XML Web Services (JAX-WS) useful for SOAP Web Services. Java API for XML Web Services SOAP Web Services Typical implementations of these APIs on Application Servers or Servlet Containers use a standard servlet for handling all interactions with the HTTP requests and responses that delegate to the web service methods for the actual business logic. HTTP JavaServer Pages JavaServer Pages JavaServer Pages (JSP) are server-side Java EE components that generate responses, typically HTML pages, to HTTP requests from clients. JSPs embed Java code in an HTML page by using the special delimiters <% and %>. A JSP is compiled to a Java servlet, a Java application in its own right, the first time it is accessed. After that, the generated servlet creates the response.[62] JSP server-side HTML HTTP clients delimiters [62] [ ] Swing application Swing (Java) Swing is a graphical user interface library for the Java SE platform. It is possible to specify a different look and feel through the pluggable look and feel system of Swing. Clones of Windows, GTK+, and Motif are supplied by Sun. Apple also provides an Aqua look and feel for macOS. Where prior implementations of these looks and feels may have been considered lacking, Swing in Java SE 6 addresses this problem by using more native GUI widget drawing routines of the underlying platforms.[63] Swing library pluggable look and feel Windows GTK+ Motif Apple Aqua macOS GUI widget [63] [ ] JavaFX application JavaFX JavaFX is a software platform for creating and delivering desktop applications, as well as rich web applications that can run across a wide variety of devices. JavaFX is intended to replace Swing as the standard graphical user interface (GUI) library for Java SE, but since JDK 11 JavaFX has not been in the core JDK and instead in a separate module.[64] JavaFX has support for desktop computers and web browsers on Microsoft Windows, Linux, and macOS. JavaFX does not have support for native OS look and feels.[65] JavaFX software platform desktop applications rich web applications Swing graphical user interface Java SE [64] [ ] desktop computers web browsers Microsoft Windows Linux macOS [65] [ ] Generics Generics in Java In 2004, generics were added to the Java language, as part of J2SE 5.0. Prior to the introduction of generics, each variable declaration had to be of a specific type. For container classes, for example, this is a problem because there is no easy way to create a container that accepts only specific types of objects. Either the container operates on all subtypes of a class or interface, usually Object, or a different container class has to be created for each contained class. Generics allow compile-time type checking without having to create many container classes, each containing almost identical code. In addition to enabling more efficient code, certain runtime exceptions are prevented from occurring, by issuing compile-time errors. If Java prevented all runtime type errors (ClassCastExceptions) from occurring, it would be type safe. generics type safe In 2016, the type system of Java was proven unsound in that it is possible to use generics to construct classes and methods that allow assignment of an instance of one class to a variable of another unrelated class. Such code is accepted by the compiler, but fails at run time with a class cast exception.[66] unsound [66] [ ] Criticism Criticism of Java Criticisms directed at Java include the implementation of generics,[67] speed,[50] the handling of unsigned numbers,[68] the implementation of floating-point arithmetic,[69] and a history of security vulnerabilities in the primary Java VM implementation HotSpot.[70] Developers have criticized the complexity and verbosity of the Java Persistence API (JPA), a standard part of Java EE. This has led to increased adoption of higher-level abstractions like Spring Data JPA, which aims to simplify database operations and reduce boilerplate code. The growing popularity of such frameworks suggests limitations in the standard JPA implementation's ease-of-use for modern Java development.[71] [67] [ ] [50] [ ] [68] [ ] [69] [ ] HotSpot [70] [ ] [71] [ ] Class libraries Java Class Library The Java Class Library is the standard library, developed to support application development in Java. It is controlled by Oracle in cooperation with others through the Java Community Process program.[72] Companies or individuals participating in this process can influence the design and development of the APIs. This process has been a subject of controversy during the 2010s.[73] The class library contains features such as: Java Class Library standard library Oracle Java Community Process [72] [ ] [73] [ ] The core libraries, which include: Input/output (I/O or IO)[74] and non-blocking I/O (NIO), or IO/NIO[75] Networking[76] (new user agent (HTTP client) since Java 11[77]) Reflective programming (reflection) Concurrent computing (concurrency)[74] Generics Scripting, Compiler Functional programming (Lambda, streaming) Collection libraries that implement data structures such as lists, dictionaries, trees, sets, queues and double-ended queue, or stacks[78] XML Processing (Parsing, Transforming, Validating) libraries Security[79] Internationalization and localization libraries[80] Input/output (I/O or IO)[74] and non-blocking I/O (NIO), or IO/NIO[75] Input/output [74] [ ] non-blocking I/O [75] [ ] Networking[76] (new user agent (HTTP client) since Java 11[77]) Networking [76] [ ] user agent [77] [ ] Reflective programming (reflection) Reflective programming Concurrent computing (concurrency)[74] Concurrent computing [74] [ ] Generics Generics Scripting, Compiler Functional programming (Lambda, streaming) Functional programming Lambda Collection libraries that implement data structures such as lists, dictionaries, trees, sets, queues and double-ended queue, or stacks[78] Collection libraries data structures lists dictionaries trees sets queues double-ended queue stacks [78] [ ] XML Processing (Parsing, Transforming, Validating) libraries XML Security[79] Security [79] [ ] Internationalization and localization libraries[80] Internationalization and localization [80] [ ] The integration libraries, which allow the application writer to communicate with external systems. These libraries include: The Java Database Connectivity (JDBC) API for database access Java Naming and Directory Interface (JNDI) for lookup and discovery Java remote method invocation (RMI) and Common Object Request Broker Architecture (CORBA) for distributed application development Java Management Extensions (JMX) for managing and monitoring applications The Java Database Connectivity (JDBC) API for database access Java Database Connectivity API Java Naming and Directory Interface (JNDI) for lookup and discovery Java Naming and Directory Interface Java remote method invocation (RMI) and Common Object Request Broker Architecture (CORBA) for distributed application development Java remote method invocation Common Object Request Broker Architecture Java Management Extensions (JMX) for managing and monitoring applications Java Management Extensions User interface libraries, which include: The (heavyweight, or native) Abstract Window Toolkit (AWT), which provides GUI components, the means for laying out those components and the means for handling events from those components The (lightweight) Swing libraries, which are built on AWT but provide (non-native) implementations of the AWT widgetry APIs for audio capture, processing, and playback JavaFX User interface The (heavyweight, or native) Abstract Window Toolkit (AWT), which provides GUI components, the means for laying out those components and the means for handling events from those components native Abstract Window Toolkit GUI The (lightweight) Swing libraries, which are built on AWT but provide (non-native) implementations of the AWT widgetry Swing APIs for audio capture, processing, and playback JavaFX JavaFX A platform dependent implementation of the Java virtual machine that is the means by which the bytecodes of the Java libraries and third-party applications are executed Plugins, which enable applets to be run in web browsers applets Java Web Start, which allows Java applications to be efficiently distributed to end users across the Internet Java Web Start end users Licensing and documentation Documentation Javadoc Javadoc is a comprehensive documentation system, created by Sun Microsystems. It provides developers with an organized system for documenting their code. Javadoc comments have an extra asterisk at the beginning, i.e. the delimiters are /** and */, whereas the normal multi-line comments in Java are delimited by /* and */, and single-line comments start with //.[81] Sun Microsystems [81] [ ] Implementations Free Java implementations List of Java compilers verification Please help improve this article by adding citations to reliable sources in this section. Unsourced material may be challenged and removed.Find sources: \"Java\" programming language \u2013 news \u00b7 newspapers \u00b7 books \u00b7 scholar \u00b7 JSTOR improve this article adding citations to reliable sources Find sources: \"Java\" programming language \u2013 news \u00b7 newspapers \u00b7 books \u00b7 scholar \u00b7 JSTOR \"Java\" programming language news newspapers books scholar JSTOR (September 2023) September 2023 (Learn how and when to remove this message) Learn how and when to remove this message Oracle Corporation owns the official implementation of the Java SE platform, due to its acquisition of Sun Microsystems on January 27, 2010. This implementation is based on the original implementation of Java by Sun. The Oracle implementation is available for Windows, macOS, Linux, and Solaris. Because Java lacks any formal standardization recognized by Ecma International, ISO/IEC, ANSI, or other third-party standards organizations, the Oracle implementation is the de facto standard. Oracle Corporation Sun Microsystems Windows macOS Linux Solaris Ecma International de facto standard The Oracle implementation is packaged into two different distributions: The Java Runtime Environment (JRE) which contains the parts of the Java SE platform required to run Java programs and is intended for end users, and the Java Development Kit (JDK), which is intended for software developers and includes development tools such as the Java compiler, Javadoc, Jar, and a debugger. Oracle has also released GraalVM, a high performance Java dynamic compiler and interpreter. Java Development Kit Java compiler Javadoc Jar debugger GraalVM OpenJDK is another Java SE implementation that is licensed under the GNU GPL. The implementation started when Sun began releasing the Java source code under the GPL. As of Java SE 7, OpenJDK is the official Java reference implementation. OpenJDK The goal of Java is to make all implementations of Java compatible. Historically, Sun's trademark license for usage of the Java brand insists that all implementations be compatible. This resulted in a legal dispute with Microsoft after Sun claimed that the Microsoft implementation did not support Java remote method invocation (RMI) or Java Native Interface (JNI) and had added platform-specific features of their own. Sun sued in 1997, and, in 2001, won a settlement of US$20 million, as well as a court order enforcing the terms of the license from Sun.[82] As a result, Microsoft no longer ships Java with Windows. Microsoft Java remote method invocation Java Native Interface [82] [ ] Windows Platform-independent Java is essential to Java EE, and an even more rigorous validation is required to certify an implementation. This environment enables portable server-side applications. Java EE Use outside the Java platform The Java programming language requires the presence of a software platform in order for compiled programs to be executed. Oracle supplies the Java platform for use with Java. The Android SDK is an alternative software platform, used primarily for developing Android applications with its own GUI system. Java platform Android SDK Android applications Android The Java language is a key pillar in Android, an open source mobile operating system. Although Android, built on the Linux kernel, is written largely in C, the Android SDK uses the Java language as the basis for Android applications but does not use any of its standard GUI, SE, ME or other established Java standards.[83] The bytecode language supported by the Android SDK is incompatible with Java bytecode and runs on its own virtual machine, optimized for low-memory devices such as smartphones and tablet computers. Depending on the Android version, the bytecode is either interpreted by the Dalvik virtual machine or compiled into native code by the Android Runtime. Android open source mobile operating system Linux kernel Android SDK [83] [ ] smartphones tablet computers Dalvik virtual machine Android Runtime Android does not provide the full Java SE standard library, although the Android SDK does include an independent implementation of a large subset of it. It supports Java 6 and some Java 7 features, offering an implementation compatible with the standard library (Apache Harmony). Apache Harmony Controversy Oracle America, Inc. v. Google, Inc. The use of Java-related technology in Android led to a legal dispute between Oracle and Google. On May 7, 2012, a San Francisco jury found that if APIs could be copyrighted, then Google had infringed Oracle's copyrights by the use of Java in Android devices.[84] District Judge William Alsup ruled on May 31, 2012, that APIs cannot be copyrighted,[85] but this was reversed by the United States Court of Appeals for the Federal Circuit in May 2014.[86] On May 26, 2016, the district court decided in favor of Google, ruling the copyright infringement of the Java API in Android constitutes fair use.[87] In March 2018, this ruling was overturned by the Appeals Court, which sent down the case of determining the damages to federal court in San Francisco.[88] Google filed a petition for writ of certiorari with the Supreme Court of the United States in January 2019 to challenge the two rulings that were made by the Appeals Court in Oracle's favor.[89] On April 5, 2021, the Court ruled 6\u20132 in Google's favor, that its use of Java APIs should be considered fair use. However, the court refused to rule on the copyrightability of APIs, choosing instead to determine their ruling by considering Java's API copyrightable \"purely for argument's sake.\"[90] [84] [ ] William Alsup [85] [ ] [86] [ ] [87] [ ] [88] [ ] writ of certiorari Supreme Court of the United States [89] [ ] fair use [90] [ ] See also Computer programming portal Computer programming portal Computer programming portal C# C# C++ C++ Dalvik, used in old Android versions, replaced by non-JIT Android Runtime Dalvik Android Runtime Java Heterogeneous Distributed Computing Java Heterogeneous Distributed Computing List of Java APIs List of Java APIs List of Java frameworks List of Java frameworks List of JVM languages List of JVM languages List of Java virtual machines List of Java virtual machines Comparison of C# and Java Comparison of C# and Java Comparison of Java and C++ Comparison of Java and C++ Comparison of programming languages Comparison of programming languages References ^ Binstock, Andrew (May 20, 2015). \"Java's 20 Years of Innovation\". Forbes. Archived from the original on March 14, 2016. Retrieved March 18, 2016. ^ ^ Binstock, Andrew (May 20, 2015). \"Java's 20 Years of Innovation\". Forbes. Archived from the original on March 14, 2016. Retrieved March 18, 2016. \"Java's 20 Years of Innovation\" Archived . Retrieved March 18, 2016 March 18, ^ a b c d Barbara Liskov with John Guttag (2000). Program Development in Java \u2013 Abstraction, Specification, and Object-Oriented Design. USA, Addison Wesley. ISBN 978-0-201-65768-5. ^ a b c d a b c d Barbara Liskov with John Guttag (2000). Program Development in Java \u2013 Abstraction, Specification, and Object-Oriented Design. USA, Addison Wesley. ISBN 978-0-201-65768-5. Barbara Liskov John Guttag ISBN 978-0-201-65768-5 ^ Chaudhary, Harry H. (July 28, 2014). \"Cracking The Java Programming Interview :: 2000+ Java Interview Que/Ans\". Archived from the original on September 29, 2023. Retrieved May 29, 2016. ^ ^ Chaudhary, Harry H. (July 28, 2014). \"Cracking The Java Programming Interview :: 2000+ Java Interview Que/Ans\". Archived from the original on September 29, 2023. Retrieved May 29, 2016. \"Cracking The Java Programming Interview :: 2000+ Java Interview Que/Ans\" Archived . Retrieved May 29, 2016 May 29, ^ Java 5.0 added several new language features (the enhanced for loop, autoboxing, varargs and annotations), after they were introduced in the similar (and competing) C# language. [1] Archived March 19, 2011, at the Wayback Machine [2] Archived January 7, 2006, at the Wayback Machine ^ ^ Java 5.0 added several new language features (the enhanced for loop, autoboxing, varargs and annotations), after they were introduced in the similar (and competing) C# language. [1] Archived March 19, 2011, at the Wayback Machine [2] Archived January 7, 2006, at the Wayback Machine enhanced for loop autoboxing varargs annotations C# [1] Archived Wayback Machine [2] Archived Wayback Machine ^ Gosling, James; McGilton, Henry (May 1996). \"The Java Language Environment\". Archived from the original on May 6, 2014. Retrieved May 6, 2014. ^ ^ Gosling, James; McGilton, Henry (May 1996). \"The Java Language Environment\". Archived from the original on May 6, 2014. Retrieved May 6, 2014. \"The Java Language Environment\" Archived . Retrieved May 6, 2014 May 6, ^ Gosling, James; Joy, Bill; Steele, Guy; Bracha, Gilad. \"The Java Language Specification, 2nd Edition\". Archived from the original on August 5, 2011. Retrieved February 8, 2008. ^ ^ Gosling, James; Joy, Bill; Steele, Guy; Bracha, Gilad. \"The Java Language Specification, 2nd Edition\". Archived from the original on August 5, 2011. Retrieved February 8, 2008. \"The Java Language Specification, 2nd Edition\" Archived . Retrieved February 8, 2008 February 8, ^ \"The A-Z of Programming Languages: Modula-3\". Computerworld. Archived from the original on January 5, 2009. Retrieved June 9, 2010. ^ ^ \"The A-Z of Programming Languages: Modula-3\". Computerworld. Archived from the original on January 5, 2009. Retrieved June 9, 2010. \"The A-Z of Programming Languages: Modula-3\" the original . Retrieved June 9, 2010 June 9, ^ Niklaus Wirth stated on a number of public occasions, e.g. in a lecture at the Polytechnic Museum, Moscow in September 2005 (several independent first-hand accounts in Russian exist, e.g. one with an audio recording: Filippova, Elena (September 22, 2005). \"Niklaus Wirth's lecture at the Polytechnic Museum in Moscow\". Archived from the original on December 1, 2020. Retrieved November 20, 2011.), that the Sun Java design team licensed the Oberon compiler sources a number of years prior to the release of Java and examined it: a (relative) compactness, type safety, garbage collection, no multiple inheritance for classes \u2013 all these key overall design features are shared by Java and Oberon. ^ ^ Niklaus Wirth stated on a number of public occasions, e.g. in a lecture at the Polytechnic Museum, Moscow in September 2005 (several independent first-hand accounts in Russian exist, e.g. one with an audio recording: Filippova, Elena (September 22, 2005). \"Niklaus Wirth's lecture at the Polytechnic Museum in Moscow\". Archived from the original on December 1, 2020. Retrieved November 20, 2011.), that the Sun Java design team licensed the Oberon compiler sources a number of years prior to the release of Java and examined it: a (relative) compactness, type safety, garbage collection, no multiple inheritance for classes \u2013 all these key overall design features are shared by Java and Oberon. Niklaus Wirth \"Niklaus Wirth's lecture at the Polytechnic Museum in Moscow\" Archived . Retrieved November 20, 2011 November 20, ^ Patrick Naughton cites Objective-C as a strong influence on the design of the Java programming language, stating that notable direct derivatives include Java interfaces (derived from Objective-C's protocol) and primitive wrapper classes. [3] Archived July 13, 2011, at the Wayback Machine ^ ^ Patrick Naughton cites Objective-C as a strong influence on the design of the Java programming language, stating that notable direct derivatives include Java interfaces (derived from Objective-C's protocol) and primitive wrapper classes. [3] Archived July 13, 2011, at the Wayback Machine Patrick Naughton Objective-C protocol [3] Archived Wayback Machine ^ TechMetrix Research (1999). \"History of Java\" (PDF). Java Application Servers Report. Archived from the original (PDF) on December 29, 2010. The project went ahead under the name green and the language was based on an old model of UCSD Pascal, which makes it possible to generate interpretive code. ^ ^ TechMetrix Research (1999). \"History of Java\" (PDF). Java Application Servers Report. Archived from the original (PDF) on December 29, 2010. The project went ahead under the name green and the language was based on an old model of UCSD Pascal, which makes it possible to generate interpretive code. \"History of Java\" (PDF) the original (PDF) UCSD Pascal ^ \"A Conversation with James Gosling \u2013 ACM Queue\". Queue.acm.org. August 31, 2004. Archived from the original on July 16, 2015. Retrieved June 9, 2010. ^ ^ \"A Conversation with James Gosling \u2013 ACM Queue\". Queue.acm.org. August 31, 2004. Archived from the original on July 16, 2015. Retrieved June 9, 2010. \"A Conversation with James Gosling \u2013 ACM Queue\" Archived . Retrieved June 9, 2010 June 9, ^ The Java Language Team. About Microsoft's \"Delegates\" (White Paper). JavaSoft, Sun Microsystems, Inc. Archived from the original on June 27, 2012. In the summer of 1996, Sun was designing the precursor to what is now the event model of the AWT and the JavaBeans component architecture. Borland contributed greatly to this process. We looked very carefully at Delphi Object Pascal and built a working prototype of bound method references in order to understand their interaction with the Java programming language and its APIs. ^ ^ The Java Language Team. About Microsoft's \"Delegates\" (White Paper). JavaSoft, Sun Microsystems, Inc. Archived from the original on June 27, 2012. In the summer of 1996, Sun was designing the precursor to what is now the event model of the AWT and the JavaBeans component architecture. Borland contributed greatly to this process. We looked very carefully at Delphi Object Pascal and built a working prototype of bound method references in order to understand their interaction with the Java programming language and its APIs. About Microsoft's \"Delegates\" the original ^ \"Chapel spec (Acknowledgements)\" (PDF). Cray Inc. October 1, 2015. Archived (PDF) from the original on February 5, 2016. Retrieved January 14, 2016. ^ ^ \"Chapel spec (Acknowledgements)\" (PDF). Cray Inc. October 1, 2015. Archived (PDF) from the original on February 5, 2016. Retrieved January 14, 2016. \"Chapel spec (Acknowledgements)\" (PDF) Archived (PDF) . Retrieved January 14, 2016 January 14, ^ \"Gambas Documentation Introduction\". Gambas Website. Archived from the original on October 9, 2017. Retrieved October 9, 2017. ^ ^ \"Gambas Documentation Introduction\". Gambas Website. Archived from the original on October 9, 2017. Retrieved October 9, 2017. \"Gambas Documentation Introduction\" Archived . Retrieved October 9, 2017 October 9, ^ \"Facebook Q&A: Hack brings static typing to PHP world\". InfoWorld. March 26, 2014. Archived from the original on February 13, 2015. Retrieved January 11, 2015. ^ ^ \"Facebook Q&A: Hack brings static typing to PHP world\". InfoWorld. March 26, 2014. Archived from the original on February 13, 2015. Retrieved January 11, 2015. \"Facebook Q&A: Hack brings static typing to PHP world\" Archived . Retrieved January 11, 2015 January 11, ^ \"Write once, run anywhere?\". Computer Weekly. May 2, 2002. Archived from the original on August 13, 2021. Retrieved July 27, 2009. ^ ^ \"Write once, run anywhere?\". Computer Weekly. May 2, 2002. Archived from the original on August 13, 2021. Retrieved July 27, 2009. \"Write once, run anywhere?\" Computer Weekly Archived . Retrieved July 27, 2009 July 27, ^ a b c \"1.2 Design Goals of the Java Programming Language\". Oracle. January 1, 1999. Archived from the original on January 23, 2013. Retrieved January 14, 2013. ^ a b c a b c \"1.2 Design Goals of the Java Programming Language\". Oracle. January 1, 1999. Archived from the original on January 23, 2013. Retrieved January 14, 2013. \"1.2 Design Goals of the Java Programming Language\" Archived . Retrieved January 14, 2013 January 14, ^ Melanson, Mike (August 9, 2022). \"Don't call it a comeback: Why Java is still champ\". GitHub. Archived from the original on August 25, 2023. Retrieved October 15, 2023. ^ ^ Melanson, Mike (August 9, 2022). \"Don't call it a comeback: Why Java is still champ\". GitHub. Archived from the original on August 25, 2023. Retrieved October 15, 2023. \"Don't call it a comeback: Why Java is still champ\" GitHub Archived . Retrieved October 15, 2023 October 15, ^ \"The top programming languages\". The State of the Octoverse. GitHub. Archived from the original on August 2, 2023. Retrieved October 15, 2023. ^ ^ \"The top programming languages\". The State of the Octoverse. GitHub. Archived from the original on August 2, 2023. Retrieved October 15, 2023. \"The top programming languages\" GitHub Archived . Retrieved October 15, 2023 October 15, ^ McMillan, Robert (August 1, 2013). \"Is Java Losing Its Mojo?\". Wired. Archived from the original on February 15, 2017. Retrieved October 15, 2023. ^ ^ McMillan, Robert (August 1, 2013). \"Is Java Losing Its Mojo?\". Wired. Archived from the original on February 15, 2017. Retrieved October 15, 2023. \"Is Java Losing Its Mojo?\" \"Is Java Losing Its Mojo?\" Wired Archived . Retrieved October 15, 2023 October 15, ^ Byous, Jon (c. 1998). \"Java technology: The early years\". Sun Developer Network. Sun Microsystems. Archived from the original on April 20, 2005. Retrieved April 22, 2005. ^ ^ Byous, Jon (c. 1998). \"Java technology: The early years\". Sun Developer Network. Sun Microsystems. Archived from the original on April 20, 2005. Retrieved April 22, 2005. \"Java technology: The early years\" Sun Microsystems the original . Retrieved April 22, 2005 April 22, ^ Object-oriented programming \"The History of Java Technology\". Sun Developer Network. c. 1995. Archived from the original on February 10, 2010. Retrieved April 30, 2010. ^ ^ Object-oriented programming \"The History of Java Technology\". Sun Developer Network. c. 1995. Archived from the original on February 10, 2010. Retrieved April 30, 2010. \"The History of Java Technology\" the original . Retrieved April 30, 2010 April 30, ^ Murphy, Kieron (October 4, 1996). \"So why did they decide to call it Java?\". JavaWorld. Archived from the original on July 13, 2020. Retrieved July 13, 2020. ^ ^ Murphy, Kieron (October 4, 1996). \"So why did they decide to call it Java?\". JavaWorld. Archived from the original on July 13, 2020. Retrieved July 13, 2020. \"So why did they decide to call it Java?\" JavaWorld Archived . Retrieved July 13, 2020 July 13, ^ Kabutz, Heinz; Once Upon an Oak Archived April 13, 2007, at the Wayback Machine. Artima. Retrieved April 29, 2007. ^ ^ Kabutz, Heinz; Once Upon an Oak Archived April 13, 2007, at the Wayback Machine. Artima. Retrieved April 29, 2007. Once Upon an Oak Archived Wayback Machine ^ \"JAVASOFT SHIPS JAVA 1.0\". Archived from the original on March 10, 2007. Retrieved May 13, 2018. ^ ^ \"JAVASOFT SHIPS JAVA 1.0\". Archived from the original on March 10, 2007. Retrieved May 13, 2018. \"JAVASOFT SHIPS JAVA 1.0\" the original . Retrieved May 13, 2018 May 13, ^ Object-oriented Programming with Java: Essentials and Applications. Tata McGraw-Hill Education. p. 34. ^ ^ Object-oriented Programming with Java: Essentials and Applications. Tata McGraw-Hill Education. p. 34. Object-oriented Programming with Java: Essentials and Applications ^ \"JSG \u2013 Java Study Group\". open-std.org. Archived from the original on August 25, 2006. Retrieved August 2, 2006. ^ ^ \"JSG \u2013 Java Study Group\". open-std.org. Archived from the original on August 25, 2006. Retrieved August 2, 2006. \"JSG \u2013 Java Study Group\" Archived . Retrieved August 2, 2006 August 2, ^ \"Why Java Was \u2013 Not \u2013 Standardized Twice\" (PDF). Archived (PDF) from the original on January 13, 2014. Retrieved June 3, 2018. ^ ^ \"Why Java Was \u2013 Not \u2013 Standardized Twice\" (PDF). Archived (PDF) from the original on January 13, 2014. Retrieved June 3, 2018. \"Why Java Was \u2013 Not \u2013 Standardized Twice\" (PDF) Archived (PDF) . Retrieved June 3, 2018 June 3, ^ \"What is ECMA\u2014and why Microsoft cares\". ZDNet. Archived from the original on May 6, 2014. Retrieved May 6, 2014. ^ ^ \"What is ECMA\u2014and why Microsoft cares\". ZDNet. Archived from the original on May 6, 2014. Retrieved May 6, 2014. \"What is ECMA\u2014and why Microsoft cares\" ZDNet Archived . Retrieved May 6, 2014 May 6, ^ \"Java Community Process website\". Jcp.org. May 24, 2010. Archived from the original on August 8, 2006. Retrieved June 9, 2010. ^ ^ \"Java Community Process website\". Jcp.org. May 24, 2010. Archived from the original on August 8, 2006. Retrieved June 9, 2010. \"Java Community Process website\" Archived . Retrieved June 9, 2010 June 9, ^ \"JAVAONE: Sun \u2013 The bulk of Java is open sourced\". GrnLight.net. Archived from the original on May 27, 2014. Retrieved May 26, 2014. ^ ^ \"JAVAONE: Sun \u2013 The bulk of Java is open sourced\". GrnLight.net. Archived from the original on May 27, 2014. Retrieved May 26, 2014. \"JAVAONE: Sun \u2013 The bulk of Java is open sourced\" Archived . Retrieved May 26, 2014 May 26, ^ \"Sun's Evolving Role as Java Evangelist\". O'Reilly Media. Archived from the original on September 15, 2010. Retrieved August 2, 2009. ^ ^ \"Sun's Evolving Role as Java Evangelist\". O'Reilly Media. Archived from the original on September 15, 2010. Retrieved August 2, 2009. \"Sun's Evolving Role as Java Evangelist\" O'Reilly Media Archived . Retrieved August 2, 2009 August 2, ^ \"Oracle and Java\". oracle.com. Oracle Corporation. Archived from the original on January 31, 2010. Retrieved August 23, 2010. Oracle has been a leading and substantive supporter of Java since its emergence in 1995 and takes on the new role as steward of Java technology with a relentless commitment to fostering a community of participation and transparency. ^ ^ \"Oracle and Java\". oracle.com. Oracle Corporation. Archived from the original on January 31, 2010. Retrieved August 23, 2010. Oracle has been a leading and substantive supporter of Java since its emergence in 1995 and takes on the new role as steward of Java technology with a relentless commitment to fostering a community of participation and transparency. \"Oracle and Java\" the original . Retrieved August 23, 2010 August 23, ^ Gosling, James (April 9, 2010). \"Time to move on...\" On a New Road. Archived from the original on November 5, 2010. Retrieved November 16, 2011. ^ ^ Gosling, James (April 9, 2010). \"Time to move on...\" On a New Road. Archived from the original on November 5, 2010. Retrieved November 16, 2011. Gosling, James \"Time to move on...\" the original . Retrieved November 16, 2011 November 16, ^ Topic, Dalibor. \"Moving to a Plugin-Free Web\". Archived from the original on March 16, 2016. Retrieved March 15, 2016. ^ ^ Topic, Dalibor. \"Moving to a Plugin-Free Web\". Archived from the original on March 16, 2016. Retrieved March 15, 2016. \"Moving to a Plugin-Free Web\" Archived . Retrieved March 15, 2016 March 15, ^ \"Learn About Java Technology\". Oracle. Archived from the original on November 24, 2011. Retrieved November 21, 2011. ^ ^ \"Learn About Java Technology\". Oracle. Archived from the original on November 24, 2011. Retrieved November 21, 2011. \"Learn About Java Technology\" Archived . Retrieved November 21, 2011 November 21, ^ \"Why should I uninstall older versions of Java from my system?\". Oracle. Archived from the original on February 12, 2018. Retrieved September 24, 2021. ^ ^ \"Why should I uninstall older versions of Java from my system?\". Oracle. Archived from the original on February 12, 2018. Retrieved September 24, 2021. \"Why should I uninstall older versions of Java from my system?\" Archived . Retrieved September 24, 2021 September 24, ^ \"Oracle Java SE Support Roadmap\". Oracle. September 13, 2021. Archived from the original on September 19, 2021. Retrieved September 18, 2021. ^ ^ \"Oracle Java SE Support Roadmap\". Oracle. September 13, 2021. Archived from the original on September 19, 2021. Retrieved September 18, 2021. \"Oracle Java SE Support Roadmap\" Archived . Retrieved September 18, 2021 September 18, ^ \"Temurin\u2122 Support; Adoptium\". adoptium.net. Archived from the original on March 29, 2024. Retrieved March 29, 2024. ^ ^ \"Temurin\u2122 Support; Adoptium\". adoptium.net. Archived from the original on March 29, 2024. Retrieved March 29, 2024. \"Temurin\u2122 Support; Adoptium\" Archived . Retrieved March 29, 2024 March 29, ^ \"JAVASOFT SHIPS JAVA 1.0\". sun.com. Archived from the original on March 10, 2007. Retrieved February 5, 2008. ^ ^ \"JAVASOFT SHIPS JAVA 1.0\". sun.com. Archived from the original on March 10, 2007. Retrieved February 5, 2008. \"JAVASOFT SHIPS JAVA 1.0\" the original . Retrieved February 5, 2008 February 5, ^ Chander, Sharat. \"Introducing Java SE 11\". oracle.com. Archived from the original on September 26, 2018. Retrieved September 26, 2018. ^ ^ Chander, Sharat. \"Introducing Java SE 11\". oracle.com. Archived from the original on September 26, 2018. Retrieved September 26, 2018. \"Introducing Java SE 11\" Archived . Retrieved September 26, 2018 September 26, ^ Chander, Sharat (September 15, 2020). \"The Arrival of Java 15!\". Oracle. Archived from the original on September 16, 2020. Retrieved September 15, 2020. ^ ^ Chander, Sharat (September 15, 2020). \"The Arrival of Java 15!\". Oracle. Archived from the original on September 16, 2020. Retrieved September 15, 2020. \"The Arrival of Java 15!\" Oracle Archived . Retrieved September 15, 2020 September 15, ^ \"JDK 21\". openjdk.org. Archived from the original on September 20, 2023. Retrieved September 20, 2023. ^ ^ \"JDK 21\". openjdk.org. Archived from the original on September 20, 2023. Retrieved September 20, 2023. \"JDK 21\" Archived . Retrieved September 20, 2023 September 20, ^ \"Java Card Overview\". Oracle Technology Network. Oracle. Archived from the original on January 7, 2015. Retrieved December 18, 2014. ^ ^ \"Java Card Overview\". Oracle Technology Network. Oracle. Archived from the original on January 7, 2015. Retrieved December 18, 2014. \"Java Card Overview\" Archived . Retrieved December 18, 2014 December 18, ^ \"Java Platform, Micro Edition (Java ME)\". Oracle Technology Network. Oracle. Archived from the original on January 4, 2015. Retrieved December 18, 2014. ^ ^ \"Java Platform, Micro Edition (Java ME)\". Oracle Technology Network. Oracle. Archived from the original on January 4, 2015. Retrieved December 18, 2014. \"Java Platform, Micro Edition (Java ME)\" Archived . Retrieved December 18, 2014 December 18, ^ \"Java SE\". Oracle Technology Network. Oracle. Archived from the original on December 24, 2014. Retrieved December 18, 2014. ^ ^ \"Java SE\". Oracle Technology Network. Oracle. Archived from the original on December 24, 2014. Retrieved December 18, 2014. \"Java SE\" Archived . Retrieved December 18, 2014 December 18, ^ \"Java Platform, Enterprise Edition (Java EE)\". Oracle Technology Network. Oracle. Archived from the original on December 17, 2014. Retrieved December 18, 2014. ^ ^ \"Java Platform, Enterprise Edition (Java EE)\". Oracle Technology Network. Oracle. Archived from the original on December 17, 2014. Retrieved December 18, 2014. \"Java Platform, Enterprise Edition (Java EE)\" Archived . Retrieved December 18, 2014 December 18, ^ \"Deep Dive Into the New Java JIT Compiler \u2013 Graal | Baeldung\". www.baeldung.com. August 6, 2021. Archived from the original on October 28, 2021. Retrieved October 13, 2021. ^ ^ \"Deep Dive Into the New Java JIT Compiler \u2013 Graal | Baeldung\". www.baeldung.com. August 6, 2021. Archived from the original on October 28, 2021. Retrieved October 13, 2021. \"Deep Dive Into the New Java JIT Compiler \u2013 Graal | Baeldung\" Archived . Retrieved October 13, 2021 October 13, ^ \"Is the JVM (Java Virtual Machine) platform dependent or platform independent? What is the advantage of using the JVM, and having Java be a translated language?\". Programmer Interview. Archived from the original on January 19, 2015. Retrieved January 19, 2015. ^ ^ \"Is the JVM (Java Virtual Machine) platform dependent or platform independent? What is the advantage of using the JVM, and having Java be a translated language?\". Programmer Interview. Archived from the original on January 19, 2015. Retrieved January 19, 2015. \"Is the JVM (Java Virtual Machine) platform dependent or platform independent? What is the advantage of using the JVM, and having Java be a translated language?\" Archived . Retrieved January 19, 2015 January 19, ^ a b Jelovic, Dejan. \"Why Java will always be slower than C++\". Archived from the original on February 11, 2008. Retrieved February 15, 2008. ^ a b a b Jelovic, Dejan. \"Why Java will always be slower than C++\". Archived from the original on February 11, 2008. Retrieved February 15, 2008. \"Why Java will always be slower than C++\" the original . Retrieved February 15, 2008 February 15, ^ Hundt, Robert. \"Loop Recognition in C++/Java/Go/Scala\" (PDF). Archived (PDF) from the original on November 16, 2011. Retrieved July 12, 2012. ^ ^ Hundt, Robert. \"Loop Recognition in C++/Java/Go/Scala\" (PDF). Archived (PDF) from the original on November 16, 2011. Retrieved July 12, 2012. \"Loop Recognition in C++/Java/Go/Scala\" (PDF) Archived (PDF) . Retrieved July 12, 2012 July 12, ^ \"Symantec's Just-In-Time Java Compiler To Be Integrated into Sun JDK 1.1\". Archived from the original on June 28, 2010. Retrieved August 1, 2009. ^ ^ \"Symantec's Just-In-Time Java Compiler To Be Integrated into Sun JDK 1.1\". Archived from the original on June 28, 2010. Retrieved August 1, 2009. \"Symantec's Just-In-Time Java Compiler To Be Integrated into Sun JDK 1.1\" the original . Retrieved August 1, 2009 August 1, ^ Salcic, Zoran; Park, Heejong; Teich, J\u00fcrgen; Malik, Avinash; Nadeem, Muhammad (July 22, 2017). \"Noc-HMP: A Heterogeneous Multicore Processor for Embedded Systems Designed in SystemJ\". ACM Transactions on Design Automation of Electronic Systems. 22 (4): 73. doi:10.1145/3073416. ISSN 1084-4309. S2CID 11150290. ^ ^ Salcic, Zoran; Park, Heejong; Teich, J\u00fcrgen; Malik, Avinash; Nadeem, Muhammad (July 22, 2017). \"Noc-HMP: A Heterogeneous Multicore Processor for Embedded Systems Designed in SystemJ\". ACM Transactions on Design Automation of Electronic Systems. 22 (4): 73. doi:10.1145/3073416. ISSN 1084-4309. S2CID 11150290. doi 10.1145/3073416 ISSN 1084-4309 S2CID 11150290 ^ a b c Bloch 2018, p. 26-28, \u00a7Item 7: Eliminate obsolete object references. ^ a b c a b c Bloch 2018, p. 26-28, \u00a7Item 7: Eliminate obsolete object references. Bloch 2018 ^ \"NullPointerException\". Oracle. Archived from the original on May 6, 2014. Retrieved May 6, 2014. ^ ^ \"NullPointerException\". Oracle. Archived from the original on May 6, 2014. Retrieved May 6, 2014. \"NullPointerException\" Archived . Retrieved May 6, 2014 May 6, ^ \"Exceptions in Java\". Artima.com. Archived from the original on January 21, 2009. Retrieved August 10, 2010. ^ ^ \"Exceptions in Java\". Artima.com. Archived from the original on January 21, 2009. Retrieved August 10, 2010. \"Exceptions in Java\" Archived . Retrieved August 10, 2010 August 10, ^ \"Java HotSpot\u2122 Virtual Machine Performance Enhancements\". Oracle.com. Archived from the original on May 29, 2017. Retrieved April 26, 2017. ^ ^ \"Java HotSpot\u2122 Virtual Machine Performance Enhancements\". Oracle.com. Archived from the original on May 29, 2017. Retrieved April 26, 2017. \"Java HotSpot\u2122 Virtual Machine Performance Enhancements\" Archived . Retrieved April 26, 2017 April 26, ^ \"Operator Overloading (C# vs Java)\". C# for Java Developers. Microsoft. Archived from the original on January 7, 2015. Retrieved December 10, 2014. ^ ^ \"Operator Overloading (C# vs Java)\". C# for Java Developers. Microsoft. Archived from the original on January 7, 2015. Retrieved December 10, 2014. \"Operator Overloading (C# vs Java)\" Archived . Retrieved December 10, 2014 December 10, ^ \"Multiple Inheritance of State, Implementation, and Type\". The Java Tutorials. Oracle. Archived from the original on November 9, 2014. Retrieved December 10, 2014. ^ ^ \"Multiple Inheritance of State, Implementation, and Type\". The Java Tutorials. Oracle. Archived from the original on November 9, 2014. Retrieved December 10, 2014. \"Multiple Inheritance of State, Implementation, and Type\" Archived . Retrieved December 10, 2014 December 10, ^ \"Deprecated APIs, Features, and Options\". Oracle. Archived from the original on June 19, 2019. Retrieved May 31, 2019. ^ ^ \"Deprecated APIs, Features, and Options\". Oracle. Archived from the original on June 19, 2019. Retrieved May 31, 2019. \"Deprecated APIs, Features, and Options\" Archived . Retrieved May 31, 2019 May 31, ^ \"Applet (Java Platform SE 7)\". Docs. Oracle. Archived from the original on August 2, 2020. Retrieved May 1, 2020. ^ ^ \"Applet (Java Platform SE 7)\". Docs. Oracle. Archived from the original on August 2, 2020. Retrieved May 1, 2020. \"Applet (Java Platform SE 7)\" Archived . Retrieved May 1, 2020 May 1, ^ \"What Is a JSP Page? - The Java EE 5 Tutorial\". docs.oracle.com. Archived from the original on August 2, 2020. Retrieved May 1, 2020. ^ ^ \"What Is a JSP Page? - The Java EE 5 Tutorial\". docs.oracle.com. Archived from the original on August 2, 2020. Retrieved May 1, 2020. \"What Is a JSP Page? - The Java EE 5 Tutorial\" Archived . Retrieved May 1, 2020 May 1, ^ \"Trail: Creating a GUI With JFC/Swing (The Java Tutorials)\". docs.oracle.com. Archived from the original on April 29, 2020. Retrieved May 1, 2020. ^ ^ \"Trail: Creating a GUI With JFC/Swing (The Java Tutorials)\". docs.oracle.com. Archived from the original on April 29, 2020. Retrieved May 1, 2020. \"Trail: Creating a GUI With JFC/Swing (The Java Tutorials)\" Archived . Retrieved May 1, 2020 May 1, ^ \"Removed from JDK 11, JavaFX 11 arrives as a standalone module\". InfoWorld. September 20, 2018. Archived from the original on October 14, 2020. Retrieved October 13, 2020. ^ ^ \"Removed from JDK 11, JavaFX 11 arrives as a standalone module\". InfoWorld. September 20, 2018. Archived from the original on October 14, 2020. Retrieved October 13, 2020. \"Removed from JDK 11, JavaFX 11 arrives as a standalone module\" Archived . Retrieved October 13, 2020 October 13, ^ \"Getting Started with JavaFX: Hello World, JavaFX Style\". JavaFX 2 Tutorials and Documentation. Oracle. Archived from the original on August 2, 2020. Retrieved May 1, 2020. ^ ^ \"Getting Started with JavaFX: Hello World, JavaFX Style\". JavaFX 2 Tutorials and Documentation. Oracle. Archived from the original on August 2, 2020. Retrieved May 1, 2020. \"Getting Started with JavaFX: Hello World, JavaFX Style\" Archived . Retrieved May 1, 2020 May 1, ^ \"Java and Scala's Type Systems are Unsound\" (PDF). Archived (PDF) from the original on November 28, 2016. Retrieved February 20, 2017. ^ ^ \"Java and Scala's Type Systems are Unsound\" (PDF). Archived (PDF) from the original on November 28, 2016. Retrieved February 20, 2017. \"Java and Scala's Type Systems are Unsound\" (PDF) Archived (PDF) . Retrieved February 20, 2017 February 20, ^ Arnold, Ken (June 27, 2005). \"Generics Considered Harmful\". java.net. Archived from the original on October 10, 2007. Retrieved September 10, 2015. ^ ^ Arnold, Ken (June 27, 2005). \"Generics Considered Harmful\". java.net. Archived from the original on October 10, 2007. Retrieved September 10, 2015. \"Generics Considered Harmful\" the original . Retrieved September 10, 2015 September 10, ^ Owens, Sean R. \"Java and unsigned int, unsigned short, unsigned byte, unsigned long, etc. (Or rather, the lack thereof)\". Archived from the original on February 20, 2009. Retrieved July 4, 2011. ^ ^ Owens, Sean R. \"Java and unsigned int, unsigned short, unsigned byte, unsigned long, etc. (Or rather, the lack thereof)\". Archived from the original on February 20, 2009. Retrieved July 4, 2011. \"Java and unsigned int, unsigned short, unsigned byte, unsigned long, etc. (Or rather, the lack thereof)\" the original . Retrieved July 4, 2011 July 4, ^ Kahan, William (March 1, 1998). \"How Java's Floating-Point Hurts Everyone Everywhere \u2013 ACM 1998 Workshop on Java (Stanford)\" (PDF). Electrical Engineering & Computer Science, University of California at Berkeley. Archived (PDF) from the original on September 5, 2012. Retrieved June 4, 2011. ^ ^ Kahan, William (March 1, 1998). \"How Java's Floating-Point Hurts Everyone Everywhere \u2013 ACM 1998 Workshop on Java (Stanford)\" (PDF). Electrical Engineering & Computer Science, University of California at Berkeley. Archived (PDF) from the original on September 5, 2012. Retrieved June 4, 2011. \"How Java's Floating-Point Hurts Everyone Everywhere \u2013 ACM 1998 Workshop on Java (Stanford)\" (PDF) Archived (PDF) . Retrieved June 4, 2011 June 4, ^ \"Have you checked the Java?\". Archived from the original on September 21, 2012. Retrieved December 23, 2011. ^ ^ \"Have you checked the Java?\". Archived from the original on September 21, 2012. Retrieved December 23, 2011. \"Have you checked the Java?\" the original . Retrieved December 23, 2011 December 23, ^ Chidester, Ashlan. Java Persistence API, Jenkins and AWS. ISBN 9798224253951. Retrieved September 16, 2024. ^ ^ Chidester, Ashlan. Java Persistence API, Jenkins and AWS. ISBN 9798224253951. Retrieved September 16, 2024. Java Persistence API, Jenkins and AWS ISBN 9798224253951 . Retrieved September 16, 2024 September 16, ^ Cadenhead, Rogers (November 20, 2017), Understanding How Java Programs Work, archived from the original on August 13, 2021, retrieved March 26, 2019 ^ ^ Cadenhead, Rogers (November 20, 2017), Understanding How Java Programs Work, archived from the original on August 13, 2021, retrieved March 26, 2019 Understanding How Java Programs Work archived , retrieved March 26, 2019 March 26, ^ Woolf, Nicky (May 26, 2016). \"Google wins six-year legal battle with Oracle over Android code copyright\". The Guardian. ISSN 0261-3077. Archived from the original on March 26, 2019. Retrieved March 26, 2019. ^ ^ Woolf, Nicky (May 26, 2016). \"Google wins six-year legal battle with Oracle over Android code copyright\". The Guardian. ISSN 0261-3077. Archived from the original on March 26, 2019. Retrieved March 26, 2019. \"Google wins six-year legal battle with Oracle over Android code copyright\" ISSN 0261-3077 Archived . Retrieved March 26, 2019 March 26, ^ a b Bloch 2018, pp. 1\u20134, \u00a7 1 Introduction. ^ a b a b Bloch 2018, pp. 1\u20134, \u00a7 1 Introduction. Bloch 2018 ^ \"java.nio (Java Platform SE 8)\". docs.oracle.com. ^ ^ \"java.nio (Java Platform SE 8)\". docs.oracle.com. \"java.nio (Java Platform SE 8)\" ^ \"Java Networking\". docs.oracle.com. ^ ^ \"Java Networking\". docs.oracle.com. \"Java Networking\" ^ \"HttpClient (Java SE 11 & JDK 11)\". docs.oracle.com. ^ ^ \"HttpClient (Java SE 11 & JDK 11)\". docs.oracle.com. \"HttpClient (Java SE 11 & JDK 11)\" ^ \"Collections Framework Overview\". Java Documentation. Oracle. Archived from the original on December 31, 2014. Retrieved December 18, 2014. ^ ^ \"Collections Framework Overview\". Java Documentation. Oracle. Archived from the original on December 31, 2014. Retrieved December 18, 2014. \"Collections Framework Overview\" Archived . Retrieved December 18, 2014 December 18, ^ \"Java Security Overview\". Java Documentation. Oracle. Archived from the original on January 3, 2015. Retrieved December 18, 2014. ^ ^ \"Java Security Overview\". Java Documentation. Oracle. Archived from the original on January 3, 2015. Retrieved December 18, 2014. \"Java Security Overview\" Archived . Retrieved December 18, 2014 December 18, ^ \"Trail: Internationalization\". The Java Tutorials. Oracle. Archived from the original on December 31, 2014. Retrieved December 18, 2014. ^ ^ \"Trail: Internationalization\". The Java Tutorials. Oracle. Archived from the original on December 31, 2014. Retrieved December 18, 2014. \"Trail: Internationalization\" Archived . Retrieved December 18, 2014 December 18, ^ \"How to Write Doc Comments for the Javadoc Tool\". Oracle Technology Network. Oracle. Archived from the original on December 18, 2014. Retrieved December 18, 2014. ^ ^ \"How to Write Doc Comments for the Javadoc Tool\". Oracle Technology Network. Oracle. Archived from the original on December 18, 2014. Retrieved December 18, 2014. \"How to Write Doc Comments for the Javadoc Tool\" Archived . Retrieved December 18, 2014 December 18, ^ Niccolai, James (January 24, 2001). \"Sun, Microsoft settle Java lawsuit\". JavaWorld. IDG News Service. Archived from the original on July 14, 2020. Retrieved July 13, 2020. ^ ^ Niccolai, James (January 24, 2001). \"Sun, Microsoft settle Java lawsuit\". JavaWorld. IDG News Service. Archived from the original on July 14, 2020. Retrieved July 13, 2020. \"Sun, Microsoft settle Java lawsuit\" JavaWorld IDG News Service Archived . Retrieved July 13, 2020 July 13, ^ van Gurp, Jilles (November 13, 2007). \"Google Android: Initial Impressions and Criticism\". Javalobby. Archived from the original on August 28, 2008. Retrieved March 7, 2009. Frankly, I don't understand why Google intends to ignore the vast amount of existing implementation out there. It seems like a bad case of \"not invented here\" to me. Ultimately, this will slow adoption. There are already too many Java platforms for the mobile world and this is yet another one ^ ^ van Gurp, Jilles (November 13, 2007). \"Google Android: Initial Impressions and Criticism\". Javalobby. Archived from the original on August 28, 2008. Retrieved March 7, 2009. Frankly, I don't understand why Google intends to ignore the vast amount of existing implementation out there. It seems like a bad case of \"not invented here\" to me. Ultimately, this will slow adoption. There are already too many Java platforms for the mobile world and this is yet another one \"Google Android: Initial Impressions and Criticism\" . Retrieved March 7, 2009 March 7, ^ Mullin, Joe (May 7, 2012). \"Google guilty of infringement in Oracle trial; future legal headaches loom\". Law & Disorder. Ars Technica. Archived from the original on May 8, 2012. Retrieved May 8, 2012. ^ ^ Mullin, Joe (May 7, 2012). \"Google guilty of infringement in Oracle trial; future legal headaches loom\". Law & Disorder. Ars Technica. Archived from the original on May 8, 2012. Retrieved May 8, 2012. \"Google guilty of infringement in Oracle trial; future legal headaches loom\" Archived . Retrieved May 8, 2012 May 8, ^ Mullin, Joe (May 31, 2012). \"Google wins crucial API ruling, Oracle's case decimated\". Ars Technica. Archived from the original on March 12, 2017. Retrieved June 1, 2012. ^ ^ Mullin, Joe (May 31, 2012). \"Google wins crucial API ruling, Oracle's case decimated\". Ars Technica. Archived from the original on March 12, 2017. Retrieved June 1, 2012. \"Google wins crucial API ruling, Oracle's case decimated\" Archived . Retrieved June 1, 2012 June 1, ^ Rosenblatt, Seth (May 9, 2014). \"Court sides with Oracle over Android in Java patent appeal\". CNET. Archived from the original on May 10, 2014. Retrieved May 10, 2014. ^ ^ Rosenblatt, Seth (May 9, 2014). \"Court sides with Oracle over Android in Java patent appeal\". CNET. Archived from the original on May 10, 2014. Retrieved May 10, 2014. \"Court sides with Oracle over Android in Java patent appeal\" Archived . Retrieved May 10, 2014 May 10, ^ Mullin, Joe (May 26, 2016). \"Google beats Oracle\u2014Android makes \"fair use\" of Java APIs\". Ars Technica. Archived from the original on January 20, 2017. Retrieved May 26, 2016. ^ ^ Mullin, Joe (May 26, 2016). \"Google beats Oracle\u2014Android makes \"fair use\" of Java APIs\". Ars Technica. Archived from the original on January 20, 2017. Retrieved May 26, 2016. \"Google beats Oracle\u2014Android makes \"fair use\" of Java APIs\" Archived . Retrieved May 26, 2016 May 26, ^ Farivar, Cyrus (March 27, 2018). \"\"Google's use of the Java API packages was not fair,\" appeals court rules\". Ars Technica. Archived from the original on September 24, 2019. Retrieved August 6, 2019. ^ ^ Farivar, Cyrus (March 27, 2018). \"\"Google's use of the Java API packages was not fair,\" appeals court rules\". Ars Technica. Archived from the original on September 24, 2019. Retrieved August 6, 2019. \"\"Google's use of the Java API packages was not fair,\" appeals court rules\" Archived . Retrieved August 6, 2019 August 6, ^ Lee, Timothy (April 23, 2019). \"Google asks Supreme Court to overrule disastrous ruling on API copyrights\". Ars Technica. Archived from the original on April 23, 2019. Retrieved April 23, 2019. ^ ^ Lee, Timothy (April 23, 2019). \"Google asks Supreme Court to overrule disastrous ruling on API copyrights\". Ars Technica. Archived from the original on April 23, 2019. Retrieved April 23, 2019. \"Google asks Supreme Court to overrule disastrous ruling on API copyrights\" Ars Technica Archived . Retrieved April 23, 2019 April 23, ^ \"Google LLC v. Oracle America, Inc 593 U. S. ____ (2021)\" (PDF). Archived (PDF) from the original on April 5, 2021. Retrieved April 6, 2021. ^ ^ \"Google LLC v. Oracle America, Inc 593 U. S. ____ (2021)\" (PDF). Archived (PDF) from the original on April 5, 2021. Retrieved April 6, 2021. \"Google LLC v. Oracle America, Inc 593 U. S. ____ (2021)\" (PDF) Archived (PDF) . Retrieved April 6, 2021 April 6, Bibliography Bloch, Joshua (2018). \"Effective Java: Programming Language Guide\" (third ed.). Addison-Wesley. ISBN 978-0-13-468599-1. ISBN 978-0-13-468599-1 Gosling, James; Joy, Bill; Steele, Guy; Bracha, Gilad; Buckley, Alex (2014). The Java\u00ae Language Specification (PDF) (Java SE 8 ed.). Archived (PDF) from the original on October 21, 2014. Retrieved November 18, 2014. The Java\u00ae Language Specification (PDF) Archived (PDF) . Retrieved November 18, 2014 November 18, Gosling, James; Joy, Bill; Steele, Guy L. Jr.; Bracha, Gilad (2005). The Java Language Specification (3rd ed.). Addison-Wesley. ISBN 0-321-24678-0. Archived from the original on February 14, 2012. Retrieved February 8, 2019. Joy, Bill Steele, Guy L. Jr. Bracha, Gilad The Java Language Specification ISBN 0-321-24678-0 Archived . Retrieved February 8, 2019 February 8, Lindholm, Tim; Yellin, Frank (1999). The Java Virtual Machine Specification (2nd ed.). Addison-Wesley. ISBN 0-201-43294-3. Archived from the original on September 25, 2011. Retrieved February 8, 2019. The Java Virtual Machine Specification ISBN 0-201-43294-3 Archived . Retrieved February 8, 2019 February 8, External links sister projects sister projects Definitions from Wiktionary Definitions from Wiktionary Definitions Media from Commons Media from Commons Media Quotations from Wikiquote Quotations from Wikiquote Quotations Textbooks from Wikibooks Textbooks from Wikibooks Textbooks Resources from Wikiversity Resources from Wikiversity Resources Data from Wikidata Data from Wikidata Data Java Platform, Enterprise Edition/Java EE Tutorial Java Weekly Java Weekly v v t t e e Java (software platform) JVM JVM Java ME (Micro) Java ME Java SE (Standard) Java SE Jakarta EE (Enterprise) Jakarta EE Java Card Java Card Android SDK Android SDK GraalVM GraalVM Squawk Squawk Java Development Kit Java Development Kit OpenJDK OpenJDK Java virtual machine Java virtual machine JavaFX JavaFX Maxine VM Maxine VM Servlets Servlets MIDlets MIDlets JSP JSP JSF JSF Web Start (JNLP) Web Start Pack200 Pack200 Modules Modules applets applets Eclipse Eclipse GNU Classpath GNU Classpath Hibernate Hibernate Spring Spring GWT GWT IcedTea IcedTea Jazelle Jazelle Struts Struts TopLink TopLink WildFly WildFly blackdown blackdown harmony harmony Java version history Java version history Java Community Process Java Community Process Sun Microsystems Sun Microsystems Free Java implementations Free Java implementations JVM languages Java Java BeanShell BeanShell Clojure Clojure Groovy Groovy JRuby JRuby Jython Jython Kotlin Kotlin Processing Processing Rhino Rhino Scala Scala Oxygene Oxygene Conferences JavaOne JavaOne Devoxx Devoxx Apache Software Foundation Apache Software Foundation Eclipse Foundation Eclipse Foundation Java Community Process Java Community Process Oracle Corporation Oracle Corporation Sun Microsystems, Sun Microsystems Laboratories Sun Microsystems Sun Microsystems Laboratories James Gosling James Gosling Arthur van Hoff Arthur van Hoff Urs H\u00f6lzle Urs H\u00f6lzle Patrick Naughton Patrick Naughton Category Computer programming portal v v t t e e Sun Microsystems Acquired by Oracle Sun-1 Sun-1 Sun-2 Sun-2 Sun-3 Sun-3 Sun386i Sun386i Sun-4 Sun-4 SPARCstation/server/center 1 2 4 5 10 20 IPC IPX LX ZX SPARCstation/server/center 1 1 2 2 4 4 5 5 10 10 20 20 IPC IPC IPX IPX LX LX ZX ZX SPARCclassic SPARCclassic Netra Netra Ultra 1 2 5 10 24 30 60 80 Ultra 1 1 2 2 5 5 10 10 24 24 30 30 60 60 80 80 Enterprise 10000 Enterprise 10000 10000 Blade server Blade server Blade workstation Blade workstation Fire 12K 15K E25K T2000 X4500 Fire 12K 12K 15K 15K E25K E25K T2000 T2000 X4500 X4500 SPARC Enterprise SPARC Enterprise JavaStation JavaStation Java Workstation Java Workstation Ray Ray Cobalt Qube Cobalt Qube Cobalt RaQ Cobalt RaQ SPARC SPARC MB86900 MB86900 microSPARC microSPARC SuperSPARC SuperSPARC UltraSPARC UltraSPARC UltraSPARC II UltraSPARC IIe UltraSPARC IIi Gemini UltraSPARC II UltraSPARC IIe UltraSPARC IIi Gemini UltraSPARC III UltraSPARC III Cu UltraSPARC IIIi UltraSPARC III UltraSPARC III Cu UltraSPARC IIIi UltraSPARC IV UltraSPARC IV UltraSPARC T1 UltraSPARC T1 UltraSPARC T2 UltraSPARC T2 SPARC T3 SPARC T3 SPARC T4 SPARC T4 SPARC T5 SPARC T5 Rock Rock MAJC MAJC Sun4d Sun4d SBus SBus Fireplane Fireplane LOM port LOM port MBus MBus Modular Datacenter Modular Datacenter Neptune Neptune System Service Processor System Service Processor SPARC T series SPARC T series SPOT SPOT Ultra Port Architecture Ultra Port Architecture Visual Instruction Set Visual Instruction Set SunOS SunOS Solaris Solaris NIS NIS+ NIS NIS+ NIS+ NFS NFS ZFS ZFS+ ZFS ZFS+ ZFS+ SunView SunView NeWS NeWS OpenWindows OpenWindows Java Desktop System Java Desktop System Studio Studio Java Java StarOffice StarOffice iPlanet iPlanet Java System Java System Sun Secure Global Desktop Sun Secure Global Desktop MySQL MySQL xVM xVM GlassFish GlassFish VirtualBox VirtualBox StorageTek 5800 System StorageTek 5800 System StorageTek SL8500 StorageTek SL8500 Open Storage Open Storage QFS QFS ZFS ZFS Cloud Cloud Constellation System Constellation System Visualization System Visualization System Grid Engine Grid Engine Lustre Lustre Sun Microsystems Laboratories Sun Microsystems Laboratories picoJava picoJava Fortress Fortress Project Looking Glass Project Looking Glass SCPs SCPs BlueJ BlueJ Common Development and Distribution License Common Development and Distribution License Java Community Process Java Community Process NetBeans NetBeans OpenOffice.org OpenOffice.org OpenSolaris OpenSolaris OpenSPARC OpenSPARC OpenJDK OpenJDK Open Source University Meetup Open Source University Meetup People Bill Joy Bill Joy Andy Bechtolsheim Andy Bechtolsheim Scott McNealy Scott McNealy Vinod Khosla Vinod Khosla Afara Websystems Afara Websystems Cobalt Networks Cobalt Networks Gridware Gridware Lighthouse Design Lighthouse Design Montalvo Systems Montalvo Systems MySQL AB MySQL AB Pixo Pixo Procom Technology Procom Technology SavaJe SavaJe StorageTek StorageTek Tarantella Tarantella The Network is the Computer The Network is the Computer Write once, run anywhere Write once, run anywhere Category v v t t e e Oracle Corporation Jeffrey Berg Jeffrey Berg H. Raymond Bingham H. Raymond Bingham Michael Boskin Michael Boskin Safra Catz Safra Catz Larry Ellison Larry Ellison H\u00e9ctor Garc\u00eda-Molina H\u00e9ctor Garc\u00eda-Molina Joseph Grundfest Joseph Grundfest Jeffrey O. Henley Jeffrey O. Henley Mark Hurd Mark Hurd Jack F. Kemp Jack F. Kemp Donald L. Lucas Donald L. Lucas Naomi O. Seligman list Sun Sun PeopleSoft PeopleSoft Hyperion Hyperion Siebel Siebel BEA BEA JD Edwards JD Edwards RightNow RightNow Virtual Iron Virtual Iron TimesTen TimesTen Sunopsis Sunopsis NetSuite NetSuite Cerner Cerner Oracle Database Oracle Database MySQL MySQL InnoDB InnoDB Berkeley DB Berkeley DB TimesTen TimesTen Rdb Rdb Essbase Essbase Java Java PL/SQL PL/SQL JDeveloper JDeveloper Forms Forms NetBeans NetBeans Apex Apex SQL Developer SQL Developer Developer Studio Developer Studio Fusion Middleware Fusion Middleware WebCenter WebCenter SOA Suite SOA Suite WebLogic Server WebLogic Server Coherence Coherence Tuxedo Tuxedo GlassFish GlassFish Oracle Linux Oracle Linux Oracle Solaris Oracle Solaris Sun Fire Sun Fire SPARC (T-Series, Enterprise) SPARC T-Series Enterprise StorageTek StorageTek Oracle Exadata Oracle Exadata Oracle Exalogic Oracle Exalogic Big Data Appliance Big Data Appliance Oracle Certification Program Oracle Certification Program Category Category v v t t e e Oracle free and open-source software (FOSS) OpenOffice.org OpenOffice.org VirtualBox VirtualBox Java Java Berkeley DB Berkeley DB InnoDB InnoDB MySQL MySQL Oracle NoSQL Database Oracle NoSQL Database DTrace DTrace GlassFish GlassFish HotSpot HotSpot Image Packaging System Image Packaging System Java (software platform) Java (software platform) Maxine Virtual Machine Maxine Virtual Machine Memory Memory Nashorn Nashorn NetBeans NetBeans OpenJDK OpenJDK VisualVM VisualVM OpenSolaris OpenSolaris Oracle Linux Oracle Linux Java Desktop System Java Desktop System OCFS2 OCFS2 Oracle Grid Engine Oracle Grid Engine Oracle VM Server for x86 Oracle VM Server for x86 Google LLC v. Oracle America, Inc. Google LLC v. Oracle America, Inc. v v t t e e Programming languages Comparison Comparison Timeline Timeline History History Ada Ada ALGOL Simula ALGOL Simula Simula APL APL Assembly Assembly BASIC Visual Basic classic .NET BASIC Visual Basic classic .NET Visual Basic classic classic .NET .NET C C C++ C++ C# C# COBOL COBOL Erlang Erlang Forth Forth Fortran Fortran Go Go Haskell Haskell Java Java JavaScript JavaScript Julia Julia Kotlin Kotlin Lisp Lisp Lua Lua MATLAB MATLAB ML ML Pascal Object Pascal Pascal Object Pascal Object Pascal Perl Perl PHP PHP Prolog Prolog Python Python R R Ruby Ruby Rust Rust SQL SQL Scratch Scratch Shell Shell Smalltalk Smalltalk Swift Swift more... more... Lists: Alphabetical Alphabetical Categorical Categorical Generational Generational Non-English-based Non-English-based Category Category Authority control databases Germany Germany Germany United States United States United States France France France BnF data BnF data BnF data Czech Republic Czech Republic Czech Republic Czech Republic Spain Spain Spain Israel Israel Israel ELMCIP ELMCIP ELMCIP https://en.wikipedia.org/w/index.php?title=Java_(programming_language)&oldid=1266787868 Categories Java (programming language) Java (programming language) C programming language family C programming language family Class-based programming languages Class-based programming languages Concurrent programming languages Concurrent programming languages Java platform Java platform Java specification requests Java specification requests JVM programming languages JVM programming languages Multi-paradigm programming languages Multi-paradigm programming languages Object-oriented programming languages Object-oriented programming languages Programming languages created in 1995 Programming languages created in 1995 Programming languages Programming languages Statically typed programming languages Statically typed programming languages Sun Microsystems Sun Microsystems Compiled programming languages Compiled programming languages American inventions American inventions Webarchive template wayback links Webarchive template wayback links CS1: unfit URL CS1: unfit URL Wikipedia indefinitely semi-protected pages Wikipedia indefinitely semi-protected pages Articles with short description Articles with short description Short description matches Wikidata Short description matches Wikidata Use mdy dates from August 2017 Use mdy dates from August 2017 Articles containing potentially dated statements from 2022 Articles containing potentially dated statements from 2022 All articles containing potentially dated statements All articles containing potentially dated statements Articles containing potentially dated statements from September 2024 Articles containing potentially dated statements from September 2024 Articles containing potentially dated statements from November 2024 Articles containing potentially dated statements from November 2024 Articles with example Java code Articles with example Java code Articles needing additional references from May 2019 Articles needing additional references from May 2019 All articles needing additional references All articles needing additional references Articles needing additional references from September 2023 Articles needing additional references from September 2023 Pages using Sister project links with wikidata namespace mismatch Pages using Sister project links with wikidata namespace mismatch Pages using Sister project links with hidden wikidata Pages using Sister project links with hidden wikidata This page was last edited on 2 January 2025, at 06:14 (UTC). (UTC) Text is available under the Creative Commons Attribution-ShareAlike 4.0 License; additional terms may apply. By using this site, you agree to the Terms of Use and Privacy Policy. Wikipedia\u00ae is a registered trademark of the Wikimedia Foundation, Inc., a non-profit organization. Creative Commons Attribution-ShareAlike 4.0 License Terms of Use Privacy Policy Wikimedia Foundation, Inc. Privacy policy Privacy policy About Wikipedia About Wikipedia Disclaimers Disclaimers Contact Wikipedia Contact Wikipedia Code of Conduct Code of Conduct Developers Developers Statistics Statistics Cookie statement Cookie statement Mobile view Mobile view",
    "https://en.wikipedia.org/wiki/C_(programming_language)": "Jump to content Main menu Main page Main page Main page Contents Contents Contents Current events Current events Current events Random article Random article Random article About Wikipedia About Wikipedia About Wikipedia Contact us Contact us Contact us Help Help Help Learn to edit Learn to edit Learn to edit Community portal Community portal Community portal Recent changes Recent changes Recent changes Upload file Upload file Upload file Search Search Appearance Donate Donate Donate Create account Create account Create account Log in Log in Log in Personal tools Donate Donate Donate Create account Create account Create account Log in Log in Log in learn more learn more Contributions Contributions Contributions Talk Talk Talk Contents (Top) (Top) 1 Overview Toggle Overview subsection 1.1 Relations to other languages 1 Overview 1 Overview Toggle Overview subsection 1.1 Relations to other languages 1.1 Relations to other languages 1.1 Relations to other languages 2 History Toggle History subsection 2.1 Early developments 2.1.1 B 2.1.2 New B and first C release 2.1.3 Structures and Unix kernel re-write 2.2 K&R C 2.3 ANSI C and ISO C 2.4 C99 2.5 C11 2.6 C17 2.7 C23 2.8 C2Y 2.9 Embedded C 2 History 2 History Toggle History subsection 2.1 Early developments 2.1.1 B 2.1.2 New B and first C release 2.1.3 Structures and Unix kernel re-write 2.1 Early developments 2.1 Early developments 2.1.1 B 2.1.1 B 2.1.1 B 2.1.2 New B and first C release 2.1.2 New B and first C release 2.1.2 New B and first C release 2.1.3 Structures and Unix kernel re-write 2.1.3 Structures and Unix kernel re-write 2.1.3 Structures and Unix kernel re-write 2.2 K&R C 2.2 K&R C 2.2 K&R C 2.3 ANSI C and ISO C 2.3 ANSI C and ISO C 2.3 ANSI C and ISO C 2.4 C99 2.4 C99 2.4 C99 2.5 C11 2.5 C11 2.5 C11 2.6 C17 2.6 C17 2.6 C17 2.7 C23 2.7 C23 2.7 C23 2.8 C2Y 2.8 C2Y 2.8 C2Y 2.9 Embedded C 2.9 Embedded C 2.9 Embedded C 3 Syntax Toggle Syntax subsection 3.1 Character set 3.2 Reserved words 3.3 Operators 3 Syntax 3 Syntax Toggle Syntax subsection 3.1 Character set 3.1 Character set 3.1 Character set 3.2 Reserved words 3.2 Reserved words 3.2 Reserved words 3.3 Operators 3.3 Operators 3.3 Operators 4 \"Hello, world\" example 4 \"Hello, world\" example 4 \"Hello, world\" example 5 Data types Toggle Data types subsection 5.1 Pointers 5.2 Arrays 5.3 Array\u2013pointer interchangeability 5 Data types 5 Data types Toggle Data types subsection 5.1 Pointers 5.1 Pointers 5.1 Pointers 5.2 Arrays 5.2 Arrays 5.2 Arrays 5.3 Array\u2013pointer interchangeability 5.3 Array\u2013pointer interchangeability 5.3 Array\u2013pointer interchangeability 6 Memory management 6 Memory management 6 Memory management 7 Libraries Toggle Libraries subsection 7.1 File handling and streams 7 Libraries 7 Libraries Toggle Libraries subsection 7.1 File handling and streams 7.1 File handling and streams 7.1 File handling and streams 8 Language tools 8 Language tools 8 Language tools 9 Uses Toggle Uses subsection 9.1 Rationale for use in systems programming 9.2 Used for computationally-intensive libraries 9.3 C as an intermediate language 9.4 Other languages written in C 9.5 Once used for web development 9.6 Web servers 9.7 End-user applications 9 Uses 9 Uses Toggle Uses subsection 9.1 Rationale for use in systems programming 9.1 Rationale for use in systems programming 9.1 Rationale for use in systems programming 9.2 Used for computationally-intensive libraries 9.2 Used for computationally-intensive libraries 9.2 Used for computationally-intensive libraries 9.3 C as an intermediate language 9.3 C as an intermediate language 9.3 C as an intermediate language 9.4 Other languages written in C 9.4 Other languages written in C 9.4 Other languages written in C 9.5 Once used for web development 9.5 Once used for web development 9.5 Once used for web development 9.6 Web servers 9.6 Web servers 9.6 Web servers 9.7 End-user applications 9.7 End-user applications 9.7 End-user applications 10 Limitations 10 Limitations 10 Limitations 11 Related languages 11 Related languages 11 Related languages 12 See also 12 See also 12 See also 13 Notes 13 Notes 13 Notes 14 References 14 References 14 References 15 Sources 15 Sources 15 Sources 16 Further reading 16 Further reading 16 Further reading 17 External links 17 External links 17 External links Toggle the table of contents C (programming language) C (programming language) 123 languages Afrikaans Afrikaans Afrikaans Alemannisch Alemannisch Alemannisch \u12a0\u121b\u122d\u129b \u12a0\u121b\u122d\u129b \u12a0\u121b\u122d\u129b \u0627\u0644\u0639\u0631\u0628\u064a\u0629 \u0627\u0644\u0639\u0631\u0628\u064a\u0629 \u0627\u0644\u0639\u0631\u0628\u064a\u0629 Aragon\u00e9s Aragon\u00e9s Aragon\u00e9s \u0985\u09b8\u09ae\u09c0\u09af\u09bc\u09be \u0985\u09b8\u09ae\u09c0\u09af\u09bc\u09be \u0985\u09b8\u09ae\u09c0\u09af\u09bc\u09be Asturianu Asturianu Asturianu Az\u0259rbaycanca Az\u0259rbaycanca Az\u0259rbaycanca \u062a\u06c6\u0631\u06a9\u062c\u0647 \u062a\u06c6\u0631\u06a9\u062c\u0647 \u062a\u06c6\u0631\u06a9\u062c\u0647 \u09ac\u09be\u0982\u09b2\u09be \u09ac\u09be\u0982\u09b2\u09be \u09ac\u09be\u0982\u09b2\u09be \u95a9\u5357\u8a9e / B\u00e2n-l\u00e2m-g\u00fa \u95a9\u5357\u8a9e / B\u00e2n-l\u00e2m-g\u00fa \u95a9\u5357\u8a9e / B\u00e2n-l\u00e2m-g\u00fa \u0411\u0435\u043b\u0430\u0440\u0443\u0441\u043a\u0430\u044f \u0411\u0435\u043b\u0430\u0440\u0443\u0441\u043a\u0430\u044f \u0411\u0435\u043b\u0430\u0440\u0443\u0441\u043a\u0430\u044f \u0411\u0435\u043b\u0430\u0440\u0443\u0441\u043a\u0430\u044f (\u0442\u0430\u0440\u0430\u0448\u043a\u0435\u0432\u0456\u0446\u0430) \u0411\u0435\u043b\u0430\u0440\u0443\u0441\u043a\u0430\u044f (\u0442\u0430\u0440\u0430\u0448\u043a\u0435\u0432\u0456\u0446\u0430) \u0411\u0435\u043b\u0430\u0440\u0443\u0441\u043a\u0430\u044f (\u0442\u0430\u0440\u0430\u0448\u043a\u0435\u0432\u0456\u0446\u0430) \u092d\u094b\u091c\u092a\u0941\u0930\u0940 \u092d\u094b\u091c\u092a\u0941\u0930\u0940 \u092d\u094b\u091c\u092a\u0941\u0930\u0940 \u0411\u044a\u043b\u0433\u0430\u0440\u0441\u043a\u0438 \u0411\u044a\u043b\u0433\u0430\u0440\u0441\u043a\u0438 \u0411\u044a\u043b\u0433\u0430\u0440\u0441\u043a\u0438 Bosanski Bosanski Bosanski Brezhoneg Brezhoneg Brezhoneg Catal\u00e0 Catal\u00e0 Catal\u00e0 \u0427\u04d1\u0432\u0430\u0448\u043b\u0430 \u0427\u04d1\u0432\u0430\u0448\u043b\u0430 \u0427\u04d1\u0432\u0430\u0448\u043b\u0430 \u010ce\u0161tina \u010ce\u0161tina \u010ce\u0161tina Corsu Corsu Corsu Cymraeg Cymraeg Cymraeg Dansk Dansk Dansk \u0627\u0644\u062f\u0627\u0631\u062c\u0629 \u0627\u0644\u062f\u0627\u0631\u062c\u0629 \u0627\u0644\u062f\u0627\u0631\u062c\u0629 Deutsch Deutsch Deutsch Eesti Eesti Eesti \u0395\u03bb\u03bb\u03b7\u03bd\u03b9\u03ba\u03ac \u0395\u03bb\u03bb\u03b7\u03bd\u03b9\u03ba\u03ac \u0395\u03bb\u03bb\u03b7\u03bd\u03b9\u03ba\u03ac Espa\u00f1ol Espa\u00f1ol Espa\u00f1ol Esperanto Esperanto Esperanto Euskara Euskara Euskara \u0641\u0627\u0631\u0633\u06cc \u0641\u0627\u0631\u0633\u06cc \u0641\u0627\u0631\u0633\u06cc Fran\u00e7ais Fran\u00e7ais Fran\u00e7ais Gaeilge Gaeilge Gaeilge Galego Galego Galego \u0a97\u0ac1\u0a9c\u0ab0\u0abe\u0aa4\u0ac0 \u0a97\u0ac1\u0a9c\u0ab0\u0abe\u0aa4\u0ac0 \u0a97\u0ac1\u0a9c\u0ab0\u0abe\u0aa4\u0ac0 \u5ba2\u5bb6\u8a9e / Hak-k\u00e2-ng\u00ee \u5ba2\u5bb6\u8a9e / Hak-k\u00e2-ng\u00ee \u5ba2\u5bb6\u8a9e / Hak-k\u00e2-ng\u00ee \ud55c\uad6d\uc5b4 \ud55c\uad6d\uc5b4 \ud55c\uad6d\uc5b4 Hausa Hausa Hausa \u0540\u0561\u0575\u0565\u0580\u0565\u0576 \u0540\u0561\u0575\u0565\u0580\u0565\u0576 \u0540\u0561\u0575\u0565\u0580\u0565\u0576 \u0939\u093f\u0928\u094d\u0926\u0940 \u0939\u093f\u0928\u094d\u0926\u0940 \u0939\u093f\u0928\u094d\u0926\u0940 Hrvatski Hrvatski Hrvatski Ido Ido Ido Bahasa Indonesia Bahasa Indonesia Bahasa Indonesia Interlingua Interlingua Interlingua IsiXhosa IsiXhosa IsiXhosa \u00cdslenska \u00cdslenska \u00cdslenska Italiano Italiano Italiano \u05e2\u05d1\u05e8\u05d9\u05ea \u05e2\u05d1\u05e8\u05d9\u05ea \u05e2\u05d1\u05e8\u05d9\u05ea Jawa Jawa Jawa \u0c95\u0ca8\u0ccd\u0ca8\u0ca1 \u0c95\u0ca8\u0ccd\u0ca8\u0ca1 \u0c95\u0ca8\u0ccd\u0ca8\u0ca1 \u10e5\u10d0\u10e0\u10d7\u10e3\u10da\u10d8 \u10e5\u10d0\u10e0\u10d7\u10e3\u10da\u10d8 \u10e5\u10d0\u10e0\u10d7\u10e3\u10da\u10d8 \u049a\u0430\u0437\u0430\u049b\u0448\u0430 \u049a\u0430\u0437\u0430\u049b\u0448\u0430 \u049a\u0430\u0437\u0430\u049b\u0448\u0430 Kiswahili Kiswahili Kiswahili Kurd\u00ee Kurd\u00ee Kurd\u00ee \u041a\u044b\u0440\u0433\u044b\u0437\u0447\u0430 \u041a\u044b\u0440\u0433\u044b\u0437\u0447\u0430 \u041a\u044b\u0440\u0433\u044b\u0437\u0447\u0430 \u0ea5\u0eb2\u0ea7 \u0ea5\u0eb2\u0ea7 \u0ea5\u0eb2\u0ea7 Latina Latina Latina Latvie\u0161u Latvie\u0161u Latvie\u0161u Lietuvi\u0173 Lietuvi\u0173 Lietuvi\u0173 Lombard Lombard Lombard Magyar Magyar Magyar \u092e\u0948\u0925\u093f\u0932\u0940 \u092e\u0948\u0925\u093f\u0932\u0940 \u092e\u0948\u0925\u093f\u0932\u0940 \u041c\u0430\u043a\u0435\u0434\u043e\u043d\u0441\u043a\u0438 \u041c\u0430\u043a\u0435\u0434\u043e\u043d\u0441\u043a\u0438 \u041c\u0430\u043a\u0435\u0434\u043e\u043d\u0441\u043a\u0438 Malagasy Malagasy Malagasy \u0d2e\u0d32\u0d2f\u0d3e\u0d33\u0d02 \u0d2e\u0d32\u0d2f\u0d3e\u0d33\u0d02 \u0d2e\u0d32\u0d2f\u0d3e\u0d33\u0d02 \u092e\u0930\u093e\u0920\u0940 \u092e\u0930\u093e\u0920\u0940 \u092e\u0930\u093e\u0920\u0940 Bahasa Melayu Bahasa Melayu Bahasa Melayu \uabc3\uabe4\uabc7\uabe9 \uabc2\uabe3\uabdf \uabc3\uabe4\uabc7\uabe9 \uabc2\uabe3\uabdf \uabc3\uabe4\uabc7\uabe9 \uabc2\uabe3\uabdf \u95a9\u6771\u8a9e / M\u00ecng-d\u0115\u0324ng-ng\u1e73\u0304 \u95a9\u6771\u8a9e / M\u00ecng-d\u0115\u0324ng-ng\u1e73\u0304 \u95a9\u6771\u8a9e / M\u00ecng-d\u0115\u0324ng-ng\u1e73\u0304 \u041c\u043e\u043d\u0433\u043e\u043b \u041c\u043e\u043d\u0433\u043e\u043b \u041c\u043e\u043d\u0433\u043e\u043b \u1019\u103c\u1014\u103a\u1019\u102c\u1018\u102c\u101e\u102c \u1019\u103c\u1014\u103a\u1019\u102c\u1018\u102c\u101e\u102c \u1019\u103c\u1014\u103a\u1019\u102c\u1018\u102c\u101e\u102c Nederlands Nederlands Nederlands \u0928\u0947\u092a\u093e\u0932\u0940 \u0928\u0947\u092a\u093e\u0932\u0940 \u0928\u0947\u092a\u093e\u0932\u0940 \u65e5\u672c\u8a9e \u65e5\u672c\u8a9e \u65e5\u672c\u8a9e Norsk bokm\u00e5l Norsk bokm\u00e5l Norsk bokm\u00e5l Norsk nynorsk Norsk nynorsk Norsk nynorsk Occitan Occitan Occitan \u0b13\u0b21\u0b3c\u0b3f\u0b06 \u0b13\u0b21\u0b3c\u0b3f\u0b06 \u0b13\u0b21\u0b3c\u0b3f\u0b06 O\u02bbzbekcha / \u045e\u0437\u0431\u0435\u043a\u0447\u0430 O\u02bbzbekcha / \u045e\u0437\u0431\u0435\u043a\u0447\u0430 O\u02bbzbekcha / \u045e\u0437\u0431\u0435\u043a\u0447\u0430 \u0a2a\u0a70\u0a1c\u0a3e\u0a2c\u0a40 \u0a2a\u0a70\u0a1c\u0a3e\u0a2c\u0a40 \u0a2a\u0a70\u0a1c\u0a3e\u0a2c\u0a40 \u067e\u0646\u062c\u0627\u0628\u06cc \u067e\u0646\u062c\u0627\u0628\u06cc \u067e\u0646\u062c\u0627\u0628\u06cc \u1015\u1021\u102d\u102f\u101d\u103a\u108f\u1018\u102c\u108f\u101e\u102c\u108f \u1015\u1021\u102d\u102f\u101d\u103a\u108f\u1018\u102c\u108f\u101e\u102c\u108f \u1015\u1021\u102d\u102f\u101d\u103a\u108f\u1018\u102c\u108f\u101e\u102c\u108f \u1797\u17b6\u179f\u17b6\u1781\u17d2\u1798\u17c2\u179a \u1797\u17b6\u179f\u17b6\u1781\u17d2\u1798\u17c2\u179a \u1797\u17b6\u179f\u17b6\u1781\u17d2\u1798\u17c2\u179a Piemont\u00e8is Piemont\u00e8is Piemont\u00e8is Polski Polski Polski Portugu\u00eas Portugu\u00eas Portugu\u00eas Qaraqalpaqsha Qaraqalpaqsha Qaraqalpaqsha Rom\u00e2n\u0103 Rom\u00e2n\u0103 Rom\u00e2n\u0103 Runa Simi Runa Simi Runa Simi \u0420\u0443\u0441\u0441\u043a\u0438\u0439 \u0420\u0443\u0441\u0441\u043a\u0438\u0439 \u0420\u0443\u0441\u0441\u043a\u0438\u0439 \u0421\u0430\u0445\u0430 \u0442\u044b\u043b\u0430 \u0421\u0430\u0445\u0430 \u0442\u044b\u043b\u0430 \u0421\u0430\u0445\u0430 \u0442\u044b\u043b\u0430 \u1c65\u1c5f\u1c71\u1c5b\u1c5f\u1c72\u1c64 \u1c65\u1c5f\u1c71\u1c5b\u1c5f\u1c72\u1c64 \u1c65\u1c5f\u1c71\u1c5b\u1c5f\u1c72\u1c64 Scots Scots Scots Shqip Shqip Shqip \u0dc3\u0dd2\u0d82\u0dc4\u0dbd \u0dc3\u0dd2\u0d82\u0dc4\u0dbd \u0dc3\u0dd2\u0d82\u0dc4\u0dbd Simple English Simple English Simple English Sloven\u010dina Sloven\u010dina Sloven\u010dina Sloven\u0161\u010dina Sloven\u0161\u010dina Sloven\u0161\u010dina \u0421\u0440\u043f\u0441\u043a\u0438 / srpski \u0421\u0440\u043f\u0441\u043a\u0438 / srpski \u0421\u0440\u043f\u0441\u043a\u0438 / srpski Srpskohrvatski / \u0441\u0440\u043f\u0441\u043a\u043e\u0445\u0440\u0432\u0430\u0442\u0441\u043a\u0438 Srpskohrvatski / \u0441\u0440\u043f\u0441\u043a\u043e\u0445\u0440\u0432\u0430\u0442\u0441\u043a\u0438 Srpskohrvatski / \u0441\u0440\u043f\u0441\u043a\u043e\u0445\u0440\u0432\u0430\u0442\u0441\u043a\u0438 Suomi Suomi Suomi Svenska Svenska Svenska Tagalog Tagalog Tagalog \u0ba4\u0bae\u0bbf\u0bb4\u0bcd \u0ba4\u0bae\u0bbf\u0bb4\u0bcd \u0ba4\u0bae\u0bbf\u0bb4\u0bcd Taqbaylit Taqbaylit Taqbaylit \u0422\u0430\u0442\u0430\u0440\u0447\u0430 / tatar\u00e7a \u0422\u0430\u0442\u0430\u0440\u0447\u0430 / tatar\u00e7a \u0422\u0430\u0442\u0430\u0440\u0447\u0430 / tatar\u00e7a \u0c24\u0c46\u0c32\u0c41\u0c17\u0c41 \u0c24\u0c46\u0c32\u0c41\u0c17\u0c41 \u0c24\u0c46\u0c32\u0c41\u0c17\u0c41 \u0e44\u0e17\u0e22 \u0e44\u0e17\u0e22 \u0e44\u0e17\u0e22 \u0422\u043e\u04b7\u0438\u043a\u04e3 \u0422\u043e\u04b7\u0438\u043a\u04e3 \u0422\u043e\u04b7\u0438\u043a\u04e3 T\u00fcrk\u00e7e T\u00fcrk\u00e7e T\u00fcrk\u00e7e Basa Ugi Basa Ugi Basa Ugi \u0423\u043a\u0440\u0430\u0457\u043d\u0441\u044c\u043a\u0430 \u0423\u043a\u0440\u0430\u0457\u043d\u0441\u044c\u043a\u0430 \u0423\u043a\u0440\u0430\u0457\u043d\u0441\u044c\u043a\u0430 \u0627\u0631\u062f\u0648 \u0627\u0631\u062f\u0648 \u0627\u0631\u062f\u0648 V\u00e8neto V\u00e8neto V\u00e8neto Ti\u1ebfng Vi\u1ec7t Ti\u1ebfng Vi\u1ec7t Ti\u1ebfng Vi\u1ec7t \u6587\u8a00 \u6587\u8a00 \u6587\u8a00 Winaray Winaray Winaray \u5434\u8bed \u5434\u8bed \u5434\u8bed \u05d9\u05d9\u05b4\u05d3\u05d9\u05e9 \u05d9\u05d9\u05b4\u05d3\u05d9\u05e9 \u05d9\u05d9\u05b4\u05d3\u05d9\u05e9 \u7cb5\u8a9e \u7cb5\u8a9e \u7cb5\u8a9e \u017demait\u0117\u0161ka \u017demait\u0117\u0161ka \u017demait\u0117\u0161ka \u4e2d\u6587 \u4e2d\u6587 \u4e2d\u6587 \u2d5c\u2d30\u2d4e\u2d30\u2d63\u2d49\u2d56\u2d5c \u2d5c\u2d30\u2d4f\u2d30\u2d61\u2d30\u2d62\u2d5c \u2d5c\u2d30\u2d4e\u2d30\u2d63\u2d49\u2d56\u2d5c \u2d5c\u2d30\u2d4f\u2d30\u2d61\u2d30\u2d62\u2d5c \u2d5c\u2d30\u2d4e\u2d30\u2d63\u2d49\u2d56\u2d5c \u2d5c\u2d30\u2d4f\u2d30\u2d61\u2d30\u2d62\u2d5c Edit links Edit links Article Article Article Talk Talk Talk English Read Read Read Edit Edit Edit View history View history View history Tools Read Read Read Edit Edit Edit View history View history View history What links here What links here What links here Related changes Related changes Related changes Upload file Upload file Upload file Special pages Special pages Special pages Permanent link Permanent link Permanent link Page information Page information Page information Cite this page Cite this page Cite this page Get shortened URL Get shortened URL Get shortened URL Download QR code Download QR code Download QR code Download as PDF Download as PDF Download as PDF Printable version Printable version Printable version Wikimedia Commons Wikimedia Commons Wikimedia Commons Wikibooks Wikibooks Wikibooks Wikiquote Wikiquote Wikiquote Wikiversity Wikiversity Wikiversity Wikidata item Wikidata item Wikidata item Page version status This is an accepted version of this page latest accepted revision reviewed The C Programming Language C++ C# The C Programming Language [1] [ ] Paradigm Multi-paradigm imperative procedural structured Designed by Dennis Ritchie Developer ANSI C ISO/IEC JTC 1 (Joint Technical Committee 1) / SC 22 (Subcommittee 22) ; 53 years ago (1972) 1972 [2] [ ] Stable release C23 ; 2 months ago (2024-10-31) 2024-10-31 Preview release ; 10 months ago (2024-02-21) 2024-02-21 [3] [ ] Typing discipline Static weak manifest nominal OS Cross-platform Filename extensions www.iso.org/standard/82075.html www.iso.org/standard/82075.html www.open-std.org/jtc1/sc22/wg14/ www.open-std.org/jtc1/sc22/wg14/ implementations pcc GCC Clang Intel C Intel C C++Builder Microsoft Visual C++ Microsoft Visual C++ Watcom C Watcom C Dialects Cyclone Unified Parallel C Split-C Cilk C* B BCPL CPL ALGOL 68 [4] [ ] PL/I FORTRAN Numerous AMPL AWK csh C++ C-- C# Objective-C D Go Java JavaScript JS++ Julia Limbo LPC Perl PHP Pike Processing Python Rust Seed7 V (Vlang) Vala Verilog [5] [ ] Nim Zig C Programming at Wikibooks C Programming C (pronounced /\u02c8si\u02d0/ \u2013 like the letter c)[6] is a general-purpose programming language. It was created in the 1970s by Dennis Ritchie and remains very widely used and influential. By design, C's features cleanly reflect the capabilities of the targeted CPUs. It has found lasting use in operating systems code (especially in kernels[7]), device drivers, and protocol stacks, but its use in application software has been decreasing.[8] C is commonly used on computer architectures that range from the largest supercomputers to the smallest microcontrollers and embedded systems. /\u02c8si\u02d0/ /\u02c8si\u02d0/ /\u02c8si\u02d0/ \u02c8si\u02d0 \u02c8 s i\u02d0 c [6] [ ] general-purpose programming language Dennis Ritchie CPUs operating systems kernels [7] [ ] device drivers protocol stacks application software [8] [ ] supercomputers microcontrollers embedded systems A successor to the programming language B, C was originally developed at Bell Labs by Ritchie between 1972 and 1973 to construct utilities running on Unix. It was applied to re-implementing the kernel of the Unix operating system.[9] During the 1980s, C gradually gained popularity. It has become one of the most widely used programming languages,[10][11] with C compilers available for practically all modern computer architectures and operating systems. The book The C Programming Language, co-authored by the original language designer, served for many years as the de facto standard for the language.[12][1] C has been standardized since 1989 by the American National Standards Institute (ANSI) and, subsequently, jointly by the International Organization for Standardization (ISO) and the International Electrotechnical Commission (IEC). B Bell Labs Unix [9] [ ] programming languages [10] [ ] [11] [ ] compilers computer architectures operating systems The C Programming Language [12] [ ] [1] [ ] American National Standards Institute International Organization for Standardization International Electrotechnical Commission C is an imperative procedural language, supporting structured programming, lexical variable scope, and recursion, with a static type system. It was designed to be compiled to provide low-level access to memory and language constructs that map efficiently to machine instructions, all with minimal runtime support. Despite its low-level capabilities, the language was designed to encourage cross-platform programming. A standards-compliant C program written with portability in mind can be compiled for a wide variety of computer platforms and operating systems with few changes to its source code. imperative procedural structured programming lexical variable scope recursion static type system compiled low-level memory machine instructions runtime support standards portability Since 2000, C has consistently ranked among the top four languages in the TIOBE index, a measure of the popularity of programming languages.[13] TIOBE index [13] [ ] Overview [edit] [ edit edit ] Dennis Ritchie Ken Thompson C is an imperative, procedural language in the ALGOL tradition. It has a static type system. In C, all executable code is contained within subroutines (also called \"functions\", though not in the sense of functional programming). Function parameters are passed by value, although arrays are passed as pointers, i.e. the address of the first item in the array. Pass-by-reference is simulated in C by explicitly passing pointers to the thing being referenced. imperative ALGOL type system executable code subroutines functional programming Function parameters arrays pointers C program source text is free-form code. Semicolons terminate statements, while curly braces are used to group statements into blocks. free-form Semicolons statements curly braces blocks The C language also exhibits the following characteristics: The language has a small, fixed number of keywords, including a full set of control flow primitives: if/else, for, do/while, while, and switch. User-defined names are not distinguished from keywords by any kind of sigil. control flow if/else for do/while while switch sigil It has a large number of arithmetic, bitwise, and logic operators: +,+=,++,&,||, etc. bitwise More than one assignment may be performed in a single statement. assignment Functions: Function return values can be ignored, when not needed. Function and data pointers permit ad hoc run-time polymorphism. Functions may not be defined within the lexical scope of other functions. Variables may be defined within a function, with scope. A function may call itself, so recursion is supported. Function return values can be ignored, when not needed. Function and data pointers permit ad hoc run-time polymorphism. run-time polymorphism Functions may not be defined within the lexical scope of other functions. Variables may be defined within a function, with scope. scope A function may call itself, so recursion is supported. recursion Data typing is static, but weakly enforced; all data has a type, but implicit conversions are possible. static weakly enforced implicit conversions User-defined (typedef) and compound types are possible. Heterogeneous aggregate data types (struct) allow related data elements to be accessed and assigned as a unit. The contents of whole structs cannot be compared using a single built-in operator (the elements must be compared individually). Union is a structure with overlapping members; it allows multiple data types to share the same memory location. Array indexing is a secondary notation, defined in terms of pointer arithmetic. Whole arrays cannot be assigned or compared using a single built-in operator. There is no \"array\" keyword in use or definition; instead, square brackets indicate arrays syntactically, for example month[11]. Enumerated types are possible with the enum keyword. They are freely interconvertible with integers. Strings are not a distinct data type, but are conventionally implemented as null-terminated character arrays. typedef Heterogeneous aggregate data types (struct) allow related data elements to be accessed and assigned as a unit. The contents of whole structs cannot be compared using a single built-in operator (the elements must be compared individually). struct Union is a structure with overlapping members; it allows multiple data types to share the same memory location. Union Array indexing is a secondary notation, defined in terms of pointer arithmetic. Whole arrays cannot be assigned or compared using a single built-in operator. There is no \"array\" keyword in use or definition; instead, square brackets indicate arrays syntactically, for example month[11]. Array Enumerated types are possible with the enum keyword. They are freely interconvertible with integers. Enumerated types Strings are not a distinct data type, but are conventionally implemented as null-terminated character arrays. Strings implemented null-terminated Low-level access to computer memory is possible by converting machine addresses to pointers. computer memory pointers Procedures (subroutines not returning values) are a special case of function, with an empty return type void. Procedures Memory can be allocated to a program with calls to library routines. allocated library routines A preprocessor performs macro definition, source code file inclusion, and conditional compilation. preprocessor macro source code conditional compilation There is a basic form of modularity: files can be compiled separately and linked together, with control over which functions and data objects are visible to other files via static and extern attributes. modularity linked static Complex functionality such as I/O, string manipulation, and mathematical functions are consistently delegated to library routines. I/O string library routines The generated code after compilation has relatively straightforward needs on the underlying platform, which makes it suitable for creating operating systems and for use in embedded systems. embedded systems While C does not include certain features found in other languages (such as object orientation and garbage collection), these can be implemented or emulated, often through the use of external libraries (e.g., the GLib Object System or the Boehm garbage collector). object orientation garbage collection GLib Object System Boehm garbage collector Relations to other languages [edit] [ edit edit ] List of C-family programming languages Many later languages have borrowed directly or indirectly from C, including C++, C#, Unix's C shell, D, Go, Java, JavaScript (including transpilers), Julia, Limbo, LPC, Objective-C, Perl, PHP, Python, Ruby, Rust, Swift, Verilog and SystemVerilog (hardware description languages).[5] These languages have drawn many of their control structures and other basic features from C. Most of them also express highly similar syntax to C, and they tend to combine the recognizable expression and statement syntax of C with underlying type systems, data models, and semantics that can be radically different. C++ C# C shell D Go Java JavaScript transpilers Julia Limbo LPC Objective-C Perl PHP Python Ruby Rust Swift Verilog SystemVerilog [5] [ ] control structures syntax syntax of C History [edit] [ edit edit ] Early developments [edit] [ edit edit ] K&R C ANSI C C99 C11 C17 C23 C2Y The origin of C is closely tied to the development of the Unix operating system, originally implemented in assembly language on a PDP-7 by Dennis Ritchie and Ken Thompson, incorporating several ideas from colleagues. Eventually, they decided to port the operating system to a PDP-11. The original PDP-11 version of Unix was also developed in assembly language.[9] Unix assembly language PDP-7 Dennis Ritchie Ken Thompson PDP-11 [9] [ ] B [edit] [ edit edit ] B (programming language) Thompson wanted a programming language for developing utilities for the new platform. He first tried writing a Fortran compiler, but he soon gave up the idea and instead created a cut-down version of the recently developed systems programming language called BCPL. The official description of BCPL was not available at the time,[14] and Thompson modified the syntax to be less 'wordy' and similar to a simplified ALGOL known as SMALGOL.[15] He called the result B,[9] describing it as \"BCPL semantics with a lot of SMALGOL syntax\".[15] Like BCPL, B had a bootstrapping compiler to facilitate porting to new machines.[15] Ultimately, few utilities were written in B because it was too slow and could not take advantage of PDP-11 features such as byte addressability. Fortran systems programming language BCPL [14] [ ] ALGOL [15] [ ] B [9] [ ] [15] [ ] bootstrapping [15] [ ] byte New B and first C release [edit] [ edit edit ] In 1971 Ritchie started to improve B, to use the features of the more-powerful PDP-11. A significant addition was a character data type. He called this New B (NB).[15] Thompson started to use NB to write the Unix kernel, and his requirements shaped the direction of the language development.[15][16] Through to 1972, richer types were added to the NB language: NB had arrays of int and char. Pointers, the ability to generate pointers to other types, arrays of all types, and types to be returned from functions were all also added. Arrays within expressions became pointers. A new compiler was written, and the language was renamed C.[9] [15] [ ] Unix [15] [ ] [16] [ ] [9] [ ] The C compiler and some utilities made with it were included in Version 2 Unix, which is also known as Research Unix.[17] Version 2 Unix Research Unix [17] [ ] Structures and Unix kernel re-write [edit] [ edit edit ] At Version 4 Unix, released in November 1973, the Unix kernel was extensively re-implemented in C.[9] By this time, the C language had acquired some powerful features such as struct types. Version 4 Unix Unix kernel [9] [ ] The preprocessor was introduced around 1973 at the urging of Alan Snyder and also in recognition of the usefulness of the file-inclusion mechanisms available in BCPL and PL/I. Its original version provided only included files and simple string replacements: #include and #define of parameterless macros. Soon after that, it was extended, mostly by Mike Lesk and then by John Reiser, to incorporate macros with arguments and conditional compilation.[9] preprocessor Alan Snyder PL/I Mike Lesk conditional compilation [9] [ ] Unix was one of the first operating system kernels implemented in a language other than assembly. Earlier instances include the Multics system (which was written in PL/I) and Master Control Program (MCP) for the Burroughs B5000 (which was written in ALGOL) in 1961. In around 1977, Ritchie and Stephen C. Johnson made further changes to the language to facilitate portability of the Unix operating system. Johnson's Portable C Compiler served as the basis for several implementations of C on new platforms.[16] assembly Multics PL/I Master Control Program Burroughs B5000 ALGOL Stephen C. Johnson portability Portable C Compiler [16] [ ] K&R C [edit] [ edit edit ] Brian Kernighan Dennis Ritchie In 1978 Brian Kernighan and Dennis Ritchie published the first edition of The C Programming Language.[18] Known as K&R from the initials of its authors, the book served for many years as an informal specification of the language. The version of C that it describes is commonly referred to as \"K&R C\". As this was released in 1978, it is now also referred to as C78.[19] The second edition of the book[20] covers the later ANSI C standard, described below. Brian Kernighan Dennis Ritchie The C Programming Language [18] [ ] specification [19] [ ] [20] [ ] ANSI C K&R introduced several language features: Standard I/O library Standard I/O library long int data type long int unsigned int data type Compound assignment operators of the form =op (such as =-) were changed to the form op= (that is, -=) to remove the semantic ambiguity created by constructs such as i=-10, which had been interpreted as i =- 10 (decrement i by 10) instead of the possibly intended i = -10 (let i be \u221210). Even after the publication of the 1989 ANSI standard, for many years K&R C was still considered the \"lowest common denominator\" to which C programmers restricted themselves when maximum portability was desired, since many older compilers were still in use, and because carefully written K&R C code can be legal Standard C as well. lowest common denominator In early versions of C, only functions that return types other than int must be declared if used before the function definition; functions used without prior declaration were presumed to return type int. For example: long some_function (); /* This is a function declaration, so the compiler can know the name and return type of this function. */ /* int */ other_function (); /* Another function declaration. Because this is an early version of C, there is an implicit 'int' type here. A comment shows where the explicit 'int' type specifier would be required in later versions. */ /* int */ calling_function () /* This is a function definition, including the body of the code following in the { curly brackets }. Because no return type is specified, the function implicitly returns an 'int' in this early version of C. */ { long test1 ; register /* int */ test2 ; /* Again, note that 'int' is not required here. The 'int' type specifier */ /* in the comment would be required in later versions of C. */ /* The 'register' keyword indicates to the compiler that this variable should */ /* ideally be stored in a register as opposed to within the stack frame. */ test1 = some_function (); if ( test1 > 1 ) test2 = 0 ; else test2 = other_function (); return test2 ; } The int type specifiers which are commented out could be omitted in K&R C, but are required in later standards. Since K&R function declarations did not include any information about function arguments, function parameter type checks were not performed, although some compilers would issue a warning message if a local function was called with the wrong number of arguments, or if different calls to an external function used different numbers or types of arguments. Separate tools such as Unix's lint utility were developed that (among other things) could check for consistency of function use across multiple source files. type checks lint In the years following the publication of K&R C, several features were added to the language, supported by compilers from AT&T (in particular PCC[21]) and some other vendors. These included: PCC [21] [ ] void functions (i.e., functions with no return value) void functions returning struct or union types (previously only a single pointer, integer or float could be returned) struct union assignment for struct data types assignment enumerated types (previously, preprocessor definitions for integer fixed values were used, e.g. #define GREEN 3) enumerated types The large number of extensions and lack of agreement on a standard library, together with the language popularity and the fact that not even the Unix compilers precisely implemented the K&R specification, led to the necessity of standardization.[22] standard library [22] [ ] ANSI C and ISO C [edit] [ edit edit ] ANSI C During the late 1970s and 1980s, versions of C were implemented for a wide variety of mainframe computers, minicomputers, and microcomputers, including the IBM PC, as its popularity began to increase significantly. mainframe computers minicomputers microcomputers IBM PC In 1983 the American National Standards Institute (ANSI) formed a committee, X3J11, to establish a standard specification of C. X3J11 based the C standard on the Unix implementation; however, the non-portable portion of the Unix C library was handed off to the IEEE working group 1003 to become the basis for the 1988 POSIX standard. In 1989, the C standard was ratified as ANSI X3.159-1989 \"Programming Language C\". This version of the language is often referred to as ANSI C, Standard C, or sometimes C89. American National Standards Institute IEEE working group POSIX ANSI C In 1990 the ANSI C standard (with formatting changes) was adopted by the International Organization for Standardization (ISO) as ISO/IEC 9899:1990, which is sometimes called C90. Therefore, the terms \"C89\" and \"C90\" refer to the same programming language. International Organization for Standardization ANSI, like other national standards bodies, no longer develops the C standard independently, but defers to the international C standard, maintained by the working group ISO/IEC JTC1/SC22/WG14. National adoption of an update to the international standard typically occurs within a year of ISO publication. ISO/IEC JTC1/SC22 One of the aims of the C standardization process was to produce a superset of K&R C, incorporating many of the subsequently introduced unofficial features. The standards committee also included several additional features such as function prototypes (borrowed from C++), void pointers, support for international character sets and locales, and preprocessor enhancements. Although the syntax for parameter declarations was augmented to include the style used in C++, the K&R interface continued to be permitted, for compatibility with existing source code. superset function prototypes character sets locales syntax C89 is supported by current C compilers, and most modern C code is based on it. Any program written only in Standard C and without any hardware-dependent assumptions will run correctly on any platform with a conforming C implementation, within its resource limits. Without such precautions, programs may compile only on a certain platform or with a particular compiler, due, for example, to the use of non-standard libraries, such as GUI libraries, or to a reliance on compiler- or platform-specific attributes such as the exact size of data types and byte endianness. platform GUI endianness In cases where code must be compilable by either standard-conforming or K&R C-based compilers, the __STDC__ macro can be used to split the code into Standard and K&R sections to prevent the use on a K&R C-based compiler of features available only in Standard C. After the ANSI/ISO standardization process, the C language specification remained relatively static for several years. In 1995, Normative Amendment 1 to the 1990 C standard (ISO/IEC 9899/AMD1:1995, known informally as C95) was published, to correct some details and to add more extensive support for international character sets.[23] [23] [ ] C99 [edit] [ edit edit ] C99 The C standard was further revised in the late 1990s, leading to the publication of ISO/IEC 9899:1999 in 1999, which is commonly referred to as \"C99\". It has since been amended three times by Technical Corrigenda.[24] C99 [24] [ ] C99 introduced several new features, including inline functions, several new data types (including long long int and a complex type to represent complex numbers), variable-length arrays and flexible array members, improved support for IEEE 754 floating point, support for variadic macros (macros of variable arity), and support for one-line comments beginning with //, as in BCPL or C++. Many of these had already been implemented as extensions in several C compilers. inline functions data types complex numbers variable-length arrays flexible array members IEEE 754 variadic macros arity C99 is for the most part backward compatible with C90, but is stricter in some ways; in particular, a declaration that lacks a type specifier no longer has int implicitly assumed. A standard macro __STDC_VERSION__ is defined with value 199901L to indicate that C99 support is available. GCC, Solaris Studio, and other C compilers now[when?] support many or all of the new features of C99. The C compiler in Microsoft Visual C++, however, implements the C89 standard and those parts of C99 that are required for compatibility with C++11.[25][needs update] GCC Solaris Studio when? when? Microsoft Visual C++ C++11 [25] [ ] needs update needs update In addition, the C99 standard requires support for identifiers using Unicode in the form of escaped characters (e.g. \\u0040 or \\U0001f431) and suggests support for raw Unicode names. identifiers Unicode C11 [edit] [ edit edit ] C11 (C standard revision) Work began in 2007 on another revision of the C standard, informally called \"C1X\" until its official publication of ISO/IEC 9899:2011 on December 8, 2011. The C standards committee adopted guidelines to limit the adoption of new features that had not been tested by existing implementations. The C11 standard adds numerous new features to C and the library, including type generic macros, anonymous structures, improved Unicode support, atomic operations, multi-threading, and bounds-checked functions. It also makes some portions of the existing C99 library optional, and improves compatibility with C++. The standard macro __STDC_VERSION__ is defined as 201112L to indicate that C11 support is available. C17 [edit] [ edit edit ] C17 (C standard revision) C17 is an informal name for ISO/IEC 9899:2018, a standard for the C programming language published in June 2018. It introduces no new language features, only technical corrections, and clarifications to defects in C11. The standard macro __STDC_VERSION__ is defined as 201710L to indicate that C17 support is available. C23 [edit] [ edit edit ] C23 (C standard revision) C23 is an informal name for the current major C language standard revision. It was informally known as \"C2X\" through most of its development. C23 was published in October 2024 as ISO/IEC 9899:2024.[26] The standard macro __STDC_VERSION__ is defined as 202311L to indicate that C23 support is available. [26] [ ] C2Y [edit] [ edit edit ] C2Y is an informal name for the next major C language standard revision, after C23 (C2X), that is hoped to be released later in the 2020s decade, hence the '2' in \"C2Y\". An early working draft of C2Y was released in February 2024 as N3220 by the working group ISO/IEC JTC1/SC22/WG14.[27] ISO/IEC JTC1/SC22 [27] [ ] Embedded C [edit] [ edit edit ] Embedded C Historically, embedded C programming requires non-standard extensions to the C language to support exotic features such as fixed-point arithmetic, multiple distinct memory banks, and basic I/O operations. fixed-point arithmetic memory banks In 2008, the C Standards Committee published a technical report extending the C language[28] to address these issues by providing a common standard for all implementations to adhere to. It includes a number of features not available in normal C, such as fixed-point arithmetic, named address spaces, and basic I/O hardware addressing. technical report [28] [ ] Syntax [edit] [ edit edit ] C syntax C has a formal grammar specified by the C standard.[29] Line endings are generally not significant in C; however, line boundaries do have significance during the preprocessing phase. Comments may appear either between the delimiters /* and */, or (since C99) following // until the end of the line. Comments delimited by /* and */ do not nest, and these sequences of characters are not interpreted as comment delimiters if they appear inside string or character literals.[30] formal grammar [29] [ ] string [30] [ ] C source files contain declarations and function definitions. Function definitions, in turn, contain declarations and statements. Declarations either define new types using keywords such as struct, union, and enum, or assign types to and perhaps reserve storage for new variables, usually by writing the type followed by the variable name. Keywords such as char and int specify built-in types. Sections of code are enclosed in braces ({ and }, sometimes called \"curly brackets\") to limit the scope of declarations and to act as a single statement for control structures. statements As an imperative language, C uses statements to specify actions. The most common statement is an expression statement, consisting of an expression to be evaluated, followed by a semicolon; as a side effect of the evaluation, functions may be called and variables assigned new values. To modify the normal sequential execution of statements, C provides several control-flow statements identified by reserved keywords. Structured programming is supported by if ... [else] conditional execution and by do ... while, while, and for iterative execution (looping). The for statement has separate initialization, testing, and reinitialization expressions, any or all of which can be omitted. break and continue can be used within the loop. Break is used to leave the innermost enclosing loop statement and continue is used to skip to its reinitialisation. There is also a non-structured goto statement which branches directly to the designated label within the function. switch selects a case to be executed based on the value of an integer expression. Different from many other languages, control-flow will fall through to the next case unless terminated by a break. side effect functions may be called variables assigned Structured programming goto label switch fall through Expressions can use a variety of built-in operators and may contain function calls. The order in which arguments to functions and operands to most operators are evaluated is unspecified. The evaluations may even be interleaved. However, all side effects (including storage to variables) will occur before the next \"sequence point\"; sequence points include the end of each expression statement, and the entry to and return from each function call. Sequence points also occur during evaluation of expressions containing certain operators (&&, ||, ?: and the comma operator). This permits a high degree of object code optimization by the compiler, but requires C programmers to take more care to obtain reliable results than is needed for other programming languages. sequence point ?: comma operator Kernighan and Ritchie say in the Introduction of The C Programming Language: \"C, like any other language, has its blemishes. Some of the operators have the wrong precedence; some parts of the syntax could be better.\"[31] The C standard did not attempt to correct many of these blemishes, because of the impact of such changes on already existing software. [31] [ ] Character set [edit] [ edit edit ] The basic C source character set includes the following characters: Lowercase and uppercase letters of the ISO basic Latin alphabet: a\u2013z, A\u2013Z ISO basic Latin alphabet Decimal digits: 0\u20139 Graphic characters: ! \" # % & ' ( ) * + , - . / : ; < = > ? [ \\ ] ^ _ { | } ~ Whitespace characters: space, horizontal tab, vertical tab, form feed, newline Whitespace characters space horizontal tab vertical tab form feed newline The newline character indicates the end of a text line; it need not correspond to an actual single character, although for convenience C treats it as such. Additional multi-byte encoded characters may be used in string literals, but they are not entirely portable. The latest C standard (C11) allows multi-national Unicode characters to be embedded portably within C source text by using \\uXXXX or \\UXXXXXXXX encoding (where X denotes a hexadecimal character), although this feature is not yet widely implemented.[needs update] string literals portable C11 needs update needs update The basic C execution character set contains the same characters, along with representations for alert, backspace, and carriage return. Run-time support for extended character sets has increased with each revision of the C standard. alert backspace carriage return Run-time Reserved words [edit] [ edit edit ] The following reserved words are case sensitive. case sensitive C89 has 32 reserved words, also known as 'keywords', which cannot be used for any purposes other than those for which they are predefined: auto break break case char const const continue continue default do double double else else enum enum extern extern float float for for goto goto if if int int long long register register return return short short signed signed sizeof sizeof static static struct struct switch switch typedef typedef union union unsigned unsigned void void volatile volatile while while C99 added five more reserved words: (\u2021 indicates an alternative spelling alias for a C23 keyword) inline inline restrict restrict _Bool \u2021 _Complex _Complex _Imaginary _Imaginary C11 added seven more reserved words:[32] (\u2021 indicates an alternative spelling alias for a C23 keyword) [32] [ ] _Alignas \u2021 _Alignof \u2021 _Atomic _Generic _Noreturn _Static_assert \u2021 _Thread_local \u2021 C23 reserved fifteen more words: alignas alignof bool constexpr false nullptr static_assert thread_local true typeof typeof_unqual _BitInt _Decimal32 _Decimal64 _Decimal128 Most of the recently reserved words begin with an underscore followed by a capital letter, because identifiers of that form were previously reserved by the C standard for use only by implementations. Since existing program source code should not have been using these identifiers, it would not be affected when C implementations started supporting these extensions to the programming language. Some standard headers do define more convenient synonyms for underscored identifiers. Some of those words were added as keywords with their conventional spelling in C23 and the corresponding macros were removed. Prior to C89, entry was reserved as a keyword. In the second edition of their book The C Programming Language, which describes what became known as C89, Kernighan and Ritchie wrote, \"The ... [keyword] entry, formerly reserved but never used, is no longer reserved.\" and \"The stillborn entry keyword is withdrawn.\"[33] The C Programming Language [33] [ ] Operators [edit] [ edit edit ] Operators in C and C++ C supports a rich set of operators, which are symbols used within an expression to specify the manipulations to be performed while evaluating that expression. C has operators for: operators expression arithmetic: +, -, *, /, % arithmetic + - * / % assignment: = assignment augmented assignment: +=, -=, *=, /=, %=, &=, |=, ^=, <<=, >>= augmented assignment bitwise logic: ~, &, |, ^ bitwise logic bitwise shifts: <<, >> bitwise shifts Boolean logic: !, &&, || Boolean logic conditional evaluation: ? : conditional evaluation ? : equality testing: ==, != == != calling functions: ( ) calling functions increment and decrement: ++, -- increment and decrement member selection: ., -> member selection object size: sizeof sizeof type: typeof, typeof_unqual since C23 typeof order relations: <, <=, >, >= order relations reference and dereference: &, *, [ ] reference and dereference sequencing: , , subexpression grouping: ( ) subexpression grouping type conversion: (typename) type conversion C uses the operator = (used in mathematics to express equality) to indicate assignment, following the precedent of Fortran and PL/I, but unlike ALGOL and its derivatives. C uses the operator == to test for equality. The similarity between the operators for assignment and equality may result in the accidental use of one in place of the other, and in many cases the mistake does not produce an error message (although some compilers produce warnings). For example, the conditional expression if (a == b + 1) might mistakenly be written as if (a = b + 1), which will be evaluated as true unless the value of a is 0 after the assignment.[34] Fortran PL/I ALGOL [34] [ ] The C operator precedence is not always intuitive. For example, the operator == binds more tightly than (is executed prior to) the operators & (bitwise AND) and | (bitwise OR) in expressions such as x & 1 == 0, which must be written as (x & 1) == 0 if that is the coder's intent.[35] operator precedence [35] [ ] \"Hello, world\" example [edit] [ edit edit ] \"Hello, World!\" program Brian Kernighan The \"hello, world\" example that appeared in the first edition of K&R has become the model for an introductory program in most programming textbooks. The program prints \"hello, world\" to the standard output, which is usually a terminal or screen display. K&R standard output The original version was:[36] [36] [ ] main () { printf ( \"hello, world \\n \" ); } A standard-conforming \"hello, world\" program is:[a] [a] [ ] #include <stdio.h> int main ( void ) { printf ( \"hello, world \\n \" ); } The first line of the program contains a preprocessing directive, indicated by #include. This causes the compiler to replace that line of code with the entire text of the stdio.h header file, which contains declarations for standard input and output functions such as printf and scanf. The angle brackets surrounding stdio.h indicate that the header file can be located using a search strategy that prefers headers provided with the compiler to other headers having the same name (as opposed to double quotes which typically include local or project-specific header files). preprocessing directive stdio.h The second line indicates that a function named main is being defined. The main function serves a special purpose in C programs; the run-time environment calls the main function to begin program execution. The type specifier int indicates that the value returned to the invoker (in this case the run-time environment) as a result of evaluating the main function, is an integer. The keyword void as a parameter list indicates that the main function takes no arguments.[b] main run-time environment [b] [ ] The opening curly brace indicates the beginning of the code that defines the main function. The next line of the program is a statement that calls (i.e. diverts execution to) a function named printf, which in this case is supplied from a system library. In this call, the printf function is passed (i.e. provided with) a single argument, which is the address of the first character in the string literal \"hello, world\\n\". The string literal is an unnamed array set up automatically by the compiler, with elements of type char and a final NULL character (ASCII value 0) marking the end of the array (to allow printf to determine the length of the string). The NULL character can also be written as the escape sequence \\0. The \\n is a standard escape sequence that C translates to a newline character, which, on output, signifies the end of the current line. The return value of the printf function is of type int, but it is silently discarded since it is not used. (A more careful program might test the return value to check that the printf function succeeded.) The semicolon ; terminates the statement. printf library address string literal array NULL character escape sequence newline The closing curly brace indicates the end of the code for the main function. According to the C99 specification and newer, the main function (unlike any other function) will implicitly return a value of 0 upon reaching the } that terminates the function.[c] The return value of 0 is interpreted by the run-time system as an exit code indicating successful execution of the function.[37] [c] [ ] [37] [ ] Data types [edit] [ edit edit ] C data types verification Please help improve this article by adding citations to reliable sources in this section. Unsourced material may be challenged and removed. improve this article adding citations to reliable sources (October 2012) October 2012 (Learn how and when to remove this message) Learn how and when to remove this message The type system in C is static and weakly typed, which makes it similar to the type system of ALGOL descendants such as Pascal.[38] There are built-in types for integers of various sizes, both signed and unsigned, floating-point numbers, and enumerated types (enum). Integer type char is often used for single-byte characters. C99 added a Boolean data type. There are also derived types including arrays, pointers, records (struct), and unions (union). type system static weakly typed ALGOL Pascal [38] [ ] floating-point numbers Boolean data type arrays pointers records struct unions C is often used in low-level systems programming where escapes from the type system may be necessary. The compiler attempts to ensure type correctness of most expressions, but the programmer can override the checks in various ways, either by using a type cast to explicitly convert a value from one type to another, or by using pointers or unions to reinterpret the underlying bits of a data object in some other way. type cast Some find C's declaration syntax unintuitive, particularly for function pointers. (Ritchie's idea was to declare identifiers in contexts resembling their use: \"declaration reflects use\".)[39] function pointers declaration reflects use [39] [ ] C's usual arithmetic conversions allow for efficient code to be generated, but can sometimes produce unexpected results. For example, a comparison of signed and unsigned integers of equal width requires a conversion of the signed value to unsigned. This can generate unexpected results if the signed value is negative. Pointers [edit] [ edit edit ] C supports the use of pointers, a type of reference that records the address or location of an object or function in memory. Pointers can be dereferenced to access data stored at the address pointed to, or to invoke a pointed-to function. Pointers can be manipulated using assignment or pointer arithmetic. The run-time representation of a pointer value is typically a raw memory address (perhaps augmented by an offset-within-word field), but since a pointer's type includes the type of the thing pointed to, expressions including pointers can be type-checked at compile time. Pointer arithmetic is automatically scaled by the size of the pointed-to data type. pointers reference pointer arithmetic Pointers are used for many purposes in C. Text strings are commonly manipulated using pointers into arrays of characters. Dynamic memory allocation is performed using pointers; the result of a malloc is usually cast to the data type of the data to be stored. Many data types, such as trees, are commonly implemented as dynamically allocated struct objects linked together using pointers. Pointers to other pointers are often used in multi-dimensional arrays and arrays of struct objects. Pointers to functions (function pointers) are useful for passing functions as arguments to higher-order functions (such as qsort or bsearch), in dispatch tables, or as callbacks to event handlers.[37] Text strings Dynamic memory allocation cast trees function pointers higher-order functions qsort bsearch dispatch tables callbacks event handlers [37] [ ] A null pointer value explicitly points to no valid location. Dereferencing a null pointer value is undefined, often resulting in a segmentation fault. Null pointer values are useful for indicating special cases such as no \"next\" pointer in the final node of a linked list, or as an error indication from functions returning pointers. In appropriate contexts in source code, such as for assigning to a pointer variable, a null pointer constant can be written as 0, with or without explicit casting to a pointer type, as the NULL macro defined by several standard headers or, since C23 with the constant nullptr. In conditional contexts, null pointer values evaluate to false, while all other pointer values evaluate to true. null pointer segmentation fault linked list Void pointers (void *) point to objects of unspecified type, and can therefore be used as \"generic\" data pointers. Since the size and type of the pointed-to object is not known, void pointers cannot be dereferenced, nor is pointer arithmetic on them allowed, although they can easily be (and in many contexts implicitly are) converted to and from any other object pointer type.[37] [37] [ ] Careless use of pointers is potentially dangerous. Because they are typically unchecked, a pointer variable can be made to point to any arbitrary location, which can cause undesirable effects. Although properly used pointers point to safe places, they can be made to point to unsafe places by using invalid pointer arithmetic; the objects they point to may continue to be used after deallocation (dangling pointers); they may be used without having been initialized (wild pointers); or they may be directly assigned an unsafe value using a cast, union, or through another corrupt pointer. In general, C is permissive in allowing manipulation of and conversion between pointer types, although compilers typically provide options for various levels of checking. Some other programming languages address these problems by using more restrictive reference types. pointer arithmetic dangling pointers wild pointers reference Arrays [edit] [ edit edit ] C string handling Array types in C are traditionally of a fixed, static size specified at compile time. The more recent C99 standard also allows a form of variable-length arrays. However, it is also possible to allocate a block of memory (of arbitrary size) at run-time, using the standard library's malloc function, and treat it as an array. Array Since arrays are always accessed (in effect) via pointers, array accesses are typically not checked against the underlying array size, although some compilers may provide bounds checking as an option.[40][41] Array bounds violations are therefore possible and can lead to various repercussions, including illegal memory accesses, corruption of data, buffer overruns, and run-time exceptions. bounds checking [40] [ ] [41] [ ] buffer overruns C does not have a special provision for declaring multi-dimensional arrays, but rather relies on recursion within the type system to declare arrays of arrays, which effectively accomplishes the same thing. The index values of the resulting \"multi-dimensional array\" can be thought of as increasing in row-major order. Multi-dimensional arrays are commonly used in numerical algorithms (mainly from applied linear algebra) to store matrices. The structure of the C array is well suited to this particular task. However, in early versions of C the bounds of the array must be known fixed values or else explicitly passed to any subroutine that requires them, and dynamically sized arrays of arrays cannot be accessed using double indexing. (A workaround for this was to allocate the array with an additional \"row vector\" of pointers to the columns.) C99 introduced \"variable-length arrays\" which address this issue. multi-dimensional arrays recursion row-major order linear algebra The following example using modern C (C99 or later) shows allocation of a two-dimensional array on the heap and the use of multi-dimensional array indexing for accesses (which can use bounds-checking on many C compilers): int func ( int N , int M ) { float ( * p )[ N ] [ M ] = malloc ( sizeof * p ); if ( p == 0 ) return -1 ; for ( int i = 0 ; i < N ; i ++ ) for ( int j = 0 ; j < M ; j ++ ) ( * p )[ i ] [ j ] = i + j ; print_array ( N , M , p ); free ( p ); return 1 ; } And here is a similar implementation using C99's Auto VLA feature:[d] VLA [d] [ ] int func ( int N , int M ) { // Caution: checks should be made to ensure N*M*sizeof(float) does NOT exceed limitations for auto VLAs and is within available size of stack. float p [ N ] [ M ]; // auto VLA is held on the stack, and sized when the function is invoked for ( int i = 0 ; i < N ; i ++ ) for ( int j = 0 ; j < M ; j ++ ) p [ i ] [ j ] = i + j ; print_array ( N , M , p ); // no need to free(p) since it will disappear when the function exits, along with the rest of the stack frame return 1 ; } Array\u2013pointer interchangeability [edit] [ edit edit ] The subscript notation x[i] (where x designates a pointer) is syntactic sugar for *(x+i).[42] Taking advantage of the compiler's knowledge of the pointer type, the address that x + i points to is not the base address (pointed to by x) incremented by i bytes, but rather is defined to be the base address incremented by i multiplied by the size of an element that x points to. Thus, x[i] designates the i+1th element of the array. syntactic sugar [42] [ ] Furthermore, in most expression contexts (a notable exception is as operand of sizeof), an expression of array type is automatically converted to a pointer to the array's first element. This implies that an array is never copied as a whole when named as an argument to a function, but rather only the address of its first element is passed. Therefore, although function calls in C use pass-by-value semantics, arrays are in effect passed by reference. sizeof pass-by-value reference The total size of an array x can be determined by applying sizeof to an expression of array type. The size of an element can be determined by applying the operator sizeof to any dereferenced element of an array A, as in n = sizeof A[0]. Thus, the number of elements in a declared array A can be determined as sizeof A / sizeof A[0]. Note, that if only a pointer to the first element is available as it is often the case in C code because of the automatic conversion described above, the information about the full type of the array and its length are lost. Memory management [edit] [ edit edit ] One of the most important functions of a programming language is to provide facilities for managing memory and the objects that are stored in memory. C provides three principal ways to allocate memory for objects:[37] memory [37] [ ] Static memory allocation: space for the object is provided in the binary at compile-time; these objects have an extent (or lifetime) as long as the binary which contains them is loaded into memory. Static memory allocation extent Automatic memory allocation: temporary objects can be stored on the stack, and this space is automatically freed and reusable after the block in which they are declared is exited. Automatic memory allocation stack Dynamic memory allocation: blocks of memory of arbitrary size can be requested at run-time using library functions such as malloc from a region of memory called the heap; these blocks persist until subsequently freed for reuse by calling the library function realloc or free. Dynamic memory allocation heap These three approaches are appropriate in different situations and have various trade-offs. For example, static memory allocation has little allocation overhead, automatic allocation may involve slightly more overhead, and dynamic memory allocation can potentially have a great deal of overhead for both allocation and deallocation. The persistent nature of static objects is useful for maintaining state information across function calls, automatic allocation is easy to use but stack space is typically much more limited and transient than either static memory or heap space, and dynamic memory allocation allows convenient allocation of objects whose size is known only at run-time. Most C programs make extensive use of all three. Where possible, automatic or static allocation is usually simplest because the storage is managed by the compiler, freeing the programmer of the potentially error-prone chore of manually allocating and releasing storage. However, many data structures can change in size at runtime, and since static allocations (and automatic allocations before C99) must have a fixed size at compile-time, there are many situations in which dynamic allocation is necessary.[37] Prior to the C99 standard, variable-sized arrays were a common example of this. (See the article on C dynamic memory allocation for an example of dynamically allocated arrays.) Unlike automatic allocation, which can fail at run time with uncontrolled consequences, the dynamic allocation functions return an indication (in the form of a null pointer value) when the required storage cannot be allocated. (Static allocation that is too large is usually detected by the linker or loader, before the program can even begin execution.) [37] [ ] C dynamic memory allocation linker loader Unless otherwise specified, static objects contain zero or null pointer values upon program startup. Automatically and dynamically allocated objects are initialized only if an initial value is explicitly specified; otherwise they initially have indeterminate values (typically, whatever bit pattern happens to be present in the storage, which might not even represent a valid value for that type). If the program attempts to access an uninitialized value, the results are undefined. Many modern compilers try to detect and warn about this problem, but both false positives and false negatives can occur. bit pattern storage false positives and false negatives Heap memory allocation has to be synchronized with its actual usage in any program to be reused as much as possible. For example, if the only pointer to a heap memory allocation goes out of scope or has its value overwritten before it is deallocated explicitly, then that memory cannot be recovered for later reuse and is essentially lost to the program, a phenomenon known as a memory leak. Conversely, it is possible for memory to be freed, but is referenced subsequently, leading to unpredictable results. Typically, the failure symptoms appear in a portion of the program unrelated to the code that causes the error, making it difficult to diagnose the failure. Such issues are ameliorated in languages with automatic garbage collection. memory leak automatic garbage collection Libraries [edit] [ edit edit ] The C programming language uses libraries as its primary method of extension. In C, a library is a set of functions contained within a single \"archive\" file. Each library typically has a header file, which contains the prototypes of the functions contained within the library that may be used by a program, and declarations of special data types and macro symbols used with these functions. For a program to use a library, it must include the library's header file, and the library must be linked with the program, which in many cases requires compiler flags (e.g., -lm, shorthand for \"link the math library\").[37] libraries header file compiler flags [37] [ ] The most common C library is the C standard library, which is specified by the ISO and ANSI C standards and comes with every C implementation (implementations which target limited environments such as embedded systems may provide only a subset of the standard library). This library supports stream input and output, memory allocation, mathematics, character strings, and time values. Several separate standard headers (for example, stdio.h) specify the interfaces for these and other standard library facilities. C standard library ISO ANSI C embedded systems Another common set of C library functions are those used by applications specifically targeted for Unix and Unix-like systems, especially functions which provide an interface to the kernel. These functions are detailed in various standards such as POSIX and the Single UNIX Specification. Unix Unix-like kernel POSIX Single UNIX Specification Since many programs have been written in C, there are a wide variety of other libraries available. Libraries are often written in C because C compilers generate efficient object code; programmers then create interfaces to the library so that the routines can be used from higher-level languages like Java, Perl, and Python.[37] object code Java Perl Python [37] [ ] File handling and streams [edit] [ edit edit ] File input and output (I/O) is not part of the C language itself but instead is handled by libraries (such as the C standard library) and their associated header files (e.g. stdio.h). File handling is generally implemented through high-level I/O which works through streams. A stream is from this perspective a data flow that is independent of devices, while a file is a concrete device. The high-level I/O is done through the association of a stream to a file. In the C standard library, a buffer (a memory area or queue) is temporarily used to store data before it is sent to the final destination. This reduces the time spent waiting for slower devices, for example a hard drive or solid-state drive. Low-level I/O functions are not part of the standard C library[clarification needed] but are generally part of \"bare metal\" programming (programming that is independent of any operating system such as most embedded programming). With few exceptions, implementations include low-level I/O. streams buffer hard drive solid-state drive clarification needed clarification needed operating system embedded programming Language tools [edit] [ edit edit ] verification Please help improve this article by adding citations to reliable sources in this section. Unsourced material may be challenged and removed. improve this article adding citations to reliable sources (July 2014) July 2014 (Learn how and when to remove this message) Learn how and when to remove this message A number of tools have been developed to help C programmers find and fix statements with undefined behavior or possibly erroneous expressions, with greater rigor than that provided by the compiler. The tool lint was the first such, leading to many others. lint Automated source code checking and auditing are beneficial in any language, and for C many such tools exist, such as Lint. A common practice is to use Lint to detect questionable code when a program is first written. Once a program passes Lint, it is then compiled using the C compiler. Also, many compilers can optionally warn about syntactically valid constructs that are likely to actually be errors. MISRA C is a proprietary set of guidelines to avoid such questionable code, developed for embedded systems.[43] Lint MISRA C [43] [ ] There are also compilers, libraries, and operating system level mechanisms for performing actions that are not a standard part of C, such as bounds checking for arrays, detection of buffer overflow, serialization, dynamic memory tracking, and automatic garbage collection. bounds checking buffer overflow serialization dynamic memory automatic garbage collection Tools such as Purify or Valgrind and linking with libraries containing special versions of the memory allocation functions can help uncover runtime errors in memory usage.[44][45] Purify Valgrind memory allocation functions [44] [ ] [45] [ ] Uses [edit] [ edit edit ] Rationale for use in systems programming [edit] [ edit edit ] C is widely used for systems programming in implementing operating systems and embedded system applications.[46] This is for several reasons: systems programming operating systems embedded system [46] [ ] The C language permits platform hardware and memory to be accessed with pointers and type punning, so system-specific features (e.g. Control/Status Registers, I/O registers) can be configured and used with code written in C \u2013 it allows fullest control of the platform it is running on. type punning Control/Status Registers I/O registers The code generated after compilation does not demand many system features, and can be invoked from some boot code in a straightforward manner \u2013 it is simple to execute. system features The C language statements and expressions typically map well on to sequences of instructions for the target processor, and consequently there is a low run-time demand on system resources \u2013 it is fast to execute. run-time With its rich set of operators, the C language can use many of the features of target CPUs. Where a particular CPU has more esoteric instructions, a language variant can be constructed with perhaps intrinsic functions to exploit those instructions \u2013 it can use practically all the target CPU's features. intrinsic functions The language makes it easy to overlay structures onto blocks of binary data, allowing the data to be comprehended, navigated and modified \u2013 it can write data structures, even file systems. The language supports a rich set of operators, including bit manipulation, for integer arithmetic and logic, and perhaps different sizes of floating point numbers \u2013 it can process appropriately-structured data effectively. C is a fairly small language, with only a handful of statements, and without too many features that generate extensive target code \u2013 it is comprehensible. C has direct control over memory allocation and deallocation, which gives reasonable efficiency and predictable timing to memory-handling operations, without any concerns for sporadic stop-the-world garbage collection events \u2013 it has predictable performance. stop-the-world C permits the use and implementation of different memory allocation schemes, including a typical malloc and free; a more sophisticated mechanism with arenas; or a version for an OS kernel that may suit DMA, use within interrupt handlers, or integrated with the virtual memory system. memory allocation malloc free arenas OS kernel DMA interrupt handlers virtual memory Depending on the linker and environment, C code can also call libraries written in assembly language, and may be called from assembly language \u2013 it interoperates well with other lower-level code. assembly language C and its calling conventions and linker structures are commonly used in conjunction with other high-level languages, with calls both to C and from C supported \u2013 it interoperates well with other high-level code. calling conventions C has a very mature and broad ecosystem, including libraries, frameworks, open source compilers, debuggers and utilities, and is the de facto standard. It is likely the drivers already exist in C, or that there is a similar CPU architecture as a back-end of a C compiler, so there is reduced incentive to choose another language. Used for computationally-intensive libraries [edit] [ edit edit ] C enables programmers to create efficient implementations of algorithms and data structures, because the layer of abstraction from hardware is thin, and its overhead is low, an important criterion for computationally intensive programs. For example, the GNU Multiple Precision Arithmetic Library, the GNU Scientific Library, Mathematica, and MATLAB are completely or partially written in C. Many languages support calling library functions in C, for example, the Python-based framework NumPy uses C for the high-performance and hardware-interacting aspects. GNU Multiple Precision Arithmetic Library GNU Scientific Library Mathematica MATLAB Python NumPy C as an intermediate language [edit] [ edit edit ] C is sometimes used as an intermediate language by implementations of other languages. This approach may be used for portability or convenience; by using C as an intermediate language, additional machine-specific code generators are not necessary. C has some features, such as line-number preprocessor directives and optional superfluous commas at the end of initializer lists, that support compilation of generated code. However, some of C's shortcomings have prompted the development of other C-based languages specifically designed for use as intermediate languages, such as C--. Also, contemporary major compilers GCC and LLVM both feature an intermediate representation that is not C, and those compilers support front ends for many languages including C. intermediate language C-based languages C-- GCC LLVM intermediate representation Other languages written in C [edit] [ edit edit ] A consequence of C's wide availability and efficiency is that compilers, libraries and interpreters of other programming languages are often implemented in C.[47] For example, the reference implementations of Python,[48] Perl,[49] Ruby,[50] and PHP[51] are written in C. compilers interpreters [47] [ ] reference implementations Python [48] [ ] Perl [49] [ ] Ruby [50] [ ] PHP [51] [ ] Once used for web development [edit] [ edit edit ] Historically, C was sometimes used for web development using the Common Gateway Interface (CGI) as a \"gateway\" for information between the web application, the server, and the browser.[52] C may have been chosen over interpreted languages because of its speed, stability, and near-universal availability.[53] It is no longer common practice for web development to be done in C,[54] and many other web development tools exist. web development Common Gateway Interface [52] [ ] interpreted languages [53] [ ] [54] [ ] web development tools Web servers [edit] [ edit edit ] The two most popular web servers, Apache HTTP Server and Nginx, are both written in C. These web servers interact with the operating system, listen on TCP ports for HTTP requests, and then serve up static web content, or cause the execution of other languages handling to 'render' content such as PHP, which is itself primarily written in C. C's close-to-the-metal approach allows for the construction of these high-performance software systems. web servers Apache HTTP Server Nginx PHP End-user applications [edit] [ edit edit ] C has also been widely used to implement end-user applications.[55] However, such applications can also be written in newer, higher-level languages. end-user [55] [ ] Limitations [edit] [ edit edit ] the power of assembly language and the convenience of ... assembly language [56] [ ] While C has been popular, influential and hugely successful, it has drawbacks, including: The standard dynamic memory handling with malloc and free is error prone. Improper use can lead to memory leaks and dangling pointers. dynamic memory memory leaks dangling pointers The use of pointers and the direct manipulation of memory means corruption of memory is possible, perhaps due to programmer error, or insufficient checking of bad data. There is some type checking, but it does not apply to areas like variadic functions, and the type checking can be trivially or inadvertently circumvented. It is weakly typed. type checking variadic functions weakly typed Since the code generated by the compiler contains few checks itself, there is a burden on the programmer to consider all possible outcomes, to protect against buffer overruns, array bounds checking, stack overflows, memory exhaustion, and consider race conditions, thread isolation, etc. stack overflows race conditions The use of pointers and the run-time manipulation of these means there may be two ways to access the same data (aliasing), which is not determinable at compile time. This means that some optimisations that may be available to other languages are not possible in C. FORTRAN is considered faster. Some of the standard library functions, e.g. scanf or strncat, can lead to buffer overruns. buffer overruns There is limited standardisation in support for low-level variants in generated code, for example: different function calling conventions and ABI; different structure packing conventions; different byte ordering within larger integers (including endianness). In many language implementations, some of these options may be handled with the preprocessor directive #pragma,[57][58] and some with additional keywords e.g. use __cdecl calling convention. The directive and options are not consistently supported.[59] calling conventions ABI structure packing #pragma [57] [ ] [58] [ ] __cdecl [59] [ ] String handling using the standard library is code-intensive, with explicit memory management required. String handling The language does not directly support object orientation, introspection, run-time expression evaluation, generics, etc. introspection There are few guards against inappropriate use of language features, which may lead to unmaintainable code. In particular, the C preprocessor can hide troubling effects such as double evaluation and worse.[60] This facility for tricky code has been celebrated with competitions such as the International Obfuscated C Code Contest and the Underhanded C Contest. unmaintainable C preprocessor [60] [ ] International Obfuscated C Code Contest Underhanded C Contest C lacks standard support for exception handling and only offers return codes for error checking. The setjmp and longjmp standard library functions have been used[61] to implement a try-catch mechanism via macros. exception handling return codes setjmp and longjmp [61] [ ] For some purposes, restricted styles of C have been adopted, e.g. MISRA C or CERT C, in an attempt to reduce the opportunity for bugs. Databases such as CWE attempt to count the ways C etc. has vulnerabilities, along with recommendations for mitigation. MISRA C CERT C CWE There are tools that can mitigate against some of the drawbacks. Contemporary C compilers include checks which may generate warnings to help identify many potential bugs. tools Related languages [edit] [ edit edit ] TIOBE index [62] [ ] List of C-family programming languages C has both directly and indirectly influenced many later languages such as C++ and Java.[63] The most pervasive influence has been syntactical; all of the languages mentioned combine the statement and (more or less recognizably) expression syntax of C with type systems, data models or large-scale program structures that differ from those of C, sometimes radically. C++ Java [63] [ ] syntax of C Several C or near-C interpreters exist, including Ch and CINT, which can also be used for scripting. Ch CINT When object-oriented programming languages became popular, C++ and Objective-C were two different extensions of C that provided object-oriented capabilities. Both languages were originally implemented as source-to-source compilers; source code was translated into C, and then compiled with a C compiler.[64] object-oriented programming C++ Objective-C source-to-source compilers [64] [ ] The C++ programming language (originally named \"C with Classes\") was devised by Bjarne Stroustrup as an approach to providing object-oriented functionality with a C-like syntax.[65] C++ adds greater typing strength, scoping, and other tools useful in object-oriented programming, and permits generic programming via templates. Nearly a superset of C, C++ now[when?] supports most of C, with a few exceptions. C++ Classes Bjarne Stroustrup object-oriented [65] [ ] generic programming when? when? a few exceptions Objective-C was originally a very \"thin\" layer on top of C, and remains a strict superset of C that permits object-oriented programming using a hybrid dynamic/static typing paradigm. Objective-C derives its syntax from both C and Smalltalk: syntax that involves preprocessing, expressions, function declarations, and function calls is inherited from C, while the syntax for object-oriented features was originally taken from Smalltalk. Objective-C superset Smalltalk In addition to C++ and Objective-C, Ch, Cilk, and Unified Parallel C are nearly supersets of C. C++ Objective-C Ch Cilk Unified Parallel C See also [edit] [ edit edit ] Computer programming portal Computer programming portal Computer programming portal Free and open-source software portal Free and open-source software portal Free and open-source software portal Compatibility of C and C++ Compatibility of C and C++ Comparison of Pascal and C Comparison of Pascal and C Comparison of programming languages Comparison of programming languages International Obfuscated C Code Contest International Obfuscated C Code Contest List of C-family programming languages List of C-family programming languages List of C compilers List of C compilers Notes [edit] [ edit edit ] ^ The original example code will compile on most modern compilers that are not in strict standard compliance mode, but it does not fully conform to the requirements of either C89 or C99. In fact, C99 requires that a diagnostic message be produced. ^ ^ The original example code will compile on most modern compilers that are not in strict standard compliance mode, but it does not fully conform to the requirements of either C89 or C99. In fact, C99 requires that a diagnostic message be produced. ^ The main function actually has two arguments, int argc and char *argv[], respectively, which can be used to handle command-line arguments. The ISO C standard (section 5.1.2.2.1) requires both forms of main to be supported, which is special treatment not afforded to any other function. ^ ^ The main function actually has two arguments, int argc and char *argv[], respectively, which can be used to handle command-line arguments. The ISO C standard (section 5.1.2.2.1) requires both forms of main to be supported, which is special treatment not afforded to any other function. command-line arguments ^ Prior to C99, an explicit return 0; statement was required at the end of the main function. ^ ^ Prior to C99, an explicit return 0; statement was required at the end of the main function. ^ Code of print_array (not shown) slightly differs,[why?] too. ^ ^ Code of print_array (not shown) slightly differs,[why?] too. why? why? References [edit] [ edit edit ] ^ a b Prinz, Peter; Crawford, Tony (December 16, 2005). C in a Nutshell. O'Reilly Media, Inc. p. 3. ISBN 9780596550714. ^ a b a b Prinz, Peter; Crawford, Tony (December 16, 2005). C in a Nutshell. O'Reilly Media, Inc. p. 3. ISBN 9780596550714. C in a Nutshell ISBN 9780596550714 ^ Ritchie (1993): \"Thompson had made a brief attempt to produce a system coded in an early version of C\u2014before structures\u2014in 1972, but gave up the effort.\" ^ ^ Ritchie (1993): \"Thompson had made a brief attempt to produce a system coded in an early version of C\u2014before structures\u2014in 1972, but gave up the effort.\" Ritchie (1993) ^ \"N3221 \u2013 Editor's Report, Post January 2024 Strasbourg France Meeting\". ISO/IEC JTC1/SC22/WG14. Open Standards. February 21, 2024. Retrieved May 24, 2024. ^ ^ \"N3221 \u2013 Editor's Report, Post January 2024 Strasbourg France Meeting\". ISO/IEC JTC1/SC22/WG14. Open Standards. February 21, 2024. Retrieved May 24, 2024. \"N3221 \u2013 Editor's Report, Post January 2024 Strasbourg France Meeting\" . Retrieved May 24, 2024 May 24, ^ Ritchie (1993): \"The scheme of type composition adopted by C owes considerable debt to Algol 68, although it did not, perhaps, emerge in a form that Algol's adherents would approve of.\" ^ ^ Ritchie (1993): \"The scheme of type composition adopted by C owes considerable debt to Algol 68, although it did not, perhaps, emerge in a form that Algol's adherents would approve of.\" Ritchie (1993) ^ a b \"Verilog HDL (and C)\" (PDF). The Research School of Computer Science at the Australian National University. June 3, 2010. Archived from the original (PDF) on November 6, 2013. Retrieved August 19, 2013. 1980s: Verilog first introduced; Verilog inspired by the C programming language ^ a b a b \"Verilog HDL (and C)\" (PDF). The Research School of Computer Science at the Australian National University. June 3, 2010. Archived from the original (PDF) on November 6, 2013. Retrieved August 19, 2013. 1980s: Verilog first introduced; Verilog inspired by the C programming language \"Verilog HDL (and C)\" (PDF) the original (PDF) . Retrieved August 19, 2013 August 19, ^ \"The name is based on, and pronounced like the letter C in the English alphabet\". the c programming language sound. English Chinese Dictionary. Archived from the original on November 17, 2022. Retrieved November 17, 2022. ^ ^ \"The name is based on, and pronounced like the letter C in the English alphabet\". the c programming language sound. English Chinese Dictionary. Archived from the original on November 17, 2022. Retrieved November 17, 2022. \"The name is based on, and pronounced like the letter C in the English alphabet\" Archived . Retrieved November 17, 2022 November 17, ^ Munoz, Daniel. \"After All These Years, the World is Still Powered by C Programming | Toptal\". Toptal Engineering Blog. Retrieved June 15, 2024. ^ ^ Munoz, Daniel. \"After All These Years, the World is Still Powered by C Programming | Toptal\". Toptal Engineering Blog. Retrieved June 15, 2024. \"After All These Years, the World is Still Powered by C Programming | Toptal\" . Retrieved June 15, 2024 June 15, ^ \"C Language Drops to Lowest Popularity Rating\". Developer.com. August 9, 2016. Archived from the original on August 22, 2022. Retrieved August 1, 2022. ^ ^ \"C Language Drops to Lowest Popularity Rating\". Developer.com. August 9, 2016. Archived from the original on August 22, 2022. Retrieved August 1, 2022. \"C Language Drops to Lowest Popularity Rating\" the original . Retrieved August 1, 2022 August 1, ^ a b c d e f Ritchie (1993) ^ a b c d e f a b c d e f Ritchie (1993) Ritchie (1993) ^ \"Programming Language Popularity\". 2009. Archived from the original on January 16, 2009. Retrieved January 16, 2009. ^ ^ \"Programming Language Popularity\". 2009. Archived from the original on January 16, 2009. Retrieved January 16, 2009. \"Programming Language Popularity\" the original . Retrieved January 16, 2009 January 16, ^ \"TIOBE Programming Community Index\". 2009. Archived from the original on May 4, 2009. Retrieved May 6, 2009. ^ ^ \"TIOBE Programming Community Index\". 2009. Archived from the original on May 4, 2009. Retrieved May 6, 2009. \"TIOBE Programming Community Index\" the original . Retrieved May 6, 2009 May 6, ^ Ward, Terry A. (August 1983). \"Annotated C / A Bibliography of the C Language\". Byte. p. 268. Retrieved January 31, 2015. ^ ^ Ward, Terry A. (August 1983). \"Annotated C / A Bibliography of the C Language\". Byte. p. 268. Retrieved January 31, 2015. \"Annotated C / A Bibliography of the C Language\" . Retrieved January 31, 2015 January 31, ^ \"TIOBE Index for September 2024\". Archived from the original on September 18, 2024. Retrieved September 20, 2024. ^ ^ \"TIOBE Index for September 2024\". Archived from the original on September 18, 2024. Retrieved September 20, 2024. \"TIOBE Index for September 2024\" Archived . Retrieved September 20, 2024 September 20, ^ Ritchie, Dennis. \"BCPL to B to C\". lysator.liu.se. Archived from the original on December 12, 2019. Retrieved September 10, 2019. ^ ^ Ritchie, Dennis. \"BCPL to B to C\". lysator.liu.se. Archived from the original on December 12, 2019. Retrieved September 10, 2019. \"BCPL to B to C\" Archived . Retrieved September 10, 2019 September 10, ^ a b c d e Jensen, Richard (December 9, 2020). \"\"A damn stupid thing to do\"\u2014the origins of C\". Ars Technica. Archived from the original on March 28, 2022. Retrieved March 28, 2022. ^ a b c d e a b c d e Jensen, Richard (December 9, 2020). \"\"A damn stupid thing to do\"\u2014the origins of C\". Ars Technica. Archived from the original on March 28, 2022. Retrieved March 28, 2022. \"\"A damn stupid thing to do\"\u2014the origins of C\" Archived . Retrieved March 28, 2022 March 28, ^ a b Johnson, S. C.; Ritchie, D. M. (1978). \"Portability of C Programs and the UNIX System\". Bell System Tech. J. 57 (6): 2021\u20132048. CiteSeerX 10.1.1.138.35. doi:10.1002/j.1538-7305.1978.tb02141.x. ISSN 0005-8580. S2CID 17510065. (Note: The PDF is an OCR scan of the original, and contains a rendering of \"IBM 370\" as \"IBM 310\".) ^ a b a b Johnson, S. C.; Ritchie, D. M. (1978). \"Portability of C Programs and the UNIX System\". Bell System Tech. J. 57 (6): 2021\u20132048. CiteSeerX 10.1.1.138.35. doi:10.1002/j.1538-7305.1978.tb02141.x. ISSN 0005-8580. S2CID 17510065. (Note: The PDF is an OCR scan of the original, and contains a rendering of \"IBM 370\" as \"IBM 310\".) Johnson, S. C. Ritchie, D. M. 2021\u2013 CiteSeerX 10.1.1.138.35 10.1.1.138.35 doi 10.1002/j.1538-7305.1978.tb02141.x ISSN 0005-8580 S2CID 17510065 ^ McIlroy, M. D. (1987). A Research Unix reader: annotated excerpts from the Programmer's Manual, 1971\u20131986 (PDF) (Technical report). CSTR. Bell Labs. p. 10. 139. Archived (PDF) from the original on November 11, 2017. Retrieved February 1, 2015. ^ ^ McIlroy, M. D. (1987). A Research Unix reader: annotated excerpts from the Programmer's Manual, 1971\u20131986 (PDF) (Technical report). CSTR. Bell Labs. p. 10. 139. Archived (PDF) from the original on November 11, 2017. Retrieved February 1, 2015. McIlroy, M. D. A Research Unix reader: annotated excerpts from the Programmer's Manual, 1971\u20131986 (PDF) Archived (PDF) . Retrieved February 1, 2015 February 1, ^ Kernighan, Brian W.; Ritchie, Dennis M. (February 1978). The C Programming Language (1st ed.). Englewood Cliffs, NJ: Prentice Hall. ISBN 978-0-13-110163-0. ^ ^ Kernighan, Brian W.; Ritchie, Dennis M. (February 1978). The C Programming Language (1st ed.). Englewood Cliffs, NJ: Prentice Hall. ISBN 978-0-13-110163-0. Kernighan, Brian W. Ritchie, Dennis M. The C Programming Language Englewood Cliffs, NJ Prentice Hall ISBN 978-0-13-110163-0 ^ \"C manual pages\". FreeBSD Miscellaneous Information Manual (FreeBSD 13.0 ed.). May 30, 2011. Archived from the original on January 21, 2021. Retrieved January 15, 2021. [1] Archived January 21, 2021, at the Wayback Machine ^ ^ \"C manual pages\". FreeBSD Miscellaneous Information Manual (FreeBSD 13.0 ed.). May 30, 2011. Archived from the original on January 21, 2021. Retrieved January 15, 2021. [1] Archived January 21, 2021, at the Wayback Machine FreeBSD Miscellaneous Information Manual Archived . Retrieved January 15, 2021 January 15, [1] Archived Wayback Machine ^ Kernighan, Brian W.; Ritchie, Dennis M. (March 1988). The C Programming Language (2nd ed.). Englewood Cliffs, NJ: Prentice Hall. ISBN 978-0-13-110362-7. ^ ^ Kernighan, Brian W.; Ritchie, Dennis M. (March 1988). The C Programming Language (2nd ed.). Englewood Cliffs, NJ: Prentice Hall. ISBN 978-0-13-110362-7. Kernighan, Brian W. Ritchie, Dennis M. The C Programming Language Englewood Cliffs, NJ Prentice Hall ISBN 978-0-13-110362-7 ^ Stroustrup, Bjarne (2002). Sibling rivalry: C and C++ (PDF) (Report). AT&T Labs. Archived (PDF) from the original on August 24, 2014. Retrieved April 14, 2014. ^ ^ Stroustrup, Bjarne (2002). Sibling rivalry: C and C++ (PDF) (Report). AT&T Labs. Archived (PDF) from the original on August 24, 2014. Retrieved April 14, 2014. Stroustrup, Bjarne Sibling rivalry: C and C++ (PDF) Archived (PDF) . Retrieved April 14, 2014 April 14, ^ \"Rationale for American National Standard for Information Systems \u2013 Programming Language \u2013 C\". Archived from the original on July 17, 2024. Retrieved July 17, 2024. ^ ^ \"Rationale for American National Standard for Information Systems \u2013 Programming Language \u2013 C\". Archived from the original on July 17, 2024. Retrieved July 17, 2024. \"Rationale for American National Standard for Information Systems \u2013 Programming Language \u2013 C\" the original . Retrieved July 17, 2024 July 17, ^ C Integrity. International Organization for Standardization. March 30, 1995. Archived from the original on July 25, 2018. Retrieved July 24, 2018. ^ ^ C Integrity. International Organization for Standardization. March 30, 1995. Archived from the original on July 25, 2018. Retrieved July 24, 2018. C Integrity Archived . Retrieved July 24, 2018 July 24, ^ \"JTC1/SC22/WG14 \u2013 C\". Home page. ISO/IEC. Archived from the original on February 12, 2018. Retrieved June 2, 2011. ^ ^ \"JTC1/SC22/WG14 \u2013 C\". Home page. ISO/IEC. Archived from the original on February 12, 2018. Retrieved June 2, 2011. \"JTC1/SC22/WG14 \u2013 C\" Archived . Retrieved June 2, 2011 June 2, ^ Andrew Binstock (October 12, 2011). \"Interview with Herb Sutter\". Dr. Dobbs. Archived from the original on August 2, 2013. Retrieved September 7, 2013. ^ ^ Andrew Binstock (October 12, 2011). \"Interview with Herb Sutter\". Dr. Dobbs. Archived from the original on August 2, 2013. Retrieved September 7, 2013. \"Interview with Herb Sutter\" Dr. Dobbs Archived . Retrieved September 7, 2013 September 7, ^ \"WG14-N3132 : Revised C23 Schedule\" (PDF). open-std.org. June 4, 2023. Archived (PDF) from the original on June 9, 2023. ^ ^ \"WG14-N3132 : Revised C23 Schedule\" (PDF). open-std.org. June 4, 2023. Archived (PDF) from the original on June 9, 2023. \"WG14-N3132 : Revised C23 Schedule\" (PDF) Archived (PDF) ^ \"WG14-N3220 : Working Draft, C2y\" (PDF). open-std.org. February 21, 2024. Archived (PDF) from the original on February 26, 2024. ^ ^ \"WG14-N3220 : Working Draft, C2y\" (PDF). open-std.org. February 21, 2024. Archived (PDF) from the original on February 26, 2024. \"WG14-N3220 : Working Draft, C2y\" (PDF) Archived (PDF) ^ \"TR 18037: Embedded C\" (PDF). open-std.org. April 4, 2006. ISO/IEC JTC1 SC22 WG14 N1169. Archived (PDF) from the original on February 25, 2021. Retrieved July 26, 2011. ^ ^ \"TR 18037: Embedded C\" (PDF). open-std.org. April 4, 2006. ISO/IEC JTC1 SC22 WG14 N1169. Archived (PDF) from the original on February 25, 2021. Retrieved July 26, 2011. \"TR 18037: Embedded C\" (PDF) Archived (PDF) . Retrieved July 26, 2011 July 26, ^ Harbison, Samuel P.; Steele, Guy L. (2002). C: A Reference Manual (5th ed.). Englewood Cliffs, NJ: Prentice Hall. ISBN 978-0-13-089592-9. Contains a BNF grammar for C. ^ ^ Harbison, Samuel P.; Steele, Guy L. (2002). C: A Reference Manual (5th ed.). Englewood Cliffs, NJ: Prentice Hall. ISBN 978-0-13-089592-9. Contains a BNF grammar for C. Steele, Guy L. Englewood Cliffs, NJ Prentice Hall ISBN 978-0-13-089592-9 BNF ^ Kernighan & Ritchie (1988), p. 192. ^ ^ Kernighan & Ritchie (1988), p. 192. Kernighan & Ritchie (1988) ^ Kernighan & Ritchie (1978), p. 3. ^ ^ Kernighan & Ritchie (1978), p. 3. Kernighan & Ritchie (1978) ^ \"ISO/IEC 9899:201x (ISO C11) Committee Draft\" (PDF). open-std.org. December 2, 2010. Archived (PDF) from the original on December 22, 2017. Retrieved September 16, 2011. ^ ^ \"ISO/IEC 9899:201x (ISO C11) Committee Draft\" (PDF). open-std.org. December 2, 2010. Archived (PDF) from the original on December 22, 2017. Retrieved September 16, 2011. \"ISO/IEC 9899:201x (ISO C11) Committee Draft\" (PDF) Archived (PDF) . Retrieved September 16, 2011 September 16, ^ Kernighan & Ritchie (1988), pp. 192, 259. ^ ^ Kernighan & Ritchie (1988), pp. 192, 259. Kernighan & Ritchie (1988) ^ \"10 Common Programming Mistakes in C++\". Cs.ucr.edu. Archived from the original on October 21, 2008. Retrieved June 26, 2009. ^ ^ \"10 Common Programming Mistakes in C++\". Cs.ucr.edu. Archived from the original on October 21, 2008. Retrieved June 26, 2009. \"10 Common Programming Mistakes in C++\" Archived . Retrieved June 26, 2009 June 26, ^ Schultz, Thomas (2004). C and the 8051 (3rd ed.). Otsego, MI: PageFree Publishing Inc. p. 20. ISBN 978-1-58961-237-2. Retrieved February 10, 2012. ^ ^ Schultz, Thomas (2004). C and the 8051 (3rd ed.). Otsego, MI: PageFree Publishing Inc. p. 20. ISBN 978-1-58961-237-2. Retrieved February 10, 2012. C and the 8051 ISBN 978-1-58961-237-2 . Retrieved February 10, 2012 February 10, ^ Kernighan & Ritchie (1978), p. 6. ^ ^ Kernighan & Ritchie (1978), p. 6. Kernighan & Ritchie (1978) ^ a b c d e f g Klemens, Ben (2013). 21st Century C. O'Reilly Media. ISBN 978-1-4493-2714-9. ^ a b c d e f g a b c d e f g Klemens, Ben (2013). 21st Century C. O'Reilly Media. ISBN 978-1-4493-2714-9. Klemens, Ben O'Reilly Media ISBN 978-1-4493-2714-9 ^ Feuer, Alan R.; Gehani, Narain H. (March 1982). \"Comparison of the Programming Languages C and Pascal\". ACM Computing Surveys. 14 (1): 73\u201392. doi:10.1145/356869.356872. S2CID 3136859. ^ ^ Feuer, Alan R.; Gehani, Narain H. (March 1982). \"Comparison of the Programming Languages C and Pascal\". ACM Computing Surveys. 14 (1): 73\u201392. doi:10.1145/356869.356872. S2CID 3136859. 73\u2013 doi 10.1145/356869.356872 S2CID 3136859 ^ Kernighan & Ritchie (1988), p. 122. ^ ^ Kernighan & Ritchie (1988), p. 122. Kernighan & Ritchie (1988) ^ For example, gcc provides _FORTIFY_SOURCE. \"Security Features: Compile Time Buffer Checks (FORTIFY_SOURCE)\". fedoraproject.org. Archived from the original on January 7, 2007. Retrieved August 5, 2012. ^ ^ For example, gcc provides _FORTIFY_SOURCE. \"Security Features: Compile Time Buffer Checks (FORTIFY_SOURCE)\". fedoraproject.org. Archived from the original on January 7, 2007. Retrieved August 5, 2012. \"Security Features: Compile Time Buffer Checks (FORTIFY_SOURCE)\" Archived . Retrieved August 5, 2012 August 5, ^ \u0e40\u0e2d\u0e35\u0e48\u0e22\u0e21\u0e2a\u0e34\u0e23\u0e34\u0e27\u0e07\u0e28\u0e4c, \u0e42\u0e2d\u0e20\u0e32\u0e28 (2016). Programming with C. Bangkok, Thailand: SE-EDUCATION PUBLIC COMPANY LIMITED. pp. 225\u2013230. ISBN 978-616-08-2740-4. ^ ^ \u0e40\u0e2d\u0e35\u0e48\u0e22\u0e21\u0e2a\u0e34\u0e23\u0e34\u0e27\u0e07\u0e28\u0e4c, \u0e42\u0e2d\u0e20\u0e32\u0e28 (2016). Programming with C. Bangkok, Thailand: SE-EDUCATION PUBLIC COMPANY LIMITED. pp. 225\u2013230. ISBN 978-616-08-2740-4. 225\u2013 ISBN 978-616-08-2740-4 ^ Raymond, Eric S. (October 11, 1996). The New Hacker's Dictionary (3rd ed.). MIT Press. p. 432. ISBN 978-0-262-68092-9. Retrieved August 5, 2012. ^ ^ Raymond, Eric S. (October 11, 1996). The New Hacker's Dictionary (3rd ed.). MIT Press. p. 432. ISBN 978-0-262-68092-9. Retrieved August 5, 2012. Raymond, Eric S. The New Hacker's Dictionary ISBN 978-0-262-68092-9 . Retrieved August 5, 2012 August 5, ^ \"Man Page for lint (freebsd Section 1)\". unix.com. May 24, 2001. Retrieved July 15, 2014. ^ ^ \"Man Page for lint (freebsd Section 1)\". unix.com. May 24, 2001. Retrieved July 15, 2014. \"Man Page for lint (freebsd Section 1)\" . Retrieved July 15, 2014 July 15, ^ \"CS107 Valgrind Memcheck\". web.stanford.edu. Retrieved June 23, 2023. ^ ^ \"CS107 Valgrind Memcheck\". web.stanford.edu. Retrieved June 23, 2023. \"CS107 Valgrind Memcheck\" . Retrieved June 23, 2023 June 23, ^ Hastings, Reed; Joyce, Bob. \"Purify: Fast Detection of Memory Leaks and Access Errors\" (PDF). Pure Software Inc.: 9. ^ ^ Hastings, Reed; Joyce, Bob. \"Purify: Fast Detection of Memory Leaks and Access Errors\" (PDF). Pure Software Inc.: 9. \"Purify: Fast Detection of Memory Leaks and Access Errors\" (PDF) ^ Dale, Nell B.; Weems, Chip (2014). Programming and problem solving with C++ (6th ed.). Burlington, Massachusetts: Jones & Bartlett Learning. ISBN 978-1449694289. OCLC 894992484. ^ ^ Dale, Nell B.; Weems, Chip (2014). Programming and problem solving with C++ (6th ed.). Burlington, Massachusetts: Jones & Bartlett Learning. ISBN 978-1449694289. OCLC 894992484. ISBN 978-1449694289 OCLC 894992484 ^ \"C \u2013 the mother of all languages\". ICT Academy at IITK. November 13, 2018. Archived from the original on May 31, 2021. Retrieved October 11, 2022. ^ ^ \"C \u2013 the mother of all languages\". ICT Academy at IITK. November 13, 2018. Archived from the original on May 31, 2021. Retrieved October 11, 2022. \"C \u2013 the mother of all languages\" the original . Retrieved October 11, 2022 October 11, ^ \"1. Extending Python with C or C++\". Python 3.10.7 documentation. Archived from the original on November 5, 2012. Retrieved October 11, 2022. ^ ^ \"1. Extending Python with C or C++\". Python 3.10.7 documentation. Archived from the original on November 5, 2012. Retrieved October 11, 2022. \"1. Extending Python with C or C++\" Archived . Retrieved October 11, 2022 October 11, ^ Conrad, Michael (January 22, 2018). \"An overview of the Perl 5 engine\". Opensource.com. Archived from the original on May 26, 2022. Retrieved October 11, 2022. ^ ^ Conrad, Michael (January 22, 2018). \"An overview of the Perl 5 engine\". Opensource.com. Archived from the original on May 26, 2022. Retrieved October 11, 2022. \"An overview of the Perl 5 engine\" Archived . Retrieved October 11, 2022 October 11, ^ \"To Ruby From C and C++\". Ruby Programming Language. Archived from the original on August 12, 2013. Retrieved October 11, 2022. ^ ^ \"To Ruby From C and C++\". Ruby Programming Language. Archived from the original on August 12, 2013. Retrieved October 11, 2022. \"To Ruby From C and C++\" Archived . Retrieved October 11, 2022 October 11, ^ Para, Michael (August 3, 2022). \"What is PHP? How to Write Your First PHP Program\". freeCodeCamp. Archived from the original on August 4, 2022. Retrieved October 11, 2022. ^ ^ Para, Michael (August 3, 2022). \"What is PHP? How to Write Your First PHP Program\". freeCodeCamp. Archived from the original on August 4, 2022. Retrieved October 11, 2022. \"What is PHP? How to Write Your First PHP Program\" Archived . Retrieved October 11, 2022 October 11, ^ Dr. Dobb's Sourcebook. U.S.: Miller Freeman, Inc. November\u2013December 1995. ^ ^ Dr. Dobb's Sourcebook. U.S.: Miller Freeman, Inc. November\u2013December 1995. ^ \"Using C for CGI Programming\". linuxjournal.com. March 1, 2005. Archived from the original on February 13, 2010. Retrieved January 4, 2010. ^ ^ \"Using C for CGI Programming\". linuxjournal.com. March 1, 2005. Archived from the original on February 13, 2010. Retrieved January 4, 2010. \"Using C for CGI Programming\" Archived . Retrieved January 4, 2010 January 4, ^ Perkins, Luc (September 17, 2013). \"Web development in C: crazy? Or crazy like a fox?\". Medium. Archived from the original on October 4, 2014. Retrieved April 8, 2022. ^ ^ Perkins, Luc (September 17, 2013). \"Web development in C: crazy? Or crazy like a fox?\". Medium. Archived from the original on October 4, 2014. Retrieved April 8, 2022. \"Web development in C: crazy? Or crazy like a fox?\" Archived . Retrieved April 8, 2022 April 8, ^ Munoz, Daniel. \"After All These Years, the World is Still Powered by C Programming\". Toptal Engineering Blog. Retrieved November 17, 2023. ^ ^ Munoz, Daniel. \"After All These Years, the World is Still Powered by C Programming\". Toptal Engineering Blog. Retrieved November 17, 2023. \"After All These Years, the World is Still Powered by C Programming\" . Retrieved November 17, 2023 November 17, ^ Metz, Cade. \"Dennis Ritchie: The Shoulders Steve Jobs Stood On\". Wired. Archived from the original on April 12, 2022. Retrieved April 19, 2022. ^ ^ Metz, Cade. \"Dennis Ritchie: The Shoulders Steve Jobs Stood On\". Wired. Archived from the original on April 12, 2022. Retrieved April 19, 2022. \"Dennis Ritchie: The Shoulders Steve Jobs Stood On\" Archived . Retrieved April 19, 2022 April 19, ^ corob-msft (March 31, 2022). \"Pragma directives and the __pragma and _Pragma keywords\". Microsoft Learn. Archived from the original on September 24, 2022. Retrieved September 24, 2022. ^ ^ corob-msft (March 31, 2022). \"Pragma directives and the __pragma and _Pragma keywords\". Microsoft Learn. Archived from the original on September 24, 2022. Retrieved September 24, 2022. \"Pragma directives and the __pragma and _Pragma keywords\" Archived . Retrieved September 24, 2022 September 24, ^ \"Pragmas (The C Preprocessor)\". GCC, the GNU Compiler Collection. Archived from the original on June 17, 2002. Retrieved September 24, 2022. ^ ^ \"Pragmas (The C Preprocessor)\". GCC, the GNU Compiler Collection. Archived from the original on June 17, 2002. Retrieved September 24, 2022. \"Pragmas (The C Preprocessor)\" Archived . Retrieved September 24, 2022 September 24, ^ \"Pragmas\". Intel C++ Compiler Classic Developer Guide and Reference. Intel. Archived from the original on April 10, 2022. Retrieved April 10, 2022. ^ ^ \"Pragmas\". Intel C++ Compiler Classic Developer Guide and Reference. Intel. Archived from the original on April 10, 2022. Retrieved April 10, 2022. \"Pragmas\" Archived . Retrieved April 10, 2022 April 10, ^ \"In praise of the C preprocessor\". apenwarr. August 13, 2007. Retrieved July 9, 2023. ^ ^ \"In praise of the C preprocessor\". apenwarr. August 13, 2007. Retrieved July 9, 2023. \"In praise of the C preprocessor\" . Retrieved July 9, 2023 July 9, ^ Roberts, Eric S. (March 21, 1989). \"Implementing Exceptions in C\" (PDF). DEC Systems Research Center. SRC-RR-40. Archived (PDF) from the original on January 15, 2017. Retrieved January 4, 2022. ^ ^ Roberts, Eric S. (March 21, 1989). \"Implementing Exceptions in C\" (PDF). DEC Systems Research Center. SRC-RR-40. Archived (PDF) from the original on January 15, 2017. Retrieved January 4, 2022. \"Implementing Exceptions in C\" (PDF) DEC Systems Research Center Archived (PDF) . Retrieved January 4, 2022 January 4, ^ McMillan, Robert (August 1, 2013). \"Is Java Losing Its Mojo?\". Wired. Archived from the original on February 15, 2017. Retrieved March 5, 2017. ^ ^ McMillan, Robert (August 1, 2013). \"Is Java Losing Its Mojo?\". Wired. Archived from the original on February 15, 2017. Retrieved March 5, 2017. \"Is Java Losing Its Mojo?\" Wired Archived . Retrieved March 5, 2017 March 5, ^ O'Regan, Gerard (September 24, 2015). Pillars of computing : a compendium of select, pivotal technology firms. Springer. ISBN 978-3319214641. OCLC 922324121. ^ ^ O'Regan, Gerard (September 24, 2015). Pillars of computing : a compendium of select, pivotal technology firms. Springer. ISBN 978-3319214641. OCLC 922324121. ISBN 978-3319214641 OCLC 922324121 ^ Rauchwerger, Lawrence (2004). Languages and compilers for parallel computing : 16th international workshop, LCPC 2003, College Station, TX, USA, October 2\u20134, 2003 : revised papers. Springer. ISBN 978-3540246442. OCLC 57965544. ^ ^ Rauchwerger, Lawrence (2004). Languages and compilers for parallel computing : 16th international workshop, LCPC 2003, College Station, TX, USA, October 2\u20134, 2003 : revised papers. Springer. ISBN 978-3540246442. OCLC 57965544. ISBN 978-3540246442 OCLC 57965544 ^ Stroustrup, Bjarne (1993). \"A History of C++: 1979\u20131991\" (PDF). Archived (PDF) from the original on February 2, 2019. Retrieved June 9, 2011. ^ ^ Stroustrup, Bjarne (1993). \"A History of C++: 1979\u20131991\" (PDF). Archived (PDF) from the original on February 2, 2019. Retrieved June 9, 2011. Stroustrup, Bjarne \"A History of C++: 1979\u20131991\" (PDF) Archived (PDF) . Retrieved June 9, 2011 June 9, Sources [edit] [ edit edit ] Ritchie, Dennis M. (March 1993). \"The Development of the C Language\". ACM SIGPLAN Notices. 28 (3). ACM: 201\u2013208. doi:10.1145/155360.155580. By courtesy of the author, also at Ritchie, Dennis M. \"Chistory\". www.bell-labs.com. Retrieved March 29, 2022. Ritchie, Dennis M. \"The Development of the C Language\" 201\u2013 doi 10.1145/155360.155580 10.1145/155360.155580 By courtesy of the author, also at Ritchie, Dennis M. \"Chistory\". www.bell-labs.com. Retrieved March 29, 2022. \"Chistory\" . Retrieved March 29, 2022 March 29, Ritchie, Dennis M. (1993). \"The Development of the C Language\". The Second ACM SIGPLAN Conference on History of Programming Languages (HOPL-II). ACM. pp. 201\u2013208. doi:10.1145/154766.155580. ISBN 0-89791-570-4. Archived from the original on April 11, 2019. Retrieved November 4, 2014. \"The Development of the C Language\" ACM 201\u2013 doi 10.1145/154766.155580 ISBN 0-89791-570-4 the original . Retrieved November 4, 2014 November 4, Kernighan, Brian W.; Ritchie, Dennis M. (1988). The C Programming Language (2nd ed.). Prentice Hall. ISBN 0-13-110362-8. Kernighan, Brian W. Ritchie, Dennis M. Prentice Hall ISBN 0-13-110362-8 Further reading [edit] [ edit edit ] Plauger, P.J. (1992). The Standard C Library (1 ed.). Prentice Hall. ISBN 978-0131315099. (source) Plauger, P.J. ISBN 978-0131315099 (source) Banahan, M.; Brady, D.; Doran, M. (1991). The C Book: Featuring the ANSI C Standard (2 ed.). Addison-Wesley. ISBN 978-0201544336. (free) ISBN 978-0201544336 (free) Feuer, Alan R. (1985). The C Puzzle Book (1 ed.). Prentice Hall. ISBN 0131099345. ISBN 0131099345 Harbison, Samuel; Steele, Guy Jr. (2002). C: A Reference Manual (5 ed.). Pearson. ISBN 978-0130895929. (archive) ISBN 978-0130895929 (archive) King, K.N. (2008). C Programming: A Modern Approach (2 ed.). W. W. Norton. ISBN 978-0393979503. (archive) ISBN 978-0393979503 (archive) Griffiths, David; Griffiths, Dawn (2012). Head First C (1 ed.). O'Reilly. ISBN 978-1449399917. ISBN 978-1449399917 Perry, Greg; Miller, Dean (2013). C Programming: Absolute Beginner's Guide (3 ed.). Que. ISBN 978-0789751980. ISBN 978-0789751980 Deitel, Paul; Deitel, Harvey (2015). C: How to Program (8 ed.). Pearson. ISBN 978-0133976892. ISBN 978-0133976892 Gustedt, Jens (2019). Modern C (2 ed.). Manning. ISBN 978-1617295812. (free) ISBN 978-1617295812 (free) External links [edit] [ edit edit ] sister projects sister projects Media from Commons Media from Commons Media News from Wikinews News from Wikinews News Quotations from Wikiquote Quotations from Wikiquote Quotations Textbooks from Wikibooks Textbooks from Wikibooks Textbooks Resources from Wikiversity Resources from Wikiversity Resources ISO C Working Group official website ISO/IEC 9899, publicly available official C documents, including the C99 Rationale \"C99 with Technical corrigenda TC1, TC2, and TC3 included\" (PDF). Archived (PDF) from the original on October 25, 2007. (3.61 MB) ISO C Working Group official website ISO/IEC 9899, publicly available official C documents, including the C99 Rationale ISO/IEC 9899 \"C99 with Technical corrigenda TC1, TC2, and TC3 included\" (PDF). Archived (PDF) from the original on October 25, 2007. (3.61 MB) \"C99 with Technical corrigenda TC1, TC2, and TC3 included\" (PDF) Archived (PDF) (3.61 MB) comp.lang.c Frequently Asked Questions comp.lang.c Frequently Asked Questions A History of C, by Dennis Ritchie A History of C C Library Reference and Examples C Library Reference and Examples v v t t e e C programming language ANSI C ANSI C C99 C99 C11 C11 C17 C17 C23 C23 Embedded C Embedded C MISRA C MISRA C Functions Functions Header files Header files Operators Operators String String Syntax Syntax Preprocessor Preprocessor Data types Data types Standard library Char Char File I/O File I/O Math Math Dynamic memory Dynamic memory String String Time Time Variadic Variadic POSIX POSIX Bionic libhybris Bionic libhybris libhybris dietlibc dietlibc glibc EGLIBC glibc EGLIBC EGLIBC klibc klibc Windows CRT Windows CRT musl musl Newlib Newlib uClibc uClibc Compilers ACK ACK Borland Turbo C Borland Turbo C Clang Clang GCC GCC ICC ICC LCC LCC Norcroft C Norcroft C PCC PCC SDCC SDCC TCC TCC Visual Studio, Express, C++ Visual Studio Express C++ Watcom C/C++ Watcom C/C++ IDEs Anjuta Anjuta CLion CLion Code::Blocks Code::Blocks CodeLite CodeLite Eclipse Eclipse Geany Geany GNOME Builder GNOME Builder KDevelop KDevelop Visual Studio Visual Studio NetBeans NetBeans Compatibility of C and C++ Compatibility of C and C++ Comparison with Pascal Comparison with Pascal C++ C++ C# C# D D Objective-C Objective-C Alef Alef Limbo Limbo Go Go Vala Vala Dennis Ritchie Dennis Ritchie Category Category v v t t e e Integrated development environments C C++ Open source Arduino IDE Arduino IDE Code::Blocks Code::Blocks CodeLite CodeLite Dev-C++ Dev-C++ Eclipse Eclipse Emacs Emacs Geany Geany GNOME Builder GNOME Builder Kakoune Kakoune KDevelop KDevelop NetBeans NetBeans QDevelop QDevelop Qt Creator Qt Creator TheIDE TheIDE Vi\u2013Vim Vi Vim OpenWatcom OpenWatcom Freeware DevEco Studio DevEco Studio Oracle Developer Studio Oracle Developer Studio Visual Studio Code Visual Studio Code Visual Studio Community Visual Studio Community Xcode Xcode Retail C++Builder C++Builder Eclipse-based CodeWarrior MyEclipse Eclipse CodeWarrior CodeWarrior MyEclipse MyEclipse Visual Studio Visual Studio By JetBrains IntelliJ IDEA AppCode CLion JetBrains IntelliJ IDEA IntelliJ IDEA AppCode AppCode CLion CLion LabWindows/CVI LabWindows/CVI IBM Rational Software Architect Rational Software Architect Understand Understand SlickEdit SlickEdit Anjuta Anjuta VisualAge VisualAge Visual C++ Express Visual C++ Express MonoDevelop MonoDevelop SharpDevelop SharpDevelop Borland Turbo C, C++ Borland Turbo C C++ QuickC QuickC Java Open source Anjuta Anjuta BlueJ BlueJ DrJava DrJava Eclipse Eclipse Geany Geany Greenfoot Greenfoot IntelliJ IDEA Community Edition Android Studio IntelliJ IDEA Android Studio Android Studio NetBeans NetBeans DevEco Studio DevEco Studio Freeware jGRASP jGRASP JDeveloper JDeveloper Retail JCreator MyEclipse MyEclipse JetBrains IntelliJ IDEA JetBrains IntelliJ IDEA SlickEdit SlickEdit Understand Understand Metrowerks CodeWarrior Pro for Java CodeWarrior JBuilder JBuilder Sun Java Studio Creator (superseded by NetBeans) Sun Java Studio Creator NetBeans VisualAge (superseded by Eclipse) VisualAge Eclipse Visual Caf\u00e9 (aka Espresso, superseded by JBuilder) Visual Caf\u00e9 JBuilder Visual J++ Visual J++ Xelfi (became NetBeans) Xelfi NetBeans JavaScript Open source Vim Vim Visual Studio Code Visual Studio Code Atom Atom Chromium Chromium DevEco Studio DevEco Studio CLI .NET Open source Visual Studio Code Visual Studio Code PascalABC.NET PascalABC.NET Freeware Visual Studio Community Visual Studio Community Retail Visual Studio Visual Studio Rider Rider Understand Understand Xamarin Studio Xamarin Studio MonoDevelop MonoDevelop SharpDevelop SharpDevelop Visual Basic Express Visual Basic Express Visual Web Developer Express Visual Web Developer Express Visual J# Express Visual J# Express Visual Studio Express for Windows Phone Visual Studio Express for Windows Phone Visual C++ Express Visual C++ Express Visual C# Express Visual C# Express Express for Desktop Express for Desktop Express for Web Express for Web Express for Windows Express for Windows Flash Adobe Flash Builder Adobe Flash Builder FlashDevelop Powerflasher FDT Powerflasher FDT PHP Open source Aptana Aptana NetBeans NetBeans Komodo Edit Komodo Edit Komodo IDE Komodo IDE KDevelop KDevelop Proprietary Codelobster Codelobster PhpStorm PhpStorm PHPEdit PHPEdit SlickEdit SlickEdit Zend Studio Zend Studio R RStudio RStudio R Tools for Visual Studio R Tools for Visual Studio Python Open source Anjuta Anjuta IDLE IDLE Eric Eric Light Table Light Table Ninja-IDE Ninja-IDE PyDev PyDev PyScripter PIDA Spyder Spyder Komodo Edit Komodo Edit Komodo IDE Komodo IDE KDevelop KDevelop Vim Vim Visual Studio Code Visual Studio Code Thonny Thonny DevEco Studio DevEco Studio Proprietary PyCharm PyCharm PythonAnywhere PythonAnywhere Visual Studio Code Visual Studio Code SlickEdit SlickEdit Wing IDE Wing IDE Pascal Object Pascal Open source Free Pascal IDE Free Pascal Lazarus Lazarus Dev-Pascal Dev-Pascal GNAVI PascalABC.NET PascalABC.NET Freeware Delphi Community Delphi Community Retail Delphi Delphi Turbo Pascal Turbo Pascal Virtual Pascal Virtual Pascal Borland Kylix Borland Kylix QuickPascal QuickPascal BASIC Open source Basic-256 Basic-256 Microsoft Small Basic Microsoft Small Basic SdlBasic SdlBasic Gambas Gambas Basic4GL Basic4GL VisualFBEditor / WinFBE VisualFBEditor / WinFBE InForm InForm Freeware FutureBASIC FutureBASIC RapidQ RapidQ Visual Studio Community Visual Studio Community Retail Visual Studio Visual Studio NS Basic NS Basic PureBasic PureBasic GLBasic GLBasic Liberty BASIC Liberty BASIC Xojo Xojo CA-Realizer CA-Realizer MonoDevelop MonoDevelop QuickBASIC QuickBASIC QBasic QBasic SharpDevelop SharpDevelop Visual Basic Visual Basic Visual Basic Express Visual Basic Express Go Open source Vim Vim Visual Studio Code (VSCode) Visual Studio Code Eclipse Eclipse Freeware Cloud9 IDE Cloud9 IDE Retail GoLand GoLand Haxe IntelliJ IDEA IntelliJ IDEA VS Code VS Code Sublime Text Sublime Text Powerflasher FDT Powerflasher FDT POP-11 Poplog Poplog Online AWS Cloud9 IDE Cloud9 IDE Eclipse Che Eclipse Che SourceLair SourceLair Comparison Comparison Category Category v v t t e e Programming languages Comparison Comparison Timeline Timeline History History Ada Ada ALGOL Simula ALGOL Simula Simula APL APL Assembly Assembly BASIC Visual Basic classic .NET BASIC Visual Basic classic .NET Visual Basic classic classic .NET .NET C C C++ C++ C# C# COBOL COBOL Erlang Erlang Forth Forth Fortran Fortran Go Go Haskell Haskell Java Java JavaScript JavaScript Julia Julia Kotlin Kotlin Lisp Lisp Lua Lua MATLAB MATLAB ML ML Pascal Object Pascal Pascal Object Pascal Object Pascal Perl Perl PHP PHP Prolog Prolog Python Python R R Ruby Ruby Rust Rust SQL SQL Scratch Scratch Shell Shell Smalltalk Smalltalk Swift Swift more... more... Lists: Alphabetical Alphabetical Categorical Categorical Generational Generational Non-English-based Non-English-based Category Category Authority control databases FAST FAST FAST Germany Germany Germany United States United States United States France France France BnF data BnF data BnF data Czech Republic Czech Republic Czech Republic Czech Republic Israel Israel Israel IdRef IdRef IdRef https://en.wikipedia.org/w/index.php?title=C_(programming_language)&oldid=1266176654 Categories C (programming language) C (programming language) American inventions American inventions C programming language family C programming language family Cross-platform software Cross-platform software High-level programming languages High-level programming languages Procedural programming languages Procedural programming languages Structured programming languages Structured programming languages Programming languages created in 1972 Programming languages created in 1972 Programming languages with an ISO standard Programming languages with an ISO standard Statically typed programming languages Statically typed programming languages Systems programming languages Systems programming languages Wikipedia articles needing clarification from November 2023 Wikipedia articles needing clarification from November 2023 Webarchive template wayback links Webarchive template wayback links Articles with short description Articles with short description Short description matches Wikidata Short description matches Wikidata Wikipedia pending changes protected pages Wikipedia pending changes protected pages Use mdy dates from October 2024 Use mdy dates from October 2024 All articles with vague or ambiguous time All articles with vague or ambiguous time Vague or ambiguous time from August 2022 Vague or ambiguous time from August 2022 Wikipedia articles in need of updating from February 2021 Wikipedia articles in need of updating from February 2021 All Wikipedia articles in need of updating All Wikipedia articles in need of updating Wikipedia articles in need of updating from October 2024 Wikipedia articles in need of updating from October 2024 Articles needing additional references from October 2012 Articles needing additional references from October 2012 All articles needing additional references All articles needing additional references Wikipedia articles needing clarification from October 2021 Wikipedia articles needing clarification from October 2021 Articles needing additional references from July 2014 Articles needing additional references from July 2014 Pages using Sister project links with default search Pages using Sister project links with default search Pages using Sister project links with wikidata mismatch Pages using Sister project links with wikidata mismatch Pages using Sister project links with hidden wikidata Pages using Sister project links with hidden wikidata Articles with example C code Articles with example C code This page was last edited on 30 December 2024, at 12:32 (UTC). (UTC) Text is available under the Creative Commons Attribution-ShareAlike 4.0 License; additional terms may apply. By using this site, you agree to the Terms of Use and Privacy Policy. Wikipedia\u00ae is a registered trademark of the Wikimedia Foundation, Inc., a non-profit organization. Creative Commons Attribution-ShareAlike 4.0 License Terms of Use Privacy Policy Wikimedia Foundation, Inc. Privacy policy Privacy policy About Wikipedia About Wikipedia Disclaimers Disclaimers Contact Wikipedia Contact Wikipedia Code of Conduct Code of Conduct Developers Developers Statistics Statistics Cookie statement Cookie statement Mobile view Mobile view",
    "https://en.wikipedia.org/wiki/Ruby_(programming_language)": "Jump to content Main menu Main page Main page Main page Contents Contents Contents Current events Current events Current events Random article Random article Random article About Wikipedia About Wikipedia About Wikipedia Contact us Contact us Contact us Help Help Help Learn to edit Learn to edit Learn to edit Community portal Community portal Community portal Recent changes Recent changes Recent changes Upload file Upload file Upload file Search Search Appearance Donate Donate Donate Create account Create account Create account Log in Log in Log in Personal tools Donate Donate Donate Create account Create account Create account Log in Log in Log in learn more learn more Contributions Contributions Contributions Talk Talk Talk Contents (Top) (Top) 1 History Toggle History subsection 1.1 Early concept 1.2 Early releases 1.3 Ruby 1.8 and 1.9 1.4 Ruby 2 1.5 Ruby 3 1 History 1 History Toggle History subsection 1.1 Early concept 1.1 Early concept 1.1 Early concept 1.2 Early releases 1.2 Early releases 1.2 Early releases 1.3 Ruby 1.8 and 1.9 1.3 Ruby 1.8 and 1.9 1.3 Ruby 1.8 and 1.9 1.4 Ruby 2 1.4 Ruby 2 1.4 Ruby 2 1.5 Ruby 3 1.5 Ruby 3 1.5 Ruby 3 2 Semantics and philosophy 2 Semantics and philosophy 2 Semantics and philosophy 3 Features 3 Features 3 Features 4 Syntax 4 Syntax 4 Syntax 5 Implementations Toggle Implementations subsection 5.1 Matz's Ruby interpreter 5.2 Alternative implementations 5.3 Platform support 5 Implementations 5 Implementations Toggle Implementations subsection 5.1 Matz's Ruby interpreter 5.1 Matz's Ruby interpreter 5.1 Matz's Ruby interpreter 5.2 Alternative implementations 5.2 Alternative implementations 5.2 Alternative implementations 5.3 Platform support 5.3 Platform support 5.3 Platform support 6 Repositories and libraries 6 Repositories and libraries 6 Repositories and libraries 7 See also 7 See also 7 See also 8 References 8 References 8 References 9 Further reading 9 Further reading 9 Further reading 10 External links 10 External links 10 External links Toggle the table of contents Ruby (programming language) Ruby (programming language) 74 languages \u0627\u0644\u0639\u0631\u0628\u064a\u0629 \u0627\u0644\u0639\u0631\u0628\u064a\u0629 \u0627\u0644\u0639\u0631\u0628\u064a\u0629 Aragon\u00e9s Aragon\u00e9s Aragon\u00e9s Asturianu Asturianu Asturianu Az\u0259rbaycanca Az\u0259rbaycanca Az\u0259rbaycanca \u062a\u06c6\u0631\u06a9\u062c\u0647 \u062a\u06c6\u0631\u06a9\u062c\u0647 \u062a\u06c6\u0631\u06a9\u062c\u0647 \u09ac\u09be\u0982\u09b2\u09be \u09ac\u09be\u0982\u09b2\u09be \u09ac\u09be\u0982\u09b2\u09be \u0411\u0435\u043b\u0430\u0440\u0443\u0441\u043a\u0430\u044f \u0411\u0435\u043b\u0430\u0440\u0443\u0441\u043a\u0430\u044f \u0411\u0435\u043b\u0430\u0440\u0443\u0441\u043a\u0430\u044f \u0411\u044a\u043b\u0433\u0430\u0440\u0441\u043a\u0438 \u0411\u044a\u043b\u0433\u0430\u0440\u0441\u043a\u0438 \u0411\u044a\u043b\u0433\u0430\u0440\u0441\u043a\u0438 Bosanski Bosanski Bosanski Catal\u00e0 Catal\u00e0 Catal\u00e0 \u010ce\u0161tina \u010ce\u0161tina \u010ce\u0161tina Dansk Dansk Dansk Deutsch Deutsch Deutsch Eesti Eesti Eesti \u0395\u03bb\u03bb\u03b7\u03bd\u03b9\u03ba\u03ac \u0395\u03bb\u03bb\u03b7\u03bd\u03b9\u03ba\u03ac \u0395\u03bb\u03bb\u03b7\u03bd\u03b9\u03ba\u03ac Espa\u00f1ol Espa\u00f1ol Espa\u00f1ol Esperanto Esperanto Esperanto Euskara Euskara Euskara \u0641\u0627\u0631\u0633\u06cc \u0641\u0627\u0631\u0633\u06cc \u0641\u0627\u0631\u0633\u06cc Fran\u00e7ais Fran\u00e7ais Fran\u00e7ais Galego Galego Galego \ud55c\uad6d\uc5b4 \ud55c\uad6d\uc5b4 \ud55c\uad6d\uc5b4 Hausa Hausa Hausa \u0540\u0561\u0575\u0565\u0580\u0565\u0576 \u0540\u0561\u0575\u0565\u0580\u0565\u0576 \u0540\u0561\u0575\u0565\u0580\u0565\u0576 Hrvatski Hrvatski Hrvatski Ido Ido Ido Bahasa Indonesia Bahasa Indonesia Bahasa Indonesia Interlingua Interlingua Interlingua Italiano Italiano Italiano \u05e2\u05d1\u05e8\u05d9\u05ea \u05e2\u05d1\u05e8\u05d9\u05ea \u05e2\u05d1\u05e8\u05d9\u05ea \u10e5\u10d0\u10e0\u10d7\u10e3\u10da\u10d8 \u10e5\u10d0\u10e0\u10d7\u10e3\u10da\u10d8 \u10e5\u10d0\u10e0\u10d7\u10e3\u10da\u10d8 Kiswahili Kiswahili Kiswahili \u041a\u044b\u0440\u0433\u044b\u0437\u0447\u0430 \u041a\u044b\u0440\u0433\u044b\u0437\u0447\u0430 \u041a\u044b\u0440\u0433\u044b\u0437\u0447\u0430 Latvie\u0161u Latvie\u0161u Latvie\u0161u Lietuvi\u0173 Lietuvi\u0173 Lietuvi\u0173 Lombard Lombard Lombard Magyar Magyar Magyar \u0d2e\u0d32\u0d2f\u0d3e\u0d33\u0d02 \u0d2e\u0d32\u0d2f\u0d3e\u0d33\u0d02 \u0d2e\u0d32\u0d2f\u0d3e\u0d33\u0d02 \u092e\u0930\u093e\u0920\u0940 \u092e\u0930\u093e\u0920\u0940 \u092e\u0930\u093e\u0920\u0940 Bahasa Melayu Bahasa Melayu Bahasa Melayu \u1019\u103c\u1014\u103a\u1019\u102c\u1018\u102c\u101e\u102c \u1019\u103c\u1014\u103a\u1019\u102c\u1018\u102c\u101e\u102c \u1019\u103c\u1014\u103a\u1019\u102c\u1018\u102c\u101e\u102c Nederlands Nederlands Nederlands \u65e5\u672c\u8a9e \u65e5\u672c\u8a9e \u65e5\u672c\u8a9e Norsk bokm\u00e5l Norsk bokm\u00e5l Norsk bokm\u00e5l Norsk nynorsk Norsk nynorsk Norsk nynorsk \u0b13\u0b21\u0b3c\u0b3f\u0b06 \u0b13\u0b21\u0b3c\u0b3f\u0b06 \u0b13\u0b21\u0b3c\u0b3f\u0b06 O\u02bbzbekcha / \u045e\u0437\u0431\u0435\u043a\u0447\u0430 O\u02bbzbekcha / \u045e\u0437\u0431\u0435\u043a\u0447\u0430 O\u02bbzbekcha / \u045e\u0437\u0431\u0435\u043a\u0447\u0430 \u0a2a\u0a70\u0a1c\u0a3e\u0a2c\u0a40 \u0a2a\u0a70\u0a1c\u0a3e\u0a2c\u0a40 \u0a2a\u0a70\u0a1c\u0a3e\u0a2c\u0a40 \u067e\u0646\u062c\u0627\u0628\u06cc \u067e\u0646\u062c\u0627\u0628\u06cc \u067e\u0646\u062c\u0627\u0628\u06cc Polski Polski Polski Portugu\u00eas Portugu\u00eas Portugu\u00eas Rom\u00e2n\u0103 Rom\u00e2n\u0103 Rom\u00e2n\u0103 \u0420\u0443\u0441\u0441\u043a\u0438\u0439 \u0420\u0443\u0441\u0441\u043a\u0438\u0439 \u0420\u0443\u0441\u0441\u043a\u0438\u0439 Shqip Shqip Shqip Simple English Simple English Simple English Sloven\u010dina Sloven\u010dina Sloven\u010dina \u06a9\u0648\u0631\u062f\u06cc \u06a9\u0648\u0631\u062f\u06cc \u06a9\u0648\u0631\u062f\u06cc \u0421\u0440\u043f\u0441\u043a\u0438 / srpski \u0421\u0440\u043f\u0441\u043a\u0438 / srpski \u0421\u0440\u043f\u0441\u043a\u0438 / srpski Srpskohrvatski / \u0441\u0440\u043f\u0441\u043a\u043e\u0445\u0440\u0432\u0430\u0442\u0441\u043a\u0438 Srpskohrvatski / \u0441\u0440\u043f\u0441\u043a\u043e\u0445\u0440\u0432\u0430\u0442\u0441\u043a\u0438 Srpskohrvatski / \u0441\u0440\u043f\u0441\u043a\u043e\u0445\u0440\u0432\u0430\u0442\u0441\u043a\u0438 Suomi Suomi Suomi Svenska Svenska Svenska \u0ba4\u0bae\u0bbf\u0bb4\u0bcd \u0ba4\u0bae\u0bbf\u0bb4\u0bcd \u0ba4\u0bae\u0bbf\u0bb4\u0bcd \u0422\u0430\u0442\u0430\u0440\u0447\u0430 / tatar\u00e7a \u0422\u0430\u0442\u0430\u0440\u0447\u0430 / tatar\u00e7a \u0422\u0430\u0442\u0430\u0440\u0447\u0430 / tatar\u00e7a \u0c24\u0c46\u0c32\u0c41\u0c17\u0c41 \u0c24\u0c46\u0c32\u0c41\u0c17\u0c41 \u0c24\u0c46\u0c32\u0c41\u0c17\u0c41 \u0e44\u0e17\u0e22 \u0e44\u0e17\u0e22 \u0e44\u0e17\u0e22 \u0422\u043e\u04b7\u0438\u043a\u04e3 \u0422\u043e\u04b7\u0438\u043a\u04e3 \u0422\u043e\u04b7\u0438\u043a\u04e3 T\u00fcrk\u00e7e T\u00fcrk\u00e7e T\u00fcrk\u00e7e \u0423\u043a\u0440\u0430\u0457\u043d\u0441\u044c\u043a\u0430 \u0423\u043a\u0440\u0430\u0457\u043d\u0441\u044c\u043a\u0430 \u0423\u043a\u0440\u0430\u0457\u043d\u0441\u044c\u043a\u0430 \u0627\u0631\u062f\u0648 \u0627\u0631\u062f\u0648 \u0627\u0631\u062f\u0648 Ti\u1ebfng Vi\u1ec7t Ti\u1ebfng Vi\u1ec7t Ti\u1ebfng Vi\u1ec7t \u5434\u8bed \u5434\u8bed \u5434\u8bed \u7cb5\u8a9e \u7cb5\u8a9e \u7cb5\u8a9e \u017demait\u0117\u0161ka \u017demait\u0117\u0161ka \u017demait\u0117\u0161ka \u4e2d\u6587 \u4e2d\u6587 \u4e2d\u6587 Edit links Edit links Article Article Article Talk Talk Talk English Read Read Read Edit Edit Edit View history View history View history Tools Read Read Read Edit Edit Edit View history View history View history What links here What links here What links here Related changes Related changes Related changes Upload file Upload file Upload file Special pages Special pages Special pages Permanent link Permanent link Permanent link Page information Page information Page information Cite this page Cite this page Cite this page Get shortened URL Get shortened URL Get shortened URL Download QR code Download QR code Download QR code Download as PDF Download as PDF Download as PDF Printable version Printable version Printable version Wikimedia Commons Wikimedia Commons Wikimedia Commons Wikibooks Wikibooks Wikibooks Wikiquote Wikiquote Wikiquote Wikiversity Wikiversity Wikiversity Wikidata item Wikidata item Wikidata item Ruby on Rails Please help improve it to make it understandable to non-experts, without removing the technical details. help improve it make it understandable to non-experts (June 2022) June 2022 (Learn how and when to remove this message) Learn how and when to remove this message Paradigm Multi-paradigm functional imperative object-oriented reflective Designed by Yukihiro Matsumoto Developer ; 30 years ago (1995) 1995 Stable release [1] [ ] ; 7 days ago (25 December 2024) 25 December 2024 Typing discipline Duck dynamic strong Scope C OS Cross-platform License Ruby License Filename extensions ruby-lang.org ruby-lang.org implementations Ruby MRI TruffleRuby YARV Rubinius JRuby RubyMotion mruby Ada [2] [ ] Basic [3] [ ] C++ [2] [ ] CLU [4] [ ] Dylan [4] [ ] Eiffel [2] [ ] Lisp [4] [ ] Lua Perl [4] [ ] Python [4] [ ] Smalltalk [4] [ ] Clojure CoffeeScript Crystal D Elixir Groovy Julia [5] [ ] Mirah Nu [6] [ ] Ring [7] [ ] Rust [8] [ ] Swift [9] [ ] Ruby Programming at Wikibooks Ruby Programming Ruby is an interpreted, high-level, general-purpose programming language. It was designed with an emphasis on programming productivity and simplicity. In Ruby, everything is an object, including primitive data types. It was developed in the mid-1990s by Yukihiro \"Matz\" Matsumoto in Japan. interpreted high-level general-purpose programming language object primitive data types Yukihiro \"Matz\" Matsumoto Japan Ruby is dynamically typed and uses garbage collection and just-in-time compilation. It supports multiple programming paradigms, including procedural, object-oriented, and functional programming. According to the creator, Ruby was influenced by Perl, Smalltalk, Eiffel, Ada, BASIC, Java, and Lisp.[10][3] dynamically typed garbage collection just-in-time compilation procedural object-oriented functional programming Perl Smalltalk Eiffel Ada BASIC Java Lisp [10] [ ] [3] [ ] History [edit] [ edit edit ] History of Ruby Early concept [edit] [ edit edit ] Matsumoto has said that Ruby was conceived in 1993. In a 1999 post to the ruby-talk mailing list, he describes some of his early ideas about the language: I was talking with my colleague about the possibility of an object-oriented scripting language. I knew Perl (Perl4, not Perl5), but I didn't like it really, because it had the smell of a toy language (it still has). The object-oriented language seemed very promising. I knew Python then. But I didn't like it, because I didn't think it was a true object-oriented language \u2013 OO features appeared to be add-on to the language. As a language maniac and OO fan for 15 years, I really wanted a genuine object-oriented, easy-to-use scripting language. I looked for but couldn't find one. So I decided to make it. toy language Python Matsumoto describes the design of Ruby as being like a simple Lisp language at its core, with an object system like that of Smalltalk, blocks inspired by higher-order functions, and practical utility like that of Perl.[11] Lisp higher-order functions [11] [ ] The name \"Ruby\" originated during an online chat session between Matsumoto and Keiju Ishitsuka on February 24, 1993, before any code had been written for the language.[12] Initially two names were proposed: \"Coral\" and \"Ruby\". Matsumoto chose the latter in a later e-mail to Ishitsuka.[13] Matsumoto later noted a factor in choosing the name \"Ruby\"\u2013it was the birthstone of one of his colleagues.[14][15] [12] [ ] Coral Ruby [13] [ ] birthstone [14] [ ] [15] [ ] Early releases [edit] [ edit edit ] The first public release of Ruby 0.95 was announced on Japanese domestic newsgroups on December 21, 1995.[16][17] Subsequently, three more versions of Ruby were released in two days.[12] The release coincided with the launch of the Japanese-language ruby-list mailing list, which was the first mailing list for the new language. newsgroups [16] [ ] [17] [ ] [12] [ ] Japanese-language Already present at this stage of development were many of the features familiar in later releases of Ruby, including object-oriented design, classes with inheritance, mixins, iterators, closures, exception handling and garbage collection.[18] object-oriented classes mixins iterators closures exception handling garbage collection [18] [ ] After the release of Ruby 0.95 in 1995, several stable versions of Ruby were released in these years: Ruby 1.0: December 25, 1996[12] [12] [ ] Ruby 1.2: December 1998 Ruby 1.4: August 1999 Ruby 1.6: September 2000 In 1997, the first article about Ruby was published on the Web. In the same year, Matsumoto was hired by netlab.jp to work on Ruby as a full-time developer.[12] [12] [ ] In 1998, the Ruby Application Archive was launched by Matsumoto, along with a simple English-language homepage for Ruby.[12] [12] [ ] In 1999, the first English language mailing list ruby-talk began, which signaled a growing interest in the language outside Japan.[19] In this same year, Matsumoto and Keiju Ishitsuka wrote the first book on Ruby, The Object-oriented Scripting Language Ruby (\u30aa\u30d6\u30b8\u30a7\u30af\u30c8\u6307\u5411\u30b9\u30af\u30ea\u30d7\u30c8\u8a00\u8a9e Ruby), which was published in Japan in October 1999. It would be followed in the early 2000s by around 20 books on Ruby published in Japanese.[12] [19] [ ] [12] [ ] By 2000, Ruby was more popular than Python in Japan.[20] In September 2000, the first English language book Programming Ruby was printed, which was later freely released to the public, further widening the adoption of Ruby amongst English speakers. In early 2002, the English-language ruby-talk mailing list was receiving more messages than the Japanese-language ruby-list, demonstrating Ruby's increasing popularity in the non-Japanese speaking world. [20] [ ] Programming Ruby Ruby 1.8 and 1.9 [edit] [ edit edit ] Ruby 1.8 was initially released August 2003, was stable for a long time, and was retired June 2013.[21] Although deprecated, there is still code based on it. Ruby 1.8 is only partially compatible with Ruby 1.9. [21] [ ] Ruby 1.8 has been the subject of several industry standards. The language specifications for Ruby were developed by the Open Standards Promotion Center of the Information-Technology Promotion Agency (a Japanese government agency) for submission to the Japanese Industrial Standards Committee (JISC) and then to the International Organization for Standardization (ISO). It was accepted as a Japanese Industrial Standard (JIS X 3017) in 2011[22] and an international standard (ISO/IEC 30170) in 2012.[23][24] Japanese government Japanese Industrial Standards Committee International Organization for Standardization [22] [ ] [23] [ ] [24] [ ] Ruby on Rails Around 2005, interest in the Ruby language surged in tandem with Ruby on Rails, a web framework written in Ruby. Rails is frequently credited with increasing awareness of Ruby.[25] Ruby on Rails web framework [25] [ ] Effective with Ruby 1.9.3, released October 31, 2011,[26] Ruby switched from being dual-licensed under the Ruby License and the GPL to being dual-licensed under the Ruby License and the two-clause BSD license.[27] Adoption of 1.9 was slowed by changes from 1.8 that required many popular third party gems to be rewritten. Ruby 1.9 introduces many significant changes over the 1.8 series. Examples include:[28] [26] [ ] [27] [ ] gems [28] [ ] block local variables (variables that are local to the block in which they are declared) block local block an additional lambda syntax: f = ->(a,b) { puts a + b } lambda f = -> ( a , b ) { puts a + b } an additional Hash literal syntax using colons for symbol keys: {symbol_key: \"value\"} == {:symbol_key => \"value\"} Hash { symbol_key : \"value\" } == { :symbol_key => \"value\" } per-string character encodings are supported character encodings new socket API [IPv6 support] IPv6 require_relative import security Ruby 2 [edit] [ edit edit ] Ruby 2.0 was intended to be fully backward compatible with Ruby 1.9.3. As of the official 2.0.0 release on February 24, 2013, there were only five known (minor) incompatibilities.[29] Ruby 2.0 added several new features, including: [29] [ ] Method keyword arguments Method A new method, Module#prepend, to extend a class A new literal to create an array of symbols New API for lazy evaluation of Enumerables lazy evaluation A new convention of using #to_h to convert objects to Hashes[30] [30] [ ] Starting with 2.1.0, Ruby's versioning policy changed to be more similar to semantic versioning.[31] semantic versioning [31] [ ] Ruby 2.2.0 includes speed-ups, bugfixes, and library updates and removes some deprecated APIs. Most notably, Ruby 2.2.0 introduces changes to memory handling \u2013 an incremental garbage collector, support for garbage collection of symbols and the option to compile directly against jemalloc. It also contains experimental support for using vfork(2) with system() and spawn(), and added support for the Unicode 7.0 specification. Since version 2.2.1,[32] Ruby MRI performance on PowerPC64 was improved.[33][34][35] Features that were made obsolete or removed include callcc, the DL library, Digest::HMAC, lib/rational.rb, lib/complex.rb, GServer, Logger::Application as well as various C API functions.[36] vfork Unicode [32] [ ] Ruby MRI PowerPC64 [33] [ ] [34] [ ] [35] [ ] [36] [ ] Ruby 2.3.0 includes many performance improvements, updates, and bugfixes including changes to Proc#call, Socket and IO use of exception keywords, Thread#name handling, default passive Net::FTP connections, and Rake being removed from stdlib.[37] Other notable changes include: [37] [ ] The ability to mark all string literals as frozen by default with a consequently large performance increase in string operations.[38] string literals [38] [ ] Hash comparison to allow direct checking of key/value pairs instead of just keys. A new safe navigation operator &. that can ease nil handling (e.g. instead of if obj && obj.foo && obj.foo.bar, we can use if obj&.foo&.bar). safe navigation operator if obj && obj . foo && obj . foo . bar The did_you_mean gem is now bundled by default and required on startup to automatically suggest similar name matches on a NameError or NoMethodError. Hash#dig and Array#dig to easily extract deeply nested values (e.g. given profile = { social: { wikipedia: { name: 'Foo Baz' } } }, the value Foo Baz can now be retrieved by profile.dig(:social, :wikipedia, :name)). profile = { social : { wikipedia : { name : 'Foo Baz' } } } .grep_v(regexp) which will match all negative examples of a given regular expression in addition to other new features. Ruby 2.4.0 includes performance improvements to hash table, Array#max, Array#min, and instance variable access.[39] Other notable changes include: [39] [ ] Binding#irb: Start a REPL session similar to binding.pry Unify Fixnum and Bignum into Integer class String supports Unicode case mappings, not just ASCII A new method, Regexp#match?, which is a faster Boolean version of Regexp#match Thread deadlock detection now shows threads with their backtrace and dependency A few notable changes in Ruby 2.5.0 include rescue and ensure statements automatically use a surrounding do-end block (less need for extra begin-end blocks), method-chaining with yield_self, support for branch coverage and method coverage measurement, and easier Hash transformations with Hash#slice and Hash#transform_keys On top of that come a lot of performance improvements like faster block passing (3 times faster), faster Mutexes, faster ERB templates and improvements on some concatenation methods. A few notable changes in Ruby 2.6.0 include an experimental just-in-time compiler (JIT), and RubyVM::AbstractSyntaxTree (experimental). just-in-time compiler A few notable changes in Ruby 2.7.0 include pattern Matching (experimental), REPL improvements, a compaction GC, and separation of positional and keyword arguments. Ruby 3 [edit] [ edit edit ] Ruby 3.0.0 was released on Christmas Day in 2020.[40] It is known as Ruby 3x3 which means that programs would run three times faster in Ruby 3.0 comparing to Ruby 2.0.[41] and some had already implemented in intermediate releases on the road from 2 to 3. To achieve 3x3, Ruby 3 comes with MJIT, and later YJIT, Just-In-Time Compilers, to make programs faster, although they are described as experimental and remain disabled by default (enabled by flags at runtime). Christmas [40] [ ] [41] [ ] Another goal of Ruby 3.0 is to improve concurrency and two more utilities Fibre Scheduler, and experimental Ractor facilitate the goal.[40] Ractor is light-weight and thread-safe as it is achieved by exchanging messages rather than shared objects. concurrency [40] [ ] Ruby 3.0 introduces RBS language to describe the types of Ruby programs for static analysis.[40] It is separated from general Ruby programs. static analysis [40] [ ] There are some syntax enhancements and library changes in Ruby 3.0 as well.[40] [40] [ ] Ruby 3.1 was released on December 25, 2021.[42] It includes YJIT, a new, experimental, Just-In-Time Compiler developed by Shopify, to enhance the performance of real world business applications. A new debugger is also included. There are some syntax enhancements and other improvements in this release. Network libraries for FTP, SMTP, IMAP, and POP are moved from default gems to bundled gems.[43] [42] [ ] Shopify debugger FTP SMTP IMAP POP [43] [ ] Ruby 3.2 was released on December 25, 2022.[44] It brings support for being run inside of a WebAssembly environment via a WASI interface. Regular expressions also receives some improvements, including a faster, memoized matching algorithm to protect against certain ReDoS attacks, and configurable timeouts for regular expression matching. Additional debugging and syntax features are also included in this release, which include syntax suggestion, as well as error highlighting. The MJIT compiler has been re-implemented as a standard library module, while the YJIT, a Rust-based JIT compiler now supports more architectures on Linux. [44] [ ] WebAssembly Regular expressions memoized ReDoS Rust JIT Ruby 3.3 was released on December 25, 2023.[1] Ruby 3.3 introduces significant enhancements and performance improvements to the language. Key features include the introduction of the Prism parser for portable and maintainable parsing, the addition of the pure-Ruby JIT compiler RJIT, and major performance boosts in the YJIT compiler. Additionally, improvements in memory usage, the introduction of an M:N thread scheduler, and updates to the standard library contribute to a more efficient and developer-friendly Ruby ecosystem. [1] [ ] Semantics and philosophy [edit] [ edit edit ] Yukihiro Matsumoto Matsumoto has said that Ruby is designed for programmer productivity and fun, following the principles of good user interface design.[45] At a Google Tech Talk in 2008 he said, \"I hope to see Ruby help every programmer in the world to be productive, and to enjoy programming, and to be happy. That is the primary purpose of Ruby language.\"[46] He stresses that systems design needs to emphasize human, rather than computer, needs:[47] user interface [45] [ ] [46] [ ] [47] [ ] Often people, especially computer engineers, focus on the machines. They think, \"By doing this, the machine will run fast. By doing this, the machine will run more effectively. By doing this, the machine will something something something.\" They are focusing on machines. But in fact we need to focus on humans, on how humans care about doing programming or operating the application of the machines. We are the masters. They are the slaves. Matsumoto has said his primary design goal was to make a language that he himself enjoyed using, by minimizing programmer work and possible confusion. He has said that he had not applied the principle of least astonishment (POLA) to the design of Ruby;[47] in a May 2005 discussion on the newsgroup comp.lang.ruby, Matsumoto attempted to distance Ruby from POLA, explaining that because any design choice will be surprising to someone, he uses a personal standard in evaluating surprise. If that personal standard remains consistent, there would be few surprises for those familiar with the standard.[48] principle of least astonishment [47] [ ] [48] [ ] Matsumoto defined it this way in an interview:[47] [47] [ ] Everyone has an individual background. Someone may come from Python, someone else may come from Perl, and they may be surprised by different aspects of the language. Then they come up to me and say, 'I was surprised by this feature of the language, so Ruby violates the principle of least surprise.' Wait. Wait. The principle of least surprise is not for you only. The principle of least surprise means principle of least my surprise. And it means the principle of least surprise after you learn Ruby very well. For example, I was a C++ programmer before I started designing Ruby. I programmed in C++ exclusively for two or three years. And after two years of C++ programming, it still surprises me. Ruby is object-oriented: every value is an object, including classes and instances of types that many other languages designate as primitives (such as integers, Booleans, and \"null\"). Because everything in Ruby is an object, everything in Ruby has certain built-in abilities called methods. Every function is a method and methods are always called on an object. Methods defined at the top level scope become methods of the Object class. Since this class is an ancestor of every other class, such methods can be called on any object. They are also visible in all scopes, effectively serving as \"global\" procedures. Ruby supports inheritance with dynamic dispatch, mixins and singleton methods (belonging to, and defined for, a single instance rather than being defined on the class). Though Ruby does not support multiple inheritance, classes can import modules as mixins. object-oriented integers null function method inheritance dynamic dispatch mixins instance multiple inheritance modules Ruby has been described as a multi-paradigm programming language: it allows procedural programming (defining functions/variables outside classes makes them part of the root, 'self' Object), with object orientation (everything is an object) or functional programming (it has anonymous functions, closures, and continuations; statements all have values, and functions return the last evaluation). It has support for introspection, reflective programming, metaprogramming, and interpreter-based threads. Ruby features dynamic typing, and supports parametric polymorphism. multi-paradigm programming language functional programming anonymous functions closures continuations introspection reflective programming metaprogramming threads dynamic typing parametric polymorphism According to the Ruby FAQ, the syntax is similar to Perl's and the semantics are similar to Smalltalk's, but the design philosophy differs greatly from Python's.[49] Perl Smalltalk's Python [49] [ ] Features [edit] [ edit edit ] Thoroughly object-oriented with inheritance, mixins and metaclasses[50] object-oriented inheritance mixins metaclasses [50] [ ] Dynamic typing and duck typing Dynamic typing duck typing Everything is an expression (even statements) and everything is executed imperatively (even declarations) expression statements imperatively declarations Succinct and flexible syntax[51] that minimizes syntactic noise and serves as a foundation for domain-specific languages[52] [51] [ ] syntactic noise domain-specific languages [52] [ ] Dynamic reflection and alteration of objects to facilitate metaprogramming[53] reflection alteration metaprogramming [53] [ ] Lexical closures, iterators and generators, with a block syntax[54] Lexical closures iterators generators block syntax [54] [ ] Literal notation for arrays, hashes, regular expressions and symbols arrays hashes regular expressions symbols Embedding code in strings (interpolation) interpolation Default arguments Default arguments Four levels of variable scope (global, class, instance, and local) denoted by sigils or the lack thereof global class instance local sigils Garbage collection Garbage collection First-class continuations First-class continuations Strict Boolean coercion rules (everything is true except false and nil) coercion nil Exception handling Exception handling Operator overloading[55] Operator overloading [55] [ ] Built-in support for rational numbers, complex numbers and arbitrary-precision arithmetic rational numbers complex numbers arbitrary-precision arithmetic Custom dispatch behavior (through method_missing and const_missing) Native threads and cooperative fibers (fibers are a 1.9/YARV feature) threads fibers YARV Support for Unicode and multiple character encodings. Unicode character encodings Native plug-in API in C plug-in C Interactive Ruby Shell, an interactive command-line interpreter that can be used to test code quickly (REPL) REPL Centralized package management through RubyGems RubyGems Implemented on all major platforms Large standard library, including modules for YAML, JSON, XML, CGI, OpenSSL, HTTP, FTP, RSS, curses, zlib and Tk[56] YAML JSON XML CGI OpenSSL HTTP FTP RSS curses zlib Tk [56] [ ] Just-in-time compilation Just-in-time compilation Syntax [edit] [ edit edit ] Ruby syntax The syntax of Ruby is broadly similar to that of Perl and Python. Class and method definitions are signaled by keywords, whereas code blocks can be defined by either keywords or braces. In contrast to Perl, variables are not obligatorily prefixed with a sigil. When used, the sigil changes the semantics of scope of the variable. For practical purposes there is no distinction between expressions and statements.[57][58] Line breaks are significant and taken as the end of a statement; a semicolon may be equivalently used. Unlike Python, indentation is not significant. Perl Python sigil expressions statements [57] [ ] [58] [ ] One of the differences from Python and Perl is that Ruby keeps all of its instance variables completely private to the class and only exposes them through accessor methods (attr_writer, attr_reader, etc.). Unlike the \"getter\" and \"setter\" methods of other languages like C++ or Java, accessor methods in Ruby can be created with a single line of code via metaprogramming; however, accessor methods can also be created in the traditional fashion of C++ and Java. As invocation of these methods does not require the use of parentheses, it is trivial to change an instance variable into a full function, without modifying a single line of calling code or having to do any refactoring achieving similar functionality to C# and VB.NET property members. C++ Java metaprogramming C# VB.NET Python's property descriptors are similar, but come with a trade-off in the development process. If one begins in Python by using a publicly exposed instance variable, and later changes the implementation to use a private instance variable exposed through a property descriptor, code internal to the class may need to be adjusted to use the private variable rather than the public property. Ruby's design forces all instance variables to be private, but also provides a simple way to declare set and get methods. This is in keeping with the idea that in Ruby, one never directly accesses the internal members of a class from outside the class; rather, one passes a message to the class and receives a response. Implementations [edit] [ edit edit ] Ruby MRI \u00a7 Operating systems List of Ruby compilers Matz's Ruby interpreter [edit] [ edit edit ] The original Ruby interpreter is often referred to as Matz's Ruby Interpreter or MRI. This implementation is written in C and uses its own Ruby-specific virtual machine. interpreter Matz's Ruby Interpreter virtual machine The standardized and retired Ruby 1.8 implementation was written in C, as a single-pass interpreted language.[21] implementation C interpreted language [21] [ ] Starting with Ruby 1.9, and continuing with Ruby 2.x and above, the official Ruby interpreter has been YARV (\"Yet Another Ruby VM\"), and this implementation has superseded the slower virtual machine used in previous releases of MRI. YARV Alternative implementations [edit] [ edit edit ] As of 2018[update], there are a number of alternative implementations of Ruby, including JRuby, Rubinius, and mruby. Each takes a different approach, with JRuby and Rubinius providing just-in-time compilation and mruby also providing ahead-of-time compilation. [update] JRuby Rubinius mruby just-in-time compilation ahead-of-time compilation Ruby has three major alternative implementations: JRuby, a mixed Java and Ruby implementation that runs on the Java virtual machine. JRuby currently targets Ruby 3.1.x. JRuby Java Java virtual machine TruffleRuby, a Java implementation using the Truffle language implementation framework with GraalVM TruffleRuby GraalVM Rubinius, a C++ bytecode virtual machine that uses LLVM to compile to machine code at runtime. The bytecode compiler and most core classes are written in pure Ruby. Rubinius currently targets Ruby 2.3.1. Rubinius C++ LLVM Other Ruby implementations include: MagLev, a Smalltalk implementation that runs on GemTalk Systems' GemStone/S VM MagLev Smalltalk GemTalk Systems GemStone/S mruby, an implementation designed to be embedded into C code, in a similar vein to Lua. It is currently being developed by Yukihiro Matsumoto and others mruby Lua Yukihiro Matsumoto RGSS, or Ruby Game Scripting System, a proprietary implementation that is used by the RPG Maker series of software for game design and modification of the RPG Maker engine RGSS proprietary RPG Maker julializer, a transpiler (partial) from Ruby to Julia. It can be used for a large speedup over e.g. Ruby or JRuby implementations (may only be useful for numerical code).[59] julializer transpiler Julia [59] [ ] Topaz, a Ruby implementation written in Python Topaz Python Opal, a web-based interpreter that compiles Ruby to JavaScript Opal JavaScript Other now defunct Ruby implementations were: MacRuby, a Mac OS X implementation on the Objective-C runtime. Its iOS counterpart is called RubyMotion MacRuby Mac OS X Objective-C RubyMotion IronRuby an implementation on the .NET Framework IronRuby .NET Framework Cardinal, an implementation for the Parrot virtual machine Parrot virtual machine Ruby Enterprise Edition, often shortened to ree, an implementation optimized to handle large-scale Ruby on Rails projects Ruby Enterprise Edition Ruby on Rails HotRuby, a JavaScript and ActionScript implementation of the Ruby programming language HotRuby JavaScript ActionScript Ruby programming language The maturity of Ruby implementations tends to be measured by their ability to run the Ruby on Rails (Rails) framework, because it is complex to implement and uses many Ruby-specific features. The point when a particular implementation achieves this goal is called \"the Rails singularity\". The reference implementation, JRuby, and Rubinius[60] are all able to run Rails unmodified in a production environment. Ruby on Rails [60] [ ] Platform support [edit] [ edit edit ] Matsumoto originally developed Ruby on the 4.3BSD-based Sony NEWS-OS 3.x, but later migrated his work to SunOS 4.x, and finally to Linux.[61][62] By 1999, Ruby was known to work across many different operating systems. Modern Ruby versions and implementations are available on all major desktop, mobile and server-based operating systems. Ruby is also supported across a number of cloud hosting platforms like Jelastic, Heroku, Google Cloud Platform and others. 4.3BSD Sony NEWS-OS SunOS Linux [61] [ ] [62] [ ] operating systems Jelastic Heroku Google Cloud Platform Tools such as RVM and RBEnv, facilitate installation and partitioning of multiple ruby versions, and multiple 'gemsets' on one machine. RVM RBEnv Repositories and libraries [edit] [ edit edit ] RubyGems is Ruby's package manager. A Ruby package is called a \"gem\" and can be installed via the command line. Most gems are libraries, though a few exist that are applications, such as IDEs.[63] There are over 100,000 Ruby gems hosted on RubyGems.org.[64] RubyGems IDEs [63] [ ] RubyGems.org [64] [ ] Many new and existing Ruby libraries are hosted on GitHub, a service that offers version control repository hosting for Git. GitHub version control Git The Ruby Application Archive, which hosted applications, documentation, and libraries for Ruby programming, was maintained until 2013, when its function was transferred to RubyGems.[65] [65] [ ] See also [edit] [ edit edit ] Free and open-source software portal Free and open-source software portal Free and open-source software portal Computer programming portal Computer programming portal Computer programming portal Comparison of programming languages Comparison of programming languages Metasploit Metasploit Why's (poignant) Guide to Ruby Why's (poignant) Guide to Ruby Crystal (programming language) Crystal (programming language) Ruby on Rails Ruby on Rails References [edit] [ edit edit ] ^ a b \"Ruby 3.3.0 Released\". Archived from the original on 2023-12-25. Retrieved 2023-12-25. ^ a b a b \"Ruby 3.3.0 Released\". Archived from the original on 2023-12-25. Retrieved 2023-12-25. \"Ruby 3.3.0 Released\" Archived . Retrieved 2023-12-25 2023-12-25 ^ a b c Cooper, Peter (2009). Beginning Ruby: From Novice to Professional. Beginning from Novice to Professional (2nd ed.). Berkeley: APress. p. 101. ISBN 978-1-4302-2363-4. To a lesser extent, Python, LISP, Eiffel, Ada, and C++ have also influenced Ruby. ^ a b c a b c Cooper, Peter (2009). Beginning Ruby: From Novice to Professional. Beginning from Novice to Professional (2nd ed.). Berkeley: APress. p. 101. ISBN 978-1-4302-2363-4. To a lesser extent, Python, LISP, Eiffel, Ada, and C++ have also influenced Ruby. ISBN 978-1-4302-2363-4 ^ a b \"Reasons behind Ruby\". Ruby Conference 2008. Confreaks (YouTube). 15 April 2013. ^ a b a b \"Reasons behind Ruby\". Ruby Conference 2008. Confreaks (YouTube). 15 April 2013. \"Reasons behind Ruby\" ^ a b c d e f Bini, Ola (2007). Practical JRuby on Rails Web 2.0 Projects: Bringing Ruby on Rails to Java. Berkeley: APress. p. 3. ISBN 978-1-59059-881-8. It draws primarily on features from Perl, Smalltalk, Python, Lisp, Dylan, and CLU. ^ a b c d e f a b c d e f Bini, Ola (2007). Practical JRuby on Rails Web 2.0 Projects: Bringing Ruby on Rails to Java. Berkeley: APress. p. 3. ISBN 978-1-59059-881-8. It draws primarily on features from Perl, Smalltalk, Python, Lisp, Dylan, and CLU. Practical JRuby on Rails Web 2.0 Projects: Bringing Ruby on Rails to Java 3 ISBN 978-1-59059-881-8 ^ \"Julia 1.0 Documentation: Introduction\". Archived from the original on 16 August 2018. Retrieved 6 October 2018. ^ ^ \"Julia 1.0 Documentation: Introduction\". Archived from the original on 16 August 2018. Retrieved 6 October 2018. \"Julia 1.0 Documentation: Introduction\" the original . Retrieved 6 October 2018 6 October ^ Burks, Tim. \"About Nu\u2122\". Programming Nu\u2122. Neon Design Technology, Inc. Archived from the original on 2018-12-25. Retrieved 2011-07-21. ^ ^ Burks, Tim. \"About Nu\u2122\". Programming Nu\u2122. Neon Design Technology, Inc. Archived from the original on 2018-12-25. Retrieved 2011-07-21. \"About Nu\u2122\" Archived . Retrieved 2011-07-21 2011-07-21 ^ Ring Team (3 December 2017). \"Ring and other languages\". ring-lang.net. ring-lang. Archived from the original on 25 December 2018. Retrieved 3 December 2017. ^ ^ Ring Team (3 December 2017). \"Ring and other languages\". ring-lang.net. ring-lang. Archived from the original on 25 December 2018. Retrieved 3 December 2017. \"Ring and other languages\" ring-lang Archived . Retrieved 3 December 2017 3 December ^ \"Influences - The Rust Reference\". The Rust Reference. Archived from the original on 2019-01-26. Retrieved 2023-04-18. ^ ^ \"Influences - The Rust Reference\". The Rust Reference. Archived from the original on 2019-01-26. Retrieved 2023-04-18. \"Influences - The Rust Reference\" Archived . Retrieved 2023-04-18 2023-04-18 ^ Lattner, Chris (2014-06-03). \"Chris Lattner's Homepage\". Chris Lattner. Archived from the original on 2018-12-25. Retrieved 2014-06-03. The Swift language is the product of tireless effort from a team of language experts, documentation gurus, compiler optimization ninjas, and an incredibly important internal dogfooding group who provided feedback to help refine and battle-test ideas. Of course, it also greatly benefited from the experiences hard-won by many other languages in the field, drawing ideas from Objective-C, Rust, Haskell, Ruby, Python, C#, CLU, and far too many others to list. ^ ^ Lattner, Chris (2014-06-03). \"Chris Lattner's Homepage\". Chris Lattner. Archived from the original on 2018-12-25. Retrieved 2014-06-03. The Swift language is the product of tireless effort from a team of language experts, documentation gurus, compiler optimization ninjas, and an incredibly important internal dogfooding group who provided feedback to help refine and battle-test ideas. Of course, it also greatly benefited from the experiences hard-won by many other languages in the field, drawing ideas from Objective-C, Rust, Haskell, Ruby, Python, C#, CLU, and far too many others to list. \"Chris Lattner's Homepage\" Archived . Retrieved 2014-06-03 2014-06-03 ^ \"About Ruby\". Archived from the original on 9 October 2014. Retrieved 15 February 2020. ^ ^ \"About Ruby\". Archived from the original on 9 October 2014. Retrieved 15 February 2020. \"About Ruby\" Archived . Retrieved 15 February 2020 15 February ^ Matsumoto, Yukihiro (13 February 2006). \"Re: Ruby's lisp features\". Archived from the original on 2018-10-27. Retrieved 15 February 2020. ^ ^ Matsumoto, Yukihiro (13 February 2006). \"Re: Ruby's lisp features\". Archived from the original on 2018-10-27. Retrieved 15 February 2020. Matsumoto, Yukihiro \"Re: Ruby's lisp features\" the original . Retrieved 15 February 2020 15 February ^ a b c d e f \"History of Ruby\". Archived from the original on 2011-07-14. Retrieved 2008-08-14. ^ a b c d e f a b c d e f \"History of Ruby\". Archived from the original on 2011-07-14. Retrieved 2008-08-14. \"History of Ruby\" Archived . Retrieved 2008-08-14 2008-08-14 ^ \"[FYI: historic] The decisive moment of the language name Ruby. (Re: [ANN] ruby 1.8.1)\" (E-mail from Hiroshi Sugihara to ruby-talk). Archived from the original on 2011-07-17. Retrieved 2008-08-14. ^ ^ \"[FYI: historic] The decisive moment of the language name Ruby. (Re: [ANN] ruby 1.8.1)\" (E-mail from Hiroshi Sugihara to ruby-talk). Archived from the original on 2011-07-17. Retrieved 2008-08-14. \"[FYI: historic] The decisive moment of the language name Ruby. (Re: [ANN] ruby 1.8.1)\" the original . Retrieved 2008-08-14 2008-08-14 ^ \"1.3 Why the name 'Ruby'?\". The Ruby Language FAQ. Ruby-Doc.org. Archived from the original on May 28, 2012. Retrieved April 10, 2012. ^ ^ \"1.3 Why the name 'Ruby'?\". The Ruby Language FAQ. Ruby-Doc.org. Archived from the original on May 28, 2012. Retrieved April 10, 2012. \"1.3 Why the name 'Ruby'?\" Archived . Retrieved April 10, 2012 April 10, ^ Yukihiro Matsumoto (June 11, 1999). \"Re: the name of Ruby?\". Ruby-Talk (Mailing list). Archived from the original on December 25, 2018. Retrieved April 10, 2012. ^ ^ Yukihiro Matsumoto (June 11, 1999). \"Re: the name of Ruby?\". Ruby-Talk (Mailing list). Archived from the original on December 25, 2018. Retrieved April 10, 2012. Yukihiro Matsumoto \"Re: the name of Ruby?\" the original . Retrieved April 10, 2012 April 10, ^ \"More archeolinguistics: unearthing proto-Ruby\". Archived from the original on 6 November 2015. Retrieved 2 May 2015. ^ ^ \"More archeolinguistics: unearthing proto-Ruby\". Archived from the original on 6 November 2015. Retrieved 2 May 2015. \"More archeolinguistics: unearthing proto-Ruby\" the original . Retrieved 2 May 2015 2 May ^ \"[ruby-talk:00382] Re: history of ruby\". Archived from the original on 16 July 2011. Retrieved 2 May 2015. ^ ^ \"[ruby-talk:00382] Re: history of ruby\". Archived from the original on 16 July 2011. Retrieved 2 May 2015. \"[ruby-talk:00382] Re: history of ruby\" the original . Retrieved 2 May 2015 2 May ^ \"[ruby-list:124] TUTORIAL \u2014 ruby's features\". Archived from the original on 24 May 2003. Retrieved 2 May 2015. ^ ^ \"[ruby-list:124] TUTORIAL \u2014 ruby's features\". Archived from the original on 24 May 2003. Retrieved 2 May 2015. \"[ruby-list:124] TUTORIAL \u2014 ruby's features\" the original . Retrieved 2 May 2015 2 May ^ \"An Interview with the Creator of Ruby\". Archived from the original on 2008-02-08. Retrieved 2007-07-11. ^ ^ \"An Interview with the Creator of Ruby\". Archived from the original on 2008-02-08. Retrieved 2007-07-11. \"An Interview with the Creator of Ruby\" Archived . Retrieved 2007-07-11 2007-07-11 ^ Yukihiro Matsumoto (October 2000). \"Programming Ruby: Forward\". Archived from the original on 25 December 2018. Retrieved 5 March 2014. ^ ^ Yukihiro Matsumoto (October 2000). \"Programming Ruby: Forward\". Archived from the original on 25 December 2018. Retrieved 5 March 2014. \"Programming Ruby: Forward\" Archived . Retrieved 5 March 2014 5 March ^ a b \"We retire Ruby 1.8.7\". Archived from the original on 6 June 2015. Retrieved 2 May 2015. ^ a b a b \"We retire Ruby 1.8.7\". Archived from the original on 6 June 2015. Retrieved 2 May 2015. \"We retire Ruby 1.8.7\" Archived . Retrieved 2 May 2015 2 May ^ \"IPA \u72ec\u7acb\u884c\u653f\u6cd5\u4eba \u60c5\u5831\u51e6\u7406\u63a8\u9032\u6a5f\u69cb\uff1a\u30d7\u30ec\u30b9\u767a\u8868 \u30d7\u30ed\u30b0\u30e9\u30e0\u8a00\u8a9eRuby\u306eJIS\u898f\u683c\uff08JIS X 3017\uff09\u5236\u5b9a\u306b\u3064\u3044\u3066\". Archived from the original on 2 February 2015. Retrieved 2 May 2015. ^ ^ \"IPA \u72ec\u7acb\u884c\u653f\u6cd5\u4eba \u60c5\u5831\u51e6\u7406\u63a8\u9032\u6a5f\u69cb\uff1a\u30d7\u30ec\u30b9\u767a\u8868 \u30d7\u30ed\u30b0\u30e9\u30e0\u8a00\u8a9eRuby\u306eJIS\u898f\u683c\uff08JIS X 3017\uff09\u5236\u5b9a\u306b\u3064\u3044\u3066\". Archived from the original on 2 February 2015. Retrieved 2 May 2015. \"IPA \u72ec\u7acb\u884c\u653f\u6cd5\u4eba \u60c5\u5831\u51e6\u7406\u63a8\u9032\u6a5f\u69cb\uff1a\u30d7\u30ec\u30b9\u767a\u8868 \u30d7\u30ed\u30b0\u30e9\u30e0\u8a00\u8a9eRuby\u306eJIS\u898f\u683c\uff08JIS X 3017\uff09\u5236\u5b9a\u306b\u3064\u3044\u3066\" the original . Retrieved 2 May 2015 2 May ^ \"IPA \u72ec\u7acb\u884c\u653f\u6cd5\u4eba \u60c5\u5831\u51e6\u7406\u63a8\u9032\u6a5f\u69cb\uff1a\u30d7\u30ec\u30b9\u767a\u8868 \u30d7\u30ed\u30b0\u30e9\u30e0\u8a00\u8a9eRuby\u3001\u56fd\u969b\u898f\u683c\u3068\u3057\u3066\u627f\u8a8d\". Archived from the original on 1 February 2015. Retrieved 2 May 2015. ^ ^ \"IPA \u72ec\u7acb\u884c\u653f\u6cd5\u4eba \u60c5\u5831\u51e6\u7406\u63a8\u9032\u6a5f\u69cb\uff1a\u30d7\u30ec\u30b9\u767a\u8868 \u30d7\u30ed\u30b0\u30e9\u30e0\u8a00\u8a9eRuby\u3001\u56fd\u969b\u898f\u683c\u3068\u3057\u3066\u627f\u8a8d\". Archived from the original on 1 February 2015. Retrieved 2 May 2015. \"IPA \u72ec\u7acb\u884c\u653f\u6cd5\u4eba \u60c5\u5831\u51e6\u7406\u63a8\u9032\u6a5f\u69cb\uff1a\u30d7\u30ec\u30b9\u767a\u8868 \u30d7\u30ed\u30b0\u30e9\u30e0\u8a00\u8a9eRuby\u3001\u56fd\u969b\u898f\u683c\u3068\u3057\u3066\u627f\u8a8d\" the original . Retrieved 2 May 2015 2 May ^ \"ISO/IEC 30170:2012\". Archived from the original on 2017-03-12. Retrieved 2017-03-10. ^ ^ \"ISO/IEC 30170:2012\". Archived from the original on 2017-03-12. Retrieved 2017-03-10. \"ISO/IEC 30170:2012\" Archived . Retrieved 2017-03-10 2017-03-10 ^ Web Development: Ruby on Rails Archived 2009-02-24 at the Wayback Machine. Devarticles.com (2007-03-22). Retrieved on 2013-07-17. ^ ^ Web Development: Ruby on Rails Archived 2009-02-24 at the Wayback Machine. Devarticles.com (2007-03-22). Retrieved on 2013-07-17. Web Development: Ruby on Rails Archived Wayback Machine ^ \"Ruby 1.9.3 p0 is released\". ruby-lang.org. October 31, 2011. Archived from the original on January 14, 2013. Retrieved February 20, 2013. ^ ^ \"Ruby 1.9.3 p0 is released\". ruby-lang.org. October 31, 2011. Archived from the original on January 14, 2013. Retrieved February 20, 2013. \"Ruby 1.9.3 p0 is released\" Archived . Retrieved February 20, 2013 February 20, ^ \"v1_9_3_0/NEWS\". Ruby Subversion source repository. ruby-lang.org. September 17, 2011. Archived from the original on November 6, 2015. Retrieved February 20, 2013. ^ ^ \"v1_9_3_0/NEWS\". Ruby Subversion source repository. ruby-lang.org. September 17, 2011. Archived from the original on November 6, 2015. Retrieved February 20, 2013. \"v1_9_3_0/NEWS\" Subversion the original . Retrieved February 20, 2013 February 20, ^ Ruby 1.9: What to Expect Archived 2016-03-04 at the Wayback Machine. slideshow.rubyforge.org. Retrieved on 2013-07-17. ^ ^ Ruby 1.9: What to Expect Archived 2016-03-04 at the Wayback Machine. slideshow.rubyforge.org. Retrieved on 2013-07-17. Ruby 1.9: What to Expect Archived Wayback Machine ^ Endoh, Yusuke. (2013-02-24) Ruby 2.0.0-p0 is released Archived 2013-02-27 at the Wayback Machine. Ruby-lang.org. Retrieved on 2013-07-17. ^ ^ Endoh, Yusuke. (2013-02-24) Ruby 2.0.0-p0 is released Archived 2013-02-27 at the Wayback Machine. Ruby-lang.org. Retrieved on 2013-07-17. Ruby 2.0.0-p0 is released Archived Wayback Machine ^ Endoh, Yusuke. (2013-02-24) Ruby 2.0.0-p0 is released Archived 2016-01-17 at the Wayback Machine. Ruby-lang.org. Retrieved on 2013-07-17. ^ ^ Endoh, Yusuke. (2013-02-24) Ruby 2.0.0-p0 is released Archived 2016-01-17 at the Wayback Machine. Ruby-lang.org. Retrieved on 2013-07-17. Ruby 2.0.0-p0 is released Archived Wayback Machine ^ \"Semantic Versioning starting with Ruby 2.1.0\". December 21, 2013. Archived from the original on February 13, 2014. Retrieved December 27, 2013. ^ ^ \"Semantic Versioning starting with Ruby 2.1.0\". December 21, 2013. Archived from the original on February 13, 2014. Retrieved December 27, 2013. \"Semantic Versioning starting with Ruby 2.1.0\" the original . Retrieved December 27, 2013 December 27, ^ Gustavo Frederico Temple Pedrosa, Vitor de Lima, Leonardo Bianconi (2015). \"Ruby 2.2.1 Released\". Archived from the original on 16 May 2016. Retrieved 12 July 2016.{{cite web}}: CS1 maint: multiple names: authors list (link) ^ ^ Gustavo Frederico Temple Pedrosa, Vitor de Lima, Leonardo Bianconi (2015). \"Ruby 2.2.1 Released\". Archived from the original on 16 May 2016. Retrieved 12 July 2016.{{cite web}}: CS1 maint: multiple names: authors list (link) \"Ruby 2.2.1 Released\" Archived . Retrieved 12 July 2016 12 July {{cite web}}: CS1 maint: multiple names: authors list (link) cite web link ^ Gustavo Frederico Temple Pedrosa, Vitor de Lima, Leonardo Bianconi (2015). \"v2.2.1 ChangeLog\". Archived from the original on 26 February 2017. Retrieved 12 July 2016.{{cite web}}: CS1 maint: multiple names: authors list (link) ^ ^ Gustavo Frederico Temple Pedrosa, Vitor de Lima, Leonardo Bianconi (2015). \"v2.2.1 ChangeLog\". Archived from the original on 26 February 2017. Retrieved 12 July 2016.{{cite web}}: CS1 maint: multiple names: authors list (link) \"v2.2.1 ChangeLog\" the original . Retrieved 12 July 2016 12 July {{cite web}}: CS1 maint: multiple names: authors list (link) cite web link ^ Gustavo Frederico Temple Pedrosa, Vitor de Lima, Leonardo Bianconi (2014). \"Specifying non volatile registers for increase performance in ppc64\". Archived from the original on 17 September 2016. Retrieved 12 July 2016.{{cite web}}: CS1 maint: multiple names: authors list (link) ^ ^ Gustavo Frederico Temple Pedrosa, Vitor de Lima, Leonardo Bianconi (2014). \"Specifying non volatile registers for increase performance in ppc64\". Archived from the original on 17 September 2016. Retrieved 12 July 2016.{{cite web}}: CS1 maint: multiple names: authors list (link) \"Specifying non volatile registers for increase performance in ppc64\" Archived . Retrieved 12 July 2016 12 July {{cite web}}: CS1 maint: multiple names: authors list (link) cite web link ^ Gustavo Frederico Temple Pedrosa, Vitor de Lima, Leonardo Bianconi (2014). \"Specifying MACRO for increase performance in ppc64\". Archived from the original on 17 September 2016. Retrieved 12 July 2016.{{cite web}}: CS1 maint: multiple names: authors list (link) ^ ^ Gustavo Frederico Temple Pedrosa, Vitor de Lima, Leonardo Bianconi (2014). \"Specifying MACRO for increase performance in ppc64\". Archived from the original on 17 September 2016. Retrieved 12 July 2016.{{cite web}}: CS1 maint: multiple names: authors list (link) \"Specifying MACRO for increase performance in ppc64\" Archived . Retrieved 12 July 2016 12 July {{cite web}}: CS1 maint: multiple names: authors list (link) cite web link ^ \"ruby/NEWS at v2_2_0 \u00b7 ruby/ruby \u00b7 GitHub\". GitHub. Archived from the original on 1 January 2015. Retrieved 2 May 2015. ^ ^ \"ruby/NEWS at v2_2_0 \u00b7 ruby/ruby \u00b7 GitHub\". GitHub. Archived from the original on 1 January 2015. Retrieved 2 May 2015. \"ruby/NEWS at v2_2_0 \u00b7 ruby/ruby \u00b7 GitHub\" Archived . Retrieved 2 May 2015 2 May ^ \"Ruby/NEWS at v.2_3_0 - ruby/ruby\". GitHub. Archived from the original on 1 March 2017. Retrieved 25 December 2015. ^ ^ \"Ruby/NEWS at v.2_3_0 - ruby/ruby\". GitHub. Archived from the original on 1 March 2017. Retrieved 25 December 2015. \"Ruby/NEWS at v.2_3_0 - ruby/ruby\" Archived . Retrieved 25 December 2015 25 December ^ \"Ruby 2.3.0 changes and features\". Running with Ruby. dev.mensfeld.pl. 14 November 2015. Archived from the original on 5 January 2016. Retrieved 27 December 2015. ^ ^ \"Ruby 2.3.0 changes and features\". Running with Ruby. dev.mensfeld.pl. 14 November 2015. Archived from the original on 5 January 2016. Retrieved 27 December 2015. \"Ruby 2.3.0 changes and features\" Archived . Retrieved 27 December 2015 27 December ^ \"Ruby 2.4.0 Released\". www.ruby-lang.org. Archived from the original on 2017-02-17. Retrieved 2016-12-30. ^ ^ \"Ruby 2.4.0 Released\". www.ruby-lang.org. Archived from the original on 2017-02-17. Retrieved 2016-12-30. \"Ruby 2.4.0 Released\" Archived . Retrieved 2016-12-30 2016-12-30 ^ a b c d \"Ruby 3.0.0 Released\". Ruby Programming Language. 2020-12-25. Archived from the original on 2020-12-25. Retrieved 2020-12-25. ^ a b c d a b c d \"Ruby 3.0.0 Released\". Ruby Programming Language. 2020-12-25. Archived from the original on 2020-12-25. Retrieved 2020-12-25. \"Ruby 3.0.0 Released\" Archived . Retrieved 2020-12-25 2020-12-25 ^ Scheffler, Jonan (November 10, 2016). \"Ruby 3x3: Matz, Koichi, and Tenderlove on the future of Ruby Performance\". Ruby. Archived from the original on May 10, 2019. Retrieved May 18, 2019. ^ ^ Scheffler, Jonan (November 10, 2016). \"Ruby 3x3: Matz, Koichi, and Tenderlove on the future of Ruby Performance\". Ruby. Archived from the original on May 10, 2019. Retrieved May 18, 2019. \"Ruby 3x3: Matz, Koichi, and Tenderlove on the future of Ruby Performance\" Archived . Retrieved May 18, 2019 May 18, ^ \"Ruby 3.1.0 Released\". ruby-lang.org. Archived from the original on 25 December 2021. Retrieved 25 Dec 2021. ^ ^ \"Ruby 3.1.0 Released\". ruby-lang.org. Archived from the original on 25 December 2021. Retrieved 25 Dec 2021. \"Ruby 3.1.0 Released\" Archived . Retrieved 25 Dec 2021 25 Dec ^ \"Ruby 3.1.0 Released\". Archived from the original on 2021-12-26. Retrieved 2021-12-26. ^ ^ \"Ruby 3.1.0 Released\". Archived from the original on 2021-12-26. Retrieved 2021-12-26. \"Ruby 3.1.0 Released\" Archived . Retrieved 2021-12-26 2021-12-26 ^ \"Ruby 3.2.0 Released\". Archived from the original on 2022-12-25. Retrieved 2022-12-25. ^ ^ \"Ruby 3.2.0 Released\". Archived from the original on 2022-12-25. Retrieved 2022-12-25. \"Ruby 3.2.0 Released\" Archived . Retrieved 2022-12-25 2022-12-25 ^ \"The Ruby Programming Language\". Archived from the original on 18 January 2020. Retrieved 2 May 2015. ^ ^ \"The Ruby Programming Language\". Archived from the original on 18 January 2020. Retrieved 2 May 2015. \"The Ruby Programming Language\" Archived . Retrieved 2 May 2015 2 May ^ Google Tech Talks \u2013 Ruby 1.9 on YouTube ^ ^ Google Tech Talks \u2013 Ruby 1.9 on YouTube Google Tech Talks \u2013 Ruby 1.9 Google Tech Talks \u2013 Ruby 1.9 YouTube ^ a b c Bill Venners. \"The Philosophy of Ruby\". Archived from the original on 5 July 2019. Retrieved 2 May 2015. ^ a b c a b c Bill Venners. \"The Philosophy of Ruby\". Archived from the original on 5 July 2019. Retrieved 2 May 2015. \"The Philosophy of Ruby\" Archived . Retrieved 2 May 2015 2 May ^ \"Welcome to RUBYWEEKLYNEWS.ORG\". 4 July 2017. Archived from the original on 4 July 2017.{{cite web}}: CS1 maint: bot: original URL status unknown (link) ^ ^ \"Welcome to RUBYWEEKLYNEWS.ORG\". 4 July 2017. Archived from the original on 4 July 2017.{{cite web}}: CS1 maint: bot: original URL status unknown (link) \"Welcome to RUBYWEEKLYNEWS.ORG\" {{cite web}}: CS1 maint: bot: original URL status unknown (link) cite web link ^ \"The Ruby Language FAQ: How Does Ruby Stack Up Against...?\". Archived from the original on 8 May 2015. Retrieved 2 May 2015. ^ ^ \"The Ruby Language FAQ: How Does Ruby Stack Up Against...?\". Archived from the original on 8 May 2015. Retrieved 2 May 2015. \"The Ruby Language FAQ: How Does Ruby Stack Up Against...?\" Archived . Retrieved 2 May 2015 2 May ^ Bruce Stewart (29 November 2001). \"An Interview with the Creator of Ruby\". O'Reilly Media. Archived from the original on 6 May 2015. Retrieved 2 May 2015. ^ ^ Bruce Stewart (29 November 2001). \"An Interview with the Creator of Ruby\". O'Reilly Media. Archived from the original on 6 May 2015. Retrieved 2 May 2015. \"An Interview with the Creator of Ruby\" Archived . Retrieved 2 May 2015 2 May ^ Bill Venners. \"Dynamic Productivity with Ruby\". Archived from the original on 5 September 2015. Retrieved 2 May 2015. ^ ^ Bill Venners. \"Dynamic Productivity with Ruby\". Archived from the original on 5 September 2015. Retrieved 2 May 2015. \"Dynamic Productivity with Ruby\" Archived . Retrieved 2 May 2015 2 May ^ \"Language Workbenches: The Killer-App for Domain Specific Languages?\". martinfowler.com. Archived from the original on 2 May 2021. Retrieved 2 May 2015. ^ ^ \"Language Workbenches: The Killer-App for Domain Specific Languages?\". martinfowler.com. Archived from the original on 2 May 2021. Retrieved 2 May 2015. \"Language Workbenches: The Killer-App for Domain Specific Languages?\" Archived . Retrieved 2 May 2015 2 May ^ \"Ruby \u2013 Add class methods at runtime\". Archived from the original on 2007-09-22. Retrieved 2007-11-01. ^ ^ \"Ruby \u2013 Add class methods at runtime\". Archived from the original on 2007-09-22. Retrieved 2007-11-01. \"Ruby \u2013 Add class methods at runtime\" Archived . Retrieved 2007-11-01 2007-11-01 ^ Bill Venners. \"Blocks and Closures in Ruby\". Archived from the original on 18 April 2015. Retrieved 2 May 2015. ^ ^ Bill Venners. \"Blocks and Closures in Ruby\". Archived from the original on 18 April 2015. Retrieved 2 May 2015. \"Blocks and Closures in Ruby\" Archived . Retrieved 2 May 2015 2 May ^ \"Methods\". Official Ruby FAQ. Archived from the original on 2022-06-28. Retrieved 2021-06-20. ^ ^ \"Methods\". Official Ruby FAQ. Archived from the original on 2022-06-28. Retrieved 2021-06-20. \"Methods\" Archived . Retrieved 2021-06-20 2021-06-20 ^ Britt, James. \"Ruby 2.0.0 Standard Library Documentation\". Archived from the original on 2013-11-13. Retrieved 2013-12-09. ^ ^ Britt, James. \"Ruby 2.0.0 Standard Library Documentation\". Archived from the original on 2013-11-13. Retrieved 2013-12-09. \"Ruby 2.0.0 Standard Library Documentation\" Archived . Retrieved 2013-12-09 2013-12-09 ^ \"[ruby-talk:01120] Re: The value of while...\" Archived from the original on 2011-07-17. Retrieved 2008-12-06. In Ruby's syntax, statement is just a special case of an expression that cannot appear as an argument (e.g. multiple assignment). ^ ^ \"[ruby-talk:01120] Re: The value of while...\" Archived from the original on 2011-07-17. Retrieved 2008-12-06. In Ruby's syntax, statement is just a special case of an expression that cannot appear as an argument (e.g. multiple assignment). \"[ruby-talk:01120] Re: The value of while...\" the original . Retrieved 2008-12-06 2008-12-06 ^ \"[ruby-talk:02460] Re: Precedence question\". Archived from the original on 2004-07-22. Retrieved 2008-12-06. statement [...] can not be part of expression unless grouped within parentheses. ^ ^ \"[ruby-talk:02460] Re: Precedence question\". Archived from the original on 2004-07-22. Retrieved 2008-12-06. statement [...] can not be part of expression unless grouped within parentheses. \"[ruby-talk:02460] Re: Precedence question\" the original . Retrieved 2008-12-06 2008-12-06 ^ \"remove/virtual_module: Born to make your Ruby Code more than 3x faster. Hopefully\". GitHub. 21 February 2020. Archived from the original on 1 March 2017. Retrieved 29 August 2016. ^ ^ \"remove/virtual_module: Born to make your Ruby Code more than 3x faster. Hopefully\". GitHub. 21 February 2020. Archived from the original on 1 March 2017. Retrieved 29 August 2016. \"remove/virtual_module: Born to make your Ruby Code more than 3x faster. Hopefully\" GitHub Archived . Retrieved 29 August 2016 29 August ^ Peter Cooper (2010-05-18). \"The Why, What, and How of Rubinius 1.0's Release\". Archived from the original on 2010-05-24. Retrieved 2010-05-23. ^ ^ Peter Cooper (2010-05-18). \"The Why, What, and How of Rubinius 1.0's Release\". Archived from the original on 2010-05-24. Retrieved 2010-05-23. \"The Why, What, and How of Rubinius 1.0's Release\" Archived . Retrieved 2010-05-23 2010-05-23 ^ Maya Stodte (February 2000). \"IBM developerWorks \u2013 Ruby: a new language\". Archived from the original on August 18, 2000. Retrieved 3 March 2014. ^ ^ Maya Stodte (February 2000). \"IBM developerWorks \u2013 Ruby: a new language\". Archived from the original on August 18, 2000. Retrieved 3 March 2014. \"IBM developerWorks \u2013 Ruby: a new language\" the original . Retrieved 3 March 2014 3 March ^ Yukihiro Matsumoto (August 2002). \"lang-ruby-general: Re: question about Ruby initial development\". Archived from the original on 3 March 2014. Retrieved 3 March 2014. ^ ^ Yukihiro Matsumoto (August 2002). \"lang-ruby-general: Re: question about Ruby initial development\". Archived from the original on 3 March 2014. Retrieved 3 March 2014. \"lang-ruby-general: Re: question about Ruby initial development\" Archived . Retrieved 3 March 2014 3 March ^ \"The Ruby Toolbox\". Archived from the original on 2015-04-03. Retrieved 2015-04-04. ^ ^ \"The Ruby Toolbox\". Archived from the original on 2015-04-03. Retrieved 2015-04-04. \"The Ruby Toolbox\" Archived . Retrieved 2015-04-04 2015-04-04 ^ \"Stats RubyGems.org your community gem host\". rubygems.org. Archived from the original on 10 December 2021. Retrieved 10 December 2021. ^ ^ \"Stats RubyGems.org your community gem host\". rubygems.org. Archived from the original on 10 December 2021. Retrieved 10 December 2021. \"Stats RubyGems.org your community gem host\" Archived . Retrieved 10 December 2021 10 December ^ \"We retire raa.ruby-lang.org\". 2013-08-08. Archived from the original on 2015-12-31. Retrieved 2016-01-03. ^ ^ \"We retire raa.ruby-lang.org\". 2013-08-08. Archived from the original on 2015-12-31. Retrieved 2016-01-03. \"We retire raa.ruby-lang.org\" Archived . Retrieved 2016-01-03 2016-01-03 Further reading [edit] [ edit edit ] Black, David; Leo, Joseph (March 15, 2019), The Well-Grounded Rubyist (Third ed.), Manning Publications, p. 584, ISBN 978-1617295218 Manning Publications ISBN 978-1617295218 Metz, Sandi (August 22, 2018), Practical Object-Oriented Design: An Agile Primer Using Ruby (Second ed.), Addison-Wesley Professional, p. 288, ISBN 978-0-13-445647-8, archived from the original on February 13, 2020, retrieved February 13, 2020 Practical Object-Oriented Design: An Agile Primer Using Ruby Addison-Wesley Professional ISBN 978-0-13-445647-8 archived , retrieved February 13, 2020 February 13, Cooper, Peter (July 12, 2016), Beginning Ruby: From Novice to Professional (Third ed.), Apress, p. 492, ISBN 978-1484212790 Apress ISBN 978-1484212790 Carlson, Lucas; Richardson, Leonard (April 3, 2015), Ruby Cookbook: Recipes for Object-Oriented Scripting (Second ed.), O'Reilly Media, p. 963, ISBN 978-1449373719 Ruby Cookbook: Recipes for Object-Oriented Scripting O'Reilly Media ISBN 978-1449373719 Fulton, Hal; Arko, Andr\u00e9 (March 2, 2015), The Ruby Way: Solutions and Techniques in Ruby Programming (Third ed.), Addison-Wesley Professional, p. 816, ISBN 978-0-321-71463-3, archived from the original on February 13, 2020, retrieved February 13, 2020 The Ruby Way: Solutions and Techniques in Ruby Programming Addison-Wesley Professional ISBN 978-0-321-71463-3 archived , retrieved February 13, 2020 February 13, Thomas, Dave; Fowler, Chad; Hunt, Andy (July 7, 2013), Programming Ruby 1.9 & 2.0: The Pragmatic Programmers' Guide (Fourth ed.), Pragmatic Bookshelf, p. 888, ISBN 978-1937785499 Pragmatic Bookshelf ISBN 978-1937785499 McAnally, Jeremy; Arkin, Assaf (March 28, 2009), Ruby in Practice (First ed.), Manning Publications, p. 360, ISBN 978-1933988474 Manning Publications ISBN 978-1933988474 Flanagan, David; Matsumoto, Yukihiro (January 25, 2008), The Ruby Programming Language (First ed.), O'Reilly Media, p. 446, ISBN 978-0-596-51617-8 The Ruby Programming Language O'Reilly Media 446 ISBN 978-0-596-51617-8 Baird, Kevin (June 8, 2007), Ruby by Example: Concepts and Code (First ed.), No Starch Press, p. 326, ISBN 978-1593271480, archived from the original on January 13, 2020, retrieved February 13, 2020 Ruby by Example: Concepts and Code No Starch Press ISBN 978-1593271480 archived , retrieved February 13, 2020 February 13, Fitzgerald, Michael (May 14, 2007), Learning Ruby (First ed.), O'Reilly Media, p. 255, ISBN 978-0-596-52986-4 Learning Ruby O'Reilly Media 255 ISBN 978-0-596-52986-4 External links [edit] [ edit edit ] Ruby (programming language) Ruby programming language Ruby programming language Topic:Ruby Official website Official website Official website Official website Ruby documentation Ruby documentation v v t t e e Ruby programming language Implementations Ruby MRI; YARV Ruby MRI YARV mruby mruby JRuby JRuby RubyMotion RubyMotion MacRuby MacRuby IronRuby IronRuby XRuby HotRuby HotRuby MagLev MagLev Rubinius Rubinius IDE Aptana Studio Aptana Studio Komodo IDE Komodo IDE RubyMine RubyMine Applications Capistrano Capistrano Hackety Hack Hackety Hack Interactive Ruby Shell Interactive Ruby Shell Pry Pry Rake Rake Redmine Redmine RSpec RSpec RubyGems RubyGems RVM RVM Chef Chef Puppet Puppet Vagrant Vagrant Homebrew Homebrew CocoaPods CocoaPods Libraries frameworks Adhearsion Adhearsion eRuby (RHTML) eRuby Merb Merb Padrino Padrino RubyCocoa RubyCocoa Ruby on Rails Ruby on Rails Shoes Shoes Sinatra Sinatra QtRuby QtRuby Server mod_ruby mod_ruby Mongrel Mongrel Phusion Passenger (mod_rails/mod_rack) Phusion Passenger Rack Rack WEBrick WEBrick Yukihiro Matsumoto Yukihiro Matsumoto David Heinemeier Hansson David Heinemeier Hansson why the lucky stiff why the lucky stiff Dave Thomas Dave Thomas Document format Document format why's (poignant) Guide to Ruby why's (poignant) Guide to Ruby Ruby Central Ruby Central RubyKaigi RubyKaigi RubyForge RubyForge Ruby License Ruby License Category Category Commons Commons v v t t e e Programming languages Comparison Comparison Timeline Timeline History History Ada Ada ALGOL Simula ALGOL Simula Simula APL APL Assembly Assembly BASIC Visual Basic classic .NET BASIC Visual Basic classic .NET Visual Basic classic classic .NET .NET C C C++ C++ C# C# COBOL COBOL Erlang Erlang Forth Forth Fortran Fortran Go Go Haskell Haskell Java Java JavaScript JavaScript Julia Julia Kotlin Kotlin Lisp Lisp Lua Lua MATLAB MATLAB ML ML Pascal Object Pascal Pascal Object Pascal Object Pascal Perl Perl PHP PHP Prolog Prolog Python Python R R Ruby Ruby Rust Rust SQL SQL Scratch Scratch Shell Shell Smalltalk Smalltalk Swift Swift more... more... Lists: Alphabetical Alphabetical Categorical Categorical Generational Generational Non-English-based Non-English-based Category Category v v t t e e Free and open-source software Alternative terms for free software Alternative terms for free software Comparison of open-source and closed-source software Comparison of open-source and closed-source software Comparison of source-code-hosting facilities Comparison of source-code-hosting facilities Free software Free software Free software project directories Free software project directories Gratis versus libre Gratis versus libre Long-term support Long-term support Open-source software Open-source software Open-source software development Open-source software development Outline Outline Timeline Timeline Softwarepackages Audio Audio Bioinformatics Bioinformatics Codecs Codecs Configuration management Configuration management Drivers Graphics Wireless Drivers Graphics Graphics Wireless Wireless Health Health Mathematics Mathematics Office suites Office suites Operating systems Operating systems Routing Routing Television Television Video games Video games Web applications E-commerce Web applications E-commerce E-commerce Android apps Android apps iOS apps iOS apps Commercial Commercial Formerly proprietary Formerly proprietary Formerly open-source Formerly open-source Community Free software movement Free software movement History History Open-source-software movement Open-source-software movement Events Events Advocacy Advocacy Organisations Free Software Movement of India Free Software Movement of India Free Software Foundation Free Software Foundation Licenses AFL AFL Apache Apache APSL APSL Artistic Artistic Beerware Beerware BSD BSD Creative Commons Creative Commons CDDL CDDL EPL EPL Free Software Foundation GNU GPL GNU AGPL GNU LGPL Free Software Foundation GNU GPL GNU GPL GNU AGPL GNU AGPL GNU LGPL GNU LGPL ISC ISC MIT MIT MPL MPL Python Python Python Software Foundation License Python Software Foundation License Shared Source Initiative Shared Source Initiative Sleepycat Sleepycat Unlicense Unlicense WTFPL WTFPL zlib zlib Comparison of licenses Comparison of licenses Contributor License Agreement Contributor License Agreement Copyleft Copyleft Debian Free Software Guidelines Debian Free Software Guidelines Definition of Free Cultural Works Definition of Free Cultural Works Free license Free license The Free Software Definition The Free Software Definition The Open Source Definition The Open Source Definition Open-source license Open-source license Permissive software license Permissive software license Public domain Public domain Digital rights management Digital rights management License proliferation License proliferation Mozilla software rebranding Mozilla software rebranding Proprietary device drivers Proprietary device drivers Proprietary firmware Proprietary firmware Proprietary software Proprietary software SCO/Linux controversies SCO/Linux controversies Software patents Software patents Software security Software security Tivoization Tivoization Trusted Computing Trusted Computing Forking Forking GNU Manifesto GNU Manifesto Microsoft Open Specification Promise Microsoft Open Specification Promise Open-core model Open-core model Open-source hardware Open-source hardware Shared Source Initiative Shared Source Initiative Source-available software Source-available software The Cathedral and the Bazaar The Cathedral and the Bazaar Revolution OS Revolution OS Portal Portal Category Category v v t t e e ISO by standard number ISO standards ISO romanizations IEC standards 1 1 2 2 3 3 4 4 6 6 7 7 9 9 16 16 17 17 31 -0 -1 -3 -4 -5 -6 -7 -8 -9 -10 -11 -12 -13 31 -0 -0 -1 -1 -3 -3 -4 -4 -5 -5 -6 -6 -7 -7 -8 -8 -9 -9 -10 -10 -11 -11 -12 -12 -13 -13 68-1 68-1 128 128 216 216 217 217 226 226 228 228 233 233 259 259 261 261 262 262 302 302 306 306 361 361 500 500 518 518 519 519 639 -1 -2 -3 -5 -6 639 -1 -1 -2 -2 -3 -3 -5 -5 -6 -6 646 646 657 657 668 668 690 690 704 704 732 732 764 764 838 838 843 843 860 860 898 898 965 965 999 999 1000 1000 1004 1004 1007 1007 1073-1 1073-1 1073-2 1073-2 1155 1155 1413 1413 1538 1538 1629 1629 1745 1745 1989 1989 2014 2014 2015 2015 2022 2022 2033 2033 2047 2047 2108 2108 2145 2145 2146 2146 2240 2240 2281 2281 2533 2533 2709 2709 2711 2711 2720 2720 2788 2788 2848 2848 2852 2852 2921 2921 3029 3029 3103 3103 3166 -1 -2 -3 3166 -1 -1 -2 -2 -3 -3 3297 3297 3307 3307 3601 3601 3602 3602 3864 3864 3901 3901 3950 3950 3977 3977 4031 4031 4157 4157 4165 4165 4217 4217 4909 4909 5218 5218 5426 5426 5427 5427 5428 5428 5725 5725 5775 5775 5776 5776 5800 5800 5807 5807 5964 5964 6166 6166 6344 6344 6346 6346 6373 6373 6385 6385 6425 6425 6429 6429 6438 6438 6523 6523 6709 6709 6943 6943 7001 7001 7002 7002 7010 7010 7027 7027 7064 7064 7098 7098 7185 7185 7200 7200 7498 -1 7498 -1 -1 7637 7637 7736 7736 7810 7810 7811 7811 7812 7812 7813 7813 7816 7816 7942 7942 8000 8000 8093 8093 8178 8178 8217 8217 8373 8373 8501-1 8501-1 8571 8571 8583 8583 8601 8601 8613 8613 8632 8632 8651 8651 8652 8652 8691 8691 8805/8806 8805/8806 8807 8807 8820-5 8820-5 8859 -1 -2 -3 -4 -5 -6 -7 -8 -8-I -9 -10 -11 -12 -13 -14 -15 -16 8859 -1 -1 -2 -2 -3 -3 -4 -4 -5 -5 -6 -6 -7 -7 -8 -8 -8-I -8-I -9 -9 -10 -10 -11 -11 -12 -12 -13 -13 -14 -14 -15 -15 -16 -16 8879 8879 9000/9001 9000/9001 9036 9036 9075 9075 9126 9126 9141 9141 9227 9227 9241 9241 9293 9293 9314 9314 9362 9362 9407 9407 9496 9496 9506 9506 9529 9529 9564 9564 9592/9593 9592/9593 9594 9594 9660 9660 9797-1 9797-1 9897 9897 9899 9899 9945 9945 9984 9984 9985 9985 9995 9995 10006 10006 10007 10007 10116 10116 10118-3 10118-3 10160 10160 10161 10161 10165 10165 10179 10179 10206 10206 10218 10218 10279 10279 10303 -11 -21 -22 -28 -238 10303 -11 -11 -21 -21 -22 -22 -28 -28 -238 -238 10383 10383 10585 10585 10589 10589 10628 10628 10646 10646 10664 10664 10746 10746 10861 10861 10957 10957 10962 10962 10967 10967 11073 11073 11170 11170 11172 11172 11179 11179 11404 11404 11544 11544 11783 11783 11784 11784 11785 11785 11801 11801 11889 11889 11898 11898 11940 (-2) 11940 -2 11941 11941 11941 (TR) 11941 (TR) 11992 11992 12006 12006 12052 12052 12182 12182 12207 12207 12234-2 12234-2 12620 12620 13211 -1 -2 13211 -1 -1 -2 -2 13216 13216 13250 13250 13399 13399 13406-2 13406-2 13450 13450 13485 13485 13490 13490 13567 13567 13568 13568 13584 13584 13616 13616 13816 13816 13818 13818 14000 14000 14031 14031 14224 14224 14289 14289 14396 14396 14443 14443 14496 -2 -3 -6 -10 -11 -12 -14 -17 -20 14496 -2 -2 -3 -3 -6 -6 -10 -10 -11 -11 -12 -12 -14 -14 -17 -17 -20 -20 14617 14617 14644 14644 14649 14649 14651 14651 14698 14698 14764 14764 14882 14882 14971 14971 15022 15022 15189 15189 15288 15288 15291 15291 15398 15398 15408 15408 15444 -3 -9 15444 -3 -3 -9 -9 15445 15445 15438 15438 15504 15504 15511 15511 15686 15686 15693 15693 15706 -2 15706 -2 -2 15707 15707 15897 15897 15919 15919 15924 15924 15926 15926 15926 WIP 15926 WIP 15930 15930 15938 15938 16023 16023 16262 16262 16355-1 16355-1 16485 16485 16612-2 16612-2 16750 16750 16949 (TS) 16949 (TS) 17024 17024 17025 17025 17100 17100 17203 17203 17369 17369 17442 17442 17506 17506 17799 17799 18004 18004 18014 18014 18181 18181 18245 18245 18629 18629 18916 18916 19005 19005 19011 19011 19092 -1 -2 19092 -1 -1 -2 -2 19114 19114 19115 19115 19125 19125 19136 19136 19407 19407 19439 19439 19500 19500 19501 19501 19502 19502 19503 19503 19505 19505 19506 19506 19507 19507 19508 19508 19509 19509 19510 19510 19600 19600 19752 19752 19757 19757 19770 19770 19775-1 19775-1 19794-5 19794-5 19831 19831 20000 20000 20022 20022 20121 20121 20400 20400 20802 20802 20830 20830 21000 21000 21001 21001 21047 21047 21122 21122 21500 21500 21827 21827 22000 22000 22275 22275 22300 22300 22301 22301 22395 22395 22537 22537 23000 23000 23003 23003 23008 23008 23009 23009 23090-3 23090-3 23092 23092 23094-1 23094-1 23094-2 23094-2 23270 23270 23271 23271 23360 23360 23941 23941 24517 24517 24613 24613 24617 24617 24707 24707 24728 24728 25178 25178 25964 25964 26000 26000 26262 26262 26300 26300 26324 26324 27000 series 27000 series 27000 27000 27001 27001 27002 27002 27005 27005 27006 27006 27729 27729 28000 28000 29110 29148 29148 29199-2 29199-2 29500 29500 30170 30170 31000 31000 32000 32000 37001 37001 38500 38500 39075 39075 40500 40500 42010 42010 45001 45001 50001 50001 55000 55000 56000 56000 80000 80000 Category Category Authority control databases Germany Germany Germany United States United States United States France France France BnF data BnF data BnF data Czech Republic Czech Republic Czech Republic Czech Republic Spain Spain Spain Israel Israel Israel IdRef IdRef IdRef https://en.wikipedia.org/w/index.php?title=Ruby_(programming_language)&oldid=1256598228 Categories Ruby (programming language) Ruby (programming language) Class-based programming languages Class-based programming languages Dynamic programming languages Dynamic programming languages Dynamically typed programming languages Dynamically typed programming languages Free and open source interpreters Free and open source interpreters Functional languages Functional languages Free software programmed in C Free software programmed in C ISO standards ISO standards Japanese inventions Japanese inventions Multi-paradigm programming languages Multi-paradigm programming languages Object-oriented programming languages Object-oriented programming languages Programming languages created in 1995 Programming languages created in 1995 Programming languages with an ISO standard Programming languages with an ISO standard Scripting languages Scripting languages Software using the BSD license Software using the BSD license Text-oriented programming languages Text-oriented programming languages Webarchive template wayback links Webarchive template wayback links CS1 maint: multiple names: authors list CS1 maint: multiple names: authors list CS1 maint: bot: original URL status unknown CS1 maint: bot: original URL status unknown Articles with short description Articles with short description Short description matches Wikidata Short description matches Wikidata Wikipedia articles that are too technical from June 2022 Wikipedia articles that are too technical from June 2022 All articles that are too technical All articles that are too technical Articles containing potentially dated statements from 2018 Articles containing potentially dated statements from 2018 All articles containing potentially dated statements All articles containing potentially dated statements Commons category link is on Wikidata Commons category link is on Wikidata Official website different in Wikidata and Wikipedia Official website different in Wikidata and Wikipedia Articles with example Ruby code Articles with example Ruby code This page was last edited on 10 November 2024, at 18:35 (UTC). (UTC) Text is available under the Creative Commons Attribution-ShareAlike 4.0 License; additional terms may apply. By using this site, you agree to the Terms of Use and Privacy Policy. Wikipedia\u00ae is a registered trademark of the Wikimedia Foundation, Inc., a non-profit organization. Creative Commons Attribution-ShareAlike 4.0 License Terms of Use Privacy Policy Wikimedia Foundation, Inc. Privacy policy Privacy policy About Wikipedia About Wikipedia Disclaimers Disclaimers Contact Wikipedia Contact Wikipedia Code of Conduct Code of Conduct Developers Developers Statistics Statistics Cookie statement Cookie statement Mobile view Mobile view",
    "https://en.wikipedia.org/wiki/HTML": "Jump to content Main menu Main page Main page Main page Contents Contents Contents Current events Current events Current events Random article Random article Random article About Wikipedia About Wikipedia About Wikipedia Contact us Contact us Contact us Help Help Help Learn to edit Learn to edit Learn to edit Community portal Community portal Community portal Recent changes Recent changes Recent changes Upload file Upload file Upload file Search Search Appearance Donate Donate Donate Create account Create account Create account Log in Log in Log in Personal tools Donate Donate Donate Create account Create account Create account Log in Log in Log in learn more learn more Contributions Contributions Contributions Talk Talk Talk Contents (Top) (Top) 1 History Toggle History subsection 1.1 Development 1.2 HTML version timeline 1.2.1 HTML 2 1.2.2 HTML 3 1.2.3 HTML 4 1.2.4 HTML 5 1.3 HTML draft version timeline 1.3.1 XHTML versions 1.4 Transition of HTML publication to WHATWG 1 History 1 History Toggle History subsection 1.1 Development 1.1 Development 1.1 Development 1.2 HTML version timeline 1.2.1 HTML 2 1.2.2 HTML 3 1.2.3 HTML 4 1.2.4 HTML 5 1.2 HTML version timeline 1.2 HTML version timeline 1.2.1 HTML 2 1.2.1 HTML 2 1.2.1 HTML 2 1.2.2 HTML 3 1.2.2 HTML 3 1.2.2 HTML 3 1.2.3 HTML 4 1.2.3 HTML 4 1.2.3 HTML 4 1.2.4 HTML 5 1.2.4 HTML 5 1.2.4 HTML 5 1.3 HTML draft version timeline 1.3.1 XHTML versions 1.3 HTML draft version timeline 1.3 HTML draft version timeline 1.3.1 XHTML versions 1.3.1 XHTML versions 1.3.1 XHTML versions 1.4 Transition of HTML publication to WHATWG 1.4 Transition of HTML publication to WHATWG 1.4 Transition of HTML publication to WHATWG 2 Markup Toggle Markup subsection 2.1 Elements 2.1.1 Element examples 2.1.1.1 Headings 2.1.1.2 Line breaks 2.1.1.3 Links 2.1.1.4 Inputs 2.1.2 Attributes 2.2 Character and entity references 2.3 Data types 2.4 Document type declaration 2 Markup 2 Markup Toggle Markup subsection 2.1 Elements 2.1.1 Element examples 2.1.1.1 Headings 2.1.1.2 Line breaks 2.1.1.3 Links 2.1.1.4 Inputs 2.1.2 Attributes 2.1 Elements 2.1 Elements 2.1.1 Element examples 2.1.1.1 Headings 2.1.1.2 Line breaks 2.1.1.3 Links 2.1.1.4 Inputs 2.1.1 Element examples 2.1.1 Element examples 2.1.1.1 Headings 2.1.1.1 Headings 2.1.1.1 Headings 2.1.1.2 Line breaks 2.1.1.2 Line breaks 2.1.1.2 Line breaks 2.1.1.3 Links 2.1.1.3 Links 2.1.1.3 Links 2.1.1.4 Inputs 2.1.1.4 Inputs 2.1.1.4 Inputs 2.1.2 Attributes 2.1.2 Attributes 2.1.2 Attributes 2.2 Character and entity references 2.2 Character and entity references 2.2 Character and entity references 2.3 Data types 2.3 Data types 2.3 Data types 2.4 Document type declaration 2.4 Document type declaration 2.4 Document type declaration 3 Semantic HTML 3 Semantic HTML 3 Semantic HTML 4 Delivery Toggle Delivery subsection 4.1 HTTP 4.2 HTML e-mail 4.3 Naming conventions 4.4 HTML Application 4 Delivery 4 Delivery Toggle Delivery subsection 4.1 HTTP 4.1 HTTP 4.1 HTTP 4.2 HTML e-mail 4.2 HTML e-mail 4.2 HTML e-mail 4.3 Naming conventions 4.3 Naming conventions 4.3 Naming conventions 4.4 HTML Application 4.4 HTML Application 4.4 HTML Application 5 HTML4 variations Toggle HTML4 variations subsection 5.1 SGML-based versus XML-based HTML 5.2 Transitional versus strict 5.3 Frameset versus transitional 5.4 Summary of specification versions 5 HTML4 variations 5 HTML4 variations Toggle HTML4 variations subsection 5.1 SGML-based versus XML-based HTML 5.1 SGML-based versus XML-based HTML 5.1 SGML-based versus XML-based HTML 5.2 Transitional versus strict 5.2 Transitional versus strict 5.2 Transitional versus strict 5.3 Frameset versus transitional 5.3 Frameset versus transitional 5.3 Frameset versus transitional 5.4 Summary of specification versions 5.4 Summary of specification versions 5.4 Summary of specification versions 6 WHATWG HTML versus HTML5 6 WHATWG HTML versus HTML5 6 WHATWG HTML versus HTML5 7 WYSIWYG editors 7 WYSIWYG editors 7 WYSIWYG editors 8 See also 8 See also 8 See also 9 Notes 9 Notes 9 Notes 10 References 10 References 10 References 11 External links 11 External links 11 External links Toggle the table of contents HTML HTML 135 languages Afrikaans Afrikaans Afrikaans Alemannisch Alemannisch Alemannisch \u00c6nglisc \u00c6nglisc \u00c6nglisc \u0627\u0644\u0639\u0631\u0628\u064a\u0629 \u0627\u0644\u0639\u0631\u0628\u064a\u0629 \u0627\u0644\u0639\u0631\u0628\u064a\u0629 Aragon\u00e9s Aragon\u00e9s Aragon\u00e9s \u0985\u09b8\u09ae\u09c0\u09af\u09bc\u09be \u0985\u09b8\u09ae\u09c0\u09af\u09bc\u09be \u0985\u09b8\u09ae\u09c0\u09af\u09bc\u09be Asturianu Asturianu Asturianu Az\u0259rbaycanca Az\u0259rbaycanca Az\u0259rbaycanca \u062a\u06c6\u0631\u06a9\u062c\u0647 \u062a\u06c6\u0631\u06a9\u062c\u0647 \u062a\u06c6\u0631\u06a9\u062c\u0647 Basa Bali Basa Bali Basa Bali \u09ac\u09be\u0982\u09b2\u09be \u09ac\u09be\u0982\u09b2\u09be \u09ac\u09be\u0982\u09b2\u09be \u95a9\u5357\u8a9e / B\u00e2n-l\u00e2m-g\u00fa \u95a9\u5357\u8a9e / B\u00e2n-l\u00e2m-g\u00fa \u95a9\u5357\u8a9e / B\u00e2n-l\u00e2m-g\u00fa \u0411\u0430\u0448\u04a1\u043e\u0440\u0442\u0441\u0430 \u0411\u0430\u0448\u04a1\u043e\u0440\u0442\u0441\u0430 \u0411\u0430\u0448\u04a1\u043e\u0440\u0442\u0441\u0430 \u0411\u0435\u043b\u0430\u0440\u0443\u0441\u043a\u0430\u044f \u0411\u0435\u043b\u0430\u0440\u0443\u0441\u043a\u0430\u044f \u0411\u0435\u043b\u0430\u0440\u0443\u0441\u043a\u0430\u044f \u0411\u0435\u043b\u0430\u0440\u0443\u0441\u043a\u0430\u044f (\u0442\u0430\u0440\u0430\u0448\u043a\u0435\u0432\u0456\u0446\u0430) \u0411\u0435\u043b\u0430\u0440\u0443\u0441\u043a\u0430\u044f (\u0442\u0430\u0440\u0430\u0448\u043a\u0435\u0432\u0456\u0446\u0430) \u0411\u0435\u043b\u0430\u0440\u0443\u0441\u043a\u0430\u044f (\u0442\u0430\u0440\u0430\u0448\u043a\u0435\u0432\u0456\u0446\u0430) \u0411\u044a\u043b\u0433\u0430\u0440\u0441\u043a\u0438 \u0411\u044a\u043b\u0433\u0430\u0440\u0441\u043a\u0438 \u0411\u044a\u043b\u0433\u0430\u0440\u0441\u043a\u0438 Boarisch Boarisch Boarisch Bosanski Bosanski Bosanski Brezhoneg Brezhoneg Brezhoneg Catal\u00e0 Catal\u00e0 Catal\u00e0 \u0427\u04d1\u0432\u0430\u0448\u043b\u0430 \u0427\u04d1\u0432\u0430\u0448\u043b\u0430 \u0427\u04d1\u0432\u0430\u0448\u043b\u0430 \u010ce\u0161tina \u010ce\u0161tina \u010ce\u0161tina Corsu Corsu Corsu Cymraeg Cymraeg Cymraeg Dansk Dansk Dansk Davvis\u00e1megiella Davvis\u00e1megiella Davvis\u00e1megiella Deutsch Deutsch Deutsch Dolnoserbski Dolnoserbski Dolnoserbski Eesti Eesti Eesti \u0395\u03bb\u03bb\u03b7\u03bd\u03b9\u03ba\u03ac \u0395\u03bb\u03bb\u03b7\u03bd\u03b9\u03ba\u03ac \u0395\u03bb\u03bb\u03b7\u03bd\u03b9\u03ba\u03ac Espa\u00f1ol Espa\u00f1ol Espa\u00f1ol Esperanto Esperanto Esperanto Euskara Euskara Euskara \u0641\u0627\u0631\u0633\u06cc \u0641\u0627\u0631\u0633\u06cc \u0641\u0627\u0631\u0633\u06cc Fiji Hindi Fiji Hindi Fiji Hindi F\u00f8royskt F\u00f8royskt F\u00f8royskt Fran\u00e7ais Fran\u00e7ais Fran\u00e7ais Frysk Frysk Frysk Furlan Furlan Furlan Gaeilge Gaeilge Gaeilge G\u00e0idhlig G\u00e0idhlig G\u00e0idhlig Galego Galego Galego \u06af\u06cc\u0644\u06a9\u06cc \u06af\u06cc\u0644\u06a9\u06cc \u06af\u06cc\u0644\u06a9\u06cc \u0a97\u0ac1\u0a9c\u0ab0\u0abe\u0aa4\u0ac0 \u0a97\u0ac1\u0a9c\u0ab0\u0abe\u0aa4\u0ac0 \u0a97\u0ac1\u0a9c\u0ab0\u0abe\u0aa4\u0ac0 \ud55c\uad6d\uc5b4 \ud55c\uad6d\uc5b4 \ud55c\uad6d\uc5b4 \u0540\u0561\u0575\u0565\u0580\u0565\u0576 \u0540\u0561\u0575\u0565\u0580\u0565\u0576 \u0540\u0561\u0575\u0565\u0580\u0565\u0576 \u0939\u093f\u0928\u094d\u0926\u0940 \u0939\u093f\u0928\u094d\u0926\u0940 \u0939\u093f\u0928\u094d\u0926\u0940 Hornjoserbsce Hornjoserbsce Hornjoserbsce Hrvatski Hrvatski Hrvatski Ido Ido Ido Bahasa Indonesia Bahasa Indonesia Bahasa Indonesia Interlingua Interlingua Interlingua \u00cdslenska \u00cdslenska \u00cdslenska Italiano Italiano Italiano \u05e2\u05d1\u05e8\u05d9\u05ea \u05e2\u05d1\u05e8\u05d9\u05ea \u05e2\u05d1\u05e8\u05d9\u05ea Jawa Jawa Jawa \u10e5\u10d0\u10e0\u10d7\u10e3\u10da\u10d8 \u10e5\u10d0\u10e0\u10d7\u10e3\u10da\u10d8 \u10e5\u10d0\u10e0\u10d7\u10e3\u10da\u10d8 \u049a\u0430\u0437\u0430\u049b\u0448\u0430 \u049a\u0430\u0437\u0430\u049b\u0448\u0430 \u049a\u0430\u0437\u0430\u049b\u0448\u0430 Kiswahili Kiswahili Kiswahili Kurd\u00ee Kurd\u00ee Kurd\u00ee \u041a\u044b\u0440\u0433\u044b\u0437\u0447\u0430 \u041a\u044b\u0440\u0433\u044b\u0437\u0447\u0430 \u041a\u044b\u0440\u0433\u044b\u0437\u0447\u0430 Latga\u013cu Latga\u013cu Latga\u013cu Latina Latina Latina Latvie\u0161u Latvie\u0161u Latvie\u0161u L\u00ebtzebuergesch L\u00ebtzebuergesch L\u00ebtzebuergesch Lietuvi\u0173 Lietuvi\u0173 Lietuvi\u0173 Ligure Ligure Ligure Lingua Franca Nova Lingua Franca Nova Lingua Franca Nova Lombard Lombard Lombard Magyar Magyar Magyar \u092e\u0948\u0925\u093f\u0932\u0940 \u092e\u0948\u0925\u093f\u0932\u0940 \u092e\u0948\u0925\u093f\u0932\u0940 \u041c\u0430\u043a\u0435\u0434\u043e\u043d\u0441\u043a\u0438 \u041c\u0430\u043a\u0435\u0434\u043e\u043d\u0441\u043a\u0438 \u041c\u0430\u043a\u0435\u0434\u043e\u043d\u0441\u043a\u0438 Malagasy Malagasy Malagasy \u0d2e\u0d32\u0d2f\u0d3e\u0d33\u0d02 \u0d2e\u0d32\u0d2f\u0d3e\u0d33\u0d02 \u0d2e\u0d32\u0d2f\u0d3e\u0d33\u0d02 \u092e\u0930\u093e\u0920\u0940 \u092e\u0930\u093e\u0920\u0940 \u092e\u0930\u093e\u0920\u0940 \u0645\u0635\u0631\u0649 \u0645\u0635\u0631\u0649 \u0645\u0635\u0631\u0649 Bahasa Melayu Bahasa Melayu Bahasa Melayu Minangkabau Minangkabau Minangkabau \u95a9\u6771\u8a9e / M\u00ecng-d\u0115\u0324ng-ng\u1e73\u0304 \u95a9\u6771\u8a9e / M\u00ecng-d\u0115\u0324ng-ng\u1e73\u0304 \u95a9\u6771\u8a9e / M\u00ecng-d\u0115\u0324ng-ng\u1e73\u0304 \u041c\u043e\u043d\u0433\u043e\u043b \u041c\u043e\u043d\u0433\u043e\u043b \u041c\u043e\u043d\u0433\u043e\u043b \u1019\u103c\u1014\u103a\u1019\u102c\u1018\u102c\u101e\u102c \u1019\u103c\u1014\u103a\u1019\u102c\u1018\u102c\u101e\u102c \u1019\u103c\u1014\u103a\u1019\u102c\u1018\u102c\u101e\u102c Nederlands Nederlands Nederlands \u0928\u0947\u092a\u093e\u0932\u0940 \u0928\u0947\u092a\u093e\u0932\u0940 \u0928\u0947\u092a\u093e\u0932\u0940 \u0928\u0947\u092a\u093e\u0932 \u092d\u093e\u0937\u093e \u0928\u0947\u092a\u093e\u0932 \u092d\u093e\u0937\u093e \u0928\u0947\u092a\u093e\u0932 \u092d\u093e\u0937\u093e \u65e5\u672c\u8a9e \u65e5\u672c\u8a9e \u65e5\u672c\u8a9e \u041d\u043e\u0445\u0447\u0438\u0439\u043d \u041d\u043e\u0445\u0447\u0438\u0439\u043d \u041d\u043e\u0445\u0447\u0438\u0439\u043d Nordfriisk Nordfriisk Nordfriisk Norsk bokm\u00e5l Norsk bokm\u00e5l Norsk bokm\u00e5l Norsk nynorsk Norsk nynorsk Norsk nynorsk \u041e\u043b\u044b\u043a \u043c\u0430\u0440\u0438\u0439 \u041e\u043b\u044b\u043a \u043c\u0430\u0440\u0438\u0439 \u041e\u043b\u044b\u043a \u043c\u0430\u0440\u0438\u0439 O\u02bbzbekcha / \u045e\u0437\u0431\u0435\u043a\u0447\u0430 O\u02bbzbekcha / \u045e\u0437\u0431\u0435\u043a\u0447\u0430 O\u02bbzbekcha / \u045e\u0437\u0431\u0435\u043a\u0447\u0430 \u0a2a\u0a70\u0a1c\u0a3e\u0a2c\u0a40 \u0a2a\u0a70\u0a1c\u0a3e\u0a2c\u0a40 \u0a2a\u0a70\u0a1c\u0a3e\u0a2c\u0a40 \u067e\u0646\u062c\u0627\u0628\u06cc \u067e\u0646\u062c\u0627\u0628\u06cc \u067e\u0646\u062c\u0627\u0628\u06cc \u1015\u1021\u102d\u102f\u101d\u103a\u108f\u1018\u102c\u108f\u101e\u102c\u108f \u1015\u1021\u102d\u102f\u101d\u103a\u108f\u1018\u102c\u108f\u101e\u102c\u108f \u1015\u1021\u102d\u102f\u101d\u103a\u108f\u1018\u102c\u108f\u101e\u102c\u108f \u1797\u17b6\u179f\u17b6\u1781\u17d2\u1798\u17c2\u179a \u1797\u17b6\u179f\u17b6\u1781\u17d2\u1798\u17c2\u179a \u1797\u17b6\u179f\u17b6\u1781\u17d2\u1798\u17c2\u179a Polski Polski Polski Portugu\u00eas Portugu\u00eas Portugu\u00eas Qaraqalpaqsha Qaraqalpaqsha Qaraqalpaqsha Rom\u00e2n\u0103 Rom\u00e2n\u0103 Rom\u00e2n\u0103 Runa Simi Runa Simi Runa Simi \u0420\u0443\u0441\u0438\u043d\u044c\u0441\u043a\u044b\u0439 \u0420\u0443\u0441\u0438\u043d\u044c\u0441\u043a\u044b\u0439 \u0420\u0443\u0441\u0438\u043d\u044c\u0441\u043a\u044b\u0439 \u0420\u0443\u0441\u0441\u043a\u0438\u0439 \u0420\u0443\u0441\u0441\u043a\u0438\u0439 \u0420\u0443\u0441\u0441\u043a\u0438\u0439 Scots Scots Scots Shqip Shqip Shqip \u0dc3\u0dd2\u0d82\u0dc4\u0dbd \u0dc3\u0dd2\u0d82\u0dc4\u0dbd \u0dc3\u0dd2\u0d82\u0dc4\u0dbd Simple English Simple English Simple English \u0633\u0646\u068c\u064a \u0633\u0646\u068c\u064a \u0633\u0646\u068c\u064a Sloven\u010dina Sloven\u010dina Sloven\u010dina Sloven\u0161\u010dina Sloven\u0161\u010dina Sloven\u0161\u010dina Soomaaliga Soomaaliga Soomaaliga \u06a9\u0648\u0631\u062f\u06cc \u06a9\u0648\u0631\u062f\u06cc \u06a9\u0648\u0631\u062f\u06cc \u0421\u0440\u043f\u0441\u043a\u0438 / srpski \u0421\u0440\u043f\u0441\u043a\u0438 / srpski \u0421\u0440\u043f\u0441\u043a\u0438 / srpski Srpskohrvatski / \u0441\u0440\u043f\u0441\u043a\u043e\u0445\u0440\u0432\u0430\u0442\u0441\u043a\u0438 Srpskohrvatski / \u0441\u0440\u043f\u0441\u043a\u043e\u0445\u0440\u0432\u0430\u0442\u0441\u043a\u0438 Srpskohrvatski / \u0441\u0440\u043f\u0441\u043a\u043e\u0445\u0440\u0432\u0430\u0442\u0441\u043a\u0438 Suomi Suomi Suomi Svenska Svenska Svenska Tagalog Tagalog Tagalog \u0ba4\u0bae\u0bbf\u0bb4\u0bcd \u0ba4\u0bae\u0bbf\u0bb4\u0bcd \u0ba4\u0bae\u0bbf\u0bb4\u0bcd \u107d\u1083\u1087\u101e\u1083\u1087\u1010\u1086\u1038 \u107d\u1083\u1087\u101e\u1083\u1087\u1010\u1086\u1038 \u107d\u1083\u1087\u101e\u1083\u1087\u1010\u1086\u1038 \u0c24\u0c46\u0c32\u0c41\u0c17\u0c41 \u0c24\u0c46\u0c32\u0c41\u0c17\u0c41 \u0c24\u0c46\u0c32\u0c41\u0c17\u0c41 \u0e44\u0e17\u0e22 \u0e44\u0e17\u0e22 \u0e44\u0e17\u0e22 \u0422\u043e\u04b7\u0438\u043a\u04e3 \u0422\u043e\u04b7\u0438\u043a\u04e3 \u0422\u043e\u04b7\u0438\u043a\u04e3 T\u00fcrk\u00e7e T\u00fcrk\u00e7e T\u00fcrk\u00e7e T\u00fcrkmen\u00e7e T\u00fcrkmen\u00e7e T\u00fcrkmen\u00e7e \u0423\u043a\u0440\u0430\u0457\u043d\u0441\u044c\u043a\u0430 \u0423\u043a\u0440\u0430\u0457\u043d\u0441\u044c\u043a\u0430 \u0423\u043a\u0440\u0430\u0457\u043d\u0441\u044c\u043a\u0430 \u0627\u0631\u062f\u0648 \u0627\u0631\u062f\u0648 \u0627\u0631\u062f\u0648 V\u00e8neto V\u00e8neto V\u00e8neto Ti\u1ebfng Vi\u1ec7t Ti\u1ebfng Vi\u1ec7t Ti\u1ebfng Vi\u1ec7t Winaray Winaray Winaray Wolof Wolof Wolof \u5434\u8bed \u5434\u8bed \u5434\u8bed \u05d9\u05d9\u05b4\u05d3\u05d9\u05e9 \u05d9\u05d9\u05b4\u05d3\u05d9\u05e9 \u05d9\u05d9\u05b4\u05d3\u05d9\u05e9 Yor\u00f9b\u00e1 Yor\u00f9b\u00e1 Yor\u00f9b\u00e1 \u7cb5\u8a9e \u7cb5\u8a9e \u7cb5\u8a9e \u017demait\u0117\u0161ka \u017demait\u0117\u0161ka \u017demait\u0117\u0161ka \u4e2d\u6587 \u4e2d\u6587 \u4e2d\u6587 Edit links Edit links Article Article Article Talk Talk Talk English Read Read Read View source View source View source View history View history View history Tools Read Read Read View source View source View source View history View history View history What links here What links here What links here Related changes Related changes Related changes Upload file Upload file Upload file Special pages Special pages Special pages Permanent link Permanent link Permanent link Page information Page information Page information Cite this page Cite this page Cite this page Get shortened URL Get shortened URL Get shortened URL Download QR code Download QR code Download QR code Download as PDF Download as PDF Download as PDF Printable version Printable version Printable version Wikimedia Commons Wikimedia Commons Wikimedia Commons MediaWiki MediaWiki MediaWiki Wikibooks Wikibooks Wikibooks Wikiversity Wikiversity Wikiversity Wikidata item Wikidata item Wikidata item HTM HTML5 [1] [ ] Filename extension .html .htm Internet media type Type code Uniform Type Identifier (UTI) WHATWG WHATWG World Wide Web Consortium (W3C; formerly) World Wide Web Consortium ; 32 years ago (1993) 1993 Latest release Living Standard Document file format Container for HTML elements Web browser SGML XHTML Open format? Open format html.spec.whatwg.org html.spec.whatwg.org HTML Dynamic HTML Dynamic HTML HTML5 article audio canvas video HTML5 article article audio audio canvas canvas video video XHTML Basic Mobile Profile XHTML Basic Basic Mobile Profile Mobile Profile HTML element meta div and span blink marquee HTML element meta meta div and span div and span blink blink marquee marquee HTML attribute alt attribute HTML attribute alt attribute alt attribute HTML frame HTML frame HTML editor HTML editor Character encodings named characters Unicode Character encodings named characters named characters Unicode Unicode Language code Language code Document Object Model Document Object Model Browser Object Model Browser Object Model Style sheets CSS Style sheets CSS CSS Font family Font family Web colors Web colors JavaScript WebCL HTMX JavaScript WebCL WebCL HTMX HTMX Web3D WebGL WebGPU WebXR Web3D WebGL WebGL WebGPU WebGPU WebXR WebXR W3C Validator W3C Validator Validator WHATWG WHATWG Quirks mode Quirks mode Web storage Web storage Rendering engine Rendering engine Document markup languages Document markup languages Comparison of browser engines Comparison of browser engines v v t t e e Hypertext Markup Language (HTML) is the standard markup language[a] for documents designed to be displayed in a web browser. It defines the content and structure of web content. It is often assisted by technologies such as Cascading Style Sheets (CSS) and scripting languages such as JavaScript, a programming language. markup language [a] [ ] web browser web content Cascading Style Sheets scripting languages JavaScript Web browsers receive HTML documents from a web server or from local storage and render the documents into multimedia web pages. HTML describes the structure of a web page semantically and originally included cues for its appearance. Web browsers web server render web page semantically HTML elements are the building blocks of HTML pages. With HTML constructs, images and other objects such as interactive forms may be embedded into the rendered page. HTML provides a means to create structured documents by denoting structural semantics for text such as headings, paragraphs, lists, links, quotes, and other items. HTML elements are delineated by tags, written using angle brackets. Tags such as <img> and <input> directly introduce content into the page. Other tags such as <p> and </p> surround and provide information about document text and may include sub-element tags. Browsers do not display the HTML tags but use them to interpret the content of the page. HTML elements images interactive forms structured documents semantics links angle brackets < img > < input > < p > </ p > Browsers HTML can embed programs written in a scripting language such as JavaScript, which affects the behavior and content of web pages. The inclusion of CSS defines the look and layout of content. The World Wide Web Consortium (W3C), former maintainer of the HTML and current maintainer of the CSS standards, has encouraged the use of CSS over explicit presentational HTML since 1997.[update][3] A form of HTML, known as HTML5, is used to display video and audio, primarily using the <canvas> element, together with JavaScript. scripting language JavaScript World Wide Web Consortium CSS [update] [3] [ ] HTML5 < canvas > History Development Tim Berners-Lee In 1980, physicist Tim Berners-Lee, a contractor at CERN, proposed and prototyped ENQUIRE, a system for CERN researchers to use and share documents. In 1989, Berners-Lee wrote a memo proposing an Internet-based hypertext system.[4] Berners-Lee specified HTML and wrote the browser and server software in late 1990. That year, Berners-Lee and CERN data systems engineer Robert Cailliau collaborated on a joint request for funding, but the project was not formally adopted by CERN. In his personal notes of 1990, Berners-Lee listed \"some of the many areas in which hypertext is used\"; an encyclopedia is the first entry.[5] physicist Tim Berners-Lee CERN ENQUIRE Internet hypertext [4] [ ] data systems Robert Cailliau encyclopedia [5] [ ] The first publicly available description of HTML was a document called \"HTML Tags\",[6] first mentioned on the Internet by Tim Berners-Lee in late 1991.[7][8] It describes 18 elements comprising the initial, relatively simple design of HTML. Except for the hyperlink tag, these were strongly influenced by SGMLguid, an in-house Standard Generalized Markup Language (SGML)-based documentation format at CERN. Eleven of these elements still exist in HTML 4.[9] [6] [ ] [7] [ ] [8] [ ] SGMLguid Standard Generalized Markup Language [9] [ ] HTML is a markup language that web browsers use to interpret and compose text, images, and other material into visible or audible web pages. Default characteristics for every item of HTML markup are defined in the browser, and these characteristics can be altered or enhanced by the web page designer's additional use of CSS. Many of the text elements are mentioned in the 1988 ISO technical report TR 9537 Techniques for using SGML, which describes the features of early text formatting languages such as that used by the RUNOFF command developed in the early 1960s for the CTSS (Compatible Time-Sharing System) operating system. These formatting commands were derived from the commands used by typesetters to manually format documents. However, the SGML concept of generalized markup is based on elements (nested annotated ranges with attributes) rather than merely print effects, with separate structure and markup. HTML has been progressively moved in this direction with CSS. markup language web browsers compose CSS RUNOFF command CTSS Berners-Lee considered HTML to be an application of SGML. It was formally defined as such by the Internet Engineering Task Force (IETF) with the mid-1993 publication of the first proposal for an HTML specification, the \"Hypertext Markup Language (HTML)\" Internet Draft by Berners-Lee and Dan Connolly, which included an SGML Document type definition to define the syntax.[10][11] The draft expired after six months, but was notable for its acknowledgment of the NCSA Mosaic browser's custom tag for embedding in-line images, reflecting the IETF's philosophy of basing standards on successful prototypes. Similarly, Dave Raggett's competing Internet Draft, \"HTML+ (Hypertext Markup Format)\", from late 1993, suggested standardizing already-implemented features like tables and fill-out forms.[12] Internet Engineering Task Force Dan Connolly Document type definition [10] [ ] [11] [ ] NCSA Mosaic Dave Raggett [12] [ ] After the HTML and HTML+ drafts expired in early 1994, the IETF created an HTML Working Group. In 1995, this working group completed \"HTML 2.0\", the first HTML specification intended to be treated as a standard against which future implementations should be based.[13] [13] [ ] Further development under the auspices of the IETF was stalled by competing interests. Since 1996,[update] the HTML specifications have been maintained, with input from commercial software vendors, by the World Wide Web Consortium (W3C).[14] In 2000, HTML became an international standard (ISO/IEC 15445:2000). HTML 4.01 was published in late 1999, with further errata published through 2001. In 2004, development began on HTML5 in the Web Hypertext Application Technology Working Group (WHATWG), which became a joint deliverable with the W3C in 2008, and was completed and standardized on 28 October 2014.[15] [update] World Wide Web Consortium [14] [ ] ISO IEC Web Hypertext Application Technology Working Group [15] [ ] HTML version timeline HTML 2 RFC 1866 RFCs November 25, 1995: RFC 1867 (form-based file upload) RFC 1867 May 1996: RFC 1942 (tables) RFC 1942 August 1996: RFC 1980 (client-side image maps) RFC 1980 January 1997: RFC 2070 (internationalization) RFC 2070 internationalization HTML 3 [16] [ ] W3C Recommendation [17] [ ] [18] [ ] Netscape blink element Microsoft marquee element [14] [ ] MathML HTML 4 [19] [ ] Strict, in which deprecated elements are forbidden Transitional, in which deprecated elements are allowed Frameset, in which mostly only frame related elements are allowed. frame Initially code-named \"Cougar\",[18] HTML 4.0 adopted many browser-specific element types and attributes, but also sought to phase out Netscape's visual markup features by marking them as deprecated in favor of style sheets. HTML 4 is an SGML application conforming to ISO 8879 \u2013 SGML.[20] [18] [ ] deprecated [20] [ ] [21] [ ] [22] [ ] [23] [ ] [24] [ ] ISO [25] [ ] ISO/IEC JTC 1/SC 34 [24] [ ] HTML 5 HTML5 [26] [ ] [27] [ ] [28] [ ] [29] [ ] [30] [ ] [31] [ ] [32] [ ] [33] [ ] HTML draft version timeline [7] [ ] [34] [ ] [35] [ ] [36] [ ] [37] [ ] [37] [ ] [38] [ ] IETF [39] [ ] HTML+ [40] [ ] [41] [ ] [42] [ ] RFC 1866 [43] [ ] [44] [ ] [45] [ ] [45] [ ] Arena browser test bed [46] [ ] [47] [ ] [48] [ ] [14] [ ] [14] [ ] browser wars [14] [ ] [14] [ ] HTML5 Working Draft [49] [ ] SGML HTML5 [50] [ ] [51] [ ] [52] [ ] W3C [53] [ ] [54] [ ] W3C Recommendation [55] [ ] [56] [ ] [57] [ ] [58] [ ] XHTML versions XHTML XHTML is a separate language that began as a reformulation of HTML 4.01 using XML 1.0. It is now referred to as the XML syntax for HTML and is no longer being developed as a separate standard.[59] XML [59] [ ] XHTML 1.0 was published as a W3C Recommendation on January 26, 2000,[60] and was later revised and republished on August 1, 2002. It offers the same three variations as HTML 4.0 and 4.01, reformulated in XML, with minor restrictions. [60] [ ] XHTML 1.1[61] was published as a W3C Recommendation on May 31, 2001. It is based on XHTML 1.0 Strict, but includes minor changes, can be customized, and is reformulated using modules in the W3C recommendation \"Modularization of XHTML\", which was published on April 10, 2001.[62] [61] [ ] [62] [ ] XHTML 2.0 was a working draft. Work on it was abandoned in 2009 in favor of work on HTML5 and XHTML5.[63][64][65] XHTML 2.0 was incompatible with XHTML 1.x and, therefore, would be more accurately characterized as an XHTML-inspired new language than an update to XHTML 1.x. HTML5 XHTML5 [63] [ ] [64] [ ] [65] [ ] Transition of HTML publication to WHATWG HTML5 \u00a7 W3C and WHATWG conflict On 28 May 2019, the W3C announced that WHATWG would be the sole publisher of the HTML and DOM standards.[66][67][68][69] The W3C and WHATWG had been publishing competing standards since 2012. While the W3C standard was identical to the WHATWG in 2007 the standards have since progressively diverged due to different design decisions.[70] The WHATWG \"Living Standard\" had been the de facto web standard for some time.[71] [66] [ ] [67] [ ] [68] [ ] [69] [ ] [70] [ ] [71] [ ] Markup HTML markup consists of several key components, including those called tags (and their attributes), character-based data types, character references and entity references. HTML tags most commonly come in pairs like <h1> and </h1>, although some represent empty elements and so are unpaired, for example <img>. The first tag in such a pair is the start tag, and the second is the end tag (they are also called opening tags and closing tags). < h1 > </ h1 > < img > Another important component is the HTML document type declaration, which triggers standards mode rendering. document type declaration standards mode The following is an example of the classic \"Hello, World!\" program: \"Hello, World!\" program <!DOCTYPE html> < html > < head > < title > </ title > </ head > < body > < div > < p > </ p > </ div > </ body > </ html > The text between <html> and </html> describes the web page, and the text between <body> and </body> is the visible page content. The markup text <title>This is a title</title> defines the browser page title shown on browser tabs and window titles and the tag <div> defines a division of the page used for easy styling. Between <head> and </head>, a <meta> element can be used to define webpage metadata. < html > </ html > < body > </ body > < title > </ title > browser tabs window < div > < head > </ head > < meta > The Document Type Declaration <!DOCTYPE html> is for HTML5. If a declaration is not included, various browsers will revert to \"quirks mode\" for rendering.[72] <!DOCTYPE html> quirks mode [72] [ ] Elements HTML element HTML documents imply a structure of nested HTML elements. These are indicated in the document by HTML tags, enclosed in angle brackets thus: <p>.[73][better source needed] HTML elements < p > [73] [ ] better source needed better source needed In the simple, general case, the extent of an element is indicated by a pair of tags: a \"start tag\" <p> and \"end tag\" </p>. The text content of the element, if any, is placed between these tags. < p > </ p > Tags may also enclose further tag markup between the start and end, including a mixture of tags and text. This indicates further (nested) elements, as children of the parent element. The start tag may also include the element's attributes within the tag. These indicate other information, such as identifiers for sections within the document, identifiers used to bind style information to the presentation of the document, and for some tags such as the <img> used to embed images, the reference to the image resource in the format like this: <img src=\"example.com/example.jpg\"> < img > < img src = \"example.com/example.jpg\" > Some elements, such as the line break <br /> do not permit any embedded content, either text or further tags. These require only a single empty tag (akin to a start tag) and do not use an end tag. line break < br /> Many tags, particularly the closing end tag for the very commonly used paragraph element <p>, are optional. An HTML browser or other agent can infer the closure for the end of an element from the context and the structural rules defined by the HTML standard. These rules are complex and not widely understood by most HTML authors. < p > The general form of an HTML element is therefore: <tag attribute1=\"value1\" attribute2=\"value2\">''content''</tag>. Some HTML elements are defined as empty elements and take the form <tag attribute1=\"value1\" attribute2=\"value2\">. Empty elements may enclose no content, for instance, the <br /> tag or the inline <img> tag. The name of an HTML element is the name used in the tags. The end tag's name is preceded by a slash character, /, and that in empty elements the end tag is neither required nor allowed. If attributes are not mentioned, default values are used in each case. < tag attribute1 = \"value1\" attribute2 = \"value2\" > </ tag > < tag attribute1 = \"value1\" attribute2 = \"value2\" > < br /> < img > Element examples HTML element Header of the HTML document: <head>...</head>. The title is included in the head, for example: < head > </ head > < head > < title > </ title > < link rel = \"stylesheet\" href = \"stylebyjimbowales.css\" > <!-- Imports Stylesheets --> </ head > Headings HTML headings are defined with the <h1> to <h6> tags with H1 being the highest (or most important) level and H6 the least: < h1 > < h6 > < h1 > </ h1 > < h2 > </ h2 > < h3 > </ h3 > < h4 > </ h4 > < h5 > </ h5 > < h6 > </ h6 > The effects are: CSS can substantially change the rendering. Paragraphs: < p > </ p > < p > </ p > Line breaks <br />. The difference between <br /> and <p> is that <br /> breaks a line without altering the semantic structure of the page, whereas <p> sections the page into paragraphs. The element <br /> is an empty element in that, although it may have attributes, it can take no content and it may not have an end tag. < br /> < br /> < p > < br /> breaks a line < p > paragraphs < br /> < p > < br > < br > < br > </ p > Links This is a link in HTML. To create a link the <a> tag is used. The href attribute holds the URL address of the link. < a > URL < a href = \"https://www.wikipedia.org/\" > </ a > Inputs There are many possible ways a user can give inputs like: < input type = \"text\" > <!-- This is for text input --> < input type = \"file\" > <!-- This is for uploading files --> < input type = \"checkbox\" > <!-- This is for checkboxes --> Comments: <!-- This is a comment --> Comments can help in the understanding of the markup and do not display in the webpage. There are several types of markup elements used in HTML: < h2 > </ h2 > heading Cascading Style Sheets [74] [ ] < b > </ b > < b > </ b > < i > </ i > < strong > </ strong > < em > </ em > deprecated CSS hyperlink URL < a href = \"https://en.wikipedia.org/\" > </ a > Wikipedia Wikipedia < a href = \"https://example.org\" >< img src = \"image.gif\" alt = \"descriptive text\" width = \"50\" height = \"50\" border = \"0\" ></ a > Attributes HTML attribute Most of the attributes of an element are name\u2013value pairs, separated by = and written within the start tag of an element after the element's name. The value may be enclosed in single or double quotes, although values consisting of certain characters can be left unquoted in HTML (but not XHTML).[75][76] Leaving attribute values unquoted is considered unsafe.[77] In contrast with name-value pair attributes, there are some attributes that affect the element simply by their presence in the start tag of the element,[7] like the ismap attribute for the img element.[78] name\u2013value pairs [75] [ ] [76] [ ] [77] [ ] [7] [ ] [78] [ ] There are several common attributes that may appear in many elements : The id attribute provides a document-wide unique identifier for an element. This is used to identify the element so that stylesheets can alter its presentational properties, and scripts may alter, animate or delete its contents or presentation. Appended to the URL of the page, it provides a globally unique identifier for the element, typically a sub-section of the page. For example, the ID \"Attributes\" in https://en.wikipedia.org/wiki/HTML#Attributes. The class attribute provides a way of classifying similar elements. This can be used for semantic or presentation purposes. For example, an HTML document might semantically use the designation <class=\"notation\"> to indicate that all elements with this class value are subordinate to the main text of the document. In presentation, such elements might be gathered together and presented as footnotes on a page instead of appearing in the place where they occur in the HTML source. Class attributes are used semantically in microformats. Multiple class values may be specified; for example <class=\"notation important\"> puts the element into both the notation and the important classes. semantic < class =\" notation \" > microformats < class =\" notation important \" > An author may use the style attribute to assign presentational properties to a particular element. It is considered better practice to use an element's id or class attributes to select the element from within a stylesheet, though sometimes this can be too cumbersome for a simple, specific, or ad hoc styling. stylesheet The title attribute is used to attach a subtextual explanation to an element. In most browsers this attribute is displayed as a tooltip. browsers tooltip The lang attribute identifies the natural language of the element's contents, which may be different from that of the rest of the document. For example, in an English-language document: <p>Oh well, <span lang=\"fr\">c'est la vie</span>, as they say in France.</p> < p > < span lang = \"fr\" > </ span > </ p > The abbreviation element, abbr, can be used to demonstrate some of these attributes: < abbr id = \"anId\" class = \"jargon\" style = \"color:purple;\" title = \"Hypertext Markup Language\" > </ abbr > This example displays as HTML; in most browsers, pointing the cursor at the abbreviation should display the title text \"Hypertext Markup Language.\" Most elements take the language-related attribute dir to specify text direction, such as with \"rtl\" for right-to-left text in, for example, Arabic, Persian or Hebrew.[79] Arabic Persian Hebrew [79] [ ] Character and entity references List of XML and HTML character entity references Unicode and HTML As of version 4.0, HTML defines a set of 252 character entity references and a set of 1,114,050 numeric character references, both of which allow individual characters to be written via simple markup, rather than literally. A literal character and its markup counterpart are considered equivalent and are rendered identically. character entity references numeric character references The ability to \"escape\" characters in this way allows for the characters < and & (when written as &lt; and &amp;, respectively) to be interpreted as character data, rather than markup. For example, a literal < normally indicates the start of a tag, and & normally indicates the start of a character entity reference or numeric character reference; writing it as &amp; or &#x26; or &#38; allows & to be included in the content of an element or in the value of an attribute. The double-quote character (\"), when not used to quote an attribute value, must also be escaped as &quot; or &#x22; or &#34; when it appears within the attribute value itself. Equivalently, the single-quote character ('), when not used to quote an attribute value, must also be escaped as &#x27; or &#39; (or as &apos; in HTML5 or XHTML documents[80][81]) when it appears within the attribute value itself. If document authors overlook the need to escape such characters, some browsers can be very forgiving and try to use context to guess their intent. The result is still invalid markup, which makes the document less accessible to other browsers and to other user agents that may try to parse the document for search and indexing purposes for example. escape [80] [ ] [81] [ ] user agents search and indexing Escaping also allows for characters that are not easily typed, or that are not available in the document's character encoding, to be represented within the element and attribute content. For example, the acute-accented e (\u00e9), a character typically found only on Western European and South American keyboards, can be written in any HTML document as the entity reference &eacute; or as the numeric references &#xE9; or &#233;, using characters that are available on all keyboards and are supported in all character encodings. Unicode character encodings such as UTF-8 are compatible with all modern browsers and allow direct access to almost all the characters of the world's writing systems.[82] character encoding Unicode UTF-8 [82] [ ] Ampersand Less Than Greater Than Double Quote Single Quote Non-Breaking Space Copyright Registered Trademark Dagger Double dagger Trademark Data types HTML defines several data types for element content, such as script data and stylesheet data, and a plethora of types for attribute values, including IDs, names, URIs, numbers, units of length, languages, media descriptors, colors, character encodings, dates and times, and so on. All of these data types are specializations of character data. data types URIs Document type declaration HTML documents are required to start with a document type declaration (informally, a \"doctype\"). In browsers, the doctype helps to define the rendering mode\u2014particularly whether to use quirks mode. ocument type declaration quirks mode The original purpose of the doctype was to enable the parsing and validation of HTML documents by SGML tools based on the document type definition (DTD). The DTD to which the DOCTYPE refers contains a machine-readable grammar specifying the permitted and prohibited content for a document conforming to such a DTD. Browsers, on the other hand, do not implement HTML as an application of SGML and as consequence do not read the DTD. document type definition HTML5 does not define a DTD; therefore, in HTML5 the doctype declaration is simpler and shorter:[83] HTML5 [83] [ ] <!DOCTYPE html > An example of an HTML 4 doctype <!DOCTYPE HTML PUBLIC \"-//W3C//DTD HTML 4.01//EN\" \"https://www.w3.org/TR/html4/strict.dtd\" > This declaration references the DTD for the \"strict\" version of HTML 4.01. SGML-based validators read the DTD in order to properly parse the document and to perform validation. In modern browsers, a valid doctype activates standards mode as opposed to quirks mode. quirks mode In addition, HTML 4.01 provides Transitional and Frameset DTDs, as explained below. The transitional type is the most inclusive, incorporating current tags as well as older or \"deprecated\" tags, with the Strict DTD excluding deprecated tags. The frameset has all tags necessary to make frames on a page along with the tags included in transitional type.[84] as explained below [84] [ ] Semantic HTML Semantic HTML Semantic HTML is a way of writing HTML that emphasizes the meaning of the encoded information over its presentation (look). HTML has included semantic markup from its inception,[85] but has also included presentational markup, such as <font>, <i> and <center> tags. There are also the semantically neutral div and span tags. Since the late 1990s, when Cascading Style Sheets were beginning to work in most browsers, web authors have been encouraged to avoid the use of presentational HTML markup with a view to the separation of content and presentation.[86] [85] [ ] < font > < i > < center > div and span Cascading Style Sheets separation of content and presentation [86] [ ] In a 2001 discussion of the Semantic Web, Tim Berners-Lee and others gave examples of ways in which intelligent software \"agents\" may one day automatically crawl the web and find, filter, and correlate previously unrelated, published facts for the benefit of human users.[87] Such agents are not commonplace even now, but some of the ideas of Web 2.0, mashups and price comparison websites may be coming close. The main difference between these web application hybrids and Berners-Lee's semantic agents lies in the fact that the current aggregation and hybridization of information is usually designed by web developers, who already know the web locations and the API semantics of the specific data they wish to mash, compare and combine. Semantic Web Tim Berners-Lee [87] [ ] Web 2.0 mashups price comparison websites aggregation web developers API semantics An important type of web agent that does crawl and read web pages automatically, without prior knowledge of what it might find, is the web crawler or search-engine spider. These software agents are dependent on the semantic clarity of web pages they find as they use various techniques and algorithms to read and index millions of web pages a day and provide web users with search facilities without which the World Wide Web's usefulness would be greatly reduced. web crawler algorithms search facilities In order for search engine spiders to be able to rate the significance of pieces of text they find in HTML documents, and also for those creating mashups and other hybrids as well as for more automated agents as they are developed, the semantic structures that exist in HTML need to be widely and uniformly applied to bring out the meaning of the published text.[88] [88] [ ] Presentational markup tags are deprecated in current HTML and XHTML recommendations. The majority of presentational features from previous versions of HTML are no longer allowed as they lead to poorer accessibility, higher cost of site maintenance, and larger document sizes.[89] deprecated XHTML [89] [ ] Good semantic HTML also improves the accessibility of web documents (see also Web Content Accessibility Guidelines). For example, when a screen reader or audio browser can correctly ascertain the structure of a document, it will not waste the visually impaired user's time by reading out repeated or irrelevant information when it has been marked up correctly. accessibility Web Content Accessibility Guidelines Delivery HTML documents can be delivered by the same means as any other computer file. However, they are most often delivered either by HTTP from a web server or by email. HTTP web server email HTTP Hypertext Transfer Protocol The World Wide Web is composed primarily of HTML documents transmitted from web servers to web browsers using the Hypertext Transfer Protocol (HTTP). However, HTTP is used to serve images, sound, and other content, in addition to HTML. To allow the web browser to know how to handle each document it receives, other information is transmitted along with the document. This meta data usually includes the MIME type (e.g., text/html or application/xhtml+xml) and the character encoding (see Character encodings in HTML). World Wide Web Hypertext Transfer Protocol meta data MIME type Character encodings in HTML In modern browsers, the MIME type that is sent with the HTML document may affect how the document is initially interpreted. A document sent with the XHTML MIME type is expected to be well-formed XML; syntax errors may cause the browser to fail to render it. The same document sent with the HTML MIME type might be displayed successfully since some browsers are more lenient with HTML. well-formed The W3C recommendations state that XHTML 1.0 documents that follow guidelines set forth in the recommendation's Appendix C may be labeled with either MIME Type.[90] XHTML 1.1 also states that XHTML 1.1 documents should[91] be labeled with either MIME type.[92] [90] [ ] [91] [ ] [92] [ ] HTML e-mail HTML email Most graphical email clients allow the use of a subset of HTML (often ill-defined) to provide formatting and semantic markup not available with plain text. This may include typographic information like colored headings, emphasized and quoted text, inline images and diagrams. Many such clients include both a GUI editor for composing HTML e-mail messages and a rendering engine for displaying them. Use of HTML in e-mail is criticized by some because of compatibility issues, because it can help disguise phishing attacks, because of accessibility issues for blind or visually impaired people, because it can confuse spam filters and because the message size is larger than plain text. semantic plain text GUI phishing spam Naming conventions The most common filename extension for files containing HTML is .html. A common abbreviation of this is .htm, which originated because some early operating systems and file systems, such as DOS and the limitations imposed by FAT data structure, limited file extensions to three letters.[93] filename extension files DOS FAT three letters [93] [ ] HTML Application HTML Application An HTML Application (HTA; file extension .hta) is a Microsoft Windows application that uses HTML and Dynamic HTML in a browser to provide the application's graphical interface. A regular HTML file is confined to the security model of the web browser's security, communicating only to web servers and manipulating only web page objects and site cookies. An HTA runs as a fully trusted application and therefore has more privileges, like creation/editing/removal of files and Windows Registry entries. Because they operate outside the browser's security model, HTAs cannot be executed via HTTP, but must be downloaded (just like an EXE file) and executed from local file system. Microsoft Windows browser web browser's security site cookies Windows Registry EXE HTML4 variations Since its inception, HTML and its associated protocols gained acceptance relatively quickly. However, no clear standards existed in the early years of the language. Though its creators originally conceived of HTML as a semantic language devoid of presentation details,[94] practical uses pushed many presentational elements and attributes into the language, driven largely by the various browser vendors. The latest standards surrounding HTML reflect efforts to overcome the sometimes chaotic development of the language[95] and to create a rational foundation for building both meaningful and well-presented documents. To return HTML to its role as a semantic language, the W3C has developed style languages such as CSS and XSL to shoulder the burden of presentation. In conjunction, the HTML specification has slowly reined in the presentational elements. [94] [ ] [95] [ ] W3C CSS XSL There are two axes differentiating various variations of HTML as currently specified: SGML-based HTML versus XML-based HTML (referred to as XHTML) on one axis, and strict versus transitional (loose) versus frameset on the other axis. SGML-based versus XML-based HTML One difference in the latest[when?] HTML specifications lies in the distinction between the SGML-based specification and the XML-based specification. The XML-based specification is usually called XHTML to distinguish it clearly from the more traditional definition. However, the root element name continues to be \"html\" even in the XHTML-specified HTML. The W3C intended XHTML 1.0 to be identical to HTML 4.01 except where limitations of XML over the more complex SGML require workarounds. Because XHTML and HTML are closely related, they are sometimes documented in parallel. In such circumstances, some authors conflate the two names as (X)HTML or X(HTML). when? when? XHTML conflate the two names Like HTML 4.01, XHTML 1.0 has three sub-specifications: strict, transitional, and frameset. Aside from the different opening declarations for a document, the differences between an HTML 4.01 and XHTML 1.0 document\u2014in each of the corresponding DTDs\u2014are largely syntactic. The underlying syntax of HTML allows many shortcuts that XHTML does not, such as elements with optional opening or closing tags, and even empty elements which must not have an end tag. By contrast, XHTML requires all elements to have an opening tag and a closing tag. XHTML, however, also introduces a new shortcut: an XHTML tag may be opened and closed within the same tag, by including a slash before the end of the tag like this: <br/>. The introduction of this shorthand, which is not used in the SGML declaration for HTML 4.01, may confuse earlier software unfamiliar with this new convention. A fix for this is to include a space before closing the tag, as such: <br />.[96] < br /> < br /> [96] [ ] To understand the subtle differences between HTML and XHTML, consider the transformation of a valid and well-formed XHTML 1.0 document that adheres to Appendix C (see below) into a valid HTML 4.01 document. Making this translation requires the following steps: The language for an element should be specified with a lang attribute rather than the XHTML xml:lang attribute. XHTML uses XML's built-in language-defining functionality attribute. Remove the XML namespace (xmlns=URI). HTML has no facilities for namespaces. Change the document type declaration from XHTML 1.0 to HTML 4.01. (see DTD section for further explanation). DTD section If present, remove the XML declaration. (Typically this is: <?xml version=\"1.0\" encoding=\"utf-8\"?>). <?xml version=\"1.0\" encoding=\"utf-8\"?> Ensure that the document's MIME type is set to text/html. For both HTML and XHTML, this comes from the HTTP Content-Type header sent by the server. Change the XML empty-element syntax to an HTML style empty element (<br/> to <br />). < br /> < br /> Those are the main changes necessary to translate a document from XHTML 1.0 to HTML 4.01. To translate from HTML to XHTML would also require the addition of any omitted opening or closing tags. Whether coding in HTML or XHTML it may just be best to always include the optional tags within an HTML document rather than remembering which tags can be omitted. A well-formed XHTML document adheres to all the syntax requirements of XML. A valid document adheres to the content specification for XHTML, which describes the document structure. The W3C recommends several conventions to ensure an easy migration between HTML and XHTML (see HTML Compatibility Guidelines). The following steps can be applied to XHTML 1.0 documents only: HTML Compatibility Guidelines Include both xml:lang and lang attributes on any elements assigning language. Use the empty-element syntax only for elements specified as empty in HTML. Include an extra space in empty-element tags: for example <br /> instead of <br/>. < br /> < br /> Include explicit close tags for elements that permit content but are left empty (for example, <div></div>, not <div />). < div ></ div > < div /> Omit the XML declaration. By carefully following the W3C's compatibility guidelines, a user agent should be able to interpret the document equally as HTML or XHTML. For documents that are XHTML 1.0 and have been made compatible in this way, the W3C permits them to be served either as HTML (with a text/html MIME type), or as XHTML (with an application/xhtml+xml or application/xml MIME type). When delivered as XHTML, browsers should use an XML parser, which adheres strictly to the XML specifications for parsing the document's contents. MIME type Transitional versus strict HTML 4 defined three different versions of the language: Strict, Transitional (once called Loose), and Frameset. The Strict version is intended for new documents and is considered best practice, while the Transitional and Frameset versions were developed to make it easier to transition documents that conformed to older HTML specifications or did not conform to any specification to a version of HTML 4. The Transitional and Frameset versions allow for presentational markup, which is omitted in the Strict version. Instead, cascading style sheets are encouraged to improve the presentation of HTML documents. Because XHTML 1 only defines an XML syntax for the language defined by HTML 4, the same differences apply to XHTML 1 as well. cascading style sheets The Transitional version allows the following parts of the vocabulary, which are not included in the Strict version: A looser content model Inline elements and plain text are allowed directly in: body, blockquote, form, noscript and noframes Inline elements and plain text are allowed directly in: body, blockquote, form, noscript and noframes Presentation related elements underline (u) (Deprecated. can confuse a visitor with a hyperlink.) strike-through (s) center (Deprecated. use CSS instead.) font (Deprecated. use CSS instead.) basefont (Deprecated. use CSS instead.) underline (u) (Deprecated. can confuse a visitor with a hyperlink.) strike-through (s) center (Deprecated. use CSS instead.) font (Deprecated. use CSS instead.) basefont (Deprecated. use CSS instead.) Presentation related attributes background (Deprecated. use CSS instead.) and bgcolor (Deprecated. use CSS instead.) attributes for body (required element according to the W3C.) element. align (Deprecated. use CSS instead.) attribute on div, form, paragraph (p) and heading (h1...h6) elements align (Deprecated. use CSS instead.), noshade (Deprecated. use CSS instead.), size (Deprecated. use CSS instead.) and width (Deprecated. use CSS instead.) attributes on hr element align (Deprecated. use CSS instead.), border, vspace and hspace attributes on img and object (caution: the object element is only supported in Internet Explorer (from the major browsers)) elements align (Deprecated. use CSS instead.) attribute on legend and caption elements align (Deprecated. use CSS instead.) and bgcolor (Deprecated. use CSS instead.) on table element nowrap (Obsolete), bgcolor (Deprecated. use CSS instead.), width, height on td and th elements bgcolor (Deprecated. use CSS instead.) attribute on tr element clear (Obsolete) attribute on br element compact attribute on dl, dir and menu elements type (Deprecated. use CSS instead.), compact (Deprecated. use CSS instead.) and start (Deprecated. use CSS instead.) attributes on ol and ul elements type and value attributes on li element width attribute on pre element background (Deprecated. use CSS instead.) and bgcolor (Deprecated. use CSS instead.) attributes for body (required element according to the W3C.) element. align (Deprecated. use CSS instead.) attribute on div, form, paragraph (p) and heading (h1...h6) elements align (Deprecated. use CSS instead.), noshade (Deprecated. use CSS instead.), size (Deprecated. use CSS instead.) and width (Deprecated. use CSS instead.) attributes on hr element align (Deprecated. use CSS instead.), border, vspace and hspace attributes on img and object (caution: the object element is only supported in Internet Explorer (from the major browsers)) elements align (Deprecated. use CSS instead.) attribute on legend and caption elements align (Deprecated. use CSS instead.) and bgcolor (Deprecated. use CSS instead.) on table element nowrap (Obsolete), bgcolor (Deprecated. use CSS instead.), width, height on td and th elements bgcolor (Deprecated. use CSS instead.) attribute on tr element clear (Obsolete) attribute on br element compact attribute on dl, dir and menu elements type (Deprecated. use CSS instead.), compact (Deprecated. use CSS instead.) and start (Deprecated. use CSS instead.) attributes on ol and ul elements type and value attributes on li element width attribute on pre element Additional elements in Transitional specification menu (Deprecated. use CSS instead.) list (no substitute, though the unordered list, is recommended) dir (Deprecated. use CSS instead.) list (no substitute, though the unordered list is recommended) isindex (Deprecated.) (element requires server-side support and is typically added to documents server-side, form and input elements can be used as a substitute) applet (Deprecated. use the object element instead.) menu (Deprecated. use CSS instead.) list (no substitute, though the unordered list, is recommended) dir (Deprecated. use CSS instead.) list (no substitute, though the unordered list is recommended) isindex (Deprecated.) (element requires server-side support and is typically added to documents server-side, form and input elements can be used as a substitute) applet (Deprecated. use the object element instead.) The language (Obsolete) attribute on script element (redundant with the type attribute). Frame related entities iframe noframes target (Deprecated in the map, link and form elements.) attribute on a, client-side image-map (map), link, form and base elements iframe noframes target (Deprecated in the map, link and form elements.) attribute on a, client-side image-map (map), link, form and base elements The Frameset version includes everything in the Transitional version, as well as the frameset element (used instead of body) and the frame element. Frameset versus transitional In addition to the above transitional differences, the frameset specifications (whether XHTML 1.0 or HTML 4.01) specify a different content model, with frameset replacing body, that contains either frame elements, or optionally noframes with a body. Summary of specification versions As this list demonstrates, the loose versions of the specification are maintained for legacy support. However, contrary to popular misconceptions, the move to XHTML does not imply a removal of this legacy support. Rather the X in XML stands for extensible and the W3C is modularizing the entire specification and opens it up to independent extensions. The primary achievement in the move from XHTML 1.0 to XHTML 1.1 is the modularization of the entire specification. The strict version of HTML is deployed in XHTML 1.1 through a set of modular extensions to the base XHTML 1.1 specification. Likewise, someone looking for the loose (transitional) or frameset specifications will find similar extended XHTML 1.1 support (much of it is contained in the legacy or frame modules). Modularization also allows for separate features to develop on their own timetable. So for example, XHTML 1.1 will allow quicker migration to emerging XML standards such as MathML (a presentational and semantic math language based on XML) and XForms\u2014a new highly advanced web-form technology to replace the existing HTML forms. MathML XForms In summary, the HTML 4 specification primarily reined in all the various HTML implementations into a single clearly written specification based on SGML. XHTML 1.0, ported this specification, as is, to the new XML-defined specification. Next, XHTML 1.1 takes advantage of the extensible nature of XML and modularizes the whole specification. XHTML 2.0 was intended to be the first step in adding new features to the specification in a standards-body-based approach. WHATWG HTML versus HTML5 \u00a7 Transition of HTML Publication to WHATWG The HTML Living Standard, which is developed by WHATWG, is the official version, while W3C HTML5 is no longer separate from WHATWG. WYSIWYG editors Please expand the article by making an edit requestto include this information . Further details may exist on the talk page. by making an edit request by making an edit request by making an edit request to include this information talk page (January 2021) January 2021 There are some WYSIWYG editors (what you see is what you get), in which the user lays out everything as it is to appear in the HTML document using a graphical user interface (GUI), often similar to word processors. The editor renders the document rather than showing the code, so authors do not require extensive knowledge of HTML. WYSIWYG graphical user interface word processors The WYSIWYG editing model has been criticized,[97][98] primarily because of the low quality of the generated code; there are voices[who?] advocating a change to the WYSIWYM model (what you see is what you mean). [97] [ ] [98] [ ] who? who? WYSIWYM WYSIWYG editors remain a controversial topic because of their perceived flaws such as: Relying mainly on the layout as opposed to meaning, often using markup that does not convey the intended meaning but simply copies the layout.[99] [99] [ ] Often producing extremely verbose and redundant code that fails to make use of the cascading nature of HTML and CSS. CSS Often producing ungrammatical markup, called tag soup or semantically incorrect markup (such as <em> for italics). tag soup < em > As a great deal of the information in HTML documents is not in the layout, the model has been criticized for its \"what you see is all you get\"-nature.[100] [100] [ ] See also Breadcrumb navigation Breadcrumb navigation Cellpadding Cellpadding Comparison of HTML parsers Comparison of HTML parsers Dynamic web page Dynamic web page HTML Application HTML Application HTML character references HTML character references List of document markup languages List of document markup languages List of XML and HTML character entity references List of XML and HTML character entity references Microdata (HTML) Microdata (HTML) Microformat Microformat Polyglot markup Polyglot markup Semantic HTML Semantic HTML W3C (X)HTML Validator W3C (X)HTML Validator Web colors Web colors Notes ^ Even though HTML is executable, it is not viewed as a programming language in programming language discourse.[2] ^ ^ Even though HTML is executable, it is not viewed as a programming language in programming language discourse.[2] programming language [2] [ ] References ^ \"W3C Html\". ^ ^ \"W3C Html\". \"W3C Html\" ^ Hermans, Felienne; Schlesinger, Ari (2024-10-17). \"A Case for Feminism in Programming Language Design\". OOPSLA. ACM: 205\u2013222. doi:10.1145/3689492.3689809. ISBN 979-8-4007-1215-9. ^ ^ Hermans, Felienne; Schlesinger, Ari (2024-10-17). \"A Case for Feminism in Programming Language Design\". OOPSLA. ACM: 205\u2013222. doi:10.1145/3689492.3689809. ISBN 979-8-4007-1215-9. \"A Case for Feminism in Programming Language Design\" OOPSLA 205\u2013 doi 10.1145/3689492.3689809 ISBN 979-8-4007-1215-9 ^ \"HTML 4.0 Specification \u2014 W3C Recommendation \u2014 Conformance: requirements and recommendations\". World Wide Web Consortium. December 18, 1997. Archived from the original on July 5, 2015. Retrieved July 6, 2015. ^ ^ \"HTML 4.0 Specification \u2014 W3C Recommendation \u2014 Conformance: requirements and recommendations\". World Wide Web Consortium. December 18, 1997. Archived from the original on July 5, 2015. Retrieved July 6, 2015. \"HTML 4.0 Specification \u2014 W3C Recommendation \u2014 Conformance: requirements and recommendations\" Archived . Retrieved July 6, 2015 July 6, ^ Tim Berners-Lee, \"Information Management: A Proposal\". CERN (March 1989, May 1990). W3C. ^ ^ Tim Berners-Lee, \"Information Management: A Proposal\". CERN (March 1989, May 1990). W3C. Information Management: A Proposal ^ Berners-Lee, Tim. \"Intended Uses\". W3C. ^ ^ Berners-Lee, Tim. \"Intended Uses\". W3C. \"Intended Uses\" ^ \"Tags used in HTML\". info.cern.ch. October 1991. Retrieved 2 March 2023. ^ ^ \"Tags used in HTML\". info.cern.ch. October 1991. Retrieved 2 March 2023. \"Tags used in HTML\" . Retrieved 2 March 2023 2 March ^ a b c \"Tags used in HTML\". World Wide Web Consortium. November 3, 1992. Archived from the original on January 31, 2010. Retrieved November 16, 2008. ^ a b c a b c \"Tags used in HTML\". World Wide Web Consortium. November 3, 1992. Archived from the original on January 31, 2010. Retrieved November 16, 2008. \"Tags used in HTML\" Archived . Retrieved November 16, 2008 November 16, ^ Berners-Lee, Tim (October 29, 1991). \"Re: status. Re: X11 BROWSER for WWW\". World Wide Web Consortium. Archived from the original on May 24, 2007. Retrieved April 8, 2007. ^ ^ Berners-Lee, Tim (October 29, 1991). \"Re: status. Re: X11 BROWSER for WWW\". World Wide Web Consortium. Archived from the original on May 24, 2007. Retrieved April 8, 2007. \"Re: status. Re: X11 BROWSER for WWW\" Archived . Retrieved April 8, 2007 April 8, ^ \"Index of the HTML 4 elements\". World Wide Web Consortium. December 24, 1999. Archived from the original on May 5, 2007. Retrieved April 8, 2007. ^ ^ \"Index of the HTML 4 elements\". World Wide Web Consortium. December 24, 1999. Archived from the original on May 5, 2007. Retrieved April 8, 2007. \"Index of the HTML 4 elements\" Archived . Retrieved April 8, 2007 April 8, ^ Berners-Lee, Tim (December 9, 1991). \"Re: SGML/HTML docs, X Browser\". w3. Archived from the original on December 22, 2007. Retrieved June 16, 2007. SGML is very general. HTML is a specific application of the SGML basic syntax applied to hypertext documents with simple structure. ^ ^ Berners-Lee, Tim (December 9, 1991). \"Re: SGML/HTML docs, X Browser\". w3. Archived from the original on December 22, 2007. Retrieved June 16, 2007. SGML is very general. HTML is a specific application of the SGML basic syntax applied to hypertext documents with simple structure. \"Re: SGML/HTML docs, X Browser\" Archived . Retrieved June 16, 2007 June 16, ^ Berners-Lee, Tim; Connolly, Daniel (June 1993). \"Hypertext Markup Language (HTML): A Representation of Textual Information and MetaInformation for Retrieval and Interchange\". w3. Archived from the original on January 3, 2017. Retrieved January 4, 2017. ^ ^ Berners-Lee, Tim; Connolly, Daniel (June 1993). \"Hypertext Markup Language (HTML): A Representation of Textual Information and MetaInformation for Retrieval and Interchange\". w3. Archived from the original on January 3, 2017. Retrieved January 4, 2017. \"Hypertext Markup Language (HTML): A Representation of Textual Information and MetaInformation for Retrieval and Interchange\" Archived . Retrieved January 4, 2017 January 4, ^ Raggett, Dave. \"A Review of the HTML+ Document Format\". w3. Archived from the original on February 29, 2000. Retrieved May 22, 2020. The hypertext markup language HTML was developed as a simple non-proprietary delivery format for global hypertext. HTML+ is a set of modular extensions to HTML and has been developed in response to a growing understanding of the needs of information providers. These extensions include text flow around floating figures, fill-out forms, tables, and mathematical equations. ^ ^ Raggett, Dave. \"A Review of the HTML+ Document Format\". w3. Archived from the original on February 29, 2000. Retrieved May 22, 2020. The hypertext markup language HTML was developed as a simple non-proprietary delivery format for global hypertext. HTML+ is a set of modular extensions to HTML and has been developed in response to a growing understanding of the needs of information providers. These extensions include text flow around floating figures, fill-out forms, tables, and mathematical equations. Raggett, Dave \"A Review of the HTML+ Document Format\" Archived . Retrieved May 22, 2020 May 22, ^ Berners-Lee, Tim; Connolly, Daniel W. (November 1995). Hypertext Markup Language - 2.0. Network Working Group. doi:10.17487/RFC1866. RFC 1866. Historic. Obsoleted by RFC 2854. This document thus defines an HTML 2.0 (to distinguish it from the previous informal specifications). Future (generally upwardly compatible) versions of HTML with new features will be released with higher version numbers. ^ ^ Berners-Lee, Tim; Connolly, Daniel W. (November 1995). Hypertext Markup Language - 2.0. Network Working Group. doi:10.17487/RFC1866. RFC 1866. Historic. Obsoleted by RFC 2854. This document thus defines an HTML 2.0 (to distinguish it from the previous informal specifications). Future (generally upwardly compatible) versions of HTML with new features will be released with higher version numbers. Berners-Lee, Tim Hypertext Markup Language - 2.0 doi 10.17487/RFC1866 10.17487/RFC1866 RFC 1866 2854 ^ a b c d e f Raggett, Dave (1998). Raggett on HTML 4. Archived from the original on August 9, 2007. Retrieved July 9, 2007. ^ a b c d e f a b c d e f Raggett, Dave (1998). Raggett on HTML 4. Archived from the original on August 9, 2007. Retrieved July 9, 2007. Raggett on HTML 4 the original . Retrieved July 9, 2007 July 9, ^ \"HTML5 \u2013 Hypertext Markup Language \u2013 5.0\". Internet Engineering Task Force. 28 October 2014. Archived from the original on October 28, 2014. Retrieved November 25, 2014. This document recommends HTML 5.0 after completion. ^ ^ \"HTML5 \u2013 Hypertext Markup Language \u2013 5.0\". Internet Engineering Task Force. 28 October 2014. Archived from the original on October 28, 2014. Retrieved November 25, 2014. This document recommends HTML 5.0 after completion. \"HTML5 \u2013 Hypertext Markup Language \u2013 5.0\" Archived . Retrieved November 25, 2014 November 25, ^ \"HTML 3.2 Reference Specification\". World Wide Web Consortium. January 14, 1997. Retrieved November 16, 2008. ^ ^ \"HTML 3.2 Reference Specification\". World Wide Web Consortium. January 14, 1997. Retrieved November 16, 2008. \"HTML 3.2 Reference Specification\" . Retrieved November 16, 2008 November 16, ^ \"IETF HTML WG\". Retrieved June 16, 2007. Note: This working group is closed ^ ^ \"IETF HTML WG\". Retrieved June 16, 2007. Note: This working group is closed \"IETF HTML WG\" . Retrieved June 16, 2007 June 16, ^ a b Engelfriet, Arnoud. \"Introduction to Wilbur\". htmlhelp.com. Retrieved June 16, 2007. ^ a b a b Engelfriet, Arnoud. \"Introduction to Wilbur\". htmlhelp.com. Retrieved June 16, 2007. Engelfriet, Arnoud \"Introduction to Wilbur\" . Retrieved June 16, 2007 June 16, ^ \"HTML 4.0 Specification\". World Wide Web Consortium. December 18, 1997. Retrieved November 16, 2008. ^ ^ \"HTML 4.0 Specification\". World Wide Web Consortium. December 18, 1997. Retrieved November 16, 2008. \"HTML 4.0 Specification\" . Retrieved November 16, 2008 November 16, ^ \"HTML 4 \u2013 4 Conformance: requirements and recommendations\". Retrieved December 30, 2009. ^ ^ \"HTML 4 \u2013 4 Conformance: requirements and recommendations\". Retrieved December 30, 2009. \"HTML 4 \u2013 4 Conformance: requirements and recommendations\" . Retrieved December 30, 2009 December 30, ^ \"HTML 4.0 Specification\". World Wide Web Consortium. April 24, 1998. Retrieved November 16, 2008. ^ ^ \"HTML 4.0 Specification\". World Wide Web Consortium. April 24, 1998. Retrieved November 16, 2008. \"HTML 4.0 Specification\" . Retrieved November 16, 2008 November 16, ^ \"HTML 4.01 Specification\". World Wide Web Consortium. December 24, 1999. Retrieved November 16, 2008. ^ ^ \"HTML 4.01 Specification\". World Wide Web Consortium. December 24, 1999. Retrieved November 16, 2008. \"HTML 4.01 Specification\" . Retrieved November 16, 2008 November 16, ^ \"HTML 4 Errata\". W3C. Retrieved March 2, 2023. ^ ^ \"HTML 4 Errata\". W3C. Retrieved March 2, 2023. \"HTML 4 Errata\" . Retrieved March 2, 2023 March 2, ^ a b ISO (2000). \"ISO/IEC 15445:2000 \u2013 Information technology \u2013 Document description and processing languages \u2013 HyperText Markup Language (HTML)\". Retrieved March 1, 2023. ^ a b a b ISO (2000). \"ISO/IEC 15445:2000 \u2013 Information technology \u2013 Document description and processing languages \u2013 HyperText Markup Language (HTML)\". Retrieved March 1, 2023. \"ISO/IEC 15445:2000 \u2013 Information technology \u2013 Document description and processing languages \u2013 HyperText Markup Language (HTML)\" . Retrieved March 1, 2023 March 1, ^ \"ISO/IEC 15445:2000(E) ISO-HTML\". www.scss.tcd.ie. Geneva, CH: ISO/IEC. May 15, 2000. Retrieved March 1, 2023. ^ ^ \"ISO/IEC 15445:2000(E) ISO-HTML\". www.scss.tcd.ie. Geneva, CH: ISO/IEC. May 15, 2000. Retrieved March 1, 2023. \"ISO/IEC 15445:2000(E) ISO-HTML\" . Retrieved March 1, 2023 March 1, ^ \"HTML5: A vocabulary and associated APIs for HTML and XHTML\". World Wide Web Consortium. 28 October 2014. Retrieved 31 October 2014. ^ ^ \"HTML5: A vocabulary and associated APIs for HTML and XHTML\". World Wide Web Consortium. 28 October 2014. Retrieved 31 October 2014. \"HTML5: A vocabulary and associated APIs for HTML and XHTML\" . Retrieved 31 October 2014 31 October ^ \"Open Web Platform Milestone Achieved with HTML5 Recommendation\" (Press release). World Wide Web Consortium. 28 October 2014. Retrieved 31 October 2014. ^ ^ \"Open Web Platform Milestone Achieved with HTML5 Recommendation\" (Press release). World Wide Web Consortium. 28 October 2014. Retrieved 31 October 2014. \"Open Web Platform Milestone Achieved with HTML5 Recommendation\" . Retrieved 31 October 2014 31 October ^ \"HTML 5.1\". World Wide Web Consortium. 1 November 2016. Retrieved 6 January 2017. ^ ^ \"HTML 5.1\". World Wide Web Consortium. 1 November 2016. Retrieved 6 January 2017. \"HTML 5.1\" . Retrieved 6 January 2017 6 January ^ \"HTML 5.1 is a W3C Recommendation\". World Wide Web Consortium. 1 November 2016. Retrieved 6 January 2017. ^ ^ \"HTML 5.1 is a W3C Recommendation\". World Wide Web Consortium. 1 November 2016. Retrieved 6 January 2017. \"HTML 5.1 is a W3C Recommendation\" . Retrieved 6 January 2017 6 January ^ Philippe le Hegaret (17 November 2016). \"HTML 5.1 is the gold standard\". World Wide Web Consortium. Retrieved 6 January 2017. ^ ^ Philippe le Hegaret (17 November 2016). \"HTML 5.1 is the gold standard\". World Wide Web Consortium. Retrieved 6 January 2017. \"HTML 5.1 is the gold standard\" . Retrieved 6 January 2017 6 January ^ \"HTML 5.2\". World Wide Web Consortium. 14 December 2017. Retrieved 15 December 2017. ^ ^ \"HTML 5.2\". World Wide Web Consortium. 14 December 2017. Retrieved 15 December 2017. \"HTML 5.2\" . Retrieved 15 December 2017 15 December ^ \"HTML 5.2 is now a W3C Recommendation\". World Wide Web Consortium. 14 December 2017. Retrieved 15 December 2017. ^ ^ \"HTML 5.2 is now a W3C Recommendation\". World Wide Web Consortium. 14 December 2017. Retrieved 15 December 2017. \"HTML 5.2 is now a W3C Recommendation\" . Retrieved 15 December 2017 15 December ^ Charles McCathie Nevile (14 December 2017). \"HTML 5.2 is done, HTML 5.3 is coming\". World Wide Web Consortium. Retrieved 15 December 2017. ^ ^ Charles McCathie Nevile (14 December 2017). \"HTML 5.2 is done, HTML 5.3 is coming\". World Wide Web Consortium. Retrieved 15 December 2017. \"HTML 5.2 is done, HTML 5.3 is coming\" . Retrieved 15 December 2017 15 December ^ Connolly, Daniel (6 June 1992). \"MIME as a hypertext architecture\". CERN. Retrieved 24 October 2010. ^ ^ Connolly, Daniel (6 June 1992). \"MIME as a hypertext architecture\". CERN. Retrieved 24 October 2010. Connolly, Daniel \"MIME as a hypertext architecture\" . Retrieved 24 October 2010 24 October ^ Connolly, Daniel (15 July 1992). \"HTML DTD enclosed\". CERN. Retrieved 24 October 2010. ^ ^ Connolly, Daniel (15 July 1992). \"HTML DTD enclosed\". CERN. Retrieved 24 October 2010. Connolly, Daniel \"HTML DTD enclosed\" . Retrieved 24 October 2010 24 October ^ Connolly, Daniel (18 August 1992). \"document type declaration subset for Hyper Text Markup Language as defined by the World Wide Web project\". CERN. Archived from the original on 14 March 2012. Retrieved 24 October 2010. ^ ^ Connolly, Daniel (18 August 1992). \"document type declaration subset for Hyper Text Markup Language as defined by the World Wide Web project\". CERN. Archived from the original on 14 March 2012. Retrieved 24 October 2010. Connolly, Daniel \"document type declaration subset for Hyper Text Markup Language as defined by the World Wide Web project\" the original . Retrieved 24 October 2010 24 October ^ a b Connolly, Daniel (24 November 1992). \"Document Type Definition for the Hyper Text Markup Language as used by the World Wide Web application\". CERN. Archived from the original on 18 January 2012. Retrieved 24 October 2010. See section \"Revision History\" ^ a b a b Connolly, Daniel (24 November 1992). \"Document Type Definition for the Hyper Text Markup Language as used by the World Wide Web application\". CERN. Archived from the original on 18 January 2012. Retrieved 24 October 2010. See section \"Revision History\" Connolly, Daniel \"Document Type Definition for the Hyper Text Markup Language as used by the World Wide Web application\" the original . Retrieved 24 October 2010 24 October ^ Berners-Lee, Tim; Connolly, Daniel (June 1993). \"Hyper Text Markup Language (HTML) Internet-Draft version 1.1\". IETF IIIR Working Group. Retrieved 18 September 2010. ^ ^ Berners-Lee, Tim; Connolly, Daniel (June 1993). \"Hyper Text Markup Language (HTML) Internet-Draft version 1.1\". IETF IIIR Working Group. Retrieved 18 September 2010. Berners-Lee, Tim Connolly, Daniel \"Hyper Text Markup Language (HTML) Internet-Draft version 1.1\" . Retrieved 18 September 2010 18 September ^ Berners-Lee, Tim; Connolly, Daniel (June 1993). \"Hypertext Markup Language (HTML) Internet-Draft version 1.2\". IETF IIIR Working Group. Retrieved 18 September 2010. ^ ^ Berners-Lee, Tim; Connolly, Daniel (June 1993). \"Hypertext Markup Language (HTML) Internet-Draft version 1.2\". IETF IIIR Working Group. Retrieved 18 September 2010. Berners-Lee, Tim Connolly, Daniel \"Hypertext Markup Language (HTML) Internet-Draft version 1.2\" . Retrieved 18 September 2010 18 September ^ Raggett, Dave (1993-11-08). \"History for draft-raggett-www-html-00\". IETF Datatracker. Retrieved 2019-11-18. ^ ^ Raggett, Dave (1993-11-08). \"History for draft-raggett-www-html-00\". IETF Datatracker. Retrieved 2019-11-18. \"History for draft-raggett-www-html-00\" . Retrieved 2019-11-18 2019-11-18 ^ Berners-Lee, Tim; Connolly, Daniel (28 November 1994). \"HyperText Markup Language Specification \u2013 2.0 INTERNET DRAFT\". Internet Engineering Task Force. Retrieved 24 October 2010. ^ ^ Berners-Lee, Tim; Connolly, Daniel (28 November 1994). \"HyperText Markup Language Specification \u2013 2.0 INTERNET DRAFT\". Internet Engineering Task Force. Retrieved 24 October 2010. Berners-Lee, Tim Connolly, Daniel \"HyperText Markup Language Specification \u2013 2.0 INTERNET DRAFT\" . Retrieved 24 October 2010 24 October ^ Connolly, Daniel W. (1995-05-16). \"Hypertext Markup Language \u2013 2.0\". tools.ietf.org. Retrieved 2019-11-18. ^ ^ Connolly, Daniel W. (1995-05-16). \"Hypertext Markup Language \u2013 2.0\". tools.ietf.org. Retrieved 2019-11-18. \"Hypertext Markup Language \u2013 2.0\" . Retrieved 2019-11-18 2019-11-18 ^ Berners-Lee, Tim; Connolly, Daniel W. (November 1995). Hypertext Markup Language - 2.0. Network Working Group. doi:10.17487/RFC1866. RFC 1866. Historic. Obsoleted by RFC 2854. ^ ^ Berners-Lee, Tim; Connolly, Daniel W. (November 1995). Hypertext Markup Language - 2.0. Network Working Group. doi:10.17487/RFC1866. RFC 1866. Historic. Obsoleted by RFC 2854. Berners-Lee, Tim Hypertext Markup Language - 2.0 doi 10.17487/RFC1866 10.17487/RFC1866 RFC 1866 2854 ^ \"HTML 3.0 Draft (Expired!) Materials\". World Wide Web Consortium. December 21, 1995. Retrieved November 16, 2008. ^ ^ \"HTML 3.0 Draft (Expired!) Materials\". World Wide Web Consortium. December 21, 1995. Retrieved November 16, 2008. \"HTML 3.0 Draft (Expired!) Materials\" . Retrieved November 16, 2008 November 16, ^ a b \"HyperText Markup Language Specification Version 3.0\". Retrieved June 16, 2007. ^ a b a b \"HyperText Markup Language Specification Version 3.0\". Retrieved June 16, 2007. \"HyperText Markup Language Specification Version 3.0\" . Retrieved June 16, 2007 June 16, ^ Raggett, Dave (28 March 1995). \"HyperText Markup Language Specification Version 3.0\". HTML 3.0 Internet Draft Expires in six months. World Wide Web Consortium. Retrieved 17 June 2010. ^ ^ Raggett, Dave (28 March 1995). \"HyperText Markup Language Specification Version 3.0\". HTML 3.0 Internet Draft Expires in six months. World Wide Web Consortium. Retrieved 17 June 2010. \"HyperText Markup Language Specification Version 3.0\" World Wide Web Consortium . Retrieved 17 June 2010 17 June ^ Bowers, N. (1998). \"Weblint: just another perl hack\" (PDF). 1998 USENIX Annual Technical Conference (USENIX ATC 98). ^ ^ Bowers, N. (1998). \"Weblint: just another perl hack\" (PDF). 1998 USENIX Annual Technical Conference (USENIX ATC 98). \"Weblint: just another perl hack\" (PDF) ^ Lie, H\u00e5kon Wium; Bos, Bert (April 1997). Cascading style sheets: designing for the Web. Addison Wesley Longman. p. 263. ISBN 978-0-201-41998-6. Retrieved 9 June 2010. ^ ^ Lie, H\u00e5kon Wium; Bos, Bert (April 1997). Cascading style sheets: designing for the Web. Addison Wesley Longman. p. 263. ISBN 978-0-201-41998-6. Retrieved 9 June 2010. Lie, H\u00e5kon Wium Bos, Bert Cascading style sheets: designing for the Web Cascading style sheets: designing for the Web 263 ISBN 978-0-201-41998-6 . Retrieved 9 June 2010 9 June ^ \"HTML5\". World Wide Web Consortium. June 10, 2008. Retrieved November 16, 2008. ^ ^ \"HTML5\". World Wide Web Consortium. June 10, 2008. Retrieved November 16, 2008. \"HTML5\" . Retrieved November 16, 2008 November 16, ^ \"HTML5, one vocabulary, two serializations\". 15 January 2008. Retrieved February 25, 2009. ^ ^ \"HTML5, one vocabulary, two serializations\". 15 January 2008. Retrieved February 25, 2009. \"HTML5, one vocabulary, two serializations\" . Retrieved February 25, 2009 February 25, ^ \"W3C Confirms May 2011 for HTML5 Last Call, Targets 2014 for HTML5 Standard\". World Wide Web Consortium. 14 February 2011. Retrieved 18 February 2011. ^ ^ \"W3C Confirms May 2011 for HTML5 Last Call, Targets 2014 for HTML5 Standard\". World Wide Web Consortium. 14 February 2011. Retrieved 18 February 2011. \"W3C Confirms May 2011 for HTML5 Last Call, Targets 2014 for HTML5 Standard\" World Wide Web Consortium . Retrieved 18 February 2011 18 February ^ Hickson, Ian (January 19, 2011). \"HTML Is the New HTML5\". The WHATWG Blog. Archived from the original on 6 October 2019. Retrieved 21 January 2011. ^ ^ Hickson, Ian (January 19, 2011). \"HTML Is the New HTML5\". The WHATWG Blog. Archived from the original on 6 October 2019. Retrieved 21 January 2011. \"HTML Is the New HTML5\" the original . Retrieved 21 January 2011 21 January ^ Grannell, Craig (July 23, 2012). \"HTML5 gets the splits\". Net magazine. Archived from the original on Jul 25, 2012. Retrieved 23 July 2012. ^ ^ Grannell, Craig (July 23, 2012). \"HTML5 gets the splits\". Net magazine. Archived from the original on Jul 25, 2012. Retrieved 23 July 2012. \"HTML5 gets the splits\" the original . Retrieved 23 July 2012 23 July ^ \"HTML5\". W3C. 2012-12-17. Retrieved 2013-06-15. ^ ^ \"HTML5\". W3C. 2012-12-17. Retrieved 2013-06-15. \"HTML5\" . Retrieved 2013-06-15 2013-06-15 ^ \"When Will HTML5 Be Finished?\". FAQ. WHAT Working Group. Retrieved 29 November 2009. ^ ^ \"When Will HTML5 Be Finished?\". FAQ. WHAT Working Group. Retrieved 29 November 2009. \"When Will HTML5 Be Finished?\" . Retrieved 29 November 2009 29 November ^ \"Call for Review: HTML5 Proposed Recommendation Published W3C News\". W3C. 2014-09-16. Retrieved 2014-09-27. ^ ^ \"Call for Review: HTML5 Proposed Recommendation Published W3C News\". W3C. 2014-09-16. Retrieved 2014-09-27. \"Call for Review: HTML5 Proposed Recommendation Published W3C News\" . Retrieved 2014-09-27 2014-09-27 ^ \"Open Web Platform Milestone Achieved with HTML5 Recommendation\". W3C. 28 October 2014. Retrieved 29 October 2014. ^ ^ \"Open Web Platform Milestone Achieved with HTML5 Recommendation\". W3C. 28 October 2014. Retrieved 29 October 2014. \"Open Web Platform Milestone Achieved with HTML5 Recommendation\" . Retrieved 29 October 2014 29 October ^ \"HTML5 specification finalized, squabbling over specs continues\". Ars Technica. 2014-10-29. Retrieved 2014-10-29. ^ ^ \"HTML5 specification finalized, squabbling over specs continues\". Ars Technica. 2014-10-29. Retrieved 2014-10-29. \"HTML5 specification finalized, squabbling over specs continues\" . Retrieved 2014-10-29 2014-10-29 ^ \"HTML vs XML syntax\". WHATWG. Retrieved 22 March 2023. ^ ^ \"HTML vs XML syntax\". WHATWG. Retrieved 22 March 2023. \"HTML vs XML syntax\" . Retrieved 22 March 2023 22 March ^ \"XHTML 1.0: The Extensible HyperText Markup Language (Second Edition)\". World Wide Web Consortium. January 26, 2000. Retrieved November 16, 2008. ^ ^ \"XHTML 1.0: The Extensible HyperText Markup Language (Second Edition)\". World Wide Web Consortium. January 26, 2000. Retrieved November 16, 2008. \"XHTML 1.0: The Extensible HyperText Markup Language (Second Edition)\" . Retrieved November 16, 2008 November 16, ^ \"XHTML 1.1 \u2013 Module-based XHTML \u2014 Second Edition\". World Wide Web Consortium. February 16, 2007. Retrieved November 16, 2008. ^ ^ \"XHTML 1.1 \u2013 Module-based XHTML \u2014 Second Edition\". World Wide Web Consortium. February 16, 2007. Retrieved November 16, 2008. \"XHTML 1.1 \u2013 Module-based XHTML \u2014 Second Edition\" . Retrieved November 16, 2008 November 16, ^ \"Modularization of XHTML\". W3C. Retrieved 2017-01-04. ^ ^ \"Modularization of XHTML\". W3C. Retrieved 2017-01-04. \"Modularization of XHTML\" . Retrieved 2017-01-04 2017-01-04 ^ \"XHTM 2.0\". World Wide Web Consortium. July 26, 2006. Retrieved November 16, 2008. ^ ^ \"XHTM 2.0\". World Wide Web Consortium. July 26, 2006. Retrieved November 16, 2008. \"XHTM 2.0\" . Retrieved November 16, 2008 November 16, ^ \"XHTML 2 Working Group Expected to Stop Work End of 2009, W3C to Increase Resources on HTML5\". World Wide Web Consortium. July 17, 2009. Retrieved November 16, 2008. ^ ^ \"XHTML 2 Working Group Expected to Stop Work End of 2009, W3C to Increase Resources on HTML5\". World Wide Web Consortium. July 17, 2009. Retrieved November 16, 2008. \"XHTML 2 Working Group Expected to Stop Work End of 2009, W3C to Increase Resources on HTML5\" . Retrieved November 16, 2008 November 16, ^ \"W3C XHTML FAQ\". ^ ^ \"W3C XHTML FAQ\". \"W3C XHTML FAQ\" ^ Jaffe, Jeff (28 May 2019). \"W3C and WHATWG to Work Together to Advance the Open Web Platform\". W3C Blog. Archived from the original on 29 May 2019. Retrieved 29 May 2019. ^ ^ Jaffe, Jeff (28 May 2019). \"W3C and WHATWG to Work Together to Advance the Open Web Platform\". W3C Blog. Archived from the original on 29 May 2019. Retrieved 29 May 2019. \"W3C and WHATWG to Work Together to Advance the Open Web Platform\" Archived . Retrieved 29 May 2019 29 May ^ \"W3C and the WHATWG Signed an Agreement to Collaborate on a Single Version of HTML and DOM\". W3C. 28 May 2019. Archived from the original on 29 May 2019. Retrieved 29 May 2019. ^ ^ \"W3C and the WHATWG Signed an Agreement to Collaborate on a Single Version of HTML and DOM\". W3C. 28 May 2019. Archived from the original on 29 May 2019. Retrieved 29 May 2019. \"W3C and the WHATWG Signed an Agreement to Collaborate on a Single Version of HTML and DOM\" Archived . Retrieved 29 May 2019 29 May ^ \"Memorandum of Understanding Between W3C and WHATWG\". W3C. 28 May 2019. Archived from the original on 29 May 2019. Retrieved 29 May 2019. ^ ^ \"Memorandum of Understanding Between W3C and WHATWG\". W3C. 28 May 2019. Archived from the original on 29 May 2019. Retrieved 29 May 2019. \"Memorandum of Understanding Between W3C and WHATWG\" Archived . Retrieved 29 May 2019 29 May ^ Cimpanu, Catalin (29 May 2019). \"Browser vendors Win War with W3C over HTML and DOM standards\". ZDNet. Archived from the original on 29 May 2019. Retrieved 29 May 2019. ^ ^ Cimpanu, Catalin (29 May 2019). \"Browser vendors Win War with W3C over HTML and DOM standards\". ZDNet. Archived from the original on 29 May 2019. Retrieved 29 May 2019. \"Browser vendors Win War with W3C over HTML and DOM standards\" the original . Retrieved 29 May 2019 29 May ^ \"W3C \u2013 WHATWG Wiki\". WHATWG Wiki. Archived from the original on 29 May 2019. Retrieved 29 May 2019. ^ ^ \"W3C \u2013 WHATWG Wiki\". WHATWG Wiki. Archived from the original on 29 May 2019. Retrieved 29 May 2019. \"W3C \u2013 WHATWG Wiki\" the original . Retrieved 29 May 2019 29 May ^ Shankland, Stephen (July 9, 2009). \"An epitaph for the Web standard, XHTML 2\". CNET. CBS INTERACTIVE INC. ^ ^ Shankland, Stephen (July 9, 2009). \"An epitaph for the Web standard, XHTML 2\". CNET. CBS INTERACTIVE INC. \"An epitaph for the Web standard, XHTML 2\" ^ Activating Browser Modes with Doctype. Hsivonen.iki.fi. Retrieved on 2012-02-16. ^ ^ Activating Browser Modes with Doctype. Hsivonen.iki.fi. Retrieved on 2012-02-16. Activating Browser Modes with Doctype ^ \"HTML Elements\". w3schools. Retrieved 16 March 2015. ^ ^ \"HTML Elements\". w3schools. Retrieved 16 March 2015. \"HTML Elements\" . Retrieved 16 March 2015 16 March ^ \"CSS Introduction\". W3schools. Retrieved 16 March 2015. ^ ^ \"CSS Introduction\". W3schools. Retrieved 16 March 2015. \"CSS Introduction\" . Retrieved 16 March 2015 16 March ^ \"On SGML and HTML\". World Wide Web Consortium. Retrieved November 16, 2008. ^ ^ \"On SGML and HTML\". World Wide Web Consortium. Retrieved November 16, 2008. \"On SGML and HTML\" . Retrieved November 16, 2008 November 16, ^ \"XHTML 1.0 \u2013 Differences with HTML 4\". World Wide Web Consortium. Retrieved November 16, 2008. ^ ^ \"XHTML 1.0 \u2013 Differences with HTML 4\". World Wide Web Consortium. Retrieved November 16, 2008. \"XHTML 1.0 \u2013 Differences with HTML 4\" . Retrieved November 16, 2008 November 16, ^ Korpela, Jukka (July 6, 1998). \"Why attribute values should always be quoted in HTML\". Cs.tut.fi. Retrieved November 16, 2008. ^ ^ Korpela, Jukka (July 6, 1998). \"Why attribute values should always be quoted in HTML\". Cs.tut.fi. Retrieved November 16, 2008. \"Why attribute values should always be quoted in HTML\" . Retrieved November 16, 2008 November 16, ^ \"Objects, Images, and Applets in HTML documents\". World Wide Web Consortium. December 24, 1999. Retrieved November 16, 2008. ^ ^ \"Objects, Images, and Applets in HTML documents\". World Wide Web Consortium. December 24, 1999. Retrieved November 16, 2008. \"Objects, Images, and Applets in HTML documents\" . Retrieved November 16, 2008 November 16, ^ \"H56: Using the dir attribute on an inline element to resolve problems with nested directional runs\". Techniques for WCAG 2.0. W3C. Retrieved 18 September 2010. ^ ^ \"H56: Using the dir attribute on an inline element to resolve problems with nested directional runs\". Techniques for WCAG 2.0. W3C. Retrieved 18 September 2010. \"H56: Using the dir attribute on an inline element to resolve problems with nested directional runs\" . Retrieved 18 September 2010 18 September ^ \"Character Entity Reference Chart\". World Wide Web Consortium. October 24, 2012. ^ ^ \"Character Entity Reference Chart\". World Wide Web Consortium. October 24, 2012. \"Character Entity Reference Chart\" ^ \"The Named Character Reference '\". World Wide Web Consortium. January 26, 2000. ^ ^ \"The Named Character Reference '\". World Wide Web Consortium. January 26, 2000. \"The Named Character Reference '\" ^ \"The Unicode Standard: A Technical Introduction\". Unicode. Retrieved 2010-03-16. ^ ^ \"The Unicode Standard: A Technical Introduction\". Unicode. Retrieved 2010-03-16. \"The Unicode Standard: A Technical Introduction\" . Retrieved 2010-03-16 2010-03-16 ^ \"The HTML syntax\". HTML Standard. Retrieved 2013-08-19. ^ ^ \"The HTML syntax\". HTML Standard. Retrieved 2013-08-19. \"The HTML syntax\" . Retrieved 2013-08-19 2013-08-19 ^ \"HTML 4 Frameset Document Type Definition\". W3C. Retrieved 2021-12-25. ^ ^ \"HTML 4 Frameset Document Type Definition\". W3C. Retrieved 2021-12-25. \"HTML 4 Frameset Document Type Definition\" . Retrieved 2021-12-25 2021-12-25 ^ Berners-Lee, Tim; Fischetti, Mark (2000). Weaving the Web: The Original Design and Ultimate Destiny of the World Wide Web by Its Inventor. San Francisco: Harper. ISBN 978-0-06-251587-2. ^ ^ Berners-Lee, Tim; Fischetti, Mark (2000). Weaving the Web: The Original Design and Ultimate Destiny of the World Wide Web by Its Inventor. San Francisco: Harper. ISBN 978-0-06-251587-2. Weaving the Web: The Original Design and Ultimate Destiny of the World Wide Web by Its Inventor Weaving the Web: The Original Design and Ultimate Destiny of the World Wide Web by Its Inventor ISBN 978-0-06-251587-2 ^ Raggett, Dave (2002). \"Adding a touch of style\". W3C. Retrieved October 2, 2009. This article notes that presentational HTML markup may be useful when targeting browsers \"before Netscape 4.0 and Internet Explorer 4.0\". See the list of web browsers to confirm that these were both released in 1997. ^ ^ Raggett, Dave (2002). \"Adding a touch of style\". W3C. Retrieved October 2, 2009. This article notes that presentational HTML markup may be useful when targeting browsers \"before Netscape 4.0 and Internet Explorer 4.0\". See the list of web browsers to confirm that these were both released in 1997. \"Adding a touch of style\" . Retrieved October 2, 2009 October 2, list of web browsers ^ Berners-Lee, Tim; Hendler, James; Lassila, Ora (May 1, 2001). \"The Semantic Web\". Scientific American. Retrieved October 2, 2009. ^ ^ Berners-Lee, Tim; Hendler, James; Lassila, Ora (May 1, 2001). \"The Semantic Web\". Scientific American. Retrieved October 2, 2009. \"The Semantic Web\" . Retrieved October 2, 2009 October 2, ^ Nigel Shadbolt, Wendy Hall and Tim Berners-Lee (2006). \"The Semantic Web Revisited\" (PDF). IEEE Intelligent Systems. Archived from the original (PDF) on March 20, 2013. Retrieved October 2, 2009. ^ ^ Nigel Shadbolt, Wendy Hall and Tim Berners-Lee (2006). \"The Semantic Web Revisited\" (PDF). IEEE Intelligent Systems. Archived from the original (PDF) on March 20, 2013. Retrieved October 2, 2009. \"The Semantic Web Revisited\" (PDF) the original (PDF) . Retrieved October 2, 2009 October 2, ^ \"HTML: The Living Standard\". WHATWG. Retrieved 27 September 2018. ^ ^ \"HTML: The Living Standard\". WHATWG. Retrieved 27 September 2018. \"HTML: The Living Standard\" . Retrieved 27 September 2018 27 September ^ \"XHTML 1.0 The Extensible HyperText Markup Language (Second Edition)\". World Wide Web Consortium. 2002 [2000]. Retrieved December 7, 2008. XHTML Documents which follow the guidelines set forth in Appendix C, \"HTML Compatibility Guidelines\" may be labeled with the Internet Media Type \"text/html\" [RFC2854], as they are compatible with most HTML browsers. Those documents, and any other document conforming to this specification, may also be labeled with the Internet Media Type \"application/xhtml+xml\" as defined in [RFC3236]. ^ ^ \"XHTML 1.0 The Extensible HyperText Markup Language (Second Edition)\". World Wide Web Consortium. 2002 [2000]. Retrieved December 7, 2008. XHTML Documents which follow the guidelines set forth in Appendix C, \"HTML Compatibility Guidelines\" may be labeled with the Internet Media Type \"text/html\" [RFC2854], as they are compatible with most HTML browsers. Those documents, and any other document conforming to this specification, may also be labeled with the Internet Media Type \"application/xhtml+xml\" as defined in [RFC3236]. \"XHTML 1.0 The Extensible HyperText Markup Language (Second Edition)\" . Retrieved December 7, 2008 December 7, ^ S. Bradner (March 1997). Key words for use in RFCs to Indicate Requirement Levels. Network Working Group. doi:10.17487/RFC2119. BCP 14. RFC 2119. Best Current Practice 14. Updated by RFC 8174. 3. SHOULD This word, or the adjective \"RECOMMENDED\", mean that there may exist valid reasons in particular circumstances to ignore a particular item, but the full implications must be understood and carefully weighed before choosing a different course. ^ ^ S. Bradner (March 1997). Key words for use in RFCs to Indicate Requirement Levels. Network Working Group. doi:10.17487/RFC2119. BCP 14. RFC 2119. Best Current Practice 14. Updated by RFC 8174. 3. SHOULD This word, or the adjective \"RECOMMENDED\", mean that there may exist valid reasons in particular circumstances to ignore a particular item, but the full implications must be understood and carefully weighed before choosing a different course. Key words for use in RFCs to Indicate Requirement Levels doi 10.17487/RFC2119 10.17487/RFC2119 RFC 2119 8174 ^ \"XHTML 1.1 \u2013 Module-based XHTML \u2014 Second Edition\". World Wide Web Consortium. 2007. Retrieved December 7, 2008. XHTML 1.1 documents SHOULD be labeled with the Internet Media Type text/html as defined in [RFC2854] or application/xhtml+xml as defined in [RFC3236]. ^ ^ \"XHTML 1.1 \u2013 Module-based XHTML \u2014 Second Edition\". World Wide Web Consortium. 2007. Retrieved December 7, 2008. XHTML 1.1 documents SHOULD be labeled with the Internet Media Type text/html as defined in [RFC2854] or application/xhtml+xml as defined in [RFC3236]. \"XHTML 1.1 \u2013 Module-based XHTML \u2014 Second Edition\" . Retrieved December 7, 2008 December 7, ^ \"Naming Files, Paths, and Namespaces\". Microsoft. Retrieved 16 March 2015. ^ ^ \"Naming Files, Paths, and Namespaces\". Microsoft. Retrieved 16 March 2015. \"Naming Files, Paths, and Namespaces\" . Retrieved 16 March 2015 16 March ^ HTML Design Constraints, W3C Archives ^ ^ HTML Design Constraints, W3C Archives HTML Design Constraints ^ WWW: BTB \u2013 HTML, Pris Sears ^ ^ WWW: BTB \u2013 HTML, Pris Sears WWW: BTB \u2013 HTML ^ Freeman, E (2005). Head First HTML. O'Reilly. ^ ^ Freeman, E (2005). Head First HTML. O'Reilly. ^ Sauer, C.: WYSIWIKI \u2013 Questioning WYSIWYG in the Internet Age. In: Wikimania (2006) ^ ^ Sauer, C.: WYSIWIKI \u2013 Questioning WYSIWYG in the Internet Age. In: Wikimania (2006) ^ Spiesser, J., Kitchen, L.: Optimization of HTML automatically generated by WYSIWYG programs. In: 13th International Conference on World Wide Web, pp. 355\u2014364. WWW '04. ACM, New York, NY (New York, NY, U.S., May 17\u201320, 2004) ^ ^ Spiesser, J., Kitchen, L.: Optimization of HTML automatically generated by WYSIWYG programs. In: 13th International Conference on World Wide Web, pp. 355\u2014364. WWW '04. ACM, New York, NY (New York, NY, U.S., May 17\u201320, 2004) ^ XHTML Reference: blockquote Archived 2010-03-25 at the Wayback Machine. Xhtml.com. Retrieved on 2012-02-16. ^ ^ XHTML Reference: blockquote Archived 2010-03-25 at the Wayback Machine. Xhtml.com. Retrieved on 2012-02-16. XHTML Reference: blockquote Archived Wayback Machine ^ Doug Engelbart's INVISIBLE REVOLUTION. Invisiblerevolution.net. Retrieved on 2012-02-16. ^ ^ Doug Engelbart's INVISIBLE REVOLUTION. Invisiblerevolution.net. Retrieved on 2012-02-16. Doug Engelbart's INVISIBLE REVOLUTION External links HTML sister projects sister projects Definitions from Wiktionary Definitions from Wiktionary Definitions Media from Commons Media from Commons Media Textbooks from Wikibooks Textbooks from Wikibooks Textbooks Resources from Wikiversity Resources from Wikiversity Resources Data from Wikidata Data from Wikidata Data Discussions from Meta-Wiki Discussions from Meta-Wiki Discussions Documentation from MediaWiki Documentation from MediaWiki Documentation WHATWG's HTML Living Standard WHATWG HTML Living Standard Dave Raggett's Introduction to HTML Dave Raggett's Introduction to HTML Tim Berners-Lee Gives the Web a New Definition (archived 12 April 2011) Tim Berners-Lee Gives the Web a New Definition List of all HTML elements from all major versions List of all HTML elements from all major versions HTML Entities HTML Entities Sean B. Palmer. \"Early History of HTML \u2013 1990 to 1992\". Infomesh. Retrieved 2022-04-13. (Timeframe: 1980\u20131995) \"Early History of HTML \u2013 1990 to 1992\" . Retrieved 2022-04-13 2022-04-13 v v t t e e Web browsers Features standards protocols Bookmarks Bookmarks Extensions Extensions Privacy mode Privacy mode Web standards HTML v5 HTML v5 v5 CSS CSS DOM DOM JavaScript WebAssembly Web storage IndexedDB WebGL WebGPU JavaScript WebAssembly WebAssembly Web storage Web storage IndexedDB IndexedDB WebGL WebGL WebGPU WebGPU HTTP Encryption Cookies third-party HTTP Encryption Encryption Cookies third-party Cookies third-party third-party OCSP OCSP WebRTC WebRTC WebSocket WebSocket Blink Proprietary Google Chrome Google Chrome Arc Arc Avast Avast Coc Coc Coc Coc Comodo Comodo Epic Epic Maxthon Maxthon Microsoft Edge Microsoft Edge Opera (Mobile) Opera Mobile Puffin Puffin QQ QQ Samsung Samsung Silk Silk Sleipnir Sleipnir SRWare SRWare UC UC Vivaldi Vivaldi Whale Whale Yandex Yandex FOSS Chromium Chromium Brave Brave Dooble Dooble Falkon Falkon Otter Otter Supermium Supermium ungoogled ungoogled Gecko Firefox Firefox Floorp Floorp GNU IceCat GNU IceCat LibreWolf LibreWolf Midori Midori SlimBrowser SlimBrowser Tor Tor Zen Zen Gecko forks Basilisk K-Meleon Pale Moon SeaMonkey Waterfox forks Basilisk Basilisk K-Meleon K-Meleon Pale Moon Pale Moon SeaMonkey SeaMonkey Waterfox Waterfox WebKit Safari Safari GNOME Web GNOME Web iCab iCab Orion Orion engine 360 360 DuckDuckGo DuckDuckGo Konqueror Konqueror Lunascape Lunascape NetFront NetFront qutebrowser qutebrowser eww eww Flow Flow Ladybird Ladybird Links Links Lynx Lynx NetSurf NetSurf Opera Mini Opera Mini w3m w3m Blink Beaker Beaker Citrio Citrio Flock Flock Redcore Redcore Rockmelt Rockmelt SalamWeb SalamWeb Sputnik Sputnik Torch Torch Gecko Beonex Beonex Camino Camino Classilla Classilla Conkeror Conkeror Firefox Lite Firefox Lite Galeon Galeon Ghostzilla Ghostzilla IceDragon IceDragon Kazehakase Kazehakase Kylo Kylo Lotus Lotus MicroB MicroB Minimo Minimo Mozilla suite Mozilla suite PirateBrowser PirateBrowser Pogo Pogo Strata Strata Swiftfox Swiftfox Swiftweasel Swiftweasel TenFourFox TenFourFox Timberwolf Timberwolf xB xB MSHTML Internet Explorer Internet Explorer AOL AOL Deepnet Deepnet GreenBrowser GreenBrowser MediaBrowser MediaBrowser NeoPlanet NeoPlanet NetCaptor NetCaptor SpaceTime SpaceTime ZAC ZAC WebKit Arora Arora BOLT BOLT Dolphin Dolphin Fluid Fluid Google TV Google TV Iris Iris Mercury Mercury Nokia Symbian Nokia Symbian OmniWeb OmniWeb Opera Coast Opera Coast Origyn Origyn QtWeb QtWeb Shiira Shiira Steel Steel surf surf Uzbl Uzbl WebPositive WebPositive xombrero xombrero abaco abaco Amaya Amaya Arachne Arachne Arena Arena Blazer Blazer Cake Cake Charon Charon CM CM Deepfish Deepfish Dillo Dillo Edge Legacy Edge Legacy ELinks ELinks Gazelle Gazelle HotJava HotJava IBM Home Page Reader IBM Home Page Reader IBM WebExplorer IBM WebExplorer IBrowse IBrowse Internet Explorer for Mac Internet Explorer for Mac KidZui KidZui Line Mode Line Mode Mosaic Mosaic MSN TV MSN TV NetPositive NetPositive Netscape Netscape Skweezer Skweezer Skyfire Skyfire ThunderHawk ThunderHawk Vision Vision WinWAP WinWAP WorldWideWeb WorldWideWeb Category Category Comparisons Comparisons List List v v t t e e World Wide Web Consortium Recommendations ActivityPub ActivityPub Activity Streams Activity Streams ARIA ARIA Canonical XML Canonical XML CDF CDF CSS Animations Flexbox Grid CSS Animations Animations Flexbox Flexbox Grid Grid DOM DOM EXI EXI EmotionML EmotionML Geolocation API Geolocation API HTML HTML5 HTML HTML5 HTML5 IndexedDB IndexedDB ITS ITS JSON-LD JSON-LD Linked Data Notifications Linked Data Notifications MathML MathML Micropub Micropub OWL OWL PLS PLS RDF Schema RDFa RDF Schema Schema RDFa RDFa SISR SISR SKOS SKOS SMIL SMIL SOAP SOAP SRGS SRGS SRI SRI SSML SSML SVG Filter Effects SVG Filter Effects Filter Effects SCXML SCXML SHACL SHACL SPARQL SPARQL Timed text Timed text VoiceXML VoiceXML WebAssembly WebAssembly WoT TD WoT TD TD Web storage Web storage WSDL WSDL Webmention Webmention WebSub WebSub WebVTT WebVTT WOFF WOFF XHTML +RDFa XHTML +RDFa +RDFa XML Base Encryption Events Information Set Namespace Schema Signature XForms XInclude XLink XOP XPath 2.0 3.x XPointer XProc XQuery XSL XSL-FO XSLT elements XML Base Base Encryption Encryption Events Events Information Set Information Set Namespace Namespace Schema Schema Signature Signature XForms XForms XInclude XInclude XLink XLink XOP XOP XPath 2.0 3.x XPath 2.0 2.0 3.x 3.x XPointer XPointer XProc XProc XQuery XQuery XSL XSL XSL-FO XSL-FO XSLT elements XSLT elements elements IndieAuth IndieAuth XAdES XAdES XBL XBL XHTML+SMIL XHTML+SMIL XUP XUP Working drafts CCXML CCXML CURIE CURIE EME EME InkML InkML MSE MSE RIF RIF SMIL Timesheets SMIL Timesheets sXBL sXBL WebGPU WebGPU WebXR WebXR XFDL XFDL XFrames XFrames XMLHttpRequest XMLHttpRequest Web Content Accessibility Guidelines Web Content Accessibility Guidelines Markup Validation Service Markup Validation Service Web Accessibility Initiative Web Accessibility Initiative Web Components Web Components C-HTML C-HTML HDML HDML JSSS JSSS PGML PGML VML VML WebPlatform WebPlatform P3P P3P XHTML+MathML+SVG XHTML+MathML+SVG World Wide Web Foundation World Wide Web Foundation WHATWG WHATWG AB AB Board Board TAG TAG CSS CSS SVG SVG WebAssembly WebAssembly WebAuthn WebAuthn Web Advertising BG Web Advertising BG WebAssembly CG WebAssembly Device Description (DDWG) Device Description HTML HTML Multimodal Interaction Activity (MMI) Multimodal Interaction Activity CERN httpd CERN httpd Libwww Libwww Browsers Line Mode (1990\u2013) Line Mode (1990\u2013) Arena (1993\u201398) Arena (1993\u201398) Agora (1994\u201397) Agora (1994\u201397) Argo (1994\u201397) Argo (1994\u201397) Amaya (browser/editor, 1996\u20132012) Amaya (browser/editor, 1996\u20132012) International World Wide Web Conference (IW3C) Steering Committee (IW3C2) First conference (\"WWW1\", 1994) International World Wide Web Conference Steering Committee (IW3C2) Steering Committee First conference (\"WWW1\", 1994) First conference v v t t e e Document markup languages Office suite Compound Document Format Compound Document Format OOXML SpreadsheetML PresentationML WordprocessingML OOXML SpreadsheetML SpreadsheetML PresentationML PresentationML WordprocessingML WordprocessingML ODF ODF UOF UOF HTML HTML XHTML XHTML MathML MathML RTF RTF TeX TeX LaTeX LaTeX Markdown Markdown AmigaGuide AmigaGuide AsciiDoc AsciiDoc BBCode BBCode CML CML C-HTML C-HTML ConTeXt ConTeXt CrossMark CrossMark DITA DITA DocBook DocBook EAD EAD Enriched text Enriched text FHTML FHTML GML GML GuideML GuideML HDML HDML HyTime HyTime IPF IPF LilyPond LilyPond LinuxDoc LinuxDoc Lout MIF MIF MAML MAML MEI MEI MusicXML MusicXML OMDoc OMDoc OpenMath OpenMath Org-mode Org-mode POD POD ReStructuredText ReStructuredText RTML RTML RFT RFT S1000D S1000D Setext Setext TEI TEI Texinfo Texinfo troff troff Wikitext Wikitext WML WML WapTV WapTV XAML XAML List of document markup languages v v t t e e ISO by standard number ISO standards ISO romanizations IEC standards 1 1 2 2 3 3 4 4 6 6 7 7 9 9 16 16 17 17 31 -0 -1 -3 -4 -5 -6 -7 -8 -9 -10 -11 -12 -13 31 -0 -0 -1 -1 -3 -3 -4 -4 -5 -5 -6 -6 -7 -7 -8 -8 -9 -9 -10 -10 -11 -11 -12 -12 -13 -13 68-1 68-1 128 128 216 216 217 217 226 226 228 228 233 233 259 259 261 261 262 262 302 302 306 306 361 361 500 500 518 518 519 519 639 -1 -2 -3 -5 -6 639 -1 -1 -2 -2 -3 -3 -5 -5 -6 -6 646 646 657 657 668 668 690 690 704 704 732 732 764 764 838 838 843 843 860 860 898 898 965 965 999 999 1000 1000 1004 1004 1007 1007 1073-1 1073-1 1073-2 1073-2 1155 1155 1413 1413 1538 1538 1629 1629 1745 1745 1989 1989 2014 2014 2015 2015 2022 2022 2033 2033 2047 2047 2108 2108 2145 2145 2146 2146 2240 2240 2281 2281 2533 2533 2709 2709 2711 2711 2720 2720 2788 2788 2848 2848 2852 2852 2921 2921 3029 3029 3103 3103 3166 -1 -2 -3 3166 -1 -1 -2 -2 -3 -3 3297 3297 3307 3307 3601 3601 3602 3602 3864 3864 3901 3901 3950 3950 3977 3977 4031 4031 4157 4157 4165 4165 4217 4217 4909 4909 5218 5218 5426 5426 5427 5427 5428 5428 5725 5725 5775 5775 5776 5776 5800 5800 5807 5807 5964 5964 6166 6166 6344 6344 6346 6346 6373 6373 6385 6385 6425 6425 6429 6429 6438 6438 6523 6523 6709 6709 6943 6943 7001 7001 7002 7002 7010 7010 7027 7027 7064 7064 7098 7098 7185 7185 7200 7200 7498 -1 7498 -1 -1 7637 7637 7736 7736 7810 7810 7811 7811 7812 7812 7813 7813 7816 7816 7942 7942 8000 8000 8093 8093 8178 8178 8217 8217 8373 8373 8501-1 8501-1 8571 8571 8583 8583 8601 8601 8613 8613 8632 8632 8651 8651 8652 8652 8691 8691 8805/8806 8805/8806 8807 8807 8820-5 8820-5 8859 -1 -2 -3 -4 -5 -6 -7 -8 -8-I -9 -10 -11 -12 -13 -14 -15 -16 8859 -1 -1 -2 -2 -3 -3 -4 -4 -5 -5 -6 -6 -7 -7 -8 -8 -8-I -8-I -9 -9 -10 -10 -11 -11 -12 -12 -13 -13 -14 -14 -15 -15 -16 -16 8879 8879 9000/9001 9000/9001 9036 9036 9075 9075 9126 9126 9141 9141 9227 9227 9241 9241 9293 9293 9314 9314 9362 9362 9407 9407 9496 9496 9506 9506 9529 9529 9564 9564 9592/9593 9592/9593 9594 9594 9660 9660 9797-1 9797-1 9897 9897 9899 9899 9945 9945 9984 9984 9985 9985 9995 9995 10006 10006 10007 10007 10116 10116 10118-3 10118-3 10160 10160 10161 10161 10165 10165 10179 10179 10206 10206 10218 10218 10279 10279 10303 -11 -21 -22 -28 -238 10303 -11 -11 -21 -21 -22 -22 -28 -28 -238 -238 10383 10383 10585 10585 10589 10589 10628 10628 10646 10646 10664 10664 10746 10746 10861 10861 10957 10957 10962 10962 10967 10967 11073 11073 11170 11170 11172 11172 11179 11179 11404 11404 11544 11544 11783 11783 11784 11784 11785 11785 11801 11801 11889 11889 11898 11898 11940 (-2) 11940 -2 11941 11941 11941 (TR) 11941 (TR) 11992 11992 12006 12006 12052 12052 12182 12182 12207 12207 12234-2 12234-2 12620 12620 13211 -1 -2 13211 -1 -1 -2 -2 13216 13216 13250 13250 13399 13399 13406-2 13406-2 13450 13450 13485 13485 13490 13490 13567 13567 13568 13568 13584 13584 13616 13616 13816 13816 13818 13818 14000 14000 14031 14031 14224 14224 14289 14289 14396 14396 14443 14443 14496 -2 -3 -6 -10 -11 -12 -14 -17 -20 14496 -2 -2 -3 -3 -6 -6 -10 -10 -11 -11 -12 -12 -14 -14 -17 -17 -20 -20 14617 14617 14644 14644 14649 14649 14651 14651 14698 14698 14764 14764 14882 14882 14971 14971 15022 15022 15189 15189 15288 15288 15291 15291 15398 15398 15408 15408 15444 -3 -9 15444 -3 -3 -9 -9 15445 15445 15438 15438 15504 15504 15511 15511 15686 15686 15693 15693 15706 -2 15706 -2 -2 15707 15707 15897 15897 15919 15919 15924 15924 15926 15926 15926 WIP 15926 WIP 15930 15930 15938 15938 16023 16023 16262 16262 16355-1 16355-1 16485 16485 16612-2 16612-2 16750 16750 16949 (TS) 16949 (TS) 17024 17024 17025 17025 17100 17100 17203 17203 17369 17369 17442 17442 17506 17506 17799 17799 18004 18004 18014 18014 18181 18181 18245 18245 18629 18629 18916 18916 19005 19005 19011 19011 19092 -1 -2 19092 -1 -1 -2 -2 19114 19114 19115 19115 19125 19125 19136 19136 19407 19407 19439 19439 19500 19500 19501 19501 19502 19502 19503 19503 19505 19505 19506 19506 19507 19507 19508 19508 19509 19509 19510 19510 19600 19600 19752 19752 19757 19757 19770 19770 19775-1 19775-1 19794-5 19794-5 19831 19831 20000 20000 20022 20022 20121 20121 20400 20400 20802 20802 20830 20830 21000 21000 21001 21001 21047 21047 21122 21122 21500 21500 21827 21827 22000 22000 22275 22275 22300 22300 22301 22301 22395 22395 22537 22537 23000 23000 23003 23003 23008 23008 23009 23009 23090-3 23090-3 23092 23092 23094-1 23094-1 23094-2 23094-2 23270 23270 23271 23271 23360 23360 23941 23941 24517 24517 24613 24613 24617 24617 24707 24707 24728 24728 25178 25178 25964 25964 26000 26000 26262 26262 26300 26300 26324 26324 27000 series 27000 series 27000 27000 27001 27001 27002 27002 27005 27005 27006 27006 27729 27729 28000 28000 29110 29148 29148 29199-2 29199-2 29500 29500 30170 30170 31000 31000 32000 32000 37001 37001 38500 38500 39075 39075 40500 40500 42010 42010 45001 45001 50001 50001 55000 55000 56000 56000 80000 80000 Category Category v v t t e e IEC standards 60027 60027 60034 60034 60038 60038 60062 60062 60063 60063 60068 60068 60112 60112 60228 60228 60269 60269 60297 60297 60309 60309 60320 60320 60364 60364 60446 60446 60559 60559 60601 60601 60870 60870-5 60870-6 60870 60870-5 60870-5 60870-6 60870-6 60906-1 60906-1 60908 60908 60929 60929 60958 60958 60980-344 60980-344 61030 61030 61131 61131-3 61131-9 61131 61131-3 61131-3 61131-9 61131-9 61158 61158 61162 61162 61334 61334 61355 61355 61360 61360 61400 61400 61499 61499 61508 61508 61511 61511 61784 61784 61850 61850 61851 61851 61883 61883 61960 61960 61968 61968 61970 61970 62014-4 62014-4 62026 62026 62056 62056 62061 62061 62196 62196 62262 62262 62264 62264 62304 62304 62325 62325 62351 62351 62365 62365 62366 62366 62379 62379 62386 62386 62455 62455 62680 62680 62682 62682 62700 62700 63110 63110 63119 63119 63382 63382 646 646 1989 1989 2022 2022 4909 4909 5218 5218 6429 6429 6523 6523 7810 7810 7811 7811 7812 7812 7813 7813 7816 7816 7942 7942 8613 8613 8632 8632 8652 8652 8859 8859 9126 9126 9293 9293 9496 9496 9529 9529 9592 9592 9593 9593 9899 9899 9945 9945 9995 9995 10021 10021 10116 10116 10165 10165 10179 10179 10279 10279 10646 10646 10967 10967 11172 11172 11179 11179 11404 11404 11544 11544 11801 11801 12207 12207 13250 13250 13346 13346 13522-5 13522-5 13568 13568 13816 13816 13818 13818 14443 14443 14496 14496 14651 14651 14882 14882 15288 15288 15291 15291 15408 15408 15444 15444 15445 15445 15504 15504 15511 15511 15693 15693 15897 15897 15938 15938 16262 16262 16485 16485 17024 17024 17025 17025 18004 18004 18014 18014 18181 18181 19752 19752 19757 19757 19770 19770 19788 19788 20000 20000 20802 20802 21000 21000 21827 21827 22275 22275 22537 22537 23000 23000 23003 23003 23008 23008 23270 23270 23360 23360 24707 24707 24727 24727 24744 24744 24752 24752 26300 26300 27000 27000 27000-series 27000-series 27002 27002 27040 27040 29110 29119 29119 33001 33001 38500 38500 39075 39075 42010 42010 80000 80000 81346 81346 International Electrotechnical Commission International Electrotechnical Commission Authority control databases Germany Germany Germany United States United States United States United States France France France BnF data BnF data BnF data Czech Republic Czech Republic Czech Republic Czech Republic Spain Spain Spain Israel Israel Israel ELMCIP ELMCIP ELMCIP Portal: Portal Computer programming Computer programming https://en.wikipedia.org/w/index.php?title=HTML&oldid=1264571829 Categories HTML HTML Computer-related introductions in 1990 Computer-related introductions in 1990 Markup languages Markup languages Open formats Open formats Technical communication Technical communication World Wide Web Consortium standards World Wide Web Consortium standards SGML SGML Webarchive template wayback links Webarchive template wayback links Articles with short description Articles with short description Short description is different from Wikidata Short description is different from Wikidata Wikipedia pages semi-protected against vandalism Wikipedia pages semi-protected against vandalism Articles containing potentially dated statements from 1997 Articles containing potentially dated statements from 1997 All articles containing potentially dated statements All articles containing potentially dated statements Articles containing potentially dated statements from 1996 Articles containing potentially dated statements from 1996 All articles lacking reliable references All articles lacking reliable references Articles lacking reliable references from February 2019 Articles lacking reliable references from February 2019 All articles with vague or ambiguous time All articles with vague or ambiguous time Vague or ambiguous time from March 2022 Vague or ambiguous time from March 2022 Articles to be expanded from January 2021 Articles to be expanded from January 2021 All articles with specifically marked weasel-worded phrases All articles with specifically marked weasel-worded phrases Articles with specifically marked weasel-worded phrases from June 2020 Articles with specifically marked weasel-worded phrases from June 2020 Pages using Sister project links with wikidata namespace mismatch Pages using Sister project links with wikidata namespace mismatch Pages using Sister project links with hidden wikidata Pages using Sister project links with hidden wikidata Articles with example code Articles with example code This page was last edited on 22 December 2024, at 12:57 (UTC). (UTC) Text is available under the Creative Commons Attribution-ShareAlike 4.0 License; additional terms may apply. By using this site, you agree to the Terms of Use and Privacy Policy. Wikipedia\u00ae is a registered trademark of the Wikimedia Foundation, Inc., a non-profit organization. Creative Commons Attribution-ShareAlike 4.0 License Terms of Use Privacy Policy Wikimedia Foundation, Inc. Privacy policy Privacy policy About Wikipedia About Wikipedia Disclaimers Disclaimers Contact Wikipedia Contact Wikipedia Code of Conduct Code of Conduct Developers Developers Statistics Statistics Cookie statement Cookie statement Mobile view Mobile view",
    "https://en.wikipedia.org/wiki/C%2B%2B": "Jump to content Main menu Main page Main page Main page Contents Contents Contents Current events Current events Current events Random article Random article Random article About Wikipedia About Wikipedia About Wikipedia Contact us Contact us Contact us Help Help Help Learn to edit Learn to edit Learn to edit Community portal Community portal Community portal Recent changes Recent changes Recent changes Upload file Upload file Upload file Search Search Appearance Donate Donate Donate Create account Create account Create account Log in Log in Log in Personal tools Donate Donate Donate Create account Create account Create account Log in Log in Log in learn more learn more Contributions Contributions Contributions Talk Talk Talk Contents (Top) (Top) 1 History Toggle History subsection 1.1 Etymology 1.2 Philosophy 1.3 Standardization 1 History 1 History Toggle History subsection 1.1 Etymology 1.1 Etymology 1.1 Etymology 1.2 Philosophy 1.2 Philosophy 1.2 Philosophy 1.3 Standardization 1.3 Standardization 1.3 Standardization 2 Language Toggle Language subsection 2.1 Object storage 2.1.1 Static storage duration objects 2.1.2 Thread storage duration objects 2.1.3 Automatic storage duration objects 2.1.4 Dynamic storage duration objects 2.2 Templates 2.3 Objects 2.3.1 Encapsulation 2.3.2 Inheritance 2.4 Operators and operator overloading 2.5 Polymorphism 2.5.1 Static polymorphism 2.5.2 Dynamic polymorphism 2.5.2.1 Inheritance 2.5.2.2 Virtual member functions 2.6 Lambda expressions 2.7 Exception handling 2.8 Enumerated types 2 Language 2 Language Toggle Language subsection 2.1 Object storage 2.1.1 Static storage duration objects 2.1.2 Thread storage duration objects 2.1.3 Automatic storage duration objects 2.1.4 Dynamic storage duration objects 2.1 Object storage 2.1 Object storage 2.1.1 Static storage duration objects 2.1.1 Static storage duration objects 2.1.1 Static storage duration objects 2.1.2 Thread storage duration objects 2.1.2 Thread storage duration objects 2.1.2 Thread storage duration objects 2.1.3 Automatic storage duration objects 2.1.3 Automatic storage duration objects 2.1.3 Automatic storage duration objects 2.1.4 Dynamic storage duration objects 2.1.4 Dynamic storage duration objects 2.1.4 Dynamic storage duration objects 2.2 Templates 2.2 Templates 2.2 Templates 2.3 Objects 2.3.1 Encapsulation 2.3.2 Inheritance 2.3 Objects 2.3 Objects 2.3.1 Encapsulation 2.3.1 Encapsulation 2.3.1 Encapsulation 2.3.2 Inheritance 2.3.2 Inheritance 2.3.2 Inheritance 2.4 Operators and operator overloading 2.4 Operators and operator overloading 2.4 Operators and operator overloading 2.5 Polymorphism 2.5.1 Static polymorphism 2.5.2 Dynamic polymorphism 2.5.2.1 Inheritance 2.5.2.2 Virtual member functions 2.5 Polymorphism 2.5 Polymorphism 2.5.1 Static polymorphism 2.5.1 Static polymorphism 2.5.1 Static polymorphism 2.5.2 Dynamic polymorphism 2.5.2.1 Inheritance 2.5.2.2 Virtual member functions 2.5.2 Dynamic polymorphism 2.5.2 Dynamic polymorphism 2.5.2.1 Inheritance 2.5.2.1 Inheritance 2.5.2.1 Inheritance 2.5.2.2 Virtual member functions 2.5.2.2 Virtual member functions 2.5.2.2 Virtual member functions 2.6 Lambda expressions 2.6 Lambda expressions 2.6 Lambda expressions 2.7 Exception handling 2.7 Exception handling 2.7 Exception handling 2.8 Enumerated types 2.8 Enumerated types 2.8 Enumerated types 3 Standard library 3 Standard library 3 Standard library 4 C++ Core Guidelines 4 C++ Core Guidelines 4 C++ Core Guidelines 5 Compatibility Toggle Compatibility subsection 5.1 With C 5 Compatibility 5 Compatibility Toggle Compatibility subsection 5.1 With C 5.1 With C 5.1 With C 6 See also 6 See also 6 See also 7 Footnotes 7 Footnotes 7 Footnotes 8 References 8 References 8 References 9 Further reading 9 Further reading 9 Further reading 10 External links 10 External links 10 External links Toggle the table of contents C++ C++ 109 languages Afrikaans Afrikaans Afrikaans \u0627\u0644\u0639\u0631\u0628\u064a\u0629 \u0627\u0644\u0639\u0631\u0628\u064a\u0629 \u0627\u0644\u0639\u0631\u0628\u064a\u0629 Aragon\u00e9s Aragon\u00e9s Aragon\u00e9s \u0985\u09b8\u09ae\u09c0\u09af\u09bc\u09be \u0985\u09b8\u09ae\u09c0\u09af\u09bc\u09be \u0985\u09b8\u09ae\u09c0\u09af\u09bc\u09be Asturianu Asturianu Asturianu Az\u0259rbaycanca Az\u0259rbaycanca Az\u0259rbaycanca \u062a\u06c6\u0631\u06a9\u062c\u0647 \u062a\u06c6\u0631\u06a9\u062c\u0647 \u062a\u06c6\u0631\u06a9\u062c\u0647 \u09ac\u09be\u0982\u09b2\u09be \u09ac\u09be\u0982\u09b2\u09be \u09ac\u09be\u0982\u09b2\u09be \u95a9\u5357\u8a9e / B\u00e2n-l\u00e2m-g\u00fa \u95a9\u5357\u8a9e / B\u00e2n-l\u00e2m-g\u00fa \u95a9\u5357\u8a9e / B\u00e2n-l\u00e2m-g\u00fa \u0411\u0435\u043b\u0430\u0440\u0443\u0441\u043a\u0430\u044f \u0411\u0435\u043b\u0430\u0440\u0443\u0441\u043a\u0430\u044f \u0411\u0435\u043b\u0430\u0440\u0443\u0441\u043a\u0430\u044f \u0411\u0435\u043b\u0430\u0440\u0443\u0441\u043a\u0430\u044f (\u0442\u0430\u0440\u0430\u0448\u043a\u0435\u0432\u0456\u0446\u0430) \u0411\u0435\u043b\u0430\u0440\u0443\u0441\u043a\u0430\u044f (\u0442\u0430\u0440\u0430\u0448\u043a\u0435\u0432\u0456\u0446\u0430) \u0411\u0435\u043b\u0430\u0440\u0443\u0441\u043a\u0430\u044f (\u0442\u0430\u0440\u0430\u0448\u043a\u0435\u0432\u0456\u0446\u0430) \u0411\u044a\u043b\u0433\u0430\u0440\u0441\u043a\u0438 \u0411\u044a\u043b\u0433\u0430\u0440\u0441\u043a\u0438 \u0411\u044a\u043b\u0433\u0430\u0440\u0441\u043a\u0438 \u0f56\u0f7c\u0f51\u0f0b\u0f61\u0f72\u0f42 \u0f56\u0f7c\u0f51\u0f0b\u0f61\u0f72\u0f42 \u0f56\u0f7c\u0f51\u0f0b\u0f61\u0f72\u0f42 Bosanski Bosanski Bosanski Brezhoneg Brezhoneg Brezhoneg Catal\u00e0 Catal\u00e0 Catal\u00e0 \u0427\u04d1\u0432\u0430\u0448\u043b\u0430 \u0427\u04d1\u0432\u0430\u0448\u043b\u0430 \u0427\u04d1\u0432\u0430\u0448\u043b\u0430 \u010ce\u0161tina \u010ce\u0161tina \u010ce\u0161tina Dansk Dansk Dansk Deutsch Deutsch Deutsch Eesti Eesti Eesti \u0395\u03bb\u03bb\u03b7\u03bd\u03b9\u03ba\u03ac \u0395\u03bb\u03bb\u03b7\u03bd\u03b9\u03ba\u03ac \u0395\u03bb\u03bb\u03b7\u03bd\u03b9\u03ba\u03ac Espa\u00f1ol Espa\u00f1ol Espa\u00f1ol Esperanto Esperanto Esperanto Euskara Euskara Euskara \u0641\u0627\u0631\u0633\u06cc \u0641\u0627\u0631\u0633\u06cc \u0641\u0627\u0631\u0633\u06cc Fran\u00e7ais Fran\u00e7ais Fran\u00e7ais Gaeilge Gaeilge Gaeilge Galego Galego Galego \u8d1b\u8a9e \u8d1b\u8a9e \u8d1b\u8a9e \u0a97\u0ac1\u0a9c\u0ab0\u0abe\u0aa4\u0ac0 \u0a97\u0ac1\u0a9c\u0ab0\u0abe\u0aa4\u0ac0 \u0a97\u0ac1\u0a9c\u0ab0\u0abe\u0aa4\u0ac0 \ud55c\uad6d\uc5b4 \ud55c\uad6d\uc5b4 \ud55c\uad6d\uc5b4 Hausa Hausa Hausa \u0540\u0561\u0575\u0565\u0580\u0565\u0576 \u0540\u0561\u0575\u0565\u0580\u0565\u0576 \u0540\u0561\u0575\u0565\u0580\u0565\u0576 \u0939\u093f\u0928\u094d\u0926\u0940 \u0939\u093f\u0928\u094d\u0926\u0940 \u0939\u093f\u0928\u094d\u0926\u0940 Hrvatski Hrvatski Hrvatski Ido Ido Ido Bahasa Indonesia Bahasa Indonesia Bahasa Indonesia Interlingua Interlingua Interlingua \u00cdslenska \u00cdslenska \u00cdslenska Italiano Italiano Italiano \u05e2\u05d1\u05e8\u05d9\u05ea \u05e2\u05d1\u05e8\u05d9\u05ea \u05e2\u05d1\u05e8\u05d9\u05ea Jawa Jawa Jawa \u0c95\u0ca8\u0ccd\u0ca8\u0ca1 \u0c95\u0ca8\u0ccd\u0ca8\u0ca1 \u0c95\u0ca8\u0ccd\u0ca8\u0ca1 \u10e5\u10d0\u10e0\u10d7\u10e3\u10da\u10d8 \u10e5\u10d0\u10e0\u10d7\u10e3\u10da\u10d8 \u10e5\u10d0\u10e0\u10d7\u10e3\u10da\u10d8 \u049a\u0430\u0437\u0430\u049b\u0448\u0430 \u049a\u0430\u0437\u0430\u049b\u0448\u0430 \u049a\u0430\u0437\u0430\u049b\u0448\u0430 Kiswahili Kiswahili Kiswahili \u041a\u044b\u0440\u0433\u044b\u0437\u0447\u0430 \u041a\u044b\u0440\u0433\u044b\u0437\u0447\u0430 \u041a\u044b\u0440\u0433\u044b\u0437\u0447\u0430 Latina Latina Latina Latvie\u0161u Latvie\u0161u Latvie\u0161u L\u00ebtzebuergesch L\u00ebtzebuergesch L\u00ebtzebuergesch Lietuvi\u0173 Lietuvi\u0173 Lietuvi\u0173 Lombard Lombard Lombard Magyar Magyar Magyar \u041c\u0430\u043a\u0435\u0434\u043e\u043d\u0441\u043a\u0438 \u041c\u0430\u043a\u0435\u0434\u043e\u043d\u0441\u043a\u0438 \u041c\u0430\u043a\u0435\u0434\u043e\u043d\u0441\u043a\u0438 \u0d2e\u0d32\u0d2f\u0d3e\u0d33\u0d02 \u0d2e\u0d32\u0d2f\u0d3e\u0d33\u0d02 \u0d2e\u0d32\u0d2f\u0d3e\u0d33\u0d02 \u092e\u0930\u093e\u0920\u0940 \u092e\u0930\u093e\u0920\u0940 \u092e\u0930\u093e\u0920\u0940 \u0645\u0635\u0631\u0649 \u0645\u0635\u0631\u0649 \u0645\u0635\u0631\u0649 Bahasa Melayu Bahasa Melayu Bahasa Melayu Minangkabau Minangkabau Minangkabau \u041c\u043e\u043d\u0433\u043e\u043b \u041c\u043e\u043d\u0433\u043e\u043b \u041c\u043e\u043d\u0433\u043e\u043b \u1019\u103c\u1014\u103a\u1019\u102c\u1018\u102c\u101e\u102c \u1019\u103c\u1014\u103a\u1019\u102c\u1018\u102c\u101e\u102c \u1019\u103c\u1014\u103a\u1019\u102c\u1018\u102c\u101e\u102c Na Vosa Vakaviti Na Vosa Vakaviti Na Vosa Vakaviti Nederlands Nederlands Nederlands \u0928\u0947\u092a\u093e\u0932 \u092d\u093e\u0937\u093e \u0928\u0947\u092a\u093e\u0932 \u092d\u093e\u0937\u093e \u0928\u0947\u092a\u093e\u0932 \u092d\u093e\u0937\u093e \u65e5\u672c\u8a9e \u65e5\u672c\u8a9e \u65e5\u672c\u8a9e Norsk bokm\u00e5l Norsk bokm\u00e5l Norsk bokm\u00e5l Norsk nynorsk Norsk nynorsk Norsk nynorsk Occitan Occitan Occitan O\u02bbzbekcha / \u045e\u0437\u0431\u0435\u043a\u0447\u0430 O\u02bbzbekcha / \u045e\u0437\u0431\u0435\u043a\u0447\u0430 O\u02bbzbekcha / \u045e\u0437\u0431\u0435\u043a\u0447\u0430 \u0a2a\u0a70\u0a1c\u0a3e\u0a2c\u0a40 \u0a2a\u0a70\u0a1c\u0a3e\u0a2c\u0a40 \u0a2a\u0a70\u0a1c\u0a3e\u0a2c\u0a40 P\u00e4lzisch P\u00e4lzisch P\u00e4lzisch \u067e\u0646\u062c\u0627\u0628\u06cc \u067e\u0646\u062c\u0627\u0628\u06cc \u067e\u0646\u062c\u0627\u0628\u06cc Polski Polski Polski Portugu\u00eas Portugu\u00eas Portugu\u00eas Qaraqalpaqsha Qaraqalpaqsha Qaraqalpaqsha Rom\u00e2n\u0103 Rom\u00e2n\u0103 Rom\u00e2n\u0103 Runa Simi Runa Simi Runa Simi \u0420\u0443\u0441\u0441\u043a\u0438\u0439 \u0420\u0443\u0441\u0441\u043a\u0438\u0439 \u0420\u0443\u0441\u0441\u043a\u0438\u0439 \u0421\u0430\u0445\u0430 \u0442\u044b\u043b\u0430 \u0421\u0430\u0445\u0430 \u0442\u044b\u043b\u0430 \u0421\u0430\u0445\u0430 \u0442\u044b\u043b\u0430 Shqip Shqip Shqip \u0dc3\u0dd2\u0d82\u0dc4\u0dbd \u0dc3\u0dd2\u0d82\u0dc4\u0dbd \u0dc3\u0dd2\u0d82\u0dc4\u0dbd Simple English Simple English Simple English Sloven\u010dina Sloven\u010dina Sloven\u010dina Sloven\u0161\u010dina Sloven\u0161\u010dina Sloven\u0161\u010dina \u06a9\u0648\u0631\u062f\u06cc \u06a9\u0648\u0631\u062f\u06cc \u06a9\u0648\u0631\u062f\u06cc \u0421\u0440\u043f\u0441\u043a\u0438 / srpski \u0421\u0440\u043f\u0441\u043a\u0438 / srpski \u0421\u0440\u043f\u0441\u043a\u0438 / srpski Srpskohrvatski / \u0441\u0440\u043f\u0441\u043a\u043e\u0445\u0440\u0432\u0430\u0442\u0441\u043a\u0438 Srpskohrvatski / \u0441\u0440\u043f\u0441\u043a\u043e\u0445\u0440\u0432\u0430\u0442\u0441\u043a\u0438 Srpskohrvatski / \u0441\u0440\u043f\u0441\u043a\u043e\u0445\u0440\u0432\u0430\u0442\u0441\u043a\u0438 Suomi Suomi Suomi Svenska Svenska Svenska \u0ba4\u0bae\u0bbf\u0bb4\u0bcd \u0ba4\u0bae\u0bbf\u0bb4\u0bcd \u0ba4\u0bae\u0bbf\u0bb4\u0bcd \u0422\u0430\u0442\u0430\u0440\u0447\u0430 / tatar\u00e7a \u0422\u0430\u0442\u0430\u0440\u0447\u0430 / tatar\u00e7a \u0422\u0430\u0442\u0430\u0440\u0447\u0430 / tatar\u00e7a \u0c24\u0c46\u0c32\u0c41\u0c17\u0c41 \u0c24\u0c46\u0c32\u0c41\u0c17\u0c41 \u0c24\u0c46\u0c32\u0c41\u0c17\u0c41 \u0e44\u0e17\u0e22 \u0e44\u0e17\u0e22 \u0e44\u0e17\u0e22 \u0422\u043e\u04b7\u0438\u043a\u04e3 \u0422\u043e\u04b7\u0438\u043a\u04e3 \u0422\u043e\u04b7\u0438\u043a\u04e3 T\u00fcrk\u00e7e T\u00fcrk\u00e7e T\u00fcrk\u00e7e T\u00fcrkmen\u00e7e T\u00fcrkmen\u00e7e T\u00fcrkmen\u00e7e Basa Ugi Basa Ugi Basa Ugi \u0423\u043a\u0440\u0430\u0457\u043d\u0441\u044c\u043a\u0430 \u0423\u043a\u0440\u0430\u0457\u043d\u0441\u044c\u043a\u0430 \u0423\u043a\u0440\u0430\u0457\u043d\u0441\u044c\u043a\u0430 \u0627\u0631\u062f\u0648 \u0627\u0631\u062f\u0648 \u0627\u0631\u062f\u0648 Ti\u1ebfng Vi\u1ec7t Ti\u1ebfng Vi\u1ec7t Ti\u1ebfng Vi\u1ec7t \u6587\u8a00 \u6587\u8a00 \u6587\u8a00 Winaray Winaray Winaray \u5434\u8bed \u5434\u8bed \u5434\u8bed \u05d9\u05d9\u05b4\u05d3\u05d9\u05e9 \u05d9\u05d9\u05b4\u05d3\u05d9\u05e9 \u05d9\u05d9\u05b4\u05d3\u05d9\u05e9 Yor\u00f9b\u00e1 Yor\u00f9b\u00e1 Yor\u00f9b\u00e1 \u7cb5\u8a9e \u7cb5\u8a9e \u7cb5\u8a9e \u017demait\u0117\u0161ka \u017demait\u0117\u0161ka \u017demait\u0117\u0161ka \u4e2d\u6587 \u4e2d\u6587 \u4e2d\u6587 Edit links Edit links Article Article Article Talk Talk Talk English Read Read Read Edit Edit Edit View history View history View history Tools Read Read Read Edit Edit Edit View history View history View history What links here What links here What links here Related changes Related changes Related changes Upload file Upload file Upload file Special pages Special pages Special pages Permanent link Permanent link Permanent link Page information Page information Page information Cite this page Cite this page Cite this page Get shortened URL Get shortened URL Get shortened URL Download QR code Download QR code Download QR code Download as PDF Download as PDF Download as PDF Printable version Printable version Printable version Wikimedia Commons Wikimedia Commons Wikimedia Commons Wikibooks Wikibooks Wikibooks Wikiquote Wikiquote Wikiquote Wikiversity Wikiversity Wikiversity Wikidata item Wikidata item Wikidata item CXX Paradigms Multi-paradigm procedural imperative functional object-oriented generic modular C Designed by Bjarne Stroustrup Developer ISO/IEC JTC 1 (Joint Technical Committee 1) / SC 22 (Subcommittee 22) ; 40 years ago (1985) 1985 Stable release C++23 ; 2 months ago (2024-10-19) 2024-10-19 Preview release C++26 ; 2 months ago (2024-10-16) 2024-10-16 Typing discipline Static strong nominative partially inferred OS Cross-platform Filename extensions .c++ .h++ [1] [ ] isocpp.org isocpp.org implementations GCC, LLVM Clang, Microsoft Visual C++, Embarcadero C++Builder, Intel C++ Compiler, IBM XL C++, EDG GCC LLVM Clang Microsoft Visual C++ Embarcadero C++Builder Intel C++ Compiler IBM XL C++ EDG Ada ALGOL 68 [2] [ ] BCPL [3] [ ] C CLU [2] [ ] F# [4] [ ] [note 1] [ ] ML Mesa [2] [ ] Modula-2 [2] [ ] Simula Smalltalk [2] [ ] Ada 95 C# [5] [ ] C99 Carbon Chapel [6] [ ] Clojure [7] [ ] D Java [8] [ ] JS++ [9] [ ] Lua [10] [ ] Nim [11] [ ] Objective-C++ Perl PHP Python [12] [ ] Rust [13] [ ] Seed7 C++ Programming at Wikibooks C++ Programming C++ (/\u02c8si\u02d0 pl\u028cs pl\u028cs/, pronounced \"C plus plus\" and sometimes abbreviated as CPP) is a high-level, general-purpose programming language created by Danish computer scientist Bjarne Stroustrup. First released in 1985 as an extension of the C programming language, it has since expanded significantly over time; as of 1997[update], C++ has object-oriented, generic, and functional features, in addition to facilities for low-level memory manipulation for systems like microcomputers or to make operating systems like Linux or Windows. It is usually implemented as a compiled language, and many vendors provide C++ compilers, including the Free Software Foundation, LLVM, Microsoft, Intel, Embarcadero, Oracle, and IBM.[14] /\u02c8si\u02d0 pl\u028cs pl\u028cs/ /\u02c8si\u02d0 pl\u028cs pl\u028cs/ /\u02c8si\u02d0 pl\u028cs pl\u028cs/ \u02c8si\u02d0 \u02c8 s i\u02d0 pl\u028cs p l \u028c s pl\u028cs p l \u028c s high-level general-purpose programming language Bjarne Stroustrup C programming language [update] object-oriented generic functional low-level memory microcomputers Linux Windows compiled language C++ compilers Free Software Foundation LLVM Microsoft Intel Embarcadero Oracle IBM [14] [ ] C++ was designed with systems programming and embedded, resource-constrained software and large systems in mind, with performance, efficiency, and flexibility of use as its design highlights.[15] C++ has also been found useful in many other contexts, with key strengths being software infrastructure and resource-constrained applications,[15] including desktop applications, video games, servers (e.g., e-commerce, web search, or databases), and performance-critical applications (e.g., telephone switches or space probes).[16] systems programming embedded performance [15] [ ] [15] [ ] desktop applications video games servers e-commerce web search databases telephone switches space probes [16] [ ] C++ is standardized by the International Organization for Standardization (ISO), with the latest standard version ratified and published by ISO in October 2024 as ISO/IEC 14882:2024 (informally known as C++23).[17] The C++ programming language was initially standardized in 1998 as ISO/IEC 14882:1998, which was then amended by the C++03, C++11, C++14, C++17, and C++20 standards. The current C++23 standard supersedes these with new features and an enlarged standard library. Before the initial standardization in 1998, C++ was developed by Stroustrup at Bell Labs since 1979 as an extension of the C language; he wanted an efficient and flexible language similar to C that also provided high-level features for program organization.[18] Since 2012, C++ has been on a three-year release schedule[19] with C++26 as the next planned standard.[20] International Organization for Standardization C++23 [17] [ ] C++03 C++11 C++14 C++17 C++20 C++23 C++23 standard library Bell Labs high-level features [18] [ ] [19] [ ] C++26 [20] [ ] Despite its widespread adoption, some notable programmers have criticized the C++ language, including Linus Torvalds,[21] Richard Stallman,[22] Joshua Bloch, Ken Thompson,[23][24][25] and Donald Knuth.[26][27] criticized the C++ language Linus Torvalds [21] [ ] Richard Stallman [22] [ ] Joshua Bloch Ken Thompson [23] [ ] [24] [ ] [25] [ ] Donald Knuth [26] [ ] [27] [ ] History [edit] [ edit edit ] 2000 In 1979, Bjarne Stroustrup, a Danish computer scientist, began work on \"C with Classes\", the predecessor to C++.[28] The motivation for creating a new language originated from Stroustrup's experience in programming for his PhD thesis. Stroustrup found that Simula had features that were very helpful for large software development, but the language was too slow for practical use, while BCPL was fast but too low-level to be suitable for large software development. When Stroustrup started working in AT&T Bell Labs, he had the problem of analyzing the UNIX kernel with respect to distributed computing. Remembering his PhD experience, Stroustrup set out to enhance the C language with Simula-like features.[29] C was chosen because it was general-purpose, fast, portable, and widely used. In addition to C and Simula's influences, other languages influenced this new language, including ALGOL 68, Ada, CLU, and ML.[citation needed] Bjarne Stroustrup computer scientist C with Classes C with Classes Classes [28] [ ] Simula BCPL AT&T Bell Labs UNIX kernel distributed computing C Simula [29] [ ] ALGOL 68 Ada CLU ML citation needed citation needed Initially, Stroustrup's \"C with Classes\" added features to the C compiler, Cpre, including classes, derived classes, strong typing, inlining, and default arguments.[30] classes derived classes strong typing inlining default arguments [30] [ ] In 1982, Stroustrup started to develop a successor to C with Classes, which he named \"C++\" (++ being the increment operator in C) after going through several other names. New features were added, including virtual functions, function name and operator overloading, references, constants, type-safe free-store memory allocation (new/delete), improved type checking, and BCPL-style single-line comments with two forward slashes (//). Furthermore, Stroustrup developed a new, standalone compiler for C++, Cfront. ++ increment operator virtual functions operator overloading references // Cfront In 1984, Stroustrup implemented the first stream input/output library. The idea of providing an output operator rather than a named output function was suggested by Doug McIlroy[2] (who had previously suggested Unix pipes). Doug McIlroy [2] [ ] Unix pipes In 1985, the first edition of The C++ Programming Language was released, which became the definitive reference for the language, as there was not yet an official standard.[31] The first commercial implementation of C++ was released in October of the same year.[28] The C++ Programming Language [31] [ ] [28] [ ] In 1989, C++ 2.0 was released, followed by the updated second edition of The C++ Programming Language in 1991.[32] New features in 2.0 included multiple inheritance, abstract classes, static member functions, const member functions, and protected members. In 1990, The Annotated C++ Reference Manual was published. This work became the basis for the future standard. Later feature additions included templates, exceptions, namespaces, new casts, and a Boolean type. [32] [ ] const member functions templates exceptions namespaces casts Boolean type In 1998, C++98 was released, standardizing the language, and a minor update (C++03) was released in 2003. C++03 After C++98, C++ evolved relatively slowly until, in 2011, the C++11 standard was released, adding numerous new features, enlarging the standard library further, and providing more facilities to C++ programmers. After a minor C++14 update released in December 2014, various new additions were introduced in C++17.[33] After becoming finalized in February 2020,[34] a draft of the C++20 standard was approved on 4 September 2020, and officially published on 15 December 2020.[35][36] C++11 C++14 C++14 C++17 [33] [ ] [34] [ ] [35] [ ] [36] [ ] On January 3, 2018, Stroustrup was announced as the 2018 winner of the Charles Stark Draper Prize for Engineering, \"for conceptualizing and developing the C++ programming language\".[37] Charles Stark Draper Prize [37] [ ] In December 2022, C++ ranked third on the TIOBE index, surpassing Java for the first time in the history of the index. As of November 2024[update], the language ranks second after Python, with Java being in third.[38] TIOBE index Java [update] Python [38] [ ] Etymology [edit] [ edit edit ] According to Stroustrup, \"the name signifies the evolutionary nature of the changes from C.\"[39] This name is credited to Rick Mascitti (mid-1983)[30] and was first used in December 1983. When Mascitti was questioned informally in 1992 about the naming, he indicated that it was given in a tongue-in-cheek spirit. The name comes from C's ++ operator (which increments the value of a variable) and a common naming convention of using \"+\" to indicate an enhanced computer program. [39] [ ] [30] [ ] tongue-in-cheek ++ operator increments value variable naming convention During C++'s development period, the language had been referred to as \"new C\" and \"C with Classes\"[30][40] before acquiring its final name. [30] [ ] [40] [ ] Philosophy [edit] [ edit edit ] Throughout C++'s life, its development and evolution has been guided by a set of principles:[29] [29] [ ] It must be driven by actual problems and its features should be immediately useful in real world programs. Every feature should be implementable (with a reasonably obvious way to do so). Programmers should be free to pick their own programming style, and that style should be fully supported by C++. Allowing a useful feature is more important than preventing every possible misuse of C++. It should provide facilities for organising programs into separate, well-defined parts, and provide facilities for combining separately developed parts. No implicit violations of the type system (but allow explicit violations; that is, those explicitly requested by the programmer). type system User-created types need to have the same support and performance as built-in types. Unused features should not negatively impact created executables (e.g. in lower performance). There should be no language beneath C++ (except assembly language). assembly language C++ should work alongside other existing programming languages, rather than fostering its own separate and incompatible programming environment. programming languages programming environment If the programmer's intent is unknown, allow the programmer to specify it by providing manual control. Standardization [edit] [ edit edit ] C++03 C++11 C++14 C++17 C++20 C++23 C++26 [41] [ ] [42] [ ] C++03 [43] [ ] C++11 [44] [ ] C++14 [45] [ ] C++17 [46] [ ] C++20 [17] [ ] C++23 C++26 C++ is standardized by an ISO working group known as JTC1/SC22/WG21. So far, it has published seven revisions of the C++ standard and is currently working on the next revision, C++26. ISO JTC1/SC22/WG21 C++26 In 1998, the ISO working group standardized C++ for the first time as ISO/IEC 14882:1998, which is informally known as C++98. In 2003, it published a new version of the C++ standard called ISO/IEC 14882:2003, which fixed problems identified in C++98. The next major revision of the standard was informally referred to as \"C++0x\", but it was not released until 2011.[47] C++11 (14882:2011) included many additions to both the core language and the standard library.[43] [47] [ ] C++11 [43] [ ] In 2014, C++14 (also known as C++1y) was released as a small extension to C++11, featuring mainly bug fixes and small improvements.[48] The Draft International Standard ballot procedures completed in mid-August 2014.[49] C++14 [48] [ ] [49] [ ] After C++14, a major revision C++17, informally known as C++1z, was completed by the ISO C++ committee in mid July 2017 and was approved and published in December 2017.[50] C++17 [50] [ ] As part of the standardization process, ISO also publishes technical reports and specifications: technical reports and specifications ISO/IEC TR 18015:2006[51] on the use of C++ in embedded systems and on performance implications of C++ language and library features, [51] [ ] ISO/IEC TR 19768:2007[52] (also known as the C++ Technical Report 1) on library extensions mostly integrated into C++11, [52] [ ] C++ Technical Report 1 C++11 ISO/IEC TR 29124:2010[53] on special mathematical functions, integrated into C++17, [53] [ ] C++17 C++17 ISO/IEC TR 24733:2011[54] on decimal floating-point arithmetic, [54] [ ] decimal floating-point ISO/IEC TS 18822:2015[55] on the standard filesystem library, integrated into C++17, [55] [ ] C++17 ISO/IEC TS 19570:2015[56] on parallel versions of the standard library algorithms, integrated into C++17, [56] [ ] parallel C++17 ISO/IEC TS 19841:2015[57] on software transactional memory, [57] [ ] transactional memory ISO/IEC TS 19568:2015[58] on a new set of library extensions, some of which are already integrated into C++17, [58] [ ] C++17 ISO/IEC TS 19217:2015[59] on the C++ concepts, integrated into C++20, [59] [ ] concepts C++20 ISO/IEC TS 19571:2016[60] on the library extensions for concurrency, some of which are already integrated into C++20, [60] [ ] C++20 ISO/IEC TS 19568:2017[61] on a new set of general-purpose library extensions, [61] [ ] ISO/IEC TS 21425:2017[62] on the library extensions for ranges, integrated into C++20, [62] [ ] C++20 ISO/IEC TS 22277:2017[63] on coroutines, integrated into C++20, [63] [ ] C++20 ISO/IEC TS 19216:2018[64] on the networking library, [64] [ ] ISO/IEC TS 21544:2018[65] on modules, integrated into C++20, [65] [ ] C++20 ISO/IEC TS 19570:2018[66] on a new set of library extensions for parallelism, and [66] [ ] ISO/IEC TS 23619:2021[67] on new extensions for reflective programming (reflection). [67] [ ] reflective programming More technical specifications are in development and pending approval, including new set of concurrency extensions. Language [edit] [ edit edit ] The C++ language has two main components: a direct mapping of hardware features provided primarily by the C subset, and zero-overhead abstractions based on those mappings. Stroustrup describes C++ as \"a light-weight abstraction programming language [designed] for building and using efficient and elegant abstractions\";[15] and \"offering both hardware access and abstraction is the basis of C++. Doing it efficiently is what distinguishes it from other languages.\"[68] [15] [ ] [68] [ ] C++ inherits most of C's syntax. A hello world program that conforms to the C standard is also a valid C++ hello world program. The following is Bjarne Stroustrup's version of the Hello world program that uses the C++ Standard Library stream facility to write a message to standard output:[69][70][note 2] C's syntax hello world program that conforms to the C standard Hello world program C++ Standard Library standard output [69] [ ] [70] [ ] [note 2] [ ] #include <iostream> int main () { std :: cout << \"Hello, world! \\n \" ; } Object storage [edit] [ edit edit ] As in C, C++ supports four types of memory management: static storage duration objects, thread storage duration objects, automatic storage duration objects, and dynamic storage duration objects.[71] memory management [71] [ ] Static storage duration objects [edit] [ edit edit ] Static storage duration objects are created before main() is entered (see exceptions below) and destroyed in reverse order of creation after main() exits. The exact order of creation is not specified by the standard (though there are some rules defined below) to allow implementations some freedom in how to organize their implementation. More formally, objects of this type have a lifespan that \"shall last for the duration of the program\".[72] [72] [ ] Static storage duration objects are initialized in two phases. First, \"static initialization\" is performed, and only after all static initialization is performed, \"dynamic initialization\" is performed. In static initialization, all objects are first initialized with zeros; after that, all objects that have a constant initialization phase are initialized with the constant expression (i.e. variables initialized with a literal or constexpr). Though it is not specified in the standard, the static initialization phase can be completed at compile time and saved in the data partition of the executable. Dynamic initialization involves all object initialization done via a constructor or function call (unless the function is marked with constexpr, in C++11). The dynamic initialization order is defined as the order of declaration within the compilation unit (i.e. the same file). No guarantees are provided about the order of initialization between compilation units. Thread storage duration objects [edit] [ edit edit ] Variables of this type are very similar to static storage duration objects. The main difference is the creation time is just before thread creation, and destruction is done after the thread has been joined.[73] [73] [ ] Automatic storage duration objects [edit] [ edit edit ] The most common variable types in C++ are local variables inside a function or block, and temporary variables.[74] The common feature about automatic variables is that they have a lifetime that is limited to the scope of the variable. They are created and potentially initialized at the point of declaration (see below for details) and destroyed in the reverse order of creation when the scope is left. This is implemented by allocation on the stack. local variables function [74] [ ] stack Local variables are created as the point of execution passes the declaration point. If the variable has a constructor or initializer this is used to define the initial state of the object. Local variables are destroyed when the local block or function that they are declared in is closed. C++ destructors for local variables are called at the end of the object lifetime, allowing a discipline for automatic resource management termed RAII, which is widely used in C++. RAII Member variables are created when the parent object is created. Array members are initialized from 0 to the last member of the array in order. Member variables are destroyed when the parent object is destroyed in the reverse order of creation. i.e. If the parent is an \"automatic object\" then it will be destroyed when it goes out of scope which triggers the destruction of all its members. Temporary variables are created as the result of expression evaluation and are destroyed when the statement containing the expression has been fully evaluated (usually at the ; at the end of a statement). Dynamic storage duration objects [edit] [ edit edit ] new and delete (C++) These objects have a dynamic lifespan and can be created directly with a call to new and destroyed explicitly with a call to delete.[75] C++ also supports malloc and free, from C, but these are not compatible with new and delete. Use of new returns an address to the allocated memory. The C++ Core Guidelines advise against using new directly for creating dynamic objects in favor of smart pointers through make_unique<T> for single ownership and make_shared<T> for reference-counted multiple ownership,[76] which were introduced in C++11. new delete [75] [ ] new delete new new make_unique < T > make_shared < T > [76] [ ] Templates [edit] [ edit edit ] Template metaprogramming Generic programming C++ templates enable generic programming. C++ supports function, class, alias, and variable templates. Templates may be parameterized by types, compile-time constants, and other templates. Templates are implemented by instantiation at compile-time. To instantiate a template, compilers substitute specific arguments for a template's parameters to generate a concrete function or class instance. Some substitutions are not possible; these are eliminated by an overload resolution policy described by the phrase \"Substitution failure is not an error\" (SFINAE). Templates are a powerful tool that can be used for generic programming, template metaprogramming, and code optimization, but this power implies a cost. Template use may increase object code size, because each template instantiation produces a copy of the template code: one for each set of template arguments, however, this is the same or smaller amount of code that would be generated if the code were written by hand.[77] This is in contrast to run-time generics seen in other languages (e.g., Java) where at compile-time the type is erased and a single template body is preserved. C++ templates generic programming C++ Substitution failure is not an error generic programming template metaprogramming object code [77] [ ] Java Templates are different from macros: while both of these compile-time language features enable conditional compilation, templates are not restricted to lexical substitution. Templates are aware of the semantics and type system of their companion language, as well as all compile-time type definitions, and can perform high-level operations including programmatic flow control based on evaluation of strictly type-checked parameters. Macros are capable of conditional control over compilation based on predetermined criteria, but cannot instantiate new types, recurse, or perform type evaluation and in effect are limited to pre-compilation text-substitution and text-inclusion/exclusion. In other words, macros can control compilation flow based on pre-defined symbols but cannot, unlike templates, independently instantiate new symbols. Templates are a tool for static polymorphism (see below) and generic programming. macros polymorphism generic programming In addition, templates are a compile-time mechanism in C++ that is Turing-complete, meaning that any computation expressible by a computer program can be computed, in some form, by a template metaprogram before runtime. Turing-complete template metaprogram In summary, a template is a compile-time parameterized function or class written without knowledge of the specific arguments used to instantiate it. After instantiation, the resulting code is equivalent to code written specifically for the passed arguments. In this manner, templates provide a way to decouple generic, broadly applicable aspects of functions and classes (encoded in templates) from specific aspects (encoded in template parameters) without sacrificing performance due to abstraction. Objects [edit] [ edit edit ] C++ classes C++ introduces object-oriented programming (OOP) features to C. It offers classes, which provide the four features commonly present in OOP (and some non-OOP) languages: abstraction, encapsulation, inheritance, and polymorphism. One distinguishing feature of C++ classes compared to classes in other programming languages is support for deterministic destructors, which in turn provide support for the Resource Acquisition is Initialization (RAII) concept. object-oriented programming classes abstraction encapsulation inheritance polymorphism C++ destructors Resource Acquisition is Initialization Encapsulation [edit] [ edit edit ] Encapsulation is the hiding of information to ensure that data structures and operators are used as intended and to make the usage model more obvious to the developer. C++ provides the ability to define classes and functions as its primary encapsulation mechanisms. Within a class, members can be declared as either public, protected, or private to explicitly enforce encapsulation. A public member of the class is accessible to any function. A private member is accessible only to functions that are members of that class and to functions and classes explicitly granted access permission by the class (\"friends\"). A protected member is accessible to members of classes that inherit from the class in addition to the class itself and any friends. Encapsulation The object-oriented principle ensures the encapsulation of all and only the functions that access the internal representation of a type. C++ supports this principle via member functions and friend functions, but it does not enforce it. Programmers can declare parts or all of the representation of a type to be public, and they are allowed to make public entities not part of the representation of a type. Therefore, C++ supports not just object-oriented programming, but other decomposition paradigms such as modular programming. modular programming It is generally considered good practice to make all data private or protected, and to make public only those functions that are part of a minimal interface for users of the class. This can hide the details of data implementation, allowing the designer to later fundamentally change the implementation without changing the interface in any way.[78][79] data [78] [ ] [79] [ ] Inheritance [edit] [ edit edit ] Inheritance allows one data type to acquire properties of other data types. Inheritance from a base class may be declared as public, protected, or private. This access specifier determines whether unrelated and derived classes can access the inherited public and protected members of the base class. Only public inheritance corresponds to what is usually meant by \"inheritance\". The other two forms are much less frequently used. If the access specifier is omitted, a \"class\" inherits privately, while a \"struct\" inherits publicly. Base classes may be declared as virtual; this is called virtual inheritance. Virtual inheritance ensures that only one instance of a base class exists in the inheritance graph, avoiding some of the ambiguity problems of multiple inheritance. Inheritance base class virtual inheritance Multiple inheritance is a C++ feature allowing a class to be derived from more than one base class; this allows for more elaborate inheritance relationships. For example, a \"Flying Cat\" class can inherit from both \"Cat\" and \"Flying Mammal\". Some other languages, such as C# or Java, accomplish something similar (although more limited) by allowing inheritance of multiple interfaces while restricting the number of base classes to one (interfaces, unlike classes, provide only declarations of member functions, no implementation or member data). An interface as in C# and Java can be defined in C++ as a class containing only pure virtual functions, often known as an abstract base class or \"ABC\". The member functions of such an abstract base class are normally explicitly defined in the derived class, not inherited implicitly. C++ virtual inheritance exhibits an ambiguity resolution feature called dominance. Multiple inheritance C# Java interfaces C++ abstract base class dominance Operators and operator overloading [edit] [ edit edit ] :: ?: . . * sizeof sizeof typeid typeid Operators in C and C++ C++ provides more than 35 operators, covering basic arithmetic, bit manipulation, indirection, comparisons, logical operations and others. Almost all operators can be overloaded for user-defined types, with a few notable exceptions such as member access (. and .*) and the conditional operator. The rich set of overloadable operators is central to making user-defined types in C++ seem like built-in types. overloaded Overloadable operators are also an essential part of many advanced C++ programming techniques, such as smart pointers. Overloading an operator does not change the precedence of calculations involving the operator, nor does it change the number of operands that the operator uses (any operand may however be ignored by the operator, though it will be evaluated prior to execution). Overloaded \"&&\" and \"||\" operators lose their short-circuit evaluation property. smart pointers short-circuit evaluation Polymorphism [edit] [ edit edit ] Polymorphism (computer science) Polymorphism enables one common interface for many implementations, and for objects to act differently under different circumstances. Polymorphism C++ supports several kinds of static (resolved at compile-time) and dynamic (resolved at run-time) polymorphisms, supported by the language features described above. Compile-time polymorphism does not allow for certain run-time decisions, while runtime polymorphism typically incurs a performance penalty. compile-time run-time polymorphisms Compile-time polymorphism runtime polymorphism Static polymorphism [edit] [ edit edit ] Parametric polymorphism ad hoc polymorphism Function overloading allows programs to declare multiple functions having the same name but with different arguments (i.e. ad hoc polymorphism). The functions are distinguished by the number or types of their formal parameters. Thus, the same function name can refer to different functions depending on the context in which it is used. The type returned by the function is not used to distinguish overloaded functions and differing return types would result in a compile-time error message. Function overloading ad hoc polymorphism formal parameters When declaring a function, a programmer can specify for one or more parameters a default value. Doing so allows the parameters with defaults to optionally be omitted when the function is called, in which case the default arguments will be used. When a function is called with fewer arguments than there are declared parameters, explicit arguments are matched to parameters in left-to-right order, with any unmatched parameters at the end of the parameter list being assigned their default arguments. In many cases, specifying default arguments in a single function declaration is preferable to providing overloaded function definitions with different numbers of parameters. default value Templates in C++ provide a sophisticated mechanism for writing generic, polymorphic code (i.e. parametric polymorphism). In particular, through the curiously recurring template pattern, it is possible to implement a form of static polymorphism that closely mimics the syntax for overriding virtual functions. Because C++ templates are type-aware and Turing-complete, they can also be used to let the compiler resolve recursive conditionals and generate substantial programs through template metaprogramming. Contrary to some opinion, template code will not generate a bulk code after compilation with the proper compiler settings.[77] Templates in C++ parametric polymorphism curiously recurring template pattern Turing-complete template metaprogramming [77] [ ] Dynamic polymorphism [edit] [ edit edit ] Inheritance [edit] [ edit edit ] Subtyping Variable pointers and references to a base class type in C++ can also refer to objects of any derived classes of that type. This allows arrays and other kinds of containers to hold pointers to objects of differing types (references cannot be directly held in containers). This enables dynamic (run-time) polymorphism, where the referred objects can behave differently, depending on their (actual, derived) types. C++ also provides the dynamic_cast operator, which allows code to safely attempt conversion of an object, via a base reference/pointer, to a more derived type: downcasting. The attempt is necessary as often one does not know which derived type is referenced. (Upcasting, conversion to a more general type, can always be checked/performed at compile-time via static_cast, as ancestral classes are specified in the derived class's interface, visible to all callers.) dynamic_cast relies on run-time type information (RTTI), metadata in the program that enables differentiating types and their relationships. If a dynamic_cast to a pointer fails, the result is the nullptr constant, whereas if the destination is a reference (which cannot be null), the cast throws an exception. Objects known to be of a certain derived type can be cast to that with static_cast, bypassing RTTI and the safe runtime type-checking of dynamic_cast, so this should be used only if the programmer is very confident the cast is, and will always be, valid. dynamic_cast static_cast dynamic_cast run-time type information dynamic_cast nullptr static_cast dynamic_cast Virtual member functions [edit] [ edit edit ] Ordinarily, when a function in a derived class overrides a function in a base class, the function to call is determined by the type of the object. A given function is overridden when there exists no difference in the number or type of parameters between two or more definitions of that function. Hence, at compile time, it may not be possible to determine the type of the object and therefore the correct function to call, given only a base class pointer; the decision is therefore put off until runtime. This is called dynamic dispatch. Virtual member functions or methods[80] allow the most specific implementation of the function to be called, according to the actual run-time type of the object. In C++ implementations, this is commonly done using virtual function tables. If the object type is known, this may be bypassed by prepending a fully qualified class name before the function call, but in general calls to virtual functions are resolved at run time. overrides dynamic dispatch Virtual member functions [80] [ ] virtual function tables fully qualified class name In addition to standard member functions, operator overloads and destructors can be virtual. An inexact rule based on practical experience states that if any function in the class is virtual, the destructor should be as well. As the type of an object at its creation is known at compile time, constructors, and by extension copy constructors, cannot be virtual. Nonetheless, a situation may arise where a copy of an object needs to be created when a pointer to a derived object is passed as a pointer to a base object. In such a case, a common solution is to create a clone() (or similar) virtual function that creates and returns a copy of the derived class when called. clone () A member function can also be made \"pure virtual\" by appending it with = 0 after the closing parenthesis and before the semicolon. A class containing a pure virtual function is called an abstract class. Objects cannot be created from an abstract class; they can only be derived from. Any derived class inherits the virtual function as pure and must provide a non-pure definition of it (and all other pure virtual functions) before objects of the derived class can be created. A program that attempts to create an object of a class with a pure virtual member function or inherited pure virtual member function is ill-formed. = 0 Lambda expressions [edit] [ edit edit ] C++ provides support for anonymous functions, also known as lambda expressions, with the following form: anonymous functions lambda expressions [ capture ]( parameters ) -> return_type { function_body } Since C++20, the keyword template is optional for template parameters of lambda expressions: template [ capture ] < template_parameters > ( parameters ) -> return_type { function_body } If the lambda takes no parameters, and no return type or other specifiers are used, the () can be omitted; that is, [ capture ] { function_body } The return type of a lambda expression can be automatically inferred, if possible; e.g.: []( int x , int y ) { return x + y ; } // inferred []( int x , int y ) -> int { return x + y ; } // explicit The [capture] list supports the definition of closures. Such lambda expressions are defined in the standard as syntactic sugar for an unnamed function object. [ capture ] closures syntactic sugar function object Exception handling [edit] [ edit edit ] Exception handling is used to communicate the existence of a runtime problem or error from where it was detected to where the issue can be handled.[81] It permits this to be done in a uniform manner and separately from the main code, while detecting all errors.[82] Should an error occur, an exception is thrown (raised), which is then caught by the nearest suitable exception handler. The exception causes the current scope to be exited, and also each outer scope (propagation) until a suitable handler is found, calling in turn the destructors of any objects in these exited scopes.[83] At the same time, an exception is presented as an object carrying the data about the detected problem.[84] [81] [ ] [82] [ ] [83] [ ] [84] [ ] Some C++ style guides, such as Google's,[85] LLVM's,[86] and Qt's,[87] forbid the usage of exceptions. [85] [ ] [86] [ ] [87] [ ] The exception-causing code is placed inside a try block. The exceptions are handled in separate catch blocks (the handlers); each try block can have multiple exception handlers, as it is visible in the example below.[88] try catch try [88] [ ] #include <iostream> #include <vector> #include <stdexcept> int main () { try { std :: vector < int > vec { 3 , 4 , 3 , 1 }; int i { vec . at ( 4 )}; // Throws an exception, std::out_of_range (indexing for vec is from 0-3 not 1-4) } // An exception handler, catches std::out_of_range, which is thrown by vec.at(4) catch ( const std :: out_of_range & e ) { std :: cerr << \"Accessing a non-existent element: \" << e . what () << '\\n' ; } // To catch any other standard library exceptions (they derive from std::exception) catch ( const std :: exception & e ) { std :: cerr << \"Exception thrown: \" << e . what () << '\\n' ; } // Catch any unrecognised exceptions (i.e. those which don't derive from std::exception) catch (...) { std :: cerr << \"Some fatal error \\n \" ; } } It is also possible to raise exceptions purposefully, using the throw keyword; these exceptions are handled in the usual way. In some cases, exceptions cannot be used due to technical reasons. One such example is a critical component of an embedded system, where every operation must be guaranteed to complete within a specified amount of time. This cannot be determined with exceptions as no tools exist to determine the maximum time required for an exception to be handled.[89] throw [89] [ ] Unlike signal handling, in which the handling function is called from the point of failure, exception handling exits the current scope before the catch block is entered, which may be located in the current function or any of the previous function calls currently on the stack. signal handling Enumerated types [edit] [ edit edit ] Enumerated type \u00a7 C++ [edit] [ edit ] C++ has enumeration types that are directly inherited from C's and work mostly like these, except that an enumeration is a real type in C++, giving added compile-time checking. Also (as with structs), the C++ enum keyword is combined with a typedef, so that instead of naming the type enum name, simply name it name. This can be simulated in C using a typedef: typedef enum {Value1, Value2} name; typedef typedef typedef enum { Value1 , Value2 } name ; C++11 also provides a second kind of enumeration, called a scoped enumeration. These are type-safe: the enumerators are not implicitly converted to an integer type. Among other things, this allows I/O streaming to be defined for the enumeration type. Another feature of scoped enumerations is that the enumerators do not leak, so usage requires prefixing with the name of the enumeration (e.g., Color::Red for the first enumerator in the example below), unless a using enum declaration (introduced in C++20) has been used to bring the enumerators into the current scope. A scoped enumeration is specified by the phrase enum class (or enum struct). For example: C++11 C++20 enum class Color { Red , Green , Blue }; The underlying type of an enumeration is an implementation-defined integral type that is large enough to hold all enumerated values; it does not have to be the smallest possible type. The underlying type can be specified directly, which allows \"forward declarations\" of enumerations: enum class Color : long { Red , Green , Blue }; // must fit in size and memory layout the type 'long' enum class Shapes : char ; // forward declaration. If later there are values defined that don't fit in 'char' it is an error. Standard library [edit] [ edit edit ] C++ Standard Library The C++ standard consists of two parts: the core language and the standard library. C++ programmers expect the latter on every major implementation of C++; it includes aggregate types (vectors, lists, maps, sets, queues, stacks, arrays, tuples), algorithms (find, for_each, binary_search, random_shuffle, etc.), input/output facilities (iostream, for reading from and writing to the console and files), filesystem library, localisation support, smart pointers for automatic memory management, regular expression support, multi-threading library, atomics support (allowing a variable to be read or written to by at most one thread at a time without any external synchronisation), time utilities (measurement, getting current time, etc.), a system for converting error reporting that does not use C++ exceptions into C++ exceptions, a random number generator, and a slightly modified version of the C standard library (to make it comply with the C++ type system). standard vectors algorithms for_each binary_search iostream smart pointers regular expression multi-threading exceptions random number generator C standard library A large part of the C++ library is based on the Standard Template Library (STL). Useful tools provided by the STL include containers as the collections of objects (such as vectors and lists), iterators that provide array-like access to containers, and algorithms that perform operations such as searching and sorting. Standard Template Library containers vectors lists iterators algorithms Furthermore, (multi)maps (associative arrays) and (multi)sets are provided, all of which export compatible interfaces. Therefore, using templates it is possible to write generic algorithms that work with any container or on any sequence defined by iterators. As in C, the features of the library are accessed by using the #include directive to include a standard header. The C++ Standard Library provides 105 standard headers, of which 27 are deprecated. associative arrays features library #include directive standard header C++ Standard Library The standard incorporates the STL that was originally designed by Alexander Stepanov, who experimented with generic algorithms and containers for many years. When he started with C++, he finally found a language where it was possible to create generic algorithms (e.g., STL sort) that perform even better than, for example, the C standard library qsort, thanks to C++ features like using inlining and compile-time binding instead of function pointers. The standard does not refer to it as \"STL\", as it is merely a part of the standard library, but the term is still widely used to distinguish it from the rest of the standard library (input/output streams, internationalization, diagnostics, the C library subset, etc.).[90] Alexander Stepanov [90] [ ] Most C++ compilers, and all major ones, provide a standards-conforming implementation of the C++ standard library. C++ Core Guidelines [edit] [ edit edit ] The C++ Core Guidelines[91] are an initiative led by Bjarne Stroustrup, the inventor of C++, and Herb Sutter, the convener and chair of the C++ ISO Working Group, to help programmers write 'Modern C++' by using best practices for the language standards C++11 and newer, and to help developers of compilers and static checking tools to create rules for catching bad programming practices. [91] [ ] The main aim is to efficiently and consistently write type and resource safe C++. The Core Guidelines were announced[92] in the opening keynote at CPPCon 2015. [92] [ ] The Guidelines are accompanied by the Guideline Support Library (GSL),[93] a header only library of types and functions to implement the Core Guidelines and static checker tools for enforcing Guideline rules.[94] [93] [ ] [94] [ ] Compatibility [edit] [ edit edit ] To give compiler vendors greater freedom, the C++ standards committee decided not to dictate the implementation of name mangling, exception handling, and other implementation-specific features. The downside of this decision is that object code produced by different compilers is expected to be incompatible. There are, however, attempts to standardize compilers for particular machines or operating systems. For example, the Itanium C++ ABI is processor-independent (despite its name) and is implemented by GCC and Clang.[95] name mangling exception handling object code compilers operating systems [95] [ ] With C [edit] [ edit edit ] Compatibility of C and C++ C++ is often considered to be a superset of C but this is not strictly true.[96] Most C code can easily be made to compile correctly in C++ but there are a few differences that cause some valid C code to be invalid or behave differently in C++. For example, C allows implicit conversion from void* to other pointer types but C++ does not (for type safety reasons). Also, C++ defines many new keywords, such as new and class, which may be used as identifiers (for example, variable names) in a C program. C [96] [ ] void * new class Some incompatibilities have been removed by the 1999 revision of the C standard (C99), which now supports C++ features such as line comments (//) and declarations mixed with code. On the other hand, C99 introduced a number of new features that C++ did not support that were incompatible or redundant in C++, such as variable-length arrays, native complex-number types (however, the std::complex class in the C++ standard library provides similar functionality, although not code-compatible), designated initializers, compound literals, and the restrict keyword.[97] Some of the C99-introduced features were included in the subsequent version of the C++ standard, C++11 (out of those which were not redundant).[98][99][100] However, the C++11 standard introduces new incompatibilities, such as disallowing assignment of a string literal to a character pointer, which remains valid C. C99 // variable-length arrays std :: complex compound literals restrict [97] [ ] C++11 [98] [ ] [99] [ ] [100] [ ] To intermix C and C++ code, any function declaration or definition that is to be called from/used both in C and C++ must be declared with C linkage by placing it within an extern \"C\" {/*...*/} block. Such a function may not rely on features depending on name mangling (i.e., function overloading). extern \"C\" { /*...*/ } name mangling See also [edit] [ edit edit ] Computer programming portal Computer programming portal Computer programming portal Carbon (programming language) Carbon (programming language) Comparison of programming languages Comparison of programming languages List of C++ compilers List of C++ compilers Outline of C++ Outline of C++ Category:C++ libraries Category:C++ libraries Footnotes [edit] [ edit edit ] ^ For the idea of the C++20 stackless coroutines. ^ ^ For the idea of the C++20 stackless coroutines. ^ This code is copied directly from Bjarne Stroustrup's errata page (p. 633). He addresses the use of '\\n' rather than std::endl. Also see Can I write \"void main()\"? Archived 2 July 2020 at the Wayback Machine for an explanation of the implicit return 0; in the main function. This implicit return is not available in other functions. ^ ^ This code is copied directly from Bjarne Stroustrup's errata page (p. 633). He addresses the use of '\\n' rather than std::endl. Also see Can I write \"void main()\"? Archived 2 July 2020 at the Wayback Machine for an explanation of the implicit return 0; in the main function. This implicit return is not available in other functions. Can I write \"void main()\"? Archived Wayback Machine References [edit] [ edit edit ] ^ \"Overview of modules in C++\". Microsoft. 24 April 2023. ^ ^ \"Overview of modules in C++\". Microsoft. 24 April 2023. \"Overview of modules in C++\" ^ a b c d e f Stroustrup, Bjarne (1996). \"A history of C++: 1979-1991\". History of programming languages---II. ACM. pp. 699\u2013769. doi:10.1145/234286.1057836. ^ a b c d e f a b c d e f Stroustrup, Bjarne (1996). \"A history of C++: 1979-1991\". History of programming languages---II. ACM. pp. 699\u2013769. doi:10.1145/234286.1057836. ACM 699\u2013 doi 10.1145/234286.1057836 10.1145/234286.1057836 ^ Stroustrup, Bjarne (16 December 2021). \"C++20: Reaching for the Aims of C++ - Bjarne Stroustrup - CppCon 2021\". CppCon. Archived from the original on 30 December 2021. Retrieved 30 December 2021. ^ ^ Stroustrup, Bjarne (16 December 2021). \"C++20: Reaching for the Aims of C++ - Bjarne Stroustrup - CppCon 2021\". CppCon. Archived from the original on 30 December 2021. Retrieved 30 December 2021. \"C++20: Reaching for the Aims of C++ - Bjarne Stroustrup - CppCon 2021\" Archived . Retrieved 30 December 2021 30 December ^ Stroustrup, Bjarne (12 June 2020). \"Thriving in a crowded and changing world: C++ 2006\u20132020\". Proceedings of the ACM on Programming Languages. 4 (HOPL). Association for Computing Machinery (ACM): 1\u2013168. doi:10.1145/3386320. ISSN 2475-1421. S2CID 219603741. ^ ^ Stroustrup, Bjarne (12 June 2020). \"Thriving in a crowded and changing world: C++ 2006\u20132020\". Proceedings of the ACM on Programming Languages. 4 (HOPL). Association for Computing Machinery (ACM): 1\u2013168. doi:10.1145/3386320. ISSN 2475-1421. S2CID 219603741. \"Thriving in a crowded and changing world: C++ 2006\u20132020\" 1\u2013 doi 10.1145/3386320 10.1145/3386320 ISSN 2475-1421 S2CID 219603741 ^ Naugler, David (May 2007). \"C# 2.0 for C++ and Java programmer: conference workshop\". Journal of Computing Sciences in Colleges. 22 (5). Although C# has been strongly influenced by Java it has also been strongly influenced by C++ and is best viewed as a descendant of both C++ and Java. ^ ^ Naugler, David (May 2007). \"C# 2.0 for C++ and Java programmer: conference workshop\". Journal of Computing Sciences in Colleges. 22 (5). Although C# has been strongly influenced by Java it has also been strongly influenced by C++ and is best viewed as a descendant of both C++ and Java. ^ \"Chapel spec (Acknowledgements)\" (PDF). Cray Inc. 1 October 2015. Archived (PDF) from the original on 24 June 2018. Retrieved 14 January 2016. ^ ^ \"Chapel spec (Acknowledgements)\" (PDF). Cray Inc. 1 October 2015. Archived (PDF) from the original on 24 June 2018. Retrieved 14 January 2016. \"Chapel spec (Acknowledgements)\" (PDF) Archived (PDF) . Retrieved 14 January 2016 14 January ^ Fogus, Michael. \"Rich Hickey Q&A\". Code Quarterly. Archived from the original on 11 January 2017. Retrieved 11 January 2017. ^ ^ Fogus, Michael. \"Rich Hickey Q&A\". Code Quarterly. Archived from the original on 11 January 2017. Retrieved 11 January 2017. \"Rich Hickey Q&A\" the original . Retrieved 11 January 2017 11 January ^ Harry. H. Chaudhary (28 July 2014). \"Cracking The Java Programming Interview :: 2000+ Java Interview Que/Ans\". Archived from the original on 27 May 2021. Retrieved 29 May 2016. ^ ^ Harry. H. Chaudhary (28 July 2014). \"Cracking The Java Programming Interview :: 2000+ Java Interview Que/Ans\". Archived from the original on 27 May 2021. Retrieved 29 May 2016. \"Cracking The Java Programming Interview :: 2000+ Java Interview Que/Ans\" Archived . Retrieved 29 May 2016 29 May ^ Roger Poon (1 May 2017). \"Scaling JS++: Abstraction, Performance, and Readability\". Archived from the original on 11 May 2020. Retrieved 21 April 2020. ^ ^ Roger Poon (1 May 2017). \"Scaling JS++: Abstraction, Performance, and Readability\". Archived from the original on 11 May 2020. Retrieved 21 April 2020. \"Scaling JS++: Abstraction, Performance, and Readability\" Archived . Retrieved 21 April 2020 21 April ^ \"The evolution of an extension language: a history of Lua\". www.lua.org. Retrieved 4 January 2023. ^ ^ \"The evolution of an extension language: a history of Lua\". www.lua.org. Retrieved 4 January 2023. \"The evolution of an extension language: a history of Lua\" . Retrieved 4 January 2023 4 January ^ \"FAQ Nim Programming Language\". Archived from the original on 11 July 2017. Retrieved 21 April 2020. ^ ^ \"FAQ Nim Programming Language\". Archived from the original on 11 July 2017. Retrieved 21 April 2020. \"FAQ Nim Programming Language\" Archived . Retrieved 21 April 2020 21 April ^ \"9. Classes \u2014 Python 3.6.4 documentation\". docs.python.org. Archived from the original on 23 October 2012. Retrieved 9 January 2018. ^ ^ \"9. Classes \u2014 Python 3.6.4 documentation\". docs.python.org. Archived from the original on 23 October 2012. Retrieved 9 January 2018. \"9. Classes \u2014 Python 3.6.4 documentation\" Archived . Retrieved 9 January 2018 9 January ^ \"Influences - The Rust Reference\". doc.rust-lang.org. Retrieved 4 January 2023. ^ ^ \"Influences - The Rust Reference\". doc.rust-lang.org. Retrieved 4 January 2023. \"Influences - The Rust Reference\" . Retrieved 4 January 2023 4 January ^ Stroustrup, Bjarne (1997). \"1\". The C++ Programming Language (Third ed.). Addison-Wesley. ISBN 0-201-88954-4. OCLC 59193992. ^ ^ Stroustrup, Bjarne (1997). \"1\". The C++ Programming Language (Third ed.). Addison-Wesley. ISBN 0-201-88954-4. OCLC 59193992. Stroustrup, Bjarne The C++ Programming Language ISBN 0-201-88954-4 OCLC 59193992 ^ a b c Stroustrup, B. (6 May 2014). \"Lecture:The essence of C++. University of Edinburgh\". YouTube. Archived from the original on 28 April 2015. Retrieved 12 June 2015. ^ a b c a b c Stroustrup, B. (6 May 2014). \"Lecture:The essence of C++. University of Edinburgh\". YouTube. Archived from the original on 28 April 2015. Retrieved 12 June 2015. \"Lecture:The essence of C++. University of Edinburgh\" YouTube Archived . Retrieved 12 June 2015 12 June ^ Stroustrup, Bjarne (17 February 2014). \"C++ Applications\". stroustrup.com. Archived from the original on 4 April 2021. Retrieved 5 May 2014. ^ ^ Stroustrup, Bjarne (17 February 2014). \"C++ Applications\". stroustrup.com. Archived from the original on 4 April 2021. Retrieved 5 May 2014. \"C++ Applications\" Archived . Retrieved 5 May 2014 5 May ^ a b \"ISO/IEC 14882:2024\". International Organization for Standardization. Retrieved 21 October 2020. ^ a b a b \"ISO/IEC 14882:2024\". International Organization for Standardization. Retrieved 21 October 2020. \"ISO/IEC 14882:2024\" . Retrieved 21 October 2020 21 October ^ \"Bjarne Stroustrup's Homepage\". www.stroustrup.com. Archived from the original on 14 May 2019. Retrieved 15 May 2013. ^ ^ \"Bjarne Stroustrup's Homepage\". www.stroustrup.com. Archived from the original on 14 May 2019. Retrieved 15 May 2013. \"Bjarne Stroustrup's Homepage\" Archived . Retrieved 15 May 2013 15 May ^ \"C++ IS schedule\" (PDF). Archived (PDF) from the original on 10 August 2020. Retrieved 9 August 2020. ^ ^ \"C++ IS schedule\" (PDF). Archived (PDF) from the original on 10 August 2020. Retrieved 9 August 2020. \"C++ IS schedule\" (PDF) Archived (PDF) . Retrieved 9 August 2020 9 August ^ \"C++; Where it's heading\". Archived from the original on 3 December 2018. Retrieved 3 December 2018. ^ ^ \"C++; Where it's heading\". Archived from the original on 3 December 2018. Retrieved 3 December 2018. \"C++; Where it's heading\" Archived . Retrieved 3 December 2018 3 December ^ \"Re: [RFC] Convert builin-mailinfo.c to use The Better String Library\" (Mailing list). 6 September 2007. Archived from the original on 8 March 2021. Retrieved 31 March 2015. ^ ^ \"Re: [RFC] Convert builin-mailinfo.c to use The Better String Library\" (Mailing list). 6 September 2007. Archived from the original on 8 March 2021. Retrieved 31 March 2015. \"Re: [RFC] Convert builin-mailinfo.c to use The Better String Library\" Archived . Retrieved 31 March 2015 31 March ^ \"Re: Efforts to attract more users?\" (Mailing list). 12 July 2010. Archived from the original on 21 March 2015. Retrieved 31 March 2015. ^ ^ \"Re: Efforts to attract more users?\" (Mailing list). 12 July 2010. Archived from the original on 21 March 2015. Retrieved 31 March 2015. \"Re: Efforts to attract more users?\" Archived . Retrieved 31 March 2015 31 March ^ Andrew Binstock (18 May 2011). \"Dr. Dobb's: Interview with Ken Thompson\". Archived from the original on 13 March 2014. Retrieved 7 February 2014. ^ ^ Andrew Binstock (18 May 2011). \"Dr. Dobb's: Interview with Ken Thompson\". Archived from the original on 13 March 2014. Retrieved 7 February 2014. \"Dr. Dobb's: Interview with Ken Thompson\" Archived . Retrieved 7 February 2014 7 February ^ Peter Seibel (16 September 2009). Coders at Work: Reflections on the Craft of Programming. Apress. pp. 475\u2013476. ISBN 978-1-4302-1948-4. Archived from the original on 1 December 2019. Retrieved 9 November 2017. ^ ^ Peter Seibel (16 September 2009). Coders at Work: Reflections on the Craft of Programming. Apress. pp. 475\u2013476. ISBN 978-1-4302-1948-4. Archived from the original on 1 December 2019. Retrieved 9 November 2017. Coders at Work: Reflections on the Craft of Programming 475\u2013 ISBN 978-1-4302-1948-4 Archived . Retrieved 9 November 2017 9 November ^ \"C++ in Coders at Work\". 16 October 2009. Archived from the original on 10 November 2017. Retrieved 9 November 2017. ^ ^ \"C++ in Coders at Work\". 16 October 2009. Archived from the original on 10 November 2017. Retrieved 9 November 2017. \"C++ in Coders at Work\" Archived . Retrieved 9 November 2017 9 November ^ \"An Interview with Donald Knuth\". Dr. Dobb's. Archived from the original on 8 March 2021. Retrieved 18 July 2021. ^ ^ \"An Interview with Donald Knuth\". Dr. Dobb's. Archived from the original on 8 March 2021. Retrieved 18 July 2021. \"An Interview with Donald Knuth\" Archived . Retrieved 18 July 2021 18 July ^ \"(La)TeX Navigator\". Archived from the original on 20 November 2017. Retrieved 10 November 2017. ^ ^ \"(La)TeX Navigator\". Archived from the original on 20 November 2017. Retrieved 10 November 2017. \"(La)TeX Navigator\" Archived . Retrieved 10 November 2017 10 November ^ a b Stroustrup, Bjarne (7 March 2010). \"Bjarne Stroustrup's FAQ: When was C++ invented?\". stroustrup.com. Archived from the original on 6 February 2016. Retrieved 16 September 2010. ^ a b a b Stroustrup, Bjarne (7 March 2010). \"Bjarne Stroustrup's FAQ: When was C++ invented?\". stroustrup.com. Archived from the original on 6 February 2016. Retrieved 16 September 2010. \"Bjarne Stroustrup's FAQ: When was C++ invented?\" Archived . Retrieved 16 September 2010 16 September ^ a b Stroustrup, Bjarne. \"Evolving a language in and for the real world: C++ 1991-2006\" (PDF). Archived (PDF) from the original on 20 November 2007. Retrieved 14 August 2013. ^ a b a b Stroustrup, Bjarne. \"Evolving a language in and for the real world: C++ 1991-2006\" (PDF). Archived (PDF) from the original on 20 November 2007. Retrieved 14 August 2013. \"Evolving a language in and for the real world: C++ 1991-2006\" (PDF) Archived (PDF) . Retrieved 14 August 2013 14 August ^ a b c Stroustrup, Bjarne. \"A History of C ++ : 1979\u2212 1991\" (PDF). Archived (PDF) from the original on 2 February 2019. Retrieved 18 July 2013. ^ a b c a b c Stroustrup, Bjarne. \"A History of C ++ : 1979\u2212 1991\" (PDF). Archived (PDF) from the original on 2 February 2019. Retrieved 18 July 2013. \"A History of C ++ : 1979\u2212 1991\" (PDF) Archived (PDF) . Retrieved 18 July 2013 18 July ^ Stroustrup, Bjarne. \"The C++ Programming Language\" (First ed.). Archived from the original on 9 August 2012. Retrieved 16 September 2010. ^ ^ Stroustrup, Bjarne. \"The C++ Programming Language\" (First ed.). Archived from the original on 9 August 2012. Retrieved 16 September 2010. \"The C++ Programming Language\" Archived . Retrieved 16 September 2010 16 September ^ Stroustrup, Bjarne. \"The C++ Programming Language\" (Second ed.). Archived from the original on 9 August 2012. Retrieved 16 September 2010. ^ ^ Stroustrup, Bjarne. \"The C++ Programming Language\" (Second ed.). Archived from the original on 9 August 2012. Retrieved 16 September 2010. \"The C++ Programming Language\" Archived . Retrieved 16 September 2010 16 September ^ Sutter, Herb (30 June 2016). \"Trip report: Summer ISO C++ standards meeting (Oulu)\". herbsutter.com. Archived from the original on 8 October 2016. the next standard after C++17 will be C++20 ^ ^ Sutter, Herb (30 June 2016). \"Trip report: Summer ISO C++ standards meeting (Oulu)\". herbsutter.com. Archived from the original on 8 October 2016. the next standard after C++17 will be C++20 \"Trip report: Summer ISO C++ standards meeting (Oulu)\" the original ^ Dus\u00edkov\u00e1, Hana (6 November 2019). \"N4817: 2020 Prague Meeting Invitation and Information\" (PDF). Archived (PDF) from the original on 29 December 2019. Retrieved 13 February 2020. ^ ^ Dus\u00edkov\u00e1, Hana (6 November 2019). \"N4817: 2020 Prague Meeting Invitation and Information\" (PDF). Archived (PDF) from the original on 29 December 2019. Retrieved 13 February 2020. \"N4817: 2020 Prague Meeting Invitation and Information\" (PDF) Archived (PDF) . Retrieved 13 February 2020 13 February ^ \"Current Status\". isocpp.org. Archived from the original on 8 September 2020. Retrieved 7 September 2020. ^ ^ \"Current Status\". isocpp.org. Archived from the original on 8 September 2020. Retrieved 7 September 2020. \"Current Status\" Archived . Retrieved 7 September 2020 7 September ^ \"C++20 Approved -- Herb Sutter\". isocpp.org. Archived from the original on 11 September 2020. Retrieved 8 September 2020. ^ ^ \"C++20 Approved -- Herb Sutter\". isocpp.org. Archived from the original on 11 September 2020. Retrieved 8 September 2020. \"C++20 Approved -- Herb Sutter\" Archived . Retrieved 8 September 2020 8 September ^ \"Computer Science Pioneer Bjarne Stroustrup to Receive the 2018 Charles Stark Draper Prize for Engineering\" (Press release). National Academy of Engineering. 3 January 2018. Archived from the original on 3 January 2018. Retrieved 14 December 2021. ^ ^ \"Computer Science Pioneer Bjarne Stroustrup to Receive the 2018 Charles Stark Draper Prize for Engineering\" (Press release). National Academy of Engineering. 3 January 2018. Archived from the original on 3 January 2018. Retrieved 14 December 2021. \"Computer Science Pioneer Bjarne Stroustrup to Receive the 2018 Charles Stark Draper Prize for Engineering\" the original . Retrieved 14 December 2021 14 December ^ TIOBE (November 2024). \"TIOBE Index for November 2024\". TIOBE.com. TIOBE Company. Archived from the original on 18 November 2024. Retrieved 18 November 2024. ^ ^ TIOBE (November 2024). \"TIOBE Index for November 2024\". TIOBE.com. TIOBE Company. Archived from the original on 18 November 2024. Retrieved 18 November 2024. \"TIOBE Index for November 2024\" Archived . Retrieved 18 November 2024 18 November ^ \"Bjarne Stroustrup's FAQ \u2013 Where did the name \"C++\" come from?\". Archived from the original on 6 February 2016. Retrieved 16 January 2008. ^ ^ \"Bjarne Stroustrup's FAQ \u2013 Where did the name \"C++\" come from?\". Archived from the original on 6 February 2016. Retrieved 16 January 2008. \"Bjarne Stroustrup's FAQ \u2013 Where did the name \"C++\" come from?\" Archived . Retrieved 16 January 2008 16 January ^ \"C For C++ Programmers\". Northeastern University. Archived from the original on 17 November 2010. Retrieved 7 September 2015. ^ ^ \"C For C++ Programmers\". Northeastern University. Archived from the original on 17 November 2010. Retrieved 7 September 2015. \"C For C++ Programmers\" Northeastern University the original . Retrieved 7 September 2015 7 September ^ \"ISO/IEC 14882:1998\". International Organization for Standardization. Archived from the original on 15 January 2017. Retrieved 23 November 2018. ^ ^ \"ISO/IEC 14882:1998\". International Organization for Standardization. Archived from the original on 15 January 2017. Retrieved 23 November 2018. \"ISO/IEC 14882:1998\" Archived . Retrieved 23 November 2018 23 November ^ \"ISO/IEC 14882:2003\". International Organization for Standardization. Archived from the original on 13 August 2021. Retrieved 23 November 2018. ^ ^ \"ISO/IEC 14882:2003\". International Organization for Standardization. Archived from the original on 13 August 2021. Retrieved 23 November 2018. \"ISO/IEC 14882:2003\" Archived . Retrieved 23 November 2018 23 November ^ a b \"ISO/IEC 14882:2011\". International Organization for Standardization. Archived from the original on 27 May 2016. Retrieved 23 November 2018. ^ a b a b \"ISO/IEC 14882:2011\". International Organization for Standardization. Archived from the original on 27 May 2016. Retrieved 23 November 2018. \"ISO/IEC 14882:2011\" Archived . Retrieved 23 November 2018 23 November ^ \"ISO/IEC 14882:2014\". International Organization for Standardization. Archived from the original on 29 April 2016. Retrieved 23 November 2018. ^ ^ \"ISO/IEC 14882:2014\". International Organization for Standardization. Archived from the original on 29 April 2016. Retrieved 23 November 2018. \"ISO/IEC 14882:2014\" Archived . Retrieved 23 November 2018 23 November ^ \"ISO/IEC 14882:2017\". International Organization for Standardization. Archived from the original on 29 January 2013. Retrieved 2 December 2017. ^ ^ \"ISO/IEC 14882:2017\". International Organization for Standardization. Archived from the original on 29 January 2013. Retrieved 2 December 2017. \"ISO/IEC 14882:2017\" Archived . Retrieved 2 December 2017 2 December ^ \"ISO/IEC 14882:2020\". International Organization for Standardization. Archived from the original on 16 December 2020. Retrieved 16 December 2020. ^ ^ \"ISO/IEC 14882:2020\". International Organization for Standardization. Archived from the original on 16 December 2020. Retrieved 16 December 2020. \"ISO/IEC 14882:2020\" Archived . Retrieved 16 December 2020 16 December ^ \"We have an international standard: C++0x is unanimously approved\". Sutter's Mill. 12 August 2011. Archived from the original on 28 June 2018. Retrieved 23 November 2018. ^ ^ \"We have an international standard: C++0x is unanimously approved\". Sutter's Mill. 12 August 2011. Archived from the original on 28 June 2018. Retrieved 23 November 2018. \"We have an international standard: C++0x is unanimously approved\" Archived . Retrieved 23 November 2018 23 November ^ \"The Future of C++\". Archived from the original on 23 October 2018. Retrieved 23 November 2018 \u2013 via channel9.msdn.com. ^ ^ \"The Future of C++\". Archived from the original on 23 October 2018. Retrieved 23 November 2018 \u2013 via channel9.msdn.com. \"The Future of C++\" Archived . Retrieved 23 November 2018 23 November ^ \"We have C++14! : Standard C++\". isocpp.org. Archived from the original on 19 August 2014. Retrieved 19 August 2014. ^ ^ \"We have C++14! : Standard C++\". isocpp.org. Archived from the original on 19 August 2014. Retrieved 19 August 2014. \"We have C++14! : Standard C++\" Archived . Retrieved 19 August 2014 19 August ^ Sutter, Herb (15 July 2017). \"Trip report: Summer ISO C++ standards meeting (Toronto)\". Archived from the original on 6 August 2017. Retrieved 4 August 2017. ^ ^ Sutter, Herb (15 July 2017). \"Trip report: Summer ISO C++ standards meeting (Toronto)\". Archived from the original on 6 August 2017. Retrieved 4 August 2017. \"Trip report: Summer ISO C++ standards meeting (Toronto)\" Archived . Retrieved 4 August 2017 4 August ^ \"ISO/IEC TR 18015:2006\". International Organization for Standardization. Archived from the original on 15 January 2019. Retrieved 15 February 2019. ^ ^ \"ISO/IEC TR 18015:2006\". International Organization for Standardization. Archived from the original on 15 January 2019. Retrieved 15 February 2019. \"ISO/IEC TR 18015:2006\" Archived . Retrieved 15 February 2019 15 February ^ \"ISO/IEC TR 19768:2007\". International Organization for Standardization. Archived from the original on 4 March 2016. Retrieved 15 February 2019. ^ ^ \"ISO/IEC TR 19768:2007\". International Organization for Standardization. Archived from the original on 4 March 2016. Retrieved 15 February 2019. \"ISO/IEC TR 19768:2007\" Archived . Retrieved 15 February 2019 15 February ^ \"ISO/IEC TR 29124:2010\". International Organization for Standardization. Archived from the original on 12 January 2019. Retrieved 15 February 2019. ^ ^ \"ISO/IEC TR 29124:2010\". International Organization for Standardization. Archived from the original on 12 January 2019. Retrieved 15 February 2019. \"ISO/IEC TR 29124:2010\" Archived . Retrieved 15 February 2019 15 February ^ \"ISO/IEC TR 24733:2011\". International Organization for Standardization. Archived from the original on 15 January 2019. Retrieved 15 February 2019. ^ ^ \"ISO/IEC TR 24733:2011\". International Organization for Standardization. Archived from the original on 15 January 2019. Retrieved 15 February 2019. \"ISO/IEC TR 24733:2011\" Archived . Retrieved 15 February 2019 15 February ^ \"ISO/IEC TS 18822:2015\". International Organization for Standardization. Archived from the original on 15 January 2019. Retrieved 15 February 2019. ^ ^ \"ISO/IEC TS 18822:2015\". International Organization for Standardization. Archived from the original on 15 January 2019. Retrieved 15 February 2019. \"ISO/IEC TS 18822:2015\" Archived . Retrieved 15 February 2019 15 February ^ \"ISO/IEC TS 19570:2015\". International Organization for Standardization. Archived from the original on 15 January 2019. Retrieved 15 February 2019. ^ ^ \"ISO/IEC TS 19570:2015\". International Organization for Standardization. Archived from the original on 15 January 2019. Retrieved 15 February 2019. \"ISO/IEC TS 19570:2015\" Archived . Retrieved 15 February 2019 15 February ^ \"ISO/IEC TS 19841:2015\". International Organization for Standardization. Archived from the original on 15 January 2019. Retrieved 15 February 2019. ^ ^ \"ISO/IEC TS 19841:2015\". International Organization for Standardization. Archived from the original on 15 January 2019. Retrieved 15 February 2019. \"ISO/IEC TS 19841:2015\" Archived . Retrieved 15 February 2019 15 February ^ \"ISO/IEC TS 19568:2015\". International Organization for Standardization. Archived from the original on 15 January 2019. Retrieved 15 February 2019. ^ ^ \"ISO/IEC TS 19568:2015\". International Organization for Standardization. Archived from the original on 15 January 2019. Retrieved 15 February 2019. \"ISO/IEC TS 19568:2015\" Archived . Retrieved 15 February 2019 15 February ^ \"ISO/IEC TS 19217:2015\". International Organization for Standardization. Archived from the original on 15 January 2019. Retrieved 15 February 2019. ^ ^ \"ISO/IEC TS 19217:2015\". International Organization for Standardization. Archived from the original on 15 January 2019. Retrieved 15 February 2019. \"ISO/IEC TS 19217:2015\" Archived . Retrieved 15 February 2019 15 February ^ \"ISO/IEC TS 19571:2016\". International Organization for Standardization. Archived from the original on 15 January 2019. Retrieved 15 February 2019. ^ ^ \"ISO/IEC TS 19571:2016\". International Organization for Standardization. Archived from the original on 15 January 2019. Retrieved 15 February 2019. \"ISO/IEC TS 19571:2016\" Archived . Retrieved 15 February 2019 15 February ^ \"ISO/IEC TS 19568:2017\". International Organization for Standardization. Archived from the original on 15 January 2019. Retrieved 15 February 2019. ^ ^ \"ISO/IEC TS 19568:2017\". International Organization for Standardization. Archived from the original on 15 January 2019. Retrieved 15 February 2019. \"ISO/IEC TS 19568:2017\" Archived . Retrieved 15 February 2019 15 February ^ \"ISO/IEC TS 21425:2017\". International Organization for Standardization. Archived from the original on 15 January 2019. Retrieved 15 February 2019. ^ ^ \"ISO/IEC TS 21425:2017\". International Organization for Standardization. Archived from the original on 15 January 2019. Retrieved 15 February 2019. \"ISO/IEC TS 21425:2017\" Archived . Retrieved 15 February 2019 15 February ^ \"ISO/IEC TS 22277:2017\". International Organization for Standardization. Archived from the original on 15 January 2019. Retrieved 15 February 2019. ^ ^ \"ISO/IEC TS 22277:2017\". International Organization for Standardization. Archived from the original on 15 January 2019. Retrieved 15 February 2019. \"ISO/IEC TS 22277:2017\" Archived . Retrieved 15 February 2019 15 February ^ \"ISO/IEC TS 19216:2018\". International Organization for Standardization. Archived from the original on 15 January 2019. Retrieved 15 February 2019. ^ ^ \"ISO/IEC TS 19216:2018\". International Organization for Standardization. Archived from the original on 15 January 2019. Retrieved 15 February 2019. \"ISO/IEC TS 19216:2018\" Archived . Retrieved 15 February 2019 15 February ^ \"ISO/IEC TS 21544:2018\". International Organization for Standardization. Archived from the original on 15 January 2019. Retrieved 15 February 2019. ^ ^ \"ISO/IEC TS 21544:2018\". International Organization for Standardization. Archived from the original on 15 January 2019. Retrieved 15 February 2019. \"ISO/IEC TS 21544:2018\" Archived . Retrieved 15 February 2019 15 February ^ \"ISO/IEC TS 19570:2018\". International Organization for Standardization. Archived from the original on 15 January 2019. Retrieved 15 February 2019. ^ ^ \"ISO/IEC TS 19570:2018\". International Organization for Standardization. Archived from the original on 15 January 2019. Retrieved 15 February 2019. \"ISO/IEC TS 19570:2018\" Archived . Retrieved 15 February 2019 15 February ^ \"ISO/IEC TS 23619:2021\". International Organization for Standardization. Archived from the original on 15 December 2018. Retrieved 11 October 2021. ^ ^ \"ISO/IEC TS 23619:2021\". International Organization for Standardization. Archived from the original on 15 December 2018. Retrieved 11 October 2021. \"ISO/IEC TS 23619:2021\" Archived . Retrieved 11 October 2021 11 October ^ B. Stroustrup (interviewed by Sergio De Simone) (30 April 2015). \"Stroustrup: Thoughts on C++17 - An Interview\". Archived from the original on 8 July 2015. Retrieved 8 July 2015. ^ ^ B. Stroustrup (interviewed by Sergio De Simone) (30 April 2015). \"Stroustrup: Thoughts on C++17 - An Interview\". Archived from the original on 8 July 2015. Retrieved 8 July 2015. \"Stroustrup: Thoughts on C++17 - An Interview\" Archived . Retrieved 8 July 2015 8 July ^ Stroustrup, Bjarne (2000). The C++ Programming Language (Special ed.). Addison-Wesley. p. 46. ISBN 0-201-70073-5. ^ ^ Stroustrup, Bjarne (2000). The C++ Programming Language (Special ed.). Addison-Wesley. p. 46. ISBN 0-201-70073-5. ISBN 0-201-70073-5 ^ Stroustrup, Bjarne. \"Open issues for The C++ Programming Language (3rd Edition)\". Archived from the original on 5 May 2014. Retrieved 5 May 2014. ^ ^ Stroustrup, Bjarne. \"Open issues for The C++ Programming Language (3rd Edition)\". Archived from the original on 5 May 2014. Retrieved 5 May 2014. \"Open issues for The C++ Programming Language (3rd Edition)\" Archived . Retrieved 5 May 2014 5 May ^ ISO/IEC. Programming Languages \u2013 C++11 Draft (n3797) Archived 2 October 2018 at the Wayback Machine \u00a73.7 Storage duration [basic.stc] ^ ^ ISO/IEC. Programming Languages \u2013 C++11 Draft (n3797) Archived 2 October 2018 at the Wayback Machine \u00a73.7 Storage duration [basic.stc] ISO IEC Programming Languages \u2013 C++11 Draft (n3797) Archived Wayback Machine ^ ISO/IEC. Programming Languages \u2013 C++11 Draft (n3797) Archived 2 October 2018 at the Wayback Machine \u00a73.7.1 Static Storage duration [basic.stc.static] ^ ^ ISO/IEC. Programming Languages \u2013 C++11 Draft (n3797) Archived 2 October 2018 at the Wayback Machine \u00a73.7.1 Static Storage duration [basic.stc.static] ISO IEC Programming Languages \u2013 C++11 Draft (n3797) Archived Wayback Machine ^ ISO/IEC. Programming Languages \u2013 C++11 Draft (n3797) Archived 2 October 2018 at the Wayback Machine \u00a73.7.2 Thread Storage duration [basic.stc.thread] ^ ^ ISO/IEC. Programming Languages \u2013 C++11 Draft (n3797) Archived 2 October 2018 at the Wayback Machine \u00a73.7.2 Thread Storage duration [basic.stc.thread] ISO IEC Programming Languages \u2013 C++11 Draft (n3797) Archived Wayback Machine ^ ISO/IEC. Programming Languages \u2013 C++11 Draft (n3797) Archived 2 October 2018 at the Wayback Machine \u00a73.7.3 Automatic Storage duration [basic.stc.auto] ^ ^ ISO/IEC. Programming Languages \u2013 C++11 Draft (n3797) Archived 2 October 2018 at the Wayback Machine \u00a73.7.3 Automatic Storage duration [basic.stc.auto] ISO IEC Programming Languages \u2013 C++11 Draft (n3797) Archived Wayback Machine ^ ISO/IEC. Programming Languages \u2013 C++11 Draft (n3797) Archived 2 October 2018 at the Wayback Machine \u00a73.7.4 Dynamic Storage duration [basic.stc.dynamic] ^ ^ ISO/IEC. Programming Languages \u2013 C++11 Draft (n3797) Archived 2 October 2018 at the Wayback Machine \u00a73.7.4 Dynamic Storage duration [basic.stc.dynamic] ISO IEC Programming Languages \u2013 C++11 Draft (n3797) Archived Wayback Machine ^ \"C++ Core Guidelines\". isocpp.github.io. Archived from the original on 8 February 2020. Retrieved 9 February 2020. ^ ^ \"C++ Core Guidelines\". isocpp.github.io. Archived from the original on 8 February 2020. Retrieved 9 February 2020. \"C++ Core Guidelines\" Archived . Retrieved 9 February 2020 9 February ^ a b \"Nobody Understands C++: Part 5: Template Code Bloat\". articles.emptycrate.com/: EmptyCrate Software. Travel. Stuff. 6 May 2008. Archived from the original on 25 April 2016. Retrieved 8 March 2010. On occasion you will read or hear someone talking about C++ templates causing code bloat. I was thinking about it the other day and thought to myself, \"self, if the code does exactly the same thing then the compiled code cannot really be any bigger, can it?\" [...] And what about compiled code size? Each were compiled with the command g++ <filename>.cpp -O3. Non-template version: 8140 bytes, template version: 8028 bytes! ^ a b a b \"Nobody Understands C++: Part 5: Template Code Bloat\". articles.emptycrate.com/: EmptyCrate Software. Travel. Stuff. 6 May 2008. Archived from the original on 25 April 2016. Retrieved 8 March 2010. On occasion you will read or hear someone talking about C++ templates causing code bloat. I was thinking about it the other day and thought to myself, \"self, if the code does exactly the same thing then the compiled code cannot really be any bigger, can it?\" [...] And what about compiled code size? Each were compiled with the command g++ <filename>.cpp -O3. Non-template version: 8140 bytes, template version: 8028 bytes! \"Nobody Understands C++: Part 5: Template Code Bloat\" Archived . Retrieved 8 March 2010 8 March ^ Sutter, Herb; Alexandrescu, Andrei (2004). C++ Coding Standards: 101 Rules, Guidelines, and Best Practices. Addison-Wesley. ^ ^ Sutter, Herb; Alexandrescu, Andrei (2004). C++ Coding Standards: 101 Rules, Guidelines, and Best Practices. Addison-Wesley. Sutter, Herb Alexandrescu, Andrei ^ Henricson, Mats; Nyquist, Erik (1997). Industrial Strength C++. Prentice Hall. ISBN 0-13-120965-5. ^ ^ Henricson, Mats; Nyquist, Erik (1997). Industrial Strength C++. Prentice Hall. ISBN 0-13-120965-5. Industrial Strength C++ ISBN 0-13-120965-5 ^ Stroustrup, Bjarne (2000). The C++ Programming Language (Special ed.). Addison-Wesley. p. 310. ISBN 0-201-70073-5. A virtual member function is sometimes called a method. ^ ^ Stroustrup, Bjarne (2000). The C++ Programming Language (Special ed.). Addison-Wesley. p. 310. ISBN 0-201-70073-5. A virtual member function is sometimes called a method. ISBN 0-201-70073-5 ^ Mycroft, Alan (2013). \"C and C++ Exceptions | Templates\" (PDF). Cambridge Computer Laboratory - Course Materials 2013-14. Archived (PDF) from the original on 13 May 2016. Retrieved 30 August 2016. ^ ^ Mycroft, Alan (2013). \"C and C++ Exceptions | Templates\" (PDF). Cambridge Computer Laboratory - Course Materials 2013-14. Archived (PDF) from the original on 13 May 2016. Retrieved 30 August 2016. \"C and C++ Exceptions | Templates\" (PDF) Archived (PDF) . Retrieved 30 August 2016 30 August ^ Stroustrup, Bjarne (2013). The C++ Programming Language. Addison Wesley. p. 345. ISBN 9780321563842. ^ ^ Stroustrup, Bjarne (2013). The C++ Programming Language. Addison Wesley. p. 345. ISBN 9780321563842. ISBN 9780321563842 ^ Stroustrup, Bjarne (2013). The C++ Programming Language. Addison Wesley. pp. 363\u2013365. ISBN 9780321563842. ^ ^ Stroustrup, Bjarne (2013). The C++ Programming Language. Addison Wesley. pp. 363\u2013365. ISBN 9780321563842. 363\u2013 ISBN 9780321563842 ^ Stroustrup, Bjarne (2013). The C++ Programming Language. Addison Wesley. pp. 345, 363. ISBN 9780321563842. ^ ^ Stroustrup, Bjarne (2013). The C++ Programming Language. Addison Wesley. pp. 345, 363. ISBN 9780321563842. ISBN 9780321563842 ^ \"Google C++ Style Guide\". Archived from the original on 16 March 2019. Retrieved 25 June 2019. ^ ^ \"Google C++ Style Guide\". Archived from the original on 16 March 2019. Retrieved 25 June 2019. \"Google C++ Style Guide\" Archived . Retrieved 25 June 2019 25 June ^ \"LLVM Coding Standards\". LLVM 9 documentation. Archived from the original on 27 June 2019. Retrieved 25 June 2019. ^ ^ \"LLVM Coding Standards\". LLVM 9 documentation. Archived from the original on 27 June 2019. Retrieved 25 June 2019. \"LLVM Coding Standards\" Archived . Retrieved 25 June 2019 25 June ^ \"Coding Conventions\". Qt Wiki. Archived from the original on 26 June 2019. Retrieved 26 June 2019. ^ ^ \"Coding Conventions\". Qt Wiki. Archived from the original on 26 June 2019. Retrieved 26 June 2019. \"Coding Conventions\" Archived . Retrieved 26 June 2019 26 June ^ Stroustrup, Bjarne (2013). The C++ Programming Language. Addison Wesley. pp. 344, 370. ISBN 9780321563842. ^ ^ Stroustrup, Bjarne (2013). The C++ Programming Language. Addison Wesley. pp. 344, 370. ISBN 9780321563842. ISBN 9780321563842 ^ Stroustrup, Bjarne (2013). The C++ Programming Language. Addison Wesley. p. 349. ISBN 9780321563842. ^ ^ Stroustrup, Bjarne (2013). The C++ Programming Language. Addison Wesley. p. 349. ISBN 9780321563842. ISBN 9780321563842 ^ Graziano Lo Russo (2008). \"An Interview with A. Stepanov\". stlport.org. Archived from the original on 4 March 2009. Retrieved 8 October 2015. ^ ^ Graziano Lo Russo (2008). \"An Interview with A. Stepanov\". stlport.org. Archived from the original on 4 March 2009. Retrieved 8 October 2015. \"An Interview with A. Stepanov\" Archived . Retrieved 8 October 2015 8 October ^ \"C++ Core Guidelines\". isocpp.github.io. Archived from the original on 16 February 2020. Retrieved 9 February 2020. ^ ^ \"C++ Core Guidelines\". isocpp.github.io. Archived from the original on 16 February 2020. Retrieved 9 February 2020. \"C++ Core Guidelines\" Archived . Retrieved 9 February 2020 9 February ^ \"Bjarne Stroustrup announces C++ Core Guidelines : Standard C++\". isocpp.org. Archived from the original on 11 May 2020. Retrieved 31 March 2020. ^ ^ \"Bjarne Stroustrup announces C++ Core Guidelines : Standard C++\". isocpp.org. Archived from the original on 11 May 2020. Retrieved 31 March 2020. \"Bjarne Stroustrup announces C++ Core Guidelines : Standard C++\" Archived . Retrieved 31 March 2020 31 March ^ \"microsoft/GSL\". 18 July 2021. Archived from the original on 18 July 2021. Retrieved 18 July 2021 \u2013 via GitHub. ^ ^ \"microsoft/GSL\". 18 July 2021. Archived from the original on 18 July 2021. Retrieved 18 July 2021 \u2013 via GitHub. \"microsoft/GSL\" Archived . Retrieved 18 July 2021 18 July ^ \"Using the C++ Core Guidelines checkers\". Microsoft Learn. Archived from the original on 13 August 2021. Retrieved 31 March 2020. ^ ^ \"Using the C++ Core Guidelines checkers\". Microsoft Learn. Archived from the original on 13 August 2021. Retrieved 31 March 2020. \"Using the C++ Core Guidelines checkers\" Archived . Retrieved 31 March 2020 31 March ^ \"C++ ABI Summary\". 20 March 2001. Archived from the original on 10 July 2018. Retrieved 30 May 2006. ^ ^ \"C++ ABI Summary\". 20 March 2001. Archived from the original on 10 July 2018. Retrieved 30 May 2006. \"C++ ABI Summary\" Archived . Retrieved 30 May 2006 30 May ^ \"Bjarne Stroustrup's FAQ \u2013 Is C a subset of C++?\". Archived from the original on 6 February 2016. Retrieved 5 May 2014. ^ ^ \"Bjarne Stroustrup's FAQ \u2013 Is C a subset of C++?\". Archived from the original on 6 February 2016. Retrieved 5 May 2014. \"Bjarne Stroustrup's FAQ \u2013 Is C a subset of C++?\" Archived . Retrieved 5 May 2014 5 May ^ \"C9X \u2013 The New C Standard\". Archived from the original on 21 June 2018. Retrieved 27 December 2008. ^ ^ \"C9X \u2013 The New C Standard\". Archived from the original on 21 June 2018. Retrieved 27 December 2008. \"C9X \u2013 The New C Standard\" Archived . Retrieved 27 December 2008 27 December ^ \"C++0x Support in GCC\". Archived from the original on 21 July 2010. Retrieved 12 October 2010. ^ ^ \"C++0x Support in GCC\". Archived from the original on 21 July 2010. Retrieved 12 October 2010. \"C++0x Support in GCC\" Archived . Retrieved 12 October 2010 12 October ^ \"C++0x Core Language Features In VC10: The Table\". Archived from the original on 21 August 2010. Retrieved 12 October 2010. ^ ^ \"C++0x Core Language Features In VC10: The Table\". Archived from the original on 21 August 2010. Retrieved 12 October 2010. \"C++0x Core Language Features In VC10: The Table\" Archived . Retrieved 12 October 2010 12 October ^ \"Clang - C++98, C++11, and C++14 Status\". Clang.llvm.org. 12 May 2013. Archived from the original on 4 July 2013. Retrieved 10 June 2013. ^ ^ \"Clang - C++98, C++11, and C++14 Status\". Clang.llvm.org. 12 May 2013. Archived from the original on 4 July 2013. Retrieved 10 June 2013. \"Clang - C++98, C++11, and C++14 Status\" Archived . Retrieved 10 June 2013 10 June Further reading [edit] [ edit edit ] Abrahams, David; Gurtovoy, Aleksey (2005). C++ Template Metaprogramming: Concepts, Tools, and Techniques from Boost and Beyond. Addison-Wesley. ISBN 0-321-22725-5. Abrahams, David ISBN 0-321-22725-5 Alexandrescu, Andrei (2001). Modern C++ Design: Generic Programming and Design Patterns Applied. Addison-Wesley. ISBN 0-201-70431-5. Alexandrescu, Andrei ISBN 0-201-70431-5 Alexandrescu, Andrei; Sutter, Herb (2004). C++ Design and Coding Standards: Rules and Guidelines for Writing Programs. Addison-Wesley. ISBN 0-321-11358-6. Alexandrescu, Andrei Sutter, Herb ISBN 0-321-11358-6 Becker, Pete (2006). The C++ Standard Library Extensions : A Tutorial and Reference. Addison-Wesley. ISBN 0-321-41299-0. Becker, Pete ISBN 0-321-41299-0 Brokken, Frank (2010). C++ Annotations. University of Groningen. ISBN 978-90-367-0470-0. Archived from the original on 28 April 2010. Retrieved 28 April 2010. C++ Annotations ISBN 978-90-367-0470-0 Archived . Retrieved 28 April 2010 28 April Coplien, James O. (1994) [reprinted with corrections, original year of publication 1992]. Advanced C++: Programming Styles and Idioms. Addison-Wesley. ISBN 0-201-54855-0. Coplien, James O. Advanced C++: Programming Styles and Idioms ISBN 0-201-54855-0 Dewhurst, Stephen C. (2005). C++ Common Knowledge: Essential Intermediate Programming. Addison-Wesley. ISBN 0-321-32192-8. ISBN 0-321-32192-8 Information Technology Industry Council (15 October 2003). Programming languages \u2013 C++ (Second ed.). Geneva: ISO/IEC. 14882:2003(E). Information Technology Industry Council Josuttis, Nicolai M. (2012). The C++ Standard Library, A Tutorial and Reference (Second ed.). Addison-Wesley. ISBN 978-0-321-62321-8. ISBN 978-0-321-62321-8 Koenig, Andrew; Moo, Barbara E. (2000). Accelerated C++ \u2013 Practical Programming by Example. Addison-Wesley. ISBN 0-201-70353-X. Koenig, Andrew Accelerated C++ \u2013 Practical Programming by Example ISBN 0-201-70353-X Lippman, Stanley B.; Lajoie, Jos\u00e9e; Moo, Barbara E. (2011). C++ Primer (Fifth ed.). Addison-Wesley. ISBN 978-0-321-71411-4. Lippman, Stanley B. C++ Primer C++ Primer ISBN 978-0-321-71411-4 Lippman, Stanley B. (1996). Inside the C++ Object Model. Addison-Wesley. ISBN 0-201-83454-5. ISBN 0-201-83454-5 Meyers, Scott (2005). Effective C++ (Third ed.). Addison-Wesley. ISBN 0-321-33487-6. Meyers, Scott Effective C++ ISBN 0-321-33487-6 Stroustrup, Bjarne (2013). The C++ Programming Language (Fourth ed.). Addison-Wesley. ISBN 978-0-321-56384-2. Stroustrup, Bjarne The C++ Programming Language ISBN 978-0-321-56384-2 Stroustrup, Bjarne (1994). The Design and Evolution of C++. Addison-Wesley. ISBN 0-201-54330-3. Stroustrup, Bjarne The Design and Evolution of C++ ISBN 0-201-54330-3 Stroustrup, Bjarne (2014). Programming: Principles and Practice Using C++ (Second ed.). Addison-Wesley. ISBN 978-0-321-99278-9. Stroustrup, Bjarne ISBN 978-0-321-99278-9 Sutter, Herb (2001). More Exceptional C++: 40 New Engineering Puzzles, Programming Problems, and Solutions. Addison-Wesley. ISBN 0-201-70434-X. Sutter, Herb ISBN 0-201-70434-X Sutter, Herb (2004). Exceptional C++ Style. Addison-Wesley. ISBN 0-201-76042-8. Sutter, Herb ISBN 0-201-76042-8 Vandevoorde, David; Josuttis, Nicolai M. (2003). C++ Templates: The complete Guide. Addison-Wesley. ISBN 0-201-73484-2. ISBN 0-201-73484-2 External links [edit] [ edit edit ] JTC1/SC22/WG21 \u2013 the ISO/IEC C++ Standard Working Group JTC1/SC22/WG21 Standard C++ Foundation \u2013 a non-profit organization that promotes the use and understanding of standard C++. Bjarne Stroustrup is a director of the organization. Standard C++ Foundation Bjarne Stroustrup C++ Keywords C++ Keywords C++ Expressions C++ Expressions C++ Operator Precedence C++ Operator Precedence sister projects sister projects Media from Commons Media from Commons Media Quotations from Wikiquote Quotations from Wikiquote Quotations Textbooks from Wikibooks Textbooks from Wikibooks Textbooks Resources from Wikiversity Resources from Wikiversity Resources v v t t e e C++ C++ C++ Outline Outline C++98 C++98 C++03 C++03 C++11 C++11 C++14 C++14 C++17 C++17 C++20 C++20 C++23 C++23 C++26 C++26 Libraries Libraries Classes Classes Concepts Concepts Copy constructor Copy constructor Exception handling (Exception safety) Exception handling Exception safety Function overloading Function overloading Move semantics Move semantics new and delete new and delete Operator overloading Operator overloading Operators Operators References References Templates Templates Virtual functions Virtual functions Standard Library I/O Streams I/O Streams Smart pointers Smart pointers STL STL Strings Strings As-if rule As-if rule Barton\u2013Nackman trick Barton\u2013Nackman trick Curiously recurring template pattern Curiously recurring template pattern Most vexing parse Most vexing parse One Definition Rule One Definition Rule Resource acquisition is initialization Resource acquisition is initialization Rule of three Rule of three Slicing Slicing Special member functions Special member functions Substitution failure is not an error Substitution failure is not an error Template metaprogramming Template metaprogramming Compilers Comparison of C++ compilers Comparison of C++ compilers Borland C++ Borland C++ Borland Turbo C++ Borland Turbo C++ C++Builder C++Builder Clang Clang GCC GCC Intel C++ Compiler Intel C++ Compiler Oracle Solaris Studio Oracle Solaris Studio Visual C++ (MSVC) Visual C++ (MSVC) Watcom C/C++ Watcom C/C++ IDEs Comparison of C IDEs Comparison of C IDEs Anjuta Anjuta CLion CLion Code::Blocks Code::Blocks CodeLite CodeLite Dev-C++ Dev-C++ Eclipse Eclipse Geany Geany NetBeans NetBeans KDevelop KDevelop Qt Creator Qt Creator Visual Studio Visual Studio Objective-C++ Objective-C++ C++/CLI C++/CLI C++/CX C++/CX C++/WinRT C++/WinRT Ch Ch SYCL SYCL Embedded C++ Embedded C++ Comparison of programming languages Comparison of programming languages Comparison of Java and C++ Comparison of Java and C++ Comparison of ALGOL 68 and C++ Comparison of ALGOL 68 and C++ Compatibility of C and C++ Compatibility of C and C++ Criticism of C++ Criticism of C++ Designer Bjarne Stroustrup Bjarne Stroustrup Category v v t t e e Programming languages Comparison Comparison Timeline Timeline History History Ada Ada ALGOL Simula ALGOL Simula Simula APL APL Assembly Assembly BASIC Visual Basic classic .NET BASIC Visual Basic classic .NET Visual Basic classic classic .NET .NET C C C++ C++ C# C# COBOL COBOL Erlang Erlang Forth Forth Fortran Fortran Go Go Haskell Haskell Java Java JavaScript JavaScript Julia Julia Kotlin Kotlin Lisp Lisp Lua Lua MATLAB MATLAB ML ML Pascal Object Pascal Pascal Object Pascal Object Pascal Perl Perl PHP PHP Prolog Prolog Python Python R R Ruby Ruby Rust Rust SQL SQL Scratch Scratch Shell Shell Smalltalk Smalltalk Swift Swift more... more... Lists: Alphabetical Alphabetical Categorical Categorical Generational Generational Non-English-based Non-English-based Category Category v v t t e e ISO by standard number ISO standards ISO romanizations IEC standards 1 1 2 2 3 3 4 4 6 6 7 7 9 9 16 16 17 17 31 -0 -1 -3 -4 -5 -6 -7 -8 -9 -10 -11 -12 -13 31 -0 -0 -1 -1 -3 -3 -4 -4 -5 -5 -6 -6 -7 -7 -8 -8 -9 -9 -10 -10 -11 -11 -12 -12 -13 -13 68-1 68-1 128 128 216 216 217 217 226 226 228 228 233 233 259 259 261 261 262 262 302 302 306 306 361 361 500 500 518 518 519 519 639 -1 -2 -3 -5 -6 639 -1 -1 -2 -2 -3 -3 -5 -5 -6 -6 646 646 657 657 668 668 690 690 704 704 732 732 764 764 838 838 843 843 860 860 898 898 965 965 999 999 1000 1000 1004 1004 1007 1007 1073-1 1073-1 1073-2 1073-2 1155 1155 1413 1413 1538 1538 1629 1629 1745 1745 1989 1989 2014 2014 2015 2015 2022 2022 2033 2033 2047 2047 2108 2108 2145 2145 2146 2146 2240 2240 2281 2281 2533 2533 2709 2709 2711 2711 2720 2720 2788 2788 2848 2848 2852 2852 2921 2921 3029 3029 3103 3103 3166 -1 -2 -3 3166 -1 -1 -2 -2 -3 -3 3297 3297 3307 3307 3601 3601 3602 3602 3864 3864 3901 3901 3950 3950 3977 3977 4031 4031 4157 4157 4165 4165 4217 4217 4909 4909 5218 5218 5426 5426 5427 5427 5428 5428 5725 5725 5775 5775 5776 5776 5800 5800 5807 5807 5964 5964 6166 6166 6344 6344 6346 6346 6373 6373 6385 6385 6425 6425 6429 6429 6438 6438 6523 6523 6709 6709 6943 6943 7001 7001 7002 7002 7010 7010 7027 7027 7064 7064 7098 7098 7185 7185 7200 7200 7498 -1 7498 -1 -1 7637 7637 7736 7736 7810 7810 7811 7811 7812 7812 7813 7813 7816 7816 7942 7942 8000 8000 8093 8093 8178 8178 8217 8217 8373 8373 8501-1 8501-1 8571 8571 8583 8583 8601 8601 8613 8613 8632 8632 8651 8651 8652 8652 8691 8691 8805/8806 8805/8806 8807 8807 8820-5 8820-5 8859 -1 -2 -3 -4 -5 -6 -7 -8 -8-I -9 -10 -11 -12 -13 -14 -15 -16 8859 -1 -1 -2 -2 -3 -3 -4 -4 -5 -5 -6 -6 -7 -7 -8 -8 -8-I -8-I -9 -9 -10 -10 -11 -11 -12 -12 -13 -13 -14 -14 -15 -15 -16 -16 8879 8879 9000/9001 9000/9001 9036 9036 9075 9075 9126 9126 9141 9141 9227 9227 9241 9241 9293 9293 9314 9314 9362 9362 9407 9407 9496 9496 9506 9506 9529 9529 9564 9564 9592/9593 9592/9593 9594 9594 9660 9660 9797-1 9797-1 9897 9897 9899 9899 9945 9945 9984 9984 9985 9985 9995 9995 10006 10006 10007 10007 10116 10116 10118-3 10118-3 10160 10160 10161 10161 10165 10165 10179 10179 10206 10206 10218 10218 10279 10279 10303 -11 -21 -22 -28 -238 10303 -11 -11 -21 -21 -22 -22 -28 -28 -238 -238 10383 10383 10585 10585 10589 10589 10628 10628 10646 10646 10664 10664 10746 10746 10861 10861 10957 10957 10962 10962 10967 10967 11073 11073 11170 11170 11172 11172 11179 11179 11404 11404 11544 11544 11783 11783 11784 11784 11785 11785 11801 11801 11889 11889 11898 11898 11940 (-2) 11940 -2 11941 11941 11941 (TR) 11941 (TR) 11992 11992 12006 12006 12052 12052 12182 12182 12207 12207 12234-2 12234-2 12620 12620 13211 -1 -2 13211 -1 -1 -2 -2 13216 13216 13250 13250 13399 13399 13406-2 13406-2 13450 13450 13485 13485 13490 13490 13567 13567 13568 13568 13584 13584 13616 13616 13816 13816 13818 13818 14000 14000 14031 14031 14224 14224 14289 14289 14396 14396 14443 14443 14496 -2 -3 -6 -10 -11 -12 -14 -17 -20 14496 -2 -2 -3 -3 -6 -6 -10 -10 -11 -11 -12 -12 -14 -14 -17 -17 -20 -20 14617 14617 14644 14644 14649 14649 14651 14651 14698 14698 14764 14764 14882 14882 14971 14971 15022 15022 15189 15189 15288 15288 15291 15291 15398 15398 15408 15408 15444 -3 -9 15444 -3 -3 -9 -9 15445 15445 15438 15438 15504 15504 15511 15511 15686 15686 15693 15693 15706 -2 15706 -2 -2 15707 15707 15897 15897 15919 15919 15924 15924 15926 15926 15926 WIP 15926 WIP 15930 15930 15938 15938 16023 16023 16262 16262 16355-1 16355-1 16485 16485 16612-2 16612-2 16750 16750 16949 (TS) 16949 (TS) 17024 17024 17025 17025 17100 17100 17203 17203 17369 17369 17442 17442 17506 17506 17799 17799 18004 18004 18014 18014 18181 18181 18245 18245 18629 18629 18916 18916 19005 19005 19011 19011 19092 -1 -2 19092 -1 -1 -2 -2 19114 19114 19115 19115 19125 19125 19136 19136 19407 19407 19439 19439 19500 19500 19501 19501 19502 19502 19503 19503 19505 19505 19506 19506 19507 19507 19508 19508 19509 19509 19510 19510 19600 19600 19752 19752 19757 19757 19770 19770 19775-1 19775-1 19794-5 19794-5 19831 19831 20000 20000 20022 20022 20121 20121 20400 20400 20802 20802 20830 20830 21000 21000 21001 21001 21047 21047 21122 21122 21500 21500 21827 21827 22000 22000 22275 22275 22300 22300 22301 22301 22395 22395 22537 22537 23000 23000 23003 23003 23008 23008 23009 23009 23090-3 23090-3 23092 23092 23094-1 23094-1 23094-2 23094-2 23270 23270 23271 23271 23360 23360 23941 23941 24517 24517 24613 24613 24617 24617 24707 24707 24728 24728 25178 25178 25964 25964 26000 26000 26262 26262 26300 26300 26324 26324 27000 series 27000 series 27000 27000 27001 27001 27002 27002 27005 27005 27006 27006 27729 27729 28000 28000 29110 29148 29148 29199-2 29199-2 29500 29500 30170 30170 31000 31000 32000 32000 37001 37001 38500 38500 39075 39075 40500 40500 42010 42010 45001 45001 50001 50001 55000 55000 56000 56000 80000 80000 Category Category v v t t e e IEC standards 60027 60027 60034 60034 60038 60038 60062 60062 60063 60063 60068 60068 60112 60112 60228 60228 60269 60269 60297 60297 60309 60309 60320 60320 60364 60364 60446 60446 60559 60559 60601 60601 60870 60870-5 60870-6 60870 60870-5 60870-5 60870-6 60870-6 60906-1 60906-1 60908 60908 60929 60929 60958 60958 60980-344 60980-344 61030 61030 61131 61131-3 61131-9 61131 61131-3 61131-3 61131-9 61131-9 61158 61158 61162 61162 61334 61334 61355 61355 61360 61360 61400 61400 61499 61499 61508 61508 61511 61511 61784 61784 61850 61850 61851 61851 61883 61883 61960 61960 61968 61968 61970 61970 62014-4 62014-4 62026 62026 62056 62056 62061 62061 62196 62196 62262 62262 62264 62264 62304 62304 62325 62325 62351 62351 62365 62365 62366 62366 62379 62379 62386 62386 62455 62455 62680 62680 62682 62682 62700 62700 63110 63110 63119 63119 63382 63382 646 646 1989 1989 2022 2022 4909 4909 5218 5218 6429 6429 6523 6523 7810 7810 7811 7811 7812 7812 7813 7813 7816 7816 7942 7942 8613 8613 8632 8632 8652 8652 8859 8859 9126 9126 9293 9293 9496 9496 9529 9529 9592 9592 9593 9593 9899 9899 9945 9945 9995 9995 10021 10021 10116 10116 10165 10165 10179 10179 10279 10279 10646 10646 10967 10967 11172 11172 11179 11179 11404 11404 11544 11544 11801 11801 12207 12207 13250 13250 13346 13346 13522-5 13522-5 13568 13568 13816 13816 13818 13818 14443 14443 14496 14496 14651 14651 14882 14882 15288 15288 15291 15291 15408 15408 15444 15444 15445 15445 15504 15504 15511 15511 15693 15693 15897 15897 15938 15938 16262 16262 16485 16485 17024 17024 17025 17025 18004 18004 18014 18014 18181 18181 19752 19752 19757 19757 19770 19770 19788 19788 20000 20000 20802 20802 21000 21000 21827 21827 22275 22275 22537 22537 23000 23000 23003 23003 23008 23008 23270 23270 23360 23360 24707 24707 24727 24727 24744 24744 24752 24752 26300 26300 27000 27000 27000-series 27000-series 27002 27002 27040 27040 29110 29119 29119 33001 33001 38500 38500 39075 39075 42010 42010 80000 80000 81346 81346 International Electrotechnical Commission International Electrotechnical Commission Authority control databases Germany Germany Germany United States United States United States France France France BnF data BnF data BnF data Czech Republic Czech Republic Czech Republic Czech Republic Spain Spain Spain Israel Israel Israel IdRef IdRef IdRef https://en.wikipedia.org/w/index.php?title=C%2B%2B&oldid=1264775453 Categories C++ C++ Algol programming language family Algol programming language family C++ programming language family C++ programming language family Class-based programming languages Class-based programming languages Cross-platform software Cross-platform software High-level programming languages High-level programming languages Object-oriented programming languages Object-oriented programming languages Programming languages created in 1983 Programming languages created in 1983 Programming languages with an ISO standard Programming languages with an ISO standard Statically typed programming languages Statically typed programming languages Compiled programming languages Compiled programming languages Webarchive template wayback links Webarchive template wayback links Articles with short description Articles with short description Short description matches Wikidata Short description matches Wikidata Wikipedia pages move-protected due to vandalism Wikipedia pages move-protected due to vandalism Use dmy dates from January 2020 Use dmy dates from January 2020 Articles containing potentially dated statements from 1997 Articles containing potentially dated statements from 1997 All articles containing potentially dated statements All articles containing potentially dated statements All articles with unsourced statements All articles with unsourced statements Articles with unsourced statements from February 2024 Articles with unsourced statements from February 2024 Articles containing potentially dated statements from November 2024 Articles containing potentially dated statements from November 2024 Articles with example C++ code Articles with example C++ code Articles with excerpts Articles with excerpts This page was last edited on 23 December 2024, at 13:06 (UTC). (UTC) Text is available under the Creative Commons Attribution-ShareAlike 4.0 License; additional terms may apply. By using this site, you agree to the Terms of Use and Privacy Policy. Wikipedia\u00ae is a registered trademark of the Wikimedia Foundation, Inc., a non-profit organization. Creative Commons Attribution-ShareAlike 4.0 License Terms of Use Privacy Policy Wikimedia Foundation, Inc. Privacy policy Privacy policy About Wikipedia About Wikipedia Disclaimers Disclaimers Contact Wikipedia Contact Wikipedia Code of Conduct Code of Conduct Developers Developers Statistics Statistics Cookie statement Cookie statement Mobile view Mobile view",
    "https://en.wikipedia.org/wiki/Go_(programming_language)": "Jump to content Main menu Main page Main page Main page Contents Contents Contents Current events Current events Current events Random article Random article Random article About Wikipedia About Wikipedia About Wikipedia Contact us Contact us Contact us Help Help Help Learn to edit Learn to edit Learn to edit Community portal Community portal Community portal Recent changes Recent changes Recent changes Upload file Upload file Upload file Search Search Appearance Donate Donate Donate Create account Create account Create account Log in Log in Log in Personal tools Donate Donate Donate Create account Create account Create account Log in Log in Log in learn more learn more Contributions Contributions Contributions Talk Talk Talk Contents (Top) (Top) 1 History Toggle History subsection 1.1 Branding and styling 1.2 Generics 1.3 Versioning 1 History 1 History Toggle History subsection 1.1 Branding and styling 1.1 Branding and styling 1.1 Branding and styling 1.2 Generics 1.2 Generics 1.2 Generics 1.3 Versioning 1.3 Versioning 1.3 Versioning 2 Design Toggle Design subsection 2.1 Syntax 2.2 Types 2.2.1 Interface system 2.2.2 Generic code using parameterized types 2.2.3 Enumerated types 2.3 Package system 2.4 Concurrency: goroutines and channels 2.4.1 Suitability for parallel programming 2.4.2 Lack of data race safety 2.5 Binaries 2.6 Omissions 2 Design 2 Design Toggle Design subsection 2.1 Syntax 2.1 Syntax 2.1 Syntax 2.2 Types 2.2.1 Interface system 2.2.2 Generic code using parameterized types 2.2.3 Enumerated types 2.2 Types 2.2 Types 2.2.1 Interface system 2.2.1 Interface system 2.2.1 Interface system 2.2.2 Generic code using parameterized types 2.2.2 Generic code using parameterized types 2.2.2 Generic code using parameterized types 2.2.3 Enumerated types 2.2.3 Enumerated types 2.2.3 Enumerated types 2.3 Package system 2.3 Package system 2.3 Package system 2.4 Concurrency: goroutines and channels 2.4.1 Suitability for parallel programming 2.4.2 Lack of data race safety 2.4 Concurrency: goroutines and channels 2.4 Concurrency: goroutines and channels 2.4.1 Suitability for parallel programming 2.4.1 Suitability for parallel programming 2.4.1 Suitability for parallel programming 2.4.2 Lack of data race safety 2.4.2 Lack of data race safety 2.4.2 Lack of data race safety 2.5 Binaries 2.5 Binaries 2.5 Binaries 2.6 Omissions 2.6 Omissions 2.6 Omissions 3 Style 3 Style 3 Style 4 Tools 4 Tools 4 Tools 5 Examples Toggle Examples subsection 5.1 Hello world 5.2 Concurrency 5.3 Testing 5.4 Web app 5 Examples 5 Examples Toggle Examples subsection 5.1 Hello world 5.1 Hello world 5.1 Hello world 5.2 Concurrency 5.2 Concurrency 5.2 Concurrency 5.3 Testing 5.3 Testing 5.3 Testing 5.4 Web app 5.4 Web app 5.4 Web app 6 Applications 6 Applications 6 Applications 7 Reception 7 Reception 7 Reception 8 Naming dispute 8 Naming dispute 8 Naming dispute 9 See also 9 See also 9 See also 10 Notes 10 Notes 10 Notes 11 References 11 References 11 References 12 Further reading 12 Further reading 12 Further reading 13 External links 13 External links 13 External links Toggle the table of contents Go (programming language) Go (programming language) 57 languages Afrikaans Afrikaans Afrikaans \u0627\u0644\u0639\u0631\u0628\u064a\u0629 \u0627\u0644\u0639\u0631\u0628\u064a\u0629 \u0627\u0644\u0639\u0631\u0628\u064a\u0629 Az\u0259rbaycanca Az\u0259rbaycanca Az\u0259rbaycanca \u09ac\u09be\u0982\u09b2\u09be \u09ac\u09be\u0982\u09b2\u09be \u09ac\u09be\u0982\u09b2\u09be \u95a9\u5357\u8a9e / B\u00e2n-l\u00e2m-g\u00fa \u95a9\u5357\u8a9e / B\u00e2n-l\u00e2m-g\u00fa \u95a9\u5357\u8a9e / B\u00e2n-l\u00e2m-g\u00fa \u0411\u0435\u043b\u0430\u0440\u0443\u0441\u043a\u0430\u044f \u0411\u0435\u043b\u0430\u0440\u0443\u0441\u043a\u0430\u044f \u0411\u0435\u043b\u0430\u0440\u0443\u0441\u043a\u0430\u044f \u0411\u044a\u043b\u0433\u0430\u0440\u0441\u043a\u0438 \u0411\u044a\u043b\u0433\u0430\u0440\u0441\u043a\u0438 \u0411\u044a\u043b\u0433\u0430\u0440\u0441\u043a\u0438 Brezhoneg Brezhoneg Brezhoneg Catal\u00e0 Catal\u00e0 Catal\u00e0 \u010ce\u0161tina \u010ce\u0161tina \u010ce\u0161tina Dansk Dansk Dansk Deutsch Deutsch Deutsch Eesti Eesti Eesti \u0395\u03bb\u03bb\u03b7\u03bd\u03b9\u03ba\u03ac \u0395\u03bb\u03bb\u03b7\u03bd\u03b9\u03ba\u03ac \u0395\u03bb\u03bb\u03b7\u03bd\u03b9\u03ba\u03ac Espa\u00f1ol Espa\u00f1ol Espa\u00f1ol Euskara Euskara Euskara \u0641\u0627\u0631\u0633\u06cc \u0641\u0627\u0631\u0633\u06cc \u0641\u0627\u0631\u0633\u06cc Fran\u00e7ais Fran\u00e7ais Fran\u00e7ais \u0a97\u0ac1\u0a9c\u0ab0\u0abe\u0aa4\u0ac0 \u0a97\u0ac1\u0a9c\u0ab0\u0abe\u0aa4\u0ac0 \u0a97\u0ac1\u0a9c\u0ab0\u0abe\u0aa4\u0ac0 \ud55c\uad6d\uc5b4 \ud55c\uad6d\uc5b4 \ud55c\uad6d\uc5b4 \u0540\u0561\u0575\u0565\u0580\u0565\u0576 \u0540\u0561\u0575\u0565\u0580\u0565\u0576 \u0540\u0561\u0575\u0565\u0580\u0565\u0576 Hrvatski Hrvatski Hrvatski Bahasa Indonesia Bahasa Indonesia Bahasa Indonesia \u00cdslenska \u00cdslenska \u00cdslenska Italiano Italiano Italiano \u05e2\u05d1\u05e8\u05d9\u05ea \u05e2\u05d1\u05e8\u05d9\u05ea \u05e2\u05d1\u05e8\u05d9\u05ea \u041a\u044b\u0440\u0433\u044b\u0437\u0447\u0430 \u041a\u044b\u0440\u0433\u044b\u0437\u0447\u0430 \u041a\u044b\u0440\u0433\u044b\u0437\u0447\u0430 Latvie\u0161u Latvie\u0161u Latvie\u0161u Magyar Magyar Magyar \u0d2e\u0d32\u0d2f\u0d3e\u0d33\u0d02 \u0d2e\u0d32\u0d2f\u0d3e\u0d33\u0d02 \u0d2e\u0d32\u0d2f\u0d3e\u0d33\u0d02 Bahasa Melayu Bahasa Melayu Bahasa Melayu Nederlands Nederlands Nederlands \u65e5\u672c\u8a9e \u65e5\u672c\u8a9e \u65e5\u672c\u8a9e Norsk bokm\u00e5l Norsk bokm\u00e5l Norsk bokm\u00e5l O\u02bbzbekcha / \u045e\u0437\u0431\u0435\u043a\u0447\u0430 O\u02bbzbekcha / \u045e\u0437\u0431\u0435\u043a\u0447\u0430 O\u02bbzbekcha / \u045e\u0437\u0431\u0435\u043a\u0447\u0430 Polski Polski Polski Portugu\u00eas Portugu\u00eas Portugu\u00eas Qaraqalpaqsha Qaraqalpaqsha Qaraqalpaqsha Rom\u00e2n\u0103 Rom\u00e2n\u0103 Rom\u00e2n\u0103 \u0420\u0443\u0441\u0441\u043a\u0438\u0439 \u0420\u0443\u0441\u0441\u043a\u0438\u0439 \u0420\u0443\u0441\u0441\u043a\u0438\u0439 Shqip Shqip Shqip \u0dc3\u0dd2\u0d82\u0dc4\u0dbd \u0dc3\u0dd2\u0d82\u0dc4\u0dbd \u0dc3\u0dd2\u0d82\u0dc4\u0dbd Simple English Simple English Simple English Sloven\u010dina Sloven\u010dina Sloven\u010dina Sloven\u0161\u010dina Sloven\u0161\u010dina Sloven\u0161\u010dina \u0421\u0440\u043f\u0441\u043a\u0438 / srpski \u0421\u0440\u043f\u0441\u043a\u0438 / srpski \u0421\u0440\u043f\u0441\u043a\u0438 / srpski Suomi Suomi Suomi Svenska Svenska Svenska \u0ba4\u0bae\u0bbf\u0bb4\u0bcd \u0ba4\u0bae\u0bbf\u0bb4\u0bcd \u0ba4\u0bae\u0bbf\u0bb4\u0bcd \u0422\u043e\u04b7\u0438\u043a\u04e3 \u0422\u043e\u04b7\u0438\u043a\u04e3 \u0422\u043e\u04b7\u0438\u043a\u04e3 T\u00fcrk\u00e7e T\u00fcrk\u00e7e T\u00fcrk\u00e7e \u0423\u043a\u0440\u0430\u0457\u043d\u0441\u044c\u043a\u0430 \u0423\u043a\u0440\u0430\u0457\u043d\u0441\u044c\u043a\u0430 \u0423\u043a\u0440\u0430\u0457\u043d\u0441\u044c\u043a\u0430 \u0627\u0631\u062f\u0648 \u0627\u0631\u062f\u0648 \u0627\u0631\u062f\u0648 Ti\u1ebfng Vi\u1ec7t Ti\u1ebfng Vi\u1ec7t Ti\u1ebfng Vi\u1ec7t \u5434\u8bed \u5434\u8bed \u5434\u8bed \u7cb5\u8a9e \u7cb5\u8a9e \u7cb5\u8a9e \u4e2d\u6587 \u4e2d\u6587 \u4e2d\u6587 Edit links Edit links Article Article Article Talk Talk Talk English Read Read Read Edit Edit Edit View history View history View history Tools Read Read Read Edit Edit Edit View history View history View history What links here What links here What links here Related changes Related changes Related changes Upload file Upload file Upload file Special pages Special pages Special pages Permanent link Permanent link Permanent link Page information Page information Page information Cite this page Cite this page Cite this page Get shortened URL Get shortened URL Get shortened URL Download QR code Download QR code Download QR code Download as PDF Download as PDF Download as PDF Printable version Printable version Printable version Wikimedia Commons Wikimedia Commons Wikimedia Commons Wikidata item Wikidata item Wikidata item Go! (programming language) Paradigm Multi-paradigm concurrent imperative functional [1] [ ] object-oriented [2] [ ] [3] [ ] Designed by Robert Griesemer Rob Pike Ken Thompson [4] [ ] Developer [5] [ ] ; 15 years ago (2009-11-10) 2009-11-10 Stable release 1.23.3 1.23.3 ; 57 days ago (6 November 2024) 6 November 2024 Typing discipline Inferred static strong [6] [ ] structural [7] [ ] [8] [ ] nominal Memory management Garbage collection Assembly language C++ OS DragonFly BSD FreeBSD Linux macOS NetBSD OpenBSD [9] [ ] Plan 9 [10] [ ] Solaris Windows License 3-clause BSD [5] [ ] patent [11] [ ] Filename extensions go.dev go.dev implementations C, Oberon-2, Limbo, Active Oberon, communicating sequential processes, Pascal, Oberon, Smalltalk, Newsqueak, Modula-2, Alef, APL, BCPL, Modula, occam C C Oberon-2 Oberon-2 Limbo Limbo Active Oberon Active Oberon communicating sequential processes communicating sequential processes Pascal Pascal Oberon Oberon Smalltalk Smalltalk Newsqueak Newsqueak Modula-2 Modula-2 Alef Alef APL APL BCPL BCPL Modula Modula occam occam Crystal V Go is a fast[12] statically typed, compiled high-level general purpose programming language. It is known for its simplicity and efficiency.[13] Its simplicity express through its basic syntax of the language itself and its large library that help the developer to have a small stack for its project. It was designed at Google[14] in 2009 by Robert Griesemer, Rob Pike, and Ken Thompson.[4] It is syntactically similar to C, but also has memory safety, garbage collection, structural typing,[7] and CSP-style concurrency.[15] It is often referred to as Golang because of its former domain name, golang.org, but its proper name is Go.[16] [12] [ ] statically typed compiled high-level general purpose programming language [13] [ ] Google [14] [ ] Robert Griesemer Rob Pike Ken Thompson [4] [ ] syntactically C memory safety garbage collection structural typing [7] [ ] CSP concurrency [15] [ ] [16] [ ] There are two major implementations: The original, self-hosting[17] compiler toolchain, initially developed inside Google;[18] self-hosting [17] [ ] compiler toolchain [18] [ ] A frontend written in C++, called gofrontend,[19] originally a GCC frontend, providing gccgo, a GCC-based Go compiler;[20] later extended to also support LLVM, providing an LLVM-based Go compiler called gollvm.[21] C++ [19] [ ] GCC [20] [ ] LLVM [21] [ ] A third-party source-to-source compiler, GopherJS,[22] transpiles Go to JavaScript for front-end web development. source-to-source compiler [22] [ ] JavaScript front-end web development History [edit] [ edit edit ] Go was designed at Google in 2007 to improve programming productivity in an era of multicore, networked machines and large codebases.[23] The designers wanted to address criticisms of other languages in use at Google, but keep their useful characteristics:[24] Google programming productivity multicore networked machines codebases [23] [ ] [24] [ ] Static typing and run-time efficiency (like C) Static typing run-time C Readability and usability (like Python)[25] Readability usability Python [25] [ ] High-performance networking and multiprocessing networking multiprocessing Its designers were primarily motivated by their shared dislike of C++.[26][27][28] dislike of C++ [26] [ ] [27] [ ] [28] [ ] Go was publicly announced in November 2009,[29] and version 1.0 was released in March 2012.[30][31] Go is widely used in production at Google[32] and in many other organizations and open-source projects. [29] [ ] [30] [ ] [31] [ ] [32] [ ] Branding and styling [edit] [ edit edit ] Gopher The Gopher mascot was introduced in 2009 for the open source launch of the language. The design, by Ren\u00e9e French, borrowed from a c. 2000 WFMU promotion.[33] Gopher mascot open source Ren\u00e9e French WFMU [33] [ ] In November 2016, the Go and Go Mono fonts were released by type designers Charles Bigelow and Kris Holmes specifically for use by the Go project. Go is a humanist sans-serif resembling Lucida Grande, and Go Mono is monospaced. Both fonts adhere to the WGL4 character set and were designed to be legible with a large x-height and distinct letterforms. Both Go and Go Mono adhere to the DIN 1450 standard by having a slashed zero, lowercase l with a tail, and an uppercase I with serifs.[34][35] Charles Bigelow Kris Holmes humanist sans-serif Lucida Grande monospaced WGL4 x-height letterforms DIN [34] [ ] [35] [ ] In April 2018, the original logo was redesigned by brand designer Adam Smith. The new logo is a modern, stylized GO slanting right with trailing streamlines. (The Gopher mascot remained the same.[36]) [36] [ ] Generics [edit] [ edit edit ] The lack of support for generic programming in initial versions of Go drew considerable criticism.[37] The designers expressed an openness to generic programming and noted that built-in functions were in fact type-generic, but are treated as special cases; Pike called this a weakness that might be changed at some point.[38] The Google team built at least one compiler for an experimental Go dialect with generics, but did not release it.[39] generic programming [37] [ ] [38] [ ] [39] [ ] In August 2018, the Go principal contributors published draft designs for generic programming and error handling and asked users to submit feedback.[40][41] However, the error handling proposal was eventually abandoned.[42] error handling [40] [ ] [41] [ ] [42] [ ] In June 2020, a new draft design document[43] was published that would add the necessary syntax to Go for declaring generic functions and types. A code translation tool, go2go, was provided to allow users to try the new syntax, along with a generics-enabled version of the online Go Playground.[44] [43] [ ] go2go [44] [ ] Generics were finally added to Go in version 1.18 on March 15, 2022.[45] [45] [ ] Versioning [edit] [ edit edit ] Go 1 guarantees compatibility[46] for the language specification and major parts of the standard library. All versions up through the current Go 1.23 release[47] have maintained this promise. [46] [ ] [47] [ ] Go does not follow SemVer; rather, each major Go release is supported until there are two newer major releases. Unlike most software, Go calls the second number in a version the major, i.e., in 1.x x is the major version. [48] This is because Go plans to never reach 2.0, given that compatibility is one of language's major selling points.[49] SemVer [48] [ ] [49] [ ] Design [edit] [ edit edit ] Go is influenced by C (especially the Plan 9 dialect[50][failed verification \u2013 see discussion]), but with an emphasis on greater simplicity and safety. It consists of: C Plan 9 dialect [50] [ ] failed verification failed verification see discussion A syntax and environment adopting patterns more common in dynamic languages:[51] Optional concise variable declaration and initialization through type inference (x := 0 instead of var x int = 0; or var x = 0;) Fast compilation[52] Remote package management (go get)[53] and online package documentation[54] dynamic languages [51] [ ] Optional concise variable declaration and initialization through type inference (x := 0 instead of var x int = 0; or var x = 0;) type inference x := 0 var x int = 0 ; var x = 0 ; Fast compilation[52] [52] [ ] Remote package management (go get)[53] and online package documentation[54] [53] [ ] [54] [ ] Distinctive approaches to particular problems: Built-in concurrency primitives: light-weight processes (goroutines), channels, and the select statement An interface system in place of virtual inheritance, and type embedding instead of non-virtual inheritance A toolchain that, by default, produces statically linked native binaries without external Go dependencies Built-in concurrency primitives: light-weight processes (goroutines), channels, and the select statement light-weight processes channels An interface system in place of virtual inheritance, and type embedding instead of non-virtual inheritance interface virtual inheritance A toolchain that, by default, produces statically linked native binaries without external Go dependencies statically linked A desire to keep the language specification simple enough to hold in a programmer's head,[55] in part by omitting features that are common in similar languages. [55] [ ] omitting features that are common in similar languages Syntax [edit] [ edit edit ] Go's syntax includes changes from C aimed at keeping code concise and readable. A combined declaration/initialization operator was introduced that allows the programmer to write i := 3 or s := \"Hello, world!\", without specifying the types of variables used. This contrasts with C's int i = 3; and const char *s = \"Hello, world!\";. C i := 3 s := \"Hello, world!\" without specifying the types int i = 3 ; const char * s = \"Hello, world!\" ; Semicolons still terminate statements;[a] but are implicit when the end of a line occurs.[b] [a] [ ] [b] [ ] Methods may return multiple values, and returning a result, err pair is the conventional way a method indicates an error to its caller in Go.[c] Go adds literal syntaxes for initializing struct parameters by name and for initializing maps and slices. As an alternative to C's three-statement for loop, Go's range expressions allow concise iteration over arrays, slices, strings, maps, and channels.[58] result , err [c] [ ] maps slices [58] [ ] fmt.Println(\"Hello World!\") is a statement. In Go, statements are separated by ending a line (hitting the Enter key) or by a semicolon \";\". Hitting the Enter key adds \";\" to the end of the line implicitly (does not show up in the source code). The left curly bracket { cannot come at the start of a line.[59] [59] [ ] Types [edit] [ edit edit ] Go has a number of built-in types, including numeric ones (byte, int64, float32, etc.), Booleans, and byte strings (string). Strings are immutable; built-in operators and keywords (rather than functions) provide concatenation, comparison, and UTF-8 encoding/decoding.[60] Record types can be defined with the struct keyword.[61] byte int64 float32 Booleans string UTF-8 [60] [ ] Record types struct [61] [ ] For each type T and each non-negative integer constant n, there is an array type denoted [n]T; arrays of differing lengths are thus of different types. Dynamic arrays are available as \"slices\", denoted []T for some type T. These have a length and a capacity specifying when new memory needs to be allocated to expand the array. Several slices may share their underlying memory.[38][62][63] T n array type [n]T Dynamic arrays []T T [38] [ ] [62] [ ] [63] [ ] Pointers are available for all types, and the pointer-to-T type is denoted *T. Address-taking and indirection use the & and * operators, as in C, or happen implicitly through the method call or attribute access syntax.[64][65] There is no pointer arithmetic,[d] except via the special unsafe.Pointer type in the standard library.[66] Pointers T *T & * [64] [ ] [65] [ ] [d] [ ] unsafe.Pointer [66] [ ] For a pair of types K, V, the type map[K]V is the type mapping type-K keys to type-V values, though Go Programming Language specification does not give any performance guarantees or implementation requirements for map types. Hash tables are built into the language, with special syntax and built-in functions. chan T is a channel that allows sending values of type T between concurrent Go processes.[67] K V map[K]V K V chan T concurrent Go processes [67] [ ] Aside from its support for interfaces, Go's type system is nominal: the type keyword can be used to define a new named type, which is distinct from other named types that have the same layout (in the case of a struct, the same members in the same order). Some conversions between types (e.g., between the various integer types) are pre-defined and adding a new type may define additional conversions, but conversions between named types must always be invoked explicitly.[68] For example, the type keyword can be used to define a type for IPv4 addresses, based on 32-bit unsigned integers as follows: interfaces nominal type struct [68] [ ] type IPv4 type ipv4addr uint32 With this type definition, ipv4addr(x) interprets the uint32 value x as an IP address. Simply assigning x to a variable of type ipv4addr is a type error.[69] ipv4addr(x) uint32 x x ipv4addr [69] [ ] Constant expressions may be either typed or \"untyped\"; they are given a type when assigned to a typed variable if the value they represent passes a compile-time check.[70] [70] [ ] Function types are indicated by the func keyword; they take zero or more parameters and return zero or more values, all of which are typed. The parameter and return values determine a function type; thus, func(string, int32) (int, error) is the type of functions that take a string and a 32-bit signed integer, and return a signed integer (of default width) and a value of the built-in interface type error.[71] Function func parameters return func(string, int32) (int, error) string error [71] [ ] Any named type has a method set associated with it. The IP address example above can be extended with a method for checking whether its value is a known standard: method // ZeroBroadcast reports whether addr is 255.255.255.255. func ( addr ipv4addr ) ZeroBroadcast () bool { return addr == 0xFFFFFFFF } Due to nominal typing, this method definition adds a method to ipv4addr, but not on uint32. While methods have special definition and call syntax, there is no distinct method type.[72] ipv4addr uint32 [72] [ ] Interface system [edit] [ edit edit ] Go provides two features that replace class inheritance.[citation needed] class inheritance citation needed citation needed The first is embedding, which can be viewed as an automated form of composition.[73] composition [73] [ ] The second are its interfaces, which provides runtime polymorphism.[74]: 266 Interfaces are a class of types and provide a limited form of structural typing in the otherwise nominal type system of Go. An object which is of an interface type is also of another type, much like C++ objects being simultaneously of a base and derived class. The design of Go interfaces was inspired by protocols from the Smalltalk programming language.[75] Multiple sources use the term duck typing when describing Go interfaces.[76][77] Although the term duck typing is not precisely defined and therefore not wrong, it usually implies that type conformance is not statically checked. Because conformance to a Go interface is checked statically by the Go compiler (except when performing a type assertion), the Go authors prefer the term structural typing.[78] interfaces runtime polymorphism [74] [ ] : 266 structural typing C++ protocols [75] [ ] duck typing [76] [ ] [77] [ ] [78] [ ] The definition of an interface type lists required methods by name and type. Any object of type T for which functions exist matching all the required methods of interface type I is an object of type I as well. The definition of type T need not (and cannot) identify type I. For example, if Shape, Square and Circle are defined as Shape Square and Circle import \"math\" type Shape interface { Area () float64 } type Square struct { // Note: no \"implements\" declaration side float64 } func ( sq Square ) Area () float64 { return sq . side * sq . side } type Circle struct { // No \"implements\" declaration here either radius float64 } func ( c Circle ) Area () float64 { return math . Pi * math . Pow ( c . radius , 2 ) } then both a Square and a Circle are implicitly a Shape and can be assigned to a Shape-typed variable.[74]: 263\u2013268 In formal language, Go's interface system provides structural rather than nominal typing. Interfaces can embed other interfaces with the effect of creating a combined interface that is satisfied by exactly the types that implement the embedded interface and any methods that the newly defined interface adds.[74]: 270 Square Circle Shape Shape [74] [ ] : 263\u2013268 structural nominal [74] [ ] : 270 The Go standard library uses interfaces to provide genericity in several places, including the input/output system that is based on the concepts of Reader and Writer.[74]: 282\u2013283 Reader Writer [74] [ ] : 282\u2013283 Besides calling methods via interfaces, Go allows converting interface values to other types with a run-time type check. The language constructs to do so are the type assertion,[79] which checks against a single potential type: [79] [ ] var shp Shape = Square { 5 } square , ok := shp .( Square ) // Asserts Square type on shp, should work if ok { fmt . Printf ( \"%#v\\n\" , square ) } else { fmt . Println ( \"Can't print shape as Square\" ) } and the type switch,[80] which checks against multiple types:[citation needed] [80] [ ] citation needed citation needed func ( sq Square ) Diagonal () float64 { return sq . side * math . Sqrt2 } func ( c Circle ) Diameter () float64 { return 2 * c . radius } func LongestContainedLine ( shp Shape ) float64 { switch v := shp .( type ) { case Square : return v . Diagonal () // Or, with type assertion, shp.(Square).Diagonal() case Circle : return v . Diameter () // Or, with type assertion, shp.(Circle).Diameter() default : return 0 // In practice, this should be handled with errors } } The empty interface interface{} is an important base case because it can refer to an item of any concrete type. It is similar to the Object class in Java or C# and is satisfied by any type, including built-in types like int.[74]: 284 Code using the empty interface cannot simply call methods (or built-in operators) on the referred-to object, but it can store the interface{} value, try to convert it to a more useful type via a type assertion or type switch, or inspect it with Go's reflect package.[81] Because interface{} can refer to any value, it is a limited way to escape the restrictions of static typing, like void* in C but with additional run-time type checks.[citation needed] interface {} Object Java C# int [74] [ ] : 284 interface {} [81] [ ] interface {} void * citation needed citation needed The interface{} type can be used to model structured data of any arbitrary schema in Go, such as JSON or YAML data, by representing it as a map[string]interface{} (map of string to empty interface). This recursively describes data in the form of a dictionary with string keys and values of any type.[82] interface {} JSON YAML map [ string ] interface {} [82] [ ] Interface values are implemented using pointer to data and a second pointer to run-time type information.[83] Like some other types implemented using pointers in Go, interface values are nil if uninitialized.[84] [83] [ ] [84] [ ] Generic code using parameterized types [edit] [ edit edit ] Since version 1.18, Go supports generic code using parameterized types.[85] [85] [ ] Functions and types now have the ability to be generic using type parameters. These type parameters are specified within square brackets, right after the function or type name.[86] The compiler transforms the generic function or type into non-generic by substituting type arguments for the type parameters provided, either explicitly by the user or type inference by the compiler.[87] This transformation process is referred to as type instantiation.[88] [86] [ ] [87] [ ] [88] [ ] Interfaces now can define a set of types (known as type set) using | (Union) operator, as well as a set of methods. These changes were made to support type constraints in generics code. For a generic function or type, a constraint can be thought of as the type of the type argument: a meta-type. This new ~T syntax will be the first use of ~ as a token in Go. ~T means the set of all types whose underlying type is T.[89] [89] [ ] type Number interface { ~ int | ~ float64 | ~ float32 | ~ int32 | ~ int64 } func Add [ T Number ]( nums ... T ) T { var sum T for _ , v := range nums { sum += v } return sum } func main () { add := Add [ int ] // Type instantiation println ( add ( 1 , 2 , 3 , 4 , 5 )) // 15 res := Add ( 1.1 , 2.2 , 3.3 , 4.4 , 5.5 ) // Type Inference println ( res ) // +1.650000e+001 } Enumerated types [edit] [ edit edit ] Enumerated type \u00a7 Go [edit] [ edit ] Go uses the iota keyword to create enumerated constants.[90] [90] [ ] type ByteSize float64 const ( _ = iota // ignore first value by assigning to blank identifier KB ByteSize = 1 << ( 10 * iota ) MB GB ) Package system [edit] [ edit edit ] In Go's package system, each package has a path (e.g., \"compress/bzip2\" or \"golang.org/x/net/html\") and a name (e.g., bzip2 or html). References to other packages' definitions must always be prefixed with the other package's name, and only the capitalized names from other packages are accessible: io.Reader is public but bzip2.reader is not.[91] The go get command can retrieve packages stored in a remote repository[92] and developers are encouraged to develop packages inside a base path corresponding to a source repository (such as example.com/user_name/package_name) to reduce the likelihood of name collision with future additions to the standard library or other external libraries.[93] [91] [ ] [92] [ ] [93] [ ] Concurrency: goroutines and channels [edit] [ edit edit ] The Go language has built-in facilities, as well as library support, for writing concurrent programs. Concurrency refers not only to CPU parallelism, but also to asynchrony: letting slow operations like a database or network read run while the program does other work, as is common in event-based servers.[94] concurrent programs asynchrony [94] [ ] The primary concurrency construct is the goroutine, a type of green thread.[95]: 280\u2013281 A function call prefixed with the go keyword starts a function in a new goroutine. The language specification does not specify how goroutines should be implemented, but current implementations multiplex a Go process's goroutines onto a smaller set of operating-system threads, similar to the scheduling performed in Erlang.[96]: 10 green thread [95] [ ] : 280\u2013281 operating-system threads Erlang [96] [ ] : 10 While a standard library package featuring most of the classical concurrency control structures (mutex locks, etc.) is available,[96]: 151\u2013152 idiomatic concurrent programs instead prefer channels, which send messages between goroutines.[97] Optional buffers store messages in FIFO order[98]: 43 and allow sending goroutines to proceed before their messages are received.[95]: 233 concurrency control mutex [96] [ ] : 151\u2013152 send messages [97] [ ] FIFO [98] [ ] : 43 [95] [ ] : 233 Channels are typed, so that a channel of type chan T can only be used to transfer messages of type T. Special syntax is used to operate on them; <-ch is an expression that causes the executing goroutine to block until a value comes in over the channel ch, while ch <- x sends the value x (possibly blocking until another goroutine receives the value). The built-in switch-like select statement can be used to implement non-blocking communication on multiple channels; see below for an example. Go has a memory model describing how goroutines must use channels or other operations to safely share data.[99] chan T T <-ch ch ch <- x x switch select below [99] [ ] The existence of channels does not by itself set Go apart from actor model-style concurrent languages like Erlang, where messages are addressed directly to actors (corresponding to goroutines). In the actor model, channels are themselves actors, therefore addressing a channel just means to address an actor. The actor style can be simulated in Go by maintaining a one-to-one correspondence between goroutines and channels, but the language allows multiple goroutines to share a channel or a single goroutine to send and receive on multiple channels.[96]: 147 actor model [96] [ ] : 147 From these tools one can build concurrent constructs like worker pools, pipelines (in which, say, a file is decompressed and parsed as it downloads), background calls with timeout, \"fan-out\" parallel calls to a set of services, and others.[100] Channels have also found uses further from the usual notion of interprocess communication, like serving as a concurrency-safe list of recycled buffers,[101] implementing coroutines (which helped inspire the name goroutine),[102] and implementing iterators.[103] [100] [ ] [101] [ ] coroutines [102] [ ] iterators [103] [ ] Concurrency-related structural conventions of Go (channels and alternative channel inputs) are derived from Tony Hoare's communicating sequential processes model. Unlike previous concurrent programming languages such as Occam or Limbo (a language on which Go co-designer Rob Pike worked),[104] Go does not provide any built-in notion of safe or verifiable concurrency.[105] While the communicating-processes model is favored in Go, it is not the only one: all goroutines in a program share a single address space. This means that mutable objects and pointers can be shared between goroutines; see \u00a7 Lack of data race safety, below. channels Tony Hoare's communicating sequential processes Occam Limbo [104] [ ] [105] [ ] \u00a7 Lack of data race safety Suitability for parallel programming [edit] [ edit edit ] Although Go's concurrency features are not aimed primarily at parallel processing,[94] they can be used to program shared-memory multi-processor machines. Various studies have been done into the effectiveness of this approach.[106] One of these studies compared the size (in lines of code) and speed of programs written by a seasoned programmer not familiar with the language and corrections to these programs by a Go expert (from Google's development team), doing the same for Chapel, Cilk and Intel TBB. The study found that the non-expert tended to write divide-and-conquer algorithms with one go statement per recursion, while the expert wrote distribute-work-synchronize programs using one goroutine per processor core. The expert's programs were usually faster, but also longer.[107] parallel processing [94] [ ] shared-memory multi-processor [106] [ ] lines of code Chapel Cilk Intel TBB divide-and-conquer go [107] [ ] Lack of data race safety [edit] [ edit edit ] Go's approach to concurrency can be summarized as \"don't communicate by sharing memory; share memory by communicating\".[108] There are no restrictions on how goroutines access shared data, making data races possible. Specifically, unless a program explicitly synchronizes via channels or other means, writes from one goroutine might be partly, entirely, or not at all visible to another, often with no guarantees about ordering of writes.[105] Furthermore, Go's internal data structures like interface values, slice headers, hash tables, and string headers are not immune to data races, so type and memory safety can be violated in multithreaded programs that modify shared instances of those types without synchronization.[109][110] Instead of language support, safe concurrent programming thus relies on conventions; for example, Chisnall recommends an idiom called \"aliases xor mutable\", meaning that passing a mutable value (or pointer) over a channel signals a transfer of ownership over the value to its receiver.[96]: 155 The gc toolchain has an optional data race detector that can check for unsynchronized access to shared memory during runtime since version 1.1,[111] additionally a best-effort race detector is also included by default since version 1.6 of the gc runtime for access to the map data type.[112] [108] [ ] data races [105] [ ] [109] [ ] [110] [ ] xor [96] [ ] : 155 [111] [ ] [112] [ ] Binaries [edit] [ edit edit ] The linker in the gc toolchain creates statically linked binaries by default; therefore all Go binaries include the Go runtime.[113][114] [113] [ ] [114] [ ] Omissions [edit] [ edit edit ] Go deliberately omits certain features common in other languages, including (implementation) inheritance, assertions,[e] pointer arithmetic,[d] implicit type conversions, untagged unions,[f] and tagged unions.[g] The designers added only those facilities that all three agreed on.[117] (implementation) inheritance assertions [e] [ ] pointer arithmetic [d] [ ] implicit type conversions untagged unions [f] [ ] tagged unions [g] [ ] [117] [ ] Of the omitted language features, the designers explicitly argue against assertions and pointer arithmetic, while defending the choice to omit type inheritance as giving a more useful language, encouraging instead the use of interfaces to achieve dynamic dispatch[h] and composition to reuse code. Composition and delegation are in fact largely automated by struct embedding; according to researchers Schmager et al., this feature \"has many of the drawbacks of inheritance: it affects the public interface of objects, it is not fine-grained (i.e, no method-level control over embedding), methods of embedded objects cannot be hidden, and it is static\", making it \"not obvious\" whether programmers will overuse it to the extent that programmers in other languages are reputed to overuse inheritance.[73] interfaces dynamic dispatch [h] [ ] composition delegation struct [73] [ ] Exception handling was initially omitted in Go due to lack of a \"design that gives value proportionate to the complexity\".[118] An exception-like panic/recover mechanism that avoids the usual try-catch control structure was proposed[119] and released in the March 30, 2010 snapshot.[120] The Go authors advise using it for unrecoverable errors such as those that should halt an entire program or server request, or as a shortcut to propagate errors up the stack within a package.[121][122] Across package boundaries, Go includes a canonical error type, and multi-value returns using this type are the standard idiom.[4] Exception handling [118] [ ] panic recover [119] [ ] [120] [ ] [121] [ ] [122] [ ] [4] [ ] Style [edit] [ edit edit ] The Go authors put substantial effort into influencing the style of Go programs: Indentation, spacing, and other surface-level details of code are automatically standardized by the gofmt tool. It uses tabs for indentation and blanks for alignment. Alignment assumes that an editor is using a fixed-width font.[123] golint does additional style checks automatically, but has been deprecated and archived by the Go maintainers.[124] [123] [ ] [124] [ ] Tools and libraries distributed with Go suggest standard approaches to things like API documentation (godoc),[125] testing (go test), building (go build), package management (go get), and so on. [125] [ ] Go enforces rules that are recommendations in other languages, for example banning cyclic dependencies, unused variables[126] or imports,[127] and implicit type conversions. [126] [ ] [127] [ ] The omission of certain features (for example, functional-programming shortcuts like map and Java-style try/finally blocks) tends to encourage a particular explicit, concrete, and imperative programming style. On day one the Go team published a collection of Go idioms,[125] and later also collected code review comments,[128] talks,[129] and official blog posts[130] to teach Go style and coding philosophy. [125] [ ] [128] [ ] [129] [ ] [130] [ ] Tools [edit] [ edit edit ] The main Go distribution includes tools for building, testing, and analyzing code: building testing analyzing go build, which builds Go binaries using only information in the source files themselves, no separate makefiles go test, for unit testing and microbenchmarks as well as fuzzing go fmt, for formatting code go install, for retrieving and installing remote packages go vet, a static analyzer looking for potential errors in code go run, a shortcut for building and executing code godoc, for displaying documentation or serving it via HTTP gorename, for renaming variables, functions, and so on in a type-safe way go generate, a standard way to invoke code generators go mod, for creating a new module, adding dependencies, upgrading dependencies, etc. It also includes profiling and debugging support, fuzzing capabilities to detect bugs, runtime instrumentation (for example, to track garbage collection pauses), and a data race detector. profiling debugging fuzzing runtime garbage collection data race Another tool maintained by the Go team but is not included in Go distributions is gopls, a language server that provides IDE features such as intelligent code completion to Language Server Protocol compatible editors.[131] IDE intelligent code completion Language Server Protocol [131] [ ] An ecosystem of third-party tools adds to the standard distribution, such as gocode, which enables code autocompletion in many text editors, goimports, which automatically adds/removes package imports as needed, and errcheck, which detects code that might unintentionally ignore errors. Examples [edit] [ edit edit ] Hello world [edit] [ edit edit ] package main import \"fmt\" func main () { fmt . Println ( \"hello world\" ) } where \"fmt\" is the package for formatted I/O, similar to C's C file input/output.[132] I/O C file input/output [132] [ ] Concurrency [edit] [ edit edit ] The following simple program demonstrates Go's concurrency features to implement an asynchronous program. It launches two lightweight threads (\"goroutines\"): one waits for the user to type some text, while the other implements a timeout. The select statement waits for either of these goroutines to send a message to the main routine, and acts on the first message to arrive (example adapted from David Chisnall's book).[96]: 152 concurrency features select [96] [ ] : 152 package main import ( \"fmt\" \"time\" ) func readword ( ch chan string ) { fmt . Println ( \"Type a word, then hit Enter.\" ) var word string fmt . Scanf ( \"%s\" , & word ) ch <- word } func timeout ( t chan bool ) { time . Sleep ( 5 * time . Second ) t <- false } func main () { t := make ( chan bool ) go timeout ( t ) ch := make ( chan string ) go readword ( ch ) select { case word := <- ch : fmt . Println ( \"Received\" , word ) case <- t : fmt . Println ( \"Timeout.\" ) } } Testing [edit] [ edit edit ] The testing package provides support for automated testing of go packages.[133] Target function example: [133] [ ] func ExtractUsername ( email string ) string { at := strings . Index ( email , \"@\" ) return email [: at ] } Test code (note that assert keyword is missing in Go; tests live in <filename>_test.go at the same package): import ( \"testing\" ) func TestExtractUsername ( t * testing . T ) { t . Run ( \"withoutDot\" , func ( t * testing . T ) { username := ExtractUsername ( \"r@google.com\" ) if username != \"r\" { t . Fatalf ( \"Got: %v\\n\" , username ) } }) t . Run ( \"withDot\" , func ( t * testing . T ) { username := ExtractUsername ( \"jonh.smith@example.com\" ) if username != \"jonh.smith\" { t . Fatalf ( \"Got: %v\\n\" , username ) } }) } It is possible to run tests in parallel. Web app [edit] [ edit edit ] The net/http[134] package provides support for creating web applications. net/http [134] [ ] This example would show \"Hello world!\" when localhost:8080 is visited. package main import ( \"fmt\" \"log\" \"net/http\" ) func helloFunc ( w http . ResponseWriter , r * http . Request ) { fmt . Fprintf ( w , \"Hello world!\" ) } func main () { http . HandleFunc ( \"/\" , helloFunc ) log . Fatal ( http . ListenAndServe ( \":8080\" , nil )) } Applications [edit] [ edit edit ] Go has found widespread adoption in various domains due to its robust standard library and ease of use.[135] [135] [ ] Popular applications include: Caddy, a web server that automates the process of setting up HTTPS,[136] Docker, which provides a platform for containerization, aiming to ease the complexities of software development and deployment,[137] Kubernetes, which automates the deployment, scaling, and management of containerized applications,[138] CockroachDB, a distributed SQL database engineered for scalability and strong consistency,[139] and Hugo, a static site generator that prioritizes speed and flexibility, allowing developers to create websites efficiently.[140] Caddy [136] [ ] Docker [137] [ ] Kubernetes [138] [ ] CockroachDB [139] [ ] Hugo [140] [ ] Reception [edit] [ edit edit ] The interface system, and the deliberate omission of inheritance, were praised by Michele Simionato, who likened these characteristics to those of Standard ML, calling it \"a shame that no popular language has followed [this] particular route\".[141] Standard ML [141] [ ] Dave Astels at Engine Yard wrote in 2009:[142] Engine Yard [142] [ ] Go is extremely easy to dive into. There are a minimal number of fundamental language concepts and the syntax is clean and designed to be clear and unambiguous. Go is still experimental and still a little rough around the edges. syntax Go was named Programming Language of the Year by the TIOBE Programming Community Index in its first year, 2009, for having a larger 12-month increase in popularity (in only 2 months, after its introduction in November) than any other language that year, and reached 13th place by January 2010,[143] surpassing established languages like Pascal. By June 2015, its ranking had dropped to below 50th in the index, placing it lower than COBOL and Fortran.[144] But as of January 2017, its ranking had surged to 13th, indicating significant growth in popularity and adoption. Go was again awarded TIOBE Programming Language of the Year in 2016.[145] TIOBE Programming Community Index [143] [ ] Pascal COBOL Fortran [144] [ ] [145] [ ] Bruce Eckel has stated:[146] Bruce Eckel [146] [ ] The complexity of C++ (even more complexity has been added in the new C++), and the resulting impact on productivity, is no longer justified. All the hoops that the C++ programmer had to jump through in order to use a C-compatible language make no sense anymore -- they're just a waste of time and effort. Go makes much more sense for the class of problems that C++ was originally intended to solve. C++ A 2011 evaluation of the language and its gc implementation in comparison to C++ (GCC), Java and Scala by a Google engineer found: gc GCC Scala Go offers interesting language features, which also allow for a concise and standardized notation. The compilers for this language are still immature, which reflects in both performance and binary sizes. [147] [ ] The evaluation got a rebuttal from the Go development team. Ian Lance Taylor, who had improved the Go code for Hundt's paper, had not been aware of the intention to publish his code, and says that his version was \"never intended to be an example of idiomatic or efficient Go\"; Russ Cox then optimized the Go code, as well as the C++ code, and got the Go code to run almost as fast as the C++ version and more than an order of magnitude faster than the code in the paper.[148] [148] [ ] Go's nil combined with the lack of algebraic types leads to difficulty handling failures and base cases.[149][150] nil algebraic types base cases [149] [ ] [150] [ ] Go does not allow an opening brace to appear on its own line, which forces all Go programmers to use the same brace style.[151] [151] [ ] Go has been criticized for focusing on simplicity of implementation rather than correctness and flexibility; as an example, the language uses POSIX file semantics on all platforms, and therefore provides incorrect information on platforms such as Windows (which do not follow the aforementioned standard).[152][153] POSIX Windows [152] [ ] [153] [ ] A study showed that it is as easy to make concurrency bugs with message passing as with shared memory, sometimes even more.[154] [154] [ ] Naming dispute [edit] [ edit edit ] On November 10, 2009, the day of the general release of the language, Francis McCabe, developer of the Go! programming language (note the exclamation point), requested a name change of Google's language to prevent confusion with his language, which he had spent 10 years developing.[155] McCabe raised concerns that \"the 'big guy' will end up steam-rollering over\" him, and this concern resonated with the more than 120 developers who commented on Google's official issues thread saying they should change the name, with some[156] even saying the issue contradicts Google's motto of: Don't be evil.[157] Go! programming language [155] [ ] [156] [ ] Don't be evil [157] [ ] On October 12, 2010, the filed public issue ticket was closed by Google developer Russ Cox (@rsc) with the custom status \"Unfortunate\" accompanied by the following comment: \"There are many computing products and services named Go. In the 11 months since our release, there has been minimal confusion of the two languages.\"[157] [157] [ ] See also [edit] [ edit edit ] Free and open-source software portal Free and open-source software portal Free and open-source software portal Fat pointer Fat pointer Comparison of programming languages Comparison of programming languages Notes [edit] [ edit edit ] ^ But \"To allow complex statements to occupy a single line, a semicolon may be omitted before a closing ) or }\".[56] ^ ^ But \"To allow complex statements to occupy a single line, a semicolon may be omitted before a closing ) or }\".[56] [56] [ ] ^ \"if the newline comes after a token that could end a statement, [the lexer will] insert a semicolon\".[57] ^ ^ \"if the newline comes after a token that could end a statement, [the lexer will] insert a semicolon\".[57] [57] [ ] ^ Usually, exactly one of the result and error values has a value other than the type's zero value; sometimes both do, as when a read or write can only be partially completed, and sometimes neither, as when a read returns 0 bytes. See Semipredicate problem: Multivalued return. ^ ^ Usually, exactly one of the result and error values has a value other than the type's zero value; sometimes both do, as when a read or write can only be partially completed, and sometimes neither, as when a read returns 0 bytes. See Semipredicate problem: Multivalued return. Semipredicate problem: Multivalued return ^ a b Language FAQ \"Why is there no pointer arithmetic? Safety ... never derive an illegal address that succeeds incorrectly ... using array indices can be as efficient as ... pointer arithmetic ... simplify the implementation of the garbage collector....\"[4] ^ a b a b Language FAQ \"Why is there no pointer arithmetic? Safety ... never derive an illegal address that succeeds incorrectly ... using array indices can be as efficient as ... pointer arithmetic ... simplify the implementation of the garbage collector....\"[4] [4] [ ] ^ Language FAQ \"Why does Go not have assertions? ...our experience has been that programmers use them as a crutch to avoid thinking about proper error handling and reporting....\"[4] ^ ^ Language FAQ \"Why does Go not have assertions? ...our experience has been that programmers use them as a crutch to avoid thinking about proper error handling and reporting....\"[4] [4] [ ] ^ Language FAQ \"Why are there no untagged unions...? [they] would violate Go's memory safety guarantees.\"[4] ^ ^ Language FAQ \"Why are there no untagged unions...? [they] would violate Go's memory safety guarantees.\"[4] [4] [ ] ^ Language FAQ \"Why does Go not have variant types? ... We considered [them but] they overlap in confusing ways with interfaces.... [S]ome of what variant types address is already covered, ... although not as elegantly.\"[4] (The tag of an interface type[115] is accessed with a type assertion[116]). ^ ^ Language FAQ \"Why does Go not have variant types? ... We considered [them but] they overlap in confusing ways with interfaces.... [S]ome of what variant types address is already covered, ... although not as elegantly.\"[4] (The tag of an interface type[115] is accessed with a type assertion[116]). [4] [ ] [115] [ ] [116] [ ] ^ Questions \"How do I get dynamic dispatch of methods?\" and \"Why is there no type inheritance?\" in the language FAQ.[4] ^ ^ Questions \"How do I get dynamic dispatch of methods?\" and \"Why is there no type inheritance?\" in the language FAQ.[4] [4] [ ] References [edit] [ edit edit ] official Go tutorial ^ \"Codewalk: First-Class Functions in Go\". Go supports first class functions, higher-order functions, user-defined function types, function literals, closures, and multiple return values. This rich feature set supports a functional programming style in a strongly typed language. ^ ^ \"Codewalk: First-Class Functions in Go\". Go supports first class functions, higher-order functions, user-defined function types, function literals, closures, and multiple return values. This rich feature set supports a functional programming style in a strongly typed language. \"Codewalk: First-Class Functions in Go\" ^ \"Is Go an object-oriented language?\". Retrieved April 13, 2019. Although Go has types and methods and allows an object-oriented style of programming, there is no type hierarchy. ^ ^ \"Is Go an object-oriented language?\". Retrieved April 13, 2019. Although Go has types and methods and allows an object-oriented style of programming, there is no type hierarchy. \"Is Go an object-oriented language?\" . Retrieved April 13, 2019 April 13, ^ \"Go: code that grows with grace\". Retrieved June 24, 2018. Go is Object Oriented, but not in the usual way. ^ ^ \"Go: code that grows with grace\". Retrieved June 24, 2018. Go is Object Oriented, but not in the usual way. \"Go: code that grows with grace\" . Retrieved June 24, 2018 June 24, ^ a b c d e f g h \"Language Design FAQ\". The Go Programming Language. January 16, 2010. Retrieved February 27, 2010. ^ a b c d e f g h a b c d e f g h \"Language Design FAQ\". The Go Programming Language. January 16, 2010. Retrieved February 27, 2010. \"Language Design FAQ\" . Retrieved February 27, 2010 February 27, ^ a b \"Text file LICENSE\". The Go Programming Language. Retrieved October 5, 2012. ^ a b a b \"Text file LICENSE\". The Go Programming Language. Retrieved October 5, 2012. \"Text file LICENSE\" . Retrieved October 5, 2012 October 5, ^ \"The Go Programming Language Specification - the Go Programming Language\". ^ ^ \"The Go Programming Language Specification - the Go Programming Language\". \"The Go Programming Language Specification - the Go Programming Language\" ^ a b \"Why doesn't Go have \"implements\" declarations?\". The Go Programming Language. Retrieved October 1, 2015. ^ a b a b \"Why doesn't Go have \"implements\" declarations?\". The Go Programming Language. Retrieved October 1, 2015. \"Why doesn't Go have \"implements\" declarations?\" . Retrieved October 1, 2015 October 1, ^ Pike, Rob (December 22, 2014). \"Rob Pike on Twitter\". Archived from the original on April 7, 2022. Retrieved March 13, 2016. Go has structural typing, not duck typing. Full interface satisfaction is checked and required. ^ ^ Pike, Rob (December 22, 2014). \"Rob Pike on Twitter\". Archived from the original on April 7, 2022. Retrieved March 13, 2016. Go has structural typing, not duck typing. Full interface satisfaction is checked and required. \"Rob Pike on Twitter\" the original . Retrieved March 13, 2016 March 13, ^ \"lang/go: go-1.4\". OpenBSD ports. December 23, 2014. Retrieved January 19, 2015. ^ ^ \"lang/go: go-1.4\". OpenBSD ports. December 23, 2014. Retrieved January 19, 2015. \"lang/go: go-1.4\" . Retrieved January 19, 2015 January 19, ^ \"Go Porting Efforts\". Go Language Resources. cat-v. January 12, 2010. Retrieved January 18, 2010. ^ ^ \"Go Porting Efforts\". Go Language Resources. cat-v. January 12, 2010. Retrieved January 18, 2010. \"Go Porting Efforts\" . Retrieved January 18, 2010 January 18, ^ \"Additional IP Rights Grant\". The Go Programming Language. Retrieved October 5, 2012. ^ ^ \"Additional IP Rights Grant\". The Go Programming Language. Retrieved October 5, 2012. \"Additional IP Rights Grant\" . Retrieved October 5, 2012 October 5, ^ \"Go Introduction\". www.w3schools.com. Retrieved November 23, 2024. ^ ^ \"Go Introduction\". www.w3schools.com. Retrieved November 23, 2024. \"Go Introduction\" . Retrieved November 23, 2024 November 23, ^ \"Go Introduction\". www.w3schools.com. Retrieved November 23, 2024. ^ ^ \"Go Introduction\". www.w3schools.com. Retrieved November 23, 2024. \"Go Introduction\" . Retrieved November 23, 2024 November 23, ^ Kincaid, Jason (November 10, 2009). \"Google's Go: A New Programming Language That's Python Meets C++\". TechCrunch. Retrieved January 18, 2010. ^ ^ Kincaid, Jason (November 10, 2009). \"Google's Go: A New Programming Language That's Python Meets C++\". TechCrunch. Retrieved January 18, 2010. \"Google's Go: A New Programming Language That's Python Meets C++\" . Retrieved January 18, 2010 January 18, ^ Metz, Cade (May 5, 2011). \"Google Go boldly goes where no code has gone before\". The Register. ^ ^ Metz, Cade (May 5, 2011). \"Google Go boldly goes where no code has gone before\". The Register. \"Google Go boldly goes where no code has gone before\" ^ \"Is the language called Go or Golang?\". Retrieved March 16, 2022. The language is called Go. ^ ^ \"Is the language called Go or Golang?\". Retrieved March 16, 2022. The language is called Go. \"Is the language called Go or Golang?\" . Retrieved March 16, 2022 March 16, ^ \"Go 1.5 Release Notes\". Retrieved January 28, 2016. The compiler and runtime are now implemented in Go and assembler, without C. ^ ^ \"Go 1.5 Release Notes\". Retrieved January 28, 2016. The compiler and runtime are now implemented in Go and assembler, without C. \"Go 1.5 Release Notes\" . Retrieved January 28, 2016 January 28, ^ \"The Go programming language\". Retrieved November 1, 2024. ^ ^ \"The Go programming language\". Retrieved November 1, 2024. \"The Go programming language\" . Retrieved November 1, 2024 November 1, ^ \"gofrontend\". Retrieved November 1, 2024. ^ ^ \"gofrontend\". Retrieved November 1, 2024. \"gofrontend\" . Retrieved November 1, 2024 November 1, ^ \"gccgo\". Retrieved November 1, 2024. gccgo, the GNU compiler for the Go programming language ^ ^ \"gccgo\". Retrieved November 1, 2024. gccgo, the GNU compiler for the Go programming language \"gccgo\" . Retrieved November 1, 2024 November 1, ^ \"Gollvm\". Retrieved November 1, 2024. Gollvm is an LLVM-based Go compiler. ^ ^ \"Gollvm\". Retrieved November 1, 2024. Gollvm is an LLVM-based Go compiler. \"Gollvm\" . Retrieved November 1, 2024 November 1, ^ \"A compiler from Go to JavaScript for running Go code in a browser: Gopherjs/Gopherjs\". GitHub. Archived from the original on December 12, 2023. ^ ^ \"A compiler from Go to JavaScript for running Go code in a browser: Gopherjs/Gopherjs\". GitHub. Archived from the original on December 12, 2023. \"A compiler from Go to JavaScript for running Go code in a browser: Gopherjs/Gopherjs\" GitHub Archived ^ \"Go at Google: Language Design in the Service of Software Engineering\". Retrieved October 8, 2018. ^ ^ \"Go at Google: Language Design in the Service of Software Engineering\". Retrieved October 8, 2018. \"Go at Google: Language Design in the Service of Software Engineering\" . Retrieved October 8, 2018 October 8, ^ Pike, Rob (April 28, 2010). \"Another Go at Language Design\". Stanford EE Computer Systems Colloquium. Stanford University. Video available. ^ ^ Pike, Rob (April 28, 2010). \"Another Go at Language Design\". Stanford EE Computer Systems Colloquium. Stanford University. Video available. \"Another Go at Language Design\" Stanford University Video available ^ \"Frequently Asked Questions (FAQ) - The Go Programming Language\". The Go Programming Language. Retrieved February 26, 2016. ^ ^ \"Frequently Asked Questions (FAQ) - The Go Programming Language\". The Go Programming Language. Retrieved February 26, 2016. \"Frequently Asked Questions (FAQ) - The Go Programming Language\" . Retrieved February 26, 2016 February 26, ^ Binstock, Andrew (May 18, 2011). \"Dr. Dobb's: Interview with Ken Thompson\". Archived from the original on January 5, 2013. Retrieved February 7, 2014. ^ ^ Binstock, Andrew (May 18, 2011). \"Dr. Dobb's: Interview with Ken Thompson\". Archived from the original on January 5, 2013. Retrieved February 7, 2014. \"Dr. Dobb's: Interview with Ken Thompson\" the original . Retrieved February 7, 2014 February 7, ^ Pike, Rob (2012). \"Less is exponentially more\". ^ ^ Pike, Rob (2012). \"Less is exponentially more\". \"Less is exponentially more\" ^ Griesemer, Robert (2015). \"The Evolution of Go\". ^ ^ Griesemer, Robert (2015). \"The Evolution of Go\". \"The Evolution of Go\" ^ Griesemer, Robert; Pike, Rob; Thompson, Ken; Taylor, Ian; Cox, Russ; Kim, Jini; Langley, Adam. \"Hey! Ho! Let's Go!\". Google Open Source. Retrieved May 17, 2018. ^ ^ Griesemer, Robert; Pike, Rob; Thompson, Ken; Taylor, Ian; Cox, Russ; Kim, Jini; Langley, Adam. \"Hey! Ho! Let's Go!\". Google Open Source. Retrieved May 17, 2018. \"Hey! Ho! Let's Go!\" . Retrieved May 17, 2018 May 17, ^ Shankland, Stephen (March 30, 2012). \"Google's Go language turns one, wins a spot at YouTube: The lower-level programming language has matured enough to sport the 1.0 version number. And it's being used for real work at Google\". News. CNet. CBS Interactive Inc. Retrieved August 6, 2017. Google has released version 1 of its Go programming language, an ambitious attempt to improve upon giants of the lower-level programming world such as C and C++. ^ ^ Shankland, Stephen (March 30, 2012). \"Google's Go language turns one, wins a spot at YouTube: The lower-level programming language has matured enough to sport the 1.0 version number. And it's being used for real work at Google\". News. CNet. CBS Interactive Inc. Retrieved August 6, 2017. Google has released version 1 of its Go programming language, an ambitious attempt to improve upon giants of the lower-level programming world such as C and C++. \"Google's Go language turns one, wins a spot at YouTube: The lower-level programming language has matured enough to sport the 1.0 version number. And it's being used for real work at Google\" . Retrieved August 6, 2017 August 6, ^ \"Release History\". The Go Programming Language. ^ ^ \"Release History\". The Go Programming Language. \"Release History\" ^ \"Go FAQ: Is Google using Go internally?\". Retrieved March 9, 2013. ^ ^ \"Go FAQ: Is Google using Go internally?\". Retrieved March 9, 2013. \"Go FAQ: Is Google using Go internally?\" . Retrieved March 9, 2013 March 9, ^ \"The Go Gopher - The Go Programming Language\". go.dev. Retrieved February 9, 2023. ^ ^ \"The Go Gopher - The Go Programming Language\". go.dev. Retrieved February 9, 2023. \"The Go Gopher - The Go Programming Language\" . Retrieved February 9, 2023 February 9, ^ \"Go fonts\". Go. November 16, 2016. Retrieved March 12, 2019. ^ ^ \"Go fonts\". Go. November 16, 2016. Retrieved March 12, 2019. \"Go fonts\" . Retrieved March 12, 2019 March 12, ^ \"Go Font TTFs\". GitHub. Retrieved April 2, 2019. ^ ^ \"Go Font TTFs\". GitHub. Retrieved April 2, 2019. \"Go Font TTFs\" . Retrieved April 2, 2019 April 2, ^ \"Go's New Brand\". The Go Blog. Retrieved November 9, 2018. ^ ^ \"Go's New Brand\". The Go Blog. Retrieved November 9, 2018. \"Go's New Brand\" . Retrieved November 9, 2018 November 9, ^ Merrick, Alice (March 9, 2021). \"Go Developer Survey 2020 Results\". Go Programming Language. Retrieved March 16, 2022. ^ ^ Merrick, Alice (March 9, 2021). \"Go Developer Survey 2020 Results\". Go Programming Language. Retrieved March 16, 2022. \"Go Developer Survey 2020 Results\" . Retrieved March 16, 2022 March 16, ^ a b Pike, Rob (September 26, 2013). \"Arrays, slices (and strings): The mechanics of 'append'\". The Go Blog. Retrieved March 7, 2015. ^ a b a b Pike, Rob (September 26, 2013). \"Arrays, slices (and strings): The mechanics of 'append'\". The Go Blog. Retrieved March 7, 2015. \"Arrays, slices (and strings): The mechanics of 'append'\" . Retrieved March 7, 2015 March 7, ^ \"E2E: Erik Meijer and Robert Griesemer\". Channel 9. Microsoft. May 7, 2012. ^ ^ \"E2E: Erik Meijer and Robert Griesemer\". Channel 9. Microsoft. May 7, 2012. \"E2E: Erik Meijer and Robert Griesemer\" ^ \"Go 2 Draft Designs\". Retrieved September 12, 2018. ^ ^ \"Go 2 Draft Designs\". Retrieved September 12, 2018. \"Go 2 Draft Designs\" . Retrieved September 12, 2018 September 12, ^ \"The Go Blog: Go 2 Draft Designs\". August 28, 2018. ^ ^ \"The Go Blog: Go 2 Draft Designs\". August 28, 2018. \"The Go Blog: Go 2 Draft Designs\" ^ \"Proposal: A built-in Go error check function, \"try\"\". Go repository on GitHub. Retrieved March 16, 2022. ^ ^ \"Proposal: A built-in Go error check function, \"try\"\". Go repository on GitHub. Retrieved March 16, 2022. \"Proposal: A built-in Go error check function, \"try\"\" . Retrieved March 16, 2022 March 16, ^ \"Type Parameters \u2014 Draft Design\". go.googlesource.com. ^ ^ \"Type Parameters \u2014 Draft Design\". go.googlesource.com. \"Type Parameters \u2014 Draft Design\" ^ \"Generics in Go\". bitfieldconsulting.com. December 17, 2021. ^ ^ \"Generics in Go\". bitfieldconsulting.com. December 17, 2021. \"Generics in Go\" ^ \"Go 1.18 is released!\". Go Programming Language. March 15, 2022. Retrieved March 16, 2022. ^ ^ \"Go 1.18 is released!\". Go Programming Language. March 15, 2022. Retrieved March 16, 2022. \"Go 1.18 is released!\" . Retrieved March 16, 2022 March 16, ^ \"Go 1 and the Future of Go Programs\". The Go Programming Language. ^ ^ \"Go 1 and the Future of Go Programs\". The Go Programming Language. \"Go 1 and the Future of Go Programs\" ^ \"Go 1.23 Release Notes\". The Go Programming Language. ^ ^ \"Go 1.23 Release Notes\". The Go Programming Language. \"Go 1.23 Release Notes\" ^ \"Release History\". The Go Programming Language. ^ ^ \"Release History\". The Go Programming Language. \"Release History\" ^ \"Backward Compatibility, Go 1.21, and Go 2\". The Go Programming Language. ^ ^ \"Backward Compatibility, Go 1.21, and Go 2\". The Go Programming Language. \"Backward Compatibility, Go 1.21, and Go 2\" ^ \"A Quick Guide to Go's Assembler\". go.dev. Retrieved December 31, 2021. ^ ^ \"A Quick Guide to Go's Assembler\". go.dev. Retrieved December 31, 2021. \"A Quick Guide to Go's Assembler\" . Retrieved December 31, 2021 December 31, ^ Pike, Rob. \"The Go Programming Language\". YouTube. Retrieved July 1, 2011. ^ ^ Pike, Rob. \"The Go Programming Language\". YouTube. Retrieved July 1, 2011. \"The Go Programming Language\" . Retrieved July 1, 2011 July 1, ^ Pike, Rob (November 10, 2009). The Go Programming Language (flv) (Tech talk). Google. Event occurs at 8:53. ^ ^ Pike, Rob (November 10, 2009). The Go Programming Language (flv) (Tech talk). Google. Event occurs at 8:53. The Go Programming Language (flv) ^ \"Download and install packages and dependencies\". See godoc.org for addresses and documentation of some packages. ^ ^ \"Download and install packages and dependencies\". See godoc.org for addresses and documentation of some packages. \"Download and install packages and dependencies\" godoc.org ^ \"GoDoc\". godoc.org. ^ ^ \"GoDoc\". godoc.org. \"GoDoc\" ^ Pike, Rob. \"The Changelog\" (Podcast). Archived from the original on October 20, 2013. Retrieved October 7, 2013. ^ ^ Pike, Rob. \"The Changelog\" (Podcast). Archived from the original on October 20, 2013. Retrieved October 7, 2013. \"The Changelog\" the original . Retrieved October 7, 2013 October 7, ^ \"Go Programming Language Specification, \u00a7Semicolons\". The Go Programming Language. ^ ^ \"Go Programming Language Specification, \u00a7Semicolons\". The Go Programming Language. \"Go Programming Language Specification, \u00a7Semicolons\" ^ \"Effective Go, \u00a7Semicolons\". The Go Programming Language. ^ ^ \"Effective Go, \u00a7Semicolons\". The Go Programming Language. \"Effective Go, \u00a7Semicolons\" ^ \"The Go Programming Language Specification\". The Go Programming Language. ^ ^ \"The Go Programming Language Specification\". The Go Programming Language. \"The Go Programming Language Specification\" ^ \"Go Syntax\". www.w3schools.com. Retrieved November 23, 2024. ^ ^ \"Go Syntax\". www.w3schools.com. Retrieved November 23, 2024. \"Go Syntax\" . Retrieved November 23, 2024 November 23, ^ Pike, Rob (October 23, 2013). \"Strings, bytes, runes and characters in Go\". ^ ^ Pike, Rob (October 23, 2013). \"Strings, bytes, runes and characters in Go\". \"Strings, bytes, runes and characters in Go\" ^ Doxsey, Caleb. \"Structs and Interfaces \u2014 An Introduction to Programming in Go\". www.golang-book.com. Retrieved October 15, 2018. ^ ^ Doxsey, Caleb. \"Structs and Interfaces \u2014 An Introduction to Programming in Go\". www.golang-book.com. Retrieved October 15, 2018. \"Structs and Interfaces \u2014 An Introduction to Programming in Go\" . Retrieved October 15, 2018 October 15, ^ Gerrand, Andrew. \"Go Slices: usage and internals\". ^ ^ Gerrand, Andrew. \"Go Slices: usage and internals\". \"Go Slices: usage and internals\" ^ The Go Authors. \"Effective Go: Slices\". ^ ^ The Go Authors. \"Effective Go: Slices\". \"Effective Go: Slices\" ^ The Go authors. \"Selectors\". ^ ^ The Go authors. \"Selectors\". \"Selectors\" ^ The Go authors. \"Calls\". ^ ^ The Go authors. \"Calls\". \"Calls\" ^ \"Go Programming Language Specification, \u00a7Package unsafe\". The Go Programming Language. ^ ^ \"Go Programming Language Specification, \u00a7Package unsafe\". The Go Programming Language. \"Go Programming Language Specification, \u00a7Package unsafe\" ^ \"The Go Programming Language Specification\". go.dev. Retrieved December 31, 2021. ^ ^ \"The Go Programming Language Specification\". go.dev. Retrieved December 31, 2021. \"The Go Programming Language Specification\" . Retrieved December 31, 2021 December 31, ^ \"The Go Programming Language Specification\". The Go Programming Language. ^ ^ \"The Go Programming Language Specification\". The Go Programming Language. \"The Go Programming Language Specification\" ^ \"A tour of go\". go.dev. ^ ^ \"A tour of go\". go.dev. \"A tour of go\" ^ \"The Go Programming Language Specification\". The Go Programming Language. ^ ^ \"The Go Programming Language Specification\". The Go Programming Language. \"The Go Programming Language Specification\" ^ \"The Go Programming Language Specification\". go.dev. Retrieved December 31, 2021. ^ ^ \"The Go Programming Language Specification\". go.dev. Retrieved December 31, 2021. \"The Go Programming Language Specification\" . Retrieved December 31, 2021 December 31, ^ \"The Go Programming Language Specification\". The Go Programming Language. ^ ^ \"The Go Programming Language Specification\". The Go Programming Language. \"The Go Programming Language Specification\" ^ a b Schmager, Frank; Cameron, Nicholas; Noble, James (2010). GoHotDraw: evaluating the Go programming language with design patterns. Evaluation and Usability of Programming Languages and Tools. ACM. ^ a b a b Schmager, Frank; Cameron, Nicholas; Noble, James (2010). GoHotDraw: evaluating the Go programming language with design patterns. Evaluation and Usability of Programming Languages and Tools. ACM. ^ a b c d e Balbaert, Ivo (2012). The Way to Go: A Thorough Introduction to the Go Programming Language. iUniverse. ^ a b c d e a b c d e Balbaert, Ivo (2012). The Way to Go: A Thorough Introduction to the Go Programming Language. iUniverse. ^ \"The Evolution of Go\". talks.golang.org. Retrieved March 13, 2016. ^ ^ \"The Evolution of Go\". talks.golang.org. Retrieved March 13, 2016. \"The Evolution of Go\" . Retrieved March 13, 2016 March 13, ^ Diggins, Christopher (November 24, 2009). \"Duck Typing and the Go Programming Language\". Dr. Dobb's, The world of software development. Retrieved March 10, 2016. ^ ^ Diggins, Christopher (November 24, 2009). \"Duck Typing and the Go Programming Language\". Dr. Dobb's, The world of software development. Retrieved March 10, 2016. \"Duck Typing and the Go Programming Language\" . Retrieved March 10, 2016 March 10, ^ Ryer, Mat (December 1, 2015). \"Duck typing in Go\". Retrieved March 10, 2016. ^ ^ Ryer, Mat (December 1, 2015). \"Duck typing in Go\". Retrieved March 10, 2016. \"Duck typing in Go\" . Retrieved March 10, 2016 March 10, ^ \"Frequently Asked Questions (FAQ) - The Go Programming Language\". The Go Programming Language. ^ ^ \"Frequently Asked Questions (FAQ) - The Go Programming Language\". The Go Programming Language. \"Frequently Asked Questions (FAQ) - The Go Programming Language\" ^ \"The Go Programming Language Specification\". The Go Programming Language. ^ ^ \"The Go Programming Language Specification\". The Go Programming Language. \"The Go Programming Language Specification\" ^ \"The Go Programming Language Specification\". The Go Programming Language. ^ ^ \"The Go Programming Language Specification\". The Go Programming Language. \"The Go Programming Language Specification\" ^ \"reflect package\". pkg.go.dev. ^ ^ \"reflect package\". pkg.go.dev. \"reflect package\" ^ \"map[string]interface{} in Go\". bitfieldconsulting.com. June 6, 2020. ^ ^ \"map[string]interface{} in Go\". bitfieldconsulting.com. June 6, 2020. \"map[string]interface{} in Go\" ^ \"Go Data Structures: Interfaces\". Retrieved November 15, 2012. ^ ^ \"Go Data Structures: Interfaces\". Retrieved November 15, 2012. \"Go Data Structures: Interfaces\" . Retrieved November 15, 2012 November 15, ^ \"The Go Programming Language Specification\". The Go Programming Language. ^ ^ \"The Go Programming Language Specification\". The Go Programming Language. \"The Go Programming Language Specification\" ^ \"Go 1.18 Release Notes: Generics\". Go Programming Language. March 15, 2022. Retrieved March 16, 2022. ^ ^ \"Go 1.18 Release Notes: Generics\". Go Programming Language. March 15, 2022. Retrieved March 16, 2022. \"Go 1.18 Release Notes: Generics\" . Retrieved March 16, 2022 March 16, ^ \"Type Parameters Proposal\". go.googlesource.com. Retrieved June 25, 2023. ^ ^ \"Type Parameters Proposal\". go.googlesource.com. Retrieved June 25, 2023. \"Type Parameters Proposal\" . Retrieved June 25, 2023 June 25, ^ \"The Go Programming Language Specification - The Go Programming Language\". go.dev. Retrieved June 25, 2023. ^ ^ \"The Go Programming Language Specification - The Go Programming Language\". go.dev. Retrieved June 25, 2023. \"The Go Programming Language Specification - The Go Programming Language\" . Retrieved June 25, 2023 June 25, ^ \"An Introduction To Generics - The Go Programming Language\". go.dev. Retrieved June 25, 2023. ^ ^ \"An Introduction To Generics - The Go Programming Language\". go.dev. Retrieved June 25, 2023. \"An Introduction To Generics - The Go Programming Language\" . Retrieved June 25, 2023 June 25, ^ \"Type Parameters Proposal\". go.googlesource.com. Retrieved June 25, 2023. ^ ^ \"Type Parameters Proposal\". go.googlesource.com. Retrieved June 25, 2023. \"Type Parameters Proposal\" . Retrieved June 25, 2023 June 25, ^ \"Effective Go\". golang.org. The Go Authors. Retrieved May 13, 2014. ^ ^ \"Effective Go\". golang.org. The Go Authors. Retrieved May 13, 2014. \"Effective Go\" . Retrieved May 13, 2014 May 13, ^ \"A Tutorial for the Go Programming Language\". The Go Programming Language. Retrieved March 10, 2013. In Go the rule about visibility of information is simple: if a name (of a top-level type, function, method, constant or variable, or of a structure field or method) is capitalized, users of the package may see it. Otherwise, the name and hence the thing being named is visible only inside the package in which it is declared. ^ ^ \"A Tutorial for the Go Programming Language\". The Go Programming Language. Retrieved March 10, 2013. In Go the rule about visibility of information is simple: if a name (of a top-level type, function, method, constant or variable, or of a structure field or method) is capitalized, users of the package may see it. Otherwise, the name and hence the thing being named is visible only inside the package in which it is declared. \"A Tutorial for the Go Programming Language\" . Retrieved March 10, 2013 March 10, ^ \"go\". The Go Programming Language. ^ ^ \"go\". The Go Programming Language. \"go\" ^ \"How to Write Go Code\". The Go Programming Language. The packages from the standard library are given short import paths such as \"fmt\" and \"net/http\". For your own packages, you must choose a base path that is unlikely to collide with future additions to the standard library or other external libraries. If you keep your code in a source repository somewhere, then you should use the root of that source repository as your base path. For instance, if you have an Example account at example.com/user, that should be your base path ^ ^ \"How to Write Go Code\". The Go Programming Language. The packages from the standard library are given short import paths such as \"fmt\" and \"net/http\". For your own packages, you must choose a base path that is unlikely to collide with future additions to the standard library or other external libraries. If you keep your code in a source repository somewhere, then you should use the root of that source repository as your base path. For instance, if you have an Example account at example.com/user, that should be your base path \"How to Write Go Code\" ^ a b Pike, Rob (September 18, 2012). \"Concurrency is not Parallelism\". ^ a b a b Pike, Rob (September 18, 2012). \"Concurrency is not Parallelism\". \"Concurrency is not Parallelism\" ^ a b Donovan, Alan A. A.; Kernighan, Brian W. (2016). The Go programming language. Addison-Wesley professional computing series. New York, Munich: Addison-Wesley. ISBN 978-0-13-419044-0. ^ a b a b Donovan, Alan A. A.; Kernighan, Brian W. (2016). The Go programming language. Addison-Wesley professional computing series. New York, Munich: Addison-Wesley. ISBN 978-0-13-419044-0. ISBN 978-0-13-419044-0 ^ a b c d e Chisnall, David (2012). The Go Programming Language Phrasebook. Addison-Wesley. ISBN 9780132919005. ^ a b c d e a b c d e Chisnall, David (2012). The Go Programming Language Phrasebook. Addison-Wesley. ISBN 9780132919005. The Go Programming Language Phrasebook ISBN 9780132919005 ^ \"Effective Go\". The Go Programming Language. ^ ^ \"Effective Go\". The Go Programming Language. \"Effective Go\" ^ Summerfield, Mark (2012). Programming in Go: Creating Applications for the 21st Century. Addison-Wesley. ^ ^ Summerfield, Mark (2012). Programming in Go: Creating Applications for the 21st Century. Addison-Wesley. ^ \"The Go Memory Model\". Retrieved April 10, 2017. ^ ^ \"The Go Memory Model\". Retrieved April 10, 2017. \"The Go Memory Model\" . Retrieved April 10, 2017 April 10, ^ \"Go Concurrency Patterns\". The Go Programming Language. ^ ^ \"Go Concurrency Patterns\". The Go Programming Language. \"Go Concurrency Patterns\" ^ Graham-Cumming, John (August 24, 2013). \"Recycling Memory Buffers in Go\". ^ ^ Graham-Cumming, John (August 24, 2013). \"Recycling Memory Buffers in Go\". \"Recycling Memory Buffers in Go\" ^ \"tree.go\". ^ ^ \"tree.go\". \"tree.go\" ^ Cheslack-Postava, Ewen. \"Iterators in Go\". ^ ^ Cheslack-Postava, Ewen. \"Iterators in Go\". \"Iterators in Go\" ^ Kernighan, Brian W. \"A Descent Into Limbo\". ^ ^ Kernighan, Brian W. \"A Descent Into Limbo\". \"A Descent Into Limbo\" ^ a b \"The Go Memory Model\". Retrieved January 5, 2011. ^ a b a b \"The Go Memory Model\". Retrieved January 5, 2011. \"The Go Memory Model\" . Retrieved January 5, 2011 January 5, ^ Tang, Peiyi (2010). Multi-core parallel programming in Go (PDF). Proc. First International Conference on Advanced Computing and Communications. Archived from the original (PDF) on September 9, 2016. Retrieved May 14, 2015. ^ ^ Tang, Peiyi (2010). Multi-core parallel programming in Go (PDF). Proc. First International Conference on Advanced Computing and Communications. Archived from the original (PDF) on September 9, 2016. Retrieved May 14, 2015. Multi-core parallel programming in Go (PDF) the original (PDF) . Retrieved May 14, 2015 May 14, ^ Nanz, Sebastian; West, Scott; Soares Da Silveira, Kaue. Examining the expert gap in parallel programming (PDF). Euro-Par 2013. CiteSeerX 10.1.1.368.6137. ^ ^ Nanz, Sebastian; West, Scott; Soares Da Silveira, Kaue. Examining the expert gap in parallel programming (PDF). Euro-Par 2013. CiteSeerX 10.1.1.368.6137. Examining the expert gap in parallel programming (PDF) CiteSeerX 10.1.1.368.6137 10.1.1.368.6137 ^ Go Authors. \"Share Memory By Communicating\". ^ ^ Go Authors. \"Share Memory By Communicating\". \"Share Memory By Communicating\" ^ Cox, Russ. \"Off to the Races\". ^ ^ Cox, Russ. \"Off to the Races\". Cox, Russ \"Off to the Races\" ^ Pike, Rob (October 25, 2012). \"Go at Google: Language Design in the Service of Software Engineering\". Google, Inc. \"There is one important caveat: Go is not purely memory safe in the presence of concurrency.\" ^ ^ Pike, Rob (October 25, 2012). \"Go at Google: Language Design in the Service of Software Engineering\". Google, Inc. \"There is one important caveat: Go is not purely memory safe in the presence of concurrency.\" Pike, Rob \"Go at Google: Language Design in the Service of Software Engineering\" ^ \"Introducing the Go Race Detector\". The Go Blog. Retrieved June 26, 2013. ^ ^ \"Introducing the Go Race Detector\". The Go Blog. Retrieved June 26, 2013. \"Introducing the Go Race Detector\" . Retrieved June 26, 2013 June 26, ^ \"Go 1.6 Release Notes - The Go Programming Language\". go.dev. Retrieved November 17, 2023. ^ ^ \"Go 1.6 Release Notes - The Go Programming Language\". go.dev. Retrieved November 17, 2023. \"Go 1.6 Release Notes - The Go Programming Language\" . Retrieved November 17, 2023 November 17, ^ \"Frequently Asked Questions (FAQ) - the Go Programming Language\". ^ ^ \"Frequently Asked Questions (FAQ) - the Go Programming Language\". \"Frequently Asked Questions (FAQ) - the Go Programming Language\" ^ \"A Story of a Fat Go Binary\". September 21, 2018. ^ ^ \"A Story of a Fat Go Binary\". September 21, 2018. \"A Story of a Fat Go Binary\" ^ \"Go Programming Language Specification, \u00a7Interface types\". The Go Programming Language. ^ ^ \"Go Programming Language Specification, \u00a7Interface types\". The Go Programming Language. \"Go Programming Language Specification, \u00a7Interface types\" ^ \"Go Programming Language Specification, \u00a7Type assertions\". The Go Programming Language. ^ ^ \"Go Programming Language Specification, \u00a7Type assertions\". The Go Programming Language. \"Go Programming Language Specification, \u00a7Type assertions\" ^ \"All Systems Are Go\". informIT (Interview). August 17, 2010. Retrieved June 21, 2018. ^ ^ \"All Systems Are Go\". informIT (Interview). August 17, 2010. Retrieved June 21, 2018. \"All Systems Are Go\" . Retrieved June 21, 2018 June 21, ^ \"Language Design FAQ\". November 13, 2009. Archived from the original on November 13, 2009. ^ ^ \"Language Design FAQ\". November 13, 2009. Archived from the original on November 13, 2009. \"Language Design FAQ\" the original ^ \"Proposal for an exception-like mechanism\". golang-nuts. March 25, 2010. Retrieved March 25, 2010. ^ ^ \"Proposal for an exception-like mechanism\". golang-nuts. March 25, 2010. Retrieved March 25, 2010. \"Proposal for an exception-like mechanism\" . Retrieved March 25, 2010 March 25, ^ \"Weekly Snapshot History\". The Go Programming Language. ^ ^ \"Weekly Snapshot History\". The Go Programming Language. \"Weekly Snapshot History\" ^ \"Panic And Recover\". Go wiki. ^ ^ \"Panic And Recover\". Go wiki. \"Panic And Recover\" ^ \"Effective Go\". The Go Programming Language. ^ ^ \"Effective Go\". The Go Programming Language. \"Effective Go\" ^ \"gofmt\". The Go Programming Language. Retrieved February 5, 2021. ^ ^ \"gofmt\". The Go Programming Language. Retrieved February 5, 2021. \"gofmt\" . Retrieved February 5, 2021 February 5, ^ \"golang/lint public archive\". github.com. November 30, 2022. ^ ^ \"golang/lint public archive\". github.com. November 30, 2022. \"golang/lint public archive\" ^ a b \"Effective Go\". The Go Programming Language. ^ a b a b \"Effective Go\". The Go Programming Language. \"Effective Go\" ^ \"Unused local variables\". yourbasic.org. Retrieved February 11, 2021. ^ ^ \"Unused local variables\". yourbasic.org. Retrieved February 11, 2021. \"Unused local variables\" . Retrieved February 11, 2021 February 11, ^ \"Unused package imports\". yourbasic.org. Retrieved February 11, 2021. ^ ^ \"Unused package imports\". yourbasic.org. Retrieved February 11, 2021. \"Unused package imports\" . Retrieved February 11, 2021 February 11, ^ \"Code Review Comments\". GitHub. Retrieved July 3, 2018. ^ ^ \"Code Review Comments\". GitHub. Retrieved July 3, 2018. \"Code Review Comments\" GitHub . Retrieved July 3, 2018 July 3, ^ \"Talks\". Retrieved July 3, 2018. ^ ^ \"Talks\". Retrieved July 3, 2018. \"Talks\" . Retrieved July 3, 2018 July 3, ^ \"Errors Are Values\". Retrieved July 3, 2018. ^ ^ \"Errors Are Values\". Retrieved July 3, 2018. \"Errors Are Values\" . Retrieved July 3, 2018 July 3, ^ \"tools/gopls/README.md at master \u00b7 golang/tools\". GitHub. Retrieved November 17, 2023. ^ ^ \"tools/gopls/README.md at master \u00b7 golang/tools\". GitHub. Retrieved November 17, 2023. \"tools/gopls/README.md at master \u00b7 golang/tools\" . Retrieved November 17, 2023 November 17, ^ \"fmt\". The Go Programming Language. Retrieved April 8, 2019. ^ ^ \"fmt\". The Go Programming Language. Retrieved April 8, 2019. \"fmt\" . Retrieved April 8, 2019 April 8, ^ \"testing\". The Go Programming Language. Retrieved December 27, 2020. ^ ^ \"testing\". The Go Programming Language. Retrieved December 27, 2020. \"testing\" . Retrieved December 27, 2020 December 27, ^ \"http package - net/http - Go Packages\". pkg.go.dev. Retrieved November 23, 2024. ^ ^ \"http package - net/http - Go Packages\". pkg.go.dev. Retrieved November 23, 2024. \"http package - net/http - Go Packages\" . Retrieved November 23, 2024 November 23, ^ Lee, Wei-Meng (November 24, 2022). \"Introduction to the Go Programming Language\". Component Developer Magazine. Archived from the original on June 5, 2023. Retrieved September 8, 2023. ^ ^ Lee, Wei-Meng (November 24, 2022). \"Introduction to the Go Programming Language\". Component Developer Magazine. Archived from the original on June 5, 2023. Retrieved September 8, 2023. \"Introduction to the Go Programming Language\" Component Developer Magazine the original . Retrieved September 8, 2023 September 8, ^ Hoffmann, Frank; Neumeyer, Mandy (August 2018). \"Simply Secure\". Linux Magazine. No. 213. Archived from the original on May 28, 2023. Retrieved September 8, 2023. ^ ^ Hoffmann, Frank; Neumeyer, Mandy (August 2018). \"Simply Secure\". Linux Magazine. No. 213. Archived from the original on May 28, 2023. Retrieved September 8, 2023. \"Simply Secure\" Linux Magazine the original . Retrieved September 8, 2023 September 8, ^ Lee, Wei-Meng (August 31, 2022). \"Introduction to Containerization Using Docker\". CODE Magazine. Archived from the original on May 30, 2023. Retrieved September 8, 2023. ^ ^ Lee, Wei-Meng (August 31, 2022). \"Introduction to Containerization Using Docker\". CODE Magazine. Archived from the original on May 30, 2023. Retrieved September 8, 2023. \"Introduction to Containerization Using Docker\" CODE Magazine Archived . Retrieved September 8, 2023 September 8, ^ Pirker, Alexander (February 24, 2023). \"Kubernetes Security for Starters\". CODE Magazine. Archived from the original on April 1, 2023. Retrieved September 8, 2023. ^ ^ Pirker, Alexander (February 24, 2023). \"Kubernetes Security for Starters\". CODE Magazine. Archived from the original on April 1, 2023. Retrieved September 8, 2023. \"Kubernetes Security for Starters\" CODE Magazine Archived . Retrieved September 8, 2023 September 8, ^ Taft, Rebecca; Sharif, Irfan; Matei, Andrei; Van Benschoten, Nathan; Lewis, Jordan; Grieger, Tobias; Niemi, Kai; Woods, Andy; Birzin, Anne; Poss, Raphael; Bardea, Paul; Ranade, Amruta; Darnell, Ben; Gruneir, Bram; Jaffray, Justin; Zhang, Lucy; Mattis, Peter (June 11, 2020). \"CockroachDB: The Resilient Geo-Distributed SQL Database\". Proceedings of the 2020 ACM SIGMOD International Conference on Management of Data. SIGMOD '20. pp. 1493\u20131509. doi:10.1145/3318464.3386134. ISBN 978-1-4503-6735-6. ^ ^ Taft, Rebecca; Sharif, Irfan; Matei, Andrei; Van Benschoten, Nathan; Lewis, Jordan; Grieger, Tobias; Niemi, Kai; Woods, Andy; Birzin, Anne; Poss, Raphael; Bardea, Paul; Ranade, Amruta; Darnell, Ben; Gruneir, Bram; Jaffray, Justin; Zhang, Lucy; Mattis, Peter (June 11, 2020). \"CockroachDB: The Resilient Geo-Distributed SQL Database\". Proceedings of the 2020 ACM SIGMOD International Conference on Management of Data. SIGMOD '20. pp. 1493\u20131509. doi:10.1145/3318464.3386134. ISBN 978-1-4503-6735-6. SIGMOD 1493\u2013 doi 10.1145/3318464.3386134 10.1145/3318464.3386134 ISBN 978-1-4503-6735-6 ^ Hopkins, Brandon (September 13, 2022). \"Static Site Generation with Hugo\". Linux Journal. Archived from the original on April 8, 2023. Retrieved September 8, 2023. ^ ^ Hopkins, Brandon (September 13, 2022). \"Static Site Generation with Hugo\". Linux Journal. Archived from the original on April 8, 2023. Retrieved September 8, 2023. \"Static Site Generation with Hugo\" Archived . Retrieved September 8, 2023 September 8, ^ Simionato, Michele (November 15, 2009). \"Interfaces vs Inheritance (or, watch out for Go!)\". artima. Retrieved November 15, 2009. ^ ^ Simionato, Michele (November 15, 2009). \"Interfaces vs Inheritance (or, watch out for Go!)\". artima. Retrieved November 15, 2009. \"Interfaces vs Inheritance (or, watch out for Go!)\" . Retrieved November 15, 2009 November 15, ^ Astels, Dave (November 9, 2009). \"Ready, Set, Go!\". engineyard. Archived from the original on October 19, 2018. Retrieved November 9, 2009. ^ ^ Astels, Dave (November 9, 2009). \"Ready, Set, Go!\". engineyard. Archived from the original on October 19, 2018. Retrieved November 9, 2009. \"Ready, Set, Go!\" the original . Retrieved November 9, 2009 November 9, ^ jt (January 11, 2010). \"Google's Go Wins Programming Language Of The Year Award\". jaxenter. Retrieved December 5, 2012. ^ ^ jt (January 11, 2010). \"Google's Go Wins Programming Language Of The Year Award\". jaxenter. Retrieved December 5, 2012. \"Google's Go Wins Programming Language Of The Year Award\" . Retrieved December 5, 2012 December 5, ^ \"TIOBE Programming Community Index for June 2015\". TIOBE Software. June 2015. Retrieved July 5, 2015. ^ ^ \"TIOBE Programming Community Index for June 2015\". TIOBE Software. June 2015. Retrieved July 5, 2015. \"TIOBE Programming Community Index for June 2015\" . Retrieved July 5, 2015 July 5, ^ \"TIOBE Index\". TIOBE. Retrieved July 15, 2024. ^ ^ \"TIOBE Index\". TIOBE. Retrieved July 15, 2024. \"TIOBE Index\" . Retrieved July 15, 2024 July 15, ^ Eckel, Bruce (August 27, 2011). \"Calling Go from Python via JSON-RPC\". Retrieved August 29, 2011. ^ ^ Eckel, Bruce (August 27, 2011). \"Calling Go from Python via JSON-RPC\". Retrieved August 29, 2011. \"Calling Go from Python via JSON-RPC\" . Retrieved August 29, 2011 August 29, ^ Hundt, Robert (2011). Loop recognition in C++/Java/Go/Scala (PDF). Scala Days. ^ ^ Hundt, Robert (2011). Loop recognition in C++/Java/Go/Scala (PDF). Scala Days. Loop recognition in C++/Java/Go/Scala (PDF) ^ Metz, Cade (July 1, 2011). \"Google Go strikes back with C++ bake-off\". The Register. ^ ^ Metz, Cade (July 1, 2011). \"Google Go strikes back with C++ bake-off\". The Register. \"Google Go strikes back with C++ bake-off\" The Register ^ Yager, Will. \"Why Go is not Good\". Retrieved November 4, 2018. ^ ^ Yager, Will. \"Why Go is not Good\". Retrieved November 4, 2018. \"Why Go is not Good\" . Retrieved November 4, 2018 November 4, ^ Dobronszki, Janos. \"Everyday Hassles in Go\". Retrieved November 4, 2018. ^ ^ Dobronszki, Janos. \"Everyday Hassles in Go\". Retrieved November 4, 2018. \"Everyday Hassles in Go\" . Retrieved November 4, 2018 November 4, ^ \"Why are there braces but no semicolons? And why can't I put the opening brace on the next line?\". Retrieved March 26, 2020. The advantages of a single, programmatically mandated format for all Go programs greatly outweigh any perceived disadvantages of the particular style. ^ ^ \"Why are there braces but no semicolons? And why can't I put the opening brace on the next line?\". Retrieved March 26, 2020. The advantages of a single, programmatically mandated format for all Go programs greatly outweigh any perceived disadvantages of the particular style. \"Why are there braces but no semicolons? And why can't I put the opening brace on the next line?\" . Retrieved March 26, 2020 March 26, ^ \"I want off Mr. Golang's Wild Ride\". February 28, 2020. Retrieved November 17, 2020. ^ ^ \"I want off Mr. Golang's Wild Ride\". February 28, 2020. Retrieved November 17, 2020. \"I want off Mr. Golang's Wild Ride\" . Retrieved November 17, 2020 November 17, ^ \"proposal: os: Create/Open/OpenFile() set FILE_SHARE_DELETE on windows #32088\". GitHub. May 16, 2019. Retrieved November 17, 2020. ^ ^ \"proposal: os: Create/Open/OpenFile() set FILE_SHARE_DELETE on windows #32088\". GitHub. May 16, 2019. Retrieved November 17, 2020. \"proposal: os: Create/Open/OpenFile() set FILE_SHARE_DELETE on windows #32088\" GitHub . Retrieved November 17, 2020 November 17, ^ Tu, Tengfei (2019). \"Understanding Real-World Concurrency Bugs in Go\" (PDF). For example, around 58% of blocking bugs are caused by message passing. In addition to the violation of Go's channel usage rules (e.g., waiting on a channel that no one sends data to or close), many concurrency bugs are caused by the mixed usage of message passing and other new semantics and new libraries in Go, which can easily be overlooked but hard to detect ^ ^ Tu, Tengfei (2019). \"Understanding Real-World Concurrency Bugs in Go\" (PDF). For example, around 58% of blocking bugs are caused by message passing. In addition to the violation of Go's channel usage rules (e.g., waiting on a channel that no one sends data to or close), many concurrency bugs are caused by the mixed usage of message passing and other new semantics and new libraries in Go, which can easily be overlooked but hard to detect \"Understanding Real-World Concurrency Bugs in Go\" (PDF) ^ Brownlee, John (November 13, 2009). \"Google didn't google \"Go\" before naming their programming language'\". Archived from the original on December 8, 2015. Retrieved May 26, 2016. ^ ^ Brownlee, John (November 13, 2009). \"Google didn't google \"Go\" before naming their programming language'\". Archived from the original on December 8, 2015. Retrieved May 26, 2016. \"Google didn't google \"Go\" before naming their programming language'\" the original . Retrieved May 26, 2016 May 26, ^ Claburn, Thomas (November 11, 2009). \"Google 'Go' Name Brings Accusations Of Evil'\". InformationWeek. Archived from the original on July 22, 2010. Retrieved January 18, 2010. ^ ^ Claburn, Thomas (November 11, 2009). \"Google 'Go' Name Brings Accusations Of Evil'\". InformationWeek. Archived from the original on July 22, 2010. Retrieved January 18, 2010. \"Google 'Go' Name Brings Accusations Of Evil'\" the original . Retrieved January 18, 2010 January 18, ^ a b \"Issue 9 - go \u2014 I have already used the name for *MY* programming language\". Github. Google Inc. Retrieved October 12, 2010. ^ a b a b \"Issue 9 - go \u2014 I have already used the name for *MY* programming language\". Github. Google Inc. Retrieved October 12, 2010. \"Issue 9 - go \u2014 I have already used the name for *MY* programming language\" Google Inc. Retrieved October 12, 2010 October 12, Further reading [edit] [ edit edit ] Donovan, Alan; Kernighan, Brian (October 2015). The Go Programming Language (1st ed.). Addison-Wesley Professional. p. 400. ISBN 978-0-13-419044-0. The Go Programming Language Addison-Wesley Professional ISBN 978-0-13-419044-0 Bodner, Jon (March 2021). Learning Go (1st ed.). O'Reilly. p. 352. ISBN 9781492077213. Learning Go O'Reilly ISBN 9781492077213 External links [edit] [ edit edit ] Go (programming language) Go (programming language) Official website Official website Official website Official website v v t t e e Programming languages Comparison Comparison Timeline Timeline History History Ada Ada ALGOL Simula ALGOL Simula Simula APL APL Assembly Assembly BASIC Visual Basic classic .NET BASIC Visual Basic classic .NET Visual Basic classic classic .NET .NET C C C++ C++ C# C# COBOL COBOL Erlang Erlang Forth Forth Fortran Fortran Go Go Haskell Haskell Java Java JavaScript JavaScript Julia Julia Kotlin Kotlin Lisp Lisp Lua Lua MATLAB MATLAB ML ML Pascal Object Pascal Pascal Object Pascal Object Pascal Perl Perl PHP PHP Prolog Prolog Python Python R R Ruby Ruby Rust Rust SQL SQL Scratch Scratch Shell Shell Smalltalk Smalltalk Swift Swift more... more... Lists: Alphabetical Alphabetical Categorical Categorical Generational Generational Non-English-based Non-English-based Category Category v v t t e e Google Chromium Chromium Gemma Gemma OpenRefine OpenRefine Tesseract Tesseract Carbon Carbon Dart Dart Go Go Sawzall Sawzall AMP AMP Angular Angular AngularJS AngularJS Beam Beam Bazel Bazel Blockly Blockly Closure Tools Closure Tools Cpplint Cpplint FlatBuffers FlatBuffers Flutter Flutter Ganeti Ganeti Gears Gears Gerrit Gerrit GLOP GLOP gRPC gRPC Gson Gson Guava Guava Guetzli Guetzli Guice Guice gVisor gVisor Kubernetes Kubernetes LevelDB LevelDB libvpx libvpx Lighthouse Lighthouse NaCl NaCl Namebench Namebench Nomulus Nomulus OR-Tools OR-Tools Polymer Polymer Protocol Buffers Protocol Buffers TensorFlow TensorFlow V8 V8 Android Android ChromiumOS ChromiumOS Fuchsia Fuchsia gLinux gLinux Goobuntu Goobuntu Code-in Code-in Google LLC v. Oracle America, Inc. Google LLC v. Oracle America, Inc. Open Source Security Foundation Open Source Security Foundation Summer of Code Summer of Code v v t t e e Rob Pike Plan 9 from Bell Labs Plan 9 from Bell Labs Inferno Inferno Newsqueak Newsqueak Limbo Limbo Sawzall Sawzall Go Go acme acme Blit Blit sam sam rio rio 8\u00bd 8\u00bd The Practice of Programming The Practice of Programming The Unix Programming Environment The Unix Programming Environment Ren\u00e9e French Ren\u00e9e French Mark V. Shaney Mark V. Shaney UTF-8 UTF-8 v v t t e e Ken Thompson Unix Unix Plan 9 from Bell Labs Plan 9 from Bell Labs Inferno Inferno B B Go Go Belle Belle ed ed grep grep sam sam Space Travel Space Travel Thompson shell Thompson shell Bell Labs Bell Labs Google Google UTF-8 UTF-8 Authority control databases FAST FAST FAST Germany Germany Germany United States United States United States United States Israel Israel Israel https://en.wikipedia.org/w/index.php?title=Go_(programming_language)&oldid=1265734780 Categories Go (programming language) Go (programming language) American inventions American inventions C programming language family C programming language family Concurrent programming languages Concurrent programming languages Cross-platform free software Cross-platform free software Cross-platform software Cross-platform software Free and open source compilers Free and open source compilers Google software Google software High-level programming languages High-level programming languages Procedural programming languages Procedural programming languages Programming languages Programming languages Programming languages created in 2009 Programming languages created in 2009 Software using the BSD license Software using the BSD license Statically typed programming languages Statically typed programming languages Systems programming languages Systems programming languages Articles with short description Articles with short description Short description is different from Wikidata Short description is different from Wikidata Use American English from August 2022 Use American English from August 2022 All Wikipedia articles written in American English All Wikipedia articles written in American English Use mdy dates from October 2018 Use mdy dates from October 2018 All articles with failed verification All articles with failed verification Articles with failed verification from June 2022 Articles with failed verification from June 2022 All articles with unsourced statements All articles with unsourced statements Articles with unsourced statements from December 2016 Articles with unsourced statements from December 2016 Articles with excerpts Articles with excerpts Commons category link is on Wikidata Commons category link is on Wikidata Articles prone to spam from June 2013 Articles prone to spam from June 2013 This page was last edited on 28 December 2024, at 11:11 (UTC). (UTC) Text is available under the Creative Commons Attribution-ShareAlike 4.0 License; additional terms may apply. By using this site, you agree to the Terms of Use and Privacy Policy. Wikipedia\u00ae is a registered trademark of the Wikimedia Foundation, Inc., a non-profit organization. Creative Commons Attribution-ShareAlike 4.0 License Terms of Use Privacy Policy Wikimedia Foundation, Inc. Privacy policy Privacy policy About Wikipedia About Wikipedia Disclaimers Disclaimers Contact Wikipedia Contact Wikipedia Code of Conduct Code of Conduct Developers Developers Statistics Statistics Cookie statement Cookie statement Mobile view Mobile view",
    "https://en.wikipedia.org/wiki/Swift_(programming_language)": "Jump to content Main menu Main page Main page Main page Contents Contents Contents Current events Current events Current events Random article Random article Random article About Wikipedia About Wikipedia About Wikipedia Contact us Contact us Contact us Help Help Help Learn to edit Learn to edit Learn to edit Community portal Community portal Community portal Recent changes Recent changes Recent changes Upload file Upload file Upload file Search Search Appearance Donate Donate Donate Create account Create account Create account Log in Log in Log in Personal tools Donate Donate Donate Create account Create account Create account Log in Log in Log in learn more learn more Contributions Contributions Contributions Talk Talk Talk Contents (Top) (Top) 1 History Toggle History subsection 1.1 Platforms 1.2 Version history 1 History 1 History Toggle History subsection 1.1 Platforms 1.1 Platforms 1.1 Platforms 1.2 Version history 1.2 Version history 1.2 Version history 2 Features Toggle Features subsection 2.1 Basic syntax 2.2 Closure support 2.3 String support 2.4 Callable objects 2.5 Access control 2.6 Optionals and chaining 2.7 Value types 2.8 Extensions 2.9 Protocol-oriented programming 2.10 Concurrency 2.11 Libraries, runtime, development 2.12 Memory management 2.13 Debugging 2 Features 2 Features Toggle Features subsection 2.1 Basic syntax 2.1 Basic syntax 2.1 Basic syntax 2.2 Closure support 2.2 Closure support 2.2 Closure support 2.3 String support 2.3 String support 2.3 String support 2.4 Callable objects 2.4 Callable objects 2.4 Callable objects 2.5 Access control 2.5 Access control 2.5 Access control 2.6 Optionals and chaining 2.6 Optionals and chaining 2.6 Optionals and chaining 2.7 Value types 2.7 Value types 2.7 Value types 2.8 Extensions 2.8 Extensions 2.8 Extensions 2.9 Protocol-oriented programming 2.9 Protocol-oriented programming 2.9 Protocol-oriented programming 2.10 Concurrency 2.10 Concurrency 2.10 Concurrency 2.11 Libraries, runtime, development 2.11 Libraries, runtime, development 2.11 Libraries, runtime, development 2.12 Memory management 2.12 Memory management 2.12 Memory management 2.13 Debugging 2.13 Debugging 2.13 Debugging 3 Comparisons to other languages 3 Comparisons to other languages 3 Comparisons to other languages 4 Development and other implementations 4 Development and other implementations 4 Development and other implementations 5 See also 5 See also 5 See also 6 References 6 References 6 References 7 External links 7 External links 7 External links Toggle the table of contents Swift (programming language) Swift (programming language) 47 languages \u0627\u0644\u0639\u0631\u0628\u064a\u0629 \u0627\u0644\u0639\u0631\u0628\u064a\u0629 \u0627\u0644\u0639\u0631\u0628\u064a\u0629 Az\u0259rbaycanca Az\u0259rbaycanca Az\u0259rbaycanca \u09ac\u09be\u0982\u09b2\u09be \u09ac\u09be\u0982\u09b2\u09be \u09ac\u09be\u0982\u09b2\u09be \u0411\u0435\u043b\u0430\u0440\u0443\u0441\u043a\u0430\u044f \u0411\u0435\u043b\u0430\u0440\u0443\u0441\u043a\u0430\u044f \u0411\u0435\u043b\u0430\u0440\u0443\u0441\u043a\u0430\u044f \u0411\u044a\u043b\u0433\u0430\u0440\u0441\u043a\u0438 \u0411\u044a\u043b\u0433\u0430\u0440\u0441\u043a\u0438 \u0411\u044a\u043b\u0433\u0430\u0440\u0441\u043a\u0438 Catal\u00e0 Catal\u00e0 Catal\u00e0 \u010ce\u0161tina \u010ce\u0161tina \u010ce\u0161tina Deutsch Deutsch Deutsch \u0395\u03bb\u03bb\u03b7\u03bd\u03b9\u03ba\u03ac \u0395\u03bb\u03bb\u03b7\u03bd\u03b9\u03ba\u03ac \u0395\u03bb\u03bb\u03b7\u03bd\u03b9\u03ba\u03ac Espa\u00f1ol Espa\u00f1ol Espa\u00f1ol Esperanto Esperanto Esperanto \u0641\u0627\u0631\u0633\u06cc \u0641\u0627\u0631\u0633\u06cc \u0641\u0627\u0631\u0633\u06cc Fran\u00e7ais Fran\u00e7ais Fran\u00e7ais \ud55c\uad6d\uc5b4 \ud55c\uad6d\uc5b4 \ud55c\uad6d\uc5b4 \u0540\u0561\u0575\u0565\u0580\u0565\u0576 \u0540\u0561\u0575\u0565\u0580\u0565\u0576 \u0540\u0561\u0575\u0565\u0580\u0565\u0576 Bahasa Indonesia Bahasa Indonesia Bahasa Indonesia \u00cdslenska \u00cdslenska \u00cdslenska Italiano Italiano Italiano \u05e2\u05d1\u05e8\u05d9\u05ea \u05e2\u05d1\u05e8\u05d9\u05ea \u05e2\u05d1\u05e8\u05d9\u05ea \u041a\u044b\u0440\u0433\u044b\u0437\u0447\u0430 \u041a\u044b\u0440\u0433\u044b\u0437\u0447\u0430 \u041a\u044b\u0440\u0433\u044b\u0437\u0447\u0430 Latina Latina Latina Magyar Magyar Magyar \u0d2e\u0d32\u0d2f\u0d3e\u0d33\u0d02 \u0d2e\u0d32\u0d2f\u0d3e\u0d33\u0d02 \u0d2e\u0d32\u0d2f\u0d3e\u0d33\u0d02 Bahasa Melayu Bahasa Melayu Bahasa Melayu Nederlands Nederlands Nederlands \u65e5\u672c\u8a9e \u65e5\u672c\u8a9e \u65e5\u672c\u8a9e Norsk bokm\u00e5l Norsk bokm\u00e5l Norsk bokm\u00e5l O\u02bbzbekcha / \u045e\u0437\u0431\u0435\u043a\u0447\u0430 O\u02bbzbekcha / \u045e\u0437\u0431\u0435\u043a\u0447\u0430 O\u02bbzbekcha / \u045e\u0437\u0431\u0435\u043a\u0447\u0430 Polski Polski Polski Portugu\u00eas Portugu\u00eas Portugu\u00eas Qaraqalpaqsha Qaraqalpaqsha Qaraqalpaqsha Rom\u00e2n\u0103 Rom\u00e2n\u0103 Rom\u00e2n\u0103 \u0420\u0443\u0441\u0441\u043a\u0438\u0439 \u0420\u0443\u0441\u0441\u043a\u0438\u0439 \u0420\u0443\u0441\u0441\u043a\u0438\u0439 Simple English Simple English Simple English Sloven\u010dina Sloven\u010dina Sloven\u010dina Sloven\u0161\u010dina Sloven\u0161\u010dina Sloven\u0161\u010dina \u06a9\u0648\u0631\u062f\u06cc \u06a9\u0648\u0631\u062f\u06cc \u06a9\u0648\u0631\u062f\u06cc \u0421\u0440\u043f\u0441\u043a\u0438 / srpski \u0421\u0440\u043f\u0441\u043a\u0438 / srpski \u0421\u0440\u043f\u0441\u043a\u0438 / srpski Suomi Suomi Suomi Svenska Svenska Svenska \u0422\u043e\u04b7\u0438\u043a\u04e3 \u0422\u043e\u04b7\u0438\u043a\u04e3 \u0422\u043e\u04b7\u0438\u043a\u04e3 T\u00fcrk\u00e7e T\u00fcrk\u00e7e T\u00fcrk\u00e7e \u0423\u043a\u0440\u0430\u0457\u043d\u0441\u044c\u043a\u0430 \u0423\u043a\u0440\u0430\u0457\u043d\u0441\u044c\u043a\u0430 \u0423\u043a\u0440\u0430\u0457\u043d\u0441\u044c\u043a\u0430 Ti\u1ebfng Vi\u1ec7t Ti\u1ebfng Vi\u1ec7t Ti\u1ebfng Vi\u1ec7t \u5434\u8bed \u5434\u8bed \u5434\u8bed \u7cb5\u8a9e \u7cb5\u8a9e \u7cb5\u8a9e \u4e2d\u6587 \u4e2d\u6587 \u4e2d\u6587 Edit links Edit links Article Article Article Talk Talk Talk English Read Read Read Edit Edit Edit View history View history View history Tools Read Read Read Edit Edit Edit View history View history View history What links here What links here What links here Related changes Related changes Related changes Upload file Upload file Upload file Special pages Special pages Special pages Permanent link Permanent link Permanent link Page information Page information Page information Cite this page Cite this page Cite this page Get shortened URL Get shortened URL Get shortened URL Download QR code Download QR code Download QR code Download as PDF Download as PDF Download as PDF Printable version Printable version Printable version Wikimedia Commons Wikimedia Commons Wikimedia Commons Wikibooks Wikibooks Wikibooks Wikidata item Wikidata item Wikidata item Swift (parallel scripting language) Swift (CPU) Paradigm Multi-paradigm protocol-oriented object-oriented functional imperative block structured declarative concurrent Designed by Chris Lattner Apple Inc. [1] [ ] Developer Apple Inc. ; 10 years ago (2014-06-02) 2014-06-02 [2] [ ] Stable release [3] [ ] ; 20 days ago (13 December 2024) 13 December 2024 Preview release Typing discipline Static strong inferred Memory management Automatic Reference Counting OS Darwin iOS iPadOS macOS tvOS visionOS watchOS Linux Windows Android z/OS License Apache License 2.0 Proprietary [4] [ ] [5] [ ] Filename extensions www.swift.org www.swift.org www.swift.org developer.apple.com/swift/ developer.apple.com/swift/ developer.apple.com/swift/ Objective-C [6] [ ] Rust Haskell Ruby Python C# CLU [7] [ ] D [8] [ ] Rust [9] [ ] V (Vlang) [10] [ ] Swift is a high-level general-purpose, multi-paradigm, compiled programming language created by Chris Lattner in 2010 for Apple Inc. and maintained by the open-source community. Swift compiles to machine code and uses an LLVM-based compiler. Swift was first released in June 2014[11] and the Swift toolchain has shipped in Xcode since Xcode version 6, released in September 2014. high-level general-purpose multi-paradigm compiled programming language Chris Lattner Apple Inc. the open-source community LLVM [11] [ ] Xcode Apple intended Swift to support many core concepts associated with Objective-C, notably dynamic dispatch, widespread late binding, extensible programming, and similar features, but in a \"safer\" way, making it easier to catch software bugs; Swift has features addressing some common programming errors like null pointer dereferencing and provides syntactic sugar to help avoid the pyramid of doom. Swift supports the concept of protocol extensibility, an extensibility system that can be applied to types, structs and classes, which Apple promotes as a real change in programming paradigms they term \"protocol-oriented programming\"[12] (similar to traits and type classes).[13] Objective-C dynamic dispatch late binding extensible programming software bugs null pointer dereferencing syntactic sugar pyramid of doom protocol structs classes [12] [ ] traits type classes [13] [ ] Swift was introduced at Apple's 2014 Worldwide Developers Conference (WWDC).[14] It underwent an upgrade to version 1.2 during 2014 and a major upgrade to Swift 2 at WWDC 2015. It was initially a proprietary language, but version 2.2 was made open-source software under the Apache License 2.0 on December 3, 2015, for Apple's platforms and Linux.[15][16] 2014 Worldwide Developers Conference [14] [ ] proprietary language open-source software Apache License Linux [15] [ ] [16] [ ] Through version 3.0 the syntax of Swift went through significant evolution, with the core team making source stability a focus in later versions.[17][18] In the first quarter of 2018 Swift surpassed Objective-C in measured popularity.[19] syntax [17] [ ] [18] [ ] Objective-C [19] [ ] Swift 4.0, released in 2017, introduced several changes to some built-in classes and structures. Code written with previous versions of Swift can be updated using the migration functionality built into Xcode. Swift 5, released in March 2019, introduced a stable binary interface on Apple platforms, allowing the Swift runtime to be incorporated into Apple operating systems. It is source compatible with Swift 4.[20] [20] [ ] Swift 5.1 was officially released in September 2019. Swift 5.1 builds on the previous version of Swift 5 by extending the stable features of the language to compile-time with the introduction of module stability. The introduction of module stability makes it possible to create and share binary frameworks that will work with future releases of Swift.[21] [21] [ ] Swift 5.5, officially announced by Apple at the 2021 WWDC, significantly expands language support for concurrency and asynchronous code, notably introducing a unique version of the actor model.[22] WWDC concurrency asynchronous code actor model [22] [ ] Swift 5.9, was released in September 2023 and includes a macro system, generic parameter packs, and ownership features like the new consume operator.[23] [23] [ ] Swift 5.10, was released in March 2024. This version improves the language's concurrency model, allowing for full data isolation to prevent data races. It is also the last release before Swift 6.[24] Version 5.10 is currently available for macOS, Windows and for Linux.[25] data races [24] [ ] [25] [ ] Swift 6 was released in September 2024.[26] [26] [ ] History [edit] [ edit edit ] Development of Swift started in July 2010 by Chris Lattner, with the eventual collaboration of many other programmers at Apple. Swift was motivated by the need for a replacement for Apple's earlier programming language Objective-C, which had been largely unchanged since the early 1980s and lacked modern language features. Swift took language ideas \"from Objective-C, Rust, Haskell, Ruby, Python, C#, CLU, and far too many others to list\".[7] On June 2, 2014, the Apple Worldwide Developers Conference (WWDC) application became the first publicly released app written with Swift.[27] A beta version of the programming language was released to registered Apple developers at the conference, but the company did not promise that the final version of Swift would be source code compatible with the test version. Apple planned to make source code converters available if needed for the full release.[27] Chris Lattner Apple Objective-C Objective-C Rust Haskell Ruby Python C# CLU [7] [ ] Apple Worldwide Developers Conference [27] [ ] beta version programming language source code [27] [ ] The Swift Programming Language, a free 500-page manual, was also released at WWDC, and is available on the Apple Books Store and the official website.[28] Apple Books [28] [ ] Swift reached the 1.0 milestone on September 9, 2014, with the Gold Master of Xcode 6.0 for iOS.[29] Swift 1.1 was released on October 22, 2014, alongside the launch of Xcode 6.1.[30] Swift 1.2 was released on April 8, 2015, along with Xcode 6.3.[31] Swift 2.0 was announced at WWDC 2015, and was made available for publishing apps in the App Store on September 21, 2015.[32] Swift 3.0 was released on September 13, 2016.[33] Swift 4.0 was released on September 19, 2017.[34] Swift 4.1 was released on March 29, 2018.[35] Xcode iOS [29] [ ] [30] [ ] [31] [ ] [32] [ ] [33] [ ] [34] [ ] [35] [ ] Swift won first place for Most Loved Programming Language in the Stack Overflow Developer Survey 2015[36] and second place in 2016.[37] Stack Overflow [36] [ ] [37] [ ] On December 3, 2015, the Swift language, supporting libraries, debugger, and package manager were open-sourced under the Apache 2.0 license with a Runtime Library Exception,[38] and Swift.org was created to host the project. The source code is hosted on GitHub, where it is easy for anyone to get the code, build it themselves, and even create pull requests to contribute code back to the project. [38] [ ] Swift.org GitHub In December 2015, IBM announced its Swift Sandbox website, which allows developers to write Swift code in one pane and display output in another.[39][40][41] The Swift Sandbox was deprecated in January 2018.[42] IBM [39] [ ] [40] [ ] [41] [ ] [42] [ ] During the WWDC 2016, Apple announced an iPad exclusive app, named Swift Playgrounds, intended to teach people how to code in Swift. The app is presented in a 3D video game-like interface which provides feedback when lines of code are placed in a certain order and executed.[43][44][45] WWDC 2016 iPad app Swift Playgrounds 3D video game-like [43] [ ] [44] [ ] [45] [ ] In January 2017, Chris Lattner announced his departure from Apple for a new position with Tesla Motors, with the Swift project lead role going to team veteran Ted Kremenek.[46][47] Tesla Motors [46] [ ] [47] [ ] During WWDC 2019, Apple announced SwiftUI with Xcode 11, which provides a framework for declarative UI structure design across all Apple platforms.[48] SwiftUI declarative [48] [ ] Official downloads of the SDK and toolchain for the Ubuntu distribution of Linux have been available since Swift 2.2, with more distros added since Swift 5.2.4, CentOS and Amazon Linux.[49] There is an unofficial SDK and native toolchain package for Android too.[50][51] Ubuntu CentOS [49] [ ] [50] [ ] [51] [ ] Platforms [edit] [ edit edit ] The platforms Swift supports are Apple's operating systems (Darwin, iOS, iPadOS, macOS, tvOS, watchOS), Linux, Windows, and Android.[52][53] Darwin iOS iPadOS macOS tvOS watchOS Linux Windows Android [52] [ ] [53] [ ] A key aspect of Swift's design is its ability to interoperate with the huge body of existing Objective-C code developed for Apple products over the previous decades, such as Cocoa and the Cocoa Touch frameworks. On Apple platforms,[54] it links with the Objective-C runtime library, which allows C, Objective-C, C++ and Swift code to run within one program.[55] Cocoa Cocoa Touch frameworks [54] [ ] runtime library C Objective-C C++ [55] [ ] Version history [edit] [ edit edit ] macOS Linux Windows [56] [ ] [57] [ ] [58] [ ] [59] [ ] [60] [ ] [61] [ ] [62] [ ] [63] [ ] [64] [ ] [65] [ ] [66] [ ] [67] [ ] [68] [ ] [69] [ ] [24] [ ] [70] [ ] [71] [ ] Features [edit] [ edit edit ] Please help improve it to make it understandable to non-experts, without removing the technical details. help improve it make it understandable to non-experts (June 2018) June 2018 (Learn how and when to remove this message) Learn how and when to remove this message Swift is a general purpose programming language that employs modern programming-language theory concepts and strives to present a simple, yet powerful syntax. Swift incorporates innovations and conventions from various programming languages, with notable inspiration from Objective-C, which it replaced as the primary development language on Apple Platforms. Swift was designed to be safe and friendly to new programmers while not sacrificing speed. By default Swift manages all memory automatically and ensures variables are always initialized before use. Array accesses are checked for out-of-bounds errors and integer operations are checked for overflow. Parameter names allow creating clear APIs. Protocols define interfaces that types may adopt, while extensions allow developers to add more function to existing types. Swift enables object-oriented programming with the support for classes, subtyping, and method overriding. Optionals allow nil values to be handled explicitly and safely. Concurrent programs can be written using async/await syntax and actors isolate shared mutable state in order to eliminate data races.[72][73] object-oriented programming classes subtyping method overriding nil async/await actors [72] [ ] [73] [ ] Basic syntax [edit] [ edit edit ] Swift's syntax is similar to C-style languages. Code begins executing in the global scope by default.[74] Alternatively, the @main attribute can be applied a structure, class, or enumeration declaration to indicate that it contains the program's entry point.[75] syntax [74] [ ] @ main [75] [ ] Swift's \"Hello, World!\" program is: \"Hello, World!\" program print ( \"Hello, world!\" ) The print(_:separator:terminator:) function used here is included in Swift's standard library, which is available to all programs without the need to import external modules. Statements in Swift don't have to end with a semicolon, however semicolons are required to separate multiple statements written on the same line. Single-line comments begin with // and continue until the end of the current line. Multiline comments are contained by /* and */ characters. Constants are declared with the let keyword and variables with the var keyword. Values must be initialized before they are read. Values may infer their type based on the type of the provided initial value. If the initial value is set after the value's declaration, a type must be declared explicitly.[74] print ( _ : separator : terminator :) comments // /* */ let var [74] [ ] let highScoreThreshold = 1000 // A constant with type Int. The type was inferred based on the provided value. var currentScore = 980 // A variable with type Int. currentScore = 1200 // The value of variables can change over time. let playerMessage : String // A constant with explicit type String. if currentScore > highScoreThreshold { playerMessage = \"You are a top player!\" } else { playerMessage = \"Better luck next time.\" } print ( playerMessage ) // Prints \"You are a top player!\" Control flow in Swift is managed with if-else, guard, and switch statements, along with while and for-in loops. The if statements take a Boolean parameter and execute the body of the if statement if the condition is true, otherwise it executes the optional else body. if-let syntax provides syntactic sugar for checking for the existence of an optional value and unwrapping it at the same time. if-else guard switch while for-in loops if if else if - let let someNumber = 42 if someNumber % 2 == 0 { // Use the remainder operator to find the remainder of someNumber divided by 2. print ( \" \\( someNumber ) is even.\" ) } else { print ( \" \\( someNumber ) is odd.\" ) } // Prints \"42 is even.\" Functions are defined with the func keyword. Function parameters may have names which allow function calls to read like phrases. An underscore before the parameter name allows the argument label to be omitted from the call site. Tuples can be used by functions to return multiple pieces of data at once. func Tuples func constructGreeting ( for name : String ) -> String { return \"Hello \\( name ) !\" } let greeting = constructGreeting ( for : \"Craig\" ) print ( greeting ) // Prints \"Hello Craig!\" Functions, and anonymous functions known as closures, can be assigned to properties and passed around the program like any other value. closures func divideByTwo ( _ aNum : Int ) -> Int { return aNum / 2 } func multiplyByTwo ( _ aNum : Int ) -> Int { return aNum * 2 } let mathOperation = multiplyByTwo print ( mathOperation ( 21 )) // Prints \"42\" guard statements require that the given condition is true before continuing on past the guard statement, otherwise the body of the provided else clause is run. The else clause must exit control of the code block in which the guard statement appears. guard statements are useful for ensuring that certain requirements are met before continuing on with program execution. In particular they can be used to create an unwrapped version of an optional value that is guaranteed to be non-nil for the remainder of the enclosing scope. guard guard else else guard guard func divide ( numerator : Int ?, byDenominator denominator : Int ) -> Int ? { guard denominator != 0 else { print ( \"Can't divide by 0.\" ) return nil } guard let numerator else { print ( \"The provided numerator is nil.\" ) return nil } return numerator / denominator } let result = divide ( numerator : 3 , byDenominator : 0 ) print ( \"Division result is: \\( result ) \" ) // Prints: // \"Can't divide by 0.\" // \"Division result is: nil.\" switch statements compare a value with multiple potential values and then executes an associated code block. switch statements must be made exhaustive, either by including cases for all possible values or by including a default case which is run when the provided value doesn't match any of the other cases. switch cases do not implicitly fall through, although they may explicitly do so with the fallthrough keyword. Pattern matching can be used in various ways inside switch statements. Here is an example of an integer being matched against a number of potential ranges: switch Pattern matching let someNumber = 42 switch someNumber { case ..< 0 : print ( \" \\( someNumber ) negative.\" ) case 0 : print ( \" \\( someNumber ) is 0.\" ) case 1. .. 9 : print ( \" \\( someNumber ) greater than 0, but less than 10.\" ) default : print ( \" \\( someNumber ) is greater than 9.\" ) } // Prints \"42 is greater than 9.\" for-in loops iterate over a sequence of values: let names = [ \"Will\" , \"Anna\" , \"Bart\" ] for name in names { print ( name ) } // Prints: // Will // Anna // Bart while loops iterate as long as the given Boolean condition evaluates to true: while // Add together all the numbers from 1 to 5. var i = 1 var result = 0 while i <= 5 { // The loop performs its body as long as i is less than or equal to 5. result += i // Add i to the current result. i += 1 // Increment i by 1. } print ( result ) // Prints \"15\" Closure support [edit] [ edit edit ] Swift supports closures, which are self-contained blocks of functionality that can be passed around and used in code,[76] and can also be used as anonymous functions. Here are some examples: closures [76] [ ] anonymous functions // Closure type, defined by its input and output values, can be specified outside the closure: let closure1 : ( Int , Int ) -> Int = { arg1 , arg2 in return arg1 + arg2 } // \u2026or inside it: let closure2 = { ( arg1 : Int , arg2 : Int ) -> Int in return arg1 + arg2 } // In most cases, closure's return type can be inferred automatically by the compiler. let closure3 = { arg1 : Int , arg2 : Int in return arg1 + arg2 } Closures can be assigned to variables and constants, and can be passed into other functions or closures as parameters. Single-expression closures may drop the return keyword. return Swift also has a trailing closure syntax, which allows the closure to be written after the end of the function call instead of within the function's parameter list. Parentheses can be omitted altogether if the closure is the function's only parameter: // This function takes a closure which receives no input parameters and returns an integer, // evaluates it, and uses the closure's return value (an Int) as the function's return value. func foo ( closure bar : () -> Int ) -> Int { return bar () } // Without trailing closure syntax: foo ( closure : { return 1 }) // With trailing closure syntax, and implicit return: foo { 1 } Starting from version 5.3, Swift supports multiple trailing closures:[77] [77] [ ] // This function passes the return of the first closure as the parameter of the second, // and returns the second closure's result: func foo ( bar : () -> Int , baz : ( Int ) -> Int ) -> Int { return baz ( bar ()) } // With no trailing closures: foo ( bar : { return 1 }, baz : { x in return x + 1 }) // With 1 trailing closure: foo ( bar : { return 1 }) { x in return x + 1 } // With 2 trailing closures (only the first closure's argument name is omitted): foo { return 1 } baz : { x in return x + 1 } Swift will provide shorthand argument names for inline closures, removing the need to explicitly name all of the closures parameters.[78] Arguments can be referred to with the names $0, $1, $2, and so on: [78] [ ] let names = [ \"Josephine\" , \"Steve\" , \"Chris\" , \"Barbara\" ] // filter calls the given closure for each value in names. // Values with a character count less than 6 are kept, the others are dropped. let shortNames = names . filter { $0 . count < 6 } print ( shortNames ) // Prints \"[\"Steve\", \"Chris\"]\" Closures may capture values from their surrounding scope. The closure will refer to this captured value for as long as the closure exists: func makeMultiplier ( withMultiple multiple : Int ) -> ( Int ) -> ( Int ) { // Create and return a closure that takes in an Int and returns the input multiplied by the value of multiple. return { $0 * multiple } } let multiplier = makeMultiplier ( withMultiple : 3 ) print ( multiplier ( 3 )) // Prints \"9\" print ( multiplier ( 10 )) // Prints \"30\" String support [edit] [ edit edit ] The Swift standard library includes unicode-compliant String and Character types. String values can be initialized with a String literal, a sequence of characters surrounded by double quotation marks. Strings can be concatenated with the + operator: String Character + var someString = \"Hello,\" someString += \" world!\" String interpolation allows for the creation of a new string from other values and expressions. Values written between parentheses preceded by a \\ will be inserted into the enclosing string literal:[79] \\ [79] [ ] var currentScore = 980 print ( \"Your score is \\( currentScore ) .\" ) // Prints \"Your score is 980.\" A for-in loop can be used to iterate over the characters contained in a string: for character in \"Swift\" { print ( character ) } // S // w // i // f // t When the Foundation framework is imported Swift invisibly bridges the String type to NSString, the String class commonly used in Objective-C. Callable objects [edit] [ edit edit ] Callable object \u00a7 In Swift [edit] [ edit ] In Swift, callable objects are defined using callAsFunction.[80] [80] [ ] struct CallableStruct { var value : Int func callAsFunction ( _ number : Int , scale : Int ) { print ( scale * ( number + value )) } } let callable = CallableStruct ( value : 100 ) callable ( 4 , scale : 2 ) callable . callAsFunction ( 4 , scale : 2 ) // Both function calls print 208. Access control [edit] [ edit edit ] Swift supports five access control levels for symbols: open, public, internal, fileprivate, and private. Unlike many object-oriented languages, these access controls ignore inheritance hierarchies: private indicates that a symbol is accessible only in the immediate scope, fileprivate indicates it is accessible only from within the file, internal indicates it is accessible within the containing module, public indicates it is accessible from any module, and open (only for classes and their methods) indicates that the class may be subclassed outside of the module.[81] access control open public internal fileprivate private inheritance private scope fileprivate internal public open [81] [ ] Optionals and chaining [edit] [ edit edit ] An important feature in Swift is option types, which allow references or values to operate in a manner similar to the common pattern in C, where a pointer may either refer to a specific value or no value at all. This implies that non-optional types cannot result in a null-pointer error; the compiler can ensure this is not possible. option types references C pointer null-pointer error Optional types are created with the Optional enum. To make an Integer that is nullable, one would use a declaration similar to var optionalInteger: Optional<Int>. As in C#,[82] Swift also includes syntactic sugar for this, allowing one to indicate a variable is optional by placing a question mark after the type name, var optionalInteger: Int?.[83] Variables or constants that are marked optional either have a value of the underlying type or are nil. Optional types wrap the base type, resulting in a different instance. String and String? are fundamentally different types, the former is of type String while the latter is an Optional that may be holding some String value. [82] [ ] [83] [ ] To access the value inside, assuming it is not nil, it must be unwrapped to expose the instance inside. This is performed with the ! operator: let myValue = anOptionalInstance !. someMethod () In this case, the ! operator unwraps anOptionalInstance to expose the instance inside, allowing the method call to be made on it. If anOptionalInstance is nil, a null-pointer error occurs, terminating the program. This is known as force unwrapping. Optionals may be safely unwrapped using optional chaining which first tests whether the instance is nil, and then unwrap it if it is non-null: optional chaining let myValue = anOptionalInstance ?. someMethod () In this case the runtime calls someMethod only if anOptionalInstance is not nil, suppressing the error. A ? must be placed after every optional property. If any of these properties are nil the entire expression evaluates as nil. The origin of the term chaining comes from the more common case where several method calls/getters are chained together. For instance: let aTenant = aBuilding . tenantList [ 5 ] let theirLease = aTenant . leaseDetails let leaseStart = theirLease ?. startDate can be reduced to: let leaseStart = aBuilding . tenantList [ 5 ]. leaseDetails ?. startDate Swift's use of optionals allows the compiler to use static dispatch because the unwrapping action is called on a defined instance (the wrapper), versus occurring in a runtime dispatch system. static dispatch Value types [edit] [ edit edit ] In many object-oriented languages, objects are represented internally in two parts. The object is stored as a block of data placed on the heap, while the name (or \"handle\") to that object is represented by a pointer. Objects are passed between methods by copying the value of the pointer, allowing the same underlying data on the heap to be accessed by anyone with a copy. In contrast, basic types like integers and floating-point values are represented directly; the handle contains the data, not a pointer to it, and that data is passed directly to methods by copying. These styles of access are termed pass-by-reference in the case of objects, and pass-by-value for basic types. heap pointer Both concepts have their advantages and disadvantages. Objects are useful when the data is large, like the description of a window or the contents of a document. In these cases, access to that data is provided by copying a 32- or 64-bit value, versus copying an entire data structure. However, smaller values like integers are the same size as pointers (typically both are one word), so there is no advantage to passing a pointer, versus passing the value. word Swift offers built-in support for objects using either pass-by-reference or pass-by-value semantics, the former using the class declaration and the latter using struct. Structs in Swift have almost all the same features as classes: methods, implementing protocols and using the extension mechanisms. For this reason, Apple terms all data generically as instances, versus objects or values. Structs do not support inheritance, however.[84] [84] [ ] The programmer is free to choose which semantics are more appropriate for each data structure in the application. Larger structures like windows would be defined as classes, allowing them to be passed around as pointers. Smaller structures, like a 2D point, can be defined as structs, which will be pass-by-value and allow direct access to their internal data with no indirection or reference counting. The performance improvement inherent to the pass-by-value concept is such that Swift uses these types for almost all common data types, including Int and Double, and types normally represented by objects, like String and Array.[84] Using value types can result in significant performance improvements in user applications as well.[85] [84] [ ] [85] [ ] Array, Dictionary, and Set all utilize copy on write so that their data are copied only if and when the program attempts to change a value in them. This means that the various accessors have what is in effect a pointer to the same data storage. So while the data is physically stored as one instance in memory, at the level of the application, these values are separate and physical separation is enforced by copy on write only if needed.[86] copy on write [86] [ ] Extensions [edit] [ edit edit ] Extensions add new functionality to an existing type, without the need to subclass or even have access to the original source code. Extensions can add new methods, initializers, computed properties, subscripts, and protocol conformances.[87] An example might be to add a spell checker to the base String type, which means all instances of String in the program gain the ability to spell-check. The system is also widely used as an organizational technique, allowing related code to be gathered into library-like extensions. [87] [ ] Extensions are declared with the extension keyword. struct Rectangle { let width : Double let height : Double } extension Rectangle { var area : Double { return height * width } } Protocol-oriented programming [edit] [ edit edit ] Protocols promise that a particular type implements a set of methods or properties, meaning that other instances in the system can call those methods on any instance implementing that protocol. This is often used in modern object-oriented languages as a substitute for multiple inheritance, although the feature sets are not entirely similar. multiple inheritance In Objective-C, and most other languages implementing the protocol concept, it is up to the programmer to ensure that the required methods are implemented in each class.[88] Swift adds the ability to add these methods using extensions, and to use generic programming (generics) to implement them. Combined, these allow protocols to be written once and support a wide variety of instances. Also, the extension mechanism can be used to add protocol conformance to an object that does not list that protocol in its definition.[89] [88] [ ] generic programming [89] [ ] For example, a protocol might be declared called Printable, which ensures that instances that conform to the protocol implement a description property and a printDetails() method requirement: // Define a protocol named Printable protocol Printable { var description : String { get } // A read-only property requirement func printDetails () // A method requirement } This protocol can now be adopted by other types: // Adopt the Printable protocol in a class class MyClass : Printable { var description : String { return \"An instance of MyClass\" } func printDetails () { print ( description ) } } Extensions can be used to add protocol conformance to types. Protocols themselves can also be extended to provide default implementations of their requirements. Adopters may define their own implementations, or they may use the default implementation: extension Printable { // All Printable instances will receive this implementation, or they may define their own. func printDetails () { print ( description ) } } // Bool now conforms to Printable, and inherits the printDetails() implementation above. extension Bool : Printable { var description : String { return \"An instance of Bool with value: \\( self ) \" } } In Swift, like many modern languages supporting interfaces, protocols can be used as types, which means variables and methods can be defined by protocol instead of their specific type: func getSomethingPrintable () -> any Printable { return true } var someSortOfPrintableInstance = getSomethingPrintable () print ( someSortOfPrintableInstance . description ) // Prints \"An instance of Bool with value: true\" It does not matter what concrete type of someSortOfPrintableInstance is, the compiler will ensure that it conforms to the protocol and thus this code is safe. This syntax also means that collections can be based on protocols also, like let printableArray = [any Printable]. Both extensions and protocols are used extensively in Swift's standard library; in Swift 5.9, approximately 1.2 percent of all symbols within the standard library were protocols, and another 12.3 percent were protocol requirements or default implementations.[90] For instance, Swift uses extensions to add the Equatable protocol to many of their basic types, like Strings and Arrays, allowing them to be compared with the == operator. The Equatable protocol also defines this default implementation: [90] [ ] func !=< T : Equatable >( lhs : T , rhs : T ) -> Bool This function defines a method that works on any instance conforming to Equatable, providing a not equals operator. Any instance, class or struct, automatically gains this implementation simply by conforming to Equatable.[91] [91] [ ] Protocols, extensions, and generics can be combined to create sophisticated APIs. For example, constraints allow types to conditionally adopt protocols or methods based on the characteristics of the adopting type. A common use case may be adding a method on collection types only when the elements contained within the collection are Equatable: extension Array where Element : Equatable { // allEqual will be available only on instances of Array that contain Equatable elements. func allEqual () -> Bool { for element in self { if element != self . first { return false } } return true } } Concurrency [edit] [ edit edit ] Swift 5.5 introduced structured concurrency into the language.[92] Structured concurrency uses Async/await syntax similar to Kotlin, JavaScript, and Rust. An async function is defined with the async keyword after the parameter list. When calling an async function the await keyword must be written before the function to indicate that execution will potentially suspend while calling function. While a function is suspended the program may run some other concurrent function in the same program. This syntax allows programs to clearly call out potential suspension points and avoid a version of the Pyramid of doom (programming) caused by the previously widespread use of closure callbacks.[93] [92] [ ] Async/await Pyramid of doom (programming) [93] [ ] func downloadText ( name : String ) async -> String { let result = // ... some asynchronous downloading code ... return result } let text = await downloadText ( \"text1\" ) The async let syntax allows multiple functions to run in parallel. await is again used to mark the point at which the program will suspend to wait for the completion of the async functions called earlier. // Each of these calls to downloadText will run in parallel. async let text1 = downloadText ( name : \"text1\" ) async let text2 = downloadText ( name : \"text2\" ) async let text3 = downloadText ( name : \"text3\" ) let textToPrint = await [ text1 , text2 , text3 ] // Suspends until all three downloadText calls have returned. print ( textToPrint ) Tasks and TaskGroups can be created explicitly to create a dynamic number of child tasks during runtime: let taskHandle = Task { await downloadText ( name : \"someText\" ) } let result = await taskHandle . value Swift uses the Actor model to isolate mutable state, allowing different tasks to mutate shared state in a safe manner. Actors are declared with the actor keyword and are reference types, like classes. Only one task may access the mutable state of an actor at the same time. Actors may access and mutate their own internal state freely, but code running in separate tasks must mark each access with the await keyword to indicate that the code may suspend until other tasks finish accessing the actor's state. Actor model actor Directory { var names : [ String ] = [] func add ( name : String ) { names . append ( name ) } } let directory = Directory () // Code suspends until other tasks finish accessing the actor. await directory . add ( name : \"Tucker\" ) print ( await directory . names ) Libraries, runtime, development [edit] [ edit edit ] On Apple systems, Swift uses the same runtime as the extant Objective-C system, but requires iOS 7 or macOS 10.9 or higher. It also depends on Grand Central Dispatch.[94] Swift and Objective-C code can be used in one program, and by extension, C and C++ also. Beginning in Swift 5.9, C++ code can be used directly from Swift code.[95] In the case of Objective-C, Swift has considerable access to the object model, and can be used to subclass, extend and use Objective-C code to provide protocol support.[96] The converse is not true: a Swift class cannot be subclassed in Objective-C.[97] Objective-C Grand Central Dispatch [94] [ ] C++ [95] [ ] [96] [ ] [97] [ ] To aid development of such programs, and the re-use of extant code, Xcode 6 and higher offers a semi-automated system that builds and maintains a bridging header to expose Objective-C code to Swift. This takes the form of an additional header file that simply defines or imports all of the Objective-C symbols that are needed by the project's Swift code. At that point, Swift can refer to the types, functions, and variables declared in those imports as though they were written in Swift. Objective-C code can also use Swift code directly, by importing an automatically maintained header file with Objective-C declarations of the project's Swift symbols. For instance, an Objective-C file in a mixed project called \"MyApp\" could access Swift classes or functions with the code #import \"MyApp-Swift.h\". Not all symbols are available through this mechanism, however\u2014use of Swift-specific features like generic types, non-object optional types, sophisticated enums, or even Unicode identifiers may render a symbol inaccessible from Objective-C.[98] header file [98] [ ] Swift also has limited support for attributes, metadata that is read by the development environment, and is not necessarily part of the compiled code. Like Objective-C, attributes use the @ syntax, but the currently available set is small. One example is the @IBOutlet attribute, which marks a given value in the code as an outlet, available for use within Interface Builder (IB). An outlet is a device that binds the value of the on-screen display to an object in code. Interface Builder On non-Apple systems, Swift does not depend on an Objective-C runtime or other Apple system libraries; a set of Swift \"Corelib\" implementations replace them. These include a \"swift-corelibs-foundation\" to stand in for the Foundation Kit, a \"swift-corelibs-libdispatch\" to stand in for the Grand Central Dispatch, and an \"swift-corelibs-xctest\" to stand in for the XCTest APIs from Xcode.[99] Foundation Kit Xcode [99] [ ] As of 2019, with Xcode 11, Apple has also added a major new UI paradigm called SwiftUI. SwiftUI replaces the older Interface Builder paradigm with a new declarative development paradigm.[100] Interface Builder [100] [ ] Memory management [edit] [ edit edit ] Swift uses Automatic Reference Counting (ARC) to manage memory. Every instance of a class or closure maintains a reference count which keeps a running tally of the number of references the program is holding on to. When this count reaches 0 the instance is deallocated. This automatic deallocation removes the need for a garbage collector as instances are deallocated as soon as they are no longer needed. Automatic Reference Counting manage memory A strong reference cycle can occur if two instances each strongly reference each other (e.g. A references B, B references A). Since neither instances reference count can ever reach zero neither is ever deallocated, resulting in a memory leak. Swift provides the keywords weak and unowned to prevent strong reference cycles. These keywords allow an instance to be referenced without incrementing its reference count. weak references must be optional variables, since they can change and become nil.[101] Attempting to access an unowned value that has already been deallocated results in a runtime error. strong reference cycle memory leak [101] [ ] A closure within a class can also create a strong reference cycle by capturing self references. Self references to be treated as weak or unowned can be indicated using a capture list. class Person { let name : String weak var home : Home ? // Defined as a weak reference in order to break the reference cycle. weak references do not increment the reference count of the instance that they refer to. init ( name : String ) { self . name = name } deinit { print ( \"De-initialized \\( name ) \" ) } } class Home { let address : String var owner : Person ? init ( address : String , owner : Person ?) { self . address = address self . owner = owner } deinit { print ( \"De-initialized \\( address ) \" ) } } var stacy : Person ? = Person ( name : \"Stacy\" ) var house21b : Home ? = Home ( address : \"21b Baker Street\" , owner : stacy ) stacy ?. home = house21b // stacy and house42b now refer to each other. stacy = nil // The reference count for stacy is now 1, because house21b is still holding a reference to it. house21b = nil // house21b's reference count drops to 0, which in turn drops stacy's count to 0 because house21b was the last instance holding a strong reference to stacy. // Prints: // De-initialized 21b Baker Street // De-initialized Stacy Debugging [edit] [ edit edit ] A key element of the Swift system is its ability to be cleanly debugged and run within the development environment, using a read\u2013eval\u2013print loop (REPL), giving it interactive properties more in common with the scripting abilities of Python than traditional system programming languages. The REPL is further enhanced with playgrounds, interactive views running within the Xcode environment or Playgrounds app that respond to code or debugger changes on-the-fly.[102] Playgrounds allow programmers to add in Swift code along with markdown documentation. Programmers can step through code and add breakpoints using LLDB either in a console or an IDE like Xcode. read\u2013eval\u2013print loop system programming playgrounds Playgrounds [102] [ ] LLDB IDE Comparisons to other languages [edit] [ edit edit ] Swift is considered a C family programming language and is similar to C in various ways: C family programming language Most operators in C also appear in Swift, although some operators such as + have slightly different behavior. For example, in Swift, + traps on overflow, whereas &+ is used to denote the C-like behavior of wrapping on overflow. Curly braces are used to group statements. Curly braces Variables are assigned using an equals sign, but compared using two consecutive equals signs. A new identity operator, ===, is provided to check if two data elements refer to the same object. equals sign two consecutive equals signs object Control statements while, if, and switch are similar, but have extended functions, e.g., a switch that takes non-integer cases, while and if supporting pattern matching and conditionally unwrapping optionals, for uses the for i in 1...10 syntax. pattern matching for i in 1. .. 10 Square brackets are used with arrays, both to declare them and to get a value at a given index in one of them. Square brackets arrays It also has similarities to Objective-C: Basic numeric types: Int, UInt, Float, Double Class methods are inherited, like instance methods; self in class methods is the class the method was called on. Similar for...in enumeration syntax. Differences from Objective-C include: Statements need not end with semicolons (;), though these must be used to allow more than one statement on one line. No header files. Uses type inference. type inference Generic programming. Generic programming Functions are first-class objects. first-class Enumeration cases can have associated data (algebraic data types). algebraic data types Operators can be redefined for classes (operator overloading), and new operators can be defined. operator overloading Strings fully support Unicode. Most Unicode characters can be used in either identifiers or operators. Unicode No exception handling. Swift 2 introduces a different and incompatible error-handling model.[103] exception handling [103] [ ] Several features of earlier C-family languages that are easy to misuse have been removed: Pointers are not exposed by default. There is no need for the programmer to keep track of and mark names for referencing or dereferencing. Assignments return no value. This prevents the common error of writing i = 0 instead of i == 0 (which throws a compile-time error). No need to use break statements in switch blocks. Individual cases do not fall through to the next case unless the fallthrough statement is used. Variables and constants are always initialized and array bounds are always checked. Integer overflows, which result in undefined behavior for signed integers in C, are trapped as a run-time error in Swift. Programmers can choose to allow overflows by using the special arithmetical operators &+, &-, &*, &/ and &%. The properties min and max are defined in Swift for all integer types and can be used to safely check for potential overflows, versus relying on constants defined for each type in external libraries. The one-statement form of if and while, which allows for the omission of braces around the statement, is unsupported. C-style enumeration for (int i = 0; i < c; i++), which is prone to off-by-one errors, is unsupported (from Swift 3 onward).[104] The pre- and post- increment and decrement operators (i++, --i ...) are unsupported (from Swift 3 onward), more so since C-style for statements are also unsupported from Swift 3 onward.[105] C-family languages Pointers are not exposed by default. There is no need for the programmer to keep track of and mark names for referencing or dereferencing. Pointers Assignments return no value. This prevents the common error of writing i = 0 instead of i == 0 (which throws a compile-time error). No need to use break statements in switch blocks. Individual cases do not fall through to the next case unless the fallthrough statement is used. switch Variables and constants are always initialized and array bounds are always checked. Integer overflows, which result in undefined behavior for signed integers in C, are trapped as a run-time error in Swift. Programmers can choose to allow overflows by using the special arithmetical operators &+, &-, &*, &/ and &%. The properties min and max are defined in Swift for all integer types and can be used to safely check for potential overflows, versus relying on constants defined for each type in external libraries. Integer overflows undefined behavior The one-statement form of if and while, which allows for the omission of braces around the statement, is unsupported. C-style enumeration for (int i = 0; i < c; i++), which is prone to off-by-one errors, is unsupported (from Swift 3 onward).[104] off-by-one errors [104] [ ] The pre- and post- increment and decrement operators (i++, --i ...) are unsupported (from Swift 3 onward), more so since C-style for statements are also unsupported from Swift 3 onward.[105] increment and decrement operators [105] [ ] Development and other implementations [edit] [ edit edit ] Because Swift can run on Linux, it is sometimes also used as a server-side language.[106] Some web frameworks have already been developed, such as IBM's Kitura (now discontinued), Perfect and Vapor. [106] [ ] IBM Kitura Perfect Vapor An official \"Server APIs\" work group has also been started by Apple,[107] with members of the Swift developer community playing a central role.[108] [107] [ ] [108] [ ] A second free implementation of Swift that targets Cocoa, Microsoft's Common Language Infrastructure (.NET Framework, now .NET), and the Java and Android platform exists as part of the Elements Compiler from RemObjects Software.[109] Cocoa Microsoft Common Language Infrastructure .NET Framework .NET Java Android RemObjects Software [109] [ ] Subsets of Swift have been ported to additional platforms, such as Arduino[110] and Mac OS 9.[111] ported Arduino [110] [ ] Mac OS 9 [111] [ ] See also [edit] [ edit edit ] Computer programming portal Computer programming portal Computer programming portal Comparison of programming languages Comparison of programming languages Objective-C Objective-C D (programming language) D (programming language) Kotlin (programming language) Kotlin (programming language) Python (programming language) Python (programming language) Nim (programming language) Nim (programming language) References [edit] [ edit edit ] ^ U.S. patent no. 9329844 ^ ^ U.S. patent no. 9329844 ^ \"Swift Has Reached 1.0\". Apple. September 9, 2014. Retrieved March 8, 2015. ^ ^ \"Swift Has Reached 1.0\". Apple. September 9, 2014. Retrieved March 8, 2015. \"Swift Has Reached 1.0\" . Retrieved March 8, 2015 March 8, ^ \"Release 6.0.3\". December 13, 2024. Retrieved December 31, 2024. ^ ^ \"Release 6.0.3\". December 13, 2024. Retrieved December 31, 2024. \"Release 6.0.3\" . Retrieved December 31, 2024 December 31, ^ \"Swift, Objectively\". Swift is proprietary and closed: It is entirely controlled by Apple and there is no open source implementation. ^ ^ \"Swift, Objectively\". Swift is proprietary and closed: It is entirely controlled by Apple and there is no open source implementation. \"Swift, Objectively\" ^ Lattner, Chris (June 11, 2014). \"Re: [LLVMdev] [cfe-dev] [Advertisement] open positions in Apple's Swift compiler team\". Archived from the original on July 14, 2014. Retrieved June 12, 2014. You can imagine that many of us want it to be open source and part of LLVM, but the discussion hasn't happened yet, and won't for some time. ^ ^ Lattner, Chris (June 11, 2014). \"Re: [LLVMdev] [cfe-dev] [Advertisement] open positions in Apple's Swift compiler team\". Archived from the original on July 14, 2014. Retrieved June 12, 2014. You can imagine that many of us want it to be open source and part of LLVM, but the discussion hasn't happened yet, and won't for some time. Lattner, Chris \"Re: [LLVMdev] [cfe-dev] [Advertisement] open positions in Apple's Swift compiler team\" the original . Retrieved June 12, 2014 June 12, ^ \"Chris Lattner's Homepage\". Chris Lattner. June 3, 2014. Retrieved June 3, 2014. The Swift language is the product of tireless effort from a team of language experts, documentation gurus, compiler optimization ninjas, and an incredibly important internal dogfooding group who provided feedback to help refine and battle-test ideas. Of course, it also greatly benefited from the experiences hard-won by many other languages in the field, drawing ideas from Objective-C, Rust, Haskell, Ruby, Python, C#, CLU, and far too many others to list. ^ ^ \"Chris Lattner's Homepage\". Chris Lattner. June 3, 2014. Retrieved June 3, 2014. The Swift language is the product of tireless effort from a team of language experts, documentation gurus, compiler optimization ninjas, and an incredibly important internal dogfooding group who provided feedback to help refine and battle-test ideas. Of course, it also greatly benefited from the experiences hard-won by many other languages in the field, drawing ideas from Objective-C, Rust, Haskell, Ruby, Python, C#, CLU, and far too many others to list. \"Chris Lattner's Homepage\" . Retrieved June 3, 2014 June 3, ^ a b Lattner, Chris (June 3, 2014). \"Chris Lattner's Homepage\". Chris Lattner. Retrieved June 3, 2014. I started work on the Swift Programming Language in July of 2010. I implemented much of the basic language structure, with only a few people knowing of its existence. A few other (amazing) people started contributing in earnest late in 2011, and it became a major focus for the Apple Developer Tools group in July 2013 [...] drawing ideas from Objective-C, Rust, Haskell, Ruby, Python, C#, CLU, and far too many others to list. ^ a b a b Lattner, Chris (June 3, 2014). \"Chris Lattner's Homepage\". Chris Lattner. Retrieved June 3, 2014. I started work on the Swift Programming Language in July of 2010. I implemented much of the basic language structure, with only a few people knowing of its existence. A few other (amazing) people started contributing in earnest late in 2011, and it became a major focus for the Apple Developer Tools group in July 2013 [...] drawing ideas from Objective-C, Rust, Haskell, Ruby, Python, C#, CLU, and far too many others to list. Lattner, Chris \"Chris Lattner's Homepage\" . Retrieved June 3, 2014 June 3, Objective-C ^ \"Building assert() in Swift, Part 2: __FILE__ and __LINE__\". Retrieved September 25, 2014. ^ ^ \"Building assert() in Swift, Part 2: __FILE__ and __LINE__\". Retrieved September 25, 2014. \"Building assert() in Swift, Part 2: __FILE__ and __LINE__\" . Retrieved September 25, 2014 September 25, ^ \"Influences - The Rust Reference\". doc.rust-lang.org. Retrieved May 2, 2020. ^ ^ \"Influences - The Rust Reference\". doc.rust-lang.org. Retrieved May 2, 2020. \"Influences - The Rust Reference\" . Retrieved May 2, 2020 May 2, ^ \"influenced by - V documentation\". github.com. Retrieved November 3, 2023. ^ ^ \"influenced by - V documentation\". github.com. Retrieved November 3, 2023. \"influenced by - V documentation\" . Retrieved November 3, 2023 November 3, ^ Lardinois, Frederic (June 2, 2014). \"Apple Launches Swift, A New Programming Language For Writing iOS And OS X Apps\". TechCrunch. Retrieved September 7, 2022. ^ ^ Lardinois, Frederic (June 2, 2014). \"Apple Launches Swift, A New Programming Language For Writing iOS And OS X Apps\". TechCrunch. Retrieved September 7, 2022. \"Apple Launches Swift, A New Programming Language For Writing iOS And OS X Apps\" . Retrieved September 7, 2022 September 7, ^ Protocol-oriented Programming in Swift. Apple Inc. YouTube. ^ ^ Protocol-oriented Programming in Swift. Apple Inc. YouTube. Protocol-oriented Programming in Swift YouTube ^ \"Concepts are similar to Rust Traits\". ^ ^ \"Concepts are similar to Rust Traits\". \"Concepts are similar to Rust Traits\" ^ Williams, Owen (June 2, 2014). \"Tim Berners-Lee's sixtieth birthday Apple announces Swift, a new programming language for iOS\". The Next Web. Retrieved June 2, 2014. ^ ^ Williams, Owen (June 2, 2014). \"Tim Berners-Lee's sixtieth birthday Apple announces Swift, a new programming language for iOS\". The Next Web. Retrieved June 2, 2014. \"Tim Berners-Lee's sixtieth birthday Apple announces Swift, a new programming language for iOS\" . Retrieved June 2, 2014 June 2, ^ \"Apple's new programming language Swift is now open source\". The Verge. December 3, 2015. Retrieved December 5, 2015. ^ ^ \"Apple's new programming language Swift is now open source\". The Verge. December 3, 2015. Retrieved December 5, 2015. \"Apple's new programming language Swift is now open source\" . Retrieved December 5, 2015 December 5, ^ \"Apple Open Sources Swift in Latest Pitch to the Enterprise\". CIO Journal. The Wall Street Journal Blogs. December 3, 2015. Retrieved December 5, 2015. ^ ^ \"Apple Open Sources Swift in Latest Pitch to the Enterprise\". CIO Journal. The Wall Street Journal Blogs. December 3, 2015. Retrieved December 5, 2015. \"Apple Open Sources Swift in Latest Pitch to the Enterprise\" \"Apple Open Sources Swift in Latest Pitch to the Enterprise\" The Wall Street Journal . Retrieved December 5, 2015 December 5, ^ \"Looking back on Swift 3 and ahead to Swift 4\". Swift Forums. July 29, 2016. Retrieved November 19, 2018. ^ ^ \"Looking back on Swift 3 and ahead to Swift 4\". Swift Forums. July 29, 2016. Retrieved November 19, 2018. \"Looking back on Swift 3 and ahead to Swift 4\" . Retrieved November 19, 2018 November 19, ^ \"Swift-Evolution\". Swift Evolution. Retrieved November 19, 2018. ^ ^ \"Swift-Evolution\". Swift Evolution. Retrieved November 19, 2018. \"Swift-Evolution\" . Retrieved November 19, 2018 November 19, ^ \"The RedMonk Programming Language Rankings: January 2018 \u2013 tecosystems\". redmonk.com. March 7, 2018. Retrieved November 20, 2018. ^ ^ \"The RedMonk Programming Language Rankings: January 2018 \u2013 tecosystems\". redmonk.com. March 7, 2018. Retrieved November 20, 2018. \"The RedMonk Programming Language Rankings: January 2018 \u2013 tecosystems\" . Retrieved November 20, 2018 November 20, ^ Kremenek, Ted (March 25, 2019). \"Swift 5 Released!\". ^ ^ Kremenek, Ted (March 25, 2019). \"Swift 5 Released!\". \"Swift 5 Released!\" ^ Kremenek, Ted (September 20, 2019). \"Swift 5.1 Released!\". Archived from the original on February 26, 2022. Retrieved October 28, 2019. ^ ^ Kremenek, Ted (September 20, 2019). \"Swift 5.1 Released!\". Archived from the original on February 26, 2022. Retrieved October 28, 2019. \"Swift 5.1 Released!\" the original . Retrieved October 28, 2019 October 28, ^ Hudson, Paul (June 6, 2021). \"What's new in Swift 5.5?\". HackingWithSwift.com. Hacking with Swift. Retrieved June 8, 2021. ^ ^ Hudson, Paul (June 6, 2021). \"What's new in Swift 5.5?\". HackingWithSwift.com. Hacking with Swift. Retrieved June 8, 2021. \"What's new in Swift 5.5?\" . Retrieved June 8, 2021 June 8, ^ \"Swift 5.9 Released\". Swift.org. September 18, 2023. Retrieved October 9, 2023. ^ ^ \"Swift 5.9 Released\". Swift.org. September 18, 2023. Retrieved October 9, 2023. \"Swift 5.9 Released\" . Retrieved October 9, 2023 October 9, ^ a b Borla, Holly (March 5, 2024). \"Swift 5.10 Released\". Swift.org. Retrieved March 13, 2024. ^ a b a b Borla, Holly (March 5, 2024). \"Swift 5.10 Released\". Swift.org. Retrieved March 13, 2024. \"Swift 5.10 Released\" . Retrieved March 13, 2024 March 13, ^ \"Swift.org\". Swift.org. March 2014. Retrieved April 28, 2024. ^ ^ \"Swift.org\". Swift.org. March 2014. Retrieved April 28, 2024. \"Swift.org\" . Retrieved April 28, 2024 April 28, ^ Inc, Apple (September 17, 2024). \"Announcing Swift 6\". Swift.org. Retrieved October 15, 2024. {{cite web}}: |last= has generic name (help) ^ ^ Inc, Apple (September 17, 2024). \"Announcing Swift 6\". Swift.org. Retrieved October 15, 2024. {{cite web}}: |last= has generic name (help) \"Announcing Swift 6\" . Retrieved October 15, 2024 October 15, {{cite web}}: cite web |last= has generic name (help) help ^ a b Platforms State of the Union, Session 102, Apple Worldwide Developers Conference, June 2, 2014 ^ a b a b Platforms State of the Union, Session 102, Apple Worldwide Developers Conference, June 2, 2014 Apple Worldwide Developers Conference ^ The Swift Programming Language. Apple. June 2, 2014. Retrieved June 2, 2014. \"Documentation\". Swift. ^ ^ The Swift Programming Language. Apple. June 2, 2014. Retrieved June 2, 2014. \"Documentation\". Swift. The Swift Programming Language Apple . Retrieved June 2, 2014 June 2, \"Documentation\". Swift. \"Documentation\" ^ \"Swift Has Reached 1.0\". September 9, 2014. Retrieved September 10, 2014. ^ ^ \"Swift Has Reached 1.0\". September 9, 2014. Retrieved September 10, 2014. \"Swift Has Reached 1.0\" . Retrieved September 10, 2014 September 10, ^ \"Xcode 6.1 Release Notes\". October 22, 2014. Retrieved January 23, 2015. ^ ^ \"Xcode 6.1 Release Notes\". October 22, 2014. Retrieved January 23, 2015. \"Xcode 6.1 Release Notes\" . Retrieved January 23, 2015 January 23, ^ \"Xcode 6.3 Release Notes\". April 8, 2015. Retrieved April 8, 2015. ^ ^ \"Xcode 6.3 Release Notes\". April 8, 2015. Retrieved April 8, 2015. \"Xcode 6.3 Release Notes\" . Retrieved April 8, 2015 April 8, ^ \"Swift 2 Apps in the App Store\". Swift Blog. Retrieved March 13, 2016. ^ ^ \"Swift 2 Apps in the App Store\". Swift Blog. Retrieved March 13, 2016. \"Swift 2 Apps in the App Store\" . Retrieved March 13, 2016 March 13, ^ \"Swift 3.0 Released!\". Swift.org. September 13, 2016. Archived from the original on October 14, 2016. Retrieved October 26, 2016. ^ ^ \"Swift 3.0 Released!\". Swift.org. September 13, 2016. Archived from the original on October 14, 2016. Retrieved October 26, 2016. \"Swift 3.0 Released!\" the original . Retrieved October 26, 2016 October 26, ^ \"Swift 4.0 Released!\". Swift.org. September 17, 2017. Archived from the original on March 28, 2019. Retrieved March 1, 2018. ^ ^ \"Swift 4.0 Released!\". Swift.org. September 17, 2017. Archived from the original on March 28, 2019. Retrieved March 1, 2018. \"Swift 4.0 Released!\" the original . Retrieved March 1, 2018 March 1, ^ \"Swift 4.1 Released!\". Swift.org. March 29, 2018. Archived from the original on April 25, 2019. Retrieved March 30, 2018. ^ ^ \"Swift 4.1 Released!\". Swift.org. March 29, 2018. Archived from the original on April 25, 2019. Retrieved March 30, 2018. \"Swift 4.1 Released!\" the original . Retrieved March 30, 2018 March 30, ^ \"Stack Overflow Developer Survey Results 2015\". ^ ^ \"Stack Overflow Developer Survey Results 2015\". \"Stack Overflow Developer Survey Results 2015\" ^ \"Stack Overflow Developer Survey Results 2016\". ^ ^ \"Stack Overflow Developer Survey Results 2016\". \"Stack Overflow Developer Survey Results 2016\" ^ \"Swift.org and Open Source\". Swift.org. Apple Inc. Retrieved February 25, 2019. ^ ^ \"Swift.org and Open Source\". Swift.org. Apple Inc. Retrieved February 25, 2019. \"Swift.org and Open Source\" . Retrieved February 25, 2019 February 25, ^ \"Introducing the IBM Swift Sandbox \u2014 Swift\". Swift. Retrieved December 5, 2015. ^ ^ \"Introducing the IBM Swift Sandbox \u2014 Swift\". Swift. Retrieved December 5, 2015. \"Introducing the IBM Swift Sandbox \u2014 Swift\" . Retrieved December 5, 2015 December 5, ^ Mayo, Benjamin (December 4, 2015). \"Write Swift code in a web browser with the IBM Swift Sandbox\". 9to5Mac. Retrieved December 5, 2015. ^ ^ Mayo, Benjamin (December 4, 2015). \"Write Swift code in a web browser with the IBM Swift Sandbox\". 9to5Mac. Retrieved December 5, 2015. \"Write Swift code in a web browser with the IBM Swift Sandbox\" . Retrieved December 5, 2015 December 5, ^ \"After Apple open sources it, IBM puts Swift programming in the cloud\". ZDNet. Retrieved December 5, 2015. ^ ^ \"After Apple open sources it, IBM puts Swift programming in the cloud\". ZDNet. Retrieved December 5, 2015. \"After Apple open sources it, IBM puts Swift programming in the cloud\" . Retrieved December 5, 2015 December 5, ^ \"Swift Package Catalog and Swift Sandbox Deprecation\". Retrieved November 9, 2018. ^ ^ \"Swift Package Catalog and Swift Sandbox Deprecation\". Retrieved November 9, 2018. \"Swift Package Catalog and Swift Sandbox Deprecation\" . Retrieved November 9, 2018 November 9, ^ \"Swift Playgrounds\". Apple Developer. Retrieved June 19, 2016. ^ ^ \"Swift Playgrounds\". Apple Developer. Retrieved June 19, 2016. \"Swift Playgrounds\" . Retrieved June 19, 2016 June 19, ^ \"Swift Playgrounds \u2014 Preview\". Apple. Retrieved June 19, 2016. ^ ^ \"Swift Playgrounds \u2014 Preview\". Apple. Retrieved June 19, 2016. \"Swift Playgrounds \u2014 Preview\" . Retrieved June 19, 2016 June 19, ^ Mayo, Benjamin (June 13, 2016). \"Apple announces Swift Playgrounds for iPad at WWDC, public release in fall\". 9to5Mac. Retrieved June 19, 2016. ^ ^ Mayo, Benjamin (June 13, 2016). \"Apple announces Swift Playgrounds for iPad at WWDC, public release in fall\". 9to5Mac. Retrieved June 19, 2016. \"Apple announces Swift Playgrounds for iPad at WWDC, public release in fall\" . Retrieved June 19, 2016 June 19, ^ Cunningham, Andrew (January 10, 2017). \"Longtime Apple programmer and Swift creator leaves Apple for Tesla\". Ars Technica. ^ ^ Cunningham, Andrew (January 10, 2017). \"Longtime Apple programmer and Swift creator leaves Apple for Tesla\". Ars Technica. \"Longtime Apple programmer and Swift creator leaves Apple for Tesla\" ^ Wuerthele, Mike (January 13, 2017). \"New Swift project head Ted Kremenek said to be running the show behind the scenes for some time\". AppleInsider. ^ ^ Wuerthele, Mike (January 13, 2017). \"New Swift project head Ted Kremenek said to be running the show behind the scenes for some time\". AppleInsider. \"New Swift project head Ted Kremenek said to be running the show behind the scenes for some time\" ^ Daniel Eran Dilger (June 19, 2019). \"WWDC19: SwiftUI was the brightest star in a galaxy of new ideas\". AppleInsider. Retrieved July 19, 2019. ^ ^ Daniel Eran Dilger (June 19, 2019). \"WWDC19: SwiftUI was the brightest star in a galaxy of new ideas\". AppleInsider. Retrieved July 19, 2019. \"WWDC19: SwiftUI was the brightest star in a galaxy of new ideas\" . Retrieved July 19, 2019 July 19, ^ \"Swift.org - Download Swift\". Retrieved June 21, 2020. ^ ^ \"Swift.org - Download Swift\". Retrieved June 21, 2020. \"Swift.org - Download Swift\" . Retrieved June 21, 2020 June 21, ^ \"Android SDKs for Swift\". GitHub. Retrieved September 10, 2021. ^ ^ \"Android SDKs for Swift\". GitHub. Retrieved September 10, 2021. \"Android SDKs for Swift\" GitHub . Retrieved September 10, 2021 September 10, ^ \"swift-lang package versions\". Retrieved September 10, 2021. ^ ^ \"swift-lang package versions\". Retrieved September 10, 2021. \"swift-lang package versions\" . Retrieved September 10, 2021 September 10, ^ Readdle (January 15, 2020). \"Swift for Android: Our Experience and Tools\". Medium. Retrieved August 20, 2020. ^ ^ Readdle (January 15, 2020). \"Swift for Android: Our Experience and Tools\". Medium. Retrieved August 20, 2020. \"Swift for Android: Our Experience and Tools\" . Retrieved August 20, 2020 August 20, ^ Anderson, Tim (March 30, 2020). \"Official tailored Swift for Windows support promised in 5.3: Swift on more platforms \u2013 provided you do not need a GUI\". The Register. Retrieved September 18, 2020. ^ ^ Anderson, Tim (March 30, 2020). \"Official tailored Swift for Windows support promised in 5.3: Swift on more platforms \u2013 provided you do not need a GUI\". The Register. Retrieved September 18, 2020. \"Official tailored Swift for Windows support promised in 5.3: Swift on more platforms \u2013 provided you do not need a GUI\" The Register . Retrieved September 18, 2020 September 18, ^ \"The Swift Linux Port\". Swift.org. Apple Inc. December 3, 2015. Retrieved August 3, 2016. ^ ^ \"The Swift Linux Port\". Swift.org. Apple Inc. December 3, 2015. Retrieved August 3, 2016. \"The Swift Linux Port\" . Retrieved August 3, 2016 August 3, ^ Timmer, John (June 5, 2014). \"A fast look at Swift, Apple's new programming language\". Ars Technica. Cond\u00e9 Nast. Retrieved June 6, 2014. ^ ^ Timmer, John (June 5, 2014). \"A fast look at Swift, Apple's new programming language\". Ars Technica. Cond\u00e9 Nast. Retrieved June 6, 2014. \"A fast look at Swift, Apple's new programming language\" Ars Technica Cond\u00e9 Nast . Retrieved June 6, 2014 June 6, ^ Kremenek, Ted (March 25, 2019). \"Swift 5 Released!\". Swift.org. Retrieved March 28, 2019. ^ ^ Kremenek, Ted (March 25, 2019). \"Swift 5 Released!\". Swift.org. Retrieved March 28, 2019. \"Swift 5 Released!\" . Retrieved March 28, 2019 March 28, ^ \"Download Swift\". Swift.org. Apple. Retrieved December 15, 2020. ^ ^ \"Download Swift\". Swift.org. Apple. Retrieved December 15, 2020. \"Download Swift\" . Retrieved December 15, 2020 December 15, ^ Kremenek, Ted (April 26, 2021). \"Swift 5.4 Released!\". Swift.org. Apple. Archived from the original on April 26, 2021. Retrieved April 26, 2021. ^ ^ Kremenek, Ted (April 26, 2021). \"Swift 5.4 Released!\". Swift.org. Apple. Archived from the original on April 26, 2021. Retrieved April 26, 2021. \"Swift 5.4 Released!\" the original . Retrieved April 26, 2021 April 26, ^ Kremenek, Ted (March 14, 2022). \"Swift 5.6 Released!\". Swift.org. Apple. Retrieved March 14, 2022. ^ ^ Kremenek, Ted (March 14, 2022). \"Swift 5.6 Released!\". Swift.org. Apple. Retrieved March 14, 2022. \"Swift 5.6 Released!\" . Retrieved March 14, 2022 March 14, ^ \"Release Swift 5.6.1 Release \u00b7 apple/Swift\". GitHub. ^ ^ \"Release Swift 5.6.1 Release \u00b7 apple/Swift\". GitHub. \"Release Swift 5.6.1 Release \u00b7 apple/Swift\" GitHub ^ \"Release Swift 5.6.2 Release \u00b7 apple/Swift\". GitHub. ^ ^ \"Release Swift 5.6.2 Release \u00b7 apple/Swift\". GitHub. \"Release Swift 5.6.2 Release \u00b7 apple/Swift\" GitHub ^ \"Release Swift 5.6.3 Release \u00b7 apple/Swift\". GitHub. ^ ^ \"Release Swift 5.6.3 Release \u00b7 apple/Swift\". GitHub. \"Release Swift 5.6.3 Release \u00b7 apple/Swift\" GitHub ^ Borla, Holly (September 12, 2022). \"Swift 5.7 Released!\". Swift.org. Apple. Retrieved September 23, 2022. ^ ^ Borla, Holly (September 12, 2022). \"Swift 5.7 Released!\". Swift.org. Apple. Retrieved September 23, 2022. \"Swift 5.7 Released!\" . Retrieved September 23, 2022 September 23, ^ \"Release Swift 5.7.1 Release \u00b7 apple/Swift\". GitHub. ^ ^ \"Release Swift 5.7.1 Release \u00b7 apple/Swift\". GitHub. \"Release Swift 5.7.1 Release \u00b7 apple/Swift\" GitHub ^ \"Release Swift 5.8 Release \u00b7 apple/Swift\". GitHub. ^ ^ \"Release Swift 5.8 Release \u00b7 apple/Swift\". GitHub. \"Release Swift 5.8 Release \u00b7 apple/Swift\" GitHub ^ \"Release Swift 5.8.1 Release \u00b7 apple/swift\". GitHub. Retrieved June 14, 2023. ^ ^ \"Release Swift 5.8.1 Release \u00b7 apple/swift\". GitHub. Retrieved June 14, 2023. \"Release Swift 5.8.1 Release \u00b7 apple/swift\" . Retrieved June 14, 2023 June 14, ^ \"Release Swift 5.9 Release \u00b7 apple/swift\". GitHub. Retrieved September 18, 2023. ^ ^ \"Release Swift 5.9 Release \u00b7 apple/swift\". GitHub. Retrieved September 18, 2023. \"Release Swift 5.9 Release \u00b7 apple/swift\" . Retrieved September 18, 2023 September 18, ^ \"Release Swift 5.9.1 Release \u00b7 apple/swift\". GitHub. Retrieved October 19, 2023. ^ ^ \"Release Swift 5.9.1 Release \u00b7 apple/swift\". GitHub. Retrieved October 19, 2023. \"Release Swift 5.9.1 Release \u00b7 apple/swift\" . Retrieved October 19, 2023 October 19, ^ \"Release Swift 5.9.2 Release \u00b7 apple/swift\". GitHub. Retrieved December 11, 2023. ^ ^ \"Release Swift 5.9.2 Release \u00b7 apple/swift\". GitHub. Retrieved December 11, 2023. \"Release Swift 5.9.2 Release \u00b7 apple/swift\" . Retrieved December 11, 2023 December 11, ^ \"Swift 5.10.1 Release\". GitHub. Retrieved June 5, 2024. ^ ^ \"Swift 5.10.1 Release\". GitHub. Retrieved June 5, 2024. \"Swift 5.10.1 Release\" . Retrieved June 5, 2024 June 5, ^ \"Swift 6.0 Release\". GitHub. Retrieved September 17, 2024. ^ ^ \"Swift 6.0 Release\". GitHub. Retrieved September 17, 2024. \"Swift 6.0 Release\" . Retrieved September 17, 2024 September 17, ^ \"Documentation\". docs.swift.org. Retrieved November 17, 2023. ^ ^ \"Documentation\". docs.swift.org. Retrieved November 17, 2023. \"Documentation\" . Retrieved November 17, 2023 November 17, ^ \"Eliminate data races using Swift Concurrency - WWDC22 - Videos\". Apple Inc. Retrieved November 17, 2023. ^ ^ \"Eliminate data races using Swift Concurrency - WWDC22 - Videos\". Apple Inc. Retrieved November 17, 2023. \"Eliminate data races using Swift Concurrency - WWDC22 - Videos\" . Retrieved November 17, 2023 November 17, ^ a b \"Documentation\". docs.swift.org. Retrieved October 15, 2023. ^ a b a b \"Documentation\". docs.swift.org. Retrieved October 15, 2023. \"Documentation\" . Retrieved October 15, 2023 October 15, ^ \"Documentation\". docs.swift.org. Retrieved October 15, 2023. ^ ^ \"Documentation\". docs.swift.org. Retrieved October 15, 2023. \"Documentation\" . Retrieved October 15, 2023 October 15, ^ \"Closures \u2014 The Swift Programming Language (Swift 5.5)\". docs.swift.org. Retrieved August 31, 2021. ^ ^ \"Closures \u2014 The Swift Programming Language (Swift 5.5)\". docs.swift.org. Retrieved August 31, 2021. \"Closures \u2014 The Swift Programming Language (Swift 5.5)\" . Retrieved August 31, 2021 August 31, ^ Macomber, Kyle; Yaskevich, Yavel; Gregor, Doug; McCall, John. \"Multiple Trailing Closures\". GitHub. Retrieved October 19, 2020. ^ ^ Macomber, Kyle; Yaskevich, Yavel; Gregor, Doug; McCall, John. \"Multiple Trailing Closures\". GitHub. Retrieved October 19, 2020. \"Multiple Trailing Closures\" . Retrieved October 19, 2020 October 19, ^ \"Documentation\". docs.swift.org. Retrieved October 16, 2023. ^ ^ \"Documentation\". docs.swift.org. Retrieved October 16, 2023. \"Documentation\" . Retrieved October 16, 2023 October 16, ^ \"Strings and Characters\". docs.swift.org. Retrieved October 16, 2023. ^ ^ \"Strings and Characters\". docs.swift.org. Retrieved October 16, 2023. \"Strings and Characters\" . Retrieved October 16, 2023 October 16, ^ \"Declarations \u2014 The Swift Programming Language (Swift 5.6)\". docs.swift.org. Retrieved February 28, 2022. ^ ^ \"Declarations \u2014 The Swift Programming Language (Swift 5.6)\". docs.swift.org. Retrieved February 28, 2022. \"Declarations \u2014 The Swift Programming Language (Swift 5.6)\" . Retrieved February 28, 2022 February 28, ^ \"Access Control\". The Swift Programming Language. Apple Developer. Retrieved October 25, 2016. ^ ^ \"Access Control\". The Swift Programming Language. Apple Developer. Retrieved October 25, 2016. \"Access Control\" Apple . Retrieved October 25, 2016 October 25, ^ \"Nullable Types\", C# Programming Guide, Microsoft. Archived February 21, 2017, at the Wayback Machine. ^ ^ \"Nullable Types\", C# Programming Guide, Microsoft. Archived February 21, 2017, at the Wayback Machine. \"Nullable Types\" Archived Wayback Machine ^ \"Types\". The Swift Programming Language. Apple Developer. Retrieved July 16, 2014. ^ ^ \"Types\". The Swift Programming Language. Apple Developer. Retrieved July 16, 2014. \"Types\" Apple . Retrieved July 16, 2014 July 16, ^ a b \"Classes and Structures\". The Swift Programming Language. Apple Developer. Archived from the original on March 25, 2016. ^ a b a b \"Classes and Structures\". The Swift Programming Language. Apple Developer. Archived from the original on March 25, 2016. \"Classes and Structures\" Archived ^ Guhit, Fiel (February 14, 2015). \"Performance Case Study on Swift 1.1, Swift 1.2, and Objective-C\". Medium. Archived from the original on December 12, 2023. ^ ^ Guhit, Fiel (February 14, 2015). \"Performance Case Study on Swift 1.1, Swift 1.2, and Objective-C\". Medium. Archived from the original on December 12, 2023. \"Performance Case Study on Swift 1.1, Swift 1.2, and Objective-C\" Archived ^ Building Better Apps with Value Types. Apple. Archived from the original on June 21, 2015. ^ ^ Building Better Apps with Value Types. Apple. Archived from the original on June 21, 2015. Building Better Apps with Value Types the original ^ \"Extensions\". docs.swift.org. Retrieved November 28, 2023. ^ ^ \"Extensions\". docs.swift.org. Retrieved November 28, 2023. \"Extensions\" . Retrieved November 28, 2023 November 28, ^ \"Working with Protocols\". Programming with Objective-C. Apple Developer Documentation Archive. September 17, 2014. Archived from the original on April 29, 2016. ^ ^ \"Working with Protocols\". Programming with Objective-C. Apple Developer Documentation Archive. September 17, 2014. Archived from the original on April 29, 2016. \"Working with Protocols\" Archived ^ \"NSCopying Protocol Reference\". Apple. ^ ^ \"NSCopying Protocol Reference\". Apple. \"NSCopying Protocol Reference\" ^ \"Swift standard library statistics\". swiftinit.org. Swiftinit. Retrieved October 2, 2023. ^ ^ \"Swift standard library statistics\". swiftinit.org. Swiftinit. Retrieved October 2, 2023. \"Swift standard library statistics\" . Retrieved October 2, 2023 October 2, ^ Thompson, Mattt (September 2, 2014). \"Swift Default Protocol Implementations\". NSHipster. ^ ^ Thompson, Mattt (September 2, 2014). \"Swift Default Protocol Implementations\". NSHipster. \"Swift Default Protocol Implementations\" ^ \"swift-evolution/proposals/0304-structured-concurrency.md at main \u00b7 apple/swift-evolution\". GitHub. Retrieved October 16, 2023. ^ ^ \"swift-evolution/proposals/0304-structured-concurrency.md at main \u00b7 apple/swift-evolution\". GitHub. Retrieved October 16, 2023. \"swift-evolution/proposals/0304-structured-concurrency.md at main \u00b7 apple/swift-evolution\" . Retrieved October 16, 2023 October 16, ^ \"swift-evolution/proposals/0296-async-await.md at main \u00b7 apple/swift-evolution\". GitHub. Retrieved October 16, 2023. ^ ^ \"swift-evolution/proposals/0296-async-await.md at main \u00b7 apple/swift-evolution\". GitHub. Retrieved October 16, 2023. \"swift-evolution/proposals/0296-async-await.md at main \u00b7 apple/swift-evolution\" . Retrieved October 16, 2023 October 16, ^ \"Do Swift-based apps work on macOS 10.9/iOS 7 and lower?\", StackOverflow ^ ^ \"Do Swift-based apps work on macOS 10.9/iOS 7 and lower?\", StackOverflow \"Do Swift-based apps work on macOS 10.9/iOS 7 and lower?\" ^ Inc, Apple (September 18, 2023). \"Swift 5.9 Released\". Swift.org. Retrieved October 9, 2023. {{cite web}}: |last= has generic name (help) ^ ^ Inc, Apple (September 18, 2023). \"Swift 5.9 Released\". Swift.org. Retrieved October 9, 2023. {{cite web}}: |last= has generic name (help) \"Swift 5.9 Released\" . Retrieved October 9, 2023 October 9, {{cite web}}: cite web |last= has generic name (help) help ^ \"Writing Swift Classes with Objective-C Behavior\", Apple Inc. ^ ^ \"Writing Swift Classes with Objective-C Behavior\", Apple Inc. \"Writing Swift Classes with Objective-C Behavior\" ^ \"Migrating Your Objective-C Code to Swift\". ^ ^ \"Migrating Your Objective-C Code to Swift\". \"Migrating Your Objective-C Code to Swift\" ^ \"Swift and Objective-C in the Same Project\", Apple Inc. ^ ^ \"Swift and Objective-C in the Same Project\", Apple Inc. \"Swift and Objective-C in the Same Project\" ^ \"Apple: search \"corelib\"\". GitHub. ^ ^ \"Apple: search \"corelib\"\". GitHub. \"Apple: search \"corelib\"\" ^ \"Xcode - SwiftUI- Apple Developer\". developer.apple.com. Retrieved February 1, 2021. ^ ^ \"Xcode - SwiftUI- Apple Developer\". developer.apple.com. Retrieved February 1, 2021. \"Xcode - SwiftUI- Apple Developer\" . Retrieved February 1, 2021 February 1, ^ Lanier, Brian; Groff, Joe. \"Intermediate Swift\". Apple. Retrieved July 3, 2014. ^ ^ Lanier, Brian; Groff, Joe. \"Intermediate Swift\". Apple. Retrieved July 3, 2014. \"Intermediate Swift\" . Retrieved July 3, 2014 July 3, ^ Metz, Cade. \"Why Coders Are Going Nuts Over Apple's New Programming Language\". Wired. Retrieved July 16, 2014. ^ ^ Metz, Cade. \"Why Coders Are Going Nuts Over Apple's New Programming Language\". Wired. Retrieved July 16, 2014. \"Why Coders Are Going Nuts Over Apple's New Programming Language\" Wired . Retrieved July 16, 2014 July 16, ^ \"Error-Handling in Swift-Language\". stackoverflow.com. ^ ^ \"Error-Handling in Swift-Language\". stackoverflow.com. \"Error-Handling in Swift-Language\" ^ \"apple/swift-evolution\". GitHub. Retrieved April 4, 2016. ^ ^ \"apple/swift-evolution\". GitHub. Retrieved April 4, 2016. \"apple/swift-evolution\" . Retrieved April 4, 2016 April 4, ^ \"apple/swift-evolution\". GitHub. Retrieved April 4, 2016. ^ ^ \"apple/swift-evolution\". GitHub. Retrieved April 4, 2016. \"apple/swift-evolution\" . Retrieved April 4, 2016 April 4, ^ Barbosa, Greg (February 22, 2016). \"IBM brings Swift to the cloud, releases web framework Kitura written in Apple's programming language\". 9to5Mac. Retrieved May 16, 2016. ^ ^ Barbosa, Greg (February 22, 2016). \"IBM brings Swift to the cloud, releases web framework Kitura written in Apple's programming language\". 9to5Mac. Retrieved May 16, 2016. \"IBM brings Swift to the cloud, releases web framework Kitura written in Apple's programming language\" . Retrieved May 16, 2016 May 16, ^ \"Server APIs Work Group\". Swift.org. October 25, 2016. Retrieved October 28, 2016. ^ ^ \"Server APIs Work Group\". Swift.org. October 25, 2016. Retrieved October 28, 2016. \"Server APIs Work Group\" . Retrieved October 28, 2016 October 28, ^ \"Swift.org\". Swift.org. Archived from the original on May 10, 2021. Retrieved October 28, 2016. ^ ^ \"Swift.org\". Swift.org. Archived from the original on May 10, 2021. Retrieved October 28, 2016. \"Swift.org\" the original . Retrieved October 28, 2016 October 28, ^ \"RemObjects Elements Compiler\". Retrieved January 17, 2016. ^ ^ \"RemObjects Elements Compiler\". Retrieved January 17, 2016. \"RemObjects Elements Compiler\" . Retrieved January 17, 2016 January 17, ^ \"Swift for Arduino\". ^ ^ \"Swift for Arduino\". \"Swift for Arduino\" ^ Rose, Jordan (April 1, 2020). \"Swift on Mac OS 9\". -dealloc. ^ ^ Rose, Jordan (April 1, 2020). \"Swift on Mac OS 9\". -dealloc. \"Swift on Mac OS 9\" External links [edit] [ edit edit ] Official website Official website Official website Official website Swift at Apple Developer Swift Apple Developer Swift source code on GitHub Swift source code GitHub Swift Example Swift Example Server-side Swift: The Vapor Framework Server-side Swift: The Vapor Framework v v t t e e Programming languages Comparison Comparison Timeline Timeline History History Ada Ada ALGOL Simula ALGOL Simula Simula APL APL Assembly Assembly BASIC Visual Basic classic .NET BASIC Visual Basic classic .NET Visual Basic classic classic .NET .NET C C C++ C++ C# C# COBOL COBOL Erlang Erlang Forth Forth Fortran Fortran Go Go Haskell Haskell Java Java JavaScript JavaScript Julia Julia Kotlin Kotlin Lisp Lisp Lua Lua MATLAB MATLAB ML ML Pascal Object Pascal Pascal Object Pascal Object Pascal Perl Perl PHP PHP Prolog Prolog Python Python R R Ruby Ruby Rust Rust SQL SQL Scratch Scratch Shell Shell Smalltalk Smalltalk Swift Swift more... more... Lists: Alphabetical Alphabetical Categorical Categorical Generational Generational Non-English-based Non-English-based Category Category v v t t e e Apple Operatingsystems Apple DOS Apple DOS Apple III (SOS) Apple III (SOS) Classic Mac OS Classic Mac OS Darwin Darwin iOS iOS iPadOS iPadOS GS/OS GS/OS Lisa OS Lisa OS macOS (OS X) macOS MacWorks XL MacWorks XL ProDOS ProDOS tvOS tvOS visionOS visionOS watchOS watchOS Bento Bento Clips Clips Final Cut Express Final Cut Express Front Row Front Row GarageBand GarageBand iCloud iCloud iDVD iDVD iLife iLife iMovie iMovie iTunes iTunes iWork Keynote Pages Numbers iWork Keynote Keynote Pages Pages Numbers Numbers iPhoto iPhoto Logic Express Logic Express Aperture Aperture Apple Media Tool Apple Media Tool Apple Qmaster Apple Qmaster Color Color DVD Studio Pro DVD Studio Pro FileMaker FileMaker Final Cut Pro Motion Compressor Final Cut Studio Cinema Tools Final Cut Pro Motion Motion Compressor Compressor Final Cut Studio Final Cut Studio Cinema Tools Cinema Tools Logic Studio Logic Studio Logic Pro Logic Pro MacProject MacProject MacTerminal MacTerminal MainStage MainStage Shake Shake Soundtrack Pro Soundtrack Pro Classroom Classroom AppleWorks AppleWorks Books Books Calendar Calendar Contacts Contacts Mail Mail Messages Messages FaceTime FaceTime MacDraw MacDraw MacPaint MacPaint MacWrite MacWrite Music Music Passwords Passwords Photo Booth Photo Booth Photos Photos Podcasts Podcasts Safari Safari TextEdit TextEdit Apple Remote Desktop Apple Remote Desktop FoundationDB FoundationDB macOS Server macOS Server WebObjects WebObjects Xgrid Xgrid Xsan Xsan Developer HyperCard HyperCard iBooks Author iBooks Author Instruments Instruments Interface Builder Interface Builder Quartz Composer Quartz Composer ResEdit ResEdit Swift Swift Xcode Xcode .Mac .Mac Dashcode Dashcode iAd Producer iAd Producer iChat iChat iWeb iWeb MobileMe MobileMe Windows AirPort Utility AirPort Utility AppleWorks AppleWorks Bonjour Bonjour Boot Camp Boot Camp iCloud iCloud iTunes iTunes MobileMe MobileMe QuickTime QuickTime Software Update Software Update List of Apple II application software List of Mac software List of old Macintosh software Authority control databases Germany Germany Germany United States United States United States France France France BnF data BnF data BnF data Israel Israel Israel https://en.wikipedia.org/w/index.php?title=Swift_(programming_language)&oldid=1260160620 Categories Swift (programming language) Swift (programming language) Apple Inc. software Apple Inc. software Programming languages Programming languages High-level programming languages High-level programming languages Declarative programming languages Declarative programming languages Object-oriented programming languages Object-oriented programming languages Functional languages Functional languages Pattern matching programming languages Pattern matching programming languages Programming languages created in 2014 Programming languages created in 2014 Statically typed programming languages Statically typed programming languages Systems programming languages Systems programming languages Cross-platform free software Cross-platform free software Software using the Apache license Software using the Apache license CS1 errors: generic name CS1 errors: generic name Webarchive template wayback links Webarchive template wayback links Articles with short description Articles with short description Short description is different from Wikidata Short description is different from Wikidata Use mdy dates from October 2018 Use mdy dates from October 2018 Wikipedia articles that are too technical from June 2018 Wikipedia articles that are too technical from June 2018 All articles that are too technical All articles that are too technical Articles with excerpts Articles with excerpts Articles with example Swift code Articles with example Swift code This page was last edited on 29 November 2024, at 06:06 (UTC). (UTC) Text is available under the Creative Commons Attribution-ShareAlike 4.0 License; additional terms may apply. By using this site, you agree to the Terms of Use and Privacy Policy. Wikipedia\u00ae is a registered trademark of the Wikimedia Foundation, Inc., a non-profit organization. Creative Commons Attribution-ShareAlike 4.0 License Terms of Use Privacy Policy Wikimedia Foundation, Inc. Privacy policy Privacy policy About Wikipedia About Wikipedia Disclaimers Disclaimers Contact Wikipedia Contact Wikipedia Code of Conduct Code of Conduct Developers Developers Statistics Statistics Cookie statement Cookie statement Mobile view Mobile view",
    "https://en.wikipedia.org/wiki/Kotlin": "Jump to content Main menu Main page Main page Main page Contents Contents Contents Current events Current events Current events Random article Random article Random article About Wikipedia About Wikipedia About Wikipedia Contact us Contact us Contact us Help Help Help Learn to edit Learn to edit Learn to edit Community portal Community portal Community portal Recent changes Recent changes Recent changes Upload file Upload file Upload file Search Search Appearance Donate Donate Donate Create account Create account Create account Log in Log in Log in Personal tools Donate Donate Donate Create account Create account Create account Log in Log in Log in learn more learn more Contributions Contributions Contributions Talk Talk Talk Kotlin Kotlin 15 languages \u0411\u0435\u043b\u0430\u0440\u0443\u0441\u043a\u0430\u044f \u0411\u0435\u043b\u0430\u0440\u0443\u0441\u043a\u0430\u044f \u0411\u0435\u043b\u0430\u0440\u0443\u0441\u043a\u0430\u044f Cebuano Cebuano Cebuano Deutsch Deutsch Deutsch Espa\u00f1ol Espa\u00f1ol Espa\u00f1ol Fran\u00e7ais Fran\u00e7ais Fran\u00e7ais \ud55c\uad6d\uc5b4 \ud55c\uad6d\uc5b4 \ud55c\uad6d\uc5b4 Hrvatski Hrvatski Hrvatski Italiano Italiano Italiano Nederlands Nederlands Nederlands \u65e5\u672c\u8a9e \u65e5\u672c\u8a9e \u65e5\u672c\u8a9e Polski Polski Polski \u0420\u0443\u0441\u0441\u043a\u0438\u0439 \u0420\u0443\u0441\u0441\u043a\u0438\u0439 \u0420\u0443\u0441\u0441\u043a\u0438\u0439 Sloven\u010dina Sloven\u010dina Sloven\u010dina Suomi Suomi Suomi Svenska Svenska Svenska Edit links Edit links Article Article Article Talk Talk Talk English Read Read Read Edit Edit Edit View history View history View history Tools Read Read Read Edit Edit Edit View history View history View history What links here What links here What links here Related changes Related changes Related changes Upload file Upload file Upload file Special pages Special pages Special pages Permanent link Permanent link Permanent link Page information Page information Page information Cite this page Cite this page Cite this page Get shortened URL Get shortened URL Get shortened URL Download QR code Download QR code Download QR code Download as PDF Download as PDF Download as PDF Printable version Printable version Printable version Wikidata item Wikidata item Wikidata item Kotlin Kotlin may refer to: Kotlin, Greater Poland Voivodeship, a village in west-central Poland Kotlin, Greater Poland Voivodeship Kotlin Island, a Russian island near the head of the Gulf of Finland Kotlin Island Kotlin (programming language), a general-purpose programming language Kotlin (programming language) Kotlin-class destroyer, a class of destroyers built for the Soviet Navy Kotlin-class destroyer Kotlin, a brand of fruit and vegetable products made by Agros Nova, a Polish company Agros Nova disambiguation internal link https://en.wikipedia.org/w/index.php?title=Kotlin&oldid=999577926 Categories Disambiguation pages Disambiguation pages Place name disambiguation pages Place name disambiguation pages Short description is different from Wikidata Short description is different from Wikidata All article disambiguation pages All article disambiguation pages All disambiguation pages All disambiguation pages This page was last edited on 10 January 2021, at 22:20 (UTC). (UTC) Text is available under the Creative Commons Attribution-ShareAlike 4.0 License; additional terms may apply. By using this site, you agree to the Terms of Use and Privacy Policy. Wikipedia\u00ae is a registered trademark of the Wikimedia Foundation, Inc., a non-profit organization. Creative Commons Attribution-ShareAlike 4.0 License Terms of Use Privacy Policy Wikimedia Foundation, Inc. Privacy policy Privacy policy About Wikipedia About Wikipedia Disclaimers Disclaimers Contact Wikipedia Contact Wikipedia Code of Conduct Code of Conduct Developers Developers Statistics Statistics Cookie statement Cookie statement Mobile view Mobile view"
}

## Βήμα 2

- **Κατέβασμα των απαιτούμενων πόρων του NLTK**: 
Χρησιμοποιούμε το NLTK για να κατεβάσουμε τα δεδομένα που απαιτούνται για την επεξεργασία του κειμένου, όπως οι stopwords και η λέξη-ρίζα λεξικογράφηση.
- **Φόρτωση stopwords**: 
Οι stopwords είναι κοινές λέξεις που συνήθως αφαιρούνται κατά την επεξεργασία φυσικής γλώσσας, όπως "the", "is", κλπ.
- **Αρχικοποίηση του Stemming και Lemmatization**: 
ο stemming απομακρύνει τα κατάληξη των λέξεων για να επιστρέψει τη ρίζα της λέξης, ενώ το lemmatization επιστρέφει την κανονική μορφή της λέξης.
- **Διαδικασία καθαρισμού**: 
Ο καθαρισμός του κειμένου περιλαμβάνει το tokenization, την αφαίρεση stopwords, την αφαίρεση της στίξης, και την εφαρμογή του stemming και του lemmatization.
- **Αποθήκευση των επεξεργασμένων δεδομένων**: 
Τα επεξεργασμένα δεδομένα αποθηκεύονται σε ένα νέο αρχείο JSON.

In [ ]:
import json
import nltk
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer, WordNetLemmatizer

# Κατεβάζουμε τους απαιτούμενους πόρους του NLTK
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

# Ορίζουμε τις διαδρομές των αρχείων εισόδου και εξόδου
input_json_path = 'programming_languages_text.json'
output_json_path = 'processed_programming_languages_text.json'

# Φορτώνουμε τις λέξεις κλειδιά (stopwords)
stop_words = set(stopwords.words('english'))

# Αρχικοποιούμε τον αλγόριθμο stemming και lemmatization
stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()

# Debug: Εμφανίζουμε τις stopwords που χρησιμοποιούνται
print("Stopwords used:", stop_words)

# Ορίζουμε μία συνάρτηση για τον καθαρισμό του κειμένου
def clean_text(text):
    # Κάνουμε tokenization του κειμένου σε λέξεις
    words = word_tokenize(text)
    
    # Αφαιρούμε τις stopwords και την στίξη
    cleaned_words = [
        word.lower() for word in words 
        if word.lower() not in stop_words and word not in string.punctuation
    ]
    
    # Εφαρμόζουμε το stemming
    stemmed_words = [stemmer.stem(word) for word in cleaned_words]
    
    # Εφαρμόζουμε το lemmatization
    lemmatized_words = [lemmatizer.lemmatize(word) for word in stemmed_words]
    
    # Ενώνουμε τις καθαρές, stemmed και lemmatized λέξεις πίσω σε μία συμβολοσειρά
    return ' '.join(lemmatized_words)

# Διαβάζουμε το αρχείο εισόδου JSON
with open(input_json_path, 'r', encoding='utf-8') as file:
    data = json.load(file)

# Επεξεργαζόμαστε τα δεδομένα κειμένου και αποθηκεύουμε την καθαρισμένη εκδοχή μόνο
processed_data = {}
for key, value in data.items():
    processed_value = clean_text(value)
    processed_data[key] = processed_value

# Αποθηκεύουμε τα επεξεργασμένα δεδομένα στο αρχείο εξόδου JSON
with open(output_json_path, 'w', encoding='utf-8') as file:
    json.dump(processed_data, file, ensure_ascii=False, indent=4)

print(f"Τα επεξεργασμένα δεδομένα αποθηκεύτηκαν στο {output_json_path}")


![step2](./project/images/step_2.png)

Αφού τρέξουμε τον κώδικα, δημιουργείται ένα νέο JSON αρχείο, στο οποίο βρίσκεται το επεξεργασμένο κείμενο, το οποίο είναι σε πιο χρήσιμη μορφή για τη συνέχεια.

In [ ]:
{
    "https://en.wikipedia.org/wiki/Python_(programming_language)": "jump content main menu main page main page main page content content content current event current event current event random articl random articl random articl wikipedia wikipedia wikipedia contact u contact u contact u help help help learn edit learn edit learn edit commun portal commun portal commun portal recent chang recent chang recent chang upload file upload file upload file search search appear donat donat donat creat account creat account creat account log log log person tool donat donat donat creat account creat account creat account log log log learn learn contribut contribut contribut talk talk talk content top top 1 histori 1 histori 1 histori 2 design philosophi featur 2 design philosophi featur 2 design philosophi featur 3 syntax semant toggl syntax semant subsect 3.1 indent 3.2 statement control flow 3.3 express 3.4 method 3.5 type 3.6 arithmet oper 3.7 function syntax 3 syntax semant 3 syntax semant toggl syntax semant subsect 3.1 indent 3.1 indent 3.1 indent 3.2 statement control flow 3.2 statement control flow 3.2 statement control flow 3.3 express 3.3 express 3.3 express 3.4 method 3.4 method 3.4 method 3.5 type 3.5 type 3.5 type 3.6 arithmet oper 3.6 arithmet oper 3.6 arithmet oper 3.7 function syntax 3.7 function syntax 3.7 function syntax 4 program exampl 4 program exampl 4 program exampl 5 librari 5 librari 5 librari 6 develop environ 6 develop environ 6 develop environ 7 implement toggl implement subsect 7.1 refer implement 7.2 implement 7.3 longer support implement 7.4 cross-compil languag 7.5 perform 7 implement 7 implement toggl implement subsect 7.1 refer implement 7.1 refer implement 7.1 refer implement 7.2 implement 7.2 implement 7.2 implement 7.3 longer support implement 7.3 longer support implement 7.3 longer support implement 7.4 cross-compil languag 7.4 cross-compil languag 7.4 cross-compil languag 7.5 perform 7.5 perform 7.5 perform 8 develop 8 develop 8 develop 9 api document gener 9 api document gener 9 api document gener 10 name 10 name 10 name 11 popular 11 popular 11 popular 12 use 12 use 12 use 13 languag influenc python 13 languag influenc python 13 languag influenc python 14 see also 14 see also 14 see also 15 refer toggl refer subsect 15.1 sourc 15 refer 15 refer toggl refer subsect 15.1 sourc 15.1 sourc 15.1 sourc 16 read 16 read 16 read 17 extern link 17 extern link 17 extern link toggl tabl content python program languag python program languag 112 languag afrikaan afrikaan afrikaan alemannisch alemannisch alemannisch العربية العربية العربية aragoné aragoné aragoné অসমীয়া অসমীয়া অসমীয়া asturianu asturianu asturianu azərbaycanca azərbaycanca azərbaycanca تۆرکجه تۆرکجه تۆرکجه basa bali basa bali basa bali বাংলা বাংলা বাংলা 閩南語 bân-lâm-gú 閩南語 bân-lâm-gú 閩南語 bân-lâm-gú беларуская беларуская беларуская беларуская тарашкевіца беларуская тарашкевіца беларуская тарашкевіца भोजपुरी भोजपुरी भोजपुरी български български български bosanski bosanski bosanski brezhoneg brezhoneg brezhoneg català català català cebuano cebuano cebuano čeština čeština čeština cymraeg cymraeg cymraeg dansk dansk dansk deutsch deutsch deutsch eesti eesti eesti ελληνικά ελληνικά ελληνικά español español español esperanto esperanto esperanto euskara euskara euskara فارسی فارسی فارسی françai françai françai galego galego galego ગુજરાતી ગુજરાતી ગુજરાતી 한국어 한국어 한국어 hausa hausa hausa հայերեն հայերեն հայերեն हिन्दी हिन्दी हिन्दी hrvatski hrvatski hrvatski ido ido ido bahasa indonesia bahasa indonesia bahasa indonesia interlingua interlingua interlingua íslenska íslenska íslenska italiano italiano italiano עברית עברית עברית ქართული ქართული ქართული қазақша қазақша қазақша kiswahili kiswahili kiswahili kurdî kurdî kurdî кыргызча кыргызча кыргызча ລາວ ລາວ ລາວ latina latina latina latviešu latviešu latviešu lietuvių lietuvių lietuvių la .lojban la .lojban la .lojban lombard lombard lombard magyar magyar magyar македонски македонски македонски മലയാളം മലയാളം മലയാളം मराठी मराठी मराठी bahasa melayu bahasa melayu bahasa melayu монгол монгол монгол မြန်မာဘာသာ မြန်မာဘာသာ မြန်မာဘာသာ na vosa vakav na vosa vakav na vosa vakav nederland nederland nederland नेपाली नेपाली नेपाली 日本語 日本語 日本語 ߒߞߏ ߒߞߏ ߒߞߏ norsk bokmål norsk bokmål norsk bokmål norsk nynorsk norsk nynorsk norsk nynorsk ଓଡ଼ିଆ ଓଡ଼ିଆ ଓଡ଼ିଆ oʻzbekcha ўзбекча oʻzbekcha ўзбекча oʻzbekcha ўзбекча ਪੰਜਾਬੀ ਪੰਜਾਬੀ ਪੰਜਾਬੀ پنجابی پنجابی پنجابی ភាសាខ្មែរ ភាសាខ្មែរ ភាសាខ្មែរ plattdüütsch plattdüütsch plattdüütsch polski polski polski portuguê portuguê portuguê qaraqalpaqsha qaraqalpaqsha qaraqalpaqsha română română română runa simi runa simi runa simi русский русский русский саха тыла саха тыла саха тыла ᱥᱟᱱᱛᱟᱲᱤ ᱥᱟᱱᱛᱟᱲᱤ ᱥᱟᱱᱛᱟᱲᱤ scot scot scot shqip shqip shqip සිංහල සිංහල සිංහල simpl english simpl english simpl english slovenčina slovenčina slovenčina slovenščina slovenščina slovenščina کوردی کوردی کوردی српски srpski српски srpski српски srpski srpskohrvatski српскохрватски srpskohrvatski српскохрватски srpskohrvatski српскохрватски suomi suomi suomi svenska svenska svenska tagalog tagalog tagalog தமிழ் தமிழ் தமிழ் татарча tatarça татарча tatarça татарча tatarça ၽႃႇသႃႇတႆး ၽႃႇသႃႇတႆး ၽႃႇသႃႇတႆး తెలుగు తెలుగు తెలుగు ไทย ไทย ไทย тоҷикӣ тоҷикӣ тоҷикӣ türkçe türkçe türkçe basa ugi basa ugi basa ugi українська українська українська اردو اردو اردو ئۇيغۇرچە uyghurch ئۇيغۇرچە uyghurch ئۇيغۇرچە uyghurch tiếng việt tiếng việt tiếng việt walon walon walon 文言 文言 文言 winaray winaray winaray 吴语 吴语 吴语 粵語 粵語 粵語 中文 中文 中文 edit link edit link articl articl articl talk talk talk english read read read edit edit edit view histori view histori view histori tool read read read edit edit edit view histori view histori view histori link link link relat chang relat chang relat chang upload file upload file upload file special page special page special page perman link perman link perman link page inform page inform page inform cite page cite page cite page get shorten url get shorten url get shorten url download qr code download qr code download qr code download pdf download pdf download pdf printabl version printabl version printabl version wikimedia common wikimedia common wikimedia common mediawiki mediawiki mediawiki wikibook wikibook wikibook wikifunct wikifunct wikifunct wikiquot wikiquot wikiquot wikivers wikivers wikivers wikidata item wikidata item wikidata item python genu paradigm multi-paradigm object-ori 1 procedur imper function structur reflect design guido van rossum develop python softwar foundat 33 year ago 1991-02-20 1991-02-20 2 stabl releas 40 day ago 3 decemb 2024 3 decemb 2024 type disciplin duck dynam strong 3 option type annot 4 o tier 1 64-bit linux maco 64- 32-bit window 10+ 5 linux maco window 5 tier 2 e.g 32-bit webassembl wasi webassembl tier 3 64-bit android 6 io freebsd 32-bit raspberri pi osunoffici known work unix-like/bsd variant platform 7 8 9 android 6 io freebsd raspberri pi o unix-lik bsd 7 8 9 licens python softwar foundat licens filenam extens 10 python.org python.org implement cpython pypi stackless python micropython circuitpython ironpython jython dialect cython rpython starlark 11 abc 12 ada 13 algol 68 14 apl 15 c 16 c++ 17 clu 18 dylan 19 haskel 20 15 icon 21 lisp 22 modula-3 modula-3 14 17 perl 23 standard ml 15 apach groovi boo cobra coffeescript 24 f gdscript go javascript 25 26 julia 27 mojo 28 nim ring 29 rubi 30 swift 31 v 32 python program wikibook python program python high-level general-purpos program languag design philosophi emphas code readabl use signific indent 33 high-level general-purpos program languag code readabl signific indent 33 python dynam type-check garbage-collect support multipl program paradigm includ structur particularli procedur object-ori function program often describ `` batteri includ '' languag due comprehens standard librari 34 35 dynam type-check garbage-collect program paradigm structur procedur object-ori function program standard librari 34 35 guido van rossum began work python late 1980 successor abc program languag first releas 1991 python 0.9.0 36 python 2.0 releas 2000 python 3.0 releas 2008 major revis complet backward-compat earlier version python 2.7.18 releas 2020 last releas python 2 37 guido van rossum abc 36 backward-compat 37 python consist rank one popular program languag gain widespread use machin learn commun 38 39 40 41 machin learn 38 39 40 41 histori edit edit edit histori python guido van rossum python conceiv late 1980 42 guido van rossum centrum wiskund informatica cwi netherland successor abc program languag inspir setl 43 capabl except handl interfac amoeba oper system 12 implement began decemb 1989 44 van rossum shoulder sole respons project lead develop 12 juli 2018 announc `` perman vacat '' respons python 's `` benevol dictat life '' bdfl titl python commun bestow upon reflect long-term commit project 's chief decision-mak 45 sinc come retir self-titl `` bdfl-emeritu '' januari 2019 activ python core develop elect five-memb steer council lead project 46 47 42 guido van rossum centrum wiskund informatica netherland abc setl 43 except handl amoeba 12 44 benevol dictat life 45 46 47 name python said come british comedi seri monti python 's fli circu 48 monti python 's fli circu 48 python 2.0 releas 16 octob 2000 mani major new featur list comprehens cycle-detect garbag collect refer count unicod support 49 python 2.7 's end-of-lif initi set 2015 postpon 2020 concern larg bodi exist code could easili forward-port python 3 50 51 secur patch improv releas 52 53 python 2.7 older version offici unsupport differ unoffici python implement pypi continu support python 2 i.e `` 2.7.18+ '' plu 3.10 plu mean least `` backport secur updat '' 54 list comprehens cycle-detect refer count unicod 49 end-of-lif 50 51 52 53 pypi backport 54 python 3.0 releas 3 decemb 2008 new semant chang syntax least everi python releas sinc unsupport 3.5 ad syntax languag later releas drop outdat modul chang semant least minor way sinc 7 octob 2024 updat python 3.13 latest stabl releas month 3.12 releas activ support includ bug fix oppos secur python 3.9 55 oldest support version python albeit 'secur support phase due python 3.8 reach end-of-lif 56 57 start 3.13 later version 2 year full support one half follow 3 year secur support total support updat 55 end-of-lif 56 57 secur updat expedit 2021 twice 2022 fix 2023 septemb 2024 python 3.12.6 3.8.20 sinc python version insecur includ 2.7 58 secur issu lead possibl remot code execut 59 web-cach poison 60 58 remot code execut 59 web-cach poison 60 python 3.10 ad union type oper 61 match case keyword structur pattern match statement 3.11 expand except handl function python 3.12 ad new keyword type notabl chang 3.11 3.10 includ increas program execut speed improv error report 62 python 3.11 claim 10 60 faster python 3.10 python 3.12 add anoth 5 top also improv error messag improv 3.14 mani chang 61 pattern match except handl 62 python 3.13 introduc syntax type new improv interact interpret repl featur multi-lin edit color support increment garbag collector produc shorter paus collect program lot object addit improv speed 3.11 3.12 experiment just-in-tim jit compil featur can/ne enabl specif increas speed 63 experiment free-thread build mode disabl global interpret lock gil allow thread run concurr latter featur enabl python3.13t python3.13t.ex repl just-in-tim jit compil 63 global interpret lock python 3.13 introduc chang behavior i.e new `` well-defin semant '' fix bug plu mani remov deprec class function method remov c api outdat modul `` old implement local frame.f_loc slow inconsist buggi mani corner case odditi code work around may need chang code use local simpl templat print debug continu work correctli `` 64 64 standard librari modul mani deprec class function method remov python 3.15 3.16 65 66 65 66 python 3.11 add sigstor digit verif signatur cpython artifact addit pgp sinc use pgp critic secur practition python move sigstor exclus drop pgp 3.14 67 pgp 67 python 3.14 alpha 3 regard possibl chang annot `` python 3.14 __future__ import annot continu work convert annot string `` 68 68 pep 711 propos pybi standard format distribut python binari 69 69 python 3.15 `` make utf-8 mode default '' 70 mode exist current python version current need opt utf-8 alreadi use default window elsewher thing e.g open file 's enabl also make code fulli cross-platform i.e use utf-8 everyth platform 70 utf-8 design philosophi featur edit edit edit python multi-paradigm program languag object-ori program structur program fulli support mani featur support function program aspect-ori program includ metaprogram 71 metaobject 72 mani paradigm support via extens includ design contract 73 74 logic program 75 python known glue languag 76 abl work well mani languag ea access multi-paradigm program languag object-ori program structur program aspect-ori program metaprogram 71 metaobject 72 design contract 73 74 logic program 75 glue languag 76 python use dynam type combin refer count cycle-detect garbag collector memori manag 77 use dynam name resolut late bind bind method variabl name program execut dynam type refer count memori manag 77 name resolut late bind design offer support function program lisp tradit filter mapandreduc function list comprehens dictionari set gener express 78 standard librari two modul itertool functool implement function tool borrow haskel standard ml 79 lisp list comprehens dictionari gener 78 haskel standard ml 79 core philosophi summar zen python pep 20 includ aphor 80 zen python aphor 80 beauti better ugli explicit better implicit simpl better complex complex better complic readabl count howev python featur regularli violat principl receiv critic ad unnecessari languag bloat 81 respons critic zen python guidelin rather rule 82 addit new featur controversi guido van rossum resign benevol dictat life follow vitriol addit assign express oper python 3.8 83 84 81 82 83 84 nevertheless rather build function core python design highli extens via modul compact modular made particularli popular mean ad programm interfac exist applic van rossum 's vision small core languag larg standard librari easili extens interpret stem frustrat abc espous opposit approach 42 extens abc 42 python claim strive simpler less-clutt syntax grammar give develop choic code methodolog contrast perl 's `` one way '' motto python embrac `` one—and prefer one—obvi way '' philosophi 80 practic howev python provid mani way achiev task exampl least three way format string liter certainti one programm use 85 alex martelli fellow python softwar foundat python book author wrote `` describ someth 'clever consid compliment python cultur `` 86 perl one way 80 85 alex martelli fellow python softwar foundat 86 python 's develop usual strive avoid prematur optim reject patch non-crit part cpython refer implement would offer margin increas speed cost clariti 87 execut speed improv move speed-crit function extens modul written languag c use just-in-tim compil like pypi also possibl cross-compil languag either n't provid full speed-up might expect sinc python dynam languag restrict subset python compil possibl semant slightli chang 88 prematur optim cpython 87 just-in-tim compil pypi cross-compil languag dynam languag 88 python 's develop aim fun use reflect name—a tribut british comedi group monti python 89 —and occasion play approach tutori refer materi use term `` spam '' `` egg '' refer monti python sketch exampl instead often-us `` foo '' `` bar '' 90 91 common neolog python commun python wide rang mean relat program style `` python '' code may use python idiom well natur show fluenci languag conform python 's minimalist philosophi emphasi readabl code difficult understand read like rough transcript anoth program languag call unpython 92 monti python 89 monti python sketch `` foo '' `` bar '' 90 91 neolog idiom 92 syntax semant edit edit edit python syntax semant c python meant easili readabl languag format visual unclutt often use english keyword languag use punctuat unlik mani languag use curli bracket delimit block semicolon statement allow rare use fewer syntact except special case c pascal 93 curli bracket c pascal 93 indent edit edit edit python syntax semant § indent python use whitespac indent rather curli bracket keyword delimit block increas indent come certain statement decreas indent signifi end current block 94 thu program 's visual structur accur repres semant structur 95 featur sometim term off-sid rule languag use indent way indent semant mean recommend indent size four space 96 whitespac curli bracket block 94 95 off-sid rule 96 statement control flow edit edit edit python 's statement includ statement assign statement use singl equal sign assign statement condit execut block code along el elif contract else-if statement iter iter object captur element local variabl use attach block statement execut block code long condit true tri statement allow except rais attach code block caught handl except claus new syntax except python 3.11 except group 97 also ensur clean-up code final block alway run regardless block exit tri 97 rais statement use rais specifi except re-rais caught except class statement execut block code attach local namespac class use object-ori program class def statement defin function method function method statement enclos code block within context manag exampl acquir lock run releas lock open close file allow resource-acquisition-is-initi raii -like behavior replac common try/fin idiom 98 lock file resource-acquisition-is-initi 98 break statement exit loop break continu statement skip rest current iter continu next del statement remov variable—delet refer name valu produc error variabl refer redefin pas statement serv nop syntact need creat empti code block nop assert statement use debug check condit appli assert yield statement return valu gener function also oper use implement coroutin gener coroutin return statement use return valu function import statement use import modul whose function variabl use current program import match case statement analog switch statement construct compar express one case control-of-flow measur switch statement assign statement bind name refer separ dynam alloc object variabl may subsequ rebound time object python variabl name gener refer holder without fix data type howev alway refer object type call dynam typing—in contrast statically-typ languag variabl may contain valu certain type refer object data type dynam type statically-typ python support tail call optim first-class continu accord van rossum never 99 100 howev better support coroutine-lik function provid extend python 's gener 101 2.5 gener lazi iter data pas unidirect gener python 2.5 possibl pas data back gener function version 3.3 pas multipl stack level 102 tail call first-class continu 99 100 coroutin gener 101 lazi iter 102 express edit edit edit python 's express includ express oper mathemat addit subtract multipl similar languag behavior divis differ two type divis python floor divis integ divis // floating-point/divis 103 python use oper exponenti floor divis 103 python use oper string concaten python use oper duplic string specifi number time infix oper intend use librari numpi matrix multipl 104 105 numpi matrix multipl 104 105 syntax call `` walru oper '' introduc python 3.8 assign valu variabl part larger express 106 106 python == compar valu python 's oper may use compar object ident comparison refer comparison may chained—for exampl b c. b c python use boolean oper python type express name list comprehens gener express name gener express 78 list comprehens gener 78 anonym function implement use lambda express howev may one express bodi anonym function lambda express condit express written x c el 107 differ order operand c x oper common mani languag x c el 107 c x python make distinct list tupl list written 1 2 3 mutabl use key dictionari dictionari key must immut python tupl written 1 2 3 immut thu use key dictionari provid tupl 's element immut oper use concaten two tupl directli modifi content produc new tupl contain element thu given variabl initi equal 1 2 3 execut 4 5 first evalu 4 5 yield 1 2 3 4 5 assign back t—therebi effect `` modifi content '' conform immut natur tupl object parenthes option tupl unambigu context 108 list tupl 1 2 3 immut 1 2 3 1 2 3 4 5 4 5 1 2 3 4 5 108 python featur sequenc unpack multipl express evalu anyth assign variabl writabl properti etc associ ident manner form tupl literals—and whole put left-hand side equal sign assign statement statement expect iter object right-hand side equal sign produc number valu provid writabl express iter assign produc valu correspond express left 109 109 python `` string format '' oper function analog printf format string c—e.g `` spam= eggs= '' `` blah '' 2 evalu `` spam=blah eggs=2 '' python 2.6+ 3+ supplement format method str class e.g `` spam= 0 eggs= 1 '' .format `` blah '' 2 python 3.6 ad `` f-string '' spam `` blah '' egg 2 f'spam= spam eggs= egg 110 printf `` spam= eggs= `` `` blah '' 2 `` spam= 0 eggs= 1 `` format `` blah '' 2 spam `` blah '' egg 2 f 'spam= spam eggs= egg 110 string python concaten `` ad '' oper ad integ float e.g `` spam '' `` egg '' return `` spamegg '' string contain number ad string rather integ e.g `` 2 '' `` 2 '' return `` 22 '' concaten `` spam '' `` egg '' `` 2 '' `` 2 '' python variou string liter delimit singl doubl quot unlik unix shell perl perl-influenc languag singl doubl quot work use backslash escap charact string interpol becam avail python 3.6 `` format string liter '' 110 triple-quot begin end three singl doubl quot may span multipl line function like document shell perl rubi raw string varieti denot prefix string liter r. escap sequenc interpret henc raw string use liter backslash common regular express windows-styl path compar `` -quot '' c string liter delimit singl doubl quot unlik unix shell perl perl-influenc languag singl doubl quot work use backslash escap charact string interpol becam avail python 3.6 `` format string liter '' 110 unix shell perl escap charact string interpol 110 triple-quot begin end three singl doubl quot may span multipl line function like document shell perl rubi document rubi raw string varieti denot prefix string liter r. escap sequenc interpret henc raw string use liter backslash common regular express windows-styl path compar `` -quot '' c raw string regular express window c python array index array slice express list denot key start stop start stop step index zero-bas neg index rel end slice take element start index includ stop index third slice paramet call step stride allow element skip revers slice index may omitted—for exampl return copi entir list element slice shallow copi array index array slice start stop start stop step zero-bas shallow copi python distinct express statement rigidli enforc contrast languag common lisp scheme rubi lead duplic function exampl common lisp scheme rubi list comprehens vs. for-loop list comprehens condit express vs. block condit eval vs. exec built-in function python 2 exec statement former express latter statement statement part expression—so list comprehens lambda express express contain statement particular case assign statement 1 form part condit express condit statement lambda express 1 method edit edit edit method object function attach object 's class syntax instance.method argument normal method function syntact sugar class.method instanc argument python method explicit self paramet access instanc data contrast implicit self object-ori program languag e.g. c++ java objective-c rubi 111 python also provid method often call dunder method due name begin end double-underscor allow user-defin class modifi handl nativ oper includ length comparison arithmet oper type convers 112 method function instanc method argument syntact sugar class method instanc argument self instanc data c++ java objective-c rubi 111 arithmet oper 112 type edit edit edit python use duck type type object untyp variabl name type constraint check compil time rather oper object may fail signifi suitabl type despit dynam type python strongli type forbid oper well-defin exampl ad number string rather silent attempt make sen duck type compil time dynam type strongli type python allow programm defin type use class often use object-ori program new instanc class construct call class exampl spamclass eggsclass class instanc metaclass type instanc allow metaprogram reflect class object-ori program instanc spamclass eggsclass metaclass reflect version 3.0 python two kind class use syntax old-styl new-styl 113 current python version support semant new style 113 python support option type annot 4 114 annot enforc languag may use extern tool mypi catch error 115 116 mypi also support python compil call mypyc leverag type annot optim 117 option type annot 4 114 115 116 117 mutabl boolean valu true fals byte bytearray b 'some ascii bytearray b `` ascii '' bytearray 119 105 107 105 b 'some ascii b `` ascii '' byte 119 105 107 105 complex number 3 2.7 j 3 2.7 j associ array 'key1 1.0 3 fals ellipsi numpi ... ellipsi double-precis floating-point number ieee 754 118 1.33333 1.33333 set frozenset 4.0 'string true frozenset 4.0 'string true integ 119 42 list 4.0 'string true null none overload oper notimpl 120 rang − 1 10 rang 10 − 5 − 2 set 4.0 'string true set charact string 'wikipedia `` wikipedia '' `` '' '' span multipl line '' '' '' span multipl line 4.0 'string true 'singl element arithmet oper edit edit edit python usual symbol arithmet oper floor divis oper // modulo oper remaind neg e.g 4 -3 == -2 also exponenti e.g 5 3 == 125 9 0.5 == 3.0 matrix‑multipl oper 121 oper work like tradit math preced rule oper infix also unari repres posit neg number respect modulo oper exponenti 121 preced rule infix unari divis integ produc floating-point result behavior divis chang significantli time 122 122 current python i.e sinc 3.0 chang alway floating-point divis e.g 5/2 == 2.5 5 2 == 2.5 floor divis // oper introduc 7//3 == 2 -7//3 == -3 7.5//3 == 2.0 -7.5//3 == -3.0 ad __future__ import divis caus modul use python 2.7 use python 3.0 rule divis see __future__ import divis python term true divis simpli divis // floor divis version 3.0 classic divis 122 122 round toward neg infin though differ languag add consist instanc mean equat b //b == a//b 1 alway true also mean equat b a//b b == valid posit neg valu howev maintain valid equat mean result b expect half-open interv 0 b b posit integ lie interv b 0 b neg 123 b // b == // b 1 b // b b == half-open interv 123 python provid round function round float nearest integ tie-break python 3 use round even round 1.5 round 2.5 produc 2 124 version 3 use round-away-from-zero round 0.5 1.0 round -0.5 −1.0 125 round tie-break round even 124 round-away-from-zero 125 python allow boolean express multipl equal relat manner consist gener use mathemat exampl express b c test whether less b b less c. 126 c-deriv languag interpret express differ c express would first evalu b result 0 1 result would compar c. 127 126 127 python use arbitrary-precis arithmet integ oper decim type/class decim modul provid decim floating-point number pre-defin arbitrari precis sever round mode 128 fraction class fraction modul provid arbitrari precis ration number 129 arbitrary-precis arithmet decim floating-point number 128 ration number 129 due python 's extens mathemat librari third-parti librari numpi extend nativ capabl frequent use scientif script languag aid problem numer data process manipul 130 131 numpi 130 131 function syntax edit edit edit function creat python use def keyword python defin function call type function name attribut requir exampl function print whatev given def printer input1 input2 `` alreadi '' print input1 print input2 printer `` hello '' exampl output hello alreadi want attribut set valu valu given use variable-defin syntax insid function definit program exampl edit edit edit `` hello world '' program `` hello world '' program print 'hello world program calcul factori posit integ factori n int input 'type number factori print n 0 rais valueerror 'you must enter non-neg integ factori 1 rang 2 n 1 factori print factori librari edit edit edit python 's larg standard librari 132 provid tool suit mani task commonli cite one greatest strength internet-fac applic mani standard format protocol mime http support includ modul creat graphic user interfac connect relat databas gener pseudorandom number arithmet arbitrary-precis decim 128 manipul regular express unit test 132 mime http graphic user interfac relat databas gener pseudorandom number 128 regular express unit test part standard librari cover specifications—for exampl web server gateway interfac wsgi implement wsgiref follow pep 333 133 —but specifi code intern document test suit howev standard librari cross-platform python code modul need alter rewrit variant implement web server gateway interfac 133 test suit 17 march 2024 updat python packag index pypi offici repositori third-parti python softwar contain 523,000 134 packag wide rang function includ updat python packag index 134 autom autom data analyt data analyt databas databas document document graphic user interfac graphic user interfac imag process imag process machin learn machin learn mobil app mobil app multimedia multimedia comput network comput network scientif comput scientif comput system administr system administr test framework test framework text process text process web framework web framework web scrape web scrape develop environ edit edit edit comparison integr develop environ § python python implement includ cpython includ read–eval–print loop repl permit function command line interpret user enter statement sequenti receiv result immedi read–eval–print loop command line interpret python also come integr develop environ ide call idl beginner-ori integr develop environ ide idl shell includ idl ipython add abil improv auto-complet session state retent syntax highlight idl ipython syntax highlight well standard desktop integr develop environ includ pycharm intellij idea visual studio code etc web browser-bas ide includ sagemath develop science- math-rel program pythonanywher browser-bas ide host environ canopi ide commerci ide emphas scientif comput 135 integr develop environ web browser sagemath pythonanywher scientif comput 135 implement edit edit edit list python softwar § python implement refer implement edit edit edit cpython refer implement python written c meet c89 standard python 3.11 use c11 136 sever select c99 featur cpython includ c extens third-parti extens limit older c versions—e.g implement c11 c++ 137 138 cpython compil python program intermedi bytecod 139 execut virtual machin 140 cpython distribut larg standard librari written mixtur c nativ python avail mani platform includ window start python 3.9 python instal deliber fail instal window 7 8 141 142 window xp support python 3.5 modern unix-lik system includ maco appl m1 mac sinc python 3.9.1 experiment instal unoffici support vm 143 platform portabl one earliest prioriti 144 python 1 2 develop even os/2 solari support 145 support sinc drop mani platform cpython refer implement c89 c11 136 c99 c11 137 138 compil bytecod 139 virtual machin 140 window 7 141 142 window xp unix-lik appl m1 vm 143 144 os/2 solari 145 current python version i.e sinc 3.7 support oper system multi-thread support implement edit edit edit altern implement least slightli differ semant e.g may unord dictionari unlik current python version e.g larger python ecosystem support c python api pypi pypi fast compliant interpret python 2.7 3.10 146 147 just-in-tim compil often bring signific speed improv cpython librari written c use 148 e.g risc-v support pypi 146 147 just-in-tim compil 148 risc-v codon languag ahead-of-tim aot compil aot compil statically-typ python-lik languag `` syntax semant nearli ident python 's notabl differ '' 149 e.g use 64-bit machin integ speed arbitrari like python claim speedup cpython usual order 10–100x compil machin code via llvm support nativ multithread 150 codon also compil python extens modul import use python codon ahead-of-tim aot compil 149 llvm 150 stackless python signific fork cpython implement microthread use call stack way thu allow massiv concurr program pypi also stackless version 151 stackless python microthread call stack 151 micropython circuitpython python 3 variant optim microcontrol includ lego mindstorm ev3 152 micropython circuitpython microcontrol lego mindstorm ev3 152 pyston variant python runtim use just-in-tim compil speed execut python program 153 153 cinder performance-ori fork cpython 3.8 contain number optim includ bytecod inlin cach eager evalu coroutin method-at-a-tim jit experiment bytecod compil 154 jit 154 snek 155 156 157 embed comput languag compat e.g 8-bit avr microcontrol atmega 328p-base arduino well larger one compat micropython `` python-inspir python possibl write snek program run full python system python program run snek `` 158 imper languag includ oop class unlik python simplifi one number type 32-bit single-precis similar javascript except smaller snek 155 156 157 avr microcontrol atmega 328p micropython 158 oop single-precis javascript longer support implement edit edit edit just-in-tim python compil develop unsupport googl began project name unladen swallow 2009 aim speed python interpret five-fold use llvm improv multithread abil scale thousand core 159 ordinari implement suffer global interpret lock unladen swallow llvm multithread 159 global interpret lock psyco discontinu just-in-tim special compil integr cpython transform bytecod machin code runtim emit code special certain data type faster standard python code psyco support python 2.7 later psyco just-in-tim special data type pys60 python 2 interpret seri 60 mobil phone releas nokia 2005 implement mani modul standard librari addit modul integr symbian oper system nokia n900 also support python gtk widget librari enabl program written run target devic 160 pys60 seri 60 nokia symbian n900 gtk 160 cross-compil languag edit edit edit sever compilers/transpil high-level object languag either unrestrict python restrict subset python languag similar python sourc languag transpil brython 161 transcrypt 162 163 pyj latest releas 2012 compil python javascript 161 162 163 pyj javascript cython compil superset python c. result code also usabl python via direct c-level api call python interpret cython pyjl compiles/transpil subset python `` human-read maintain high-perform julia sourc code '' 88 despit claim high perform tool claim arbitrari python code i.e 's known possibl compil faster languag machin code unless semant python chang mani case speedup possibl chang python code faster julia sourc code use python compil machin code base way 88 nuitka compil python c. 164 work python 3.4 3.12 2.6 2.7 python 's main support platform window 7 even window xp android claim complet support python 3.10 support 3.11 3.12 experiment support python 3.13 support maco includ appl silicon-bas 's free compil though also commerci add-on e.g hide sourc code nuitka 164 numba use python tool enabl ad decor relev python code jit compil translat subset python numpi code fast machin code numba pythran compil subset python 3 c++ c++11 165 c++11 165 rpython compil c use build pypi interpret python rpython python → 11l → c++ transpil 166 compil subset python 3 c++ c++17 166 c++17 special myhdl python-bas hardwar descript languag hdl convert myhdl code verilog vhdl code myhdl hardwar descript languag verilog vhdl older project use python 3.x latest syntax googl 's grumpi latest releas 2017 transpil python 2 go 167 168 169 transpil go 167 168 169 ironpython allow run python 2.7 program alpha releas 2021 also avail `` python 3.4 although featur behavior later version may includ '' 170 .net common languag runtim 171 ironpython alpha 170 common languag runtim 171 jython compil python 2.7 java bytecod allow use java librari python program 172 jython 172 pyrex latest releas 2010 shed skin latest releas 2013 compil c c++ respect pyrex shed skin perform edit edit edit perform comparison variou python implement non-numer combinatori workload present euroscipi '13 173 python 's perform compar program languag also benchmark comput languag benchmark game 174 173 comput languag benchmark game 174 develop edit edit edit python 's develop conduct larg python enhanc propos pep process primari mechan propos major new featur collect commun input issu document python design decis 175 python code style cover pep 8 176 outstand pep review comment python commun steer council 175 175 176 175 enhanc languag correspond develop cpython refer implement mail list python-dev primari forum languag 's develop specif issu origin discus roundup bug tracker host foundat 177 2022 issu discus migrat github 178 develop origin took place self-host source-cod repositori run mercuri python move github januari 2017 179 roundup bug tracker 177 github 178 self-host mercuri github 179 cpython 's public releas come three type distinguish part version number increment backward-incompat version code expect break need manual port first part version number increment releas happen infrequently—vers 3.0 releas 8 year 2.0 accord guido van rossum version 4.0 unlik ever happen 180 port 180 major `` featur '' releas larg compat previou version introduc new featur second part version number increment start python 3.9 releas expect happen annual 181 182 major version support bug fix sever year releas 183 181 182 183 bug fix releas 184 introduc new featur occur everi 3 month made suffici number bug fix upstream sinc last releas secur vulner also patch releas third final part version number increment 184 184 184 mani alpha beta release-candid also releas preview test final releas although rough schedul releas often delay code readi python 's develop team monitor state code run larg unit test suit develop 185 alpha beta release-candid unit test 185 major academ confer python pycon also special python mentor program pyladi academ confer pycon pyladi python 3.12 remov wstr mean python extens 186 need modifi 187 3.10 ad pattern match languag 188 186 187 pattern match 188 python 3.12 drop outdat modul drop futur deprec 3.13 alreadi deprec array u format code emit deprecationwarn sinc 3.13 remov python 3.16 w format code use instead part ctype also deprec http.server.cgihttprequesthandl emit deprecationwarn remov 3.15 use code alreadi high potenti secur function bug part type modul deprec e.g creat typing.namedtupl class use keyword argument denot field disallow python 3.15 api document gener edit edit edit tool gener document python api includ pydoc avail part standard librari sphinx pdoc fork doxygen graphviz among other 189 api pydoc sphinx pdoc doxygen graphviz 189 name edit edit edit python 's name deriv british comedi group monti python python creator guido van rossum enjoy develop languag monti python refer appear frequent python code cultur 190 exampl metasyntact variabl often use python literatur spam egg instead tradit foo bar 190 191 offici python document also contain variou refer monti python routin 192 193 user python sometim refer `` pythonista '' 194 monti python 190 metasyntact variabl spam egg foo bar 190 191 192 193 194 prefix py- use show someth relat python exampl use prefix name python applic librari includ pygam bind simpl directmedia layer python commonli use creat game pyqt pygtk bind qt gtk python respect pypi python implement origin written python pygam bind simpl directmedia layer pyqt pygtk qt pypi popular edit edit edit sinc 2003 python consist rank top ten popular program languag tiob program commun index decemb 2022 updat popular languag ahead c c++ java 40 select program languag year `` highest rise rate year '' 2007 2010 2018 2020 languag done four time 2020 updat 195 tiob program commun index updat java 40 updat 195 larg organ use python includ wikipedia googl 196 yahoo 197 cern 198 nasa 199 facebook 200 amazon instagram 201 spotifi 202 smaller entiti like industri light magic 203 ita 204 social news network site reddit written mostli python 205 organ partial use python includ discord 206 baidu 207 wikipedia googl 196 yahoo 197 cern 198 nasa 199 facebook 200 amazon instagram 201 spotifi 202 industri light magic 203 ita 204 reddit 205 discord 206 baidu 207 use edit edit edit list python softwar python serv script languag web applic e.g via mod_wsgi apach webserv 208 web server gateway interfac standard api evolv facilit applic web framework like django pylon pyramid turbogear web2pi tornado flask bottl zope support develop design mainten complex applic pyj ironpython use develop client-sid ajax-bas applic sqlalchemi use data mapper relat databas twist framework program commun comput use exampl dropbox script languag web applic mod_wsgi apach webserv 208 web server gateway interfac web framework django pylon pyramid turbogear web2pi tornado flask zope ironpython sqlalchemi data mapper twist dropbox librari numpi scipi matplotlib allow effect use python scientif comput 209 210 special librari biopython astropi provid domain-specif function sagemath comput algebra system notebook interfac programm python librari cover mani aspect mathemat includ algebra combinator numer mathemat number theori calculu 211 opencv python bind rich set featur comput vision imag process 212 numpi scipi matplotlib 209 210 biopython astropi sagemath comput algebra system notebook interfac mathemat algebra combinator numer mathemat number theori calculu 211 opencv comput vision imag process 212 python commonli use artifici intellig project machin learn project help librari like tensorflow kera pytorch scikit-learn logic languag problog 213 214 215 216 217 script languag modular architectur simpl syntax rich text process tool python often use natur languag process 218 artifici intellig tensorflow kera pytorch scikit-learn problog 213 214 215 216 217 modular architectur natur languag process 218 combin python prolog prove particularli use ai applic prolog provid knowledg represent reason capabl janu system particular exploit similar two languag part use dynam type simpl recurs natur data structur typic applic combin includ natur languag process visual queri answer geospati reason handl semant web data 219 220 natlog system implement python use definit claus grammar dcg prompt gener text-to-text gener like gpt3 text-to-imag gener like dall- stabl diffus 221 prolog 219 220 definit claus grammar 221 python also use graphic user interfac gui use librari like tkinter 222 223 graphic user interfac tkinter 222 223 python success embed mani softwar product script languag includ finit element method softwar abaqu 3d parametr model like freecad 3d anim packag 3d max blender cinema 4d lightwav houdini maya modo motionbuild softimag visual effect compositor nuke 2d imag program like gimp 224 inkscap scribu paint shop pro 225 music notat program like scorewrit capella gnu debugg use python pretti printer show complex structur c++ contain esri promot python best choic write script arcgi 226 also use sever video game 227 228 adopt first three avail program languag googl app engin two java go 229 finit element method abaqu freecad 3d max blender cinema 4d lightwav houdini maya modo motionbuild softimag nuke gimp 224 inkscap scribu paint shop pro 225 music notat scorewrit capella gnu debugg pretti printer esri arcgi 226 227 228 program languag googl app engin java go 229 mani oper system includ python standard compon ship linux distribut 230 amigao 4 use python 2.7 freebsd packag netbsd openbsd packag use command line termin mani linux distribut use instal written python ubuntu use ubiqu instal red hat linux fedora linux use anaconda instal gentoo linux use python packag manag system portag linux distribut 230 amigao 4 freebsd netbsd openbsd ubuntu ubiqu red hat linux fedora linux anaconda gentoo linux packag manag system portag python use extens inform secur industri includ exploit develop 231 232 inform secur 231 232 sugar softwar one laptop per child xo develop sugar lab 2008 updat written python 233 raspberri pi single-board comput project adopt python main user-program languag sugar one laptop per child sugar lab updat 233 raspberri pi single-board comput libreoffic includ python intend replac java python python script provid core featur 234 sinc version 4.0 7 februari 2013 libreoffic 234 languag influenc python edit edit edit python 's design philosophi influenc mani program languag boo use indent similar syntax similar object model 235 boo 235 cobra use indent similar syntax acknowledg document list python first among languag influenc 236 cobra 236 coffeescript program languag cross-compil javascript python-inspir syntax coffeescript ecmascript–javascript borrow iter gener python 237 ecmascript javascript gener 237 gdscript script languag similar python built-in godot game engin 238 gdscript godot 238 go design `` speed work dynam languag like python '' 239 share syntax slice array go 239 groovi motiv desir bring python design philosophi java 240 groovi java 240 julia design `` usabl gener program python '' 27 julia 27 mojo non-strict 28 241 superset python e.g still miss class ad e.g struct 242 mojo 28 241 struct 242 nim use indent similar syntax 243 nim 243 rubi 's creator yukihiro matsumoto said `` want script languag power perl object-ori python 's decid design languag `` 244 rubi yukihiro matsumoto 244 swift program languag develop appl python-inspir syntax 245 swift 245 kotlin blend python java featur minim boilerpl code enhanc develop effici 246 kotlin 246 python 's develop practic also emul languag exampl practic requir document describ rational issu surround chang languag python pep also use tcl 247 erlang 248 swift 249 tcl 247 erlang 248 249 see also edit edit edit comput program portal comput program portal comput program portal free open-sourc softwar portal free open-sourc softwar portal free open-sourc softwar portal python syntax semant python syntax semant pip packag manag pip packag manag list program languag list program languag histori program languag histori program languag comparison program languag comparison program languag refer edit edit edit `` gener python faq – python 3 document '' docs.python.org retriev 7 juli 2024 `` gener python faq – python 3 document '' docs.python.org retriev 7 juli 2024 `` gener python faq – python 3 document '' retriev 7 juli 2024 7 juli `` python 0.9.1 part 01/21 '' alt.sourc archiv archiv origin 11 august 2021 retriev 11 august 2021 `` python 0.9.1 part 01/21 '' alt.sourc archiv archiv origin 11 august 2021 retriev 11 august 2021 `` python 0.9.1 part 01/21 '' archiv retriev 11 august 2021 11 august `` python dynam languag also strongli type languag '' python wiki archiv origin 14 march 2021 retriev 27 januari 2021 `` python dynam languag also strongli type languag '' python wiki archiv origin 14 march 2021 retriev 27 januari 2021 `` python dynam languag also strongli type languag '' archiv retriev 27 januari 2021 27 januari b `` pep 483 – theori type hint '' python.org archiv origin 14 june 2020 retriev 14 june 2018 b b `` pep 483 – theori type hint '' python.org archiv origin 14 june 2020 retriev 14 june 2018 `` pep 483 – theori type hint '' archiv retriev 14 june 2018 14 june `` pep 11 – cpython platform support peps.python.org '' python enhanc propos pep retriev 22 april 2024 `` pep 11 – cpython platform support peps.python.org '' python enhanc propos pep retriev 22 april 2024 `` pep 11 – cpython platform support peps.python.org '' retriev 22 april 2024 22 april `` pep 738 – ad android support platform peps.python.org '' python enhanc propos pep retriev 19 may 2024 `` pep 738 – ad android support platform peps.python.org '' python enhanc propos pep retriev 19 may 2024 `` pep 738 – ad android support platform peps.python.org '' retriev 19 may 2024 19 may `` download python platform '' python.org archiv origin 27 novemb 2020 retriev 18 august 2023 `` download python platform '' python.org archiv origin 27 novemb 2020 retriev 18 august 2023 `` download python platform '' archiv retriev 18 august 2023 18 august `` test – regress test packag python – python 3.7.13 document '' docs.python.org archiv origin 17 may 2022 retriev 17 may 2022 `` test – regress test packag python – python 3.7.13 document '' docs.python.org archiv origin 17 may 2022 retriev 17 may 2022 `` test – regress test packag python – python 3.7.13 document '' archiv retriev 17 may 2022 17 may `` platform – access underli platform 's identifi data – python 3.10.4 document '' docs.python.org archiv origin 17 may 2022 retriev 17 may 2022 `` platform – access underli platform 's identifi data – python 3.10.4 document '' docs.python.org archiv origin 17 may 2022 retriev 17 may 2022 `` platform – access underli platform 's identifi data – python 3.10.4 document '' archiv retriev 17 may 2022 17 may holth moor 30 march 2014 `` pep 0441 – improv python zip applic support '' archiv origin 26 decemb 2018 retriev 12 novemb 2015 holth moor 30 march 2014 `` pep 0441 – improv python zip applic support '' archiv origin 26 decemb 2018 retriev 12 novemb 2015 `` pep 0441 – improv python zip applic support '' archiv retriev 12 novemb 2015 12 novemb `` starlark languag '' archiv origin 15 june 2020 retriev 25 may 2019 `` starlark languag '' archiv origin 15 june 2020 retriev 25 may 2019 `` starlark languag '' archiv retriev 25 may 2019 25 may b `` python creat first place '' gener python faq python softwar foundat archiv origin 24 octob 2012 retriev 22 march 2007 extens experi implement interpret languag abc group cwi work group learn lot languag design origin mani python featur includ use indent statement group inclus high-level data type although detail differ python b b `` python creat first place '' gener python faq python softwar foundat archiv origin 24 octob 2012 retriev 22 march 2007 extens experi implement interpret languag abc group cwi work group learn lot languag design origin mani python featur includ use indent statement group inclus high-level data type although detail differ python `` python creat first place '' archiv retriev 22 march 2007 22 march `` ada 83 refer manual rais statement '' archiv origin 22 octob 2019 retriev 7 januari 2020 `` ada 83 refer manual rais statement '' archiv origin 22 octob 2019 retriev 7 januari 2020 `` ada 83 refer manual rais statement '' archiv retriev 7 januari 2020 7 januari b kuchl andrew m. 22 decemb 2006 `` interview guido van rossum juli 1998 '' amk.ca archiv origin 1 may 2007 retriev 12 march 2012 'd spent summer dec 's system research center introduc modula-2+ modula-3 final report written time learn later show python 's except handl modul fact method explicitli contain 'self paramet list string slice came algol-68 icon b b kuchl andrew m. 22 decemb 2006 `` interview guido van rossum juli 1998 '' amk.ca archiv origin 1 may 2007 retriev 12 march 2012 'd spent summer dec 's system research center introduc modula-2+ modula-3 final report written time learn later show python 's except handl modul fact method explicitli contain 'self paramet list string slice came algol-68 icon `` interview guido van rossum juli 1998 '' origin retriev 12 march 2012 12 march b c `` itertool – function creat iter effici loop – python 3.7.1 document '' docs.python.org archiv origin 14 june 2020 retriev 22 novemb 2016 modul implement number iter build block inspir construct apl haskel sml b c b c `` itertool – function creat iter effici loop – python 3.7.1 document '' docs.python.org archiv origin 14 june 2020 retriev 22 novemb 2016 modul implement number iter build block inspir construct apl haskel sml `` itertool – function creat iter effici loop – python 3.7.1 document '' archiv retriev 22 novemb 2016 22 novemb van rossum guido 1993 `` introduct python unix/c programm '' proceed nluug najaarsconferenti dutch unix user group citeseerx 10.1.1.38.2023. even though design c far ideal influenc python consider van rossum guido 1993 `` introduct python unix/c programm '' proceed nluug najaarsconferenti dutch unix user group citeseerx 10.1.1.38.2023. even though design c far ideal influenc python consider citeseerx 10.1.1.38.2023 10.1.1.38.2023 b `` class '' python tutori python softwar foundat archiv origin 23 octob 2012 retriev 20 februari 2012 mixtur class mechan found c++ modula-3 b b `` class '' python tutori python softwar foundat archiv origin 23 octob 2012 retriev 20 februari 2012 mixtur class mechan found c++ modula-3 `` class '' archiv retriev 20 februari 2012 20 februari lundh fredrik `` call object '' effbot.org archiv origin 23 novemb 2019 retriev 21 novemb 2017. replac `` clu '' `` python '' `` record '' `` instanc '' `` procedur '' `` function method '' get pretti accur descript python 's object model lundh fredrik `` call object '' effbot.org archiv origin 23 novemb 2019 retriev 21 novemb 2017. replac `` clu '' `` python '' `` record '' `` instanc '' `` procedur '' `` function method '' get pretti accur descript python 's object model `` call object '' archiv retriev 21 novemb 2017 21 novemb simionato michel `` python 2.3 method resolut order '' python softwar foundat archiv origin 20 august 2020 retriev 29 juli 2014 c3 method noth python sinc invent peopl work dylan describ paper intend lisper simionato michel `` python 2.3 method resolut order '' python softwar foundat archiv origin 20 august 2020 retriev 29 juli 2014 c3 method noth python sinc invent peopl work dylan describ paper intend lisper `` python 2.3 method resolut order '' archiv retriev 29 juli 2014 29 juli kuchl a. m. `` function program howto '' python v2.7.2 document python softwar foundat archiv origin 24 octob 2012 retriev 9 februari 2012 list comprehens gener express ... concis notat oper borrow function program languag haskel kuchl a. m. `` function program howto '' python v2.7.2 document python softwar foundat archiv origin 24 octob 2012 retriev 9 februari 2012 list comprehens gener express ... concis notat oper borrow function program languag haskel `` function program howto '' archiv retriev 9 februari 2012 9 februari schemenau neil peter tim hetland magnu lie 18 may 2001 `` pep 255 – simpl gener '' python enhanc propos python softwar foundat archiv origin 5 june 2020 retriev 9 februari 2012 schemenau neil peter tim hetland magnu lie 18 may 2001 `` pep 255 – simpl gener '' python enhanc propos python softwar foundat archiv origin 5 june 2020 retriev 9 februari 2012 `` pep 255 – simpl gener '' archiv retriev 9 februari 2012 9 februari `` control flow tool '' python 3 document python softwar foundat archiv origin 4 june 2016 retriev 24 juli 2015 popular demand featur commonli found function program languag like lisp ad python lambda keyword small anonym function creat `` control flow tool '' python 3 document python softwar foundat archiv origin 4 june 2016 retriev 24 juli 2015 popular demand featur commonli found function program languag like lisp ad python lambda keyword small anonym function creat `` control flow tool '' archiv retriev 24 juli 2015 24 juli `` – regular express oper – python 3.10.6 document '' docs.python.org archiv origin 18 juli 2018 retriev 6 septemb 2022 modul provid regular express match oper similar found perl `` – regular express oper – python 3.10.6 document '' docs.python.org archiv origin 18 juli 2018 retriev 6 septemb 2022 modul provid regular express match oper similar found perl `` – regular express oper – python 3.10.6 document '' archiv retriev 6 septemb 2022 6 septemb `` coffeescript '' coffeescript.org archiv origin 12 june 2020 retriev 3 juli 2018 `` coffeescript '' coffeescript.org archiv origin 12 june 2020 retriev 3 juli 2018 `` coffeescript '' archiv retriev 3 juli 2018 3 juli `` perl python influenc javascript '' www.2ality.com 24 februari 2013 archiv origin 26 decemb 2018 retriev 15 may 2015 `` perl python influenc javascript '' www.2ality.com 24 februari 2013 archiv origin 26 decemb 2018 retriev 15 may 2015 `` perl python influenc javascript '' archiv retriev 15 may 2015 15 may rauschmay axel `` chapter 3 natur javascript influenc '' o'reilli speak javascript archiv origin 26 decemb 2018 retriev 15 may 2015 rauschmay axel `` chapter 3 natur javascript influenc '' o'reilli speak javascript archiv origin 26 decemb 2018 retriev 15 may 2015 `` chapter 3 natur javascript influenc '' archiv retriev 15 may 2015 15 may b `` creat julia '' julia websit februari 2012 archiv origin 2 may 2020 retriev 5 june 2014 want someth usabl gener program python ... b b `` creat julia '' julia websit februari 2012 archiv origin 2 may 2020 retriev 5 june 2014 want someth usabl gener program python ... `` creat julia '' archiv retriev 5 june 2014 5 june b krill paul 4 may 2023 `` mojo languag marri python mlir ai develop '' infoworld archiv origin 5 may 2023 retriev 5 may 2023 b b krill paul 4 may 2023 `` mojo languag marri python mlir ai develop '' infoworld archiv origin 5 may 2023 retriev 5 may 2023 `` mojo languag marri python mlir ai develop '' archiv retriev 5 may 2023 5 may ring team 4 decemb 2017 `` ring languag '' ring-lang.net ring-lang archiv origin 25 decemb 2018 retriev 4 decemb 2017 ring team 4 decemb 2017 `` ring languag '' ring-lang.net ring-lang archiv origin 25 decemb 2018 retriev 4 decemb 2017 `` ring languag '' ring-lang archiv retriev 4 decemb 2017 4 decemb bini ola 2007 practic jrubi rail web 2.0 project bring rubi rail java platform berkeley apress p. 3 isbn 978-1-59059-881-8 bini ola 2007 practic jrubi rail web 2.0 project bring rubi rail java platform berkeley apress p. 3 isbn 978-1-59059-881-8 practic jrubi rail web 2.0 project bring rubi rail java platform practic jrubi rail web 2.0 project bring rubi rail java platform 3 isbn 978-1-59059-881-8 lattner chri 3 june 2014 `` chri lattner 's homepag '' chri lattner archiv origin 25 decemb 2018 retriev 3 june 2014 swift languag product tireless effort team languag expert document guru compil optim ninja incred import intern dogfood group provid feedback help refin battle-test idea cours also greatli benefit experi hard-won mani languag field draw idea objective-c rust haskel rubi python c clu far mani other list lattner chri 3 june 2014 `` chri lattner 's homepag '' chri lattner archiv origin 25 decemb 2018 retriev 3 june 2014 swift languag product tireless effort team languag expert document guru compil optim ninja incred import intern dogfood group provid feedback help refin battle-test idea cours also greatli benefit experi hard-won mani languag field draw idea objective-c rust haskel rubi python c clu far mani other list `` chri lattner 's homepag '' archiv retriev 3 june 2014 3 june `` v document introduct '' github retriev 24 decemb 2024 `` v document introduct '' github retriev 24 decemb 2024 `` v document introduct '' retriev 24 decemb 2024 24 decemb kuhlman dave `` python book begin python advanc python python exercis '' section 1.1 archiv origin pdf 23 june 2012 kuhlman dave `` python book begin python advanc python python exercis '' section 1.1 archiv origin pdf 23 june 2012 `` python book begin python advanc python python exercis '' origin pdf `` python '' python softwar foundat archiv origin 20 april 2012 retriev 24 april 2012. second section `` fan python use phrase `` batteri includ '' describ standard librari cover everyth asynchron process zip file '' `` python '' python softwar foundat archiv origin 20 april 2012 retriev 24 april 2012. second section `` fan python use phrase `` batteri includ '' describ standard librari cover everyth asynchron process zip file '' `` python '' archiv retriev 24 april 2012 24 april `` pep 206 – python advanc librari '' python.org archiv origin 5 may 2021 retriev 11 octob 2021 `` pep 206 – python advanc librari '' python.org archiv origin 5 may 2021 retriev 11 octob 2021 `` pep 206 – python advanc librari '' archiv retriev 11 octob 2021 11 octob rossum guido van 20 januari 2009 `` histori python brief timelin python '' histori python archiv origin 5 june 2020 retriev 5 march 2021 rossum guido van 20 januari 2009 `` histori python brief timelin python '' histori python archiv origin 5 june 2020 retriev 5 march 2021 `` histori python brief timelin python '' archiv retriev 5 march 2021 5 march peterson benjamin 20 april 2020 `` python 2.7.18 last releas python 2 '' python insid archiv origin 26 april 2020 retriev 27 april 2020 peterson benjamin 20 april 2020 `` python 2.7.18 last releas python 2 '' python insid archiv origin 26 april 2020 retriev 27 april 2020 `` python 2.7.18 last releas python 2 '' archiv retriev 27 april 2020 27 april `` stack overflow develop survey 2022 '' stack overflow archiv origin 27 june 2022 retriev 12 august 2022 `` stack overflow develop survey 2022 '' stack overflow archiv origin 27 june 2022 retriev 12 august 2022 `` stack overflow develop survey 2022 '' archiv retriev 12 august 2022 12 august `` state develop ecosystem 2020 infograph '' jetbrain develop tool profession team archiv origin 1 march 2021 retriev 5 march 2021 `` state develop ecosystem 2020 infograph '' jetbrain develop tool profession team archiv origin 1 march 2021 retriev 5 march 2021 `` state develop ecosystem 2020 infograph '' archiv retriev 5 march 2021 5 march b `` tiob index '' tiob archiv origin 25 februari 2018 retriev 3 januari 2023 tiob program commun index indic popular program languag updat requir b b `` tiob index '' tiob archiv origin 25 februari 2018 retriev 3 januari 2023 tiob program commun index indic popular program languag updat requir `` tiob index '' archiv retriev 3 januari 2023 3 januari `` pypl popular program languag index '' pypl.github.io archiv origin 14 march 2017 retriev 26 march 2021 `` pypl popular program languag index '' pypl.github.io archiv origin 14 march 2017 retriev 26 march 2021 `` pypl popular program languag index '' archiv retriev 26 march 2021 26 march b venner bill 13 januari 2003 `` make python '' artima develop artima archiv origin 1 septemb 2016 retriev 22 march 2007 b b venner bill 13 januari 2003 `` make python '' artima develop artima archiv origin 1 septemb 2016 retriev 22 march 2007 `` make python '' archiv retriev 22 march 2007 22 march van rossum guido 29 august 2000 `` setl lukewarm rang liter '' python-dev mail list archiv origin 14 juli 2018 retriev 13 march 2011 van rossum guido 29 august 2000 `` setl lukewarm rang liter '' python-dev mail list archiv origin 14 juli 2018 retriev 13 march 2011. van rossum guido `` setl lukewarm rang liter '' archiv retriev 13 march 2011 13 march van rossum guido 20 januari 2009 `` brief timelin python '' histori python archiv origin 5 june 2020 retriev 20 januari 2009 van rossum guido 20 januari 2009 `` brief timelin python '' histori python archiv origin 5 june 2020 retriev 20 januari 2009 `` brief timelin python '' archiv retriev 20 januari 2009 20 januari fairchild carli 12 juli 2018 `` guido van rossum step role python 's benevol dictat life '' linux journal archiv origin 13 juli 2018 retriev 13 juli 2018 fairchild carli 12 juli 2018 `` guido van rossum step role python 's benevol dictat life '' linux journal archiv origin 13 juli 2018 retriev 13 juli 2018 `` guido van rossum step role python 's benevol dictat life '' archiv retriev 13 juli 2018 13 juli `` pep 8100 '' python softwar foundat archiv origin 4 june 2020 retriev 4 may 2019 `` pep 8100 '' python softwar foundat archiv origin 4 june 2020 retriev 4 may 2019 `` pep 8100 '' archiv retriev 4 may 2019 4 may `` pep 13 – python languag govern '' python.org archiv origin 27 may 2021 retriev 25 august 2021 `` pep 13 – python languag govern '' python.org archiv origin 27 may 2021 retriev 25 august 2021 `` pep 13 – python languag govern '' archiv retriev 25 august 2021 25 august brigg jason r. lipovača miran 2013 python kid play introduct program san francisco calif starch press isbn 978-1-59327-407-8 brigg jason r. lipovača miran 2013 python kid play introduct program san francisco calif starch press isbn 978-1-59327-407-8 isbn 978-1-59327-407-8 kuchl a. m. zadka mosh 16 octob 2000 `` 's new python 2.0 '' python softwar foundat archiv origin 23 octob 2012 retriev 11 februari 2012 kuchl a. m. zadka mosh 16 octob 2000 `` 's new python 2.0 '' python softwar foundat archiv origin 23 octob 2012 retriev 11 februari 2012 `` 's new python 2.0 '' archiv retriev 11 februari 2012 11 februari `` pep 373 – python 2.7 releas schedul '' python.org archiv origin 19 may 2020 retriev 9 januari 2017 `` pep 373 – python 2.7 releas schedul '' python.org archiv origin 19 may 2020 retriev 9 januari 2017 `` pep 373 – python 2.7 releas schedul '' archiv retriev 9 januari 2017 9 januari `` pep 466 – network secur enhanc python 2.7.x '' python.org archiv origin 4 june 2020 retriev 9 januari 2017 `` pep 466 – network secur enhanc python 2.7.x '' python.org archiv origin 4 june 2020 retriev 9 januari 2017 `` pep 466 – network secur enhanc python 2.7.x '' archiv retriev 9 januari 2017 9 januari `` sunset python 2 '' python.org archiv origin 12 januari 2020 retriev 22 septemb 2019 `` sunset python 2 '' python.org archiv origin 12 januari 2020 retriev 22 septemb 2019 `` sunset python 2 '' archiv retriev 22 septemb 2019 22 septemb `` pep 373 – python 2.7 releas schedul '' python.org archiv origin 13 januari 2020 retriev 22 septemb 2019 `` pep 373 – python 2.7 releas schedul '' python.org archiv origin 13 januari 2020 retriev 22 septemb 2019 `` pep 373 – python 2.7 releas schedul '' archiv retriev 22 septemb 2019 22 septemb mattip 25 decemb 2023 `` pypi v7.3.14 releas '' pypi archiv origin 5 januari 2024 retriev 5 januari 2024 mattip 25 decemb 2023 `` pypi v7.3.14 releas '' pypi archiv origin 5 januari 2024 retriev 5 januari 2024 `` pypi v7.3.14 releas '' archiv retriev 5 januari 2024 5 januari langa łukasz 17 may 2022 `` python 3.9.13 avail '' python insid archiv origin 17 may 2022 retriev 21 may 2022 langa łukasz 17 may 2022 `` python 3.9.13 avail '' python insid archiv origin 17 may 2022 retriev 21 may 2022 `` python 3.9.13 avail '' archiv retriev 21 may 2022 21 may `` statu python version '' python develop 's guid retriev 7 octob 2024 `` statu python version '' python develop 's guid retriev 7 octob 2024 `` statu python version '' retriev 7 octob 2024 7 octob `` python '' endoflife.d 8 octob 2024 retriev 20 novemb 2024 `` python '' endoflife.d 8 octob 2024 retriev 20 novemb 2024 `` python '' retriev 20 novemb 2024 20 novemb `` cve-2021-3177 '' red hat custom portal archiv origin 6 march 2021 retriev 26 februari 2021 `` cve-2021-3177 '' red hat custom portal archiv origin 6 march 2021 retriev 26 februari 2021 `` cve-2021-3177 '' archiv retriev 26 februari 2021 26 februari `` cve-2021-3177 '' cve archiv origin 27 februari 2021 retriev 26 februari 2021 `` cve-2021-3177 '' cve archiv origin 27 februari 2021 retriev 26 februari 2021 `` cve-2021-3177 '' archiv retriev 26 februari 2021 26 februari `` cve-2021-23336 '' cve archiv origin 24 februari 2021 retriev 26 februari 2021 `` cve-2021-23336 '' cve archiv origin 24 februari 2021 retriev 26 februari 2021 `` cve-2021-23336 '' archiv retriev 26 februari 2021 26 februari `` built-in type '' `` built-in type '' `` built-in type '' corbet 24 octob 2022 `` python 3.11 releas lwn.net '' lwn.net retriev 15 novemb 2022 corbet 24 octob 2022 `` python 3.11 releas lwn.net '' lwn.net retriev 15 novemb 2022 `` python 3.11 releas lwn.net '' retriev 15 novemb 2022 15 novemb `` 's new python 3.13 '' python document retriev 30 april 2024 `` 's new python 3.13 '' python document retriev 30 april 2024 `` 's new python 3.13 '' retriev 30 april 2024 30 april `` pep 667 – consist view namespac peps.python.org '' python enhanc propos pep retriev 7 octob 2024 `` pep 667 – consist view namespac peps.python.org '' python enhanc propos pep retriev 7 octob 2024 `` pep 667 – consist view namespac peps.python.org '' retriev 7 octob 2024 7 octob wouter thoma 9 april 2024 `` python insid python 3.12.3 3.13.0a6 releas '' python insid retriev 29 april 2024 wouter thoma 9 april 2024 `` python insid python 3.12.3 3.13.0a6 releas '' python insid retriev 29 april 2024 `` python insid python 3.12.3 3.13.0a6 releas '' retriev 29 april 2024 29 april `` pep 594 – remov dead batteri standard librari '' python enhanc propos python softtwar foundat 20 may 2019 `` pep 594 – remov dead batteri standard librari '' python enhanc propos python softtwar foundat 20 may 2019 `` pep 594 – remov dead batteri standard librari '' `` pep 761 – deprec pgp signatur cpython artifact peps.python.org '' python enhanc propos pep retriev 6 januari 2025 `` pep 761 – deprec pgp signatur cpython artifact peps.python.org '' python enhanc propos pep retriev 6 januari 2025 `` pep 761 – deprec pgp signatur cpython artifact peps.python.org '' retriev 6 januari 2025 6 januari `` pep 749 – implement pep 649 peps.python.org '' python enhanc propos pep retriev 20 novemb 2024 `` pep 749 – implement pep 649 peps.python.org '' python enhanc propos pep retriev 20 novemb 2024 `` pep 749 – implement pep 649 peps.python.org '' retriev 20 novemb 2024 20 novemb `` pep 711 pybi standard format distribut python binari '' discus python.org 7 april 2023 retriev 20 novemb 2024 `` pep 711 pybi standard format distribut python binari '' discus python.org 7 april 2023 retriev 20 novemb 2024 `` pep 711 pybi standard format distribut python binari '' retriev 20 novemb 2024 20 novemb `` pep 686 – make utf-8 mode default peps.python.org '' python enhanc propos pep retriev 20 novemb 2024 `` pep 686 – make utf-8 mode default peps.python.org '' python enhanc propos pep retriev 20 novemb 2024 `` pep 686 – make utf-8 mode default peps.python.org '' retriev 20 novemb 2024 20 novemb cain gang ltd. `` python metaclass '' pdf archiv origin pdf 30 may 2009 retriev 27 june 2009 cain gang ltd. `` python metaclass '' pdf archiv origin pdf 30 may 2009 retriev 27 june 2009 `` python metaclass '' pdf origin pdf retriev 27 june 2009 27 june `` 3.3 special method name '' python languag refer python softwar foundat archiv origin 15 decemb 2018 retriev 27 june 2009 `` 3.3 special method name '' python languag refer python softwar foundat archiv origin 15 decemb 2018 retriev 27 june 2009 `` 3.3 special method name '' archiv retriev 27 june 2009 27 june `` pydbc method precondit method postcondit class invari python '' archiv origin 23 novemb 2019 retriev 24 septemb 2011 `` pydbc method precondit method postcondit class invari python '' archiv origin 23 novemb 2019 retriev 24 septemb 2011 `` pydbc method precondit method postcondit class invari python '' archiv retriev 24 septemb 2011 24 septemb `` contract python '' archiv origin 15 june 2020 retriev 24 septemb 2011 `` contract python '' archiv origin 15 june 2020 retriev 24 septemb 2011 `` contract python '' archiv retriev 24 septemb 2011 24 septemb `` pydatalog '' archiv origin 13 june 2020 retriev 22 juli 2012 `` pydatalog '' archiv origin 13 june 2020 retriev 22 juli 2012 `` pydatalog '' archiv retriev 22 juli 2012 22 juli `` glue togeth python '' python.org retriev 30 septemb 2024 `` glue togeth python '' python.org retriev 30 septemb 2024 `` glue togeth python '' retriev 30 septemb 2024 30 septemb `` extend embed python interpret refer count '' docs.python.org archiv origin 18 octob 2012 retriev 5 june 2020 sinc python make heavi use malloc free need strategi avoid memori leak well use freed memori chosen method call refer count `` extend embed python interpret refer count '' docs.python.org archiv origin 18 octob 2012 retriev 5 june 2020 sinc python make heavi use malloc free need strategi avoid memori leak well use freed memori chosen method call refer count `` extend embed python interpret refer count '' archiv retriev 5 june 2020 5 june b hetting raymond 30 januari 2002 `` pep 289 – gener express '' python enhanc propos python softwar foundat archiv origin 14 june 2020 retriev 19 februari 2012 b b hetting raymond 30 januari 2002 `` pep 289 – gener express '' python enhanc propos python softwar foundat archiv origin 14 june 2020 retriev 19 februari 2012 `` pep 289 – gener express '' archiv retriev 19 februari 2012 19 februari `` 6.5 itertool – function creat iter effici loop '' docs.python.org archiv origin 14 june 2020 retriev 22 novemb 2016 `` 6.5 itertool – function creat iter effici loop '' docs.python.org archiv origin 14 june 2020 retriev 22 novemb 2016 `` 6.5 itertool – function creat iter effici loop '' archiv retriev 22 novemb 2016 22 novemb b peter tim 19 august 2004 `` pep 20 – zen python '' python enhanc propos python softwar foundat archiv origin 26 decemb 2018 retriev 24 novemb 2008 b b peter tim 19 august 2004 `` pep 20 – zen python '' python enhanc propos python softwar foundat archiv origin 26 decemb 2018 retriev 24 novemb 2008 `` pep 20 – zen python '' archiv retriev 24 novemb 2008 24 novemb lutz mark januari 2022 `` python chang 2014+ '' learn python archiv origin 15 march 2024 retriev 25 februari 2024 lutz mark januari 2022 `` python chang 2014+ '' learn python archiv origin 15 march 2024 retriev 25 februari 2024 `` python chang 2014+ '' archiv retriev 25 februari 2024 25 februari `` confus regard rule zen python '' python help discus python.org 3 may 2022 archiv origin 25 februari 2024 retriev 25 februari 2024 `` confus regard rule zen python '' python help discus python.org 3 may 2022 archiv origin 25 februari 2024 retriev 25 februari 2024 `` confus regard rule zen python '' archiv retriev 25 februari 2024 25 februari ambi chetan 4 juli 2021 `` controversi python walru oper '' python simplifi archiv origin 27 august 2023 retriev 5 februari 2024 ambi chetan 4 juli 2021 `` controversi python walru oper '' python simplifi archiv origin 27 august 2023 retriev 5 februari 2024 `` controversi python walru oper '' archiv retriev 5 februari 2024 5 februari grifski jeremi 24 may 2020 `` controversi behind walru oper python '' renegad coder archiv origin 28 decemb 2023 retriev 25 februari 2024 grifski jeremi 24 may 2020 `` controversi behind walru oper python '' renegad coder archiv origin 28 decemb 2023 retriev 25 februari 2024 `` controversi behind walru oper python '' archiv retriev 25 februari 2024 25 februari bader dan `` python string format best practic '' real python archiv origin 18 februari 2024 retriev 25 februari 2024 bader dan `` python string format best practic '' real python archiv origin 18 februari 2024 retriev 25 februari 2024 `` python string format best practic '' archiv retriev 25 februari 2024 25 februari martelli alex ravenscroft anna ascher david 2005 python cookbook 2nd edit o'reilli medium p. 230 isbn 978-0-596-00797-3 archiv origin 23 februari 2020 retriev 14 novemb 2015 martelli alex ravenscroft anna ascher david 2005 python cookbook 2nd edit o'reilli medium p. 230 isbn 978-0-596-00797-3 archiv origin 23 februari 2020 retriev 14 novemb 2015 python cookbook 2nd edit o'reilli medium isbn 978-0-596-00797-3 archiv retriev 14 novemb 2015 14 novemb `` python cultur '' ebeab 21 januari 2014 archiv origin 30 januari 2014 `` python cultur '' ebeab 21 januari 2014 archiv origin 30 januari 2014 `` python cultur '' origin b `` transpil python julia use pyjl '' pdf archiv pdf origin 19 novemb 2023 retriev 20 septemb 2023 manual modifi one line code specifi necessari type inform obtain speedup 52.6× make translat julia code 19.5× faster origin python code b b `` transpil python julia use pyjl '' pdf archiv pdf origin 19 novemb 2023 retriev 20 septemb 2023 manual modifi one line code specifi necessari type inform obtain speedup 52.6× make translat julia code 19.5× faster origin python code `` transpil python julia use pyjl '' pdf archiv pdf retriev 20 septemb 2023 20 septemb `` call python '' gener python faq docs.python.org archiv origin 24 octob 2012 retriev 3 januari 2023 `` call python '' gener python faq docs.python.org archiv origin 24 octob 2012 retriev 3 januari 2023 `` call python '' archiv retriev 3 januari 2023 3 januari `` 15 way python power forc web '' archiv origin 11 may 2019 retriev 3 juli 2018 `` 15 way python power forc web '' archiv origin 11 may 2019 retriev 3 juli 2018 `` 15 way python power forc web '' origin retriev 3 juli 2018 3 juli `` pprint – data pretti printer – python 3.11.0 document '' docs.python.org archiv origin 22 januari 2021 retriev 5 novemb 2022. stuff= 'spam 'egg 'lumberjack 'knight 'ni `` pprint – data pretti printer – python 3.11.0 document '' docs.python.org archiv origin 22 januari 2021 retriev 5 novemb 2022. stuff= 'spam 'egg 'lumberjack 'knight 'ni `` pprint – data pretti printer – python 3.11.0 document '' archiv retriev 5 novemb 2022 5 novemb `` code style – hitchhik 's guid python '' docs.python-guide.org archiv origin 27 januari 2021 retriev 20 januari 2021 `` code style – hitchhik 's guid python '' docs.python-guide.org archiv origin 27 januari 2021 retriev 20 januari 2021 `` code style – hitchhik 's guid python '' archiv retriev 20 januari 2021 20 januari `` python good languag begin programm '' gener python faq python softwar foundat archiv origin 24 octob 2012 retriev 21 march 2007 `` python good languag begin programm '' gener python faq python softwar foundat archiv origin 24 octob 2012 retriev 21 march 2007 `` python good languag begin programm '' archiv retriev 21 march 2007 21 march `` myth indent python '' secnetix.d archiv origin 18 februari 2018 retriev 19 april 2011 `` myth indent python '' secnetix.d archiv origin 18 februari 2018 retriev 19 april 2011 `` myth indent python '' origin retriev 19 april 2011 19 april guttag john v. 12 august 2016 introduct comput program use python applic understand data mit press isbn 978-0-262-52962-4 guttag john v. 12 august 2016 introduct comput program use python applic understand data mit press isbn 978-0-262-52962-4 isbn 978-0-262-52962-4 `` pep 8 – style guid python code '' python.org archiv origin 17 april 2019 retriev 26 march 2019 `` pep 8 – style guid python code '' python.org archiv origin 17 april 2019 retriev 26 march 2019 `` pep 8 – style guid python code '' archiv retriev 26 march 2019 26 march `` 8 error except – python 3.12.0a0 document '' docs.python.org archiv origin 9 may 2022 retriev 9 may 2022 `` 8 error except – python 3.12.0a0 document '' docs.python.org archiv origin 9 may 2022 retriev 9 may 2022 `` 8 error except – python 3.12.0a0 document '' archiv retriev 9 may 2022 9 may `` highlight python 2.5 '' python.org archiv origin 4 august 2019 retriev 20 march 2018 `` highlight python 2.5 '' python.org archiv origin 4 august 2019 retriev 20 march 2018 `` highlight python 2.5 '' archiv retriev 20 march 2018 20 march van rossum guido 22 april 2009 `` tail recurs elimin '' neopythonic.blogspot.b archiv origin 19 may 2018 retriev 3 decemb 2012 van rossum guido 22 april 2009 `` tail recurs elimin '' neopythonic.blogspot.b archiv origin 19 may 2018 retriev 3 decemb 2012 `` tail recurs elimin '' archiv retriev 3 decemb 2012 3 decemb van rossum guido 9 februari 2006 `` languag design solv puzzl '' artima forum artima archiv origin 17 januari 2020 retriev 21 march 2007 van rossum guido 9 februari 2006 `` languag design solv puzzl '' artima forum artima archiv origin 17 januari 2020 retriev 21 march 2007 `` languag design solv puzzl '' archiv retriev 21 march 2007 21 march van rossum guido ebi phillip j 10 may 2005 `` pep 342 – coroutin via enhanc gener '' python enhanc propos python softwar foundat archiv origin 29 may 2020 retriev 19 februari 2012 van rossum guido ebi phillip j 10 may 2005 `` pep 342 – coroutin via enhanc gener '' python enhanc propos python softwar foundat archiv origin 29 may 2020 retriev 19 februari 2012 `` pep 342 – coroutin via enhanc gener '' archiv retriev 19 februari 2012 19 februari `` pep 380 '' python.org archiv origin 4 june 2020 retriev 3 decemb 2012 `` pep 380 '' python.org archiv origin 4 june 2020 retriev 3 decemb 2012 `` pep 380 '' archiv retriev 3 decemb 2012 3 decemb `` divis '' python.org archiv origin 20 juli 2006 retriev 30 juli 2014 `` divis '' python.org archiv origin 20 juli 2006 retriev 30 juli 2014 `` divis '' archiv retriev 30 juli 2014 30 juli `` pep 0465 – dedic infix oper matrix multipl '' python.org archiv origin 4 june 2020 retriev 1 januari 2016 `` pep 0465 – dedic infix oper matrix multipl '' python.org archiv origin 4 june 2020 retriev 1 januari 2016 `` pep 0465 – dedic infix oper matrix multipl '' archiv retriev 1 januari 2016 1 januari `` python 3.5.1 releas changelog '' python.org archiv origin 14 may 2020 retriev 1 januari 2016 `` python 3.5.1 releas changelog '' python.org archiv origin 14 may 2020 retriev 1 januari 2016 `` python 3.5.1 releas changelog '' archiv retriev 1 januari 2016 1 januari `` 's new python 3.8 '' archiv origin 8 june 2020 retriev 14 octob 2019 `` 's new python 3.8 '' archiv origin 8 june 2020 retriev 14 octob 2019 `` 's new python 3.8 '' archiv retriev 14 octob 2019 14 octob van rossum guido hetting raymond 7 februari 2003 `` pep 308 – condit express '' python enhanc propos python softwar foundat archiv origin 13 march 2016 retriev 13 juli 2011 van rossum guido hetting raymond 7 februari 2003 `` pep 308 – condit express '' python enhanc propos python softwar foundat archiv origin 13 march 2016 retriev 13 juli 2011 `` pep 308 – condit express '' archiv retriev 13 juli 2011 13 juli `` 4 built-in type – python 3.6.3rc1 document '' python.org archiv origin 14 june 2020 retriev 1 octob 2017 `` 4 built-in type – python 3.6.3rc1 document '' python.org archiv origin 14 june 2020 retriev 1 octob 2017 `` 4 built-in type – python 3.6.3rc1 document '' archiv retriev 1 octob 2017 1 octob `` 5.3 tupl sequenc – python 3.7.1rc2 document '' python.org archiv origin 10 june 2020 retriev 17 octob 2018 `` 5.3 tupl sequenc – python 3.7.1rc2 document '' python.org archiv origin 10 june 2020 retriev 17 octob 2018 `` 5.3 tupl sequenc – python 3.7.1rc2 document '' archiv retriev 17 octob 2018 17 octob b `` pep 498 – liter string interpol '' python.org archiv origin 15 june 2020 retriev 8 march 2017 b b `` pep 498 – liter string interpol '' python.org archiv origin 15 june 2020 retriev 8 march 2017 `` pep 498 – liter string interpol '' archiv retriev 8 march 2017 8 march `` must 'self use explicitli method definit call '' design histori faq python softwar foundat archiv origin 24 octob 2012 retriev 19 februari 2012 `` must 'self use explicitli method definit call '' design histori faq python softwar foundat archiv origin 24 octob 2012 retriev 19 februari 2012 `` must 'self use explicitli method definit call '' archiv retriev 19 februari 2012 19 februari sweigart al 2020 beyond basic stuff python best practic write clean code starch press p. 322 isbn 978-1-59327-966-0 archiv origin 13 august 2021 retriev 7 juli 2021 sweigart al 2020 beyond basic stuff python best practic write clean code starch press p. 322 isbn 978-1-59327-966-0 archiv origin 13 august 2021 retriev 7 juli 2021 beyond basic stuff python best practic write clean code isbn 978-1-59327-966-0 archiv retriev 7 juli 2021 7 juli `` python languag refer section 3.3 new-styl classic class releas 2.7.1 '' archiv origin 26 octob 2012 retriev 12 januari 2011 `` python languag refer section 3.3 new-styl classic class releas 2.7.1 '' archiv origin 26 octob 2012 retriev 12 januari 2011 `` python languag refer section 3.3 new-styl classic class releas 2.7.1 '' origin retriev 12 januari 2011 12 januari `` pep 484 – type hint peps.python.org '' peps.python.org archiv origin 27 novemb 2023 retriev 29 novemb 2023 `` pep 484 – type hint peps.python.org '' peps.python.org archiv origin 27 novemb 2023 retriev 29 novemb 2023 `` pep 484 – type hint peps.python.org '' archiv retriev 29 novemb 2023 29 novemb `` type — support type hint '' python document python softwar foundat archiv origin 21 februari 2020 retriev 22 decemb 2023 `` type — support type hint '' python document python softwar foundat archiv origin 21 februari 2020 retriev 22 decemb 2023 `` type — support type hint '' archiv retriev 22 decemb 2023 22 decemb `` mypi – option static type python '' archiv origin 6 june 2020 retriev 28 januari 2017 `` mypi – option static type python '' archiv origin 6 june 2020 retriev 28 januari 2017 `` mypi – option static type python '' archiv retriev 28 januari 2017 28 januari `` introduct '' mypyc.readthedocs.io archiv origin 22 decemb 2023 retriev 22 decemb 2023 `` introduct '' mypyc.readthedocs.io archiv origin 22 decemb 2023 retriev 22 decemb 2023 `` introduct '' archiv retriev 22 decemb 2023 22 decemb `` 15 float point arithmet issu limit – python 3.8.3 document '' docs.python.org archiv origin 6 june 2020 retriev 6 june 2020 almost machin today novemb 2000 use ieee-754 float point arithmet almost platform map python float ieee-754 `` doubl precis '' `` 15 float point arithmet issu limit – python 3.8.3 document '' docs.python.org archiv origin 6 june 2020 retriev 6 june 2020 almost machin today novemb 2000 use ieee-754 float point arithmet almost platform map python float ieee-754 `` doubl precis '' `` 15 float point arithmet issu limit – python 3.8.3 document '' archiv retriev 6 june 2020 6 june zadka mosh van rossum guido 11 march 2001 `` pep 237 – unifi long integ integ '' python enhanc propos python softwar foundat archiv origin 28 may 2020 retriev 24 septemb 2011 zadka mosh van rossum guido 11 march 2001 `` pep 237 – unifi long integ integ '' python enhanc propos python softwar foundat archiv origin 28 may 2020 retriev 24 septemb 2011 `` pep 237 – unifi long integ integ '' archiv retriev 24 septemb 2011 24 septemb `` built-in type '' archiv origin 14 june 2020 retriev 3 octob 2019 `` built-in type '' archiv origin 14 june 2020 retriev 3 octob 2019 `` built-in type '' archiv retriev 3 octob 2019 3 octob `` pep 465 – dedic infix oper matrix multipl '' python.org archiv origin 29 may 2020 retriev 3 juli 2018 `` pep 465 – dedic infix oper matrix multipl '' python.org archiv origin 29 may 2020 retriev 3 juli 2018 `` pep 465 – dedic infix oper matrix multipl '' archiv retriev 3 juli 2018 3 juli b zadka mosh van rossum guido 11 march 2001 `` pep 238 – chang divis oper '' python enhanc propos python softwar foundat archiv origin 28 may 2020 retriev 23 octob 2013 b b zadka mosh van rossum guido 11 march 2001 `` pep 238 – chang divis oper '' python enhanc propos python softwar foundat archiv origin 28 may 2020 retriev 23 octob 2013 `` pep 238 – chang divis oper '' archiv retriev 23 octob 2013 23 octob `` python 's integ divis floor '' 24 august 2010 archiv origin 5 june 2020 retriev 25 august 2010 `` python 's integ divis floor '' 24 august 2010 archiv origin 5 june 2020 retriev 25 august 2010 `` python 's integ divis floor '' archiv retriev 25 august 2010 25 august `` round '' python standard librari releas 3.2 §2 built-in function archiv origin 25 octob 2012 retriev 14 august 2011 `` round '' python standard librari releas 3.2 §2 built-in function archiv origin 25 octob 2012 retriev 14 august 2011 `` round '' archiv retriev 14 august 2011 14 august `` round '' python standard librari releas 2.7 §2 built-in function archiv origin 27 octob 2012 retriev 14 august 2011 `` round '' python standard librari releas 2.7 §2 built-in function archiv origin 27 octob 2012 retriev 14 august 2011 `` round '' archiv retriev 14 august 2011 14 august beazley david m. 2009 python essenti refer 4th ed. addison-wesley profession p. 66 isbn 9780672329784 beazley david m. 2009 python essenti refer 4th ed. addison-wesley profession p. 66 isbn 9780672329784 python essenti refer python essenti refer 66 isbn 9780672329784 kernighan brian w. ritchi denni m. 1988 c program languag 2nd ed. p. 206 kernighan brian w. ritchi denni m. 1988 c program languag 2nd ed. p. 206 c program languag 206 b batista facundo 17 octob 2003 `` pep 327 – decim data type '' python enhanc propos python softwar foundat archiv origin 4 june 2020 retriev 24 novemb 2008 b b batista facundo 17 octob 2003 `` pep 327 – decim data type '' python enhanc propos python softwar foundat archiv origin 4 june 2020 retriev 24 novemb 2008 `` pep 327 – decim data type '' archiv retriev 24 novemb 2008 24 novemb `` 's new python 2.6 '' python v2.6.9 document 29 octob 2013 archiv origin 23 decemb 2019 retriev 26 septemb 2015 `` 's new python 2.6 '' python v2.6.9 document 29 octob 2013 archiv origin 23 decemb 2019 retriev 26 septemb 2015 `` 's new python 2.6 '' archiv retriev 26 septemb 2015 26 septemb `` 10 reason python rock research reason n't – hoyt koepk '' univers washington depart statist archiv origin 31 may 2020 retriev 3 februari 2019 `` 10 reason python rock research reason n't – hoyt koepk '' univers washington depart statist archiv origin 31 may 2020 retriev 3 februari 2019 `` 10 reason python rock research reason n't – hoyt koepk '' origin retriev 3 februari 2019 3 februari shell scott 17 june 2014 `` introduct python scientif comput '' pdf archiv pdf origin 4 februari 2019 retriev 3 februari 2019 shell scott 17 june 2014 `` introduct python scientif comput '' pdf archiv pdf origin 4 februari 2019 retriev 3 februari 2019 `` introduct python scientif comput '' pdf archiv pdf retriev 3 februari 2019 3 februari piotrowski przemyslaw juli 2006 `` build rapid web develop environ python server page oracl '' oracl technolog network oracl archiv origin 2 april 2019 retriev 12 march 2012 piotrowski przemyslaw juli 2006 `` build rapid web develop environ python server page oracl '' oracl technolog network oracl archiv origin 2 april 2019 retriev 12 march 2012 `` build rapid web develop environ python server page oracl '' archiv retriev 12 march 2012 12 march ebi phillip j 7 decemb 2003 `` pep 333 – python web server gateway interfac v1.0 '' python enhanc propos python softwar foundat archiv origin 14 june 2020 retriev 19 februari 2012 ebi phillip j 7 decemb 2003 `` pep 333 – python web server gateway interfac v1.0 '' python enhanc propos python softwar foundat archiv origin 14 june 2020 retriev 19 februari 2012 `` pep 333 – python web server gateway interfac v1.0 '' archiv retriev 19 februari 2012 19 februari `` pypi '' pypi 17 march 2024 archiv origin 17 march 2024 `` pypi '' pypi 17 march 2024 archiv origin 17 march 2024 `` pypi '' archiv enthought canopi `` canopi '' www.enthought.com archiv origin 15 juli 2017 retriev 20 august 2016 enthought canopi `` canopi '' www.enthought.com archiv origin 15 juli 2017 retriev 20 august 2016 `` canopi '' origin retriev 20 august 2016 20 august `` pep 7 – style guid c code peps.python.org '' peps.python.org archiv origin 24 april 2022 retriev 28 april 2022 `` pep 7 – style guid c code peps.python.org '' peps.python.org archiv origin 24 april 2022 retriev 28 april 2022 `` pep 7 – style guid c code peps.python.org '' archiv retriev 28 april 2022 28 april `` 4 build c c++ extens – python 3.9.2 document '' docs.python.org archiv origin 3 march 2021 retriev 1 march 2021 `` 4 build c c++ extens – python 3.9.2 document '' docs.python.org archiv origin 3 march 2021 retriev 1 march 2021 `` 4 build c c++ extens – python 3.9.2 document '' archiv retriev 1 march 2021 1 march van rossum guido 5 june 2001 `` pep 7 – style guid c code '' python enhanc propos python softwar foundat archiv origin 1 june 2020 retriev 24 novemb 2008 van rossum guido 5 june 2001 `` pep 7 – style guid c code '' python enhanc propos python softwar foundat archiv origin 1 june 2020 retriev 24 novemb 2008 `` pep 7 – style guid c code '' archiv retriev 24 novemb 2008 24 novemb `` cpython byte code '' docs.python.org archiv origin 5 june 2020 retriev 16 februari 2016 `` cpython byte code '' docs.python.org archiv origin 5 june 2020 retriev 16 februari 2016 `` cpython byte code '' archiv retriev 16 februari 2016 16 februari `` python 2.5 intern '' pdf archiv pdf origin 6 august 2012 retriev 19 april 2011 `` python 2.5 intern '' pdf archiv pdf origin 6 august 2012 retriev 19 april 2011 `` python 2.5 intern '' pdf archiv pdf retriev 19 april 2011 19 april `` changelog – python 3.9.0 document '' docs.python.org archiv origin 7 februari 2021 retriev 8 februari 2021 `` changelog – python 3.9.0 document '' docs.python.org archiv origin 7 februari 2021 retriev 8 februari 2021 `` changelog – python 3.9.0 document '' archiv retriev 8 februari 2021 8 februari `` download python '' python.org archiv origin 8 decemb 2020 retriev 13 decemb 2020 `` download python '' python.org archiv origin 8 decemb 2020 retriev 13 decemb 2020 `` download python '' archiv retriev 13 decemb 2020 13 decemb `` histori vmspython '' www.vmspython.org archiv origin 2 decemb 2020 retriev 4 decemb 2020 `` histori vmspython '' www.vmspython.org archiv origin 2 decemb 2020 retriev 4 decemb 2020 `` histori vmspython '' archiv retriev 4 decemb 2020 4 decemb `` interview guido van rossum '' oreilly.com archiv origin 16 juli 2014 retriev 24 novemb 2008 `` interview guido van rossum '' oreilly.com archiv origin 16 juli 2014 retriev 24 novemb 2008 `` interview guido van rossum '' archiv retriev 24 novemb 2008 24 novemb `` download python platform '' python.org archiv origin 27 novemb 2020 retriev 4 decemb 2020 `` download python platform '' python.org archiv origin 27 novemb 2020 retriev 4 decemb 2020 `` download python platform '' archiv retriev 4 decemb 2020 4 decemb `` pypi compat '' pypy.org archiv origin 6 june 2020 retriev 3 decemb 2012 `` pypi compat '' pypy.org archiv origin 6 june 2020 retriev 3 decemb 2012 `` pypi compat '' archiv retriev 3 decemb 2012 3 decemb team pypi 28 decemb 2019 `` download instal '' pypi archiv origin 8 januari 2022 retriev 8 januari 2022 team pypi 28 decemb 2019 `` download instal '' pypi archiv origin 8 januari 2022 retriev 8 januari 2022 `` download instal '' archiv retriev 8 januari 2022 8 januari `` speed comparison cpython pypi '' speed.pypy.org archiv origin 10 may 2021 retriev 3 decemb 2012 `` speed comparison cpython pypi '' speed.pypy.org archiv origin 10 may 2021 retriev 3 decemb 2012 `` speed comparison cpython pypi '' archiv retriev 3 decemb 2012 3 decemb `` codon differ python '' archiv origin 25 may 2023 retriev 28 august 2023 `` codon differ python '' archiv origin 25 may 2023 retriev 28 august 2023 `` codon differ python '' archiv retriev 28 august 2023 28 august lawson lorain 14 march 2023 `` mit-creat compil speed python code '' new stack archiv origin 6 april 2023 retriev 28 august 2023 lawson lorain 14 march 2023 `` mit-creat compil speed python code '' new stack archiv origin 6 april 2023 retriev 28 august 2023 `` mit-creat compil speed python code '' archiv retriev 28 august 2023 28 august `` application-level stackless featur – pypi 2.0.2 document '' doc.pypy.org archiv origin 4 june 2020 retriev 17 juli 2013 `` application-level stackless featur – pypi 2.0.2 document '' doc.pypy.org archiv origin 4 june 2020 retriev 17 juli 2013 `` application-level stackless featur – pypi 2.0.2 document '' archiv retriev 17 juli 2013 17 juli `` python-for-ev3 '' lego educ archiv origin 7 june 2020 retriev 17 april 2019 `` python-for-ev3 '' lego educ archiv origin 7 june 2020 retriev 17 april 2019 `` python-for-ev3 '' archiv retriev 17 april 2019 17 april yegulalp serdar 29 octob 2020 `` pyston return dead speed python '' infoworld archiv origin 27 januari 2021 retriev 26 januari 2021 yegulalp serdar 29 octob 2020 `` pyston return dead speed python '' infoworld archiv origin 27 januari 2021 retriev 26 januari 2021 `` pyston return dead speed python '' infoworld archiv retriev 26 januari 2021 26 januari `` cinder instagram 's performance-ori fork cpython '' github archiv origin 4 may 2021 retriev 4 may 2021 `` cinder instagram 's performance-ori fork cpython '' github archiv origin 4 may 2021 retriev 4 may 2021 `` cinder instagram 's performance-ori fork cpython '' github archiv retriev 4 may 2021 4 may aroca rafael 7 august 2021 `` snek lang feel like python arduino '' yet anoth technolog blog archiv origin 5 januari 2024 retriev 4 januari 2024 aroca rafael 7 august 2021 `` snek lang feel like python arduino '' yet anoth technolog blog archiv origin 5 januari 2024 retriev 4 januari 2024 `` snek lang feel like python arduino '' archiv retriev 4 januari 2024 4 januari aufranc cnxsoft jean-luc 16 januari 2020 `` snekboard control lego power function circuitpython snek program languag crowdfund – cnx softwar '' cnx softwar – embed system news archiv origin 5 januari 2024 retriev 4 januari 2024 aufranc cnxsoft jean-luc 16 januari 2020 `` snekboard control lego power function circuitpython snek program languag crowdfund – cnx softwar '' cnx softwar – embed system news archiv origin 5 januari 2024 retriev 4 januari 2024 `` snekboard control lego power function circuitpython snek program languag crowdfund – cnx softwar '' archiv retriev 4 januari 2024 4 januari kennedi mkennedi michael `` readi find 're git famou '' pythonbytes.fm archiv origin 5 januari 2024 retriev 4 januari 2024 kennedi mkennedi michael `` readi find 're git famou '' pythonbytes.fm archiv origin 5 januari 2024 retriev 4 januari 2024 `` readi find 're git famou '' archiv retriev 4 januari 2024 4 januari packard keith 20 decemb 2022 `` snek program languag python-inspir embed comput languag '' pdf archiv pdf origin 4 januari 2024 retriev 4 januari 2024 packard keith 20 decemb 2022 `` snek program languag python-inspir embed comput languag '' pdf archiv pdf origin 4 januari 2024 retriev 4 januari 2024 `` snek program languag python-inspir embed comput languag '' pdf archiv pdf retriev 4 januari 2024 4 januari `` plan optim python '' googl project host 15 decemb 2009 archiv origin 11 april 2016 retriev 24 septemb 2011 `` plan optim python '' googl project host 15 decemb 2009 archiv origin 11 april 2016 retriev 24 septemb 2011 `` plan optim python '' archiv retriev 24 septemb 2011 24 septemb `` python nokia n900 '' stochast geometri 29 april 2010 archiv origin 20 june 2019 retriev 9 juli 2015 `` python nokia n900 '' stochast geometri 29 april 2010 archiv origin 20 june 2019 retriev 9 juli 2015 `` python nokia n900 '' archiv retriev 9 juli 2015 9 juli `` brython '' brython.info archiv origin 3 august 2018 retriev 21 januari 2021 `` brython '' brython.info archiv origin 3 august 2018 retriev 21 januari 2021 `` brython '' archiv retriev 21 januari 2021 21 januari `` transcrypt – python browser '' transcrypt.org archiv origin 19 august 2018 retriev 22 decemb 2020 `` transcrypt – python browser '' transcrypt.org archiv origin 19 august 2018 retriev 22 decemb 2020 `` transcrypt – python browser '' archiv retriev 22 decemb 2020 22 decemb `` transcrypt anatomi python javascript compil '' infoq archiv origin 5 decemb 2020 retriev 20 januari 2021 `` transcrypt anatomi python javascript compil '' infoq archiv origin 5 decemb 2020 retriev 20 januari 2021 `` transcrypt anatomi python javascript compil '' archiv retriev 20 januari 2021 20 januari `` nuitka home nuitka home '' nuitka.net archiv origin 30 may 2020 retriev 18 august 2017 `` nuitka home nuitka home '' nuitka.net archiv origin 30 may 2020 retriev 18 august 2017 `` nuitka home nuitka home '' archiv retriev 18 august 2017 18 august guelton serg brunet pierrick amini mehdi merlini adrien corbillon xavier raynaud alan 16 march 2015 `` pythran enabl static optim scientif python program '' comput scienc discoveri 8 1 iop publish 014001 bibcode:2015c .... 8a4001g doi:10.1088/1749-4680/8/1/014001 issn 1749-4699 guelton serg brunet pierrick amini mehdi merlini adrien corbillon xavier raynaud alan 16 march 2015 `` pythran enabl static optim scientif python program '' comput scienc discoveri 8 1 iop publish 014001 bibcode:2015c .... 8a4001g doi:10.1088/1749-4680/8/1/014001 issn 1749-4699 `` pythran enabl static optim scientif python program '' bibcod 2015c .... 8a4001g doi 10.1088/1749-4680/8/1/014001 10.1088/1749-4680/8/1/014001 issn 1749-4699 `` python → 11l → c++ transpil '' archiv origin 24 septemb 2022 retriev 17 juli 2022 `` python → 11l → c++ transpil '' archiv origin 24 septemb 2022 retriev 17 juli 2022 `` python → 11l → c++ transpil '' archiv retriev 17 juli 2022 17 juli `` google/grumpi '' 10 april 2020 archiv origin 15 april 2020 retriev 25 march 2020 – via github `` google/grumpi '' 10 april 2020 archiv origin 15 april 2020 retriev 25 march 2020 – via github `` google/grumpi '' archiv retriev 25 march 2020 25 march `` project '' opensource.googl archiv origin 24 april 2020 retriev 25 march 2020 `` project '' opensource.googl archiv origin 24 april 2020 retriev 25 march 2020 `` project '' archiv retriev 25 march 2020 25 march francisco thoma claburn san `` googl 's grumpi code make python go '' www.theregister.com archiv origin 7 march 2021 retriev 20 januari 2021 francisco thoma claburn san `` googl 's grumpi code make python go '' www.theregister.com archiv origin 7 march 2021 retriev 20 januari 2021 `` googl 's grumpi code make python go '' archiv retriev 20 januari 2021 20 januari `` github – ironlanguages/ironpython3 implement python 3.x .net framework built top dynam languag runtim '' github archiv origin 28 septemb 2021 `` github – ironlanguages/ironpython3 implement python 3.x .net framework built top dynam languag runtim '' github archiv origin 28 septemb 2021 `` github – ironlanguages/ironpython3 implement python 3.x .net framework built top dynam languag runtim '' github archiv `` ironpython.net '' ironpython.net archiv origin 17 april 2021 `` ironpython.net '' ironpython.net archiv origin 17 april 2021 `` ironpython.net '' archiv `` jython faq '' www.jython.org archiv origin 22 april 2021 retriev 22 april 2021 `` jython faq '' www.jython.org archiv origin 22 april 2021 retriev 22 april 2021 `` jython faq '' archiv retriev 22 april 2021 22 april murri riccardo 2013 perform python runtim non-numer scientif code european confer python scienc euroscipi arxiv:1404.6388 bibcode:2014arxiv1404.6388m murri riccardo 2013 perform python runtim non-numer scientif code european confer python scienc euroscipi arxiv:1404.6388 bibcode:2014arxiv1404.6388m arxiv 1404.6388 1404.6388 bibcod 2014arxiv1404.6388m `` comput languag benchmark game '' archiv origin 14 june 2020 retriev 30 april 2020 `` comput languag benchmark game '' archiv origin 14 june 2020 retriev 30 april 2020 `` comput languag benchmark game '' archiv retriev 30 april 2020 30 april b warsaw barri hylton jeremi goodger david 13 june 2000 `` pep 1 – pep purpos guidelin '' python enhanc propos python softwar foundat archiv origin 6 june 2020 retriev 19 april 2011 b b warsaw barri hylton jeremi goodger david 13 june 2000 `` pep 1 – pep purpos guidelin '' python enhanc propos python softwar foundat archiv origin 6 june 2020 retriev 19 april 2011 `` pep 1 – pep purpos guidelin '' archiv retriev 19 april 2011 19 april `` pep 8 – style guid python code '' python.org archiv origin 17 april 2019 retriev 26 march 2019 `` pep 8 – style guid python code '' python.org archiv origin 17 april 2019 retriev 26 march 2019 `` pep 8 – style guid python code '' archiv retriev 26 march 2019 26 march cannon brett `` guido guy mail list python develop '' python.org python softwar foundat archiv origin 1 june 2009 retriev 27 june 2009 cannon brett `` guido guy mail list python develop '' python.org python softwar foundat archiv origin 1 june 2009 retriev 27 june 2009 `` guido guy mail list python develop '' origin retriev 27 june 2009 27 june `` move python 's bug github lwn.net '' archiv origin 2 octob 2022 retriev 2 octob 2022 `` move python 's bug github lwn.net '' archiv origin 2 octob 2022 retriev 2 octob 2022 `` move python 's bug github lwn.net '' archiv retriev 2 octob 2022 2 octob `` python develop 's guid – python develop 's guid '' devguide.python.org archiv origin 9 novemb 2020 retriev 17 decemb 2019 `` python develop 's guid – python develop 's guid '' devguide.python.org archiv origin 9 novemb 2020 retriev 17 decemb 2019 `` python develop 's guid – python develop 's guid '' archiv retriev 17 decemb 2019 17 decemb hugh owen 24 may 2021 `` program languag python 4.0 might never arriv accord creator '' techrepubl archiv origin 14 juli 2022 retriev 16 may 2022 hugh owen 24 may 2021 `` program languag python 4.0 might never arriv accord creator '' techrepubl archiv origin 14 juli 2022 retriev 16 may 2022 `` program languag python 4.0 might never arriv accord creator '' archiv retriev 16 may 2022 16 may `` pep 602 – annual releas cycl python '' python.org archiv origin 14 june 2020 retriev 6 novemb 2019 `` pep 602 – annual releas cycl python '' python.org archiv origin 14 june 2020 retriev 6 novemb 2019 `` pep 602 – annual releas cycl python '' archiv retriev 6 novemb 2019 6 novemb `` chang python releas cadenc lwn.net '' lwn.net archiv origin 6 novemb 2019 retriev 6 novemb 2019 `` chang python releas cadenc lwn.net '' lwn.net archiv origin 6 novemb 2019 retriev 6 novemb 2019 `` chang python releas cadenc lwn.net '' archiv retriev 6 novemb 2019 6 novemb norwitz neal 8 april 2002 `` python-dev releas schedul stabil chang '' archiv origin 15 decemb 2018 retriev 27 june 2009 norwitz neal 8 april 2002 `` python-dev releas schedul stabil chang '' archiv origin 15 decemb 2018 retriev 27 june 2009 `` python-dev releas schedul stabil chang '' archiv retriev 27 june 2009 27 june b aahz baxter anthoni 15 march 2001 `` pep 6 – bug fix releas '' python enhanc propos python softwar foundat archiv origin 5 june 2020 retriev 27 june 2009 b b aahz baxter anthoni 15 march 2001 `` pep 6 – bug fix releas '' python enhanc propos python softwar foundat archiv origin 5 june 2020 retriev 27 june 2009 `` pep 6 – bug fix releas '' archiv retriev 27 june 2009 27 june `` python buildbot '' python develop 's guid python softwar foundat archiv origin 5 june 2020 retriev 24 septemb 2011 `` python buildbot '' python develop 's guid python softwar foundat archiv origin 5 june 2020 retriev 24 septemb 2011 `` python buildbot '' archiv retriev 24 septemb 2011 24 septemb `` 1 extend python c c++ – python 3.9.1 document '' docs.python.org archiv origin 23 june 2020 retriev 14 februari 2021 `` 1 extend python c c++ – python 3.9.1 document '' docs.python.org archiv origin 23 june 2020 retriev 14 februari 2021 `` 1 extend python c c++ – python 3.9.1 document '' archiv retriev 14 februari 2021 14 februari `` pep 623 – remov wstr unicod '' python.org archiv origin 5 march 2021 retriev 14 februari 2021 `` pep 623 – remov wstr unicod '' python.org archiv origin 5 march 2021 retriev 14 februari 2021 `` pep 623 – remov wstr unicod '' archiv retriev 14 februari 2021 14 februari `` pep 634 – structur pattern match specif '' python.org archiv origin 6 may 2021 retriev 14 februari 2021 `` pep 634 – structur pattern match specif '' python.org archiv origin 6 may 2021 retriev 14 februari 2021 `` pep 634 – structur pattern match specif '' archiv retriev 14 februari 2021 14 februari `` document tool '' python.org archiv origin 11 novemb 2020 retriev 22 march 2021 `` document tool '' python.org archiv origin 11 novemb 2020 retriev 22 march 2021 `` document tool '' archiv retriev 22 march 2021 22 march b `` whet appetit '' python tutori python softwar foundat archiv origin 26 octob 2012 retriev 20 februari 2012 b b `` whet appetit '' python tutori python softwar foundat archiv origin 26 octob 2012 retriev 20 februari 2012 `` whet appetit '' archiv retriev 20 februari 2012 20 februari `` python use el return block '' stack overflow stack exchang 17 februari 2011 archiv origin 20 june 2019 retriev 6 may 2011 `` python use el return block '' stack overflow stack exchang 17 februari 2011 archiv origin 20 june 2019 retriev 6 may 2011 `` python use el return block '' stack overflow archiv retriev 6 may 2011 6 may lutz mark 2009 learn python power object-ori program o'reilli medium inc. p. 17 isbn 9781449379322 archiv origin 17 juli 2017 retriev 9 may 2017 lutz mark 2009 learn python power object-ori program o'reilli medium inc. p. 17 isbn 9781449379322 archiv origin 17 juli 2017 retriev 9 may 2017 learn python power object-ori program isbn 9781449379322 archiv retriev 9 may 2017 9 may fehili chri 2002 python peachpit press p. xv isbn 9780201748840 archiv origin 17 juli 2017 retriev 9 may 2017 fehili chri 2002 python peachpit press p. xv isbn 9780201748840 archiv origin 17 juli 2017 retriev 9 may 2017 python isbn 9780201748840 archiv retriev 9 may 2017 9 may lubanov bill 2014 introduc python sebastopol ca o'reilli medium p. 305 isbn 978-1-4493-5936-2 retriev 31 juli 2023 lubanov bill 2014 introduc python sebastopol ca o'reilli medium p. 305 isbn 978-1-4493-5936-2 retriev 31 juli 2023 introduc python isbn 978-1-4493-5936-2 retriev 31 juli 2023 31 juli blake troy 18 januari 2021 `` tiob index januari 2021 '' technolog news inform seniordba archiv origin 21 march 2021 retriev 26 februari 2021 blake troy 18 januari 2021 `` tiob index januari 2021 '' technolog news inform seniordba archiv origin 21 march 2021 retriev 26 februari 2021 `` tiob index januari 2021 '' archiv retriev 26 februari 2021 26 februari `` quot python '' python softwar foundat archiv origin 3 june 2020 retriev 8 januari 2012 `` quot python '' python softwar foundat archiv origin 3 june 2020 retriev 8 januari 2012 `` quot python '' archiv retriev 8 januari 2012 8 januari `` organ use python '' python softwar foundat archiv origin 21 august 2018 retriev 15 januari 2009 `` organ use python '' python softwar foundat archiv origin 21 august 2018 retriev 15 januari 2009 `` organ use python '' archiv retriev 15 januari 2009 15 januari `` python holi grail program '' cern bulletin 31/2006 cern public 31 juli 2006 archiv origin 15 januari 2013 retriev 11 februari 2012 `` python holi grail program '' cern bulletin 31/2006 cern public 31 juli 2006 archiv origin 15 januari 2013 retriev 11 februari 2012 `` python holi grail program '' archiv retriev 11 februari 2012 11 februari shafer daniel g. 17 januari 2003 `` python streamlin space shuttl mission design '' python softwar foundat archiv origin 5 june 2020 retriev 24 novemb 2008 shafer daniel g. 17 januari 2003 `` python streamlin space shuttl mission design '' python softwar foundat archiv origin 5 june 2020 retriev 24 novemb 2008 `` python streamlin space shuttl mission design '' archiv retriev 24 novemb 2008 24 novemb `` tornado facebook 's real-tim web framework python – facebook develop '' facebook develop archiv origin 19 februari 2019 retriev 19 june 2018 `` tornado facebook 's real-tim web framework python – facebook develop '' facebook develop archiv origin 19 februari 2019 retriev 19 june 2018 `` tornado facebook 's real-tim web framework python – facebook develop '' archiv retriev 19 june 2018 19 june `` power instagram hundr instanc dozen technolog '' instagram engin 11 decemb 2016 archiv origin 15 june 2020 retriev 27 may 2019 `` power instagram hundr instanc dozen technolog '' instagram engin 11 decemb 2016 archiv origin 15 june 2020 retriev 27 may 2019 `` power instagram hundr instanc dozen technolog '' archiv retriev 27 may 2019 27 may `` use python spotifi '' spotifi lab 20 march 2013 archiv origin 10 june 2020 retriev 25 juli 2018 `` use python spotifi '' spotifi lab 20 march 2013 archiv origin 10 june 2020 retriev 25 juli 2018 `` use python spotifi '' archiv retriev 25 juli 2018 25 juli fortenberri tim 17 januari 2003 `` industri light magic run python '' python softwar foundat archiv origin 6 june 2020 retriev 11 februari 2012 fortenberri tim 17 januari 2003 `` industri light magic run python '' python softwar foundat archiv origin 6 june 2020 retriev 11 februari 2012 `` industri light magic run python '' archiv retriev 11 februari 2012 11 februari taft darryl k. 5 march 2007 `` python slither system '' eweek.com ziff davi hold archiv origin 13 august 2021 retriev 24 septemb 2011 taft darryl k. 5 march 2007 `` python slither system '' eweek.com ziff davi hold archiv origin 13 august 2021 retriev 24 septemb 2011 `` python slither system '' archiv retriev 24 septemb 2011 24 septemb github – reddit-archive/reddit histor code reddit.com. reddit archiv archiv origin 1 june 2020 retriev 20 march 2019 github – reddit-archive/reddit histor code reddit.com. reddit archiv archiv origin 1 june 2020 retriev 20 march 2019 github – reddit-archive/reddit histor code reddit.com archiv retriev 20 march 2019 20 march `` real time commun scale elixir discord '' 8 octob 2020 `` real time commun scale elixir discord '' 8 octob 2020 `` real time commun scale elixir discord '' `` program languag baidu built '' 5 juli 2018 `` program languag baidu built '' 5 juli 2018 `` program languag baidu built '' `` usag statist market share python websit '' 2012 archiv origin 13 august 2021 retriev 18 decemb 2012 `` usag statist market share python websit '' 2012 archiv origin 13 august 2021 retriev 18 decemb 2012 `` usag statist market share python websit '' archiv retriev 18 decemb 2012 18 decemb oliph travi 2007 `` python scientif comput '' comput scienc engin 9 3 10–20 bibcode:2007cs ..... 9c .. 10o citeseerx 10.1.1.474.6460. doi:10.1109/mcse.2007.58 issn 1521-9615 s2cid 206457124 archiv origin 15 june 2020 retriev 10 april 2015 oliph travi 2007 `` python scientif comput '' comput scienc engin 9 3 10–20 bibcode:2007cs ..... 9c .. 10o citeseerx 10.1.1.474.6460. doi:10.1109/mcse.2007.58 issn 1521-9615 s2cid 206457124 archiv origin 15 june 2020 retriev 10 april 2015 `` python scientif comput '' 10– bibcod 2007cse ..... 9c .. 10o citeseerx 10.1.1.474.6460 10.1.1.474.6460 doi 10.1109/mcse.2007.58 issn 1521-9615 s2cid 206457124 archiv retriev 10 april 2015 10 april millman k. jarrod aivazi michael 2011 `` python scientist engin '' comput scienc engin 13 2 9–12 bibcode:2011cs .... 13b ... 9m doi:10.1109/mcse.2011.36 archiv origin 19 februari 2019 retriev 7 juli 2014 millman k. jarrod aivazi michael 2011 `` python scientist engin '' comput scienc engin 13 2 9–12 bibcode:2011cs .... 13b ... 9m doi:10.1109/mcse.2011.36 archiv origin 19 februari 2019 retriev 7 juli 2014 `` python scientist engin '' 9– bibcod 2011cse .... 13b ... 9m doi 10.1109/mcse.2011.36 archiv retriev 7 juli 2014 7 juli scienc educ sagemath innov comput scienc educ archiv origin 15 june 2020 retriev 22 april 2019 scienc educ sagemath innov comput scienc educ archiv origin 15 june 2020 retriev 22 april 2019 scienc educ sagemath origin retriev 22 april 2019 22 april `` opencv opencv-python tutori '' docs.opencv.org archiv origin 23 septemb 2020 retriev 14 septemb 2020 `` opencv opencv-python tutori '' docs.opencv.org archiv origin 23 septemb 2020 retriev 14 septemb 2020 `` opencv opencv-python tutori '' archiv retriev 14 septemb 2020 14 septemb dean jeff monga rajat et al 9 novemb 2015 `` tensorflow large-scal machin learn heterogen system '' pdf tensorflow.org googl research archiv pdf origin 20 novemb 2015 retriev 10 novemb 2015 dean jeff monga rajat et al 9 novemb 2015 `` tensorflow large-scal machin learn heterogen system '' pdf tensorflow.org googl research archiv pdf origin 20 novemb 2015 retriev 10 novemb 2015 dean jeff `` tensorflow large-scal machin learn heterogen system '' pdf archiv pdf retriev 10 novemb 2015 10 novemb piatetski gregori `` python eat away r top softwar analyt data scienc machin learn 2018 trend analysi '' kdnugget archiv origin 15 novemb 2019 retriev 30 may 2018 piatetski gregori `` python eat away r top softwar analyt data scienc machin learn 2018 trend analysi '' kdnugget archiv origin 15 novemb 2019 retriev 30 may 2018 `` python eat away r top softwar analyt data scienc machin learn 2018 trend analysi '' archiv retriev 30 may 2018 30 may `` use scikit-learn – scikit-learn 0.20.1 document '' scikit-learn.org archiv origin 6 may 2020 retriev 30 novemb 2018 `` use scikit-learn – scikit-learn 0.20.1 document '' scikit-learn.org archiv origin 6 may 2020 retriev 30 novemb 2018 `` use scikit-learn – scikit-learn 0.20.1 document '' archiv retriev 30 novemb 2018 30 novemb jouppi norm `` googl supercharg machin learn task tpu custom chip '' googl cloud platform blog archiv origin 18 may 2016 retriev 19 may 2016 jouppi norm `` googl supercharg machin learn task tpu custom chip '' googl cloud platform blog archiv origin 18 may 2016 retriev 19 may 2016 jouppi norm `` googl supercharg machin learn task tpu custom chip '' archiv retriev 19 may 2016 19 may de raedt luc kimmig angelika 2015 `` probabilist logic program concept '' machin learn 100 1 5–47 doi:10.1007/s10994-015-5494-z s2cid 3166992 de raedt luc kimmig angelika 2015 `` probabilist logic program concept '' machin learn 100 1 5–47 doi:10.1007/s10994-015-5494-z s2cid 3166992 `` probabilist logic program concept '' 5– doi 10.1007/s10994-015-5494-z 10.1007/s10994-015-5494-z s2cid 3166992 `` natur languag toolkit – nltk 3.5b1 document '' www.nltk.org archiv origin 13 june 2020 retriev 10 april 2020 `` natur languag toolkit – nltk 3.5b1 document '' www.nltk.org archiv origin 13 june 2020 retriev 10 april 2020 `` natur languag toolkit – nltk 3.5b1 document '' archiv retriev 10 april 2020 10 april andersen c. swift t. 2023 janu system bridg new prolog applic prolog next 50 year pp 93–104 cham springer natur switzerland andersen c. swift t. 2023 janu system bridg new prolog applic prolog next 50 year pp 93–104 cham springer natur switzerland `` swi-prolog python interfac '' archiv origin 15 march 2024 retriev 15 march 2024 `` swi-prolog python interfac '' archiv origin 15 march 2024 retriev 15 march 2024 `` swi-prolog python interfac '' archiv retriev 15 march 2024 15 march tarau p. 2023 reflect autom learnabl express logic-bas program languag prolog next 50 year pp 359–371 cham springer natur switzerland tarau p. 2023 reflect autom learnabl express logic-bas program languag prolog next 50 year pp 359–371 cham springer natur switzerland `` tkinter — python interfac tcl/tk '' archiv origin 18 octob 2012 retriev 9 june 2023 `` tkinter — python interfac tcl/tk '' archiv origin 18 octob 2012 retriev 9 june 2023 `` tkinter — python interfac tcl/tk '' archiv retriev 9 june 2023 9 june `` python tkinter tutori '' 3 june 2020 archiv origin 9 june 2023 retriev 9 june 2023 `` python tkinter tutori '' 3 june 2020 archiv origin 9 june 2023 retriev 9 june 2023 `` python tkinter tutori '' archiv retriev 9 june 2023 9 june `` instal gimp window – frequent ask question '' 26 juli 2013 archiv origin 17 juli 2013 retriev 26 juli 2013 `` instal gimp window – frequent ask question '' 26 juli 2013 archiv origin 17 juli 2013 retriev 26 juli 2013 `` instal gimp window – frequent ask question '' origin retriev 26 juli 2013 26 juli `` jasc psp9compon '' archiv origin 19 march 2008 `` jasc psp9compon '' archiv origin 19 march 2008 `` jasc psp9compon '' origin `` get start write geoprocess script '' arcgi desktop help 9.2 environment system research institut 17 novemb 2006 archiv origin 5 june 2020 retriev 11 februari 2012 `` get start write geoprocess script '' arcgi desktop help 9.2 environment system research institut 17 novemb 2006 archiv origin 5 june 2020 retriev 11 februari 2012 `` get start write geoprocess script '' archiv retriev 11 februari 2012 11 februari ccp porkbelli 24 august 2010 `` stackless python 2.7 '' eve commun dev blog ccp game archiv origin 11 januari 2014 retriev 11 januari 2014 may know eve core program languag known stackless python ccp porkbelli 24 august 2010 `` stackless python 2.7 '' eve commun dev blog ccp game archiv origin 11 januari 2014 retriev 11 januari 2014 may know eve core program languag known stackless python `` stackless python 2.7 '' ccp game archiv retriev 11 januari 2014 11 januari caudil barri 20 septemb 2005 `` mod sid meier 's civil iv '' sid meier 's civil iv develop blog firaxi game archiv origin 2 decemb 2010. creat three level tool ... next level offer python xml support let modder experi manipul game world everyth caudil barri 20 septemb 2005 `` mod sid meier 's civil iv '' sid meier 's civil iv develop blog firaxi game archiv origin 2 decemb 2010. creat three level tool ... next level offer python xml support let modder experi manipul game world everyth `` mod sid meier 's civil iv '' firaxi game origin `` python languag guid v1.0 '' googl document list data api v1.0 archiv origin 15 juli 2010 `` python languag guid v1.0 '' googl document list data api v1.0 archiv origin 15 juli 2010 `` python languag guid v1.0 '' origin `` python setup usag '' python softwar foundat archiv origin 17 june 2020 retriev 10 januari 2020 `` python setup usag '' python softwar foundat archiv origin 17 june 2020 retriev 10 januari 2020 `` python setup usag '' archiv retriev 10 januari 2020 10 januari `` immun know 're secur '' archiv origin 16 februari 2009 `` immun know 're secur '' archiv origin 16 februari 2009 `` immun know 're secur '' origin `` core secur '' core secur archiv origin 9 june 2020 retriev 10 april 2020 `` core secur '' core secur archiv origin 9 june 2020 retriev 10 april 2020 `` core secur '' archiv retriev 10 april 2020 10 april `` sugar '' sugar lab archiv origin 9 januari 2009 retriev 11 februari 2012 `` sugar '' sugar lab archiv origin 9 januari 2009 retriev 11 februari 2012 `` sugar '' archiv retriev 11 februari 2012 11 februari `` 4.0 new featur fix '' libreoffice.org document foundat 2013 archiv origin 9 februari 2014 retriev 25 februari 2013 `` 4.0 new featur fix '' libreoffice.org document foundat 2013 archiv origin 9 februari 2014 retriev 25 februari 2013 `` 4.0 new featur fix '' document foundat archiv retriev 25 februari 2013 25 februari `` gotcha python user '' boo.codehaus.org codehau foundat archiv origin 11 decemb 2008 retriev 24 novemb 2008 `` gotcha python user '' boo.codehaus.org codehau foundat archiv origin 11 decemb 2008 retriev 24 novemb 2008 `` gotcha python user '' origin retriev 24 novemb 2008 24 novemb esterbrook charl `` acknowledg '' cobra-language.com cobra languag archiv origin 8 februari 2008 retriev 7 april 2010 esterbrook charl `` acknowledg '' cobra-language.com cobra languag archiv origin 8 februari 2008 retriev 7 april 2010 `` acknowledg '' origin retriev 7 april 2010 7 april `` propos iter gener es4 wiki '' wiki.ecmascript.org archiv origin 20 octob 2007 retriev 24 novemb 2008 `` propos iter gener es4 wiki '' wiki.ecmascript.org archiv origin 20 octob 2007 retriev 24 novemb 2008 `` propos iter gener es4 wiki '' origin retriev 24 novemb 2008 24 novemb `` frequent ask question '' godot engin document archiv origin 28 april 2021 retriev 10 may 2021 `` frequent ask question '' godot engin document archiv origin 28 april 2021 retriev 10 may 2021 `` frequent ask question '' archiv retriev 10 may 2021 10 may kincaid jason 10 novemb 2009 `` googl 's go new program languag 's python meet c++ '' techcrunch archiv origin 18 januari 2010 retriev 29 januari 2010 kincaid jason 10 novemb 2009 `` googl 's go new program languag 's python meet c++ '' techcrunch archiv origin 18 januari 2010 retriev 29 januari 2010 `` googl 's go new program languag 's python meet c++ '' archiv retriev 29 januari 2010 29 januari strachan jame 29 august 2003 `` groovi – birth new dynam languag java platform '' archiv origin 5 april 2007 retriev 11 june 2007 strachan jame 29 august 2003 `` groovi – birth new dynam languag java platform '' archiv origin 5 april 2007 retriev 11 june 2007 `` groovi – birth new dynam languag java platform '' origin retriev 11 june 2007 11 june `` modular doc – mojo '' docs.modular.com archiv origin 5 may 2023 retriev 5 may 2023 mojo member python famili .. embrac python massiv simplifi design effort syntax alreadi specifi .. decid right long-term goal mojo provid superset python i.e compat exist program embrac cpython immedi long-tail ecosystem enabl python programm expect hope mojo immedi familiar also provid new tool develop systems-level code enabl thing python fall back c c++ `` modular doc – mojo '' docs.modular.com archiv origin 5 may 2023 retriev 5 may 2023 mojo member python famili .. embrac python massiv simplifi design effort syntax alreadi specifi .. decid right long-term goal mojo provid superset python i.e compat exist program embrac cpython immedi long-tail ecosystem enabl python programm expect hope mojo immedi familiar also provid new tool develop systems-level code enabl thing python fall back c c++ `` modular doc – mojo '' archiv retriev 5 may 2023 5 may spencer michael 4 may 2023 `` mojo program languag '' datasciencelearningcenter.substack.com archiv origin 5 may 2023 retriev 5 may 2023 spencer michael 4 may 2023 `` mojo program languag '' datasciencelearningcenter.substack.com archiv origin 5 may 2023 retriev 5 may 2023 `` mojo program languag '' archiv retriev 5 may 2023 5 may yegulalp serdar 16 januari 2017 `` nim languag draw best python rust go lisp '' infoworld archiv origin 13 octob 2018 retriev 7 june 2020 nim 's syntax strongli reminisc python 's use indent code block syntax way if/elif/then/els block construct yegulalp serdar 16 januari 2017 `` nim languag draw best python rust go lisp '' infoworld archiv origin 13 octob 2018 retriev 7 june 2020 nim 's syntax strongli reminisc python 's use indent code block syntax way if/elif/then/els block construct `` nim languag draw best python rust go lisp '' archiv retriev 7 june 2020 7 june `` interview creator rubi '' linuxdevcenter.com archiv origin 28 april 2018 retriev 3 decemb 2012 `` interview creator rubi '' linuxdevcenter.com archiv origin 28 april 2018 retriev 3 decemb 2012 `` interview creator rubi '' archiv retriev 3 decemb 2012 3 decemb lattner chri 3 june 2014 `` chri lattner 's homepag '' chri lattner archiv origin 22 decemb 2015 retriev 3 june 2014 start work swift program languag juli 2010 implement much basic languag structur peopl know exist amaz peopl start contribut earnest late 2011 becam major focu appl develop tool group juli 2013 ... draw idea objective-c rust haskel rubi python c clu far mani other list lattner chri 3 june 2014 `` chri lattner 's homepag '' chri lattner archiv origin 22 decemb 2015 retriev 3 june 2014 start work swift program languag juli 2010 implement much basic languag structur peopl know exist amaz peopl start contribut earnest late 2011 becam major focu appl develop tool group juli 2013 ... draw idea objective-c rust haskel rubi python c clu far mani other list lattner chri `` chri lattner 's homepag '' archiv retriev 3 june 2014 3 june jalan nishant aanjaney 10 novemb 2022 `` program kotlin '' codex retriev 29 april 2024 jalan nishant aanjaney 10 novemb 2022 `` program kotlin '' codex retriev 29 april 2024 `` program kotlin '' retriev 29 april 2024 29 april kupri andrea fellow donal k. 14 septemb 2000 `` tip 3 tip format '' tcl.tk tcl develop xchang archiv origin 13 juli 2017 retriev 24 novemb 2008 kupri andrea fellow donal k. 14 septemb 2000 `` tip 3 tip format '' tcl.tk tcl develop xchang archiv origin 13 juli 2017 retriev 24 novemb 2008 `` tip 3 tip format '' archiv retriev 24 novemb 2008 24 novemb gustafsson per niskanen raimo 29 januari 2007 `` eep 1 eep purpos guidelin '' erlang.org archiv origin 15 june 2020 retriev 19 april 2011 gustafsson per niskanen raimo 29 januari 2007 `` eep 1 eep purpos guidelin '' erlang.org archiv origin 15 june 2020 retriev 19 april 2011 `` eep 1 eep purpos guidelin '' archiv retriev 19 april 2011 19 april `` swift evolut process '' swift program languag evolut repositori github 18 februari 2020 archiv origin 27 april 2020 retriev 27 april 2020 `` swift evolut process '' swift program languag evolut repositori github 18 februari 2020 archiv origin 27 april 2020 retriev 27 april 2020 `` swift evolut process '' archiv retriev 27 april 2020 27 april sourc edit edit edit `` python artifici intellig '' python wiki 19 juli 2012 archiv origin 1 novemb 2012 retriev 3 decemb 2012 `` python artifici intellig '' origin retriev 3 decemb 2012 3 decemb pain jocelyn ed august 2005 `` ai python '' ai expert newslett amzi archiv origin 26 march 2012 retriev 11 februari 2012 `` ai python '' origin retriev 11 februari 2012 11 februari `` pyaiml 0.8.5 python packag index '' pypi.python.org retriev 17 juli 2013 `` pyaiml 0.8.5 python packag index '' retriev 17 juli 2013 17 juli russel stuart j norvig peter 2009 artifici intellig modern approach 3rd ed. upper saddl river nj prentic hall isbn 978-0-13-604259-4 russel stuart j. norvig peter isbn 978-0-13-604259-4 read edit edit edit downey allen juli 2024 think python think like comput scientist 3rd ed. o'reilli medium isbn 978-1098155438 think python think like comput scientist isbn 978-1098155438 lutz mark 2013 learn python 5th ed. o'reilli medium isbn 978-0-596-15806-4 isbn 978-0-596-15806-4 summerfield mark 2009 program python 3 2nd ed. addison-wesley profession isbn 978-0-321-68056-3 isbn 978-0-321-68056-3 ramalho luciano may 2022 fluent python o'reilli medium isbn 978-1-4920-5632-4 fluent python isbn 978-1-4920-5632-4 extern link edit edit edit sister project sister project medium common medium common medium quotat wikiquot quotat wikiquot quotat textbook wikibook textbook wikibook textbook resourc wikivers resourc wikivers resourc data wikidata data wikidata data offici websit offici websit offici websit offici websit python tutori python tutori v v e e python python implement circuitpython circuitpython clpython clpython cpython cpython cython cython micropython micropython numba numba ironpython ironpython jython jython psyco psyco pypi pypi python s60 python s60 shed skin shed skin stackless python stackless python unladen swallow unladen swallow ... ide eric eric idl idl ninja-id ninja-id pycharm pycharm pydev pydev spyder spyder ... wsgi wsgi asgi asgi design guido van rossum guido van rossum softwar list softwar python softwar foundat python softwar foundat python confer pycon python confer v v e e program languag comparison comparison timelin timelin histori histori ada ada algol simula algol simula simula apl apl assembl assembl basic visual basic classic .net basic visual basic classic .net visual basic classic classic .net .net c c c++ c++ c c cobol cobol erlang erlang forth forth fortran fortran go go haskel haskel java java javascript javascript julia julia kotlin kotlin lisp lisp lua lua matlab matlab ml ml pascal object pascal pascal object pascal object pascal perl perl php php prolog prolog python python r r rubi rubi rust rust sql sql scratch scratch shell shell smalltalk smalltalk swift swift ... ... list alphabet alphabet categor categor gener gener non-english-bas non-english-bas categori categori v v e e python web framework bottl cherrypi cherrypi cubicweb cubicweb django django fastapi fastapi flask flask grok grok nevow nevow pylon pylon pyramid pyramid quixot quixot tornado tornado turbogear turbogear twistedweb twistedweb web2pi web2pi zope 2 zope 2 ... comparison comparison v v e e gener differenti program differenti program inform geometri inform geometri statist manifold statist manifold automat differenti automat differenti neuromorph comput neuromorph comput pattern recognit pattern recognit ricci calculu ricci calculu comput learn theori comput learn theori induct bia induct bia ipu ipu tpu tpu vpu vpu memristor memristor spinnak spinnak tensorflow tensorflow pytorch pytorch kera kera scikit-learn scikit-learn theano theano jax jax flux.jl flux.jl mindspor mindspor portal comput program technolog comput program comput program technolog technolog v v e e free open-sourc softwar altern term free softwar altern term free softwar comparison open-sourc closed-sourc softwar comparison open-sourc closed-sourc softwar comparison source-code-host facil comparison source-code-host facil free softwar free softwar free softwar project directori free softwar project directori grati versu libr grati versu libr long-term support long-term support open-sourc softwar open-sourc softwar open-sourc softwar develop open-sourc softwar develop outlin outlin timelin timelin softwarepackag audio audio bioinformat bioinformat codec codec configur manag configur manag driver graphic wireless driver graphic graphic wireless wireless health health mathemat mathemat offic suit offic suit oper system oper system rout rout televis televis video game video game web applic e-commerc web applic e-commerc e-commerc android app android app io app io app commerci commerci formerli proprietari formerli proprietari formerli open-sourc formerli open-sourc commun free softwar movement free softwar movement histori histori open-source-softwar movement open-source-softwar movement event event advocaci advocaci organis free softwar movement india free softwar movement india free softwar foundat free softwar foundat licens afl afl apach apach apsl apsl artist artist beerwar beerwar bsd bsd creativ common creativ common cddl cddl epl epl free softwar foundat gnu gpl gnu agpl gnu lgpl free softwar foundat gnu gpl gnu gpl gnu agpl gnu agpl gnu lgpl gnu lgpl isc isc mit mit mpl mpl python python python softwar foundat licens python softwar foundat licens share sourc initi share sourc initi sleepycat sleepycat unlicens unlicens wtfpl wtfpl zlib zlib comparison licens comparison licens contributor licens agreement contributor licens agreement copyleft copyleft debian free softwar guidelin debian free softwar guidelin definit free cultur work definit free cultur work free licens free licens free softwar definit free softwar definit open sourc definit open sourc definit open-sourc licens open-sourc licens permiss softwar licens permiss softwar licens public domain public domain digit right manag digit right manag licens prolifer licens prolifer mozilla softwar rebrand mozilla softwar rebrand proprietari devic driver proprietari devic driver proprietari firmwar proprietari firmwar proprietari softwar proprietari softwar sco/linux controversi sco/linux controversi softwar patent softwar patent softwar secur softwar secur tivoiz tivoiz trust comput trust comput fork fork gnu manifesto gnu manifesto microsoft open specif promis microsoft open specif promis open-cor model open-cor model open-sourc hardwar open-sourc hardwar share sourc initi share sourc initi source-avail softwar source-avail softwar cathedr bazaar cathedr bazaar revolut o revolut o portal portal categori categori v v e e statist softwar public domain dataplot dataplot epi info epi info cspro cspro x-12-arima x-12-arima open-sourc admb admb dap dap gretl gretl jamovi jamovi jasp jasp jag jag jmulti jmulti julia julia jupyt julia python r jupyt gnu octav gnu octav openbug openbug orang orang pspp pspp python statsmodel pymc3 ipython idl python pymc3 ipython idl r rstudio r rstudio sagemath sagemath simfit simfit sofa statist sofa statist stan stan xlispstat xlispstat freewar bv4.1 bv4.1 cumfreq cumfreq segreg segreg xplore xplore winbug winbug commerci cross-platform data desk data desk gauss gauss graphpad instat graphpad instat graphpad prism graphpad prism ibm spss statist spss ibm spss model spss model jmp jmp mapl mapl mathcad mathcad mathematica mathematica matlab matlab oxmetr oxmetr rat rat revolut analyt revolut analyt sa sa smartpl smartpl stata stata statview statview sudaan sudaan s-plu s-plu tsp tsp world program system wp world program system window bmdp bmdp eview eview genstat genstat limdep limdep lisrel lisrel medcalc medcalc microfit microfit minitab minitab mlwin mlwin ncss ncss shazam shazam sigmastat sigmastat statistica statistica statsdirect statsdirect statxact statxact systat systat unscrambl unscrambl unistat unistat excel analyse-it analyse-it unistat excel unistat xlfit xlfit rexcel rexcel categori categori comparison comparison v v e e numerical-analysi softwar advanc simul librari advanc simul librari admb admb chapel chapel euler mathemat toolbox euler mathemat toolbox freefem++ freefem++ freemat freemat geniu geniu gmsh gmsh gnu octav gnu octav gretl gretl julia julia jupyt julia python r ipython jupyt ipython mfem mfem openfoam openfoam python python r r sagemath sagemath salom salom scicoslab scicoslab scilab scilab x10 x10 weka weka fortress fortress dadisp dadisp featool multiphys featool multiphys gauss gauss labview labview mapl mapl mathcad mathcad mathematica mathematica matlab matlab speakeasi speakeasi vissim vissim comparison comparison author control databas fast fast fast germani germani germani unit state unit state unit state franc franc franc bnf data bnf data bnf data czech republ czech republ czech republ czech republ israel israel israel idref idref idref http //en.wikipedia.org/w/index.php title=python_ programming_languag oldid=1268942017 categori python program languag python program languag class-bas program languag class-bas program languag notebook interfac notebook interfac comput scienc netherland comput scienc netherland concurr program languag concurr program languag cross-platform free softwar cross-platform free softwar cross-platform softwar cross-platform softwar dutch invent dutch invent dynam type program languag dynam type program languag educ program languag educ program languag high-level program languag high-level program languag inform technolog netherland inform technolog netherland multi-paradigm program languag multi-paradigm program languag object-ori program languag object-ori program languag pattern match program languag pattern match program languag program languag program languag program languag creat 1991 program languag creat 1991 script languag script languag text-ori program languag text-ori program languag articl short descript articl short descript short descript match wikidata short descript match wikidata use dmi date novemb 2021 use dmi date novemb 2021 use american english decemb 2024 use american english decemb 2024 wikipedia articl written american english wikipedia articl written american english articl contain potenti date statement octob 2024 articl contain potenti date statement octob 2024 articl contain potenti date statement articl contain potenti date statement articl contain potenti date statement march 2024 articl contain potenti date statement march 2024 articl contain potenti date statement decemb 2022 articl contain potenti date statement decemb 2022 articl contain potenti date statement 2020 articl contain potenti date statement 2020 articl contain potenti date statement 2008 articl contain potenti date statement 2008 page use sister project link wikidata namespac mismatch page use sister project link wikidata namespac mismatch page use sister project link hidden wikidata page use sister project link hidden wikidata articl exampl python program languag code articl exampl python program languag code page last edit 12 januari 2025 07:40 utc utc text avail creativ common attribution-sharealik 4.0 licens addit term may appli use site agre term use privaci polici wikipedia® regist trademark wikimedia foundat inc. non-profit organ creativ common attribution-sharealik 4.0 licens term use privaci polici wikimedia foundat inc. privaci polici privaci polici wikipedia wikipedia disclaim disclaim contact wikipedia contact wikipedia code conduct code conduct develop develop statist statist cooki statement cooki statement mobil view mobil view",
    "https://en.wikipedia.org/wiki/JavaScript": "jump content main menu main page main page main page content content content current event current event current event random articl random articl random articl wikipedia wikipedia wikipedia contact u contact u contact u help help help learn edit learn edit learn edit commun portal commun portal commun portal recent chang recent chang recent chang upload file upload file upload file search search appear donat donat donat creat account creat account creat account log log log person tool donat donat donat creat account creat account creat account log log log learn learn contribut contribut contribut talk talk talk content top top 1 histori toggl histori subsect 1.1 creation netscap 1.2 adopt microsoft 1.3 rise jscript 1.4 growth standard 1.5 reach matur 1 histori 1 histori toggl histori subsect 1.1 creation netscap 1.1 creation netscap 1.1 creation netscap 1.2 adopt microsoft 1.2 adopt microsoft 1.2 adopt microsoft 1.3 rise jscript 1.3 rise jscript 1.3 rise jscript 1.4 growth standard 1.4 growth standard 1.4 growth standard 1.5 reach matur 1.5 reach matur 1.5 reach matur 2 trademark 2 trademark 2 trademark 3 websit client-sid usag toggl websit client-sid usag subsect 3.1 exampl script behavior 3.2 librari framework 3 websit client-sid usag 3 websit client-sid usag toggl websit client-sid usag subsect 3.1 exampl script behavior 3.1 exampl script behavior 3.1 exampl script behavior 3.2 librari framework 3.2 librari framework 3.2 librari framework 4 usag 4 usag 4 usag 5 execut toggl execut subsect 5.1 javascript engin 5.2 runtim system 5 execut 5 execut toggl execut subsect 5.1 javascript engin 5.1 javascript engin 5.1 javascript engin 5.2 runtim system 5.2 runtim system 5.2 runtim system 6 featur toggl featur subsect 6.1 imper structur 6.2 weakli type 6.3 dynam 6.3.1 type 6.3.2 run-tim evalu 6.4 object-orient prototype-bas 6.4.1 prototyp 6.4.2 function object constructor 6.4.3 function method 6.5 function 6.5.1 lexic closur 6.5.2 anonym function 6.6 deleg 6.6.1 function role trait mixin 6.6.2 object composit inherit 6.7 miscellan 6.7.1 zero-bas number 6.7.2 variad function 6.7.3 array object liter 6.7.4 regular express 6.7.5 promis async/await 6.7.5.1 promis 6.7.5.2 async/await 6.8 vendor-specif extens 6 featur 6 featur toggl featur subsect 6.1 imper structur 6.1 imper structur 6.1 imper structur 6.2 weakli type 6.2 weakli type 6.2 weakli type 6.3 dynam 6.3.1 type 6.3.2 run-tim evalu 6.3 dynam 6.3 dynam 6.3.1 type 6.3.1 type 6.3.1 type 6.3.2 run-tim evalu 6.3.2 run-tim evalu 6.3.2 run-tim evalu 6.4 object-orient prototype-bas 6.4.1 prototyp 6.4.2 function object constructor 6.4.3 function method 6.4 object-orient prototype-bas 6.4 object-orient prototype-bas 6.4.1 prototyp 6.4.1 prototyp 6.4.1 prototyp 6.4.2 function object constructor 6.4.2 function object constructor 6.4.2 function object constructor 6.4.3 function method 6.4.3 function method 6.4.3 function method 6.5 function 6.5.1 lexic closur 6.5.2 anonym function 6.5 function 6.5 function 6.5.1 lexic closur 6.5.1 lexic closur 6.5.1 lexic closur 6.5.2 anonym function 6.5.2 anonym function 6.5.2 anonym function 6.6 deleg 6.6.1 function role trait mixin 6.6.2 object composit inherit 6.6 deleg 6.6 deleg 6.6.1 function role trait mixin 6.6.1 function role trait mixin 6.6.1 function role trait mixin 6.6.2 object composit inherit 6.6.2 object composit inherit 6.6.2 object composit inherit 6.7 miscellan 6.7.1 zero-bas number 6.7.2 variad function 6.7.3 array object liter 6.7.4 regular express 6.7.5 promis async/await 6.7.5.1 promis 6.7.5.2 async/await 6.7 miscellan 6.7 miscellan 6.7.1 zero-bas number 6.7.1 zero-bas number 6.7.1 zero-bas number 6.7.2 variad function 6.7.2 variad function 6.7.2 variad function 6.7.3 array object liter 6.7.3 array object liter 6.7.3 array object liter 6.7.4 regular express 6.7.4 regular express 6.7.4 regular express 6.7.5 promis async/await 6.7.5.1 promis 6.7.5.2 async/await 6.7.5 promis async/await 6.7.5 promis async/await 6.7.5.1 promis 6.7.5.1 promis 6.7.5.1 promis 6.7.5.2 async/await 6.7.5.2 async/await 6.7.5.2 async/await 6.8 vendor-specif extens 6.8 vendor-specif extens 6.8 vendor-specif extens 7 syntax toggl syntax subsect 7.1 simpl exampl 7.2 advanc exampl 7 syntax 7 syntax toggl syntax subsect 7.1 simpl exampl 7.1 simpl exampl 7.1 simpl exampl 7.2 advanc exampl 7.2 advanc exampl 7.2 advanc exampl 8 secur toggl secur subsect 8.1 cross-sit script 8.2 cross-sit request forgeri 8.3 misplac trust client 8.4 misplac trust develop 8.5 browser plugin code error 8.6 sandbox implement error 8.7 hardwar vulner 8 secur 8 secur toggl secur subsect 8.1 cross-sit script 8.1 cross-sit script 8.1 cross-sit script 8.2 cross-sit request forgeri 8.2 cross-sit request forgeri 8.2 cross-sit request forgeri 8.3 misplac trust client 8.3 misplac trust client 8.3 misplac trust client 8.4 misplac trust develop 8.4 misplac trust develop 8.4 misplac trust develop 8.5 browser plugin code error 8.5 browser plugin code error 8.5 browser plugin code error 8.6 sandbox implement error 8.6 sandbox implement error 8.6 sandbox implement error 8.7 hardwar vulner 8.7 hardwar vulner 8.7 hardwar vulner 9 develop tool 9 develop tool 9 develop tool 10 relat technolog toggl relat technolog subsect 10.1 java 10.2 json 10.3 transpil 10.4 webassembl 10 relat technolog 10 relat technolog toggl relat technolog subsect 10.1 java 10.1 java 10.1 java 10.2 json 10.2 json 10.2 json 10.3 transpil 10.3 transpil 10.3 transpil 10.4 webassembl 10.4 webassembl 10.4 webassembl 11 refer 11 refer 11 refer 12 sourc 12 sourc 12 sourc 13 read 13 read 13 read 14 extern link 14 extern link 14 extern link toggl tabl content javascript javascript 109 languag afrikaan afrikaan afrikaan ænglisc ænglisc ænglisc العربية العربية العربية aragoné aragoné aragoné asturianu asturianu asturianu azərbaycanca azərbaycanca azərbaycanca تۆرکجه تۆرکجه تۆرکجه বাংলা বাংলা বাংলা 閩南語 bân-lâm-gú 閩南語 bân-lâm-gú 閩南語 bân-lâm-gú беларуская беларуская беларуская беларуская тарашкевіца беларуская тарашкевіца беларуская тарашкевіца български български български brezhoneg brezhoneg brezhoneg català català català чӑвашла чӑвашла чӑвашла čeština čeština čeština dansk dansk dansk deutsch deutsch deutsch eesti eesti eesti ελληνικά ελληνικά ελληνικά español español español esperanto esperanto esperanto euskara euskara euskara فارسی فارسی فارسی françai françai françai galego galego galego 客家語 hak-kâ-ngî 客家語 hak-kâ-ngî 客家語 hak-kâ-ngî 한국어 한국어 한국어 hausa hausa hausa հայերեն հայերեն հայերեն हिन्दी हिन्दी हिन्दी hrvatski hrvatski hrvatski ido ido ido bahasa indonesia bahasa indonesia bahasa indonesia interlingua interlingua interlingua íslenska íslenska íslenska italiano italiano italiano עברית עברית עברית jawa jawa jawa ქართული ქართული ქართული қазақша қазақша қазақша kiswahili kiswahili kiswahili kurdî kurdî kurdî кыргызча кыргызча кыргызча latina latina latina latviešu latviešu latviešu lietuvių lietuvių lietuvių lingua franca nova lingua franca nova lingua franca nova lombard lombard lombard magyar magyar magyar македонски македонски македонски മലയാളം മലയാളം മലയാളം मराठी मराठी मराठी مصرى مصرى مصرى مازِرونی مازِرونی مازِرونی bahasa melayu bahasa melayu bahasa melayu 閩東語 mìng-dĕ̤ng-ngṳ̄ 閩東語 mìng-dĕ̤ng-ngṳ̄ 閩東語 mìng-dĕ̤ng-ngṳ̄ монгол монгол монгол မြန်မာဘာသာ မြန်မာဘာသာ မြန်မာဘာသာ nederland nederland nederland नेपाली नेपाली नेपाली 日本語 日本語 日本語 nordfriisk nordfriisk nordfriisk norsk bokmål norsk bokmål norsk bokmål ଓଡ଼ିଆ ଓଡ଼ିଆ ଓଡ଼ିଆ oʻzbekcha ўзбекча oʻzbekcha ўзбекча oʻzbekcha ўзбекча پنجابی پنجابی پنجابی ပအိုဝ်ႏဘာႏသာႏ ပအိုဝ်ႏဘာႏသာႏ ပအိုဝ်ႏဘာႏသာႏ ភាសាខ្មែរ ភាសាខ្មែរ ភាសាខ្មែរ pinayuanan pinayuanan pinayuanan polski polski polski portuguê portuguê portuguê qaraqalpaqsha qaraqalpaqsha qaraqalpaqsha română română română runa simi runa simi runa simi русский русский русский саха тыла саха тыла саха тыла scot scot scot shqip shqip shqip සිංහල සිංහල සිංහල simpl english simpl english simpl english slovenčina slovenčina slovenčina slovenščina slovenščina slovenščina soomaaliga soomaaliga soomaaliga کوردی کوردی کوردی српски srpski српски srpski српски srpski srpskohrvatski српскохрватски srpskohrvatski српскохрватски srpskohrvatski српскохрватски sunda sunda sunda suomi suomi suomi svenska svenska svenska tagalog tagalog tagalog தமிழ் தமிழ் தமிழ் taqbaylit taqbaylit taqbaylit татарча tatarça татарча tatarça татарча tatarça తెలుగు తెలుగు తెలుగు tetun tetun tetun ไทย ไทย ไทย türkçe türkçe türkçe türkmenç türkmenç türkmenç basa ugi basa ugi basa ugi українська українська українська اردو اردو اردو vèneto vèneto vèneto tiếng việt tiếng việt tiếng việt 文言 文言 文言 吴语 吴语 吴语 粵語 粵語 粵語 žemaitėška žemaitėška žemaitėška 中文 中文 中文 edit link edit link articl articl articl talk talk talk english read read read view sourc view sourc view sourc view histori view histori view histori tool read read read view sourc view sourc view sourc view histori view histori view histori link link link relat chang relat chang relat chang upload file upload file upload file special page special page special page perman link perman link perman link page inform page inform page inform cite page cite page cite page get shorten url get shorten url get shorten url download qr code download qr code download qr code download pdf download pdf download pdf printabl version printabl version printabl version wikimedia common wikimedia common wikimedia common mediawiki mediawiki mediawiki wikibook wikibook wikibook wikifunct wikifunct wikifunct wikivers wikivers wikivers wikidata item wikidata item wikidata item page version statu accept version page latest accept revis review java program languag javanes script ecmascript paradigm multi-paradigm event-driven function imper procedur object-ori design brendan eich netscap ecmascript 29 year ago 1995-12-04 1995-12-04 1 stabl releas 2 7 month ago june 2024 june 2024 preview releas 3 9 month ago 27 march 2024 27 march 2024 type disciplin dynam weak duck memori manag garbag collect filenam extens .j .cj .mj 4 4 ecma-international.org/publications-and-standards/standards/ecma-262/ ecma-international.org/publications-and-standards/standards/ecma-262/ implement v8 javascriptcor spidermonkey chakra java 5 6 scheme 6 self 7 awk 8 hypertalk 9 actionscript arkt assemblyscript coffeescript dart hax js++ opa typescript javascript wikibook javascript javascript /ˈdʒɑːvəskrɪpt/ often abbrevi j program languag core technolog web alongsid html cs 99 websit use javascript client side webpag behavior 10 /ˈdʒɑːvəskrɪpt/ /ˈdʒɑːvəskrɪpt/ /ˈdʒɑːvəskrɪpt/ ˈdʒɑːvəskrɪpt ˈ dʒ ɑː v ə k r ɪ p program languag web html cs websit client webpag 10 web browser dedic javascript engin execut client code engin also util server varieti app popular runtim system non-brows usag node.j web browser javascript engin code server app runtim system node.j javascript high-level often just-in-tim compil languag conform ecmascript standard 11 dynam type prototype-bas object-orient first-class function multi-paradigm support event-driven function imper program style applic program interfac api work text date regular express standard data structur document object model dom high-level just-in-tim compil ecmascript 11 dynam type prototype-bas object-orient first-class function multi-paradigm event-driven function imper program style applic program interfac regular express data structur document object model ecmascript standard includ input/output i/o network storag graphic facil practic web browser runtim system provid javascript api i/o input/output network storag graphic although java javascript similar name syntax two languag distinct differ greatli design java syntax histori creation netscap first popular web browser graphic user interfac mosaic releas 1993 access non-techn peopl play promin role rapid growth earli world wide web 12 lead develop mosaic found netscap corpor releas polish browser netscap navig 1994 quickli becam most-us 13 web browser graphic user interfac mosaic world wide web 12 netscap netscap navig 13 form year web web page could static lack capabl dynam behavior page load browser desir flourish web develop scene remov limit 1995 netscap decid add program languag navig pursu two rout achiev collabor sun microsystem emb java languag also hire brendan eich emb scheme languag 6 web page program languag sun microsystem java brendan eich scheme 6 goal `` languag mass '' 14 `` help nonprogramm creat dynam interact web site '' 15 netscap manag soon decid best option eich devi new languag syntax similar java less like scheme extant script languag 5 6 although new languag interpret implement call livescript first ship part navig beta septemb 1995 name chang javascript offici releas decemb 6 1 16 17 14 web site 15 script languag 5 6 interpret beta 6 1 16 17 choic javascript name caus confus impli directli relat java time dot-com boom begun java popular new languag eich consid javascript name market ploy netscap 14 dot-com boom 14 adopt microsoft microsoft debut internet explor 1995 lead browser war netscap javascript front microsoft creat interpret call jscript 18 microsoft internet explor browser war interpret jscript 18 microsoft first releas jscript 1996 alongsid initi support cs extens html implement notic differ counterpart netscap navig 19 20 differ made difficult develop make websit work well browser lead widespread use `` best view netscap '' `` best view internet explor '' logo sever year 19 21 cs html implement netscap navig 19 20 19 21 rise jscript brendan eich later said period `` 's still kind sidekick languag 's consid slow annoy peopl pop-up scroll messag old statu bar bottom old browser `` 14 brendan eich sidekick pop-up statu bar browser 14 novemb 1996 netscap submit javascript ecma intern start point standard specif browser vendor could conform led offici releas first ecmascript languag specif june 1997 netscap ecma intern ecmascript standard process continu year releas ecmascript 2 june 1998 ecmascript 3 decemb 1999 work ecmascript 4 began 2000 18 18 howev effort fulli standard languag undermin microsoft gain increasingli domin posit browser market earli 2000 internet explor 's market share reach 95 22 meant jscript becam de facto standard client-sid script web microsoft internet explor 22 jscript client-sid script microsoft initi particip standard process implement propos jscript languag eventu stop collabor ecma work thu ecmascript 4 mothbal growth standard period internet explor domin earli 2000 client-sid script stagnant start chang 2004 successor netscap mozilla releas firefox browser firefox well receiv mani take signific market share internet explor 23 internet explor mozilla firefox 23 2005 mozilla join ecma intern work start ecmascript xml e4x standard led mozilla work jointli macromedia later acquir adob system implement e4x actionscript 3 languag base ecmascript 4 draft goal becam standard actionscript 3 new ecmascript 4 end adob system releas tamarin implement open sourc project howev tamarin actionscript 3 differ establish client-sid script without cooper microsoft ecmascript 4 never reach fruition ecmascript xml macromedia adob system tamarin open sourc microsoft meanwhil import develop occur open-sourc commun affili ecma work 2005 jess jame garrett releas white paper coin term ajax describ set technolog javascript backbon creat web applic data load background avoid need full page reload spark renaiss period javascript spearhead open-sourc librari commun form around mani new librari creat includ jqueri prototyp dojo toolkit mootool jess jame garrett ajax web applic jqueri prototyp dojo toolkit mootool googl debut chrome browser 2008 v8 javascript engin faster competit 24 25 key innov just-in-tim compil jit 26 browser vendor need overhaul engin jit 27 googl chrome v8 24 25 just-in-tim compil 26 27 juli 2008 dispar parti came togeth confer oslo led eventu agreement earli 2009 combin relev work drive languag forward result ecmascript 5 standard releas decemb 2009 oslo reach matur ambiti work languag continu sever year culmin extens collect addit refin formal public ecmascript 6 2015 28 ecmascript 6 28 creation node.j 2009 ryan dahl spark signific increas usag javascript outsid web browser node combin v8 engin event loop i/o api therebi provid stand-alon javascript runtim system 29 30 2018 node use million develop 31 npm modul packag manag world 32 node.j ryan dahl v8 event loop i/o api 29 30 31 npm packag manag 32 ecmascript draft specif current maintain openli github 33 edit produc via regular annual snapshot 33 potenti revis languag vet comprehens propos process 34 35 instead edit number develop check statu upcom featur individu 33 github 33 33 34 35 33 current javascript ecosystem mani librari framework establish program practic substanti usag javascript outsid web browser 17 plu rise single-pag applic javascript-heavi websit sever transpil creat aid develop process 36 librari framework 17 single-pag applic transpil 36 trademark `` javascript '' trademark oracl corpor unit state 37 38 trademark origin issu sun microsystem 6 may 1997 transfer oracl acquir sun 2009 39 trademark oracl corpor 37 38 sun microsystem 39 letter circul septemb 2024 spearhead ryan dahl call oracl free javascript trademark 40 brendan eich origin creator javascript among 14,000 signatori support initi ryan dahl 40 brendan eich websit client-sid usag javascript domin client-sid script languag web 99 websit use purpos 10 script embed includ html document interact dom client-sid script languag websit 10 html dom major web browser built-in javascript engin execut code user 's devic web browser javascript engin code exampl script behavior load new web page content without reload page via ajax websocket exampl user social medium send receiv messag without leav current page web page ajax websocket user social medium web page anim fade object resiz move play browser game browser game control playback stream medium playback stream medium gener pop-up ad alert box pop-up ad valid input valu web form data sent web server valid web form web server log data user 's behavior send server websit owner use data analyt ad track person analyt ad track person redirect user anoth page store retriev data user 's devic via storag indexeddb standard storag indexeddb librari framework 80 websit use third-parti javascript librari web framework part client-sid script 41 librari web framework 41 jqueri far most-us 41 notabl one includ angular bootstrap lodash modernizr react underscor vue 41 multipl option use conjunct jqueri bootstrap 42 jqueri 41 angular bootstrap lodash modernizr react underscor vue 41 42 howev term `` vanilla j '' coin websit use librari framework instead reli entir standard javascript function 43 43 usag use javascript expand beyond web browser root javascript engin embed varieti softwar system server-sid websit deploy non-brows applic web browser javascript engin server-sid applic initi attempt promot server-sid javascript usag netscap enterpris server microsoft 's internet inform servic 44 45 small nich 46 server-sid usag eventu start grow late 2000 creation node.j approach 46 netscap enterpris server microsoft internet inform servic 44 45 46 node.j approach 46 electron cordova react nativ applic framework use creat mani applic behavior implement javascript non-brows applic includ adob acrobat support script pdf document 47 gnome shell extens written javascript 48 electron cordova react nativ applic framework adob acrobat pdf 47 gnome shell 48 javascript use embed system usual leverag node.j 49 50 51 embed system 49 50 51 execut javascript engin javascript engin edit edit javascript engin softwar compon execut javascript code first javascript engin mere interpret relev modern engin use just-in-tim compil improv perform 52 javascript engin softwar compon code engin interpret just-in-tim compil 52 javascript engin typic develop web browser vendor everi major browser one browser javascript engin run concert render engin via document object model web idl bind 53 howev use javascript engin limit browser exampl v8 engin core compon node.j runtim system 54 web browser render engin document object model web idl 53 v8 engin node.j runtim system 54 ecmascript implement webassembl sandbox 55 54 runtim system javascript engin must embed within runtim system web browser standalon system enabl script interact broader environ runtim system includ necessari api input/output oper network storag graphic provid abil import script runtim system web browser input/output network storag graphic javascript single-thread languag runtim process messag queue one time call function associ new messag creat call stack frame function 's argument local variabl call stack shrink grow base function 's need call stack empti upon function complet javascript proce next messag queue call event loop describ `` run complet '' messag fulli process next messag consid howev languag 's concurr model describ event loop non-block program i/o perform use event callback function mean exampl javascript process mous click wait databas queri return inform 56 thread messag queue function call stack argument local variabl event loop concurr model non-block event callback function 56 notabl standalon runtim node.j deno bun node.j deno bun featur follow featur common conform ecmascript implement unless explicitli specifi otherwis imper structur structur program javascript support much structur program syntax c e.g. statement loop switch statement loop etc. one partial except scope origin javascript function scope var block scope ad ecmascript 2015 keyword let const like c javascript make distinct express statement one syntact differ c automat semicolon insert allow semicolon termin statement omit 57 structur program c scope function scope block scope const express statement automat semicolon insert 57 weakli type weakli type javascript weakli type mean certain type implicitli cast depend oper use 58 weakli type 58 binari oper cast operand string unless operand number addit oper doubl concaten oper binari oper alway cast operand number unari oper alway cast operand number howev alway cast number binary64 preserv bigint integ 59 binary64 integ 59 valu cast string like follow 58 58 string left as-i number convert string represent array element cast string join comma object convert string object object object name constructor object valu cast number cast string cast string number process modifi defin tostr valueof function prototyp string number cast respect prototyp javascript receiv critic way implement convers complex rule mistaken inconsist 60 58 exampl ad number string number cast string perform concaten subtract number string string cast number perform subtract 60 58 nan often also mention result 0 number mislead interpret empti code block instead empti object empti array cast number remain unari oper express wrap parenthes – curli bracket interpret empti object result express `` object object '' expect 58 58 dynam dynam program type dynam type javascript dynam type like script languag type associ valu rather express exampl variabl initi bound number may reassign string 61 javascript support variou way test type object includ duck type 62 dynam type script languag type valu variabl string 61 duck type 62 run-tim evalu eval javascript includ eval function execut statement provid string run-tim eval object-orient prototype-bas prototyp inherit javascript describ dougla crockford dougla crockford make prototyp object ... make new instanc object mutabl javascript augment new instanc give new field method act prototyp even newer object n't need class make lot similar object ... object inherit object could object orient 63 63 javascript object associ array augment prototyp see key provid name object properti two syntact way specifi name dot notat obj.x 10 bracket notat obj x 10 properti may ad rebound delet run-tim properti object properti belong object 's prototyp inherit chain enumer use ... loop object associ array properti properti prototyp prototype-bas program javascript use prototyp mani object-ori languag use class inherit 64 possibl simul mani class-bas featur prototyp javascript 65 prototyp class inherit 64 65 function object constructor function doubl object constructor along typic role prefix function call new creat instanc prototyp inherit properti method constructor includ properti object prototyp 66 ecmascript 5 offer object.cr method allow explicit creation instanc without automat inherit object prototyp older environ assign prototyp null 67 constructor 's prototyp properti determin object use new object 's intern prototyp new method ad modifi prototyp function use constructor javascript 's built-in constructor array object also prototyp modifi possibl modifi object prototyp gener consid bad practic object javascript inherit method properti object prototyp may expect prototyp modifi 68 66 67 68 function method method comput scienc unlik mani object-ori languag javascript distinct function definit method definit rather distinct occur function call function call method object function 's local keyword bound object invoc method function function program javascript function first-class function consid object 69 function may properti method .call .bind 70 function first-class 69 70 lexic closur closur comput program nest function function defin within anoth function creat time outer function invok addit nest function form lexic closur lexic scope outer function includ constant local variabl argument valu becom part intern state inner function object even execut outer function conclud 71 lexic closur lexic scope 71 anonym function anonym function javascript also support anonym function anonym function deleg deleg object-ori program javascript support implicit explicit deleg deleg function role trait mixin role-ori program trait comput scienc mixin javascript nativ support variou function-bas implement role 72 pattern like trait 73 74 mixin 75 function defin addit behavior least one method bound keyword within function bodi role deleg explicitli via call appli object need featur addit behavior share via prototyp chain role 72 trait 73 74 mixin 75 object composit inherit wherea explicit function-bas deleg cover composit javascript implicit deleg alreadi happen everi time prototyp chain walk order e.g. find method might relat directli own object method found get call within object 's context thu inherit javascript cover deleg automat bound prototyp properti constructor function composit inherit miscellan zero-bas number javascript zero-index languag zero-index variad function variad function indefinit number paramet pas function function access formal paramet also local argument object variad function also creat use bind method formal paramet variad function bind array object liter associ array object liter like mani script languag array object associ array languag creat succinct shortcut syntax fact liter form basi json data format associ array liter json regular express regular express manner similar perl javascript also support regular express provid concis power syntax text manipul sophist built-in string function 76 perl regular express 76 promis async/await javascript support promis async/await handl asynchron oper citat need promis async/await citat need citat need promis futur promis built-in promis object provid function handl promis associ handler asynchron action 's eventu result recent javascript specif introduc combin method allow develop combin multipl javascript promis oper base differ scenario method introduc promise.rac promise.al promise.allsettl promise.ani async/await async/await async/await allow asynchron non-block function structur way similar ordinari synchron function asynchron non-block code written minim overhead structur similarli tradit synchron block code vendor-specif extens histor javascript engin support non-standard featur javascript engin condit catch claus like java array comprehens gener express like python array comprehens concis function express function arg expr experiment syntax predat arrow function ecmascript xml e4x extens add nativ xml support ecmascript unsupport firefox sinc version 21 77 ecmascript xml 77 syntax javascript syntax simpl exampl variabl javascript defin use either var 78 let 79 const 80 keyword variabl defin without keyword defin global scope variabl 78 79 80 // declar function-scop variabl name x implicitli assign // special valu undefin variabl without valu automat // set undefin // var gener consid bad practic let const usual prefer var x // variabl manual set undefin like let x2 undefin // declar block-scop variabl name implicitli set // undefin let keyword introduc ecmascript 2015. let // declar block-scop un-reassign variabl name z set // string liter const keyword also introduc ecmascript 2015 // must explicitli assign // keyword const mean constant henc variabl reassign // valu constant const z `` valu reassign '' // declar global-scop variabl assign 3 gener consid // bad practic work strict mode 3 // declar variabl name mynumb assign number liter valu // 2 let mynumb 2 // reassign mynumb set string liter valu `` foo '' // javascript dynamically-typ languag legal mynumb `` foo '' note comment exampl preced two forward slash comment forward slash built-in input/output function javascript instead provid run-tim environ ecmascript specif edit 5.1 mention `` provis specif input extern data output comput result '' 81 howev runtim environ consol object use print output 82 minimalist `` hello world '' program javascript runtim environ consol object input/output 81 82 `` hello world '' program consol log `` hello world '' html document program like requir output // text node made use `` write '' method // frown upon overwrit document document fulli load document write 'foo // element made first creat dom const myelem document createel 'span // attribut like class id set well myelem classlist add 'foo myelem id 'bar // set tag look like span class= '' foo '' id= '' bar '' data-attr= '' baz '' /span myelem setattribut 'data-attr 'baz // could also written myelem.dataset.attr 'baz // final append child element bodi html document bodi appendchild myelem // element imper grab queryselector one element queryselectoral multipl element loop foreach document queryselector '.class // select first element `` class '' class document queryselector id // select first element id `` id '' document queryselector data-oth // select first element `` data-oth '' attribut document queryselectoral '.multipl // return array-lik nodelist element `` multipl '' class simpl recurs function calcul factori natur number recurs factori natur number function factori n // check argument legitimaci factori defin posit integ isnan n consol error `` non-numer argument allow '' return nan // special valu number n === 0 return 1 // 0 1 n 0 return undefin // factori neg number defin n 1 consol warn n round closest integ non-integ consid use gamma function instead. n math round n // check need repeat recurs henc defin actual recurs part separ // follow line function express recurs comput factori use arrow syntax introduc es6 const recursivelycomput 1 recursivelycomput 1 1 // note use ternari oper return recursivelycomput n factori 3 // return 6 anonym function lambda anonym function const counter function let count 0 return function return ++ count const x counter x // return 1 x // return 2 x // return 3 exampl show javascript function closur captur non-loc variabl refer function closur arrow function first introduc 6th edit – ecmascript 2015 shorten syntax write function javascript arrow function anonym variabl need refer order invok creation unless surround parenthesi execut immedi 6th edit – ecmascript 2015 exampl arrow function // arrow function let u omit function keyword // long_exampl point anonym function valu const long_exampl input1 input2 consol log `` hello world '' const output input1 input2 return output // brace arrow function simpli return express // 's input1 input2 const short_exampl input1 input2 input1 input2 long_exampl 2 3 // print `` hello world '' return 5 short_exampl 2 5 // return 7 // arrow function one paramet parenthes remov const no_parenthes input input 2 no_parenthes 3 // return 5 // arrow function like function definit execut statement creat // use write librari avoid fill global scope closur let three b b 1 2 const generate_multiplier_funct b isnan b || b b const five_multipl generate_multiplier_funct 5 // suppli argument `` seed '' express retain a. five_multipl 1 // return 5 five_multipl 3 // return 15 five_multipl 4 // return 60 javascript object creat instanc class object instanc class object class exampl class ball constructor radiu radiu radiu area math pi radiu 2 // class thu object contain function known method show consol log radiu const mybal new ball 5 // creat new instanc ball object radiu 5 mybal radiu ++ // object properti usual modifi outsid mybal show // use inherit `` show '' function log `` 6 '' javascript object instanti directli function object object function exampl function ball radiu const area math pi radiu 2 const obj radiu area // object mutabl function ad properti obj show consol log obj radiu return obj const mybal ball 5 // creat new ball object radiu 5 `` new '' keyword need mybal radiu ++ // instanc properti modifi mybal show // use `` show '' function log `` 6 '' new instanc valu variad function demonstr argument special variabl 83 variad function variabl 83 function sum let x 0 let 0 argument length ++ x += argument return x sum 1 2 // return 3 sum 1 2 3 // return 6 // es6 use rest oper function sum ... arg return arg reduc b b sum 1 2 // return 3 sum 1 2 3 // return 6 immediately-invok function express often use creat closur closur allow gather properti method namespac make privat immediately-invok function express let counter function let 0 // privat properti return // public method get function alert set function valu valu increment function alert ++ // modul counter get // return 0 counter set 6 counter increment // return 7 counter increment // return 8 gener object form gener function provid function call exit re-ent maintain intern context state 84 gener 84 function rawcount yield 1 yield 2 function dynamiccount let count 0 true // recommend util true loop case yield ++ count // instanc const counter1 rawcount const counter2 dynamiccount // implement counter1 next // valu 1 done fals counter1 next // valu 2 done fals counter1 next // valu undefin done true counter2 next // valu 1 done fals counter2 next // valu 2 done fals counter2 next // valu 3 done fals // ... infinit javascript export import modul 85 85 export exampl mymodule.j // function remain privat export let sum b return b // export variabl export let name 'alic export let age 23 // export name function export function add num1 num2 return num1 num2 // export class export class multipl constructor num1 num2 num1 num1 num2 num2 add return sum num1 num2 import exampl // import one properti import add './mymodule.j consol log add 1 2 // 3 // import multipl properti import name age './mymodule.j consol log name age // `` alic '' 23 // import properti modul import './module.j consol log name age // `` alic '' 23 consol log add 1 2 // 3 advanc exampl sampl code display variou javascript featur find lowest common multipl lcm two number function lcmcalcul x // constructor function isnan x throw new typeerror `` non-numer argument allow '' const checkint function x // inner function x 1 == 0 throw new typeerror x `` integ '' return x checkint x // semicolon ^^^^ option newlin enough b checkint // prototyp object instanc creat constructor // constructor 's `` prototyp '' properti lcmcalcul prototyp // object liter constructor lcmcalcul // reassign prototyp set constructor properti appropri gcd function // method calcul greatest common divisor // euclidean algorithm let math ab b math ab b b // swap variabl // b b b b // swap use destructur assign es6 b == 0 b b b // need calcul gcd `` redefin '' method // actual redefinition—it 's defin instanc // this.gcd refer `` redefinit '' instead lcmcalculator.prototype.gcd // note lead wrong result lcmcalcul object member `` '' `` b '' alter afterward // also 'gcd === `` gcd '' 'gcd === this.gcd 'gcd function return return // object properti name specifi string delimit doubl `` singl quot `` lcm '' function // variabl name collid object properti e.g. |lcm| |this.lcm| // use |this.a this.b| avoid fp precis issu let lcm gcd b // need calcul lcm `` redefin '' method lcm function return lcm return lcm // method also declar use es6 syntax tostr // use es6 templat liter oper concaten valu return lcmcalcul b b // defin gener output function implement work web browser function output x document bodi appendchild document createtextnod x document bodi appendchild document createel 'br // note array 's map foreach defin javascript 1.6 // use demonstr javascript 's inher function natur 25 55 21 56 22 58 28 56 map function pair // array liter map function return new lcmcalcul pair 0 pair 1 sort b lcm b lcm // sort compar function shorthand form function call `` arrow function '' foreach printresult function printresult obj output obj `` gcd `` obj gcd `` lcm `` obj lcm follow output display browser window lcmcalcul 28 b 56 gcd 28 lcm 56 lcmcalcul 21 b 56 gcd 7 lcm 168 lcmcalcul 25 b 55 gcd 5 lcm 275 lcmcalcul 22 b 58 gcd 2 lcm 638 secur browser secur javascript dom provid potenti malici author deliv script run client comput via web browser author minim risk use two restrict first script run sandbox perform web-rel action general-purpos program task like creat file second script constrain same-origin polici script one websit access inform usernam password cooki sent anoth site javascript-rel secur bug breach either origin polici sandbox dom sandbox same-origin polici subset gener javascript—adsaf secur ecmascript se —that provid greater level secur especi code creat third parti advertis 86 87 closur toolkit anoth project safe embed isol third-parti javascript html 88 86 87 88 content secur polici main intend method ensur trust code execut web page content secur polici cross-sit script cross-sit script common javascript-rel secur problem cross-sit script xss violat same-origin polici xss vulner occur attack caus target websit onlin bank websit includ malici script webpag present victim script exampl access bank applic privileg victim potenti disclos secret inform transfer money without victim 's author one import solut xss vulner html sanit cross-sit script same-origin polici html sanit browser includ partial protect reflect xss attack attack provid url includ malici script howev even user browser vulner xss attack malici code store databas correct design web applic server-sid fulli prevent xss xss vulner also occur implement mistak browser author 89 89 cross-sit request forgeri cross-sit request forgeri anoth cross-sit vulner cross-sit request forgeri csrf csrf code attack 's site trick victim 's browser take action user intend target site like transfer money bank target site reli sole cooki request authent request origin code attack 's site carri valid login credenti initi user gener solut csrf requir authent valu hidden form field cooki authent request might last effect check http referr header also help cross-sit request forgeri `` javascript hijack '' type csrf attack script tag attack 's site exploit page victim 's site return privat inform json javascript possibl solut includ json requir authent token post get paramet respons return privat inform post get misplac trust client develop client-serv applic must recogn untrust client may control attack applic author assum javascript code run intend secret embed code could extract determin adversari implic websit author perfectli conceal javascript oper raw sourc code must sent client code obfusc obfusc reverse-engin obfusc javascript form valid provid conveni user secur site verifi user agre term servic filter invalid charact field contain number must server client script select disabl javascript reli prevent oper right-click imag save 90 90 consid bad practic emb sensit inform password javascript extract attack 91 91 prototyp pollut runtim vulner attack overwrit arbitrari properti object 's prototyp prototyp pollut misplac trust develop packag manag system npm bower popular javascript develop system allow develop easili manag program 's depend upon develop program librari develop trust maintain librari keep secur date alway case vulner emerg blind trust relied-upon librari new releas caus bug vulner appear program reli upon librari invers librari go unpatch known vulner wild studi done look sampl 133,000 websit research found 37 websit includ librari least one known vulner 92 `` median lag oldest librari version use websit newest avail version librari 1,177 day alexa develop librari still activ use ceas year ago `` 92 anoth possibl maintain librari may remov librari entir occur march 2016 azer koçulu remov repositori npm caus ten thousand program websit depend upon librari break 93 94 npm 92 92 93 94 browser plugin code error buffer overflow javascript provid interfac wide rang browser capabl may flaw buffer overflow flaw allow attack write script would run code wish user 's system code mean limit anoth javascript applic exampl buffer overrun exploit allow attack gain access oper system 's api superus privileg buffer overflow api flaw affect major browser includ firefox 95 internet explor 96 safari 97 95 96 97 plugin video player adob flash wide rang activex control enabl default microsoft internet explor may also flaw exploit via javascript flaw exploit past 98 99 adob flash activex 98 99 window vista microsoft attempt contain risk bug buffer overflow run internet explor process limit privileg 100 googl chrome similarli confin page render `` sandbox '' 100 googl chrome sandbox sandbox implement error web browser capabl run javascript outsid sandbox privileg necessari exampl creat delet file privileg intend grant code web incorrectli grant privileg javascript web play role vulner internet explor 101 firefox 102 window xp servic pack 2 microsoft demot jscript 's privileg internet explor 103 101 102 103 microsoft window allow javascript sourc file comput 's hard drive launch general-purpos non-sandbox program see window script host make javascript like vbscript theoret viabl vector trojan hors although javascript trojan hors uncommon practic 104 fail verif microsoft window window script host vbscript trojan hors 104 fail verif fail verif hardwar vulner 2015 javascript-bas proof-of-concept implement rowhamm attack describ paper secur research 105 106 107 108 rowhamm 105 106 107 108 2017 javascript-bas attack via browser demonstr could bypass aslr call `` aslr⊕cach '' anc 109 110 aslr 109 110 2018 paper announc spectr attack specul execut intel processor includ javascript implement 111 spectr 111 develop tool import tool evolv languag everi major web browser built-in web develop tool includ javascript debugg web develop tool debugg static program analysi tool eslint jslint scan javascript code conform set standard guidelin static program analysi eslint jslint browser built-in profil stand-alon profil librari also creat benchmark.j jsbench 112 113 profil 112 113 mani text editor syntax highlight support javascript code text editor relat technolog java common misconcept javascript directli relat java inde c-like syntax c languag immedi common ancestor languag also typic sandbox javascript design java 's syntax standard librari mind particular java keyword reserv origin javascript javascript 's standard librari follow java 's name convent javascript 's math date object base class java 1.0 114 java sandbox math date 114 languag first appear 1995 java develop jame gosl sun microsystem javascript brendan eich netscap commun jame gosl brendan eich differ two languag promin similar java static type javascript 's type dynam java load compil bytecod javascript load human-read sourc code java 's object class-bas javascript 's prototype-bas final java support function program java 8 javascript done begin influenc scheme static type dynam compil class-bas prototype-bas scheme json json data format deriv javascript henc name javascript object notat wide use format support mani program languag json transpil mani websit javascript-heavi transpil creat convert code written languag aid develop process 36 transpil 36 typescript coffeescript two notabl languag transpil javascript typescript coffeescript webassembl webassembl newer languag bytecod format design complement javascript especi performance-crit portion web page script major javascript engin support webassembl 115 run sandbox regular javascript code webassembl bytecod web page javascript engin 115 sandbox asm.j subset javascript serv forerunn webassembl 116 asm.j 116 refer b `` netscap sun announc javascript open cross-platform object script languag enterpris network internet '' press releas 4 decemb 1995 archiv origin 16 septemb 2007 b b `` netscap sun announc javascript open cross-platform object script languag enterpris network internet '' press releas 4 decemb 1995 archiv origin 16 septemb 2007 `` netscap sun announc javascript open cross-platform object script languag enterpris network internet '' origin `` ecmascript® 2024 languag specif '' june 2024 retriev 30 august 2024 `` ecmascript® 2024 languag specif '' june 2024 retriev 30 august 2024 `` ecmascript® 2024 languag specif '' retriev 30 august 2024 30 august `` ecmascript® 2025 languag specif '' 27 march 2024 retriev 17 april 2024 `` ecmascript® 2025 languag specif '' 27 march 2024 retriev 17 april 2024 `` ecmascript® 2025 languag specif '' retriev 17 april 2024 17 april `` nodejs/node-ep '' github archiv origin 29 august 2020 retriev 5 juli 2018 `` nodejs/node-ep '' github archiv origin 29 august 2020 retriev 5 juli 2018 `` nodejs/node-ep '' archiv retriev 5 juli 2018 5 juli b seibel peter 16 septemb 2009 coder work reflect craft program apress isbn 978-1-4302-1948-4 archiv origin 24 decemb 2020 retriev 25 decemb 2018 eich immedi concern netscap must look like java b b seibel peter 16 septemb 2009 coder work reflect craft program apress isbn 978-1-4302-1948-4 archiv origin 24 decemb 2020 retriev 25 decemb 2018 eich immedi concern netscap must look like java coder work reflect craft program isbn 978-1-4302-1948-4 archiv retriev 25 decemb 2018 25 decemb b c e `` chapter 4 javascript creat '' speakingjs.com archiv origin 27 februari 2020 retriev 21 novemb 2017 b c e b c e `` chapter 4 javascript creat '' speakingjs.com archiv origin 27 februari 2020 retriev 21 novemb 2017 `` chapter 4 javascript creat '' archiv retriev 21 novemb 2017 21 novemb `` popular – brendan eich '' `` popular – brendan eich '' `` popular – brendan eich '' `` brendan eich introduct javascript jsconf 2010 '' youtub 20 januari 2013. p. 22m archiv origin 29 august 2020 retriev 25 novemb 2019 eich `` function '' eight letter influenc awk `` brendan eich introduct javascript jsconf 2010 '' youtub 20 januari 2013. p. 22m archiv origin 29 august 2020 retriev 25 novemb 2019 eich `` function '' eight letter influenc awk `` brendan eich introduct javascript jsconf 2010 '' youtub archiv retriev 25 novemb 2019 25 novemb eich brendan 1998 `` foreword '' goodman danni ed. javascript bibl 3rd ed. john wiley son isbn 0-7645-3188-3 lccn 97078208 oclc 38888873 ol 712205m eich brendan 1998 `` foreword '' goodman danni ed. javascript bibl 3rd ed. john wiley son isbn 0-7645-3188-3 lccn 97078208 oclc 38888873 ol 712205m eich brendan goodman danni javascript bibl javascript bibl john wiley son isbn 0-7645-3188-3 lccn 97078208 oclc 38888873 ol 712205m b `` usag statist javascript client-sid program languag websit '' w3tech retriev 27 februari 2024 b b `` usag statist javascript client-sid program languag websit '' w3tech retriev 27 februari 2024 `` usag statist javascript client-sid program languag websit '' retriev 27 februari 2024 27 februari `` ecmascript 2020 languag specif '' archiv origin 8 may 2020 retriev 8 may 2020 `` ecmascript 2020 languag specif '' archiv origin 8 may 2020 retriev 8 may 2020 `` ecmascript 2020 languag specif '' archiv retriev 8 may 2020 8 may `` bloomberg game changer marc andreessen '' bloomberg bloomberg 17 march 2011 archiv origin 16 may 2012 retriev 7 decemb 2011 `` bloomberg game changer marc andreessen '' bloomberg bloomberg 17 march 2011 archiv origin 16 may 2012 retriev 7 decemb 2011 `` bloomberg game changer marc andreessen '' bloomberg archiv retriev 7 decemb 2011 7 decemb enzer larri 31 august 2018 `` evolut web browser '' monmouth web develop archiv origin 31 august 2018 retriev 31 august 2018 enzer larri 31 august 2018 `` evolut web browser '' monmouth web develop archiv origin 31 august 2018 retriev 31 august 2018 `` evolut web browser '' origin retriev 31 august 2018 31 august b c fin j 17 june 2016 `` brendan eich – ceo brave '' youtub retriev 7 februari 2018 b c b c fin j 17 june 2016 `` brendan eich – ceo brave '' youtub retriev 7 februari 2018 `` brendan eich – ceo brave '' retriev 7 februari 2018 7 februari `` netscap commun corp. '' browser enhanc encyclopædia britannica 2006 ultim refer suit dvd `` netscap commun corp. '' browser enhanc encyclopædia britannica 2006 ultim refer suit dvd `` techvis innov net brendan eich javascript '' archiv origin 8 februari 2008 `` techvis innov net brendan eich javascript '' archiv origin 8 februari 2008 `` techvis innov net brendan eich javascript '' origin b han sheon 4 march 2024 `` javascript run world—mayb even liter '' wire retriev 21 august 2024 b b han sheon 4 march 2024 `` javascript run world—mayb even liter '' wire retriev 21 august 2024 `` javascript run world—mayb even liter '' retriev 21 august 2024 21 august b `` chapter 5 standard ecmascript '' speakingjs.com archiv origin 1 novemb 2021 retriev 1 novemb 2021 b b `` chapter 5 standard ecmascript '' speakingjs.com archiv origin 1 novemb 2021 retriev 1 novemb 2021 `` chapter 5 standard ecmascript '' origin retriev 1 novemb 2021 1 novemb b champeon steve 6 april 2001 `` javascript get '' oreilly.com archiv origin 19 juli 2016 retriev 16 juli 2016 b b champeon steve 6 april 2001 `` javascript get '' oreilly.com archiv origin 19 juli 2016 retriev 16 juli 2016 `` javascript get '' origin retriev 16 juli 2016 16 juli `` microsoft internet explor 3.0 beta avail '' microsoft.com microsoft 29 may 1996 archiv origin 24 novemb 2020 retriev 16 juli 2016 `` microsoft internet explor 3.0 beta avail '' microsoft.com microsoft 29 may 1996 archiv origin 24 novemb 2020 retriev 16 juli 2016 `` microsoft internet explor 3.0 beta avail '' archiv retriev 16 juli 2016 16 juli mccracken harri 16 septemb 2010 `` unwelcom return `` best view internet explor '' '' technologizer.com archiv origin 23 june 2018 retriev 16 juli 2016 mccracken harri 16 septemb 2010 `` unwelcom return `` best view internet explor '' '' technologizer.com archiv origin 23 june 2018 retriev 16 juli 2016 `` unwelcom return `` best view internet explor '' '' archiv retriev 16 juli 2016 16 juli baker loren 24 novemb 2004 `` mozilla firefox internet browser market share gain 7.4 '' search engin journal archiv origin 7 may 2021 retriev 8 may 2021 baker loren 24 novemb 2004 `` mozilla firefox internet browser market share gain 7.4 '' search engin journal archiv origin 7 may 2021 retriev 8 may 2021 `` mozilla firefox internet browser market share gain 7.4 '' archiv retriev 8 may 2021 8 may weber tim 9 may 2005 `` assault softwar giant microsoft '' bbc news archiv origin 25 septemb 2017 weber tim 9 may 2005 `` assault softwar giant microsoft '' bbc news archiv origin 25 septemb 2017 `` assault softwar giant microsoft '' bbc news origin `` big browser comparison test internet explor vs. firefox opera safari chrome '' pc game hardwar computec medium ag 3 juli 2009 archiv origin 2 may 2012 retriev 28 june 2010 `` big browser comparison test internet explor vs. firefox opera safari chrome '' pc game hardwar computec medium ag 3 juli 2009 archiv origin 2 may 2012 retriev 28 june 2010 `` big browser comparison test internet explor vs. firefox opera safari chrome '' archiv retriev 28 june 2010 28 june purdi kevin 11 june 2009 `` lifehack speed test safari 4 chrome 2 '' lifehack archiv origin 14 april 2021 retriev 8 may 2021 purdi kevin 11 june 2009 `` lifehack speed test safari 4 chrome 2 '' lifehack archiv origin 14 april 2021 retriev 8 may 2021 `` lifehack speed test safari 4 chrome 2 '' lifehack archiv retriev 8 may 2021 8 may `` tracemonkey javascript lightspe brendan eich 's blog '' archiv origin 4 decemb 2015 retriev 22 juli 2020 `` tracemonkey javascript lightspe brendan eich 's blog '' archiv origin 4 decemb 2015 retriev 22 juli 2020 `` tracemonkey javascript lightspe brendan eich 's blog '' archiv retriev 22 juli 2020 22 juli `` mozilla ask 'are fast yet '' wire archiv origin 22 june 2018 retriev 18 januari 2019 `` mozilla ask 'are fast yet '' wire archiv origin 22 june 2018 retriev 18 januari 2019 `` mozilla ask 'are fast yet '' archiv retriev 18 januari 2019 18 januari `` ecmascript 6 new featur overview comparison '' es6-features.org archiv origin 18 march 2018 retriev 19 march 2018 `` ecmascript 6 new featur overview comparison '' es6-features.org archiv origin 18 march 2018 retriev 19 march 2018 `` ecmascript 6 new featur overview comparison '' retriev 19 march 2018 19 march profession node.j build javascript base scalabl softwar archiv 2017-03-24 wayback machin john wiley son 01-oct-2012 profession node.j build javascript base scalabl softwar archiv 2017-03-24 wayback machin john wiley son 01-oct-2012 profession node.j build javascript base scalabl softwar archiv wayback machin sam teach node.j 24 hour archiv 2017-03-23 wayback machin sam publish 05-sep-2012 sam teach node.j 24 hour archiv 2017-03-23 wayback machin sam publish 05-sep-2012 sam teach node.j 24 hour archiv wayback machin lawton georg 19 juli 2018 `` secret histori behind success npm node '' theserversid archiv origin 2 august 2021 retriev 2 august 2021 lawton georg 19 juli 2018 `` secret histori behind success npm node '' theserversid archiv origin 2 august 2021 retriev 2 august 2021 `` secret histori behind success npm node '' archiv retriev 2 august 2021 2 august brown paul 13 januari 2017 `` state union npm '' linux.com archiv origin 2 august 2021 retriev 2 august 2021 brown paul 13 januari 2017 `` state union npm '' linux.com archiv origin 2 august 2021 retriev 2 august 2021 `` state union npm '' archiv retriev 2 august 2021 2 august b c branscomb mari 4 may 2016 `` javascript standard move yearli releas schedul 's new es16 '' new stack archiv origin 16 januari 2021 retriev 15 januari 2021 b c b c branscomb mari 4 may 2016 `` javascript standard move yearli releas schedul 's new es16 '' new stack archiv origin 16 januari 2021 retriev 15 januari 2021 `` javascript standard move yearli releas schedul 's new es16 '' archiv retriev 15 januari 2021 15 januari `` tc39 process '' tc39.e ecma intern archiv origin 7 februari 2021 retriev 15 januari 2021 `` tc39 process '' tc39.e ecma intern archiv origin 7 februari 2021 retriev 15 januari 2021 `` tc39 process '' archiv retriev 15 januari 2021 15 januari `` ecmascript propos '' tc39 archiv origin 4 decemb 2020 retriev 15 januari 2021 `` ecmascript propos '' tc39 archiv origin 4 decemb 2020 retriev 15 januari 2021 `` ecmascript propos '' archiv retriev 15 januari 2021 15 januari b ashkena jeremi `` list languag compil j '' github archiv origin 31 januari 2020 retriev 6 februari 2020 b b ashkena jeremi `` list languag compil j '' github archiv origin 31 januari 2020 retriev 6 februari 2020 ashkena jeremi `` list languag compil j '' github archiv retriev 6 februari 2020 6 februari `` u. trademark serial 75026640 '' uspto.gov unit state patent trademark offic 6 may 1997 archiv origin 13 juli 2021 retriev 8 may 2021 `` u. trademark serial 75026640 '' uspto.gov unit state patent trademark offic 6 may 1997 archiv origin 13 juli 2021 retriev 8 may 2021 `` u. trademark serial 75026640 '' unit state patent trademark offic archiv retriev 8 may 2021 8 may `` legal notic '' oracle.com oracl corpor archiv origin 5 june 2021 retriev 8 may 2021 `` legal notic '' oracle.com oracl corpor archiv origin 5 june 2021 retriev 8 may 2021 `` legal notic '' oracl corpor archiv retriev 8 may 2021 8 may `` oracl buy sun 7.4-bn deal '' econom time 21 april 2009 `` oracl buy sun 7.4-bn deal '' econom time 21 april 2009 `` oracl buy sun 7.4-bn deal '' `` javascript™️ '' `` javascript™️ '' `` javascript™️ '' b c `` usag statist javascript librari websit '' w3tech retriev 9 april 2021 b c b c `` usag statist javascript librari websit '' w3tech retriev 9 april 2021 `` usag statist javascript librari websit '' retriev 9 april 2021 9 april `` use jqueri bootstrap '' clouddevs.com 10 june 2019 retriev 17 march 2024 `` use jqueri bootstrap '' clouddevs.com 10 june 2019 retriev 17 march 2024 `` use jqueri bootstrap '' retriev 17 march 2024 17 march `` vanilla j '' vanilla-js.com 16 june 2020 archiv origin 16 june 2020 retriev 17 june 2020 `` vanilla j '' vanilla-js.com 16 june 2020 archiv origin 16 june 2020 retriev 17 june 2020 `` vanilla j '' archiv retriev 17 june 2020 17 june `` server-sid javascript guid '' oracle.com oracl corpor 11 decemb 1998 archiv origin 11 march 2021 retriev 8 may 2021 `` server-sid javascript guid '' oracle.com oracl corpor 11 decemb 1998 archiv origin 11 march 2021 retriev 8 may 2021 `` server-sid javascript guid '' oracl corpor archiv retriev 8 may 2021 8 may clinick andrew 14 juli 2000 `` introduc jscript .net '' microsoft develop network microsoft archiv origin 10 novemb 2017 retriev 10 april 2018 inc 1996 introduct jscript version 1.0 ... 've see steadi increas usag jscript server—particularli activ server page asp clinick andrew 14 juli 2000 `` introduc jscript .net '' microsoft develop network microsoft archiv origin 10 novemb 2017 retriev 10 april 2018 inc 1996 introduct jscript version 1.0 ... 've see steadi increas usag jscript server—particularli activ server page asp `` introduc jscript .net '' archiv retriev 10 april 2018 10 april b mahemoff michael 17 decemb 2009 `` server-sid javascript back vengeanc '' readwrite.com archiv origin 17 june 2016 retriev 16 juli 2016 b b mahemoff michael 17 decemb 2009 `` server-sid javascript back vengeanc '' readwrite.com archiv origin 17 june 2016 retriev 16 juli 2016 `` server-sid javascript back vengeanc '' archiv retriev 16 juli 2016 16 juli `` javascript acrobat '' adobe.com 7 august 2009 archiv origin 7 august 2009 retriev 18 august 2009 `` javascript acrobat '' adobe.com 7 august 2009 archiv origin 7 august 2009 retriev 18 august 2009 `` javascript acrobat '' archiv retriev 18 august 2009 18 august treitter 2 februari 2013 `` answer question `` develop app gnome '' '' livejournal.com archiv origin 11 februari 2013 retriev 7 februari 2013 treitter 2 februari 2013 `` answer question `` develop app gnome '' '' livejournal.com archiv origin 11 februari 2013 retriev 7 februari 2013 `` answer question `` develop app gnome '' '' archiv retriev 7 februari 2013 7 februari `` tessel 2 ... leverag librari node.j creat use devic minut tessel '' tessel.io archiv origin 26 may 2021 retriev 8 may 2021 `` tessel 2 ... leverag librari node.j creat use devic minut tessel '' tessel.io archiv origin 26 may 2021 retriev 8 may 2021 `` tessel 2 ... leverag librari node.j creat use devic minut tessel '' archiv retriev 8 may 2021 8 may `` node.j raspberri pi gpio introduct '' w3schools.com archiv origin 13 august 2021 retriev 3 may 2020 `` node.j raspberri pi gpio introduct '' w3schools.com archiv origin 13 august 2021 retriev 3 may 2020 `` node.j raspberri pi gpio introduct '' archiv retriev 3 may 2020 3 may `` espruino – javascript microcontrol '' espruino.com archiv origin 1 may 2020 retriev 3 may 2020 `` espruino – javascript microcontrol '' espruino.com archiv origin 1 may 2020 retriev 3 may 2020 `` espruino – javascript microcontrol '' archiv retriev 3 may 2020 3 may looper jen 21 septemb 2015 `` guid javascript engin idiot '' telerik develop network archiv origin 8 decemb 2018 retriev 8 decemb 2018 looper jen 21 septemb 2015 `` guid javascript engin idiot '' telerik develop network archiv origin 8 decemb 2018 retriev 8 decemb 2018 `` guid javascript engin idiot '' telerik origin retriev 8 decemb 2018 8 decemb `` blink work '' googl retriev 12 march 2024 `` blink work '' googl retriev 12 march 2024 `` blink work '' retriev 12 march 2024 12 march b `` document · v8 '' googl retriev 3 march 2024 b b `` document · v8 '' googl retriev 3 march 2024 `` document · v8 '' retriev 3 march 2024 3 march nelaturu keerthi `` webassembl 's big deal '' medium.com retriev 3 march 2024 nelaturu keerthi `` webassembl 's big deal '' medium.com retriev 3 march 2024 `` webassembl 's big deal '' retriev 3 march 2024 3 march `` concurr model event loop '' mozilla develop network archiv origin 5 septemb 2015 retriev 28 august 2015 `` concurr model event loop '' mozilla develop network archiv origin 5 septemb 2015 retriev 28 august 2015 `` concurr model event loop '' archiv retriev 28 august 2015 28 august flanagan david 17 august 2006 javascript definit guid definit guid `` o'reilli medium inc. '' p. 16 isbn 978-0-596-55447-7 archiv origin 1 august 2020 retriev 29 march 2019 flanagan david 17 august 2006 javascript definit guid definit guid `` o'reilli medium inc. '' p. 16 isbn 978-0-596-55447-7 archiv origin 1 august 2020 retriev 29 march 2019 javascript definit guid definit guid isbn 978-0-596-55447-7 archiv retriev 29 march 2019 29 march b c korolev mikhail 1 march 2019 `` javascript quirk one imag internet '' dev commun archiv origin 28 octob 2019 retriev 28 octob 2019 b c b c korolev mikhail 1 march 2019 `` javascript quirk one imag internet '' dev commun archiv origin 28 octob 2019 retriev 28 octob 2019 `` javascript quirk one imag internet '' archiv retriev 28 octob 2019 28 octob `` proposal-bigint/advanced.md master · tc39/proposal-bigint '' github `` proposal-bigint/advanced.md master · tc39/proposal-bigint '' github `` proposal-bigint/advanced.md master · tc39/proposal-bigint '' github bernhardt gari 2012 `` wat '' destroy softwar archiv origin 28 octob 2019 retriev 28 octob 2019 bernhardt gari 2012 `` wat '' destroy softwar archiv origin 28 octob 2019 retriev 28 octob 2019 `` wat '' archiv retriev 28 octob 2019 28 octob `` javascript data type data structur '' mdn 16 februari 2017 archiv origin 14 march 2017 retriev 24 februari 2017 `` javascript data type data structur '' mdn 16 februari 2017 archiv origin 14 march 2017 retriev 24 februari 2017 `` javascript data type data structur '' archiv retriev 24 februari 2017 24 februari flanagan 2006 pp 176–178 flanagan 2006 pp 176–178 flanagan 2006 crockford dougla `` prototyp inherit javascript '' archiv origin 13 august 2013 retriev 20 august 2013 crockford dougla `` prototyp inherit javascript '' archiv origin 13 august 2013 retriev 20 august 2013 `` prototyp inherit javascript '' archiv retriev 20 august 2013 20 august `` inherit prototyp chain '' mozilla develop network archiv origin 25 april 2013 retriev 6 april 2013 `` inherit prototyp chain '' mozilla develop network archiv origin 25 april 2013 retriev 6 april 2013 `` inherit prototyp chain '' mozilla archiv retriev 6 april 2013 6 april herman david 2013 effect javascript addison-wesley p. 83 isbn 978-0-321-81218-6 herman david 2013 effect javascript addison-wesley p. 83 isbn 978-0-321-81218-6 effect javascript isbn 978-0-321-81218-6 haverbek marijn 2011 eloqu javascript starch press pp 95–97 isbn 978-1-59327-282-1 haverbek marijn 2011 eloqu javascript starch press pp 95–97 isbn 978-1-59327-282-1 eloqu javascript 95– isbn 978-1-59327-282-1 katz yehuda 12 august 2011 `` understand `` prototyp '' javascript '' archiv origin 5 april 2013 retriev 6 april 2013 katz yehuda 12 august 2011 `` understand `` prototyp '' javascript '' archiv origin 5 april 2013 retriev 6 april 2013 `` understand `` prototyp '' javascript '' archiv retriev 6 april 2013 6 april herman david 2013 effect javascript addison-wesley pp 125–127 isbn 978-0-321-81218-6 herman david 2013 effect javascript addison-wesley pp 125–127 isbn 978-0-321-81218-6 effect javascript 125– isbn 978-0-321-81218-6 `` function – javascript '' mdn web doc retriev 30 octob 2021 `` function – javascript '' mdn web doc retriev 30 octob 2021 `` function – javascript '' mdn web doc retriev 30 octob 2021 30 octob `` properti function object '' es5.github.com archiv origin 28 januari 2013 retriev 26 may 2013 `` properti function object '' es5.github.com archiv origin 28 januari 2013 retriev 26 may 2013 `` properti function object '' archiv retriev 26 may 2013 26 may flanagan 2006 p. 141 flanagan 2006 p. 141 flanagan 2006 mani talent javascript gener role-ori program approach like trait mixin archiv 2017-10-05 wayback machin peterseliger.blogspot.d april 11 2014 mani talent javascript gener role-ori program approach like trait mixin archiv 2017-10-05 wayback machin peterseliger.blogspot.d april 11 2014 mani talent javascript gener role-ori program approach like trait mixin archiv wayback machin trait javascript archiv 2014-07-24 wayback machin 2010 trait javascript archiv 2014-07-24 wayback machin 2010 trait javascript archiv wayback machin `` home cocktailj '' cocktailjs.github.io archiv origin 4 februari 2017 retriev 24 februari 2017 `` home cocktailj '' cocktailjs.github.io archiv origin 4 februari 2017 retriev 24 februari 2017 `` home cocktailj '' archiv retriev 24 februari 2017 24 februari croll angu 31 may 2011 `` fresh look javascript mixin '' javascript javascript… archiv origin 15 april 2020 croll angu 31 may 2011 `` fresh look javascript mixin '' javascript javascript… archiv origin 15 april 2020 `` fresh look javascript mixin '' archiv haverbek marijn 2011 eloqu javascript starch press pp 139–149 isbn 978-1-59327-282-1 haverbek marijn 2011 eloqu javascript starch press pp 139–149 isbn 978-1-59327-282-1 eloqu javascript 139– isbn 978-1-59327-282-1 `` e4x – archiv obsolet content '' mozilla develop network mozilla foundat 14 februari 2014 archiv origin 24 juli 2014 retriev 13 juli 2014 `` e4x – archiv obsolet content '' mozilla develop network mozilla foundat 14 februari 2014 archiv origin 24 juli 2014 retriev 13 juli 2014 `` e4x – archiv obsolet content '' origin retriev 13 juli 2014 13 juli `` var – javascript '' mozilla develop network archiv origin 23 decemb 2012 retriev 22 decemb 2012 `` var – javascript '' mozilla develop network archiv origin 23 decemb 2012 retriev 22 decemb 2012 `` var – javascript '' mozilla develop network archiv retriev 22 decemb 2012 22 decemb `` let '' mdn web doc mozilla archiv origin 28 may 2019 retriev 27 june 2018 `` let '' mdn web doc mozilla archiv origin 28 may 2019 retriev 27 june 2018 `` let '' archiv retriev 27 june 2018 27 june `` const '' mdn web doc mozilla archiv origin 28 june 2018 retriev 27 june 2018 `` const '' mdn web doc mozilla archiv origin 28 june 2018 retriev 27 june 2018 `` const '' archiv retriev 27 june 2018 27 june `` ecmascript languag specif – ecma-262 edit 5.1 '' ecma intern archiv origin 26 novemb 2012 retriev 22 decemb 2012 `` ecmascript languag specif – ecma-262 edit 5.1 '' ecma intern archiv origin 26 novemb 2012 retriev 22 decemb 2012 `` ecmascript languag specif – ecma-262 edit 5.1 '' ecma intern archiv retriev 22 decemb 2012 22 decemb `` consol '' mozilla develop network mozilla archiv origin 28 februari 2013 retriev 6 april 2013 `` consol '' mozilla develop network mozilla archiv origin 28 februari 2013 retriev 6 april 2013 `` consol '' mozilla archiv retriev 6 april 2013 6 april `` argument '' mozilla develop network mozilla archiv origin 13 april 2013 retriev 6 april 2013 `` argument '' mozilla develop network mozilla archiv origin 13 april 2013 retriev 6 april 2013 `` argument '' mozilla archiv retriev 6 april 2013 6 april `` function javascript mdn '' developer.mozilla.org retriev 27 septemb 2022 `` function javascript mdn '' developer.mozilla.org retriev 27 septemb 2022 `` function javascript mdn '' retriev 27 septemb 2022 27 septemb `` javascript modul '' mdn web doc mozilla archiv origin 17 juli 2022 retriev 28 juli 2022 `` javascript modul '' mdn web doc mozilla archiv origin 17 juli 2022 retriev 28 juli 2022 `` javascript modul '' archiv retriev 28 juli 2022 28 juli `` make javascript safe advertis '' adsaf archiv origin 6 juli 2021 retriev 8 may 2021 `` make javascript safe advertis '' adsaf archiv origin 6 juli 2021 retriev 8 may 2021 `` make javascript safe advertis '' archiv retriev 8 may 2021 8 may `` secur ecma script se '' archiv origin 15 may 2013 retriev 26 may 2013 `` secur ecma script se '' archiv origin 15 may 2013 retriev 26 may 2013 `` secur ecma script se '' archiv retriev 26 may 2013 26 may `` googl caja project '' googl archiv origin 22 januari 2021 retriev 9 juli 2021 `` googl caja project '' googl archiv origin 22 januari 2021 retriev 9 juli 2021 `` googl caja project '' googl archiv retriev 9 juli 2021 9 juli `` mozilla cross-sit script vulner report fix – mozillazin talkback '' mozillazine.org archiv origin 21 juli 2011 retriev 24 februari 2017 `` mozilla cross-sit script vulner report fix – mozillazin talkback '' mozillazine.org archiv origin 21 juli 2011 retriev 24 februari 2017 `` mozilla cross-sit script vulner report fix – mozillazin talkback '' archiv retriev 24 februari 2017 24 februari kottelin thor 17 june 2008 `` right-click `` protect '' forget '' blog.anta.net archiv origin 9 august 2011 retriev 28 juli 2022 kottelin thor 17 june 2008 `` right-click `` protect '' forget '' blog.anta.net archiv origin 9 august 2011 retriev 28 juli 2022 `` right-click `` protect '' forget '' origin retriev 28 juli 2022 28 juli rehorik jan 29 novemb 2016 `` never put sensit data javascript '' serviceobject blog serviceobject archiv origin 3 june 2019 retriev 3 june 2019 rehorik jan 29 novemb 2016 `` never put sensit data javascript '' serviceobject blog serviceobject archiv origin 3 june 2019 retriev 3 june 2019 `` never put sensit data javascript '' archiv retriev 3 june 2019 3 june b lauinger tobia chaaban abdelberi arshad sajjad robertson william wilson christo kirda engin 21 decemb 2016 `` thou shalt depend analys use outdat javascript librari web '' pdf northeastern univers arxiv:1811.00918 doi:10.14722/ndss.2017.23414 isbn 978-1-891562-46-4 s2cid 17885720 archiv origin pdf 29 march 2017 retriev 28 juli 2022 b b lauinger tobia chaaban abdelberi arshad sajjad robertson william wilson christo kirda engin 21 decemb 2016 `` thou shalt depend analys use outdat javascript librari web '' pdf northeastern univers arxiv:1811.00918 doi:10.14722/ndss.2017.23414 isbn 978-1-891562-46-4 s2cid 17885720 archiv origin pdf 29 march 2017 retriev 28 juli 2022 `` thou shalt depend analys use outdat javascript librari web '' pdf arxiv 1811.00918 1811.00918 doi 10.14722/ndss.2017.23414 isbn 978-1-891562-46-4 s2cid 17885720 origin pdf retriev 28 juli 2022 28 juli collin keith 27 march 2016 `` one programm broke internet delet tini piec code '' quartz archiv origin 22 februari 2017 retriev 22 februari 2017 collin keith 27 march 2016 `` one programm broke internet delet tini piec code '' quartz archiv origin 22 februari 2017 retriev 22 februari 2017 `` one programm broke internet delet tini piec code '' archiv retriev 22 februari 2017 22 februari sc magazin uk develop 's 11 line delet code 'break internet archiv februari 23 2017 wayback machin sc magazin uk develop 's 11 line delet code 'break internet archiv februari 23 2017 wayback machin develop 's 11 line delet code 'break internet archiv wayback machin mozilla corpor buffer overflow crypto.signtext archiv 2014-06-04 wayback machin mozilla corpor buffer overflow crypto.signtext archiv 2014-06-04 wayback machin buffer overflow crypto.signtext archiv wayback machin festa paul 19 august 1998 `` buffer-overflow bug ie '' cnet archiv origin 25 decemb 2002 festa paul 19 august 1998 `` buffer-overflow bug ie '' cnet archiv origin 25 decemb 2002 `` buffer-overflow bug ie '' cnet origin securitytracker.com appl safari javascript buffer overflow let remot user execut arbitrari code http redirect bug let remot user access file archiv 2010-02-18 wayback machin securitytracker.com appl safari javascript buffer overflow let remot user execut arbitrari code http redirect bug let remot user access file archiv 2010-02-18 wayback machin appl safari javascript buffer overflow let remot user execut arbitrari code http redirect bug let remot user access file archiv wayback machin securityfocu microsoft webviewfoldericon activex control buffer overflow vulner archiv 2011-10-11 wayback machin securityfocu microsoft webviewfoldericon activex control buffer overflow vulner archiv 2011-10-11 wayback machin microsoft webviewfoldericon activex control buffer overflow vulner archiv wayback machin fusion author macromedia flash activex buffer overflow archiv august 13 2011 wayback machin fusion author macromedia flash activex buffer overflow archiv august 13 2011 wayback machin macromedia flash activex buffer overflow archiv wayback machin `` protect mode vista ie7 – ieblog '' blogs.msdn.com 9 februari 2006 archiv origin 23 januari 2010 retriev 24 februari 2017 `` protect mode vista ie7 – ieblog '' blogs.msdn.com 9 februari 2006 archiv origin 23 januari 2010 retriev 24 februari 2017 `` protect mode vista ie7 – ieblog '' archiv retriev 24 februari 2017 24 februari u cert vulner note vu 713878 microsoft internet explor properli valid sourc redirect frame archiv 2009-10-30 wayback machin u cert vulner note vu 713878 microsoft internet explor properli valid sourc redirect frame archiv 2009-10-30 wayback machin vulner note vu 713878 microsoft internet explor properli valid sourc redirect frame archiv wayback machin mozilla foundat mozilla foundat secur advisori 2005–41 privileg escal via dom properti overrid archiv 2014-06-04 wayback machin mozilla foundat mozilla foundat secur advisori 2005–41 privileg escal via dom properti overrid archiv 2014-06-04 wayback machin mozilla foundat secur advisori 2005–41 privileg escal via dom properti overrid archiv wayback machin andersen starr 9 august 2004 `` part 5 enhanc brow secur '' technet microsoft doc chang function window xp servic pack 2 retriev 20 octob 2021 andersen starr 9 august 2004 `` part 5 enhanc brow secur '' technet microsoft doc chang function window xp servic pack 2 retriev 20 octob 2021 `` part 5 enhanc brow secur '' technet microsoft doc retriev 20 octob 2021 20 octob one exampl rare javascript trojan hors see symantec corpor js.seeker.k archiv 2011-09-13 wayback machin one exampl rare javascript trojan hors see symantec corpor js.seeker.k archiv 2011-09-13 wayback machin js.seeker.k archiv wayback machin grus daniel mauric clémentin mangard stefan 24 juli 2015 `` rowhammer.j remot software-induc fault attack javascript '' arxiv:1507.06955 cs.cr grus daniel mauric clémentin mangard stefan 24 juli 2015 `` rowhammer.j remot software-induc fault attack javascript '' arxiv:1507.06955 cs.cr arxiv 1507.06955 1507.06955 cs.cr jean-pharun alix 30 juli 2015 `` rowhammer.j ingeni hack 've ever seen '' motherboard vice archiv origin 27 januari 2018 retriev 26 januari 2018 jean-pharun alix 30 juli 2015 `` rowhammer.j ingeni hack 've ever seen '' motherboard vice archiv origin 27 januari 2018 retriev 26 januari 2018 `` rowhammer.j ingeni hack 've ever seen '' vice archiv retriev 26 januari 2018 26 januari goodin dan 4 august 2015 `` dram 'bitflip exploit attack pc add javascript '' ar technica archiv origin 27 januari 2018 retriev 26 januari 2018 goodin dan 4 august 2015 `` dram 'bitflip exploit attack pc add javascript '' ar technica archiv origin 27 januari 2018 retriev 26 januari 2018 `` dram 'bitflip exploit attack pc add javascript '' ar technica archiv retriev 26 januari 2018 26 januari auerbach david 28 juli 2015 `` rowhamm secur exploit new secur attack truli terrifi '' slate.com archiv origin 30 juli 2015 retriev 29 juli 2015 auerbach david 28 juli 2015 `` rowhamm secur exploit new secur attack truli terrifi '' slate.com archiv origin 30 juli 2015 retriev 29 juli 2015 auerbach david `` rowhamm secur exploit new secur attack truli terrifi '' archiv retriev 29 juli 2015 29 juli anc archiv 2017-03-16 wayback machin vusec 2017 anc archiv 2017-03-16 wayback machin vusec 2017 anc archiv wayback machin new aslr-bust javascript make drive-bi exploit much nastier archiv 2017-03-16 wayback machin ar technica 2017 new aslr-bust javascript make drive-bi exploit much nastier archiv 2017-03-16 wayback machin ar technica 2017 new aslr-bust javascript make drive-bi exploit much nastier archiv wayback machin spectr attack archiv 2018-01-03 wayback machin spectr attack spectr attack archiv 2018-01-03 wayback machin spectr attack spectr attack archiv wayback machin `` benchmark.j '' benchmarkjs.com archiv origin 19 decemb 2016 retriev 6 novemb 2016 `` benchmark.j '' benchmarkjs.com archiv origin 19 decemb 2016 retriev 6 novemb 2016 `` benchmark.j '' archiv retriev 6 novemb 2016 6 novemb jsben.ch `` jsben.ch perform benchmark playground javascript '' jsben.ch archiv origin 27 februari 2021 retriev 13 august 2021 jsben.ch `` jsben.ch perform benchmark playground javascript '' jsben.ch archiv origin 27 februari 2021 retriev 13 august 2021 `` jsben.ch perform benchmark playground javascript '' archiv retriev 13 august 2021 13 august eich brendan 3 april 2008 `` popular '' archiv origin 3 juli 2011 retriev 19 januari 2012 eich brendan 3 april 2008 `` popular '' archiv origin 3 juli 2011 retriev 19 januari 2012 eich brendan `` popular '' archiv retriev 19 januari 2012 19 januari `` edg browser switch webassembl 'on -- visual studio magazin '' visual studio magazin archiv origin 10 februari 2018 retriev 9 februari 2018 `` edg browser switch webassembl 'on -- visual studio magazin '' visual studio magazin archiv origin 10 februari 2018 retriev 9 februari 2018 `` edg browser switch webassembl 'on -- visual studio magazin '' archiv retriev 9 februari 2018 9 februari `` frequent ask question '' asm.j archiv origin 4 june 2014 retriev 13 april 2014 `` frequent ask question '' asm.j archiv origin 4 june 2014 retriev 13 april 2014 `` frequent ask question '' archiv retriev 13 april 2014 13 april sourc dere mohan 21 decemb 2017 `` integr create-react-app librari need make great app '' freecodecamp retriev 14 june 2018 `` integr create-react-app librari need make great app '' retriev 14 june 2018 14 june panchal krunal 26 april 2022 `` angular v react detail comparison '' groovi web retriev 5 june 2023 `` angular v react detail comparison '' retriev 5 june 2023 5 june read ecmascript specif document flanagan david javascript definit guid 7th edit sebastopol california o'reilli 2020 isbn 978-1-491-95202-3 isbn 978-1-491-95202-3 haverbek marijn eloqu javascript 3rd edit starch press 2018 472 page isbn 978-1593279509 download isbn 978-1593279509 download zaka nichola principl object-ori javascript 1st edit starch press 2014 120 page isbn 978-1593275402 isbn 978-1593275402 extern link sister project sister project definit wiktionari definit wiktionari definit medium common medium common medium textbook wikibook textbook wikibook textbook resourc wikivers resourc wikivers resourc document mediawiki document mediawiki document listen articl 48 minut 48 audio file 2013-08-20 2013-08-20 audio help spoken articl modern javascript tutori commun maintain continu updat collect tutori entireti languag modern javascript tutori `` javascript first 20 year '' retriev 6 februari 2022 `` javascript first 20 year '' retriev 6 februari 2022 6 februari v v e e javascript code analysi eslint eslint jshint jshint jslint jslint superset js++ js++ typescript typescript arkt arkt transpil atscript atscript babel babel clojurescript clojurescript coffeescript coffeescript dart dart elm elm emscripten emscripten googl closur compil googl closur compil googl web toolkit googl web toolkit hax hax livescript livescript morfik morfik nim nim opa opa purescript purescript reason reason websharp websharp javascript librari javascript librari javascript syntax javascript syntax debugg chrome devtool chrome devtool firefox inspector firefox inspector komodo ide komodo ide microsoft edg devtool microsoft edg devtool opera devtool opera devtool safari web inspector safari web inspector doc gener jsdoc jsdoc comparison comparison ace cloud9 ide ace cloud9 ide cloud9 ide atom atom codemirror bracket light tabl codemirror bracket bracket light tabl light tabl phpstorm phpstorm orion orion visual studio visual studio express visual studio visual studio express visual studio express visual studio code visual studio code visual studio team servic visual studio team servic vim vim engin list ecmascript engin list ecmascript engin framework comparison javascript framework comparison javascript framework list javascript librari list javascript librari technolog ajax ajax assemblyscript assemblyscript asm.j asm.j cascad style sheet cascad style sheet document object model document object model html html5 html html5 html5 json json webassembl webassembl webauthn webauthn packag manag npm npm pnpm pnpm yarn yarn webpack webpack vite vite esbuild esbuild server-sid activ server page activ server page bun bun commonj commonj deno deno jsgi jsgi node.j node.j unit test list list jasmin jasmin mocha mocha qunit qunit dougla crockford dougla crockford ryan dahl ryan dahl brendan eich brendan eich john resig john resig v v e e program languag comparison comparison timelin timelin histori histori ada ada algol simula algol simula simula apl apl assembl assembl basic visual basic classic .net basic visual basic classic .net visual basic classic classic .net .net c c c++ c++ c c cobol cobol erlang erlang forth forth fortran fortran go go haskel haskel java java javascript javascript julia julia kotlin kotlin lisp lisp lua lua matlab matlab ml ml pascal object pascal pascal object pascal object pascal perl perl php php prolog prolog python python r r rubi rubi rust rust sql sql scratch scratch shell shell smalltalk smalltalk swift swift ... ... list alphabet alphabet categor categor gener gener non-english-bas non-english-bas categori categori v v e e ecmascript actionscript actionscript caja caja javascript engin asm.j javascript engin engin asm.j asm.j jscript jscript jscript .net jscript .net qtscript qtscript typescript typescript wmlscript wmlscript engin carakan carakan futhark futhark inscript inscript javascriptcor javascriptcor jscript jscript kj kj linear b linear b qtscript qtscript rhino rhino spidermonkey tracemonkey jägermonkey spidermonkey tracemonkey tracemonkey jägermonkey jägermonkey tamarin tamarin v8 v8 chakracor chakra chakracor chakra chakra jscript .net jscript .net nashorn nashorn framework client-sid dojo dojo echo echo ext j ext j googl web toolkit googl web toolkit jqueri jqueri live kernel live kernel midori mochikit mochikit mootool mootool prototyp prototyp pyj pyj qooxdoo qooxdoo sproutcor sproutcor spri spri wakanda framework server-sid node.j node.j deno deno bun bun jaxer jaxer appjet appjet wakandadb cappuccino librari backbone.j backbone.j swfobject swfobject underscore.j underscore.j brendan eich brendan eich dougla crockford dougla crockford john resig john resig scott isaac dhtml dhtml ecma intern ecma intern jsdoc jsdoc jsgi jsgi jshint jshint jslint jslint json json jsss jsss sputnik sputnik sunspid sunspid asynchron modul definit asynchron modul definit commonj commonj javascript librari ajax framework javascript framework server-sid javascript v v e e web browser featur standard protocol bookmark bookmark extens extens privaci mode privaci mode web standard html v5 html v5 v5 cs cs dom dom javascript webassembl web storag indexeddb webgl webgpu javascript webassembl webassembl web storag web storag indexeddb indexeddb webgl webgl webgpu webgpu http encrypt cooki third-parti http encrypt encrypt cooki third-parti cooki third-parti third-parti ocsp ocsp webrtc webrtc websocket websocket blink proprietari googl chrome googl chrome arc arc avast avast coc coc coc coc comodo comodo epic epic maxthon maxthon microsoft edg microsoft edg opera mobil opera mobil puffin puffin qq qq samsung samsung silk silk sleipnir sleipnir srware srware uc uc vivaldi vivaldi whale whale yandex yandex foss chromium chromium brave brave doobl doobl falkon falkon otter otter supermium supermium ungoogl ungoogl gecko firefox firefox floorp floorp gnu icecat gnu icecat librewolf librewolf midori midori slimbrows slimbrows tor tor zen zen gecko fork basilisk k-meleon pale moon seamonkey waterfox fork basilisk basilisk k-meleon k-meleon pale moon pale moon seamonkey seamonkey waterfox waterfox webkit safari safari gnome web gnome web icab icab orion orion engin 360 360 duckduckgo duckduckgo konqueror konqueror lunascap lunascap netfront netfront qutebrows qutebrows eww eww flow flow ladybird ladybird link link lynx lynx netsurf netsurf opera mini opera mini w3m w3m blink beaker beaker citrio citrio flock flock redcor redcor rockmelt rockmelt salamweb salamweb sputnik sputnik torch torch gecko beonex beonex camino camino classilla classilla conkeror conkeror firefox lite firefox lite galeon galeon ghostzilla ghostzilla icedragon icedragon kazehakas kazehakas kylo kylo lotu lotu microb microb minimo minimo mozilla suit mozilla suit piratebrows piratebrows pogo pogo stratum stratum swiftfox swiftfox swiftweasel swiftweasel tenfourfox tenfourfox timberwolf timberwolf xb xb mshtml internet explor internet explor aol aol deepnet deepnet greenbrows greenbrows mediabrows mediabrows neoplanet neoplanet netcaptor netcaptor spacetim spacetim zac zac webkit arora arora bolt bolt dolphin dolphin fluid fluid googl tv googl tv iri iri mercuri mercuri nokia symbian nokia symbian omniweb omniweb opera coast opera coast origyn origyn qtweb qtweb shiira shiira steel steel surf surf uzbl uzbl webposit webposit xombrero xombrero abaco abaco amaya amaya arachn arachn arena arena blazer blazer cake cake charon charon cm cm deepfish deepfish dillo dillo edg legaci edg legaci elink elink gazel gazel hotjava hotjava ibm home page reader ibm home page reader ibm webexplor ibm webexplor ibrows ibrows internet explor mac internet explor mac kidzui kidzui line mode line mode mosaic mosaic msn tv msn tv netposit netposit netscap netscap skweezer skweezer skyfir skyfir thunderhawk thunderhawk vision vision winwap winwap worldwideweb worldwideweb categori categori comparison comparison list list v v e e node.j node.j node.j npm npm v8 v8 commonj commonj mean mongodb express.j angularjs/angular mean mongodb mongodb express.j express.j angularjs/angular angularj angular meen substitut ember.j ember.j backbone.j backbone.j fastifi fastifi meteor meteor sails.j use express.j sails.j nestj nestj next.j next.j nuxt.j nuxt.j lodash lodash underscore.j underscore.j react.j react.j vue.j vue.j javascript javascript coffeescript coffeescript typescript typescript webassembl assemblyscript c rust webassembl assemblyscript assemblyscript c c rust rust portal portal comput program comput program author control databas fast fast fast germani germani germani unit state unit state unit state franc franc franc bnf data bnf data bnf data czech republ czech republ czech republ czech republ spain spain spain israel israel israel idref idref idref elmcip elmcip elmcip http //en.wikipedia.org/w/index.php title=javascript oldid=1263283332 categori javascript javascript american invent american invent cross-platform softwar cross-platform softwar dynam type program languag dynam type program languag function languag function languag object-bas program languag object-bas program languag high-level program languag high-level program languag program languag creat 1995 program languag creat 1995 program languag iso standard program languag iso standard prototype-bas program languag prototype-bas program languag script languag script languag web program web program program languag program languag cs1 unfit url cs1 unfit url webarch templat wayback link webarch templat wayback link articl short descript articl short descript short descript match wikidata short descript match wikidata wikipedia indefinit semi-protect page wikipedia indefinit semi-protect page use dmi date march 2024 use dmi date march 2024 articl excerpt articl excerpt articl unsourc statement articl unsourc statement articl unsourc statement august 2023 articl unsourc statement august 2023 articl fail verif articl fail verif articl fail verif march 2017 articl fail verif march 2017 page use sister project link wikidata namespac mismatch page use sister project link wikidata namespac mismatch page use sister project link hidden wikidata page use sister project link hidden wikidata page use sister project link default search page use sister project link default search articl haudio microformat articl haudio microformat spoken articl spoken articl articl exampl javascript code articl exampl javascript code page last edit 15 decemb 2024 19:41 utc utc text avail creativ common attribution-sharealik 4.0 licens addit term may appli use site agre term use privaci polici wikipedia® regist trademark wikimedia foundat inc. non-profit organ creativ common attribution-sharealik 4.0 licens term use privaci polici wikimedia foundat inc. privaci polici privaci polici wikipedia wikipedia disclaim disclaim contact wikipedia contact wikipedia code conduct code conduct develop develop statist statist cooki statement cooki statement mobil view mobil view",
    "https://en.wikipedia.org/wiki/Java_(programming_language)": "jump content main menu main page main page main page content content content current event current event current event random articl random articl random articl wikipedia wikipedia wikipedia contact u contact u contact u help help help learn edit learn edit learn edit commun portal commun portal commun portal recent chang recent chang recent chang upload file upload file upload file search search appear donat donat donat creat account creat account creat account log log log person tool donat donat donat creat account creat account creat account log log log learn learn contribut contribut contribut talk talk talk content top top 1 histori toggl histori subsect 1.1 principl 1.2 version 1 histori 1 histori toggl histori subsect 1.1 principl 1.1 principl 1.1 principl 1.2 version 1.2 version 1.2 version 2 edit 2 edit 2 edit 3 execut system toggl execut system subsect 3.1 java jvm bytecod 3.1.1 perform 3.2 non-jvm 3.3 automat memori manag 3 execut system 3 execut system toggl execut system subsect 3.1 java jvm bytecod 3.1.1 perform 3.1 java jvm bytecod 3.1 java jvm bytecod 3.1.1 perform 3.1.1 perform 3.1.1 perform 3.2 non-jvm 3.2 non-jvm 3.2 non-jvm 3.3 automat memori manag 3.3 automat memori manag 3.3 automat memori manag 4 syntax toggl syntax subsect 4.1 hello world 4 syntax 4 syntax toggl syntax subsect 4.1 hello world 4.1 hello world 4.1 hello world 5 special class toggl special class subsect 5.1 applet 5.2 servlet 5.3 javaserv page 5.4 swing applic 5.5 javafx applic 5.6 gener 5 special class 5 special class toggl special class subsect 5.1 applet 5.1 applet 5.1 applet 5.2 servlet 5.2 servlet 5.2 servlet 5.3 javaserv page 5.3 javaserv page 5.3 javaserv page 5.4 swing applic 5.4 swing applic 5.4 swing applic 5.5 javafx applic 5.5 javafx applic 5.5 javafx applic 5.6 gener 5.6 gener 5.6 gener 6 critic 6 critic 6 critic 7 class librari 7 class librari 7 class librari 8 document 8 document 8 document 9 implement 9 implement 9 implement 10 use outsid java platform toggl use outsid java platform subsect 10.1 android 10.1.1 controversi 10 use outsid java platform 10 use outsid java platform toggl use outsid java platform subsect 10.1 android 10.1.1 controversi 10.1 android 10.1 android 10.1.1 controversi 10.1.1 controversi 10.1.1 controversi 11 see also 11 see also 11 see also 12 refer 12 refer 12 refer 13 bibliographi 13 bibliographi 13 bibliographi 14 extern link 14 extern link 14 extern link toggl tabl content java program languag java program languag 117 languag acèh acèh acèh afrikaan afrikaan afrikaan አማርኛ አማርኛ አማርኛ العربية العربية العربية aragoné aragoné aragoné অসমীয়া অসমীয়া অসমীয়া asturianu asturianu asturianu azərbaycanca azərbaycanca azərbaycanca تۆرکجه تۆرکجه تۆرکجه বাংলা বাংলা বাংলা 閩南語 bân-lâm-gú 閩南語 bân-lâm-gú 閩南語 bân-lâm-gú беларуская беларуская беларуская беларуская тарашкевіца беларуская тарашкевіца беларуская тарашкевіца български български български bosanski bosanski bosanski català català català чӑвашла чӑвашла чӑвашла čeština čeština čeština corsu corsu corsu cymraeg cymraeg cymraeg dansk dansk dansk الدارجة الدارجة الدارجة deutsch deutsch deutsch eesti eesti eesti ελληνικά ελληνικά ελληνικά español español español esperanto esperanto esperanto estremeñu estremeñu estremeñu euskara euskara euskara فارسی فارسی فارسی françai françai françai gaeilg gaeilg gaeilg galego galego galego ગુજરાતી ગુજરાતી ગુજરાતી 客家語 hak-kâ-ngî 客家語 hak-kâ-ngî 客家語 hak-kâ-ngî 한국어 한국어 한국어 hausa hausa hausa հայերեն հայերեն հայերեն हिन्दी हिन्दी हिन्दी hrvatski hrvatski hrvatski ido ido ido bahasa indonesia bahasa indonesia bahasa indonesia interlingua interlingua interlingua íslenska íslenska íslenska italiano italiano italiano עברית עברית עברית ಕನ್ನಡ ಕನ್ನಡ ಕನ್ನಡ ქართული ქართული ქართული қазақша қазақша қазақша kiswahili kiswahili kiswahili kreyòl ayisyen kreyòl ayisyen kreyòl ayisyen kurdî kurdî kurdî кыргызча кыргызча кыргызча latina latina latina latviešu latviešu latviešu lietuvių lietuvių lietuvių ligur ligur ligur lombard lombard lombard magyar magyar magyar македонски македонски македонски മലയാളം മലയാളം മലയാളം मराठी मराठी मराठी مصرى مصرى مصرى bahasa melayu bahasa melayu bahasa melayu minangkabau minangkabau minangkabau 閩東語 mìng-dĕ̤ng-ngṳ̄ 閩東語 mìng-dĕ̤ng-ngṳ̄ 閩東語 mìng-dĕ̤ng-ngṳ̄ монгол монгол монгол မြန်မာဘာသာ မြန်မာဘာသာ မြန်မာဘာသာ na vosa vakav na vosa vakav na vosa vakav nederland nederland nederland नेपाली नेपाली नेपाली 日本語 日本語 日本語 norsk bokmål norsk bokmål norsk bokmål norsk nynorsk norsk nynorsk norsk nynorsk occitan occitan occitan ଓଡ଼ିଆ ଓଡ଼ିଆ ଓଡ଼ିଆ oʻzbekcha ўзбекча oʻzbekcha ўзбекча oʻzbekcha ўзбекча ਪੰਜਾਬੀ ਪੰਜਾਬੀ ਪੰਜਾਬੀ پنجابی پنجابی پنجابی polski polski polski portuguê portuguê portuguê qaraqalpaqsha qaraqalpaqsha qaraqalpaqsha română română română runa simi runa simi runa simi русский русский русский саха тыла саха тыла саха тыла संस्कृतम् संस्कृतम् संस्कृतम् shqip shqip shqip සිංහල සිංහල සිංහල simpl english simpl english simpl english سنڌي سنڌي سنڌي slovenčina slovenčina slovenčina slovenščina slovenščina slovenščina soomaaliga soomaaliga soomaaliga کوردی کوردی کوردی српски srpski српски srpski српски srpski srpskohrvatski српскохрватски srpskohrvatski српскохрватски srpskohrvatski српскохрватски suomi suomi suomi svenska svenska svenska tagalog tagalog tagalog தமிழ் தமிழ் தமிழ் taqbaylit taqbaylit taqbaylit татарча tatarça татарча tatarça татарча tatarça తెలుగు తెలుగు తెలుగు ไทย ไทย ไทย türkçe türkçe türkçe basa ugi basa ugi basa ugi українська українська українська اردو اردو اردو vèneto vèneto vèneto tiếng việt tiếng việt tiếng việt 文言 文言 文言 winaray winaray winaray 吴语 吴语 吴语 粵語 粵語 粵語 žemaitėška žemaitėška žemaitėška 中文 中文 中文 edit link edit link articl articl articl talk talk talk english read read read view sourc view sourc view sourc view histori view histori view histori tool read read read view sourc view sourc view sourc view histori view histori view histori link link link relat chang relat chang relat chang upload file upload file upload file special page special page special page perman link perman link perman link page inform page inform page inform cite page cite page cite page get shorten url get shorten url get shorten url download qr code download qr code download qr code download pdf download pdf download pdf printabl version printabl version printabl version wikimedia common wikimedia common wikimedia common wikibook wikibook wikibook wikiquot wikiquot wikiquot wikivers wikivers wikivers wikidata item wikidata item wikidata item java softwar platform javascript java javanes languag open frame paradigm multi-paradigm gener object-ori class-bas function imper reflect concurr design jame gosl develop oracl corpor 29 year ago 1995-05-23 1995-05-23 1 type disciplin static strong safe nomin manifest memori manag automat garbag collect filenam extens .class .jar .war oracle.com/java/ oracle.com/java/ oracle.com/java/ java.com java.com java.com dev.java dev.java dev.java clu 2 simula67 2 lisp 2 smalltalk 2 ada 83 c++ 3 c 4 eiffel 5 mesa 6 modula-3 7 oberon 8 objective-c 9 ucsd pascal 10 11 object pascal 12 ada 2005 beanshel c chapel 13 clojur ecmascript fantom gamba 14 groovi hack 15 hax j kotlin php python scala seed7 vala javascript js++ arkt java program wikibook java program java high-level class-bas object-ori program languag design implement depend possibl general-purpos program languag intend let programm write run anywher wora 16 mean compil java code run platform support java without need recompil 17 java applic typic compil bytecod run java virtual machin jvm regardless underli comput architectur syntax java similar c c++ fewer low-level facil either java runtim provid dynam capabl reflect runtim code modif typic avail tradit compil languag high-level class-bas object-ori program languag depend general-purpos programm wora 16 compil 17 bytecod java virtual machin comput architectur syntax c c++ low-level reflect java gain popular shortli releas popular program languag sinc 18 java third popular program languag 2022 updat accord github 19 although still wide popular gradual declin use java recent year languag use jvm gain popular 20 18 updat github 19 languag use jvm 20 java origin develop jame gosl sun microsystem releas may 1995 core compon sun 's java platform origin refer implement java compil virtual machin class librari origin releas sun proprietari licens may 2007 complianc specif java commun process sun relicens java technolog gpl-2.0-onli licens oracl offer hotspot java virtual machin howev offici refer implement openjdk jvm free open-sourc softwar use develop default jvm almost linux distribut jame gosl sun microsystem java platform refer implement compil class librari proprietari licens java commun process relicens gpl-2.0-onli oracl hotspot refer implement openjdk septemb 2024 updat java 23 latest version java 22 20 longer maintain java 8 11 17 21 previou lt version still offici support updat java 23 histori java softwar platform § histori jame gosl jame gosl mike sheridan patrick naughton initi java languag project june 1991 21 java origin design interact televis advanc digit cabl televis industri time 22 languag initi call oak oak tree stood outsid gosl 's offic later project went name green final renam java java coffe type coffe indonesia 23 gosl design java c/c++-style syntax system applic programm would find familiar 24 jame gosl patrick naughton 21 22 oak oak java coffe indonesia 23 c c++ 24 sun microsystem releas first public implement java 1.0 1996 25 promis write run anywher wora function provid no-cost run-tim popular platform fairli secur featur configur secur allow network- file-access restrict major web browser soon incorpor abil run java applet within web page java quickli becam popular java 1.0 compil re-written java arthur van hoff compli strictli java 1.0 languag specif 26 advent java 2 releas initi j2se 1.2 decemb 1998 – 1999 new version multipl configur built differ type platform j2ee includ technolog api enterpris applic typic run server environ j2me featur api optim mobil applic desktop version renam j2se 2006 market purpos sun renam new j2 version java ee java java se respect 25 write run anywher platform web browser java applet java arthur van hoff 26 j2ee java ee java java se 1997 sun microsystem approach iso/iec jtc 1 standard bodi later ecma intern formal java soon withdrew process 27 28 29 java remain de facto standard control java commun process 30 one time sun made java implement avail without charg despit proprietari softwar statu sun gener revenu java sell licens special product java enterpris system iso/iec jtc 1 ecma intern 27 28 29 de facto standard java commun process 30 proprietari softwar novemb 13 2006 sun releas much java virtual machin jvm free open-sourc softwar foss term gpl-2.0-onli licens may 8 2007 sun finish process make jvm 's core code avail free software/open-sourc distribut term asid small portion code sun hold copyright 31 free open-sourc softwar gpl-2.0-onli free softwar 31 sun 's vice-presid rich green said sun 's ideal role regard java evangelist 32 follow oracl corpor 's acquisit sun microsystem 2009–10 oracl describ steward java technolog relentless commit foster commun particip transpar 33 prevent oracl file lawsuit googl shortli use java insid android sdk see android section 32 oracl corpor 33 android sdk android april 2 2010 jame gosl resign oracl 34 oracl 34 januari 2016 oracl announc java run-tim environ base jdk 9 discontinu browser plugin 35 35 java softwar run everyth laptop data center game consol scientif supercomput 36 data center game consol supercomput 36 oracl other highli recommend uninstal outdat unsupport version java due unresolv secur issu older version 37 oracl 37 principl five primari goal creat java languag 17 17 must simpl object-ori familiar object-ori must robust secur robust must architecture-neutr portabl must execut high perform must interpret thread dynam interpret thread dynam version java version histori novemb 2024 updat java 8 11 17 21 support long-term support lt version java 25 releas septemb 2025 next schedul lt version 38 updat long-term support 38 oracl releas last zero-cost public updat legaci version java 8 lt januari 2019 commerci use although otherwis still support java 8 public updat person use indefinit vendor adoptium continu offer free build openjdk 's long-term support lt version build may includ addit secur patch bug fix 39 legaci java 8 adoptium 39 major releas version java along releas date beta 40 41 42 43 edit free java implement § class librari java platform java card java card java micro edit java java se standard edit java se jakarta ee enterpris edit jakarta ee javafx bundl oracl 's jdk version 8 10 separ sinc 11 javafx personaljava discontinu personaljava v v e e sun defin support four edit java target differ applic environ segment mani api belong one platform platform api java card smart-card 44 java card 44 java platform micro edit java – target environ limit resourc 45 java platform micro edit 45 java platform standard edit java se – target workstat environ 46 java platform standard edit 46 java platform enterpris edit java ee – target larg distribut enterpris internet environ 47 java platform enterpris edit 47 class java api organ separ group call packag packag contain set relat interfac class subpackag except class packag interfac except sun also provid edit call person java supersed later standards-bas java configuration-profil pair person java execut system java jvm bytecod java softwar platform java virtual machin one design goal java portabl mean program written java platform must run similarli combin hardwar oper system adequ run time support achiev compil java languag code intermedi represent call java bytecod instead directli architecture-specif machin code java bytecod instruct analog machin code intend execut virtual machin vm written specif host hardwar end-us commonli use java runtim environ jre instal devic standalon java applic web browser java applet portabl java bytecod machin code virtual machin end-us java runtim environ java applet standard librari provid gener way access host-specif featur graphic thread network thread network use univers bytecod make port simpl howev overhead interpret bytecod machin instruct made interpret program almost alway run slowli nativ execut just-in-tim jit compil compil byte-cod machin code runtim introduc earli stage java 's hotspot compil actual two compil one graalvm includ e.g java 11 remov java 16 allow tier compil 48 java platform-independ adapt particular platform run java virtual machin jvm translat java bytecod platform 's machin languag 49 interpret execut just-in-tim graalvm tier compil 48 java virtual machin java bytecod 49 perform java perform program written java reput slower requir memori written c++ 50 51 howev java program execut speed improv significantli introduct just-in-tim compil 1997/1998 java 1.1 52 addit languag featur support better code analysi inner class stringbuild class option assert etc optim java virtual machin hotspot becom sun 's default jvm 2000 java 1.5 perform improv addit java.util.concurr packag includ lock-fre implement concurrentmap multi-cor collect improv java 1.6 c++ 50 51 just-in-tim compil java 1.1 52 hotspot lock-fre concurrentmap non-jvm platform offer direct hardwar support java micro control run java bytecod hardwar instead softwar java virtual machin 53 arm-bas processor could hardwar support execut java bytecod jazel option though support mostli drop current implement arm 53 arm jazel automat memori manag java use automat garbag collector manag memori object lifecycl programm determin object creat java runtim respons recov memori object longer use refer object remain unreach memori becom elig freed automat garbag collector someth similar memori leak may still occur programm 's code hold refer object longer need typic object longer need store contain still use 54 method non-exist object call null pointer except thrown 55 56 automat garbag collector object lifecycl unreach memori memori leak 54 null pointer 55 56 one idea behind java 's automat memori manag model programm spare burden perform manual memori manag languag memori creation object implicitli alloc stack explicitli alloc dealloc heap latter case respons manag memori resid programm program dealloc object memori leak occur 54 program attempt access dealloc memori alreadi dealloc result undefin difficult predict program like becom unstabl crash partial remedi use smart pointer add overhead complex garbag collect prevent logic memori leak i.e memori still referenc never use 54 stack heap memori leak 54 smart pointer logic memori 54 garbag collect may happen time ideal occur program idl guarante trigger insuffici free memori heap alloc new object caus program stall momentarili explicit memori manag possibl java java support c/c++ style pointer arithmet object address arithmet manipul e.g ad subtract offset allow garbag collector reloc referenc object ensur type safeti secur pointer arithmet c++ object-ori languag variabl java 's primit data type either store directli field object stack method rather heap commonli true non-primit data type see escap analysi consciou decis java 's design perform reason primit data type stack escap analysi java contain multipl type garbag collector sinc java 9 hotspot use garbag first garbag collector g1gc default 57 howev also sever garbag collector use manag heap z garbag collector zgc introduc java 11 shenandoah gc introduc java 12 unavail oracle-produc openjdk build shenandoah instead avail third-parti build openjdk eclips temurin applic java g1gc suffici prior version java java 8 parallel garbag collector use default garbag collector garbag first garbag collector 57 eclips temurin parallel garbag collector solv memori manag problem reliev programm burden handl properli kind resourc like network databas connect file handl etc. especi presenc except syntax java syntax gephi syntax java larg influenc c++ c. unlik c++ combin syntax structur gener object-ori program java built almost exclus object-ori languag 17 code written insid class everi data item object except primit data type i.e integ floating-point number boolean valu charact object perform reason java reus popular aspect c++ printf method c++ c 17 boolean valu printf unlik c++ java support oper overload 58 multipl inherit class though multipl inherit support interfac 59 oper overload 58 multipl inherit interfac 59 java use comment similar c++ three differ style comment singl line style mark two slash // multipl line style open close javadoc comment style open close javadoc style comment allow user run javadoc execut creat document program read integr develop environ ide eclips allow develop access document within ide comment javadoc integr develop environ eclips hello world follow simpl exampl `` hello world '' program write messag standard output `` hello world '' program standard output public class exampl public static void main string arg system println `` hello world '' special class verif plea help improv articl ad citat reliabl sourc section unsourc materi may challeng removed.find sourc `` java '' program languag – news · newspap · book · scholar · jstor improv articl ad citat reliabl sourc find sourc `` java '' program languag – news · newspap · book · scholar · jstor `` java '' program languag news newspap book scholar jstor may 2019 may 2019 learn remov messag learn remov messag applet java applet java applet program embed applic mainli web page display web browser java applet api deprec releas java 9 2017 60 61 embed 60 61 servlet java servlet java servlet technolog provid web develop simpl consist mechan extend function web server access exist busi system servlet server-sid java ee compon gener respons request client time mean gener html page respons http request although number standard servlet class avail exampl websocket commun java servlet server-sid client html http websocket java servlet api extent supersed still use hood two standard java technolog web servic java api rest web servic jax-r 2.0 use ajax json rest servic java api rest web servic java api xml web servic jax-w use soap web servic java api xml web servic soap web servic typic implement api applic server servlet contain use standard servlet handl interact http request respons deleg web servic method actual busi logic http javaserv page javaserv page javaserv page jsp server-sid java ee compon gener respons typic html page http request client jsp emb java code html page use special delimit jsp compil java servlet java applic right first time access gener servlet creat respons 62 jsp server-sid html http client delimit 62 swing applic swing java swing graphic user interfac librari java se platform possibl specifi differ look feel pluggabl look feel system swing clone window gtk+ motif suppli sun appl also provid aqua look feel maco prior implement look feel may consid lack swing java se 6 address problem use nativ gui widget draw routin underli platform 63 swing librari pluggabl look feel window gtk+ motif appl aqua maco gui widget 63 javafx applic javafx javafx softwar platform creat deliv desktop applic well rich web applic run across wide varieti devic javafx intend replac swing standard graphic user interfac gui librari java se sinc jdk 11 javafx core jdk instead separ modul 64 javafx support desktop comput web browser microsoft window linux maco javafx support nativ o look feel 65 javafx softwar platform desktop applic rich web applic swing graphic user interfac java se 64 desktop comput web browser microsoft window linux maco 65 gener gener java 2004 gener ad java languag part j2se 5.0 prior introduct gener variabl declar specif type contain class exampl problem easi way creat contain accept specif type object either contain oper subtyp class interfac usual object differ contain class creat contain class gener allow compile-tim type check without creat mani contain class contain almost ident code addit enabl effici code certain runtim except prevent occur issu compile-tim error java prevent runtim type error classcastexcept occur would type safe gener type safe 2016 type system java proven unsound possibl use gener construct class method allow assign instanc one class variabl anoth unrel class code accept compil fail run time class cast except 66 unsound 66 critic critic java critic direct java includ implement gener 67 speed 50 handl unsign number 68 implement floating-point arithmet 69 histori secur vulner primari java vm implement hotspot 70 develop critic complex verbos java persist api jpa standard part java ee led increas adopt higher-level abstract like spring data jpa aim simplifi databas oper reduc boilerpl code grow popular framework suggest limit standard jpa implement 's ease-of-us modern java develop 71 67 50 68 69 hotspot 70 71 class librari java class librari java class librari standard librari develop support applic develop java control oracl cooper other java commun process program 72 compani individu particip process influenc design develop api process subject controversi 2010 73 class librari contain featur java class librari standard librari oracl java commun process 72 73 core librari includ input/output i/o io 74 non-block i/o nio io/nio 75 network 76 new user agent http client sinc java 11 77 reflect program reflect concurr comput concurr 74 gener script compil function program lambda stream collect librari implement data structur list dictionari tree set queue double-end queue stack 78 xml process par transform valid librari secur 79 internation local librari 80 input/output i/o io 74 non-block i/o nio io/nio 75 input/output 74 non-block i/o 75 network 76 new user agent http client sinc java 11 77 network 76 user agent 77 reflect program reflect reflect program concurr comput concurr 74 concurr comput 74 gener gener script compil function program lambda stream function program lambda collect librari implement data structur list dictionari tree set queue double-end queue stack 78 collect librari data structur list dictionari tree set queue double-end queue stack 78 xml process par transform valid librari xml secur 79 secur 79 internation local librari 80 internation local 80 integr librari allow applic writer commun extern system librari includ java databas connect jdbc api databas access java name directori interfac jndi lookup discoveri java remot method invoc rmi common object request broker architectur corba distribut applic develop java manag extens jmx manag monitor applic java databas connect jdbc api databas access java databas connect api java name directori interfac jndi lookup discoveri java name directori interfac java remot method invoc rmi common object request broker architectur corba distribut applic develop java remot method invoc common object request broker architectur java manag extens jmx manag monitor applic java manag extens user interfac librari includ heavyweight nativ abstract window toolkit awt provid gui compon mean lay compon mean handl event compon lightweight swing librari built awt provid non-n implement awt widgetri api audio captur process playback javafx user interfac heavyweight nativ abstract window toolkit awt provid gui compon mean lay compon mean handl event compon nativ abstract window toolkit gui lightweight swing librari built awt provid non-n implement awt widgetri swing api audio captur process playback javafx javafx platform depend implement java virtual machin mean bytecod java librari third-parti applic execut plugin enabl applet run web browser applet java web start allow java applic effici distribut end user across internet java web start end user licens document document javadoc javadoc comprehens document system creat sun microsystem provid develop organ system document code javadoc comment extra asterisk begin i.e delimit wherea normal multi-lin comment java delimit single-lin comment start // 81 sun microsystem 81 implement free java implement list java compil verif plea help improv articl ad citat reliabl sourc section unsourc materi may challeng removed.find sourc `` java '' program languag – news · newspap · book · scholar · jstor improv articl ad citat reliabl sourc find sourc `` java '' program languag – news · newspap · book · scholar · jstor `` java '' program languag news newspap book scholar jstor septemb 2023 septemb 2023 learn remov messag learn remov messag oracl corpor own offici implement java se platform due acquisit sun microsystem januari 27 2010 implement base origin implement java sun oracl implement avail window maco linux solari java lack formal standard recogn ecma intern iso/iec ansi third-parti standard organ oracl implement de facto standard oracl corpor sun microsystem window maco linux solari ecma intern de facto standard oracl implement packag two differ distribut java runtim environ jre contain part java se platform requir run java program intend end user java develop kit jdk intend softwar develop includ develop tool java compil javadoc jar debugg oracl also releas graalvm high perform java dynam compil interpret java develop kit java compil javadoc jar debugg graalvm openjdk anoth java se implement licens gnu gpl implement start sun began releas java sourc code gpl java se 7 openjdk offici java refer implement openjdk goal java make implement java compat histor sun 's trademark licens usag java brand insist implement compat result legal disput microsoft sun claim microsoft implement support java remot method invoc rmi java nativ interfac jni ad platform-specif featur sun su 1997 2001 settlement u 20 million well court order enforc term licens sun 82 result microsoft longer ship java window microsoft java remot method invoc java nativ interfac 82 window platform-independ java essenti java ee even rigor valid requir certifi implement environ enabl portabl server-sid applic java ee use outsid java platform java program languag requir presenc softwar platform order compil program execut oracl suppli java platform use java android sdk altern softwar platform use primarili develop android applic gui system java platform android sdk android applic android java languag key pillar android open sourc mobil oper system although android built linux kernel written larg c android sdk use java languag basi android applic use standard gui se establish java standard 83 bytecod languag support android sdk incompat java bytecod run virtual machin optim low-memori devic smartphon tablet comput depend android version bytecod either interpret dalvik virtual machin compil nativ code android runtim android open sourc mobil oper system linux kernel android sdk 83 smartphon tablet comput dalvik virtual machin android runtim android provid full java se standard librari although android sdk includ independ implement larg subset support java 6 java 7 featur offer implement compat standard librari apach harmoni apach harmoni controversi oracl america inc. v. googl inc use java-rel technolog android led legal disput oracl googl may 7 2012 san francisco juri found api could copyright googl infring oracl 's copyright use java android devic 84 district judg william alsup rule may 31 2012 api copyright 85 revers unit state court appeal feder circuit may 2014 86 may 26 2016 district court decid favor googl rule copyright infring java api android constitut fair use 87 march 2018 rule overturn appeal court sent case determin damag feder court san francisco 88 googl file petit writ certiorari suprem court unit state januari 2019 challeng two rule made appeal court oracl 's favor 89 april 5 2021 court rule 6–2 googl 's favor use java api consid fair use howev court refus rule copyright api choos instead determin rule consid java 's api copyright `` pure argument 's sake `` 90 84 william alsup 85 86 87 88 writ certiorari suprem court unit state 89 fair use 90 see also comput program portal comput program portal comput program portal c c c++ c++ dalvik use old android version replac non-jit android runtim dalvik android runtim java heterogen distribut comput java heterogen distribut comput list java api list java api list java framework list java framework list jvm languag list jvm languag list java virtual machin list java virtual machin comparison c java comparison c java comparison java c++ comparison java c++ comparison program languag comparison program languag refer binstock andrew may 20 2015 `` java 's 20 year innov '' forb archiv origin march 14 2016 retriev march 18 2016 binstock andrew may 20 2015 `` java 's 20 year innov '' forb archiv origin march 14 2016 retriev march 18 2016 `` java 's 20 year innov '' archiv retriev march 18 2016 march 18 b c barbara liskov john guttag 2000 program develop java – abstract specif object-ori design usa addison wesley isbn 978-0-201-65768-5 b c b c barbara liskov john guttag 2000 program develop java – abstract specif object-ori design usa addison wesley isbn 978-0-201-65768-5 barbara liskov john guttag isbn 978-0-201-65768-5 chaudhari harri h. juli 28 2014 `` crack java program interview 2000+ java interview que/an '' archiv origin septemb 29 2023 retriev may 29 2016 chaudhari harri h. juli 28 2014 `` crack java program interview 2000+ java interview que/an '' archiv origin septemb 29 2023 retriev may 29 2016 `` crack java program interview 2000+ java interview que/an '' archiv retriev may 29 2016 may 29 java 5.0 ad sever new languag featur enhanc loop autobox vararg annot introduc similar compet c languag 1 archiv march 19 2011 wayback machin 2 archiv januari 7 2006 wayback machin java 5.0 ad sever new languag featur enhanc loop autobox vararg annot introduc similar compet c languag 1 archiv march 19 2011 wayback machin 2 archiv januari 7 2006 wayback machin enhanc loop autobox vararg annot c 1 archiv wayback machin 2 archiv wayback machin gosl jame mcgilton henri may 1996 `` java languag environ '' archiv origin may 6 2014 retriev may 6 2014 gosl jame mcgilton henri may 1996 `` java languag environ '' archiv origin may 6 2014 retriev may 6 2014 `` java languag environ '' archiv retriev may 6 2014 may 6 gosl jame joy bill steel guy bracha gilad `` java languag specif 2nd edit '' archiv origin august 5 2011 retriev februari 8 2008 gosl jame joy bill steel guy bracha gilad `` java languag specif 2nd edit '' archiv origin august 5 2011 retriev februari 8 2008 `` java languag specif 2nd edit '' archiv retriev februari 8 2008 februari 8 `` a-z program languag modula-3 '' computerworld archiv origin januari 5 2009 retriev june 9 2010 `` a-z program languag modula-3 '' computerworld archiv origin januari 5 2009 retriev june 9 2010 `` a-z program languag modula-3 '' origin retriev june 9 2010 june 9 niklau wirth state number public occas e.g lectur polytechn museum moscow septemb 2005 sever independ first-hand account russian exist e.g one audio record filippova elena septemb 22 2005 `` niklau wirth 's lectur polytechn museum moscow '' archiv origin decemb 1 2020 retriev novemb 20 2011 sun java design team licens oberon compil sourc number year prior releas java examin rel compact type safeti garbag collect multipl inherit class – key overal design featur share java oberon niklau wirth state number public occas e.g lectur polytechn museum moscow septemb 2005 sever independ first-hand account russian exist e.g one audio record filippova elena septemb 22 2005 `` niklau wirth 's lectur polytechn museum moscow '' archiv origin decemb 1 2020 retriev novemb 20 2011 sun java design team licens oberon compil sourc number year prior releas java examin rel compact type safeti garbag collect multipl inherit class – key overal design featur share java oberon niklau wirth `` niklau wirth 's lectur polytechn museum moscow '' archiv retriev novemb 20 2011 novemb 20 patrick naughton cite objective-c strong influenc design java program languag state notabl direct deriv includ java interfac deriv objective-c 's protocol primit wrapper class 3 archiv juli 13 2011 wayback machin patrick naughton cite objective-c strong influenc design java program languag state notabl direct deriv includ java interfac deriv objective-c 's protocol primit wrapper class 3 archiv juli 13 2011 wayback machin patrick naughton objective-c protocol 3 archiv wayback machin techmetrix research 1999 `` histori java '' pdf java applic server report archiv origin pdf decemb 29 2010 project went ahead name green languag base old model ucsd pascal make possibl gener interpret code techmetrix research 1999 `` histori java '' pdf java applic server report archiv origin pdf decemb 29 2010 project went ahead name green languag base old model ucsd pascal make possibl gener interpret code `` histori java '' pdf origin pdf ucsd pascal `` convers jame gosl – acm queue '' queue.acm.org august 31 2004 archiv origin juli 16 2015 retriev june 9 2010 `` convers jame gosl – acm queue '' queue.acm.org august 31 2004 archiv origin juli 16 2015 retriev june 9 2010 `` convers jame gosl – acm queue '' archiv retriev june 9 2010 june 9 java languag team microsoft 's `` deleg '' white paper javasoft sun microsystem inc. archiv origin june 27 2012 summer 1996 sun design precursor event model awt javabean compon architectur borland contribut greatli process look care delphi object pascal built work prototyp bound method refer order understand interact java program languag api java languag team microsoft 's `` deleg '' white paper javasoft sun microsystem inc. archiv origin june 27 2012 summer 1996 sun design precursor event model awt javabean compon architectur borland contribut greatli process look care delphi object pascal built work prototyp bound method refer order understand interact java program languag api microsoft 's `` deleg '' origin `` chapel spec acknowledg '' pdf cray inc. octob 1 2015 archiv pdf origin februari 5 2016 retriev januari 14 2016 `` chapel spec acknowledg '' pdf cray inc. octob 1 2015 archiv pdf origin februari 5 2016 retriev januari 14 2016 `` chapel spec acknowledg '' pdf archiv pdf retriev januari 14 2016 januari 14 `` gamba document introduct '' gamba websit archiv origin octob 9 2017 retriev octob 9 2017 `` gamba document introduct '' gamba websit archiv origin octob 9 2017 retriev octob 9 2017 `` gamba document introduct '' archiv retriev octob 9 2017 octob 9 `` facebook q hack bring static type php world '' infoworld march 26 2014 archiv origin februari 13 2015 retriev januari 11 2015 `` facebook q hack bring static type php world '' infoworld march 26 2014 archiv origin februari 13 2015 retriev januari 11 2015 `` facebook q hack bring static type php world '' archiv retriev januari 11 2015 januari 11 `` write run anywher '' comput weekli may 2 2002 archiv origin august 13 2021 retriev juli 27 2009 `` write run anywher '' comput weekli may 2 2002 archiv origin august 13 2021 retriev juli 27 2009 `` write run anywher '' comput weekli archiv retriev juli 27 2009 juli 27 b c `` 1.2 design goal java program languag '' oracl januari 1 1999 archiv origin januari 23 2013 retriev januari 14 2013 b c b c `` 1.2 design goal java program languag '' oracl januari 1 1999 archiv origin januari 23 2013 retriev januari 14 2013 `` 1.2 design goal java program languag '' archiv retriev januari 14 2013 januari 14 melanson mike august 9 2022 `` n't call comeback java still champ '' github archiv origin august 25 2023 retriev octob 15 2023 melanson mike august 9 2022 `` n't call comeback java still champ '' github archiv origin august 25 2023 retriev octob 15 2023 `` n't call comeback java still champ '' github archiv retriev octob 15 2023 octob 15 `` top program languag '' state octovers github archiv origin august 2 2023 retriev octob 15 2023 `` top program languag '' state octovers github archiv origin august 2 2023 retriev octob 15 2023 `` top program languag '' github archiv retriev octob 15 2023 octob 15 mcmillan robert august 1 2013 `` java lose mojo '' wire archiv origin februari 15 2017 retriev octob 15 2023 mcmillan robert august 1 2013 `` java lose mojo '' wire archiv origin februari 15 2017 retriev octob 15 2023 `` java lose mojo '' `` java lose mojo '' wire archiv retriev octob 15 2023 octob 15 byou jon c. 1998 `` java technolog earli year '' sun develop network sun microsystem archiv origin april 20 2005 retriev april 22 2005 byou jon c. 1998 `` java technolog earli year '' sun develop network sun microsystem archiv origin april 20 2005 retriev april 22 2005 `` java technolog earli year '' sun microsystem origin retriev april 22 2005 april 22 object-ori program `` histori java technolog '' sun develop network c. 1995 archiv origin februari 10 2010 retriev april 30 2010 object-ori program `` histori java technolog '' sun develop network c. 1995 archiv origin februari 10 2010 retriev april 30 2010 `` histori java technolog '' origin retriev april 30 2010 april 30 murphi kieron octob 4 1996 `` decid call java '' javaworld archiv origin juli 13 2020 retriev juli 13 2020 murphi kieron octob 4 1996 `` decid call java '' javaworld archiv origin juli 13 2020 retriev juli 13 2020 `` decid call java '' javaworld archiv retriev juli 13 2020 juli 13 kabutz heinz upon oak archiv april 13 2007 wayback machin artima retriev april 29 2007 kabutz heinz upon oak archiv april 13 2007 wayback machin artima retriev april 29 2007 upon oak archiv wayback machin `` javasoft ship java 1.0 '' archiv origin march 10 2007 retriev may 13 2018 `` javasoft ship java 1.0 '' archiv origin march 10 2007 retriev may 13 2018 `` javasoft ship java 1.0 '' origin retriev may 13 2018 may 13 object-ori program java essenti applic tata mcgraw-hil educ p. 34 object-ori program java essenti applic tata mcgraw-hil educ p. 34 object-ori program java essenti applic `` jsg – java studi group '' open-std.org archiv origin august 25 2006 retriev august 2 2006 `` jsg – java studi group '' open-std.org archiv origin august 25 2006 retriev august 2 2006 `` jsg – java studi group '' archiv retriev august 2 2006 august 2 `` java – – standard twice '' pdf archiv pdf origin januari 13 2014 retriev june 3 2018 `` java – – standard twice '' pdf archiv pdf origin januari 13 2014 retriev june 3 2018 `` java – – standard twice '' pdf archiv pdf retriev june 3 2018 june 3 `` ecma—and microsoft care '' zdnet archiv origin may 6 2014 retriev may 6 2014 `` ecma—and microsoft care '' zdnet archiv origin may 6 2014 retriev may 6 2014 `` ecma—and microsoft care '' zdnet archiv retriev may 6 2014 may 6 `` java commun process websit '' jcp.org may 24 2010 archiv origin august 8 2006 retriev june 9 2010 `` java commun process websit '' jcp.org may 24 2010 archiv origin august 8 2006 retriev june 9 2010 `` java commun process websit '' archiv retriev june 9 2010 june 9 `` javaon sun – bulk java open sourc '' grnlight.net archiv origin may 27 2014 retriev may 26 2014 `` javaon sun – bulk java open sourc '' grnlight.net archiv origin may 27 2014 retriev may 26 2014 `` javaon sun – bulk java open sourc '' archiv retriev may 26 2014 may 26 `` sun 's evolv role java evangelist '' o'reilli medium archiv origin septemb 15 2010 retriev august 2 2009 `` sun 's evolv role java evangelist '' o'reilli medium archiv origin septemb 15 2010 retriev august 2 2009 `` sun 's evolv role java evangelist '' o'reilli medium archiv retriev august 2 2009 august 2 `` oracl java '' oracle.com oracl corpor archiv origin januari 31 2010 retriev august 23 2010 oracl lead substant support java sinc emerg 1995 take new role steward java technolog relentless commit foster commun particip transpar `` oracl java '' oracle.com oracl corpor archiv origin januari 31 2010 retriev august 23 2010 oracl lead substant support java sinc emerg 1995 take new role steward java technolog relentless commit foster commun particip transpar `` oracl java '' origin retriev august 23 2010 august 23 gosl jame april 9 2010 `` time move ... '' new road archiv origin novemb 5 2010 retriev novemb 16 2011 gosl jame april 9 2010 `` time move ... '' new road archiv origin novemb 5 2010 retriev novemb 16 2011 gosl jame `` time move ... '' origin retriev novemb 16 2011 novemb 16 topic dalibor `` move plugin-fre web '' archiv origin march 16 2016 retriev march 15 2016 topic dalibor `` move plugin-fre web '' archiv origin march 16 2016 retriev march 15 2016 `` move plugin-fre web '' archiv retriev march 15 2016 march 15 `` learn java technolog '' oracl archiv origin novemb 24 2011 retriev novemb 21 2011 `` learn java technolog '' oracl archiv origin novemb 24 2011 retriev novemb 21 2011 `` learn java technolog '' archiv retriev novemb 21 2011 novemb 21 `` uninstal older version java system '' oracl archiv origin februari 12 2018 retriev septemb 24 2021 `` uninstal older version java system '' oracl archiv origin februari 12 2018 retriev septemb 24 2021 `` uninstal older version java system '' archiv retriev septemb 24 2021 septemb 24 `` oracl java se support roadmap '' oracl septemb 13 2021 archiv origin septemb 19 2021 retriev septemb 18 2021 `` oracl java se support roadmap '' oracl septemb 13 2021 archiv origin septemb 19 2021 retriev septemb 18 2021 `` oracl java se support roadmap '' archiv retriev septemb 18 2021 septemb 18 `` temurin™ support adoptium '' adoptium.net archiv origin march 29 2024 retriev march 29 2024 `` temurin™ support adoptium '' adoptium.net archiv origin march 29 2024 retriev march 29 2024 `` temurin™ support adoptium '' archiv retriev march 29 2024 march 29 `` javasoft ship java 1.0 '' sun.com archiv origin march 10 2007 retriev februari 5 2008 `` javasoft ship java 1.0 '' sun.com archiv origin march 10 2007 retriev februari 5 2008 `` javasoft ship java 1.0 '' origin retriev februari 5 2008 februari 5 chander sharat `` introduc java se 11 '' oracle.com archiv origin septemb 26 2018 retriev septemb 26 2018 chander sharat `` introduc java se 11 '' oracle.com archiv origin septemb 26 2018 retriev septemb 26 2018 `` introduc java se 11 '' archiv retriev septemb 26 2018 septemb 26 chander sharat septemb 15 2020 `` arriv java 15 '' oracl archiv origin septemb 16 2020 retriev septemb 15 2020 chander sharat septemb 15 2020 `` arriv java 15 '' oracl archiv origin septemb 16 2020 retriev septemb 15 2020 `` arriv java 15 '' oracl archiv retriev septemb 15 2020 septemb 15 `` jdk 21 '' openjdk.org archiv origin septemb 20 2023 retriev septemb 20 2023 `` jdk 21 '' openjdk.org archiv origin septemb 20 2023 retriev septemb 20 2023 `` jdk 21 '' archiv retriev septemb 20 2023 septemb 20 `` java card overview '' oracl technolog network oracl archiv origin januari 7 2015 retriev decemb 18 2014 `` java card overview '' oracl technolog network oracl archiv origin januari 7 2015 retriev decemb 18 2014 `` java card overview '' archiv retriev decemb 18 2014 decemb 18 `` java platform micro edit java '' oracl technolog network oracl archiv origin januari 4 2015 retriev decemb 18 2014 `` java platform micro edit java '' oracl technolog network oracl archiv origin januari 4 2015 retriev decemb 18 2014 `` java platform micro edit java '' archiv retriev decemb 18 2014 decemb 18 `` java se '' oracl technolog network oracl archiv origin decemb 24 2014 retriev decemb 18 2014 `` java se '' oracl technolog network oracl archiv origin decemb 24 2014 retriev decemb 18 2014 `` java se '' archiv retriev decemb 18 2014 decemb 18 `` java platform enterpris edit java ee '' oracl technolog network oracl archiv origin decemb 17 2014 retriev decemb 18 2014 `` java platform enterpris edit java ee '' oracl technolog network oracl archiv origin decemb 17 2014 retriev decemb 18 2014 `` java platform enterpris edit java ee '' archiv retriev decemb 18 2014 decemb 18 `` deep dive new java jit compil – graal baeldung '' www.baeldung.com august 6 2021 archiv origin octob 28 2021 retriev octob 13 2021 `` deep dive new java jit compil – graal baeldung '' www.baeldung.com august 6 2021 archiv origin octob 28 2021 retriev octob 13 2021 `` deep dive new java jit compil – graal baeldung '' archiv retriev octob 13 2021 octob 13 `` jvm java virtual machin platform depend platform independ advantag use jvm java translat languag '' programm interview archiv origin januari 19 2015 retriev januari 19 2015 `` jvm java virtual machin platform depend platform independ advantag use jvm java translat languag '' programm interview archiv origin januari 19 2015 retriev januari 19 2015 `` jvm java virtual machin platform depend platform independ advantag use jvm java translat languag '' archiv retriev januari 19 2015 januari 19 b jelov dejan `` java alway slower c++ '' archiv origin februari 11 2008 retriev februari 15 2008 b b jelov dejan `` java alway slower c++ '' archiv origin februari 11 2008 retriev februari 15 2008 `` java alway slower c++ '' origin retriev februari 15 2008 februari 15 hundt robert `` loop recognit c++/java/go/scala '' pdf archiv pdf origin novemb 16 2011 retriev juli 12 2012 hundt robert `` loop recognit c++/java/go/scala '' pdf archiv pdf origin novemb 16 2011 retriev juli 12 2012 `` loop recognit c++/java/go/scala '' pdf archiv pdf retriev juli 12 2012 juli 12 `` symantec 's just-in-tim java compil integr sun jdk 1.1 '' archiv origin june 28 2010 retriev august 1 2009 `` symantec 's just-in-tim java compil integr sun jdk 1.1 '' archiv origin june 28 2010 retriev august 1 2009 `` symantec 's just-in-tim java compil integr sun jdk 1.1 '' origin retriev august 1 2009 august 1 salcic zoran park heejong teich jürgen malik avinash nadeem muhammad juli 22 2017 `` noc-hmp heterogen multicor processor embed system design systemj '' acm transact design autom electron system 22 4 73. doi:10.1145/3073416 issn 1084-4309 s2cid 11150290 salcic zoran park heejong teich jürgen malik avinash nadeem muhammad juli 22 2017 `` noc-hmp heterogen multicor processor embed system design systemj '' acm transact design autom electron system 22 4 73. doi:10.1145/3073416 issn 1084-4309 s2cid 11150290. doi 10.1145/3073416 issn 1084-4309 s2cid 11150290 b c bloch 2018 p. 26-28 §item 7 elimin obsolet object refer b c b c bloch 2018 p. 26-28 §item 7 elimin obsolet object refer bloch 2018 `` nullpointerexcept '' oracl archiv origin may 6 2014 retriev may 6 2014 `` nullpointerexcept '' oracl archiv origin may 6 2014 retriev may 6 2014 `` nullpointerexcept '' archiv retriev may 6 2014 may 6 `` except java '' artima.com archiv origin januari 21 2009 retriev august 10 2010 `` except java '' artima.com archiv origin januari 21 2009 retriev august 10 2010 `` except java '' archiv retriev august 10 2010 august 10 `` java hotspot™ virtual machin perform enhanc '' oracle.com archiv origin may 29 2017 retriev april 26 2017 `` java hotspot™ virtual machin perform enhanc '' oracle.com archiv origin may 29 2017 retriev april 26 2017 `` java hotspot™ virtual machin perform enhanc '' archiv retriev april 26 2017 april 26 `` oper overload c v java '' c java develop microsoft archiv origin januari 7 2015 retriev decemb 10 2014 `` oper overload c v java '' c java develop microsoft archiv origin januari 7 2015 retriev decemb 10 2014 `` oper overload c v java '' archiv retriev decemb 10 2014 decemb 10 `` multipl inherit state implement type '' java tutori oracl archiv origin novemb 9 2014 retriev decemb 10 2014 `` multipl inherit state implement type '' java tutori oracl archiv origin novemb 9 2014 retriev decemb 10 2014 `` multipl inherit state implement type '' archiv retriev decemb 10 2014 decemb 10 `` deprec api featur option '' oracl archiv origin june 19 2019 retriev may 31 2019 `` deprec api featur option '' oracl archiv origin june 19 2019 retriev may 31 2019 `` deprec api featur option '' archiv retriev may 31 2019 may 31 `` applet java platform se 7 '' doc oracl archiv origin august 2 2020 retriev may 1 2020 `` applet java platform se 7 '' doc oracl archiv origin august 2 2020 retriev may 1 2020 `` applet java platform se 7 '' archiv retriev may 1 2020 may 1 `` jsp page java ee 5 tutori '' docs.oracle.com archiv origin august 2 2020 retriev may 1 2020 `` jsp page java ee 5 tutori '' docs.oracle.com archiv origin august 2 2020 retriev may 1 2020 `` jsp page java ee 5 tutori '' archiv retriev may 1 2020 may 1 `` trail creat gui jfc/swing java tutori '' docs.oracle.com archiv origin april 29 2020 retriev may 1 2020 `` trail creat gui jfc/swing java tutori '' docs.oracle.com archiv origin april 29 2020 retriev may 1 2020 `` trail creat gui jfc/swing java tutori '' archiv retriev may 1 2020 may 1 `` remov jdk 11 javafx 11 arriv standalon modul '' infoworld septemb 20 2018 archiv origin octob 14 2020 retriev octob 13 2020 `` remov jdk 11 javafx 11 arriv standalon modul '' infoworld septemb 20 2018 archiv origin octob 14 2020 retriev octob 13 2020 `` remov jdk 11 javafx 11 arriv standalon modul '' archiv retriev octob 13 2020 octob 13 `` get start javafx hello world javafx style '' javafx 2 tutori document oracl archiv origin august 2 2020 retriev may 1 2020 `` get start javafx hello world javafx style '' javafx 2 tutori document oracl archiv origin august 2 2020 retriev may 1 2020 `` get start javafx hello world javafx style '' archiv retriev may 1 2020 may 1 `` java scala 's type system unsound '' pdf archiv pdf origin novemb 28 2016 retriev februari 20 2017 `` java scala 's type system unsound '' pdf archiv pdf origin novemb 28 2016 retriev februari 20 2017 `` java scala 's type system unsound '' pdf archiv pdf retriev februari 20 2017 februari 20 arnold ken june 27 2005 `` gener consid harm '' java.net archiv origin octob 10 2007 retriev septemb 10 2015 arnold ken june 27 2005 `` gener consid harm '' java.net archiv origin octob 10 2007 retriev septemb 10 2015 `` gener consid harm '' origin retriev septemb 10 2015 septemb 10 owen sean r. `` java unsign int unsign short unsign byte unsign long etc rather lack thereof '' archiv origin februari 20 2009 retriev juli 4 2011 owen sean r. `` java unsign int unsign short unsign byte unsign long etc rather lack thereof '' archiv origin februari 20 2009 retriev juli 4 2011 `` java unsign int unsign short unsign byte unsign long etc rather lack thereof '' origin retriev juli 4 2011 juli 4 kahan william march 1 1998 `` java 's floating-point hurt everyon everywher – acm 1998 workshop java stanford '' pdf electr engin comput scienc univers california berkeley archiv pdf origin septemb 5 2012 retriev june 4 2011 kahan william march 1 1998 `` java 's floating-point hurt everyon everywher – acm 1998 workshop java stanford '' pdf electr engin comput scienc univers california berkeley archiv pdf origin septemb 5 2012 retriev june 4 2011 `` java 's floating-point hurt everyon everywher – acm 1998 workshop java stanford '' pdf archiv pdf retriev june 4 2011 june 4 `` check java '' archiv origin septemb 21 2012 retriev decemb 23 2011 `` check java '' archiv origin septemb 21 2012 retriev decemb 23 2011 `` check java '' origin retriev decemb 23 2011 decemb 23 chidest ashlan java persist api jenkin aw isbn 9798224253951 retriev septemb 16 2024 chidest ashlan java persist api jenkin aw isbn 9798224253951 retriev septemb 16 2024 java persist api jenkin aw isbn 9798224253951 retriev septemb 16 2024 septemb 16 cadenhead roger novemb 20 2017 understand java program work archiv origin august 13 2021 retriev march 26 2019 cadenhead roger novemb 20 2017 understand java program work archiv origin august 13 2021 retriev march 26 2019 understand java program work archiv retriev march 26 2019 march 26 woolf nicki may 26 2016 `` googl win six-year legal battl oracl android code copyright '' guardian issn 0261-3077 archiv origin march 26 2019 retriev march 26 2019 woolf nicki may 26 2016 `` googl win six-year legal battl oracl android code copyright '' guardian issn 0261-3077 archiv origin march 26 2019 retriev march 26 2019 `` googl win six-year legal battl oracl android code copyright '' issn 0261-3077 archiv retriev march 26 2019 march 26 b bloch 2018 pp 1–4 § 1 introduct b b bloch 2018 pp 1–4 § 1 introduct bloch 2018 `` java.nio java platform se 8 '' docs.oracle.com `` java.nio java platform se 8 '' docs.oracle.com `` java.nio java platform se 8 '' `` java network '' docs.oracle.com `` java network '' docs.oracle.com `` java network '' `` httpclient java se 11 jdk 11 '' docs.oracle.com `` httpclient java se 11 jdk 11 '' docs.oracle.com `` httpclient java se 11 jdk 11 '' `` collect framework overview '' java document oracl archiv origin decemb 31 2014 retriev decemb 18 2014 `` collect framework overview '' java document oracl archiv origin decemb 31 2014 retriev decemb 18 2014 `` collect framework overview '' archiv retriev decemb 18 2014 decemb 18 `` java secur overview '' java document oracl archiv origin januari 3 2015 retriev decemb 18 2014 `` java secur overview '' java document oracl archiv origin januari 3 2015 retriev decemb 18 2014 `` java secur overview '' archiv retriev decemb 18 2014 decemb 18 `` trail internation '' java tutori oracl archiv origin decemb 31 2014 retriev decemb 18 2014 `` trail internation '' java tutori oracl archiv origin decemb 31 2014 retriev decemb 18 2014 `` trail internation '' archiv retriev decemb 18 2014 decemb 18 `` write doc comment javadoc tool '' oracl technolog network oracl archiv origin decemb 18 2014 retriev decemb 18 2014 `` write doc comment javadoc tool '' oracl technolog network oracl archiv origin decemb 18 2014 retriev decemb 18 2014 `` write doc comment javadoc tool '' archiv retriev decemb 18 2014 decemb 18 niccolai jame januari 24 2001 `` sun microsoft settl java lawsuit '' javaworld idg news servic archiv origin juli 14 2020 retriev juli 13 2020 niccolai jame januari 24 2001 `` sun microsoft settl java lawsuit '' javaworld idg news servic archiv origin juli 14 2020 retriev juli 13 2020 `` sun microsoft settl java lawsuit '' javaworld idg news servic archiv retriev juli 13 2020 juli 13 van gurp jill novemb 13 2007 `` googl android initi impress critic '' javalobbi archiv origin august 28 2008 retriev march 7 2009 frankli n't understand googl intend ignor vast amount exist implement seem like bad case `` invent '' ultim slow adopt alreadi mani java platform mobil world yet anoth one van gurp jill novemb 13 2007 `` googl android initi impress critic '' javalobbi archiv origin august 28 2008 retriev march 7 2009 frankli n't understand googl intend ignor vast amount exist implement seem like bad case `` invent '' ultim slow adopt alreadi mani java platform mobil world yet anoth one `` googl android initi impress critic '' retriev march 7 2009 march 7 mullin joe may 7 2012 `` googl guilti infring oracl trial futur legal headach loom '' law disord ar technica archiv origin may 8 2012 retriev may 8 2012 mullin joe may 7 2012 `` googl guilti infring oracl trial futur legal headach loom '' law disord ar technica archiv origin may 8 2012 retriev may 8 2012 `` googl guilti infring oracl trial futur legal headach loom '' archiv retriev may 8 2012 may 8 mullin joe may 31 2012 `` googl win crucial api rule oracl 's case decim '' ar technica archiv origin march 12 2017 retriev june 1 2012 mullin joe may 31 2012 `` googl win crucial api rule oracl 's case decim '' ar technica archiv origin march 12 2017 retriev june 1 2012 `` googl win crucial api rule oracl 's case decim '' archiv retriev june 1 2012 june 1 rosenblatt seth may 9 2014 `` court side oracl android java patent appeal '' cnet archiv origin may 10 2014 retriev may 10 2014 rosenblatt seth may 9 2014 `` court side oracl android java patent appeal '' cnet archiv origin may 10 2014 retriev may 10 2014 `` court side oracl android java patent appeal '' archiv retriev may 10 2014 may 10 mullin joe may 26 2016 `` googl beat oracle—android make `` fair use '' java api '' ar technica archiv origin januari 20 2017 retriev may 26 2016 mullin joe may 26 2016 `` googl beat oracle—android make `` fair use '' java api '' ar technica archiv origin januari 20 2017 retriev may 26 2016 `` googl beat oracle—android make `` fair use '' java api '' archiv retriev may 26 2016 may 26 farivar cyru march 27 2018 `` `` googl 's use java api packag fair '' appeal court rule '' ar technica archiv origin septemb 24 2019 retriev august 6 2019 farivar cyru march 27 2018 `` `` googl 's use java api packag fair '' appeal court rule '' ar technica archiv origin septemb 24 2019 retriev august 6 2019 `` `` googl 's use java api packag fair '' appeal court rule '' archiv retriev august 6 2019 august 6 lee timothi april 23 2019 `` googl ask suprem court overrul disastr rule api copyright '' ar technica archiv origin april 23 2019 retriev april 23 2019 lee timothi april 23 2019 `` googl ask suprem court overrul disastr rule api copyright '' ar technica archiv origin april 23 2019 retriev april 23 2019 `` googl ask suprem court overrul disastr rule api copyright '' ar technica archiv retriev april 23 2019 april 23 `` googl llc v. oracl america inc 593 u. s. ____ 2021 '' pdf archiv pdf origin april 5 2021 retriev april 6 2021 `` googl llc v. oracl america inc 593 u. s. ____ 2021 '' pdf archiv pdf origin april 5 2021 retriev april 6 2021 `` googl llc v. oracl america inc 593 u. s. ____ 2021 '' pdf archiv pdf retriev april 6 2021 april 6 bibliographi bloch joshua 2018 `` effect java program languag guid '' third ed. addison-wesley isbn 978-0-13-468599-1 isbn 978-0-13-468599-1 gosl jame joy bill steel guy bracha gilad buckley alex 2014 java® languag specif pdf java se 8 ed. archiv pdf origin octob 21 2014 retriev novemb 18 2014 java® languag specif pdf archiv pdf retriev novemb 18 2014 novemb 18 gosl jame joy bill steel guy l. jr. bracha gilad 2005 java languag specif 3rd ed. addison-wesley isbn 0-321-24678-0 archiv origin februari 14 2012 retriev februari 8 2019 joy bill steel guy l. jr. bracha gilad java languag specif isbn 0-321-24678-0 archiv retriev februari 8 2019 februari 8 lindholm tim yellin frank 1999 java virtual machin specif 2nd ed. addison-wesley isbn 0-201-43294-3 archiv origin septemb 25 2011 retriev februari 8 2019 java virtual machin specif isbn 0-201-43294-3 archiv retriev februari 8 2019 februari 8 extern link sister project sister project definit wiktionari definit wiktionari definit medium common medium common medium quotat wikiquot quotat wikiquot quotat textbook wikibook textbook wikibook textbook resourc wikivers resourc wikivers resourc data wikidata data wikidata data java platform enterpris edition/java ee tutori java weekli java weekli v v e e java softwar platform jvm jvm java micro java java se standard java se jakarta ee enterpris jakarta ee java card java card android sdk android sdk graalvm graalvm squawk squawk java develop kit java develop kit openjdk openjdk java virtual machin java virtual machin javafx javafx maxin vm maxin vm servlet servlet midlet midlet jsp jsp jsf jsf web start jnlp web start pack200 pack200 modul modul applet applet eclips eclips gnu classpath gnu classpath hibern hibern spring spring gwt gwt icedtea icedtea jazel jazel strut strut toplink toplink wildfli wildfli blackdown blackdown harmoni harmoni java version histori java version histori java commun process java commun process sun microsystem sun microsystem free java implement free java implement jvm languag java java beanshel beanshel clojur clojur groovi groovi jrubi jrubi jython jython kotlin kotlin process process rhino rhino scala scala oxygen oxygen confer javaon javaon devoxx devoxx apach softwar foundat apach softwar foundat eclips foundat eclips foundat java commun process java commun process oracl corpor oracl corpor sun microsystem sun microsystem laboratori sun microsystem sun microsystem laboratori jame gosl jame gosl arthur van hoff arthur van hoff ur hölzle ur hölzle patrick naughton patrick naughton categori comput program portal v v e e sun microsystem acquir oracl sun-1 sun-1 sun-2 sun-2 sun-3 sun-3 sun386i sun386i sun-4 sun-4 sparcstation/server/cent 1 2 4 5 10 20 ipc ipx lx zx sparcstation/server/cent 1 1 2 2 4 4 5 5 10 10 20 20 ipc ipc ipx ipx lx lx zx zx sparcclass sparcclass netra netra ultra 1 2 5 10 24 30 60 80 ultra 1 1 2 2 5 5 10 10 24 24 30 30 60 60 80 80 enterpris 10000 enterpris 10000 10000 blade server blade server blade workstat blade workstat fire 12k 15k e25k t2000 x4500 fire 12k 12k 15k 15k e25k e25k t2000 t2000 x4500 x4500 sparc enterpris sparc enterpris javast javast java workstat java workstat ray ray cobalt qube cobalt qube cobalt raq cobalt raq sparc sparc mb86900 mb86900 microsparc microsparc supersparc supersparc ultrasparc ultrasparc ultrasparc ii ultrasparc iie ultrasparc iii gemini ultrasparc ii ultrasparc iie ultrasparc iii gemini ultrasparc iii ultrasparc iii cu ultrasparc iiii ultrasparc iii ultrasparc iii cu ultrasparc iiii ultrasparc iv ultrasparc iv ultrasparc t1 ultrasparc t1 ultrasparc t2 ultrasparc t2 sparc t3 sparc t3 sparc t4 sparc t4 sparc t5 sparc t5 rock rock majc majc sun4d sun4d sbu sbu fireplan fireplan lom port lom port mbu mbu modular datacent modular datacent neptun neptun system servic processor system servic processor sparc seri sparc seri spot spot ultra port architectur ultra port architectur visual instruct set visual instruct set suno suno solari solari ni nis+ ni nis+ nis+ nf nf zf zfs+ zf zfs+ zfs+ sunview sunview news news openwindow openwindow java desktop system java desktop system studio studio java java staroffic staroffic iplanet iplanet java system java system sun secur global desktop sun secur global desktop mysql mysql xvm xvm glassfish glassfish virtualbox virtualbox storagetek 5800 system storagetek 5800 system storagetek sl8500 storagetek sl8500 open storag open storag qf qf zf zf cloud cloud constel system constel system visual system visual system grid engin grid engin lustr lustr sun microsystem laboratori sun microsystem laboratori picojava picojava fortress fortress project look glass project look glass scp scp bluej bluej common develop distribut licens common develop distribut licens java commun process java commun process netbean netbean openoffice.org openoffice.org opensolari opensolari opensparc opensparc openjdk openjdk open sourc univers meetup open sourc univers meetup peopl bill joy bill joy andi bechtolsheim andi bechtolsheim scott mcneali scott mcneali vinod khosla vinod khosla afara websystem afara websystem cobalt network cobalt network gridwar gridwar lighthous design lighthous design montalvo system montalvo system mysql ab mysql ab pixo pixo procom technolog procom technolog savaj savaj storagetek storagetek tarantella tarantella network comput network comput write run anywher write run anywher categori v v e e oracl corpor jeffrey berg jeffrey berg h. raymond bingham h. raymond bingham michael boskin michael boskin safra catz safra catz larri ellison larri ellison héctor garcía-molina héctor garcía-molina joseph grundfest joseph grundfest jeffrey o. henley jeffrey o. henley mark hurd mark hurd jack f. kemp jack f. kemp donald l. luca donald l. luca naomi o. seligman list sun sun peoplesoft peoplesoft hyperion hyperion siebel siebel bea bea jd edward jd edward rightnow rightnow virtual iron virtual iron timesten timesten sunopsi sunopsi netsuit netsuit cerner cerner oracl databas oracl databas mysql mysql innodb innodb berkeley db berkeley db timesten timesten rdb rdb essbas essbas java java pl/sql pl/sql jdevelop jdevelop form form netbean netbean apex apex sql develop sql develop develop studio develop studio fusion middlewar fusion middlewar webcent webcent soa suit soa suit weblog server weblog server coher coher tuxedo tuxedo glassfish glassfish oracl linux oracl linux oracl solari oracl solari sun fire sun fire sparc t-seri enterpris sparc t-seri enterpris storagetek storagetek oracl exadata oracl exadata oracl exalog oracl exalog big data applianc big data applianc oracl certif program oracl certif program categori categori v v e e oracl free open-sourc softwar foss openoffice.org openoffice.org virtualbox virtualbox java java berkeley db berkeley db innodb innodb mysql mysql oracl nosql databas oracl nosql databas dtrace dtrace glassfish glassfish hotspot hotspot imag packag system imag packag system java softwar platform java softwar platform maxin virtual machin maxin virtual machin memori memori nashorn nashorn netbean netbean openjdk openjdk visualvm visualvm opensolari opensolari oracl linux oracl linux java desktop system java desktop system ocfs2 ocfs2 oracl grid engin oracl grid engin oracl vm server x86 oracl vm server x86 googl llc v. oracl america inc. googl llc v. oracl america inc. v v e e program languag comparison comparison timelin timelin histori histori ada ada algol simula algol simula simula apl apl assembl assembl basic visual basic classic .net basic visual basic classic .net visual basic classic classic .net .net c c c++ c++ c c cobol cobol erlang erlang forth forth fortran fortran go go haskel haskel java java javascript javascript julia julia kotlin kotlin lisp lisp lua lua matlab matlab ml ml pascal object pascal pascal object pascal object pascal perl perl php php prolog prolog python python r r rubi rubi rust rust sql sql scratch scratch shell shell smalltalk smalltalk swift swift ... ... list alphabet alphabet categor categor gener gener non-english-bas non-english-bas categori categori author control databas germani germani germani unit state unit state unit state franc franc franc bnf data bnf data bnf data czech republ czech republ czech republ czech republ spain spain spain israel israel israel elmcip elmcip elmcip http //en.wikipedia.org/w/index.php title=java_ programming_languag oldid=1266787868 categori java program languag java program languag c program languag famili c program languag famili class-bas program languag class-bas program languag concurr program languag concurr program languag java platform java platform java specif request java specif request jvm program languag jvm program languag multi-paradigm program languag multi-paradigm program languag object-ori program languag object-ori program languag program languag creat 1995 program languag creat 1995 program languag program languag static type program languag static type program languag sun microsystem sun microsystem compil program languag compil program languag american invent american invent webarch templat wayback link webarch templat wayback link cs1 unfit url cs1 unfit url wikipedia indefinit semi-protect page wikipedia indefinit semi-protect page articl short descript articl short descript short descript match wikidata short descript match wikidata use mdi date august 2017 use mdi date august 2017 articl contain potenti date statement 2022 articl contain potenti date statement 2022 articl contain potenti date statement articl contain potenti date statement articl contain potenti date statement septemb 2024 articl contain potenti date statement septemb 2024 articl contain potenti date statement novemb 2024 articl contain potenti date statement novemb 2024 articl exampl java code articl exampl java code articl need addit refer may 2019 articl need addit refer may 2019 articl need addit refer articl need addit refer articl need addit refer septemb 2023 articl need addit refer septemb 2023 page use sister project link wikidata namespac mismatch page use sister project link wikidata namespac mismatch page use sister project link hidden wikidata page use sister project link hidden wikidata page last edit 2 januari 2025 06:14 utc utc text avail creativ common attribution-sharealik 4.0 licens addit term may appli use site agre term use privaci polici wikipedia® regist trademark wikimedia foundat inc. non-profit organ creativ common attribution-sharealik 4.0 licens term use privaci polici wikimedia foundat inc. privaci polici privaci polici wikipedia wikipedia disclaim disclaim contact wikipedia contact wikipedia code conduct code conduct develop develop statist statist cooki statement cooki statement mobil view mobil view",
    "https://en.wikipedia.org/wiki/C_(programming_language)": "jump content main menu main page main page main page content content content current event current event current event random articl random articl random articl wikipedia wikipedia wikipedia contact u contact u contact u help help help learn edit learn edit learn edit commun portal commun portal commun portal recent chang recent chang recent chang upload file upload file upload file search search appear donat donat donat creat account creat account creat account log log log person tool donat donat donat creat account creat account creat account log log log learn learn contribut contribut contribut talk talk talk content top top 1 overview toggl overview subsect 1.1 relat languag 1 overview 1 overview toggl overview subsect 1.1 relat languag 1.1 relat languag 1.1 relat languag 2 histori toggl histori subsect 2.1 earli develop 2.1.1 b 2.1.2 new b first c releas 2.1.3 structur unix kernel re-writ 2.2 k r c 2.3 ansi c iso c 2.4 c99 2.5 c11 2.6 c17 2.7 c23 2.8 c2i 2.9 embed c 2 histori 2 histori toggl histori subsect 2.1 earli develop 2.1.1 b 2.1.2 new b first c releas 2.1.3 structur unix kernel re-writ 2.1 earli develop 2.1 earli develop 2.1.1 b 2.1.1 b 2.1.1 b 2.1.2 new b first c releas 2.1.2 new b first c releas 2.1.2 new b first c releas 2.1.3 structur unix kernel re-writ 2.1.3 structur unix kernel re-writ 2.1.3 structur unix kernel re-writ 2.2 k r c 2.2 k r c 2.2 k r c 2.3 ansi c iso c 2.3 ansi c iso c 2.3 ansi c iso c 2.4 c99 2.4 c99 2.4 c99 2.5 c11 2.5 c11 2.5 c11 2.6 c17 2.6 c17 2.6 c17 2.7 c23 2.7 c23 2.7 c23 2.8 c2i 2.8 c2i 2.8 c2i 2.9 embed c 2.9 embed c 2.9 embed c 3 syntax toggl syntax subsect 3.1 charact set 3.2 reserv word 3.3 oper 3 syntax 3 syntax toggl syntax subsect 3.1 charact set 3.1 charact set 3.1 charact set 3.2 reserv word 3.2 reserv word 3.2 reserv word 3.3 oper 3.3 oper 3.3 oper 4 `` hello world '' exampl 4 `` hello world '' exampl 4 `` hello world '' exampl 5 data type toggl data type subsect 5.1 pointer 5.2 array 5.3 array–point interchang 5 data type 5 data type toggl data type subsect 5.1 pointer 5.1 pointer 5.1 pointer 5.2 array 5.2 array 5.2 array 5.3 array–point interchang 5.3 array–point interchang 5.3 array–point interchang 6 memori manag 6 memori manag 6 memori manag 7 librari toggl librari subsect 7.1 file handl stream 7 librari 7 librari toggl librari subsect 7.1 file handl stream 7.1 file handl stream 7.1 file handl stream 8 languag tool 8 languag tool 8 languag tool 9 use toggl use subsect 9.1 rational use system program 9.2 use computationally-intens librari 9.3 c intermedi languag 9.4 languag written c 9.5 use web develop 9.6 web server 9.7 end-us applic 9 use 9 use toggl use subsect 9.1 rational use system program 9.1 rational use system program 9.1 rational use system program 9.2 use computationally-intens librari 9.2 use computationally-intens librari 9.2 use computationally-intens librari 9.3 c intermedi languag 9.3 c intermedi languag 9.3 c intermedi languag 9.4 languag written c 9.4 languag written c 9.4 languag written c 9.5 use web develop 9.5 use web develop 9.5 use web develop 9.6 web server 9.6 web server 9.6 web server 9.7 end-us applic 9.7 end-us applic 9.7 end-us applic 10 limit 10 limit 10 limit 11 relat languag 11 relat languag 11 relat languag 12 see also 12 see also 12 see also 13 note 13 note 13 note 14 refer 14 refer 14 refer 15 sourc 15 sourc 15 sourc 16 read 16 read 16 read 17 extern link 17 extern link 17 extern link toggl tabl content c program languag c program languag 123 languag afrikaan afrikaan afrikaan alemannisch alemannisch alemannisch አማርኛ አማርኛ አማርኛ العربية العربية العربية aragoné aragoné aragoné অসমীয়া অসমীয়া অসমীয়া asturianu asturianu asturianu azərbaycanca azərbaycanca azərbaycanca تۆرکجه تۆرکجه تۆرکجه বাংলা বাংলা বাংলা 閩南語 bân-lâm-gú 閩南語 bân-lâm-gú 閩南語 bân-lâm-gú беларуская беларуская беларуская беларуская тарашкевіца беларуская тарашкевіца беларуская тарашкевіца भोजपुरी भोजपुरी भोजपुरी български български български bosanski bosanski bosanski brezhoneg brezhoneg brezhoneg català català català чӑвашла чӑвашла чӑвашла čeština čeština čeština corsu corsu corsu cymraeg cymraeg cymraeg dansk dansk dansk الدارجة الدارجة الدارجة deutsch deutsch deutsch eesti eesti eesti ελληνικά ελληνικά ελληνικά español español español esperanto esperanto esperanto euskara euskara euskara فارسی فارسی فارسی françai françai françai gaeilg gaeilg gaeilg galego galego galego ગુજરાતી ગુજરાતી ગુજરાતી 客家語 hak-kâ-ngî 客家語 hak-kâ-ngî 客家語 hak-kâ-ngî 한국어 한국어 한국어 hausa hausa hausa հայերեն հայերեն հայերեն हिन्दी हिन्दी हिन्दी hrvatski hrvatski hrvatski ido ido ido bahasa indonesia bahasa indonesia bahasa indonesia interlingua interlingua interlingua isixhosa isixhosa isixhosa íslenska íslenska íslenska italiano italiano italiano עברית עברית עברית jawa jawa jawa ಕನ್ನಡ ಕನ್ನಡ ಕನ್ನಡ ქართული ქართული ქართული қазақша қазақша қазақша kiswahili kiswahili kiswahili kurdî kurdî kurdî кыргызча кыргызча кыргызча ລາວ ລາວ ລາວ latina latina latina latviešu latviešu latviešu lietuvių lietuvių lietuvių lombard lombard lombard magyar magyar magyar मैथिली मैथिली मैथिली македонски македонски македонски malagasi malagasi malagasi മലയാളം മലയാളം മലയാളം मराठी मराठी मराठी bahasa melayu bahasa melayu bahasa melayu ꯃꯤꯇꯩ ꯂꯣꯟ ꯃꯤꯇꯩ ꯂꯣꯟ ꯃꯤꯇꯩ ꯂꯣꯟ 閩東語 mìng-dĕ̤ng-ngṳ̄ 閩東語 mìng-dĕ̤ng-ngṳ̄ 閩東語 mìng-dĕ̤ng-ngṳ̄ монгол монгол монгол မြန်မာဘာသာ မြန်မာဘာသာ မြန်မာဘာသာ nederland nederland nederland नेपाली नेपाली नेपाली 日本語 日本語 日本語 norsk bokmål norsk bokmål norsk bokmål norsk nynorsk norsk nynorsk norsk nynorsk occitan occitan occitan ଓଡ଼ିଆ ଓଡ଼ିଆ ଓଡ଼ିଆ oʻzbekcha ўзбекча oʻzbekcha ўзбекча oʻzbekcha ўзбекча ਪੰਜਾਬੀ ਪੰਜਾਬੀ ਪੰਜਾਬੀ پنجابی پنجابی پنجابی ပအိုဝ်ႏဘာႏသာႏ ပအိုဝ်ႏဘာႏသာႏ ပအိုဝ်ႏဘာႏသာႏ ភាសាខ្មែរ ភាសាខ្មែរ ភាសាខ្មែរ piemontèi piemontèi piemontèi polski polski polski portuguê portuguê portuguê qaraqalpaqsha qaraqalpaqsha qaraqalpaqsha română română română runa simi runa simi runa simi русский русский русский саха тыла саха тыла саха тыла ᱥᱟᱱᱛᱟᱲᱤ ᱥᱟᱱᱛᱟᱲᱤ ᱥᱟᱱᱛᱟᱲᱤ scot scot scot shqip shqip shqip සිංහල සිංහල සිංහල simpl english simpl english simpl english slovenčina slovenčina slovenčina slovenščina slovenščina slovenščina српски srpski српски srpski српски srpski srpskohrvatski српскохрватски srpskohrvatski српскохрватски srpskohrvatski српскохрватски suomi suomi suomi svenska svenska svenska tagalog tagalog tagalog தமிழ் தமிழ் தமிழ் taqbaylit taqbaylit taqbaylit татарча tatarça татарча tatarça татарча tatarça తెలుగు తెలుగు తెలుగు ไทย ไทย ไทย тоҷикӣ тоҷикӣ тоҷикӣ türkçe türkçe türkçe basa ugi basa ugi basa ugi українська українська українська اردو اردو اردو vèneto vèneto vèneto tiếng việt tiếng việt tiếng việt 文言 文言 文言 winaray winaray winaray 吴语 吴语 吴语 ייִדיש ייִדיש ייִדיש 粵語 粵語 粵語 žemaitėška žemaitėška žemaitėška 中文 中文 中文 ⵜⴰⵎⴰⵣⵉⵖⵜ ⵜⴰⵏⴰⵡⴰⵢⵜ ⵜⴰⵎⴰⵣⵉⵖⵜ ⵜⴰⵏⴰⵡⴰⵢⵜ ⵜⴰⵎⴰⵣⵉⵖⵜ ⵜⴰⵏⴰⵡⴰⵢⵜ edit link edit link articl articl articl talk talk talk english read read read edit edit edit view histori view histori view histori tool read read read edit edit edit view histori view histori view histori link link link relat chang relat chang relat chang upload file upload file upload file special page special page special page perman link perman link perman link page inform page inform page inform cite page cite page cite page get shorten url get shorten url get shorten url download qr code download qr code download qr code download pdf download pdf download pdf printabl version printabl version printabl version wikimedia common wikimedia common wikimedia common wikibook wikibook wikibook wikiquot wikiquot wikiquot wikivers wikivers wikivers wikidata item wikidata item wikidata item page version statu accept version page latest accept revis review c program languag c++ c c program languag 1 paradigm multi-paradigm imper procedur structur design denni ritchi develop ansi c iso/iec jtc 1 joint technic committe 1 sc 22 subcommitte 22 53 year ago 1972 1972 2 stabl releas c23 2 month ago 2024-10-31 2024-10-31 preview releas 10 month ago 2024-02-21 2024-02-21 3 type disciplin static weak manifest nomin o cross-platform filenam extens www.iso.org/standard/82075.html www.iso.org/standard/82075.html www.open-std.org/jtc1/sc22/wg14/ www.open-std.org/jtc1/sc22/wg14/ implement pcc gcc clang intel c intel c c++builder microsoft visual c++ microsoft visual c++ watcom c watcom c dialect cyclon unifi parallel c split-c cilk c b bcpl cpl algol 68 4 pl/i fortran numer ampl awk csh c++ c -- c objective-c go java javascript js++ julia limbo lpc perl php pike process python rust seed7 v vlang vala verilog 5 nim zig c program wikibook c program c pronounc /ˈsiː/ – like letter c 6 general-purpos program languag creat 1970 denni ritchi remain wide use influenti design c 's featur cleanli reflect capabl target cpu found last use oper system code especi kernel 7 devic driver protocol stack use applic softwar decreas 8 c commonli use comput architectur rang largest supercomput smallest microcontrol embed system /ˈsiː/ /ˈsiː/ /ˈsiː/ ˈsiː ˈ iː c 6 general-purpos program languag denni ritchi cpu oper system kernel 7 devic driver protocol stack applic softwar 8 supercomput microcontrol embed system successor program languag b c origin develop bell lab ritchi 1972 1973 construct util run unix appli re-impl kernel unix oper system 9 1980 c gradual gain popular becom one wide use program languag 10 11 c compil avail practic modern comput architectur oper system book c program languag co-author origin languag design serv mani year de facto standard languag 12 1 c standard sinc 1989 american nation standard institut ansi subsequ jointli intern organ standard iso intern electrotechn commiss iec b bell lab unix 9 program languag 10 11 compil comput architectur oper system c program languag 12 1 american nation standard institut intern organ standard intern electrotechn commiss c imper procedur languag support structur program lexic variabl scope recurs static type system design compil provid low-level access memori languag construct map effici machin instruct minim runtim support despit low-level capabl languag design encourag cross-platform program standards-compli c program written portabl mind compil wide varieti comput platform oper system chang sourc code imper procedur structur program lexic variabl scope recurs static type system compil low-level memori machin instruct runtim support standard portabl sinc 2000 c consist rank among top four languag tiob index measur popular program languag 13 tiob index 13 overview edit edit edit denni ritchi ken thompson c imper procedur languag algol tradit static type system c execut code contain within subroutin also call `` function '' though sen function program function paramet pas valu although array pas pointer i.e address first item array pass-by-refer simul c explicitli pas pointer thing referenc imper algol type system execut code subroutin function program function paramet array pointer c program sourc text free-form code semicolon termin statement curli brace use group statement block free-form semicolon statement curli brace block c languag also exhibit follow characterist languag small fix number keyword includ full set control flow primit if/els do/whil switch user-defin name distinguish keyword kind sigil control flow if/els do/whil switch sigil larg number arithmet bitwis logic oper += ++ || etc bitwis one assign may perform singl statement assign function function return valu ignor need function data pointer permit ad hoc run-tim polymorph function may defin within lexic scope function variabl may defin within function scope function may call recurs support function return valu ignor need function data pointer permit ad hoc run-tim polymorph run-tim polymorph function may defin within lexic scope function variabl may defin within function scope scope function may call recurs support recurs data type static weakli enforc data type implicit convers possibl static weakli enforc implicit convers user-defin typedef compound type possibl heterogen aggreg data type struct allow relat data element access assign unit content whole struct compar use singl built-in oper element must compar individu union structur overlap member allow multipl data type share memori locat array index secondari notat defin term pointer arithmet whole array assign compar use singl built-in oper `` array '' keyword use definit instead squar bracket indic array syntact exampl month 11 enumer type possibl enum keyword freeli interconvert integ string distinct data type convent implement null-termin charact array typedef heterogen aggreg data type struct allow relat data element access assign unit content whole struct compar use singl built-in oper element must compar individu struct union structur overlap member allow multipl data type share memori locat union array index secondari notat defin term pointer arithmet whole array assign compar use singl built-in oper `` array '' keyword use definit instead squar bracket indic array syntact exampl month 11 array enumer type possibl enum keyword freeli interconvert integ enumer type string distinct data type convent implement null-termin charact array string implement null-termin low-level access comput memori possibl convert machin address pointer comput memori pointer procedur subroutin return valu special case function empti return type void procedur memori alloc program call librari routin alloc librari routin preprocessor perform macro definit sourc code file inclus condit compil preprocessor macro sourc code condit compil basic form modular file compil separ link togeth control function data object visibl file via static extern attribut modular link static complex function i/o string manipul mathemat function consist deleg librari routin i/o string librari routin gener code compil rel straightforward need underli platform make suitabl creat oper system use embed system embed system c includ certain featur found languag object orient garbag collect implement emul often use extern librari e.g. glib object system boehm garbag collector object orient garbag collect glib object system boehm garbag collector relat languag edit edit edit list c-famili program languag mani later languag borrow directli indirectli c includ c++ c unix 's c shell go java javascript includ transpil julia limbo lpc objective-c perl php python rubi rust swift verilog systemverilog hardwar descript languag 5 languag drawn mani control structur basic featur c. also express highli similar syntax c tend combin recogniz express statement syntax c underli type system data model semant radic differ c++ c c shell go java javascript transpil julia limbo lpc objective-c perl php python rubi rust swift verilog systemverilog 5 control structur syntax syntax c histori edit edit edit earli develop edit edit edit k r c ansi c c99 c11 c17 c23 c2i origin c close tie develop unix oper system origin implement assembl languag pdp-7 denni ritchi ken thompson incorpor sever idea colleagu eventu decid port oper system pdp-11 origin pdp-11 version unix also develop assembl languag 9 unix assembl languag pdp-7 denni ritchi ken thompson pdp-11 9 b edit edit edit b program languag thompson want program languag develop util new platform first tri write fortran compil soon gave idea instead creat cut-down version recent develop system program languag call bcpl offici descript bcpl avail time 14 thompson modifi syntax less 'wordi similar simplifi algol known smalgol 15 call result b 9 describ `` bcpl semant lot smalgol syntax '' 15 like bcpl b bootstrap compil facilit port new machin 15 ultim util written b slow could take advantag pdp-11 featur byte address fortran system program languag bcpl 14 algol 15 b 9 15 bootstrap 15 byte new b first c releas edit edit edit 1971 ritchi start improv b use featur more-pow pdp-11 signific addit charact data type call new b nb 15 thompson start use nb write unix kernel requir shape direct languag develop 15 16 1972 richer type ad nb languag nb array int char pointer abil gener pointer type array type type return function also ad array within express becam pointer new compil written languag renam c. 9 15 unix 15 16 9 c compil util made includ version 2 unix also known research unix 17 version 2 unix research unix 17 structur unix kernel re-writ edit edit edit version 4 unix releas novemb 1973 unix kernel extens re-impl c. 9 time c languag acquir power featur struct type version 4 unix unix kernel 9 preprocessor introduc around 1973 urg alan snyder also recognit use file-inclus mechan avail bcpl pl/i origin version provid includ file simpl string replac includ defin parameterless macro soon extend mostli mike lesk john reiser incorpor macro argument condit compil 9 preprocessor alan snyder pl/i mike lesk condit compil 9 unix one first oper system kernel implement languag assembl earlier instanc includ multic system written pl/i master control program mcp burrough b5000 written algol 1961 around 1977 ritchi stephen c. johnson made chang languag facilit portabl unix oper system johnson 's portabl c compil serv basi sever implement c new platform 16 assembl multic pl/i master control program burrough b5000 algol stephen c. johnson portabl portabl c compil 16 k r c edit edit edit brian kernighan denni ritchi 1978 brian kernighan denni ritchi publish first edit c program languag 18 known k r initi author book serv mani year inform specif languag version c describ commonli refer `` k r c '' releas 1978 also refer c78 19 second edit book 20 cover later ansi c standard describ brian kernighan denni ritchi c program languag 18 specif 19 20 ansi c k r introduc sever languag featur standard i/o librari standard i/o librari long int data type long int unsign int data type compound assign oper form =op =- chang form op= -= remov semant ambigu creat construct i=-10 interpret =- 10 decrement 10 instead possibl intend -10 let −10 even public 1989 ansi standard mani year k r c still consid `` lowest common denomin '' c programm restrict maximum portabl desir sinc mani older compil still use care written k r c code legal standard c well lowest common denomin earli version c function return type int must declar use function definit function use without prior declar presum return type int exampl long some_funct function declar compil know name return type function int other_funct anoth function declar earli version c implicit 'int type comment show explicit 'int type specifi would requir later version int calling_funct function definit includ bodi code follow curli bracket return type specifi function implicitli return 'int earli version c. long test1 regist int test2 note 'int requir 'int type specifi comment would requir later version c. 'regist keyword indic compil variabl ideal store regist oppos within stack frame test1 some_funct test1 1 test2 0 el test2 other_funct return test2 int type specifi comment could omit k r c requir later standard sinc k r function declar includ inform function argument function paramet type check perform although compil would issu warn messag local function call wrong number argument differ call extern function use differ number type argument separ tool unix 's lint util develop among thing could check consist function use across multipl sourc file type check lint year follow public k r c sever featur ad languag support compil particular pcc 21 vendor includ pcc 21 void function i.e. function return valu void function return struct union type previous singl pointer integ float could return struct union assign struct data type assign enumer type previous preprocessor definit integ fix valu use e.g defin green 3 enumer type larg number extens lack agreement standard librari togeth languag popular fact even unix compil precis implement k r specif led necess standard 22 standard librari 22 ansi c iso c edit edit edit ansi c late 1970 1980 version c implement wide varieti mainfram comput minicomput microcomput includ ibm pc popular began increas significantli mainfram comput minicomput microcomput ibm pc 1983 american nation standard institut ansi form committe x3j11 establish standard specif c. x3j11 base c standard unix implement howev non-port portion unix c librari hand ieee work group 1003 becom basi 1988 posix standard 1989 c standard ratifi ansi x3.159-1989 `` program languag c '' version languag often refer ansi c standard c sometim c89 american nation standard institut ieee work group posix ansi c 1990 ansi c standard format chang adopt intern organ standard iso iso/iec 9899:1990 sometim call c90 therefor term `` c89 '' `` c90 '' refer program languag intern organ standard ansi like nation standard bodi longer develop c standard independ defer intern c standard maintain work group iso/iec jtc1/sc22/wg14 nation adopt updat intern standard typic occur within year iso public iso/iec jtc1/sc22 one aim c standard process produc superset k r c incorpor mani subsequ introduc unoffici featur standard committe also includ sever addit featur function prototyp borrow c++ void pointer support intern charact set local preprocessor enhanc although syntax paramet declar augment includ style use c++ k r interfac continu permit compat exist sourc code superset function prototyp charact set local syntax c89 support current c compil modern c code base program written standard c without hardware-depend assumpt run correctli platform conform c implement within resourc limit without precaut program may compil certain platform particular compil due exampl use non-standard librari gui librari relianc compiler- platform-specif attribut exact size data type byte endian platform gui endian case code must compil either standard-conform k r c-base compil __stdc__ macro use split code standard k r section prevent use k r c-base compil featur avail standard c. ansi/iso standard process c languag specif remain rel static sever year 1995 norm amend 1 1990 c standard iso/iec 9899/amd1:1995 known inform c95 publish correct detail add extens support intern charact set 23 23 c99 edit edit edit c99 c standard revis late 1990 lead public iso/iec 9899:1999 1999 commonli refer `` c99 '' sinc amend three time technic corrigenda 24 c99 24 c99 introduc sever new featur includ inlin function sever new data type includ long long int complex type repres complex number variable-length array flexibl array member improv support ieee 754 float point support variad macro macro variabl ariti support one-lin comment begin // bcpl c++ mani alreadi implement extens sever c compil inlin function data type complex number variable-length array flexibl array member ieee 754 variad macro ariti c99 part backward compat c90 stricter way particular declar lack type specifi longer int implicitli assum standard macro __stdc_version__ defin valu 199901l indic c99 support avail gcc solari studio c compil support mani new featur c99 c compil microsoft visual c++ howev implement c89 standard part c99 requir compat c++11 25 need updat gcc solari studio microsoft visual c++ c++11 25 need updat need updat addit c99 standard requir support identifi use unicod form escap charact e.g \\u0040 \\u0001f431 suggest support raw unicod name identifi unicod c11 edit edit edit c11 c standard revis work began 2007 anoth revis c standard inform call `` c1x '' offici public iso/iec 9899:2011 decemb 8 2011 c standard committe adopt guidelin limit adopt new featur test exist implement c11 standard add numer new featur c librari includ type gener macro anonym structur improv unicod support atom oper multi-thread bounds-check function also make portion exist c99 librari option improv compat c++ standard macro __stdc_version__ defin 201112l indic c11 support avail c17 edit edit edit c17 c standard revis c17 inform name iso/iec 9899:2018 standard c program languag publish june 2018 introduc new languag featur technic correct clarif defect c11 standard macro __stdc_version__ defin 201710l indic c17 support avail c23 edit edit edit c23 c standard revis c23 inform name current major c languag standard revis inform known `` c2x '' develop c23 publish octob 2024 iso/iec 9899:2024 26 standard macro __stdc_version__ defin 202311l indic c23 support avail 26 c2i edit edit edit c2i inform name next major c languag standard revis c23 c2x hope releas later 2020 decad henc 2 `` c2i '' earli work draft c2i releas februari 2024 n3220 work group iso/iec jtc1/sc22/wg14 27 iso/iec jtc1/sc22 27 embed c edit edit edit embed c histor embed c program requir non-standard extens c languag support exot featur fixed-point arithmet multipl distinct memori bank basic i/o oper fixed-point arithmet memori bank 2008 c standard committe publish technic report extend c languag 28 address issu provid common standard implement adher includ number featur avail normal c fixed-point arithmet name address space basic i/o hardwar address technic report 28 syntax edit edit edit c syntax c formal grammar specifi c standard 29 line end gener signific c howev line boundari signific preprocess phase comment may appear either delimit sinc c99 follow // end line comment delimit nest sequenc charact interpret comment delimit appear insid string charact liter 30 formal grammar 29 string 30 c sourc file contain declar function definit function definit turn contain declar statement declar either defin new type use keyword struct union enum assign type perhap reserv storag new variabl usual write type follow variabl name keyword char int specifi built-in type section code enclos brace sometim call `` curli bracket '' limit scope declar act singl statement control structur statement imper languag c use statement specifi action common statement express statement consist express evalu follow semicolon side effect evalu function may call variabl assign new valu modifi normal sequenti execut statement c provid sever control-flow statement identifi reserv keyword structur program support ... el condit execut ... iter execut loop statement separ initi test reiniti express omit break continu use within loop break use leav innermost enclos loop statement continu use skip reinitialis also non-structur goto statement branch directli design label within function switch select case execut base valu integ express differ mani languag control-flow fall next case unless termin break side effect function may call variabl assign structur program goto label switch fall express use varieti built-in oper may contain function call order argument function operand oper evalu unspecifi evalu may even interleav howev side effect includ storag variabl occur next `` sequenc point '' sequenc point includ end express statement entri return function call sequenc point also occur evalu express contain certain oper || comma oper permit high degre object code optim compil requir c programm take care obtain reliabl result need program languag sequenc point comma oper kernighan ritchi say introduct c program languag `` c like languag blemish oper wrong preced part syntax could better `` 31 c standard attempt correct mani blemish impact chang alreadi exist softwar 31 charact set edit edit edit basic c sourc charact set includ follow charact lowercas uppercas letter iso basic latin alphabet a–z a–z iso basic latin alphabet decim digit 0–9 graphic charact `` whitespac charact space horizont tab vertic tab form feed newlin whitespac charact space horizont tab vertic tab form feed newlin newlin charact indic end text line need correspond actual singl charact although conveni c treat addit multi-byt encod charact may use string liter entir portabl latest c standard c11 allow multi-n unicod charact embed portabl within c sourc text use \\uxxxx \\uxxxxxxxx encod x denot hexadecim charact although featur yet wide implement need updat string liter portabl c11 need updat need updat basic c execut charact set contain charact along represent alert backspac carriag return run-tim support extend charact set increas revis c standard alert backspac carriag return run-tim reserv word edit edit edit follow reserv word case sensit case sensit c89 32 reserv word also known 'keyword use purpos predefin auto break break case char const const continu continu default doubl doubl el el enum enum extern extern float float goto goto int int long long regist regist return return short short sign sign sizeof sizeof static static struct struct switch switch typedef typedef union union unsign unsign void void volatil volatil c99 ad five reserv word ‡ indic altern spell alia c23 keyword inlin inlin restrict restrict _bool ‡ _complex _complex _imaginari _imaginari c11 ad seven reserv word 32 ‡ indic altern spell alia c23 keyword 32 _aligna ‡ _alignof ‡ _atom _gener _noreturn _static_assert ‡ _thread_loc ‡ c23 reserv fifteen word aligna alignof bool constexpr fals nullptr static_assert thread_loc true typeof typeof_unqu _bitint _decimal32 _decimal64 _decimal128 recent reserv word begin underscor follow capit letter identifi form previous reserv c standard use implement sinc exist program sourc code use identifi would affect c implement start support extens program languag standard header defin conveni synonym underscor identifi word ad keyword convent spell c23 correspond macro remov prior c89 entri reserv keyword second edit book c program languag describ becam known c89 kernighan ritchi wrote `` ... keyword entri formerli reserv never use longer reserv '' `` stillborn entri keyword withdrawn `` 33 c program languag 33 oper edit edit edit oper c c++ c support rich set oper symbol use within express specifi manipul perform evalu express c oper oper express arithmet arithmet assign assign augment assign += -= /= |= ^= augment assign bitwis logic bitwis logic bitwis shift bitwis shift boolean logic || boolean logic condit evalu condit evalu equal test == == call function call function increment decrement ++ -- increment decrement member select member select object size sizeof sizeof type typeof typeof_unqu sinc c23 typeof order relat order relat refer derefer refer derefer sequenc subexpress group subexpress group type convers typenam type convers c use oper use mathemat express equal indic assign follow preced fortran pl/i unlik algol deriv c use oper == test equal similar oper assign equal may result accident use one place mani case mistak produc error messag although compil produc warn exampl condit express == b 1 might mistakenli written b 1 evalu true unless valu 0 assign 34 fortran pl/i algol 34 c oper preced alway intuit exampl oper == bind tightli execut prior oper bitwis bitwis express x 1 == 0 must written x 1 == 0 coder 's intent 35 oper preced 35 `` hello world '' exampl edit edit edit `` hello world '' program brian kernighan `` hello world '' exampl appear first edit k r becom model introductori program program textbook program print `` hello world '' standard output usual termin screen display k r standard output origin version 36 36 main printf `` hello world \\n `` standard-conform `` hello world '' program includ stdio.h int main void printf `` hello world \\n `` first line program contain preprocess direct indic includ caus compil replac line code entir text stdio.h header file contain declar standard input output function printf scanf angl bracket surround stdio.h indic header file locat use search strategi prefer header provid compil header name oppos doubl quot typic includ local project-specif header file preprocess direct stdio.h second line indic function name main defin main function serv special purpos c program run-tim environ call main function begin program execut type specifi int indic valu return invok case run-tim environ result evalu main function integ keyword void paramet list indic main function take argument b main run-tim environ b open curli brace indic begin code defin main function next line program statement call i.e divert execut function name printf case suppli system librari call printf function pas i.e provid singl argument address first charact string liter `` hello world\\n '' string liter unnam array set automat compil element type char final null charact ascii valu 0 mark end array allow printf determin length string null charact also written escap sequenc \\0 \\n standard escap sequenc c translat newlin charact output signifi end current line return valu printf function type int silent discard sinc use care program might test return valu check printf function succeed semicolon termin statement printf librari address string liter array null charact escap sequenc newlin close curli brace indic end code main function accord c99 specif newer main function unlik function implicitli return valu 0 upon reach termin function c return valu 0 interpret run-tim system exit code indic success execut function 37 c 37 data type edit edit edit c data type verif plea help improv articl ad citat reliabl sourc section unsourc materi may challeng remov improv articl ad citat reliabl sourc octob 2012 octob 2012 learn remov messag learn remov messag type system c static weakli type make similar type system algol descend pascal 38 built-in type integ variou size sign unsign floating-point number enumer type enum integ type char often use single-byt charact c99 ad boolean data type also deriv type includ array pointer record struct union union type system static weakli type algol pascal 38 floating-point number boolean data type array pointer record struct union c often use low-level system program escap type system may necessari compil attempt ensur type correct express programm overrid check variou way either use type cast explicitli convert valu one type anoth use pointer union reinterpret underli bit data object way type cast find c 's declar syntax unintuit particularli function pointer ritchi 's idea declar identifi context resembl use `` declar reflect use '' 39 function pointer declar reflect use 39 c 's usual arithmet convers allow effici code gener sometim produc unexpect result exampl comparison sign unsign integ equal width requir convers sign valu unsign gener unexpect result sign valu neg pointer edit edit edit c support use pointer type refer record address locat object function memori pointer dereferenc access data store address point invok pointed-to function pointer manipul use assign pointer arithmet run-tim represent pointer valu typic raw memori address perhap augment offset-within-word field sinc pointer 's type includ type thing point express includ pointer type-check compil time pointer arithmet automat scale size pointed-to data type pointer refer pointer arithmet pointer use mani purpos c. text string commonli manipul use pointer array charact dynam memori alloc perform use pointer result malloc usual cast data type data store mani data type tree commonli implement dynam alloc struct object link togeth use pointer pointer pointer often use multi-dimension array array struct object pointer function function pointer use pas function argument higher-ord function qsort bsearch dispatch tabl callback event handler 37 text string dynam memori alloc cast tree function pointer higher-ord function qsort bsearch dispatch tabl callback event handler 37 null pointer valu explicitli point valid locat dereferenc null pointer valu undefin often result segment fault null pointer valu use indic special case `` next '' pointer final node link list error indic function return pointer appropri context sourc code assign pointer variabl null pointer constant written 0 without explicit cast pointer type null macro defin sever standard header sinc c23 constant nullptr condit context null pointer valu evalu fals pointer valu evalu true null pointer segment fault link list void pointer void point object unspecifi type therefor use `` gener '' data pointer sinc size type pointed-to object known void pointer dereferenc pointer arithmet allow although easili mani context implicitli convert object pointer type 37 37 careless use pointer potenti danger typic uncheck pointer variabl made point arbitrari locat caus undesir effect although properli use pointer point safe place made point unsaf place use invalid pointer arithmet object point may continu use dealloc dangl pointer may use without initi wild pointer may directli assign unsaf valu use cast union anoth corrupt pointer gener c permiss allow manipul convers pointer type although compil typic provid option variou level check program languag address problem use restrict refer type pointer arithmet dangl pointer wild pointer refer array edit edit edit c string handl array type c tradit fix static size specifi compil time recent c99 standard also allow form variable-length array howev also possibl alloc block memori arbitrari size run-tim use standard librari 's malloc function treat array array sinc array alway access effect via pointer array access typic check underli array size although compil may provid bound check option 40 41 array bound violat therefor possibl lead variou repercuss includ illeg memori access corrupt data buffer overrun run-tim except bound check 40 41 buffer overrun c special provis declar multi-dimension array rather reli recurs within type system declar array array effect accomplish thing index valu result `` multi-dimension array '' thought increas row-major order multi-dimension array commonli use numer algorithm mainli appli linear algebra store matric structur c array well suit particular task howev earli version c bound array must known fix valu el explicitli pas subroutin requir dynam size array array access use doubl index workaround alloc array addit `` row vector '' pointer column c99 introduc `` variable-length array '' address issu multi-dimension array recurs row-major order linear algebra follow exampl use modern c c99 later show alloc two-dimension array heap use multi-dimension array index access use bounds-check mani c compil int func int n int float p n malloc sizeof p p == 0 return -1 int 0 n ++ int j 0 j j ++ p j j print_array n p free p return 1 similar implement use c99 's auto vla featur vla int func int n int // caution check made ensur n sizeof float exceed limit auto vla within avail size stack float p n // auto vla held stack size function invok int 0 n ++ int j 0 j j ++ p j j print_array n p // need free p sinc disappear function exit along rest stack frame return 1 array–point interchang edit edit edit subscript notat x x design pointer syntact sugar x+i 42 take advantag compil 's knowledg pointer type address x point base address point x increment byte rather defin base address increment multipli size element x point thu x design i+1th element array syntact sugar 42 furthermor express context notabl except operand sizeof express array type automat convert pointer array 's first element impli array never copi whole name argument function rather address first element pas therefor although function call c use pass-by-valu semant array effect pas refer sizeof pass-by-valu refer total size array x determin appli sizeof express array type size element determin appli oper sizeof dereferenc element array n sizeof 0 thu number element declar array determin sizeof sizeof 0 note pointer first element avail often case c code automat convers describ inform full type array length lost memori manag edit edit edit one import function program languag provid facil manag memori object store memori c provid three princip way alloc memori object 37 memori 37 static memori alloc space object provid binari compile-tim object extent lifetim long binari contain load memori static memori alloc extent automat memori alloc temporari object store stack space automat freed reusabl block declar exit automat memori alloc stack dynam memori alloc block memori arbitrari size request run-tim use librari function malloc region memori call heap block persist subsequ freed reus call librari function realloc free dynam memori alloc heap three approach appropri differ situat variou trade-off exampl static memori alloc littl alloc overhead automat alloc may involv slightli overhead dynam memori alloc potenti great deal overhead alloc dealloc persist natur static object use maintain state inform across function call automat alloc easi use stack space typic much limit transient either static memori heap space dynam memori alloc allow conveni alloc object whose size known run-tim c program make extens use three possibl automat static alloc usual simplest storag manag compil free programm potenti error-pron chore manual alloc releas storag howev mani data structur chang size runtim sinc static alloc automat alloc c99 must fix size compile-tim mani situat dynam alloc necessari 37 prior c99 standard variable-s array common exampl see articl c dynam memori alloc exampl dynam alloc array unlik automat alloc fail run time uncontrol consequ dynam alloc function return indic form null pointer valu requir storag alloc static alloc larg usual detect linker loader program even begin execut 37 c dynam memori alloc linker loader unless otherwis specifi static object contain zero null pointer valu upon program startup automat dynam alloc object initi initi valu explicitli specifi otherwis initi indetermin valu typic whatev bit pattern happen present storag might even repres valid valu type program attempt access uniniti valu result undefin mani modern compil tri detect warn problem fals posit fals neg occur bit pattern storag fals posit fals neg heap memori alloc synchron actual usag program reus much possibl exampl pointer heap memori alloc goe scope valu overwritten dealloc explicitli memori recov later reus essenti lost program phenomenon known memori leak convers possibl memori freed referenc subsequ lead unpredict result typic failur symptom appear portion program unrel code caus error make difficult diagnos failur issu amelior languag automat garbag collect memori leak automat garbag collect librari edit edit edit c program languag use librari primari method extens c librari set function contain within singl `` archiv '' file librari typic header file contain prototyp function contain within librari may use program declar special data type macro symbol use function program use librari must includ librari 's header file librari must link program mani case requir compil flag e.g. -lm shorthand `` link math librari '' 37 librari header file compil flag 37 common c librari c standard librari specifi iso ansi c standard come everi c implement implement target limit environ embed system may provid subset standard librari librari support stream input output memori alloc mathemat charact string time valu sever separ standard header exampl stdio.h specifi interfac standard librari facil c standard librari iso ansi c embed system anoth common set c librari function use applic specif target unix unix-lik system especi function provid interfac kernel function detail variou standard posix singl unix specif unix unix-lik kernel posix singl unix specif sinc mani program written c wide varieti librari avail librari often written c c compil gener effici object code programm creat interfac librari routin use higher-level languag like java perl python 37 object code java perl python 37 file handl stream edit edit edit file input output i/o part c languag instead handl librari c standard librari associ header file e.g stdio.h file handl gener implement high-level i/o work stream stream perspect data flow independ devic file concret devic high-level i/o done associ stream file c standard librari buffer memori area queue temporarili use store data sent final destin reduc time spent wait slower devic exampl hard drive solid-st drive low-level i/o function part standard c librari clarif need gener part `` bare metal '' program program independ oper system embed program except implement includ low-level i/o stream buffer hard drive solid-st drive clarif need clarif need oper system embed program languag tool edit edit edit verif plea help improv articl ad citat reliabl sourc section unsourc materi may challeng remov improv articl ad citat reliabl sourc juli 2014 juli 2014 learn remov messag learn remov messag number tool develop help c programm find fix statement undefin behavior possibl erron express greater rigor provid compil tool lint first lead mani other lint autom sourc code check audit benefici languag c mani tool exist lint common practic use lint detect question code program first written program pas lint compil use c compil also mani compil option warn syntact valid construct like actual error misra c proprietari set guidelin avoid question code develop embed system 43 lint misra c 43 also compil librari oper system level mechan perform action standard part c bound check array detect buffer overflow serial dynam memori track automat garbag collect bound check buffer overflow serial dynam memori automat garbag collect tool purifi valgrind link librari contain special version memori alloc function help uncov runtim error memori usag 44 45 purifi valgrind memori alloc function 44 45 use edit edit edit rational use system program edit edit edit c wide use system program implement oper system embed system applic 46 sever reason system program oper system embed system 46 c languag permit platform hardwar memori access pointer type pun system-specif featur e.g control/statu regist i/o regist configur use code written c – allow fullest control platform run type pun control/statu regist i/o regist code gener compil demand mani system featur invok boot code straightforward manner – simpl execut system featur c languag statement express typic map well sequenc instruct target processor consequ low run-tim demand system resourc – fast execut run-tim rich set oper c languag use mani featur target cpu particular cpu esoter instruct languag variant construct perhap intrins function exploit instruct – use practic target cpu 's featur intrins function languag make easi overlay structur onto block binari data allow data comprehend navig modifi – write data structur even file system languag support rich set oper includ bit manipul integ arithmet logic perhap differ size float point number – process appropriately-structur data effect c fairli small languag hand statement without mani featur gener extens target code – comprehens c direct control memori alloc dealloc give reason effici predict time memory-handl oper without concern sporad stop-the-world garbag collect event – predict perform stop-the-world c permit use implement differ memori alloc scheme includ typic malloc free sophist mechan arena version o kernel may suit dma use within interrupt handler integr virtual memori system memori alloc malloc free arena o kernel dma interrupt handler virtual memori depend linker environ c code also call librari written assembl languag may call assembl languag – interoper well lower-level code assembl languag c call convent linker structur commonli use conjunct high-level languag call c c support – interoper well high-level code call convent c matur broad ecosystem includ librari framework open sourc compil debugg util de facto standard like driver alreadi exist c similar cpu architectur back-end c compil reduc incent choos anoth languag use computationally-intens librari edit edit edit c enabl programm creat effici implement algorithm data structur layer abstract hardwar thin overhead low import criterion comput intens program exampl gnu multipl precis arithmet librari gnu scientif librari mathematica matlab complet partial written c. mani languag support call librari function c exampl python-bas framework numpi use c high-perform hardware-interact aspect gnu multipl precis arithmet librari gnu scientif librari mathematica matlab python numpi c intermedi languag edit edit edit c sometim use intermedi languag implement languag approach may use portabl conveni use c intermedi languag addit machine-specif code gener necessari c featur line-numb preprocessor direct option superflu comma end initi list support compil gener code howev c 's shortcom prompt develop c-base languag specif design use intermedi languag c -- also contemporari major compil gcc llvm featur intermedi represent c compil support front end mani languag includ c. intermedi languag c-base languag c -- gcc llvm intermedi represent languag written c edit edit edit consequ c 's wide avail effici compil librari interpret program languag often implement c. 47 exampl refer implement python 48 perl 49 rubi 50 php 51 written c. compil interpret 47 refer implement python 48 perl 49 rubi 50 php 51 use web develop edit edit edit histor c sometim use web develop use common gateway interfac cgi `` gateway '' inform web applic server browser 52 c may chosen interpret languag speed stabil near-univers avail 53 longer common practic web develop done c 54 mani web develop languag popular applic c-base web develop continu includ http configur page router iot devic similar although even project part higher-level languag e.g use lua within openwrt web develop common gateway interfac 52 interpret languag 53 54 web develop languag http router iot lua openwrt web server edit edit edit two popular web server apach http server nginx written c. web server interact oper system listen tcp port http request serv static web content caus execut languag handl 'render content php primarili written c. c 's close-to-the-met approach allow construct high-perform softwar system web server apach http server nginx php end-us applic edit edit edit c also wide use implement end-us applic 55 howev applic also written newer higher-level languag end-us 55 limit edit edit edit power assembl languag conveni ... assembl languag 56 c popular influenti huge success drawback includ standard dynam memori handl malloc free error prone improp use lead memori leak dangl pointer dynam memori memori leak dangl pointer use pointer direct manipul memori mean corrupt memori possibl perhap due programm error insuffici check bad data type check appli area like variad function type check trivial inadvert circumv weakli type type check variad function weakli type sinc code gener compil contain check burden programm consid possibl outcom protect buffer overrun array bound check stack overflow memori exhaust consid race condit thread isol etc stack overflow race condit use pointer run-tim manipul mean may two way access data alias determin compil time mean optimis may avail languag possibl c. fortran consid faster standard librari function e.g scanf strncat lead buffer overrun buffer overrun limit standardis support low-level variant gener code exampl differ function call convent abi differ structur pack convent differ byte order within larger integ includ endian mani languag implement option may handl preprocessor direct pragma 57 58 addit keyword e.g use __cdecl call convent direct option consist support 59 call convent abi structur pack pragma 57 58 __cdecl 59 string handl use standard librari code-intens explicit memori manag requir string handl languag directli support object orient introspect run-tim express evalu gener etc introspect guard inappropri use languag featur may lead unmaintain code particular c preprocessor hide troubl effect doubl evalu wors 60 facil tricki code celebr competit intern obfusc c code contest underhand c contest unmaintain c preprocessor 60 intern obfusc c code contest underhand c contest c lack standard support except handl offer return code error check setjmp longjmp standard librari function use 61 implement try-catch mechan via macro except handl return code setjmp longjmp 61 purpos restrict style c adopt e.g misra c cert c attempt reduc opportun bug databas cwe attempt count way c etc vulner along recommend mitig misra c cert c cwe tool mitig drawback contemporari c compil includ check may gener warn help identifi mani potenti bug tool relat languag edit edit edit tiob index 62 list c-famili program languag c directli indirectli influenc mani later languag c++ java 63 pervas influenc syntact languag mention combin statement less recogniz express syntax c type system data model large-scal program structur differ c sometim radic c++ java 63 syntax c sever c near-c interpret exist includ ch cint also use script ch cint object-ori program languag becam popular c++ objective-c two differ extens c provid object-ori capabl languag origin implement source-to-sourc compil sourc code translat c compil c compil 64 object-ori program c++ objective-c source-to-sourc compil 64 c++ program languag origin name `` c class '' devi bjarn stroustrup approach provid object-ori function c-like syntax 65 c++ add greater type strength scope tool use object-ori program permit gener program via templat nearli superset c c++ support c except c++ class bjarn stroustrup object-ori 65 gener program except objective-c origin `` thin '' layer top c remain strict superset c permit object-ori program use hybrid dynamic/stat type paradigm objective-c deriv syntax c smalltalk syntax involv preprocess express function declar function call inherit c syntax object-ori featur origin taken smalltalk objective-c superset smalltalk addit c++ objective-c ch cilk unifi parallel c nearli superset c. c++ objective-c ch cilk unifi parallel c see also edit edit edit comput program portal comput program portal comput program portal free open-sourc softwar portal free open-sourc softwar portal free open-sourc softwar portal compat c c++ compat c c++ comparison pascal c comparison pascal c comparison program languag comparison program languag intern obfusc c code contest intern obfusc c code contest list c-famili program languag list c-famili program languag list c compil list c compil note edit edit edit origin exampl code compil modern compil strict standard complianc mode fulli conform requir either c89 c99 fact c99 requir diagnost messag produc origin exampl code compil modern compil strict standard complianc mode fulli conform requir either c89 c99 fact c99 requir diagnost messag produc main function actual two argument int argc char argv respect use handl command-lin argument iso c standard section 5.1.2.2.1 requir form main support special treatment afford function main function actual two argument int argc char argv respect use handl command-lin argument iso c standard section 5.1.2.2.1 requir form main support special treatment afford function command-lin argument prior c99 explicit return 0 statement requir end main function prior c99 explicit return 0 statement requir end main function code print_array shown slightli differ code print_array shown slightli differ refer edit edit edit b prinz peter crawford toni decemb 16 2005 c nutshel o'reilli medium inc. p. 3 isbn 9780596550714 b b prinz peter crawford toni decemb 16 2005 c nutshel o'reilli medium inc. p. 3 isbn 9780596550714 c nutshel isbn 9780596550714 ritchi 1993 `` thompson made brief attempt produc system code earli version c—befor structures—in 1972 gave effort '' ritchi 1993 `` thompson made brief attempt produc system code earli version c—befor structures—in 1972 gave effort '' ritchi 1993 `` n3221 – editor 's report post januari 2024 strasbourg franc meet '' iso/iec jtc1/sc22/wg14 open standard februari 21 2024 retriev may 24 2024 `` n3221 – editor 's report post januari 2024 strasbourg franc meet '' iso/iec jtc1/sc22/wg14 open standard februari 21 2024 retriev may 24 2024 `` n3221 – editor 's report post januari 2024 strasbourg franc meet '' retriev may 24 2024 may 24 ritchi 1993 `` scheme type composit adopt c owe consider debt algol 68 although perhap emerg form algol 's adher would approv '' ritchi 1993 `` scheme type composit adopt c owe consider debt algol 68 although perhap emerg form algol 's adher would approv '' ritchi 1993 b `` verilog hdl c '' pdf research school comput scienc australian nation univers june 3 2010 archiv origin pdf novemb 6 2013 retriev august 19 2013 1980 verilog first introduc verilog inspir c program languag b b `` verilog hdl c '' pdf research school comput scienc australian nation univers june 3 2010 archiv origin pdf novemb 6 2013 retriev august 19 2013 1980 verilog first introduc verilog inspir c program languag `` verilog hdl c '' pdf origin pdf retriev august 19 2013 august 19 `` name base pronounc like letter c english alphabet '' c program languag sound english chine dictionari archiv origin novemb 17 2022 retriev novemb 17 2022 `` name base pronounc like letter c english alphabet '' c program languag sound english chine dictionari archiv origin novemb 17 2022 retriev novemb 17 2022 `` name base pronounc like letter c english alphabet '' archiv retriev novemb 17 2022 novemb 17 munoz daniel `` year world still power c program toptal '' toptal engin blog retriev june 15 2024 munoz daniel `` year world still power c program toptal '' toptal engin blog retriev june 15 2024 `` year world still power c program toptal '' retriev june 15 2024 june 15 `` c languag drop lowest popular rate '' developer.com august 9 2016 archiv origin august 22 2022 retriev august 1 2022 `` c languag drop lowest popular rate '' developer.com august 9 2016 archiv origin august 22 2022 retriev august 1 2022 `` c languag drop lowest popular rate '' origin retriev august 1 2022 august 1 b c e f ritchi 1993 b c e f b c e f ritchi 1993 ritchi 1993 `` program languag popular '' 2009 archiv origin januari 16 2009 retriev januari 16 2009 `` program languag popular '' 2009 archiv origin januari 16 2009 retriev januari 16 2009 `` program languag popular '' origin retriev januari 16 2009 januari 16 `` tiob program commun index '' 2009 archiv origin may 4 2009 retriev may 6 2009 `` tiob program commun index '' 2009 archiv origin may 4 2009 retriev may 6 2009 `` tiob program commun index '' origin retriev may 6 2009 may 6 ward terri august 1983 `` annot c bibliographi c languag '' byte p. 268 retriev januari 31 2015 ward terri august 1983 `` annot c bibliographi c languag '' byte p. 268 retriev januari 31 2015 `` annot c bibliographi c languag '' retriev januari 31 2015 januari 31 `` tiob index septemb 2024 '' archiv origin septemb 18 2024 retriev septemb 20 2024 `` tiob index septemb 2024 '' archiv origin septemb 18 2024 retriev septemb 20 2024 `` tiob index septemb 2024 '' archiv retriev septemb 20 2024 septemb 20 ritchi denni `` bcpl b c '' lysator.liu.s archiv origin decemb 12 2019 retriev septemb 10 2019 ritchi denni `` bcpl b c '' lysator.liu.s archiv origin decemb 12 2019 retriev septemb 10 2019 `` bcpl b c '' archiv retriev septemb 10 2019 septemb 10 b c e jensen richard decemb 9 2020 `` `` damn stupid thing '' —the origin c '' ar technica archiv origin march 28 2022 retriev march 28 2022 b c e b c e jensen richard decemb 9 2020 `` `` damn stupid thing '' —the origin c '' ar technica archiv origin march 28 2022 retriev march 28 2022 `` `` damn stupid thing '' —the origin c '' archiv retriev march 28 2022 march 28 b johnson s. c. ritchi d. m. 1978 `` portabl c program unix system '' bell system tech j 57 6 2021–2048 citeseerx 10.1.1.138.35. doi:10.1002/j.1538-7305.1978.tb02141.x issn 0005-8580 s2cid 17510065 note pdf ocr scan origin contain render `` ibm 370 '' `` ibm 310 '' b b johnson s. c. ritchi d. m. 1978 `` portabl c program unix system '' bell system tech j 57 6 2021–2048 citeseerx 10.1.1.138.35. doi:10.1002/j.1538-7305.1978.tb02141.x issn 0005-8580 s2cid 17510065 note pdf ocr scan origin contain render `` ibm 370 '' `` ibm 310 '' johnson s. c. ritchi d. m. 2021– citeseerx 10.1.1.138.35 10.1.1.138.35 doi 10.1002/j.1538-7305.1978.tb02141.x issn 0005-8580 s2cid 17510065 mcilroy m. d. 1987 research unix reader annot excerpt programm 's manual 1971–1986 pdf technic report cstr bell lab p. 10 139 archiv pdf origin novemb 11 2017 retriev februari 1 2015 mcilroy m. d. 1987 research unix reader annot excerpt programm 's manual 1971–1986 pdf technic report cstr bell lab p. 10 139 archiv pdf origin novemb 11 2017 retriev februari 1 2015 mcilroy m. d. research unix reader annot excerpt programm 's manual 1971–1986 pdf archiv pdf retriev februari 1 2015 februari 1 kernighan brian w. ritchi denni m. februari 1978 c program languag 1st ed. englewood cliff nj prentic hall isbn 978-0-13-110163-0 kernighan brian w. ritchi denni m. februari 1978 c program languag 1st ed. englewood cliff nj prentic hall isbn 978-0-13-110163-0 kernighan brian w. ritchi denni m. c program languag englewood cliff nj prentic hall isbn 978-0-13-110163-0 `` c manual page '' freebsd miscellan inform manual freebsd 13.0 ed. may 30 2011 archiv origin januari 21 2021 retriev januari 15 2021 1 archiv januari 21 2021 wayback machin `` c manual page '' freebsd miscellan inform manual freebsd 13.0 ed. may 30 2011 archiv origin januari 21 2021 retriev januari 15 2021 1 archiv januari 21 2021 wayback machin freebsd miscellan inform manual archiv retriev januari 15 2021 januari 15 1 archiv wayback machin kernighan brian w. ritchi denni m. march 1988 c program languag 2nd ed. englewood cliff nj prentic hall isbn 978-0-13-110362-7 kernighan brian w. ritchi denni m. march 1988 c program languag 2nd ed. englewood cliff nj prentic hall isbn 978-0-13-110362-7 kernighan brian w. ritchi denni m. c program languag englewood cliff nj prentic hall isbn 978-0-13-110362-7 stroustrup bjarn 2002 sibl rivalri c c++ pdf report lab archiv pdf origin august 24 2014 retriev april 14 2014 stroustrup bjarn 2002 sibl rivalri c c++ pdf report lab archiv pdf origin august 24 2014 retriev april 14 2014 stroustrup bjarn sibl rivalri c c++ pdf archiv pdf retriev april 14 2014 april 14 `` rational american nation standard inform system – program languag – c '' archiv origin juli 17 2024 retriev juli 17 2024 `` rational american nation standard inform system – program languag – c '' archiv origin juli 17 2024 retriev juli 17 2024 `` rational american nation standard inform system – program languag – c '' origin retriev juli 17 2024 juli 17 c integr intern organ standard march 30 1995 archiv origin juli 25 2018 retriev juli 24 2018 c integr intern organ standard march 30 1995 archiv origin juli 25 2018 retriev juli 24 2018 c integr archiv retriev juli 24 2018 juli 24 `` jtc1/sc22/wg14 – c '' home page iso/iec archiv origin februari 12 2018 retriev june 2 2011 `` jtc1/sc22/wg14 – c '' home page iso/iec archiv origin februari 12 2018 retriev june 2 2011 `` jtc1/sc22/wg14 – c '' archiv retriev june 2 2011 june 2 andrew binstock octob 12 2011 `` interview herb sutter '' dr. dobb archiv origin august 2 2013 retriev septemb 7 2013 andrew binstock octob 12 2011 `` interview herb sutter '' dr. dobb archiv origin august 2 2013 retriev septemb 7 2013 `` interview herb sutter '' dr. dobb archiv retriev septemb 7 2013 septemb 7 `` wg14-n3132 revis c23 schedul '' pdf open-std.org june 4 2023 archiv pdf origin june 9 2023 `` wg14-n3132 revis c23 schedul '' pdf open-std.org june 4 2023 archiv pdf origin june 9 2023 `` wg14-n3132 revis c23 schedul '' pdf archiv pdf `` wg14-n3220 work draft c2i '' pdf open-std.org februari 21 2024 archiv pdf origin februari 26 2024 `` wg14-n3220 work draft c2i '' pdf open-std.org februari 21 2024 archiv pdf origin februari 26 2024 `` wg14-n3220 work draft c2i '' pdf archiv pdf `` tr 18037 embed c '' pdf open-std.org april 4 2006 iso/iec jtc1 sc22 wg14 n1169 archiv pdf origin februari 25 2021 retriev juli 26 2011 `` tr 18037 embed c '' pdf open-std.org april 4 2006 iso/iec jtc1 sc22 wg14 n1169 archiv pdf origin februari 25 2021 retriev juli 26 2011 `` tr 18037 embed c '' pdf archiv pdf retriev juli 26 2011 juli 26 harbison samuel p. steel guy l. 2002 c refer manual 5th ed. englewood cliff nj prentic hall isbn 978-0-13-089592-9 contain bnf grammar c. harbison samuel p. steel guy l. 2002 c refer manual 5th ed. englewood cliff nj prentic hall isbn 978-0-13-089592-9 contain bnf grammar c. steel guy l. englewood cliff nj prentic hall isbn 978-0-13-089592-9 bnf kernighan ritchi 1988 p. 192 kernighan ritchi 1988 p. 192 kernighan ritchi 1988 kernighan ritchi 1978 p. 3 kernighan ritchi 1978 p. 3 kernighan ritchi 1978 `` iso/iec 9899:201x iso c11 committe draft '' pdf open-std.org decemb 2 2010 archiv pdf origin decemb 22 2017 retriev septemb 16 2011 `` iso/iec 9899:201x iso c11 committe draft '' pdf open-std.org decemb 2 2010 archiv pdf origin decemb 22 2017 retriev septemb 16 2011 `` iso/iec 9899:201x iso c11 committe draft '' pdf archiv pdf retriev septemb 16 2011 septemb 16 kernighan ritchi 1988 pp 192 259 kernighan ritchi 1988 pp 192 259 kernighan ritchi 1988 `` 10 common program mistak c++ '' cs.ucr.edu archiv origin octob 21 2008 retriev june 26 2009 `` 10 common program mistak c++ '' cs.ucr.edu archiv origin octob 21 2008 retriev june 26 2009 `` 10 common program mistak c++ '' archiv retriev june 26 2009 june 26 schultz thoma 2004 c 8051 3rd ed. otsego mi pagefre publish inc. p. 20 isbn 978-1-58961-237-2 retriev februari 10 2012 schultz thoma 2004 c 8051 3rd ed. otsego mi pagefre publish inc. p. 20 isbn 978-1-58961-237-2 retriev februari 10 2012 c 8051 isbn 978-1-58961-237-2 retriev februari 10 2012 februari 10 kernighan ritchi 1978 p. 6 kernighan ritchi 1978 p. 6 kernighan ritchi 1978 b c e f g klemen ben 2013 21st centuri c. o'reilli medium isbn 978-1-4493-2714-9 b c e f g b c e f g klemen ben 2013 21st centuri c. o'reilli medium isbn 978-1-4493-2714-9 klemen ben o'reilli medium isbn 978-1-4493-2714-9 feuer alan r. gehani narain h. march 1982 `` comparison program languag c pascal '' acm comput survey 14 1 73–92 doi:10.1145/356869.356872 s2cid 3136859 feuer alan r. gehani narain h. march 1982 `` comparison program languag c pascal '' acm comput survey 14 1 73–92 doi:10.1145/356869.356872 s2cid 3136859 73– doi 10.1145/356869.356872 s2cid 3136859 kernighan ritchi 1988 p. 122 kernighan ritchi 1988 p. 122 kernighan ritchi 1988 exampl gcc provid _fortify_sourc `` secur featur compil time buffer check fortify_sourc '' fedoraproject.org archiv origin januari 7 2007 retriev august 5 2012 exampl gcc provid _fortify_sourc `` secur featur compil time buffer check fortify_sourc '' fedoraproject.org archiv origin januari 7 2007 retriev august 5 2012 `` secur featur compil time buffer check fortify_sourc '' archiv retriev august 5 2012 august 5 เอี่ยมสิริวงศ์ โอภาศ 2016 program c. bangkok thailand se-educ public compani limit pp 225–230 isbn 978-616-08-2740-4 เอี่ยมสิริวงศ์ โอภาศ 2016 program c. bangkok thailand se-educ public compani limit pp 225–230 isbn 978-616-08-2740-4 225– isbn 978-616-08-2740-4 raymond eric s. octob 11 1996 new hacker 's dictionari 3rd ed. mit press p. 432 isbn 978-0-262-68092-9 retriev august 5 2012 raymond eric s. octob 11 1996 new hacker 's dictionari 3rd ed. mit press p. 432 isbn 978-0-262-68092-9 retriev august 5 2012 raymond eric s. new hacker 's dictionari isbn 978-0-262-68092-9 retriev august 5 2012 august 5 `` man page lint freebsd section 1 '' unix.com may 24 2001 retriev juli 15 2014 `` man page lint freebsd section 1 '' unix.com may 24 2001 retriev juli 15 2014 `` man page lint freebsd section 1 '' retriev juli 15 2014 juli 15 `` cs107 valgrind memcheck '' web.stanford.edu retriev june 23 2023 `` cs107 valgrind memcheck '' web.stanford.edu retriev june 23 2023 `` cs107 valgrind memcheck '' retriev june 23 2023 june 23 hast reed joyc bob `` purifi fast detect memori leak access error '' pdf pure softwar inc. 9 hast reed joyc bob `` purifi fast detect memori leak access error '' pdf pure softwar inc. 9 `` purifi fast detect memori leak access error '' pdf dale nell b. weem chip 2014 program problem solv c++ 6th ed. burlington massachusett jone bartlett learn isbn 978-1449694289 oclc 894992484 dale nell b. weem chip 2014 program problem solv c++ 6th ed. burlington massachusett jone bartlett learn isbn 978-1449694289 oclc 894992484 isbn 978-1449694289 oclc 894992484 `` c – mother languag '' ict academi iitk novemb 13 2018 archiv origin may 31 2021 retriev octob 11 2022 `` c – mother languag '' ict academi iitk novemb 13 2018 archiv origin may 31 2021 retriev octob 11 2022 `` c – mother languag '' origin retriev octob 11 2022 octob 11 `` 1 extend python c c++ '' python 3.10.7 document archiv origin novemb 5 2012 retriev octob 11 2022 `` 1 extend python c c++ '' python 3.10.7 document archiv origin novemb 5 2012 retriev octob 11 2022 `` 1 extend python c c++ '' archiv retriev octob 11 2022 octob 11 conrad michael januari 22 2018 `` overview perl 5 engin '' opensource.com archiv origin may 26 2022 retriev octob 11 2022 conrad michael januari 22 2018 `` overview perl 5 engin '' opensource.com archiv origin may 26 2022 retriev octob 11 2022 `` overview perl 5 engin '' archiv retriev octob 11 2022 octob 11 `` rubi c c++ '' rubi program languag archiv origin august 12 2013 retriev octob 11 2022 `` rubi c c++ '' rubi program languag archiv origin august 12 2013 retriev octob 11 2022 `` rubi c c++ '' archiv retriev octob 11 2022 octob 11 para michael august 3 2022 `` php write first php program '' freecodecamp archiv origin august 4 2022 retriev octob 11 2022 para michael august 3 2022 `` php write first php program '' freecodecamp archiv origin august 4 2022 retriev octob 11 2022 `` php write first php program '' archiv retriev octob 11 2022 octob 11 dr. dobb 's sourcebook u.s. miller freeman inc. november–decemb 1995 dr. dobb 's sourcebook u.s. miller freeman inc. november–decemb 1995 `` use c cgi program '' linuxjournal.com march 1 2005 archiv origin februari 13 2010 retriev januari 4 2010 `` use c cgi program '' linuxjournal.com march 1 2005 archiv origin februari 13 2010 retriev januari 4 2010 `` use c cgi program '' archiv retriev januari 4 2010 januari 4 perkin luc septemb 17 2013 `` web develop c crazi crazi like fox '' medium archiv origin octob 4 2014 retriev april 8 2022 perkin luc septemb 17 2013 `` web develop c crazi crazi like fox '' medium archiv origin octob 4 2014 retriev april 8 2022 `` web develop c crazi crazi like fox '' archiv retriev april 8 2022 april 8 munoz daniel `` year world still power c program '' toptal engin blog retriev novemb 17 2023 munoz daniel `` year world still power c program '' toptal engin blog retriev novemb 17 2023 `` year world still power c program '' retriev novemb 17 2023 novemb 17 metz cade `` denni ritchi shoulder steve job stood '' wire archiv origin april 12 2022 retriev april 19 2022 metz cade `` denni ritchi shoulder steve job stood '' wire archiv origin april 12 2022 retriev april 19 2022 `` denni ritchi shoulder steve job stood '' archiv retriev april 19 2022 april 19 corob-msft march 31 2022 `` pragma direct __pragma _pragma keyword '' microsoft learn archiv origin septemb 24 2022 retriev septemb 24 2022 corob-msft march 31 2022 `` pragma direct __pragma _pragma keyword '' microsoft learn archiv origin septemb 24 2022 retriev septemb 24 2022 `` pragma direct __pragma _pragma keyword '' archiv retriev septemb 24 2022 septemb 24 `` pragma c preprocessor '' gcc gnu compil collect archiv origin june 17 2002 retriev septemb 24 2022 `` pragma c preprocessor '' gcc gnu compil collect archiv origin june 17 2002 retriev septemb 24 2022 `` pragma c preprocessor '' archiv retriev septemb 24 2022 septemb 24 `` pragma '' intel c++ compil classic develop guid refer intel archiv origin april 10 2022 retriev april 10 2022 `` pragma '' intel c++ compil classic develop guid refer intel archiv origin april 10 2022 retriev april 10 2022 `` pragma '' archiv retriev april 10 2022 april 10 `` prais c preprocessor '' apenwarr august 13 2007 retriev juli 9 2023 `` prais c preprocessor '' apenwarr august 13 2007 retriev juli 9 2023 `` prais c preprocessor '' retriev juli 9 2023 juli 9 robert eric s. march 21 1989 `` implement except c '' pdf dec system research center src-rr-40 archiv pdf origin januari 15 2017 retriev januari 4 2022 robert eric s. march 21 1989 `` implement except c '' pdf dec system research center src-rr-40 archiv pdf origin januari 15 2017 retriev januari 4 2022 `` implement except c '' pdf dec system research center archiv pdf retriev januari 4 2022 januari 4 mcmillan robert august 1 2013 `` java lose mojo '' wire archiv origin februari 15 2017 retriev march 5 2017 mcmillan robert august 1 2013 `` java lose mojo '' wire archiv origin februari 15 2017 retriev march 5 2017 `` java lose mojo '' wire archiv retriev march 5 2017 march 5 o'regan gerard septemb 24 2015 pillar comput compendium select pivot technolog firm springer isbn 978-3319214641 oclc 922324121 o'regan gerard septemb 24 2015 pillar comput compendium select pivot technolog firm springer isbn 978-3319214641 oclc 922324121 isbn 978-3319214641 oclc 922324121 rauchwerg lawrenc 2004 languag compil parallel comput 16th intern workshop lcpc 2003 colleg station tx usa octob 2–4 2003 revis paper springer isbn 978-3540246442 oclc 57965544 rauchwerg lawrenc 2004 languag compil parallel comput 16th intern workshop lcpc 2003 colleg station tx usa octob 2–4 2003 revis paper springer isbn 978-3540246442 oclc 57965544 isbn 978-3540246442 oclc 57965544 stroustrup bjarn 1993 `` histori c++ 1979–1991 '' pdf archiv pdf origin februari 2 2019 retriev june 9 2011 stroustrup bjarn 1993 `` histori c++ 1979–1991 '' pdf archiv pdf origin februari 2 2019 retriev june 9 2011 stroustrup bjarn `` histori c++ 1979–1991 '' pdf archiv pdf retriev june 9 2011 june 9 sourc edit edit edit ritchi denni m. march 1993 `` develop c languag '' acm sigplan notic 28 3 acm 201–208 doi:10.1145/155360.155580 courtesi author also ritchi denni m. `` chistori '' www.bell-labs.com retriev march 29 2022 ritchi denni m. `` develop c languag '' 201– doi 10.1145/155360.155580 10.1145/155360.155580 courtesi author also ritchi denni m. `` chistori '' www.bell-labs.com retriev march 29 2022 `` chistori '' retriev march 29 2022 march 29 ritchi denni m. 1993 `` develop c languag '' second acm sigplan confer histori program languag hopl-ii acm pp 201–208 doi:10.1145/154766.155580 isbn 0-89791-570-4 archiv origin april 11 2019 retriev novemb 4 2014 `` develop c languag '' acm 201– doi 10.1145/154766.155580 isbn 0-89791-570-4 origin retriev novemb 4 2014 novemb 4 kernighan brian w. ritchi denni m. 1988 c program languag 2nd ed. prentic hall isbn 0-13-110362-8 kernighan brian w. ritchi denni m. prentic hall isbn 0-13-110362-8 read edit edit edit plauger p.j 1992 standard c librari 1 ed. prentic hall isbn 978-0131315099 sourc plauger p.j isbn 978-0131315099 sourc banahan m. bradi d. doran m. 1991 c book featur ansi c standard 2 ed. addison-wesley isbn 978-0201544336 free isbn 978-0201544336 free feuer alan r. 1985 c puzzl book 1 ed. prentic hall isbn 0131099345 isbn 0131099345 harbison samuel steel guy jr. 2002 c refer manual 5 ed. pearson isbn 978-0130895929 archiv isbn 978-0130895929 archiv king k.n 2008 c program modern approach 2 ed. w. w. norton isbn 978-0393979503 archiv isbn 978-0393979503 archiv griffith david griffith dawn 2012 head first c 1 ed. o'reilli isbn 978-1449399917 isbn 978-1449399917 perri greg miller dean 2013 c program absolut beginn 's guid 3 ed. que isbn 978-0789751980 isbn 978-0789751980 deitel paul deitel harvey 2015 c program 8 ed. pearson isbn 978-0133976892 isbn 978-0133976892 gustedt jen 2019 modern c 2 ed. man isbn 978-1617295812 free isbn 978-1617295812 free extern link edit edit edit sister project sister project medium common medium common medium news wikinew news wikinew news quotat wikiquot quotat wikiquot quotat textbook wikibook textbook wikibook textbook resourc wikivers resourc wikivers resourc iso c work group offici websit iso/iec 9899 publicli avail offici c document includ c99 rational `` c99 technic corrigenda tc1 tc2 tc3 includ '' pdf archiv pdf origin octob 25 2007 3.61 mb iso c work group offici websit iso/iec 9899 publicli avail offici c document includ c99 rational iso/iec 9899 `` c99 technic corrigenda tc1 tc2 tc3 includ '' pdf archiv pdf origin octob 25 2007 3.61 mb `` c99 technic corrigenda tc1 tc2 tc3 includ '' pdf archiv pdf 3.61 mb comp.lang.c frequent ask question comp.lang.c frequent ask question histori c denni ritchi histori c c librari refer exampl c librari refer exampl v v e e c program languag ansi c ansi c c99 c99 c11 c11 c17 c17 c23 c23 embed c embed c misra c misra c function function header file header file oper oper string string syntax syntax preprocessor preprocessor data type data type standard librari char char file i/o file i/o math math dynam memori dynam memori string string time time variad variad posix posix bionic libhybri bionic libhybri libhybri dietlibc dietlibc glibc eglibc glibc eglibc eglibc klibc klibc window crt window crt musl musl newlib newlib uclibc uclibc compil ack ack borland turbo c borland turbo c clang clang gcc gcc icc icc lcc lcc norcroft c norcroft c pcc pcc sdcc sdcc tcc tcc visual studio express c++ visual studio express c++ watcom c/c++ watcom c/c++ ide anjuta anjuta clion clion code :block code :block codelit codelit eclips eclips geani geani gnome builder gnome builder kdevelop kdevelop visual studio visual studio netbean netbean compat c c++ compat c c++ comparison pascal comparison pascal c++ c++ c c objective-c objective-c alef alef limbo limbo go go vala vala denni ritchi denni ritchi categori categori v v e e integr develop environ c c++ open sourc arduino ide arduino ide code :block code :block codelit codelit dev-c++ dev-c++ eclips eclips emac emac geani geani gnome builder gnome builder kakoun kakoun kdevelop kdevelop netbean netbean qdevelop qdevelop qt creator qt creator theid theid vi–vim vi vim openwatcom openwatcom freewar deveco studio deveco studio oracl develop studio oracl develop studio visual studio code visual studio code visual studio commun visual studio commun xcode xcode retail c++builder c++builder eclipse-bas codewarrior myeclips eclips codewarrior codewarrior myeclips myeclips visual studio visual studio jetbrain intellij idea appcod clion jetbrain intellij idea intellij idea appcod appcod clion clion labwindows/cvi labwindows/cvi ibm ration softwar architect ration softwar architect understand understand slickedit slickedit anjuta anjuta visualag visualag visual c++ express visual c++ express monodevelop monodevelop sharpdevelop sharpdevelop borland turbo c c++ borland turbo c c++ quickc quickc java open sourc anjuta anjuta bluej bluej drjava drjava eclips eclips geani geani greenfoot greenfoot intellij idea commun edit android studio intellij idea android studio android studio netbean netbean deveco studio deveco studio freewar jgrasp jgrasp jdevelop jdevelop retail jcreator myeclips myeclips jetbrain intellij idea jetbrain intellij idea slickedit slickedit understand understand metrowerk codewarrior pro java codewarrior jbuilder jbuilder sun java studio creator supersed netbean sun java studio creator netbean visualag supersed eclips visualag eclips visual café aka espresso supersed jbuilder visual café jbuilder visual j++ visual j++ xelfi becam netbean xelfi netbean javascript open sourc vim vim visual studio code visual studio code atom atom chromium chromium deveco studio deveco studio cli .net open sourc visual studio code visual studio code pascalabc.net pascalabc.net freewar visual studio commun visual studio commun retail visual studio visual studio rider rider understand understand xamarin studio xamarin studio monodevelop monodevelop sharpdevelop sharpdevelop visual basic express visual basic express visual web develop express visual web develop express visual j express visual j express visual studio express window phone visual studio express window phone visual c++ express visual c++ express visual c express visual c express express desktop express desktop express web express web express window express window flash adob flash builder adob flash builder flashdevelop powerflash fdt powerflash fdt php open sourc aptana aptana netbean netbean komodo edit komodo edit komodo ide komodo ide kdevelop kdevelop proprietari codelobst codelobst phpstorm phpstorm phpedit phpedit slickedit slickedit zend studio zend studio r rstudio rstudio r tool visual studio r tool visual studio python open sourc anjuta anjuta idl idl eric eric light tabl light tabl ninja-id ninja-id pydev pydev pyscript pida spyder spyder komodo edit komodo edit komodo ide komodo ide kdevelop kdevelop vim vim visual studio code visual studio code thonni thonni deveco studio deveco studio proprietari pycharm pycharm pythonanywher pythonanywher visual studio code visual studio code slickedit slickedit wing ide wing ide pascal object pascal open sourc free pascal ide free pascal lazaru lazaru dev-pasc dev-pasc gnavi pascalabc.net pascalabc.net freewar delphi commun delphi commun retail delphi delphi turbo pascal turbo pascal virtual pascal virtual pascal borland kylix borland kylix quickpasc quickpasc basic open sourc basic-256 basic-256 microsoft small basic microsoft small basic sdlbasic sdlbasic gamba gamba basic4gl basic4gl visualfbeditor winfb visualfbeditor winfb inform inform freewar futurebas futurebas rapidq rapidq visual studio commun visual studio commun retail visual studio visual studio n basic n basic purebas purebas glbasic glbasic liberti basic liberti basic xojo xojo ca-real ca-real monodevelop monodevelop quickbas quickbas qbasic qbasic sharpdevelop sharpdevelop visual basic visual basic visual basic express visual basic express go open sourc vim vim visual studio code vscode visual studio code eclips eclips freewar cloud9 ide cloud9 ide retail goland goland hax intellij idea intellij idea v code v code sublim text sublim text powerflash fdt powerflash fdt pop-11 poplog poplog onlin aw cloud9 ide cloud9 ide eclips che eclips che sourcelair sourcelair comparison comparison categori categori v v e e program languag comparison comparison timelin timelin histori histori ada ada algol simula algol simula simula apl apl assembl assembl basic visual basic classic .net basic visual basic classic .net visual basic classic classic .net .net c c c++ c++ c c cobol cobol erlang erlang forth forth fortran fortran go go haskel haskel java java javascript javascript julia julia kotlin kotlin lisp lisp lua lua matlab matlab ml ml pascal object pascal pascal object pascal object pascal perl perl php php prolog prolog python python r r rubi rubi rust rust sql sql scratch scratch shell shell smalltalk smalltalk swift swift ... ... list alphabet alphabet categor categor gener gener non-english-bas non-english-bas categori categori author control databas fast fast fast germani germani germani unit state unit state unit state franc franc franc bnf data bnf data bnf data czech republ czech republ czech republ czech republ israel israel israel idref idref idref http //en.wikipedia.org/w/index.php title=c_ programming_languag oldid=1268541771 categori c program languag c program languag american invent american invent c program languag famili c program languag famili cross-platform softwar cross-platform softwar high-level program languag high-level program languag procedur program languag procedur program languag structur program languag structur program languag program languag creat 1972 program languag creat 1972 program languag iso standard program languag iso standard static type program languag static type program languag system program languag system program languag wikipedia articl need clarif novemb 2023 wikipedia articl need clarif novemb 2023 webarch templat wayback link webarch templat wayback link articl short descript articl short descript short descript match wikidata short descript match wikidata wikipedia pend chang protect page wikipedia pend chang protect page use mdi date octob 2024 use mdi date octob 2024 articl vagu ambigu time articl vagu ambigu time vagu ambigu time august 2022 vagu ambigu time august 2022 wikipedia articl need updat februari 2021 wikipedia articl need updat februari 2021 wikipedia articl need updat wikipedia articl need updat wikipedia articl need updat octob 2024 wikipedia articl need updat octob 2024 articl need addit refer octob 2012 articl need addit refer octob 2012 articl need addit refer articl need addit refer wikipedia articl need clarif octob 2021 wikipedia articl need clarif octob 2021 articl need addit refer juli 2014 articl need addit refer juli 2014 page use sister project link default search page use sister project link default search page use sister project link wikidata mismatch page use sister project link wikidata mismatch page use sister project link hidden wikidata page use sister project link hidden wikidata articl exampl c code articl exampl c code page last edit 10 januari 2025 08:10 utc utc text avail creativ common attribution-sharealik 4.0 licens addit term may appli use site agre term use privaci polici wikipedia® regist trademark wikimedia foundat inc. non-profit organ creativ common attribution-sharealik 4.0 licens term use privaci polici wikimedia foundat inc. privaci polici privaci polici wikipedia wikipedia disclaim disclaim contact wikipedia contact wikipedia code conduct code conduct develop develop statist statist cooki statement cooki statement mobil view mobil view",
    "https://en.wikipedia.org/wiki/Ruby_(programming_language)": "jump content main menu main page main page main page content content content current event current event current event random articl random articl random articl wikipedia wikipedia wikipedia contact u contact u contact u help help help learn edit learn edit learn edit commun portal commun portal commun portal recent chang recent chang recent chang upload file upload file upload file search search appear donat donat donat creat account creat account creat account log log log person tool donat donat donat creat account creat account creat account log log log learn learn contribut contribut contribut talk talk talk content top top 1 histori toggl histori subsect 1.1 earli concept 1.2 earli releas 1.3 rubi 1.8 1.9 1.4 rubi 2 1.5 rubi 3 1 histori 1 histori toggl histori subsect 1.1 earli concept 1.1 earli concept 1.1 earli concept 1.2 earli releas 1.2 earli releas 1.2 earli releas 1.3 rubi 1.8 1.9 1.3 rubi 1.8 1.9 1.3 rubi 1.8 1.9 1.4 rubi 2 1.4 rubi 2 1.4 rubi 2 1.5 rubi 3 1.5 rubi 3 1.5 rubi 3 2 semant philosophi 2 semant philosophi 2 semant philosophi 3 featur 3 featur 3 featur 4 syntax 4 syntax 4 syntax 5 implement toggl implement subsect 5.1 matz 's rubi interpret 5.2 altern implement 5.3 platform support 5 implement 5 implement toggl implement subsect 5.1 matz 's rubi interpret 5.1 matz 's rubi interpret 5.1 matz 's rubi interpret 5.2 altern implement 5.2 altern implement 5.2 altern implement 5.3 platform support 5.3 platform support 5.3 platform support 6 repositori librari 6 repositori librari 6 repositori librari 7 see also 7 see also 7 see also 8 refer 8 refer 8 refer 9 read 9 read 9 read 10 extern link 10 extern link 10 extern link toggl tabl content rubi program languag rubi program languag 74 languag العربية العربية العربية aragoné aragoné aragoné asturianu asturianu asturianu azərbaycanca azərbaycanca azərbaycanca تۆرکجه تۆرکجه تۆرکجه বাংলা বাংলা বাংলা беларуская беларуская беларуская български български български bosanski bosanski bosanski català català català čeština čeština čeština dansk dansk dansk deutsch deutsch deutsch eesti eesti eesti ελληνικά ελληνικά ελληνικά español español español esperanto esperanto esperanto euskara euskara euskara فارسی فارسی فارسی françai françai françai galego galego galego 한국어 한국어 한국어 hausa hausa hausa հայերեն հայերեն հայերեն hrvatski hrvatski hrvatski ido ido ido bahasa indonesia bahasa indonesia bahasa indonesia interlingua interlingua interlingua italiano italiano italiano עברית עברית עברית ქართული ქართული ქართული kiswahili kiswahili kiswahili кыргызча кыргызча кыргызча latviešu latviešu latviešu lietuvių lietuvių lietuvių lombard lombard lombard magyar magyar magyar മലയാളം മലയാളം മലയാളം मराठी मराठी मराठी bahasa melayu bahasa melayu bahasa melayu မြန်မာဘာသာ မြန်မာဘာသာ မြန်မာဘာသာ nederland nederland nederland 日本語 日本語 日本語 norsk bokmål norsk bokmål norsk bokmål norsk nynorsk norsk nynorsk norsk nynorsk ଓଡ଼ିଆ ଓଡ଼ିଆ ଓଡ଼ିଆ oʻzbekcha ўзбекча oʻzbekcha ўзбекча oʻzbekcha ўзбекча ਪੰਜਾਬੀ ਪੰਜਾਬੀ ਪੰਜਾਬੀ پنجابی پنجابی پنجابی polski polski polski portuguê portuguê portuguê română română română русский русский русский shqip shqip shqip simpl english simpl english simpl english slovenčina slovenčina slovenčina کوردی کوردی کوردی српски srpski српски srpski српски srpski srpskohrvatski српскохрватски srpskohrvatski српскохрватски srpskohrvatski српскохрватски suomi suomi suomi svenska svenska svenska தமிழ் தமிழ் தமிழ் татарча tatarça татарча tatarça татарча tatarça తెలుగు తెలుగు తెలుగు ไทย ไทย ไทย тоҷикӣ тоҷикӣ тоҷикӣ türkçe türkçe türkçe українська українська українська اردو اردو اردو tiếng việt tiếng việt tiếng việt 吴语 吴语 吴语 粵語 粵語 粵語 žemaitėška žemaitėška žemaitėška 中文 中文 中文 edit link edit link articl articl articl talk talk talk english read read read edit edit edit view histori view histori view histori tool read read read edit edit edit view histori view histori view histori link link link relat chang relat chang relat chang upload file upload file upload file special page special page special page perman link perman link perman link page inform page inform page inform cite page cite page cite page get shorten url get shorten url get shorten url download qr code download qr code download qr code download pdf download pdf download pdf printabl version printabl version printabl version wikimedia common wikimedia common wikimedia common wikibook wikibook wikibook wikiquot wikiquot wikiquot wikivers wikivers wikivers wikidata item wikidata item wikidata item rubi rail plea help improv make understand non-expert without remov technic detail help improv make understand non-expert june 2022 june 2022 learn remov messag learn remov messag paradigm multi-paradigm function imper object-ori reflect design yukihiro matsumoto develop 30 year ago 1995 1995 stabl releas 1 15 day ago 25 decemb 2024 25 decemb 2024 type disciplin duck dynam strong scope c o cross-platform licens rubi licens filenam extens ruby-lang.org ruby-lang.org implement rubi mri trufflerubi yarv rubiniu jrubi rubymot mrubi ada 2 basic 3 c++ 2 clu 4 dylan 4 eiffel 2 lisp 4 lua perl 4 python 4 smalltalk 4 clojur coffeescript crystal elixir groovi julia 5 mirah nu 6 ring 7 rust 8 swift 9 rubi program wikibook rubi program rubi interpret high-level general-purpos program languag design emphasi program product simplic rubi everyth object includ primit data type develop mid-1990 yukihiro `` matz '' matsumoto japan interpret high-level general-purpos program languag object primit data type yukihiro `` matz '' matsumoto japan rubi dynam type use garbag collect just-in-tim compil support multipl program paradigm includ procedur object-ori function program accord creator rubi influenc perl smalltalk eiffel ada basic lisp 10 3 dynam type garbag collect just-in-tim compil procedur object-ori function program perl smalltalk eiffel ada basic lisp 10 3 histori edit edit edit histori rubi earli concept edit edit edit matsumoto said rubi conceiv 1993 1999 post ruby-talk mail list describ earli idea languag talk colleagu possibl object-ori script languag knew perl perl4 perl5 n't like realli smell toy languag still object-ori languag seem promis knew python n't like n't think true object-ori languag – oo featur appear add-on languag languag maniac oo fan 15 year realli want genuin object-ori easy-to-us script languag look could n't find one decid make toy languag python matsumoto describ design rubi like simpl lisp languag core object system like smalltalk block inspir higher-ord function practic util like perl 11 lisp higher-ord function 11 name `` rubi '' origin onlin chat session matsumoto keiju ishitsuka februari 24 1993 code written languag 12 initi two name propos `` coral '' `` rubi '' matsumoto chose latter later e-mail ishitsuka 13 matsumoto later note factor choos name `` rubi '' –it birthston one colleagu 14 15 12 coral rubi 13 birthston 14 15 earli releas edit edit edit first public releas rubi 0.95 announc japanes domest newsgroup decemb 21 1995 16 17 subsequ three version rubi releas two day 12 releas coincid launch japanese-languag ruby-list mail list first mail list new languag newsgroup 16 17 12 japanese-languag alreadi present stage develop mani featur familiar later releas rubi includ object-ori design class inherit mixin iter closur except handl garbag collect 18 object-ori class mixin iter closur except handl garbag collect 18 releas rubi 0.95 1995 sever stabl version rubi releas year rubi 1.0 decemb 25 1996 12 12 rubi 1.2 decemb 1998 rubi 1.4 august 1999 rubi 1.6 septemb 2000 1997 first articl rubi publish web year matsumoto hire netlab.jp work rubi full-tim develop 12 12 1998 rubi applic archiv launch matsumoto along simpl english-languag homepag rubi 12 12 1999 first english languag mail list ruby-talk began signal grow interest languag outsid japan 19 year matsumoto keiju ishitsuka wrote first book rubi object-ori script languag rubi オブジェクト指向スクリプト言語 rubi publish japan octob 1999 would follow earli 2000 around 20 book rubi publish japanes 12 19 12 2000 rubi popular python japan 20 septemb 2000 first english languag book program rubi print later freeli releas public widen adopt rubi amongst english speaker earli 2002 english-languag ruby-talk mail list receiv messag japanese-languag ruby-list demonstr rubi 's increas popular non-japanes speak world 20 program rubi rubi 1.8 1.9 edit edit edit rubi 1.8 initi releas august 2003 stabl long time retir june 2013 21 although deprec still code base rubi 1.8 partial compat rubi 1.9 21 rubi 1.8 subject sever industri standard languag specif rubi develop open standard promot center information-technolog promot agenc japanes govern agenc submiss japanes industri standard committe jisc intern organ standard iso accept japanes industri standard ji x 3017 2011 22 intern standard iso/iec 30170 2012 23 24 japanes govern japanes industri standard committe intern organ standard 22 23 24 rubi rail around 2005 interest rubi languag surg tandem rubi rail web framework written rubi rail frequent credit increas awar rubi 25 rubi rail web framework 25 effect rubi 1.9.3 releas octob 31 2011 26 rubi switch dual-licens rubi licens gpl dual-licens rubi licens two-claus bsd licens 27 adopt 1.9 slow chang 1.8 requir mani popular third parti gem rewritten rubi 1.9 introduc mani signific chang 1.8 seri exampl includ 28 26 27 gem 28 block local variabl variabl local block declar block local block addit lambda syntax f b put b lambda f b put b addit hash liter syntax use colon symbol key symbol_key `` valu '' == symbol_key `` valu '' hash symbol_key `` valu '' == symbol_key `` valu '' per-str charact encod support charact encod new socket api ipv6 support ipv6 require_rel import secur rubi 2 edit edit edit rubi 2.0 intend fulli backward compat rubi 1.9.3 offici 2.0.0 releas februari 24 2013 five known minor incompat 29 rubi 2.0 ad sever new featur includ 29 method keyword argument method new method modul prepend extend class new liter creat array symbol new api lazi evalu enumer lazi evalu new convent use to_h convert object hash 30 30 start 2.1.0 rubi 's version polici chang similar semant version 31 semant version 31 rubi 2.2.0 includ speed-up bugfix librari updat remov deprec api notabl rubi 2.2.0 introduc chang memori handl – increment garbag collector support garbag collect symbol option compil directli jemalloc also contain experiment support use vfork 2 system spawn ad support unicod 7.0 specif sinc version 2.2.1 32 rubi mri perform powerpc64 improv 33 34 35 featur made obsolet remov includ callcc dl librari digest :hmac lib/rational.rb lib/complex.rb gserver logger :applic well variou c api function 36 vfork unicod 32 rubi mri powerpc64 33 34 35 36 rubi 2.3.0 includ mani perform improv updat bugfix includ chang proc call socket io use except keyword thread name handl default passiv net :ftp connect rake remov stdlib 37 notabl chang includ 37 abil mark string liter frozen default consequ larg perform increas string oper 38 string liter 38 hash comparison allow direct check key/valu pair instead key new safe navig oper ea nil handl e.g instead obj obj.foo obj.foo.bar use obj .foo .bar safe navig oper obj obj foo obj foo bar did_you_mean gem bundl default requir startup automat suggest similar name match nameerror nomethoderror hash dig array dig easili extract deepli nest valu e.g given profil social wikipedia name 'foo baz valu foo baz retriev profile.dig social wikipedia name profil social wikipedia name 'foo baz .grep_v regexp match neg exampl given regular express addit new featur rubi 2.4.0 includ perform improv hash tabl array max array min instanc variabl access 39 notabl chang includ 39 bind irb start repl session similar binding.pri unifi fixnum bignum integ class string support unicod case map ascii new method regexp match faster boolean version regexp match thread deadlock detect show thread backtrac depend notabl chang rubi 2.5.0 includ rescu ensur statement automat use surround do-end block less need extra begin-end block method-chain yield_self support branch coverag method coverag measur easier hash transform hash slice hash transform_key top come lot perform improv like faster block pas 3 time faster faster mutex faster erb templat improv concaten method notabl chang rubi 2.6.0 includ experiment just-in-tim compil jit rubyvm :abstractsyntaxtre experiment just-in-tim compil notabl chang rubi 2.7.0 includ pattern match experiment repl improv compact gc separ posit keyword argument rubi 3 edit edit edit rubi 3.0.0 releas christma day 2020 40 known rubi 3x3 mean program would run three time faster rubi 3.0 compar rubi 2.0 41 alreadi implement intermedi releas road 2 3 achiev 3x3 rubi 3 come mjit later yjit just-in-tim compil make program faster although describ experiment remain disabl default enabl flag runtim christma 40 41 anoth goal rubi 3.0 improv concurr two util fibr schedul experiment ractor facilit goal 40 ractor light-weight thread-saf achiev exchang messag rather share object concurr 40 rubi 3.0 introduc rb languag describ type rubi program static analysi 40 separ gener rubi program static analysi 40 syntax enhanc librari chang rubi 3.0 well 40 40 rubi 3.1 releas decemb 25 2021 42 includ yjit new experiment just-in-tim compil develop shopifi enhanc perform real world busi applic new debugg also includ syntax enhanc improv releas network librari ftp smtp imap pop move default gem bundl gem 43 42 shopifi debugg ftp smtp imap pop 43 rubi 3.2 releas decemb 25 2022 44 bring support run insid webassembl environ via wasi interfac regular express also receiv improv includ faster memoiz match algorithm protect certain redo attack configur timeout regular express match addit debug syntax featur also includ releas includ syntax suggest well error highlight mjit compil re-impl standard librari modul yjit rust-bas jit compil support architectur linux 44 webassembl regular express memoiz redo rust jit rubi 3.3 releas decemb 25 2023 1 rubi 3.3 introduc signific enhanc perform improv languag key featur includ introduct prism parser portabl maintain par addit pure-rubi jit compil rjit major perform boost yjit compil addit improv memori usag introduct n thread schedul updat standard librari contribut effici developer-friendli rubi ecosystem 1 semant philosophi edit edit edit yukihiro matsumoto matsumoto said rubi design programm product fun follow principl good user interfac design 45 googl tech talk 2008 said `` hope see rubi help everi programm world product enjoy program happi primari purpos rubi languag `` 46 stress system design need emphas human rather comput need 47 user interfac 45 46 47 often peopl especi comput engin focu machin think `` machin run fast machin run effect machin someth someth someth '' focus machin fact need focu human human care program oper applic machin master slave matsumoto said primari design goal make languag enjoy use minim programm work possibl confus said appli principl least astonish pola design rubi 47 may 2005 discus newsgroup comp.lang.rubi matsumoto attempt distanc rubi pola explain design choic surpris someon use person standard evalu surpris person standard remain consist would surpris familiar standard 48 principl least astonish 47 48 matsumoto defin way interview 47 47 everyon individu background someon may come python someon el may come perl may surpris differ aspect languag come say surpris featur languag rubi violat principl least surpris wait wait principl least surpris principl least surpris mean principl least surpris mean principl least surpris learn rubi well exampl c++ programm start design rubi program c++ exclus two three year two year c++ program still surpris rubi object-ori everi valu object includ class instanc type mani languag design primit integ boolean `` null '' everyth rubi object everyth rubi certain built-in abil call method everi function method method alway call object method defin top level scope becom method object class sinc class ancestor everi class method call object also visibl scope effect serv `` global '' procedur rubi support inherit dynam dispatch mixin singleton method belong defin singl instanc rather defin class though rubi support multipl inherit class import modul mixin object-ori integ null function method inherit dynam dispatch mixin instanc multipl inherit modul rubi describ multi-paradigm program languag allow procedur program defin functions/vari outsid class make part root 'self object object orient everyth object function program anonym function closur continu statement valu function return last evalu support introspect reflect program metaprogram interpreter-bas thread rubi featur dynam type support parametr polymorph multi-paradigm program languag function program anonym function closur continu introspect reflect program metaprogram thread dynam type parametr polymorph accord rubi faq syntax similar perl 's semant similar smalltalk 's design philosophi differ greatli python 's 49 perl smalltalk 's python 49 featur edit edit edit thoroughli object-ori inherit mixin metaclass 50 object-ori inherit mixin metaclass 50 dynam type duck type dynam type duck type everyth express even statement everyth execut imper even declar express statement imper declar succinct flexibl syntax 51 minim syntact nois serv foundat domain-specif languag 52 51 syntact nois domain-specif languag 52 dynam reflect alter object facilit metaprogram 53 reflect alter metaprogram 53 lexic closur iter gener block syntax 54 lexic closur iter gener block syntax 54 liter notat array hash regular express symbol array hash regular express symbol embed code string interpol interpol default argument default argument four level variabl scope global class instanc local denot sigil lack thereof global class instanc local sigil garbag collect garbag collect first-class continu first-class continu strict boolean coercion rule everyth true except fals nil coercion nil except handl except handl oper overload 55 oper overload 55 built-in support ration number complex number arbitrary-precis arithmet ration number complex number arbitrary-precis arithmet custom dispatch behavior method_miss const_miss nativ thread cooper fiber fiber 1.9/yarv featur thread fiber yarv support unicod multipl charact encod unicod charact encod nativ plug-in api c plug-in c interact rubi shell interact command-lin interpret use test code quickli repl repl central packag manag rubygem rubygem implement major platform larg standard librari includ modul yaml json xml cgi openssl http ftp rss cur zlib tk 56 yaml json xml cgi openssl http ftp rss cur zlib tk 56 just-in-tim compil just-in-tim compil syntax edit edit edit rubi syntax syntax rubi broadli similar perl python class method definit signal keyword wherea code block defin either keyword brace contrast perl variabl obligatorili prefix sigil use sigil chang semant scope variabl practic purpos distinct express statement 57 58 line break signific taken end statement semicolon may equival use unlik python indent signific perl python sigil express statement 57 58 one differ python perl rubi keep instanc variabl complet privat class expo accessor method attr_writ attr_read etc. unlik `` getter '' `` setter '' method languag like c++ java accessor method rubi creat singl line code via metaprogram howev accessor method also creat tradit fashion c++ java invoc method requir use parenthes trivial chang instanc variabl full function without modifi singl line call code refactor achiev similar function c vb.net properti member c++ java metaprogram c vb.net python 's properti descriptor similar come trade-off develop process one begin python use publicli expo instanc variabl later chang implement use privat instanc variabl expo properti descriptor code intern class may need adjust use privat variabl rather public properti rubi 's design forc instanc variabl privat also provid simpl way declar set get method keep idea rubi one never directli access intern member class outsid class rather one pas messag class receiv respons implement edit edit edit rubi mri § oper system list rubi compil matz 's rubi interpret edit edit edit origin rubi interpret often refer matz 's rubi interpret mri implement written c use ruby-specif virtual machin interpret matz 's rubi interpret virtual machin standard retir rubi 1.8 implement written c single-pass interpret languag 21 implement c interpret languag 21 start rubi 1.9 continu rubi 2.x offici rubi interpret yarv `` yet anoth rubi vm '' implement supersed slower virtual machin use previou releas mri yarv altern implement edit edit edit 2018 updat number altern implement rubi includ jrubi rubiniu mrubi take differ approach jrubi rubiniu provid just-in-tim compil mrubi also provid ahead-of-tim compil updat jrubi rubiniu mrubi just-in-tim compil ahead-of-tim compil rubi three major altern implement jrubi mix java rubi implement run java virtual machin jrubi current target rubi 3.1.x jrubi java java virtual machin trufflerubi java implement use truffl languag implement framework graalvm trufflerubi graalvm rubiniu c++ bytecod virtual machin use llvm compil machin code runtim bytecod compil core class written pure rubi rubiniu current target rubi 2.3.1 rubiniu c++ llvm rubi implement includ maglev smalltalk implement run gemtalk system gemstone/ vm maglev smalltalk gemtalk system gemstone/ mrubi implement design embed c code similar vein lua current develop yukihiro matsumoto other mrubi lua yukihiro matsumoto rgss rubi game script system proprietari implement use rpg maker seri softwar game design modif rpg maker engin rgss proprietari rpg maker julial transpil partial rubi julia use larg speedup e.g rubi jrubi implement may use numer code 59 julial transpil julia 59 topaz rubi implement written python topaz python opal web-bas interpret compil rubi javascript opal javascript defunct rubi implement macrubi mac o x implement objective-c runtim io counterpart call rubymot macrubi mac o x objective-c rubymot ironrubi implement .net framework ironrubi .net framework cardin implement parrot virtual machin parrot virtual machin rubi enterpris edit often shorten ree implement optim handl large-scal rubi rail project rubi enterpris edit rubi rail hotrubi javascript actionscript implement rubi program languag hotrubi javascript actionscript rubi program languag matur rubi implement tend measur abil run rubi rail rail framework complex implement use mani ruby-specif featur point particular implement achiev goal call `` rail singular '' refer implement jrubi rubiniu 60 abl run rail unmodifi product environ rubi rail 60 platform support edit edit edit matsumoto origin develop rubi 4.3bsd-base soni news-o 3.x later migrat work suno 4.x final linux 61 62 1999 rubi known work across mani differ oper system modern rubi version implement avail major desktop mobil server-bas oper system rubi also support across number cloud host platform like jelast heroku googl cloud platform other 4.3bsd soni news-o suno linux 61 62 oper system jelast heroku googl cloud platform tool rvm rbenv facilit instal partit multipl rubi version multipl 'gemset one machin rvm rbenv repositori librari edit edit edit rubygem rubi 's packag manag rubi packag call `` gem '' instal via command line gem librari though exist applic ide 63 100,000 rubi gem host rubygems.org 64 rubygem ide 63 rubygems.org 64 mani new exist rubi librari host github servic offer version control repositori host git github version control git rubi applic archiv host applic document librari rubi program maintain 2013 function transfer rubygem 65 65 see also edit edit edit free open-sourc softwar portal free open-sourc softwar portal free open-sourc softwar portal comput program portal comput program portal comput program portal comparison program languag comparison program languag metasploit metasploit 's poignant guid rubi 's poignant guid rubi crystal program languag crystal program languag rubi rail rubi rail refer edit edit edit b `` rubi 3.3.0 releas '' archiv origin 2023-12-25 retriev 2023-12-25 b b `` rubi 3.3.0 releas '' archiv origin 2023-12-25 retriev 2023-12-25 `` rubi 3.3.0 releas '' archiv retriev 2023-12-25 2023-12-25 b c cooper peter 2009 begin rubi novic profession begin novic profession 2nd ed. berkeley apress p. 101 isbn 978-1-4302-2363-4 lesser extent python lisp eiffel ada c++ also influenc rubi b c b c cooper peter 2009 begin rubi novic profession begin novic profession 2nd ed. berkeley apress p. 101 isbn 978-1-4302-2363-4 lesser extent python lisp eiffel ada c++ also influenc rubi isbn 978-1-4302-2363-4 b `` reason behind rubi '' rubi confer 2008 confreak youtub 15 april 2013 b b `` reason behind rubi '' rubi confer 2008 confreak youtub 15 april 2013 `` reason behind rubi '' b c e f bini ola 2007 practic jrubi rail web 2.0 project bring rubi rail java berkeley apress p. 3 isbn 978-1-59059-881-8 draw primarili featur perl smalltalk python lisp dylan clu b c e f b c e f bini ola 2007 practic jrubi rail web 2.0 project bring rubi rail java berkeley apress p. 3 isbn 978-1-59059-881-8 draw primarili featur perl smalltalk python lisp dylan clu practic jrubi rail web 2.0 project bring rubi rail java 3 isbn 978-1-59059-881-8 `` julia 1.0 document introduct '' archiv origin 16 august 2018 retriev 6 octob 2018 `` julia 1.0 document introduct '' archiv origin 16 august 2018 retriev 6 octob 2018 `` julia 1.0 document introduct '' origin retriev 6 octob 2018 6 octob burk tim `` nu™ '' program nu™ neon design technolog inc. archiv origin 2018-12-25 retriev 2011-07-21 burk tim `` nu™ '' program nu™ neon design technolog inc. archiv origin 2018-12-25 retriev 2011-07-21 `` nu™ '' archiv retriev 2011-07-21 2011-07-21 ring team 3 decemb 2017 `` ring languag '' ring-lang.net ring-lang archiv origin 25 decemb 2018 retriev 3 decemb 2017 ring team 3 decemb 2017 `` ring languag '' ring-lang.net ring-lang archiv origin 25 decemb 2018 retriev 3 decemb 2017 `` ring languag '' ring-lang archiv retriev 3 decemb 2017 3 decemb `` influenc rust refer '' rust refer archiv origin 2019-01-26 retriev 2023-04-18 `` influenc rust refer '' rust refer archiv origin 2019-01-26 retriev 2023-04-18 `` influenc rust refer '' archiv retriev 2023-04-18 2023-04-18 lattner chri 2014-06-03 `` chri lattner 's homepag '' chri lattner archiv origin 2018-12-25 retriev 2014-06-03 swift languag product tireless effort team languag expert document guru compil optim ninja incred import intern dogfood group provid feedback help refin battle-test idea cours also greatli benefit experi hard-won mani languag field draw idea objective-c rust haskel rubi python c clu far mani other list lattner chri 2014-06-03 `` chri lattner 's homepag '' chri lattner archiv origin 2018-12-25 retriev 2014-06-03 swift languag product tireless effort team languag expert document guru compil optim ninja incred import intern dogfood group provid feedback help refin battle-test idea cours also greatli benefit experi hard-won mani languag field draw idea objective-c rust haskel rubi python c clu far mani other list `` chri lattner 's homepag '' archiv retriev 2014-06-03 2014-06-03 `` rubi '' archiv origin 9 octob 2014 retriev 15 februari 2020 `` rubi '' archiv origin 9 octob 2014 retriev 15 februari 2020 `` rubi '' archiv retriev 15 februari 2020 15 februari matsumoto yukihiro 13 februari 2006 `` rubi 's lisp featur '' archiv origin 2018-10-27 retriev 15 februari 2020 matsumoto yukihiro 13 februari 2006 `` rubi 's lisp featur '' archiv origin 2018-10-27 retriev 15 februari 2020 matsumoto yukihiro `` rubi 's lisp featur '' origin retriev 15 februari 2020 15 februari b c e f `` histori rubi '' archiv origin 2011-07-14 retriev 2008-08-14 b c e f b c e f `` histori rubi '' archiv origin 2011-07-14 retriev 2008-08-14 `` histori rubi '' archiv retriev 2008-08-14 2008-08-14 `` fyi histor decis moment languag name rubi ann rubi 1.8.1 '' e-mail hiroshi sugihara ruby-talk archiv origin 2011-07-17 retriev 2008-08-14 `` fyi histor decis moment languag name rubi ann rubi 1.8.1 '' e-mail hiroshi sugihara ruby-talk archiv origin 2011-07-17 retriev 2008-08-14 `` fyi histor decis moment languag name rubi ann rubi 1.8.1 '' origin retriev 2008-08-14 2008-08-14 `` 1.3 name 'rubi '' rubi languag faq ruby-doc.org archiv origin may 28 2012 retriev april 10 2012 `` 1.3 name 'rubi '' rubi languag faq ruby-doc.org archiv origin may 28 2012 retriev april 10 2012 `` 1.3 name 'rubi '' archiv retriev april 10 2012 april 10 yukihiro matsumoto june 11 1999 `` name rubi '' ruby-talk mail list archiv origin decemb 25 2018 retriev april 10 2012 yukihiro matsumoto june 11 1999 `` name rubi '' ruby-talk mail list archiv origin decemb 25 2018 retriev april 10 2012 yukihiro matsumoto `` name rubi '' origin retriev april 10 2012 april 10 `` archeolinguist unearth proto-rubi '' archiv origin 6 novemb 2015 retriev 2 may 2015 `` archeolinguist unearth proto-rubi '' archiv origin 6 novemb 2015 retriev 2 may 2015 `` archeolinguist unearth proto-rubi '' origin retriev 2 may 2015 2 may `` ruby-talk:00382 histori rubi '' archiv origin 16 juli 2011 retriev 2 may 2015 `` ruby-talk:00382 histori rubi '' archiv origin 16 juli 2011 retriev 2 may 2015 `` ruby-talk:00382 histori rubi '' origin retriev 2 may 2015 2 may `` ruby-list:124 tutori — rubi 's featur '' archiv origin 24 may 2003 retriev 2 may 2015 `` ruby-list:124 tutori — rubi 's featur '' archiv origin 24 may 2003 retriev 2 may 2015 `` ruby-list:124 tutori — rubi 's featur '' origin retriev 2 may 2015 2 may `` interview creator rubi '' archiv origin 2008-02-08 retriev 2007-07-11 `` interview creator rubi '' archiv origin 2008-02-08 retriev 2007-07-11 `` interview creator rubi '' archiv retriev 2007-07-11 2007-07-11 yukihiro matsumoto octob 2000 `` program rubi forward '' archiv origin 25 decemb 2018 retriev 5 march 2014 yukihiro matsumoto octob 2000 `` program rubi forward '' archiv origin 25 decemb 2018 retriev 5 march 2014 `` program rubi forward '' archiv retriev 5 march 2014 5 march b `` retir rubi 1.8.7 '' archiv origin 6 june 2015 retriev 2 may 2015 b b `` retir rubi 1.8.7 '' archiv origin 6 june 2015 retriev 2 may 2015 `` retir rubi 1.8.7 '' archiv retriev 2 may 2015 2 may `` ipa 独立行政法人 情報処理推進機構：プレス発表 プログラム言語rubyのjis規格（ji x 3017）制定について '' archiv origin 2 februari 2015 retriev 2 may 2015 `` ipa 独立行政法人 情報処理推進機構：プレス発表 プログラム言語rubyのjis規格（ji x 3017）制定について '' archiv origin 2 februari 2015 retriev 2 may 2015 `` ipa 独立行政法人 情報処理推進機構：プレス発表 プログラム言語rubyのjis規格（ji x 3017）制定について '' origin retriev 2 may 2015 2 may `` ipa 独立行政法人 情報処理推進機構：プレス発表 プログラム言語ruby、国際規格として承認 '' archiv origin 1 februari 2015 retriev 2 may 2015 `` ipa 独立行政法人 情報処理推進機構：プレス発表 プログラム言語ruby、国際規格として承認 '' archiv origin 1 februari 2015 retriev 2 may 2015 `` ipa 独立行政法人 情報処理推進機構：プレス発表 プログラム言語ruby、国際規格として承認 '' origin retriev 2 may 2015 2 may `` iso/iec 30170:2012 '' archiv origin 2017-03-12 retriev 2017-03-10 `` iso/iec 30170:2012 '' archiv origin 2017-03-12 retriev 2017-03-10 `` iso/iec 30170:2012 '' archiv retriev 2017-03-10 2017-03-10 web develop rubi rail archiv 2009-02-24 wayback machin devarticles.com 2007-03-22 retriev 2013-07-17 web develop rubi rail archiv 2009-02-24 wayback machin devarticles.com 2007-03-22 retriev 2013-07-17 web develop rubi rail archiv wayback machin `` rubi 1.9.3 p0 releas '' ruby-lang.org octob 31 2011 archiv origin januari 14 2013 retriev februari 20 2013 `` rubi 1.9.3 p0 releas '' ruby-lang.org octob 31 2011 archiv origin januari 14 2013 retriev februari 20 2013 `` rubi 1.9.3 p0 releas '' archiv retriev februari 20 2013 februari 20 `` v1_9_3_0/new '' rubi subvers sourc repositori ruby-lang.org septemb 17 2011 archiv origin novemb 6 2015 retriev februari 20 2013 `` v1_9_3_0/new '' rubi subvers sourc repositori ruby-lang.org septemb 17 2011 archiv origin novemb 6 2015 retriev februari 20 2013 `` v1_9_3_0/new '' subvers origin retriev februari 20 2013 februari 20 rubi 1.9 expect archiv 2016-03-04 wayback machin slideshow.rubyforge.org retriev 2013-07-17 rubi 1.9 expect archiv 2016-03-04 wayback machin slideshow.rubyforge.org retriev 2013-07-17 rubi 1.9 expect archiv wayback machin endoh yusuk 2013-02-24 rubi 2.0.0-p0 releas archiv 2013-02-27 wayback machin ruby-lang.org retriev 2013-07-17 endoh yusuk 2013-02-24 rubi 2.0.0-p0 releas archiv 2013-02-27 wayback machin ruby-lang.org retriev 2013-07-17 rubi 2.0.0-p0 releas archiv wayback machin endoh yusuk 2013-02-24 rubi 2.0.0-p0 releas archiv 2016-01-17 wayback machin ruby-lang.org retriev 2013-07-17 endoh yusuk 2013-02-24 rubi 2.0.0-p0 releas archiv 2016-01-17 wayback machin ruby-lang.org retriev 2013-07-17 rubi 2.0.0-p0 releas archiv wayback machin `` semant version start rubi 2.1.0 '' decemb 21 2013 archiv origin februari 13 2014 retriev decemb 27 2013 `` semant version start rubi 2.1.0 '' decemb 21 2013 archiv origin februari 13 2014 retriev decemb 27 2013 `` semant version start rubi 2.1.0 '' origin retriev decemb 27 2013 decemb 27 gustavo frederico templ pedrosa vitor de lima leonardo bianconi 2015 `` rubi 2.2.1 releas '' archiv origin 16 may 2016 retriev 12 juli 2016 cite web cs1 maint multipl name author list link gustavo frederico templ pedrosa vitor de lima leonardo bianconi 2015 `` rubi 2.2.1 releas '' archiv origin 16 may 2016 retriev 12 juli 2016 cite web cs1 maint multipl name author list link `` rubi 2.2.1 releas '' archiv retriev 12 juli 2016 12 juli cite web cs1 maint multipl name author list link cite web link gustavo frederico templ pedrosa vitor de lima leonardo bianconi 2015 `` v2.2.1 changelog '' archiv origin 26 februari 2017 retriev 12 juli 2016 cite web cs1 maint multipl name author list link gustavo frederico templ pedrosa vitor de lima leonardo bianconi 2015 `` v2.2.1 changelog '' archiv origin 26 februari 2017 retriev 12 juli 2016 cite web cs1 maint multipl name author list link `` v2.2.1 changelog '' origin retriev 12 juli 2016 12 juli cite web cs1 maint multipl name author list link cite web link gustavo frederico templ pedrosa vitor de lima leonardo bianconi 2014 `` specifi non volatil regist increas perform ppc64 '' archiv origin 17 septemb 2016 retriev 12 juli 2016 cite web cs1 maint multipl name author list link gustavo frederico templ pedrosa vitor de lima leonardo bianconi 2014 `` specifi non volatil regist increas perform ppc64 '' archiv origin 17 septemb 2016 retriev 12 juli 2016 cite web cs1 maint multipl name author list link `` specifi non volatil regist increas perform ppc64 '' archiv retriev 12 juli 2016 12 juli cite web cs1 maint multipl name author list link cite web link gustavo frederico templ pedrosa vitor de lima leonardo bianconi 2014 `` specifi macro increas perform ppc64 '' archiv origin 17 septemb 2016 retriev 12 juli 2016 cite web cs1 maint multipl name author list link gustavo frederico templ pedrosa vitor de lima leonardo bianconi 2014 `` specifi macro increas perform ppc64 '' archiv origin 17 septemb 2016 retriev 12 juli 2016 cite web cs1 maint multipl name author list link `` specifi macro increas perform ppc64 '' archiv retriev 12 juli 2016 12 juli cite web cs1 maint multipl name author list link cite web link `` ruby/new v2_2_0 · ruby/rubi · github '' github archiv origin 1 januari 2015 retriev 2 may 2015 `` ruby/new v2_2_0 · ruby/rubi · github '' github archiv origin 1 januari 2015 retriev 2 may 2015 `` ruby/new v2_2_0 · ruby/rubi · github '' archiv retriev 2 may 2015 2 may `` ruby/new v.2_3_0 ruby/rubi '' github archiv origin 1 march 2017 retriev 25 decemb 2015 `` ruby/new v.2_3_0 ruby/rubi '' github archiv origin 1 march 2017 retriev 25 decemb 2015 `` ruby/new v.2_3_0 ruby/rubi '' archiv retriev 25 decemb 2015 25 decemb `` rubi 2.3.0 chang featur '' run rubi dev.mensfeld.pl 14 novemb 2015 archiv origin 5 januari 2016 retriev 27 decemb 2015 `` rubi 2.3.0 chang featur '' run rubi dev.mensfeld.pl 14 novemb 2015 archiv origin 5 januari 2016 retriev 27 decemb 2015 `` rubi 2.3.0 chang featur '' archiv retriev 27 decemb 2015 27 decemb `` rubi 2.4.0 releas '' www.ruby-lang.org archiv origin 2017-02-17 retriev 2016-12-30 `` rubi 2.4.0 releas '' www.ruby-lang.org archiv origin 2017-02-17 retriev 2016-12-30 `` rubi 2.4.0 releas '' archiv retriev 2016-12-30 2016-12-30 b c `` rubi 3.0.0 releas '' rubi program languag 2020-12-25 archiv origin 2020-12-25 retriev 2020-12-25 b c b c `` rubi 3.0.0 releas '' rubi program languag 2020-12-25 archiv origin 2020-12-25 retriev 2020-12-25 `` rubi 3.0.0 releas '' archiv retriev 2020-12-25 2020-12-25 scheffler jonan novemb 10 2016 `` rubi 3x3 matz koichi tenderlov futur rubi perform '' rubi archiv origin may 10 2019 retriev may 18 2019 scheffler jonan novemb 10 2016 `` rubi 3x3 matz koichi tenderlov futur rubi perform '' rubi archiv origin may 10 2019 retriev may 18 2019 `` rubi 3x3 matz koichi tenderlov futur rubi perform '' archiv retriev may 18 2019 may 18 `` rubi 3.1.0 releas '' ruby-lang.org archiv origin 25 decemb 2021 retriev 25 dec 2021 `` rubi 3.1.0 releas '' ruby-lang.org archiv origin 25 decemb 2021 retriev 25 dec 2021 `` rubi 3.1.0 releas '' archiv retriev 25 dec 2021 25 dec `` rubi 3.1.0 releas '' archiv origin 2021-12-26 retriev 2021-12-26 `` rubi 3.1.0 releas '' archiv origin 2021-12-26 retriev 2021-12-26 `` rubi 3.1.0 releas '' archiv retriev 2021-12-26 2021-12-26 `` rubi 3.2.0 releas '' archiv origin 2022-12-25 retriev 2022-12-25 `` rubi 3.2.0 releas '' archiv origin 2022-12-25 retriev 2022-12-25 `` rubi 3.2.0 releas '' archiv retriev 2022-12-25 2022-12-25 `` rubi program languag '' archiv origin 18 januari 2020 retriev 2 may 2015 `` rubi program languag '' archiv origin 18 januari 2020 retriev 2 may 2015 `` rubi program languag '' archiv retriev 2 may 2015 2 may googl tech talk – rubi 1.9 youtub googl tech talk – rubi 1.9 youtub googl tech talk – rubi 1.9 googl tech talk – rubi 1.9 youtub b c bill venner `` philosophi rubi '' archiv origin 5 juli 2019 retriev 2 may 2015 b c b c bill venner `` philosophi rubi '' archiv origin 5 juli 2019 retriev 2 may 2015 `` philosophi rubi '' archiv retriev 2 may 2015 2 may `` welcom rubyweeklynews.org '' 4 juli 2017 archiv origin 4 juli 2017 cite web cs1 maint bot origin url statu unknown link `` welcom rubyweeklynews.org '' 4 juli 2017 archiv origin 4 juli 2017 cite web cs1 maint bot origin url statu unknown link `` welcom rubyweeklynews.org '' cite web cs1 maint bot origin url statu unknown link cite web link `` rubi languag faq rubi stack ... '' archiv origin 8 may 2015 retriev 2 may 2015 `` rubi languag faq rubi stack ... '' archiv origin 8 may 2015 retriev 2 may 2015 `` rubi languag faq rubi stack ... '' archiv retriev 2 may 2015 2 may bruce stewart 29 novemb 2001 `` interview creator rubi '' o'reilli medium archiv origin 6 may 2015 retriev 2 may 2015 bruce stewart 29 novemb 2001 `` interview creator rubi '' o'reilli medium archiv origin 6 may 2015 retriev 2 may 2015 `` interview creator rubi '' archiv retriev 2 may 2015 2 may bill venner `` dynam product rubi '' archiv origin 5 septemb 2015 retriev 2 may 2015 bill venner `` dynam product rubi '' archiv origin 5 septemb 2015 retriev 2 may 2015 `` dynam product rubi '' archiv retriev 2 may 2015 2 may `` languag workbench killer-app domain specif languag '' martinfowler.com archiv origin 2 may 2021 retriev 2 may 2015 `` languag workbench killer-app domain specif languag '' martinfowler.com archiv origin 2 may 2021 retriev 2 may 2015 `` languag workbench killer-app domain specif languag '' archiv retriev 2 may 2015 2 may `` rubi – add class method runtim '' archiv origin 2007-09-22 retriev 2007-11-01 `` rubi – add class method runtim '' archiv origin 2007-09-22 retriev 2007-11-01 `` rubi – add class method runtim '' archiv retriev 2007-11-01 2007-11-01 bill venner `` block closur rubi '' archiv origin 18 april 2015 retriev 2 may 2015 bill venner `` block closur rubi '' archiv origin 18 april 2015 retriev 2 may 2015 `` block closur rubi '' archiv retriev 2 may 2015 2 may `` method '' offici rubi faq archiv origin 2022-06-28 retriev 2021-06-20 `` method '' offici rubi faq archiv origin 2022-06-28 retriev 2021-06-20 `` method '' archiv retriev 2021-06-20 2021-06-20 britt jame `` rubi 2.0.0 standard librari document '' archiv origin 2013-11-13 retriev 2013-12-09 britt jame `` rubi 2.0.0 standard librari document '' archiv origin 2013-11-13 retriev 2013-12-09 `` rubi 2.0.0 standard librari document '' archiv retriev 2013-12-09 2013-12-09 `` ruby-talk:01120 valu ... '' archiv origin 2011-07-17 retriev 2008-12-06 rubi 's syntax statement special case express appear argument e.g multipl assign `` ruby-talk:01120 valu ... '' archiv origin 2011-07-17 retriev 2008-12-06 rubi 's syntax statement special case express appear argument e.g multipl assign `` ruby-talk:01120 valu ... '' origin retriev 2008-12-06 2008-12-06 `` ruby-talk:02460 preced question '' archiv origin 2004-07-22 retriev 2008-12-06. statement ... part express unless group within parenthes `` ruby-talk:02460 preced question '' archiv origin 2004-07-22 retriev 2008-12-06. statement ... part express unless group within parenthes `` ruby-talk:02460 preced question '' origin retriev 2008-12-06 2008-12-06 `` remove/virtual_modul born make rubi code 3x faster hope '' github 21 februari 2020 archiv origin 1 march 2017 retriev 29 august 2016 `` remove/virtual_modul born make rubi code 3x faster hope '' github 21 februari 2020 archiv origin 1 march 2017 retriev 29 august 2016 `` remove/virtual_modul born make rubi code 3x faster hope '' github archiv retriev 29 august 2016 29 august peter cooper 2010-05-18 `` rubiniu 1.0 's releas '' archiv origin 2010-05-24 retriev 2010-05-23 peter cooper 2010-05-18 `` rubiniu 1.0 's releas '' archiv origin 2010-05-24 retriev 2010-05-23 `` rubiniu 1.0 's releas '' archiv retriev 2010-05-23 2010-05-23 maya stodt februari 2000 `` ibm developerwork – rubi new languag '' archiv origin august 18 2000 retriev 3 march 2014 maya stodt februari 2000 `` ibm developerwork – rubi new languag '' archiv origin august 18 2000 retriev 3 march 2014 `` ibm developerwork – rubi new languag '' origin retriev 3 march 2014 3 march yukihiro matsumoto august 2002 `` lang-ruby-gener question rubi initi develop '' archiv origin 3 march 2014 retriev 3 march 2014 yukihiro matsumoto august 2002 `` lang-ruby-gener question rubi initi develop '' archiv origin 3 march 2014 retriev 3 march 2014 `` lang-ruby-gener question rubi initi develop '' archiv retriev 3 march 2014 3 march `` rubi toolbox '' archiv origin 2015-04-03 retriev 2015-04-04 `` rubi toolbox '' archiv origin 2015-04-03 retriev 2015-04-04 `` rubi toolbox '' archiv retriev 2015-04-04 2015-04-04 `` stat rubygems.org commun gem host '' rubygems.org archiv origin 10 decemb 2021 retriev 10 decemb 2021 `` stat rubygems.org commun gem host '' rubygems.org archiv origin 10 decemb 2021 retriev 10 decemb 2021 `` stat rubygems.org commun gem host '' archiv retriev 10 decemb 2021 10 decemb `` retir raa.ruby-lang.org '' 2013-08-08 archiv origin 2015-12-31 retriev 2016-01-03 `` retir raa.ruby-lang.org '' 2013-08-08 archiv origin 2015-12-31 retriev 2016-01-03 `` retir raa.ruby-lang.org '' archiv retriev 2016-01-03 2016-01-03 read edit edit edit black david leo joseph march 15 2019 well-ground rubyist third ed man public p. 584 isbn 978-1617295218 man public isbn 978-1617295218 metz sandi august 22 2018 practic object-ori design agil primer use rubi second ed addison-wesley profession p. 288 isbn 978-0-13-445647-8 archiv origin februari 13 2020 retriev februari 13 2020 practic object-ori design agil primer use rubi addison-wesley profession isbn 978-0-13-445647-8 archiv retriev februari 13 2020 februari 13 cooper peter juli 12 2016 begin rubi novic profession third ed apress p. 492 isbn 978-1484212790 apress isbn 978-1484212790 carlson luca richardson leonard april 3 2015 rubi cookbook recip object-ori script second ed o'reilli medium p. 963 isbn 978-1449373719 rubi cookbook recip object-ori script o'reilli medium isbn 978-1449373719 fulton hal arko andré march 2 2015 rubi way solut techniqu rubi program third ed addison-wesley profession p. 816 isbn 978-0-321-71463-3 archiv origin februari 13 2020 retriev februari 13 2020 rubi way solut techniqu rubi program addison-wesley profession isbn 978-0-321-71463-3 archiv retriev februari 13 2020 februari 13 thoma dave fowler chad hunt andi juli 7 2013 program rubi 1.9 2.0 pragmat programm guid fourth ed pragmat bookshelf p. 888 isbn 978-1937785499 pragmat bookshelf isbn 978-1937785499 mcanal jeremi arkin assaf march 28 2009 rubi practic first ed man public p. 360 isbn 978-1933988474 man public isbn 978-1933988474 flanagan david matsumoto yukihiro januari 25 2008 rubi program languag first ed o'reilli medium p. 446 isbn 978-0-596-51617-8 rubi program languag o'reilli medium 446 isbn 978-0-596-51617-8 baird kevin june 8 2007 rubi exampl concept code first ed starch press p. 326 isbn 978-1593271480 archiv origin januari 13 2020 retriev februari 13 2020 rubi exampl concept code starch press isbn 978-1593271480 archiv retriev februari 13 2020 februari 13 fitzgerald michael may 14 2007 learn rubi first ed o'reilli medium p. 255 isbn 978-0-596-52986-4 learn rubi o'reilli medium 255 isbn 978-0-596-52986-4 extern link edit edit edit rubi program languag rubi program languag rubi program languag topic rubi offici websit offici websit offici websit offici websit rubi document rubi document v v e e rubi program languag implement rubi mri yarv rubi mri yarv mrubi mrubi jrubi jrubi rubymot rubymot macrubi macrubi ironrubi ironrubi xrubi hotrubi hotrubi maglev maglev rubiniu rubiniu ide aptana studio aptana studio komodo ide komodo ide rubymin rubymin applic capistrano capistrano hacketi hack hacketi hack interact rubi shell interact rubi shell pri pri rake rake redmin redmin rspec rspec rubygem rubygem rvm rvm chef chef puppet puppet vagrant vagrant homebrew homebrew cocoapod cocoapod librari framework adhears adhears erubi rhtml erubi merb merb padrino padrino rubycocoa rubycocoa rubi rail rubi rail shoe shoe sinatra sinatra qtrubi qtrubi server mod_rubi mod_rubi mongrel mongrel phusion passeng mod_rails/mod_rack phusion passeng rack rack webrick webrick yukihiro matsumoto yukihiro matsumoto david heinemei hansson david heinemei hansson lucki stiff lucki stiff dave thoma dave thoma document format document format 's poignant guid rubi 's poignant guid rubi rubi central rubi central rubykaigi rubykaigi rubyforg rubyforg rubi licens rubi licens categori categori common common v v e e program languag comparison comparison timelin timelin histori histori ada ada algol simula algol simula simula apl apl assembl assembl basic visual basic classic .net basic visual basic classic .net visual basic classic classic .net .net c c c++ c++ c c cobol cobol erlang erlang forth forth fortran fortran go go haskel haskel java java javascript javascript julia julia kotlin kotlin lisp lisp lua lua matlab matlab ml ml pascal object pascal pascal object pascal object pascal perl perl php php prolog prolog python python r r rubi rubi rust rust sql sql scratch scratch shell shell smalltalk smalltalk swift swift ... ... list alphabet alphabet categor categor gener gener non-english-bas non-english-bas categori categori v v e e free open-sourc softwar altern term free softwar altern term free softwar comparison open-sourc closed-sourc softwar comparison open-sourc closed-sourc softwar comparison source-code-host facil comparison source-code-host facil free softwar free softwar free softwar project directori free softwar project directori grati versu libr grati versu libr long-term support long-term support open-sourc softwar open-sourc softwar open-sourc softwar develop open-sourc softwar develop outlin outlin timelin timelin softwarepackag audio audio bioinformat bioinformat codec codec configur manag configur manag driver graphic wireless driver graphic graphic wireless wireless health health mathemat mathemat offic suit offic suit oper system oper system rout rout televis televis video game video game web applic e-commerc web applic e-commerc e-commerc android app android app io app io app commerci commerci formerli proprietari formerli proprietari formerli open-sourc formerli open-sourc commun free softwar movement free softwar movement histori histori open-source-softwar movement open-source-softwar movement event event advocaci advocaci organis free softwar movement india free softwar movement india free softwar foundat free softwar foundat licens afl afl apach apach apsl apsl artist artist beerwar beerwar bsd bsd creativ common creativ common cddl cddl epl epl free softwar foundat gnu gpl gnu agpl gnu lgpl free softwar foundat gnu gpl gnu gpl gnu agpl gnu agpl gnu lgpl gnu lgpl isc isc mit mit mpl mpl python python python softwar foundat licens python softwar foundat licens share sourc initi share sourc initi sleepycat sleepycat unlicens unlicens wtfpl wtfpl zlib zlib comparison licens comparison licens contributor licens agreement contributor licens agreement copyleft copyleft debian free softwar guidelin debian free softwar guidelin definit free cultur work definit free cultur work free licens free licens free softwar definit free softwar definit open sourc definit open sourc definit open-sourc licens open-sourc licens permiss softwar licens permiss softwar licens public domain public domain digit right manag digit right manag licens prolifer licens prolifer mozilla softwar rebrand mozilla softwar rebrand proprietari devic driver proprietari devic driver proprietari firmwar proprietari firmwar proprietari softwar proprietari softwar sco/linux controversi sco/linux controversi softwar patent softwar patent softwar secur softwar secur tivoiz tivoiz trust comput trust comput fork fork gnu manifesto gnu manifesto microsoft open specif promis microsoft open specif promis open-cor model open-cor model open-sourc hardwar open-sourc hardwar share sourc initi share sourc initi source-avail softwar source-avail softwar cathedr bazaar cathedr bazaar revolut o revolut o portal portal categori categori v v e e iso standard number iso standard iso roman iec standard 1 1 2 2 3 3 4 4 6 6 7 7 9 9 16 16 17 17 31 -0 -1 -3 -4 -5 -6 -7 -8 -9 -10 -11 -12 -13 31 -0 -0 -1 -1 -3 -3 -4 -4 -5 -5 -6 -6 -7 -7 -8 -8 -9 -9 -10 -10 -11 -11 -12 -12 -13 -13 68-1 68-1 128 128 216 216 217 217 226 226 228 228 233 233 259 259 261 261 262 262 302 302 306 306 361 361 500 500 518 518 519 519 639 -1 -2 -3 -5 -6 639 -1 -1 -2 -2 -3 -3 -5 -5 -6 -6 646 646 657 657 668 668 690 690 704 704 732 732 764 764 838 838 843 843 860 860 898 898 965 965 999 999 1000 1000 1004 1004 1007 1007 1073-1 1073-1 1073-2 1073-2 1155 1155 1413 1413 1538 1538 1629 1629 1745 1745 1989 1989 2014 2014 2015 2015 2022 2022 2033 2033 2047 2047 2108 2108 2145 2145 2146 2146 2240 2240 2281 2281 2533 2533 2709 2709 2711 2711 2720 2720 2788 2788 2848 2848 2852 2852 2921 2921 3029 3029 3103 3103 3166 -1 -2 -3 3166 -1 -1 -2 -2 -3 -3 3297 3297 3307 3307 3601 3601 3602 3602 3864 3864 3901 3901 3950 3950 3977 3977 4031 4031 4157 4157 4165 4165 4217 4217 4909 4909 5218 5218 5426 5426 5427 5427 5428 5428 5725 5725 5775 5775 5776 5776 5800 5800 5807 5807 5964 5964 6166 6166 6344 6344 6346 6346 6373 6373 6385 6385 6425 6425 6429 6429 6438 6438 6523 6523 6709 6709 6943 6943 7001 7001 7002 7002 7010 7010 7027 7027 7064 7064 7098 7098 7185 7185 7200 7200 7498 -1 7498 -1 -1 7637 7637 7736 7736 7810 7810 7811 7811 7812 7812 7813 7813 7816 7816 7942 7942 8000 8000 8093 8093 8178 8178 8217 8217 8373 8373 8501-1 8501-1 8571 8571 8583 8583 8601 8601 8613 8613 8632 8632 8651 8651 8652 8652 8691 8691 8805/8806 8805/8806 8807 8807 8820-5 8820-5 8859 -1 -2 -3 -4 -5 -6 -7 -8 -8-i -9 -10 -11 -12 -13 -14 -15 -16 8859 -1 -1 -2 -2 -3 -3 -4 -4 -5 -5 -6 -6 -7 -7 -8 -8 -8-i -8-i -9 -9 -10 -10 -11 -11 -12 -12 -13 -13 -14 -14 -15 -15 -16 -16 8879 8879 9000/9001 9000/9001 9036 9036 9075 9075 9126 9126 9141 9141 9227 9227 9241 9241 9293 9293 9314 9314 9362 9362 9407 9407 9496 9496 9506 9506 9529 9529 9564 9564 9592/9593 9592/9593 9594 9594 9660 9660 9797-1 9797-1 9897 9897 9899 9899 9945 9945 9984 9984 9985 9985 9995 9995 10006 10006 10007 10007 10116 10116 10118-3 10118-3 10160 10160 10161 10161 10165 10165 10179 10179 10206 10206 10218 10218 10279 10279 10303 -11 -21 -22 -28 -238 10303 -11 -11 -21 -21 -22 -22 -28 -28 -238 -238 10383 10383 10585 10585 10589 10589 10628 10628 10646 10646 10664 10664 10746 10746 10861 10861 10957 10957 10962 10962 10967 10967 11073 11073 11170 11170 11172 11172 11179 11179 11404 11404 11544 11544 11783 11783 11784 11784 11785 11785 11801 11801 11889 11889 11898 11898 11940 -2 11940 -2 11941 11941 11941 tr 11941 tr 11992 11992 12006 12006 12052 12052 12182 12182 12207 12207 12234-2 12234-2 12620 12620 13211 -1 -2 13211 -1 -1 -2 -2 13216 13216 13250 13250 13399 13399 13406-2 13406-2 13450 13450 13485 13485 13490 13490 13567 13567 13568 13568 13584 13584 13616 13616 13816 13816 13818 13818 14000 14000 14031 14031 14224 14224 14289 14289 14396 14396 14443 14443 14496 -2 -3 -6 -10 -11 -12 -14 -17 -20 14496 -2 -2 -3 -3 -6 -6 -10 -10 -11 -11 -12 -12 -14 -14 -17 -17 -20 -20 14617 14617 14644 14644 14649 14649 14651 14651 14698 14698 14764 14764 14882 14882 14971 14971 15022 15022 15189 15189 15288 15288 15291 15291 15398 15398 15408 15408 15444 -3 -9 15444 -3 -3 -9 -9 15445 15445 15438 15438 15504 15504 15511 15511 15686 15686 15693 15693 15706 -2 15706 -2 -2 15707 15707 15897 15897 15919 15919 15924 15924 15926 15926 15926 wip 15926 wip 15930 15930 15938 15938 16023 16023 16262 16262 16355-1 16355-1 16485 16485 16612-2 16612-2 16750 16750 16949 t 16949 t 17024 17024 17025 17025 17100 17100 17203 17203 17369 17369 17442 17442 17506 17506 17799 17799 18004 18004 18014 18014 18181 18181 18245 18245 18629 18629 18916 18916 19005 19005 19011 19011 19092 -1 -2 19092 -1 -1 -2 -2 19114 19114 19115 19115 19125 19125 19136 19136 19407 19407 19439 19439 19500 19500 19501 19501 19502 19502 19503 19503 19505 19505 19506 19506 19507 19507 19508 19508 19509 19509 19510 19510 19600 19600 19752 19752 19757 19757 19770 19770 19775-1 19775-1 19794-5 19794-5 19831 19831 20000 20000 20022 20022 20121 20121 20400 20400 20802 20802 20830 20830 21000 21000 21001 21001 21047 21047 21122 21122 21500 21500 21827 21827 22000 22000 22275 22275 22300 22300 22301 22301 22395 22395 22537 22537 23000 23000 23003 23003 23008 23008 23009 23009 23090-3 23090-3 23092 23092 23094-1 23094-1 23094-2 23094-2 23270 23270 23271 23271 23360 23360 23941 23941 24517 24517 24613 24613 24617 24617 24707 24707 24728 24728 25178 25178 25964 25964 26000 26000 26262 26262 26300 26300 26324 26324 27000 seri 27000 seri 27000 27000 27001 27001 27002 27002 27005 27005 27006 27006 27729 27729 28000 28000 29110 29148 29148 29199-2 29199-2 29500 29500 30170 30170 31000 31000 32000 32000 37001 37001 38500 38500 39075 39075 40500 40500 42010 42010 45001 45001 50001 50001 55000 55000 56000 56000 80000 80000 categori categori author control databas germani germani germani unit state unit state unit state franc franc franc bnf data bnf data bnf data czech republ czech republ czech republ czech republ spain spain spain israel israel israel idref idref idref http //en.wikipedia.org/w/index.php title=ruby_ programming_languag oldid=1268195105 categori rubi program languag rubi program languag class-bas program languag class-bas program languag dynam program languag dynam program languag dynam type program languag dynam type program languag free open sourc interpret free open sourc interpret function languag function languag free softwar program c free softwar program c iso standard iso standard japanes invent japanes invent multi-paradigm program languag multi-paradigm program languag object-ori program languag object-ori program languag program languag creat 1995 program languag creat 1995 program languag iso standard program languag iso standard script languag script languag softwar use bsd licens softwar use bsd licens text-ori program languag text-ori program languag webarch templat wayback link webarch templat wayback link cs1 maint multipl name author list cs1 maint multipl name author list cs1 maint bot origin url statu unknown cs1 maint bot origin url statu unknown articl short descript articl short descript short descript match wikidata short descript match wikidata wikipedia articl technic june 2022 wikipedia articl technic june 2022 articl technic articl technic articl contain potenti date statement 2018 articl contain potenti date statement 2018 articl contain potenti date statement articl contain potenti date statement common categori link wikidata common categori link wikidata offici websit differ wikidata wikipedia offici websit differ wikidata wikipedia articl exampl rubi code articl exampl rubi code page last edit 8 januari 2025 15:36 utc utc text avail creativ common attribution-sharealik 4.0 licens addit term may appli use site agre term use privaci polici wikipedia® regist trademark wikimedia foundat inc. non-profit organ creativ common attribution-sharealik 4.0 licens term use privaci polici wikimedia foundat inc. privaci polici privaci polici wikipedia wikipedia disclaim disclaim contact wikipedia contact wikipedia code conduct code conduct develop develop statist statist cooki statement cooki statement mobil view mobil view",
    "https://en.wikipedia.org/wiki/HTML": "jump content main menu main page main page main page content content content current event current event current event random articl random articl random articl wikipedia wikipedia wikipedia contact u contact u contact u help help help learn edit learn edit learn edit commun portal commun portal commun portal recent chang recent chang recent chang upload file upload file upload file search search appear donat donat donat creat account creat account creat account log log log person tool donat donat donat creat account creat account creat account log log log learn learn contribut contribut contribut talk talk talk content top top 1 histori toggl histori subsect 1.1 develop 1.2 html version timelin 1.2.1 html 2 1.2.2 html 3 1.2.3 html 4 1.2.4 html 5 1.3 html draft version timelin 1.3.1 xhtml version 1.4 transit html public whatwg 1 histori 1 histori toggl histori subsect 1.1 develop 1.1 develop 1.1 develop 1.2 html version timelin 1.2.1 html 2 1.2.2 html 3 1.2.3 html 4 1.2.4 html 5 1.2 html version timelin 1.2 html version timelin 1.2.1 html 2 1.2.1 html 2 1.2.1 html 2 1.2.2 html 3 1.2.2 html 3 1.2.2 html 3 1.2.3 html 4 1.2.3 html 4 1.2.3 html 4 1.2.4 html 5 1.2.4 html 5 1.2.4 html 5 1.3 html draft version timelin 1.3.1 xhtml version 1.3 html draft version timelin 1.3 html draft version timelin 1.3.1 xhtml version 1.3.1 xhtml version 1.3.1 xhtml version 1.4 transit html public whatwg 1.4 transit html public whatwg 1.4 transit html public whatwg 2 markup toggl markup subsect 2.1 element 2.1.1 element exampl 2.1.1.1 head 2.1.1.2 line break 2.1.1.3 link 2.1.1.4 input 2.1.2 attribut 2.2 charact entiti refer 2.3 data type 2.4 document type declar 2 markup 2 markup toggl markup subsect 2.1 element 2.1.1 element exampl 2.1.1.1 head 2.1.1.2 line break 2.1.1.3 link 2.1.1.4 input 2.1.2 attribut 2.1 element 2.1 element 2.1.1 element exampl 2.1.1.1 head 2.1.1.2 line break 2.1.1.3 link 2.1.1.4 input 2.1.1 element exampl 2.1.1 element exampl 2.1.1.1 head 2.1.1.1 head 2.1.1.1 head 2.1.1.2 line break 2.1.1.2 line break 2.1.1.2 line break 2.1.1.3 link 2.1.1.3 link 2.1.1.3 link 2.1.1.4 input 2.1.1.4 input 2.1.1.4 input 2.1.2 attribut 2.1.2 attribut 2.1.2 attribut 2.2 charact entiti refer 2.2 charact entiti refer 2.2 charact entiti refer 2.3 data type 2.3 data type 2.3 data type 2.4 document type declar 2.4 document type declar 2.4 document type declar 3 semant html 3 semant html 3 semant html 4 deliveri toggl deliveri subsect 4.1 http 4.2 html e-mail 4.3 name convent 4.4 html applic 4 deliveri 4 deliveri toggl deliveri subsect 4.1 http 4.1 http 4.1 http 4.2 html e-mail 4.2 html e-mail 4.2 html e-mail 4.3 name convent 4.3 name convent 4.3 name convent 4.4 html applic 4.4 html applic 4.4 html applic 5 html4 variat toggl html4 variat subsect 5.1 sgml-base versu xml-base html 5.2 transit versu strict 5.3 frameset versu transit 5.4 summari specif version 5 html4 variat 5 html4 variat toggl html4 variat subsect 5.1 sgml-base versu xml-base html 5.1 sgml-base versu xml-base html 5.1 sgml-base versu xml-base html 5.2 transit versu strict 5.2 transit versu strict 5.2 transit versu strict 5.3 frameset versu transit 5.3 frameset versu transit 5.3 frameset versu transit 5.4 summari specif version 5.4 summari specif version 5.4 summari specif version 6 whatwg html versu html5 6 whatwg html versu html5 6 whatwg html versu html5 7 wysiwyg editor 7 wysiwyg editor 7 wysiwyg editor 8 see also 8 see also 8 see also 9 note 9 note 9 note 10 refer 10 refer 10 refer 11 extern link 11 extern link 11 extern link toggl tabl content html html 135 languag afrikaan afrikaan afrikaan alemannisch alemannisch alemannisch ænglisc ænglisc ænglisc العربية العربية العربية aragoné aragoné aragoné অসমীয়া অসমীয়া অসমীয়া asturianu asturianu asturianu azərbaycanca azərbaycanca azərbaycanca تۆرکجه تۆرکجه تۆرکجه basa bali basa bali basa bali বাংলা বাংলা বাংলা 閩南語 bân-lâm-gú 閩南語 bân-lâm-gú 閩南語 bân-lâm-gú башҡортса башҡортса башҡортса беларуская беларуская беларуская беларуская тарашкевіца беларуская тарашкевіца беларуская тарашкевіца български български български boarisch boarisch boarisch bosanski bosanski bosanski brezhoneg brezhoneg brezhoneg català català català чӑвашла чӑвашла чӑвашла čeština čeština čeština corsu corsu corsu cymraeg cymraeg cymraeg dansk dansk dansk davvisámegiella davvisámegiella davvisámegiella deutsch deutsch deutsch dolnoserbski dolnoserbski dolnoserbski eesti eesti eesti ελληνικά ελληνικά ελληνικά español español español esperanto esperanto esperanto euskara euskara euskara فارسی فارسی فارسی fiji hindi fiji hindi fiji hindi føroyskt føroyskt føroyskt françai françai françai frysk frysk frysk furlan furlan furlan gaeilg gaeilg gaeilg gàidhlig gàidhlig gàidhlig galego galego galego گیلکی گیلکی گیلکی ગુજરાતી ગુજરાતી ગુજરાતી 한국어 한국어 한국어 հայերեն հայերեն հայերեն हिन्दी हिन्दी हिन्दी hornjoserbsc hornjoserbsc hornjoserbsc hrvatski hrvatski hrvatski ido ido ido bahasa indonesia bahasa indonesia bahasa indonesia interlingua interlingua interlingua íslenska íslenska íslenska italiano italiano italiano עברית עברית עברית jawa jawa jawa ქართული ქართული ქართული қазақша қазақша қазақша kiswahili kiswahili kiswahili kurdî kurdî kurdî кыргызча кыргызча кыргызча latgaļu latgaļu latgaļu latina latina latina latviešu latviešu latviešu lëtzebuergesch lëtzebuergesch lëtzebuergesch lietuvių lietuvių lietuvių ligur ligur ligur lingua franca nova lingua franca nova lingua franca nova lombard lombard lombard magyar magyar magyar मैथिली मैथिली मैथिली македонски македонски македонски malagasi malagasi malagasi മലയാളം മലയാളം മലയാളം मराठी मराठी मराठी مصرى مصرى مصرى bahasa melayu bahasa melayu bahasa melayu minangkabau minangkabau minangkabau 閩東語 mìng-dĕ̤ng-ngṳ̄ 閩東語 mìng-dĕ̤ng-ngṳ̄ 閩東語 mìng-dĕ̤ng-ngṳ̄ монгол монгол монгол မြန်မာဘာသာ မြန်မာဘာသာ မြန်မာဘာသာ nederland nederland nederland नेपाली नेपाली नेपाली नेपाल भाषा नेपाल भाषा नेपाल भाषा 日本語 日本語 日本語 нохчийн нохчийн нохчийн nordfriisk nordfriisk nordfriisk norsk bokmål norsk bokmål norsk bokmål norsk nynorsk norsk nynorsk norsk nynorsk олык марий олык марий олык марий oʻzbekcha ўзбекча oʻzbekcha ўзбекча oʻzbekcha ўзбекча ਪੰਜਾਬੀ ਪੰਜਾਬੀ ਪੰਜਾਬੀ پنجابی پنجابی پنجابی ပအိုဝ်ႏဘာႏသာႏ ပအိုဝ်ႏဘာႏသာႏ ပအိုဝ်ႏဘာႏသာႏ ភាសាខ្មែរ ភាសាខ្មែរ ភាសាខ្មែរ polski polski polski portuguê portuguê portuguê qaraqalpaqsha qaraqalpaqsha qaraqalpaqsha română română română runa simi runa simi runa simi русиньскый русиньскый русиньскый русский русский русский scot scot scot shqip shqip shqip සිංහල සිංහල සිංහල simpl english simpl english simpl english سنڌي سنڌي سنڌي slovenčina slovenčina slovenčina slovenščina slovenščina slovenščina soomaaliga soomaaliga soomaaliga کوردی کوردی کوردی српски srpski српски srpski српски srpski srpskohrvatski српскохрватски srpskohrvatski српскохрватски srpskohrvatski српскохрватски suomi suomi suomi svenska svenska svenska tagalog tagalog tagalog தமிழ் தமிழ் தமிழ் ၽႃႇသႃႇတႆး ၽႃႇသႃႇတႆး ၽႃႇသႃႇတႆး తెలుగు తెలుగు తెలుగు ไทย ไทย ไทย тоҷикӣ тоҷикӣ тоҷикӣ türkçe türkçe türkçe türkmenç türkmenç türkmenç українська українська українська اردو اردو اردو vèneto vèneto vèneto tiếng việt tiếng việt tiếng việt winaray winaray winaray wolof wolof wolof 吴语 吴语 吴语 ייִדיש ייִדיש ייִדיש yorùbá yorùbá yorùbá 粵語 粵語 粵語 žemaitėška žemaitėška žemaitėška 中文 中文 中文 edit link edit link articl articl articl talk talk talk english read read read view sourc view sourc view sourc view histori view histori view histori tool read read read view sourc view sourc view sourc view histori view histori view histori link link link relat chang relat chang relat chang upload file upload file upload file special page special page special page perman link perman link perman link page inform page inform page inform cite page cite page cite page get shorten url get shorten url get shorten url download qr code download qr code download qr code download pdf download pdf download pdf printabl version printabl version printabl version wikimedia common wikimedia common wikimedia common mediawiki mediawiki mediawiki wikibook wikibook wikibook wikivers wikivers wikivers wikidata item wikidata item wikidata item htm html5 1 filenam extens .html .htm internet medium type type code uniform type identifi uti whatwg whatwg world wide web consortium w3c formerli world wide web consortium 32 year ago 1993 1993 latest releas live standard document file format contain html element web browser sgml xhtml open format open format html.spec.whatwg.org html.spec.whatwg.org html dynam html dynam html html5 articl audio canva video html5 articl articl audio audio canva canva video video xhtml basic mobil profil xhtml basic basic mobil profil mobil profil html element meta div span blink marque html element meta meta div span div span blink blink marque marque html attribut alt attribut html attribut alt attribut alt attribut html frame html frame html editor html editor charact encod name charact unicod charact encod name charact name charact unicod unicod languag code languag code document object model document object model browser object model browser object model style sheet cs style sheet cs cs font famili font famili web color web color javascript webcl htmx javascript webcl webcl htmx htmx web3d webgl webgpu webxr web3d webgl webgl webgpu webgpu webxr webxr w3c valid w3c valid valid whatwg whatwg quirk mode quirk mode web storag web storag render engin render engin document markup languag document markup languag comparison browser engin comparison browser engin v v e e hypertext markup languag html standard markup languag document design display web browser defin content structur web content often assist technolog cascad style sheet cs script languag javascript program languag markup languag web browser web content cascad style sheet script languag javascript web browser receiv html document web server local storag render document multimedia web page html describ structur web page semant origin includ cue appear web browser web server render web page semant html element build block html page html construct imag object interact form may embed render page html provid mean creat structur document denot structur semant text head paragraph list link quot item html element delin tag written use angl bracket tag img input directli introduc content page tag p /p surround provid inform document text may includ sub-el tag browser display html tag use interpret content page html element imag interact form structur document semant link angl bracket img input p p browser html emb program written script languag javascript affect behavior content web page inclus cs defin look layout content world wide web consortium w3c former maintain html current maintain cs standard encourag use cs explicit present html sinc 1997 updat 3 form html known html5 use display video audio primarili use canva element togeth javascript script languag javascript world wide web consortium cs updat 3 html5 canva histori develop tim berners-le 1980 physicist tim berners-le contractor cern propos prototyp enquir system cern research use share document 1989 berners-le wrote memo propos internet-bas hypertext system 4 berners-le specifi html wrote browser server softwar late 1990 year berners-le cern data system engin robert cailliau collabor joint request fund project formal adopt cern person note 1990 berners-le list `` mani area hypertext use '' encyclopedia first entri 5 physicist tim berners-le cern enquir internet hypertext 4 data system robert cailliau encyclopedia 5 first publicli avail descript html document call `` html tag '' 6 first mention internet tim berners-le late 1991 7 8 describ 18 element compris initi rel simpl design html except hyperlink tag strongli influenc sgmlguid in-hous standard gener markup languag sgml -base document format cern eleven element still exist html 4 9 6 7 8 sgmlguid standard gener markup languag 9 html markup languag web browser use interpret compos text imag materi visibl audibl web page default characterist everi item html markup defin browser characterist alter enhanc web page design 's addit use cs mani text element mention 1988 iso technic report tr 9537 techniqu use sgml describ featur earli text format languag use runoff command develop earli 1960 ctss compat time-shar system oper system format command deriv command use typesett manual format document howev sgml concept gener markup base element nest annot rang attribut rather mere print effect separ structur markup html progress move direct cs markup languag web browser compos cs runoff command ctss berners-le consid html applic sgml formal defin internet engin task forc ietf mid-1993 public first propos html specif `` hypertext markup languag html '' internet draft berners-le dan connolli includ sgml document type definit defin syntax 10 11 draft expir six month notabl acknowledg ncsa mosaic browser 's custom tag embed in-lin imag reflect ietf 's philosophi base standard success prototyp similarli dave raggett 's compet internet draft `` html+ hypertext markup format '' late 1993 suggest standard already-impl featur like tabl fill-out form 12 internet engin task forc dan connolli document type definit 10 11 ncsa mosaic dave raggett 12 html html+ draft expir earli 1994 ietf creat html work group 1995 work group complet `` html 2.0 '' first html specif intend treat standard futur implement base 13 13 develop auspic ietf stall compet interest sinc 1996 updat html specif maintain input commerci softwar vendor world wide web consortium w3c 14 2000 html becam intern standard iso/iec 15445:2000 html 4.01 publish late 1999 erratum publish 2001 2004 develop began html5 web hypertext applic technolog work group whatwg becam joint deliver w3c 2008 complet standard 28 octob 2014 15 updat world wide web consortium 14 iso iec web hypertext applic technolog work group 15 html version timelin html 2 rfc 1866 rfc novemb 25 1995 rfc 1867 form-bas file upload rfc 1867 may 1996 rfc 1942 tabl rfc 1942 august 1996 rfc 1980 client-sid imag map rfc 1980 januari 1997 rfc 2070 internation rfc 2070 internation html 3 16 w3c recommend 17 18 netscap blink element microsoft marque element 14 mathml html 4 19 strict deprec element forbidden transit deprec element allow frameset mostli frame relat element allow frame initi code-nam `` cougar '' 18 html 4.0 adopt mani browser-specif element type attribut also sought phase netscap 's visual markup featur mark deprec favor style sheet html 4 sgml applic conform iso 8879 – sgml 20 18 deprec 20 21 22 23 24 iso 25 iso/iec jtc 1/sc 34 24 html 5 html5 26 27 28 29 30 31 32 33 html draft version timelin 7 34 35 36 37 37 38 ietf 39 html+ 40 41 42 rfc 1866 43 44 45 45 arena browser test bed 46 47 48 14 14 browser war 14 14 html5 work draft 49 sgml html5 50 51 52 w3c 53 54 w3c recommend 55 56 57 58 xhtml version xhtml xhtml separ languag began reformul html 4.01 use xml 1.0 refer xml syntax html longer develop separ standard 59 xml 59 xhtml 1.0 publish w3c recommend januari 26 2000 60 later revis republish august 1 2002 offer three variat html 4.0 4.01 reformul xml minor restrict 60 xhtml 1.1 61 publish w3c recommend may 31 2001 base xhtml 1.0 strict includ minor chang custom reformul use modul w3c recommend `` modular xhtml '' publish april 10 2001 62 61 62 xhtml 2.0 work draft work abandon 2009 favor work html5 xhtml5 63 64 65 xhtml 2.0 incompat xhtml 1.x therefor would accur character xhtml-inspir new languag updat xhtml 1.x html5 xhtml5 63 64 65 transit html public whatwg html5 § w3c whatwg conflict 28 may 2019 w3c announc whatwg would sole publish html dom standard 66 67 68 69 w3c whatwg publish compet standard sinc 2012 w3c standard ident whatwg 2007 standard sinc progress diverg due differ design decis 70 whatwg `` live standard '' de facto web standard time 71 66 67 68 69 70 71 markup html markup consist sever key compon includ call tag attribut character-bas data type charact refer entiti refer html tag commonli come pair like h1 /h1 although repres empti element unpair exampl img first tag pair start tag second end tag also call open tag close tag h1 h1 img anoth import compon html document type declar trigger standard mode render document type declar standard mode follow exampl classic `` hello world '' program `` hello world '' program doctyp html html head titl titl head bodi div p p div bodi html text html /html describ web page text bodi /bodi visibl page content markup text titl titl /titl defin browser page titl shown browser tab window titl tag div defin divis page use easi style head /head meta element use defin webpag metadata html html bodi bodi titl titl browser tab window div head head meta document type declar doctyp html html5 declar includ variou browser revert `` quirk mode '' render 72 doctyp html quirk mode 72 element html element html document impli structur nest html element indic document html tag enclos angl bracket thu p 73 better sourc need html element p 73 better sourc need better sourc need simpl gener case extent element indic pair tag `` start tag '' p `` end tag '' /p text content element place tag p p tag may also enclos tag markup start end includ mixtur tag text indic nest element child parent element start tag may also includ element 's attribut within tag indic inform identifi section within document identifi use bind style inform present document tag img use emb imag refer imag resourc format like img src= '' example.com/example.jpg '' img img src `` example.com/example.jpg '' element line break br permit embed content either text tag requir singl empti tag akin start tag use end tag line break br mani tag particularli close end tag commonli use paragraph element p option html browser agent infer closur end element context structur rule defin html standard rule complex wide understood html author p gener form html element therefor tag attribute1= '' value1 '' attribute2= '' value2 '' '' content '' /tag html element defin empti element take form tag attribute1= '' value1 '' attribute2= '' value2 '' empti element may enclos content instanc br tag inlin img tag name html element name use tag end tag 's name preced slash charact empti element end tag neither requir allow attribut mention default valu use case tag attribute1 `` value1 '' attribute2 `` value2 '' tag tag attribute1 `` value1 '' attribute2 `` value2 '' br img element exampl html element header html document head ... /head titl includ head exampl head head head titl titl link rel `` stylesheet '' href `` stylebyjimbowales.css '' -- import stylesheet -- head head html head defin h1 h6 tag h1 highest import level h6 least h1 h6 h1 h1 h2 h2 h3 h3 h4 h4 h5 h5 h6 h6 effect cs substanti chang render paragraph p p p p line break br differ br p br break line without alter semant structur page wherea p section page paragraph element br empti element although may attribut take content may end tag br br p br break line p paragraph br p br br br p link link html creat link tag use href attribut hold url address link url href `` http //www.wikipedia.org/ '' input mani possibl way user give input like input type `` text '' -- text input -- input type `` file '' -- upload file -- input type `` checkbox '' -- checkbox -- comment -- comment -- comment help understand markup display webpag sever type markup element use html h2 h2 head cascad style sheet 74 b b b b strong strong em em deprec cs hyperlink url href `` http //en.wikipedia.org/ '' wikipedia wikipedia href `` http //example.org '' img src `` image.gif '' alt `` descript text '' width `` 50 '' height `` 50 '' border `` 0 '' attribut html attribut attribut element name–valu pair separ written within start tag element element 's name valu may enclos singl doubl quot although valu consist certain charact left unquot html xhtml 75 76 leav attribut valu unquot consid unsaf 77 contrast name-valu pair attribut attribut affect element simpli presenc start tag element 7 like ismap attribut img element 78 name–valu pair 75 76 77 7 78 sever common attribut may appear mani element id attribut provid document-wid uniqu identifi element use identifi element stylesheet alter present properti script may alter anim delet content present append url page provid global uniqu identifi element typic sub-sect page exampl id `` attribut '' http //en.wikipedia.org/wiki/html attribut class attribut provid way classifi similar element use semant present purpos exampl html document might semant use design class= '' notat '' indic element class valu subordin main text document present element might gather togeth present footnot page instead appear place occur html sourc class attribut use semant microformat multipl class valu may specifi exampl class= '' notat import '' put element notat import class semant class '' notat `` microformat class '' notat import `` author may use style attribut assign present properti particular element consid better practic use element 's id class attribut select element within stylesheet though sometim cumbersom simpl specif ad hoc style stylesheet titl attribut use attach subtextu explan element browser attribut display tooltip browser tooltip lang attribut identifi natur languag element 's content may differ rest document exampl english-languag document p oh well span lang= '' fr '' c'est la vie /span say france. /p p span lang `` fr '' span p abbrevi element abbr use demonstr attribut abbr id `` anid '' class `` jargon '' style `` color purpl '' titl `` hypertext markup languag '' abbr exampl display html browser point cursor abbrevi display titl text `` hypertext markup languag '' element take language-rel attribut dir specifi text direct `` rtl '' right-to-left text exampl arab persian hebrew 79 arab persian hebrew 79 charact entiti refer list xml html charact entiti refer unicod html version 4.0 html defin set 252 charact entiti refer set 1,114,050 numer charact refer allow individu charact written via simpl markup rather liter liter charact markup counterpart consid equival render ident charact entiti refer numer charact refer abil `` escap '' charact way allow charact written lt amp respect interpret charact data rather markup exampl liter normal indic start tag normal indic start charact entiti refer numer charact refer write amp x26 38 allow includ content element valu attribut double-quot charact `` use quot attribut valu must also escap quot x22 34 appear within attribut valu equival single-quot charact use quot attribut valu must also escap x27 39 apo html5 xhtml document 80 81 appear within attribut valu document author overlook need escap charact browser forgiv tri use context guess intent result still invalid markup make document less access browser user agent may tri par document search index purpos exampl escap 80 81 user agent search index escap also allow charact easili type avail document 's charact encod repres within element attribut content exampl acute-acc e é charact typic found western european south american keyboard written html document entiti refer eacut numer refer xe9 233 use charact avail keyboard support charact encod unicod charact encod utf-8 compat modern browser allow direct access almost charact world 's write system 82 charact encod unicod utf-8 82 ampersand less greater doubl quot singl quot non-break space copyright regist trademark dagger doubl dagger trademark data type html defin sever data type element content script data stylesheet data plethora type attribut valu includ id name uri number unit length languag medium descriptor color charact encod date time data type special charact data data type uri document type declar html document requir start document type declar inform `` doctyp '' browser doctyp help defin render mode—particularli whether use quirk mode ocument type declar quirk mode origin purpos doctyp enabl par valid html document sgml tool base document type definit dtd dtd doctyp refer contain machine-read grammar specifi permit prohibit content document conform dtd browser hand implement html applic sgml consequ read dtd document type definit html5 defin dtd therefor html5 doctyp declar simpler shorter 83 html5 83 doctyp html exampl html 4 doctyp doctyp html public `` -//w3c//dtd html 4.01//en '' `` http //www.w3.org/tr/html4/strict.dtd '' declar refer dtd `` strict '' version html 4.01 sgml-base valid read dtd order properli par document perform valid modern browser valid doctyp activ standard mode oppos quirk mode quirk mode addit html 4.01 provid transit frameset dtd explain transit type inclus incorpor current tag well older `` deprec '' tag strict dtd exclud deprec tag frameset tag necessari make frame page along tag includ transit type 84 explain 84 semant html semant html semant html way write html emphas mean encod inform present look html includ semant markup incept 85 also includ present markup font center tag also semant neutral div span tag sinc late 1990 cascad style sheet begin work browser web author encourag avoid use present html markup view separ content present 86 85 font center div span cascad style sheet separ content present 86 2001 discus semant web tim berners-le other gave exampl way intellig softwar `` agent '' may one day automat crawl web find filter correl previous unrel publish fact benefit human user 87 agent commonplac even idea web 2.0 mashup price comparison websit may come close main differ web applic hybrid berners-le 's semant agent lie fact current aggreg hybrid inform usual design web develop alreadi know web locat api semant specif data wish mash compar combin semant web tim berners-le 87 web 2.0 mashup price comparison websit aggreg web develop api semant import type web agent crawl read web page automat without prior knowledg might find web crawler search-engin spider softwar agent depend semant clariti web page find use variou techniqu algorithm read index million web page day provid web user search facil without world wide web 's use would greatli reduc web crawler algorithm search facil order search engin spider abl rate signific piec text find html document also creat mashup hybrid well autom agent develop semant structur exist html need wide uniformli appli bring mean publish text 88 88 present markup tag deprec current html xhtml recommend major present featur previou version html longer allow lead poorer access higher cost site mainten larger document size 89 deprec xhtml 89 good semant html also improv access web document see also web content access guidelin exampl screen reader audio browser correctli ascertain structur document wast visual impair user 's time read repeat irrelev inform mark correctli access web content access guidelin deliveri html document deliv mean comput file howev often deliv either http web server email http web server email http hypertext transfer protocol world wide web compos primarili html document transmit web server web browser use hypertext transfer protocol http howev http use serv imag sound content addit html allow web browser know handl document receiv inform transmit along document meta data usual includ mime type e.g. text/html application/xhtml+xml charact encod see charact encod html world wide web hypertext transfer protocol meta data mime type charact encod html modern browser mime type sent html document may affect document initi interpret document sent xhtml mime type expect well-form xml syntax error may caus browser fail render document sent html mime type might display success sinc browser lenient html well-form w3c recommend state xhtml 1.0 document follow guidelin set forth recommend 's appendix c may label either mime type 90 xhtml 1.1 also state xhtml 1.1 document 91 label either mime type 92 90 91 92 html e-mail html email graphic email client allow use subset html often ill-defin provid format semant markup avail plain text may includ typograph inform like color head emphas quot text inlin imag diagram mani client includ gui editor compos html e-mail messag render engin display use html e-mail critic compat issu help disguis phish attack access issu blind visual impair peopl confus spam filter messag size larger plain text semant plain text gui phish spam name convent common filenam extens file contain html .html common abbrevi .htm origin earli oper system file system do limit impos fat data structur limit file extens three letter 93 filenam extens file do fat three letter 93 html applic html applic html applic hta file extens .hta microsoft window applic use html dynam html browser provid applic 's graphic interfac regular html file confin secur model web browser 's secur commun web server manipul web page object site cooki hta run fulli trust applic therefor privileg like creation/editing/remov file window registri entri oper outsid browser 's secur model hta execut via http must download like exe file execut local file system microsoft window browser web browser 's secur site cooki window registri exe html4 variat sinc incept html associ protocol gain accept rel quickli howev clear standard exist earli year languag though creator origin conceiv html semant languag devoid present detail 94 practic use push mani present element attribut languag driven larg variou browser vendor latest standard surround html reflect effort overcom sometim chaotic develop languag 95 creat ration foundat build meaning well-pres document return html role semant languag w3c develop style languag cs xsl shoulder burden present conjunct html specif slowli rein present element 94 95 w3c cs xsl two axe differenti variou variat html current specifi sgml-base html versu xml-base html refer xhtml one axi strict versu transit loo versu frameset axi sgml-base versu xml-base html one differ latest html specif lie distinct sgml-base specif xml-base specif xml-base specif usual call xhtml distinguish clearli tradit definit howev root element name continu `` html '' even xhtml-specifi html w3c intend xhtml 1.0 ident html 4.01 except limit xml complex sgml requir workaround xhtml html close relat sometim document parallel circumst author conflat two name x html x html xhtml conflat two name like html 4.01 xhtml 1.0 three sub-specif strict transit frameset asid differ open declar document differ html 4.01 xhtml 1.0 document—in correspond dtds—are larg syntact underli syntax html allow mani shortcut xhtml element option open close tag even empti element must end tag contrast xhtml requir element open tag close tag xhtml howev also introduc new shortcut xhtml tag may open close within tag includ slash end tag like br/ introduct shorthand use sgml declar html 4.01 may confus earlier softwar unfamiliar new convent fix includ space close tag br 96 br br 96 understand subtl differ html xhtml consid transform valid well-form xhtml 1.0 document adher appendix c see valid html 4.01 document make translat requir follow step languag element specifi lang attribut rather xhtml xml lang attribut xhtml use xml 's built-in language-defin function attribut remov xml namespac xmlns=uri html facil namespac chang document type declar xhtml 1.0 html 4.01 see dtd section explan dtd section present remov xml declar typic xml version= '' 1.0 '' encoding= '' utf-8 '' xml version= '' 1.0 '' encoding= '' utf-8 '' ensur document 's mime type set text/html html xhtml come http content-typ header sent server chang xml empty-el syntax html style empti element br/ br br br main chang necessari translat document xhtml 1.0 html 4.01 translat html xhtml would also requir addit omit open close tag whether code html xhtml may best alway includ option tag within html document rather rememb tag omit well-form xhtml document adher syntax requir xml valid document adher content specif xhtml describ document structur w3c recommend sever convent ensur easi migrat html xhtml see html compat guidelin follow step appli xhtml 1.0 document html compat guidelin includ xml lang lang attribut element assign languag use empty-el syntax element specifi empti html includ extra space empty-el tag exampl br instead br/ br br includ explicit close tag element permit content left empti exampl div /div div div div div omit xml declar care follow w3c 's compat guidelin user agent abl interpret document equal html xhtml document xhtml 1.0 made compat way w3c permit serv either html text/html mime type xhtml application/xhtml+xml application/xml mime type deliv xhtml browser use xml parser adher strictli xml specif par document 's content mime type transit versu strict html 4 defin three differ version languag strict transit call loo frameset strict version intend new document consid best practic transit frameset version develop make easier transit document conform older html specif conform specif version html 4 transit frameset version allow present markup omit strict version instead cascad style sheet encourag improv present html document xhtml 1 defin xml syntax languag defin html 4 differ appli xhtml 1 well cascad style sheet transit version allow follow part vocabulari includ strict version looser content model inlin element plain text allow directli bodi blockquot form noscript nofram inlin element plain text allow directli bodi blockquot form noscript nofram present relat element underlin u deprec confus visitor hyperlink strike-through center deprec use cs instead font deprec use cs instead basefont deprec use cs instead underlin u deprec confus visitor hyperlink strike-through center deprec use cs instead font deprec use cs instead basefont deprec use cs instead present relat attribut background deprec use cs instead bgcolor deprec use cs instead attribut bodi requir element accord w3c element align deprec use cs instead attribut div form paragraph p head h1 ... h6 element align deprec use cs instead noshad deprec use cs instead size deprec use cs instead width deprec use cs instead attribut hr element align deprec use cs instead border vspace hspace attribut img object caution object element support internet explor major browser element align deprec use cs instead attribut legend caption element align deprec use cs instead bgcolor deprec use cs instead tabl element nowrap obsolet bgcolor deprec use cs instead width height td th element bgcolor deprec use cs instead attribut tr element clear obsolet attribut br element compact attribut dl dir menu element type deprec use cs instead compact deprec use cs instead start deprec use cs instead attribut ol ul element type valu attribut li element width attribut pre element background deprec use cs instead bgcolor deprec use cs instead attribut bodi requir element accord w3c element align deprec use cs instead attribut div form paragraph p head h1 ... h6 element align deprec use cs instead noshad deprec use cs instead size deprec use cs instead width deprec use cs instead attribut hr element align deprec use cs instead border vspace hspace attribut img object caution object element support internet explor major browser element align deprec use cs instead attribut legend caption element align deprec use cs instead bgcolor deprec use cs instead tabl element nowrap obsolet bgcolor deprec use cs instead width height td th element bgcolor deprec use cs instead attribut tr element clear obsolet attribut br element compact attribut dl dir menu element type deprec use cs instead compact deprec use cs instead start deprec use cs instead attribut ol ul element type valu attribut li element width attribut pre element addit element transit specif menu deprec use cs instead list substitut though unord list recommend dir deprec use cs instead list substitut though unord list recommend isindex deprec element requir server-sid support typic ad document server-sid form input element use substitut applet deprec use object element instead menu deprec use cs instead list substitut though unord list recommend dir deprec use cs instead list substitut though unord list recommend isindex deprec element requir server-sid support typic ad document server-sid form input element use substitut applet deprec use object element instead languag obsolet attribut script element redund type attribut frame relat entiti ifram nofram target deprec map link form element attribut client-sid image-map map link form base element ifram nofram target deprec map link form element attribut client-sid image-map map link form base element frameset version includ everyth transit version well frameset element use instead bodi frame element frameset versu transit addit transit differ frameset specif whether xhtml 1.0 html 4.01 specifi differ content model frameset replac bodi contain either frame element option nofram bodi summari specif version list demonstr loo version specif maintain legaci support howev contrari popular misconcept move xhtml impli remov legaci support rather x xml stand extens w3c modular entir specif open independ extens primari achiev move xhtml 1.0 xhtml 1.1 modular entir specif strict version html deploy xhtml 1.1 set modular extens base xhtml 1.1 specif likewis someon look loo transit frameset specif find similar extend xhtml 1.1 support much contain legaci frame modul modular also allow separ featur develop timet exampl xhtml 1.1 allow quicker migrat emerg xml standard mathml present semant math languag base xml xforms—a new highli advanc web-form technolog replac exist html form mathml xform summari html 4 specif primarili rein variou html implement singl clearli written specif base sgml xhtml 1.0 port specif new xml-defin specif next xhtml 1.1 take advantag extens natur xml modular whole specif xhtml 2.0 intend first step ad new featur specif standards-body-bas approach whatwg html versu html5 § transit html public whatwg html live standard develop whatwg offici version w3c html5 longer separ whatwg wysiwyg editor plea expand articl make edit requestto includ inform detail may exist talk page make edit request make edit request make edit request includ inform talk page januari 2021 januari 2021 wysiwyg editor see get user lay everyth appear html document use graphic user interfac gui often similar word processor editor render document rather show code author requir extens knowledg html wysiwyg graphic user interfac word processor wysiwyg edit model critic 97 98 primarili low qualiti gener code voic advoc chang wysiwym model see mean 97 98 wysiwym wysiwyg editor remain controversi topic perceiv flaw reli mainli layout oppos mean often use markup convey intend mean simpli copi layout 99 99 often produc extrem verbos redund code fail make use cascad natur html cs cs often produc ungrammat markup call tag soup semant incorrect markup em ital tag soup em great deal inform html document layout model critic `` see get '' -natur 100 100 see also breadcrumb navig breadcrumb navig cellpad cellpad comparison html parser comparison html parser dynam web page dynam web page html applic html applic html charact refer html charact refer list document markup languag list document markup languag list xml html charact entiti refer list xml html charact entiti refer microdata html microdata html microformat microformat polyglot markup polyglot markup semant html semant html w3c x html valid w3c x html valid web color web color note even though html run browser view program languag program languag discours 2 even though html run browser view program languag program languag discours 2 program languag 2 refer `` w3c html '' `` w3c html '' `` w3c html '' herman felienn schlesing ari 2024-10-17 `` case femin program languag design '' proceed 2024 acm sigplan intern symposium new idea new paradigm reflect program softwar acm pp 205–222 doi:10.1145/3689492.3689809 isbn 979-8-4007-1215-9 cite book |journal= ignor help herman felienn schlesing ari 2024-10-17 `` case femin program languag design '' proceed 2024 acm sigplan intern symposium new idea new paradigm reflect program softwar acm pp 205–222 doi:10.1145/3689492.3689809 isbn 979-8-4007-1215-9 cite book |journal= ignor help `` case femin program languag design '' 205– doi 10.1145/3689492.3689809 isbn 979-8-4007-1215-9 cite book cite book |journal= ignor help help `` html 4.0 specif — w3c recommend — conform requir recommend '' world wide web consortium decemb 18 1997 archiv origin juli 5 2015 retriev juli 6 2015 `` html 4.0 specif — w3c recommend — conform requir recommend '' world wide web consortium decemb 18 1997 archiv origin juli 5 2015 retriev juli 6 2015 `` html 4.0 specif — w3c recommend — conform requir recommend '' archiv retriev juli 6 2015 juli 6 tim berners-le `` inform manag propos '' cern march 1989 may 1990 w3c tim berners-le `` inform manag propos '' cern march 1989 may 1990 w3c inform manag propos berners-le tim `` intend use '' w3c berners-le tim `` intend use '' w3c `` intend use '' `` tag use html '' info.cern.ch octob 1991 retriev 2 march 2023 `` tag use html '' info.cern.ch octob 1991 retriev 2 march 2023 `` tag use html '' retriev 2 march 2023 2 march b c `` tag use html '' world wide web consortium novemb 3 1992 archiv origin januari 31 2010 retriev novemb 16 2008 b c b c `` tag use html '' world wide web consortium novemb 3 1992 archiv origin januari 31 2010 retriev novemb 16 2008 `` tag use html '' archiv retriev novemb 16 2008 novemb 16 berners-le tim octob 29 1991 `` statu x11 browser www '' world wide web consortium archiv origin may 24 2007 retriev april 8 2007 berners-le tim octob 29 1991 `` statu x11 browser www '' world wide web consortium archiv origin may 24 2007 retriev april 8 2007 `` statu x11 browser www '' archiv retriev april 8 2007 april 8 `` index html 4 element '' world wide web consortium decemb 24 1999 archiv origin may 5 2007 retriev april 8 2007 `` index html 4 element '' world wide web consortium decemb 24 1999 archiv origin may 5 2007 retriev april 8 2007 `` index html 4 element '' archiv retriev april 8 2007 april 8 berners-le tim decemb 9 1991 `` sgml/html doc x browser '' w3 archiv origin decemb 22 2007 retriev june 16 2007 sgml gener html specif applic sgml basic syntax appli hypertext document simpl structur berners-le tim decemb 9 1991 `` sgml/html doc x browser '' w3 archiv origin decemb 22 2007 retriev june 16 2007 sgml gener html specif applic sgml basic syntax appli hypertext document simpl structur `` sgml/html doc x browser '' archiv retriev june 16 2007 june 16 berners-le tim connolli daniel june 1993 `` hypertext markup languag html represent textual inform metainform retriev interchang '' w3 archiv origin januari 3 2017 retriev januari 4 2017 berners-le tim connolli daniel june 1993 `` hypertext markup languag html represent textual inform metainform retriev interchang '' w3 archiv origin januari 3 2017 retriev januari 4 2017 `` hypertext markup languag html represent textual inform metainform retriev interchang '' archiv retriev januari 4 2017 januari 4 raggett dave `` review html+ document format '' w3 archiv origin februari 29 2000 retriev may 22 2020 hypertext markup languag html develop simpl non-proprietari deliveri format global hypertext html+ set modular extens html develop respons grow understand need inform provid extens includ text flow around float figur fill-out form tabl mathemat equat raggett dave `` review html+ document format '' w3 archiv origin februari 29 2000 retriev may 22 2020 hypertext markup languag html develop simpl non-proprietari deliveri format global hypertext html+ set modular extens html develop respons grow understand need inform provid extens includ text flow around float figur fill-out form tabl mathemat equat raggett dave `` review html+ document format '' archiv retriev may 22 2020 may 22 berners-le tim connolli daniel w. novemb 1995 hypertext markup languag 2.0 network work group doi:10.17487/rfc1866 rfc 1866 histor obsolet rfc 2854 document thu defin html 2.0 distinguish previou inform specif futur gener upwardli compat version html new featur releas higher version number berners-le tim connolli daniel w. novemb 1995 hypertext markup languag 2.0 network work group doi:10.17487/rfc1866 rfc 1866 histor obsolet rfc 2854 document thu defin html 2.0 distinguish previou inform specif futur gener upwardli compat version html new featur releas higher version number berners-le tim hypertext markup languag 2.0 doi 10.17487/rfc1866 10.17487/rfc1866 rfc 1866 2854 b c e f raggett dave 1998 raggett html 4 archiv origin august 9 2007 retriev juli 9 2007 b c e f b c e f raggett dave 1998 raggett html 4 archiv origin august 9 2007 retriev juli 9 2007 raggett html 4 origin retriev juli 9 2007 juli 9 `` html5 – hypertext markup languag – 5.0 '' internet engin task forc 28 octob 2014 archiv origin octob 28 2014 retriev novemb 25 2014 document recommend html 5.0 complet `` html5 – hypertext markup languag – 5.0 '' internet engin task forc 28 octob 2014 archiv origin octob 28 2014 retriev novemb 25 2014 document recommend html 5.0 complet `` html5 – hypertext markup languag – 5.0 '' archiv retriev novemb 25 2014 novemb 25 `` html 3.2 refer specif '' world wide web consortium januari 14 1997 retriev novemb 16 2008 `` html 3.2 refer specif '' world wide web consortium januari 14 1997 retriev novemb 16 2008 `` html 3.2 refer specif '' retriev novemb 16 2008 novemb 16 `` ietf html wg '' retriev june 16 2007 note work group close `` ietf html wg '' retriev june 16 2007 note work group close `` ietf html wg '' retriev june 16 2007 june 16 b engelfriet arnoud `` introduct wilbur '' htmlhelp.com retriev june 16 2007 b b engelfriet arnoud `` introduct wilbur '' htmlhelp.com retriev june 16 2007 engelfriet arnoud `` introduct wilbur '' retriev june 16 2007 june 16 `` html 4.0 specif '' world wide web consortium decemb 18 1997 retriev novemb 16 2008 `` html 4.0 specif '' world wide web consortium decemb 18 1997 retriev novemb 16 2008 `` html 4.0 specif '' retriev novemb 16 2008 novemb 16 `` html 4 – 4 conform requir recommend '' retriev decemb 30 2009 `` html 4 – 4 conform requir recommend '' retriev decemb 30 2009 `` html 4 – 4 conform requir recommend '' retriev decemb 30 2009 decemb 30 `` html 4.0 specif '' world wide web consortium april 24 1998 retriev novemb 16 2008 `` html 4.0 specif '' world wide web consortium april 24 1998 retriev novemb 16 2008 `` html 4.0 specif '' retriev novemb 16 2008 novemb 16 `` html 4.01 specif '' world wide web consortium decemb 24 1999 retriev novemb 16 2008 `` html 4.01 specif '' world wide web consortium decemb 24 1999 retriev novemb 16 2008 `` html 4.01 specif '' retriev novemb 16 2008 novemb 16 `` html 4 erratum '' w3c retriev march 2 2023 `` html 4 erratum '' w3c retriev march 2 2023 `` html 4 erratum '' retriev march 2 2023 march 2 b iso 2000 `` iso/iec 15445:2000 – inform technolog – document descript process languag – hypertext markup languag html '' retriev march 1 2023 b b iso 2000 `` iso/iec 15445:2000 – inform technolog – document descript process languag – hypertext markup languag html '' retriev march 1 2023 `` iso/iec 15445:2000 – inform technolog – document descript process languag – hypertext markup languag html '' retriev march 1 2023 march 1 `` iso/iec 15445:2000 e iso-html '' www.scss.tcd.ie geneva ch iso/iec may 15 2000 retriev march 1 2023 `` iso/iec 15445:2000 e iso-html '' www.scss.tcd.ie geneva ch iso/iec may 15 2000 retriev march 1 2023 `` iso/iec 15445:2000 e iso-html '' retriev march 1 2023 march 1 `` html5 vocabulari associ api html xhtml '' world wide web consortium 28 octob 2014 retriev 31 octob 2014 `` html5 vocabulari associ api html xhtml '' world wide web consortium 28 octob 2014 retriev 31 octob 2014 `` html5 vocabulari associ api html xhtml '' retriev 31 octob 2014 31 octob `` open web platform mileston achiev html5 recommend '' press releas world wide web consortium 28 octob 2014 retriev 31 octob 2014 `` open web platform mileston achiev html5 recommend '' press releas world wide web consortium 28 octob 2014 retriev 31 octob 2014 `` open web platform mileston achiev html5 recommend '' retriev 31 octob 2014 31 octob `` html 5.1 '' world wide web consortium 1 novemb 2016 retriev 6 januari 2017 `` html 5.1 '' world wide web consortium 1 novemb 2016 retriev 6 januari 2017 `` html 5.1 '' retriev 6 januari 2017 6 januari `` html 5.1 w3c recommend '' world wide web consortium 1 novemb 2016 retriev 6 januari 2017 `` html 5.1 w3c recommend '' world wide web consortium 1 novemb 2016 retriev 6 januari 2017 `` html 5.1 w3c recommend '' retriev 6 januari 2017 6 januari philipp le hegaret 17 novemb 2016 `` html 5.1 gold standard '' world wide web consortium retriev 6 januari 2017 philipp le hegaret 17 novemb 2016 `` html 5.1 gold standard '' world wide web consortium retriev 6 januari 2017 `` html 5.1 gold standard '' retriev 6 januari 2017 6 januari `` html 5.2 '' world wide web consortium 14 decemb 2017 retriev 15 decemb 2017 `` html 5.2 '' world wide web consortium 14 decemb 2017 retriev 15 decemb 2017 `` html 5.2 '' retriev 15 decemb 2017 15 decemb `` html 5.2 w3c recommend '' world wide web consortium 14 decemb 2017 retriev 15 decemb 2017 `` html 5.2 w3c recommend '' world wide web consortium 14 decemb 2017 retriev 15 decemb 2017 `` html 5.2 w3c recommend '' retriev 15 decemb 2017 15 decemb charl mccathi nevil 14 decemb 2017 `` html 5.2 done html 5.3 come '' world wide web consortium retriev 15 decemb 2017 charl mccathi nevil 14 decemb 2017 `` html 5.2 done html 5.3 come '' world wide web consortium retriev 15 decemb 2017 `` html 5.2 done html 5.3 come '' retriev 15 decemb 2017 15 decemb connolli daniel 6 june 1992 `` mime hypertext architectur '' cern retriev 24 octob 2010 connolli daniel 6 june 1992 `` mime hypertext architectur '' cern retriev 24 octob 2010 connolli daniel `` mime hypertext architectur '' retriev 24 octob 2010 24 octob connolli daniel 15 juli 1992 `` html dtd enclos '' cern retriev 24 octob 2010 connolli daniel 15 juli 1992 `` html dtd enclos '' cern retriev 24 octob 2010 connolli daniel `` html dtd enclos '' retriev 24 octob 2010 24 octob connolli daniel 18 august 1992 `` document type declar subset hyper text markup languag defin world wide web project '' cern archiv origin 14 march 2012 retriev 24 octob 2010 connolli daniel 18 august 1992 `` document type declar subset hyper text markup languag defin world wide web project '' cern archiv origin 14 march 2012 retriev 24 octob 2010 connolli daniel `` document type declar subset hyper text markup languag defin world wide web project '' origin retriev 24 octob 2010 24 octob b connolli daniel 24 novemb 1992 `` document type definit hyper text markup languag use world wide web applic '' cern archiv origin 18 januari 2012 retriev 24 octob 2010 see section `` revis histori '' b b connolli daniel 24 novemb 1992 `` document type definit hyper text markup languag use world wide web applic '' cern archiv origin 18 januari 2012 retriev 24 octob 2010 see section `` revis histori '' connolli daniel `` document type definit hyper text markup languag use world wide web applic '' origin retriev 24 octob 2010 24 octob berners-le tim connolli daniel june 1993 `` hyper text markup languag html internet-draft version 1.1 '' ietf iiir work group retriev 18 septemb 2010 berners-le tim connolli daniel june 1993 `` hyper text markup languag html internet-draft version 1.1 '' ietf iiir work group retriev 18 septemb 2010 berners-le tim connolli daniel `` hyper text markup languag html internet-draft version 1.1 '' retriev 18 septemb 2010 18 septemb berners-le tim connolli daniel june 1993 `` hypertext markup languag html internet-draft version 1.2 '' ietf iiir work group retriev 18 septemb 2010 berners-le tim connolli daniel june 1993 `` hypertext markup languag html internet-draft version 1.2 '' ietf iiir work group retriev 18 septemb 2010 berners-le tim connolli daniel `` hypertext markup languag html internet-draft version 1.2 '' retriev 18 septemb 2010 18 septemb raggett dave 1993-11-08 `` histori draft-raggett-www-html-00 '' ietf datatrack retriev 2019-11-18 raggett dave 1993-11-08 `` histori draft-raggett-www-html-00 '' ietf datatrack retriev 2019-11-18 `` histori draft-raggett-www-html-00 '' retriev 2019-11-18 2019-11-18 berners-le tim connolli daniel 28 novemb 1994 `` hypertext markup languag specif – 2.0 internet draft '' internet engin task forc retriev 24 octob 2010 berners-le tim connolli daniel 28 novemb 1994 `` hypertext markup languag specif – 2.0 internet draft '' internet engin task forc retriev 24 octob 2010 berners-le tim connolli daniel `` hypertext markup languag specif – 2.0 internet draft '' retriev 24 octob 2010 24 octob connolli daniel w. 1995-05-16 `` hypertext markup languag – 2.0 '' tools.ietf.org retriev 2019-11-18 connolli daniel w. 1995-05-16 `` hypertext markup languag – 2.0 '' tools.ietf.org retriev 2019-11-18 `` hypertext markup languag – 2.0 '' retriev 2019-11-18 2019-11-18 berners-le tim connolli daniel w. novemb 1995 hypertext markup languag 2.0 network work group doi:10.17487/rfc1866 rfc 1866 histor obsolet rfc 2854 berners-le tim connolli daniel w. novemb 1995 hypertext markup languag 2.0 network work group doi:10.17487/rfc1866 rfc 1866 histor obsolet rfc 2854 berners-le tim hypertext markup languag 2.0 doi 10.17487/rfc1866 10.17487/rfc1866 rfc 1866 2854 `` html 3.0 draft expir materi '' world wide web consortium decemb 21 1995 retriev novemb 16 2008 `` html 3.0 draft expir materi '' world wide web consortium decemb 21 1995 retriev novemb 16 2008 `` html 3.0 draft expir materi '' retriev novemb 16 2008 novemb 16 b `` hypertext markup languag specif version 3.0 '' retriev june 16 2007 b b `` hypertext markup languag specif version 3.0 '' retriev june 16 2007 `` hypertext markup languag specif version 3.0 '' retriev june 16 2007 june 16 raggett dave 28 march 1995 `` hypertext markup languag specif version 3.0 '' html 3.0 internet draft expir six month world wide web consortium retriev 17 june 2010 raggett dave 28 march 1995 `` hypertext markup languag specif version 3.0 '' html 3.0 internet draft expir six month world wide web consortium retriev 17 june 2010 `` hypertext markup languag specif version 3.0 '' world wide web consortium retriev 17 june 2010 17 june bower n. 1998 `` weblint anoth perl hack '' pdf 1998 usenix annual technic confer usenix atc 98 bower n. 1998 `` weblint anoth perl hack '' pdf 1998 usenix annual technic confer usenix atc 98 `` weblint anoth perl hack '' pdf lie håkon wium bo bert april 1997 cascad style sheet design web addison wesley longman p. 263 isbn 978-0-201-41998-6 retriev 9 june 2010 lie håkon wium bo bert april 1997 cascad style sheet design web addison wesley longman p. 263 isbn 978-0-201-41998-6 retriev 9 june 2010 lie håkon wium bo bert cascad style sheet design web cascad style sheet design web 263 isbn 978-0-201-41998-6 retriev 9 june 2010 9 june `` html5 '' world wide web consortium june 10 2008 retriev novemb 16 2008 `` html5 '' world wide web consortium june 10 2008 retriev novemb 16 2008 `` html5 '' retriev novemb 16 2008 novemb 16 `` html5 one vocabulari two serial '' 15 januari 2008 retriev februari 25 2009 `` html5 one vocabulari two serial '' 15 januari 2008 retriev februari 25 2009 `` html5 one vocabulari two serial '' retriev februari 25 2009 februari 25 `` w3c confirm may 2011 html5 last call target 2014 html5 standard '' world wide web consortium 14 februari 2011 retriev 18 februari 2011 `` w3c confirm may 2011 html5 last call target 2014 html5 standard '' world wide web consortium 14 februari 2011 retriev 18 februari 2011 `` w3c confirm may 2011 html5 last call target 2014 html5 standard '' world wide web consortium retriev 18 februari 2011 18 februari hickson ian januari 19 2011 `` html new html5 '' whatwg blog archiv origin 6 octob 2019 retriev 21 januari 2011 hickson ian januari 19 2011 `` html new html5 '' whatwg blog archiv origin 6 octob 2019 retriev 21 januari 2011 `` html new html5 '' origin retriev 21 januari 2011 21 januari grannel craig juli 23 2012 `` html5 get split '' net magazin archiv origin jul 25 2012 retriev 23 juli 2012 grannel craig juli 23 2012 `` html5 get split '' net magazin archiv origin jul 25 2012 retriev 23 juli 2012 `` html5 get split '' origin retriev 23 juli 2012 23 juli `` html5 '' w3c 2012-12-17 retriev 2013-06-15 `` html5 '' w3c 2012-12-17 retriev 2013-06-15 `` html5 '' retriev 2013-06-15 2013-06-15 `` html5 finish '' faq work group retriev 29 novemb 2009 `` html5 finish '' faq work group retriev 29 novemb 2009 `` html5 finish '' retriev 29 novemb 2009 29 novemb `` call review html5 propos recommend publish w3c news '' w3c 2014-09-16 retriev 2014-09-27 `` call review html5 propos recommend publish w3c news '' w3c 2014-09-16 retriev 2014-09-27 `` call review html5 propos recommend publish w3c news '' retriev 2014-09-27 2014-09-27 `` open web platform mileston achiev html5 recommend '' w3c 28 octob 2014 retriev 29 octob 2014 `` open web platform mileston achiev html5 recommend '' w3c 28 octob 2014 retriev 29 octob 2014 `` open web platform mileston achiev html5 recommend '' retriev 29 octob 2014 29 octob `` html5 specif final squabbl spec continu '' ar technica 2014-10-29 retriev 2014-10-29 `` html5 specif final squabbl spec continu '' ar technica 2014-10-29 retriev 2014-10-29 `` html5 specif final squabbl spec continu '' retriev 2014-10-29 2014-10-29 `` html v xml syntax '' whatwg retriev 22 march 2023 `` html v xml syntax '' whatwg retriev 22 march 2023 `` html v xml syntax '' retriev 22 march 2023 22 march `` xhtml 1.0 extens hypertext markup languag second edit '' world wide web consortium januari 26 2000 retriev novemb 16 2008 `` xhtml 1.0 extens hypertext markup languag second edit '' world wide web consortium januari 26 2000 retriev novemb 16 2008 `` xhtml 1.0 extens hypertext markup languag second edit '' retriev novemb 16 2008 novemb 16 `` xhtml 1.1 – module-bas xhtml — second edit '' world wide web consortium februari 16 2007 retriev novemb 16 2008 `` xhtml 1.1 – module-bas xhtml — second edit '' world wide web consortium februari 16 2007 retriev novemb 16 2008 `` xhtml 1.1 – module-bas xhtml — second edit '' retriev novemb 16 2008 novemb 16 `` modular xhtml '' w3c retriev 2017-01-04 `` modular xhtml '' w3c retriev 2017-01-04 `` modular xhtml '' retriev 2017-01-04 2017-01-04 `` xhtm 2.0 '' world wide web consortium juli 26 2006 retriev novemb 16 2008 `` xhtm 2.0 '' world wide web consortium juli 26 2006 retriev novemb 16 2008 `` xhtm 2.0 '' retriev novemb 16 2008 novemb 16 `` xhtml 2 work group expect stop work end 2009 w3c increas resourc html5 '' world wide web consortium juli 17 2009 retriev novemb 16 2008 `` xhtml 2 work group expect stop work end 2009 w3c increas resourc html5 '' world wide web consortium juli 17 2009 retriev novemb 16 2008 `` xhtml 2 work group expect stop work end 2009 w3c increas resourc html5 '' retriev novemb 16 2008 novemb 16 `` w3c xhtml faq '' `` w3c xhtml faq '' `` w3c xhtml faq '' jaff jeff 28 may 2019 `` w3c whatwg work togeth advanc open web platform '' w3c blog archiv origin 29 may 2019 retriev 29 may 2019 jaff jeff 28 may 2019 `` w3c whatwg work togeth advanc open web platform '' w3c blog archiv origin 29 may 2019 retriev 29 may 2019 `` w3c whatwg work togeth advanc open web platform '' archiv retriev 29 may 2019 29 may `` w3c whatwg sign agreement collabor singl version html dom '' w3c 28 may 2019 archiv origin 29 may 2019 retriev 29 may 2019 `` w3c whatwg sign agreement collabor singl version html dom '' w3c 28 may 2019 archiv origin 29 may 2019 retriev 29 may 2019 `` w3c whatwg sign agreement collabor singl version html dom '' archiv retriev 29 may 2019 29 may `` memorandum understand w3c whatwg '' w3c 28 may 2019 archiv origin 29 may 2019 retriev 29 may 2019 `` memorandum understand w3c whatwg '' w3c 28 may 2019 archiv origin 29 may 2019 retriev 29 may 2019 `` memorandum understand w3c whatwg '' archiv retriev 29 may 2019 29 may cimpanu catalin 29 may 2019 `` browser vendor win war w3c html dom standard '' zdnet archiv origin 29 may 2019 retriev 29 may 2019 cimpanu catalin 29 may 2019 `` browser vendor win war w3c html dom standard '' zdnet archiv origin 29 may 2019 retriev 29 may 2019 `` browser vendor win war w3c html dom standard '' origin retriev 29 may 2019 29 may `` w3c – whatwg wiki '' whatwg wiki archiv origin 29 may 2019 retriev 29 may 2019 `` w3c – whatwg wiki '' whatwg wiki archiv origin 29 may 2019 retriev 29 may 2019 `` w3c – whatwg wiki '' origin retriev 29 may 2019 29 may shankland stephen juli 9 2009 `` epitaph web standard xhtml 2 '' cnet cb interact inc. shankland stephen juli 9 2009 `` epitaph web standard xhtml 2 '' cnet cb interact inc. `` epitaph web standard xhtml 2 '' activ browser mode doctyp hsivonen.iki.fi retriev 2012-02-16 activ browser mode doctyp hsivonen.iki.fi retriev 2012-02-16 activ browser mode doctyp `` html element '' w3school retriev 16 march 2015 `` html element '' w3school retriev 16 march 2015 `` html element '' retriev 16 march 2015 16 march `` cs introduct '' w3school retriev 16 march 2015 `` cs introduct '' w3school retriev 16 march 2015 `` cs introduct '' retriev 16 march 2015 16 march `` sgml html '' world wide web consortium retriev novemb 16 2008 `` sgml html '' world wide web consortium retriev novemb 16 2008 `` sgml html '' retriev novemb 16 2008 novemb 16 `` xhtml 1.0 – differ html 4 '' world wide web consortium retriev novemb 16 2008 `` xhtml 1.0 – differ html 4 '' world wide web consortium retriev novemb 16 2008 `` xhtml 1.0 – differ html 4 '' retriev novemb 16 2008 novemb 16 korpela jukka juli 6 1998 `` attribut valu alway quot html '' cs.tut.fi retriev novemb 16 2008 korpela jukka juli 6 1998 `` attribut valu alway quot html '' cs.tut.fi retriev novemb 16 2008 `` attribut valu alway quot html '' retriev novemb 16 2008 novemb 16 `` object imag applet html document '' world wide web consortium decemb 24 1999 retriev novemb 16 2008 `` object imag applet html document '' world wide web consortium decemb 24 1999 retriev novemb 16 2008 `` object imag applet html document '' retriev novemb 16 2008 novemb 16 `` h56 use dir attribut inlin element resolv problem nest direct run '' techniqu wcag 2.0 w3c retriev 18 septemb 2010 `` h56 use dir attribut inlin element resolv problem nest direct run '' techniqu wcag 2.0 w3c retriev 18 septemb 2010 `` h56 use dir attribut inlin element resolv problem nest direct run '' retriev 18 septemb 2010 18 septemb `` charact entiti refer chart '' world wide web consortium octob 24 2012 `` charact entiti refer chart '' world wide web consortium octob 24 2012 `` charact entiti refer chart '' `` name charact refer '' world wide web consortium januari 26 2000 `` name charact refer '' world wide web consortium januari 26 2000 `` name charact refer '' `` unicod standard technic introduct '' unicod retriev 2010-03-16 `` unicod standard technic introduct '' unicod retriev 2010-03-16 `` unicod standard technic introduct '' retriev 2010-03-16 2010-03-16 `` html syntax '' html standard retriev 2013-08-19 `` html syntax '' html standard retriev 2013-08-19 `` html syntax '' retriev 2013-08-19 2013-08-19 `` html 4 frameset document type definit '' w3c retriev 2021-12-25 `` html 4 frameset document type definit '' w3c retriev 2021-12-25 `` html 4 frameset document type definit '' retriev 2021-12-25 2021-12-25 berners-le tim fischetti mark 2000 weav web origin design ultim destini world wide web inventor san francisco harper isbn 978-0-06-251587-2 berners-le tim fischetti mark 2000 weav web origin design ultim destini world wide web inventor san francisco harper isbn 978-0-06-251587-2 weav web origin design ultim destini world wide web inventor weav web origin design ultim destini world wide web inventor isbn 978-0-06-251587-2 raggett dave 2002 `` ad touch style '' w3c retriev octob 2 2009 articl note present html markup may use target browser `` netscap 4.0 internet explor 4.0 '' see list web browser confirm releas 1997 raggett dave 2002 `` ad touch style '' w3c retriev octob 2 2009 articl note present html markup may use target browser `` netscap 4.0 internet explor 4.0 '' see list web browser confirm releas 1997 `` ad touch style '' retriev octob 2 2009 octob 2 list web browser berners-le tim hendler jame lassila os may 1 2001 `` semant web '' scientif american retriev octob 2 2009 berners-le tim hendler jame lassila os may 1 2001 `` semant web '' scientif american retriev octob 2 2009 `` semant web '' retriev octob 2 2009 octob 2 nigel shadbolt wendi hall tim berners-le 2006 `` semant web revisit '' pdf ieee intellig system archiv origin pdf march 20 2013 retriev octob 2 2009 nigel shadbolt wendi hall tim berners-le 2006 `` semant web revisit '' pdf ieee intellig system archiv origin pdf march 20 2013 retriev octob 2 2009 `` semant web revisit '' pdf origin pdf retriev octob 2 2009 octob 2 `` html live standard '' whatwg retriev 27 septemb 2018 `` html live standard '' whatwg retriev 27 septemb 2018 `` html live standard '' retriev 27 septemb 2018 27 septemb `` xhtml 1.0 extens hypertext markup languag second edit '' world wide web consortium 2002 2000 retriev decemb 7 2008 xhtml document follow guidelin set forth appendix c `` html compat guidelin '' may label internet medium type `` text/html '' rfc2854 compat html browser document document conform specif may also label internet medium type `` application/xhtml+xml '' defin rfc3236 `` xhtml 1.0 extens hypertext markup languag second edit '' world wide web consortium 2002 2000 retriev decemb 7 2008 xhtml document follow guidelin set forth appendix c `` html compat guidelin '' may label internet medium type `` text/html '' rfc2854 compat html browser document document conform specif may also label internet medium type `` application/xhtml+xml '' defin rfc3236 `` xhtml 1.0 extens hypertext markup languag second edit '' retriev decemb 7 2008 decemb 7 s. bradner march 1997 key word use rfc indic requir level network work group doi:10.17487/rfc2119 bcp 14 rfc 2119 best current practic 14 updat rfc 8174 3 word adject `` recommend '' mean may exist valid reason particular circumst ignor particular item full implic must understood care weigh choos differ cours s. bradner march 1997 key word use rfc indic requir level network work group doi:10.17487/rfc2119 bcp 14 rfc 2119 best current practic 14 updat rfc 8174 3 word adject `` recommend '' mean may exist valid reason particular circumst ignor particular item full implic must understood care weigh choos differ cours s. bradner key word use rfc indic requir level doi 10.17487/rfc2119 10.17487/rfc2119 rfc 2119 8174 `` xhtml 1.1 – module-bas xhtml — second edit '' world wide web consortium 2007 retriev decemb 7 2008 xhtml 1.1 document label internet medium type text/html defin rfc2854 application/xhtml+xml defin rfc3236 `` xhtml 1.1 – module-bas xhtml — second edit '' world wide web consortium 2007 retriev decemb 7 2008 xhtml 1.1 document label internet medium type text/html defin rfc2854 application/xhtml+xml defin rfc3236 `` xhtml 1.1 – module-bas xhtml — second edit '' retriev decemb 7 2008 decemb 7 `` name file path namespac '' microsoft retriev 16 march 2015 `` name file path namespac '' microsoft retriev 16 march 2015 `` name file path namespac '' retriev 16 march 2015 16 march html design constraint w3c archiv html design constraint w3c archiv html design constraint www btb – html pri sear www btb – html pri sear www btb – html freeman e 2005 head first html o'reilli freeman e 2005 head first html o'reilli sauer c. wysiwiki – question wysiwyg internet age wikimania 2006 sauer c. wysiwiki – question wysiwyg internet age wikimania 2006 spiesser j. kitchen l. optim html automat gener wysiwyg program 13th intern confer world wide web pp 355—364 www '04 acm new york ny new york ny u.s. may 17–20 2004 spiesser j. kitchen l. optim html automat gener wysiwyg program 13th intern confer world wide web pp 355—364 www '04 acm new york ny new york ny u.s. may 17–20 2004 xhtml refer blockquot archiv 2010-03-25 wayback machin xhtml.com retriev 2012-02-16 xhtml refer blockquot archiv 2010-03-25 wayback machin xhtml.com retriev 2012-02-16 xhtml refer blockquot archiv wayback machin doug engelbart 's invis revolut invisiblerevolution.net retriev 2012-02-16 doug engelbart 's invis revolut invisiblerevolution.net retriev 2012-02-16 doug engelbart 's invis revolut extern link html sister project sister project definit wiktionari definit wiktionari definit medium common medium common medium textbook wikibook textbook wikibook textbook resourc wikivers resourc wikivers resourc data wikidata data wikidata data discus meta-wiki discus meta-wiki discus document mediawiki document mediawiki document whatwg 's html live standard whatwg html live standard dave raggett 's introduct html dave raggett 's introduct html tim berners-le give web new definit archiv 12 april 2011 tim berners-le give web new definit list html element major version list html element major version html entiti html entiti sean b. palmer `` earli histori html – 1990 1992 '' infomesh retriev 2022-04-13 timefram 1980–1995 `` earli histori html – 1990 1992 '' retriev 2022-04-13 2022-04-13 v v e e web browser featur standard protocol bookmark bookmark extens extens privaci mode privaci mode web standard html v5 html v5 v5 cs cs dom dom javascript webassembl web storag indexeddb webgl webgpu javascript webassembl webassembl web storag web storag indexeddb indexeddb webgl webgl webgpu webgpu http encrypt cooki third-parti http encrypt encrypt cooki third-parti cooki third-parti third-parti ocsp ocsp webrtc webrtc websocket websocket blink proprietari googl chrome googl chrome arc arc avast avast coc coc coc coc comodo comodo epic epic maxthon maxthon microsoft edg microsoft edg opera mobil opera mobil puffin puffin qq qq samsung samsung silk silk sleipnir sleipnir srware srware uc uc vivaldi vivaldi whale whale yandex yandex foss chromium chromium brave brave doobl doobl falkon falkon otter otter supermium supermium ungoogl ungoogl gecko firefox firefox floorp floorp gnu icecat gnu icecat librewolf librewolf midori midori slimbrows slimbrows tor tor zen zen gecko fork basilisk k-meleon pale moon seamonkey waterfox fork basilisk basilisk k-meleon k-meleon pale moon pale moon seamonkey seamonkey waterfox waterfox webkit safari safari gnome web gnome web icab icab orion orion engin 360 360 duckduckgo duckduckgo konqueror konqueror lunascap lunascap netfront netfront qutebrows qutebrows eww eww flow flow ladybird ladybird link link lynx lynx netsurf netsurf opera mini opera mini w3m w3m blink beaker beaker citrio citrio flock flock redcor redcor rockmelt rockmelt salamweb salamweb sputnik sputnik torch torch gecko beonex beonex camino camino classilla classilla conkeror conkeror firefox lite firefox lite galeon galeon ghostzilla ghostzilla icedragon icedragon kazehakas kazehakas kylo kylo lotu lotu microb microb minimo minimo mozilla suit mozilla suit piratebrows piratebrows pogo pogo stratum stratum swiftfox swiftfox swiftweasel swiftweasel tenfourfox tenfourfox timberwolf timberwolf xb xb mshtml internet explor internet explor aol aol deepnet deepnet greenbrows greenbrows mediabrows mediabrows neoplanet neoplanet netcaptor netcaptor spacetim spacetim zac zac webkit arora arora bolt bolt dolphin dolphin fluid fluid googl tv googl tv iri iri mercuri mercuri nokia symbian nokia symbian omniweb omniweb opera coast opera coast origyn origyn qtweb qtweb shiira shiira steel steel surf surf uzbl uzbl webposit webposit xombrero xombrero abaco abaco amaya amaya arachn arachn arena arena blazer blazer cake cake charon charon cm cm deepfish deepfish dillo dillo edg legaci edg legaci elink elink gazel gazel hotjava hotjava ibm home page reader ibm home page reader ibm webexplor ibm webexplor ibrows ibrows internet explor mac internet explor mac kidzui kidzui line mode line mode mosaic mosaic msn tv msn tv netposit netposit netscap netscap skweezer skweezer skyfir skyfir thunderhawk thunderhawk vision vision winwap winwap worldwideweb worldwideweb categori categori comparison comparison list list v v e e world wide web consortium recommend activitypub activitypub activ stream activ stream aria aria canon xml canon xml cdf cdf cs anim flexbox grid cs anim anim flexbox flexbox grid grid dom dom exi exi emotionml emotionml geoloc api geoloc api html html5 html html5 html5 indexeddb indexeddb json-ld json-ld link data notif link data notif mathml mathml micropub micropub owl owl pl pl rdf schema rdfa rdf schema schema rdfa rdfa sisr sisr sko sko smil smil soap soap srg srg sri sri ssml ssml svg filter effect svg filter effect filter effect scxml scxml shacl shacl sparql sparql time text time text voicexml voicexml webassembl webassembl wot td wot td td web storag web storag wsdl wsdl webment webment websub websub webvtt webvtt woff woff xhtml +rdfa xhtml +rdfa +rdfa xml base encrypt event inform set namespac schema signatur xform xinclud xlink xop xpath 2.0 3.x xpointer xproc xqueri xsl xsl-fo xslt element xml base base encrypt encrypt event event inform set inform set namespac namespac schema schema signatur signatur xform xform xinclud xinclud xlink xlink xop xop xpath 2.0 3.x xpath 2.0 2.0 3.x 3.x xpointer xpointer xproc xproc xqueri xqueri xsl xsl xsl-fo xsl-fo xslt element xslt element element indieauth indieauth xade xade xbl xbl xhtml+smil xhtml+smil xup xup work draft ccxml ccxml curi curi eme eme inkml inkml mse mse rif rif smil timesheet smil timesheet sxbl sxbl webgpu webgpu webxr webxr xfdl xfdl xframe xframe xmlhttprequest xmlhttprequest web content access guidelin web content access guidelin markup valid servic markup valid servic web access initi web access initi web compon web compon c-html c-html hdml hdml jsss jsss pgml pgml vml vml webplatform webplatform p3p p3p xhtml+mathml+svg xhtml+mathml+svg world wide web foundat world wide web foundat whatwg whatwg ab ab board board tag tag cs cs svg svg webassembl webassembl webauthn webauthn web advertis bg web advertis bg webassembl cg webassembl devic descript ddwg devic descript html html multimod interact activ mmi multimod interact activ cern httpd cern httpd libwww libwww browser line mode 1990– line mode 1990– arena 1993–98 arena 1993–98 agora 1994–97 agora 1994–97 argo 1994–97 argo 1994–97 amaya browser/editor 1996–2012 amaya browser/editor 1996–2012 intern world wide web confer iw3c steer committe iw3c2 first confer `` www1 '' 1994 intern world wide web confer steer committe iw3c2 steer committe first confer `` www1 '' 1994 first confer v v e e document markup languag offic suit compound document format compound document format ooxml spreadsheetml presentationml wordprocessingml ooxml spreadsheetml spreadsheetml presentationml presentationml wordprocessingml wordprocessingml odf odf uof uof html html xhtml xhtml mathml mathml rtf rtf tex tex latex latex markdown markdown amigaguid amigaguid asciidoc asciidoc bbcode bbcode cml cml c-html c-html context context crossmark crossmark dita dita docbook docbook ead ead enrich text enrich text fhtml fhtml gml gml guideml guideml hdml hdml hytim hytim ipf ipf lilypond lilypond linuxdoc linuxdoc lout mif mif maml maml mei mei musicxml musicxml omdoc omdoc openmath openmath org-mod org-mod pod pod restructuredtext restructuredtext rtml rtml rft rft s1000d s1000d setext setext tei tei texinfo texinfo troff troff wikitext wikitext wml wml waptv waptv xaml xaml list document markup languag v v e e iso standard number iso standard iso roman iec standard 1 1 2 2 3 3 4 4 6 6 7 7 9 9 16 16 17 17 31 -0 -1 -3 -4 -5 -6 -7 -8 -9 -10 -11 -12 -13 31 -0 -0 -1 -1 -3 -3 -4 -4 -5 -5 -6 -6 -7 -7 -8 -8 -9 -9 -10 -10 -11 -11 -12 -12 -13 -13 68-1 68-1 128 128 216 216 217 217 226 226 228 228 233 233 259 259 261 261 262 262 302 302 306 306 361 361 500 500 518 518 519 519 639 -1 -2 -3 -5 -6 639 -1 -1 -2 -2 -3 -3 -5 -5 -6 -6 646 646 657 657 668 668 690 690 704 704 732 732 764 764 838 838 843 843 860 860 898 898 965 965 999 999 1000 1000 1004 1004 1007 1007 1073-1 1073-1 1073-2 1073-2 1155 1155 1413 1413 1538 1538 1629 1629 1745 1745 1989 1989 2014 2014 2015 2015 2022 2022 2033 2033 2047 2047 2108 2108 2145 2145 2146 2146 2240 2240 2281 2281 2533 2533 2709 2709 2711 2711 2720 2720 2788 2788 2848 2848 2852 2852 2921 2921 3029 3029 3103 3103 3166 -1 -2 -3 3166 -1 -1 -2 -2 -3 -3 3297 3297 3307 3307 3601 3601 3602 3602 3864 3864 3901 3901 3950 3950 3977 3977 4031 4031 4157 4157 4165 4165 4217 4217 4909 4909 5218 5218 5426 5426 5427 5427 5428 5428 5725 5725 5775 5775 5776 5776 5800 5800 5807 5807 5964 5964 6166 6166 6344 6344 6346 6346 6373 6373 6385 6385 6425 6425 6429 6429 6438 6438 6523 6523 6709 6709 6943 6943 7001 7001 7002 7002 7010 7010 7027 7027 7064 7064 7098 7098 7185 7185 7200 7200 7498 -1 7498 -1 -1 7637 7637 7736 7736 7810 7810 7811 7811 7812 7812 7813 7813 7816 7816 7942 7942 8000 8000 8093 8093 8178 8178 8217 8217 8373 8373 8501-1 8501-1 8571 8571 8583 8583 8601 8601 8613 8613 8632 8632 8651 8651 8652 8652 8691 8691 8805/8806 8805/8806 8807 8807 8820-5 8820-5 8859 -1 -2 -3 -4 -5 -6 -7 -8 -8-i -9 -10 -11 -12 -13 -14 -15 -16 8859 -1 -1 -2 -2 -3 -3 -4 -4 -5 -5 -6 -6 -7 -7 -8 -8 -8-i -8-i -9 -9 -10 -10 -11 -11 -12 -12 -13 -13 -14 -14 -15 -15 -16 -16 8879 8879 9000/9001 9000/9001 9036 9036 9075 9075 9126 9126 9141 9141 9227 9227 9241 9241 9293 9293 9314 9314 9362 9362 9407 9407 9496 9496 9506 9506 9529 9529 9564 9564 9592/9593 9592/9593 9594 9594 9660 9660 9797-1 9797-1 9897 9897 9899 9899 9945 9945 9984 9984 9985 9985 9995 9995 10006 10006 10007 10007 10116 10116 10118-3 10118-3 10160 10160 10161 10161 10165 10165 10179 10179 10206 10206 10218 10218 10279 10279 10303 -11 -21 -22 -28 -238 10303 -11 -11 -21 -21 -22 -22 -28 -28 -238 -238 10383 10383 10585 10585 10589 10589 10628 10628 10646 10646 10664 10664 10746 10746 10861 10861 10957 10957 10962 10962 10967 10967 11073 11073 11170 11170 11172 11172 11179 11179 11404 11404 11544 11544 11783 11783 11784 11784 11785 11785 11801 11801 11889 11889 11898 11898 11940 -2 11940 -2 11941 11941 11941 tr 11941 tr 11992 11992 12006 12006 12052 12052 12182 12182 12207 12207 12234-2 12234-2 12620 12620 13211 -1 -2 13211 -1 -1 -2 -2 13216 13216 13250 13250 13399 13399 13406-2 13406-2 13450 13450 13485 13485 13490 13490 13567 13567 13568 13568 13584 13584 13616 13616 13816 13816 13818 13818 14000 14000 14031 14031 14224 14224 14289 14289 14396 14396 14443 14443 14496 -2 -3 -6 -10 -11 -12 -14 -17 -20 14496 -2 -2 -3 -3 -6 -6 -10 -10 -11 -11 -12 -12 -14 -14 -17 -17 -20 -20 14617 14617 14644 14644 14649 14649 14651 14651 14698 14698 14764 14764 14882 14882 14971 14971 15022 15022 15189 15189 15288 15288 15291 15291 15398 15398 15408 15408 15444 -3 -9 15444 -3 -3 -9 -9 15445 15445 15438 15438 15504 15504 15511 15511 15686 15686 15693 15693 15706 -2 15706 -2 -2 15707 15707 15897 15897 15919 15919 15924 15924 15926 15926 15926 wip 15926 wip 15930 15930 15938 15938 16023 16023 16262 16262 16355-1 16355-1 16485 16485 16612-2 16612-2 16750 16750 16949 t 16949 t 17024 17024 17025 17025 17100 17100 17203 17203 17369 17369 17442 17442 17506 17506 17799 17799 18004 18004 18014 18014 18181 18181 18245 18245 18629 18629 18916 18916 19005 19005 19011 19011 19092 -1 -2 19092 -1 -1 -2 -2 19114 19114 19115 19115 19125 19125 19136 19136 19407 19407 19439 19439 19500 19500 19501 19501 19502 19502 19503 19503 19505 19505 19506 19506 19507 19507 19508 19508 19509 19509 19510 19510 19600 19600 19752 19752 19757 19757 19770 19770 19775-1 19775-1 19794-5 19794-5 19831 19831 20000 20000 20022 20022 20121 20121 20400 20400 20802 20802 20830 20830 21000 21000 21001 21001 21047 21047 21122 21122 21500 21500 21827 21827 22000 22000 22275 22275 22300 22300 22301 22301 22395 22395 22537 22537 23000 23000 23003 23003 23008 23008 23009 23009 23090-3 23090-3 23092 23092 23094-1 23094-1 23094-2 23094-2 23270 23270 23271 23271 23360 23360 23941 23941 24517 24517 24613 24613 24617 24617 24707 24707 24728 24728 25178 25178 25964 25964 26000 26000 26262 26262 26300 26300 26324 26324 27000 seri 27000 seri 27000 27000 27001 27001 27002 27002 27005 27005 27006 27006 27729 27729 28000 28000 29110 29148 29148 29199-2 29199-2 29500 29500 30170 30170 31000 31000 32000 32000 37001 37001 38500 38500 39075 39075 40500 40500 42010 42010 45001 45001 50001 50001 55000 55000 56000 56000 80000 80000 categori categori v v e e iec standard 60027 60027 60034 60034 60038 60038 60062 60062 60063 60063 60068 60068 60112 60112 60228 60228 60269 60269 60297 60297 60309 60309 60320 60320 60364 60364 60446 60446 60559 60559 60601 60601 60870 60870-5 60870-6 60870 60870-5 60870-5 60870-6 60870-6 60906-1 60906-1 60908 60908 60929 60929 60958 60958 60980-344 60980-344 61030 61030 61131 61131-3 61131-9 61131 61131-3 61131-3 61131-9 61131-9 61158 61158 61162 61162 61334 61334 61355 61355 61360 61360 61400 61400 61499 61499 61508 61508 61511 61511 61784 61784 61850 61850 61851 61851 61883 61883 61960 61960 61968 61968 61970 61970 62014-4 62014-4 62026 62026 62056 62056 62061 62061 62196 62196 62262 62262 62264 62264 62304 62304 62325 62325 62351 62351 62365 62365 62366 62366 62379 62379 62386 62386 62455 62455 62680 62680 62682 62682 62700 62700 63110 63110 63119 63119 63382 63382 646 646 1989 1989 2022 2022 4909 4909 5218 5218 6429 6429 6523 6523 7810 7810 7811 7811 7812 7812 7813 7813 7816 7816 7942 7942 8613 8613 8632 8632 8652 8652 8859 8859 9126 9126 9293 9293 9496 9496 9529 9529 9592 9592 9593 9593 9899 9899 9945 9945 9995 9995 10021 10021 10116 10116 10165 10165 10179 10179 10279 10279 10646 10646 10967 10967 11172 11172 11179 11179 11404 11404 11544 11544 11801 11801 12207 12207 13250 13250 13346 13346 13522-5 13522-5 13568 13568 13816 13816 13818 13818 14443 14443 14496 14496 14651 14651 14882 14882 15288 15288 15291 15291 15408 15408 15444 15444 15445 15445 15504 15504 15511 15511 15693 15693 15897 15897 15938 15938 16262 16262 16485 16485 17024 17024 17025 17025 18004 18004 18014 18014 18181 18181 19752 19752 19757 19757 19770 19770 19788 19788 20000 20000 20802 20802 21000 21000 21827 21827 22275 22275 22537 22537 23000 23000 23003 23003 23008 23008 23270 23270 23360 23360 24707 24707 24727 24727 24744 24744 24752 24752 26300 26300 27000 27000 27000-seri 27000-seri 27002 27002 27040 27040 29110 29119 29119 33001 33001 38500 38500 39075 39075 42010 42010 80000 80000 81346 81346 intern electrotechn commiss intern electrotechn commiss author control databas germani germani germani unit state unit state unit state unit state franc franc franc bnf data bnf data bnf data czech republ czech republ czech republ czech republ spain spain spain israel israel israel elmcip elmcip elmcip portal portal comput program comput program http //en.wikipedia.org/w/index.php title=html oldid=1268733701 categori html html computer-rel introduct 1990 computer-rel introduct 1990 markup languag markup languag open format open format technic commun technic commun world wide web consortium standard world wide web consortium standard sgml sgml cs1 error period ignor cs1 error period ignor webarch templat wayback link webarch templat wayback link articl short descript articl short descript short descript differ wikidata short descript differ wikidata wikipedia page semi-protect vandal wikipedia page semi-protect vandal articl contain potenti date statement 1997 articl contain potenti date statement 1997 articl contain potenti date statement articl contain potenti date statement articl contain potenti date statement 1996 articl contain potenti date statement 1996 articl lack reliabl refer articl lack reliabl refer articl lack reliabl refer februari 2019 articl lack reliabl refer februari 2019 articl vagu ambigu time articl vagu ambigu time vagu ambigu time march 2022 vagu ambigu time march 2022 articl expand januari 2021 articl expand januari 2021 articl specif mark weasel-word phrase articl specif mark weasel-word phrase articl specif mark weasel-word phrase june 2020 articl specif mark weasel-word phrase june 2020 page use sister project link wikidata namespac mismatch page use sister project link wikidata namespac mismatch page use sister project link hidden wikidata page use sister project link hidden wikidata articl exampl code articl exampl code page last edit 11 januari 2025 07:39 utc utc text avail creativ common attribution-sharealik 4.0 licens addit term may appli use site agre term use privaci polici wikipedia® regist trademark wikimedia foundat inc. non-profit organ creativ common attribution-sharealik 4.0 licens term use privaci polici wikimedia foundat inc. privaci polici privaci polici wikipedia wikipedia disclaim disclaim contact wikipedia contact wikipedia code conduct code conduct develop develop statist statist cooki statement cooki statement mobil view mobil view",
    "https://en.wikipedia.org/wiki/C%2B%2B": "jump content main menu main page main page main page content content content current event current event current event random articl random articl random articl wikipedia wikipedia wikipedia contact u contact u contact u help help help learn edit learn edit learn edit commun portal commun portal commun portal recent chang recent chang recent chang upload file upload file upload file search search appear donat donat donat creat account creat account creat account log log log person tool donat donat donat creat account creat account creat account log log log learn learn contribut contribut contribut talk talk talk content top top 1 histori toggl histori subsect 1.1 etymolog 1.2 philosophi 1.3 standard 1 histori 1 histori toggl histori subsect 1.1 etymolog 1.1 etymolog 1.1 etymolog 1.2 philosophi 1.2 philosophi 1.2 philosophi 1.3 standard 1.3 standard 1.3 standard 2 languag toggl languag subsect 2.1 object storag 2.1.1 static storag durat object 2.1.2 thread storag durat object 2.1.3 automat storag durat object 2.1.4 dynam storag durat object 2.2 templat 2.3 object 2.3.1 encapsul 2.3.2 inherit 2.4 oper oper overload 2.5 polymorph 2.5.1 static polymorph 2.5.2 dynam polymorph 2.5.2.1 inherit 2.5.2.2 virtual member function 2.6 lambda express 2.7 except handl 2.8 enumer type 2 languag 2 languag toggl languag subsect 2.1 object storag 2.1.1 static storag durat object 2.1.2 thread storag durat object 2.1.3 automat storag durat object 2.1.4 dynam storag durat object 2.1 object storag 2.1 object storag 2.1.1 static storag durat object 2.1.1 static storag durat object 2.1.1 static storag durat object 2.1.2 thread storag durat object 2.1.2 thread storag durat object 2.1.2 thread storag durat object 2.1.3 automat storag durat object 2.1.3 automat storag durat object 2.1.3 automat storag durat object 2.1.4 dynam storag durat object 2.1.4 dynam storag durat object 2.1.4 dynam storag durat object 2.2 templat 2.2 templat 2.2 templat 2.3 object 2.3.1 encapsul 2.3.2 inherit 2.3 object 2.3 object 2.3.1 encapsul 2.3.1 encapsul 2.3.1 encapsul 2.3.2 inherit 2.3.2 inherit 2.3.2 inherit 2.4 oper oper overload 2.4 oper oper overload 2.4 oper oper overload 2.5 polymorph 2.5.1 static polymorph 2.5.2 dynam polymorph 2.5.2.1 inherit 2.5.2.2 virtual member function 2.5 polymorph 2.5 polymorph 2.5.1 static polymorph 2.5.1 static polymorph 2.5.1 static polymorph 2.5.2 dynam polymorph 2.5.2.1 inherit 2.5.2.2 virtual member function 2.5.2 dynam polymorph 2.5.2 dynam polymorph 2.5.2.1 inherit 2.5.2.1 inherit 2.5.2.1 inherit 2.5.2.2 virtual member function 2.5.2.2 virtual member function 2.5.2.2 virtual member function 2.6 lambda express 2.6 lambda express 2.6 lambda express 2.7 except handl 2.7 except handl 2.7 except handl 2.8 enumer type 2.8 enumer type 2.8 enumer type 3 standard librari 3 standard librari 3 standard librari 4 c++ core guidelin 4 c++ core guidelin 4 c++ core guidelin 5 compat toggl compat subsect 5.1 c 5 compat 5 compat toggl compat subsect 5.1 c 5.1 c 5.1 c 6 see also 6 see also 6 see also 7 footnot 7 footnot 7 footnot 8 refer 8 refer 8 refer 9 read 9 read 9 read 10 extern link 10 extern link 10 extern link toggl tabl content c++ c++ 109 languag afrikaan afrikaan afrikaan العربية العربية العربية aragoné aragoné aragoné অসমীয়া অসমীয়া অসমীয়া asturianu asturianu asturianu azərbaycanca azərbaycanca azərbaycanca تۆرکجه تۆرکجه تۆرکجه বাংলা বাংলা বাংলা 閩南語 bân-lâm-gú 閩南語 bân-lâm-gú 閩南語 bân-lâm-gú беларуская беларуская беларуская беларуская тарашкевіца беларуская тарашкевіца беларуская тарашкевіца български български български བོད་ཡིག བོད་ཡིག བོད་ཡིག bosanski bosanski bosanski brezhoneg brezhoneg brezhoneg català català català чӑвашла чӑвашла чӑвашла čeština čeština čeština dansk dansk dansk deutsch deutsch deutsch eesti eesti eesti ελληνικά ελληνικά ελληνικά español español español esperanto esperanto esperanto euskara euskara euskara فارسی فارسی فارسی françai françai françai gaeilg gaeilg gaeilg galego galego galego 贛語 贛語 贛語 ગુજરાતી ગુજરાતી ગુજરાતી 한국어 한국어 한국어 hausa hausa hausa հայերեն հայերեն հայերեն हिन्दी हिन्दी हिन्दी hrvatski hrvatski hrvatski ido ido ido bahasa indonesia bahasa indonesia bahasa indonesia interlingua interlingua interlingua íslenska íslenska íslenska italiano italiano italiano עברית עברית עברית jawa jawa jawa ಕನ್ನಡ ಕನ್ನಡ ಕನ್ನಡ ქართული ქართული ქართული қазақша қазақша қазақша kiswahili kiswahili kiswahili кыргызча кыргызча кыргызча latina latina latina latviešu latviešu latviešu lëtzebuergesch lëtzebuergesch lëtzebuergesch lietuvių lietuvių lietuvių lombard lombard lombard magyar magyar magyar македонски македонски македонски മലയാളം മലയാളം മലയാളം मराठी मराठी मराठी مصرى مصرى مصرى bahasa melayu bahasa melayu bahasa melayu minangkabau minangkabau minangkabau монгол монгол монгол မြန်မာဘာသာ မြန်မာဘာသာ မြန်မာဘာသာ na vosa vakav na vosa vakav na vosa vakav nederland nederland nederland नेपाल भाषा नेपाल भाषा नेपाल भाषा 日本語 日本語 日本語 norsk bokmål norsk bokmål norsk bokmål norsk nynorsk norsk nynorsk norsk nynorsk occitan occitan occitan oʻzbekcha ўзбекча oʻzbekcha ўзбекча oʻzbekcha ўзбекча ਪੰਜਾਬੀ ਪੰਜਾਬੀ ਪੰਜਾਬੀ pälzisch pälzisch pälzisch پنجابی پنجابی پنجابی polski polski polski portuguê portuguê portuguê qaraqalpaqsha qaraqalpaqsha qaraqalpaqsha română română română runa simi runa simi runa simi русский русский русский саха тыла саха тыла саха тыла shqip shqip shqip සිංහල සිංහල සිංහල simpl english simpl english simpl english slovenčina slovenčina slovenčina slovenščina slovenščina slovenščina کوردی کوردی کوردی српски srpski српски srpski српски srpski srpskohrvatski српскохрватски srpskohrvatski српскохрватски srpskohrvatski српскохрватски suomi suomi suomi svenska svenska svenska தமிழ் தமிழ் தமிழ் татарча tatarça татарча tatarça татарча tatarça తెలుగు తెలుగు తెలుగు ไทย ไทย ไทย тоҷикӣ тоҷикӣ тоҷикӣ türkçe türkçe türkçe türkmenç türkmenç türkmenç basa ugi basa ugi basa ugi українська українська українська اردو اردو اردو tiếng việt tiếng việt tiếng việt 文言 文言 文言 winaray winaray winaray 吴语 吴语 吴语 ייִדיש ייִדיש ייִדיש yorùbá yorùbá yorùbá 粵語 粵語 粵語 žemaitėška žemaitėška žemaitėška 中文 中文 中文 edit link edit link articl articl articl talk talk talk english read read read edit edit edit view histori view histori view histori tool read read read edit edit edit view histori view histori view histori link link link relat chang relat chang relat chang upload file upload file upload file special page special page special page perman link perman link perman link page inform page inform page inform cite page cite page cite page get shorten url get shorten url get shorten url download qr code download qr code download qr code download pdf download pdf download pdf printabl version printabl version printabl version wikimedia common wikimedia common wikimedia common wikibook wikibook wikibook wikiquot wikiquot wikiquot wikivers wikivers wikivers wikidata item wikidata item wikidata item cxx paradigm multi-paradigm procedur imper function object-ori gener modular c design bjarn stroustrup develop iso/iec jtc 1 joint technic committe 1 sc 22 subcommitte 22 40 year ago 1985 1985 stabl releas c++23 2 month ago 2024-10-19 2024-10-19 preview releas c++26 2 month ago 2024-10-16 2024-10-16 type disciplin static strong nomin partial infer o cross-platform filenam extens .c++ .h++ 1 isocpp.org isocpp.org implement gcc llvm clang microsoft visual c++ embarcadero c++builder intel c++ compil ibm xl c++ edg gcc llvm clang microsoft visual c++ embarcadero c++builder intel c++ compil ibm xl c++ edg ada algol 68 2 bcpl 3 c clu 2 f 4 note 1 ml mesa 2 modula-2 2 simula smalltalk 2 ada 95 c 5 c99 carbon chapel 6 clojur 7 java 8 js++ 9 lua 10 nim 11 objective-c++ perl php python 12 rust 13 seed7 c++ program wikibook c++ program c++ /ˈsiː plʌ plʌs/ pronounc `` c plu plu '' sometim abbrevi cpp high-level general-purpos program languag creat danish comput scientist bjarn stroustrup first releas 1985 extens c program languag sinc expand significantli time 1997 updat c++ object-ori gener function featur addit facil low-level memori manipul system like microcomput make oper system like linux window usual implement compil languag mani vendor provid c++ compil includ free softwar foundat llvm microsoft intel embarcadero oracl ibm 14 /ˈsiː plʌ plʌs/ /ˈsiː plʌ plʌs/ /ˈsiː plʌ plʌs/ ˈsiː ˈ iː plʌ p l ʌ plʌ p l ʌ high-level general-purpos program languag bjarn stroustrup c program languag updat object-ori gener function low-level memori microcomput linux window compil languag c++ compil free softwar foundat llvm microsoft intel embarcadero oracl ibm 14 c++ design system program embed resource-constrain softwar larg system mind perform effici flexibl use design highlight 15 c++ also found use mani context key strength softwar infrastructur resource-constrain applic 15 includ desktop applic video game server e.g. e-commerc web search databas performance-crit applic e.g. telephon switch space probe 16 system program embed perform 15 15 desktop applic video game server e-commerc web search databas telephon switch space probe 16 c++ standard intern organ standard iso latest standard version ratifi publish iso octob 2024 iso/iec 14882:2024 inform known c++23 17 c++ program languag initi standard 1998 iso/iec 14882:1998 amend c++03 c++11 c++14 c++17 c++20 standard current c++23 standard supersed new featur enlarg standard librari initi standard 1998 c++ develop stroustrup bell lab sinc 1979 extens c languag want effici flexibl languag similar c also provid high-level featur program organ 18 sinc 2012 c++ three-year releas schedul 19 c++26 next plan standard 20 intern organ standard c++23 17 c++03 c++11 c++14 c++17 c++20 c++23 c++23 standard librari bell lab high-level featur 18 19 c++26 20 despit widespread adopt notabl programm critic c++ languag includ linu torvald 21 richard stallman 22 joshua bloch ken thompson 23 24 25 donald knuth 26 27 critic c++ languag linu torvald 21 richard stallman 22 joshua bloch ken thompson 23 24 25 donald knuth 26 27 histori edit edit edit 2000 1979 bjarn stroustrup danish comput scientist began work `` c class '' predecessor c++ 28 motiv creat new languag origin stroustrup 's experi program phd thesi stroustrup found simula featur help larg softwar develop languag slow practic use bcpl fast low-level suitabl larg softwar develop stroustrup start work bell lab problem analyz unix kernel respect distribut comput rememb phd experi stroustrup set enhanc c languag simula-lik featur 29 c chosen general-purpos fast portabl wide use addit c simula 's influenc languag influenc new languag includ algol 68 ada clu ml citat need bjarn stroustrup comput scientist c class c class class 28 simula bcpl bell lab unix kernel distribut comput c simula 29 algol 68 ada clu ml citat need citat need initi stroustrup 's `` c class '' ad featur c compil cpre includ class deriv class strong type inlin default argument 30 class deriv class strong type inlin default argument 30 1982 stroustrup start develop successor c class name `` c++ '' ++ increment oper c go sever name new featur ad includ virtual function function name oper overload refer constant type-saf free-stor memori alloc new/delet improv type check bcpl-style single-lin comment two forward slash // furthermor stroustrup develop new standalon compil c++ cfront ++ increment oper virtual function oper overload refer // cfront 1984 stroustrup implement first stream input/output librari idea provid output oper rather name output function suggest doug mcilroy 2 previous suggest unix pipe doug mcilroy 2 unix pipe 1985 first edit c++ program languag releas becam definit refer languag yet offici standard 31 first commerci implement c++ releas octob year 28 c++ program languag 31 28 1989 c++ 2.0 releas follow updat second edit c++ program languag 1991 32 new featur 2.0 includ multipl inherit abstract class static member function const member function protect member 1990 annot c++ refer manual publish work becam basi futur standard later featur addit includ templat except namespac new cast boolean type 32 const member function templat except namespac cast boolean type 1998 c++98 releas standard languag minor updat c++03 releas 2003 c++03 c++98 c++ evolv rel slowli 2011 c++11 standard releas ad numer new featur enlarg standard librari provid facil c++ programm minor c++14 updat releas decemb 2014 variou new addit introduc c++17 33 becom final februari 2020 34 draft c++20 standard approv 4 septemb 2020 offici publish 15 decemb 2020 35 36 c++11 c++14 c++14 c++17 33 34 35 36 januari 3 2018 stroustrup announc 2018 winner charl stark draper prize engin `` conceptu develop c++ program languag '' 37 charl stark draper prize 37 decemb 2022 c++ rank third tiob index surpass java first time histori index novemb 2024 updat languag rank second python java third 38 tiob index java updat python 38 etymolog edit edit edit accord stroustrup `` name signifi evolutionari natur chang c. '' 39 name credit rick mascitti mid-1983 30 first use decemb 1983 mascitti question inform 1992 name indic given tongue-in-cheek spirit name come c 's ++ oper increment valu variabl common name convent use `` '' indic enhanc comput program 39 30 tongue-in-cheek ++ oper increment valu variabl name convent c++ 's develop period languag refer `` new c '' `` c class '' 30 40 acquir final name 30 40 philosophi edit edit edit throughout c++ 's life develop evolut guid set principl 29 29 must driven actual problem featur immedi use real world program everi featur implement reason obviou way programm free pick program style style fulli support c++ allow use featur import prevent everi possibl misus c++ provid facil organis program separ well-defin part provid facil combin separ develop part implicit violat type system allow explicit violat explicitli request programm type system user-cr type need support perform built-in type unus featur neg impact creat execut e.g lower perform languag beneath c++ except assembl languag assembl languag c++ work alongsid exist program languag rather foster separ incompat program environ program languag program environ programm 's intent unknown allow programm specifi provid manual control standard edit edit edit c++03 c++11 c++14 c++17 c++20 c++23 c++26 41 42 c++03 43 c++11 44 c++14 45 c++17 46 c++20 17 c++23 c++26 c++ standard iso work group known jtc1/sc22/wg21 far publish seven revis c++ standard current work next revis c++26 iso jtc1/sc22/wg21 c++26 1998 iso work group standard c++ first time iso/iec 14882:1998 inform known c++98 2003 publish new version c++ standard call iso/iec 14882:2003 fix problem identifi c++98 next major revis standard inform refer `` c++0x '' releas 2011 47 c++11 14882:2011 includ mani addit core languag standard librari 43 47 c++11 43 2014 c++14 also known c++1i releas small extens c++11 featur mainli bug fix small improv 48 draft intern standard ballot procedur complet mid-august 2014 49 c++14 48 49 c++14 major revis c++17 inform known c++1z complet iso c++ committe mid juli 2017 approv publish decemb 2017 50 c++17 50 part standard process iso also publish technic report specif technic report specif iso/iec tr 18015:2006 51 use c++ embed system perform implic c++ languag librari featur 51 iso/iec tr 19768:2007 52 also known c++ technic report 1 librari extens mostli integr c++11 52 c++ technic report 1 c++11 iso/iec tr 29124:2010 53 special mathemat function integr c++17 53 c++17 c++17 iso/iec tr 24733:2011 54 decim floating-point arithmet 54 decim floating-point iso/iec t 18822:2015 55 standard filesystem librari integr c++17 55 c++17 iso/iec t 19570:2015 56 parallel version standard librari algorithm integr c++17 56 parallel c++17 iso/iec t 19841:2015 57 softwar transact memori 57 transact memori iso/iec t 19568:2015 58 new set librari extens alreadi integr c++17 58 c++17 iso/iec t 19217:2015 59 c++ concept integr c++20 59 concept c++20 iso/iec t 19571:2016 60 librari extens concurr alreadi integr c++20 60 c++20 iso/iec t 19568:2017 61 new set general-purpos librari extens 61 iso/iec t 21425:2017 62 librari extens rang integr c++20 62 c++20 iso/iec t 22277:2017 63 coroutin integr c++20 63 c++20 iso/iec t 19216:2018 64 network librari 64 iso/iec t 21544:2018 65 modul integr c++20 65 c++20 iso/iec t 19570:2018 66 new set librari extens parallel 66 iso/iec t 23619:2021 67 new extens reflect program reflect 67 reflect program technic specif develop pend approv includ new set concurr extens languag edit edit edit c++ languag two main compon direct map hardwar featur provid primarili c subset zero-overhead abstract base map stroustrup describ c++ `` light-weight abstract program languag design build use effici eleg abstract '' 15 `` offer hardwar access abstract basi c++ effici distinguish languag `` 68 15 68 c++ inherit c 's syntax hello world program conform c standard also valid c++ hello world program follow bjarn stroustrup 's version hello world program use c++ standard librari stream facil write messag standard output 69 70 note 2 c 's syntax hello world program conform c standard hello world program c++ standard librari standard output 69 70 note 2 includ iostream int main std cout `` hello world \\n `` object storag edit edit edit c c++ support four type memori manag static storag durat object thread storag durat object automat storag durat object dynam storag durat object 71 memori manag 71 static storag durat object edit edit edit static storag durat object creat main enter see except destroy revers order creation main exit exact order creation specifi standard though rule defin allow implement freedom organ implement formal object type lifespan `` shall last durat program '' 72 72 static storag durat object initi two phase first `` static initi '' perform static initi perform `` dynam initi '' perform static initi object first initi zero object constant initi phase initi constant express i.e variabl initi liter constexpr though specifi standard static initi phase complet compil time save data partit execut dynam initi involv object initi done via constructor function call unless function mark constexpr c++11 dynam initi order defin order declar within compil unit i.e file guarante provid order initi compil unit thread storag durat object edit edit edit variabl type similar static storag durat object main differ creation time thread creation destruct done thread join 73 73 automat storag durat object edit edit edit common variabl type c++ local variabl insid function block temporari variabl 74 common featur automat variabl lifetim limit scope variabl creat potenti initi point declar see detail destroy revers order creation scope left implement alloc stack local variabl function 74 stack local variabl creat point execut pas declar point variabl constructor initi use defin initi state object local variabl destroy local block function declar close c++ destructor local variabl call end object lifetim allow disciplin automat resourc manag term raii wide use c++ raii member variabl creat parent object creat array member initi 0 last member array order member variabl destroy parent object destroy revers order creation i.e parent `` automat object '' destroy goe scope trigger destruct member temporari variabl creat result express evalu destroy statement contain express fulli evalu usual end statement dynam storag durat object edit edit edit new delet c++ object dynam lifespan creat directli call new destroy explicitli call delet 75 c++ also support malloc free c compat new delet use new return address alloc memori c++ core guidelin advis use new directli creat dynam object favor smart pointer make_uniqu singl ownership make_shar reference-count multipl ownership 76 introduc c++11 new delet 75 new delet new new make_uniqu make_shar 76 templat edit edit edit templat metaprogram gener program c++ templat enabl gener program c++ support function class alia variabl templat templat may parameter type compile-tim constant templat templat implement instanti compile-tim instanti templat compil substitut specif argument templat 's paramet gener concret function class instanc substitut possibl elimin overload resolut polici describ phrase `` substitut failur error '' sfina templat power tool use gener program templat metaprogram code optim power impli cost templat use may increas object code size templat instanti produc copi templat code one set templat argument howev smaller amount code would gener code written hand 77 contrast run-tim gener seen languag e.g. java compile-tim type era singl templat bodi preserv c++ templat gener program c++ substitut failur error gener program templat metaprogram object code 77 java templat differ macro compile-tim languag featur enabl condit compil templat restrict lexic substitut templat awar semant type system companion languag well compile-tim type definit perform high-level oper includ programmat flow control base evalu strictli type-check paramet macro capabl condit control compil base predetermin criterion instanti new type recurs perform type evalu effect limit pre-compil text-substitut text-inclusion/exclus word macro control compil flow base pre-defin symbol unlik templat independ instanti new symbol templat tool static polymorph see gener program macro polymorph gener program addit templat compile-tim mechan c++ turing-complet mean comput express comput program comput form templat metaprogram runtim turing-complet templat metaprogram summari templat compile-tim parameter function class written without knowledg specif argument use instanti instanti result code equival code written specif pas argument manner templat provid way decoupl gener broadli applic aspect function class encod templat specif aspect encod templat paramet without sacrif perform due abstract object edit edit edit c++ class c++ introduc object-ori program oop featur c. offer class provid four featur commonli present oop non-oop languag abstract encapsul inherit polymorph one distinguish featur c++ class compar class program languag support determinist destructor turn provid support resourc acquisit initi raii concept object-ori program class abstract encapsul inherit polymorph c++ destructor resourc acquisit initi encapsul edit edit edit encapsul hide inform ensur data structur oper use intend make usag model obviou develop c++ provid abil defin class function primari encapsul mechan within class member declar either public protect privat explicitli enforc encapsul public member class access function privat member access function member class function class explicitli grant access permiss class `` friend '' protect member access member class inherit class addit class friend encapsul object-ori principl ensur encapsul function access intern represent type c++ support principl via member function friend function enforc programm declar part represent type public allow make public entiti part represent type therefor c++ support object-ori program decomposit paradigm modular program modular program gener consid good practic make data privat protect make public function part minim interfac user class hide detail data implement allow design later fundament chang implement without chang interfac way 78 79 data 78 79 inherit edit edit edit inherit allow one data type acquir properti data type inherit base class may declar public protect privat access specifi determin whether unrel deriv class access inherit public protect member base class public inherit correspond usual meant `` inherit '' two form much less frequent use access specifi omit `` class '' inherit privat `` struct '' inherit publicli base class may declar virtual call virtual inherit virtual inherit ensur one instanc base class exist inherit graph avoid ambigu problem multipl inherit inherit base class virtual inherit multipl inherit c++ featur allow class deriv one base class allow elabor inherit relationship exampl `` fli cat '' class inherit `` cat '' `` fli mammal '' languag c java accomplish someth similar although limit allow inherit multipl interfac restrict number base class one interfac unlik class provid declar member function implement member data interfac c java defin c++ class contain pure virtual function often known abstract base class `` abc '' member function abstract base class normal explicitli defin deriv class inherit implicitli c++ virtual inherit exhibit ambigu resolut featur call domin multipl inherit c java interfac c++ abstract base class domin oper oper overload edit edit edit sizeof sizeof typeid typeid oper c c++ c++ provid 35 oper cover basic arithmet bit manipul indirect comparison logic oper other almost oper overload user-defin type notabl except member access condit oper rich set overload oper central make user-defin type c++ seem like built-in type overload overload oper also essenti part mani advanc c++ program techniqu smart pointer overload oper chang preced calcul involv oper chang number operand oper use operand may howev ignor oper though evalu prior execut overload `` '' `` || '' oper lose short-circuit evalu properti smart pointer short-circuit evalu polymorph edit edit edit polymorph comput scienc polymorph enabl one common interfac mani implement object act differ differ circumst polymorph c++ support sever kind static resolv compile-tim dynam resolv run-tim polymorph support languag featur describ compile-tim polymorph allow certain run-tim decis runtim polymorph typic incur perform penalti compile-tim run-tim polymorph compile-tim polymorph runtim polymorph static polymorph edit edit edit parametr polymorph ad hoc polymorph function overload allow program declar multipl function name differ argument i.e ad hoc polymorph function distinguish number type formal paramet thu function name refer differ function depend context use type return function use distinguish overload function differ return type would result compile-tim error messag function overload ad hoc polymorph formal paramet declar function programm specifi one paramet default valu allow paramet default option omit function call case default argument use function call fewer argument declar paramet explicit argument match paramet left-to-right order unmatch paramet end paramet list assign default argument mani case specifi default argument singl function declar prefer provid overload function definit differ number paramet default valu templat c++ provid sophist mechan write gener polymorph code i.e parametr polymorph particular curious recur templat pattern possibl implement form static polymorph close mimic syntax overrid virtual function c++ templat type-awar turing-complet also use let compil resolv recurs condit gener substanti program templat metaprogram contrari opinion templat code gener bulk code compil proper compil set 77 templat c++ parametr polymorph curious recur templat pattern turing-complet templat metaprogram 77 dynam polymorph edit edit edit inherit edit edit edit subtyp variabl pointer refer base class type c++ also refer object deriv class type allow array kind contain hold pointer object differ type refer directli held contain enabl dynam run-tim polymorph refer object behav differ depend actual deriv type c++ also provid dynamic_cast oper allow code safe attempt convers object via base reference/point deriv type downcast attempt necessari often one know deriv type referenc upcast convers gener type alway checked/perform compile-tim via static_cast ancestr class specifi deriv class 's interfac visibl caller dynamic_cast reli run-tim type inform rtti metadata program enabl differenti type relationship dynamic_cast pointer fail result nullptr constant wherea destin refer null cast throw except object known certain deriv type cast static_cast bypass rtti safe runtim type-check dynamic_cast use programm confid cast alway valid dynamic_cast static_cast dynamic_cast run-tim type inform dynamic_cast nullptr static_cast dynamic_cast virtual member function edit edit edit ordinarili function deriv class overrid function base class function call determin type object given function overridden exist differ number type paramet two definit function henc compil time may possibl determin type object therefor correct function call given base class pointer decis therefor put runtim call dynam dispatch virtual member function method 80 allow specif implement function call accord actual run-tim type object c++ implement commonli done use virtual function tabl object type known may bypass prepend fulli qualifi class name function call gener call virtual function resolv run time overrid dynam dispatch virtual member function 80 virtual function tabl fulli qualifi class name addit standard member function oper overload destructor virtual inexact rule base practic experi state function class virtual destructor well type object creation known compil time constructor extens copi constructor virtual nonetheless situat may aris copi object need creat pointer deriv object pas pointer base object case common solut creat clone similar virtual function creat return copi deriv class call clone member function also made `` pure virtual '' append 0 close parenthesi semicolon class contain pure virtual function call abstract class object creat abstract class deriv deriv class inherit virtual function pure must provid non-pur definit pure virtual function object deriv class creat program attempt creat object class pure virtual member function inherit pure virtual member function ill-form 0 lambda express edit edit edit c++ provid support anonym function also known lambda express follow form anonym function lambda express captur paramet return_typ function_bodi sinc c++20 keyword templat option templat paramet lambda express templat captur template_paramet paramet return_typ function_bodi lambda take paramet return type specifi use omit captur function_bodi return type lambda express automat infer possibl e.g int x int return x // infer int x int int return x // explicit captur list support definit closur lambda express defin standard syntact sugar unnam function object captur closur syntact sugar function object except handl edit edit edit except handl use commun exist runtim problem error detect issu handl 81 permit done uniform manner separ main code detect error 82 error occur except thrown rais caught nearest suitabl except handler except caus current scope exit also outer scope propag suitabl handler found call turn destructor object exit scope 83 time except present object carri data detect problem 84 81 82 83 84 c++ style guid googl 's 85 llvm 's 86 qt 's 87 forbid usag except 85 86 87 exception-caus code place insid tri block except handl separ catch block handler tri block multipl except handler visibl exampl 88 tri catch tri 88 includ iostream includ vector includ stdexcept int main tri std vector int vec 3 4 3 1 int vec 4 // throw except std :out_of_rang index vec 0-3 1-4 // except handler catch std :out_of_rang thrown vec.at 4 catch const std out_of_rang e std cerr `` access non-exist element `` e '\\n // catch standard librari except deriv std :except catch const std except e std cerr `` except thrown `` e '\\n // catch unrecognis except i.e n't deriv std :except catch ... std cerr `` fatal error \\n `` also possibl rais except purpos use throw keyword except handl usual way case except use due technic reason one exampl critic compon embed system everi oper must guarante complet within specifi amount time determin except tool exist determin maximum time requir except handl 89 throw 89 unlik signal handl handl function call point failur except handl exit current scope catch block enter may locat current function previou function call current stack signal handl enumer type edit edit edit enumer type § c++ edit edit c++ enumer type directli inherit c 's work mostli like except enumer real type c++ give ad compile-tim check also struct c++ enum keyword combin typedef instead name type enum name simpli name name simul c use typedef typedef enum value1 value2 name typedef typedef typedef enum value1 value2 name c++11 also provid second kind enumer call scope enumer type-saf enumer implicitli convert integ type among thing allow i/o stream defin enumer type anoth featur scope enumer enumer leak usag requir prefix name enumer e.g. color :red first enumer exampl unless use enum declar introduc c++20 use bring enumer current scope scope enumer specifi phrase enum class enum struct exampl c++11 c++20 enum class color red green blue underli type enumer implementation-defin integr type larg enough hold enumer valu smallest possibl type underli type specifi directli allow `` forward declar '' enumer enum class color long red green blue // must fit size memori layout type 'long enum class shape char // forward declar later valu defin n't fit 'char error standard librari edit edit edit c++ standard librari c++ standard consist two part core languag standard librari c++ programm expect latter everi major implement c++ includ aggreg type vector list map set queue stack array tupl algorithm find for_each binary_search random_shuffl etc input/output facil iostream read write consol file filesystem librari localis support smart pointer automat memori manag regular express support multi-thread librari atom support allow variabl read written one thread time without extern synchronis time util measur get current time etc system convert error report use c++ except c++ except random number gener slightli modifi version c standard librari make compli c++ type system standard vector algorithm for_each binary_search iostream smart pointer regular express multi-thread except random number gener c standard librari larg part c++ librari base standard templat librari stl use tool provid stl includ contain collect object vector list iter provid array-lik access contain algorithm perform oper search sort standard templat librari contain vector list iter algorithm furthermor multi map associ array multi set provid export compat interfac therefor use templat possibl write gener algorithm work contain sequenc defin iter c featur librari access use includ direct includ standard header c++ standard librari provid 105 standard header 27 deprec associ array featur librari includ direct standard header c++ standard librari standard incorpor stl origin design alexand stepanov experi gener algorithm contain mani year start c++ final found languag possibl creat gener algorithm e.g. stl sort perform even better exampl c standard librari qsort thank c++ featur like use inlin compile-tim bind instead function pointer standard refer `` stl '' mere part standard librari term still wide use distinguish rest standard librari input/output stream internation diagnost c librari subset etc. 90 alexand stepanov 90 c++ compil major one provid standards-conform implement c++ standard librari c++ core guidelin edit edit edit c++ core guidelin 91 initi led bjarn stroustrup inventor c++ herb sutter conven chair c++ iso work group help programm write 'modern c++ use best practic languag standard c++11 newer help develop compil static check tool creat rule catch bad program practic 91 main aim effici consist write type resourc safe c++ core guidelin announc 92 open keynot cppcon 2015 92 guidelin accompani guidelin support librari gsl 93 header librari type function implement core guidelin static checker tool enforc guidelin rule 94 93 94 compat edit edit edit give compil vendor greater freedom c++ standard committe decid dictat implement name mangl except handl implementation-specif featur downsid decis object code produc differ compil expect incompat howev attempt standard compil particular machin oper system exampl itanium c++ abi processor-independ despit name implement gcc clang 95 name mangl except handl object code compil oper system 95 c edit edit edit compat c c++ c++ often consid superset c strictli true 96 c code easili made compil correctli c++ differ caus valid c code invalid behav differ c++ exampl c allow implicit convers void pointer type c++ type safeti reason also c++ defin mani new keyword new class may use identifi exampl variabl name c program c 96 void new class incompat remov 1999 revis c standard c99 support c++ featur line comment // declar mix code hand c99 introduc number new featur c++ support incompat redund c++ variable-length array nativ complex-numb type howev std :complex class c++ standard librari provid similar function although code-compat design initi compound liter restrict keyword 97 c99-introduc featur includ subsequ version c++ standard c++11 redund 98 99 100 howev c++11 standard introduc new incompat disallow assign string liter charact pointer remain valid c. c99 // variable-length array std complex compound liter restrict 97 c++11 98 99 100 intermix c c++ code function declar definit call from/us c c++ must declar c linkag place within extern `` c '' ... block function may reli featur depend name mangl i.e. function overload extern `` c '' ... name mangl see also edit edit edit comput program portal comput program portal comput program portal carbon program languag carbon program languag comparison program languag comparison program languag list c++ compil list c++ compil outlin c++ outlin c++ categori c++ librari categori c++ librari footnot edit edit edit idea c++20 stackless coroutin idea c++20 stackless coroutin code copi directli bjarn stroustrup 's erratum page p. 633 address use '\\n rather std :endl also see write `` void main '' archiv 2 juli 2020 wayback machin explan implicit return 0 main function implicit return avail function code copi directli bjarn stroustrup 's erratum page p. 633 address use '\\n rather std :endl also see write `` void main '' archiv 2 juli 2020 wayback machin explan implicit return 0 main function implicit return avail function write `` void main '' archiv wayback machin refer edit edit edit `` overview modul c++ '' microsoft 24 april 2023 `` overview modul c++ '' microsoft 24 april 2023 `` overview modul c++ '' b c e f stroustrup bjarn 1996 `` histori c++ 1979-1991 '' histori program languag -- -ii acm pp 699–769 doi:10.1145/234286.1057836 b c e f b c e f stroustrup bjarn 1996 `` histori c++ 1979-1991 '' histori program languag -- -ii acm pp 699–769 doi:10.1145/234286.1057836 acm 699– doi 10.1145/234286.1057836 10.1145/234286.1057836 stroustrup bjarn 16 decemb 2021 `` c++20 reach aim c++ bjarn stroustrup cppcon 2021 '' cppcon archiv origin 30 decemb 2021 retriev 30 decemb 2021 stroustrup bjarn 16 decemb 2021 `` c++20 reach aim c++ bjarn stroustrup cppcon 2021 '' cppcon archiv origin 30 decemb 2021 retriev 30 decemb 2021 `` c++20 reach aim c++ bjarn stroustrup cppcon 2021 '' archiv retriev 30 decemb 2021 30 decemb stroustrup bjarn 12 june 2020 `` thrive crowd chang world c++ 2006–2020 '' proceed acm program languag 4 hopl associ comput machineri acm 1–168 doi:10.1145/3386320 issn 2475-1421 s2cid 219603741 stroustrup bjarn 12 june 2020 `` thrive crowd chang world c++ 2006–2020 '' proceed acm program languag 4 hopl associ comput machineri acm 1–168 doi:10.1145/3386320 issn 2475-1421 s2cid 219603741 `` thrive crowd chang world c++ 2006–2020 '' 1– doi 10.1145/3386320 10.1145/3386320 issn 2475-1421 s2cid 219603741 naugler david may 2007 `` c 2.0 c++ java programm confer workshop '' journal comput scienc colleg 22 5 although c strongli influenc java also strongli influenc c++ best view descend c++ java naugler david may 2007 `` c 2.0 c++ java programm confer workshop '' journal comput scienc colleg 22 5 although c strongli influenc java also strongli influenc c++ best view descend c++ java `` chapel spec acknowledg '' pdf cray inc. 1 octob 2015 archiv pdf origin 24 june 2018 retriev 14 januari 2016 `` chapel spec acknowledg '' pdf cray inc. 1 octob 2015 archiv pdf origin 24 june 2018 retriev 14 januari 2016 `` chapel spec acknowledg '' pdf archiv pdf retriev 14 januari 2016 14 januari fogu michael `` rich hickey q '' code quarterli archiv origin 11 januari 2017 retriev 11 januari 2017 fogu michael `` rich hickey q '' code quarterli archiv origin 11 januari 2017 retriev 11 januari 2017 `` rich hickey q '' origin retriev 11 januari 2017 11 januari harri h. chaudhari 28 juli 2014 `` crack java program interview 2000+ java interview que/an '' archiv origin 27 may 2021 retriev 29 may 2016 harri h. chaudhari 28 juli 2014 `` crack java program interview 2000+ java interview que/an '' archiv origin 27 may 2021 retriev 29 may 2016 `` crack java program interview 2000+ java interview que/an '' archiv retriev 29 may 2016 29 may roger poon 1 may 2017 `` scale js++ abstract perform readabl '' archiv origin 11 may 2020 retriev 21 april 2020 roger poon 1 may 2017 `` scale js++ abstract perform readabl '' archiv origin 11 may 2020 retriev 21 april 2020 `` scale js++ abstract perform readabl '' archiv retriev 21 april 2020 21 april `` evolut extens languag histori lua '' www.lua.org retriev 4 januari 2023 `` evolut extens languag histori lua '' www.lua.org retriev 4 januari 2023 `` evolut extens languag histori lua '' retriev 4 januari 2023 4 januari `` faq nim program languag '' archiv origin 11 juli 2017 retriev 21 april 2020 `` faq nim program languag '' archiv origin 11 juli 2017 retriev 21 april 2020 `` faq nim program languag '' archiv retriev 21 april 2020 21 april `` 9 class — python 3.6.4 document '' docs.python.org archiv origin 23 octob 2012 retriev 9 januari 2018 `` 9 class — python 3.6.4 document '' docs.python.org archiv origin 23 octob 2012 retriev 9 januari 2018 `` 9 class — python 3.6.4 document '' archiv retriev 9 januari 2018 9 januari `` influenc rust refer '' doc.rust-lang.org retriev 4 januari 2023 `` influenc rust refer '' doc.rust-lang.org retriev 4 januari 2023 `` influenc rust refer '' retriev 4 januari 2023 4 januari stroustrup bjarn 1997 `` 1 '' c++ program languag third ed. addison-wesley isbn 0-201-88954-4 oclc 59193992 stroustrup bjarn 1997 `` 1 '' c++ program languag third ed. addison-wesley isbn 0-201-88954-4 oclc 59193992 stroustrup bjarn c++ program languag isbn 0-201-88954-4 oclc 59193992 b c stroustrup b 6 may 2014 `` lectur essenc c++ univers edinburgh '' youtub archiv origin 28 april 2015 retriev 12 june 2015 b c b c stroustrup b 6 may 2014 `` lectur essenc c++ univers edinburgh '' youtub archiv origin 28 april 2015 retriev 12 june 2015 `` lectur essenc c++ univers edinburgh '' youtub archiv retriev 12 june 2015 12 june stroustrup bjarn 17 februari 2014 `` c++ applic '' stroustrup.com archiv origin 4 april 2021 retriev 5 may 2014 stroustrup bjarn 17 februari 2014 `` c++ applic '' stroustrup.com archiv origin 4 april 2021 retriev 5 may 2014 `` c++ applic '' archiv retriev 5 may 2014 5 may b `` iso/iec 14882:2024 '' intern organ standard retriev 21 octob 2020 b b `` iso/iec 14882:2024 '' intern organ standard retriev 21 octob 2020 `` iso/iec 14882:2024 '' retriev 21 octob 2020 21 octob `` bjarn stroustrup 's homepag '' www.stroustrup.com archiv origin 14 may 2019 retriev 15 may 2013 `` bjarn stroustrup 's homepag '' www.stroustrup.com archiv origin 14 may 2019 retriev 15 may 2013 `` bjarn stroustrup 's homepag '' archiv retriev 15 may 2013 15 may `` c++ schedul '' pdf archiv pdf origin 10 august 2020 retriev 9 august 2020 `` c++ schedul '' pdf archiv pdf origin 10 august 2020 retriev 9 august 2020 `` c++ schedul '' pdf archiv pdf retriev 9 august 2020 9 august `` c++ 's head '' archiv origin 3 decemb 2018 retriev 3 decemb 2018 `` c++ 's head '' archiv origin 3 decemb 2018 retriev 3 decemb 2018 `` c++ 's head '' archiv retriev 3 decemb 2018 3 decemb `` rfc convert builin-mailinfo.c use better string librari '' mail list 6 septemb 2007 archiv origin 8 march 2021 retriev 31 march 2015 `` rfc convert builin-mailinfo.c use better string librari '' mail list 6 septemb 2007 archiv origin 8 march 2021 retriev 31 march 2015 `` rfc convert builin-mailinfo.c use better string librari '' archiv retriev 31 march 2015 31 march `` effort attract user '' mail list 12 juli 2010 archiv origin 21 march 2015 retriev 31 march 2015 `` effort attract user '' mail list 12 juli 2010 archiv origin 21 march 2015 retriev 31 march 2015 `` effort attract user '' archiv retriev 31 march 2015 31 march andrew binstock 18 may 2011 `` dr. dobb 's interview ken thompson '' archiv origin 13 march 2014 retriev 7 februari 2014 andrew binstock 18 may 2011 `` dr. dobb 's interview ken thompson '' archiv origin 13 march 2014 retriev 7 februari 2014 `` dr. dobb 's interview ken thompson '' archiv retriev 7 februari 2014 7 februari peter seibel 16 septemb 2009 coder work reflect craft program apress pp 475–476 isbn 978-1-4302-1948-4 archiv origin 1 decemb 2019 retriev 9 novemb 2017 peter seibel 16 septemb 2009 coder work reflect craft program apress pp 475–476 isbn 978-1-4302-1948-4 archiv origin 1 decemb 2019 retriev 9 novemb 2017 coder work reflect craft program 475– isbn 978-1-4302-1948-4 archiv retriev 9 novemb 2017 9 novemb `` c++ coder work '' 16 octob 2009 archiv origin 10 novemb 2017 retriev 9 novemb 2017 `` c++ coder work '' 16 octob 2009 archiv origin 10 novemb 2017 retriev 9 novemb 2017 `` c++ coder work '' archiv retriev 9 novemb 2017 9 novemb `` interview donald knuth '' dr. dobb 's archiv origin 8 march 2021 retriev 18 juli 2021 `` interview donald knuth '' dr. dobb 's archiv origin 8 march 2021 retriev 18 juli 2021 `` interview donald knuth '' archiv retriev 18 juli 2021 18 juli `` la tex navig '' archiv origin 20 novemb 2017 retriev 10 novemb 2017 `` la tex navig '' archiv origin 20 novemb 2017 retriev 10 novemb 2017 `` la tex navig '' archiv retriev 10 novemb 2017 10 novemb b stroustrup bjarn 7 march 2010 `` bjarn stroustrup 's faq c++ invent '' stroustrup.com archiv origin 6 februari 2016 retriev 16 septemb 2010 b b stroustrup bjarn 7 march 2010 `` bjarn stroustrup 's faq c++ invent '' stroustrup.com archiv origin 6 februari 2016 retriev 16 septemb 2010 `` bjarn stroustrup 's faq c++ invent '' archiv retriev 16 septemb 2010 16 septemb b stroustrup bjarn `` evolv languag real world c++ 1991-2006 '' pdf archiv pdf origin 20 novemb 2007 retriev 14 august 2013 b b stroustrup bjarn `` evolv languag real world c++ 1991-2006 '' pdf archiv pdf origin 20 novemb 2007 retriev 14 august 2013 `` evolv languag real world c++ 1991-2006 '' pdf archiv pdf retriev 14 august 2013 14 august b c stroustrup bjarn `` histori c ++ 1979− 1991 '' pdf archiv pdf origin 2 februari 2019 retriev 18 juli 2013 b c b c stroustrup bjarn `` histori c ++ 1979− 1991 '' pdf archiv pdf origin 2 februari 2019 retriev 18 juli 2013 `` histori c ++ 1979− 1991 '' pdf archiv pdf retriev 18 juli 2013 18 juli stroustrup bjarn `` c++ program languag '' first ed. archiv origin 9 august 2012 retriev 16 septemb 2010 stroustrup bjarn `` c++ program languag '' first ed. archiv origin 9 august 2012 retriev 16 septemb 2010 `` c++ program languag '' archiv retriev 16 septemb 2010 16 septemb stroustrup bjarn `` c++ program languag '' second ed. archiv origin 9 august 2012 retriev 16 septemb 2010 stroustrup bjarn `` c++ program languag '' second ed. archiv origin 9 august 2012 retriev 16 septemb 2010 `` c++ program languag '' archiv retriev 16 septemb 2010 16 septemb sutter herb 30 june 2016 `` trip report summer iso c++ standard meet oulu '' herbsutter.com archiv origin 8 octob 2016. next standard c++17 c++20 sutter herb 30 june 2016 `` trip report summer iso c++ standard meet oulu '' herbsutter.com archiv origin 8 octob 2016. next standard c++17 c++20 `` trip report summer iso c++ standard meet oulu '' origin dusíková hana 6 novemb 2019 `` n4817 2020 pragu meet invit inform '' pdf archiv pdf origin 29 decemb 2019 retriev 13 februari 2020 dusíková hana 6 novemb 2019 `` n4817 2020 pragu meet invit inform '' pdf archiv pdf origin 29 decemb 2019 retriev 13 februari 2020 `` n4817 2020 pragu meet invit inform '' pdf archiv pdf retriev 13 februari 2020 13 februari `` current statu '' isocpp.org archiv origin 8 septemb 2020 retriev 7 septemb 2020 `` current statu '' isocpp.org archiv origin 8 septemb 2020 retriev 7 septemb 2020 `` current statu '' archiv retriev 7 septemb 2020 7 septemb `` c++20 approv -- herb sutter '' isocpp.org archiv origin 11 septemb 2020 retriev 8 septemb 2020 `` c++20 approv -- herb sutter '' isocpp.org archiv origin 11 septemb 2020 retriev 8 septemb 2020 `` c++20 approv -- herb sutter '' archiv retriev 8 septemb 2020 8 septemb `` comput scienc pioneer bjarn stroustrup receiv 2018 charl stark draper prize engin '' press releas nation academi engin 3 januari 2018 archiv origin 3 januari 2018 retriev 14 decemb 2021 `` comput scienc pioneer bjarn stroustrup receiv 2018 charl stark draper prize engin '' press releas nation academi engin 3 januari 2018 archiv origin 3 januari 2018 retriev 14 decemb 2021 `` comput scienc pioneer bjarn stroustrup receiv 2018 charl stark draper prize engin '' origin retriev 14 decemb 2021 14 decemb tiob novemb 2024 `` tiob index novemb 2024 '' tiobe.com tiob compani archiv origin 18 novemb 2024 retriev 18 novemb 2024 tiob novemb 2024 `` tiob index novemb 2024 '' tiobe.com tiob compani archiv origin 18 novemb 2024 retriev 18 novemb 2024 `` tiob index novemb 2024 '' archiv retriev 18 novemb 2024 18 novemb `` bjarn stroustrup 's faq – name `` c++ '' come '' archiv origin 6 februari 2016 retriev 16 januari 2008 `` bjarn stroustrup 's faq – name `` c++ '' come '' archiv origin 6 februari 2016 retriev 16 januari 2008 `` bjarn stroustrup 's faq – name `` c++ '' come '' archiv retriev 16 januari 2008 16 januari `` c c++ programm '' northeastern univers archiv origin 17 novemb 2010 retriev 7 septemb 2015 `` c c++ programm '' northeastern univers archiv origin 17 novemb 2010 retriev 7 septemb 2015 `` c c++ programm '' northeastern univers origin retriev 7 septemb 2015 7 septemb `` iso/iec 14882:1998 '' intern organ standard archiv origin 15 januari 2017 retriev 23 novemb 2018 `` iso/iec 14882:1998 '' intern organ standard archiv origin 15 januari 2017 retriev 23 novemb 2018 `` iso/iec 14882:1998 '' archiv retriev 23 novemb 2018 23 novemb `` iso/iec 14882:2003 '' intern organ standard archiv origin 13 august 2021 retriev 23 novemb 2018 `` iso/iec 14882:2003 '' intern organ standard archiv origin 13 august 2021 retriev 23 novemb 2018 `` iso/iec 14882:2003 '' archiv retriev 23 novemb 2018 23 novemb b `` iso/iec 14882:2011 '' intern organ standard archiv origin 27 may 2016 retriev 23 novemb 2018 b b `` iso/iec 14882:2011 '' intern organ standard archiv origin 27 may 2016 retriev 23 novemb 2018 `` iso/iec 14882:2011 '' archiv retriev 23 novemb 2018 23 novemb `` iso/iec 14882:2014 '' intern organ standard archiv origin 29 april 2016 retriev 23 novemb 2018 `` iso/iec 14882:2014 '' intern organ standard archiv origin 29 april 2016 retriev 23 novemb 2018 `` iso/iec 14882:2014 '' archiv retriev 23 novemb 2018 23 novemb `` iso/iec 14882:2017 '' intern organ standard archiv origin 29 januari 2013 retriev 2 decemb 2017 `` iso/iec 14882:2017 '' intern organ standard archiv origin 29 januari 2013 retriev 2 decemb 2017 `` iso/iec 14882:2017 '' archiv retriev 2 decemb 2017 2 decemb `` iso/iec 14882:2020 '' intern organ standard archiv origin 16 decemb 2020 retriev 16 decemb 2020 `` iso/iec 14882:2020 '' intern organ standard archiv origin 16 decemb 2020 retriev 16 decemb 2020 `` iso/iec 14882:2020 '' archiv retriev 16 decemb 2020 16 decemb `` intern standard c++0x unanim approv '' sutter 's mill 12 august 2011 archiv origin 28 june 2018 retriev 23 novemb 2018 `` intern standard c++0x unanim approv '' sutter 's mill 12 august 2011 archiv origin 28 june 2018 retriev 23 novemb 2018 `` intern standard c++0x unanim approv '' archiv retriev 23 novemb 2018 23 novemb `` futur c++ '' archiv origin 23 octob 2018 retriev 23 novemb 2018 – via channel9.msdn.com `` futur c++ '' archiv origin 23 octob 2018 retriev 23 novemb 2018 – via channel9.msdn.com `` futur c++ '' archiv retriev 23 novemb 2018 23 novemb `` c++14 standard c++ '' isocpp.org archiv origin 19 august 2014 retriev 19 august 2014 `` c++14 standard c++ '' isocpp.org archiv origin 19 august 2014 retriev 19 august 2014 `` c++14 standard c++ '' archiv retriev 19 august 2014 19 august sutter herb 15 juli 2017 `` trip report summer iso c++ standard meet toronto '' archiv origin 6 august 2017 retriev 4 august 2017 sutter herb 15 juli 2017 `` trip report summer iso c++ standard meet toronto '' archiv origin 6 august 2017 retriev 4 august 2017 `` trip report summer iso c++ standard meet toronto '' archiv retriev 4 august 2017 4 august `` iso/iec tr 18015:2006 '' intern organ standard archiv origin 15 januari 2019 retriev 15 februari 2019 `` iso/iec tr 18015:2006 '' intern organ standard archiv origin 15 januari 2019 retriev 15 februari 2019 `` iso/iec tr 18015:2006 '' archiv retriev 15 februari 2019 15 februari `` iso/iec tr 19768:2007 '' intern organ standard archiv origin 4 march 2016 retriev 15 februari 2019 `` iso/iec tr 19768:2007 '' intern organ standard archiv origin 4 march 2016 retriev 15 februari 2019 `` iso/iec tr 19768:2007 '' archiv retriev 15 februari 2019 15 februari `` iso/iec tr 29124:2010 '' intern organ standard archiv origin 12 januari 2019 retriev 15 februari 2019 `` iso/iec tr 29124:2010 '' intern organ standard archiv origin 12 januari 2019 retriev 15 februari 2019 `` iso/iec tr 29124:2010 '' archiv retriev 15 februari 2019 15 februari `` iso/iec tr 24733:2011 '' intern organ standard archiv origin 15 januari 2019 retriev 15 februari 2019 `` iso/iec tr 24733:2011 '' intern organ standard archiv origin 15 januari 2019 retriev 15 februari 2019 `` iso/iec tr 24733:2011 '' archiv retriev 15 februari 2019 15 februari `` iso/iec t 18822:2015 '' intern organ standard archiv origin 15 januari 2019 retriev 15 februari 2019 `` iso/iec t 18822:2015 '' intern organ standard archiv origin 15 januari 2019 retriev 15 februari 2019 `` iso/iec t 18822:2015 '' archiv retriev 15 februari 2019 15 februari `` iso/iec t 19570:2015 '' intern organ standard archiv origin 15 januari 2019 retriev 15 februari 2019 `` iso/iec t 19570:2015 '' intern organ standard archiv origin 15 januari 2019 retriev 15 februari 2019 `` iso/iec t 19570:2015 '' archiv retriev 15 februari 2019 15 februari `` iso/iec t 19841:2015 '' intern organ standard archiv origin 15 januari 2019 retriev 15 februari 2019 `` iso/iec t 19841:2015 '' intern organ standard archiv origin 15 januari 2019 retriev 15 februari 2019 `` iso/iec t 19841:2015 '' archiv retriev 15 februari 2019 15 februari `` iso/iec t 19568:2015 '' intern organ standard archiv origin 15 januari 2019 retriev 15 februari 2019 `` iso/iec t 19568:2015 '' intern organ standard archiv origin 15 januari 2019 retriev 15 februari 2019 `` iso/iec t 19568:2015 '' archiv retriev 15 februari 2019 15 februari `` iso/iec t 19217:2015 '' intern organ standard archiv origin 15 januari 2019 retriev 15 februari 2019 `` iso/iec t 19217:2015 '' intern organ standard archiv origin 15 januari 2019 retriev 15 februari 2019 `` iso/iec t 19217:2015 '' archiv retriev 15 februari 2019 15 februari `` iso/iec t 19571:2016 '' intern organ standard archiv origin 15 januari 2019 retriev 15 februari 2019 `` iso/iec t 19571:2016 '' intern organ standard archiv origin 15 januari 2019 retriev 15 februari 2019 `` iso/iec t 19571:2016 '' archiv retriev 15 februari 2019 15 februari `` iso/iec t 19568:2017 '' intern organ standard archiv origin 15 januari 2019 retriev 15 februari 2019 `` iso/iec t 19568:2017 '' intern organ standard archiv origin 15 januari 2019 retriev 15 februari 2019 `` iso/iec t 19568:2017 '' archiv retriev 15 februari 2019 15 februari `` iso/iec t 21425:2017 '' intern organ standard archiv origin 15 januari 2019 retriev 15 februari 2019 `` iso/iec t 21425:2017 '' intern organ standard archiv origin 15 januari 2019 retriev 15 februari 2019 `` iso/iec t 21425:2017 '' archiv retriev 15 februari 2019 15 februari `` iso/iec t 22277:2017 '' intern organ standard archiv origin 15 januari 2019 retriev 15 februari 2019 `` iso/iec t 22277:2017 '' intern organ standard archiv origin 15 januari 2019 retriev 15 februari 2019 `` iso/iec t 22277:2017 '' archiv retriev 15 februari 2019 15 februari `` iso/iec t 19216:2018 '' intern organ standard archiv origin 15 januari 2019 retriev 15 februari 2019 `` iso/iec t 19216:2018 '' intern organ standard archiv origin 15 januari 2019 retriev 15 februari 2019 `` iso/iec t 19216:2018 '' archiv retriev 15 februari 2019 15 februari `` iso/iec t 21544:2018 '' intern organ standard archiv origin 15 januari 2019 retriev 15 februari 2019 `` iso/iec t 21544:2018 '' intern organ standard archiv origin 15 januari 2019 retriev 15 februari 2019 `` iso/iec t 21544:2018 '' archiv retriev 15 februari 2019 15 februari `` iso/iec t 19570:2018 '' intern organ standard archiv origin 15 januari 2019 retriev 15 februari 2019 `` iso/iec t 19570:2018 '' intern organ standard archiv origin 15 januari 2019 retriev 15 februari 2019 `` iso/iec t 19570:2018 '' archiv retriev 15 februari 2019 15 februari `` iso/iec t 23619:2021 '' intern organ standard archiv origin 15 decemb 2018 retriev 11 octob 2021 `` iso/iec t 23619:2021 '' intern organ standard archiv origin 15 decemb 2018 retriev 11 octob 2021 `` iso/iec t 23619:2021 '' archiv retriev 11 octob 2021 11 octob b. stroustrup interview sergio de simon 30 april 2015 `` stroustrup thought c++17 interview '' archiv origin 8 juli 2015 retriev 8 juli 2015 b. stroustrup interview sergio de simon 30 april 2015 `` stroustrup thought c++17 interview '' archiv origin 8 juli 2015 retriev 8 juli 2015 `` stroustrup thought c++17 interview '' archiv retriev 8 juli 2015 8 juli stroustrup bjarn 2000 c++ program languag special ed. addison-wesley p. 46 isbn 0-201-70073-5 stroustrup bjarn 2000 c++ program languag special ed. addison-wesley p. 46 isbn 0-201-70073-5 isbn 0-201-70073-5 stroustrup bjarn `` open issu c++ program languag 3rd edit '' archiv origin 5 may 2014 retriev 5 may 2014 stroustrup bjarn `` open issu c++ program languag 3rd edit '' archiv origin 5 may 2014 retriev 5 may 2014 `` open issu c++ program languag 3rd edit '' archiv retriev 5 may 2014 5 may iso/iec program languag – c++11 draft n3797 archiv 2 octob 2018 wayback machin §3.7 storag durat basic.stc iso/iec program languag – c++11 draft n3797 archiv 2 octob 2018 wayback machin §3.7 storag durat basic.stc iso iec program languag – c++11 draft n3797 archiv wayback machin iso/iec program languag – c++11 draft n3797 archiv 2 octob 2018 wayback machin §3.7.1 static storag durat basic.stc.stat iso/iec program languag – c++11 draft n3797 archiv 2 octob 2018 wayback machin §3.7.1 static storag durat basic.stc.stat iso iec program languag – c++11 draft n3797 archiv wayback machin iso/iec program languag – c++11 draft n3797 archiv 2 octob 2018 wayback machin §3.7.2 thread storag durat basic.stc.thread iso/iec program languag – c++11 draft n3797 archiv 2 octob 2018 wayback machin §3.7.2 thread storag durat basic.stc.thread iso iec program languag – c++11 draft n3797 archiv wayback machin iso/iec program languag – c++11 draft n3797 archiv 2 octob 2018 wayback machin §3.7.3 automat storag durat basic.stc.auto iso/iec program languag – c++11 draft n3797 archiv 2 octob 2018 wayback machin §3.7.3 automat storag durat basic.stc.auto iso iec program languag – c++11 draft n3797 archiv wayback machin iso/iec program languag – c++11 draft n3797 archiv 2 octob 2018 wayback machin §3.7.4 dynam storag durat basic.stc.dynam iso/iec program languag – c++11 draft n3797 archiv 2 octob 2018 wayback machin §3.7.4 dynam storag durat basic.stc.dynam iso iec program languag – c++11 draft n3797 archiv wayback machin `` c++ core guidelin '' isocpp.github.io archiv origin 8 februari 2020 retriev 9 februari 2020 `` c++ core guidelin '' isocpp.github.io archiv origin 8 februari 2020 retriev 9 februari 2020 `` c++ core guidelin '' archiv retriev 9 februari 2020 9 februari b `` nobodi understand c++ part 5 templat code bloat '' articles.emptycrate.com/ emptycr softwar travel stuff 6 may 2008 archiv origin 25 april 2016 retriev 8 march 2010 occas read hear someon talk c++ templat caus code bloat think day thought `` self code exactli thing compil code realli bigger '' ... compil code size compil command g++ filenam .cpp -o3 non-templ version 8140 byte templat version 8028 byte b b `` nobodi understand c++ part 5 templat code bloat '' articles.emptycrate.com/ emptycr softwar travel stuff 6 may 2008 archiv origin 25 april 2016 retriev 8 march 2010 occas read hear someon talk c++ templat caus code bloat think day thought `` self code exactli thing compil code realli bigger '' ... compil code size compil command g++ filenam .cpp -o3 non-templ version 8140 byte templat version 8028 byte `` nobodi understand c++ part 5 templat code bloat '' archiv retriev 8 march 2010 8 march sutter herb alexandrescu andrei 2004 c++ code standard 101 rule guidelin best practic addison-wesley sutter herb alexandrescu andrei 2004 c++ code standard 101 rule guidelin best practic addison-wesley sutter herb alexandrescu andrei henricson mat nyquist erik 1997 industri strength c++ prentic hall isbn 0-13-120965-5 henricson mat nyquist erik 1997 industri strength c++ prentic hall isbn 0-13-120965-5 industri strength c++ isbn 0-13-120965-5 stroustrup bjarn 2000 c++ program languag special ed. addison-wesley p. 310 isbn 0-201-70073-5 virtual member function sometim call method stroustrup bjarn 2000 c++ program languag special ed. addison-wesley p. 310 isbn 0-201-70073-5 virtual member function sometim call method isbn 0-201-70073-5 mycroft alan 2013 `` c c++ except templat '' pdf cambridg comput laboratori cours materi 2013-14 archiv pdf origin 13 may 2016 retriev 30 august 2016 mycroft alan 2013 `` c c++ except templat '' pdf cambridg comput laboratori cours materi 2013-14 archiv pdf origin 13 may 2016 retriev 30 august 2016 `` c c++ except templat '' pdf archiv pdf retriev 30 august 2016 30 august stroustrup bjarn 2013 c++ program languag addison wesley p. 345 isbn 9780321563842 stroustrup bjarn 2013 c++ program languag addison wesley p. 345 isbn 9780321563842 isbn 9780321563842 stroustrup bjarn 2013 c++ program languag addison wesley pp 363–365 isbn 9780321563842 stroustrup bjarn 2013 c++ program languag addison wesley pp 363–365 isbn 9780321563842 363– isbn 9780321563842 stroustrup bjarn 2013 c++ program languag addison wesley pp 345 363 isbn 9780321563842 stroustrup bjarn 2013 c++ program languag addison wesley pp 345 363 isbn 9780321563842 isbn 9780321563842 `` googl c++ style guid '' archiv origin 16 march 2019 retriev 25 june 2019 `` googl c++ style guid '' archiv origin 16 march 2019 retriev 25 june 2019 `` googl c++ style guid '' archiv retriev 25 june 2019 25 june `` llvm code standard '' llvm 9 document archiv origin 27 june 2019 retriev 25 june 2019 `` llvm code standard '' llvm 9 document archiv origin 27 june 2019 retriev 25 june 2019 `` llvm code standard '' archiv retriev 25 june 2019 25 june `` code convent '' qt wiki archiv origin 26 june 2019 retriev 26 june 2019 `` code convent '' qt wiki archiv origin 26 june 2019 retriev 26 june 2019 `` code convent '' archiv retriev 26 june 2019 26 june stroustrup bjarn 2013 c++ program languag addison wesley pp 344 370 isbn 9780321563842 stroustrup bjarn 2013 c++ program languag addison wesley pp 344 370 isbn 9780321563842 isbn 9780321563842 stroustrup bjarn 2013 c++ program languag addison wesley p. 349 isbn 9780321563842 stroustrup bjarn 2013 c++ program languag addison wesley p. 349 isbn 9780321563842 isbn 9780321563842 graziano lo russo 2008 `` interview a. stepanov '' stlport.org archiv origin 4 march 2009 retriev 8 octob 2015 graziano lo russo 2008 `` interview a. stepanov '' stlport.org archiv origin 4 march 2009 retriev 8 octob 2015 `` interview a. stepanov '' archiv retriev 8 octob 2015 8 octob `` c++ core guidelin '' isocpp.github.io archiv origin 16 februari 2020 retriev 9 februari 2020 `` c++ core guidelin '' isocpp.github.io archiv origin 16 februari 2020 retriev 9 februari 2020 `` c++ core guidelin '' archiv retriev 9 februari 2020 9 februari `` bjarn stroustrup announc c++ core guidelin standard c++ '' isocpp.org archiv origin 11 may 2020 retriev 31 march 2020 `` bjarn stroustrup announc c++ core guidelin standard c++ '' isocpp.org archiv origin 11 may 2020 retriev 31 march 2020 `` bjarn stroustrup announc c++ core guidelin standard c++ '' archiv retriev 31 march 2020 31 march `` microsoft/gsl '' 18 juli 2021 archiv origin 18 juli 2021 retriev 18 juli 2021 – via github `` microsoft/gsl '' 18 juli 2021 archiv origin 18 juli 2021 retriev 18 juli 2021 – via github `` microsoft/gsl '' archiv retriev 18 juli 2021 18 juli `` use c++ core guidelin checker '' microsoft learn archiv origin 13 august 2021 retriev 31 march 2020 `` use c++ core guidelin checker '' microsoft learn archiv origin 13 august 2021 retriev 31 march 2020 `` use c++ core guidelin checker '' archiv retriev 31 march 2020 31 march `` c++ abi summari '' 20 march 2001 archiv origin 10 juli 2018 retriev 30 may 2006 `` c++ abi summari '' 20 march 2001 archiv origin 10 juli 2018 retriev 30 may 2006 `` c++ abi summari '' archiv retriev 30 may 2006 30 may `` bjarn stroustrup 's faq – c subset c++ '' archiv origin 6 februari 2016 retriev 5 may 2014 `` bjarn stroustrup 's faq – c subset c++ '' archiv origin 6 februari 2016 retriev 5 may 2014 `` bjarn stroustrup 's faq – c subset c++ '' archiv retriev 5 may 2014 5 may `` c9x – new c standard '' archiv origin 21 june 2018 retriev 27 decemb 2008 `` c9x – new c standard '' archiv origin 21 june 2018 retriev 27 decemb 2008 `` c9x – new c standard '' archiv retriev 27 decemb 2008 27 decemb `` c++0x support gcc '' archiv origin 21 juli 2010 retriev 12 octob 2010 `` c++0x support gcc '' archiv origin 21 juli 2010 retriev 12 octob 2010 `` c++0x support gcc '' archiv retriev 12 octob 2010 12 octob `` c++0x core languag featur vc10 tabl '' archiv origin 21 august 2010 retriev 12 octob 2010 `` c++0x core languag featur vc10 tabl '' archiv origin 21 august 2010 retriev 12 octob 2010 `` c++0x core languag featur vc10 tabl '' archiv retriev 12 octob 2010 12 octob `` clang c++98 c++11 c++14 statu '' clang.llvm.org 12 may 2013 archiv origin 4 juli 2013 retriev 10 june 2013 `` clang c++98 c++11 c++14 statu '' clang.llvm.org 12 may 2013 archiv origin 4 juli 2013 retriev 10 june 2013 `` clang c++98 c++11 c++14 statu '' archiv retriev 10 june 2013 10 june read edit edit edit abraham david gurtovoy aleksey 2005 c++ templat metaprogram concept tool techniqu boost beyond addison-wesley isbn 0-321-22725-5 abraham david isbn 0-321-22725-5 alexandrescu andrei 2001 modern c++ design gener program design pattern appli addison-wesley isbn 0-201-70431-5 alexandrescu andrei isbn 0-201-70431-5 alexandrescu andrei sutter herb 2004 c++ design code standard rule guidelin write program addison-wesley isbn 0-321-11358-6 alexandrescu andrei sutter herb isbn 0-321-11358-6 becker pete 2006 c++ standard librari extens tutori refer addison-wesley isbn 0-321-41299-0 becker pete isbn 0-321-41299-0 brokken frank 2010 c++ annot univers groningen isbn 978-90-367-0470-0 archiv origin 28 april 2010 retriev 28 april 2010 c++ annot isbn 978-90-367-0470-0 archiv retriev 28 april 2010 28 april coplien jame 1994 reprint correct origin year public 1992 advanc c++ program style idiom addison-wesley isbn 0-201-54855-0 coplien jame advanc c++ program style idiom isbn 0-201-54855-0 dewhurst stephen c. 2005 c++ common knowledg essenti intermedi program addison-wesley isbn 0-321-32192-8 isbn 0-321-32192-8 inform technolog industri council 15 octob 2003 program languag – c++ second ed. geneva iso/iec 14882:2003 e inform technolog industri council josutti nicolai m. 2012 c++ standard librari tutori refer second ed. addison-wesley isbn 978-0-321-62321-8 isbn 978-0-321-62321-8 koenig andrew moo barbara e. 2000 acceler c++ – practic program exampl addison-wesley isbn 0-201-70353-x koenig andrew acceler c++ – practic program exampl isbn 0-201-70353-x lippman stanley b. lajoi josé moo barbara e. 2011 c++ primer fifth ed. addison-wesley isbn 978-0-321-71411-4 lippman stanley b. c++ primer c++ primer isbn 978-0-321-71411-4 lippman stanley b 1996 insid c++ object model addison-wesley isbn 0-201-83454-5 isbn 0-201-83454-5 meyer scott 2005 effect c++ third ed. addison-wesley isbn 0-321-33487-6 meyer scott effect c++ isbn 0-321-33487-6 stroustrup bjarn 2013 c++ program languag fourth ed. addison-wesley isbn 978-0-321-56384-2 stroustrup bjarn c++ program languag isbn 978-0-321-56384-2 stroustrup bjarn 1994 design evolut c++ addison-wesley isbn 0-201-54330-3 stroustrup bjarn design evolut c++ isbn 0-201-54330-3 stroustrup bjarn 2014 program principl practic use c++ second ed. addison-wesley isbn 978-0-321-99278-9 stroustrup bjarn isbn 978-0-321-99278-9 sutter herb 2001 except c++ 40 new engin puzzl program problem solut addison-wesley isbn 0-201-70434-x sutter herb isbn 0-201-70434-x sutter herb 2004 except c++ style addison-wesley isbn 0-201-76042-8 sutter herb isbn 0-201-76042-8 vandevoord david josutti nicolai m. 2003 c++ templat complet guid addison-wesley isbn 0-201-73484-2 isbn 0-201-73484-2 extern link edit edit edit jtc1/sc22/wg21 – iso/iec c++ standard work group jtc1/sc22/wg21 standard c++ foundat – non-profit organ promot use understand standard c++ bjarn stroustrup director organ standard c++ foundat bjarn stroustrup c++ keyword c++ keyword c++ express c++ express c++ oper preced c++ oper preced sister project sister project medium common medium common medium quotat wikiquot quotat wikiquot quotat textbook wikibook textbook wikibook textbook resourc wikivers resourc wikivers resourc v v e e c++ c++ c++ outlin outlin c++98 c++98 c++03 c++03 c++11 c++11 c++14 c++14 c++17 c++17 c++20 c++20 c++23 c++23 c++26 c++26 librari librari class class concept concept copi constructor copi constructor except handl except safeti except handl except safeti function overload function overload move semant move semant new delet new delet oper overload oper overload oper oper refer refer templat templat virtual function virtual function standard librari i/o stream i/o stream smart pointer smart pointer stl stl string string as-if rule as-if rule barton–nackman trick barton–nackman trick curious recur templat pattern curious recur templat pattern vex par vex par one definit rule one definit rule resourc acquisit initi resourc acquisit initi rule three rule three slice slice special member function special member function substitut failur error substitut failur error templat metaprogram templat metaprogram compil comparison c++ compil comparison c++ compil borland c++ borland c++ borland turbo c++ borland turbo c++ c++builder c++builder clang clang gcc gcc intel c++ compil intel c++ compil oracl solari studio oracl solari studio visual c++ msvc visual c++ msvc watcom c/c++ watcom c/c++ ide comparison c ide comparison c ide anjuta anjuta clion clion code :block code :block codelit codelit dev-c++ dev-c++ eclips eclips geani geani netbean netbean kdevelop kdevelop qt creator qt creator visual studio visual studio objective-c++ objective-c++ c++/cli c++/cli c++/cx c++/cx c++/winrt c++/winrt ch ch sycl sycl embed c++ embed c++ comparison program languag comparison program languag comparison java c++ comparison java c++ comparison algol 68 c++ comparison algol 68 c++ compat c c++ compat c c++ critic c++ critic c++ design bjarn stroustrup bjarn stroustrup categori v v e e program languag comparison comparison timelin timelin histori histori ada ada algol simula algol simula simula apl apl assembl assembl basic visual basic classic .net basic visual basic classic .net visual basic classic classic .net .net c c c++ c++ c c cobol cobol erlang erlang forth forth fortran fortran go go haskel haskel java java javascript javascript julia julia kotlin kotlin lisp lisp lua lua matlab matlab ml ml pascal object pascal pascal object pascal object pascal perl perl php php prolog prolog python python r r rubi rubi rust rust sql sql scratch scratch shell shell smalltalk smalltalk swift swift ... ... list alphabet alphabet categor categor gener gener non-english-bas non-english-bas categori categori v v e e iso standard number iso standard iso roman iec standard 1 1 2 2 3 3 4 4 6 6 7 7 9 9 16 16 17 17 31 -0 -1 -3 -4 -5 -6 -7 -8 -9 -10 -11 -12 -13 31 -0 -0 -1 -1 -3 -3 -4 -4 -5 -5 -6 -6 -7 -7 -8 -8 -9 -9 -10 -10 -11 -11 -12 -12 -13 -13 68-1 68-1 128 128 216 216 217 217 226 226 228 228 233 233 259 259 261 261 262 262 302 302 306 306 361 361 500 500 518 518 519 519 639 -1 -2 -3 -5 -6 639 -1 -1 -2 -2 -3 -3 -5 -5 -6 -6 646 646 657 657 668 668 690 690 704 704 732 732 764 764 838 838 843 843 860 860 898 898 965 965 999 999 1000 1000 1004 1004 1007 1007 1073-1 1073-1 1073-2 1073-2 1155 1155 1413 1413 1538 1538 1629 1629 1745 1745 1989 1989 2014 2014 2015 2015 2022 2022 2033 2033 2047 2047 2108 2108 2145 2145 2146 2146 2240 2240 2281 2281 2533 2533 2709 2709 2711 2711 2720 2720 2788 2788 2848 2848 2852 2852 2921 2921 3029 3029 3103 3103 3166 -1 -2 -3 3166 -1 -1 -2 -2 -3 -3 3297 3297 3307 3307 3601 3601 3602 3602 3864 3864 3901 3901 3950 3950 3977 3977 4031 4031 4157 4157 4165 4165 4217 4217 4909 4909 5218 5218 5426 5426 5427 5427 5428 5428 5725 5725 5775 5775 5776 5776 5800 5800 5807 5807 5964 5964 6166 6166 6344 6344 6346 6346 6373 6373 6385 6385 6425 6425 6429 6429 6438 6438 6523 6523 6709 6709 6943 6943 7001 7001 7002 7002 7010 7010 7027 7027 7064 7064 7098 7098 7185 7185 7200 7200 7498 -1 7498 -1 -1 7637 7637 7736 7736 7810 7810 7811 7811 7812 7812 7813 7813 7816 7816 7942 7942 8000 8000 8093 8093 8178 8178 8217 8217 8373 8373 8501-1 8501-1 8571 8571 8583 8583 8601 8601 8613 8613 8632 8632 8651 8651 8652 8652 8691 8691 8805/8806 8805/8806 8807 8807 8820-5 8820-5 8859 -1 -2 -3 -4 -5 -6 -7 -8 -8-i -9 -10 -11 -12 -13 -14 -15 -16 8859 -1 -1 -2 -2 -3 -3 -4 -4 -5 -5 -6 -6 -7 -7 -8 -8 -8-i -8-i -9 -9 -10 -10 -11 -11 -12 -12 -13 -13 -14 -14 -15 -15 -16 -16 8879 8879 9000/9001 9000/9001 9036 9036 9075 9075 9126 9126 9141 9141 9227 9227 9241 9241 9293 9293 9314 9314 9362 9362 9407 9407 9496 9496 9506 9506 9529 9529 9564 9564 9592/9593 9592/9593 9594 9594 9660 9660 9797-1 9797-1 9897 9897 9899 9899 9945 9945 9984 9984 9985 9985 9995 9995 10006 10006 10007 10007 10116 10116 10118-3 10118-3 10160 10160 10161 10161 10165 10165 10179 10179 10206 10206 10218 10218 10279 10279 10303 -11 -21 -22 -28 -238 10303 -11 -11 -21 -21 -22 -22 -28 -28 -238 -238 10383 10383 10585 10585 10589 10589 10628 10628 10646 10646 10664 10664 10746 10746 10861 10861 10957 10957 10962 10962 10967 10967 11073 11073 11170 11170 11172 11172 11179 11179 11404 11404 11544 11544 11783 11783 11784 11784 11785 11785 11801 11801 11889 11889 11898 11898 11940 -2 11940 -2 11941 11941 11941 tr 11941 tr 11992 11992 12006 12006 12052 12052 12182 12182 12207 12207 12234-2 12234-2 12620 12620 13211 -1 -2 13211 -1 -1 -2 -2 13216 13216 13250 13250 13399 13399 13406-2 13406-2 13450 13450 13485 13485 13490 13490 13567 13567 13568 13568 13584 13584 13616 13616 13816 13816 13818 13818 14000 14000 14031 14031 14224 14224 14289 14289 14396 14396 14443 14443 14496 -2 -3 -6 -10 -11 -12 -14 -17 -20 14496 -2 -2 -3 -3 -6 -6 -10 -10 -11 -11 -12 -12 -14 -14 -17 -17 -20 -20 14617 14617 14644 14644 14649 14649 14651 14651 14698 14698 14764 14764 14882 14882 14971 14971 15022 15022 15189 15189 15288 15288 15291 15291 15398 15398 15408 15408 15444 -3 -9 15444 -3 -3 -9 -9 15445 15445 15438 15438 15504 15504 15511 15511 15686 15686 15693 15693 15706 -2 15706 -2 -2 15707 15707 15897 15897 15919 15919 15924 15924 15926 15926 15926 wip 15926 wip 15930 15930 15938 15938 16023 16023 16262 16262 16355-1 16355-1 16485 16485 16612-2 16612-2 16750 16750 16949 t 16949 t 17024 17024 17025 17025 17100 17100 17203 17203 17369 17369 17442 17442 17506 17506 17799 17799 18004 18004 18014 18014 18181 18181 18245 18245 18629 18629 18916 18916 19005 19005 19011 19011 19092 -1 -2 19092 -1 -1 -2 -2 19114 19114 19115 19115 19125 19125 19136 19136 19407 19407 19439 19439 19500 19500 19501 19501 19502 19502 19503 19503 19505 19505 19506 19506 19507 19507 19508 19508 19509 19509 19510 19510 19600 19600 19752 19752 19757 19757 19770 19770 19775-1 19775-1 19794-5 19794-5 19831 19831 20000 20000 20022 20022 20121 20121 20400 20400 20802 20802 20830 20830 21000 21000 21001 21001 21047 21047 21122 21122 21500 21500 21827 21827 22000 22000 22275 22275 22300 22300 22301 22301 22395 22395 22537 22537 23000 23000 23003 23003 23008 23008 23009 23009 23090-3 23090-3 23092 23092 23094-1 23094-1 23094-2 23094-2 23270 23270 23271 23271 23360 23360 23941 23941 24517 24517 24613 24613 24617 24617 24707 24707 24728 24728 25178 25178 25964 25964 26000 26000 26262 26262 26300 26300 26324 26324 27000 seri 27000 seri 27000 27000 27001 27001 27002 27002 27005 27005 27006 27006 27729 27729 28000 28000 29110 29148 29148 29199-2 29199-2 29500 29500 30170 30170 31000 31000 32000 32000 37001 37001 38500 38500 39075 39075 40500 40500 42010 42010 45001 45001 50001 50001 55000 55000 56000 56000 80000 80000 categori categori v v e e iec standard 60027 60027 60034 60034 60038 60038 60062 60062 60063 60063 60068 60068 60112 60112 60228 60228 60269 60269 60297 60297 60309 60309 60320 60320 60364 60364 60446 60446 60559 60559 60601 60601 60870 60870-5 60870-6 60870 60870-5 60870-5 60870-6 60870-6 60906-1 60906-1 60908 60908 60929 60929 60958 60958 60980-344 60980-344 61030 61030 61131 61131-3 61131-9 61131 61131-3 61131-3 61131-9 61131-9 61158 61158 61162 61162 61334 61334 61355 61355 61360 61360 61400 61400 61499 61499 61508 61508 61511 61511 61784 61784 61850 61850 61851 61851 61883 61883 61960 61960 61968 61968 61970 61970 62014-4 62014-4 62026 62026 62056 62056 62061 62061 62196 62196 62262 62262 62264 62264 62304 62304 62325 62325 62351 62351 62365 62365 62366 62366 62379 62379 62386 62386 62455 62455 62680 62680 62682 62682 62700 62700 63110 63110 63119 63119 63382 63382 646 646 1989 1989 2022 2022 4909 4909 5218 5218 6429 6429 6523 6523 7810 7810 7811 7811 7812 7812 7813 7813 7816 7816 7942 7942 8613 8613 8632 8632 8652 8652 8859 8859 9126 9126 9293 9293 9496 9496 9529 9529 9592 9592 9593 9593 9899 9899 9945 9945 9995 9995 10021 10021 10116 10116 10165 10165 10179 10179 10279 10279 10646 10646 10967 10967 11172 11172 11179 11179 11404 11404 11544 11544 11801 11801 12207 12207 13250 13250 13346 13346 13522-5 13522-5 13568 13568 13816 13816 13818 13818 14443 14443 14496 14496 14651 14651 14882 14882 15288 15288 15291 15291 15408 15408 15444 15444 15445 15445 15504 15504 15511 15511 15693 15693 15897 15897 15938 15938 16262 16262 16485 16485 17024 17024 17025 17025 18004 18004 18014 18014 18181 18181 19752 19752 19757 19757 19770 19770 19788 19788 20000 20000 20802 20802 21000 21000 21827 21827 22275 22275 22537 22537 23000 23000 23003 23003 23008 23008 23270 23270 23360 23360 24707 24707 24727 24727 24744 24744 24752 24752 26300 26300 27000 27000 27000-seri 27000-seri 27002 27002 27040 27040 29110 29119 29119 33001 33001 38500 38500 39075 39075 42010 42010 80000 80000 81346 81346 intern electrotechn commiss intern electrotechn commiss author control databas germani germani germani unit state unit state unit state franc franc franc bnf data bnf data bnf data czech republ czech republ czech republ czech republ spain spain spain israel israel israel idref idref idref http //en.wikipedia.org/w/index.php title=c 2b 2b oldid=1267808714 categori c++ c++ algol program languag famili algol program languag famili c++ program languag famili c++ program languag famili class-bas program languag class-bas program languag cross-platform softwar cross-platform softwar high-level program languag high-level program languag object-ori program languag object-ori program languag program languag creat 1983 program languag creat 1983 program languag iso standard program languag iso standard static type program languag static type program languag compil program languag compil program languag webarch templat wayback link webarch templat wayback link articl short descript articl short descript short descript match wikidata short descript match wikidata wikipedia page move-protect due vandal wikipedia page move-protect due vandal use dmi date januari 2020 use dmi date januari 2020 articl contain potenti date statement 1997 articl contain potenti date statement 1997 articl contain potenti date statement articl contain potenti date statement articl unsourc statement articl unsourc statement articl unsourc statement februari 2024 articl unsourc statement februari 2024 articl contain potenti date statement novemb 2024 articl contain potenti date statement novemb 2024 articl exampl c++ code articl exampl c++ code articl excerpt articl excerpt page last edit 6 januari 2025 20:19 utc utc text avail creativ common attribution-sharealik 4.0 licens addit term may appli use site agre term use privaci polici wikipedia® regist trademark wikimedia foundat inc. non-profit organ creativ common attribution-sharealik 4.0 licens term use privaci polici wikimedia foundat inc. privaci polici privaci polici wikipedia wikipedia disclaim disclaim contact wikipedia contact wikipedia code conduct code conduct develop develop statist statist cooki statement cooki statement mobil view mobil view",
    "https://en.wikipedia.org/wiki/Go_(programming_language)": "jump content main menu main page main page main page content content content current event current event current event random articl random articl random articl wikipedia wikipedia wikipedia contact u contact u contact u help help help learn edit learn edit learn edit commun portal commun portal commun portal recent chang recent chang recent chang upload file upload file upload file search search appear donat donat donat creat account creat account creat account log log log person tool donat donat donat creat account creat account creat account log log log learn learn contribut contribut contribut talk talk talk content top top 1 histori toggl histori subsect 1.1 brand style 1.2 gener 1.3 version 1 histori 1 histori toggl histori subsect 1.1 brand style 1.1 brand style 1.1 brand style 1.2 gener 1.2 gener 1.2 gener 1.3 version 1.3 version 1.3 version 2 design toggl design subsect 2.1 syntax 2.2 type 2.2.1 interfac system 2.2.2 gener code use parameter type 2.2.3 enumer type 2.3 packag system 2.4 concurr goroutin channel 2.4.1 suitabl parallel program 2.4.2 lack data race safeti 2.5 binari 2.6 omiss 2 design 2 design toggl design subsect 2.1 syntax 2.1 syntax 2.1 syntax 2.2 type 2.2.1 interfac system 2.2.2 gener code use parameter type 2.2.3 enumer type 2.2 type 2.2 type 2.2.1 interfac system 2.2.1 interfac system 2.2.1 interfac system 2.2.2 gener code use parameter type 2.2.2 gener code use parameter type 2.2.2 gener code use parameter type 2.2.3 enumer type 2.2.3 enumer type 2.2.3 enumer type 2.3 packag system 2.3 packag system 2.3 packag system 2.4 concurr goroutin channel 2.4.1 suitabl parallel program 2.4.2 lack data race safeti 2.4 concurr goroutin channel 2.4 concurr goroutin channel 2.4.1 suitabl parallel program 2.4.1 suitabl parallel program 2.4.1 suitabl parallel program 2.4.2 lack data race safeti 2.4.2 lack data race safeti 2.4.2 lack data race safeti 2.5 binari 2.5 binari 2.5 binari 2.6 omiss 2.6 omiss 2.6 omiss 3 style 3 style 3 style 4 tool 4 tool 4 tool 5 exampl toggl exampl subsect 5.1 hello world 5.2 concurr 5.3 test 5.4 web app 5 exampl 5 exampl toggl exampl subsect 5.1 hello world 5.1 hello world 5.1 hello world 5.2 concurr 5.2 concurr 5.2 concurr 5.3 test 5.3 test 5.3 test 5.4 web app 5.4 web app 5.4 web app 6 applic 6 applic 6 applic 7 recept 7 recept 7 recept 8 name disput 8 name disput 8 name disput 9 see also 9 see also 9 see also 10 note 10 note 10 note 11 refer 11 refer 11 refer 12 read 12 read 12 read 13 extern link 13 extern link 13 extern link toggl tabl content go program languag go program languag 57 languag afrikaan afrikaan afrikaan العربية العربية العربية azərbaycanca azərbaycanca azərbaycanca বাংলা বাংলা বাংলা 閩南語 bân-lâm-gú 閩南語 bân-lâm-gú 閩南語 bân-lâm-gú беларуская беларуская беларуская български български български brezhoneg brezhoneg brezhoneg català català català čeština čeština čeština dansk dansk dansk deutsch deutsch deutsch eesti eesti eesti ελληνικά ελληνικά ελληνικά español español español euskara euskara euskara فارسی فارسی فارسی françai françai françai ગુજરાતી ગુજરાતી ગુજરાતી 한국어 한국어 한국어 հայերեն հայերեն հայերեն hrvatski hrvatski hrvatski bahasa indonesia bahasa indonesia bahasa indonesia íslenska íslenska íslenska italiano italiano italiano עברית עברית עברית кыргызча кыргызча кыргызча latviešu latviešu latviešu magyar magyar magyar മലയാളം മലയാളം മലയാളം bahasa melayu bahasa melayu bahasa melayu nederland nederland nederland 日本語 日本語 日本語 norsk bokmål norsk bokmål norsk bokmål oʻzbekcha ўзбекча oʻzbekcha ўзбекча oʻzbekcha ўзбекча polski polski polski portuguê portuguê portuguê qaraqalpaqsha qaraqalpaqsha qaraqalpaqsha română română română русский русский русский shqip shqip shqip සිංහල සිංහල සිංහල simpl english simpl english simpl english slovenčina slovenčina slovenčina slovenščina slovenščina slovenščina српски srpski српски srpski српски srpski suomi suomi suomi svenska svenska svenska தமிழ் தமிழ் தமிழ் тоҷикӣ тоҷикӣ тоҷикӣ türkçe türkçe türkçe українська українська українська اردو اردو اردو tiếng việt tiếng việt tiếng việt 吴语 吴语 吴语 粵語 粵語 粵語 中文 中文 中文 edit link edit link articl articl articl talk talk talk english read read read edit edit edit view histori view histori view histori tool read read read edit edit edit view histori view histori view histori link link link relat chang relat chang relat chang upload file upload file upload file special page special page special page perman link perman link perman link page inform page inform page inform cite page cite page cite page get shorten url get shorten url get shorten url download qr code download qr code download qr code download pdf download pdf download pdf printabl version printabl version printabl version wikimedia common wikimedia common wikimedia common wikidata item wikidata item wikidata item go program languag paradigm multi-paradigm concurr imper function 1 object-ori 2 3 design robert griesem rob pike ken thompson 4 develop 5 15 year ago 2009-11-10 2009-11-10 stabl releas 1.23.4 1.23.4 41 day ago 3 decemb 2024 3 decemb 2024 type disciplin infer static strong 6 structur 7 8 nomin memori manag garbag collect assembl languag c++ o dragonfli bsd freebsd linux maco netbsd openbsd 9 plan 9 10 solari window licens 3-claus bsd 5 patent 11 filenam extens go.dev go.dev implement c oberon-2 limbo activ oberon commun sequenti process pascal oberon smalltalk newsqueak modula-2 alef apl bcpl modula occam c c oberon-2 oberon-2 limbo limbo activ oberon activ oberon commun sequenti process commun sequenti process pascal pascal oberon oberon smalltalk smalltalk newsqueak newsqueak modula-2 modula-2 alef alef apl apl bcpl bcpl modula modula occam occam crystal v go fast 12 static type compil high-level gener purpos program languag known simplic effici 13 simplic express basic syntax languag larg librari help develop small stack project design googl 14 2009 robert griesem rob pike ken thompson 4 syntact similar c also memori safeti garbag collect structur type 7 csp-style concurr 15 often refer golang avoid ambigu former domain name golang.org proper name go 16 12 static type compil high-level gener purpos program languag 13 googl 14 robert griesem rob pike ken thompson 4 syntact c memori safeti garbag collect structur type 7 csp concurr 15 16 two major implement origin self-host 17 compil toolchain initi develop insid googl 18 self-host 17 compil toolchain 18 frontend written c++ call gofrontend 19 origin gcc frontend provid gccgo gcc-base go compil 20 later extend also support llvm provid llvm-base go compil call gollvm 21 c++ 19 gcc 20 llvm 21 third-parti source-to-sourc compil gopherj 22 transpil go javascript front-end web develop source-to-sourc compil 22 javascript front-end web develop histori edit edit edit go design googl 2007 improv program product era multicor network machin larg codebas 23 design want address critic languag use googl keep use characterist 24 googl program product multicor network machin codebas 23 24 static type run-tim effici like c static type run-tim c readabl usabl like python 25 readabl usabl python 25 high-perform network multiprocess network multiprocess design primarili motiv share dislik c++ 26 27 28 dislik c++ 26 27 28 go publicli announc novemb 2009 29 version 1.0 releas march 2012 30 31 go wide use product googl 32 mani organ open-sourc project 29 30 31 32 brand style edit edit edit gopher gopher mascot introduc 2009 open sourc launch languag design rené french borrow c. 2000 wfmu promot 33 gopher mascot open sourc rené french wfmu 33 novemb 2016 go go mono font releas type design charl bigelow kri holm specif use go project go humanist sans-serif resembl lucida grand go mono monospac font adher wgl4 charact set design legibl larg x-height distinct letterform go go mono adher din 1450 standard slash zero lowercas l tail uppercas serif 34 35 charl bigelow kri holm humanist sans-serif lucida grand monospac wgl4 x-height letterform din 34 35 april 2018 origin logo redesign brand design adam smith new logo modern styliz go slant right trail streamlin gopher mascot remain 36 36 gener edit edit edit lack support gener program initi version go drew consider critic 37 design express open gener program note built-in function fact type-gener treat special case pike call weak might chang point 38 googl team built least one compil experiment go dialect gener releas 39 gener program 37 38 39 august 2018 go princip contributor publish draft design gener program error handl ask user submit feedback 40 41 howev error handl propos eventu abandon 42 error handl 40 41 42 june 2020 new draft design document 43 publish would add necessari syntax go declar gener function type code translat tool go2go provid allow user tri new syntax along generics-en version onlin go playground 44 43 go2go 44 gener final ad go version 1.18 march 15 2022 45 45 version edit edit edit go 1 guarante compat 46 languag specif major part standard librari version current go 1.23 releas 47 maintain promis 46 47 go follow semver rather major go releas support two newer major releas unlik softwar go call second number version major i.e. 1.x x major version 48 go plan never reach 2.0 given compat one languag 's major sell point 49 semver 48 49 design edit edit edit go influenc c especi plan 9 dialect 50 fail verif – see discus emphasi greater simplic safeti consist c plan 9 dialect 50 fail verif fail verif see discus syntax environ adopt pattern common dynam languag 51 option concis variabl declar initi type infer x 0 instead var x int 0 var x 0 fast compil 52 remot packag manag go get 53 onlin packag document 54 dynam languag 51 option concis variabl declar initi type infer x 0 instead var x int 0 var x 0 type infer x 0 var x int 0 var x 0 fast compil 52 52 remot packag manag go get 53 onlin packag document 54 53 54 distinct approach particular problem built-in concurr primit light-weight process goroutin channel select statement interfac system place virtual inherit type embed instead non-virtu inherit toolchain default produc static link nativ binari without extern go depend built-in concurr primit light-weight process goroutin channel select statement light-weight process channel interfac system place virtual inherit type embed instead non-virtu inherit interfac virtual inherit toolchain default produc static link nativ binari without extern go depend static link desir keep languag specif simpl enough hold programm 's head 55 part omit featur common similar languag 55 omit featur common similar languag syntax edit edit edit go 's syntax includ chang c aim keep code concis readabl combin declaration/initi oper introduc allow programm write 3 `` hello world `` without specifi type variabl use contrast c 's int 3 const char `` hello world '' c 3 `` hello world '' without specifi type int 3 const char `` hello world '' semicolon still termin statement implicit end line occur b b method may return multipl valu return result err pair convent way method indic error caller go c go add liter syntax initi struct paramet name initi map slice altern c 's three-stat loop go 's rang express allow concis iter array slice string map channel 58 result err c map slice 58 fmt.println `` hello world '' statement go statement separ end line hit enter key semicolon `` '' hit enter key add `` '' end line implicitli show sourc code left curli bracket come start line 59 59 type edit edit edit go number built-in type includ numer one byte int64 float32 etc boolean byte string string string immut built-in oper keyword rather function provid concaten comparison utf-8 encoding/decod 60 record type defin struct keyword 61 byte int64 float32 boolean string utf-8 60 record type struct 61 type non-neg integ constant n array type denot n array differ length thu differ type dynam array avail `` slice '' denot type t. length capac specifi new memori need alloc expand array sever slice may share underli memori 38 62 63 n array type n dynam array 38 62 63 pointer avail type pointer-to-t type denot t. address-tak indirect use oper c happen implicitli method call attribut access syntax 64 65 pointer arithmet except via special unsafe.point type standard librari 66 pointer 64 65 unsafe.point 66 pair type k v type map k v type map type-k key type-v valu though go program languag specif give perform guarante implement requir map type hash tabl built languag special syntax built-in function chan channel allow send valu type concurr go process 67 k v map k v k v chan concurr go process 67 asid support interfac go 's type system nomin type keyword use defin new name type distinct name type layout case struct member order convers type e.g. variou integ type pre-defin ad new type may defin addit convers convers name type must alway invok explicitli 68 exampl type keyword use defin type ipv4 address base 32-bit unsign integ follow interfac nomin type struct 68 type ipv4 type ipv4addr uint32 type definit ipv4addr x interpret uint32 valu x ip address simpli assign x variabl type ipv4addr type error 69 ipv4addr x uint32 x x ipv4addr 69 constant express may either type `` untyp '' given type assign type variabl valu repres pas compile-tim check 70 70 function type indic func keyword take zero paramet return zero valu type paramet return valu determin function type thu func string int32 int error type function take string 32-bit sign integ return sign integ default width valu built-in interfac type error 71 function func paramet return func string int32 int error string error 71 name type method set associ ip address exampl extend method check whether valu known standard method // zerobroadcast report whether addr 255.255.255.255. func addr ipv4addr zerobroadcast bool return addr == 0xffffffff due nomin type method definit add method ipv4addr uint32 method special definit call syntax distinct method type 72 ipv4addr uint32 72 interfac system edit edit edit go provid two featur replac class inherit citat need class inherit citat need citat need first embed view autom form composit 73 composit 73 second interfac provid runtim polymorph 74 266 interfac class type provid limit form structur type otherwis nomin type system go object interfac type also anoth type much like c++ object simultan base deriv class design go interfac inspir protocol smalltalk program languag 75 multipl sourc use term duck type describ go interfac 76 77 although term duck type precis defin therefor wrong usual impli type conform static check conform go interfac check static go compil except perform type assert go author prefer term structur type 78 interfac runtim polymorph 74 266 structur type c++ protocol 75 duck type 76 77 78 definit interfac type list requir method name type object type function exist match requir method interfac type object type well definit type need identifi type exampl shape squar circl defin shape squar circl import `` math '' type shape interfac area float64 type squar struct // note `` implement '' declar side float64 func sq squar area float64 return sq side sq side type circl struct // `` implement '' declar either radiu float64 func c circl area float64 return math pi math pow c radiu 2 squar circl implicitli shape assign shape-typ variabl 74 263–268 formal languag go 's interfac system provid structur rather nomin type interfac emb interfac effect creat combin interfac satisfi exactli type implement embed interfac method newli defin interfac add 74 270 squar circl shape shape 74 263–268 structur nomin 74 270 go standard librari use interfac provid gener sever place includ input/output system base concept reader writer 74 282–283 reader writer 74 282–283 besid call method via interfac go allow convert interfac valu type run-tim type check languag construct type assert 79 check singl potenti type 79 var shp shape squar 5 squar ok shp squar // assert squar type shp work ok fmt printf `` v\\n '' squar el fmt println `` ca n't print shape squar '' type switch 80 check multipl type citat need 80 citat need citat need func sq squar diagon float64 return sq side math sqrt2 func c circl diamet float64 return 2 c radiu func longestcontainedlin shp shape float64 switch v shp type case squar return v diagon // type assert shp squar .diagon case circl return v diamet // type assert shp circl .diamet default return 0 // practic handl error empti interfac interfac import base case refer item concret type similar object class java c satisfi type includ built-in type like int 74 284 code use empti interfac simpli call method built-in oper referred-to object store interfac valu tri convert use type via type assert type switch inspect go 's reflect packag 81 interfac refer valu limit way escap restrict static type like void c addit run-tim type check citat need interfac object java c int 74 284 interfac 81 interfac void citat need citat need interfac type use model structur data arbitrari schema go json yaml data repres map string interfac map string empti interfac recurs describ data form dictionari string key valu type 82 interfac json yaml map string interfac 82 interfac valu implement use pointer data second pointer run-tim type inform 83 like type implement use pointer go interfac valu nil uniniti 84 83 84 gener code use parameter type edit edit edit sinc version 1.18 go support gener code use parameter type 85 85 function type abil gener use type paramet type paramet specifi within squar bracket right function type name 86 compil transform gener function type non-gener substitut type argument type paramet provid either explicitli user type infer compil 87 transform process refer type instanti 88 86 87 88 interfac defin set type known type set use union oper well set method chang made support type constraint gener code gener function type constraint thought type type argument meta-typ new ~t syntax first use token go ~t mean set type whose underli type t. 89 89 type number interfac int float64 float32 int32 int64 func add number num ... var sum v rang num sum += v return sum func main add add int // type instanti println add 1 2 3 4 5 // 15 re add 1.1 2.2 3.3 4.4 5.5 // type infer println re // +1.650000e+001 enumer type edit edit edit enumer type § go edit edit go use iota keyword creat enumer constant 90 90 type byte float64 const iota // ignor first valu assign blank identifi kb byte 1 10 iota mb gb packag system edit edit edit go 's packag system packag path e.g. `` compress/bzip2 '' `` golang.org/x/net/html '' name e.g. bzip2 html refer packag definit must alway prefix packag 's name capit name packag access io.read public bzip2.read 91 go get command retriev packag store remot repositori 92 develop encourag develop packag insid base path correspond sourc repositori example.com/user_name/package_nam reduc likelihood name collis futur addit standard librari extern librari 93 91 92 93 concurr goroutin channel edit edit edit go languag built-in facil well librari support write concurr program concurr refer cpu parallel also asynchroni let slow oper like databas network read run program work common event-bas server 94 concurr program asynchroni 94 primari concurr construct goroutin type green thread 95 280–281 function call prefix go keyword start function new goroutin languag specif specifi goroutin implement current implement multiplex go process 's goroutin onto smaller set operating-system thread similar schedul perform erlang 96 10 green thread 95 280–281 operating-system thread erlang 96 10 standard librari packag featur classic concurr control structur mutex lock etc avail 96 151–152 idiomat concurr program instead prefer channel send messag goroutin 97 option buffer store messag fifo order 98 43 allow send goroutin proceed messag receiv 95 233 concurr control mutex 96 151–152 send messag 97 fifo 98 43 95 233 channel type channel type chan use transfer messag type t. special syntax use oper -ch express caus execut goroutin block valu come channel ch ch x send valu x possibl block anoth goroutin receiv valu built-in switch-lik select statement use implement non-block commun multipl channel see exampl go memori model describ goroutin must use channel oper safe share data 99 chan -ch ch ch x x switch select 99 exist channel set go apart actor model-styl concurr languag like erlang messag address directli actor correspond goroutin actor model channel actor therefor address channel mean address actor actor style simul go maintain one-to-on correspond goroutin channel languag allow multipl goroutin share channel singl goroutin send receiv multipl channel 96 147 actor model 96 147 tool one build concurr construct like worker pool pipelin say file decompress par download background call timeout `` fan-out '' parallel call set servic other 100 channel also found use usual notion interprocess commun like serv concurrency-saf list recycl buffer 101 implement coroutin help inspir name goroutin 102 implement iter 103 100 101 coroutin 102 iter 103 concurrency-rel structur convent go channel altern channel input deriv toni hoar 's commun sequenti process model unlik previou concurr program languag occam limbo languag go co-design rob pike work 104 go provid built-in notion safe verifi concurr 105 communicating-process model favor go one goroutin program share singl address space mean mutabl object pointer share goroutin see § lack data race safeti channel toni hoar 's commun sequenti process occam limbo 104 105 § lack data race safeti suitabl parallel program edit edit edit although go 's concurr featur aim primarili parallel process 94 use program shared-memori multi-processor machin variou studi done effect approach 106 one studi compar size line code speed program written season programm familiar languag correct program go expert googl 's develop team chapel cilk intel tbb studi found non-expert tend write divide-and-conqu algorithm one go statement per recurs expert wrote distribute-work-synchron program use one goroutin per processor core expert 's program usual faster also longer 107 parallel process 94 shared-memori multi-processor 106 line code chapel cilk intel tbb divide-and-conqu go 107 lack data race safeti edit edit edit go 's approach concurr summar `` n't commun share memori share memori commun '' 108 restrict goroutin access share data make data race possibl specif unless program explicitli synchron via channel mean write one goroutin might partli entir visibl anoth often guarante order write 105 furthermor go 's intern data structur like interfac valu slice header hash tabl string header immun data race type memori safeti violat multithread program modifi share instanc type without synchron 109 110 instead languag support safe concurr program thu reli convent exampl chisnal recommend idiom call `` alias xor mutabl '' mean pas mutabl valu pointer channel signal transfer ownership valu receiv 96 155 gc toolchain option data race detector check unsynchron access share memori runtim sinc version 1.1 111 addit best-effort race detector also includ default sinc version 1.6 gc runtim access map data type 112 108 data race 105 109 110 xor 96 155 111 112 binari edit edit edit linker gc toolchain creat static link binari default therefor go binari includ go runtim 113 114 113 114 omiss edit edit edit go deliber omit certain featur common languag includ implement inherit assert e pointer arithmet implicit type convers untag union f tag union g design ad facil three agre 117 implement inherit assert e pointer arithmet implicit type convers untag union f tag union g 117 omit languag featur design explicitli argu assert pointer arithmet defend choic omit type inherit give use languag encourag instead use interfac achiev dynam dispatch h composit reus code composit deleg fact larg autom struct embed accord research schmager et al. featur `` mani drawback inherit affect public interfac object fine-grain i.e method-level control embed method embed object hidden static '' make `` obviou '' whether programm overus extent programm languag reput overus inherit 73 interfac dynam dispatch h composit deleg struct 73 except handl initi omit go due lack `` design give valu proportion complex '' 118 exception-lik panic/recov mechan avoid usual try-catch control structur propos 119 releas march 30 2010 snapshot 120 go author advis use unrecover error halt entir program server request shortcut propag error stack within packag 121 122 across packag boundari go includ canon error type multi-valu return use type standard idiom 4 except handl 118 panic recov 119 120 121 122 4 style edit edit edit go author put substanti effort influenc style go program indent space surface-level detail code automat standard gofmt tool use tab indent blank align align assum editor use fixed-width font 123 golint addit style check automat deprec archiv go maintain 124 123 124 tool librari distribut go suggest standard approach thing like api document godoc 125 test go test build go build packag manag go get 125 go enforc rule recommend languag exampl ban cyclic depend unus variabl 126 import 127 implicit type convers 126 127 omiss certain featur exampl functional-program shortcut like map java-styl try/fin block tend encourag particular explicit concret imper program style day one go team publish collect go idiom 125 later also collect code review comment 128 talk 129 offici blog post 130 teach go style code philosophi 125 128 129 130 tool edit edit edit main go distribut includ tool build test analyz code build test analyz go build build go binari use inform sourc file separ makefil go test unit test microbenchmark well fuzz go fmt format code go instal retriev instal remot packag go vet static analyz look potenti error code go run shortcut build execut code godoc display document serv via http gorenam renam variabl function type-saf way go gener standard way invok code gener go mod creat new modul ad depend upgrad depend etc also includ profil debug support fuzz capabl detect bug runtim instrument exampl track garbag collect paus data race detector profil debug fuzz runtim garbag collect data race anoth tool maintain go team includ go distribut gopl languag server provid ide featur intellig code complet languag server protocol compat editor 131 ide intellig code complet languag server protocol 131 ecosystem third-parti tool add standard distribut gocod enabl code autocomplet mani text editor goimport automat adds/remov packag import need errcheck detect code might unintent ignor error exampl edit edit edit hello world edit edit edit packag main import `` fmt '' func main fmt println `` hello world '' `` fmt '' packag format i/o similar c 's c file input/output 132 i/o c file input/output 132 concurr edit edit edit follow simpl program demonstr go 's concurr featur implement asynchron program launch two lightweight thread `` goroutin '' one wait user type text implement timeout select statement wait either goroutin send messag main routin act first messag arriv exampl adapt david chisnal 's book 96 152 concurr featur select 96 152 packag main import `` fmt '' `` time '' func readword ch chan string fmt println `` type word hit enter '' var word string fmt scanf `` '' word ch word func timeout chan bool time sleep 5 time second fals func main make chan bool go timeout ch make chan string go readword ch select case word ch fmt println `` receiv '' word case fmt println `` timeout '' test edit edit edit test packag provid support autom test go packag 133 target function exampl 133 func extractusernam email string string string index email `` '' return email test code note assert keyword miss go test live filenam _test.go packag import `` test '' func testextractusernam test run `` withoutdot '' func test usernam extractusernam `` r google.com '' usernam `` r '' fatalf `` got v\\n '' usernam run `` withdot '' func test usernam extractusernam `` jonh.smith example.com '' usernam `` jonh.smith '' fatalf `` got v\\n '' usernam possibl run test parallel web app edit edit edit net/http 134 packag provid support creat web applic net/http 134 exampl would show `` hello world '' localhost:8080 visit packag main import `` fmt '' `` log '' `` net/http '' func hellofunc w http responsewrit r http request fmt fprintf w `` hello world '' func main http handlefunc `` '' hellofunc log fatal http listenandserv `` :8080 '' nil applic edit edit edit go found widespread adopt variou domain due robust standard librari ea use 135 135 popular applic includ caddi web server autom process set http 136 docker provid platform container aim ea complex softwar develop deploy 137 kubernet autom deploy scale manag container applic 138 cockroachdb distribut sql databas engin scalabl strong consist 139 hugo static site gener priorit speed flexibl allow develop creat websit effici 140 caddi 136 docker 137 kubernet 138 cockroachdb 139 hugo 140 recept edit edit edit interfac system deliber omiss inherit prais michel simionato liken characterist standard ml call `` shame popular languag follow particular rout '' 141 standard ml 141 dave astel engin yard wrote 2009 142 engin yard 142 go extrem easi dive minim number fundament languag concept syntax clean design clear unambigu go still experiment still littl rough around edg syntax go name program languag year tiob program commun index first year 2009 larger 12-month increas popular 2 month introduct novemb languag year reach 13th place januari 2010 143 surpass establish languag like pascal june 2015 rank drop 50th index place lower cobol fortran 144 januari 2017 rank surg 13th indic signific growth popular adopt go award tiob program languag year 2016 145 tiob program commun index 143 pascal cobol fortran 144 145 bruce eckel state 146 bruce eckel 146 complex c++ even complex ad new c++ result impact product longer justifi hoop c++ programm jump order use c-compat languag make sen anymor -- 're wast time effort go make much sen class problem c++ origin intend solv c++ 2011 evalu languag gc implement comparison c++ gcc java scala googl engin found gc gcc scala go offer interest languag featur also allow concis standard notat compil languag still immatur reflect perform binari size 147 evalu got rebutt go develop team ian lanc taylor improv go code hundt 's paper awar intent publish code say version `` never intend exampl idiomat effici go '' rus cox optim go code well c++ code got go code run almost fast c++ version order magnitud faster code paper 148 148 go 's nil combin lack algebra type lead difficulti handl failur base case 149 150 nil algebra type base case 149 150 go allow open brace appear line forc go programm use brace style 151 151 go critic focus simplic implement rather correct flexibl exampl languag use posix file semant platform therefor provid incorrect inform platform window follow aforement standard 152 153 posix window 152 153 studi show easi make concurr bug messag pas share memori sometim even 154 154 name disput edit edit edit novemb 10 2009 day gener releas languag franci mccabe develop go program languag note exclam point request name chang googl 's languag prevent confus languag spent 10 year develop 155 mccabe rais concern `` 'big guy end steam-rol '' concern reson 120 develop comment googl 's offici issu thread say chang name 156 even say issu contradict googl 's motto n't evil 157 go program languag 155 156 n't evil 157 octob 12 2010 file public issu ticket close googl develop rus cox rsc custom statu `` unfortun '' accompani follow comment `` mani comput product servic name go 11 month sinc releas minim confus two languag `` 157 157 see also edit edit edit free open-sourc softwar portal free open-sourc softwar portal free open-sourc softwar portal fat pointer fat pointer comparison program languag comparison program languag note edit edit edit `` allow complex statement occupi singl line semicolon may omit close '' 56 `` allow complex statement occupi singl line semicolon may omit close '' 56 56 `` newlin come token could end statement lexer insert semicolon '' 57 `` newlin come token could end statement lexer insert semicolon '' 57 57 usual exactli one result error valu valu type 's zero valu sometim read write partial complet sometim neither read return 0 byte see semipred problem multivalu return usual exactli one result error valu valu type 's zero valu sometim read write partial complet sometim neither read return 0 byte see semipred problem multivalu return semipred problem multivalu return b languag faq `` pointer arithmet safeti ... never deriv illeg address succe incorrectli ... use array indic effici ... pointer arithmet ... simplifi implement garbag collector .... '' 4 b b languag faq `` pointer arithmet safeti ... never deriv illeg address succe incorrectli ... use array indic effici ... pointer arithmet ... simplifi implement garbag collector .... '' 4 4 languag faq `` go assert ... experi programm use crutch avoid think proper error handl report .... '' 4 languag faq `` go assert ... experi programm use crutch avoid think proper error handl report .... '' 4 4 languag faq `` untag union ... would violat go 's memori safeti guarante `` 4 languag faq `` untag union ... would violat go 's memori safeti guarante `` 4 4 languag faq `` go variant type ... consid overlap confus way interfac .... ome variant type address alreadi cover ... although elegantli `` 4 tag interfac type 115 access type assert 116 languag faq `` go variant type ... consid overlap confus way interfac .... ome variant type address alreadi cover ... although elegantli `` 4 tag interfac type 115 access type assert 116 4 115 116 question `` get dynam dispatch method '' `` type inherit '' languag faq 4 question `` get dynam dispatch method '' `` type inherit '' languag faq 4 4 refer edit edit edit offici go tutori `` codewalk first-class function go '' go support first class function higher-ord function user-defin function type function liter closur multipl return valu rich featur set support function program style strongli type languag `` codewalk first-class function go '' go support first class function higher-ord function user-defin function type function liter closur multipl return valu rich featur set support function program style strongli type languag `` codewalk first-class function go '' `` go object-ori languag '' retriev april 13 2019 although go type method allow object-ori style program type hierarchi `` go object-ori languag '' retriev april 13 2019 although go type method allow object-ori style program type hierarchi `` go object-ori languag '' retriev april 13 2019 april 13 `` go code grow grace '' retriev june 24 2018 go object orient usual way `` go code grow grace '' retriev june 24 2018 go object orient usual way `` go code grow grace '' retriev june 24 2018 june 24 b c e f g h `` languag design faq '' go program languag januari 16 2010 retriev februari 27 2010 b c e f g h b c e f g h `` languag design faq '' go program languag januari 16 2010 retriev februari 27 2010 `` languag design faq '' retriev februari 27 2010 februari 27 b `` text file licens '' go program languag retriev octob 5 2012 b b `` text file licens '' go program languag retriev octob 5 2012 `` text file licens '' retriev octob 5 2012 octob 5 `` go program languag specif go program languag '' `` go program languag specif go program languag '' `` go program languag specif go program languag '' b `` n't go `` implement '' declar '' go program languag retriev octob 1 2015 b b `` n't go `` implement '' declar '' go program languag retriev octob 1 2015 `` n't go `` implement '' declar '' retriev octob 1 2015 octob 1 pike rob decemb 22 2014 `` rob pike twitter '' archiv origin april 7 2022 retriev march 13 2016 go structur type duck type full interfac satisfact check requir pike rob decemb 22 2014 `` rob pike twitter '' archiv origin april 7 2022 retriev march 13 2016 go structur type duck type full interfac satisfact check requir `` rob pike twitter '' origin retriev march 13 2016 march 13 `` lang/go go-1.4 '' openbsd port decemb 23 2014 retriev januari 19 2015 `` lang/go go-1.4 '' openbsd port decemb 23 2014 retriev januari 19 2015 `` lang/go go-1.4 '' retriev januari 19 2015 januari 19 `` go port effort '' go languag resourc cat-v. januari 12 2010 retriev januari 18 2010 `` go port effort '' go languag resourc cat-v. januari 12 2010 retriev januari 18 2010 `` go port effort '' retriev januari 18 2010 januari 18 `` addit ip right grant '' go program languag retriev octob 5 2012 `` addit ip right grant '' go program languag retriev octob 5 2012 `` addit ip right grant '' retriev octob 5 2012 octob 5 `` go introduct '' www.w3schools.com retriev novemb 23 2024 `` go introduct '' www.w3schools.com retriev novemb 23 2024 `` go introduct '' retriev novemb 23 2024 novemb 23 `` go introduct '' www.w3schools.com retriev novemb 23 2024 `` go introduct '' www.w3schools.com retriev novemb 23 2024 `` go introduct '' retriev novemb 23 2024 novemb 23 kincaid jason novemb 10 2009 `` googl 's go new program languag 's python meet c++ '' techcrunch retriev januari 18 2010 kincaid jason novemb 10 2009 `` googl 's go new program languag 's python meet c++ '' techcrunch retriev januari 18 2010 `` googl 's go new program languag 's python meet c++ '' retriev januari 18 2010 januari 18 metz cade may 5 2011 `` googl go boldli goe code gone '' regist metz cade may 5 2011 `` googl go boldli goe code gone '' regist `` googl go boldli goe code gone '' `` languag call go golang '' retriev march 16 2022 languag call go `` languag call go golang '' retriev march 16 2022 languag call go `` languag call go golang '' retriev march 16 2022 march 16 `` go 1.5 releas note '' retriev januari 28 2016 compil runtim implement go assembl without c. `` go 1.5 releas note '' retriev januari 28 2016 compil runtim implement go assembl without c. `` go 1.5 releas note '' retriev januari 28 2016 januari 28 `` go program languag '' retriev novemb 1 2024 `` go program languag '' retriev novemb 1 2024 `` go program languag '' retriev novemb 1 2024 novemb 1 `` gofrontend '' retriev novemb 1 2024 `` gofrontend '' retriev novemb 1 2024 `` gofrontend '' retriev novemb 1 2024 novemb 1 `` gccgo '' retriev novemb 1 2024. gccgo gnu compil go program languag `` gccgo '' retriev novemb 1 2024. gccgo gnu compil go program languag `` gccgo '' retriev novemb 1 2024 novemb 1 `` gollvm '' retriev novemb 1 2024 gollvm llvm-base go compil `` gollvm '' retriev novemb 1 2024 gollvm llvm-base go compil `` gollvm '' retriev novemb 1 2024 novemb 1 `` compil go javascript run go code browser gopherjs/gopherj '' github archiv origin decemb 12 2023 `` compil go javascript run go code browser gopherjs/gopherj '' github archiv origin decemb 12 2023 `` compil go javascript run go code browser gopherjs/gopherj '' github archiv `` go googl languag design servic softwar engin '' retriev octob 8 2018 `` go googl languag design servic softwar engin '' retriev octob 8 2018 `` go googl languag design servic softwar engin '' retriev octob 8 2018 octob 8 pike rob april 28 2010 `` anoth go languag design '' stanford ee comput system colloquium stanford univers video avail pike rob april 28 2010 `` anoth go languag design '' stanford ee comput system colloquium stanford univers video avail `` anoth go languag design '' stanford univers video avail `` frequent ask question faq go program languag '' go program languag retriev februari 26 2016 `` frequent ask question faq go program languag '' go program languag retriev februari 26 2016 `` frequent ask question faq go program languag '' retriev februari 26 2016 februari 26 binstock andrew may 18 2011 `` dr. dobb 's interview ken thompson '' archiv origin januari 5 2013 retriev februari 7 2014 binstock andrew may 18 2011 `` dr. dobb 's interview ken thompson '' archiv origin januari 5 2013 retriev februari 7 2014 `` dr. dobb 's interview ken thompson '' origin retriev februari 7 2014 februari 7 pike rob 2012 `` less exponenti '' pike rob 2012 `` less exponenti '' `` less exponenti '' griesem robert 2015 `` evolut go '' griesem robert 2015 `` evolut go '' `` evolut go '' griesem robert pike rob thompson ken taylor ian cox rus kim jini langley adam `` hey ho let 's go '' googl open sourc retriev may 17 2018 griesem robert pike rob thompson ken taylor ian cox rus kim jini langley adam `` hey ho let 's go '' googl open sourc retriev may 17 2018 `` hey ho let 's go '' retriev may 17 2018 may 17 shankland stephen march 30 2012 `` googl 's go languag turn one win spot youtub lower-level program languag matur enough sport 1.0 version number 's use real work googl '' news cnet cb interact inc. retriev august 6 2017 googl releas version 1 go program languag ambiti attempt improv upon giant lower-level program world c c++ shankland stephen march 30 2012 `` googl 's go languag turn one win spot youtub lower-level program languag matur enough sport 1.0 version number 's use real work googl '' news cnet cb interact inc. retriev august 6 2017 googl releas version 1 go program languag ambiti attempt improv upon giant lower-level program world c c++ `` googl 's go languag turn one win spot youtub lower-level program languag matur enough sport 1.0 version number 's use real work googl '' retriev august 6 2017 august 6 `` releas histori '' go program languag `` releas histori '' go program languag `` releas histori '' `` go faq googl use go intern '' retriev march 9 2013 `` go faq googl use go intern '' retriev march 9 2013 `` go faq googl use go intern '' retriev march 9 2013 march 9 `` go gopher go program languag '' go.dev retriev februari 9 2023 `` go gopher go program languag '' go.dev retriev februari 9 2023 `` go gopher go program languag '' retriev februari 9 2023 februari 9 `` go font '' go novemb 16 2016 retriev march 12 2019 `` go font '' go novemb 16 2016 retriev march 12 2019 `` go font '' retriev march 12 2019 march 12 `` go font ttf '' github retriev april 2 2019 `` go font ttf '' github retriev april 2 2019 `` go font ttf '' retriev april 2 2019 april 2 `` go 's new brand '' go blog retriev novemb 9 2018 `` go 's new brand '' go blog retriev novemb 9 2018 `` go 's new brand '' retriev novemb 9 2018 novemb 9 merrick alic march 9 2021 `` go develop survey 2020 result '' go program languag retriev march 16 2022 merrick alic march 9 2021 `` go develop survey 2020 result '' go program languag retriev march 16 2022 `` go develop survey 2020 result '' retriev march 16 2022 march 16 b pike rob septemb 26 2013 `` array slice string mechan 'append '' go blog retriev march 7 2015 b b pike rob septemb 26 2013 `` array slice string mechan 'append '' go blog retriev march 7 2015 `` array slice string mechan 'append '' retriev march 7 2015 march 7 `` e2e erik meijer robert griesem '' channel 9 microsoft may 7 2012 `` e2e erik meijer robert griesem '' channel 9 microsoft may 7 2012 `` e2e erik meijer robert griesem '' `` go 2 draft design '' retriev septemb 12 2018 `` go 2 draft design '' retriev septemb 12 2018 `` go 2 draft design '' retriev septemb 12 2018 septemb 12 `` go blog go 2 draft design '' august 28 2018 `` go blog go 2 draft design '' august 28 2018 `` go blog go 2 draft design '' `` propos built-in go error check function `` tri '' '' go repositori github retriev march 16 2022 `` propos built-in go error check function `` tri '' '' go repositori github retriev march 16 2022 `` propos built-in go error check function `` tri '' '' retriev march 16 2022 march 16 `` type paramet — draft design '' go.googlesource.com `` type paramet — draft design '' go.googlesource.com `` type paramet — draft design '' `` gener go '' bitfieldconsulting.com decemb 17 2021 `` gener go '' bitfieldconsulting.com decemb 17 2021 `` gener go '' `` go 1.18 releas '' go program languag march 15 2022 retriev march 16 2022 `` go 1.18 releas '' go program languag march 15 2022 retriev march 16 2022 `` go 1.18 releas '' retriev march 16 2022 march 16 `` go 1 futur go program '' go program languag `` go 1 futur go program '' go program languag `` go 1 futur go program '' `` go 1.23 releas note '' go program languag `` go 1.23 releas note '' go program languag `` go 1.23 releas note '' `` releas histori '' go program languag `` releas histori '' go program languag `` releas histori '' `` backward compat go 1.21 go 2 '' go program languag `` backward compat go 1.21 go 2 '' go program languag `` backward compat go 1.21 go 2 '' `` quick guid go 's assembl '' go.dev retriev decemb 31 2021 `` quick guid go 's assembl '' go.dev retriev decemb 31 2021 `` quick guid go 's assembl '' retriev decemb 31 2021 decemb 31 pike rob `` go program languag '' youtub retriev juli 1 2011 pike rob `` go program languag '' youtub retriev juli 1 2011 `` go program languag '' retriev juli 1 2011 juli 1 pike rob novemb 10 2009 go program languag flv tech talk googl event occur 8:53 pike rob novemb 10 2009 go program languag flv tech talk googl event occur 8:53 go program languag flv `` download instal packag depend '' see godoc.org address document packag `` download instal packag depend '' see godoc.org address document packag `` download instal packag depend '' godoc.org `` godoc '' godoc.org `` godoc '' godoc.org `` godoc '' pike rob `` changelog '' podcast archiv origin octob 20 2013 retriev octob 7 2013 pike rob `` changelog '' podcast archiv origin octob 20 2013 retriev octob 7 2013 `` changelog '' origin retriev octob 7 2013 octob 7 `` go program languag specif §semicolon '' go program languag `` go program languag specif §semicolon '' go program languag `` go program languag specif §semicolon '' `` effect go §semicolon '' go program languag `` effect go §semicolon '' go program languag `` effect go §semicolon '' `` go program languag specif '' go program languag `` go program languag specif '' go program languag `` go program languag specif '' `` go syntax '' www.w3schools.com retriev novemb 23 2024 `` go syntax '' www.w3schools.com retriev novemb 23 2024 `` go syntax '' retriev novemb 23 2024 novemb 23 pike rob octob 23 2013 `` string byte rune charact go '' pike rob octob 23 2013 `` string byte rune charact go '' `` string byte rune charact go '' doxsey caleb `` struct interfac — introduct program go '' www.golang-book.com retriev octob 15 2018 doxsey caleb `` struct interfac — introduct program go '' www.golang-book.com retriev octob 15 2018 `` struct interfac — introduct program go '' retriev octob 15 2018 octob 15 gerrand andrew `` go slice usag intern '' gerrand andrew `` go slice usag intern '' `` go slice usag intern '' go author `` effect go slice '' go author `` effect go slice '' `` effect go slice '' go author `` selector '' go author `` selector '' `` selector '' go author `` call '' go author `` call '' `` call '' `` go program languag specif §packag unsaf '' go program languag `` go program languag specif §packag unsaf '' go program languag `` go program languag specif §packag unsaf '' `` go program languag specif '' go.dev retriev decemb 31 2021 `` go program languag specif '' go.dev retriev decemb 31 2021 `` go program languag specif '' retriev decemb 31 2021 decemb 31 `` go program languag specif '' go program languag `` go program languag specif '' go program languag `` go program languag specif '' `` tour go '' go.dev `` tour go '' go.dev `` tour go '' `` go program languag specif '' go program languag `` go program languag specif '' go program languag `` go program languag specif '' `` go program languag specif '' go.dev retriev decemb 31 2021 `` go program languag specif '' go.dev retriev decemb 31 2021 `` go program languag specif '' retriev decemb 31 2021 decemb 31 `` go program languag specif '' go program languag `` go program languag specif '' go program languag `` go program languag specif '' b schmager frank cameron nichola nobl jame 2010 gohotdraw evalu go program languag design pattern evalu usabl program languag tool acm b b schmager frank cameron nichola nobl jame 2010 gohotdraw evalu go program languag design pattern evalu usabl program languag tool acm b c e balbaert ivo 2012 way go thorough introduct go program languag iunivers b c e b c e balbaert ivo 2012 way go thorough introduct go program languag iunivers `` evolut go '' talks.golang.org retriev march 13 2016 `` evolut go '' talks.golang.org retriev march 13 2016 `` evolut go '' retriev march 13 2016 march 13 diggin christoph novemb 24 2009 `` duck type go program languag '' dr. dobb 's world softwar develop retriev march 10 2016 diggin christoph novemb 24 2009 `` duck type go program languag '' dr. dobb 's world softwar develop retriev march 10 2016 `` duck type go program languag '' retriev march 10 2016 march 10 ryer mat decemb 1 2015 `` duck type go '' retriev march 10 2016 ryer mat decemb 1 2015 `` duck type go '' retriev march 10 2016 `` duck type go '' retriev march 10 2016 march 10 `` frequent ask question faq go program languag '' go program languag `` frequent ask question faq go program languag '' go program languag `` frequent ask question faq go program languag '' `` go program languag specif '' go program languag `` go program languag specif '' go program languag `` go program languag specif '' `` go program languag specif '' go program languag `` go program languag specif '' go program languag `` go program languag specif '' `` reflect packag '' pkg.go.dev `` reflect packag '' pkg.go.dev `` reflect packag '' `` map string interfac go '' bitfieldconsulting.com june 6 2020 `` map string interfac go '' bitfieldconsulting.com june 6 2020 `` map string interfac go '' `` go data structur interfac '' retriev novemb 15 2012 `` go data structur interfac '' retriev novemb 15 2012 `` go data structur interfac '' retriev novemb 15 2012 novemb 15 `` go program languag specif '' go program languag `` go program languag specif '' go program languag `` go program languag specif '' `` go 1.18 releas note gener '' go program languag march 15 2022 retriev march 16 2022 `` go 1.18 releas note gener '' go program languag march 15 2022 retriev march 16 2022 `` go 1.18 releas note gener '' retriev march 16 2022 march 16 `` type paramet propos '' go.googlesource.com retriev june 25 2023 `` type paramet propos '' go.googlesource.com retriev june 25 2023 `` type paramet propos '' retriev june 25 2023 june 25 `` go program languag specif go program languag '' go.dev retriev june 25 2023 `` go program languag specif go program languag '' go.dev retriev june 25 2023 `` go program languag specif go program languag '' retriev june 25 2023 june 25 `` introduct gener go program languag '' go.dev retriev june 25 2023 `` introduct gener go program languag '' go.dev retriev june 25 2023 `` introduct gener go program languag '' retriev june 25 2023 june 25 `` type paramet propos '' go.googlesource.com retriev june 25 2023 `` type paramet propos '' go.googlesource.com retriev june 25 2023 `` type paramet propos '' retriev june 25 2023 june 25 `` effect go '' golang.org go author retriev may 13 2014 `` effect go '' golang.org go author retriev may 13 2014 `` effect go '' retriev may 13 2014 may 13 `` tutori go program languag '' go program languag retriev march 10 2013 go rule visibl inform simpl name top-level type function method constant variabl structur field method capit user packag may see otherwis name henc thing name visibl insid packag declar `` tutori go program languag '' go program languag retriev march 10 2013 go rule visibl inform simpl name top-level type function method constant variabl structur field method capit user packag may see otherwis name henc thing name visibl insid packag declar `` tutori go program languag '' retriev march 10 2013 march 10 `` go '' go program languag `` go '' go program languag `` go '' `` write go code '' go program languag packag standard librari given short import path `` fmt '' `` net/http '' packag must choos base path unlik collid futur addit standard librari extern librari keep code sourc repositori somewher use root sourc repositori base path instanc exampl account example.com/us base path `` write go code '' go program languag packag standard librari given short import path `` fmt '' `` net/http '' packag must choos base path unlik collid futur addit standard librari extern librari keep code sourc repositori somewher use root sourc repositori base path instanc exampl account example.com/us base path `` write go code '' b pike rob septemb 18 2012 `` concurr parallel '' b b pike rob septemb 18 2012 `` concurr parallel '' `` concurr parallel '' b donovan alan a. kernighan brian w. 2016 go program languag addison-wesley profession comput seri new york munich addison-wesley isbn 978-0-13-419044-0 b b donovan alan a. kernighan brian w. 2016 go program languag addison-wesley profession comput seri new york munich addison-wesley isbn 978-0-13-419044-0 isbn 978-0-13-419044-0 b c e chisnal david 2012 go program languag phrasebook addison-wesley isbn 9780132919005 b c e b c e chisnal david 2012 go program languag phrasebook addison-wesley isbn 9780132919005 go program languag phrasebook isbn 9780132919005 `` effect go '' go program languag `` effect go '' go program languag `` effect go '' summerfield mark 2012 program go creat applic 21st centuri addison-wesley summerfield mark 2012 program go creat applic 21st centuri addison-wesley `` go memori model '' retriev april 10 2017 `` go memori model '' retriev april 10 2017 `` go memori model '' retriev april 10 2017 april 10 `` go concurr pattern '' go program languag `` go concurr pattern '' go program languag `` go concurr pattern '' graham-cum john august 24 2013 `` recycl memori buffer go '' graham-cum john august 24 2013 `` recycl memori buffer go '' `` recycl memori buffer go '' `` tree.go '' `` tree.go '' `` tree.go '' cheslack-postava ewen `` iter go '' cheslack-postava ewen `` iter go '' `` iter go '' kernighan brian w. `` descent limbo '' kernighan brian w. `` descent limbo '' `` descent limbo '' b `` go memori model '' retriev januari 5 2011 b b `` go memori model '' retriev januari 5 2011 `` go memori model '' retriev januari 5 2011 januari 5 tang peiyi 2010 multi-cor parallel program go pdf proc first intern confer advanc comput commun archiv origin pdf septemb 9 2016 retriev may 14 2015 tang peiyi 2010 multi-cor parallel program go pdf proc first intern confer advanc comput commun archiv origin pdf septemb 9 2016 retriev may 14 2015 multi-cor parallel program go pdf origin pdf retriev may 14 2015 may 14 nanz sebastian west scott soar da silveira kaue examin expert gap parallel program pdf euro-par 2013 citeseerx 10.1.1.368.6137 nanz sebastian west scott soar da silveira kaue examin expert gap parallel program pdf euro-par 2013 citeseerx 10.1.1.368.6137 examin expert gap parallel program pdf citeseerx 10.1.1.368.6137 10.1.1.368.6137 go author `` share memori commun '' go author `` share memori commun '' `` share memori commun '' cox rus `` race '' cox rus `` race '' cox rus `` race '' pike rob octob 25 2012 `` go googl languag design servic softwar engin '' googl inc. `` one import caveat go pure memori safe presenc concurr '' pike rob octob 25 2012 `` go googl languag design servic softwar engin '' googl inc. `` one import caveat go pure memori safe presenc concurr '' pike rob `` go googl languag design servic softwar engin '' `` introduc go race detector '' go blog retriev june 26 2013 `` introduc go race detector '' go blog retriev june 26 2013 `` introduc go race detector '' retriev june 26 2013 june 26 `` go 1.6 releas note go program languag '' go.dev retriev novemb 17 2023 `` go 1.6 releas note go program languag '' go.dev retriev novemb 17 2023 `` go 1.6 releas note go program languag '' retriev novemb 17 2023 novemb 17 `` frequent ask question faq go program languag '' `` frequent ask question faq go program languag '' `` frequent ask question faq go program languag '' `` stori fat go binari '' septemb 21 2018 `` stori fat go binari '' septemb 21 2018 `` stori fat go binari '' `` go program languag specif §interfac type '' go program languag `` go program languag specif §interfac type '' go program languag `` go program languag specif §interfac type '' `` go program languag specif §type assert '' go program languag `` go program languag specif §type assert '' go program languag `` go program languag specif §type assert '' `` system go '' informit interview august 17 2010 retriev june 21 2018 `` system go '' informit interview august 17 2010 retriev june 21 2018 `` system go '' retriev june 21 2018 june 21 `` languag design faq '' novemb 13 2009 archiv origin novemb 13 2009 `` languag design faq '' novemb 13 2009 archiv origin novemb 13 2009 `` languag design faq '' origin `` propos exception-lik mechan '' golang-nut march 25 2010 retriev march 25 2010 `` propos exception-lik mechan '' golang-nut march 25 2010 retriev march 25 2010 `` propos exception-lik mechan '' retriev march 25 2010 march 25 `` weekli snapshot histori '' go program languag `` weekli snapshot histori '' go program languag `` weekli snapshot histori '' `` panic recov '' go wiki `` panic recov '' go wiki `` panic recov '' `` effect go '' go program languag `` effect go '' go program languag `` effect go '' `` gofmt '' go program languag retriev februari 5 2021 `` gofmt '' go program languag retriev februari 5 2021 `` gofmt '' retriev februari 5 2021 februari 5 `` golang/lint public archiv '' github.com novemb 30 2022 `` golang/lint public archiv '' github.com novemb 30 2022 `` golang/lint public archiv '' b `` effect go '' go program languag b b `` effect go '' go program languag `` effect go '' `` unus local variabl '' yourbasic.org retriev februari 11 2021 `` unus local variabl '' yourbasic.org retriev februari 11 2021 `` unus local variabl '' retriev februari 11 2021 februari 11 `` unus packag import '' yourbasic.org retriev februari 11 2021 `` unus packag import '' yourbasic.org retriev februari 11 2021 `` unus packag import '' retriev februari 11 2021 februari 11 `` code review comment '' github retriev juli 3 2018 `` code review comment '' github retriev juli 3 2018 `` code review comment '' github retriev juli 3 2018 juli 3 `` talk '' retriev juli 3 2018 `` talk '' retriev juli 3 2018 `` talk '' retriev juli 3 2018 juli 3 `` error valu '' retriev juli 3 2018 `` error valu '' retriev juli 3 2018 `` error valu '' retriev juli 3 2018 juli 3 `` tools/gopls/readme.md master · golang/tool '' github retriev novemb 17 2023 `` tools/gopls/readme.md master · golang/tool '' github retriev novemb 17 2023 `` tools/gopls/readme.md master · golang/tool '' retriev novemb 17 2023 novemb 17 `` fmt '' go program languag retriev april 8 2019 `` fmt '' go program languag retriev april 8 2019 `` fmt '' retriev april 8 2019 april 8 `` test '' go program languag retriev decemb 27 2020 `` test '' go program languag retriev decemb 27 2020 `` test '' retriev decemb 27 2020 decemb 27 `` http packag net/http go packag '' pkg.go.dev retriev novemb 23 2024 `` http packag net/http go packag '' pkg.go.dev retriev novemb 23 2024 `` http packag net/http go packag '' retriev novemb 23 2024 novemb 23 lee wei-meng novemb 24 2022 `` introduct go program languag '' compon develop magazin archiv origin june 5 2023 retriev septemb 8 2023 lee wei-meng novemb 24 2022 `` introduct go program languag '' compon develop magazin archiv origin june 5 2023 retriev septemb 8 2023 `` introduct go program languag '' compon develop magazin origin retriev septemb 8 2023 septemb 8 hoffmann frank neumey mandi august 2018 `` simpli secur '' linux magazin 213 archiv origin may 28 2023 retriev septemb 8 2023 hoffmann frank neumey mandi august 2018 `` simpli secur '' linux magazin 213 archiv origin may 28 2023 retriev septemb 8 2023 `` simpli secur '' linux magazin origin retriev septemb 8 2023 septemb 8 lee wei-meng august 31 2022 `` introduct container use docker '' code magazin archiv origin may 30 2023 retriev septemb 8 2023 lee wei-meng august 31 2022 `` introduct container use docker '' code magazin archiv origin may 30 2023 retriev septemb 8 2023 `` introduct container use docker '' code magazin archiv retriev septemb 8 2023 septemb 8 pirker alexand februari 24 2023 `` kubernet secur starter '' code magazin archiv origin april 1 2023 retriev septemb 8 2023 pirker alexand februari 24 2023 `` kubernet secur starter '' code magazin archiv origin april 1 2023 retriev septemb 8 2023 `` kubernet secur starter '' code magazin archiv retriev septemb 8 2023 septemb 8 taft rebecca sharif irfan matei andrei van benschoten nathan lewi jordan grieger tobia niemi kai wood andi birzin ann poss raphael bardea paul ranad amruta darnel ben gruneir bram jaffray justin zhang luci matti peter june 11 2020 `` cockroachdb resili geo-distribut sql databas '' proceed 2020 acm sigmod intern confer manag data sigmod '20 pp 1493–1509 doi:10.1145/3318464.3386134 isbn 978-1-4503-6735-6 taft rebecca sharif irfan matei andrei van benschoten nathan lewi jordan grieger tobia niemi kai wood andi birzin ann poss raphael bardea paul ranad amruta darnel ben gruneir bram jaffray justin zhang luci matti peter june 11 2020 `` cockroachdb resili geo-distribut sql databas '' proceed 2020 acm sigmod intern confer manag data sigmod '20 pp 1493–1509 doi:10.1145/3318464.3386134 isbn 978-1-4503-6735-6 sigmod 1493– doi 10.1145/3318464.3386134 10.1145/3318464.3386134 isbn 978-1-4503-6735-6 hopkin brandon septemb 13 2022 `` static site gener hugo '' linux journal archiv origin april 8 2023 retriev septemb 8 2023 hopkin brandon septemb 13 2022 `` static site gener hugo '' linux journal archiv origin april 8 2023 retriev septemb 8 2023 `` static site gener hugo '' archiv retriev septemb 8 2023 septemb 8 simionato michel novemb 15 2009 `` interfac v inherit watch go '' artima retriev novemb 15 2009 simionato michel novemb 15 2009 `` interfac v inherit watch go '' artima retriev novemb 15 2009 `` interfac v inherit watch go '' retriev novemb 15 2009 novemb 15 astel dave novemb 9 2009 `` readi set go '' engineyard archiv origin octob 19 2018 retriev novemb 9 2009 astel dave novemb 9 2009 `` readi set go '' engineyard archiv origin octob 19 2018 retriev novemb 9 2009 `` readi set go '' origin retriev novemb 9 2009 novemb 9 jt januari 11 2010 `` googl 's go win program languag year award '' jaxent retriev decemb 5 2012 jt januari 11 2010 `` googl 's go win program languag year award '' jaxent retriev decemb 5 2012 `` googl 's go win program languag year award '' retriev decemb 5 2012 decemb 5 `` tiob program commun index june 2015 '' tiob softwar june 2015 retriev juli 5 2015 `` tiob program commun index june 2015 '' tiob softwar june 2015 retriev juli 5 2015 `` tiob program commun index june 2015 '' retriev juli 5 2015 juli 5 `` tiob index '' tiob retriev juli 15 2024 `` tiob index '' tiob retriev juli 15 2024 `` tiob index '' retriev juli 15 2024 juli 15 eckel bruce august 27 2011 `` call go python via json-rpc '' retriev august 29 2011 eckel bruce august 27 2011 `` call go python via json-rpc '' retriev august 29 2011 `` call go python via json-rpc '' retriev august 29 2011 august 29 hundt robert 2011 loop recognit c++/java/go/scala pdf scala day hundt robert 2011 loop recognit c++/java/go/scala pdf scala day loop recognit c++/java/go/scala pdf metz cade juli 1 2011 `` googl go strike back c++ bake-off '' regist metz cade juli 1 2011 `` googl go strike back c++ bake-off '' regist `` googl go strike back c++ bake-off '' regist yager `` go good '' retriev novemb 4 2018 yager `` go good '' retriev novemb 4 2018 `` go good '' retriev novemb 4 2018 novemb 4 dobronszki jano `` everyday hassl go '' retriev novemb 4 2018 dobronszki jano `` everyday hassl go '' retriev novemb 4 2018 `` everyday hassl go '' retriev novemb 4 2018 novemb 4 `` brace semicolon ca n't put open brace next line '' retriev march 26 2020 advantag singl programmat mandat format go program greatli outweigh perceiv disadvantag particular style `` brace semicolon ca n't put open brace next line '' retriev march 26 2020 advantag singl programmat mandat format go program greatli outweigh perceiv disadvantag particular style `` brace semicolon ca n't put open brace next line '' retriev march 26 2020 march 26 `` want mr. golang 's wild ride '' februari 28 2020 retriev novemb 17 2020 `` want mr. golang 's wild ride '' februari 28 2020 retriev novemb 17 2020 `` want mr. golang 's wild ride '' retriev novemb 17 2020 novemb 17 `` propos o create/open/openfil set file_share_delet window 32088 '' github may 16 2019 retriev novemb 17 2020 `` propos o create/open/openfil set file_share_delet window 32088 '' github may 16 2019 retriev novemb 17 2020 `` propos o create/open/openfil set file_share_delet window 32088 '' github retriev novemb 17 2020 novemb 17 tu tengfei 2019 `` understand real-world concurr bug go '' pdf exampl around 58 block bug caus messag pas addit violat go 's channel usag rule e.g. wait channel one send data close mani concurr bug caus mix usag messag pas new semant new librari go easili overlook hard detect tu tengfei 2019 `` understand real-world concurr bug go '' pdf exampl around 58 block bug caus messag pas addit violat go 's channel usag rule e.g. wait channel one send data close mani concurr bug caus mix usag messag pas new semant new librari go easili overlook hard detect `` understand real-world concurr bug go '' pdf brownle john novemb 13 2009 `` googl n't googl `` go '' name program languag '' archiv origin decemb 8 2015 retriev may 26 2016 brownle john novemb 13 2009 `` googl n't googl `` go '' name program languag '' archiv origin decemb 8 2015 retriev may 26 2016 `` googl n't googl `` go '' name program languag '' origin retriev may 26 2016 may 26 claburn thoma novemb 11 2009 `` googl 'go name bring accus evil '' informationweek archiv origin juli 22 2010 retriev januari 18 2010 claburn thoma novemb 11 2009 `` googl 'go name bring accus evil '' informationweek archiv origin juli 22 2010 retriev januari 18 2010 `` googl 'go name bring accus evil '' origin retriev januari 18 2010 januari 18 b `` issu 9 go — alreadi use name program languag '' github googl inc. retriev octob 12 2010 b b `` issu 9 go — alreadi use name program languag '' github googl inc. retriev octob 12 2010 `` issu 9 go — alreadi use name program languag '' googl inc. retriev octob 12 2010 octob 12 read edit edit edit donovan alan kernighan brian octob 2015 go program languag 1st ed. addison-wesley profession p. 400 isbn 978-0-13-419044-0 go program languag addison-wesley profession isbn 978-0-13-419044-0 bodner jon march 2021 learn go 1st ed. o'reilli p. 352 isbn 9781492077213 learn go o'reilli isbn 9781492077213 extern link edit edit edit go program languag go program languag offici websit offici websit offici websit offici websit v v e e program languag comparison comparison timelin timelin histori histori ada ada algol simula algol simula simula apl apl assembl assembl basic visual basic classic .net basic visual basic classic .net visual basic classic classic .net .net c c c++ c++ c c cobol cobol erlang erlang forth forth fortran fortran go go haskel haskel java java javascript javascript julia julia kotlin kotlin lisp lisp lua lua matlab matlab ml ml pascal object pascal pascal object pascal object pascal perl perl php php prolog prolog python python r r rubi rubi rust rust sql sql scratch scratch shell shell smalltalk smalltalk swift swift ... ... list alphabet alphabet categor categor gener gener non-english-bas non-english-bas categori categori v v e e googl chromium chromium gemma gemma openrefin openrefin tesseract tesseract carbon carbon dart dart go go sawzal sawzal amp amp angular angular angularj angularj beam beam bazel bazel blockli blockli closur tool closur tool cpplint cpplint flatbuff flatbuff flutter flutter ganeti ganeti gear gear gerrit gerrit glop glop grpc grpc gson gson guava guava guetzli guetzli guic guic gvisor gvisor kubernet kubernet leveldb leveldb libvpx libvpx lighthous lighthous nacl nacl namebench namebench nomulu nomulu or-tool or-tool polym polym protocol buffer protocol buffer tensorflow tensorflow v8 v8 android android chromiumo chromiumo fuchsia fuchsia glinux glinux goobuntu goobuntu code-in code-in googl llc v. oracl america inc. googl llc v. oracl america inc. open sourc secur foundat open sourc secur foundat summer code summer code v v e e rob pike plan 9 bell lab plan 9 bell lab inferno inferno newsqueak newsqueak limbo limbo sawzal sawzal go go acm acm blit blit sam sam rio rio 8½ 8½ practic program practic program unix program environ unix program environ rené french rené french mark v. shaney mark v. shaney utf-8 utf-8 v v e e ken thompson unix unix plan 9 bell lab plan 9 bell lab inferno inferno b b go go bell bell ed ed grep grep sam sam space travel space travel thompson shell thompson shell bell lab bell lab googl googl utf-8 utf-8 author control databas fast fast fast germani germani germani unit state unit state unit state unit state israel israel israel http //en.wikipedia.org/w/index.php title=go_ programming_languag oldid=1269146554 categori go program languag go program languag american invent american invent c program languag famili c program languag famili concurr program languag concurr program languag cross-platform free softwar cross-platform free softwar cross-platform softwar cross-platform softwar free open sourc compil free open sourc compil googl softwar googl softwar high-level program languag high-level program languag procedur program languag procedur program languag program languag program languag program languag creat 2009 program languag creat 2009 softwar use bsd licens softwar use bsd licens static type program languag static type program languag system program languag system program languag articl short descript articl short descript short descript differ wikidata short descript differ wikidata use american english august 2022 use american english august 2022 wikipedia articl written american english wikipedia articl written american english use mdi date octob 2018 use mdi date octob 2018 articl fail verif articl fail verif articl fail verif june 2022 articl fail verif june 2022 articl unsourc statement articl unsourc statement articl unsourc statement decemb 2016 articl unsourc statement decemb 2016 articl excerpt articl excerpt common categori link wikidata common categori link wikidata articl prone spam june 2013 articl prone spam june 2013 page last edit 13 januari 2025 07:15 utc utc text avail creativ common attribution-sharealik 4.0 licens addit term may appli use site agre term use privaci polici wikipedia® regist trademark wikimedia foundat inc. non-profit organ creativ common attribution-sharealik 4.0 licens term use privaci polici wikimedia foundat inc. privaci polici privaci polici wikipedia wikipedia disclaim disclaim contact wikipedia contact wikipedia code conduct code conduct develop develop statist statist cooki statement cooki statement mobil view mobil view",
    "https://en.wikipedia.org/wiki/Swift_(programming_language)": "jump content main menu main page main page main page content content content current event current event current event random articl random articl random articl wikipedia wikipedia wikipedia contact u contact u contact u help help help learn edit learn edit learn edit commun portal commun portal commun portal recent chang recent chang recent chang upload file upload file upload file search search appear donat donat donat creat account creat account creat account log log log person tool donat donat donat creat account creat account creat account log log log learn learn contribut contribut contribut talk talk talk content top top 1 histori toggl histori subsect 1.1 platform 1.2 version histori 1 histori 1 histori toggl histori subsect 1.1 platform 1.1 platform 1.1 platform 1.2 version histori 1.2 version histori 1.2 version histori 2 featur toggl featur subsect 2.1 basic syntax 2.2 closur support 2.3 string support 2.4 callabl object 2.5 access control 2.6 option chain 2.7 valu type 2.8 extens 2.9 protocol-ori program 2.10 concurr 2.11 librari runtim develop 2.12 memori manag 2.13 debug 2 featur 2 featur toggl featur subsect 2.1 basic syntax 2.1 basic syntax 2.1 basic syntax 2.2 closur support 2.2 closur support 2.2 closur support 2.3 string support 2.3 string support 2.3 string support 2.4 callabl object 2.4 callabl object 2.4 callabl object 2.5 access control 2.5 access control 2.5 access control 2.6 option chain 2.6 option chain 2.6 option chain 2.7 valu type 2.7 valu type 2.7 valu type 2.8 extens 2.8 extens 2.8 extens 2.9 protocol-ori program 2.9 protocol-ori program 2.9 protocol-ori program 2.10 concurr 2.10 concurr 2.10 concurr 2.11 librari runtim develop 2.11 librari runtim develop 2.11 librari runtim develop 2.12 memori manag 2.12 memori manag 2.12 memori manag 2.13 debug 2.13 debug 2.13 debug 3 comparison languag 3 comparison languag 3 comparison languag 4 develop implement 4 develop implement 4 develop implement 5 see also 5 see also 5 see also 6 refer 6 refer 6 refer 7 extern link 7 extern link 7 extern link toggl tabl content swift program languag swift program languag 47 languag العربية العربية العربية azərbaycanca azərbaycanca azərbaycanca বাংলা বাংলা বাংলা беларуская беларуская беларуская български български български català català català čeština čeština čeština deutsch deutsch deutsch ελληνικά ελληνικά ελληνικά español español español esperanto esperanto esperanto فارسی فارسی فارسی françai françai françai 한국어 한국어 한국어 հայերեն հայերեն հայերեն bahasa indonesia bahasa indonesia bahasa indonesia íslenska íslenska íslenska italiano italiano italiano עברית עברית עברית кыргызча кыргызча кыргызча latina latina latina magyar magyar magyar മലയാളം മലയാളം മലയാളം bahasa melayu bahasa melayu bahasa melayu nederland nederland nederland 日本語 日本語 日本語 norsk bokmål norsk bokmål norsk bokmål oʻzbekcha ўзбекча oʻzbekcha ўзбекча oʻzbekcha ўзбекча polski polski polski portuguê portuguê portuguê qaraqalpaqsha qaraqalpaqsha qaraqalpaqsha română română română русский русский русский simpl english simpl english simpl english slovenčina slovenčina slovenčina slovenščina slovenščina slovenščina کوردی کوردی کوردی српски srpski српски srpski српски srpski suomi suomi suomi svenska svenska svenska тоҷикӣ тоҷикӣ тоҷикӣ türkçe türkçe türkçe українська українська українська tiếng việt tiếng việt tiếng việt 吴语 吴语 吴语 粵語 粵語 粵語 中文 中文 中文 edit link edit link articl articl articl talk talk talk english read read read edit edit edit view histori view histori view histori tool read read read edit edit edit view histori view histori view histori link link link relat chang relat chang relat chang upload file upload file upload file special page special page special page perman link perman link perman link page inform page inform page inform cite page cite page cite page get shorten url get shorten url get shorten url download qr code download qr code download qr code download pdf download pdf download pdf printabl version printabl version printabl version wikimedia common wikimedia common wikimedia common wikibook wikibook wikibook wikidata item wikidata item wikidata item swift parallel script languag swift cpu paradigm multi-paradigm protocol-ori object-ori function imper block structur declar concurr design chri lattner appl inc. 1 develop appl inc. 10 year ago 2014-06-02 2014-06-02 2 stabl releas 3 30 day ago 13 decemb 2024 13 decemb 2024 preview releas type disciplin static strong infer memori manag automat refer count o darwin io ipado maco tvo visiono watcho linux window android z/o licens apach licens 2.0 proprietari 4 5 filenam extens www.swift.org www.swift.org www.swift.org developer.apple.com/swift/ developer.apple.com/swift/ developer.apple.com/swift/ objective-c 6 rust haskel rubi python c clu 7 8 rust 9 v vlang 10 swift high-level general-purpos multi-paradigm compil program languag creat chri lattner 2010 appl inc. maintain open-sourc commun swift compil machin code use llvm-base compil swift first releas june 2014 11 swift toolchain ship xcode sinc xcode version 6 releas septemb 2014. high-level general-purpos multi-paradigm compil program languag chri lattner appl inc. open-sourc commun llvm 11 xcode appl intend swift support mani core concept associ objective-c notabl dynam dispatch widespread late bind extens program similar featur `` safer '' way make easier catch softwar bug swift featur address common program error like null pointer dereferenc provid syntact sugar help avoid pyramid doom swift support concept protocol extens extens system appli type struct class appl promot real chang program paradigm term `` protocol-ori program '' 12 similar trait type class 13 objective-c dynam dispatch late bind extens program softwar bug null pointer dereferenc syntact sugar pyramid doom protocol struct class 12 trait type class 13 swift introduc appl 's 2014 worldwid develop confer wwdc 14 underw upgrad version 1.2 2014 major upgrad swift 2 wwdc 2015 initi proprietari languag version 2.2 made open-sourc softwar apach licens 2.0 decemb 3 2015 appl 's platform linux 15 16 2014 worldwid develop confer 14 proprietari languag open-sourc softwar apach licens linux 15 16 version 3.0 syntax swift went signific evolut core team make sourc stabil focu later version 17 18 first quarter 2018 swift surpass objective-c measur popular 19 syntax 17 18 objective-c 19 swift 4.0 releas 2017 introduc sever chang built-in class structur code written previou version swift updat use migrat function built xcode swift 5 releas march 2019 introduc stabl binari interfac appl platform allow swift runtim incorpor appl oper system sourc compat swift 4 20 20 swift 5.1 offici releas septemb 2019 swift 5.1 build previou version swift 5 extend stabl featur languag compile-tim introduct modul stabil introduct modul stabil make possibl creat share binari framework work futur releas swift 21 21 swift 5.5 offici announc appl 2021 wwdc significantli expand languag support concurr asynchron code notabl introduc uniqu version actor model 22 wwdc concurr asynchron code actor model 22 swift 5.9 releas septemb 2023 includ macro system gener paramet pack ownership featur like new consum oper 23 23 swift 5.10 releas march 2024 version improv languag 's concurr model allow full data isol prevent data race also last releas swift 6 24 version 5.10 current avail maco window linux 25 data race 24 25 swift 6 releas septemb 2024 26 26 histori edit edit edit develop swift start juli 2010 chri lattner eventu collabor mani programm appl swift motiv need replac appl 's earlier program languag objective-c larg unchang sinc earli 1980 lack modern languag featur swift took languag idea `` objective-c rust haskel rubi python c clu far mani other list '' 7 june 2 2014 appl worldwid develop confer wwdc applic becam first publicli releas app written swift 27 beta version program languag releas regist appl develop confer compani promis final version swift would sourc code compat test version appl plan make sourc code convert avail need full releas 27 chri lattner appl objective-c objective-c rust haskel rubi python c clu 7 appl worldwid develop confer 27 beta version program languag sourc code 27 swift program languag free 500-page manual also releas wwdc avail appl book store offici websit 28 appl book 28 swift reach 1.0 mileston septemb 9 2014 gold master xcode 6.0 io 29 swift 1.1 releas octob 22 2014 alongsid launch xcode 6.1 30 swift 1.2 releas april 8 2015 along xcode 6.3 31 swift 2.0 announc wwdc 2015 made avail publish app app store septemb 21 2015 32 swift 3.0 releas septemb 13 2016 33 swift 4.0 releas septemb 19 2017 34 swift 4.1 releas march 29 2018 35 xcode io 29 30 31 32 33 34 35 swift first place love program languag stack overflow develop survey 2015 36 second place 2016 37 stack overflow 36 37 decemb 3 2015 swift languag support librari debugg packag manag open-sourc apach 2.0 licens runtim librari except 38 swift.org creat host project sourc code host github easi anyon get code build even creat pull request contribut code back project 38 swift.org github decemb 2015 ibm announc swift sandbox websit allow develop write swift code one pane display output anoth 39 40 41 swift sandbox deprec januari 2018 42 ibm 39 40 41 42 wwdc 2016 appl announc ipad exclus app name swift playground intend teach peopl code swift app present 3d video game-lik interfac provid feedback line code place certain order execut 43 44 45 wwdc 2016 ipad app swift playground 3d video game-lik 43 44 45 januari 2017 chri lattner announc departur appl new posit tesla motor swift project lead role go team veteran ted kremenek 46 47 tesla motor 46 47 wwdc 2019 appl announc swiftui xcode 11 provid framework declar ui structur design across appl platform 48 swiftui declar 48 offici download sdk toolchain ubuntu distribut linux avail sinc swift 2.2 distro ad sinc swift 5.2.4 cento amazon linux 49 unoffici sdk nativ toolchain packag android 50 51 ubuntu cento 49 50 51 platform edit edit edit platform swift support appl 's oper system darwin io ipado maco tvo watcho linux window android 52 53 darwin io ipado maco tvo watcho linux window android 52 53 key aspect swift 's design abil interoper huge bodi exist objective-c code develop appl product previou decad cocoa cocoa touch framework appl platform 54 link objective-c runtim librari allow c objective-c c++ swift code run within one program 55 cocoa cocoa touch framework 54 runtim librari c objective-c c++ 55 version histori edit edit edit maco linux window 56 57 58 59 60 61 62 63 64 65 66 67 68 69 24 70 71 featur edit edit edit plea help improv make understand non-expert without remov technic detail help improv make understand non-expert june 2018 june 2018 learn remov messag learn remov messag swift gener purpos program languag employ modern programming-languag theori concept strive present simpl yet power syntax swift incorpor innov convent variou program languag notabl inspir objective-c replac primari develop languag appl platform swift design safe friendli new programm sacrif speed default swift manag memori automat ensur variabl alway initi use array access check out-of-bound error integ oper check overflow paramet name allow creat clear api protocol defin interfac type may adopt extens allow develop add function exist type swift enabl object-ori program support class subtyp method overrid option allow nil valu handl explicitli safe concurr program written use async/await syntax actor isol share mutabl state order elimin data race 72 73 object-ori program class subtyp method overrid nil async/await actor 72 73 basic syntax edit edit edit swift 's syntax similar c-style languag code begin execut global scope default 74 altern main attribut appli structur class enumer declar indic contain program 's entri point 75 syntax 74 main 75 swift 's `` hello world '' program `` hello world '' program print `` hello world '' print separ termin function use includ swift 's standard librari avail program without need import extern modul statement swift n't end semicolon howev semicolon requir separ multipl statement written line single-lin comment begin // continu end current line multilin comment contain charact constant declar let keyword variabl var keyword valu must initi read valu may infer type base type provid initi valu initi valu set valu 's declar type must declar explicitli 74 print separ termin comment // let var 74 let highscorethreshold 1000 // constant type int type infer base provid valu var currentscor 980 // variabl type int currentscor 1200 // valu variabl chang time let playermessag string // constant explicit type string currentscor highscorethreshold playermessag `` top player '' el playermessag `` better luck next time '' print playermessag // print `` top player '' control flow swift manag if-els guard switch statement along for-in loop statement take boolean paramet execut bodi statement condit true otherwis execut option el bodi if-let syntax provid syntact sugar check exist option valu unwrap time if-els guard switch for-in loop el let let somenumb 42 somenumb 2 == 0 // use remaind oper find remaind somenumb divid 2. print `` somenumb even '' el print `` somenumb odd '' // print `` 42 even '' function defin func keyword function paramet may name allow function call read like phrase underscor paramet name allow argument label omit call site tupl use function return multipl piec data func tupl func constructgreet name string string return `` hello name '' let greet constructgreet `` craig '' print greet // print `` hello craig '' function anonym function known closur assign properti pas around program like valu closur func dividebytwo anum int int return anum 2 func multiplybytwo anum int int return anum 2 let mathoper multiplybytwo print mathoper 21 // print `` 42 '' guard statement requir given condit true continu past guard statement otherwis bodi provid el claus run el claus must exit control code block guard statement appear guard statement use ensur certain requir met continu program execut particular use creat unwrap version option valu guarante non-nil remaind enclos scope guard guard el el guard guard func divid numer int bydenomin denomin int int guard denomin 0 el print `` ca n't divid 0 '' return nil guard let numer el print `` provid numer nil '' return nil return numer denomin let result divid numer 3 bydenomin 0 print `` divis result result `` // print // `` ca n't divid 0 '' // `` divis result nil '' switch statement compar valu multipl potenti valu execut associ code block switch statement must made exhaust either includ case possibl valu includ default case run provid valu n't match case switch case implicitli fall although may explicitli fallthrough keyword pattern match use variou way insid switch statement exampl integ match number potenti rang switch pattern match let somenumb 42 switch somenumb case .. 0 print `` somenumb neg '' case 0 print `` somenumb 0 '' case 1 .. 9 print `` somenumb greater 0 less 10 '' default print `` somenumb greater 9 '' // print `` 42 greater 9 '' for-in loop iter sequenc valu let name `` '' `` anna '' `` bart '' name name print name // print // // anna // bart loop iter long given boolean condit evalu true // add togeth number 1 5. var 1 var result 0 5 // loop perform bodi long less equal 5. result += // add current result += 1 // increment 1 print result // print `` 15 '' closur support edit edit edit swift support closur self-contain block function pas around use code 76 also use anonym function exampl closur 76 anonym function // closur type defin input output valu specifi outsid closur let closure1 int int int arg1 arg2 return arg1 arg2 // …or insid let closure2 arg1 int arg2 int int return arg1 arg2 // case closur 's return type infer automat compil let closure3 arg1 int arg2 int return arg1 arg2 closur assign variabl constant pas function closur paramet single-express closur may drop return keyword return swift also trail closur syntax allow closur written end function call instead within function 's paramet list parenthes omit altogeth closur function 's paramet // function take closur receiv input paramet return integ // evalu use closur 's return valu int function 's return valu func foo closur bar int int return bar // without trail closur syntax foo closur return 1 // trail closur syntax implicit return foo 1 start version 5.3 swift support multipl trail closur 77 77 // function pas return first closur paramet second // return second closur 's result func foo bar int baz int int int return baz bar // trail closur foo bar return 1 baz x return x 1 // 1 trail closur foo bar return 1 x return x 1 // 2 trail closur first closur 's argument name omit foo return 1 baz x return x 1 swift provid shorthand argument name inlin closur remov need explicitli name closur paramet 78 argument refer name 0 1 2 78 let name `` josephin '' `` steve '' `` chri '' `` barbara '' // filter call given closur valu name // valu charact count less 6 kept other drop let shortnam name filter 0 count 6 print shortnam // print `` `` steve '' `` chri '' '' closur may captur valu surround scope closur refer captur valu long closur exist func makemultipli withmultipl multipl int int int // creat return closur take int return input multipli valu multipl return 0 multipl let multipli makemultipli withmultipl 3 print multipli 3 // print `` 9 '' print multipli 10 // print `` 30 '' string support edit edit edit swift standard librari includ unicode-compli string charact type string valu initi string liter sequenc charact surround doubl quotat mark string concaten oper string charact var somestr `` hello '' somestr += `` world '' string interpol allow creation new string valu express valu written parenthes preced insert enclos string liter 79 79 var currentscor 980 print `` score currentscor '' // print `` score 980 '' for-in loop use iter charact contain string charact `` swift '' print charact // // w // // f // foundat framework import swift invis bridg string type nsstring string class commonli use objective-c. callabl object edit edit edit callabl object § swift edit edit swift callabl object defin use callasfunct 80 80 struct callablestruct var valu int func callasfunct number int scale int print scale number valu let callabl callablestruct valu 100 callabl 4 scale 2 callabl callasfunct 4 scale 2 // function call print 208 access control edit edit edit swift support five access control level symbol open public intern filepriv privat unlik mani object-ori languag access control ignor inherit hierarchi privat indic symbol access immedi scope filepriv indic access within file intern indic access within contain modul public indic access modul open class method indic class may subclass outsid modul 81 access control open public intern filepriv privat inherit privat scope filepriv intern public open 81 option chain edit edit edit import featur swift option type allow refer valu oper manner similar common pattern c pointer may either refer specif valu valu impli non-opt type result null-point error compil ensur possibl option type refer c pointer null-point error option type creat option enum make integ nullabl one would use declar similar var optionalinteg option int c 82 swift also includ syntact sugar allow one indic variabl option place question mark type name var optionalinteg int 83 variabl constant mark option either valu underli type nil option type wrap base type result differ instanc string string fundament differ type former type string latter option may hold string valu 82 83 access valu insid assum nil must unwrap expo instanc insid perform oper let myvalu anoptionalinst somemethod case oper unwrap anoptionalinst expo instanc insid allow method call made anoptionalinst nil null-point error occur termin program known forc unwrap option may safe unwrap use option chain first test whether instanc nil unwrap non-nul option chain let myvalu anoptionalinst somemethod case runtim call somemethod anoptionalinst nil suppress error must place everi option properti properti nil entir express evalu nil origin term chain come common case sever method calls/gett chain togeth instanc let aten abuild tenantlist 5 let theirleas aten leasedetail let leasestart theirleas startdat reduc let leasestart abuild tenantlist 5 leasedetail startdat swift 's use option allow compil use static dispatch unwrap action call defin instanc wrapper versu occur runtim dispatch system static dispatch valu type edit edit edit mani object-ori languag object repres intern two part object store block data place heap name `` handl '' object repres pointer object pas method copi valu pointer allow underli data heap access anyon copi contrast basic type like integ floating-point valu repres directli handl contain data pointer data pas directli method copi style access term pass-by-refer case object pass-by-valu basic type heap pointer concept advantag disadvantag object use data larg like descript window content document case access data provid copi 32- 64-bit valu versu copi entir data structur howev smaller valu like integ size pointer typic one word advantag pas pointer versu pas valu word swift offer built-in support object use either pass-by-refer pass-by-valu semant former use class declar latter use struct struct swift almost featur class method implement protocol use extens mechan reason appl term data gener instanc versu object valu struct support inherit howev 84 84 programm free choos semant appropri data structur applic larger structur like window would defin class allow pas around pointer smaller structur like 2d point defin struct pass-by-valu allow direct access intern data indirect refer count perform improv inher pass-by-valu concept swift use type almost common data type includ int doubl type normal repres object like string array 84 use valu type result signific perform improv user applic well 85 84 85 array dictionari set util copi write data copi program attempt chang valu mean variou accessor effect pointer data storag data physic store one instanc memori level applic valu separ physic separ enforc copi write need 86 copi write 86 extens edit edit edit extens add new function exist type without need subclass even access origin sourc code extens add new method initi comput properti subscript protocol conform 87 exampl might add spell checker base string type mean instanc string program gain abil spell-check system also wide use organiz techniqu allow relat code gather library-lik extens 87 extens declar extens keyword struct rectangl let width doubl let height doubl extens rectangl var area doubl return height width protocol-ori program edit edit edit protocol promis particular type implement set method properti mean instanc system call method instanc implement protocol often use modern object-ori languag substitut multipl inherit although featur set entir similar multipl inherit objective-c languag implement protocol concept programm ensur requir method implement class 88 swift add abil add method use extens use gener program gener implement combin allow protocol written support wide varieti instanc also extens mechan use add protocol conform object list protocol definit 89 88 gener program 89 exampl protocol might declar call printabl ensur instanc conform protocol implement descript properti printdetail method requir // defin protocol name printabl protocol printabl var descript string get // read-onli properti requir func printdetail // method requir protocol adopt type // adopt printabl protocol class class myclass printabl var descript string return `` instanc myclass '' func printdetail print descript extens use add protocol conform type protocol also extend provid default implement requir adopt may defin implement may use default implement extens printabl // printabl instanc receiv implement may defin func printdetail print descript // bool conform printabl inherit printdetail implement extens bool printabl var descript string return `` instanc bool valu self `` swift like mani modern languag support interfac protocol use type mean variabl method defin protocol instead specif type func getsomethingprint printabl return true var somesortofprintableinst getsomethingprint print somesortofprintableinst descript // print `` instanc bool valu true '' matter concret type somesortofprintableinst compil ensur conform protocol thu code safe syntax also mean collect base protocol also like let printablearray printabl extens protocol use extens swift 's standard librari swift 5.9 approxim 1.2 percent symbol within standard librari protocol anoth 12.3 percent protocol requir default implement 90 instanc swift use extens add equat protocol mani basic type like string array allow compar == oper equat protocol also defin default implement 90 func equat lh rh bool function defin method work instanc conform equat provid equal oper instanc class struct automat gain implement simpli conform equat 91 91 protocol extens gener combin creat sophist api exampl constraint allow type condit adopt protocol method base characterist adopt type common use case may ad method collect type element contain within collect equat extens array element equat // allequ avail instanc array contain equat element func allequ bool element self element self first return fals return true concurr edit edit edit swift 5.5 introduc structur concurr languag 92 structur concurr use async/await syntax similar kotlin javascript rust async function defin async keyword paramet list call async function await keyword must written function indic execut potenti suspend call function function suspend program may run concurr function program syntax allow program clearli call potenti suspens point avoid version pyramid doom program caus previous widespread use closur callback 93 92 async/await pyramid doom program 93 func downloadtext name string async string let result // ... asynchron download code ... return result let text await downloadtext `` text1 '' async let syntax allow multipl function run parallel await use mark point program suspend wait complet async function call earlier // call downloadtext run parallel async let text1 downloadtext name `` text1 '' async let text2 downloadtext name `` text2 '' async let text3 downloadtext name `` text3 '' let texttoprint await text1 text2 text3 // suspend three downloadtext call return print texttoprint task taskgroup creat explicitli creat dynam number child task runtim let taskhandl task await downloadtext name `` sometext '' let result await taskhandl valu swift use actor model isol mutabl state allow differ task mutat share state safe manner actor declar actor keyword refer type like class one task may access mutabl state actor time actor may access mutat intern state freeli code run separ task must mark access await keyword indic code may suspend task finish access actor 's state actor model actor directori var name string func add name string name append name let directori directori // code suspend task finish access actor await directori add name `` tucker '' print await directori name librari runtim develop edit edit edit appl system swift use runtim extant objective-c system requir io 7 maco 10.9 higher also depend grand central dispatch 94 swift objective-c code use one program extens c c++ also begin swift 5.9 c++ code use directli swift code 95 case objective-c swift consider access object model use subclass extend use objective-c code provid protocol support 96 convers true swift class subclass objective-c. 97 objective-c grand central dispatch 94 c++ 95 96 97 aid develop program re-us extant code xcode 6 higher offer semi-autom system build maintain bridg header expo objective-c code swift take form addit header file simpli defin import objective-c symbol need project 's swift code point swift refer type function variabl declar import though written swift objective-c code also use swift code directli import automat maintain header file objective-c declar project 's swift symbol instanc objective-c file mix project call `` myapp '' could access swift class function code import `` myapp-swift.h '' symbol avail mechan however—us swift-specif featur like gener type non-object option type sophist enum even unicod identifi may render symbol inaccess objective-c. 98 header file 98 swift also limit support attribut metadata read develop environ necessarili part compil code like objective-c attribut use syntax current avail set small one exampl iboutlet attribut mark given valu code outlet avail use within interfac builder ib outlet devic bind valu on-screen display object code interfac builder non-appl system swift depend objective-c runtim appl system librari set swift `` corelib '' implement replac includ `` swift-corelibs-found '' stand foundat kit `` swift-corelibs-libdispatch '' stand grand central dispatch `` swift-corelibs-xctest '' stand xctest api xcode 99 foundat kit xcode 99 2019 xcode 11 appl also ad major new ui paradigm call swiftui swiftui replac older interfac builder paradigm new declar develop paradigm 100 interfac builder 100 memori manag edit edit edit swift use automat refer count arc manag memori everi instanc class closur maintain refer count keep run talli number refer program hold count reach 0 instanc dealloc automat dealloc remov need garbag collector instanc dealloc soon longer need automat refer count manag memori strong refer cycl occur two instanc strongli refer e.g refer b b refer sinc neither instanc refer count ever reach zero neither ever dealloc result memori leak swift provid keyword weak unown prevent strong refer cycl keyword allow instanc referenc without increment refer count weak refer must option variabl sinc chang becom nil 101 attempt access unown valu alreadi dealloc result runtim error strong refer cycl memori leak 101 closur within class also creat strong refer cycl captur self refer self refer treat weak unown indic use captur list class person let name string weak var home home // defin weak refer order break refer cycl weak refer increment refer count instanc refer init name string self name name deinit print `` de-initi name `` class home let address string var owner person init address string owner person self address address self owner owner deinit print `` de-initi address `` var staci person person name `` staci '' var house21b home home address `` 21b baker street '' owner staci staci home house21b // staci house42b refer staci nil // refer count staci 1 house21b still hold refer house21b nil // house21b 's refer count drop 0 turn drop staci 's count 0 house21b last instanc hold strong refer staci // print // de-initi 21b baker street // de-initi staci debug edit edit edit key element swift system abil cleanli debug run within develop environ use read–eval–print loop repl give interact properti common script abil python tradit system program languag repl enhanc playground interact view run within xcode environ playground app respond code debugg chang on-the-fli 102 playground allow programm add swift code along markdown document programm step code add breakpoint use lldb either consol ide like xcode read–eval–print loop system program playground playground 102 lldb ide comparison languag edit edit edit swift consid c famili program languag similar c variou way c famili program languag oper c also appear swift although oper slightli differ behavior exampl swift trap overflow wherea use denot c-like behavior wrap overflow curli brace use group statement curli brace variabl assign use equal sign compar use two consecut equal sign new ident oper === provid check two data element refer object equal sign two consecut equal sign object control statement switch similar extend function e.g. switch take non-integ case support pattern match condit unwrap option use 1 ... 10 syntax pattern match 1 .. 10 squar bracket use array declar get valu given index one squar bracket array also similar objective-c basic numer type int uint float doubl class method inherit like instanc method self class method class method call similar ... enumer syntax differ objective-c includ statement need end semicolon though must use allow one statement one line header file use type infer type infer gener program gener program function first-class object first-class enumer case associ data algebra data type algebra data type oper redefin class oper overload new oper defin oper overload string fulli support unicod unicod charact use either identifi oper unicod except handl swift 2 introduc differ incompat error-handl model 103 except handl 103 sever featur earlier c-famili languag easi misus remov pointer expo default need programm keep track mark name referenc dereferenc assign return valu prevent common error write 0 instead == 0 throw compile-tim error need use break statement switch block individu case fall next case unless fallthrough statement use variabl constant alway initi array bound alway check integ overflow result undefin behavior sign integ c trap run-tim error swift programm choos allow overflow use special arithmet oper properti min max defin swift integ type use safe check potenti overflow versu reli constant defin type extern librari one-stat form allow omiss brace around statement unsupport c-style enumer int 0 c i++ prone off-by-on error unsupport swift 3 onward 104 pre- post- increment decrement oper i++ -- ... unsupport swift 3 onward sinc c-style statement also unsupport swift 3 onward 105 c-famili languag pointer expo default need programm keep track mark name referenc dereferenc pointer assign return valu prevent common error write 0 instead == 0 throw compile-tim error need use break statement switch block individu case fall next case unless fallthrough statement use switch variabl constant alway initi array bound alway check integ overflow result undefin behavior sign integ c trap run-tim error swift programm choos allow overflow use special arithmet oper properti min max defin swift integ type use safe check potenti overflow versu reli constant defin type extern librari integ overflow undefin behavior one-stat form allow omiss brace around statement unsupport c-style enumer int 0 c i++ prone off-by-on error unsupport swift 3 onward 104 off-by-on error 104 pre- post- increment decrement oper i++ -- ... unsupport swift 3 onward sinc c-style statement also unsupport swift 3 onward 105 increment decrement oper 105 develop implement edit edit edit swift run linux sometim also use server-sid languag 106 web framework alreadi develop ibm 's kitura discontinu perfect vapor 106 ibm kitura perfect vapor offici `` server api '' work group also start appl 107 member swift develop commun play central role 108 107 108 second free implement swift target cocoa microsoft 's common languag infrastructur .net framework .net java android platform exist part element compil remobject softwar 109 cocoa microsoft common languag infrastructur .net framework .net java android remobject softwar 109 subset swift port addit platform arduino 110 mac o 9 111 port arduino 110 mac o 9 111 see also edit edit edit comput program portal comput program portal comput program portal comparison program languag comparison program languag objective-c objective-c program languag program languag kotlin program languag kotlin program languag python program languag python program languag nim program languag nim program languag refer edit edit edit u.s. patent 9329844 u.s. patent 9329844 `` swift reach 1.0 '' appl septemb 9 2014 retriev march 8 2015 `` swift reach 1.0 '' appl septemb 9 2014 retriev march 8 2015 `` swift reach 1.0 '' retriev march 8 2015 march 8 `` releas 6.0.3 '' decemb 13 2024 retriev decemb 31 2024 `` releas 6.0.3 '' decemb 13 2024 retriev decemb 31 2024 `` releas 6.0.3 '' retriev decemb 31 2024 decemb 31 `` swift object '' swift proprietari close entir control appl open sourc implement `` swift object '' swift proprietari close entir control appl open sourc implement `` swift object '' lattner chri june 11 2014 `` llvmdev cfe-dev advertis open posit appl 's swift compil team '' archiv origin juli 14 2014 retriev june 12 2014 imagin mani u want open sourc part llvm discus n't happen yet wo n't time lattner chri june 11 2014 `` llvmdev cfe-dev advertis open posit appl 's swift compil team '' archiv origin juli 14 2014 retriev june 12 2014 imagin mani u want open sourc part llvm discus n't happen yet wo n't time lattner chri `` llvmdev cfe-dev advertis open posit appl 's swift compil team '' origin retriev june 12 2014 june 12 `` chri lattner 's homepag '' chri lattner june 3 2014 retriev june 3 2014 swift languag product tireless effort team languag expert document guru compil optim ninja incred import intern dogfood group provid feedback help refin battle-test idea cours also greatli benefit experi hard-won mani languag field draw idea objective-c rust haskel rubi python c clu far mani other list `` chri lattner 's homepag '' chri lattner june 3 2014 retriev june 3 2014 swift languag product tireless effort team languag expert document guru compil optim ninja incred import intern dogfood group provid feedback help refin battle-test idea cours also greatli benefit experi hard-won mani languag field draw idea objective-c rust haskel rubi python c clu far mani other list `` chri lattner 's homepag '' retriev june 3 2014 june 3 b lattner chri june 3 2014 `` chri lattner 's homepag '' chri lattner retriev june 3 2014 start work swift program languag juli 2010 implement much basic languag structur peopl know exist amaz peopl start contribut earnest late 2011 becam major focu appl develop tool group juli 2013 ... draw idea objective-c rust haskel rubi python c clu far mani other list b b lattner chri june 3 2014 `` chri lattner 's homepag '' chri lattner retriev june 3 2014 start work swift program languag juli 2010 implement much basic languag structur peopl know exist amaz peopl start contribut earnest late 2011 becam major focu appl develop tool group juli 2013 ... draw idea objective-c rust haskel rubi python c clu far mani other list lattner chri `` chri lattner 's homepag '' retriev june 3 2014 june 3 objective-c `` build assert swift part 2 __file__ __line__ '' retriev septemb 25 2014 `` build assert swift part 2 __file__ __line__ '' retriev septemb 25 2014 `` build assert swift part 2 __file__ __line__ '' retriev septemb 25 2014 septemb 25 `` influenc rust refer '' doc.rust-lang.org retriev may 2 2020 `` influenc rust refer '' doc.rust-lang.org retriev may 2 2020 `` influenc rust refer '' retriev may 2 2020 may 2 `` influenc v document '' github.com retriev novemb 3 2023 `` influenc v document '' github.com retriev novemb 3 2023 `` influenc v document '' retriev novemb 3 2023 novemb 3 lardinoi freder june 2 2014 `` appl launch swift new program languag write io o x app '' techcrunch retriev septemb 7 2022 lardinoi freder june 2 2014 `` appl launch swift new program languag write io o x app '' techcrunch retriev septemb 7 2022 `` appl launch swift new program languag write io o x app '' retriev septemb 7 2022 septemb 7 protocol-ori program swift appl inc. youtub protocol-ori program swift appl inc. youtub protocol-ori program swift youtub `` concept similar rust trait '' `` concept similar rust trait '' `` concept similar rust trait '' william owen june 2 2014 `` tim berners-le 's sixtieth birthday appl announc swift new program languag io '' next web retriev june 2 2014 william owen june 2 2014 `` tim berners-le 's sixtieth birthday appl announc swift new program languag io '' next web retriev june 2 2014 `` tim berners-le 's sixtieth birthday appl announc swift new program languag io '' retriev june 2 2014 june 2 `` appl 's new program languag swift open sourc '' verg decemb 3 2015 retriev decemb 5 2015 `` appl 's new program languag swift open sourc '' verg decemb 3 2015 retriev decemb 5 2015 `` appl 's new program languag swift open sourc '' retriev decemb 5 2015 decemb 5 `` appl open sourc swift latest pitch enterpris '' cio journal wall street journal blog decemb 3 2015 retriev decemb 5 2015 `` appl open sourc swift latest pitch enterpris '' cio journal wall street journal blog decemb 3 2015 retriev decemb 5 2015 `` appl open sourc swift latest pitch enterpris '' `` appl open sourc swift latest pitch enterpris '' wall street journal retriev decemb 5 2015 decemb 5 `` look back swift 3 ahead swift 4 '' swift forum juli 29 2016 retriev novemb 19 2018 `` look back swift 3 ahead swift 4 '' swift forum juli 29 2016 retriev novemb 19 2018 `` look back swift 3 ahead swift 4 '' retriev novemb 19 2018 novemb 19 `` swift-evolut '' swift evolut retriev novemb 19 2018 `` swift-evolut '' swift evolut retriev novemb 19 2018 `` swift-evolut '' retriev novemb 19 2018 novemb 19 `` redmonk program languag rank januari 2018 – tecosystem '' redmonk.com march 7 2018 retriev novemb 20 2018 `` redmonk program languag rank januari 2018 – tecosystem '' redmonk.com march 7 2018 retriev novemb 20 2018 `` redmonk program languag rank januari 2018 – tecosystem '' retriev novemb 20 2018 novemb 20 kremenek ted march 25 2019 `` swift 5 releas '' kremenek ted march 25 2019 `` swift 5 releas '' `` swift 5 releas '' kremenek ted septemb 20 2019 `` swift 5.1 releas '' archiv origin februari 26 2022 retriev octob 28 2019 kremenek ted septemb 20 2019 `` swift 5.1 releas '' archiv origin februari 26 2022 retriev octob 28 2019 `` swift 5.1 releas '' origin retriev octob 28 2019 octob 28 hudson paul june 6 2021 `` 's new swift 5.5 '' hackingwithswift.com hack swift retriev june 8 2021 hudson paul june 6 2021 `` 's new swift 5.5 '' hackingwithswift.com hack swift retriev june 8 2021 `` 's new swift 5.5 '' retriev june 8 2021 june 8 `` swift 5.9 releas '' swift.org septemb 18 2023 retriev octob 9 2023 `` swift 5.9 releas '' swift.org septemb 18 2023 retriev octob 9 2023 `` swift 5.9 releas '' retriev octob 9 2023 octob 9 b borla holli march 5 2024 `` swift 5.10 releas '' swift.org retriev march 13 2024 b b borla holli march 5 2024 `` swift 5.10 releas '' swift.org retriev march 13 2024 `` swift 5.10 releas '' retriev march 13 2024 march 13 `` swift.org '' swift.org march 2014 retriev april 28 2024 `` swift.org '' swift.org march 2014 retriev april 28 2024 `` swift.org '' retriev april 28 2024 april 28 inc appl septemb 17 2024 `` announc swift 6 '' swift.org retriev octob 15 2024 cite web |last= gener name help inc appl septemb 17 2024 `` announc swift 6 '' swift.org retriev octob 15 2024 cite web |last= gener name help `` announc swift 6 '' retriev octob 15 2024 octob 15 cite web cite web |last= gener name help help b platform state union session 102 appl worldwid develop confer june 2 2014 b b platform state union session 102 appl worldwid develop confer june 2 2014 appl worldwid develop confer swift program languag appl june 2 2014 retriev june 2 2014 `` document '' swift swift program languag appl june 2 2014 retriev june 2 2014 `` document '' swift swift program languag appl retriev june 2 2014 june 2 `` document '' swift `` document '' `` swift reach 1.0 '' septemb 9 2014 retriev septemb 10 2014 `` swift reach 1.0 '' septemb 9 2014 retriev septemb 10 2014 `` swift reach 1.0 '' retriev septemb 10 2014 septemb 10 `` xcode 6.1 releas note '' octob 22 2014 retriev januari 23 2015 `` xcode 6.1 releas note '' octob 22 2014 retriev januari 23 2015 `` xcode 6.1 releas note '' retriev januari 23 2015 januari 23 `` xcode 6.3 releas note '' april 8 2015 retriev april 8 2015 `` xcode 6.3 releas note '' april 8 2015 retriev april 8 2015 `` xcode 6.3 releas note '' retriev april 8 2015 april 8 `` swift 2 app app store '' swift blog retriev march 13 2016 `` swift 2 app app store '' swift blog retriev march 13 2016 `` swift 2 app app store '' retriev march 13 2016 march 13 `` swift 3.0 releas '' swift.org septemb 13 2016 archiv origin octob 14 2016 retriev octob 26 2016 `` swift 3.0 releas '' swift.org septemb 13 2016 archiv origin octob 14 2016 retriev octob 26 2016 `` swift 3.0 releas '' origin retriev octob 26 2016 octob 26 `` swift 4.0 releas '' swift.org septemb 17 2017 archiv origin march 28 2019 retriev march 1 2018 `` swift 4.0 releas '' swift.org septemb 17 2017 archiv origin march 28 2019 retriev march 1 2018 `` swift 4.0 releas '' origin retriev march 1 2018 march 1 `` swift 4.1 releas '' swift.org march 29 2018 archiv origin april 25 2019 retriev march 30 2018 `` swift 4.1 releas '' swift.org march 29 2018 archiv origin april 25 2019 retriev march 30 2018 `` swift 4.1 releas '' origin retriev march 30 2018 march 30 `` stack overflow develop survey result 2015 '' `` stack overflow develop survey result 2015 '' `` stack overflow develop survey result 2015 '' `` stack overflow develop survey result 2016 '' `` stack overflow develop survey result 2016 '' `` stack overflow develop survey result 2016 '' `` swift.org open sourc '' swift.org appl inc. retriev februari 25 2019 `` swift.org open sourc '' swift.org appl inc. retriev februari 25 2019 `` swift.org open sourc '' retriev februari 25 2019 februari 25 `` introduc ibm swift sandbox — swift '' swift retriev decemb 5 2015 `` introduc ibm swift sandbox — swift '' swift retriev decemb 5 2015 `` introduc ibm swift sandbox — swift '' retriev decemb 5 2015 decemb 5 mayo benjamin decemb 4 2015 `` write swift code web browser ibm swift sandbox '' 9to5mac retriev decemb 5 2015 mayo benjamin decemb 4 2015 `` write swift code web browser ibm swift sandbox '' 9to5mac retriev decemb 5 2015 `` write swift code web browser ibm swift sandbox '' retriev decemb 5 2015 decemb 5 `` appl open sourc ibm put swift program cloud '' zdnet retriev decemb 5 2015 `` appl open sourc ibm put swift program cloud '' zdnet retriev decemb 5 2015 `` appl open sourc ibm put swift program cloud '' retriev decemb 5 2015 decemb 5 `` swift packag catalog swift sandbox deprec '' retriev novemb 9 2018 `` swift packag catalog swift sandbox deprec '' retriev novemb 9 2018 `` swift packag catalog swift sandbox deprec '' retriev novemb 9 2018 novemb 9 `` swift playground '' appl develop retriev june 19 2016 `` swift playground '' appl develop retriev june 19 2016 `` swift playground '' retriev june 19 2016 june 19 `` swift playground — preview '' appl retriev june 19 2016 `` swift playground — preview '' appl retriev june 19 2016 `` swift playground — preview '' retriev june 19 2016 june 19 mayo benjamin june 13 2016 `` appl announc swift playground ipad wwdc public releas fall '' 9to5mac retriev june 19 2016 mayo benjamin june 13 2016 `` appl announc swift playground ipad wwdc public releas fall '' 9to5mac retriev june 19 2016 `` appl announc swift playground ipad wwdc public releas fall '' retriev june 19 2016 june 19 cunningham andrew januari 10 2017 `` longtim appl programm swift creator leav appl tesla '' ar technica cunningham andrew januari 10 2017 `` longtim appl programm swift creator leav appl tesla '' ar technica `` longtim appl programm swift creator leav appl tesla '' wuerthel mike januari 13 2017 `` new swift project head ted kremenek said run show behind scene time '' appleinsid wuerthel mike januari 13 2017 `` new swift project head ted kremenek said run show behind scene time '' appleinsid `` new swift project head ted kremenek said run show behind scene time '' daniel eran dilger june 19 2019 `` wwdc19 swiftui brightest star galaxi new idea '' appleinsid retriev juli 19 2019 daniel eran dilger june 19 2019 `` wwdc19 swiftui brightest star galaxi new idea '' appleinsid retriev juli 19 2019 `` wwdc19 swiftui brightest star galaxi new idea '' retriev juli 19 2019 juli 19 `` swift.org download swift '' retriev june 21 2020 `` swift.org download swift '' retriev june 21 2020 `` swift.org download swift '' retriev june 21 2020 june 21 `` android sdk swift '' github retriev septemb 10 2021 `` android sdk swift '' github retriev septemb 10 2021 `` android sdk swift '' github retriev septemb 10 2021 septemb 10 `` swift-lang packag version '' retriev septemb 10 2021 `` swift-lang packag version '' retriev septemb 10 2021 `` swift-lang packag version '' retriev septemb 10 2021 septemb 10 readdl januari 15 2020 `` swift android experi tool '' medium retriev august 20 2020 readdl januari 15 2020 `` swift android experi tool '' medium retriev august 20 2020 `` swift android experi tool '' retriev august 20 2020 august 20 anderson tim march 30 2020 `` offici tailor swift window support promis 5.3 swift platform – provid need gui '' regist retriev septemb 18 2020 anderson tim march 30 2020 `` offici tailor swift window support promis 5.3 swift platform – provid need gui '' regist retriev septemb 18 2020 `` offici tailor swift window support promis 5.3 swift platform – provid need gui '' regist retriev septemb 18 2020 septemb 18 `` swift linux port '' swift.org appl inc. decemb 3 2015 retriev august 3 2016 `` swift linux port '' swift.org appl inc. decemb 3 2015 retriev august 3 2016 `` swift linux port '' retriev august 3 2016 august 3 timmer john june 5 2014 `` fast look swift appl 's new program languag '' ar technica condé nast retriev june 6 2014 timmer john june 5 2014 `` fast look swift appl 's new program languag '' ar technica condé nast retriev june 6 2014 `` fast look swift appl 's new program languag '' ar technica condé nast retriev june 6 2014 june 6 kremenek ted march 25 2019 `` swift 5 releas '' swift.org retriev march 28 2019 kremenek ted march 25 2019 `` swift 5 releas '' swift.org retriev march 28 2019 `` swift 5 releas '' retriev march 28 2019 march 28 `` download swift '' swift.org appl retriev decemb 15 2020 `` download swift '' swift.org appl retriev decemb 15 2020 `` download swift '' retriev decemb 15 2020 decemb 15 kremenek ted april 26 2021 `` swift 5.4 releas '' swift.org appl archiv origin april 26 2021 retriev april 26 2021 kremenek ted april 26 2021 `` swift 5.4 releas '' swift.org appl archiv origin april 26 2021 retriev april 26 2021 `` swift 5.4 releas '' origin retriev april 26 2021 april 26 kremenek ted march 14 2022 `` swift 5.6 releas '' swift.org appl retriev march 14 2022 kremenek ted march 14 2022 `` swift 5.6 releas '' swift.org appl retriev march 14 2022 `` swift 5.6 releas '' retriev march 14 2022 march 14 `` releas swift 5.6.1 releas · apple/swift '' github `` releas swift 5.6.1 releas · apple/swift '' github `` releas swift 5.6.1 releas · apple/swift '' github `` releas swift 5.6.2 releas · apple/swift '' github `` releas swift 5.6.2 releas · apple/swift '' github `` releas swift 5.6.2 releas · apple/swift '' github `` releas swift 5.6.3 releas · apple/swift '' github `` releas swift 5.6.3 releas · apple/swift '' github `` releas swift 5.6.3 releas · apple/swift '' github borla holli septemb 12 2022 `` swift 5.7 releas '' swift.org appl retriev septemb 23 2022 borla holli septemb 12 2022 `` swift 5.7 releas '' swift.org appl retriev septemb 23 2022 `` swift 5.7 releas '' retriev septemb 23 2022 septemb 23 `` releas swift 5.7.1 releas · apple/swift '' github `` releas swift 5.7.1 releas · apple/swift '' github `` releas swift 5.7.1 releas · apple/swift '' github `` releas swift 5.8 releas · apple/swift '' github `` releas swift 5.8 releas · apple/swift '' github `` releas swift 5.8 releas · apple/swift '' github `` releas swift 5.8.1 releas · apple/swift '' github retriev june 14 2023 `` releas swift 5.8.1 releas · apple/swift '' github retriev june 14 2023 `` releas swift 5.8.1 releas · apple/swift '' retriev june 14 2023 june 14 `` releas swift 5.9 releas · apple/swift '' github retriev septemb 18 2023 `` releas swift 5.9 releas · apple/swift '' github retriev septemb 18 2023 `` releas swift 5.9 releas · apple/swift '' retriev septemb 18 2023 septemb 18 `` releas swift 5.9.1 releas · apple/swift '' github retriev octob 19 2023 `` releas swift 5.9.1 releas · apple/swift '' github retriev octob 19 2023 `` releas swift 5.9.1 releas · apple/swift '' retriev octob 19 2023 octob 19 `` releas swift 5.9.2 releas · apple/swift '' github retriev decemb 11 2023 `` releas swift 5.9.2 releas · apple/swift '' github retriev decemb 11 2023 `` releas swift 5.9.2 releas · apple/swift '' retriev decemb 11 2023 decemb 11 `` swift 5.10.1 releas '' github retriev june 5 2024 `` swift 5.10.1 releas '' github retriev june 5 2024 `` swift 5.10.1 releas '' retriev june 5 2024 june 5 `` swift 6.0 releas '' github retriev septemb 17 2024 `` swift 6.0 releas '' github retriev septemb 17 2024 `` swift 6.0 releas '' retriev septemb 17 2024 septemb 17 `` document '' docs.swift.org retriev novemb 17 2023 `` document '' docs.swift.org retriev novemb 17 2023 `` document '' retriev novemb 17 2023 novemb 17 `` elimin data race use swift concurr wwdc22 video '' appl inc. retriev novemb 17 2023 `` elimin data race use swift concurr wwdc22 video '' appl inc. retriev novemb 17 2023 `` elimin data race use swift concurr wwdc22 video '' retriev novemb 17 2023 novemb 17 b `` document '' docs.swift.org retriev octob 15 2023 b b `` document '' docs.swift.org retriev octob 15 2023 `` document '' retriev octob 15 2023 octob 15 `` document '' docs.swift.org retriev octob 15 2023 `` document '' docs.swift.org retriev octob 15 2023 `` document '' retriev octob 15 2023 octob 15 `` closur — swift program languag swift 5.5 '' docs.swift.org retriev august 31 2021 `` closur — swift program languag swift 5.5 '' docs.swift.org retriev august 31 2021 `` closur — swift program languag swift 5.5 '' retriev august 31 2021 august 31 macomb kyle yaskevich yavel gregor doug mccall john `` multipl trail closur '' github retriev octob 19 2020 macomb kyle yaskevich yavel gregor doug mccall john `` multipl trail closur '' github retriev octob 19 2020 `` multipl trail closur '' retriev octob 19 2020 octob 19 `` document '' docs.swift.org retriev octob 16 2023 `` document '' docs.swift.org retriev octob 16 2023 `` document '' retriev octob 16 2023 octob 16 `` string charact '' docs.swift.org retriev octob 16 2023 `` string charact '' docs.swift.org retriev octob 16 2023 `` string charact '' retriev octob 16 2023 octob 16 `` declar — swift program languag swift 5.6 '' docs.swift.org retriev februari 28 2022 `` declar — swift program languag swift 5.6 '' docs.swift.org retriev februari 28 2022 `` declar — swift program languag swift 5.6 '' retriev februari 28 2022 februari 28 `` access control '' swift program languag appl develop retriev octob 25 2016 `` access control '' swift program languag appl develop retriev octob 25 2016 `` access control '' appl retriev octob 25 2016 octob 25 `` nullabl type '' c program guid microsoft archiv februari 21 2017 wayback machin `` nullabl type '' c program guid microsoft archiv februari 21 2017 wayback machin `` nullabl type '' archiv wayback machin `` type '' swift program languag appl develop retriev juli 16 2014 `` type '' swift program languag appl develop retriev juli 16 2014 `` type '' appl retriev juli 16 2014 juli 16 b `` class structur '' swift program languag appl develop archiv origin march 25 2016 b b `` class structur '' swift program languag appl develop archiv origin march 25 2016 `` class structur '' archiv guhit fiel februari 14 2015 `` perform case studi swift 1.1 swift 1.2 objective-c '' medium archiv origin decemb 12 2023 guhit fiel februari 14 2015 `` perform case studi swift 1.1 swift 1.2 objective-c '' medium archiv origin decemb 12 2023 `` perform case studi swift 1.1 swift 1.2 objective-c '' archiv build better app valu type appl archiv origin june 21 2015 build better app valu type appl archiv origin june 21 2015 build better app valu type origin `` extens '' docs.swift.org retriev novemb 28 2023 `` extens '' docs.swift.org retriev novemb 28 2023 `` extens '' retriev novemb 28 2023 novemb 28 `` work protocol '' program objective-c. appl develop document archiv septemb 17 2014 archiv origin april 29 2016 `` work protocol '' program objective-c. appl develop document archiv septemb 17 2014 archiv origin april 29 2016 `` work protocol '' archiv `` nscopi protocol refer '' appl `` nscopi protocol refer '' appl `` nscopi protocol refer '' `` swift standard librari statist '' swiftinit.org swiftinit retriev octob 2 2023 `` swift standard librari statist '' swiftinit.org swiftinit retriev octob 2 2023 `` swift standard librari statist '' retriev octob 2 2023 octob 2 thompson mattt septemb 2 2014 `` swift default protocol implement '' nshipster thompson mattt septemb 2 2014 `` swift default protocol implement '' nshipster `` swift default protocol implement '' `` swift-evolution/proposals/0304-structured-concurrency.md main · apple/swift-evolut '' github retriev octob 16 2023 `` swift-evolution/proposals/0304-structured-concurrency.md main · apple/swift-evolut '' github retriev octob 16 2023 `` swift-evolution/proposals/0304-structured-concurrency.md main · apple/swift-evolut '' retriev octob 16 2023 octob 16 `` swift-evolution/proposals/0296-async-await.md main · apple/swift-evolut '' github retriev octob 16 2023 `` swift-evolution/proposals/0296-async-await.md main · apple/swift-evolut '' github retriev octob 16 2023 `` swift-evolution/proposals/0296-async-await.md main · apple/swift-evolut '' retriev octob 16 2023 octob 16 `` swift-bas app work maco 10.9/io 7 lower `` stackoverflow `` swift-bas app work maco 10.9/io 7 lower `` stackoverflow `` swift-bas app work maco 10.9/io 7 lower '' inc appl septemb 18 2023 `` swift 5.9 releas '' swift.org retriev octob 9 2023 cite web |last= gener name help inc appl septemb 18 2023 `` swift 5.9 releas '' swift.org retriev octob 9 2023 cite web |last= gener name help `` swift 5.9 releas '' retriev octob 9 2023 octob 9 cite web cite web |last= gener name help help `` write swift class objective-c behavior '' appl inc. `` write swift class objective-c behavior '' appl inc. `` write swift class objective-c behavior '' `` migrat objective-c code swift '' `` migrat objective-c code swift '' `` migrat objective-c code swift '' `` swift objective-c project '' appl inc. `` swift objective-c project '' appl inc. `` swift objective-c project '' `` appl search `` corelib '' '' github `` appl search `` corelib '' '' github `` appl search `` corelib '' '' `` xcode swiftui- appl develop '' developer.apple.com retriev februari 1 2021 `` xcode swiftui- appl develop '' developer.apple.com retriev februari 1 2021 `` xcode swiftui- appl develop '' retriev februari 1 2021 februari 1 lanier brian groff joe `` intermedi swift '' appl retriev juli 3 2014 lanier brian groff joe `` intermedi swift '' appl retriev juli 3 2014 `` intermedi swift '' retriev juli 3 2014 juli 3 metz cade `` coder go nut appl 's new program languag '' wire retriev juli 16 2014 metz cade `` coder go nut appl 's new program languag '' wire retriev juli 16 2014 `` coder go nut appl 's new program languag '' wire retriev juli 16 2014 juli 16 `` error-handl swift-languag '' stackoverflow.com `` error-handl swift-languag '' stackoverflow.com `` error-handl swift-languag '' `` apple/swift-evolut '' github retriev april 4 2016 `` apple/swift-evolut '' github retriev april 4 2016 `` apple/swift-evolut '' retriev april 4 2016 april 4 `` apple/swift-evolut '' github retriev april 4 2016 `` apple/swift-evolut '' github retriev april 4 2016 `` apple/swift-evolut '' retriev april 4 2016 april 4 barbosa greg februari 22 2016 `` ibm bring swift cloud releas web framework kitura written appl 's program languag '' 9to5mac retriev may 16 2016 barbosa greg februari 22 2016 `` ibm bring swift cloud releas web framework kitura written appl 's program languag '' 9to5mac retriev may 16 2016 `` ibm bring swift cloud releas web framework kitura written appl 's program languag '' retriev may 16 2016 may 16 `` server api work group '' swift.org octob 25 2016 retriev octob 28 2016 `` server api work group '' swift.org octob 25 2016 retriev octob 28 2016 `` server api work group '' retriev octob 28 2016 octob 28 `` swift.org '' swift.org archiv origin may 10 2021 retriev octob 28 2016 `` swift.org '' swift.org archiv origin may 10 2021 retriev octob 28 2016 `` swift.org '' origin retriev octob 28 2016 octob 28 `` remobject element compil '' retriev januari 17 2016 `` remobject element compil '' retriev januari 17 2016 `` remobject element compil '' retriev januari 17 2016 januari 17 `` swift arduino '' `` swift arduino '' `` swift arduino '' rose jordan april 1 2020 `` swift mac o 9 '' -dealloc rose jordan april 1 2020 `` swift mac o 9 '' -dealloc `` swift mac o 9 '' extern link edit edit edit offici websit offici websit offici websit offici websit swift appl develop swift appl develop swift sourc code github swift sourc code github swift exampl swift exampl server-sid swift vapor framework server-sid swift vapor framework v v e e program languag comparison comparison timelin timelin histori histori ada ada algol simula algol simula simula apl apl assembl assembl basic visual basic classic .net basic visual basic classic .net visual basic classic classic .net .net c c c++ c++ c c cobol cobol erlang erlang forth forth fortran fortran go go haskel haskel java java javascript javascript julia julia kotlin kotlin lisp lisp lua lua matlab matlab ml ml pascal object pascal pascal object pascal object pascal perl perl php php prolog prolog python python r r rubi rubi rust rust sql sql scratch scratch shell shell smalltalk smalltalk swift swift ... ... list alphabet alphabet categor categor gener gener non-english-bas non-english-bas categori categori v v e e appl operatingsystem appl do appl do appl iii so appl iii so classic mac o classic mac o darwin darwin io io ipado ipado gs/o gs/o lisa o lisa o maco o x maco macwork xl macwork xl prodo prodo tvo tvo visiono visiono watcho watcho bento bento clip clip final cut express final cut express front row front row garageband garageband icloud icloud idvd idvd ilif ilif imovi imovi itun itun iwork keynot page number iwork keynot keynot page page number number iphoto iphoto logic express logic express apertur apertur appl medium tool appl medium tool appl qmaster appl qmaster color color dvd studio pro dvd studio pro filemak filemak final cut pro motion compressor final cut studio cinema tool final cut pro motion motion compressor compressor final cut studio final cut studio cinema tool cinema tool logic studio logic studio logic pro logic pro macproject macproject mactermin mactermin mainstag mainstag shake shake soundtrack pro soundtrack pro classroom classroom applework applework book book calendar calendar contact contact mail mail messag messag facetim facetim macdraw macdraw macpaint macpaint macwrit macwrit music music password password photo booth photo booth photo photo podcast podcast safari safari textedit textedit appl remot desktop appl remot desktop foundationdb foundationdb maco server maco server webobject webobject xgrid xgrid xsan xsan develop hypercard hypercard ibook author ibook author instrument instrument interfac builder interfac builder quartz compos quartz compos resedit resedit swift swift xcode xcode .mac .mac dashcod dashcod iad produc iad produc ichat ichat iweb iweb mobilem mobilem window airport util airport util applework applework bonjour bonjour boot camp boot camp icloud icloud itun itun mobilem mobilem quicktim quicktim softwar updat softwar updat list appl ii applic softwar list mac softwar list old macintosh softwar author control databas germani germani germani unit state unit state unit state franc franc franc bnf data bnf data bnf data israel israel israel http //en.wikipedia.org/w/index.php title=swift_ programming_languag oldid=1260160620 categori swift program languag swift program languag appl inc. softwar appl inc. softwar program languag program languag high-level program languag high-level program languag declar program languag declar program languag object-ori program languag object-ori program languag function languag function languag pattern match program languag pattern match program languag program languag creat 2014 program languag creat 2014 static type program languag static type program languag system program languag system program languag cross-platform free softwar cross-platform free softwar softwar use apach licens softwar use apach licens cs1 error gener name cs1 error gener name webarch templat wayback link webarch templat wayback link articl short descript articl short descript short descript differ wikidata short descript differ wikidata use mdi date octob 2018 use mdi date octob 2018 wikipedia articl technic june 2018 wikipedia articl technic june 2018 articl technic articl technic articl excerpt articl excerpt articl exampl swift code articl exampl swift code page last edit 29 novemb 2024 06:06 utc utc text avail creativ common attribution-sharealik 4.0 licens addit term may appli use site agre term use privaci polici wikipedia® regist trademark wikimedia foundat inc. non-profit organ creativ common attribution-sharealik 4.0 licens term use privaci polici wikimedia foundat inc. privaci polici privaci polici wikipedia wikipedia disclaim disclaim contact wikipedia contact wikipedia code conduct code conduct develop develop statist statist cooki statement cooki statement mobil view mobil view",
    "https://en.wikipedia.org/wiki/Kotlin": "jump content main menu main page main page main page content content content current event current event current event random articl random articl random articl wikipedia wikipedia wikipedia contact u contact u contact u help help help learn edit learn edit learn edit commun portal commun portal commun portal recent chang recent chang recent chang upload file upload file upload file search search appear donat donat donat creat account creat account creat account log log log person tool donat donat donat creat account creat account creat account log log log learn learn contribut contribut contribut talk talk talk kotlin kotlin 15 languag беларуская беларуская беларуская cebuano cebuano cebuano deutsch deutsch deutsch español español español françai françai françai 한국어 한국어 한국어 hrvatski hrvatski hrvatski italiano italiano italiano nederland nederland nederland 日本語 日本語 日本語 polski polski polski русский русский русский slovenčina slovenčina slovenčina suomi suomi suomi svenska svenska svenska edit link edit link articl articl articl talk talk talk english read read read edit edit edit view histori view histori view histori tool read read read edit edit edit view histori view histori view histori link link link relat chang relat chang relat chang upload file upload file upload file special page special page special page perman link perman link perman link page inform page inform page inform cite page cite page cite page get shorten url get shorten url get shorten url download qr code download qr code download qr code download pdf download pdf download pdf printabl version printabl version printabl version wikidata item wikidata item wikidata item kotlin kotlin may refer kotlin greater poland voivodeship villag west-centr poland kotlin greater poland voivodeship kotlin island russian island near head gulf finland kotlin island kotlin program languag general-purpos program languag kotlin program languag kotlin-class destroy class destroy built soviet navi kotlin-class destroy kotlin brand fruit veget product made agro nova polish compani agro nova disambigu intern link http //en.wikipedia.org/w/index.php title=kotlin oldid=999577926 categori disambigu page disambigu page place name disambigu page place name disambigu page short descript differ wikidata short descript differ wikidata articl disambigu page articl disambigu page disambigu page disambigu page page last edit 10 januari 2021 22:20 utc utc text avail creativ common attribution-sharealik 4.0 licens addit term may appli use site agre term use privaci polici wikipedia® regist trademark wikimedia foundat inc. non-profit organ creativ common attribution-sharealik 4.0 licens term use privaci polici wikimedia foundat inc. privaci polici privaci polici wikipedia wikipedia disclaim disclaim contact wikipedia contact wikipedia code conduct code conduct develop develop statist statist cooki statement cooki statement mobil view mobil view"
}

## Βήμα 3

Για το βήμα 3, χρειάστηκε να εκτελέσουμε την παρακάτω εντολή (για τη βιβλιοθήκη nltk):
- pip install nltk
##
- **Διαβάζουμε τα προεπεξεργασμένα δεδομένα**: Ανοίγουμε το αρχείο JSON που περιέχει τα επεξεργασμένα δεδομένα κειμένου που έχουμε αποθηκεύσει προηγουμένως.

- **Αρχικοποιούμε το ανεστραμμένο ευρετήριο (Inverted Index)**: Το ανεστραμμένο ευρετήριο είναι μία δομή δεδομένων που μας επιτρέπει να εντοπίσουμε γρήγορα σε ποια έγγραφα εμφανίζεται κάθε λέξη.

- **Δημιουργία του ανεστραμμένου ευρετηρίου**: Για κάθε κείμενο που έχουμε επεξεργαστεί, διαχωρίζουμε το κείμενο σε λέξεις και προσθέτουμε την κάθε λέξη στο ανεστραμμένο ευρετήριο. Κάθε λέξη θα δείχνει σε ποια έγγραφα (ταυτοποιημένα με doc_id) εμφανίζεται.

- **Αποθήκευση του ανεστραμμένου ευρετηρίου**: Αποθηκεύουμε το ανεστραμμένο ευρετήριο σε ένα νέο αρχείο JSON για μελλοντική χρήση.


In [ ]:
import json

# Διαβάζουμε τα προεπεξεργασμένα δεδομένα
with open('processed_programming_languages_text.json', 'r', encoding='utf-8') as file:
    processed_data = json.load(file)

# Αρχικοποιούμε το ανεστραμμένο ευρετήριο (Inverted Index)
inverted_index = {}

# Δημιουργούμε το ανεστραμμένο ευρετήριο
for doc_id, text in processed_data.items():
    # Διαχωρίζουμε το κείμενο σε λέξεις
    words = text.split()
    
    for word in words:
        # Προσθέτουμε τη λέξη στο ανεστραμμένο ευρετήριο
        if word not in inverted_index:
            inverted_index[word] = []  # Αρχικοποιούμε μία κενή λίστα για τη λέξη
        if doc_id not in inverted_index[word]:
            inverted_index[word].append(doc_id)

# Αποθηκεύουμε το ανεστραμμένο ευρετήριο σε αρχείο
output_index_path = 'inverted_index.json'
with open(output_index_path, 'w', encoding='utf-8') as file:
    json.dump(inverted_index, file, ensure_ascii=False, indent=4)

print(f"Το ανεστραμμένο ευρετήριο αποθηκεύτηκε στο {output_index_path}")



![step 3](./project/images/step_3.png)

Το νέο JSON αρχείο:

In [ ]:
{
    "jump": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)",
        "https://en.wikipedia.org/wiki/Kotlin"
    ],
    "content": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)",
        "https://en.wikipedia.org/wiki/Kotlin"
    ],
    "main": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)",
        "https://en.wikipedia.org/wiki/Kotlin"
    ],
    "menu": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)",
        "https://en.wikipedia.org/wiki/Kotlin"
    ],
    "page": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)",
        "https://en.wikipedia.org/wiki/Kotlin"
    ],
    "current": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)",
        "https://en.wikipedia.org/wiki/Kotlin"
    ],
    "event": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)",
        "https://en.wikipedia.org/wiki/Kotlin"
    ],
    "random": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)",
        "https://en.wikipedia.org/wiki/Kotlin"
    ],
    "articl": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)",
        "https://en.wikipedia.org/wiki/Kotlin"
    ],
    "wikipedia": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)",
        "https://en.wikipedia.org/wiki/Kotlin"
    ],
    "contact": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)",
        "https://en.wikipedia.org/wiki/Kotlin"
    ],
    "u": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)",
        "https://en.wikipedia.org/wiki/Kotlin"
    ],
    "help": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)",
        "https://en.wikipedia.org/wiki/Kotlin"
    ],
    "learn": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)",
        "https://en.wikipedia.org/wiki/Kotlin"
    ],
    "edit": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)",
        "https://en.wikipedia.org/wiki/Kotlin"
    ],
    "commun": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)",
        "https://en.wikipedia.org/wiki/Kotlin"
    ],
    "portal": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)",
        "https://en.wikipedia.org/wiki/Kotlin"
    ],
    "recent": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)",
        "https://en.wikipedia.org/wiki/Kotlin"
    ],
    "chang": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)",
        "https://en.wikipedia.org/wiki/Kotlin"
    ],
    "upload": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)",
        "https://en.wikipedia.org/wiki/Kotlin"
    ],
    "file": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)",
        "https://en.wikipedia.org/wiki/Kotlin"
    ],
    "search": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)",
        "https://en.wikipedia.org/wiki/Kotlin"
    ],
    "appear": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)",
        "https://en.wikipedia.org/wiki/Kotlin"
    ],
    "donat": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)",
        "https://en.wikipedia.org/wiki/Kotlin"
    ],
    "creat": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)",
        "https://en.wikipedia.org/wiki/Kotlin"
    ],
    "account": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)",
        "https://en.wikipedia.org/wiki/Kotlin"
    ],
    "log": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)",
        "https://en.wikipedia.org/wiki/Kotlin"
    ],
    "person": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)",
        "https://en.wikipedia.org/wiki/Kotlin"
    ],
    "tool": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)",
        "https://en.wikipedia.org/wiki/Kotlin"
    ],
    "contribut": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)",
        "https://en.wikipedia.org/wiki/Kotlin"
    ],
    "talk": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)",
        "https://en.wikipedia.org/wiki/Kotlin"
    ],
    "top": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "1": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "histori": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)",
        "https://en.wikipedia.org/wiki/Kotlin"
    ],
    "2": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "design": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "philosophi": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "featur": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "3": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "syntax": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "semant": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "toggl": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "subsect": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "3.1": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "indent": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "3.2": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "statement": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)",
        "https://en.wikipedia.org/wiki/Kotlin"
    ],
    "control": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "flow": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "3.3": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "express": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "3.4": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "method": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "3.5": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "type": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "3.6": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "arithmet": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "oper": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "3.7": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "function": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "4": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "program": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)",
        "https://en.wikipedia.org/wiki/Kotlin"
    ],
    "exampl": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "5": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "librari": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "6": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "develop": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)",
        "https://en.wikipedia.org/wiki/Kotlin"
    ],
    "environ": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "7": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "implement": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "7.1": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "refer": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)",
        "https://en.wikipedia.org/wiki/Kotlin"
    ],
    "7.2": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "7.3": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "longer": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "support": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "7.4": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "cross-compil": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "languag": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)",
        "https://en.wikipedia.org/wiki/Kotlin"
    ],
    "7.5": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "perform": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "8": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "9": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "api": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "document": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "gener": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "10": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)",
        "https://en.wikipedia.org/wiki/Kotlin"
    ],
    "name": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)",
        "https://en.wikipedia.org/wiki/Kotlin"
    ],
    "11": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "popular": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "12": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "use": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)",
        "https://en.wikipedia.org/wiki/Kotlin"
    ],
    "13": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "influenc": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "python": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "14": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "see": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "also": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "15": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)",
        "https://en.wikipedia.org/wiki/Kotlin"
    ],
    "15.1": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "sourc": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "16": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "read": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)",
        "https://en.wikipedia.org/wiki/Kotlin"
    ],
    "17": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "extern": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "link": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)",
        "https://en.wikipedia.org/wiki/Kotlin"
    ],
    "tabl": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "112": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "afrikaan": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "alemannisch": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "العربية": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "aragoné": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "অসমীয়া": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "asturianu": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "azərbaycanca": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "تۆرکجه": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "basa": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "bali": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "বাংলা": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "閩南語": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "bân-lâm-gú": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "беларуская": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)",
        "https://en.wikipedia.org/wiki/Kotlin"
    ],
    "тарашкевіца": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "भोजपुरी": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "български": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "bosanski": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "brezhoneg": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "català": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "cebuano": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Kotlin"
    ],
    "čeština": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "cymraeg": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "dansk": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "deutsch": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)",
        "https://en.wikipedia.org/wiki/Kotlin"
    ],
    "eesti": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "ελληνικά": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "español": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)",
        "https://en.wikipedia.org/wiki/Kotlin"
    ],
    "esperanto": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "euskara": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "فارسی": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "françai": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)",
        "https://en.wikipedia.org/wiki/Kotlin"
    ],
    "galego": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "ગુજરાતી": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "한국어": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)",
        "https://en.wikipedia.org/wiki/Kotlin"
    ],
    "hausa": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "հայերեն": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "हिन्दी": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "hrvatski": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Kotlin"
    ],
    "ido": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "bahasa": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "indonesia": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "interlingua": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "íslenska": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "italiano": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)",
        "https://en.wikipedia.org/wiki/Kotlin"
    ],
    "עברית": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "ქართული": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "қазақша": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "kiswahili": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "kurdî": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "кыргызча": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "ລາວ": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "latina": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "latviešu": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "lietuvių": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "la": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    ".lojban": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "lombard": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "magyar": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "македонски": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "മലയാളം": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "मराठी": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "melayu": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "монгол": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "မြန်မာဘာသာ": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "na": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "vosa": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "vakav": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "nederland": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)",
        "https://en.wikipedia.org/wiki/Kotlin"
    ],
    "नेपाली": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "日本語": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)",
        "https://en.wikipedia.org/wiki/Kotlin"
    ],
    "ߒߞߏ": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "norsk": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "bokmål": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "nynorsk": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "ଓଡ଼ିଆ": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "oʻzbekcha": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "ўзбекча": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "ਪੰਜਾਬੀ": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "پنجابی": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "ភាសាខ្មែរ": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "plattdüütsch": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "polski": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)",
        "https://en.wikipedia.org/wiki/Kotlin"
    ],
    "portuguê": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "qaraqalpaqsha": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "română": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "runa": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "simi": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "русский": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)",
        "https://en.wikipedia.org/wiki/Kotlin"
    ],
    "саха": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "тыла": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "ᱥᱟᱱᱛᱟᱲᱤ": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "scot": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "shqip": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "සිංහල": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "simpl": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "english": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)",
        "https://en.wikipedia.org/wiki/Kotlin"
    ],
    "slovenčina": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)",
        "https://en.wikipedia.org/wiki/Kotlin"
    ],
    "slovenščina": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "کوردی": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "српски": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "srpski": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "srpskohrvatski": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "српскохрватски": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "suomi": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)",
        "https://en.wikipedia.org/wiki/Kotlin"
    ],
    "svenska": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)",
        "https://en.wikipedia.org/wiki/Kotlin"
    ],
    "tagalog": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "தமிழ்": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "татарча": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "tatarça": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "ၽႃႇသႃႇတႆး": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "తెలుగు": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "ไทย": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "тоҷикӣ": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "türkçe": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "ugi": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "українська": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "اردو": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "ئۇيغۇرچە": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "uyghurch": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "tiếng": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "việt": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "walon": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "文言": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "winaray": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "吴语": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "粵語": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "中文": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "view": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)",
        "https://en.wikipedia.org/wiki/Kotlin"
    ],
    "relat": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)",
        "https://en.wikipedia.org/wiki/Kotlin"
    ],
    "special": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)",
        "https://en.wikipedia.org/wiki/Kotlin"
    ],
    "perman": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)",
        "https://en.wikipedia.org/wiki/Kotlin"
    ],
    "inform": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)",
        "https://en.wikipedia.org/wiki/Kotlin"
    ],
    "cite": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)",
        "https://en.wikipedia.org/wiki/Kotlin"
    ],
    "get": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)",
        "https://en.wikipedia.org/wiki/Kotlin"
    ],
    "shorten": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)",
        "https://en.wikipedia.org/wiki/Kotlin"
    ],
    "url": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)",
        "https://en.wikipedia.org/wiki/Kotlin"
    ],
    "download": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)",
        "https://en.wikipedia.org/wiki/Kotlin"
    ],
    "qr": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)",
        "https://en.wikipedia.org/wiki/Kotlin"
    ],
    "code": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)",
        "https://en.wikipedia.org/wiki/Kotlin"
    ],
    "pdf": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)",
        "https://en.wikipedia.org/wiki/Kotlin"
    ],
    "printabl": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)",
        "https://en.wikipedia.org/wiki/Kotlin"
    ],
    "version": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)",
        "https://en.wikipedia.org/wiki/Kotlin"
    ],
    "wikimedia": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)",
        "https://en.wikipedia.org/wiki/Kotlin"
    ],
    "common": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)",
        "https://en.wikipedia.org/wiki/Kotlin"
    ],
    "mediawiki": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "wikibook": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "wikifunct": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "wikiquot": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "wikivers": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "wikidata": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)",
        "https://en.wikipedia.org/wiki/Kotlin"
    ],
    "item": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)",
        "https://en.wikipedia.org/wiki/Kotlin"
    ],
    "genu": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "paradigm": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "multi-paradigm": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "object-ori": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "procedur": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "imper": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "structur": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "reflect": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "guido": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "van": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "rossum": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "softwar": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "foundat": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)",
        "https://en.wikipedia.org/wiki/Kotlin"
    ],
    "33": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "year": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "ago": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "1991-02-20": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "stabl": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "releas": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "40": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "day": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "decemb": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "2024": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "disciplin": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "duck": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "dynam": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "strong": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "option": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "annot": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "o": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "tier": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "64-bit": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "linux": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "maco": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "64-": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "32-bit": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "window": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "10+": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "e.g": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "webassembl": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "wasi": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "android": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "io": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "freebsd": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "raspberri": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "pi": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "osunoffici": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "known": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "work": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "unix-like/bsd": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "variant": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "platform": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "unix-lik": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "bsd": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "licens": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)",
        "https://en.wikipedia.org/wiki/Kotlin"
    ],
    "filenam": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "extens": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "python.org": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "cpython": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "pypi": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "stackless": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "micropython": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "circuitpython": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "ironpython": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "jython": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "dialect": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "cython": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "rpython": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "starlark": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "abc": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "ada": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "algol": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "68": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "apl": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "c": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "c++": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "clu": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "18": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "dylan": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "19": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "haskel": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "20": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "icon": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "21": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "lisp": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "22": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "modula-3": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "perl": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "23": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "standard": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "ml": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "apach": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "groovi": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "boo": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "cobra": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "coffeescript": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "24": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "f": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "gdscript": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "go": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "javascript": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "25": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "26": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "julia": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "27": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "mojo": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "28": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "nim": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "ring": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "29": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "rubi": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "30": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "swift": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "31": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "v": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "32": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "high-level": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "general-purpos": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)",
        "https://en.wikipedia.org/wiki/Kotlin"
    ],
    "emphas": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "readabl": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "signific": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "type-check": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "garbage-collect": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "multipl": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "includ": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "particularli": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "often": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "describ": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "``": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "batteri": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "''": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "due": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "comprehens": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "34": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "35": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "began": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "late": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "1980": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "successor": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "first": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "1991": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "0.9.0": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "36": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "2.0": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "2000": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "3.0": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "2008": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "major": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "revis": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "complet": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "backward-compat": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "earlier": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "2.7.18": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "2020": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "last": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)",
        "https://en.wikipedia.org/wiki/Kotlin"
    ],
    "37": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "consist": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "rank": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "one": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "gain": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "widespread": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "machin": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "38": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "39": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "41": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "conceiv": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "42": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "centrum": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "wiskund": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "informatica": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "cwi": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "netherland": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "inspir": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "setl": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "43": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "capabl": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "except": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "handl": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "interfac": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "amoeba": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "system": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "1989": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "44": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "shoulder": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "sole": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "respons": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "project": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "lead": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "juli": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "2018": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "announc": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "vacat": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "'s": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "benevol": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "dictat": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "life": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "bdfl": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "titl": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "bestow": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "upon": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "long-term": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "commit": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "chief": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "decision-mak": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "45": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "sinc": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "come": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "retir": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "self-titl": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "bdfl-emeritu": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "januari": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)",
        "https://en.wikipedia.org/wiki/Kotlin"
    ],
    "2019": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "activ": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "core": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "elect": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "five-memb": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "steer": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "council": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "46": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "47": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "said": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "british": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "comedi": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "seri": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "monti": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "fli": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "circu": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "48": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "octob": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "mani": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "new": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "list": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "cycle-detect": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "garbag": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "collect": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "count": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "unicod": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "49": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "2.7": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "end-of-lif": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "initi": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "set": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "2015": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "postpon": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "concern": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "larg": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "bodi": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "exist": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "could": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "easili": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "forward-port": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "50": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "51": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "secur": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "patch": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "improv": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "52": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "53": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "older": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "offici": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "unsupport": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "differ": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)",
        "https://en.wikipedia.org/wiki/Kotlin"
    ],
    "unoffici": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "continu": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "i.e": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "2.7.18+": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "plu": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "3.10": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "mean": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "least": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "backport": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "updat": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "54": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "everi": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "ad": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "later": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "drop": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "outdat": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "modul": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "minor": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "way": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "3.13": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "latest": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "month": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "3.12": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "bug": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "fix": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "oppos": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "3.9": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "55": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "oldest": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "albeit": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "'secur": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "phase": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "3.8": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "reach": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "56": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "57": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "start": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "full": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "half": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "follow": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "total": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "expedit": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "2021": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)",
        "https://en.wikipedia.org/wiki/Kotlin"
    ],
    "twice": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "2022": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "2023": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "septemb": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "3.12.6": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "3.8.20": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "insecur": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "58": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "issu": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "possibl": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "remot": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "execut": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "59": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "web-cach": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "poison": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "60": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "union": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "61": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "match": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "case": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "keyword": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "pattern": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "3.11": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "expand": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "notabl": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "increas": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "speed": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "error": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "report": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "62": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "claim": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "faster": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "add": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "anoth": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "messag": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "3.14": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "introduc": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "interact": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "interpret": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "repl": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "multi-lin": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "color": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "increment": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "collector": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "produc": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "shorter": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "paus": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "lot": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "object": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "addit": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)",
        "https://en.wikipedia.org/wiki/Kotlin"
    ],
    "experiment": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "just-in-tim": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "jit": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "compil": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "can/ne": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "enabl": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "specif": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "63": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "free-thread": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "build": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "mode": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "disabl": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "global": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "lock": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "gil": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "allow": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "thread": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "run": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "concurr": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "latter": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "python3.13t": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "python3.13t.ex": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "behavior": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "well-defin": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "remov": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "deprec": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "class": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)",
        "https://en.wikipedia.org/wiki/Kotlin"
    ],
    "old": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "local": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "frame.f_loc": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "slow": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "inconsist": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "buggi": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "corner": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "odditi": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "around": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "may": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)",
        "https://en.wikipedia.org/wiki/Kotlin"
    ],
    "need": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "templat": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "print": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "debug": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "correctli": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "64": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "3.15": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "3.16": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "65": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "66": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "sigstor": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "digit": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "verif": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "signatur": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "artifact": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "pgp": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "critic": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "practition": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "move": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "exclus": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "67": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "alpha": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "regard": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "__future__": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "import": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "convert": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "string": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "pep": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "711": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "propos": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "pybi": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "format": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "distribut": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "binari": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "69": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "make": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "utf-8": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "default": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "70": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "opt": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "alreadi": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "elsewher": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "thing": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "open": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "fulli": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "cross-platform": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "everyth": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "aspect-ori": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "metaprogram": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "71": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "metaobject": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "72": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "via": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "contract": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "73": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "74": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "logic": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "75": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "glue": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "76": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "abl": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "well": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "ea": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "access": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "combin": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "memori": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "manag": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "77": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "resolut": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "bind": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "variabl": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "offer": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "tradit": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "filter": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "mapandreduc": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "dictionari": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "78": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "two": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "itertool": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "functool": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "borrow": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "79": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "summar": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "zen": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "aphor": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "80": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "beauti": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "better": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "ugli": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "explicit": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "implicit": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "complex": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "complic": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "howev": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "regularli": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "violat": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "principl": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "receiv": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "unnecessari": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "bloat": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "81": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "guidelin": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "rather": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "rule": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "82": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "controversi": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "resign": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "vitriol": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "assign": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "83": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "84": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "nevertheless": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "highli": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "compact": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "modular": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "made": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)",
        "https://en.wikipedia.org/wiki/Kotlin"
    ],
    "programm": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "applic": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "vision": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "small": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "stem": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "frustrat": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "espous": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "opposit": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "approach": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "strive": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "simpler": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "less-clutt": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "grammar": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "give": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "choic": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "methodolog": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "contrast": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "motto": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "embrac": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "one—and": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "prefer": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "one—obvi": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "practic": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "provid": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "achiev": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "task": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "three": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "liter": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "certainti": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "85": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "alex": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "martelli": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "fellow": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "book": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "author": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "wrote": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "someth": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "'clever": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "consid": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "compliment": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "cultur": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "86": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "usual": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "avoid": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "prematur": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "optim": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "reject": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "non-crit": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "part": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "would": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "margin": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "cost": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "clariti": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "87": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "speed-crit": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "written": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "like": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "either": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "n't": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "speed-up": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "might": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "expect": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "restrict": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "subset": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "slightli": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "88": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "aim": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "fun": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "name—a": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "tribut": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "group": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "89": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "—and": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "occasion": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "play": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "tutori": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "materi": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "term": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)",
        "https://en.wikipedia.org/wiki/Kotlin"
    ],
    "spam": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "egg": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "sketch": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "instead": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "often-us": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "foo": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "bar": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "90": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "91": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "neolog": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "wide": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "rang": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "style": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "idiom": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "natur": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "show": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "fluenci": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "conform": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "minimalist": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "emphasi": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "difficult": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "understand": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "rough": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "transcript": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "call": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "unpython": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "92": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "meant": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "visual": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "unclutt": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "punctuat": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "unlik": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "curli": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "bracket": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "delimit": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "block": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "semicolon": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "rare": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "fewer": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "syntact": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "pascal": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "93": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "§": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "whitespac": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "certain": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "decreas": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "signifi": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "end": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "94": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "thu": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "accur": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "repres": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "95": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "sometim": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "off-sid": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "recommend": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "size": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "four": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "space": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "96": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "singl": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "equal": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "sign": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "condit": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "along": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "el": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "elif": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "else-if": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "iter": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "captur": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "element": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "attach": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "long": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "true": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "tri": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "rais": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "caught": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "claus": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "97": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "ensur": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "clean-up": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "final": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "alway": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "regardless": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "exit": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "specifi": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "re-rais": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "namespac": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "def": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "defin": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "enclos": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "within": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "context": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "acquir": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "close": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "resource-acquisition-is-initi": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "raii": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "-like": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "replac": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "try/fin": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "98": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "break": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "loop": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "skip": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "rest": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "next": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "del": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "variable—delet": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "valu": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "redefin": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "pas": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "serv": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "nop": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "empti": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "assert": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "check": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "appli": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)",
        "https://en.wikipedia.org/wiki/Kotlin"
    ],
    "yield": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "return": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "coroutin": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "whose": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "analog": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "switch": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "construct": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "compar": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "control-of-flow": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "measur": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "separ": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "alloc": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "subsequ": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "rebound": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "time": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "holder": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "without": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "data": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "typing—in": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "statically-typ": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "contain": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "tail": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "first-class": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "accord": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "never": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "99": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "100": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "coroutine-lik": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "extend": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "101": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "2.5": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "lazi": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "unidirect": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "back": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "stack": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "level": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "102": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "mathemat": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "subtract": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "similar": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "divis": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "floor": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "integ": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "//": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "floating-point/divis": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "103": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "exponenti": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "concaten": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "duplic": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "number": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "infix": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "intend": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "numpi": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "matrix": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "104": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "105": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "walru": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "larger": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "106": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "==": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "ident": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "comparison": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "chained—for": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "b": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "c.": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "boolean": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "anonym": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "lambda": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "x": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "107": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "order": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "operand": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "distinct": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "tupl": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "mutabl": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "key": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "must": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "immut": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "directli": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "modifi": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "given": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "evalu": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "t—therebi": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "effect": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "parenthes": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "unambigu": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "108": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "sequenc": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "unpack": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "anyth": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "writabl": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "properti": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "etc": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "associ": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "manner": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "form": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "literals—and": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "whole": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "put": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "left-hand": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "side": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "right-hand": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "correspond": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "left": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "109": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "printf": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "c—e.g": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "spam=": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "eggs=": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "blah": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "spam=blah": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "eggs=2": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "2.6+": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "3+": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "supplement": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "str": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "0": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    ".format": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "f-string": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "f'spam=": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "110": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "'spam=": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "float": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "spamegg": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "variou": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "doubl": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "quot": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "unix": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "shell": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "perl-influenc": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "backslash": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "escap": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "charact": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "interpol": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "becam": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "avail": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)",
        "https://en.wikipedia.org/wiki/Kotlin"
    ],
    "triple-quot": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "begin": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "span": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "line": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "raw": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "varieti": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "denot": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "prefix": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "r.": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "henc": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "regular": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "windows-styl": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "path": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "-quot": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "array": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "index": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "slice": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "stop": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "step": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "zero-bas": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "neg": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "rel": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "take": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "third": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "paramet": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "stride": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "revers": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "omitted—for": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "copi": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "entir": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "shallow": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "rigidli": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "enforc": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "scheme": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "vs.": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "for-loop": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "eval": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "exec": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "built-in": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "former": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "expression—so": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "particular": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "instance.method": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "argument": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "normal": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "sugar": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "class.method": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "instanc": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "self": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "e.g.": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "java": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "objective-c": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "111": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "dunder": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "double-underscor": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "user-defin": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "nativ": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "length": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "convers": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "untyp": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "constraint": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "fail": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "suitabl": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "despit": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "strongli": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "forbid": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "silent": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "attempt": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "sen": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "spamclass": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "eggsclass": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "metaclass": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "kind": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "old-styl": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "new-styl": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "113": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "114": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "mypi": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "catch": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "115": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "116": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "mypyc": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "leverag": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "117": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "fals": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "byte": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "bytearray": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "'some": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "ascii": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "119": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "j": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "'key1": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "1.0": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "ellipsi": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "...": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "double-precis": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "floating-point": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "ieee": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "754": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "118": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "1.33333": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "frozenset": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "4.0": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)",
        "https://en.wikipedia.org/wiki/Kotlin"
    ],
    "'string": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "null": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "none": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "overload": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "notimpl": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "120": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "−": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "'wikipedia": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "'singl": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "symbol": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "modulo": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "remaind": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "-3": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "-2": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "125": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "0.5": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "matrix‑multipl": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "121": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "math": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "preced": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "unari": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "posit": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "respect": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "result": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "significantli": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "122": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "5/2": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "7//3": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "-7//3": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "7.5//3": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "-7.5//3": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "-3.0": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "caus": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "simpli": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "classic": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "round": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "toward": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "infin": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "though": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "equat": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "//b": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "a//b": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "valid": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "maintain": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "half-open": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "interv": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "lie": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "123": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "nearest": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "tie-break": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "even": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "1.5": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "124": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "round-away-from-zero": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "-0.5": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "−1.0": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "test": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "whether": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "less": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "126": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "c-deriv": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "127": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "arbitrary-precis": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "decim": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "type/class": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "pre-defin": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "arbitrari": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "precis": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "sever": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "128": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "fraction": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "ration": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "129": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "third-parti": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "frequent": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "scientif": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "script": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "aid": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "problem": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "numer": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "process": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "manipul": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "130": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "131": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "attribut": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "requir": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "whatev": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "printer": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "input1": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "input2": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "hello": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "output": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "want": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "variable-defin": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "insid": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "definit": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "world": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "'hello": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "calcul": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "factori": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "n": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "int": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "input": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "'type": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "valueerror": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "'you": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "enter": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "non-neg": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "132": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "suit": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "commonli": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "greatest": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "strength": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "internet-fac": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "protocol": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "mime": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "http": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)",
        "https://en.wikipedia.org/wiki/Kotlin"
    ],
    "graphic": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "user": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "connect": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "databas": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "pseudorandom": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "unit": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "cover": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "specifications—for": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "web": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "server": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "gateway": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "wsgi": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "wsgiref": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "333": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "133": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "—but": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "intern": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)",
        "https://en.wikipedia.org/wiki/Kotlin"
    ],
    "alter": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "rewrit": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "march": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "packag": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "repositori": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "523,000": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "134": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "autom": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "analyt": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "imag": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "mobil": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)",
        "https://en.wikipedia.org/wiki/Kotlin"
    ],
    "app": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "multimedia": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "comput": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "network": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "administr": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "framework": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "text": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)",
        "https://en.wikipedia.org/wiki/Kotlin"
    ],
    "scrape": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "integr": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "read–eval–print": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "permit": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "command": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "sequenti": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "immedi": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "ide": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "idl": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "beginner-ori": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "ipython": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "abil": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "auto-complet": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "session": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "state": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "retent": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "highlight": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "desktop": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "pycharm": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "intellij": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "idea": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "studio": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "browser-bas": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "sagemath": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "science-": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "math-rel": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "pythonanywher": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "host": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "canopi": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "commerci": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "135": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "browser": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "meet": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "c89": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "c11": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "136": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "select": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "c99": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "limit": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "versions—e.g": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "137": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "138": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "intermedi": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "bytecod": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "139": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "virtual": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "140": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "mixtur": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "instal": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "deliber": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "141": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "142": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "xp": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "modern": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "appl": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "m1": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "mac": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "3.9.1": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "vm": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "143": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "portabl": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "earliest": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "prioriti": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "144": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "os/2": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "solari": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "145": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "multi-thread": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "altern": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "unord": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "ecosystem": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "fast": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "compliant": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "146": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "147": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "bring": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "148": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "risc-v": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "codon": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "ahead-of-tim": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "aot": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "python-lik": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "nearli": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "149": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "speedup": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "10–100x": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "llvm": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "multithread": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "150": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "fork": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "microthread": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "massiv": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "151": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "microcontrol": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "lego": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "mindstorm": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "ev3": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "152": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "pyston": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "runtim": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "153": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "cinder": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "performance-ori": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "inlin": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "cach": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "eager": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "method-at-a-tim": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "154": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "snek": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "155": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "156": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "157": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "embed": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "compat": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "8-bit": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "avr": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "atmega": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "328p-base": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "arduino": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "python-inspir": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "write": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "158": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "oop": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "simplifi": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "single-precis": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "smaller": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "328p": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "googl": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "unladen": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "swallow": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "2009": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "five-fold": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "scale": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "thousand": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "159": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "ordinari": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "suffer": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "psyco": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "discontinu": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "transform": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "emit": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "pys60": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "phone": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "nokia": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "2005": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "symbian": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "n900": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "gtk": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "widget": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "target": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "devic": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "160": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "compilers/transpil": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "unrestrict": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "transpil": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "brython": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "161": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "transcrypt": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "162": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "163": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "pyj": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "2012": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "superset": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "usabl": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "direct": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "c-level": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "pyjl": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "compiles/transpil": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "human-read": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "high-perform": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "high": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "unless": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "base": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "nuitka": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "164": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "2.6": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "silicon-bas": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "free": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "add-on": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "hide": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "numba": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "decor": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "relev": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "translat": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "pythran": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "c++11": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "165": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "→": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "11l": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "166": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "c++17": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "myhdl": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "python-bas": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "hardwar": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "descript": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)",
        "https://en.wikipedia.org/wiki/Kotlin"
    ],
    "hdl": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "verilog": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "vhdl": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "3.x": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "grumpi": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "2017": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "167": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "168": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "169": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "although": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "170": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    ".net": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "171": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "172": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "pyrex": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "2010": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "shed": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "skin": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "2013": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "non-numer": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "combinatori": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "workload": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "present": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "euroscipi": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "'13": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "173": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "benchmark": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "game": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "174": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "conduct": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)",
        "https://en.wikipedia.org/wiki/Kotlin"
    ],
    "enhanc": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "primari": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "mechan": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "decis": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "175": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "176": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "outstand": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "review": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "comment": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "mail": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "python-dev": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "forum": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "origin": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "discus": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "roundup": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "tracker": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "177": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "migrat": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "github": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "178": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "took": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "place": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)",
        "https://en.wikipedia.org/wiki/Kotlin"
    ],
    "self-host": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "source-cod": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "mercuri": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "179": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "public": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "distinguish": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "backward-incompat": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "manual": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "port": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "happen": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "infrequently—vers": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "ever": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "180": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "previou": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "second": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "annual": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "181": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "182": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "183": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "184": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "occur": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "suffici": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "upstream": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "vulner": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "beta": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "release-candid": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "preview": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "schedul": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "delay": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "readi": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "team": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "monitor": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "185": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "academ": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "confer": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "pycon": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "mentor": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "pyladi": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "wstr": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "186": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "187": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "188": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "futur": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "deprecationwarn": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "w": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "ctype": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "http.server.cgihttprequesthandl": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "potenti": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "typing.namedtupl": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "field": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "disallow": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "pydoc": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "sphinx": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "pdoc": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "doxygen": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "graphviz": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "among": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "other": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "189": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "deriv": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "creator": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "enjoy": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "190": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "metasyntact": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "literatur": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "191": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "routin": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "192": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "193": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "pythonista": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "194": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "py-": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "pygam": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "directmedia": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "layer": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "pyqt": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "pygtk": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "qt": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "2003": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "ten": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "tiob": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "ahead": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "highest": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "rise": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "rate": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "2007": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "done": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "195": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "organ": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)",
        "https://en.wikipedia.org/wiki/Kotlin"
    ],
    "196": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "yahoo": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "197": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "cern": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "198": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "nasa": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "199": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "facebook": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "200": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "amazon": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "instagram": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "201": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "spotifi": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "202": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "entiti": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "industri": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "light": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "magic": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "203": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "ita": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "204": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "social": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "news": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "site": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)",
        "https://en.wikipedia.org/wiki/Kotlin"
    ],
    "reddit": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "mostli": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "205": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "partial": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "discord": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "206": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "baidu": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "207": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "mod_wsgi": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "webserv": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "208": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "evolv": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "facilit": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "django": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "pylon": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "pyramid": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "turbogear": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "web2pi": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "tornado": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "flask": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "bottl": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "zope": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "mainten": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "client-sid": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "ajax-bas": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "sqlalchemi": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "mapper": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "twist": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "dropbox": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "scipi": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "matplotlib": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "209": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "210": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "biopython": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "astropi": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "domain-specif": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "algebra": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "notebook": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "aspect": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "combinator": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "theori": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "calculu": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "211": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "opencv": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "rich": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "212": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "artifici": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "intellig": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "tensorflow": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "kera": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "pytorch": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "scikit-learn": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "problog": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "213": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "214": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "215": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "216": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "217": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "architectur": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "218": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "prolog": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "prove": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "ai": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "knowledg": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "represent": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "reason": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "janu": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "exploit": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "recurs": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "typic": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "queri": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "answer": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "geospati": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "219": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "220": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "natlog": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "dcg": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "prompt": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "text-to-text": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "gpt3": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "text-to-imag": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "dall-": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "diffus": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "221": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "gui": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "tkinter": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "222": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "223": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "success": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "product": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)",
        "https://en.wikipedia.org/wiki/Kotlin"
    ],
    "finit": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "abaqu": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "3d": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "parametr": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "model": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "freecad": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "anim": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "max": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "blender": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "cinema": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "4d": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "lightwav": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "houdini": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "maya": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "modo": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "motionbuild": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "softimag": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "compositor": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "nuke": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "2d": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "gimp": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "224": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "inkscap": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "scribu": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "paint": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "shop": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "pro": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "225": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "music": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "notat": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "scorewrit": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "capella": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "gnu": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "debugg": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "pretti": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "esri": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "promot": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "best": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "arcgi": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "226": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "video": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "227": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "228": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "adopt": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "engin": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "229": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "compon": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "ship": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "230": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "amigao": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "netbsd": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "openbsd": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "termin": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "ubuntu": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "ubiqu": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "red": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "hat": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "fedora": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "anaconda": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "gentoo": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "portag": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "231": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "232": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "laptop": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "per": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "child": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "xo": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "lab": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "233": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "single-board": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "user-program": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "libreoffic": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "234": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "februari": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "235": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "acknowledg": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "236": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "ecmascript–javascript": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "237": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "ecmascript": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "godot": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "238": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "239": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "share": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "motiv": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "desir": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "240": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "non-strict": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "241": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "still": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "miss": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "struct": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "242": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "243": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "yukihiro": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "matsumoto": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "power": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "decid": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "244": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "245": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "kotlin": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)",
        "https://en.wikipedia.org/wiki/Kotlin"
    ],
    "blend": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "minim": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "boilerpl": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "effici": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "246": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "emul": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "rational": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "surround": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "tcl": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "247": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "erlang": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "248": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "249": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "open-sourc": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "pip": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "faq": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "–": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "docs.python.org": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "retriev": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "0.9.1": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "01/21": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "alt.sourc": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "archiv": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "august": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "wiki": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "483": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "hint": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "june": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "peps.python.org": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "april": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "738": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "novemb": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "regress": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "3.7.13": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "underli": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "identifi": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "3.10.4": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "holth": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "moor": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "2014": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "0441": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "zip": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "experi": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "inclus": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "detail": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "kuchl": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "andrew": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "m.": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "2006": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "interview": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "1998": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "amk.ca": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "'d": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "spent": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "summer": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "dec": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "research": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "center": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "modula-2+": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "fact": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "explicitli": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "'self": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "came": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "algol-68": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "3.7.1": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "2016": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "sml": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "1993": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "introduct": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "unix/c": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "proceed": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "nluug": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "najaarsconferenti": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "dutch": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "citeseerx": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "10.1.1.38.2023.": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "far": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "ideal": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "consider": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "10.1.1.38.2023": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "found": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "lundh": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "fredrik": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "effbot.org": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "2017.": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "record": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "simionato": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "michel": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "2.3": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "c3": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "noth": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "invent": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "peopl": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "paper": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "lisper": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "a.": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "howto": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "v2.7.2": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "concis": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "schemenau": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "neil": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "peter": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "tim": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "hetland": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "magnu": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "2001": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "255": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "demand": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "3.10.6": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "coffeescript.org": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "www.2ality.com": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "rauschmay": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "axel": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "chapter": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "o'reilli": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "speak": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "websit": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "krill": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "paul": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "marri": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "mlir": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "infoworld": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "ring-lang.net": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "ring-lang": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "bini": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "ola": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "jrubi": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "rail": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "berkeley": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "apress": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "p.": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "isbn": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "978-1-59059-881-8": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "lattner": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "chri": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "homepag": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "tireless": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "effort": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "expert": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "guru": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "ninja": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "incred": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "dogfood": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "feedback": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "refin": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "battle-test": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "cours": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "greatli": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "benefit": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "hard-won": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "draw": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "rust": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "kuhlman": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "dave": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "advanc": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "exercis": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "section": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "1.1": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "2012.": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "fan": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "phrase": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "asynchron": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "brief": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "timelin": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "peterson": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "benjamin": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "overflow": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "survey": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "infograph": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "jetbrain": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "profession": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "indic": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "pypl": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "pypl.github.io": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "venner": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "bill": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "artima": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "lukewarm": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "2011": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "2011.": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "fairchild": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "carli": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "role": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "journal": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "8100": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "govern": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "brigg": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "jason": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "lipovača": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "miran": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "kid": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "san": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "francisco": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "calif": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "starch": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "press": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "978-1-59327-407-8": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "zadka": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "mosh": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "373": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "466": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "2.7.x": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "sunset": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "mattip": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "v7.3.14": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "langa": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "łukasz": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "3.9.13": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "statu": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "guid": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "endoflife.d": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "cve-2021-3177": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "custom": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "cve": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "cve-2021-23336": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "corbet": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "lwn.net": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "667": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "wouter": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "thoma": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "3.12.3": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "3.13.0a6": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "594": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "dead": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "softtwar": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "761": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "2025": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "749": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "649": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "686": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "cain": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "gang": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "ltd.": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "pydbc": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "precondit": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "postcondit": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "invari": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "pydatalog": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "togeth": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "heavi": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "malloc": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "strategi": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "leak": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "freed": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "chosen": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "hetting": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "raymond": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "2002": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "289": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "6.5": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "2004": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "lutz": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "mark": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "2014+": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "confus": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "ambi": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "chetan": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "grifski": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "jeremi": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "behind": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "renegad": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "coder": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "bader": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "dan": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "real": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "ravenscroft": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "anna": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "ascher": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "david": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "cookbook": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "2nd": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "medium": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "978-0-596-00797-3": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "ebeab": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "necessari": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "obtain": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "52.6×": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "19.5×": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "forc": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "pprint": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "3.11.0": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "2022.": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "stuff=": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "'spam": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "'egg": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "'lumberjack": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "'knight": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "'ni": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "hitchhik": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "docs.python-guide.org": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "good": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "myth": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "secnetix.d": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "guttag": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "john": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "v.": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "mit": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "978-0-262-52962-4": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "3.12.0a0": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "elimin": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "neopythonic.blogspot.b": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "solv": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "puzzl": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "ebi": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "phillip": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "342": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "380": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "0465": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "dedic": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "3.5.1": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "changelog": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "308": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "3.6.3rc1": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "5.3": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "3.7.1rc2": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "498": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "sweigart": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "al": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "beyond": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "basic": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "stuff": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "clean": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "322": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "978-1-59327-966-0": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "2.7.1": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "484": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "—": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "static": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "mypyc.readthedocs.io": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "point": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "3.8.3": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "almost": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "today": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "ieee-754": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "map": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "unifi": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "465": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "§2": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "beazley": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "essenti": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "4th": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "ed.": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "addison-wesley": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "9780672329784": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "kernighan": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "brian": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "w.": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "ritchi": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "denni": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "1988": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "batista": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "facundo": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "327": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "v2.6.9": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "rock": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "hoyt": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "koepk": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "univers": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "washington": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "depart": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "statist": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)",
        "https://en.wikipedia.org/wiki/Kotlin"
    ],
    "scott": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "piotrowski": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "przemyslaw": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "rapid": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "oracl": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "technolog": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "v1.0": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "enthought": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "www.enthought.com": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "3.9.2": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "3.9.0": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "vmspython": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "www.vmspython.org": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "oreilly.com": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "pypy.org": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "speed.pypy.org": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "lawson": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "lorain": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "mit-creat": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "application-level": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "2.0.2": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "doc.pypy.org": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "python-for-ev3": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "educ": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "yegulalp": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "serdar": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "aroca": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "rafael": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "lang": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "feel": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "yet": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "blog": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "aufranc": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "cnxsoft": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "jean-luc": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "snekboard": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "crowdfund": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "cnx": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "kennedi": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "mkennedi": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "michael": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "find": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "'re": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "git": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "famou": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "pythonbytes.fm": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "packard": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "keith": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "plan": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "stochast": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "geometri": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "brython.info": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "transcrypt.org": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "anatomi": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "infoq": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "home": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "nuitka.net": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "guelton": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "serg": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "brunet": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "pierrick": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "amini": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "mehdi": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "merlini": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "adrien": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "corbillon": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "xavier": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "raynaud": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "alan": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "scienc": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "discoveri": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "iop": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "publish": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "014001": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "bibcode:2015c": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "....": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "8a4001g": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "doi:10.1088/1749-4680/8/1/014001": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "issn": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "1749-4699": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "bibcod": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "2015c": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "doi": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "10.1088/1749-4680/8/1/014001": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "google/grumpi": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "opensource.googl": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "claburn": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "www.theregister.com": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "ironlanguages/ironpython3": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "built": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)",
        "https://en.wikipedia.org/wiki/Kotlin"
    ],
    "ironpython.net": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "www.jython.org": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "murri": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "riccardo": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "european": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "arxiv:1404.6388": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "bibcode:2014arxiv1404.6388m": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "arxiv": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "1404.6388": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "2014arxiv1404.6388m": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "warsaw": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "barri": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "hylton": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "goodger": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "purpos": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "cannon": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "brett": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "guy": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "devguide.python.org": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "hugh": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "owen": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "arriv": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "techrepubl": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "602": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "cycl": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "cadenc": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "norwitz": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "neal": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "stabil": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "aahz": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "baxter": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "anthoni": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "buildbot": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "623": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "634": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "whet": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "appetit": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "exchang": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "inc.": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)",
        "https://en.wikipedia.org/wiki/Kotlin"
    ],
    "9781449379322": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "fehili": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "peachpit": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "xv": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "9780201748840": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "lubanov": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "sebastopol": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "ca": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "305": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "978-1-4493-5936-2": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "blake": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "troy": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "seniordba": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "holi": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "grail": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "bulletin": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "31/2006": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "shafer": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "daniel": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "g.": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "streamlin": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "shuttl": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "mission": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "real-tim": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "hundr": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "dozen": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "fortenberri": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "taft": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "darryl": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "k.": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "slither": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "eweek.com": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "ziff": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "davi": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "hold": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "reddit-archive/reddit": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "histor": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "reddit.com.": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "reddit.com": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "elixir": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "usag": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "market": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "oliph": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "travi": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "10–20": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "bibcode:2007cs": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    ".....": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "9c": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "..": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "10o": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "10.1.1.474.6460.": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "doi:10.1109/mcse.2007.58": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "1521-9615": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "s2cid": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "206457124": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "10–": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "2007cse": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "10.1.1.474.6460": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "10.1109/mcse.2007.58": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "millman": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "jarrod": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "aivazi": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "scientist": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "9–12": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "bibcode:2011cs": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "13b": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "9m": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "doi:10.1109/mcse.2011.36": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "9–": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "2011cse": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "10.1109/mcse.2011.36": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "innov": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "opencv-python": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "docs.opencv.org": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "dean": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "jeff": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "monga": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "rajat": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "et": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "large-scal": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "heterogen": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "tensorflow.org": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "piatetski": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "gregori": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "eat": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "away": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "r": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "trend": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "analysi": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "kdnugget": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "0.20.1": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "scikit-learn.org": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "jouppi": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "norm": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "supercharg": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "tpu": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "chip": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "cloud": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "de": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "raedt": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "luc": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "kimmig": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "angelika": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "probabilist": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "concept": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "5–47": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "doi:10.1007/s10994-015-5494-z": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "3166992": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "5–": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "10.1007/s10994-015-5494-z": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "toolkit": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "nltk": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "3.5b1": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "www.nltk.org": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "andersen": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "t.": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "bridg": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "pp": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "93–104": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "cham": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "springer": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "switzerland": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "swi-prolog": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "tarau": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "learnabl": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "logic-bas": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "359–371": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "tcl/tk": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "ask": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "question": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "jasc": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "psp9compon": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "geoprocess": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "9.2": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "environment": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "institut": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "ccp": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "porkbelli": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "eve": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "dev": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "know": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "caudil": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "mod": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "sid": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "meier": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "civil": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "iv": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "firaxi": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "2010.": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "xml": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "let": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "modder": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "setup": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "immun": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "libreoffice.org": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "gotcha": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "boo.codehaus.org": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "codehau": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "esterbrook": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "charl": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "cobra-language.com": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "es4": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "wiki.ecmascript.org": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "kincaid": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "techcrunch": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "strachan": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "jame": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "birth": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "doc": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "docs.modular.com": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "member": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "famili": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "right": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "goal": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "long-tail": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "hope": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "familiar": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "systems-level": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "fall": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "spencer": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "datasciencelearningcenter.substack.com": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "reminisc": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "if/elif/then/els": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "linuxdevcenter.com": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "much": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "amaz": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "earnest": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "focu": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "jalan": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "nishant": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "aanjaney": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "codex": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "kupri": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "andrea": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "donal": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "tip": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "tcl.tk": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "xchang": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "gustafsson": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "niskanen": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "raimo": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "eep": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "erlang.org": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "evolut": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "pain": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "jocelyn": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "ed": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "newslett": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "amzi": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "pyaiml": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "0.8.5": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "pypi.python.org": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "russel": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "stuart": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "norvig": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "3rd": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "upper": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "saddl": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "river": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "nj": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "prentic": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "hall": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "978-0-13-604259-4": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "j.": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "downey": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "allen": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "think": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "978-1098155438": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "5th": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "978-0-596-15806-4": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "summerfield": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "978-0-321-68056-3": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "ramalho": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "luciano": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "fluent": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "978-1-4920-5632-4": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "sister": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "quotat": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "textbook": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "resourc": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "e": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "clpython": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "s60": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "eric": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "ninja-id": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "pydev": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "spyder": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "asgi": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "simula": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "assembl": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "cobol": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "forth": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "fortran": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "lua": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "matlab": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "php": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "sql": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "scratch": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "smalltalk": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "alphabet": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "categor": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "non-english-bas": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "categori": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)",
        "https://en.wikipedia.org/wiki/Kotlin"
    ],
    "cherrypi": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "cubicweb": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "fastapi": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "grok": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "nevow": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "quixot": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "twistedweb": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "differenti": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "manifold": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "automat": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "neuromorph": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "recognit": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "ricci": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "induct": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "bia": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "ipu": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "vpu": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "memristor": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "spinnak": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "theano": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "jax": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "flux.jl": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "mindspor": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "closed-sourc": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "source-code-host": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "facil": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "directori": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "grati": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "versu": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "libr": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "outlin": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "softwarepackag": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "audio": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "bioinformat": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "codec": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "configur": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "driver": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "wireless": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "health": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "offic": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "rout": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "televis": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "e-commerc": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "formerli": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "proprietari": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "movement": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "open-source-softwar": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "advocaci": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "organis": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "india": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "afl": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "apsl": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "artist": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "beerwar": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "creativ": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)",
        "https://en.wikipedia.org/wiki/Kotlin"
    ],
    "cddl": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "epl": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "gpl": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "agpl": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "lgpl": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "isc": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "mpl": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "sleepycat": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "unlicens": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "wtfpl": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "zlib": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "contributor": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "agreement": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "copyleft": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "debian": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "permiss": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "domain": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "prolifer": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "mozilla": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "rebrand": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "firmwar": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "sco/linux": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "patent": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "tivoiz": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "trust": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "manifesto": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "microsoft": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "promis": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "open-cor": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "source-avail": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "cathedr": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "bazaar": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "revolut": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "dataplot": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "epi": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "info": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "cspro": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "x-12-arima": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "admb": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "dap": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "gretl": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "jamovi": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "jasp": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "jag": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "jmulti": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "jupyt": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "octav": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "openbug": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "orang": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "pspp": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "statsmodel": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "pymc3": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "rstudio": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "simfit": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "sofa": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "stan": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "xlispstat": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "freewar": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "bv4.1": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "cumfreq": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "segreg": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "xplore": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "winbug": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "desk": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "gauss": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "graphpad": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "instat": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "prism": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "ibm": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "spss": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "jmp": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "mapl": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "mathcad": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "mathematica": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "oxmetr": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "rat": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "sa": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "smartpl": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "stata": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "statview": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "sudaan": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "s-plu": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "tsp": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "wp": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "bmdp": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "eview": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "genstat": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "limdep": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "lisrel": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "medcalc": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "microfit": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "minitab": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "mlwin": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "ncss": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "shazam": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "sigmastat": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "statistica": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "statsdirect": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "statxact": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "systat": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "unscrambl": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "unistat": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "excel": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "analyse-it": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "xlfit": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "rexcel": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "numerical-analysi": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "simul": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "chapel": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "euler": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "toolbox": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "freefem++": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "freemat": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "geniu": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "gmsh": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "mfem": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "openfoam": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "salom": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "scicoslab": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "scilab": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "x10": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "weka": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "fortress": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "dadisp": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "featool": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "multiphys": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "labview": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "speakeasi": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "vissim": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "germani": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "franc": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "bnf": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "czech": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "republ": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "israel": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "idref": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "//en.wikipedia.org/w/index.php": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)",
        "https://en.wikipedia.org/wiki/Kotlin"
    ],
    "title=python_": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "programming_languag": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "oldid=1268942017": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "class-bas": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "text-ori": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "short": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)",
        "https://en.wikipedia.org/wiki/Kotlin"
    ],
    "dmi": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "date": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "american": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "mismatch": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "hidden": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "07:40": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)"
    ],
    "utc": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)",
        "https://en.wikipedia.org/wiki/Kotlin"
    ],
    "attribution-sharealik": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)",
        "https://en.wikipedia.org/wiki/Kotlin"
    ],
    "agre": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)",
        "https://en.wikipedia.org/wiki/Kotlin"
    ],
    "privaci": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)",
        "https://en.wikipedia.org/wiki/Kotlin"
    ],
    "polici": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)",
        "https://en.wikipedia.org/wiki/Kotlin"
    ],
    "wikipedia®": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)",
        "https://en.wikipedia.org/wiki/Kotlin"
    ],
    "regist": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)",
        "https://en.wikipedia.org/wiki/Kotlin"
    ],
    "trademark": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)",
        "https://en.wikipedia.org/wiki/Kotlin"
    ],
    "non-profit": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)",
        "https://en.wikipedia.org/wiki/Kotlin"
    ],
    "disclaim": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)",
        "https://en.wikipedia.org/wiki/Kotlin"
    ],
    "cooki": [
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)",
        "https://en.wikipedia.org/wiki/Kotlin"
    ],
    "creation": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "netscap": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "1.2": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "1.3": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "jscript": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "1.4": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "growth": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "matur": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "5.1": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "5.2": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "6.1": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "6.2": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "weakli": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "6.3": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "6.3.1": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "6.3.2": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "run-tim": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "6.4": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "object-orient": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "prototype-bas": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "6.4.1": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "prototyp": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "6.4.2": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "constructor": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "6.4.3": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "6.5.1": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "lexic": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "closur": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "6.5.2": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "6.6": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "deleg": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "6.6.1": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "trait": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "mixin": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "6.6.2": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "composit": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "inherit": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "6.7": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "miscellan": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "6.7.1": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "6.7.2": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "variad": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "6.7.3": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "6.7.4": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "6.7.5": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "async/await": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "6.7.5.1": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "6.7.5.2": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "6.8": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "vendor-specif": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "8.1": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "cross-sit": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "8.2": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "request": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "forgeri": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "8.3": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "misplac": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "client": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "8.4": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "8.5": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "plugin": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "8.6": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "sandbox": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "8.7": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "10.1": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "10.2": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "json": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "10.3": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "10.4": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "ænglisc": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "чӑвашла": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "客家語": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "hak-kâ-ngî": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "jawa": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "lingua": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "franca": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "nova": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/Kotlin"
    ],
    "مصرى": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "مازِرونی": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "閩東語": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "mìng-dĕ̤ng-ngṳ̄": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "nordfriisk": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "ပအိုဝ်ႏဘာႏသာႏ": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "pinayuanan": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "soomaaliga": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "sunda": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "taqbaylit": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "tetun": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "türkmenç": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "vèneto": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "žemaitėška": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "accept": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "javanes": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "event-driven": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "brendan": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "eich": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "1995-12-04": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "weak": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    ".j": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    ".cj": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    ".mj": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "ecma-international.org/publications-and-standards/standards/ecma-262/": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "v8": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "javascriptcor": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "spidermonkey": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "chakra": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "awk": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "hypertalk": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "actionscript": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "arkt": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "assemblyscript": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "dart": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "hax": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "js++": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "opa": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "typescript": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "/ˈdʒɑːvəskrɪpt/": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "abbrevi": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "alongsid": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "html": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "cs": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "webpag": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "ˈdʒɑːvəskrɪpt": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "ˈ": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "dʒ": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "ɑː": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "ə": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "k": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "ɪ": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "p": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "util": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "non-brows": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "node.j": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "dom": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "input/output": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "i/o": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "storag": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "mosaic": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "non-techn": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "promin": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "earli": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "corpor": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "polish": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Kotlin"
    ],
    "navig": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "1994": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "quickli": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "most-us": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "lack": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "load": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "flourish": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "scene": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "1995": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "pursu": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "collabor": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "sun": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "microsystem": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "emb": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "hire": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "mass": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "nonprogramm": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "soon": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "devi": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "extant": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "livescript": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "impli": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "dot-com": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "boom": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "begun": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "ploy": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "debut": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "internet": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "explor": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "war": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "front": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "1996": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "notic": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "counterpart": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "logo": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "period": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "sidekick": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "annoy": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "pop-up": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "scroll": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "bottom": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "submit": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "ecma": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "vendor": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "led": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "1997": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "1999": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "undermin": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "increasingli": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "domin": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "facto": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "particip": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "eventu": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "mothbal": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "stagnant": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "firefox": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "join": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "e4x": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "jointli": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "macromedia": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "adob": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "draft": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "tamarin": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "establish": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "cooper": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "fruition": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "meanwhil": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "affili": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "jess": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "garrett": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "white": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "coin": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "ajax": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "backbon": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "background": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "reload": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "spark": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "renaiss": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "spearhead": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "jqueri": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "dojo": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "mootool": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "chrome": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "competit": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "overhaul": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "dispar": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "parti": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "oslo": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "drive": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "forward": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "ambiti": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "culmin": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "formal": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "ryan": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "dahl": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "outsid": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "node": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "therebi": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "stand-alon": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "million": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "npm": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "openli": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "snapshot": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "vet": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "upcom": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "individu": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "substanti": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "single-pag": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "javascript-heavi": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "transfer": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "letter": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "circul": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "14,000": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "signatori": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "websocket": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "send": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "leav": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "fade": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "resiz": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "playback": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "stream": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "alert": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "box": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "sent": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "owner": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "track": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "redirect": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "store": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "indexeddb": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "angular": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "bootstrap": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "lodash": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "modernizr": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "react": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "underscor": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "vue": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "conjunct": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "vanilla": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "reli": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "root": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "server-sid": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "deploy": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "enterpris": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "servic": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "nich": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "grow": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "electron": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "cordova": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "acrobat": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "gnome": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "mere": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "concert": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "render": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "standalon": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "broader": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "single-thread": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "queue": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "frame": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "shrink": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "proce": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "non-block": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "callback": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "mous": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "click": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "wait": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "deno": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "bun": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "otherwis": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "etc.": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "scope": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "var": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "const": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "insert": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "omit": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "implicitli": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "cast": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "depend": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "binary64": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "preserv": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "bigint": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "as-i": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "comma": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "tostr": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "valueof": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "mistaken": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "nan": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "mention": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "mislead": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "remain": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "wrap": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "bound": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "reassign": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "dougla": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "crockford": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "augment": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "act": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "newer": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "orient": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "dot": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "obj.x": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "obj": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "delet": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "belong": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "chain": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "enumer": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "object.cr": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "determin": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "bad": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "invoc": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    ".call": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    ".bind": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "nest": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "outer": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "invok": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "constant": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "becom": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "inner": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "conclud": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "role-ori": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "function-bas": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "wherea": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "walk": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "own": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "zero-index": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "indefinit": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "succinct": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "shortcut": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "basi": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "sophist": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "citat": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "handler": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "action": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "scenario": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "promise.rac": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "promise.al": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "promise.allsettl": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "promise.ani": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "synchron": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "overhead": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "similarli": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "non-standard": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "arg": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "expr": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "predat": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "arrow": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "declar": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "function-scop": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "undefin": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "x2": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "block-scop": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "2015.": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "un-reassign": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "z": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "global-scop": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "strict": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "mynumb": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "dynamically-typ": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "legal": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "note": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "slash": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "provis": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "consol": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "frown": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "overwrit": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "'foo": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "myelem": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "createel": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "'span": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "id": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "classlist": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "'bar": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "tag": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "look": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "class=": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "id=": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "data-attr=": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "baz": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "/span": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "setattribut": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "'data-attr": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "'baz": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "myelem.dataset.attr": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "append": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "appendchild": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "grab": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "queryselector": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "queryselectoral": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "foreach": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "'.class": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "data-oth": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "'.multipl": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "array-lik": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "nodelist": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "legitimaci": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "isnan": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "===": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "warn": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "closest": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "non-integ": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "gamma": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "instead.": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "repeat": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "actual": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "es6": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "recursivelycomput": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "ternari": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "counter": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "++": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "non-loc": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "6th": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "parenthesi": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "long_exampl": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "brace": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "short_exampl": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "no_parenthes": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "fill": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "generate_multiplier_funct": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "||": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "five_multipl": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "suppli": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "seed": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "retain": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "ball": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "radiu": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "area": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "mybal": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "instanti": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "demonstr": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "sum": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "+=": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "reduc": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "immediately-invok": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "gather": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "privat": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "re-ent": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "rawcount": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "dynamiccount": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "counter1": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "counter2": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "infinit": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "export": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "mymodule.j": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "'alic": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "age": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "num1": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "num2": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "'./mymodule.j": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "alic": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "'./module.j": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "sampl": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "display": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "lowest": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "lcm": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "lcmcalcul": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "throw": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "typeerror": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "checkint": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "^^^^": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "newlin": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "enough": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "appropri": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "gcd": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "divisor": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "euclidean": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "algorithm": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "ab": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "swap": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "destructur": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "redefinition—it": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "this.gcd": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "redefinit": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "lcmcalculator.prototype.gcd": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "wrong": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "afterward": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "'gcd": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "collid": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "|lcm|": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "|this.lcm|": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "|this.a": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "this.b|": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "fp": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "createtextnod": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "'br": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "1.6": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "inher": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "pair": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "sort": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "shorthand": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "printresult": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "275": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "638": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "malici": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "deliv": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "risk": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "web-rel": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "constrain": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "same-origin": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "usernam": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "password": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "javascript-rel": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "breach": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "javascript—adsaf": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "se": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "—that": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "greater": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)",
        "https://en.wikipedia.org/wiki/Kotlin"
    ],
    "especi": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "advertis": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "safe": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "isol": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "xss": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "attack": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "onlin": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "bank": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "victim": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "privileg": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "disclos": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "secret": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "money": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "solut": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "sanit": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "protect": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "correct": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "prevent": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "mistak": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "csrf": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "trick": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "authent": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "carri": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "login": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "credenti": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "referr": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "header": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "hijack": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "token": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "post": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "client-serv": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "recogn": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "untrust": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "assum": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "extract": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "adversari": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "implic": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "perfectli": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "conceal": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "obfusc": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "reverse-engin": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "conveni": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "verifi": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "invalid": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "right-click": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "save": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "sensit": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "pollut": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "bower": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "keep": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "emerg": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "blind": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "relied-upon": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "invers": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "unpatch": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "wild": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "studi": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "133,000": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "median": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "lag": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "newest": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "1,177": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "alexa": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "ceas": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "azer": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "koçulu": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "buffer": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "flaw": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "wish": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "overrun": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "superus": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "affect": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "safari": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "player": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "flash": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "activex": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "past": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "vista": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "confin": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "grant": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "incorrectli": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "pack": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "demot": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "hard": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "launch": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "non-sandbox": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "vbscript": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "theoret": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "viabl": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "vector": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "trojan": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "hors": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "uncommon": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "javascript-bas": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "proof-of-concept": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "rowhamm": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "bypass": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "aslr": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "aslr⊕cach": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "anc": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "spectr": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "specul": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "intel": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "processor": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "eslint": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "jslint": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "scan": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "profil": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "benchmark.j": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "jsbench": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "editor": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "misconcept": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "inde": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "c-like": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "ancestor": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "mind": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "reserv": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "convent": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "gosl": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "complement": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "performance-crit": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "portion": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "asm.j": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "forerunn": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "ecmascript®": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "nodejs/node-ep": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "seibel": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "craft": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "978-1-4302-1948-4": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "speakingjs.com": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "jsconf": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "youtub": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "2013.": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "22m": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "eight": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "foreword": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "goodman": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "danni": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "bibl": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "wiley": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "son": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "0-7645-3188-3": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "lccn": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "97078208": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "oclc": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "38888873": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "ol": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "712205m": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "w3tech": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "bloomberg": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "changer": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "marc": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "andreessen": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "enzer": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "larri": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "monmouth": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "fin": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "ceo": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "brave": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "corp.": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "encyclopædia": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "britannica": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "ultim": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "dvd": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "techvis": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "net": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "han": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "sheon": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "world—mayb": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "wire": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "champeon": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "steve": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "microsoft.com": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "mccracken": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "harri": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "unwelcom": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "technologizer.com": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "baker": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "loren": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "weber": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "assault": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "giant": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "bbc": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "big": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "opera": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "pc": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "computec": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "ag": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "purdi": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "kevin": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "lifehack": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "tracemonkey": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "lightspe": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "'are": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "overview": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "es6-features.org": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "scalabl": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "2017-03-24": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "wayback": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "01-oct-2012": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "sam": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "teach": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "hour": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "2017-03-23": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "05-sep-2012": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "lawton": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "georg": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "theserversid": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "brown": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "linux.com": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "branscomb": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "mari": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "yearli": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "es16": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "tc39": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "tc39.e": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "ashkena": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "u.": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "serial": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "75026640": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "uspto.gov": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "oracle.com": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "buy": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "7.4-bn": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "deal": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "econom": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "javascript™️": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "clouddevs.com": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "vanilla-js.com": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "clinick": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "inc": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "'ve": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "steadi": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "server—particularli": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "asp": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "mahemoff": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "vengeanc": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "readwrite.com": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "adobe.com": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "treitter": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "livejournal.com": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "tessel": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "minut": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "tessel.io": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "gpio": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "w3schools.com": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "espruino": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "espruino.com": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "looper": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "jen": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "idiot": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "telerik": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "blink": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "·": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "nelaturu": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "keerthi": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "medium.com": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "flanagan": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "978-0-596-55447-7": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "korolev": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "mikhail": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "quirk": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "proposal-bigint/advanced.md": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "master": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "tc39/proposal-bigint": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "bernhardt": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "gari": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "wat": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "destroy": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Kotlin"
    ],
    "mdn": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "176–178": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "herman": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "978-0-321-81218-6": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "haverbek": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "marijn": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "eloqu": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "95–97": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "978-1-59327-282-1": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "95–": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "katz": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "yehuda": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "125–127": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "125–": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "es5.github.com": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "talent": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "2017-10-05": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "peterseliger.blogspot.d": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "2014-07-24": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "cocktailj": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "cocktailjs.github.io": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "croll": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "angu": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "fresh": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "javascript…": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "139–149": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "139–": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "obsolet": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "ecma-262": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "developer.mozilla.org": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "adsaf": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "caja": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "mozillazin": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "talkback": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "mozillazine.org": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "kottelin": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "thor": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "forget": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "blog.anta.net": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "rehorik": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "jan": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "serviceobject": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "lauinger": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "tobia": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "chaaban": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "abdelberi": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "arshad": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "sajjad": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "robertson": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "william": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "wilson": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "christo": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "kirda": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "thou": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "shalt": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "analys": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "northeastern": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "arxiv:1811.00918": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "doi:10.14722/ndss.2017.23414": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "978-1-891562-46-4": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "17885720": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "1811.00918": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "10.14722/ndss.2017.23414": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "collin": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "broke": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "tini": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "piec": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "quartz": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "sc": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "magazin": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "uk": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "'break": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "crypto.signtext": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "2014-06-04": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "festa": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "buffer-overflow": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "ie": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "cnet": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "securitytracker.com": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "2010-02-18": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "securityfocu": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "webviewfoldericon": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "2011-10-11": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "fusion": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "ie7": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "ieblog": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "blogs.msdn.com": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "cert": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "vu": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "713878": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "properli": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "2009-10-30": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "advisori": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "2005–41": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "escal": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "overrid": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "starr": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "brow": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "technet": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "symantec": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "js.seeker.k": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "2011-09-13": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "grus": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "mauric": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "clémentin": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "mangard": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "stefan": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "rowhammer.j": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "software-induc": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "fault": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "arxiv:1507.06955": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "cs.cr": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "1507.06955": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "jean-pharun": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "alix": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "ingeni": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "hack": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "seen": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "motherboard": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "vice": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "goodin": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "dram": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "'bitflip": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "ar": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "technica": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "auerbach": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "truli": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "terrifi": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "slate.com": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "2017-03-16": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "vusec": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "aslr-bust": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "drive-bi": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "nastier": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "2018-01-03": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "benchmarkjs.com": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "jsben.ch": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "playground": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "edg": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "'on": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "--": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "dere": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "mohan": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "create-react-app": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "great": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "freecodecamp": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "panchal": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "krunal": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "7th": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "california": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "978-1-491-95202-3": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "472": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "978-1593279509": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "zaka": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "nichola": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "1st": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "978-1593275402": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "wiktionari": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "listen": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "2013-08-20": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "spoken": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "entireti": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "jshint": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "atscript": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "babel": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "clojurescript": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "elm": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "emscripten": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "morfik": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "purescript": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "websharp": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "devtool": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "inspector": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "komodo": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "jsdoc": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "ace": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "cloud9": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "atom": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "codemirror": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "phpstorm": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "orion": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "vim": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "cascad": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "sheet": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "html5": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "webauthn": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "pnpm": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "yarn": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "webpack": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "vite": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "esbuild": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "commonj": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "jsgi": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "jasmin": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "mocha": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "qunit": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "resig": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "qtscript": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "wmlscript": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "carakan": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "futhark": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "inscript": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "kj": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "linear": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "rhino": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "jägermonkey": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "chakracor": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "nashorn": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "echo": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "ext": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "live": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "kernel": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "midori": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "mochikit": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "qooxdoo": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "sproutcor": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "spri": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "wakanda": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "jaxer": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "appjet": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "wakandadb": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "cappuccino": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "backbone.j": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "swfobject": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "underscore.j": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "isaac": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "dhtml": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "jsss": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "sputnik": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "sunspid": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "bookmark": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "v5": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "webgl": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "webgpu": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "encrypt": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "ocsp": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "webrtc": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "arc": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "avast": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "coc": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "comodo": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "epic": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "maxthon": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "puffin": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "qq": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "samsung": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "silk": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "sleipnir": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "srware": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "uc": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "vivaldi": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "whale": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "yandex": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "foss": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "chromium": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "doobl": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "falkon": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "otter": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "supermium": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "ungoogl": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "gecko": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "floorp": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "icecat": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "librewolf": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "slimbrows": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "tor": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "basilisk": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "k-meleon": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "pale": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "moon": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "seamonkey": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "waterfox": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "webkit": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "icab": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "360": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "duckduckgo": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "konqueror": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "lunascap": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "netfront": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "qutebrows": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "eww": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "ladybird": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "lynx": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "netsurf": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "mini": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "w3m": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "beaker": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "citrio": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "flock": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "redcor": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "rockmelt": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "salamweb": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "torch": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "beonex": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "camino": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "classilla": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "conkeror": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "lite": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "galeon": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "ghostzilla": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "icedragon": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "kazehakas": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "kylo": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "lotu": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "microb": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "minimo": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "piratebrows": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "pogo": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "stratum": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "swiftfox": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "swiftweasel": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "tenfourfox": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "timberwolf": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "xb": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "mshtml": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "aol": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "deepnet": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "greenbrows": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "mediabrows": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "neoplanet": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "netcaptor": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "spacetim": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "zac": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "arora": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "bolt": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "dolphin": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "fluid": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "tv": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "iri": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "omniweb": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "coast": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "origyn": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "qtweb": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "shiira": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "steel": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "surf": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "uzbl": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "webposit": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "xombrero": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "abaco": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "amaya": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "arachn": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "arena": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "blazer": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "cake": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "charon": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "cm": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "deepfish": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "dillo": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "legaci": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "elink": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "gazel": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "hotjava": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "reader": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "webexplor": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "ibrows": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "kidzui": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "msn": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "netposit": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "skweezer": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "skyfir": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "thunderhawk": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "winwap": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "worldwideweb": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "mongodb": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "express.j": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "angularjs/angular": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "angularj": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "meen": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "substitut": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "ember.j": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "fastifi": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "meteor": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "sails.j": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "nestj": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "next.j": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "nuxt.j": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "react.j": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "vue.j": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "spain": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "elmcip": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "title=javascript": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "oldid=1263283332": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "object-bas": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "iso": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "cs1": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "unfit": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "webarch": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "semi-protect": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "excerpt": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "unsourc": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "haudio": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "microformat": [
        "https://en.wikipedia.org/wiki/JavaScript",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "19:41": [
        "https://en.wikipedia.org/wiki/JavaScript"
    ],
    "jvm": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "3.1.1": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "non-jvm": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "4.1": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "applet": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "servlet": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "javaserv": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "5.4": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "swing": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "5.5": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "javafx": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "5.6": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "10.1.1": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "bibliographi": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "acèh": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "አማርኛ": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "corsu": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "الدارجة": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "estremeñu": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "gaeilg": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "ಕನ್ನಡ": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "kreyòl": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "ayisyen": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "ligur": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "minangkabau": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "occitan": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "संस्कृतम्": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "سنڌي": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "1995-05-23": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "nomin": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "manifest": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    ".class": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    ".jar": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    ".war": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "oracle.com/java/": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "java.com": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "dev.java": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "simula67": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "eiffel": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "mesa": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "oberon": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "ucsd": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "beanshel": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "clojur": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "fantom": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "gamba": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "scala": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "seed7": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "vala": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "anywher": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "wora": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "recompil": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "low-level": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "modif": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "shortli": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "gradual": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "declin": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "complianc": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "relicens": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "gpl-2.0-onli": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "hotspot": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "openjdk": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "lt": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "mike": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "sheridan": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "patrick": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "naughton": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "cabl": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "oak": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "tree": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "stood": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "went": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "green": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "renam": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "coffe": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "c/c++-style": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "no-cost": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "fairli": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "network-": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "file-access": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "incorpor": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "re-written": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "arthur": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "hoff": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "compli": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "strictli": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "advent": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "j2se": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "j2ee": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "j2me": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "j2": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "ee": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "iso/iec": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "jtc": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "withdrew": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "charg": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "revenu": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "sell": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "finish": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "software/open-sourc": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "asid": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "copyright": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "vice-presid": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "evangelist": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "acquisit": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "2009–10": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "steward": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "relentless": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "foster": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "transpar": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "lawsuit": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "sdk": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "jdk": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "supercomput": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "uninstal": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "unresolv": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "five": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "robust": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "architecture-neutr": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "zero-cost": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "adoptium": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "card": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "micro": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "jakarta": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "bundl": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "personaljava": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "segment": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "smart-card": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "workstat": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "subpackag": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "supersed": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "standards-bas": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "configuration-profil": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "adequ": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "architecture-specif": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "instruct": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "end-us": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "jre": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "host-specif": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "slowli": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "byte-cod": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "stage": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "graalvm": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "platform-independ": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "adapt": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "reput": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "slower": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "1997/1998": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "stringbuild": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "java.util.concurr": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "lock-fre": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "concurrentmap": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "multi-cor": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "arm-bas": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "jazel": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "arm": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "lifecycl": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "recov": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "unreach": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "elig": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "non-exist": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "pointer": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "thrown": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "spare": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "burden": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "dealloc": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "heap": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "resid": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "predict": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "unstabl": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "crash": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "remedi": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "smart": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "referenc": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "guarante": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "trigger": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "insuffici": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "stall": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "momentarili": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "c/c++": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "address": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "offset": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "reloc": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "safeti": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "primit": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "non-primit": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "consciou": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "g1gc": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "zgc": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "shenandoah": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "gc": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "unavail": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "oracle-produc": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "eclips": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "temurin": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "prior": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "parallel": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "reliev": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "presenc": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "gephi": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "reus": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "javadoc": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "void": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "println": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "plea": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "reliabl": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "challeng": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "removed.find": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "newspap": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "scholar": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "jstor": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "mainli": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "busi": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "extent": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "hood": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "jax-r": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "jax-w": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "soap": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "jsp": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "pluggabl": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "clone": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "gtk+": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "motif": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "aqua": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "across": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "5.0": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "easi": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "subtyp": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "compile-tim": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "classcastexcept": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "proven": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "unsound": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "unrel": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "unsign": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "verbos": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "persist": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "jpa": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "higher-level": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "abstract": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "spring": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "suggest": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "ease-of-us": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "compani": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)",
        "https://en.wikipedia.org/wiki/Kotlin"
    ],
    "subject": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "nio": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "io/nio": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "agent": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "double-end": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "par": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "internation": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "writer": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "jdbc": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "jndi": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "lookup": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "rmi": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "broker": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "corba": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "jmx": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "heavyweight": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "awt": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "lay": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "lightweight": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "non-n": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "widgetri": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "extra": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "asterisk": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "single-lin": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "ansi": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "kit": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "jar": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "brand": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Kotlin"
    ],
    "insist": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "disput": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "jni": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "platform-specif": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "su": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "settlement": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "court": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "rigor": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "certifi": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "primarili": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "pillar": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "incompat": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "low-memori": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "smartphon": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "tablet": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "dalvik": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "independ": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "harmoni": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "america": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "java-rel": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "juri": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "infring": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "district": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "judg": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "alsup": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "appeal": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "feder": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "circuit": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "favor": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "constitut": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "fair": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "overturn": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "damag": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "petit": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "writ": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "certiorari": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "suprem": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "6–2": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "refus": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "choos": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "pure": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "sake": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "non-jit": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "binstock": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "forb": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "barbara": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "liskov": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "usa": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "addison": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "wesley": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "978-0-201-65768-5": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "chaudhari": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "h.": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "crack": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "2000+": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "que/an": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "autobox": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "vararg": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "compet": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "mcgilton": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "henri": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "joy": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "bracha": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "gilad": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "a-z": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "computerworld": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "niklau": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "wirth": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "occas": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "lectur": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "polytechn": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "museum": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "moscow": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "first-hand": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "russian": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/Kotlin"
    ],
    "filippova": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "elena": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "examin": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "overal": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "wrapper": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "techmetrix": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "acm": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "queue.acm.org": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "javasoft": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "precursor": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "javabean": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "borland": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "care": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "delphi": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "spec": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "cray": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "q": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "weekli": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "melanson": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "comeback": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "champ": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "octovers": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "mcmillan": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "robert": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "lose": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "byou": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "jon": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "murphi": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "kieron": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "javaworld": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "kabutz": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "heinz": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "tata": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "mcgraw-hil": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "jsg": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "open-std.org": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "ecma—and": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "zdnet": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "jcp.org": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "javaon": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "bulk": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "grnlight.net": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "substant": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "road": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "topic": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "dalibor": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "plugin-fre": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "roadmap": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "temurin™": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "adoptium.net": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "sun.com": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "chander": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "sharat": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "openjdk.org": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "deep": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "dive": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "graal": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "baeldung": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "www.baeldung.com": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "advantag": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "jelov": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "dejan": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "hundt": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "c++/java/go/scala": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "salcic": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "zoran": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "park": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "heejong": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "teich": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "jürgen": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "malik": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "avinash": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "nadeem": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "muhammad": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "noc-hmp": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "multicor": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "systemj": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "transact": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "73.": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "doi:10.1145/3073416": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "1084-4309": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "11150290": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "11150290.": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "10.1145/3073416": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "bloch": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "26-28": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "§item": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "nullpointerexcept": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "artima.com": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "hotspot™": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "docs.oracle.com": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "trail": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "jfc/swing": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "arnold": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "ken": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "harm": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "java.net": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "sean": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "thereof": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "kahan": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "hurt": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "everyon": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "everywher": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "workshop": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "stanford": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "electr": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "chidest": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "ashlan": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "jenkin": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "aw": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "9798224253951": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "cadenhead": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "roger": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "woolf": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "nicki": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "win": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "six-year": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "battl": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "guardian": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "0261-3077": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "1–4": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "java.nio": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "httpclient": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "niccolai": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "settl": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "idg": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "gurp": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "jill": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "impress": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "javalobbi": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "frankli": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "ignor": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "vast": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "amount": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "seem": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "mullin": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "joe": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "guilti": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "trial": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "headach": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "loom": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "law": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "disord": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "crucial": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "rosenblatt": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "seth": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "beat": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "oracle—android": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "farivar": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "cyru": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "lee": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "timothi": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "overrul": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "disastr": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "llc": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "593": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "s.": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "____": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "joshua": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "978-0-13-468599-1": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "buckley": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "java®": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "l.": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "jr.": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "0-321-24678-0": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "lindholm": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "yellin": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "frank": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "0-201-43294-3": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "edition/java": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "squawk": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "maxin": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "midlet": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "jsf": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "jnlp": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "pack200": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "classpath": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "hibern": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "gwt": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "icedtea": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "strut": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "toplink": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "wildfli": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "blackdown": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "oxygen": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "devoxx": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "laboratori": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "ur": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "hölzle": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "sun-1": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "sun-2": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "sun-3": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "sun386i": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "sun-4": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "sparcstation/server/cent": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "ipc": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "ipx": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "lx": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "zx": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "sparcclass": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "netra": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "ultra": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "10000": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "blade": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "fire": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "12k": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "15k": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "e25k": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "t2000": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "x4500": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "sparc": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "javast": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "ray": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "cobalt": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "qube": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "raq": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "mb86900": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "microsparc": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "supersparc": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "ultrasparc": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "ii": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "iie": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "iii": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "gemini": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "cu": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "iiii": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "t1": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "t2": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "t3": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "t4": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "t5": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "majc": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "sun4d": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "sbu": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "fireplan": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "lom": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "mbu": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "datacent": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "neptun": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "spot": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "suno": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "ni": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "nis+": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "nf": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "zf": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "zfs+": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "sunview": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "openwindow": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "staroffic": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "iplanet": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "mysql": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "xvm": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "glassfish": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "virtualbox": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "storagetek": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "5800": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "sl8500": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "qf": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "constel": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "grid": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "lustr": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "picojava": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "glass": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "scp": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "bluej": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "netbean": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "openoffice.org": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "opensolari": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "opensparc": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "meetup": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "andi": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "bechtolsheim": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "mcneali": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "vinod": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "khosla": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "afara": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "websystem": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "gridwar": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "lighthous": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "montalvo": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "pixo": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "procom": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "savaj": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "tarantella": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "jeffrey": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "berg": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "bingham": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "boskin": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "safra": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "catz": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "ellison": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "héctor": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "garcía-molina": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "joseph": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "grundfest": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "o.": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "henley": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "hurd": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "jack": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "f.": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "kemp": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "donald": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "luca": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "naomi": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "seligman": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "peoplesoft": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "hyperion": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "siebel": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "bea": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "jd": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "edward": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "rightnow": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "iron": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "timesten": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "sunopsi": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "netsuit": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "cerner": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "innodb": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "db": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "rdb": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "essbas": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "pl/sql": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "jdevelop": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "apex": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "middlewar": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "webcent": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "soa": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "weblog": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "coher": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "tuxedo": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "t-seri": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "exadata": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "exalog": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "applianc": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "certif": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "nosql": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "dtrace": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "visualvm": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "ocfs2": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "x86": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "title=java_": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "oldid=1266787868": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "mdi": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)",
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "06:14": [
        "https://en.wikipedia.org/wiki/Java_(programming_language)"
    ],
    "2.1": [
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "2.1.1": [
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "2.1.2": [
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "2.1.3": [
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "re-writ": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "2.2": [
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "2.4": [
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "c17": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "c23": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "2.8": [
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "c2i": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "2.9": [
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "word": [
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "array–point": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "interchang": [
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "9.1": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "computationally-intens": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "9.3": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "9.4": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "9.5": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "9.6": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "9.7": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "isixhosa": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "मैथिली": [
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "malagasi": [
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "ꯃꯤꯇꯩ": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "ꯂꯣꯟ": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "piemontèi": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "ייִדיש": [
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "ⵜⴰⵎⴰⵣⵉⵖⵜ": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "ⵜⴰⵏⴰⵡⴰⵢⵜ": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "joint": [
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "technic": [
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "committe": [
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "subcommitte": [
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "1972": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "2024-10-31": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "2024-02-21": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "www.iso.org/standard/82075.html": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "www.open-std.org/jtc1/sc22/wg14/": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "pcc": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "gcc": [
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "clang": [
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "c++builder": [
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "watcom": [
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "cyclon": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "split-c": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "cilk": [
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "bcpl": [
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "cpl": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "pl/i": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "ampl": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "csh": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "limbo": [
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "lpc": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "pike": [
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "vlang": [
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "zig": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "pronounc": [
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "/ˈsiː/": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "1970": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "influenti": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "cleanli": [
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "cpu": [
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "largest": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "smallest": [
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "ˈsiː": [
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "iː": [
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "bell": [
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "1973": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "re-impl": [
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "co-author": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "nation": [
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "electrotechn": [
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "commiss": [
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "iec": [
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "encourag": [
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "standards-compli": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "thompson": [
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "subroutin": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "pass-by-refer": [
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "free-form": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "exhibit": [
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "characterist": [
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "if/els": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "do/whil": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "sigil": [
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "bitwis": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "hoc": [
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "polymorph": [
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "typedef": [
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "compound": [
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "aggreg": [
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "overlap": [
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "locat": [
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "secondari": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "squar": [
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "enum": [
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "freeli": [
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "interconvert": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "null-termin": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "preprocessor": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "macro": [
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "visibl": [
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "straightforward": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "glib": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "boehm": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "c-famili": [
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "indirectli": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "systemverilog": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "drawn": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "tend": [
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "recogniz": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "radic": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "tie": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "pdp-7": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "colleagu": [
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "pdp-11": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "gave": [
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "cut-down": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "'wordi": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "smalgol": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "1971": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "more-pow": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "nb": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "shape": [
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "richer": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "char": [
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "urg": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "snyder": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "file-inclus": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "parameterless": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "lesk": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "reiser": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "multic": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "mcp": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "burrough": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "b5000": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "1961": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "1977": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "stephen": [
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "johnson": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "1978": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "c78": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "=op": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "=-": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "op=": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "-=": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "ambigu": [
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "i=-10": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "decrement": [
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "-10": [
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "−10": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "denomin": [
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "maximum": [
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "presum": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "some_funct": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "other_funct": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "'int": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "calling_funct": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "test1": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "test2": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "'regist": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "lint": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "i.e.": [
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "previous": [
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "necess": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "mainfram": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "minicomput": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "microcomput": [
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "1983": [
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "x3j11": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "non-port": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "hand": [
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "1003": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "posix": [
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "ratifi": [
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "x3.159-1989": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "1990": [
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "9899:1990": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "c90": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "therefor": [
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "defer": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "jtc1/sc22/wg14": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "jtc1/sc22": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "hardware-depend": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "assumpt": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "precaut": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "relianc": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "compiler-": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "exact": [
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "endian": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "standard-conform": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "c-base": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "__stdc__": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "split": [
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "ansi/iso": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "amend": [
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "9899/amd1:1995": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "c95": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "9899:1999": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "corrigenda": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "variable-length": [
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "flexibl": [
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "ariti": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "one-lin": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "backward": [
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "stricter": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "__stdc_version__": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "199901l": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "\\u0040": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "\\u0001f431": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "c1x": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "9899:2011": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "bounds-check": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "201112l": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "9899:2018": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "clarif": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "defect": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "201710l": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "c2x": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "9899:2024": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "202311l": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "decad": [
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "n3220": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "exot": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "fixed-point": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "adher": [
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "boundari": [
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "preprocess": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "turn": [
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "perhap": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "control-flow": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "reiniti": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "innermost": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "reinitialis": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "non-structur": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "goto": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "branch": [
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "label": [
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "unspecifi": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "interleav": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "entri": [
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "degre": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "say": [
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "blemish": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "impact": [
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "lowercas": [
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "uppercas": [
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "latin": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "a–z": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "0–9": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "horizont": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "tab": [
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "vertic": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "feed": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "treat": [
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "multi-byt": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "encod": [
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "multi-n": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "\\uxxxx": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "\\uxxxxxxxx": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "hexadecim": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "backspac": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "carriag": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "'keyword": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "predefin": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "auto": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "sizeof": [
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "volatil": [
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "‡": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "spell": [
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "alia": [
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "_bool": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "_complex": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "_imaginari": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "seven": [
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "_aligna": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "_alignof": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "_atom": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "_gener": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "_noreturn": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "_static_assert": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "_thread_loc": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "fifteen": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "aligna": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "alignof": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "bool": [
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "constexpr": [
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "nullptr": [
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "static_assert": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "thread_loc": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "typeof": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "typeof_unqu": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "_bitint": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "_decimal32": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "_decimal64": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "_decimal128": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "capit": [
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "synonym": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "stillborn": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "withdrawn": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "/=": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "|=": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "^=": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "shift": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "derefer": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "subexpress": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "typenam": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "accident": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "mistakenli": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "intuit": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "tightli": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "intent": [
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "introductori": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "screen": [
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "\\n": [
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "stdio.h": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "scanf": [
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "angl": [
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "project-specif": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "divert": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "world\\n": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "unnam": [
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "\\0": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "discard": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "succeed": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "descend": [
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "single-byt": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "reinterpret": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "bit": [
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "unintuit": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "resembl": [
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "unexpect": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "width": [
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "dereferenc": [
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "pointed-to": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "offset-within-word": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "multi-dimension": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "higher-ord": [
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "qsort": [
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "bsearch": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "dispatch": [
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "careless": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "danger": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "uncheck": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "undesir": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "unsaf": [
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "dangl": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "corrupt": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "repercuss": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "illeg": [
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "accomplish": [
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "thought": [
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "row-major": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "matric": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "workaround": [
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "row": [
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "column": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "two-dimension": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "func": [
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "-1": [
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "print_array": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "vla": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "caution": [
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "exceed": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "held": [
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "disappear": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "subscript": [
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "x+i": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "multipli": [
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "i+1th": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "furthermor": [
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "pass-by-valu": [
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "lost": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "princip": [
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "lifetim": [
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "temporari": [
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "reusabl": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "region": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "realloc": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "situat": [
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "trade-off": [
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "littl": [
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "involv": [
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "transient": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "simplest": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "error-pron": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "chore": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "variable-s": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "uncontrol": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "consequ": [
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "detect": [
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "linker": [
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "loader": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "zero": [
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "startup": [
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "indetermin": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "uniniti": [
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "goe": [
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "overwritten": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "phenomenon": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "unpredict": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "failur": [
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "symptom": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "diagnos": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "amelior": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "flag": [
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "-lm": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "perspect": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "concret": [
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "temporarili": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "destin": [
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "solid-st": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "bare": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "metal": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "erron": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "audit": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "benefici": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "misra": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "purifi": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "valgrind": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "uncov": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "pun": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "system-specif": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "control/statu": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "fullest": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "boot": [
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "low": [
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "esoter": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "intrins": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "overlay": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "onto": [
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "comprehend": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "appropriately-structur": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "memory-handl": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "sporad": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "stop-the-world": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "dma": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "interrupt": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "interoper": [
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "lower-level": [
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "broad": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "back-end": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "incent": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "thin": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "criterion": [
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "intens": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "hardware-interact": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "machine-specif": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "line-numb": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "superflu": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "shortcom": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "contemporari": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "cgi": [
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "near-univers": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "router": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "iot": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "openwrt": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "nginx": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "tcp": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "'render": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "close-to-the-met": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "huge": [
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "drawback": [
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "prone": [
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "improp": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "trivial": [
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "inadvert": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "circumv": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "outcom": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "exhaust": [
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "race": [
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "alias": [
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "optimis": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "strncat": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "standardis": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "abi": [
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "pragma": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "__cdecl": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "code-intens": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "introspect": [
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "guard": [
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "inappropri": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "unmaintain": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "troubl": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "wors": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "tricki": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "celebr": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "contest": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "underhand": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "setjmp": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "longjmp": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "try-catch": [
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "opportun": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "cwe": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "mitig": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "pervas": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "near-c": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "ch": [
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "cint": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "source-to-sourc": [
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "bjarn": [
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "stroustrup": [
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "hybrid": [
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "dynamic/stat": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "taken": [
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "diagnost": [
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "argc": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "argv": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "command-lin": [
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "5.1.2.2.1": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "treatment": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "afford": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "shown": [
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "prinz": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "crawford": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "toni": [
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "nutshel": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "9780596550714": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "c—befor": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "structures—in": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "n3221": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "strasbourg": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "owe": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "debt": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "approv": [
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "school": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "australian": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "sound": [
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "chine": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "munoz": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "toptal": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "developer.com": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "ward": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "terri": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "268": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "lysator.liu.s": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "jensen": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "richard": [
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "damn": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "stupid": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "—the": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "d.": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "tech": [
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "2021–2048": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "10.1.1.138.35.": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "doi:10.1002/j.1538-7305.1978.tb02141.x": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "0005-8580": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "17510065": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "ocr": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "370": [
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "310": [
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "2021–": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "10.1.1.138.35": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "10.1002/j.1538-7305.1978.tb02141.x": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "mcilroy": [
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "1987": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "1971–1986": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "cstr": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "englewood": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "cliff": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "978-0-13-110163-0": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "13.0": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "978-0-13-110362-7": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "sibl": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "rivalri": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "herb": [
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "sutter": [
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "dr.": [
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "dobb": [
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "wg14-n3132": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "wg14-n3220": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "tr": [
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "18037": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "jtc1": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "sc22": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "wg14": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "n1169": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "harbison": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "samuel": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "978-0-13-089592-9": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "9899:201x": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "259": [
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "cs.ucr.edu": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "schultz": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "8051": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "otsego": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "mi": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "pagefre": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "978-1-58961-237-2": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "g": [
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "klemen": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "ben": [
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "21st": [
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "centuri": [
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "978-1-4493-2714-9": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "feuer": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "gehani": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "narain": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "1982": [
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "73–92": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "doi:10.1145/356869.356872": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "3136859": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "73–": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "10.1145/356869.356872": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "_fortify_sourc": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "fortify_sourc": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "fedoraproject.org": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "เอี่ยมสิริวงศ์": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "โอภาศ": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "bangkok": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "thailand": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "se-educ": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "225–230": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "978-616-08-2740-4": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "225–": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "hacker": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "432": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "978-0-262-68092-9": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "man": [
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "unix.com": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "cs107": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "memcheck": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "web.stanford.edu": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "hast": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "reed": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "joyc": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "bob": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "dale": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "nell": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "b.": [
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "weem": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "burlington": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "massachusett": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "jone": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "bartlett": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "978-1449694289": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "894992484": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "mother": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "ict": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "academi": [
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "iitk": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "3.10.7": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "conrad": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "opensource.com": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "para": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "sourcebook": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "u.s.": [
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "miller": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "freeman": [
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "november–decemb": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "linuxjournal.com": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "perkin": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "crazi": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "fox": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "metz": [
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "cade": [
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "job": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "corob-msft": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "__pragma": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "_pragma": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "prais": [
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "apenwarr": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "src-rr-40": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "o'regan": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "gerard": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "compendium": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "pivot": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "firm": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "978-3319214641": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "922324121": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "rauchwerg": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "lawrenc": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "16th": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "lcpc": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "colleg": [
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "station": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "tx": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "2–4": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "978-3540246442": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "57965544": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "1979–1991": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "sigplan": [
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "201–208": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "doi:10.1145/155360.155580": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "courtesi": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "chistori": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "www.bell-labs.com": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "201–": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "10.1145/155360.155580": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "hopl-ii": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "doi:10.1145/154766.155580": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "0-89791-570-4": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "10.1145/154766.155580": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "0-13-110362-8": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "plauger": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "p.j": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "1992": [
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "978-0131315099": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "banahan": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "bradi": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "doran": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "978-0201544336": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "1985": [
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "0131099345": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "pearson": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "978-0130895929": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "king": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "k.n": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "norton": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "978-0393979503": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "griffith": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "dawn": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "head": [
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)",
        "https://en.wikipedia.org/wiki/Kotlin"
    ],
    "978-1449399917": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "perri": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "greg": [
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "absolut": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "beginn": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "que": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "978-0789751980": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "deitel": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "harvey": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "978-0133976892": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "gustedt": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "978-1617295812": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "wikinew": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "9899": [
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "publicli": [
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "tc1": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "tc2": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "tc3": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "3.61": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "mb": [
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "comp.lang.c": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "bionic": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "libhybri": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "dietlibc": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "glibc": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "eglibc": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "klibc": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "crt": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "musl": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "newlib": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "uclibc": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "ack": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "turbo": [
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "icc": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "lcc": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "norcroft": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "sdcc": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "tcc": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "anjuta": [
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "clion": [
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    ":block": [
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "codelit": [
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "geani": [
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "builder": [
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "kdevelop": [
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "alef": [
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "dev-c++": [
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "emac": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "kakoun": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "qdevelop": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "theid": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "vi–vim": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "vi": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "openwatcom": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "deveco": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "xcode": [
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "retail": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "eclipse-bas": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "codewarrior": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "myeclips": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "appcod": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "labwindows/cvi": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "architect": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "slickedit": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "visualag": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "monodevelop": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "sharpdevelop": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "quickc": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "drjava": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "greenfoot": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "jgrasp": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "jcreator": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "metrowerk": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "jbuilder": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "café": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "aka": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "espresso": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "j++": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "xelfi": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "cli": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "pascalabc.net": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "rider": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "xamarin": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "flashdevelop": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "powerflash": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "fdt": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "aptana": [
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "codelobst": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "phpedit": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "zend": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "pyscript": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "pida": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "thonni": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "wing": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "lazaru": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "dev-pasc": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "gnavi": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "kylix": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "quickpasc": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "basic-256": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "sdlbasic": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "basic4gl": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "visualfbeditor": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "winfb": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "futurebas": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "rapidq": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "purebas": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "glbasic": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "liberti": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "xojo": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "ca-real": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "quickbas": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "qbasic": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "vscode": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "goland": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "sublim": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "pop-11": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "poplog": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "che": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "sourcelair": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "title=c_": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "oldid=1268541771": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "pend": [
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "vagu": [
        "https://en.wikipedia.org/wiki/C_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "08:10": [
        "https://en.wikipedia.org/wiki/C_(programming_language)"
    ],
    "1.8": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "1.9": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "matz": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "non-expert": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "ruby-lang.org": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "mri": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "trufflerubi": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "yarv": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "rubiniu": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "rubymot": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "mrubi": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "crystal": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "mirah": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "nu": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "simplic": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "mid-1990": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "japan": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "ruby-talk": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "knew": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "perl4": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "perl5": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "realli": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "smell": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "toy": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "oo": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "maniac": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "genuin": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "easy-to-us": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "chat": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "keiju": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "ishitsuka": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "coral": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "chose": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "e-mail": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "factor": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "–it": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "birthston": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "0.95": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "japanes": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "domest": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "newsgroup": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "coincid": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "japanese-languag": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "ruby-list": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "netlab.jp": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "full-tim": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "english-languag": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "signal": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "interest": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "オブジェクト指向スクリプト言語": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "widen": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "amongst": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "speaker": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "non-japanes": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "information-technolog": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "agenc": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "submiss": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "jisc": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "ji": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "3017": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "30170": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "surg": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "tandem": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "credit": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "awar": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "1.9.3": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "dual-licens": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "two-claus": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "gem": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "rewritten": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "hash": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "colon": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "symbol_key": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "per-str": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "socket": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "ipv6": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "require_rel": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "2.0.0": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "prepend": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "to_h": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "2.1.0": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "2.2.0": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "bugfix": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "jemalloc": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "vfork": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "spawn": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "7.0": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "2.2.1": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "powerpc64": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "callcc": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "dl": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "digest": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    ":hmac": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "lib/rational.rb": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "lib/complex.rb": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "gserver": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "logger": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    ":applic": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "2.3.0": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "proc": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "passiv": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    ":ftp": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "rake": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "stdlib": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "frozen": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "key/valu": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "nil": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "obj.foo": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "obj.foo.bar": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    ".foo": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    ".bar": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "did_you_mean": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "nameerror": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "nomethoderror": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "dig": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "deepli": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "profile.dig": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    ".grep_v": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "regexp": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "2.4.0": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "min": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "irb": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "binding.pri": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "fixnum": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "bignum": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "deadlock": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "backtrac": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "2.5.0": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "rescu": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "do-end": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "begin-end": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "method-chain": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "yield_self": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "coverag": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "easier": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "transform_key": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "mutex": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "erb": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "2.6.0": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "rubyvm": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    ":abstractsyntaxtre": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "2.7.0": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "3.0.0": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "christma": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "3x3": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "mjit": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "yjit": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "fibr": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "ractor": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "light-weight": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "thread-saf": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "rb": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "shopifi": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "ftp": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "smtp": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "imap": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "pop": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "memoiz": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "redo": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "timeout": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "rust-bas": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "parser": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "pure-rubi": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "rjit": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "boost": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "developer-friendli": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "happi": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "stress": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "human": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "focus": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "slave": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "astonish": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "pola": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "comp.lang.rubi": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "distanc": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "explain": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "surpris": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "someon": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "singleton": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "functions/vari": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "interpreter-bas": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "thoroughli": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "nois": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "coercion": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "method_miss": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "const_miss": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "fiber": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "1.9/yarv": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "plug-in": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "central": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "rubygem": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "yaml": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "openssl": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "rss": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "cur": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "tk": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "broadli": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "obligatorili": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "equival": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "expo": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "accessor": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "attr_writ": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "attr_read": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "getter": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "setter": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "fashion": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "refactor": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "vb.net": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "descriptor": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "adjust": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "ruby-specif": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "single-pass": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "2.x": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "mix": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "3.1.x": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "truffl": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "2.3.1": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "maglev": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "gemtalk": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "gemstone/": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "vein": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "rgss": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "rpg": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "maker": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "julial": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "topaz": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "opal": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "web-bas": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "defunct": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "macrubi": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "ironrubi": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "cardin": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "parrot": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "ree": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "hotrubi": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "singular": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "unmodifi": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "4.3bsd-base": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "soni": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "news-o": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "4.x": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "server-bas": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "jelast": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "heroku": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "4.3bsd": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "rvm": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "rbenv": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "partit": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "'gemset": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "100,000": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "rubygems.org": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "metasploit": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "poignant": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "3.3.0": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "2023-12-25": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "novic": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "978-1-4302-2363-4": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "lesser": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "confreak": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "burk": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "nu™": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "neon": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "2018-12-25": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "2011-07-21": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "2019-01-26": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "2023-04-18": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "2014-06-03": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "2018-10-27": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "2011-07-14": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "2008-08-14": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "fyi": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "moment": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "ann": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "1.8.1": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "hiroshi": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "sugihara": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "2011-07-17": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "'rubi": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "ruby-doc.org": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "archeolinguist": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "unearth": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "proto-rubi": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "ruby-talk:00382": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "ruby-list:124": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "2008-02-08": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "2007-07-11": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "1.8.7": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "ipa": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "独立行政法人": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "情報処理推進機構：プレス発表": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "プログラム言語rubyのjis規格（ji": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "3017）制定について": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "プログラム言語ruby、国際規格として承認": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "30170:2012": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "2017-03-12": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "2017-03-10": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "2009-02-24": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "devarticles.com": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "2007-03-22": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "2013-07-17": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "p0": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "v1_9_3_0/new": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "subvers": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "2016-03-04": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "slideshow.rubyforge.org": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "endoh": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "yusuk": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "2013-02-24": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "2.0.0-p0": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "2013-02-27": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "2016-01-17": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "gustavo": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "frederico": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "templ": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "pedrosa": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "vitor": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "lima": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "leonardo": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "bianconi": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "maint": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "v2.2.1": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "non": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "ppc64": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "ruby/new": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "v2_2_0": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "ruby/rubi": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "v.2_3_0": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "dev.mensfeld.pl": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "www.ruby-lang.org": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "2017-02-17": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "2016-12-30": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "2020-12-25": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "scheffler": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "jonan": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "koichi": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "tenderlov": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "3.1.0": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "2021-12-26": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "3.2.0": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "2022-12-25": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "welcom": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "rubyweeklynews.org": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "bot": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "unknown": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "bruce": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "stewart": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "workbench": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "killer-app": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "martinfowler.com": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "2007-09-22": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "2007-11-01": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "2022-06-28": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "2021-06-20": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "britt": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "2013-11-13": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "2013-12-09": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "ruby-talk:01120": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "2008-12-06": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "ruby-talk:02460": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "2004-07-22": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "2008-12-06.": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "remove/virtual_modul": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "born": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "3x": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "2010-05-18": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "2010-05-24": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "2010-05-23": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "stodt": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "developerwork": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "lang-ruby-gener": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "2015-04-03": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "2015-04-04": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "stat": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "raa.ruby-lang.org": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "2013-08-08": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "2015-12-31": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "2016-01-03": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "black": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "leo": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "well-ground": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "rubyist": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "584": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "978-1617295218": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "sandi": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "agil": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "primer": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "288": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "978-0-13-445647-8": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "492": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "978-1484212790": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "carlson": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "richardson": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "leonard": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "recip": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "963": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "978-1449373719": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "fulton": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "hal": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "arko": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "andré": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "techniqu": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "816": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "978-0-321-71463-3": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "fowler": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "chad": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "hunt": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "pragmat": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "fourth": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "bookshelf": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "888": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "978-1937785499": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "mcanal": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "arkin": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "assaf": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "978-1933988474": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "446": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "978-0-596-51617-8": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "baird": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "326": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "978-1593271480": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "fitzgerald": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "978-0-596-52986-4": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "xrubi": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "rubymin": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "capistrano": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "hacketi": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "pri": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "redmin": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "rspec": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "chef": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "puppet": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "vagrant": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "homebrew": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "cocoapod": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "adhears": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "erubi": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "rhtml": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "merb": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "padrino": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "rubycocoa": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "shoe": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "sinatra": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "qtrubi": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "mod_rubi": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "mongrel": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "phusion": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "passeng": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "mod_rails/mod_rack": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "rack": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "webrick": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "heinemei": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "hansson": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "lucki": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "stiff": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "rubykaigi": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "rubyforg": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "roman": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "-0": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "-4": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "-5": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "-6": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "-7": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "-8": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "-9": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "-11": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "-12": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "-13": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "68-1": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "261": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "262": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "302": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "306": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "361": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "500": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "518": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "519": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "639": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "646": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "657": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "668": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "690": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "704": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "732": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "764": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "838": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "843": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "860": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "898": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "965": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "999": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "1000": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "1004": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "1007": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "1073-1": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "1073-2": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "1155": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "1413": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "1538": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "1629": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "1745": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "2033": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "2047": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "2108": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "2145": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "2146": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "2240": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "2281": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "2533": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "2709": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "2711": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "2720": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "2788": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "2848": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "2852": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "2921": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "3029": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "3103": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "3166": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "3297": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "3307": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "3601": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "3602": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "3864": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "3901": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "3950": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "3977": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "4031": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "4157": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "4165": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "4217": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "4909": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "5218": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "5426": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "5427": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "5428": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "5725": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "5775": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "5776": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "5807": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "5964": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "6166": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "6344": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "6346": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "6373": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "6385": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "6425": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "6429": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "6438": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "6523": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "6709": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "6943": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "7001": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "7002": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "7010": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "7027": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "7064": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "7098": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "7185": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "7200": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "7498": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "7637": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "7736": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "7810": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "7811": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "7812": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "7813": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "7816": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "7942": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "8000": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "8093": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "8178": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "8217": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "8373": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "8501-1": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "8571": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "8583": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "8601": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "8613": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "8632": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "8651": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "8652": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "8691": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "8805/8806": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "8807": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "8820-5": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "8859": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "-8-i": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "-14": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "-15": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "-16": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "8879": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "9000/9001": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "9036": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "9075": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "9126": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "9141": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "9227": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "9241": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "9293": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "9314": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "9362": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "9407": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "9496": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "9506": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "9529": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "9564": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "9592/9593": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "9594": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "9660": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "9797-1": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "9897": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "9945": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "9984": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "9985": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "9995": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "10006": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "10007": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "10116": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "10118-3": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "10160": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "10161": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "10165": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "10179": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "10206": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "10218": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "10279": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "10303": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "-21": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "-22": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "-28": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "-238": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "10383": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "10585": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "10589": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "10628": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "10646": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "10664": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "10746": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "10861": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "10957": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "10962": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "10967": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "11073": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "11170": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "11172": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "11179": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "11404": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "11544": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "11783": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "11784": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "11785": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "11801": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "11889": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "11898": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "11940": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "11941": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "11992": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "12006": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "12052": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "12182": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "12207": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "12234-2": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "12620": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "13211": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "13216": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "13250": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "13399": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "13406-2": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "13450": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "13485": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "13490": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "13567": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "13568": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "13584": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "13616": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "13816": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "13818": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "14000": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "14031": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "14224": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "14289": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "14396": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "14443": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "14496": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "-17": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "-20": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "14617": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "14644": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "14649": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "14651": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "14698": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "14764": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "14882": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "14971": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "15022": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "15189": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "15288": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "15291": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "15398": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "15408": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "15444": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "15445": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "15438": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "15504": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "15511": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "15686": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "15693": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "15706": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "15707": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "15897": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "15919": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "15924": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "15926": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "wip": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "15930": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "15938": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "16023": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "16262": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "16355-1": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "16485": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "16612-2": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "16750": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "16949": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "t": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "17024": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "17025": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "17100": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "17203": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "17369": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "17442": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "17506": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "17799": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "18004": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "18014": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "18181": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "18245": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "18629": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "18916": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "19005": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "19011": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "19092": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "19114": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "19115": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "19125": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "19136": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "19407": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "19439": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "19500": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "19501": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "19502": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "19503": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "19505": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "19506": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "19507": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "19508": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "19509": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "19510": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "19600": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "19752": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "19757": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "19770": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "19775-1": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "19794-5": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "19831": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "20000": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "20022": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "20121": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "20400": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "20802": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "20830": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "21000": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "21001": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "21047": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "21122": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "21500": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "21827": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "22000": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "22275": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "22300": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "22301": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "22395": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "22537": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "23000": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "23003": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "23008": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "23009": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "23090-3": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "23092": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "23094-1": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "23094-2": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "23270": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "23271": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "23360": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "23941": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "24517": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "24613": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "24617": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "24707": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "24728": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "25178": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "25964": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "26000": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "26262": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "26300": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "26324": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "27000": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "27001": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "27002": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "27005": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "27006": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "27729": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "28000": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "29110": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "29148": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "29199-2": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "29500": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "31000": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "32000": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "37001": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "38500": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "39075": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "40500": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "42010": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "45001": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "50001": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "55000": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "56000": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "80000": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)",
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "title=ruby_": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "oldid=1268195105": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "15:36": [
        "https://en.wikipedia.org/wiki/Ruby_(programming_language)"
    ],
    "1.2.1": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "1.2.2": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "1.2.3": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "1.2.4": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "1.3.1": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "xhtml": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "transit": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "whatwg": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "markup": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "2.1.1.1": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "2.1.1.2": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "2.1.1.3": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "2.1.1.4": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "deliveri": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "4.2": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "4.3": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "4.4": [
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "html4": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "variat": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "sgml-base": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "xml-base": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "frameset": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "summari": [
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "wysiwyg": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "башҡортса": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "boarisch": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "davvisámegiella": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "dolnoserbski": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "fiji": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "hindi": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "føroyskt": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "frysk": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "furlan": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "gàidhlig": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "گیلکی": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "hornjoserbsc": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "latgaļu": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "lëtzebuergesch": [
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "नेपाल": [
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "भाषा": [
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "нохчийн": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "олык": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "марий": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "русиньскый": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "wolof": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "yorùbá": [
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "htm": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    ".html": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    ".htm": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "uniform": [
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "uti": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "consortium": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "w3c": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "sgml": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "html.spec.whatwg.org": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "canva": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "meta": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "div": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "marque": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "alt": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "font": [
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "webcl": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "htmx": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "web3d": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "webxr": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "hypertext": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "assist": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "cue": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "paragraph": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "delin": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "img": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "/p": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "sub-el": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "layout": [
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "berners-le": [
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "physicist": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "contractor": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "enquir": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "memo": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "internet-bas": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "cailliau": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "fund": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "encyclopedia": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "compris": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "hyperlink": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "sgmlguid": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "in-hous": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "-base": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "eleven": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "compos": [
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "audibl": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "9537": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "runoff": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "1960": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "ctss": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "time-shar": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "typesett": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "progress": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "ietf": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "mid-1993": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "connolli": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "expir": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "six": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "ncsa": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "in-lin": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "raggett": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "html+": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "already-impl": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "fill-out": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "auspic": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "15445:2000": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "4.01": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "erratum": [
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "deliver": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "rfc": [
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "1866": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "1867": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "form-bas": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "1942": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "2070": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "mathml": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "forbidden": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "code-nam": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "cougar": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "browser-specif": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "sought": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "1/sc": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "bed": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "reformul": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "republish": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "abandon": [
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "xhtml5": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "1.x": [
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "character": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "xhtml-inspir": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "conflict": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "diverg": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "character-bas": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "h1": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "/h1": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "unpair": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "doctyp": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "/html": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "/bodi": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "/titl": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "/head": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "metadata": [
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "revert": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "parent": [
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "src=": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "example.com/example.jpg": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "src": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "br": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "akin": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "infer": [
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "understood": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "attribute1=": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "value1": [
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "attribute2=": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "value2": [
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "/tag": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "neither": [
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "attribute1": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "attribute2": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "stylesheet": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "href": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "stylebyjimbowales.css": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "h6": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "h2": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "h3": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "h4": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "h5": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "//www.wikipedia.org/": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "checkbox": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "em": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "//en.wikipedia.org/": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "//example.org": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "image.gif": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "height": [
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "border": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "name–valu": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "unquot": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "name-valu": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "ismap": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "document-wid": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "uniqu": [
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "sub-sect": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "//en.wikipedia.org/wiki/html": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "classifi": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "subordin": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "footnot": [
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "cumbersom": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "subtextu": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "explan": [
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "tooltip": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "oh": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "lang=": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "fr": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "c'est": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "vie": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "france.": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "abbr": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "anid": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "jargon": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "purpl": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "cursor": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "language-rel": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "dir": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "rtl": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "right-to-left": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "arab": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "persian": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "hebrew": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "252": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "1,114,050": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "amp": [
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "x26": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "double-quot": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "x22": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "single-quot": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "x27": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "apo": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "overlook": [
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "forgiv": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "guess": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "acute-acc": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "é": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "western": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "south": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "keyboard": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "eacut": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "xe9": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "ampersand": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "non-break": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "dagger": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "plethora": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "uri": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "mode—particularli": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "ocument": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "dtd": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "machine-read": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "prohibit": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "-//w3c//dtd": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "4.01//en": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "//www.w3.org/tr/html4/strict.dtd": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "exclud": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "incept": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "neutral": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "crawl": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "correl": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "commonplac": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "mashup": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "price": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "mash": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "crawler": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "search-engin": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "spider": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "uniformli": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "poorer": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "higher": [
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "ascertain": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "wast": [
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "impair": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "irrelev": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "email": [
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "transmit": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "text/html": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "application/xhtml+xml": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "well-form": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "lenient": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "appendix": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "ill-defin": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "plain": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "typograph": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "diagram": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "disguis": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "phish": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "do": [
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "impos": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "fat": [
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "hta": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    ".hta": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "creation/editing/remov": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "registri": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "exe": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "clear": [
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "devoid": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "push": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "driven": [
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "overcom": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "chaotic": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "meaning": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "well-pres": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "xsl": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "rein": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "axe": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "axi": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "loo": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "clearli": [
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "xhtml-specifi": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "circumst": [
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "conflat": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "sub-specif": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "document—in": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "dtds—are": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "br/": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "unfamiliar": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "subtl": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "language-defin": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "xmlns=uri": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "version=": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "encoding=": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "content-typ": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "empty-el": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "rememb": [
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "/div": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "application/xml": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "vocabulari": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "looser": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "blockquot": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "noscript": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "nofram": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "underlin": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "visitor": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "strike-through": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "basefont": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "bgcolor": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "align": [
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "noshad": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "hr": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "vspace": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "hspace": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "legend": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "caption": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "nowrap": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "td": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "th": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "ul": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "li": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "pre": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "isindex": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "redund": [
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "ifram": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "image-map": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "contrari": [
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "stand": [
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "likewis": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "timet": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "quicker": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "xforms—a": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "web-form": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "xform": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "xml-defin": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "standards-body-bas": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "requestto": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "qualiti": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "voic": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "advoc": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "wysiwym": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "perceiv": [
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "convey": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "extrem": [
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "ungrammat": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "soup": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "incorrect": [
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "ital": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "-natur": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "breadcrumb": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "cellpad": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "microdata": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "polyglot": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "discours": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "felienn": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "schlesing": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "ari": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "2024-10-17": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "femin": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "symposium": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "205–222": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "doi:10.1145/3689492.3689809": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "979-8-4007-1215-9": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "|journal=": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "205–": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "10.1145/3689492.3689809": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "info.cern.ch": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "x11": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "www": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "sgml/html": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "w3": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "textual": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "metainform": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "non-proprietari": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "figur": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "doi:10.17487/rfc1866": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "2854": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "upwardli": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "10.17487/rfc1866": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "wg": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "engelfriet": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "arnoud": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "wilbur": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "htmlhelp.com": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "iso-html": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "www.scss.tcd.ie": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "geneva": [
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "mileston": [
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "philipp": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "le": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "hegaret": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "gold": [
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "mccathi": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "nevil": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "hyper": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "internet-draft": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "iiir": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "1993-11-08": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "draft-raggett-www-html-00": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "datatrack": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "2019-11-18": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "1995-05-16": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "tools.ietf.org": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "n.": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "weblint": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "usenix": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "atc": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "håkon": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "wium": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "bo": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "bert": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "longman": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "263": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "978-0-201-41998-6": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "confirm": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "hickson": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "ian": [
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "grannel": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "craig": [
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "jul": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "2012-12-17": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "2013-06-15": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "2014-09-16": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "2014-09-27": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "squabbl": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "2014-10-29": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "module-bas": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "2017-01-04": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "xhtm": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "jaff": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "memorandum": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "cimpanu": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "catalin": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "shankland": [
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "epitaph": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "cb": [
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "hsivonen.iki.fi": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "2012-02-16": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "w3school": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "korpela": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "jukka": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "cs.tut.fi": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "h56": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "resolv": [
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "wcag": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "chart": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "2010-03-16": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "2013-08-19": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "2021-12-25": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "fischetti": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "weav": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "destini": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "inventor": [
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "harper": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "978-0-06-251587-2": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "touch": [
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "hendler": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "lassila": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "os": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "nigel": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "shadbolt": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "wendi": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "revisit": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "rfc2854": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "rfc3236": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "bradner": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "doi:10.17487/rfc2119": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "bcp": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "2119": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "8174": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "adject": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "weigh": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "10.17487/rfc2119": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "btb": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "sear": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "sauer": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "wysiwiki": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "wikimania": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "spiesser": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "kitchen": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "13th": [
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "355—364": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "'04": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "york": [
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "ny": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "17–20": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "2010-03-25": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "xhtml.com": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "doug": [
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "engelbart": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "invis": [
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "invisiblerevolution.net": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "meta-wiki": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "palmer": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "infomesh": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "2022-04-13": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "timefram": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "1980–1995": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "activitypub": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "aria": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "canon": [
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "cdf": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "flexbox": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "exi": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "emotionml": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "geoloc": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "json-ld": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "notif": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "micropub": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "owl": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "pl": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "rdf": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "schema": [
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "rdfa": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "sisr": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "sko": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "smil": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "srg": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "sri": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "ssml": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "svg": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "scxml": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "shacl": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "sparql": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "voicexml": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "wot": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "wsdl": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "webment": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "websub": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "webvtt": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "woff": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "+rdfa": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "xinclud": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "xlink": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "xop": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "xpath": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "xpointer": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "xproc": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "xqueri": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "xsl-fo": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "xslt": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "indieauth": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "xade": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "xbl": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "xhtml+smil": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "xup": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "ccxml": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "curi": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "eme": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "inkml": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "mse": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "rif": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "timesheet": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "sxbl": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "xfdl": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "xframe": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "xmlhttprequest": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "c-html": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "hdml": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "pgml": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "vml": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "webplatform": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "p3p": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "xhtml+mathml+svg": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "board": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "bg": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "cg": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "ddwg": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "multimod": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "mmi": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "httpd": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "libwww": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "1990–": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "1993–98": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "agora": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "1994–97": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "argo": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "browser/editor": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "1996–2012": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "iw3c": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "iw3c2": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "www1": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "ooxml": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "spreadsheetml": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "presentationml": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "wordprocessingml": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "odf": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "uof": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "rtf": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "tex": [
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "latex": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "markdown": [
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "amigaguid": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "asciidoc": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "bbcode": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "cml": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "crossmark": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "dita": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "docbook": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "ead": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "enrich": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "fhtml": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "gml": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "guideml": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "hytim": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "ipf": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "lilypond": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "linuxdoc": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "lout": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "mif": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "maml": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "mei": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "musicxml": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "omdoc": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "openmath": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "org-mod": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "pod": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "restructuredtext": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "rtml": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "rft": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "s1000d": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "setext": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "tei": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "texinfo": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "troff": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "wikitext": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "wml": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "waptv": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "xaml": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "60027": [
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "60034": [
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "60038": [
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "60062": [
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "60063": [
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "60068": [
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "60112": [
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "60228": [
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "60269": [
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "60297": [
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "60309": [
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "60320": [
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "60364": [
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "60446": [
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "60559": [
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "60601": [
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "60870": [
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "60870-5": [
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "60870-6": [
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "60906-1": [
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "60908": [
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "60929": [
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "60958": [
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "60980-344": [
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "61030": [
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "61131": [
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "61131-3": [
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "61131-9": [
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "61158": [
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "61162": [
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "61334": [
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "61355": [
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "61360": [
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "61400": [
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "61499": [
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "61508": [
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "61511": [
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "61784": [
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "61850": [
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "61851": [
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "61883": [
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "61960": [
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "61968": [
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "61970": [
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "62014-4": [
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "62026": [
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "62056": [
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "62061": [
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "62196": [
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "62262": [
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "62264": [
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "62304": [
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "62325": [
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "62351": [
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "62365": [
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "62366": [
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "62379": [
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "62386": [
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "62455": [
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "62680": [
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "62682": [
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "62700": [
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "63110": [
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "63119": [
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "63382": [
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "9592": [
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "9593": [
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "10021": [
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "13346": [
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "13522-5": [
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "19788": [
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "24727": [
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "24744": [
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "24752": [
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "27000-seri": [
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "27040": [
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "29119": [
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "33001": [
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "81346": [
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "title=html": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "oldid=1268733701": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "computer-rel": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "vandal": [
        "https://en.wikipedia.org/wiki/HTML",
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "weasel-word": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "07:39": [
        "https://en.wikipedia.org/wiki/HTML"
    ],
    "etymolog": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "durat": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "2.1.4": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "encapsul": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "2.3.2": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "2.5.1": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "2.5.2": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "2.5.2.1": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "2.5.2.2": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "བོད་ཡིག": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "贛語": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "pälzisch": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "cxx": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "c++23": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "2024-10-19": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "c++26": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "2024-10-16": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    ".c++": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    ".h++": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "isocpp.org": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "embarcadero": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "xl": [
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "modula-2": [
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "carbon": [
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "objective-c++": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "/ˈsiː": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "plʌ": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "plʌs/": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "cpp": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "danish": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "l": [
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "ʌ": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "resource-constrain": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "infrastructur": [
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "telephon": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "probe": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "14882:2024": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "14882:1998": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "c++03": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "c++14": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "c++20": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "enlarg": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "1979": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "three-year": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "linu": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "torvald": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "stallman": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "knuth": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "predecessor": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "phd": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "thesi": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "analyz": [
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "simula-lik": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "cpre": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "type-saf": [
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "free-stor": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "new/delet": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "bcpl-style": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "cfront": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "1984": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "pipe": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "c++98": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "winner": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "stark": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "draper": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "prize": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "conceptu": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "surpass": [
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "evolutionari": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "rick": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "mascitti": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "mid-1983": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "tongue-in-cheek": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "spirit": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "throughout": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "obviou": [
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "pick": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "misus": [
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "user-cr": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "unus": [
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "lower": [
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "beneath": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "jtc1/sc22/wg21": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "14882:2003": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "c++0x": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "14882:2011": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "c++1i": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "ballot": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "mid-august": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "c++1z": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "mid": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "18015:2006": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "19768:2007": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "29124:2010": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "24733:2011": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "18822:2015": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "filesystem": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "19570:2015": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "19841:2015": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "19568:2015": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "19217:2015": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "19571:2016": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "19568:2017": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "21425:2017": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "22277:2017": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "19216:2018": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "21544:2018": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "19570:2018": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "23619:2021": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "zero-overhead": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "eleg": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "iostream": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "std": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "cout": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "freedom": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "lifespan": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "shall": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "destruct": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "destructor": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "advis": [
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "make_uniqu": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "ownership": [
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "make_shar": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "reference-count": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "parameter": [
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "sfina": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "era": [
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "companion": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "programmat": [
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "predetermin": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "pre-compil": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "text-substitut": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "text-inclusion/exclus": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "turing-complet": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "decoupl": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "sacrif": [
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "non-oop": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "determinist": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "friend": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "decomposit": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "fundament": [
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "graph": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "elabor": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "relationship": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "cat": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "mammal": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "typeid": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "indirect": [
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "short-circuit": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "incur": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "penalti": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "left-to-right": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "unmatch": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "curious": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "recur": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "mimic": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "type-awar": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "opinion": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "proper": [
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "behav": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "dynamic_cast": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "reference/point": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "downcast": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "upcast": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "checked/perform": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "static_cast": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "ancestr": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "caller": [
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "rtti": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "confid": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "ordinarili": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "overridden": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "qualifi": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "inexact": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "nonetheless": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "aris": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "non-pur": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "ill-form": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "return_typ": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "function_bodi": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "template_paramet": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "propag": [
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "exception-caus": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "stdexcept": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "vec": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    ":out_of_rang": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "0-3": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "1-4": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "vec.at": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "out_of_rang": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "cerr": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "'\\n": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    ":except": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "unrecognis": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "fatal": [
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    ":red": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "blue": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "implementation-defin": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "fit": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "'long": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "'char": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "for_each": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "binary_search": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "random_shuffl": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "localis": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "synchronis": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "stl": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "multi": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "alexand": [
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "stepanov": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "thank": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "standards-conform": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "conven": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "chair": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "'modern": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "keynot": [
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "cppcon": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "accompani": [
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "gsl": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "checker": [
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "mangl": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "implementation-specif": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "downsid": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "itanium": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "processor-independ": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "complex-numb": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    ":complex": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "code-compat": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "c99-introduc": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "intermix": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "from/us": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "linkag": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "633": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    ":endl": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "1979-1991": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "-ii": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "699–769": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "doi:10.1145/234286.1057836": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "699–": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "10.1145/234286.1057836": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "thrive": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "crowd": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "2006–2020": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "hopl": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "machineri": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "1–168": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "doi:10.1145/3386320": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "2475-1421": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "219603741": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "1–": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "10.1145/3386320": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "naugler": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "fogu": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "hickey": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "quarterli": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "poon": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "www.lua.org": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "3.6.4": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "doc.rust-lang.org": [
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "0-201-88954-4": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "59193992": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "essenc": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "edinburgh": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "stroustrup.com": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "www.stroustrup.com": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "builin-mailinfo.c": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "attract": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "475–476": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "475–": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "1991-2006": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "1979−": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "trip": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "oulu": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "herbsutter.com": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "2016.": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "dusíková": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "hana": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "n4817": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "pragu": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "invit": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "pioneer": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "tiobe.com": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "14882:2014": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "14882:2017": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "14882:2020": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "unanim": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "mill": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "channel9.msdn.com": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "toronto": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "sergio": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "simon": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "0-201-70073-5": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "n3797": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "§3.7": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "basic.stc": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "§3.7.1": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "basic.stc.stat": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "§3.7.2": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "basic.stc.thread": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "§3.7.3": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "basic.stc.auto": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "§3.7.4": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "basic.stc.dynam": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "isocpp.github.io": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "nobodi": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "articles.emptycrate.com/": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "emptycr": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "travel": [
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "hear": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "exactli": [
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "bigger": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "g++": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    ".cpp": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "-o3": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "non-templ": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "8140": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "8028": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "alexandrescu": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "andrei": [
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "henricson": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "mat": [
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "nyquist": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "erik": [
        "https://en.wikipedia.org/wiki/C%2B%2B",
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "0-13-120965-5": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "mycroft": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "cambridg": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "2013-14": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "345": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "9780321563842": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "363–365": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "363–": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "363": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "344": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "349": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "graziano": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "lo": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "russo": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "stlport.org": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "microsoft/gsl": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "c9x": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "vc10": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "clang.llvm.org": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "abraham": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "gurtovoy": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "aleksey": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "0-321-22725-5": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "0-201-70431-5": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "0-321-11358-6": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "becker": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "pete": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "0-321-41299-0": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "brokken": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "groningen": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "978-90-367-0470-0": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "coplien": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "reprint": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "0-201-54855-0": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "dewhurst": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "0-321-32192-8": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "josutti": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "nicolai": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "978-0-321-62321-8": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "koenig": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "moo": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "e.": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "acceler": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "0-201-70353-x": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "lippman": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "stanley": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "lajoi": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "josé": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "fifth": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "978-0-321-71411-4": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "0-201-83454-5": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "meyer": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "0-321-33487-6": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "978-0-321-56384-2": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "0-201-54330-3": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "978-0-321-99278-9": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "0-201-70434-x": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "0-201-76042-8": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "vandevoord": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "0-201-73484-2": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "director": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "as-if": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "barton–nackman": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "vex": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "msvc": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "c++/cli": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "c++/cx": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "c++/winrt": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "sycl": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "title=c": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "2b": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "oldid=1267808714": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "move-protect": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "20:19": [
        "https://en.wikipedia.org/wiki/C%2B%2B"
    ],
    "2.2.2": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "2.2.3": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "goroutin": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "channel": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "2.4.1": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "2.4.2": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "omiss": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "recept": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "griesem": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "rob": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "2009-11-10": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "1.23.4": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "dragonfli": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "3-claus": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "go.dev": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "oberon-2": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "newsqueak": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "modula": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "occam": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "csp-style": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "golang": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "golang.org": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "csp": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "toolchain": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "frontend": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "gofrontend": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "gccgo": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "gcc-base": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "llvm-base": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "gollvm": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "gopherj": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "front-end": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "codebas": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "multiprocess": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "dislik": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "gopher": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "mascot": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "rené": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "french": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "wfmu": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "mono": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "bigelow": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "kri": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "holm": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "humanist": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "sans-serif": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "lucida": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "grand": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "monospac": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "wgl4": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "legibl": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "x-height": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "letterform": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "din": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "1450": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "serif": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "redesign": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "adam": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "smith": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "styliz": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "slant": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "drew": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "type-gener": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "go2go": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "generics-en": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "1.18": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "1.23": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "semver": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "non-virtu": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "declaration/initi": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "err": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "three-stat": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "fmt.println": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "hit": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "int64": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "float32": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "encoding/decod": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "capac": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "pointer-to-t": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "address-tak": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "unsafe.point": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "type-k": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "type-v": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "chan": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "ipv4": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "ipv4addr": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "uint32": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "ip": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "int32": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "zerobroadcast": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "addr": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "255.255.255.255.": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "0xffffffff": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "266": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "simultan": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "circl": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "float64": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "sq": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "pow": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "shape-typ": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "263–268": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "satisfi": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "newli": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "270": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "282–283": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "besid": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "shp": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "ok": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "fmt": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "v\\n": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "diagon": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "sqrt2": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "diamet": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "longestcontainedlin": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    ".diagon": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    ".diamet": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "284": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "referred-to": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "inspect": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "non-gener": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "meta-typ": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "~t": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "num": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "re": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "+1.650000e+001": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "iota": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "blank": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "kb": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "gb": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "compress/bzip2": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "golang.org/x/net/html": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "bzip2": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "io.read": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "bzip2.read": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "example.com/user_name/package_nam": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "likelihood": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "collis": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "asynchroni": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "event-bas": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "280–281": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "multiplex": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "operating-system": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "151–152": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "idiomat": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "fifo": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "-ch": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "switch-lik": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "apart": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "actor": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "model-styl": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "one-to-on": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "worker": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "pool": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "pipelin": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "decompress": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "fan-out": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "notion": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "interprocess": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "concurrency-saf": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "recycl": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "concurrency-rel": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "hoar": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "co-design": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "communicating-process": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "shared-memori": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "multi-processor": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "season": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "tbb": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "divide-and-conqu": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "distribute-work-synchron": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "partli": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "chisnal": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "xor": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "detector": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "unsynchron": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "best-effort": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "untag": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "argu": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "defend": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "h": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "schmager": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "al.": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "fine-grain": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "method-level": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "overus": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "proportion": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "exception-lik": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "panic/recov": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "unrecover": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "halt": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "multi-valu": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "panic": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "surface-level": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "gofmt": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "fixed-width": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "golint": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "godoc": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "ban": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "cyclic": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "functional-program": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "java-styl": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "makefil": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "microbenchmark": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "fuzz": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "gorenam": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "upgrad": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "instrument": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "gopl": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "gocod": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "autocomplet": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "goimport": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "adds/remov": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "errcheck": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "unintent": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "readword": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "sleep": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "extractusernam": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "_test.go": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "testextractusernam": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "withoutdot": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "google.com": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "fatalf": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "got": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "withdot": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "jonh.smith": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "example.com": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "net/http": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "localhost:8080": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "visit": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "hellofunc": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "responsewrit": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "fprintf": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "handlefunc": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "listenandserv": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    ":8080": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "caddi": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "docker": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "container": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "kubernet": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "cockroachdb": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "hugo": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "priorit": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "liken": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "shame": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "astel": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "yard": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "12-month": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "50th": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "award": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "eckel": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "justifi": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "hoop": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "c-compat": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "anymor": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "immatur": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "rebutt": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "lanc": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "taylor": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "rus": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "cox": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "magnitud": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "difficulti": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "aforement": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "franci": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "mccabe": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "exclam": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "'big": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "steam-rol": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "reson": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "contradict": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "evil": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "ticket": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "rsc": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "unfortun": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "occupi": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "lexer": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "semipred": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "multivalu": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "succe": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "crutch": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "ome": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "elegantli": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "codewalk": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "hierarchi": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "grace": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "twitter": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "satisfact": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "lang/go": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "go-1.4": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "cat-v.": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "www.w3schools.com": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "boldli": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "gone": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "2024.": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "gopherjs/gopherj": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "colloquium": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "kim": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "jini": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "langley": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "hey": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "ho": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "sport": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "ttf": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "merrick": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "'append": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "e2e": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "meijer": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "go.googlesource.com": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "bitfieldconsulting.com": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "1.21": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "quick": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "flv": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "8:53": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "godoc.org": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "podcast": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "§semicolon": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "rune": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "doxsey": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "caleb": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "www.golang-book.com": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "gerrand": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "selector": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "§packag": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "tour": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "cameron": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "nobl": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "gohotdraw": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "balbaert": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "ivo": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "thorough": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "iunivers": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "talks.golang.org": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "diggin": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "christoph": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "ryer": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "pkg.go.dev": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "top-level": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "somewher": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "example.com/us": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "donovan": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "munich": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "978-0-13-419044-0": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "phrasebook": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "9780132919005": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "graham-cum": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "tree.go": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "cheslack-postava": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "ewen": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "descent": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "tang": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "peiyi": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "nanz": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "sebastian": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "west": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "soar": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "da": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "silveira": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "kaue": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "gap": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "euro-par": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "10.1.1.368.6137": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "caveat": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "stori": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "§interfac": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "§type": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "informit": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "golang-nut": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "golang/lint": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "github.com": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "yourbasic.org": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "tools/gopls/readme.md": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "golang/tool": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "wei-meng": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "hoffmann": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "neumey": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "mandi": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "pirker": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "starter": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "rebecca": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "sharif": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "irfan": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "matei": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "benschoten": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "nathan": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "lewi": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "jordan": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "grieger": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "niemi": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "kai": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "wood": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "birzin": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "poss": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "raphael": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "bardea": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "ranad": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "amruta": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "darnel": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "gruneir": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "bram": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "jaffray": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "justin": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "zhang": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "luci": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "matti": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "resili": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "geo-distribut": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "sigmod": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "'20": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "1493–1509": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "doi:10.1145/3318464.3386134": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "978-1-4503-6735-6": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "1493–": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "10.1145/3318464.3386134": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "hopkin": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "brandon": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "watch": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "engineyard": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "jt": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "jaxent": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "json-rpc": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "strike": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "bake-off": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "yager": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "dobronszki": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "jano": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "everyday": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "hassl": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "mandat": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "outweigh": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "disadvantag": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)",
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "mr.": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "ride": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "create/open/openfil": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "file_share_delet": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "32088": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "tu": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "tengfei": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "real-world": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "brownle": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "'go": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "accus": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "informationweek": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "400": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "bodner": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "352": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "9781492077213": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "gemma": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "openrefin": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "tesseract": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "sawzal": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "beam": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "bazel": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "blockli": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "cpplint": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "flatbuff": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "flutter": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "ganeti": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "gear": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "gerrit": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "glop": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "grpc": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "gson": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "guava": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "guetzli": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "guic": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "gvisor": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "leveldb": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "libvpx": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "nacl": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "namebench": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "nomulu": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "or-tool": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "polym": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "chromiumo": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "fuchsia": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "glinux": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "goobuntu": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "code-in": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "inferno": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "blit": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "rio": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "8½": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "shaney": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "grep": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "title=go_": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "oldid=1269146554": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "07:15": [
        "https://en.wikipedia.org/wiki/Go_(programming_language)"
    ],
    "callabl": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "protocol-ori": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "2.10": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "2.11": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "2.12": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "2.13": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "2014-06-02": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "darwin": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "ipado": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "tvo": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "visiono": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "watcho": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "z/o": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "www.swift.org": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "developer.apple.com/swift/": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "2014.": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "safer": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "doom": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "worldwid": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "wwdc": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "underw": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "quarter": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "5.9": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "consum": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "5.10": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "unchang": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "500-page": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "6.0": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "love": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "swift.org": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "anyon": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "pull": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "pane": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "ipad": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "game-lik": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "departur": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "tesla": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "motor": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "veteran": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "ted": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "kremenek": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "swiftui": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "ui": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "distro": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "5.2.4": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "cento": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "cocoa": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "employ": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "programming-languag": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "friendli": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "out-of-bound": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "c-style": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "multilin": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "highscorethreshold": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "currentscor": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "980": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "1200": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "playermessag": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "luck": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "if-els": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "for-in": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "if-let": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "unwrap": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "somenumb": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "divid": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "2.": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "odd": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "constructgreet": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "greet": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "dividebytwo": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "anum": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "multiplybytwo": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "mathoper": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "met": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "non-nil": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "bydenomin": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "fallthrough": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "bart": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "5.": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "self-contain": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "closure1": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "arg1": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "arg2": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "…or": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "closure2": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "closure3": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "single-express": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "altogeth": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "josephin": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "kept": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "shortnam": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "makemultipli": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "withmultipl": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "unicode-compli": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "somestr": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "score": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "nsstring": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "objective-c.": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "callasfunct": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "callablestruct": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "filepriv": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "subclass": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "non-opt": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "null-point": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "nullabl": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "optionalinteg": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "myvalu": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "anoptionalinst": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "somemethod": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "non-nul": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "suppress": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "calls/gett": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "aten": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "abuild": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "tenantlist": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "theirleas": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "leasedetail": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "leasestart": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "startdat": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "32-": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "physic": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "spell-check": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "organiz": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "library-lik": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "rectangl": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "printdetail": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "read-onli": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "myclass": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "getsomethingprint": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "somesortofprintableinst": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "matter": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "printablearray": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "approxim": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "percent": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "12.3": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "lh": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "rh": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "allequ": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "async": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "await": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "suspend": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "suspens": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "downloadtext": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "text1": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "text2": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "text3": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "texttoprint": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "taskgroup": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "taskhandl": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "sometext": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "mutat": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "tucker": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "10.9": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "re-us": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "semi-autom": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "myapp": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "myapp-swift.h": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "however—us": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "swift-specif": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "non-object": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "inaccess": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "necessarili": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "iboutlet": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "outlet": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "ib": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "on-screen": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "non-appl": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "corelib": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "swift-corelibs-found": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "swift-corelibs-libdispatch": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "swift-corelibs-xctest": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "xctest": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "talli": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "unown": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "init": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "deinit": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "de-initi": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "staci": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "house21b": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "21b": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "street": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "house42b": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "respond": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "on-the-fli": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "breakpoint": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "lldb": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "trap": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "consecut": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "uint": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "error-handl": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "one-stat": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "i++": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "off-by-on": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "onward": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "pre-": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "post-": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "kitura": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "perfect": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "vapor": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "remobject": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "9329844": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "6.0.3": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "llvmdev": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "cfe-dev": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "imagin": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "wo": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "__file__": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "__line__": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "lardinoi": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "freder": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "sixtieth": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "birthday": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "verg": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "pitch": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "cio": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "wall": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "swift-evolut": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "redmonk": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "tecosystem": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "redmonk.com": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "hudson": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "hackingwithswift.com": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "borla": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "holli": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "|last=": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "mayo": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "9to5mac": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "catalog": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "cunningham": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "longtim": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "wuerthel": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "appleinsid": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "eran": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "dilger": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "wwdc19": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "brightest": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "star": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "galaxi": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "swift-lang": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "readdl": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "anderson": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "tailor": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "timmer": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "condé": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "nast": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "5.6.1": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "apple/swift": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "5.6.2": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "5.6.3": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "5.7": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "5.7.1": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "5.8": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "5.8.1": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "5.9.1": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "5.9.2": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "5.10.1": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "docs.swift.org": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "wwdc22": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "macomb": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "kyle": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "yaskevich": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "yavel": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "gregor": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "mccall": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "guhit": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "fiel": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "nscopi": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "swiftinit.org": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "swiftinit": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "mattt": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "nshipster": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "swift-evolution/proposals/0304-structured-concurrency.md": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "apple/swift-evolut": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "swift-evolution/proposals/0296-async-await.md": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "swift-bas": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "10.9/io": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "stackoverflow": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "swiftui-": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "developer.apple.com": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "lanier": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "groff": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "nut": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "swift-languag": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "stackoverflow.com": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "barbosa": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "rose": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "-dealloc": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "operatingsystem": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "so": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "gs/o": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "lisa": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "macwork": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "prodo": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "bento": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "clip": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "cut": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "garageband": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "icloud": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "idvd": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "ilif": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "imovi": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "itun": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "iwork": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "iphoto": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "apertur": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "qmaster": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "filemak": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "motion": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "compressor": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "macproject": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "mactermin": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "mainstag": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "shake": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "soundtrack": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "classroom": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "applework": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "calendar": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "facetim": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "macdraw": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "macpaint": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "macwrit": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "photo": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "booth": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "textedit": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "foundationdb": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "webobject": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "xgrid": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "xsan": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "hypercard": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "ibook": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "resedit": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    ".mac": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "dashcod": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "iad": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "ichat": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "iweb": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "mobilem": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "airport": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "bonjour": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "camp": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "quicktim": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "macintosh": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "title=swift_": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "oldid=1260160620": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "06:06": [
        "https://en.wikipedia.org/wiki/Swift_(programming_language)"
    ],
    "poland": [
        "https://en.wikipedia.org/wiki/Kotlin"
    ],
    "voivodeship": [
        "https://en.wikipedia.org/wiki/Kotlin"
    ],
    "villag": [
        "https://en.wikipedia.org/wiki/Kotlin"
    ],
    "west-centr": [
        "https://en.wikipedia.org/wiki/Kotlin"
    ],
    "island": [
        "https://en.wikipedia.org/wiki/Kotlin"
    ],
    "near": [
        "https://en.wikipedia.org/wiki/Kotlin"
    ],
    "gulf": [
        "https://en.wikipedia.org/wiki/Kotlin"
    ],
    "finland": [
        "https://en.wikipedia.org/wiki/Kotlin"
    ],
    "kotlin-class": [
        "https://en.wikipedia.org/wiki/Kotlin"
    ],
    "soviet": [
        "https://en.wikipedia.org/wiki/Kotlin"
    ],
    "navi": [
        "https://en.wikipedia.org/wiki/Kotlin"
    ],
    "fruit": [
        "https://en.wikipedia.org/wiki/Kotlin"
    ],
    "veget": [
        "https://en.wikipedia.org/wiki/Kotlin"
    ],
    "agro": [
        "https://en.wikipedia.org/wiki/Kotlin"
    ],
    "disambigu": [
        "https://en.wikipedia.org/wiki/Kotlin"
    ],
    "title=kotlin": [
        "https://en.wikipedia.org/wiki/Kotlin"
    ],
    "oldid=999577926": [
        "https://en.wikipedia.org/wiki/Kotlin"
    ],
    "22:20": [
        "https://en.wikipedia.org/wiki/Kotlin"
    ]
}

## Βήμα 4

- **Διαβάζουμε το ανεστραμμένο ευρετήριο**: Φορτώνουμε το ανεστραμμένο ευρετήριο από το αρχείο JSON που δημιουργήσαμε προηγουμένως.

- **Επεξεργασία Boolean Query**: Η συνάρτηση boolean_query_processor επεξεργάζεται ένα Boolean query (όπως "python AND java") και επιστρέφει τα έγγραφα που ταιριάζουν με την αναζήτηση. Χρησιμοποιεί τελεστές όπως "AND", "OR", "NOT".

- **Διαχωρισμός του query σε όρους και τελεστές**: Στην συνάρτηση parse_query, το query διαχωρίζεται σε όρους και τελεστές (AND, OR, NOT). Αυτοί οι τελεστές χρησιμοποιούνται για να συνδυαστούν τα αποτελέσματα από το ανεστραμμένο ευρετήριο.

- **Αναζήτηση για κάθε όρο**: Η συνάρτηση get_documents επιστρέφει τα έγγραφα που περιέχουν κάθε όρο από το ανεστραμμένο ευρετήριο.

- **Συνδυασμός των αποτελεσμάτων με βάση τους τελεστές**: Χρησιμοποιούμε τελεστές για να συνδυάσουμε τα αποτελέσματα των όρων και να υπολογίσουμε ποια έγγραφα ταιριάζουν με το query. Αν ο τελεστής είναι "AND", παίρνουμε τα κοινά έγγραφα (διασταύρωση), αν είναι "OR", παίρνουμε τη ένωση, και αν είναι "NOT", παίρνουμε τη διαφορά.

- **Παράδειγμα ελέγχου**: Ενδεικτικά, εκτελούμε το query "python AND java" και εκτυπώνουμε τα έγγραφα που ταιριάζουν με το query.





*boolean_query_processor.py*

In [ ]:
import json

# Διαβάζουμε το ανεστραμμένο ευρετήριο
with open('inverted_index.json', 'r', encoding='utf-8') as file:
    inverted_index = json.load(file)

def boolean_query_processor(query, inverted_index):
    """
    Επεξεργάζεται ένα Boolean query και επιστρέφει τα αναγνωριστικά των εγγράφων που ταιριάζουν.
    """
    def parse_query(query):
        # Διαχωρίζουμε το query σε όρους και τελεστές
        tokens = query.split()
        terms = []
        operators = []
        for token in tokens:
            if token in {"AND", "OR", "NOT"}:
                operators.append(token)
            else:
                terms.append(token)
        return terms, operators

    def get_documents(term):
        # Παίρνουμε τα έγγραφα για έναν όρο από το ανεστραμμένο ευρετήριο
        return set(inverted_index.get(term, []))

    terms, operators = parse_query(query)
    if not terms:
        return set()  # Αν δεν υπάρχουν όροι, επιστρέφουμε κενό σύνολο

    # Αρχικοποιούμε το σύνολο των αποτελεσμάτων με τον πρώτο όρο
    result = get_documents(terms[0])

    # Επεξεργαζόμαστε τους τελεστές και συνδυάζουμε τα αποτελέσματα
    for i, operator in enumerate(operators):
        next_term_docs = get_documents(terms[i + 1])
        if operator == "AND":
            result = result.intersection(next_term_docs)  # Διασταύρωση (AND)
        elif operator == "OR":
            result = result.union(next_term_docs)  # Ένωση (OR)
        elif operator == "NOT":
            result = result.difference(next_term_docs)  # Διαφορά (NOT)

    return result

# Παράδειγμα ελέγχου
query = "python AND java"
matching_docs = boolean_query_processor(query, inverted_index)
print(f"Έγγραφα που ταιριάζουν με το '{query}': {matching_docs}")



![step 4i](./project/images/step_4_i.png)

---
- **Διαβάζουμε το ανεστραμμένο ευρετήριο**: Ανοίγουμε το αρχείο *inverted_index.json* και φορτώνουμε το αντίστροφο ευρετήριο.

- **Διαβάζουμε τα επεξεργασμένα δεδομένα**: Φορτώνουμε το αρχείο με τα προεπεξεργασμένα δεδομένα κειμένου (*processed_programming_languages_text.json*).

- **Συνάρτηση για να βρούμε τα έγγραφα που ταιριάζουν με το query**: Η συνάρτηση `get_matching_docs` παίρνει το query, το διαχωρίζει σε όρους και βρίσκει ποια έγγραφα περιέχουν αυτούς τους όρους με τη βοήθεια του ανεστραμμένου ευρετηρίου.

- **Συνάρτηση για να υπολογίσουμε το TF-IDF και να κατατάξουμε τα έγγραφα**: Η συνάρτηση `compute_tf_idf` υπολογίζει την ομοιότητα συνημίτονου (cosine similarity) ανάμεσα στο query και τα έγγραφα. Αυτό γίνεται μέσω του υπολογισμού του TF-IDF των λέξεων του query και των εγγράφων.

- **Εμφάνιση των αποτελεσμάτων**: Τα έγγραφα κατατάσσονται με βάση την ομοιότητα τους με το query και εμφανίζονται στον χρήστη.

*tfidf_ranking.py*

In [ ]:
import json
from sklearn.feature_extraction.text import TfidfVectorizer

# Διαβάζουμε το ανεστραμμένο ευρετήριο
with open('inverted_index.json', 'r', encoding='utf-8') as file:
    inverted_index = json.load(file)

# Διαβάζουμε τα επεξεργασμένα δεδομένα
with open('processed_programming_languages_text.json', 'r', encoding='utf-8') as file:
    processed_data = json.load(file)

# Συνάρτηση για να βρούμε τα έγγραφα που ταιριάζουν με το query
def get_matching_docs(query):
    query_terms = query.lower().split()  # Διαχωρίζουμε το query σε όρους
    matching_docs = set()

    # Ελέγχουμε ποια έγγραφα ταιριάζουν με τους όρους του query
    for term in query_terms:
        if term in inverted_index:
            matching_docs.update(inverted_index[term])

    return list(matching_docs)

# Συνάρτηση για να υπολογίσουμε το TF-IDF και να κατατάξουμε τα έγγραφα
def compute_tf_idf(query, matching_docs, processed_data):
    # Προετοιμάζουμε το corpus: περιλαμβάνουμε μόνο τα έγγραφα που ταιριάζουν
    corpus = [processed_data[doc_id] for doc_id in matching_docs]
    
    # Προσθέτουμε το query ως τελευταίο στοιχείο στο corpus
    corpus.append(query)

    # Χρησιμοποιούμε τον TfidfVectorizer
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(corpus)

    # Παίρνουμε το vector του query (τελευταία γραμμή του matrix)
    query_vector = tfidf_matrix[-1]

    # Υπολογίζουμε την ομοιότητα συνημίτονου (cosine similarity) μεταξύ του query και κάθε εγγράφου
    cosine_similarities = (tfidf_matrix[:-1] @ query_vector.T).toarray().flatten()

    # Ταιριάζουμε κάθε έγγραφο με την βαθμολογία της ομοιότητας
    ranked_docs = sorted(
        zip(matching_docs, cosine_similarities),
        key=lambda x: x[1],
        reverse=True
    )

    return ranked_docs

if __name__ == "__main__":
    # Ζητάμε από τον χρήστη να εισάγει ένα query
    query = input("Εισάγετε το query σας (κατάταξη TF-IDF): ").strip()

    # Βρίσκουμε τα έγγραφα που ταιριάζουν με το query
    matching_docs = get_matching_docs(query)
    
    if not matching_docs:
        print("Δεν βρέθηκαν έγγραφα που να ταιριάζουν.")
    else:
        # Υπολογίζουμε και κατατάσσουμε τα έγγραφα με βάση το TF-IDF
        ranked_docs = compute_tf_idf(query, matching_docs, processed_data)

        # Εμφανίζουμε τα αποτελέσματα
        print("\nΚαταταγμένα αποτελέσματα:")
        for doc_id, score in ranked_docs:
            print(f"{doc_id}: {score:.4f}")


![step 4ii](./project/images/step_4_ii.png)


---
- **Διαβάζουμε το ανεστραμμένο ευρετήριο**: Φορτώνουμε το ανεστραμμένο ευρετήριο από το αρχείο *inverted_index.json*.

- **Διαβάζουμε τα επεξεργασμένα δεδομένα**: Φορτώνουμε το αρχείο με τα προεπεξεργασμένα δεδομένα κειμένου από το αρχείο *processed_programming_languages_text.json*.

- **Συνάρτηση για κατάταξη με BM25**: Η συνάρτηση `bm25_ranking` χρησιμοποιεί τον αλγόριθμο BM25 για να κατατάξει τα έγγραφα με βάση την ομοιότητα του query με τα έγγραφα. Για κάθε έγγραφο, υπολογίζουμε τη βαθμολογία του και τα ταξινομούμε σε φθίνουσα σειρά.

- **Κύρια συνάρτηση της μηχανής αναζήτησης**: Η συνάρτηση `search_engine` είναι η κεντρική λειτουργία της μηχανής αναζήτησης. Επιτρέπει στον χρήστη να εισάγει ένα query και να επιλέξει τη μέθοδο κατάταξης (Boolean, TF-IDF ή BM25). Παρουσιάζει τα αποτελέσματα του query με την επιλεγμένη μέθοδο κατάταξης.

- **Επιλογή μεθόδου κατάταξης**: Ο χρήστης μπορεί να επιλέξει την μέθοδο κατάταξης (Boolean, TF-IDF, BM25) για τα αποτελέσματα της αναζήτησης.

- **Εμφάνιση καταταγμένων αποτελεσμάτων**: Τα αποτελέσματα του query εμφανίζονται με την επιλεγμένη μέθοδο κατάταξης και κάθε έγγραφο εμφανίζεται με τη σχετική του βαθμολογία.

*search_engine.py*

In [ ]:
import json
from tfidf_ranking import compute_tf_idf
from boolean_query_processor import boolean_query_processor
from rank_bm25 import BM25Okapi

# Διαβάζουμε το ανεστραμμένο ευρετήριο
with open("inverted_index.json", "r", encoding="utf-8") as f:
    inverted_index = json.load(f)

# Διαβάζουμε τα επεξεργασμένα δεδομένα
with open("processed_programming_languages_text.json", "r", encoding="utf-8") as f:
    processed_data = json.load(f)

# Συνάρτηση για κατάταξη με BM25
def bm25_ranking(query, matching_docs, processed_data):
    # Tokenize (διαχωρίζουμε σε λέξεις) τα έγγραφα
    tokenized_docs = [doc.split() for doc in processed_data.values()]
    
    # Δημιουργούμε το αντικείμενο BM25
    bm25 = BM25Okapi(tokenized_docs)
    
    # Διαχωρίζουμε το query σε λέξεις
    query_tokens = query.split()
    
    # Υπολογίζουμε τις βαθμολογίες (scores) του query
    scores = bm25.get_scores(query_tokens)
    
    # Συνδυάζουμε τα έγγραφα με τις αντίστοιχες βαθμολογίες τους
    doc_score_pairs = [(doc, score) for doc, score in zip(processed_data.keys(), scores) if doc in matching_docs]
    
    # Ταξινομούμε τα έγγραφα κατά φθίνουσα βαθμολογία
    ranked_results = sorted(doc_score_pairs, key=lambda x: x[1], reverse=True)
    
    return [doc for doc, _ in ranked_results]

# Κύρια συνάρτηση της μηχανής αναζήτησης
def search_engine():
    print("Καλώς ήρθατε στην Μηχανή Αναζήτησης!")
    print("Πληκτρολογήστε το query σας χρησιμοποιώντας Boolean τελεστές (AND, OR, NOT).")
    print("Παράδειγμα: python AND java OR javascript\n")
    
    while True:
        query = input("Εισάγετε το query σας (ή πληκτρολογήστε 'exit' για έξοδο): ").strip()
        if query.lower() == "exit":
            print("Αντίο!")
            break

        # Επεξεργαζόμαστε το query χρησιμοποιώντας την Boolean αναζήτηση
        matching_docs = boolean_query_processor(query, inverted_index)
        if not matching_docs:
            print(f"Δεν βρέθηκαν έγγραφα που να ταιριάζουν με το query: {query}")
            continue

        print(f"\nΈγγραφα που ταιριάζουν με το '{query}': {matching_docs}")

        # Επιλογή μεθόδου κατάταξης από τον χρήστη
        print("\nΕπιλέξτε μέθοδο κατάταξης:")
        print("1. Boolean Αναζήτηση (Προεπιλογή)")
        print("2. Κατάταξη TF-IDF")
        print("3. Κατάταξη BM25")
        choice = input("Εισάγετε τον αριθμό της επιλογής σας (1, 2 ή 3): ").strip()

        if choice == "2":
            ranked_docs = compute_tf_idf(query, matching_docs, processed_data)
            print("\nΚαταταγμένα αποτελέσματα με TF-IDF:")
        elif choice == "3":
            ranked_docs = bm25_ranking(query, matching_docs, processed_data)
            print("\nΚαταταγμένα αποτελέσματα με BM25:")
        else:
            ranked_docs = list(matching_docs)
            print("\nΑποτελέσματα Boolean Αναζήτησης:")

        # Εμφάνιση των καταταγμένων αποτελεσμάτων
        for rank, doc in enumerate(ranked_docs, start=1):
            print(f"{rank}. {doc}")

# Εκκίνηση της μηχανής αναζήτησης
if __name__ == "__main__":
    search_engine()

Ακολουθεί ενδεικτική εκτέλεση.








C:\Users\Gregory\Desktop\project>python search_engine.py
Documents matching 'python AND java': {'https://en.wikipedia.org/wiki/Ruby_(programming_language)', 'https://en.wikipedia.org/wiki/Java_(programming_language)', 'https://en.wikipedia.org/wiki/C_(programming_language)', 'https://en.wikipedia.org/wiki/Python_(programming_language)', 'https://en.wikipedia.org/wiki/Swift_(programming_language)', 'https://en.wikipedia.org/wiki/JavaScript', 'https://en.wikipedia.org/wiki/C%2B%2B', 'https://en.wikipedia.org/wiki/Go_(programming_language)'}
Welcome to the Search Engine!
Type your query using Boolean operators (AND, OR, NOT).
Example: python AND java OR javascript

Enter your query (or type 'exit' to quit): java or hello

Documents matching 'java or hello': {'https://en.wikipedia.org/wiki/Ruby_(programming_language)', 'https://en.wikipedia.org/wiki/Java_(programming_language)', 'https://en.wikipedia.org/wiki/C_(programming_language)', 'https://en.wikipedia.org/wiki/Python_(programming_language)', 'https://en.wikipedia.org/wiki/Swift_(programming_language)', 'https://en.wikipedia.org/wiki/JavaScript', 'https://en.wikipedia.org/wiki/C%2B%2B', 'https://en.wikipedia.org/wiki/Go_(programming_language)'}

Choose a ranking method:
1. Boolean Retrieval (Default)
2. TF-IDF Ranking
3. BM25 Ranking
Enter the number of your choice (1, 2, or 3): 1

Boolean Retrieval Results:
1. https://en.wikipedia.org/wiki/Ruby_(programming_language)
2. https://en.wikipedia.org/wiki/Java_(programming_language)
3. https://en.wikipedia.org/wiki/C_(programming_language)
4. https://en.wikipedia.org/wiki/Python_(programming_language)
5. https://en.wikipedia.org/wiki/Swift_(programming_language)
6. https://en.wikipedia.org/wiki/JavaScript
7. https://en.wikipedia.org/wiki/C%2B%2B
8. https://en.wikipedia.org/wiki/Go_(programming_language)
Enter your query (or type 'exit' to quit): java or hello

Documents matching 'java or hello': {'https://en.wikipedia.org/wiki/Ruby_(programming_language)', 'https://en.wikipedia.org/wiki/Java_(programming_language)', 'https://en.wikipedia.org/wiki/C_(programming_language)', 'https://en.wikipedia.org/wiki/Python_(programming_language)', 'https://en.wikipedia.org/wiki/Swift_(programming_language)', 'https://en.wikipedia.org/wiki/JavaScript', 'https://en.wikipedia.org/wiki/C%2B%2B', 'https://en.wikipedia.org/wiki/Go_(programming_language)'}

Choose a ranking method:
1. Boolean Retrieval (Default)
2. TF-IDF Ranking
3. BM25 Ranking
Enter the number of your choice (1, 2, or 3): 2

TF-IDF Ranked Results:
1. ('https://en.wikipedia.org/wiki/Java_(programming_language)', np.float64(0.25966958427207704))
2. ('https://en.wikipedia.org/wiki/JavaScript', np.float64(0.01689162517913656))
3. ('https://en.wikipedia.org/wiki/C_(programming_language)', np.float64(0.015636927175054036))
4. ('https://en.wikipedia.org/wiki/C%2B%2B', np.float64(0.01499599173627299))
5. ('https://en.wikipedia.org/wiki/Go_(programming_language)', np.float64(0.012284060668750124))
6. ('https://en.wikipedia.org/wiki/Ruby_(programming_language)', np.float64(0.007483638885609637))
7. ('https://en.wikipedia.org/wiki/Swift_(programming_language)', np.float64(0.006847389333614802))
8. ('https://en.wikipedia.org/wiki/Python_(programming_language)', np.float64(0.005532624249020468))
Enter your query (or type 'exit' to quit): java or hello

Documents matching 'java or hello': {'https://en.wikipedia.org/wiki/Ruby_(programming_language)', 'https://en.wikipedia.org/wiki/Java_(programming_language)', 'https://en.wikipedia.org/wiki/C_(programming_language)', 'https://en.wikipedia.org/wiki/Python_(programming_language)', 'https://en.wikipedia.org/wiki/Swift_(programming_language)', 'https://en.wikipedia.org/wiki/JavaScript', 'https://en.wikipedia.org/wiki/C%2B%2B', 'https://en.wikipedia.org/wiki/Go_(programming_language)'}

Choose a ranking method:
1. Boolean Retrieval (Default)
2. TF-IDF Ranking
3. BM25 Ranking
Enter the number of your choice (1, 2, or 3): 3

BM25 Ranked Results:
1. https://en.wikipedia.org/wiki/Java_(programming_language)
2. https://en.wikipedia.org/wiki/C_(programming_language)
3. https://en.wikipedia.org/wiki/C%2B%2B
4. https://en.wikipedia.org/wiki/JavaScript
5. https://en.wikipedia.org/wiki/Go_(programming_language)
6. https://en.wikipedia.org/wiki/Swift_(programming_language)
7. https://en.wikipedia.org/wiki/Python_(programming_language)
8. https://en.wikipedia.org/wiki/Ruby_(programming_language)

## Βήμα 5

SearchEngineEvaluator

1. **Φόρτωση του Αντίστροφου Ευρετηρίου**:

Η πρώτη συνάρτηση, *load_inverted_index*, φορτώνει το αντίστροφο ευρετήριο από ένα αρχείο JSON. Αν το αρχείο δεν υπάρχει, η συνάρτηση επιστρέφει None και εκτυπώνει μήνυμα λάθους. Διαφορετικά, το αρχείο διαβάζεται και επιστρέφεται το αντίστροφο ευρετήριο.

In [ ]:
import json
import os

# Φόρτωση του ανεστραμμένου ευρετηρίου από το αρχείο JSON
def load_inverted_index(file_path):
    if not os.path.exists(file_path):
        print(f"Error: File {file_path} not found.")
        return None
    with open(file_path, 'r', encoding='utf-8') as f:
        inverted_index = json.load(f)
    return inverted_index

2. **Επεξεργασία Boolean Queries**:

Η συνάρτηση `boolean_query_processor` είναι υπεύθυνη για την επεξεργασία του Boolean query που παρέχεται από τον χρήστη. Αν το query περιέχει τις λέξεις κλειδιά AND, OR, ή NOT, η συνάρτηση θα επεξεργαστεί τα αντίστοιχα αποτελέσματα χρησιμοποιώντας τις αντίστοιχες λογικές πράξεις. Αν το query δεν περιέχει κανένα από αυτά τα λογικά τελεστικά, απλώς επιστρέφει τα έγγραφα που περιέχουν τον πρώτο όρο του query.

- Αν το query περιέχει AND, επιστρέφονται τα έγγραφα που περιέχουν όλους τους όρους του query.
- Αν το query περιέχει OR, επιστρέφονται τα έγγραφα που περιέχουν τουλάχιστον έναν από τους όρους.
- Αν το query περιέχει NOT, επιστρέφονται τα έγγραφα που περιέχουν τον πρώτο όρο, αλλά χωρίς τους υπόλοιπους.

In [ ]:
# Συνάρτηση για την επεξεργασία Boolean queries και την επιστροφή των ταιριαστών εγγράφων
def boolean_query_processor(query, inverted_index):
    query_terms = query.lower().split(' ')
    if 'and' in query_terms:
        terms = [term for term in query_terms if term != 'and']
        matching_docs = set(inverted_index.get(terms[0], []))
        for term in terms[1:]:
            matching_docs &= set(inverted_index.get(term, []))
    elif 'or' in query_terms:
        terms = [term for term in query_terms if term != 'or']
        matching_docs = set(inverted_index.get(terms[0], []))
        for term in terms[1:]:
            matching_docs |= set(inverted_index.get(term, []))
    elif 'not' in query_terms:
        terms = [term for term in query_terms if term != 'not']
        matching_docs = set(inverted_index.get(terms[0], []))
        for term in terms[1:]:
            matching_docs -= set(inverted_index.get(term, []))
    else:
        matching_docs = set(inverted_index.get(query_terms[0], []))

    return matching_docs


3. **Υπολογισμός Ακρίβειας, Ανάκτησης και F1-Score**:

Η συνάρτηση `evaluate_metrics` υπολογίζει την ακρίβεια, την ανάκτηση και το F1-Score για τα έγγραφα που ταιριάζουν με το query. Αυτά τα μέτρα χρησιμοποιούνται συχνά για την αξιολόγηση της απόδοσης ενός συστήματος αναζήτησης:

- **Ακρίβεια (Precision)**: Το ποσοστό των ταιριαστών εγγράφων που είναι πραγματικά συναφή με το query.
- **Ανάκτηση (Recall)**: Το ποσοστό των συναφών εγγράφων που ανακτήθηκαν από το σύστημα.
- **F1-Score**: Το αρμονικό μέσο της ακρίβειας και της ανάκτησης, το οποίο προσφέρει μια ενιαία μέτρηση.


In [ ]:
# Συνάρτηση για τον υπολογισμό της ακρίβειας, ανάκτησης και F1-Score
def evaluate_metrics(matching_docs, relevant_docs):
    relevant_doc_keys = set(relevant_docs.keys())
    
    tp = len(matching_docs & relevant_doc_keys)
    fp = len(matching_docs - relevant_doc_keys)
    fn = len(relevant_doc_keys - matching_docs)
    
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0.0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0.0
    f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0.0
    
    return precision, recall, f1_score

4. **Υπολογισμός του Mean Average Precision (MAP)**:

Η συνάρτηση `mean_average_precision` υπολογίζει το Mean Average Precision για μια σειρά από queries. Ο μέσος όρος των AP (Average Precision) για κάθε query υπολογίζεται και χρησιμοποιείται για να πάρουμε το MAP score.

- **Average Precision (AP)**: Ο υπολογισμός της ακρίβειας για κάθε θέση στο σύνολο των αποτελεσμάτων, με βάση τη σειρά κατάταξης.

In [ ]:
# Συνάρτηση για τον υπολογισμό του Mean Average Precision (MAP)
def mean_average_precision(query_results, relevant_docs_for_queries):
    average_precision_scores = []
    for query, retrieved_docs in query_results.items():
        relevant_docs = relevant_docs_for_queries.get(query, {})
        ap = average_precision(retrieved_docs, relevant_docs)
        average_precision_scores.append(ap)
    
    map_score = sum(average_precision_scores) / len(average_precision_scores) if average_precision_scores else 0.0
    return map_score

5. **Υπολογισμός του Average Precision (AP)**: 

Η συνάρτηση `average_precision` υπολογίζει το AP για κάθε query, μετρώντας την ακρίβεια σε κάθε θέση στην κατάταξη των αποτελεσμάτων και παίρνοντας τον μέσο όρο αυτών των τιμών.

In [ ]:
# Συνάρτηση για τον υπολογισμό του Average Precision (AP) για ένα μόνο query
def average_precision(retrieved_docs, relevant_docs):
    retrieved_docs = list(retrieved_docs)
    relevant_docs_set = set(relevant_docs.keys())
    relevant_count = 0
    precision_at_k = []
    
    for i, doc in enumerate(retrieved_docs, start=1):
        if doc in relevant_docs_set:
            relevant_count += 1
            precision_at_k.append(relevant_count / i)
    
    ap = sum(precision_at_k) / len(relevant_docs) if relevant_docs else 0.0
    return ap

6. **Αξιολόγηση της Μηχανής Αναζήτησης**:

Η συνάρτηση `evaluate_search_engine` αξιολογεί τη μηχανή αναζήτησης για μια σειρά από queries. Για κάθε query, χρησιμοποιείται η συνάρτηση `boolean_query_processor` για να βρούμε τα έγγραφα που ταιριάζουν με το query και στη συνέχεια υπολογίζονται οι επιδόσεις (precision, recall, F1-Score).

Επίσης, αποθηκεύονται τα αποτελέσματα των queries για τον υπολογισμό του Mean Average Precision (MAP).

In [ ]:
# Συνάρτηση για την αξιολόγηση της μηχανής αναζήτησης για μια σειρά queries
def evaluate_search_engine(inverted_index):
    queries = [
        "python AND java",
        "content OR jump",
        "python NOT java"
    ]
    
    relevant_docs_for_queries = {
        "python AND java": {
            "https://en.wikipedia.org/wiki/Python_(programming_language)": 1,
            "https://en.wikipedia.org/wiki/Java_(programming_language)": 1
        },
        "content OR jump": {
            "https://en.wikipedia.org/wiki/Python_(programming_language)": 1,
            "https://en.wikipedia.org/wiki/JavaScript": 1
        },
        "python NOT java": {
            "https://en.wikipedia.org/wiki/Python_(programming_language)": 1
        }
    }

    query_results = {}
    print("Αποτελέσματα Αξιολόγησης:\n")
    for query in queries:
        print(f"Ερώτημα: {query}")
        matching_docs = boolean_query_processor(query, inverted_index)
        relevant_docs = relevant_docs_for_queries.get(query, {})
        precision, recall, f1_score = evaluate_metrics(matching_docs, relevant_docs)
        
        print(f"Ταιριαστά Έγγραφα: {matching_docs}")
        print(f"Ακρίβεια: {precision:.2f}")
        print(f"Ανάκτηση: {recall:.2f}")
        print(f"F1-Score: {f1_score:.2f}\n")
        
        # Αποθήκευση των αποτελεσμάτων των queries για τον υπολογισμό του MAP
        query_results[query] = matching_docs

    # Υπολογισμός του Mean Average Precision (MAP)
    map_score = mean_average_precision(query_results, relevant_docs_for_queries)
    print(f"Mean Average Precision (MAP): {map_score:.2f}")

7. **Κύρια Συνάρτηση**:

Η κύρια συνάρτηση `main` φορτώνει το ανεστραμμένο ευρετήριο από το αρχείο *inverted_index.json* και αν είναι έγκυρο, καλεί τη συνάρτηση `evaluate_search_engine` για να αξιολογήσει τη μηχανή αναζήτησης.

In [ ]:
# Κύρια συνάρτηση για τη φόρτωση του ανεστραμμένου ευρετηρίου και την εκτέλεση της αξιολόγησης
def main():
    inverted_index = load_inverted_index('inverted_index.json')
    if inverted_index is not None:
        evaluate_search_engine(inverted_index)

if __name__ == "__main__":
    main()

![step 5](./project/images/step_5.png)